### Uploading necessary libraries

In [1]:
import spacy
import re
import random
from spacy.util import minibatch, compounding
from spacy.lang.en import English
from spacy.training.example import Example
from spacy.tokens import DocBin
from tqdm import tqdm
import spacy
import re
import random
import os

### Loading spacy language vector corpus

In [3]:
nlp=spacy.load("en_core_web_lg")
nlp.pipe_names
ner=nlp.get_pipe("ner")
# ner.add_label(LABEL)

### Defining custom label

In [ ]:
LABEL="FOREIGN ORG"

### Loading the training dataset (491 unique GOS + DOI)

In [ ]:
train_data=[("SAME DAY SAME AMOUNT GOT DEBITED THROUGH REMITTANCE TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG",{"entities":[(55,82,"FOREIGN ORG")]}),
            ("AS PER AVAILABLE OPINION REPORT WHITE RABBIT MOTION PICTURES PRODUCTION LLC AND MAGNETIC MOTION PICTURE PRODUCTION LLC ARE GROUP ENTITIES",{"entities":[(32,75,"FOREIGN ORG"),(80,118,"FOREIGN ORG")]}),
            ("OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, DIJLLAH JEWELLERY FZCO OF U.A.E. IN USD",{"entities":[(130,152,"FOREIGN ORG"),(27,67,"FOREIGN ORG")]}),
            ("REMITTANCE FROM HTF HAMBURG LEATHER GMBH IN EUR FROM GERMANY FOR WHICH PURPOSE WAS MENTIONED AS ADV RECPT AGAINST EXPORT-OTHER THAN NEPAL AND BHUTAN AND ON 10-09-2020 AND 15-09-2020 THE ACCOUNT GOT DEBITED WITH RS.2.98 CRORES THROUGH TRANSFER TO ASG LEATHER PRIVATE LIMITED",{"entities":[(16,40,"FOREIGN ORG")]}),
            ("IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.178.24 CR MAINLY FROM PREMIUM PETRO PRODUCTS RS.142.08 CR, IDBI - 00131020000080662, VCI EXPORTS P LTD RS.3.09 CR, IDFC 10039338826, SURYA NAGARI TRADING RS.2.97 CR, YES BANK  004163700002899, ABHA PRECISION FARMING LLP RS.2.20 CR, IDBI  0013102000077552 AND BHEEM RAJ BOHRA RS.2.02 CR, IDBI  0131063538000. 2 EXPORT ADVANCE REMITTANCES AMOUNTED TO RS.1.24 CR IN THE A/CS OF INDIA STONE HOUSE LLP AND VCI EXPORTS P LTD MAINLY FROM WONDERSTONE LLC - USA RS.0.29 CR, TRANQUILLO GMBH RS.0.21 CR, GRAMACO SOURCE AND MARBLE PVT LTD. RS.0.18 CR, STONE SOURCE INTERNATIONAL INC  USA RS.0.14 CR AND STONELAND INC - USA RS.0.10 CR. AS PER WEB CHECK WONDERSTONE LLC  USA IS INTO BUSINESS OF JEWELLERY STORES HTTPS://WWW.DNB.COM/BUSINESS-DIRECTORY/COMPANY-PROFILES.WONDERSTONE LLC.4D21F25E0AAE301E36AA8061651D299F.",{"entities":[(469, 484,"FOREIGN ORG"),(503,518,"FOREIGN ORG"),(578,608,"FOREIGN ORG"),(629,642,"FOREIGN ORG"),(678, 693,"FOREIGN ORG"),(792,807,"FOREIGN ORG")]}),
            ("TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 1,201.34 LAKHS OF WHICH CASH WAS NIL, CLEARING RS. 471.63 LAKHS, RTGS RS. 253.30 LAKHS, REMITTANCE RS. 343.39 LAKHS MAJORLY TO SHENZHEN DEDA HEALTH CO LTD, MEDMSL LIMITED, ZHENGZHOU OLIVE ELECTRONIC, INTERNET FUND TRANSFER RS. 126.18 LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS. 5.69 LAKHS IN 5 INSTANCES, AND OTHERS RS. 1.16 LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS. 3,431.77 LAKHS OF WHICH CASH WAS RS. 3.77 LAKHS MAJORLY IN DELHI KAROL BAGH GURDWARA ROAD, BULDHANA, KAROL BAGH N.D (6291) BRANCH, TRANSFERS RS. 62.40 LAKHS MAJORLY FROM BORA MOBILITY, PRASHANT PRAKASH BORA, SEANA, CLEARING RS. 328.83 LAKHS MAJORLY FROM SSK INCORPORATION, BORA MOBILITY LLP, A Z LUNAWAT AND, NEFT RS. 87.01 LAKHS MAJORLY FROM OMKAR COMPUTERS PVT LTD, BITFLIGHT ELECTRONIC SYSTEMS, BORA MOBILITY, RTGS RS. 1,460.11 LAKHS MAJORLY FROM BORA MOBILITY, BATHLA TELETECH PRIVATE, E PAO GST REFUNDS, REMITTANCE RS. 27.75 LAKHS MAJORLY FROM CAPTAINS FREIGHT SERVICES HK, INTERNET FUND TRANSFERS RS. 1,445.05 LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS. 14.92 LAKHS IN 34 INSTANCES, UPI TRANSFERS RS. 0.43 LAKHS IN 21 INSTANCES, FIXED DEPOSIT RS. 1.50 LAKHS,. TOTAL DEBIT SIDE RS. 3,405.75 LAKHS OF WHICH CASH WAS RS. 1.00 LAKH BY BRANCH, TRANSFERS RS. 2.50 LAKHS MAJORLY TO BORA MOBILITY, CLEARING RS. 821.39 LAKHS, RTGS RS. 818.00 LAKHS MAJORLY TO BORA AND BORA TECHNOLOGIES, BORA MOBILITY, REMITTANCE RS. 1,090.27 LAKHS MAJORLY TO ESTAR TECHNOLOGY GROUP CO LTD, WONDAFONE TECHNOLOGY HONGKONG CO, AYAAN ELECTRONICS LLC",{"entities":[(171,197,"FOREIGN ORG"),(200,213,"FOREIGN ORG"),(216,241,"FOREIGN ORG"),(295,319,"FOREIGN ORG"),(1677,1708,"FOREIGN ORG"),(1711,1732,"FOREIGN ORG")]}),
            ("IN CHINA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 73211910 INDUCTION CHAFER. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON 08-11-2019 FROM PREMIER HOTEL MANAGEMENT CO LTD FROM BANGLADESH DETAILS OF WHOOM IS NOT AVAILBLE IN PUBLIC DOMAIN. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF  MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT 21-06-2019 TO 20-09-2019 TO TUNE OF EUR 4265 TO ERNST NEUMARKER GMBH AND CO(ENTITY DEALS IN FOOD SUBSTITUES, SNACKS MAKING MACHINES) WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS 84385000 KITCHENWARE. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON 05-11-2019 TO THE TUNE OF USD4742.",{"entities":[(269,300,"FOREIGN ORG"),(547, 573,"FOREIGN ORG")]}),
            ("DETAILS OF TRANSACTION IS AS UNDER:TXN. REF. NO. AMOUNT (EUR) DATE BENEFICIARY0896518AP0000106 30088 06/04/18 AVL LIST GMBH HANS LIST PLATZ, AUSTRIA0896519AP0000093 13516 26/04/19 AA MODELLERIA SRLVIA ROMA",{"entities":[(110,123,"FOREIGN ORG"),(180,204,"FOREIGN ORG"),(124,138,"FOREIGN ORG")]}),
            ("HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED ON 08-05-2020 TO TUNE OF USD 2900297 FROM MUNDRA GENERAL TRADING FZE IN UAE((DEALS IN METAL SCRAP-HTTPS://PANJIVA.COM/MUNDRA-GENERAL-TRADING-F-Z-E/42896816). WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY OF MORE THAN 120 DAYS, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON 09-01-2020 USD 2891712 TO TSAI SHEN TRADING LIMITED IN HONGKONG",{"entities":[(497,522,"FOREIGN ORG"),(105,131,"FOREIGN ORG")]}),
            ("AS PER PUBLIC DOMAIN INFORMATION ON OVERSEAS ENTITIES NAFDA TRADING FZE, UAE AND CAMELOT OVERSEAS PTE LTD, SINGAPORE, BELOW RED FLAGS WERE OBSERVED:PUBLIC DOMAIN INFORMATION OF OVERSEAS ENTITY NAFDA TRADING FZE, UAE IS INSUFFICIENT MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS - 101 KITCHENER ROAD, 03-43, JALAN BESAE PLAZA, SINGAPORE-208511 OF OVERSEAS ENTITY CAMELOT OVERSEAS PTE LTDCAMELOT OVERSEAS PTE LTD HAD PREVIOUSLY CHANGED NAMES MULTIPLE TIMES (REFER: HTTPS:WWW.COMPANIES.SG/ZH/BUSINESS/201936480H/CAMELOT-OVERSEAS-PTE-LTD-)WEBSITE OF CAMELOT OVERSEAS PTE LTD, SINGAPORE IS NEWLY REGISTERED IN THE YEAR OCTOBER 2020 AND REGISTRATION LOCATION IS NEW DELHI,",{"entities":[(54,71,"FOREIGN ORG"),(81,105,"FOREIGN ORG")]}),
            ("SINCE JANUARY 2021 CUSTOMER HAS UNDERTAKEN 09 TRANSACTIONS OF GOLD DORE IMPORTS FROM PERU AMOUNTING TO INR 38.16 CR TO THE PARTIES (SMRL AMERICAN ZAFIRO EIRL ( INR 17.63 CR) , CONSORCIO PACAYPAMPA SAC ( INR5.15 CR) , CONSORCIO VILLA DEL MAR S A C ( INR 5.20 CR) , MINERA HOLDER E I R L (INR 6.03CR), EMPRESA MINERA JCHH E I R L (INR INR 4.13CR)) WHOSE PUBLIC DOMAIN INFORMATION ARE NOT AVAILABLE EXPLICITLY AND ARE ESTABLISHED RECENTLY",{"entities":[(133,158,"FOREIGN ORG"),(177,201,"FOREIGN ORG"),(301,328,"FOREIGN ORG"),(218,247,"FOREIGN ORG"),(265,286,"FOREIGN ORG")]}),
            ("OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO CUSTOMERS EDPMS AND ADVANCE AGAINST EXPORT PENDENCY, WHICH THE CUSTOMER IS NOT SUBMITTING DESPITE MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASE.THE MAJOR PENDENCY ARE WITH FOLLOWING PARTIESNUANCE COMMUNICATIONS INC (UNITED STATES OF AMERICA), HAVING SAME LINE OF BUSINESS.6. CUSTOMER HAS NOT PROCESSED ANY EXPORT WITH HDFC BANK. HENCE THE ADVANCE AGAINST EXPORT PENDENCY IS 100%. SINCE THE SHIPPING BILLS ARE GENERATED, THE EXPORTS HAS ALREADY TAKEN PLACE. NEITHER CUSTOMER HAS SUBMITTED EXPORT DOCUMENT WITH US NOR ANY OTHER BANK HAS REQUESTED FOR AD TRANSFER OF THE SHIPPING BILLS. ACCORDINGLY POSSIBLY EXPORT IS DONE ONLY FOR VALUE TRANSFER FROM INDIA TO OVERSEAS.7. CUSTOMER HAS NEITHER SUBMITTED EXPORT DOCUMENT WITH US NOR REQUESTED US TO ENABLE E-FIRC IN EDPMS FOR SUBMISSION OF DOCUMENTS TO ANOTHER BANK. HENCE POSSIBLY EXPORT SHIPMENT IS NOT PROCESSED AND ADVANCE IS RECEIVED ONLY FOR FUND TRANSFER FROM OVERSEAS TO INDIA8. NO NEGATIVE MEDIA NEW/ INFORMATION FOUND FOR THE CUSTOMER IN THE WEB SEARCH. 9. RECOMMENDED FOR STR TO BE FILED BASIS A. CUSTOMER HAS HIGH VOLUME PENDENCY OF EDPMS OF RS. 593.67 LAKHS AND ADVANCE AGAINST EXPORT OF RS. 438.58 LAKHS.B. CUSTOMER IS NOT SUBMITTED ANY RELEVANT DOCUMENTS AFTER MULTIPLE FOLLOW-UPS AND REMINDERS.C. CUSTOMER IS ENLISTED IN THE CAUTION LIST OF RBI",{"entities":[(244, 276,"FOREIGN ORG"),(3647, 3667,"FOREIGN ORG"),(3890, 3923,"FOREIGN ORG"),(3924, 3948,"FOREIGN ORG"),(3949, 3968,"FOREIGN ORG"),(3677, 3689,"FOREIGN ORG")]}),
            ("CHANGZHOU XIETONG INDUSTRIES CO LTD AND GOODS MENTIONED AS HS ORTHOPAEDIC GOODS HS DRILLING MACHINE",{"entities":[(0,35,"FOREIGN ORG")]}),
            ("FUNDS CREDITED IN THE ACCOUNT THROUGH REMITTANCE, RTGS AND DEPLOYED THROUGH REMITTANCE, RTS SEEMS UNUSUAL. FOR EXAMPLE ON 24-03-2021 FUNDS OF RS 2.23 CRORE WERE CREDITED IN THE ACCOUNT THROUGH REMITTANCE FROM MEGHA ENGINEERING AND INFRASTRUCTURE LTD, INDIA AND ON THE NEXT DAY RS 7 CRORE WERE DEPLOYED THROUGH RTGS TOWARDS RASHMI METALIKS LIMITED (ACC NO:39826430992, IFSC: SBIN0007502 ). ALSO ON 09-03-2021 FUNDS OF RS 2.8 CRORE WERE CREDITED IN THE ACCOUNT THROUGH CLOSURE PROCEEDS OF FD AND ON THE SAME DAY RS 2.46 CRORE WERE DEPLOYED THROUGH REMITTANCE TOWARDS FERROPEM, 517 AVENUE, FRANCE IN EURO FOR PIPES AND SPARES AS PAYMT TOWARDS IMP SETTLMT OTR THAN NEPL BHUTAN. SAME PATTERN IS SEEN IN ACCOUNT. AS PER GOOGLE SEARCH, THE RASHMI GROUP HAS BEEN SLAPPED WITH RS 660 CRORE NOTICE BY THE INDIAN RAILWAYS FOR EXPORTING LAKHS OF TONNES OF IRON ORE THAT IT SAID WAS DOMESTIC CONSUMPTION BY DECEITFULLY SUBMITTING MISLEADING DOCUMENTS TO THE RAILWAYS. WEB LINK: HTTPS://WWW.ONEINDIA.COM/2011/09/30/RS-660-CRORE-INDIAN-RAILWAY-SCAM-UNEARTHED.HTML. ALSO 26 BILLS FOR IMPORT PAYMENT ARE PENDING FOR REGULARISATION OUT OF WHICH 10 ARE OVERDUE. IN VIEW OF LARGE VALUE FOREX TRANSACTIONS IN THE ACCOUNT, NEGATIVE MEDIA NEWS WHEREIN CUSTOMER GROUP IS SUSPECTED TO BE INVOLVED IN ILLEGAL ACTIVITY AND LARGE NUMBER OF IMPORT BILL PENDING FOR REGULARISATION, STR IS BEING REPORTED.RASHMI METALIKS LIMITED, A PUBLIC LTD COMPANY, HAVING ADDRESS : 39 SHAKESPEARE SARANI 6TH FLOOR, PREMLATA BUILDING, KOLKATA, WEST BENGAL, INDIA, 700017, CONTACT NUMBER: +9830302479 AND EMAIL ID: TREASURY@RASHMIGROUP.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 129305000471 WITH ICICI BANK LTD, 8 AJC BOSE ROAD, CIRCULAR COURT, KOLKATA, WEST BENGAL, 700017, SINCE 08052014. DATE OF INCORPORATION IS 30012004. PAN OF THE ACCOUNT IS AACCR7183E. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0205014305. GSTIN AS PER BANK RECORDS IS 19AACCR7183E1Z6. TRANSACTION LINKED ACCOUNTS ARE 129310002451, 129310002133. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE CHANDI GHARAN NEMUYA (ALNPN1730E), CHANDAN KUMAR MODI (AADHAR:XXXXXXXX9157), DHARMENDRA KUMAR SINGH (AITPS4400P). DIRECTORS OF THE REPORTED ACCOUNT ARE SURENDRA JHA (AKNPJ8666N), SALIL RAY (BENPR3036M) AND CHANDICHARAN NEMUYA (ALNPN1730E). THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION. KYC UPDATION IS DUE. THE FIRM IS INVOLVED INTO MANUFACTURING OF SPONGE IRON, MILD STEEL BILLET, INGOT, DUCTILE IRON PIPE TMT BAR AND WIRE RODS. ANNUAL TURN OVER DECLARED BY CUSTOMER IS NIL. ALERT WAS TRIGGERED FOR LARGE VALUE REMITTANCE TRANSACTIONS. BALANCE IN THE ACCOUNT AT 02052021 WAS RS. 304.55 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 02052021 SHOWS TOTAL CREDIT RS. 1,533.45 LAKHS OF WHICH REMITTANCE RS. 1,215.70 LAKHS MAJORLY FROM MEGHA ENGINEERING AND INFRASTURES LTD, SUBHASH INFRAENGINEERS PVT LTD, FIXED DEPOSIT RS. 315.38 LAKHS. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 1,368.12 LAKHS OF WHICH RTGS RS. 1,000.00 LAKHS MAJORLY TO RASHMI METALS, REMITTANCE RS. 356.56 LAKHS MAJORLY TO GODREJ AND BOYCE MFG. CO. LTD, DALMIA SEVEN REFRACTORIES LTD, KHOSLA ENGINEERING PVT. LTD. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021 : TOTAL CREDIT SIDE RS. 10,857.49 LAKHS OF WHICH NEFT RS. 4,744.71 LAKHS MAJORLY FROM RASHMI METALIKS LTD, REMITTANCE RS. 3,567.45 LAKHS MAJORLY FROM SUBHASH INFRAENGINEERS PVT LTD, SUBHASH INFRAENGINEERS, SUBHASH METALIKS LIMITED, FIXED DEPOSIT RS. 2,544.51 LAKHS. TOTAL DEBIT SIDE RS. 10,718.37 LAKHS OF WHICH TRANSFERS RS. 3,088.37 LAKHS MAJORLY TO RASHMI METALIKS LIMITED, RTGS RS. 3,205.93 LAKHS MAJORLY TO RASHMI METALS, REMITTANCE RS. 2,540.98 LAKHS MAJORLY TO POLYSUN TONGLING EXTRUSION, ETERNAL SEA GROUP LIMITED AND OTHERS RS. 1,868.29 LAKHS. FY 2019-20 FROM 01042019 TO 31032020 : TOTAL CREDIT SIDE RS. 4,250.49 LAKHS OF WHICH CASH WAS RS. 0.30 LAKHS, REMITTANCE RS. 4,220.59 LAKHS . TOTAL DEBIT SIDE RS. 4,250.88 LAKHS OF WHICH CASH WAS RS. 36.58 LAKHS, REMITTANCE RS. 47.30 LAKHS. FY 2018-19 FROM 01042018 TO 31032019 : TOTAL CREDIT SIDE RS. 42.40 LAKHS OF WHICH CASH WAS RS. 0.40 LAKHS. TOTAL DEBIT SIDE RS. 42.12 LAKHS OF WHICH CASH WAS RS. 42.12 LAKHS. FY 2017-18 FROM 01042017 TO 31032018 : TOTAL CREDIT SIDE RS. 22.60 LAKHS OF WHICH CASH WAS RS. 0.40 LAKHS. TOTAL DEBIT SIDE RS. 22.87 LAKHS OF WHICH CASH WAS RS. 22.87 LAKHS.",{"entities":[(3315, 3344,"FOREIGN ORG"),(3996, 4022,"FOREIGN ORG"),(4024, 4049,"FOREIGN ORG"),(3284,3313,"FOREIGN ORG")]}),
            (" ZIOFI EXPORTER, A PROPRIETORSHIP FIRM, HAVING ADDRESS: VILL, MURADPUR P.O., CHAKMAHIMA P.O., BELDA MURADPUR, PASCHIM BARDHAMAN, MEDINIPUR, WEST BENGAL, INDIA, 721424, CONTACT NUMBER: +919874889010 AND EMAIL ID: ZIOFIEXPORTER@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 037105007658 WITH ICICI BANK LTD, SWASTIK, GROUND FLOOR, NAZRUL ISLAM AVENUE, TEGHARIA, VIP ROAD, KOLKATA, WEST BENGAL, 700059, SINCE 04062021. DATE OF FORMATION IS 24032021. PAN NUMBER OF THE ACCOUNT IS APBPK7473B. PAN LINKED ACCOUNTS ARE 037106000022, 634205002966. TRANSACTION LINKED ACCOUNTS ARE 037105007381, 333105500239. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE MOHAMMAD HAMIDULLA KHAN (PAN- APBPK7473B) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN- APBPK7473B. KYC UPDATION IS NOT DUE. AS PER ENHANCED DUE DILIGENCE, CUSTOMER IS INTO BUSINESS OF IMPORT EXPORT OF HUMAN HAIR. PROPRIETOR OF FIRM IS MOHAMMAD HAMIDULLA KHAN. VISIT HAS BEEN CONDUCTED BY BRANCH, ADDRESS IS TRACEABLE AND BUSINESS BOARD IN NAME OF FIRM WAS ALSO SEEN.DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. 40 LAKHS TO LESS THAN RS. 5 CRORES. ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT. BALANCE IN THE ACCOUNT AT 31082021 WAS RS.1.99 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 04062021 TILL 31082021 SHOWS TOTAL CREDIT RS. 903.80 LAKHS OF WHICH CLEARING RS. 1.50 LAKHS MAJORLY FROM CHQ, NEFT RS. 51.50 LAKHS MAJORLY FROM PRASADAM VENKATASURESH KUMAR, HS ENTERPRISE, MUSTAFIJUR RAHAMAN, RTGS RS. 23.87 LAKHS MAJORLY FROM NADIA HUMAN HAIR DOT C, SEIKH NABAB ALI, MD ALOMGIR, REMITTANCE RS. 778.50 LAKHS MAJORLY FROM DHAKA GLOBAL PTE LTD, MU MU INTERNATIONAL TRADING P, CTY TNHH MTV TAN NAM A 059 PH, INTERNET FUND TRANSFERS RS. 22.66 LAKHS FROM ENLIAN SANG NGAIHTE (131401511807), MOBILE MONEY TRANSFERS RS. 24.97 LAKHS IN 20 INSTANCES, UPI TRANSFERS RS. 0.81 LAKHS IN 4 INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 901.80 LAKHS OF WHICH RTGS RS. 155.01 LAKHS MAJORLY TO ANSHU ENTERPRISE, INTERNET FUND TRANSFER RS. 712.31 LAKHS TO KSIX CARGO INTERNATIONAL (037105007381), R H HAIR ENTERPRISE (333105500239), RAJUDDIN SK (090205025024), MOBILE MONEY TRANSFERS RS. 34.13 LAKHS IN 24 INSTANCES. TRANSACTION PATTERN SHOWS THERE WERE HIGH VALUE CREDITS FOLLOWED BY DEBITS IN THE ACCOUNT. FOR EXAMPLE, ON 13-08-2021, RS. 25.89 LAKHS WERE CREDITED THROUGH REMITTANCES FROM DHAKA GLOBAL PTE LTD AT SINGAPORE IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS PAYMENT FOR WASTE HUMAN HAIR AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH INTERNET FUND TRANSFERS TO RAJUDDIN SK (090205025024)SIMILARLY, ON 02-08-2021, RS. 33.35 LAKHS WERE CREDITED THROUGH REMITTANCES FROM DHAKA GLOBAL PTE LTD AT SINGAPORE IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS PAYMENT FOR WASTE HUMAN HAIR AND ON 03-08-2021, RS. 30 LAKHS WERE DEBITED THROUGH INTENET FUND TRANSFERS TO R H HAIR ENTERPRISE (333105500239), KSIX CARGO INTERNATIONAL (037105007381). SIMILAR TRANSACTIONS ARE BEING REPEATED IN THE ACCOUNT. HIGH TURNOVER (RS. 903.80 LAKHS) IN THE ACCOUNTS WITH IN A VERY SHORT SPAN OF TIME (3 MONTHS) AND HENCE SEEM TO BE SUSPICIOUS IN NATURE. IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT AND HIGH TURNOVER IN THE ACCOUNT WITHIN SPAN OF 3 MONTHS, STR IS BEING FILED.",{"entities":[(1791, 1811, 'FOREIGN ORG'),(1813, 1842,'FOREIGN ORG'),(2217, 2241,'FOREIGN ORG'),(3069, 3093,'FOREIGN ORG')]}),
            ("FOR EXAMPLE, ON 28-09-2021, AN AMOUNT OF RS 30 LAKHS WERE CREDITED THROUGH RTGS FROM RIDDHYMIC TECHNOLOGIE SPRIVATE LIMITED (A/C 50200036609140, IFSC HDFC0000240) AND ON 29-09-2021 RS 28.36 LAKHS WERE DEBITED THROUGH CMS TRANSFER IN 3 INSTANCES. ALSO ON 27-10-2021, RS 19 LAKHS WERE CREDITED THROUGH RTGS FROM RIDDHYMIC TECHNOLOGIES PRIVATE LIMITED (A/C 50200036609140, IFSC HDFC0000240) AND ON 30-10-2021 RS 14.87 LAKHS WERE DEBITED THROUGH CMS TRANSFER IN 4 INSTANCES. OVERALL ACCOUNT ACTIVITY SEEMS TO BE SUSPICIOUS. IN VIEW OF HIGH VALUE FUNDS BEING ROUTED, STR IS BEING FILED.RIDDHYMIC TECHNOLOGIES PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS : 171/B MITTAL TOWER, 210, NARIMAN POINT, MUMBAI, MAHARASHTRA, INDIA, 400021, CONTACT NUMBER: +919821038771 AND EMAIL ID: AMITTAL@RIDDHYMIC.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000405111318 WITH ICICI BANK LTD, 215, FREE PRESS HOUSE, FREE PRESS MARG, NARIMAN POINT, MUMBAI, MAHARASHTRA, 400021, SINCE 15022016. DATE OF INCORPORATION IS 23032000. PAN OF THE ACCOUNT IS AABCI0121N. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0316977900. GSTIN AS PER BANK RECORDS IS 27AABCI0121N1ZS. OTHER RELATIONSHIP OF CUSTOMER ARE 000405071677, 000406000878, 000410421880, 000414863995, 000414943708. TRANSACTION LINKED ACCOUNTS ARE 000205001402, 000401737660. BENEFICIAL OWNER/ AUTHORISED SIGNATORY/ DIRECTOR OF THE REPORTED ACCOUNT IS ASHWIN RAMESH MITTAL WITH PAN NO AAEPM3835F. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION. KYC UPDATION IS DUE. MANUAL ALERT WAS RAISED FOR HIGH VALUE FUNDS BEING ROUTED. BALANCE IN THE ACCOUNT AT 31102021 WAS RS.5.54 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31102021 SHOWS TOTAL CREDIT RS. 1,896.00 LAKHS OF WHICH TRANSFERS RS. 14.50 LAKHS MAJORLY FROM RIDDHYMIC TECHNOSERVE LLP, RAMESH BRAHMADUTT MITTAL JT1, NEFT RS. 5.12 LAKHS MAJORLY FROM RIDDHYMIC TECHNOLOGIES PRIVATE LTD, MR KUMAR MEHTA, E PAO GST REFUNDS, RTGS RS. 325.01 LAKHS MAJORLY FROM RIDDHYMICTECHNOLOGIESPVTLTD, E PAO GST REFUNDS, REMITTANCE RS. 169.13 LAKHS MAJORLY FROM TRADE TECH CHINA CO LTD A39 3, INTERNET FUND TRANSFERS RS. 1,382.15 LAKHS FROM COURSES INTELLIGENCE PVT LTD ETC. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 1,904.85 LAKHS OF WHICH CLEARING RS. 463.15 LAKHS, RTGS RS. 1,382.00 LAKHS MAJORLY TO RIDDHYMIC, INTERNET FUND TRANSFER RS. 49.62 LAKH TO DEEPAL MARUTI KAYANDEKA ETC.  FY 2020-21 FROM 01042020 TO 31032021 : TOTAL CREDIT SIDE RS. 383.76 LAKHS OF WHICH TRANSFERS RS. 9.20 LAKHS MAJORLY FROM MAYURI SAMEER ADHIKARI, RAMESH BRAHMADUTT MITTAL JT1, NEFT RS. 19.07 LAKHS MAJORLY FROM WELLNESS FORE UNPAID, RTGS RS. 135.10 LAKHS MAJORLY FROM RIDDHYMICTECHNOLOGIESP, REMITTANCE RS. 216.07 LAKHS. TOTAL DEBIT SIDE RS. 381.44 LAKHS OF WHICH CASH WAS RS. 17.20 LAKHS BY BRANCH, TRANSFERS RS. 17.21 LAKHS MAJORLY TO RAMESH BRAHMADUTT MITTAL JT1, ANUPAM MITTAL, EMPLOYEES PROVIDENT FUND ORGANISATION, CLEARING RS. 93.77 LAKHS, INTERNET FUND TRANSFER RS. 245.54 LAKHS TO DEEPAK MARUTI KAYANDEKA ETC. FY 2019-20 FROM 01042019 TO 31032020 : TOTAL CREDIT SIDE RS. 408.19 LAKHS OF WHICH RS. 0 BY CASH, REMITTANCE RS. 194.96 LAKHS. TOTAL DEBIT SIDE RS. 396.20 LAKHS OF WHICH CASH WAS RS. 3.72 LAKHS, REMITTANCE RS. 0.13 LAKHS. FY 2018-19 FROM 01042018 TO 31032019 : TOTAL CREDIT SIDE RS. 2,699.60 LAKHS OF WHICH RS. 0 BY CASH, REMITTANCE RS. 175.67 LAKHS. TOTAL DEBIT SIDE RS. 2,704.60 LAKHS OF WHICH CASH WAS RS. 10.69 LAKHS, REMITTANCE RS. 0.14 LAKHS. FY 2017-18 FROM 01042017 TO 31032018 : TOTAL CREDIT SIDE RS. 1,213.39 LAKHS OF WHICH RS. 0 BY CASH , REMITTANCE RS. 113.32 LAKHS . TOTAL DEBIT SIDE RS. 1,229.45 LAKHS OF WHICH CASH WAS RS. 13.34 LAKHS, REMITTANCE RS. 0.10 LAKHS . AS PER EDD, THE COMPANY IS ENGAGED IN THE BUSINESS OF PROVIDING BPO SERVICES. IT IS ALSO ENGAGED IN IT HOSTING OR INFRASTRUCTURE PROVISIONING SERVICES. FUNDS WERE CREDITED MAJORLY FROM OWN OTHER ACCOUNT AND DEBITED THROUGH CMS.",{"entities":[(2296,2323,"FOREIGN ORG")]}),
            ("FOLLOWING ARE THE OBSERVATIONS NOTED:1. ACCOUNT ACTIVITY AND TURNOVER IN A NEWLY OPENED RELATIONSHIP ARE NOT IN LINE WITH THE DECLARED TURNOVER AND KNOWN PROFILE OF THE CUSTOMER.2. INTERNET SEARCH DID NOT REVEAL ANY PROPER WEBSITE FOR THE INDIAN COUNTERPARTIES FROM WHERE MAJORITY OF CREDITS RECEIVED IN THE ACCOUNT.3. MAJORITY OF THE FUNDS CREDITED IN THE ACCOUNT FOLLOWED BY IMMEDIATE OUTWARD REMITTANCES IN A SHORT SPAN OF TIME APPEARED UNUSUAL.IN VIEW OF THE AFORESAID REASONS, AN STR IS BEING FILED.3 WINGS PRODUCTION (PAN: AKZPR7517G; GSTN: 27AKZPR7517G1ZC) HOLDS A CURRENT ACCOUNT 006-366728-001 OPENED ON 30082021 WITH THE M G ROAD BRANCH, MUMBAI.IT IS A SOLE PROPRIETORSHIP FIRM INCORPORATED ON 26042018. IT IS INTO THE BUSINESS OF MEDIA/VIDEO PRODUCTION, EXECUTING ASSIGNMENTS FOR MAJOR BRANDS, AD AGENCIES, CORPORATE COMPANIES AND SMALL BUSINESSES NATIONWIDE (HTTPS://3WINGSPRODUCTION.COM/ABOUT.PHP). THE ANNUAL TURNOVER IS DECLARED AS BELOW RS.1 CRORE. ITS ADDRESS IS: 406 ABHISHEK CHS LTD, OFF LINK ROAD, PLOT NO C 5 DALIA INDUSTRIAL, ANDHERI WEST, MUMBAI - 400053, AND PHONE NUMBER IS 9619690070 (M). MR TSANTOSH TKAMRAJU RAO (DOB: 02011981) IS THE PROPRIETOR OF THE FIRM WITH A DECLARED ANNUAL INCOME OF RS.20 LAKHS. HE ALSO HOLDS A SAVINGS ACCOUNT 006-358741-006 SINCE 17082021 WITH THE M G ROAD BRANCH, MUMBAI. HIS ADDRESS IS: FLAT NO A103 MANODAY BUILDING, DATTAPADA ROAD, NEAR SBI BANK, BORIVALI EAST, MUMBAI - 400066, AND PHONE NUMBER IS 9152990026 (M). AN ALERT WAS RAISED BY A STAFF MEMBER AS TRANSACTIONS IN THE ACCOUNT WERE NOTED TO BE SIGNIFICANTLY HIGH COMPARED TO THE DECLARED EXPECTED TURNOVER OF THE FIRM. SIGNIFICANT AMOUNT OF CREDITS NOTED IN THE ACCOUNT WERE SUBSEQUENTLY REMITTED TO OVERSEAS COUNTERPARTIES. IN VIEW OF THE SAME, THE TRANSACTIONS IN THE ABOVE MENTIONED ACCOUNTS WERE REVIEWED, AS BELOW:1. CURRENT ACCOUNT 006-366728-001 (REVIEW PERIOD: 27082021 TO 05122021) CREDITS WERE BY WAY OF:O RTGS/NEFT/IMPS AGGREGATING TO RS.22.48 CRORES (75 TRANCHES), OF WHICH:? RS.12.34 CRORES (32 TRANCHES) WERE FROM RAPID ENTERTAINMENT AND MEDIAWORKS (A/C: 2121234033182742 OF AU SMALL FINANCE BANK LIMITED)? RS.8.98 CRORES (34 TRANCHES) WERE FROM CAPITAL ENTERTAINMENT (A/C:10067204005 OF IDFC BANK & A/C: 200001375028 OF EQUITAS SMALL FINANCE BANK LIMITED)? RS.58 LAKHS (3 TRANCHES) WERE FROM TRIFOLD FILMS (A/C: 2121234033430278 OF AU SMALL FINANCE BANK LIMITED)? RS.32.88 LAKHS (4 TRANCHES) WERE FROM Z S ENTERPRISES (A/C: 0100102000094540 OF IDBI BANK), AND? RS.25 LAKHS (2 TRANCHE) WERE FROM HENAL ENTERPRISES (A/C: 0179360000001649 OF DBS BANK LTDO A CHEQUE DEPOSIT OF RS.50,000 FROM ITS OWN ACCOUNT HELD WITH ICICI BANK TOWARDS OPENING THE ACCOUNT. DEBITS WERE BY WAY OF:O OUTWARD REMITTANCES AGGREGATING TO RS.20.36 CRORES (35 TRANCHES) BEING ADVISED AS AUDIO VISUAL RELATED SERVICES LIKE MOTION PICTURES, VIDEO TAPE PRODUCTION, DISTRIBUTION AND PRODUCTION SERVICES, OF WHICH:? RS.10.36 CRORES (17 TRANCHES) WERE TOWARDS FILMNATION ENTERTAINMENT LLC, UNITED ARAB EMIRATES? RS.7.68 CRORES (14 TRANCHES) WERE TOWARDS SUNRISE AUDIO VISUALS PTE LTD, SINGAPORE? RS.1.76 CRORES (3 TRANCHES) WERE TOWARDS MARK1 GLOBAL ENTERPRISES PTE LTD, SINGAPORE, AND ? RS.56.41 LAKHS WERE TOWARDS EFFORT POINT INTERNATIONAL GROUP COMPANY LIMITED, CHINA O RTGS/NEFT AGGREGATING TO RS.1.30 CRORES (9 TRANCHES), OF WHICH:? RS.60 LAKHS WERE TOWARDS CAPITAL ENTERTAINMENT (VENDOR PAYMENT)? RS.58.65 LAKHS WERE TOWARDS FRIENDS FILMS AND ENTERTAINMENT (VENDOR PAYMENT)? RS.6.30 LAKHS (6 TRANCHES) WERE TOWARDS THE PROPRIETORS PERSONAL ACCOUNT HELD WITH ICICI BANK (A/C: 041801004577) AND INDIAN BANK (A/C: 6024524004)? RS.5 LAKHS WERE TOWARDS THE STANDARD FILM COMPANY (VENDOR PAYMENT)O CASH WITHDRAWALS AGGREGATING TO RS.2.01 LAKHS (3 TRANCHES) O CHEQUES ISSUED AGGREGATING TO RS.1.15 LAKHS (3 TRANCHES)2. SAVINGS ACCOUNT 006-358741-006 (REVIEW PERIOD: 17082021 TO 05122021) CREDITS WERE BY WAY OF IMPS FOR RS.1.50 LAKHS FROM 3 WINGS PRODUCTION DEBITS WERE BY WAY OF IMPS AGGREGATING TO RS.1.27 LAKHS (9 TRANCHES) TO HIS OWN ACCOUNT HELD WITH ICICI BANK.",{"entities":[(2251,2285,"FOREIGN ORG"),(2958, 2986,"FOREIGN ORG"),(3048,3076,"FOREIGN ORG"),(3135,3167,"FOREIGN ORG")]}),
            ("? DEBITS AGGREGATED TO RS.258.61 CRORES (IN 435 TRANCHES) COMPRISING OF:  RS.115.78 CRORES (IN 19 TRANCHES) TOWARD IMPORT BILL PAYMENTS AND IMPORT BILL CHARGES. NEFT OF RS.80.10 CRORES (IN 3 TRANCHES) TOWARDS CUSTOMERS OWN ACCOUNT 10324773582 HELD WITH SBI AND ACCOUNT 00030330017866 HELD WITH HDFC. OUTWARD REMITTANCE OF RS.14.35 CRORES (IN 3 TRANCHES) TO HSBC SINGAPORE ACCOUNT 000038865 TOWARDS REPAYMENT OF INTEREST AND PRINCIPAL OF EXTERNAL COMMERCIAL BORROWINGS. RS.37.20 LAKHS (IN 10 TRANCHES) TOWARDS REPAYMENT OF LOAN FACILITIES AVAILED FROM HSBC OUTWARD REMITTANCE OF RS.1.86 CRORES (IN 2 TRANCHES) TO HSBC BANK MAURITIUS LTD. TOWARDS INTEREST ON EXTERNAL COMMERCIAL BORROWINGS. OUTWARD REMITTANCE OF RS.8.03 LAKHS (IN 2 TRANCHES) TO PRECURSOR GROUP PTE. LTD. (629160177001, OVERSEA-CHINESE BANKING CORP LTD.) WITH THE NARRATION AS ACCOUNTING, AUDITING AND BOOK KEEPING).2. REVIEW OF TRADE TRANSACTIONS NO IMPORTS OR EXPORTS OBSERVED DURING THE REVIEW PERIOD.  TOTAL IDPMS OUTSTANDING IS RS.102.20 CRORES (IN 56 TRANCHES) OF WHICH RS.38.41 CRORES (IN 30 TRANCHES) PERTAINS TO THE YEAR 2016, RS. 1.89 CRORES (IN 2 TRANCHES) PERTAINS TO THE YEAR 2017 AND RS.4.09 CRORES (IN 3 TRANCHES) PERTAINS TO THE YEAR 2020, RS.58.24 CRORES (IN 18 TRANCHES) PERTAINS TO THE YEAR 2021.3. NO TRANSACTIONS OBSERVED IN THE ESCROW ACCOUNT 166-176743-002 DURING THE REVIEW PERIOD. TRANSACTIONS IN THE ACCOUNT DO NOT INDICATE ANY UNUSUAL PATTERN. HOWEVER, THE NOTICE RECEIVED FROM CBI, APPEARS TO BE IN THE SAME CASE PERTAINING TO THE TERMINAL EXCISE DUTY REFUNDS CASE FOR WHICH NOTICES WERE RECEIVED EARLIER FROM OTHER AUTHORITIES. ACCORDINGLY, A SUPPLEMENTARY STR IS BEING FILED.CRYSTAL CROP PROTECTION LTD. (CCPL) (PAN: AABCJ3574E, DATE OF INCORPORATION: 13071994) HOLDS BELOW ACCOUNTS WITH THE NEW DELHI BRANCH - CURRENT ACCOUNT 166-176743-001, SINCE 20012011. ESCROW ACCOUNT 166-176743-002 SINCE 06102016 MULTIPLE TERM DEPOSITS AGGREGATING TO RS.4.83 CRORES LOAN ACCOUNT 041-024134-700 FOR USD 50 LAKH WORKING CAPITAL LOAN OF RS.123 CRORESITS DIRECTORS ARE MR. MOHIT KUMAR GOEL, MR. ANIL JAIN, MR. SARTAJ SEWA SINGH, MR. CHETAN RAMESHCHANDRA DESAI, MRS SANGEETA KAPIL JIT SINGH. ITS KEY CONTROLLERS ARE MR. NAND KISHORE AGGARWAL, MR. ANKUR AGGARWAL, MR. SUMEET SOOD, MR. DINESH KUMAR GUPTA AND MR. PIYUSH JAIN AND THE BENEFICIAL OWNER IS MR. KANAK AGGARWAL. THE REGISTERED ADDRESS IS B-95, WAZIPUR INDUSTRIAL AREA, DELHI, INDIA 110052 AND CONTACT NUMBERS ARE (M) 9958793104 AND (O) 011-27006800/11. COMPANYS TURNOVER FOR FINANCIAL YEAR 2019-2020 WAS RS.1,450 CRORES.STRS WERE PREVIOUSLY FILED ON CCPL, AS UNDER: 25012018 VIDE FIU REFERENCE NUMBER 1801255099, IN VIEW OF MULTIPLE DISCREPANCIES IN REGULARIZATION OF EXPORT BILLS OF 2014 AND SIGNIFICANT VALUE OF OLD EXPORT BILLS PENDING FOR REGULARIZATION. 03062020 VIDE FIU REFERENCE NUMBER 2006035602, IN VIEW OF ADVERSE MEDIA REPORTS REGARDING INVESTIGATION BY CBI AGAINST MR. A K SINGH, THE THEN JOINT DIRECTOR GENERAL OF FOREIGN TRADE, AHMEDABAD AND CCPL AND ITS DIRECTORS IN THE MATTER OF FRAUDULENT CLAIM OF TERMINAL EXCISE DUTY BENEFITS THAT CAUSED WRONGFUL LOSS TO THE GOVERNMENT OF INDIA AND IN VIEW OF HIGH VALUE OF OLD OUTSTANDING IMPORT BILLS 02022021 VIDE FIU REFERENCE NUMBER 2102029851, IN VIEW OF FRESH NEGATIVE NEWS REGARDING CASE FILED BY ED UNDER PMLA IN RELATION TO ALLEGED FAKE REFUND CLAIMS OF TERMINAL EXCISE DUTY.ALERT WAS GENERATED ON CCPL SINCE BANK HAS RECEIVED A NOTICE ISSUED BY SHRI. S S BHADORIYA, DY. SUPDT. OF POLICE, CBI-ACB, GANDHINAGAR, DATED 29052021, IN CONNECTION TO A CASE, REGISTERED AGAINST SHRI A K SINGH (FORMER JOINT DIRECTOR, DGFT, AHMEDABAD), CCPL, ITS DIRECTORS AND UNKNOWN OTHERS, U/S 120B R/W 420 OF IPC AND SEC. 13(2) R/W SEC. 13(1)(D) OF THE PREVENTION OF CORRUPTION ACT, 1988. ACCORDINGLY, ACCOUNTS WERE REVIEWED FROM 20012021 TO 02092021, WHICH HIGHLIGHTED THE FOLLOWING - 1. IN CURRENT ACCOUNT NO. 166-176743-001? CREDITS AGGREGATED TO RS.268.08 CRORES (IN 63 TRANCHES) COMPRISING OF: NEFT/RTGS OF RS.181.62 CRORES (IN 49 TRANCHES) FROM OWN ACCOUNT 00030330017866 HELD WITH HDFC. INWARD REMITTANCE OF RS.37.12 CRORES FROM CUSTOMERS NOSTRO ACCOUNT AS SURPLUS FUNDS TRANSFERRED. LOAN DISBURSAL OF RS.30.49 CRORES (IN 10 TRANCHES) AND RS.49.45 LAKHS (IN 9 TRANCHES) AS DRAWDOWN FROM 166-176743-451, 166-176743-452, 166-176743-453 AND 166-176743-454. INWARD REMITTANCE OF RS.17.84 CRORES FROM LOTUS GLOBAL PTE. LTD.S ACCOUNT 054361452 HELD WITH HSBC, SINGAPORE. INWARD REMITTANCE OF RS.92.25 LAKHS ON 22042021 FROM HEBEI BESTAR COMMERCE AND TRADE (ACCOUNT: 50354014040000583, AGRICULTURAL BANK OF CHINA) WITH THE NARRATION AS PAYMENT FOR CONSULTING FEE. INWARD REMITTANCE OF RS.7.39 LAKHS ON 04082021 FROM BEIJING LUCKYSTAR CO. LTD. (ACCOUNT: 77012025000010095, BANK OF NINGBO CO LTD, CHINA).",{"entities":[(357,371,"FOREIGN ORG"),(612,635,"FOREIGN ORG"),(785,817,"FOREIGN ORG"),(4387,4404,"FOREIGN ORG"),(4509,4540,"FOREIGN ORG"),(4570,4596,"FOREIGN ORG"),(4700,4725,"FOREIGN ORG"),(4756,4777,"FOREIGN ORG")]}),
            ("ON COMPARISON OF SAMPLE DOCUMENTS INVOICES FURNISHED BY VIEW CINECRAFT TECHNOLOGY P LTD ON WHOM BANK HAS ALREADY FILED STR AND SAMPLE DOCUMENTS FURNISHED BY ARROW SERVICES, IT WAS NOTED THAT WHILE THE FUNDS WERE REMITTED TO A COMMON OVERSEAS PARTY NETCLICKS MEDIA S PTE LTD SINGAPORE, THE INVOICES HAD DIFFERENT FORMATS. ALSO THE INVOICE NUMBERS DID NOT APPEAR TO BE IN SERIAL ORDER WHICH APPEARS TO BE UNUSUAL. DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS OF ARROW SERVICES, IT WAS NOTED THAT THE OFFICE SETUP WAS A 600 SQUARE FEET RENTED PREMISE WITH TEMPORARILY SIGN BOARD NAME OF ENTITY PRINTED ON BANNER, CHAIR, TABLE, LAPTOP AND TWO EMPLOYEES. HOWEVER, NO EQUIPMENT OR ANY BANNER/PICTURES RELATED TO ADVERTISEMENT SERVICES GIVEN BY THE CUSTOMER WAS OBSERVED IN THE OFFICE. IT IS PERTINENT TO NOTE THAT THE ENTITY IS FORMED ON 20.7.2021, ACCOUNT IS OPENED WITHIN 9 DAYS I.E., ON 29.7.2021 AND WITHIN 2 MONTHS OF ACCOUNT OPENING TRANSACTIONS ROUTED AMOUNTED TO RS.4.33 CR. HENCE IT APPEARS DOUBTFUL THAT THE CUSTOMER HAS THE NET WORTH / GENUINE BUSINESS ACTIVITY TO ROUTE SUCH HIGH VALUE TRANSACTIONS AND CUSTOMER MAY HAVE DELIBERATELY INDICATED INFLATED TURNOVER TO AVOID BEING TRACKED ON ROUTING HIGH VALUE TRANSACTIONS. ALSO, NO MAJOR BUSINESS RELATED TRANSACTIONS APPEARS TO BE ROUTED THROUGH THE ACCOUNT LIKE PAYMENTS TOWARDS SALARY, RENT AND ETC., WHICH IS UNUSUAL WHILE RUNNING A COMMERCIAL BUSINESS. BASIS THE TRANSACTION PATTERN AND INFORMATION AVAILABLE, IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN NAME OF ARROW SERVICES AND ACCOUNT MAY HAVE BEEN OPENED TO ROUTE TRANSACTIONS FOR LAYERING OF FUNDS AND TO REMIT FUNDS OVERSEAS UNDER PRETEXT OF ADVERTISING SERVICES WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENT. QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS 5.17 CR. IN CASH RS 0.00 CR. AND CREDIT: RS 5.20 CR. IN CASH RS 0.00 CR.. THE ACCOUNT OF ARROW SERVICES IS ACTIVE WITH BALANCE OF RS.2.82 LAKHS AS ON 17.09.2021.DAYA KISHAN AS A PROPRIETOR MAINTAINS A CURRENT ACCOUNT BEARING NO. 0945488284 IN THE NAME OF A SOLE PROPRIETORSHIP CONCERN ARROW SERVICES SINCE 29.07.2021 AT OUR BRANCH LOCATED AT C-2/11, ASHOK VIHAR-2, 110052 - NEW DELHI. THE ENTITY WAS ESTABLISHED ON 20.07.2021 AND HAS SUBMITTED COPIES OF PAN GYLPK7247C, GST REG. CERT. 07GYLPK7247C1Z6, IEC CERT. GYLPK7247C AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 5TH FLOOR, UNIT NO. 514. P. NO. 2, AMBA TOWER COMMUNITY CENTER, SEC-9, ROHINI, NORTH WEST DELHI, 110085 - DELHI AND HAS REGISTERED MOBILE NUMBER  7840015796. THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND CUSTOMER IS NOT YET DUE FOR PERIODIC UPDATION OF KYC. IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF ADVERTISING WITH ANNUAL TURNOVER OF RS.10.00 CR. WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES NON-IMPORT DONE THROUGH THE BANK, IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NON-IMPORT WERE MADE THROUGH THE NEWLY OPENED ACCOUNT OF ARROW SERVICES FAVORING SINGAPORE BASED ENTITY NAMED NETCLICKS. BANK HAS ALREADY FILED STRS ON ANOTHER CUSTOMER VIEW CINECRAFT TECHNOLOGY PRIVATE LIMITED A/C NO. 5545467301 VIDE BATCH ID 2107035260 WHEREIN FUNDS WERE REMITTED TO THE SAID OVERSEAS PARTY AND THE ACCOUNT ACTIVITY APPEARED TO BE SUSPICION. HENCE DUE DILIGENCE WAS CARRIED OUT ON THE ACCOUNT OF ARROW SERVICES AND THIS REPORT IS FILED ON THE FOLLOWING GROUNDS OF SUSPICION: ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL 17.09.2021, IT WAS NOTED THAT RTGS/NEFT/IMPS CREDITS AMOUNTED  TO RS. 5.19 CR MAINLY FROM OWN ACCOUNT OF ARROW SERVICES MAINTAINED WITH OTHER BANK RS. 3.11 CR. , YES BANK - 107563300002139, BALAJI ENTERPRISES RS.1.57 CR, PUNJAB NATIONAL BANK - 50591132000948 AND SHRI RAM ENTERPRISES RS.0.51 CR, PUNJAB NATIONAL BANK - 50591132000893. SUBSEQUENT TO SUCH CREDIT, FOREIGN OUTWARD REMITTANCE NON-IMPORT AMOUNTED TO RS.3.98 CR MAINLY FAVORING ADVANCE MEDIA FILM PRODUCTION LLC- UAE RS. 1.73 CR., QUICKODE PTE LTD- SINGAPORE RS. 1.04 CR., DREAMBOX STUDIOS LIMITED- HONG KONG RS. 0.88 CR. AND NETCLICKS - SINGAPORE RS.0.31 CR TOWARDS ADVERTISEMENT SERVICES. RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS.0.76 CR. MAINLY FAVORING  ARROW SERVICES RS. 0.47 CR. , YES BANK - 107563300002139, SPARX TRADING RS.0.22 CR, PUNJAB NATIONAL BANK - 0120002100765727 AND VIVEK ENTERPRISES RS.0.20 CR, THE RATNAKAR BANK LTD BANK, SAILOGISTICS SHIPPING LLP,INFINITY DIGITAL SOLUTIONS,GREENLAB DIAMONDS LLP,HINDVA DEVELOPERS LLP,CLASSIC LOGISTICS PTE LTD,ALANA FOOD STUFF TRADING LLC,DIGITAL TRADE FINANCE COMPANY, R H S FOODSTUFF TRADING LLC,OCEAN NETWORK EXPRESS PTE,DYJ LOGISTICS LTD,A2S LOGISTICS CO LTD,KINGS AIR LOGISTICS COMPANY,DLOCAL LLP NO,SHAKTI PUMPS FZE,SHAKTI PUMPS USA LLC,MUTLUSU POMPA SAN",{"entities":[(248,283,"FOREIGN ORG"),(3083,3115,"FOREIGN ORG"),(4067,4083,"FOREIGN ORG"),(4109,4125,"FOREIGN ORG"),(4162, 4183,"FOREIGN ORG"),(4350,4362,"FOREIGN ORG"),(4014,4055,"FOREIGN ORG"),(4478,4503,"FOREIGN ORG"),(4505,4530,"FOREIGN ORG"),(4531,4552,"FOREIGN ORG"),(4553,4574,"FOREIGN ORG"),(4575,4600,"FOREIGN ORG"),(4601,4629,"FOREIGN ORG"),(4630,4659,"FOREIGN ORG"),(4689,4714,"FOREIGN ORG"),(4715,4732,"FOREIGN ORG"),(4733,4753,"FOREIGN ORG"),(4754,4781,"FOREIGN ORG"),(4782,4792,"FOREIGN ORG"),(4796,4812,"FOREIGN ORG"),(4813, 4833,"FOREIGN ORG"),(4662,4688,"FOREIGN ORG"),(4834,4851,"FOREIGN ORG")]}),
            ("BEETREAT LOGISTICS PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS: OFFICE NO 102, 1ST FLOOR, SECTOR 11, CBD BELAPUR, PLOT NO 28, NAVI MUMBAI, MAHARASHTRA, INDIA, 400614, CONTACT NUMBER: +918828759083 AND EMAIL ID: BEETREATLOGISTICS2535@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 087305003472 WITH ICICI BANK LTD, BALAJI BHAVAN, NEAR BELAPUR RAILWAY STN., CBD BELAPUR, NAVI MUMBAI, MAHARASHTRA, 400614, SINCE 16092020. DATE OF INCORPORATION IS 14032020. PAN NUMBER OF THE ACCOUNT IS AAICB9599G. GSTIN AS PER BANK RECORDS IS 27AAICB9599G1ZZ. TRANSACTION LINKED ACCOUNTS ARE 087305003470, 087301517236. AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS/ DIRECTORS OF THE REPORTED ACCOUNT ARE DINESH KUMAR WITH PAN NO ANXPK9639BVIKAS NARAWAT (PAN: AIJPV5516D). THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON 06022021. KYC DOCUMENTS OBTAINED WERE PAN - AAICB9599G, COI: U51909MH2020PTC338829, MOA AOA AND PAN OF AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS. CUSTOMER IS INTO THE BUSINESS OF LOGISTICS SERVICES, TRADING, WHOLES RETAILS, IMPORT AND EXPORT OF GOODS AND SERVICES FOR MAKING AND EARING PROFITS, WORKS ON COMMISSION BASIS FOR GOODS TRADING. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS GREATER THAN EQUALS TO RS. 100 CRORES. ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 16092020 TO 06022021: TOTAL CREDIT SIDE RS. 6,325.65 LAKHS OF WHICH TRANSFERS RS. 5.00 LAKHS MAJORLY FROM DINESH KUMAR, NEFT RS. 201.45 LAKHS MAJORLY FROM BONIRY LOGISTICS, RTGS RS. 6,118.92 LAKHS MAJORLY FROM BONIRY LOGISTICS, AEGIS LOGISTICS, MOBILE MONEY TRANSFERS RS. 0.16 LAKHS IN 18 INSTANCES. TOTAL DEBIT SIDE RS. 6,325.65 LAKHS OF WHICH CASH WAS RS. 8.55 LAKHS BY ATM AT THANE, NEW BOMBAY 1 - BELAPUR CIBD BRANCH, CLEARING RS. 12.85 LAKHS MAJORLY TO BEETREAT LOGISTICS PRIVATE LIMITED, RTGS RS. 827.87 LAKHS MAJORLY TO BONIRY LOGISTICS, REMITTANCE RS. 5,450.93 LAKHS MAJORLY TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG, IN USD CURRENCY FOR WHICH PURPOSE WAS MENTIONED AS TOWARDS OTHER PAYMENTS BY SHIPPING COMPANIES, KM LOGISTICS AND SHIPPING PTE LTD IN USD TO SINGAPORE FOR OTHER PAYMENTS BY SHIPPING COMPANIES, HMMA LOGISTICS CO LIMITED IN USD TO HONG KONG FOR OTHER PAYMENTS BY SHIPPING COMPANIES, LUCKY GAINS LIMITED IN USD TO HONG KONG FOR WHICH PURPOSE WAS MENTIONED AS PAYMENT TOWARDS SHIPPING CHARGES, INTERNET FUND TRANSFER RS. 23.05 LAKHS TO KEMPS LOGISTICS PRIVATE LIMITED (087305003470), MOBILE MONEY TRANSFERS RS. 2.20 LAKHS IN 6 INSTANCES.ON ANALYZING TRANSACTION PATTERN IN THE ACCOUNT, IT HAS BEEN OBSERVED THAT THERE WERE HIGH VALUE NON CASH CREDITS FOLLOWED BY DEBITS THROUGH REMITTANCES. FOR EXAMPLE, ON 16-12-2020, RS. 110.02 LAKHS WERE CREDITED THROUGH RTGS FROM BONIRY LOGISTICS (ACCOUNT NO: 920020063370822, IFSC CODE-UTIB0001532) AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH REMITTANCE TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG, IN USD CURRENCY FOR WHICH PURPOSE WAS MENTIONED AS TOWARDS OTHER PAYMENTS BY SHIPPING COMPANIES. ON 19-10-2020, RS. 60 LAKHS WERE CREDITED THROUGH RTGS FROM AEGIS LOGISTICS (ACCOUNT NO: 052012100000508, IFSC CODE: ASBL0000052) AND ON 20-10-2020 RS. 62.41 LAKHS WERE DEBITED THROUGH REMITTANCE TO LUCKY GAINS LIMITED IN USD TO HONG KONG FOR WHICH PURPOSE WAS MENTIONED AS PAYMENT TOWARDS SHIPPING CHARGES. SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT WITHOUT ANY BUSINESS EXPENDITURE. TRANSACTIONS SEEM UNUSUAL AND NO SATISFACTORY EXPLANATION ABOUT IT CAN BE OBTAINED FROM THE CUSTOMER. ACCOUNT CLOSED BY CUSTOMER WITHIN SPAN OF 5 MONTHS AND HIGH TURNOVER IN THE ACCOUNT WITHIN SPAN OF 5 MONTHS (RS. 6,325.65 LAKHS), WHICH RAISED FURTHER SUSPICION. IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT, STR IS BEING FILED.",{"entities":[(3071,3098,"FOREIGN ORG"),(2939, 2955,"FOREIGN ORG"),(2368, 2393,"FOREIGN ORG"),(2272, 2305,"FOREIGN ORG"),(2133, 2160,"FOREIGN ORG"),(1067,1101,"FOREIGN ORG")]}),
            ("3 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER KYTEUS P LTD, IT WAS NOTED THAT THE OFFICE WAS SHIFTED TO AN ALTERNATE LOCATION I.E., 503, DDA 2 BUILDING, DISTRICT CENTRE, JANAKPURI, WEST DELHI. AT THE MENTIONED ADDRESS SMALL SIGN BOARDS IN THE NAMES OF KYTEUS P LTD AND GLOBUS HOLIDAYS WERE SEEN BY BRANCH OFFICIAL WITH OFFICE SET UP AND FEW FREELANCERS NO PERMANENT EMPLOYEES IN 700 SQ. FT. RENTED PREMISE. 4 ALSO MIDDLE ASIA OVERSEAS AND PROCON INTERNATIONAL SHARE A COMMON ADDRESS - A-49 FIRST FLOOR MAHIPALPUR EXTN NEW DELHI SOUTH WEST DELHI 110037. DURING VISIT BY BRANCH OFFICIAL AT THE SAID ADDRESS TWO DIFFERENT TEMPORARY SIGN BOARDS WERE SEEN IN THE NAMES OF MIDDLE ASIA OVERSEAS AND IN THE NAME OF PROCON INTERNATIONAL IMPORT AND EXPORT. OFFICE SET UP WAS SEEN WITH TABLES, CHAIRS AND FEW DESKTOPS HOWEVER NO MAJOR BUSINESS ACTIVITY IN LINE WITH THE TURNOVER INDICATED OR TRANSACTIONS ROUTED COULD BE SIGHTED. 5 IT IS PERTINENT TO NOTE THAT THE CUSTOMER PROCON INTERNATIONAL HAS INDICATED NATURE OF BUSINESS AS AUTOMOBILE WHOLESALER AND THE SIGN BOARD ALSO INDICATES IMPORT AND EXPORT BUSINESS HOWEVER CUSTOMER HAS ROUTED TRANSACTIONS RELATED TO FREIGHT PAYMENTS. FURTHER, KYTEUS P LTD WHICH IS INTO BUSINESS OF IT SERVICES HAS RECEIVED RTGS CREDITS FROM MIDDLE ASIA OVERSEAS WHICH HAS DECLARED BUSINESS AS COURIER AND FREIGHT SERVICES. HENCE, IT APPEARS THAT THE ENTITIES KYTEUS P LTD, MIDDLE ASIA OVERSEAS AND PROCON INTERNATIONAL HAVE ROUTED TRANSACTIONS UNRELATED TO BUSINESS ACTIVITY WHICH MAY HAVE NO ECONOMIC RATIONALE AND SHELL COMPANIES MAY HAVE BEEN FORMED WITH DIFFERENT NATURE OF BUSINESSES BY A COMMON GROUP OF INDIVIDUALS WITHOUT GENUINE COMMERCIAL BUSINESS. 6 A PATTERN OF TRANSACTIONS WERE NOTED IN THE ACCOUNTS WHEREIN HIGH VALUE ROUND SUM RTGS/NEFT CREDITS RECEIVED MAINLY FROM OWN ACCOUNTS MAINTAINED WITH OTHER BANKS WERE REMITTED OVERSEAS AS FOREIGN OUTWARD REMITTANCES NON-IMPORT TOWARDS FREIGHT PAYMENTS OR TOWARDS SOFTWARE CONSULTANCY. NO PAYMENTS TOWARDS BUSINESS EXPENSES LIKE TAXES, RENT, UTILITY PAYMENTS COULD BE SEEN THROUGH THE ACCOUNTS DURING REVIEW PERIOD. 7 DURING SCRUTINY OF SAMPLE DOCUMENTS FURNISHED BY THE CUSTOMER IT WAS NOTED THAT ALL INVOICES WERE COMPUTER GENERATED WITHOUT THE SIGN AND STAMP OF OVERSEAS PARTIES. IT COULD BE POSSIBLE THAT THE DOCUMENTS MAY HAVE BEEN FABRICATED MERELY TO SATISFY BANK OFFICIALS TO REMIT FUNDS OVERSEAS. 8 POST QUERIES, NO MAJOR TRANSACTIONS WERE ROUTED THROUGH THE CURRENT ACCOUNTS. 9 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS IT SERVICES / FREIGHT PAYMENTS WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF LAYERING AND TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENTS.  ACCOUNT OF KYTEUS P LTDIS ACTIVE WITH BALANCE OF RS.23,562/- AS ON NOVEMBER 9, 2021. QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS.6.65 CR IN CASH RS.0 .00 CR AND CREDIT: RS.6.69 CR IN CASH RS.0.00 CR.A CURRENT ACCOUNT BEARING NUMBER 0345219266 WAS OPENED IN THE NAME OF KYTEUS PRIVATE LIMITED ON 06.07.2021 AT OUR BRANCH LOCATED AT B-1 BLOCK, JANAKPURI, NEW DELHI PIN CODE  110027. THE ENTITY WAS ESTABLISHED ON 01.08.2020 AND HAS SUBMITTED COPIES OF PAN AAICK2043E, BOARD RESOLUTION, MOA/ AOA, CERT OF INCORP U52520DL2020PTC367254, GST REG. CERT. 07AAICK2043E1ZV, IEC CERT. AAICK2043E AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING. SARANSH SHARMA AND AMIT KANOJIA ARE THE DIRECTORS AND UBOS IN THE COMPANY. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 322 JAINA TOWER 1 DISTRICT CENTRE JANAKPURI NEW DELHI WEST DELHI, 110058 AND HAS REGISTERED MOBILE NUMBER 9971331110. THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND THE ACCOUNT IS NOT YET DUE FOR PERIODIC UPDATION OF KYC. IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF IT INDUSTRY - SERVICES WITH ANNUAL TURNOVER OF RS.3.00 CR. BASIS INTER-CONNECTEDNESS WITH ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT ACCOUNTS IN THE NAMES OF MIDDLE ASIA OVERSEAS A/C NO.0945390907, CONSTITUTION-SOLE PROPRIETORSHIP, NATURE OF BUSINESS- LOGISTICS/COURIER - SERVICES, ANNUAL TURNOVER-RS.15.00 CR, PAN-NFWPS0876E, DATE OF INCORPORATION-21-OCT-20, PROCON INTERNATIONAL A/C NO.1345552377, CONSTITUTION-PARTNERSHIP FIRM, NATURE OF BUSINESS - AUTOMOBILES - WHOLESALERS, ANNUAL TURNOVER- RS.60.00 CR, PAN-AAZFP2473P, DATE OF INCORPORATION - 30-SEP-20 AND THE SWISS TOURS A/C NO.3411625497, CONSTITUTION-PARTNERSHIP, NATURE OF BUSINESS - TRAVEL AGENCIES, ANNUAL TURNOVER-RS.1.00 CR, PAN-AAKFT1288M, DATE OF INCORPORATION-08-MAY-15. THE SAID ACCOUNTS WERE OPENED DURING THE PERIOD 28-JUL-21 TILL 19-AUG-21 AT OUR ABOVE MENTIONED BRANCH EXCEPT THE SWISS TOURS WHICH WAS OPENED ON 22-JUL-15 AND CLOSED SINCE 30-JUL-2016. GAURAV SEHGAL, JASPAL SINGH BINDRA, SARANSH SHARMA AND AMIT KANOJIA ARE THE COMMON PROPRIETORS/ PARTNERS IN THE MENTIONED ACCOUNTS. WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES NON-IMPORT DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES WERE DONE FROM THE ABOVE MENTIONED NEWLY OPENED ACCOUNT OF KYTEUS P LTD. THOUGH KYTEUS P LTD IS INTO BUSINESS OF IT SERVICES HAD RECEIVED RTGS/NEFT CREDITS FROM MIDDLE ASIA OVERSEAS WHICH ALSO MAINTAINS AN ACCOUNT BEARING NUMBER 0945390907. HENCE DUE DILIGENCE WAS CARRIED OUT AND THIS CONSOLIDATED REPORT IS FILED ON THE ABOVE MENTIONED ACCOUNTS BASIS INTER-CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL NOVEMBER 5, 2021 IT WAS NOTED THAT IMPS/ RTGS/NEFT CREDITS TOTALLING TO RS.6.65 CR WERE RECEIVED IN THE ACCOUNTS BY WAY MAINLY FROM KYTEUS PRIVATE LIMITED	RS.2.66 CR, FINCARE SMALL FINANCE BANK  21200000014691, HDFC  50200053989840, ICICI  663005601270, AXIS BANK - 920020058932574, MIDDLE ASIA OVERSEAS RS.1.80 CR, BANDHAN BANK  11210001677347, LAKSHMI VILAS BANK - 0499360000005708 AND PROCON INTERNATIONAL RS.0.60 CR, BANDHAN BANK  11210001951731/ ICICI - ICI8882032973. 2 SUBSEQUENT TO SUCH CREDITS FOREIGN OUTWARD REMITTANCES NON-IMPORT AMOUNTED TO RS.6.17 CR MAINLY FAVOURING TROJEN SG PTE LTD - SINGAPORE RS.1.97 CR FROM MIDDLE ASIA OVERSEAS TOWARDS FREIGHT PAYMENTS, SUCCESS BLOSSOM SG PTE LTD  SINGAPORE RS.1.39 CR FROM A/C OF KYTEUS P LTD TOWARDS SOFTWARE CONSULTANCY AND BLISS QUAY SG PTE LTD - SINGAPORE RS.0.98 CR FROM A/C OF PROCON INTERNATIONAL TOWARDS FREIGHT ON IMPORTS.",{"entities":[(6510,6543,"FOREIGN ORG"),(6403,6429,"FOREIGN ORG"),(6300,6339,"FOREIGN ORG"),(6116,6147,"FOREIGN ORG")]}),
            ("1. OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO HIGH VOLUME OF SHIPPING BILLS PENDING AS PER EDPMS AMOUNTING TO RS. 495.41 LAKHS. CUSTOMER IS NOT SUBMITTING DOCUMENTS AFTER MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASE.. 2. SINCE THE SHIPPING BILLS ARE GENERATED, THE EXPORTS HAS ALREADY TAKEN PLACE. NEITHER CUSTOMER HAS SUBMITTED EXPORT DOCUMENT WITH US NOR ANY OTHER BANK HAS REQUESTED FOR AD TRANSFER OF THE SHIPPING BILLS. ACCORDINGLY, POSSIBLY EXPORT IS DONE ONLY FOR VALUE TRANSFER FROM INDIA TO OVERSEAS. 3. SHIPPING BILLS PENDING (FOR FIVE YEARS 2016-2021) FOR INR 381.73 LAKHS WHILE TOTAL EXPORT DONE BY CUSTOMER AMOUNTING TO INR 3307.17 LAKHS 4. THE EXPORT PENDENCY REFLECTING IN EDPMS ALSO (FOR FOUR YEARS 2017-2021) FOR INR 495.41 LAKHS. 5..CUSTOMER HAS RECEIVED REMITTANCE MAINLY FROM TOPMOST SHIPPING LLC, CN INTERNATIONAL, YH TRADING, MOH TRADING CO LTD MAINLY FROM UNITED STATES , DUBAI, SOUTH KOREA ONLY. TRANSACTIONS SEEMS IN LINE WITH CONTRA PARTIES. 6. ) CUSTOMER HAS DONE TRANSACTIONS WITHIN GROUP ACCOUNT ONLY. 3. STR IS FILED IN LIGHT OF A). SHIPPING BILLS PENDING AS PER EDPMS AGGREGATING TO RS.495.41 LAKHS.. B) CUSTOMER IS NOT READY TO SUBMIT SUPPORTIVE DOCUMENTS. C) CUSTOMER HAS DONE TRANSACTIONS WITHIN GROUP ACCOUNT ONLY. C). THE CASE IS BEING REPORTED UNDER TRADE BASED MONEY LAUNDERING (TBML) SCENARIO.1.) IN OUR REGULAR TRANSACTION MONITORING, CUSTOMER U S CLOTHING INDIA PVT LTD MAINTAINING A/C 50200010273600 GOT IDENTIFIED UNDER HIGH VOLUME OF SHIPPING BILLS PENDING AS PER EDPMS AMOUNTING TO RS. 495.41 LAKHS. BASIS WHICH CASE IS CONSIDERED FOR REPORTING. 2).CUSTOMER U S CLOTHING INDIA PVT LTD (DOI-13-DEC-01) HAS DECLARED UNDER KYC DOCUMENT AS A MANUFACTURER OF GARMENT WITH ANNUAL TURNOVER OF RS. 500 LAKHS. CUSTOMER IS MAINTAINING A A/C 50200010273600 WITH KANDLA KANDLA GUJARAT BRANCH SINCE 09 FEBRUARY 2015 CUSTOMER PAN, MOBILE AND REGISTERED ADDRESS ARE AAACU4836H,+ 919825251414 AND PLOT NO 274 SECTOR - 4 KASEZ GANDHIDHAM 370201. RESPECTIVELY. CUSTOMER ACCOUNT STATUS IS REGULAR. CUSTOMER IS CORPORATE CONCERN RISK CATEGORIZATION IS LOW CUSTOMER ACCOUNT KYC HAS BEEN DONE. CURRENT AVAILABLE BALANCE IN THE ACCOUNT IS RS -4906182.00. GURVINDER SINGH TOOR AND HANIF SALE MOHAMMAD TALU ARE THE AUTHORIZED SIGNATORY OF ACCOUNT. 2.DETAILS OF LINKED ACCOUNT_ 1.CUSTOMER IS HAVING ACCOUNT NUMBER 06098730000018 NAME OF US CLOTHING INDIA PVT LTD SINCE 03-NOV-2012 . 2. CUSTOMER IS HAVING ACCOUNT NUMBER 06092320000074 NAME OF US CLOTHING INDIA PVT LTD SINCE 22-SEP-2007 . 3.CUSTOMER IS HAVING ACCOUNT NUMBER 23128730000121 NAME OF US CLOTHING INDIA PVT LTD SINCE 07-MAY-2013 . 4.CUSTOMER IS HAVING ACCOUNT NUMBER 50200016042550 NAME OF US CLOTHING INDIA PVT LTD SINCE 05-DEC-2015 . 3) SCRUTINY OF CUSTOMER U S CLOTHING INDIA PVT LTD ACCOUNT 50200010273600 STATEMENTS FOR THE PERIOD 01-07-2020 TO TILL DATE, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 366.73 LAKHS WHICH LARGELY COMPRISES OF, FUND TRANSFER AGGREGATING TO RS. 26.12 LAKHS MAINLY FROM ACCOUNT NUMBER 50200034627071 - B V H MANUFACTURING INDIA PVT LTD, RTGS/NEFT AGGREGATING TO RS. 122.50 LAKHS MAINLY FROM OWN GROUP ACCOUNT (U S CLOTHING INDIA PVT LTD) -MAINTAINED ACCOUNT WITH AXIS BANK (IFSC CODE--UTIB0000178), INWARD REMITTANCE AGGREGATING TO RS. 203.52 LAKHS MAINLY FROM TOPMOST SHIPPING LLC FROM UNITED STATES, ALOYS NIYONAMBAZA FROM UNITED STATE. OUT OF TOTAL DEPOSITS, WHICH ARE UTILIZED TOWARDS RTGS/NEFT AGGREGATING TO RS. 332.51 LAKHS MAINLY TO OWN GROUP ACCOUNT (U S CLOTHING INDIA PVT LTD) -MAINTAINED ACCOUNT WITH AXIS BANK (IFSC CODE--UTIB0000178),NEXO AG",{"entities":[(816,836,"FOREIGN ORG"),(838,854,"FOREIGN ORG"),(3304,3324,"FOREIGN ORG"),(3345,3362,"FOREIGN ORG"),(3578, 3598,"FOREIGN ORG")]}),
            ("ORGANIZTIONS THAT ARE OPERATES FROM OUTSIDE OF INDIA ARE QUICKODE PTE LTD, DREAMBOX STUDIOS LIMITED, NETCLICKS MEDIA S PTE LTD, ADVANCE MEDIA FILM PRODUCTION LLC,TK FUJIKIN CORPORATION,TRANQUILLO GMBH,STONELAND INC,WONDERSTONE LLC, STONE SOURCE INTERNATIONAL INC",{"entities":[(55, 71,"FOREIGN ORG"),(73, 97,"FOREIGN ORG"),(99, 124,"FOREIGN ORG"),(126,158,"FOREIGN ORG"),(160,182,"FOREIGN ORG"),(185,200,"FOREIGN ORG"),(201,214,"FOREIGN ORG"),(215,230,"FOREIGN ORG") ]}),
            ("MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGS/NEFT TOTALLING TO RS.2.69 CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT  KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER: 2413200206  ON 8 INSTANCES, FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS.65.47 LAKHS, CHEQUES ISSUANCE TOTALLING TO RS.8.29 LAKHS, IMPORT PAYMENT TOTALLING TO RS.6.41 LAKHS TO TANU STAR HK LTD, HONG KONG ETC",{"entities":[(554,570,"FOREIGN ORG")]}),
            (" MAJORITY OF THE FUNDS CREDITED IN THE ACCOUNT of FAISHING-INDUSTRIAL-HK-CO-LTD ,TITOMA MANUFACTURING LTD,FAISHING INDUSTRIAL HK CO LTD,AMAZON WEB SERVICES,SEMNOX SOLUTIONS LLC  LU,CARDY INTELLIGENT TECHNOLOGY CO LTD ,KSIX CARGO ,DHAKA GLOBAL PTE LTD INTERNATIONAL , ETERNAL SEA GROUP LIMITED,SHAANXI BOHU ZHICHUANG ELECTRONIC TECHNOLOGY CO, RAINBOW COMMUNICATION CO",{"entities":[(50,79,"FOREIGN ORG"),(81, 105,"FOREIGN ORG"),(106, 135,"FOREIGN ORG"),(136, 155,"FOREIGN ORG"),(156, 176,"FOREIGN ORG"),(181, 216,"FOREIGN ORG"),(218, 228,"FOREIGN ORG"),(230, 264,"FOREIGN ORG"),(267, 292,"FOREIGN ORG"),(293, 341,"FOREIGN ORG"),(342, 363,"FOREIGN ORG")]}),
            ("FOREIGN REMITTANCE RS. . LAKHS FROM WORLD METALS AND ALLOYS FZC, SAMANCOR MARKETING PTE. LTD, HWASUNG METAL CO. LTD., SEASHORE STEEL, ETC., BILLS PAYMENT RS. . CRORES, IMPS TRANSFER RS. . CRORE FROM SUN STEEL, SHREE BALAJI ALUMNIC, ALTRUISTIC METAL PRIVATE LTD., DIVYA IMPORTERS, ETC., INTEREST RS. . LAKHS, UPI TRANSFER RS. . LAKH FROM KOLAWAT ENTERPRISES, SHRI RAGHAVENDRA STEELS, ANAND RAJESH, ETC., COMMISSION RS. ,/-. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING FOREIGN REMITTANCES RS. . CRORES TO COMPLEJO INDUSTRIAL MOLYNO, GLENCORE INTERNATIONAL AG, SIZER METALS PTE LTD., AMBANT LIMITED, ETC., CMS DEBITS RS. . CRORES, FUND TRANSFER RS. . LAKHS TO OSWAL ALLOYS PRIVATE LTD., MOHANLAL BHARATH KUMAR, SANJAY PITLIYA AND COMPANY, ETC., RTGS TRANSFER RS. . CRORES TO OSWAL MINERALS LIMITED YES BANK LTD, D S ALLOYD PVT LTD STATE BANK OF INDIA, LALWANI FERRO ALLOYS LTD. AXIS BANK, SHYAM FERRO ALLOYS LIMITED HDFC BANK LTD., ETC., NEFT TRANSFER RS. . CRORES TO LALWANI FERRO ALLOYS LIMITED AXIS BANK, MANGANESE PRODUCTS CORPORATION UNION BANK OF INDIA, ISPAT DAMODAR PVT. LTD. STATE BANK OF INDIA, FERRO ALLOYS CORPORATION LTD. STATE BANK OF INDIA, ETC., CASH RS. . CRORES, LOAN REPAYMENT RS. . CRORE, TAX RS. . LAKHS, CHARGES RS. . LAKHS, CHEQUE RS. . LAKHS, SALARY RS. . LAKHS, BILLS RS. . LAKHS, INTEREST RS. . LAKHS, FEE RS. . LAKH, CREDIT CARD RS. ,/-, ONLINE TRANSACTION RS. ,/-, COMMISSION RS. ,/-.PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FOR THE PERIOD MENTIONED BELOW: FY: -  TO : TOTAL CREDITS RS. . CRORES CASH  . LAKHS AND TOTAL DEBITS RS. . CRORES CASH - . CRORES. FY: -  TO : TOTAL CREDITS RS. . CRORES CASH  . LAKHS AND TOTAL DEBITS RS. . CRORES CASH  . CRORES.FY: -  TO : TOTAL CREDITS RS. . CRORES CASH - , AND TOTAL DEBITS RS. . CRORES CASH - . CRORES.BALANCE AS ON DATE OF FILING IS RS. ./-.WE HAVE ALREADY FILED STR ON ACCOUNT NO.:  STR NO.:, DATE  , BATCH ID  , REASON - HUGE AMOUNT OF FUNDS HAVE BEEN ROUTED THROUGH THE ACCOUNT BY INWARD AND OUTWARD RTGS/NEFT AND INTERNAL FUND TRANSFERS WITHOUT ANY ECONOMIC RATIONALEWHILE CONDUCTING DISCREET CHECK, WE HAVE OBSERVED FEW DISCREPANCIES. CUSTOMER HAS RECEIVED HIGH VALUE REFUND AGAINST DIRECT IMPORT BILL PAYMENT AS QUALITY CLAIM AND CONSOLIDATED VALUE OF REFUND AMOUNTED TO USD . MILLION, HOWEVER THE SAME WAS NOT HANDLED BY CLIENT AS PER RBI DIRECTIONS TOWARDS WRITE OFF OF IMPORT DUES. FURTHER, HIGH VALUE TRANSACTIONS HANDLED UNDER DIRECT IMPORT BILL PAYMENT AS OPEN ACCOUNT PAYMENT AND PAYMENT TERMS APPEARING ON INVOICE SEEMS MODIFIED WHILE PAYMENT. THERE IS FREQUENT TRADING BETWEEN THIRD COUNTRY AND PAYMENT MADE MAINLY WITH UAE ENTITY I.E M/S WORLD METALS AND ALLOYS FZC, WHEREIN MANUFACTURER OF THE GOODS OR ORIGIN DETAILS ARE NOT SPECIFIED PROPERLY. IT APPARENTLY SEEMS THAT CUSTOMER HAS INTENTIONALLY BREACHED THE FEMA GUIDELINES.HIGH VALUE RTGS/NEFT/FUND TRANSFER/BILLS RECEIPT CREDITS FOLLOWED BY HIGH VALUE TRANSFER TO SPECIFIC THIRD PARTIES, ANOMALIES OBSERVED RAISING SUSPICION ON UNDERLYING INTENTION. HENCE, STR FILED.OSWAL MINERALS LIMITED ACCOUNT NO.: , ,  AND  ADDRESS:  ND FLOOR DR RAJKUMAR ROAD,TH BLOCK RAJAJINAGAR, OPPOSITE TO PLANET HONDA SERVICE CENTRE BANGALORE - , DATE OF INCORPORATION:- , PAN: AACCMG, AADHAAR NUMBER REFERENCE KEY: - IN VIEW OF THE RESTRICTION OF SHARING THE AADHAAR NUMBER AS PER REGULATION  OF AADHAAR SHARING OF INFORMATION REGULATIONS , WE ARE PROVIDING REFERENCE KEY ASSIGNED TO THE AADHAAR NUMBER, WHEREVER APPLICABLE, MOBILE NUMBER: , RISK CATEGORY: HIGH RISK, AUTHORISED SIGNATORY - MOHANLAL BHARATHKUMAR JAIN, SUBHASH CHAND MOHANLAL, KYC COMPLIANCE STATUS: YES, DATE OF LAST KYC UPDATION:  AND NA OPENED AN EXCHANGE EARNERS FOREIGN CURRENCY A/C - EEFC ACCOUNT AND CASH CREDIT ACCOUNT RESPECTIVELY ON , ,  AND  AT RBL BANK, BANGALORE BRANCH, , TH CROSS, GANDHINAGAR, BANGLORE   AND FORT MUMBAI BRANCH, GROUND FLOOR, , A DHANNUR, , SIR FIROZ SHAH MEHTA ROAD, MUMBAI, FORT - . CONSTITUTION TYPE: PUBLIC LIMITED COMPANY, ACCOUNT STATUS: INACTIVE, ACTIVE, DORMANT AND ACTIVE RESPECTIVELYTHE CUSTOMER HAS DECLARED PROFILE AS MANUFACTURER OF METALS AND MINERALS HAVING EXPECTED ANNUAL TURNOVER IS GREATER THAN  CRORE. SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FOR THE PERIOD FROM  UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS. . LAKHS CONSISTING OF FOREIGN REMITTANCES RS. . LAKHS FROM WORLD METALS ALLOYS, KUBOTA SAUDI COMPANY FACTORY, AL GARHOUD GENERAL TRADING, PINK LOTUS IRON AND STEELS, ETC. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS. . LAKHS, FOREIGN REMITTANCES RS. . LAKHS TO ALLIANZ BULK CFR PTE LTD., GLENCORE INTERNATIONAL AG, PMB SILICON SDN BHD, SIZER METALS PTE LTD.PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FOR THE PERIOD MENTIONED BELOW: FY: -  TO : TOTAL CREDITS RS. . LAKHS CASH - NIL AND TOTAL DEBITS RS. . LAKHS CASH - NIL.FY: -  TO : TOTAL CREDITS RS. . LAKHS CASH - NIL AND TOTAL DEBITS RS. . LAKHS CASH - NIL.FY: -  TO : TOTAL CREDITS RS. . LAKHS CASH - NIL AND TOTAL DEBITS RS. . LAKHS CASH - NIL.BALANCE AS ON DATE OF FILING IS NIL.THERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FROM THE PERIOD OF  UPTO DATE OF FILING STR.SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FOR THE PERIOD FROM  UPTO  REVEALED THAT THE TOTAL CREDITS OF RS. . LAKHS CONSISTING OF FOREIGN REMITTANCES RS. . LAKHS FROM STAIN LESS METALS GENERAL. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS. . LAKHS.BALANCE AS ON DATE OF FILING IS NIL.THERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FROM THE PERIOD OF  UPTO DATE OF FILING STR.BALANCE AS ON DATE OF FILING IS NIL.SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO.:  FOR THE PERIOD FROM  UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS. . CRORES CONSISTING OF RTGS TRANSFER RS. . CRORES FROM OSWAL MINERALS YES BANK LTD., SONIC THERMAL PVT LTD. ALLAHABAD BANK, AJAY ENTERPRISES YES BANK LTD., MUKAND LTD YES BANK LTD., ETC., FUND TRANSFER RS. . CRORES FROM MOHANLAL, SEEMA JAIN, SUBHASHCHAN, SRIPAL KUMAR, SARITHA DEV, ETC., NEFT TRANSFER RS. . CRORES FROM OSWAL MINERALS LIMITED HDFC BANK LTD, ARCELORMITTAL NIPPON STEEL INDIA ICICI BANK LTD, SHYAM FERRO ALLOYS LIMITED HDFC BANK LTD, MACTECH ENGINEERING CORPORATION BANK, ETC., CMS RS. . CRORES, CHEQUE RS. . CRORES RETURNED AMOUNTED RS. . LAKHS",{"entities":[(37,63,"FOREIGN ORG"),(65,92,"FOREIGN ORG"),(94,116,"FOREIGN ORG"),(118,132,"FOREIGN ORG") ,(558, 578,"FOREIGN ORG") ,(503, 556,"FOREIGN ORG"),(4410, 4429,"FOREIGN ORG"),(4431, 4459,"FOREIGN ORG"),(4461, 4487,"FOREIGN ORG"),(4620, 4644,"FOREIGN ORG"),(4674, 4693,"FOREIGN ORG"),(4695, 4722,"FOREIGN ORG")]}),
            ("ON REVIEWING THE TRANSACTION PATTERN OF YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED AC.  AND EVERGREEN INTERNATIONAL AC . IT WAS OBSERVED THAT CLIENT, YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED, STATED ITS ANNUAL TURNOVER IS BETWEEN RS.CR-CR, HOWEVER IN LESS THAN  MONTHS BETWEEN ST JAN  TO MARCH , THE CLIENT HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF INR . CRORES APPROX..AS PER ACCOUNT STATEMENT REVIEW FOR THE PERIOD FROM ST APRIL  TO TH MAR , AFOREMENTIONED ENTITIES RECEIVED ACCOUNT CREDITS TO THE TUNE OF INR .CR WITH MAJOR CREDITS VIA RTGS FROM EVERGREEN INTERNATIONAL VALUING INR .CR IDFC BANK A/C NO , IMPS FROM SAWARIYA MULTITREDE VALUING INR LACS AND FUND TRANSFER FROM EVERGREEN INTERNATIONAL YES BANK A/C NO  VALUING INR LACS.THESE WERE DEBITED MAJORLY VIA TRADE REMITTANCE VALUING INR .LACS TO NAVISTAR LOGISTICS LIMITED IN HONG KONG, FUND TRANSFER TO EVERGREEN INTERNATIONAL YES BANK A/C NO  INR  LACS , ASIA WORLD INTERNATIONAL YES BANK A/C NO  INR .LACS AND RTGS/NEFT TO EVERGREEN INTERNATIONAL VALUING INR LACS IDFC BANK A/C NO   , KALYAN MULTITRADE VALUING INR LACS EQUITAS SMALL FINANCE BANK A/C NO   AND SEA LINERS SHIPPING PVT LTD VALUING INR .LACS  ICICI BANK A/C AS PER PUBLIC DOMAIN, NAVISTAR LOGISTICS LIMITED HONG KONG IS INCORPORATED ON TH SEPT . HTTPS:WWW.HKGBUSINESS.COM/EN/COMPANY/NAVISTAR-LOGISTICS-LIMITED HOWEVER THERE IS NO COMPANY WEBSITE OR OTHER INFORMATION AVAILABLE.THERE ARE NO CREDIBLE INFORMATION AVAILABLE ON THE LOCAL AS WELL AS OVERSEAS ENTITIES IN PUBLIC DOMAINAS PER TRANSACTION REF MISC GOODS HOT ROLLED STAINLESS STEEL SHEET ARE SHIPPED FROM CHINA TO INDIA MUMBAI HOWEVER BILL OF LADING IS ISSUED FROM SINGAPORE AND THE ISSUING ENTITY IS BASED IN HONG KONG AND SHIPMENT IS FROM CHINA. IN ANOTHER BILL OF LADING GOODS HOT ROLLED STAINLESS STEEL SHEET ARE SHIPPED FROM CHINA TO INDIA MUMBAI HOWEVER BILL OF LADING IS ISSUED FROM HONG KONG AND THE ISSUING ENTITY IS BASED IN HONG KONGTHE TRANSPORT DOCUMENTS AWB PROVIDED IN THE REMITTANCES MISC ARE ALSO NOT COMPLETE AND DO NOT PROVIDE COMPLETE INFORMATION ON THE ACTUAL IMPORTER NAME, DATE OF BOL ISSUED AND APPEAR SUSPICIOUS. THERE ARE NO OTHER BUSINESS TRANSACTIONS IN THE ACCOUNTS AND ARE SUSPECTED TO BE USED WITH THE SOLE INTENSION OF REMITTING THE FUNDS ABROAD.THE TRANSACTION PATTERN POINTS TOWARDS LAYERING OF FUNDS AMONGST VARIOUS BANK ACCOUNTS TO CONCEAL THE ACTUAL SOURCE OF FUNDS. THERE ARE NO OTHER BUSINESS-RELATED TRANSACTIONS IN THE ACCOUNTS. IT IS HENCE, SUSPECTED THAT CUSTOMER MAY BE INVOLVED IN TRADE BASED MONEY LAUNDERING ACTIVITY UNDER THE GUISE OF FREIGHT PAYMENTS.IN LIGHT OF THE ABOVE FACTS, IT HAS BEEN DECIDED ON -APRIL-  TO FILE A STR ON THE SAID CUSTOMER. THE CUSTOMER YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED PAN- AABCYF, DOI , COMMUNICATION PERMANENT ADDRESS- TH FLOOR OFFICE NO  A WING PRANIK CHAMBERS SAKI VIHAR ROAD MUMBAI  , REGISTRATION NUMBER- , CIN- UMHPTC, IEC CODE - AABCYF, MOBILE NUMBER-  IS MAINTAINING A CURRENT AC NO.  SINCE  WITH SAKINAKA BRANCH ADDRESS- SHOP NO. ,SAGAR TECH PLAZA,ANDHERI KURLA ROAD,SAKINAKA JUNCTION, MUMBAI . THE CUSTOMER FALLS UNDER MEDIUM RISK CATEGORY WITH KYC LAST UPDATED IN JAN . PANKRAJ RAJENDRA PRASAD KESHARI AJIT YADAV ARE THE AUTHORIZED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT. CUSTOMER IS INTO LOGISTICS AND FRIEGHT WITH AN ANNUAL GROSS TURNOVER BETWEEN RS. CR-CR. THERE IS  MORE ACCOUNT LINKED TO THE ACCOUNT OF YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED AC NO.  BY COUNTER PARTY FUND TRANSFERAACCOUNT NO  NAME :: EVERGREEN INTERNATIONAL STATUS : ACTIVEAOD:: -DEC- PAN NO :: ATUPGB  IN THE ACCOUNT OF YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED AC NO.  AND  LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM  TILL , SHOWS  CREDIT TRANSACTIONS AMOUNTING TO RS. ,, AND  DEBIT TRANSACTIONS AMOUNTING TO RS. ,,OUT OF THE TOTAL CREDITS OF RS. ,, IN THE ACCOUNTS, FUND TRANSFER AMOUNTS TO RS. ,, CONSTITUTING AROUND  PERCENT OF THE TOTAL CREDITS, RTGS/NEFT/IMPS AMOUNTS TO RS. ,, CONSTITUTING AROUND  PERCENT OF THE TOTAL CREDITSOUT OF THE TOTAL DEBITS OF RS. ,, IN THE ACCOUNTS, FUND TRANSFER AMOUNTS TO RS. ,, CONSTITUTING AROUND  PERCENT OF THE TOTAL DEBITS, RTGS/NEFT/IMPS AMOUNTS TO RS. ,, CONSTITUTING AROUND  PERCENT OF THE TOTAL DEBITS, TRADE TRANSACTION AMOUNTS TO RS. ,, CONSTITUTING AROUND  PERCENT OF THE TOTAL DEBITS NO TRANSACTIONS WERE OBSERVED FOR THE PERIOD  TILL  AS ACCOUNT WAS OPENED ON BALANCE IN THE ACCOUNT AS ON  IS RS. .",{"entities":[(825, 851,"FOREIGN ORG"),(1226, 1252,"FOREIGN ORG")  ]}),
            (" SUBSEQUENT TO SUCH CREDITS FOREIGN OUTWARD REMITTANCES NON-IMPORT AMOUNTED TO RS.. CR FAVOURING SPEED STREAM FREIGHT BROK - UAE RS.. CR, STATICE INTERNATIONAL PTE LTD  - SINGAPORE RS.. CR AND EXCEL SEA SHIPPING LLC - UAE RS.. CR TOWARDS FREIGHT ON IMPORTS.  BANK HAS ALREADY STR ON OTHER ACCOUNTS NAMELY BARA AFRICA ON SHORE SERVICES PRIVATE LIMITED , BATCH ID , BOXTRON SHIPPING AND LOGISTICS PVT LTD , BATCH ID  AND MILE LOGISTICS P LIMITED , BATCH ID  WHEREIN SIMILAR PATTERN OF TRANSACTIONS WERE NOTED AND FUNDS WERE REMITTED TO THE ABOVE MENTIONED COMMON OVERSEAS PARTY EXCEL SEA SHIPPING LLC  UAE.  IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.. CR MAINLY FAVOURING NEXASHIP LOGISTICS PRIVATE LIMITED RS.. CR, IDFC - , LIBERTON LOGISTICS PRIVATE LIMITED RS.. CR, IDFC -  AND T S GLOBAL IMPEX RS.. CR, IDFC - .  LIBERTON LOGISTICS PRIVATE LIMITED AND NEXASHIP LOGISTICS PRIVATE LIMITED HAVE UPDATED A COMMON MAILING ADDRESS. DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS SIGN BOARDS IN THE NAMES OF THE COMPANIES WERE SEEN AT THE PREMISE AND OFFICE SET UP WITH CHAIRS AND TABLES COULD BE SEEN. HOWEVER NO MAJOR BUSINESS ACTIVITY OR EMPLOYEES COULD BE SEEN TO SUBSTANTIATE THE HIGH VALUE TRANSACTIONS ROUTED THROUGH ACCOUNT. IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED TO ROUTE TRANSACTIONS WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY.  A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE ROUND SUM RTGS/NEFT CREDITS WERE FOLLOWED BY IMMEDIATE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS. NO OTHER BUSINESS RELATED TRANSACTIONS COULD BE NOTED THROUGH THE ACCOUNT. WITHIN  MONTHS OF ACCOUNT OPENING HIGH VALUE AND VOLUME OF TRANSACTIONS WERE ROUTED THROUGH ALL  ACCOUNTS.  BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING. ALSO THE PROPRIETORS AND DIRECTORS ARE IN THE AGE GROUP OF - YEARS AND MAY NOT BE THE ACTUAL BENEFICIAL OWNERS OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNTS. ACCOUNT OF LIBERTON LOGISTICS P LIMITED IS ACTIVE WITH BALANCE OF RS.,,/-  AS ON APRIL , . QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR - IS DEBIT: RS.. IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. - IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR.A CURRENT ACCOUNT BEARING NUMBER  WAS OPENED IN THE NAME OF LIBERTON LOGISTICS P LTD ON -FEB- AT OUR BRANCH LOCATED IN SHARDA TERRACES, PLOT NO , SEC-, CBD BELAPUR, NAVI MUMBAI - . THE ENTITY WAS FORMED ON -JAN- AND HAS SUBMITTED COPIES OF PAN AAGCAR, ALONG WITH BOARD RESOLUTION, MOA/ AOA AND CERT OF INCORP UMHPTC AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO. B  SHELTON SAPPHIRE PL   SECTOR  CBD BELAPUR NAVI MUMBAI . DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS AND ULTIMATE BENEFICIAL OWNERS. DIRECTORS HAVE REGISTERED THEIR MOBILE NUMBERS  AND  WITH THE BANK. THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS.. CR AND THAT THE INDIVIDUALS HAVE ANNUAL INCOME IN THE RANGE OF RS.. CR TO RS.. CR. BASIS INTER-CONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED IN THE NAMES OF NEXASHIP LOGISTICS PRIVATE LIMITED  AND A SOLE PROPRIETORSHIP CONCERN T S GLOBAL IMPEX  OPENED ON -FEB- AND -FEB-. DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS IN NEXASHIP LOGISTICS PRIVATE LIMITED WHILE TSHERINGLA SHERPA IS THE PROPRIETOR IN T S GLOBAL IMPEX. IT WAS INDICATED THAT THE ENTITIES ARE INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS.. CR AND . CR RESPECTIVELY. WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NON-IMPORT WERE ROUTED THROUGH THE NEWLY OPENED ACCOUNT OF LIBERTON LOGISTICS PRIVATE LIMITED. SIMILAR PATTERN OF TRANSACTIONS AND COMMON OVERSEAS BENEFICIARIES WERE NOTED IN THE ACCOUNTS OF NEXASHIP LOGISTICS PRIVATE LIMITED AND T S GLOBAL IMPEX WHICH AS PER CUSTOMER ARE RELATED ACCOUNTS. HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTER-CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION:  ON REVIEWING CONSOLIDATED ACCOUNT TRANSACTIONS SINCE INCEPTION TILL APRIL ,  IT WAS NOTED THAT ROUND SUM HIGH VALUE IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.. CR MAINLY FROM NAMO SAI INTERNATIONAL RS.. CR, EQUITAS SMALL FINANCE BANK LIMITED  A/C NO   AND NEXASHIP LOGISTICS PRIVATE LIMITED RS.. CR, IDBI  , IDFC  , SBI  , YES BANK  .",{"entities":[(193, 221,"FOREIGN ORG"),(138, 167,"FOREIGN ORG"),(576, 603,"FOREIGN ORG")     ]}),
            (" SRI SNJ JEWELS, A PROPRIETORSHIP FIRM, HAVING ADDRESS : , OLD NO  SHOP NO /, RD FLR, NSC BOSE ROAD, SOWCARPET, TARUS TOWER, CHENNAI, TAMIL NADU, INDIA, , CONTACT NUMBER: + AND EMAIL ID: SRISNJJEWELS@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, , NSC BOSE ROAD, SOWCARPET, CHENNAI, TAMIL NADU, , SINCE . DATE OF BIRTH IS . PAN OF THE ACCOUNT IS BADPJR. PAN LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS NIKET SOHANLAL JAIN PROPRIETOR - BADPJR. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN -BADPJR AND LETTER OF DECLARATION OF PROPRIETORSHIP FIRM. KYC UPDATION IS NOT DUE. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN  LACS. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT. BALANCE IN THE ACCOUNT AT  WAS RS. . LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY FROM KANTILAL, JAI GANESH JEWELLERY, NEFT RS. . LAKHS MAJORLY FROM DAIMOND, SANGITA MADAN JAIN, RTGS RS. ,. LAKHS MAJORLY FROM LEO JEWELS, DIAMOND, REMITTANCE RS. ,. LAKHS MAJORLY FROM CHAIPHET ANYAMANEE CO LTD, SUAY THAI THONG CO LTD, INTERNET FUND TRANSFERS RS. ,. LAKHS FROM MOKSHA JEWELLERY. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RTGS RS. ,. LAKHS MAJORLY TO DIAMOND INDIA LIMITED, REMITTANCE RS. . LAKHS MAJORLY TO CHAIPHET ANYAMANEE CO LTD, INTERNET FUND TRANSFER RS. ,. LAKHS TO MOKSHA JEWELLERY ETC, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH RS.  BY CASH, REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RS.  BY CASH, REMITTANCE RS. ,. LAKHS. CUSTOMER IS INTO GOLD ORNAMENTS MANUFACTURING. FOR MANUFACTURING PURPOSE, CUSTOMER IS USING JOB WORK VENDORS AND GOLD SMITHS. EXCEPT OFFICE SET UP, COULD NOT ASCERTAIN ANY BUSINESS ACTIVITIES. HIGH VALUE NON CASHC CREDITS AND DEBITS WERE SEEN IN THE ACCOUNT. THE ACTIVITY IS NOT LOOKING USUAL AS THE TURNOVER IN JUST  YEAR IS MUCH HIGHER IN COMPARE TO THE DECLARED TURNOVER WHICH LESS THAN RS.  LACS. FOR EXAMPLE, ON  RS . LACS WERE CREDITED THROUGH RTGS FROM J B GOLD , KKBK AND ON THE SAME DAY RS . CRORE WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PVT LTD , INDB. ALSO ON  RS . LACS WERE CREDITED THROUGH RTGS FROM A V JEWELLERS , YESB, LEO JEWELS , HDFC ETC AND ON THE SAME DAY RS . LACS WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PRIVATE LIMITED. SIMILAR PATTERN WERE SEEN IN THE ACCOUNT. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT WHICH SEEMS LIKE ROUTING OF FUNDS WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILED.",{"entities":[(1373,1395,"FOREIGN ORG"),(1346, 1371,"FOREIGN ORG") ]}),
            ("GROUNDS OF SUSPICION: THIS IS AN LEA CASE AND A SUPPLEMENTARY CASE. BANK WAS IN RECEIPT OF A SUMMON DATED // VIDE REFERENCE NUMBER ITBA/INV/S//-/   U/S  A  FROM NARENDRA MOLAI PRASAD, DY.DIRECTOR OF INCOME TAX  INV.  UNIT:   , INCOME TAX DEPARTMENT, OFFICE OF THE ASSISTANT DIRECTOR OF INCOME TAX, INVESTIGATION. THE NOTICE DIRECTED THE BANK TO PROVIDE ACCOUNT RELATED INFORMATION OF R N EXPORTS HOLDING ACCOUNT NUMBERS ,  AND  FOR INVESTIGATION PURPOSE. THE INFORMATION SOUGHT HAS BEEN PROVIDED TO THE INCOME TAX DEPARTMENT. REVIEW OF THE TRANSACTIONS FOR ACCOUNT  FOR THE PERIOD FROM // TO // REVEALED TOTAL CREDITS OF RS.. CRORES AND TOTAL DEBITS OF RS.. CRORES. MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF INTERNAL FUND TRANSFERS TOTALLING TO RS.. CRORES ON  INSTANCES WHICH WERE RECEIVED FROM OWN USD DIAMOND DOLLAR ACCOUNT  ACCOUNT NUMBER:  , RTGS/NEFT CREDITS TOTALLING TO RS.. LAKHS ON  INSTANCES OF WHICH RS.. LAKHS FROM OWN ACCOUNT  KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER:   ON  INSTANCES, RS.. LAKHS FROM CARAT LANE TRADING PRIVATE LIMITED  ICICI BANK LTD, ACCOUNT NUMBER: SLNEFTPL  ON  INSTANCES, RS.. LAKHS FROM KANTILAL CHHOTALAL  BANK OF INDIA, ACCOUNT NUMBER:   ON  INSTANCES, RS.. LAKHS FROM NAICE JEWELS PRIVATE LIMITED  KOTAK MAHINDRA BANK LIMITED, ACCOUNT NUMBER:   ON  INSTANCES, RS.. LAKHS FROM CREATIVE GEMS AND JEWELLERY LTD  STATE BANK OF INDIA, ACCOUNT NUMBER:   ON  INSTANCES, RTGS RETURNS TOTALLING TO RS.. LAKHS. MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGS/NEFT TOTALLING TO RS.. CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT  KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER:   ON  INSTANCES, FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS.. LAKHS, CHEQUES ISSUANCE TOTALLING TO RS.. LAKHS, IMPORT PAYMENT TOTALLING TO RS.. LAKHS TO TANU STAR HK LTD, HONG KONG ETC. A REVIEW OF THE TRANSACTIONS IN USD DIAMOND DOLLAR ACCOUNT  FOR ABOVE REVIEW PERIOD REVEALED BUSINESS RELATED TRANSACTIONS. SEARCHES CONDUCTED IN PUBLIC DOMAIN DID NOT REVEAL ANY ADVERSE MEDIA ON THE CUSTOMER. CASE IS BEING REPORTED TO FIU-IND DUE TO THE NOTICE RECEIVED FROM THE INCOME TAX DEPARTMENT.CUSTOMER R N EXPORTS  DATE OF INCORPORATION: //; PAN: AAAFRB  WITH IEC CODE:  ISSUED BY DGFT, GSTIN: AAAFRBZ ISSUED BY CBEC HAS ADDRESS AS , PETIT TOWER, RD FLOOR, CUMBALLA HILL, A K MARG, MUMBAI- MOBILE NUMBER:  AND LANDLINE NUMBER: -. THE CUSTOMER HAS BEEN CATEGORISED AS MEDIUM RISK AND THE LAST KYC WAS UPDATED ON // WITH THE STATUS AS COMPLETED. THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER  WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT SHREE PANT BHUVAN, SANDHURUST BRIDGE, MUMBAI- SINCE // WITH THE STATUS AS ACTIVE. THE CUSTOMER ALSO HOLDS AN USD EEFC ACCOUNT NUMBER  SINCE // WHICH IS IN DORMANT STATUS SINCE // AND USD DIAMOND DOLLAR ACCOUNT NUMBER  SINCE // WHICH IS IN ACTIVE STATUS. AS PER THE KYC RECORDS HELD WITH THE BANK, CUSTOMER R N EXPORTS IS A PARTNERSHIP FIRM ENGAGED IN THE BUSINESS OF MANUFACTURING OF JEWELLERY AND WORKING OF DIAMONDS. RAHUL ROHIT JHAVERI AND SONALI RAHUL JHAVERI ARE THE BENEFICIAL OWNERS AND AUTHORISED SIGNATORIES OF THE FIRM. THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS.. CRORES. REVIEW OF THE TRANSACTIONS FOR ACCOUNT  FOR THE CURRENT FINANCIAL YEAR FROM // TO // REVEALED TOTAL CREDITS OF   IN CASH   AND TOTAL DEBITS OF   IN CASH  . REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM // TO // REVEALED TOTAL CREDITS OF RS.. CRORES  IN CASH   AND TOTAL DEBITS OF RS.. CRORES  IN CASH RS., ; REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM // TO // REVEALED TOTAL CREDITS OF RS.. CRORES  IN CASH   AND TOTAL DEBITS OF RS.. CRORES  IN CASH RS. LAKHS . BALANCE IN THE ACCOUNT AS ON // WAS . A REVIEW OF THE INTERNAL RECORDS REVEALED THAT AN STR WAS REPORTED PREVIOUSLY VIDE BATCH ID: , S.NO.:  WITH THE REPORTED ACCOUNTS ,  AND  IN THE NAME OF R N EXPORTS DUE TO CIRCULATION OF FUNDS BETWEEN THE CLIENT AND A HONG KONG BASED ENTITY ON ACCOUNT OF PURCHASE AND SALE OF ROUGH AND POLISHED DIAMONDS.",{"entities":[(1773,1789,"FOREIGN ORG"),(1210,1238,"INIDAN ORG")]}),
            ("HIGH VALUE ADV. IMP. REM. AMOUNTED TO RS.. CR FROM THE A/C OF PREMIUM PETRO PRODUCTS MAINLY FAVOURING UAE BASED ENTITIES TASHBULAK TRADING LLC, MOTOTIVE GENERAL TRADING LLC RS.. CR, PETRO STAR FZE RS.. CR, KIARA PETROLEUM FZE RS.. CR, OULA ALNUHA INTERNATIONAL RS.. CR, TIME TECHNO PACKAGING LLC RS.. CR, TASHBULAK TRADING LLC RS.. CR AND MAN TRADING LLC RS.. CR TOWARDS IMPORT OF BITUMEN.  PREMIUM PETRO PRODUCTS IS YET TO SUBMIT BOES PERTAINING TO  CASES TOTALLING TO USD ,, WHICH COMPRISES OF  CASES TOTALLING TO USD , PENDING FOR A PERIOD ABOVE  MONTHS UPTO  MONTHS AND  CASES TOTALLING TO USD ,, PENDING FOR A PERIOD LESS THAN  MONTHS.  IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.. CR MAINLY FAVOURING PREMIUM PETRO PRODUCTS RS.. CR, PP SOFTTECH P LTD RS.. CR, ICICI - , VISHAL BOHRA RS.. CR, HDFC  / , IDBI  , YES BANK - , VINAYAK EXIMS RS.. CR, AXIS BANK -  AND AUM KRIPA ENTPS RS.. CR, HDFC - .  INTER-CONNECTED FUNDS TRANSFER TXNS WITHIN GROUP A/CS AMOUNTED TO RS.. CR. U-TURN TRANSACTIONS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS.. CR WERE RECEIVED AND RS.. CR WERE SENT OUT TO A/CS OF PREMIUM PETRO PRODUCTS, VISHAL BOHRA, ABHA PRECISION FARMING LLP, BHEEM RAJ BOHRA, MANISH BOHRA IDBI   AND VCI EXPORTS P LTD MAINTAINED WITH OTHER BANKS.  PREMIUM PETRO PRODUCTS, B L INVESTMENT, RISHI ENTPS, HERO PETRO PRODUCTS, REEVA BUILDCON P LTD, REALTECH BUILDHOME P LTD, ABHA PRECISION FARMING P LTD, AONE BUILDHOME LLP AND BODHI FARM DEVELOPERS P LTD HAVE UPDATED A COMMON MAILING ADDRESS - HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN - . DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT IT WAS A  SQ. FT. OWNED PREMISE WITH SIGN BOARD, EMPLOYEES AND OFFICE SET UP SEEN. HOWEVER NO STOCK OF GOODS WERE SIGHTED. ON ENQUIRY IT WAS LEARNT FROM CUSTOMER THAT GOODS WERE STOCKED AT MOKHAMPURA, TEHSIL-MOZAMABAD, DISTRICT JAIPUR.  PAN OF INTER-CONNECTED A/CS - SHREEBALAJI INFRAPROJECTS P LTD AAPCSK AND DENIS LAND DEVELOPERS P LTD AACCDC ARE REFLECTING IN THE LIST OF POSSIBLE SHELL ENTITIES CIRCULATED BY FIU INDIA.  TOTAL TAX PAYMENTS OF RS.. CR WAS NOTED THROUGH THE A/CS OF UJJWAL BHANDARI HUF  AND VCI EXPORTS P LTD  DURING REVIEW PERIOD, WHICH APPEARS TO BE DISPROPORTIONATE TO HIGH VALUE AND VOLUME OF TRANSACTIONS ROUTED THROUGH A/CS. BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH MULTIPLE A/CS OPENED BY THE SAME GROUP OF INDIVIDUALS FOR LAYERING OF FUNDS AND POSSIBLY TO REMIT FUNDS OVERSEAS WITHOUT ACTUAL IMPORTS TAKING PLACE POSSIBLY WITH THE INTENTION OF TBML AND HAWALA SETTLEMENT. IT ALSO APPEARS THAT SUCH HIGH VALUE AND VOLUME OF TRANSACTIONS MAY HAVE BEEN CARRIED OUT WITHIN GROUP A/CS WITHOUT ANY ECONOMIC RATIONALE POSSIBLY WITH INTENTION OF TAX EVASION. A/C OF PREMIUM PETRO PRODUCTS BEARING NUMBER  IS ACTIVE WITH BALANCE OF RS./- AND  HAS OUTSTANDING BALANCE OF RS.,,,/- ON CREDIT LIMIT OF RS.,,,/- AS ON APRIL , .QUANTUM OF TRANSACTIONS FOR CURRENT FY - IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FY : - IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. - IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. -: DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR.CURRENT A/CS WERE OPENED IN THE NAME OF A PARTNERSHIP FIRM M/S PREMIUM PETRO PRODUCTS NOS.  AND   ON -SEP- AT OUR BRANCH LOCATED IN  KRISHNA TOWER, BASEMENT, ST FLOOR, SARDAR PATEL MARGH JAIPUR RAJASTHAN - . THE ENTITY WAS FORMED ON -MAY- AND HAS SUBMITTED COPIES OF PAN AAUFPN, PARTNERSHIP DEED, CERT. OF REG. NO.//, IEC AAUFPN AND GST REG. CERT. AAUFPNZF AS KYC DOCS AT THE TIME OF A/C OPENING. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS - HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN - . RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA AND LALIT KUMAR NAREDI ARE THE PARTNERS WHILE MANISH KUMAR BOHRA AND UJJWAL BHANDARI ARE THE UBOS IN THE SAID FIRM. CUSTOMER HAS REGISTERED MOBILE NO.  WITH THE BANK. THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF PETROLEUM PRODUCTS WITH ANNUAL TURNOVER OF RS.. CR. BASIS INTER-CONNECTEDNESS, OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF  CURRENT A/CS IN THE NAMES OF  REGISTERED COS.,  NBFC,  PARTNERSHIP FIRMS,  LLP FIRMS AND  INDIVIDUAL CURRENT A/C. BESIDES  SAVINGS A/CS WERE OPENED IN THE NAMES OF INDIVIDUALS AND  HUF. DETAILS OF A/CS ARE UPDATED IN XML FILE. THE MENTIONED A/CS WERE OPENED DURING THE PERIOD -SEP- TILL -NOV- AT ABOVE MENTIONED AJMER ROAD, RAJA PARK, NORTH AVENUE, VAISHALI NAGAR BRANCHES IN JAIPUR - RAJASTHAN AND SECTOR  VASHI BRANCH IN NAVI MUMBAI. IT WAS INDICATED THAT THE ENTITIES ARE INTO VARIED BUSINESSES LIKE REAL ESTATE SERVICES, WHOLESALER OF CHARCOAL/ SODA / ASH, INFRASTRUCTURE, GENERAL TRADING, TEXTILE PRODUCTS, CONSTRUCTION, FINANCE AND INVESTMENT, DEALING IN STONES/MARBLE/GRANITE, TRADE/SALE OF HANDICRAFTS WITH ANNUAL TURNOVER IN THE RANGE OF RS.. CR TO RS.. CR. RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL  BHANDARI, VISHAL  BOHRA, LALIT KUMAR NAREDI AND BHEEM RAJ BOHRA ARE THE COMMON DIRECTORS/ PARTNERS/ KARTA IN THE ABOVE MENTIONED A/CS WITH OTHER INDIVIDUALS IN DIFFERENT COMBINATIONS. NOTICING HIGH VALUE ROUND SUM NEFT/RTGS CREDITS FROM OWN A/C AND SUBSEQUENT HIGH VALUE ADVANCE IMP. REM. THROUGH THE A/C OF PREMIUM PETRO PRODUCTS , AN ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD AND THIS CONSOLIDATED REPORT IS FILED BASIS INTER-CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION:  ON REVIEWING A/C TRANSACTIONS SINCE APRIL ,  - APRIL ,  IT WAS NOTED THAT IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.. CR MAINLY FROM PREMIUM PETRO PRODUCTS RS.. CR, IDBI - , VCI EXPORTS P LTD RS.. CR, IDFC , SURYA NAGARI TRADING RS.. CR, YES BANK  , ABHA PRECISION FARMING LLP RS.. CR, IDBI   AND BHEEM RAJ BOHRA RS.. CR, IDBI  .  EXPORT ADVANCE REMITTANCES AMOUNTED TO RS.. CR IN THE A/CS OF INDIA STONE HOUSE LLP AND VCI EXPORTS P LTD MAINLY FROM WONDERSTONE LLC - USA RS.. CR, TRANQUILLO GMBH RS.. CR, GRAMACO SOURCE AND MARBLE RS.. CR, STONE SOURCE INTERNATIONAL INC  USA RS.. CR AND STONELAND INC - USA RS.. CR. AS PER WEB CHECK WONDERSTONE LLC  USA IS INTO BUSINESS OF JEWELLERY STORES HTTPS://WWW.DNB.COM/BUSINESS-DIRECTORY/COMPANY-PROFILES.WONDERSTONE LLC.DFEAAEEAADF.HTML, TRANQUILLO GMBH IS INTO CLOTHING STORE INDUSTRY, STONE SOURCE INTERNATIONAL INC IS INTO BUSINESS OF STONES/ GRANITES AND STONELAND INC IS INTO GRANITE/ MARBLE BUSINESS. IT APPEARS THAT INDIA STONE HOUSE LLP WHICH HAS INDICATED THAT IT IS INTO BUSINESS OF STONES/ GRANITES MAY HAVE RECEIVED EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS LIKE WONDERSTONE LLC  USA JEWELLERY STONES AND TRANQUILLO GMBH CLOTHING STORE INDUSTRY AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE.",{"entities":[(121, 142,"FOREIGN ORG"),(144, 175,"FOREIGN ORG"),(182, 196,"FOREIGN ORG"),(206, 228,"FOREIGN ORG"),(235, 260,"FOREIGN ORG"),(270, 295,"FOREIGN ORG"),(305, 326,"FOREIGN ORG"),(339, 357,"FOREIGN ORG")    ,(6066, 6084,"FOREIGN ORG") ]}),
            ("NO LINKED ACCOUNTS FOUND EMAIL ID, MOBILE NO AND PAN WERE CHECKED.PART B DETAILS OF INVESTIGATION. CUSTOMER HAS MADE FOREIGN OUTWARD REMITTANCES AMOUNTING TO INR . LAKHS AGAINST PURPOSE CODE S-AUDIO-VISUAL AND RELATED SERVICES LIKE MOTION PICTURE AND VIDEO TAPE PRODUCTION, DISTRIBUTION AND PROJECTION SERVICES TO JUST TWO OVERSEAS PARTIES NAMED LIVING FILMS PTE LTD, SG AND GATE STUDIOS PTE LTD, SG, WITHIN MONTHS OF OPENING THE ACCOUNT WITH OUR BANK.",{"entities":[(347,365,"FOREIGN ORG"),(375,395,"FOREIGN ORG")]}),
            ("IN CHINA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  INDUCTION CHAFER. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON  FROM PREMIER HOTEL MANAGEMENT CO LTD FROM BANGLADESH DETAILS OF WHOOM IS NOT AVAILBLE IN PUBLIC DOMAIN. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF  MTT CYCLE NOT COMPLETED IN  DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT  TO  TO TUNE OF EUR  TO ERNST NEUMARKER GMBH AND COENTITY DEALS IN FOOD SUBSTITUES, SNACKS MAKING MACHINES WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS  KITCHENWARE. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON  TO THE TUNE OF USD. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF EUR .LACS, GBP, USD.LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF KITCHINWARE.. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.A H INTERNATIONAL, PARTNERSHIP FIRM, HAVING ADDRESS:ST FLOOR / GANDHI,PATH,CHITRAKOOT SCHEME, JAIPUR, RAJASTHAN, INDIA,  ,  CONTACT NUMBER: + AND EMAIL ID: VINAY@AHHOSPITALITY.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, JAIPUR - VAISHALI NAGAR,GROUND FLOOR, SAURAV TOWERS, PLOT NO. C, C BLOCK, VAISHALI NAGAR, JAIPUR, RAJASTHAN, ,  SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AATFAB. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . GSTIN AS PER BANK RECORDS IS AATFABZR. OTHER RELATIONSHIP OF CUSTOMER ARE , , , , , , , , . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VINAY LODHA AUTHORISED SIGNATORY- ALTPLN. BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS AMRAW DEVI LODHA,HIMANSHU LODHAACWPLM. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, GSTIN. CUSTOMER INVOLVE IN BUSINESS OF HOSPITALITY SERVICE AND PROVIDES PRODUCTS RELATED TO HOSPITALITY. CUSTOMER IMPORT MATERIAL LIKE SCHOTT ZWIESEL PURE LOOP I.E. WINE GLASS ETC. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO CR.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL  YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS MAJORLY IN JAIPUR,VAISHALI NAGAR BRANCH,  TRANSFERS RS. . LAKHS MAJORLY FROM HIMANSHU LODHA,AMRAW DEVI LODHA JT,CLEARING RS. . LAKHS MAJORLY FROM APA HOTELS PVT LTD,THE INDIAHOTREL,CROWN PROBUILD,  NEFT RS. . LAKHS MAJORLY FROM RELIANCE INDUSTRIES,A H INTERNATIONAL,VINAY LODHA,  RTGS RS. . LAKHS MAJORLY FROM A H INTERNATIONAL,BAID LEASING AND  FINANCE,RSB WEALTH PRIVATE LIMITED,  REMITTANCE RS. . LAKHS MAJORLY FROM MEYER INTERNATIONAL MARKETING PTE, ALETA AZUL CAPITAL SL   C,K D U ADVENTURES PRIVATE LIMITED,  INTERNET FUND TRANSFERS RS. . LAKHS,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS.  ,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY TO KAMAL NANDLAL MANSINGKA,PINKCITY MOTORS PVT.LTD.,M.K.TRADERS,  CLEARING RS. . LAKHS MAJORLY TO YASH BHANDARI,CHAIN ROOP BAID,NIRMALA ASHOKRAJ,  REMITTANCE RS. . LAKHS MAJORLY TO ABERT SPA,SAMBONET PADERNO INDUSTRIE SPA,ABERT S.P.A,  INTERNET FUND TRANSFER RS. . LAKHS,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. FURTHER, THE QUANTUM OF    TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS, LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF EUR .LACS, GBP, USD.LACS. FOR EXAMPLE, VIOLATION OF LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN  TO - -USD . REMITTED TO JLIP GROUP CO., LTD  CLIENT DEALS TO HOSPITALITY INDUSTRY . ITS I TRADER OF FURNITURE ADN RELATED ITEMS-HTTP://WWW.JLIP.COM.CN/ENGLISH/",{"entities":[(3054,3087,"FOREIGN ORG"),(3553, 3564,"FOREIGN ORG"),(3522, 3552,"FOREIGN ORG"),(4486, 4502,"FOREIGN ORG")]}),
            ("FAISHING-INDUSTRIAL-HK-CO-LTD IN HONGKONG WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  TAGS. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED ON  FROM FUNSTATION LIMITEDFUNSTATION ARE FAMILY ENTERTAINMENT CENTRES WITH CARNIVAL GAMES, LASER TAG, ADVENTURE GOLF, BOWLING, VR AND MUCH MORE FUN GAMES- HTTPS://WWW.FUNSTATIONUK.COM/. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN  DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN  TO  TO TUNE OF USD TO CARDY INTELLIGENT TECHNOLOGY CO LTD CUSTOMER IS INTO MANUFACTURE OF RFID TAGS, WRISTBAND ETC.HTTPS://WWW.SMARTCARDY.COM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  RFID CARDS. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN  AND  TO THE TUNE OF USD. LACS FROM BAHJAT BAHLA BAHLA AL MAMOORAH DETAILS OF THE WHOM IS NOT AVAILABLE. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD. LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.SEMNOX SOLUTIONS PVT. LTD, PRIVATE LTD, HAVING ADDRESS:RD FLOOR FLR,MG ROAD LALBAGH,MANGALORE,PUNJA BUILDING ANNEXE, MANGALORE, KARNATAKA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: IQBAL@SEMNOX.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, MANGALORE - P M RAO ROAD,BHARATH BUILDING,GROUND FLOOR, P.M.RAO ROAD,MANGALORE, KARNATAKA, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AALCSB. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . TRANSACTION LINKED ACCOUNTS ARE . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE KIRAN KARANKI AUTHORISED SIGNATORY, SAMPATH KUMAR DIRECTOR, IQBAL MOHAMMAD DIRECTOR- AAHPIA ,RAJEEV GOPALAKRISHNAN DIRECTOR, BALAJI VENUGOPAL DIRECTOR.BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS KIRAN KARANKADZPKQ,MOHAMMAD IQBALAAHPIA. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI. DATE OF LAST KYC UPDATION .SEMNOX SOLUTIONS PRIVATE LIMITED SSPL IS ENGAGED IN THE BUSINESS OF PROVIDING IT SOLUTIONS FOR THE ENTERTAINMENT AND LEISURE INDUSTRY. SSPL OFFERS SOLUTIONS SUCH AS FAMILY ENTERTAINMENT, WATER AND THEME PARKS, AND FOOD AND BEVERAGES. THE COMPANY CATERS TO WATER PARKS, MUSEUMS, KIDS PLAY AREAS, RESTAURANTS, FOOD COURTS, AND AQUARIUMS, AMONG OTHERS. SSPLS FEC AND PARK CLIENTS INCLUDE SUCH AS COWPLAY CPWMOO, SINGAPORE, ARENA REKREASI KELUARGA, AEON FANTASY, MALAYSIA, AND SPARKS, INDONESIA, AND CAMAYA COAST, PHILIPPINES, AMONG OTHERS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO CR.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH TRANSFERS RS. . LAKHS, REMITTANCE RS. . LAKHS MAJORLY FROM SEMNOX SOLUTIONS LLC  LU,AEON FANTASY MALAYSIA SDN BHD,SEMNOX SOLUTIONS DMCC P O BOX. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY TO SEMNOX SOLUTIONS PRIVATE LIMITED, REMITTANCE RS. . LAKHS MAJORLY TO FAISHING INDUSTRIAL HK CO LIMITED,TITOMA MANUFACTURING LTD,AMAZON WEB SERVICES, INC. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS, LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF US.LACS. FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON  USD  REMITTED TO FAISHING INDUSTRIAL HK CO LTDTHIS COMPANY DEALS IN EXPORT, UNDER DISCS, TAPES, SOLID-STATE NON-VOLATILE STORAGE DEVICES, SMART CARDS AND OTHER MEDIA FOR THE RECORDING OF SOUND OR OF OTHER PHENOMENA, WHETHER OR NOT RECORDED, INCLUDING MATRICES AND MASTERS FOR THE PRODUCTION OF DISCS, BUT EXCLUDING PRODUCTS OF CHAPTER  CATEGORY.",{"entities":[(596, 640,"FOREIGN ORG"),(2759, 2784,"FOREIGN ORG"),(2811, 2833,"FOREIGN ORG"),(2839, 2856,"FOREIGN ORG"),(2862,2887,"FOREIGN ORG"),(3344,3364,"FOREIGN ORG"),(3603,3627,"FOREIGN ORG"),(3569,3602,"FOREIGN ORG")]}),
            ("IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD .LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF MACHINE TOOL AND ACCESSORIES. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.THERMAX BABCOCK  WILCOX ENERGY SOLU.P.LTD,PRIVATE LTD., HAVING ADDRESS:ADISA,ICON,GAT NO FLR,/,, BANGALORE HIGHWAY,BAVDHAN BUDRUK,OPPOSITE HEMRL,MUMBAI, PUNE, MAHARASHTRA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: RAMNATH.NAYAK@TBWES.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, PUNE - BUND GARDEN,A WING SHANGRILA GARDENS BUND GARDEN ROAD, PUNE, MAHARASHTRA, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AADCTA. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . GSTIN AS PER BANK RECORDS IS AADCTAZJ. TRANSACTION LINKED ACCOUNTS ARE ,.AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NIRANJAN PAWGI AUTHORISED SIGNATORY- ABKPPN, NIRANJAN PAWGI AUTHORISED SIGNATORY- ABKPPN ,LATA KUMAR AUTHORISED SIGNATORY, ASHISH MOHAN LAL BHANDARI AUTHORISED SIGNATORY- AEXPBF ,MOHAN LAL BHANDARI ASHISH AUTHORISED SIGNATORY- AEXPBF . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA,AOA, COI. THERMAX BABCOCK  WILCOX ENERGY SOLUTIONS PRIVATE LIMITED TBWES IS A JOINT VENTURE BETWEEN THERMAX LIMITED, PUNE, INDIA AND BABCOCK  WILCOX POWER GENERATION GROUP, INC., USA. THE COMPANY IS ENGAGED IN MANUFACTURING SUBCRITICAL AND SUPERCRITICAL BOILERS FOR THERMAL POWER GENERATION. THE COMPANYS OTHER PRODUCTS INCLUDE LOW NOX COMBUSTION SYSTEMS AND ROLL WHEEL PULVERIZERS.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH, CLEARING RS. . LAKHS MAJORLY FROM ICICI PRUDENTIAL M, NEFT RS. . LAKHS MAJORLY FROM MAHARASHTRA VALUE ADDED TAX,UTI MUTUAL FUND,THERMAX LIMITED, RTGS RS. ,. LAKHS MAJORLY FROM UTI MF COMMON FUND,SBI MUTUAL FUND,DSP MUTUAL FUND REDEMP, INTERNET FUND TRANSFERS RS. . LAKHS AND OTHERS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY TO THERMAX BABCOCK  WILCOX ENERGY SOLU.P.LTD, CLEARING RS. . LAKHS, RTGS RS. ., REMITTANCE RS. . LAKHS MAJORLY TO MANTRA SWITCHGEAR CO LTD,LSIS CO LTD, INTERNET FUND TRANSFER RS. ,. LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RS.  BY CASH , REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. ., REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH REMITTANCE RS. ,. LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH REMITTANCE RS. ,. LAKHS.  VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E  MTT CYCLE NO COMPLETED IN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD .LACS. FOR EXAMPLE, MTT CYCLE NOT COMPLETED IN  DAYS WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN ON  USD  CREDITED FROM LSIS CO LTD  AUTOMATION, PLC, ELECTRONIC DEVICES MANUFACTURING FROM SOUTH KOREA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HSOTHERS. HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED FROM THE BABCOCK + WILCOX COMPANY FROM USA.  THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE",{"entities":[(315, 362,"FOREIGN ORG") ,(2529, 2553,"FOREIGN ORG"),(3427, 3455,"FOREIGN ORG"),(1494, 1538,"FOREIGN ORG")]}),
            ("AND GOODS MENTIONED AS HS  ORTHOPAEDIC GOODS,HS  DRILLING MACHINE,HS  CONNECTING RODS. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED  TO  FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS - HTTPS://WWW.DNB.COM/BUSINESS-DIRECTORY/COMPANY-PROFILES.INNOVATION_ORTHO_LINE_LIMITED.CABBABBECCEA.HTML. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN  DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN  AND  TO TUNE OF USD .LACS TO JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS  DRILLING, HS  SELF DRILLING. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN  AND  TO THE TUNE OF USD.LACS FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS - HTTPS://WWW.DNB.COM/BUSINESS-DIRECTORY/COMPANY-PROFILES.INNOVATION_ORTHO_LINE_LIMITED.CABBABBECCEA.HTML. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD.LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT,STR IS BEING FILED.AUXEIN MEDICAL PRIVATE LIMITED,PRIVATE LTD, HAVING ADDRESS:PLOT NO ,HSIIDC FLR,PHASE IV,KUNDLI SONIPAT, SONIPAT, HARYANA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: ACCOUNTS@AUXEINMEDICAL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, NEW DELHI - MODEL TOWN,H-, MODEL TOWN III, DELHI, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAKCAK. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . THERE ARE NO LINKED ACCOUNT. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE RAHUL LUTHRA AUTHORISED SIGNATORY, DEEPAK LUTHRA AUTHORISED SIGNATORY, GAURAV LUTHRA, ASHOK LUTHRA.BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK LUTHRAACBPLA,RAHUL LUTHRAADYPLJ. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI. AS PER ENHANCED DUE DILIGENCE AUXEIN MEDICAL PRIVATE LIMITED AMPL IS ENGAGED IN MANUFACTURING OF ORTHOPEDIC IMPLANTS. THE COMPANYS PRODUCTS INCLUDE MINI FRAGMENT, HIP PROSTHESIS, INTERLOCKING NAILS, INSTRUMENT AND IMPLANT BOX, SPINE IMPLANTS, MAXILLOFACIAL IMPLANTS, WISE LOCKING PLATES, AND BONE PLATES. THE COMPANYS UNIT IS EQUIPPED WITH CNC MACHINES, DRILL MACHINES, COMPARATORS, CUTTING MACHINES AND SPECIAL PURPOSE MACHINES. AMPL HAS A RANGE OF + IMPLANTS AVAILABLE IN + COUNTRIES IN ASSOCIATION WITH + PARTNERS.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. . LAKHS OF WHICH REMITTANCE RS. . LAKHS MAJORLY FROM  FIXIER SA DE CV  CARR DR M, INNOVATION ORTHO LINE LTD. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. .. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH REMITTANCE RS. . LAKHS MAJORLY FROM  ZOLDAN CORPOREISHON CIA L, MEDICAL ORTOVIT SRL  BUCUR, ATRON HEALTH S A   LEOFO. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH  REMITTANCE RS. . LAKHS MAJORLY TO CHANGZHOU NUBE INTERNATIONAL,CHANGZHOU XIETONG INDUSTRIES CO,CHANGZHOU XIETONG INDUSTRIES CO LTD. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH REMITTANCE RS. . LAKHS.TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS . TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD.LACS. FOR EXAMPLE, VIOLATION OF FUNDS OUTLAY MORE THAN  DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN  BETWEEN  TO  USD . LACS REMITTED IN PARTS FROM JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE",{"entities":[(874, 903,"FOREIGN ORG"),(3565, 3600,"FOREIGN ORG"),(3070, 3095,"FOREIGN ORG"),(4480, 4527,"FOREIGN ORG")]}),
            ("OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS REFINED SUNFLOWER OIL . HOWEVER THE RESPECTIVE EXPORT WAS RECEIVED ON BETWEEN  AND  TO THE TUNE OF RS..LACS FROM D.J.T IMPORT EXPORT IN MADAGASCAR WHOSSE DETAILS ARE NOT FOUND IN PUBLIC DOMAIN. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN  DAYS, WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN  AND  TO TUNE OF RS. .LACS WAS REMITTED TO HEBEI LUEDING IMPORT AND EXPORT CO LTDCOMPANY DEALS IN PPGI, GALVANISED STEEL COIL, ROLL FORMING MACHINE, ROOFING SHEETS, WELDING ELECTRODE-HTTPS://WWW.MACHINERYOFFERS.COM/COMPANY/LEADINGROOFING/CONTACT.HTML AND SHOUGUANG DASEN WOOD CO LTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED ASHS  PLYWOOD, GALVANIZED CORRUGATED SHEET, HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN  AND  TO THE TUNE OF EUR . IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF .CRORES WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS . CLIENT IS DEALING IN MERCHANT TRADE OF DIVERSIFIDE PRODUCTS OILS, GA SHEETS, WOOD ETC. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.THAKKAR EXIM,PROPRIETORSHIP FIRM, HAVING ADDRESS:THAKKAR HOUSE,RAJDEV STREET, SANGANVA CHOWK, RAJKOT, GUJARAT, INDIA,  ,  CONTACT NUMBER: + AND EMAIL ID: INFO@THAKKAREXIM.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, RAJKOT - YAGNIK ROAD, SHOP NO.  , CONVENTION CENTER, NR HARIBHAI HALL,OFF YAGNIK, ROAD, RAJKOT, GUJARAT, ,  SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAXPTQ. PAN LINKED ACCOUNTS ARE , . IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NITESH HARIDAS THAKKAR PROP- AAXPTQ ,NITESH HARIDAS THAKKAR AUTHORISED SIGNATORY- AAXPTQ THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN,MOA,AOA,IE CODE. DATE OF LAST KYC UPDATION . CUSTOMER IS INTO IMPORT, EXPORT AND MERCHANTING TRADER. HAVING BEGUN WITH ONLY ONE TEXTILE PRODUCT, THE COMPANY HAS NOW WELL DEVELOPED INTO AN STAR EXPORT HOUSE DEALING IN VARIOUS PRODUCTS LIKE AGRICULTURAL COMMODITIES SUCH AS RICE OF DIFFERENT VARIETIES, WHEAT FLOUR, BAKERY FLOUR, BISCUITS, PLASTIC HOUSEHOLD PRODUCTS, KITCHEN APPLIANCES AND HOME APPLIANCES. FIRM IS HAVING WEBSITE WWW.THAKKAREXIM.COM. ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS  CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL  YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH  TRANSFERS RS.. LAKHS MAJORLY FROM NITESH HARIDAS THAKKAR,THAKKER ELECTRICALS,THAKKAR EXIM,CLEARING RS. . LAKHS,  NEFT RS. . LAKHS MAJORLY FROM THAKKAR EXIM,GOPALSWAMY  BASKARAN,E PAO GST REFUNDS,  RTGS RS. . LAKHS MAJORLY FROM THAKKAR EXIM,PMS ENTERPRISES,ARTI INTERNATIONAL,  REMITTANCE RS. ,. LAKHS MAJORLY FROM THAKKAR EXIM,THAKKAR EXIM  RAJDEV STREET,SOCIETE RAG,  INTERNET FUND TRANSFERS RS. . LAKHS,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS.  ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BRANCH,  TRANSFERS RS. . LAKHS MAJORLY TO THAKKAR EXIM,BHAVNA MAHESH PARMAR,PARMAR RAJDEEP MAHESH,  CLEARING RS. . LAKHS,  RTGS RS. ,. LAKHS MAJORLY TO VIRAJ IMPEX,  REMITTANCE RS. . LAKHS MAJORLY TO HEBEI TOMATO INDUSTRY CO LTD. FURTHER, THE QUANTUM OF    TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE:    FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS.THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER  IN THE NAME OF THAKKAR EXIM, REPORT SERIAL NUMBER .  VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS, HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF .CRORES. FOR EXAMPLE, VIOLATION OF MTT CYCLE NOT COMPLETED IN  DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON  TO THE TUNE OF USD TO OIL TEC FOR OILS AND DETERGENTS MANUFACTURER OF SOAPS ADN OILS- HTTPS://WWW.GULFOOD.COM/EXHIBITORS/OIL-TEC-FOR-OILS-AND-DETERGENTS IN EGYPT WITH PURPOSE",{"entities":[(240, 262,"FOREIGN ORG"),(504, 549,"FOREIGN ORG"),(716, 743,"FOREIGN ORG") ,(3654, 3682,"FOREIGN ORG")]}),
            ("OF THE VALUE OF USD .LACS. FOR EXAMPLE, VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN  DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT  BETWEEN  TO  USD . LACS REMITTED IN PARTS TK FUJIKIN CORPORATION ENTITY DEALS IN PIPES AND FITTINGS HTTP://TK-FUJIKIN.COM/TKSCT//ENG/SUB/HISTORY.PHP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  CONTROLLER. HOWEVER RESPECTIVE INWARDS REMITTANCE TOWARDS EXPORTS RECEIVED ON  TO TUNE OF USD .LACS CARBON CRAFT ANG DETAILS OF WHOM ARE NOT AVAILABLE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY OF MORE THAN  DAYS, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON  USD  TO  TK FUJIKIN CORPORATION ENTITY DEALS IN PIPES AND FITTINGS HTTP://TK-FUJIKIN.COM/TKSCT//ENG/SUB/HISTORY.PHP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  FITTINGS. THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD .LACS. WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS . CLIENT IS DEALING IN MERCHANT TRADE OF PIPES, FITTINGS. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.PRAMA INSTRUMENTS PVT LTD, PRIVATE LTD, HAVING ADDRESS:PLOT R,RABALE, NAVI MUMBAI, MAHARASHTRA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: MADAN@PRAMAGROUP.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, MUMBAI - MAHAPE,SHOP NO  , PLOT NO X, TECHNOCITY , NEXT TO SAROVAR PORTICO, MAHAPE, THANE DIST., NAVI MUMBAI, MAHARASHTRA, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAACPF. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . OTHER RELATIONSHIP OF CUSTOMER ARE , . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORISED SIGNATORY AND DIRETOR OF THE REPORTED ACCOUNT IS  MADANMOHAN AMBIKAPRASAD MISHRAAABPMB,VIMLADEVI MADANMOHAN MISHRAAHWPMH. BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MADAN MOHAN A MISHRAAABPMB,MADANMOHAN A MISHRAAABPMB. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, IEC, COI. ENTITY IS ENGAGED IN MANUFACTURING AND TRADING OF LABORATORY INSTRUMENTS AND FILTRATION SYSTEMS. THE COMPANYS PRODUCT PORTFOLIO INCLUDES CHROMATOGRAPHY AND INSTRUMENTS, FILTER HOLDER, STERILITY TEST MANIFOLD SYSTEM, WATER JET CUTTING MACHINE, TURN OVER SILICON SEPTA, SYRINGE FILTER HOLDER, STERILITY MANIFOLD, TURN OVER SILICONE SEPTA AND STABILITY CHAMBER. PIPL CATERS TO PHARMACEUTICAL, CHEMICAL, FOOD, FRAGRANCES, FORENSIC AND ENVIRONMENT MONITORING SECTORS. PIPL WAS INCORPORATED IN  AND HAS ITS REGISTERED OFFICE LOCATED IN MUMBAI, MAHARASHTRA. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. LAC TO RS. CR. ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. .. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. . LAKHS OF WHICH INTERNET FUND TRANSFER RS. . LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY FROM BIORADIUS THERAPEUTIC RESEARCH PVT. LTD,PRESSHOT ENGINEERS,HOTEL ROYAL ELITE, CLEARING RS. . LAKHS MAJORLY FROM GUJARAT FLUOROCHEMICALS LIMITED,PRAMA INSTRUMENTS PVT LTD,ABB INDIA LTD, NEFT RS. ,. LAKHS MAJORLY FROM PRAMA INSTRUMENTS PVT LTD,PRAMA INSTRUMENTS PRIVATE LTD,ABB INDIA LIMITED, RTGS RS. . LAKHS MAJORLY FROM TRUSHNA EXIM,PRAMA INSTRUMENTS PVT LTD, INTERNET FUND TRANSFERS RS. . LAKHS, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH  TRANSFERS RS. . LAKHS MAJORLY TO ICICI PRUD LIFE INS CO LTD, EMPLOYEES PROVIDENT FUND ORGANISATION,ESI FUND A/C NO , CLEARING RS. . LAKHS MAJORLY TO DEUTSCHE BANK,MR ASHOK BHUTA, RTGS RS. . LAKHS MAJORLY TO PRAMA INSTRUMENTS PVT, REMITTANCE RS. ,. LAKHS MAJORLY TO TK FUJIKIN CORPORATION,GARDNER DENVER THOMAS PNEUMATIC, INTERNET FUND TRANSFER RS. . LAKHS, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES AND OTHERS RS. . LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS . TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH IS NIL, REMITTANCE RS. . LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL.TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS, FUNDS OUTLAY MORE THAN  DAYS, LOSS ON MTT CYCLE FUNDS OUTLAY OF MORE THAN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE ",{"entities":[(4402, 4424,"FOREIGN ORG"),(4425, 4456,"FOREIGN ORG"),(887, 910,"FOREIGN ORG"),(211,233,"FOREIGN ORG")]}),
            ("PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HSNIGER SEED FREE. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON  TO TUNE OF USD  WAS RECEIVED FROM SARL RENAUDAT J C LES CHAUMES WHOSES DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN  DAYS, WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN  AND TO TUNE OF USD TO BLACK SEA GROUP LOGISTICS IN RUSSIA DETAILS OF WHICH IS NOT AVAILABE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSCHICKPEAS, HSRED MILLET. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON   TO THE TUNE OF USD. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD .LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF SEEDS, MILLETS, MAIZE ETC.. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.AGRICOM IMPEX,PROPRIETORSHIP FIRM, HAVING ADDRESS:F NO ,AMALTAS APPT,RAJ NAGAR, NAGPUR, MAHARASHTRA, INDIA,  ,  CONTACT NUMBER: + AND EMAIL ID: AGRICOM.IMPEX@HOTMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, NAGPUR - RAMDASPETH,AKARSHAN BUSIPLEX, , CENTRAL BAZAAR ROAD, RAMDASPETH, NAGPUR, MAHARASHTRA, ,  SINCE . DATE OF INCORPORATION IS .  PAN NUMBER OF THE ACCOUNT IS AJRPGD. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . GSTIN AS PER BANK RECORDS IS AJRPGDZY.TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKASH GHADSE PROP- AJRPGD.THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, GSTIN. DATE OF LAST KYC UPDATION . AGRICOM IMPEX IS A LEADING WHOLESALE MANUFACTURER AND EXPORTER OF AGRICULTURAL OIL SEEDS, BIRD SEEDS LIKE YELLOW MILLET, SAFFLOWER SEED, NIGER SEED, SORGHUM, SESAME SEEDS, CHICK PEAS AND PEANUTS IN INTERNATIONAL MARKET. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. LAC TO RS. CR.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS  CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL  YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH  TRANSFERS RS. . LAKHS MAJORLY FROM AGRICOM IMPEX,SHRI BALAJI INDUSTRIES,  CLEARING RS. . LAKHS MAJORLY FROM FWD CANC GAIN, NEFT RS. . LAKHS MAJORLY FROM NARAYANA SAI TRADERS,SURGO RESOURCES,  RTGS RS. . LAKHS MAJORLY FROM TALOJA AGRO FOODS PVT LTD,HITASHI AGRI TRADE,AGRICOM IMPEX,  REMITTANCE RS. ,. LAKHS MAJORLY FROM EEFC,EEFC CONVERSION,AGRICOM IMPEX,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS.  ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BRANCH,  TRANSFERS RS. . LAKHS MAJORLY TO BHAIYAJI RAMBHAUJI ROKDE JEWELLERS,ICICI PRUD LIFE INS CO LTD-CASH COLLN,AGRICOM IMPEX,  CLEARING RS. . LAKHS MAJORLY TO PANKAJ GULATI AND ASSOCIA,M S E D C L,V P SALES AND SERVICES,  NEFT RS. . LAKHS MAJORLY TO B NARAYANA,THAPARSONS K V SHIPP,GIDDDIAH TRADERS,RTGS RS. . LAKHS MAJORLY TO SHREE BHAGAWATI AGRO P,NARAYAN SAI TRADER,MILAP TRADING,  REMITTANCE RS. . LAKHS MAJORLY TO GENERAL COMMODITIES LTD,SPRINGHAUS LLC,KAMAL LLC,  INTERNET FUND TRANSFER RS. . LAKHS,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. FURTHER, THE QUANTUM OF    TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE:    FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL,  REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS,  REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH  CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS, LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD .LACS. FOR EXAMPLE, VIOLATION OF LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON  USD  WAS DEBITED TO KAMAL GLOBAL TRADING PTE LTD IN SINGAPORE WHOSE DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN WITH",{"entities":[(3458, 3481,"FOREIGN ORG"),(3482, 3496,"FOREIGN ORG"),(3497, 3506,"FOREIGN ORG"),(4640, 4666,"FOREIGN ORG")]}),
            ("STANDARD C,CENTRAL BO,GE POWER I, REMITTANCE RS. ,. LAKHS MAJORLY TO JINDAL STEEL AND POWER LIMITED,ATI FLAT ROLLED PRODUCTS,KANEMATSU CORPORATION, INTERNET FUND TRANSFER RS. . LAKHS, , AND OTHERS RS. ,. LAKHS.FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. ,. LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. ,. LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. ,. LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD .LACS, EUR .LACS, CNY .LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN  DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON ON  USD  WAS REMITTED TO VICENTE BERRIZBEITIA S L CLIENT IS INTO SUPPLY OF BOLTING MATERIALS- HTTP://BERRIZBEITIA.COM/EN/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HSTUBES AND PIPES. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN  DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT ON  TO TUNE OF USD.LACS TO GE HYDRO CHINA CO. LTDENTITY IS JV WITH  YALONG RIVER HYDROPOWER DEVELOPMENT CO. LTD. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HSHYDRO SET. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD .LACS, EUR .LACS, CNY .LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS.CLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT,STR IS BEING FILED.GE POWER INDIA LIMITED,PUBLIC LTD., HAVING ADDRESS:T-  T- PLOT  TO ,JAYPEE WISHTOWN,, SECTOR ,AXIS HOUSE, NOIDA, UTTAR PRADESH, INDIA, , CONTACT NUMBER: + AND EMAIL ID: CHANDNI.SISODIA@POWER.ALSTOM.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, NEW DELHI - CONNAUGHT PLACE,A, PHELPS BUILDING, CONNAUGHT PLACE,NEW DELHI, DELHI, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AABCAF. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE P JAYALAKSHMI AUTHORISED SIGNATORY- AAEPPB ,AMARESH SINGH AUTHORISED SIGNATORY- AICPSL ,RAJESH AGGARWAL AUTHORISED SIGNATORY- AGIPAC ,HIREN RAJNIKANT VYAS AUTHORISED SIGNATORY- ABTPVP ,NAVEEN SHANKARAN AUTHORISED SIGNATORY- AIOPNM. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN- AABCAF,MOA,AOA. DATE OF LAST KYC UPDATION . GE POWER INDIA LIMITED GPIL, FORMERLY KNOWN AS ALSTOM INDIA LIMITED, IS ENGAGED IN ENGINEERING, PROCUREMENT, MANUFACTURING, CONSTRUCTION, AND SERVICING, OF POWER PLANTS AND POWER EQUIPMENT IN BOTH THERMAL AND HYDRO SEGMENTS. THE COMPANY DESIGNS MANUFACTURE, AND DELIVERS TURNKEY POWER PLANTS, POWER GENERATION EQUIPMENT, AND AIR QUALITY CONTROL SYSTEMS AND SERVICES; AND ALSO PROVIDES RENEWABLE POWER GENERATION SOLUTIONS FOR INTEGRATED POWER PLANTS COVERING HYDROELECTRICITY, WIND, GEOTHERMAL, BIOMASS, SOLAR, AND WAVE AND TIDAL STREAM ENERGIES. IT ALSO OFFERS INDIVIDUAL COMPONENTS FOR A RANGE OF TURBINES AND GENERATORS, AS WELL AS VARIOUS SERVICES, INCLUDING PLANT MODERNIZATION, MAINTENANCE, AND OPERATIONAL SUPPORT; AND PROVIDES PRODUCTS AND SYSTEMS FOR ELECTRICAL UTILITIES AND INDUSTRIES.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH RS.  BY CASH, NEFT RS. . LAKHS MAJORLY FROM LALITPUR POWER GENERATION CO,TORRENT POWER LIMITED, RTGS RS. ,. LAKHS MAJORLY FROM BHARAT HEAVY ELECTRICAL. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH RS.  BY CASH, RTGS RS. ,. LAKHS MAJORLY TO GE POWER I, REMITTANCE RS. . LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS MAJORLY IN GULBARGA BRANCH, TRANSFERS RS. ,. LAKHS MAJORLY FROM GE POWER INDIA LTD TELENGANA FGD PROJECT - ESCROW ACCOUNT,GE POWER INDIA LTD-IGSTPP JHAJJAR FGD PROJECT - ESCROW ACCOUNT,GE POWER INDIA LTD-SIPAT FGD PROJECT - ESCROW ACCOUNT, CLEARING RS. ,. LAKHS MAJORLY FROM LT MHPS BOILERS PRIVATE LIMIT,NTPC LIMITED,VEDANTA LIMITED, NEFT RS. ,. LAKHS MAJORLY FROM GENERAL ELECTRIC COMPANY,NHPC LIMITED,MEJA URJA NIGAM PVT LTD, RTGS RS. ,. LAKHS MAJORLY FROM M/S DOOSAN POWER SYSTE,BHARAT HEAVY ELECTRICA,GE POWER INDIA LIMITED, REMITTANCE RS. ,. LAKHS MAJORLY FROM ARAVALI POWER COMPANY PRIVATE LTD,GENERAL ELECTRIC SWITZERLAN,NTPC LIMITED, INTERNET FUND TRANSFERS RS. . LAKHS, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES , UPI TRANSFERS RS. . LAKHS IN  INSTANCE AND OTHERS RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY TO GE POWER INDIA LTD UNPAID DIVIDEND ACCOUNT  ,GE POWER INDIA LIMITED,GE POWER SYSTEMS INDIA PVT LTD, CLEARING RS. . LAKHS, NEFT RS. ,. LAKHS MAJORLY TO GE POWER INDIA LIMIT,CENTRAL EXCISE SERVI,WEST BENGAL STATE EL, RTGS RS. ,,. LAKHS MAJORLY",{"entities":[(22, 32,"FOREIGN ORG"),(125, 146,"FOREIGN ORG"),(1139, 1163,"FOREIGN ORG"),(1615, 1632,"FOREIGN ORG"),(1656, 1699,"FOREIGN ORG")]}),
            ("ALSO ON  RS. . LACS GOT CREDITED BY INWARD REMITTANCE FROM WORLD FIRST ASIA LTD, HONG KONG WITH INFORMATION AS IMPORT OF GARMENTS AND ON  RS.  LACS GOT DEBITED BY CMS CLEARING WITH REMARK AS SDI S.D INTERNATIONAL. IN OTHER EXAMPLE, ON  RS.  LACS WAS CREDITED THROUGH RTGS FROM SD INTERNATIONAL ACC-, IFSC-INDB AND ON THE SAME DAY RS. . LACS ARE IMMEDIATELY DEPLOYED THROUGH CMS CLEARING WITH REMARK AS SDI S.D INTERNATIONAL. SIMILAR SUCH TRANSACTIONS ARE OBSERVED THROUGHOUT THE ACCOUNT FOR WHICH NO RATIONAL FOUND. TOTAL TURNOVER IN THE FINANCIAL YEAR , WAS RS. ,. LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF LESS THAN RS. LACS. IN VIEW OF INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS, LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF HIGH VALUE FUNDS AND HIGH TURNOVER, STR IS BEING FILED.S.D. INTERNATIONAL, PROPRIETORSHIP FIRM, HAVING ADDRESS: ND FLOOR FLR, /B, NIMTOLA GHAT, STREET NEAR JORABAGAN MORE, KOLKATA, WEST BENGAL, INDIA, , CONTACT NUMBER: + AND EMAIL ID: BANKS@SDWORLD.IN IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, KOLKATA - BURRA BAZAR, ICICI BANK LTD, P, KALAKAR STREET, BURRABAZAR, KOLKATA, WEST BENGAL, , SINCE . DATE OF INCORPORATION IS . RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT. PAN NUMBER OF THE ACCOUNT IS AFYPDH. OTHER RELATIONSHIP OF CUSTOMER ARE , , , , , , , , , , , , , , , . TRANSACTION LINKED ACCOUNTS ARE  AND . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE SANJOY DEY PROPRIETOR - AFYPDH. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN - AFYPDH, AADHAR. DATE OF LAST KYC UPDATION . KYC UPDATION IS NOT DUE. ENTITY IS INTO BUSINESS OF KIDS GARMENTS AND WOMEN SUIT PCS SET TRADERS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS. LACS. ALERT WAS TRIGGERED FOR INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY FROM S D INTERNATIONAL, CLEARING RS. . LAKHS MAJORLY FROM S D, S D TRADE IMPEX PRIVATE LIMITE, CMS_, RTGS RS. ,. LAKHS MAJORLY FROM S D INTERNATIONAL, SDINTERNATIONAL, REMITTANCE RS. ,. LAKHS MAJORLY FROM ADVANTAGE GENERAL TRADING LIMITED, COSMIC EDGE GENERAL TRADING LIMITED, LIGHTWAYS GENERAL TRADING LLC. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BRANCH, TRANSFERS RS. . LAKHS MAJORLY TO HETAL SYNTEX, AMIT ROADLINES, S D INTERNATIONAL, CLEARING RS. ,. LAKHS MAJORL TO SDI_RTGS___, SDI_RTGS___, SDI_TRF___, RTGS RS. ,. LAKHS MAJORLY TO NITFUL COMMERCIAL PVT, LAXMIWAN SUPPLIERS PVT, SURAKSHIT HOUSING ADVI. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS WERE: FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS, TRANSFERS RS. ,. LAKHS, RTGS RS. ,. LAKHS, REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS, CLEARING RS. ,. LAKHS, RTGS RS. ,. LAKHS. FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH RS.  BY CASH, REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. AS PER THE TRANSACTION PATTERN, THERE ARE FREQUENT HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS. FOR EXAMPLE, ON  RS.  LACS WAS CREDITED THROUGH RTGS FROM S D INTERNATIONAL ACCOUNT NO- , BANK NAME-FEDERAL BANK, IFSC CODE- ICIC AND ON  RS.  LACS WAS DEPLOYED THROUGH RTGS TOWARDS S D INTERNATIONAL ACC-, IFSC-INDB AND THE REST OF THE FUNDS WERE DEPLOYED THROUGH CMS WITH REMARK AS SDI S.D INTERNATIONAL.",{"entities":[(59, 79,"FOREIGN ORG"),(406,423,"FOREIGN ORG"),(2388, 2403,"FOREIGN ORG"),(2514, 2543,"FOREIGN ORG"),(2477, 2511,"FOREIGN ORG"),(2442, 2474,"FOREIGN ORG")]}),
            ("BOSTON_MEGA_TEXTILE_LLC_IN_KARAKALPAKSTAN.HTML WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HSCOTTON YARN. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN  AND  TO THE TUNE OF USD. LACS FROM CHENG CHIH TRANSCEND ENTERPRISE CORPORATION CLIENT DEAAL SIN TRADING OF CONTTON YARNS-HTTPS://PANJIVA.COM/CHENG-CHIH-TRANSCEND-ENTERPRISE/. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD. LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS . CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.SQUARE CORPORATION, PROPRIETORSHIP FIRM, HAVING ADDRESS: C BRS NAGAR,LUDHIANA,PUNJAB, LUDHIANA, PUNJAB, INDIA, , CONTACT NUMBER: + AND EMAIL ID: ACCOUNTS@SQUARE-CORP.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, LUDHIANA - FEROZE GANDHI MARKET,NEHRU SIDHANT KENDER TRUST BUILDING, FEROZ GANDHI MARKET, LUDHIANA, PUNJAB, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAOPVG. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . GSTIN AS PER BANK RECORDS IS AAOPVGZF. TRANSACTION LINKED ACCOUNTS ARE . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VIVEK VERMA PROP, RAHUL TALWAR AUTHORISED SIGNATORY- ADBPTB, VANDANA VERMA AUTHORISED SIGNATORY- AADPVM THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE . DATE OF LAST KYC UPDATION . CUSTOMER IS INTO YARN BUSINESS. HTTP://WWW.SQUARE-CORP.COM/. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS EQUAL TO  CR. ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.. DEBIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY FROM PROMINENT EXPORTS INTERNATIONAL, REMITTANCE RS. . LAKHS MAJORLY FROM THERMAX YARN DYED FABRICS LIMITED.,EVINCE TEXTILES LTD,MEGHNA KNIT COMPOSITE LIMITED. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH REMITTANCE RS. . LAKHS MAJORLY TO WEIQIAO TEXTILE COMPANY LIMITED,WEIHAI WEIQIAO INDUSTRIAL PARK CO,MIF INTERNATIONAL LIMITED.FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS.TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS . TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN  DAYS, LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF US.LACS. FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN  TO  USD . LACS REMITTED TO IRS UNIVERSAL PTE LTDIRS UNIVERSAL PTE LTD. ISIN INTERNATIONAL PTE LTD WAS FOUNDED IN . THE COMPANYS LINE OF BUSINESS INCLUDES THE WHOLESALE DISTRIBUTION OF NON-DURABLE GOODS.-HTTPS://WWW.BLOOMBERG.COM/PROFILE/COMPANY/Z:SP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HSYARN. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT IS DUE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN  DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN  TO  TO TUNE OF USD.LACS TO BOSTON MEGA TEKSTIL LLC ENTITY IS INTO MANUFACTURE OF TEXTILE",{"entities":[(0,22,"FOREIGN ORG"),(2192,2246,"FOREIGN ORG"),(2247, 2276,"FOREIGN ORG"),(2350, 2381,"FOREIGN ORG"),(2382, 2415,"FOREIGN ORG"),(2416, 2441,"FOREIGN ORG"),(3329, 3371,"FOREIGN ORG"),(3373, 3399,"FOREIGN ORG"),(3928, 3958,"FOREIGN ORG")]}),
            ("HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED ON  TO TUNE OF USD  FROM MUNDRA GENERAL TRADING FZE IN UAEDEALS IN METAL SCRAP-HTTPS://PANJIVA.COM/MUNDRA-GENERAL-TRADING-F-Z-E/. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY OF MORE THAN  DAYS, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON  USD  TO TSAI SHEN TRADING LIMITED IN HONGKONGCLIENT IS INTO CERAMIC AND AGRO PRODUCTS RELATED -HTTPS://TSAISHENTRADING.COM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  STORAGE DEVICES. THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED USD  RECEIVED ON  FROM MUNDRA GENERAL TRADING FZE IN UAEDEALS IN METAL SCRAP-HTTPS://PANJIVA.COM/MUNDRA-GENERAL-TRADING-F-Z-E/. IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD .CRORES. WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.KOTHARI PRODUCTS LIMITED,PRIVATE LTD., HAVING ADDRESS:/, THE MALL, KANPUR,UTTAR PRADESH,INDIA,, CONTACT NUMBER: + AND EMAIL ID: KOTHARILW@HOTMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, KANPUR,/, J.S.TOWERS, THE MALL, KANPUR, UTTAR PRADESH, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAACKF. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . THERE ARE NO LINKED ACCOUNT. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE DEEPAK KOTHARI DIRECTOR- AAYPKC ,MITESH KOTHARI DIRECTOR- AFUPKM ,VIJAY KUMAR MODI DIRECTOR- ACVPMF ,SAURABH BHATNAGAR AUTHORISED SIGNATORY- AHQPBK ,MITESH KOTHARI AUTHORISED SIGNATORY- AGUPKM. BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK KOTHARIAAYPKC. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI, IEC. DATE OF LAST KYC UPDATION . ENTITY IS PRIMARILY ENGAGED IN THE INTERNATIONAL TRADING OF COMMODITIES. IT IS A PART OF KOTHARI GROUP. KPL ALSO EARNS REVENUE FROM REAL ESTATE THROUGH VARIOUS COMMERCIAL PROPERTIES OWNED BY ITS SUBSIDIARIES. THE COMPANY IS ALSO ENGAGED IN IMPORT EXPORT TRADE OF VARIOUS PRODUCTS/COMMODITIES INCLUDING AGRO BASED COMMODITIES, MINERALS, METALS, PETROLEUM PRODUCTS AND COAL, TILES, NOTEBOOK, COMPUTER STORAGE DEVICES, COMPUTER HARDWARE, ELECTRONIC COMPONENTS, TRANSFORMERS, COPIER PAPER, STEEL, SCRAP AND PVC. THE COMPANY WAS INCORPORATED IN  AND HAS ITS REGISTERED OFFICE LOCATED IN KANPUR, UTTAR PRADESH.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT  WAS RS.,. LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH  RTGS RS. ,. LAKHS MAJORLY FROM KOTHARI PRODUCTS LIMIT,KOTHARI PRODUCTS LIM,KPL EXPORTS LIMITED, REMITTANCE RS. ,. LAKHS MAJORLY FROM MUNDRA GENERAL TRADING F Z E,HIMALAYAN GENERAL TRADING FZE,MUNDRA GENERAL TRADING FZE,OFFICE NUMBER , . TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RTGS RS. ,. LAKHS MAJORLY TO KOTHARI PR,KPL EXPORT, REMITTANCE RS. ,. LAKHS MAJORLY TO KINGSTON PTE LTD,TSAI SHEN TRADING LIMITED,KINGSTON PTE LTD, , AND OTHERS RS. . LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. ,. LAKHS . TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH, REMITTANCE RS. ,. LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS TOTAL DEBIT SIDE RS. . LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E FUNDS OUTLAY OF MORE THAN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD .CRORES. FOR EXAMPLE, VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN  DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON  USD  LACS REMITTED TOTSAI SHEN TRADING LIMITED IN HONGKONGCLIENT IS INTO CERAMIC AND AGRO PRODUCTS RELATED -HTTPS://TSAISHENTRADING.COM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS  STORAGE DEVICE.",{"entities":[(88, 114,"FOREIGN ORG"),(449, 474,"FOREIGN ORG"),(789, 832,"FOREIGN ORG"),(3240, 3269,"FOREIGN ORG"),(3271, 3296,"FOREIGN ORG"),(3484, 3500,"FOREIGN ORG"),(3458, 3483,"FOREIGN ORG"),(3441, 3457,"FOREIGN ORG"),(4125, 4152,"FOREIGN ORG")]}),
            (" BITEX TECHNOLOGIES INDIA PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS : OFF  KANAKIA WALL, B -WING, STREET CHAKALA SIGNAL, ANDHERI EAST, OPP CENTRAL PARK, MUMBAI, MAHARASHTRA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: BITEXINDIA@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, SAGAR AVENUE, GROUND FLOOR, OPP. SHOPPERS STOP, S.V. ROAD, ANDHERI W, MUMBAI, MAHARASHTRA, , SINCE . DATE OF INCORPORATION IS . PAN OF THE ACCOUNT IS AAJCBN. GSTIN AS PER BANK RECORDS IS AAJCBNZA. OTHER RELATIONSHIP OF CUSTOMER ARE , , , . TRANSACTION LINKED ACCOUNT IS . BENEFICIAL OWNER OF THE REPORTED ACCOUNT ARE CHIRAG ASHVINBHAI RUPARELIYA WITH PAN NO BLTPRH, MONARK PARESHBHAI MODI AUTHORISED SIGNATORY/ DIRECTOR WITH PAN NO CLAPMM. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION, COI - AAJCBNZA. AS PER ENHANCED DUE DILIGENCE, CUSTOMER IS INTO PROVIDING SOFTWARE SERVICES RELATED TO TRADING INTO CRYPTO CURRENCIES HENCE RECOMMENDING ACCOUNT FOR CLOSURE. REFER WEBSITE WWW.BITEX.COM. BITEX TECH IN INDIA COLLECTS PAYMENTS FROM DIFFERENT USERS INTERESTED TO DEAL IN CRYPTOCURRENCY TRADING AND COLLECTS PAYMENTS FOR ACCOUNT MAINTAINED WITH KOTAK BANK AND DOES OWN REMITTANCE TRANSACTION WITH ICICI BANK. INITIALLY THEY HAVE CONFIRMED DEALING INTO SOFTWARE SERVICES BUT ON DEEPER INVESTIGATION, CAME TO KNOW OF TRADING IN CRYPTO CURRENCY. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO CR. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS. BALANCE IN THE ACCOUNT AT  WAS NIL. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. . LAKHS OF WHICH CLEARING RS. . LAKHS MAJORLY FROM BITEX TECHNOLOGIES, NEFT RS. . LAKHS MAJORLY FROM YBL SIGNET PAYMENT S, BITEX TECHNOLOGIES INDIA PVT LTD, RTGS RS. . LAKHS MAJORLY FROM BITEX TECHNOLOGIES INDUSTRIES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. . LAKHS OF WHICH RTGS RS. . LAKHS TO BITEX TECHNOLOGIES INDIA PVT LTD, REMITTANCE RS. . LAKHS MAJORLY TO NICHOLAS LEVENSTEIN AND COMPANY, TRADE BIT COMMERCIAL BROKERS LLC. TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT. FOR EXAMPLE, ON  RS. . LACS WERE CREDITED THROUGH NEFT FROM YBL SIGNET PAYMENTS AND SAME WAS DEBITED THROUGH RTGS TO BITEX TECHNOLOGIES INDIA PVT LTD , YESB ON . ALSO ON  RS. . WERE CREDITED THROUGH NEFT FROM YBL SIGNET PAYMENTS , YESB AND ON THE NEXT DAY SAME AMOUNT WAS DEBITED THROUGH RTGS TO SIGNET PAYMENT SOLUTIONS PRIVATE LIMITED , AUBL. ROUTING OF FUNDS OBSERVED. SIMILAR PATTERN WERE OBSERVED THROUGHOUT THE PERIOD. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS WHERE ROUTING OF FUNDS OBSERVED, STR IS BEING FILED.",{"entities":[(2161, 2192,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN SHOWS, FUNDS ARE RECEIVED FROM DIFFERENT ENTITIES AND ARE IMMEDIATELY DEBITED TOWARDS OUTWARD REMITTANCE. FOR EG, ON  ACCOUNT WAS CREDITED WITH RS. . CRORES IN PARTS THROUGH RTGS FROM SHREEJI JEWELLERS ACCOUNT NO: , IFSC CODE: UTIB, BANK: AXIS BANK AND THE SAME IS DEBITED IN PARTS THROUGH RTGS TO DIAMAX ACCOUNT NO: , IFSC CODE: INDB, BANK: INDUSIND BANK LTD. SIMILARLY, ON  ACCOUNT WAS CREDITED WITH RS. . CRORES IN PARTS THROUGH RTGS FROM DIAMAX ACCOUNT NO: , IFSC CODE: HDFC, PURVI EXPORTS ACCOUNT NO: , IFSC CODE: KKBK, VANSH GEMS, ACCOUNT NO: , IFSC CODE: KKBK AND ON THE SAME DATE ACCOUNT WAS DEBITED WITH RS. . CRORES APPROX. IN PARTS THROUGH REMITTANCE TO MARS VENTURE LIMITED, HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT . FUNDS WERE CREDITED IN THE ACCOUNT FROM MAHAPUJA PRODUCTS LTD, NYSNA MULTIVENTURES PVT LTD, SHREEJI JEWELLERS ETC AND MAJORITY REMITTANCES MADE TO MARS VENTURE LIMITED SINCE ACCOUNT OPENING. OVERSEAS ENTITIES EXISTENCE CHECKED HOWEVER NO FURTHER INFORMATION AVAILABLE ON THEM REGARDING WHAT PRODUCT THEY SELL. NO BUSINESS RELATED PAYMENTS ARE MADE FROM THIS ACCOUNT. IN VIEW OF LARGE VALUE FOREX DEBIT DAILY AND NEGATIVE FINDINGS, STR IS BEING FILED.DIAMAX, A PROPRIETORSHIP FIRM, HAVING ADDRESS:/, DAIMOND APPARTMENT, SAIYEDPURA, NAGORIVAD, SURAT, GUJARAT, INDIA, , CONTACT NUMBER: + AND EMAIL ID: DIAMAX@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, SHUBHAM POINT, RUWALA TEKRA, HARIPURA BAGAL, LAL GATE, SURAT GUJARAT, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS FOJPSR. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . GSTIN AS PER BANK RECORDS IS FOJPSRZQ. THERE ARE NO PAN LINKED ACCOUNTS. TRANSACTION LINKED ACCOUNT IS . AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS DHRUMIL DHARNENDRA SHAH PAN- FOJPSR. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN: FOJPSR, AADHAAR OF PROPRIETOR, IMPORT EXPORT CERTIFICATE AND GST REGISTRATION CERTIFICATE. CUSTOMER IS AN IMPORTER OF LAB GROWN THAT IS SYNTHETIC DIAMONDS AND FURTHER HE IS SELLING IN LOCAL MARKET. AS PER ENHANCED DUE DILIGENCE EDD, NAME BOARD AFFIXED OUTSIDE OFFICE WHICH IS PRINTED ON A SIZE PAPER. ALSO THERE ARE CREDITS FROM NYSNA MULTIVENTRES PVT LTD SEEN AND IS ONE SUCH COMPANY WHICH HAS TRANSACTED WITH MANY SUSPICIOUS DIAMOND COMPANIES OF SURAT AND WHOSE ACCOUNTS ARE CLOSED UNDER BANK INDUCED PROCESS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS.  CRORES TO LESS THAN RS.  CRORES. ALERT WAS TRIGGERED FOR LARGE VALUE FOREX DEBIT DAILY. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH CLEARING RS. . LAKHS MAJORLY FROM RAHIL GEM, NEFT RS. . LAKHS MAJORLY FROM SAHAJ DIAM, INTEGRITY JEWELS, RTGS RS. ,. LAKHS MAJORLY FROM DIAMAX, VANSH GEMS, SHREEJI JEWELLERS, MOBILE MONEY TRANSFERS RS. . LAKHS. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH RTGS RS. . LAKHS MAJORLY TO PARI IMPEX, RADHE KRISHNA IMPEX, DIAMAX, REMITTANCE RS. ,. LAKHS MAJORLY TO MARS VENTURE LIMITED, HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT , AARAS DIAMONDS LLC, ALICAT SCIENTIFIC INC IN USD TO UNITED STATES FOR ADV PAYMENT AGAINST IMPORT OTHER THAN NEPAL BHUTAN , NEFT RS. . LAKHS TO MAYAN EXPORTS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CLEARING RS. . LAKHS FROM DIAMAX, NEFT RS. . LAKHS FROM SHREE RAM DIAMOND, RTGS RS. ,. LAKHS FROM NYSNA MULTIVENTURES PVT LTD, SHREEJI JEWELLERS, INTERNET FUND TRANSFERS RS. . LAKHS FROM MAHAPUJA PRODUCTS LTD, MOBILE MONEY TRANSFERS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS TO SURAT BHAGAL BRANCH, RTGS RS. . LAKHS, REMITTANCE RS. ,. LAKHS TO MARS VENTURE LIMITED, HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT .",{"entities":[(928, 948,"FOREIGN ORG"),(685,705 ,"FOREIGN ORG") ,(3336, 3354,"FOREIGN ORG"),(3356, 3380,"FOREIGN ORG")]}),
            ("ROUTING OF LARGE VOLUME OF FUNDS WERE OBSERVED IN ACCOUNT LEADING TO HIGH TURNOVER WHICH SEEMS UNUSUAL. IN VIEW OF, LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS, STR IS BEING FILED.WANDERLAND REAL ESTATE PRIVATE LIMITED, SMEAG-PRIVATE LTD COMPANY , HAVING ADDRESS:TH FLR TREASURE ISLAND, TUKOGANJ MAIN,RD, INDORE, MADHYAPRADESH, INDIA,  ,  CONTACT NUMBER: + IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, INDORE - MALAV PARISAR, ICICI BANK LTD, , ,CHHOTI KHAJRANI, MALAV PARISAR, INDORE, INDORE, MADHYA PRADESH, ,  SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAACWF.  OTHER RELATIONSHIP OF CUSTOMER ARE , , , , , . TRANSACTION LINKED ACCOUNTS ARE  ,. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VINAYAK KALANI AUTHORISED SIGNATORY- ATGPKP ,QURESH YUSUF MATKAWALA AUTHORISED SIGNATORY- AGSPMA ,YOGESH GOSWAMI AUTHORISED SIGNATORY- ACFPGK ,NIRMAL KUMAR SHRIVASTAVA DIRECTOR, KAPIL GOPLANI DIRECTOR- BSEPGN. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN - AAACWF, MOA-AOA, BOARD RESOLUTION ETC.  KYC UPDATION IS NOT DUE. AS PER ENHANCED DUE DILIGENCE:  CUSTOMER DEALS IN EXPORT OF PP BAGS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO LESS THAN CR.  ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS  CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL  YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH TRANSFERS RS. . LAKH MAJORLY FROM WANDERLAND REAL ESTATES PVT LTD,  CLEARING RS. . LAKHS MAJORLY FROM NEFT,WANDERLAND REAL ESTATE PRIVATE,CMS_,  NEFT RS. . LAKHS MAJORLY FROM TATWA PLASTCO EXTRUSIONS PVT,MAERSK LINE INDIA PRIVATE LI,  REMITTANCE RS. ,. LAKHS MAJORLY FROM FLEXI GLOBAL PTE LTD A BE,PANKTI INTERNATIONAL PTE  LTD,DHAMPUR INTERNATIONAL PTE  LT,  INTERNET FUND TRANSFERS RS. . LAKHS, , AND OTHERS RS. . LAKHS . TOTAL DEBITS DURING THE SAME PERIOD WAS RS.  ,. LAKHS OF WHICH ,  TRANSFERS RS. . LAKHS MAJORLY TO WANDERLAND REAL ESTATE PRIVATE LIMITED,  CLEARING RS. ,. LAKHS,  RTGS RS. ,. LAKHS MAJORLY TO FLEXITUFF VENTURES IND,FLEXITUFF VENTURES INT,SBIN,  REMITTANCE RS. . LAKHS, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES , , AND OTHERS RS. . LAKHS .FURTHER, THE QUANTUM OF    TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE:    FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. TRANSACTION PATTERN SHOWS FUNDS WERE CREDITED THROUGH INWARD REMITTANCES AND THOSE FUNDS WERE DEPLOYED TOWARDS OTHER ENTITIES. FOR EXAMPLE, ON , RS. .LAKHS CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE. LTD.  SHENTON WAY SHENTON HOUSE  - SINGAPORE WHERE THE PURPOSE MENTIONED AS PAYMENT OF IMPORT BILL  AND ON , RS.  LAKHS DEBITED IN PARTS THROUGH RTGS TO FLEXITUFF VENTURES INTERNATIONAL LTD  ACCOUNT NO.  IFSC SBIN.  ALSO ON , RS. LAKHS CREDITED THROUGH TRANSFER FROM NANOFIL TECHNOLOGIES PRIVATE LIMITED WHERE THOSE FUNDS WERE CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE. LTD.  SHENTON WAY SHENTON HOUSE - SINGAPORE WHERE THE PURPOSE MENTIONED AS EXPORT BILLS REALN COLLN AND ON , RS. .CRORES DEBITED IN PARTS THROUGH CMS TRANSFER AS PAYMENTS. ALSO ON , RS. . LACS WERE CREDITED BY REMITTANCE FROM FLEXI GLOBAL PTE LTD A BE, SINGAPORE AND ON SAME DAY ENTIRE AMOUNT WAS DEBITED IN  PARTS BY RTGS TOWARDS FLEXITUFF VENTURES INTERNATIONAL LTD ACCOUNT NO.  IFSC SBIN.",{"entities":[(1767, 1795,"FOREIGN ORG"),(1890, 1919,"FOREIGN ORG"),(1920, 1949,"FOREIGN ORG"),(2209, 2231,"FOREIGN ORG"),(2232, 2254,"FOREIGN ORG"),(2803, 2828,"FOREIGN ORG") ,(3449, 3471,"FOREIGN ORG"),(3554, 3590,"FOREIGN ORG")]}),
            ("BHANU VYAPAAR PVT LTD, A PRIVATE LTD COMPANY, HAVING ADDRESS  ANANDAPUR, E.M. BYPASS TH FLR, KOLKATA, WEST BENGAL, INDIA, , CONTACT NUMBER: + AND EMAIL ID: RAJESHAGARWAL@EMANIGROUP.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, , SIR R.N. MUKHERJEE ROAD, KOLKATA, WEST BENGAL, , SINCE . DATE OF INCORPORATION IS . RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT. PAN NUMBER OF THE ACCOUNT IS AABCBQ. OTHER RELATIONSHIP OF CUSTOMER ARE , , , , , , , . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE MOHAN GOENKA PTR- ADZPGG, MOHAN GOENKA AUTHORISED SIGNATORY- ADZPGG, MANISH GOENKA DIRECTOR- ADXPGB, MANISH GOENKA AUTHORISED SIGNATORY- ADXPGB, RADHE SHYAM GOENKA DIRECTOR- AEGPGB. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN - AABCBQ, MOHAN GOENKA ADZPGG, MANISH GOENKA ADXPGB, RADHE SHYAM GOENKA AEGPGB, COI- UWBPTC, MOA, AOA, BOARD RESOLUTION. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY FROM PREMIER FERRO ALLOYS AND SECURITIES LTD, EMAMI REALTY LIMITED, EMAMI AGROTECH LIMITED, CLEARING RS. . LAKHS MAJORLY FROM CITIBANK DD, NEFT RS. ,. LAKHS MAJORLY FROM BHANU VYAPAAR PRIVATE LIMITE, PYRITE POWER PRIVATE LIMITED, EMAMI FRANK ROSS LTD, RTGS RS. ,. LAKHS MAJORLY FROM AMRI HOSPITALS LTD, IIFL WEALTH FINANCE LIMIT, DEUTSCHE INVESTMENTS, INTERNET FUND TRANSFERS RS. ,. LAKHS FROM RAVIRAJ VINIYOG PRIVATE LIMITED. TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY TO MIDKOT INVESTMENTS PRIVATE LIMITED, BENGAL NRI COMPLEX LTD, EMAMI CAPITAL MARKETS LIMITED, RTGS RS. ,. LAKHS MAJORLY TO EMAMI REALTY LIMITED, AMRI HOSPITALS LTD, DEUTSCHE I, INTERNET FUND TRANSFER RS. ,. LAKHS TO EMAMI REALTY LIMITED. FY - FROM  TO : TOTAL CREDIT SIDE RS. ,,. LAKHS TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH REMITTANCE RS. . LAKHS. FY - FROM  TO : TOTAL CREDIT SIDE RS. ,,. LAKHS TOTAL DEBIT SIDE RS. ,,. LAKHS. FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS.  THIS IS THE FIRST REPORTING OF THE CUSTOMER. CUSTOMER MANUFACTURES BEVERAGES. THE COMPANY OFFERS FOOD PRODUCTS. CONSUMER STAPLES. FREQUENT HIGH VALUE CREDITS FOLLOWED BY DEBITS SEEMS LIKE ROUTING OF FUNDS AMONG MULTIPLE INDIVIDUALS AND ENTITIES. FOR EXAMPLE, ON , RS. . LACS CREDITED THROUGH RTGS NAME EMAMI REALTY LTD ACCOUNT NO.  IFSC KKBK AND ON SAME DAY, RS. . CRS DEBITED THROUGH RTGS NAME RADHE SHYAM GOENKA ACCOUNT NO.  IFSC KKBK. IN OTHER EXAMPLE, ON , RS. . CRS CREDITED BY TRANSFERS FROM -RAVIRAJ VINIYOG PRIVATE LIMITED AND ON SAME DAY, RS. . CRS DEBITED BY TRANSFERS TO -EMAMI REALTY LIMITED. SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED LIKE ROUTING OF FUNDS. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT, ROUTING OF FUNDS, STR IS BEING FILED.",{"entities":[(1767, 1795,"FOREIGN ORG")]}),
            ("SHIPMENT MADE EXPORT BILLS ARE PENDING FOR THE SUBMISSION OUT OF WHICH  BILLS WERE OVERDUE AND  IS IN CAUTIONED LISTED FOR MORE THAN TWO YEARS SEEMS SUSPICIOUS. IN VIEW OF BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE, STR IS BEING REPORTED.KGR RIGS AND MINING EQUIPMENT, PARTNERSHIP, HAVING ADDRESS: PLOTNO:/ /, IDA PHASE-II, CHERLAPALLY, HCL POST, RANGAREDDI, HYDERABAD, TELANGANA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: GOVINDREDDY_K@YAHOO.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, PLOT NO. , H.NO. /, SUBHASH NAGAR, OPP JUNA NAGAR, KUSHAIGUDA, KAPRA, RANGAREDDY DIST., HYDERABAD, TELANGANA, , SINCE . DATE OF INCORPORATION IS . NA. PAN NUMBER OF THE ACCOUNT IS AAHFKN. OTHER RELATIONSHIP OF CUSTOMER ARE , ,  AND . TRANSACTION LINKED ACCOUNTS ARE  AND . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE GOVINDAREDDY KATIREDDY PARTNER- ADFPKK AND KATIREDDY NITHIN REDDY PARTNER- DECPKF. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PARTNERSHIP DEED AND PAN. DATE OF LAST KYC UPDATION . KYC UPDATION IS NOT DUE.ENTITY IS A MANUFACTURER AND TRADER OF DRILLING EQUIPMENT SUCH AS DRILLING RIGS, DTH HAMMERS, BUTTON BITS ETC. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS.  CRORES TO LESS THAN  CRORES.ALERT WAS TRIGGERED FOR MANUAL STR FOR BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS DEBIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH CASH WAS RS.  LAKHS, TRANSFERS RS. . LAKHS MAJORLY FROM KGR RIGS MINING EQUIPMENT, KGR INDUSTRIES, KATIREDDY GOVINDA REDDY, CLEARING RS. . LAKHS MAJORLY FROM BIMIDI MAHENDER, LAXMI NARASIMHA RAO, NEFT RS. . LAKHS MAJORLY FROM COMINT SYSTEMS AND SOLUTIONS, E-PAO GST REFUNDS, MASTER DRILLERS, RTGS RS. ,. LAKHS MAJORLY FROM E PAO GST REFUNDS, BHAVIKA PLAS TEK PRIVA, NANO POLYMERS INDIA PVT LTD, REMITTANCE RS. ,. LAKHS MAJORLY FROM BAKORI MEGA SERVICES, ROYAL TECHNO INDUSTRIES LIMITED, PROACTIVE ENERGY TRADING , INTERNET FUND TRANSFERS RS. . LAKHS FROM KGR INDUCTION DRILL PIPES COMPANY , MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES, UPI TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY KAPRA BRAN CH, TRANSFERS RS. ,. LAKHS MAJORLY TO SEKO BEC PRIVATE LIMITED, KGR RIGS MINING EQUIPMENT, G-ROC MINING AND CONSTRUCTION PRIVATE LIMITED, CLEARING RS. . LAKHS MAJORLY TO ELGI EQUIPMENTS LIMITED, TSSPDCL, NEFT RS. . LAKHS MAJORLY TO OSMAN PIPES AND TUBE, RTGS RS. ,. LAKHS MAJORLY TO ELGI EQUIPMENTS LTD, ASHOK LEYLAND LIMITED, REMITTANCE RS. . LAKHS, INTERNET FUND TRANSFER RS. ,. LAKHS TO SOUTHERN STEELS FORGINGS , MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES, AND OTHERS RS. . LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS WERE: FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL. FY - FROM  TO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS.TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS.AS PER THE TRANSACTION PATTERN, FUNDS ARE GETTING CREDITED FROM DOMESTIC AND INTERNATIONAL ENTITIES AND THOSE FUNDS ARE GETTING DEBITED TO CUSTOMER OTHER BANK ACCOUNT, OD ACCOUNT IS WITH US AND VARIOUS SUPPLIERS, VENDORS AND OPERATIONAL EXPENSES ETC SEEMS INLINE. HOWEVER, CUSTOMER IS HAVING  ADVANCES RECEIVED, THERE ARE EXPORT BILLS ARE PENDING FOR SUBMISSION OUT OF WHICH  BILLS ARE OVERDUE .",{"entities":[(2087, 2118,"FOREIGN ORG"),(1716, 1730,"FOREIGN ORG"),(2065, 2085,"FOREIGN ORG"),(2582, 2605,"FOREIGN ORG"),(2188, 2221,"FOREIGN ORG")]}),
            ("ON  FUNDS OF RS  LACS APPROX CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM SUYOG SYNTHETICSACC NO:, IFSC :UTIB AND DEPLOYED THROUGH RTGS TOWARDS SHREE BALAJI CREATIONACC NO:, IFSC: KKBK SEEMS UNUSUAL, RATIONALE FOR SUCH ROUTING OF FUNDS ARE NOT CLEAR. ACCOUNT TURN OVER IS TOO HIGH MORE THAN  CRORES IS SEEN IN ACCOUNT. IN VIEW OF SUSPICIOUS LARGE VALUE NON CASH DEPOSIT TRANSACTIONS AND SUSPECTED ROUTING OF FUNDS, STR IS BEING REPORTED.SUYOG SYNTHETICS, PROPRIETOR, HAVING ADDRESS: RAGHUVEER BUSINESS,EMPIRE,AAI MATA CHOWK, PARVAT PATIYA, SURAT, GUJARAT, INDIA,  ,  CONTACT NUMBER: + AND EMAIL ID: JAGDISHGARG@TRIBENIGROUP.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, SURAT - RING ROAD, ICICI BANK LTD SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAPPGJ. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . PAN LINKED ACCOUNTS ARE  AND . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE JAGDISHPRASAD RAMKISHAN GARG PROPREITOR- AAPPGJ. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN CARD-AAPPGJ, GSTIN-AAPPGJZE, IEC- AND AADHAR CARD. DATE OF LAST KYC UPDATION . HENCE, KYC UPDATION IS NOT DUE. CUSTOMER IS INTO IMPORTING AND TRADING OF YARN. ANNUAL TURN OVER DECLARED BY CUSTOMER IS RS  LAC TO  CRORE. ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS. BALANCE IN THE ACCOUNT AT  WAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY - FROM  TILL  SHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS MAJORLY IN JUHU , SURAT TEXTILE MARKET BRANCH,  TRANSFERS RS. . LAKHS MAJORLY FROM RAJ TEXTILE, SHREE GANESH TEXTILE,TRIBENI WORLDWIDE PVT LTD,  CLEARING RS. . LAKHS MAJORLY FROM SUMILON, BHATHIJI SILK MILLS,GEETA RAYON,  NEFT RS. . LAKHS MAJORLY FROM PADMAVATI INTERNATIONAL ,GURUKRUPA RAYON, GURU FIBERS,  RTGS RS. ,. LAKHS MAJORLY FROM SUYOG SYNTHETICS, R R TRADING CO, SAKSHI YARNS PVT LTD,  REMITTANCE RS. . LAKHS MAJORLY FROM INDORAMA SYNTHETICS TBK PT PP,UNITED RAW MATERIAL PTE. LTD,  INTERNET FUND TRANSFERS RS. , . LAKHS,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS.  ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BR ANCH,  TRANSFERS RS. . LAKHS MAJORLY TO SINGHAL BROTHERS, S K KABRA AND COMPANY,  CLEARING RS. . LAKHS MAJORLY TO MONIT TRADING PRIVATE LIM,S R GROUP, KAYAVLON IMPEX PRIVATE LIMITED,  RTGS RS. ,. LAKHS MAJORLY TO DEEP SHIPPING AGENCY,SUYOG AXIS CC, NIRANJAN SHIPPING AGENCY,  REMITTANCE RS. ,. LAKHS MAJORLY TO F-TEX ASIA INTERNATIONAL LIMITED, PT INDORAMA POLYCHEM INDONESIA, INTERNET FUND TRANSFER RS.  ,. LAKHS,  CARD TRANSACTIONS RS. . LAKHS,  MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES AND OTHERS RS. . LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS .FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. ON ANALYZING THE TRANSACTION PATTERN, IT HAS BEEN NOTICED THAT THERE ARE HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS. FOR EXAMPLE ON  FUNDS OF RS  LACS APPROX CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM M/S ULTIMATEACC NO:, IFSC : SCBL, NEFT FROM SUYOG SYNTHETICSACC NO:, IFSC :UTIB AND DEPLOYED THROUGH TRANSFER TOWARDS TRIBENI WORLDWIDE PVT LTD  AND REMITTANCE TOWARDS F TEX ASIA INTERNATIONAL LIMITED , CHINA FOR ADV PAYMENT AGAINST IMP OTR THAN NEPL BHUT AN AND",{"entities":[(2720, 2750,"FOREIGN ORG") ,(2134, 2163,"FOREIGN ORG"),(2164, 2188,"FOREIGN ORG")]}),
            ("ACCOUNT OPENED ON , CURRENT STATUS OF THE ACCOUNT IS .TRANSACTIONS NOTICED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES, CHEQUE DEPOSITS, IMPS CREDITS, CASH DEPOSITS, ETC, CREDITS IN THE ACCOUNT VIA VARIOUS MODES ARE UTILISED VIA RTGS/NEFT TO LOCAL PARTIES IMMEDIATELY. BANK FILED STR ON  FIU BATCH ID  . .RELATED ACCOUNTS ARE VALIDATED VIA UCIC  & DIRECTOR MR. SUSHIL JAIN IS ALSO PROPRIETOR IN THE FIRMS SHANTI OVERSEAS  & SUNSHINE ENTERPRISES . DIRECTOR MR.RISHABH JAIN IS ONE OF THE PROPRIETOR/DIRECTOR IN THE BULK CIRCLE LLP , SHALINI TRADEWELL INDIA PVT LTDINR, GBP,EUR,  USD & SHALINI ENTERPRISES . RELATED ACCOUNT DETAILS: .SHANTI OVERSEAS  OPENED ACCOUNT WITH US ON .CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS .TOTAL CREDIT TURNOVER AS ON DATE IN THE ACCOUNT IS RS..CRORES,AVAILABLE BALANCE IN THE ACCOUNT IS RS.,..TRANSACTIONS NOTICED ARE HIGH VALUE CREDITS FROM E PAO GST REFUNDS,DUTY DBK DISBURSAL ACCOUNT & RTGS/NEFT FROM LOCAL PARTIES, INWARD REMITTANCES ARE UTILISED IMMEDIATELY. BANK FILED STR ON  VIA FIU BATCH ID . .SUNSHINE ENTERPRISES , ACCOUNT OPENED ON .CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS .AVAILABLE BALANCE IN THE ACCOUNT IS RS.. CREDIT. BANK FILED STR ON  VIA FIU BATCH ID . .BULK CIRCLE LLP , ACCOUNT OPENED ON ,CURRENT STATUS OF THE ACCOUNT CLOSED ON . .SHALINI TRADEWELL INDIA PVT LTDINR,  GBP, EUR, USD,ACCOUNT OPENED ON ,CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS .AS ON  TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RS..CRORES & AVAILABLE BALANCE IN THE ACCOUNT IS RS.,. CREDIT.TRANSACTIONS OBSERVED ARE HIGH VALUE CREDITS VIA E PAO GST REFUNDS & INWARD REMITTANCES UTILISED TO MAKE IMMEDIATELY RTGS TO LOCAL PARTIES, DESPITE OF VOLUMES CLIENT IS HOLDING VERY MINIMAL BALANCE IN THE ACCOUNT, ACCOUNT WAS UTILISED TO ROUTING FUNDS IMMEDIATELY TO OTHER LOCAL PARTIES & KEEPING MINIMAL BALANCE IN THE ACCOUNT BRINGS SUSPICION IN THE ACCOUNT PATTERN. .SHALINI ENTERPRISES , ACCOUNT OPENED WITH US ON .CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS .AS ON  TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RS..CRORES & AVAILABLE BALANCE IN THE ACCOUNT IS RS.. CREDIT. BANK FILED STR ON  VIA FIU BATCH ID . HIGH VALUE CASH DEPOSITS AMOUNTING TO INR . CRORES WERE EVIDENCED SINCE TH NOV  IN THE ACCOUNT. NO OTHER CREDITS OTHER THAN CASH DEPOSITS WERE EVIDENCED IN THE ACCOUNT. TURNOVER IN THE ACCOUNT HAS SURPASSED THE DECLARED TURNOVER. .MULTIPLE CURRENT ACCOUNTS ARE OPENED TOWARDS ROUTING FUNDS. .CREDITS RECEIVED IN THE ACCOUNT VIA INWARD REMITTANCES ARE UTILISED TO MAKE LOCAL PARTIES PAYMENTS VIA RTGS/NEFT. SAME ACCOUNT PATTERN EVIDENCED IN THE ALL THE RELATED ACCOUNTS. HIGH TURNOVER IN THE ACCOUNT HOWEVER VERY MINIMAL BALANCE MAINTAINED IN THE ACCOUNT. SUSPICION RAISED DUE TO BELOW MENTIONED RED FLAGS. .UNABLE TO ASCERTAIN OVERSEAS PARTY PROFILE THROUGH WEB SEARCH. .MAJORITY OF INWARD REMITTANCES TOWARDS EXPORT OF MOBILE PHONES RECEIVED FROM LOGISTICS FIRMS. .IMMEDIATE UTILISATION OF HIGH VALUE RTGS/NEFT CREDITS, INWARD REMITTANCE PROCEEDS TO MAKE RTGS/NEFT TO LOCAL PARTIES. . HIGH CREDIT TURNOVER IN A NEWLY INCORPORATED ACCOUNT. . MULTIPLE CURRENT ACCOUNTS ARE OPENED FOR ROUTING FUNDS FOR MAKING HIGH VALUE TRANSACTIONS. . DESPITE HIGH CREDIT TURNOVER IN RELATED ACCOUNT VERY MINIMAL BALANCE MAINTAINED. . FUNDS TRANSFER BETWEEN STR FILED ACCOUNTS. .STR FILED FOR THE RELATED ACCOUNTS TOWARDS SERIOUS RED FLAGS.CLIENT HAS BEEN REVIEWED FROM AML PERSPECTIVE BASIS HIGH VALUE INWARD REMITTANCES & OUR OBSERVATIONS ARE: .M/S AVIK TELEVENTURES PVT LTD IS A PVT LTD FIRM INCORPORATED ON  AT FIRST FLOOR A ,NARAINA INDUSTRIAL AREA PHASE , SOUTH WEST DELHI INDIA . CLIENT OPENED CURRENT ACCOUNT  WITH US ON  AT CHANDANI CHOWK BRANCH, NEW DELHI INDUSIND BANK LTD, MUNICIPAL NO - TO , LAXMI NIWAS BUILDING, CHANDNI CHOWK, DELHI  . AS ON , STATUS OF ACCOUNT IS ACTIVE, CUMULATIVE DEBIT IS RS. ,,,. & CUMULATIVE CREDIT IS RS. ,,,. & AVAILABLE BALANCE IS RS. ,,. CREDIT. LAST CUSTOMER INDUCED TRANSACTION DATE, AT THE TIME OF INVESTIGATION, IS . NATURE OF BUSINESS IS IMPORT EXPORT OF TELECOMMUNICATION ITEMS, MOBILE/TELEPHONE ITEMS/ELECTRICAL/ELECTRONIC ITEMS. DECLARED TURNOVER IS ABOVE RS.CRORES PER ANNUM. DIRECTORS - MR. SAHIL JAIN MOBILE: & MR.RISHABH JAINMOBILE:. PAN & IEC IS AATCAH. RISK CATEGORY OF THE CLIENT IS HIGH & LAST KYC DONE ON . DOCUMENTS OBTAINED DURING ACCOUNT OPENING ARE PAN, BOARD RESOLUTION, CERTIFICATE OF INCORPORATION, ASSOCIATION MEMO ARTICLE, GST REGISTRATION FORM. GSTIN IS AATCAHZC & CURRENT STATUS IS ACTIVE AS PER GST SITE. .AS ON , CLIENT HAD DONE  DIRECT/ADVANCE IMPORTS FOR RS..LAKHS TOWARDS IMPORT OF OXYMETER, GAS SENSOR, OUTDOOR PM MONITOR, POLLUTION MONITOR, SQUAIR POLLUTION MONITOR,CAIR MONITOR TO OVERSEAS PARTIES RAINBOW COMMUNICATION CO & SHAANXI BOHU ZHICHUANG ELECTRONIC TECHNOLOGY CO BASED IN CHINA & HONGKONG. UNABLE AS FETCH OVERSEAS PARTIES PROFILE THROUGH WEB SEARCH. DURING THE SAME PERIOD CLIENT HAS RECEIVED  EXPORT ADVANCE FOR RS..CRORES TOWARDS EXPORT OF MOBILE PHONES FROM THE OVERSEAS PARTIES BASED IN UAE & UNITED STATES.MAJORITY OF REMITTANCES ARE RECEIVED FROM LOGISTICS FIRMS, RANDOM CHECK DONE ON ONE OF THE OVERSEAS PARTY PROFILE OF LOGIX ONE FZE, & PARTY IS INTO LOGISTICS SERVICES HTTPS://CONNECT.AE/PROFILES/DUBAI/LOGIX-ONE-FZE/PROFILE-ID=ER. . AS ON , CLIENT HAS ALL BOE OUTSTANDING AND  EXPORT ADVANCE OUTSTANDING..TOTAL CREDIT TURNOVER BETWEEN  TO  IS RS..CRORES WHICH INCLUDES TOTAL INWARD REMITTANCES RECEIVED FOR RS..CRORES, OTHER CREDITS DETAILS ARE: A.TOTAL RTGS CREDITS RECEIVED FOR RS..CRORES, OF WHICH RS..CRORES RECEIVED FROM E PAO GST REFUNDS, ACCOUNT WITH STATE BANK OF INDIA, ACCOUNT NO  & RS..LAKHS RECEIVED FROM INDIVIDUAL ACCOUNT RAJ KUMAR SANJEEV KUM ACCOUNT WITH BANDHAN BANK LIMITED, ACCOUNT NO . B.MAJORITY OF INWARD REMITTANCES ARE RECEIVED FROM LOGISTICS FIRM.. TOTAL DEBIT TURNOVER BETWEEN  TO  IS RS..CRORES: A. RS..CRORES VIA RTGS/NEFT,OF WHICH MAJORLY SENT TO CLIENT OWN ACCOUNT FOR RS..CRORES, HELD ACCOUNT WITH AU SMALL FINANCE BANK LIMITED, RS..CRORES TO BLUE BERRY, ACCOUNT WITH ICICI BANK & RS..CRORES TO HEMANG ELECTRONICS ACCOUNT WITH PUNJAB NATIONAL BANK & LAXMI VILAS BANK . B. TOTAL TRANSFER DEBIT FOR RS..CRORES OF WHICH MAJORLY SENT TO YASHOM INTERNATIONAL TRADERS PVT LTDACCOUNT NO  FOR RS..LAKHS & RS..LAKHS TO JAY JALARAM ENTERPRISE ACCOUNT NO . YASHOM INTERNATIONAL TRADERS PVT LTD, ACCOUNT OPENED ON . STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS . TOTAL CREDIT BALANCE IN THE ACCOUNT IS RS..CRORES.AVAILABLE BALANCE AS ON DATE IS RS.,. CREDIT.TRANSACTIONS OBSERVED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES UTILISED VIA RTGS/NEFT TO LOCAL PARTIES IMMEDIATELY, DESPITE OF HIGH VOLUMES CLIENT HOLDING MINIMAL BALANCE IN THE ACCOUNT, BRINGS MORE SUSPICION IN THE ACCOUNT PATTERN. UNABLE TO ASCERTAIN RELATIONSHIP BETWEEN OUR CLIENT & YASHOM INTERNATIONAL TRADERS PVT LTD. C. JAY JALARAM ENTERPRISEACCOUNT NO ",{"entities":[(4822, 4871,"FOREIGN ORG"),(5263, 5276,"FOREIGN ORG") ]}),
            ("BHARAT TRADING CO, A PROPRIETORSHIP FIRM, HAVING ADDRESS : NEW COLONY HOUSE NO A , PANCHWATI, BLOCK A, GHAZIABAD, UTTAR PRADESH, INDIA, , CONTACT NUMBER: + AND EMAIL ID: CANNYCOCFA@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, R/, RAJ NAGAR, GHAZIABAD, UTTAR PRADESH, , SINCE . DATE OF INCORPORATION IS . PAN OF THE ACCOUNT IS BHIPAE. GSTIN AS PER BANK RECORDS IS BHIPAEZO. OTHER RELATIONSHIP OF CUSTOMER IS . TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS AMAR AGARWAL PROPRIETOR - BHIPAE. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN- BHIPAE, GSTIN: BHIPAEZO. KYC UPDATION IS NOT DUE. THE CUSTOMER IS INVOLVE IN BAMBOO STICKS TRADING BUSINESS. ANNUAL TURN OVER DECLARED BY CUSTOMER IS LESS THAN  LACS. ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS. BALANCE IN THE ACCOUNT AT  WAS RS. . LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS OF WHICH NEFT RS. . LAKHS MAJORLY FROM RAJA TRADERS PROP MOHAMMAD R, GUPTA TRADERS, SHUBHAM KUMAR, RTGS RS. . LAKHS MAJORLY FROM SHUBHAM KUMAR, BHARAT TRADING CO, DHARISH ENTERPRISES, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH RTGS RS. . LAKHS MAJORLY TO MERINOAIR CONTAINER LIMITED, BHARAT TRADING CO, REMITTANCE RS. . LAKHS MAJORLY TO CV AGUNG MAKMUR JAYA, CV DUNIA GLOBAL NUSANTARA, PT NUANSA DUA WARNA, INTERNET FUND TRANSFER RS. . LAKHS TO MANOJ. FY - FROM  TO  : TOTAL CREDIT SIDE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS OF WHICH REMITTANCE RS. . LAKHS. HIGH VALUE CREDITS FOLLOWED BY IMMEDIATE DEBITS ARE SEEN IN ACCOUNT WHICH SEEMS UNUSUAL. FOR EXAMPLE ON  FUNDS OF RS . LACS CREDITED IN THE ACCOUNT THROUGH RTGS FROM BHARAT TRADING CO ACC NO: , IFSC: BARBUTTAMX AND ON THE SAME DAY RS . LACS WERE DEPLOYED THROUGH REMITTANCE TOWARDS PT NUANSA DUA WARNA, INDONESIA IN USD FOR BAMBOOS FOR ADV PAYMENT AGAINST IMP OTR THAN NEPL BHUTAN. ALSO ON  RS  LACS WERE CREDITED THROUGH RTGS FROM BHARAT TRADING COMPANY AND ON THE SAME DAY RS . LACS WERE DEBITED THROUGH OUTWARD REMITTANCE TO PT NUANSA DUA WARNA, INDONESIA IN USD FOR ADVANCE PAYMENT AGAINST IMPORT. AS PRE EDD VISIT FINDINGS, IT WAS RESIDENTIAL ADDRESS. CUSTOMER ALSO NOT HAVING ANY OTHER OFFICE. NO BUSINESS SETUP, BUSINESS ACTIVITY, STAFFS , STOCKS SIGN BOARD WAS SEEN. TRIGGERED TRANSACTIONS ARE SAME LINE OF BUSINESS HOWEVER VOLUME OF TRANSACTIONS ARE NOT MATCHING WITH BUSINESS SETUP, HUGE TRANSACTIONS OBSERVED IN ACCOUNT AND IMMEDIATE DEBIT SEEMS SUSPICIOUS. IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT AND NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS WITH NEGATIVE EDD FINDINGS, STR IS BEING REPORTED.",{"entities":[(1645, 1663,"FOREIGN ORG"),(2348, 2367,"FOREIGN ORG"),(2102, 2121,"FOREIGN ORG")]}),
            ("SRI SNJ JEWELS, A PROPRIETORSHIP FIRM, HAVING ADDRESS : , OLD NO  SHOP NO /, RD FLR, NSC BOSE ROAD, SOWCARPET, TARUS TOWER, CHENNAI, TAMIL NADU, INDIA, , CONTACT NUMBER: + AND EMAIL ID: SRISNJJEWELS@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD, , NSC BOSE ROAD, SOWCARPET, CHENNAI, TAMIL NADU, , SINCE . DATE OF BIRTH IS . PAN OF THE ACCOUNT IS BADPJR. PAN LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS NIKET SOHANLAL JAIN PROPRIETOR - BADPJR. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN -BADPJR AND LETTER OF DECLARATION OF PROPRIETORSHIP FIRM. KYC UPDATION IS NOT DUE. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN  LACS. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT. BALANCE IN THE ACCOUNT AT  WAS RS. . LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY FROM KANTILAL, JAI GANESH JEWELLERY, NEFT RS. . LAKHS MAJORLY FROM DAIMOND, SANGITA MADAN JAIN, RTGS RS. ,. LAKHS MAJORLY FROM LEO JEWELS, DIAMOND, REMITTANCE RS. ,. LAKHS MAJORLY FROM CHAIPHET ANYAMANEE CO LTD, SUAY THAI THONG CO LTD, INTERNET FUND TRANSFERS RS. ,. LAKHS FROM MOKSHA JEWELLERY. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RTGS RS. ,. LAKHS MAJORLY TO DIAMOND INDIA LIMITED, REMITTANCE RS. . LAKHS MAJORLY TO CHAIPHET ANYAMANEE CO LTD, INTERNET FUND TRANSFER RS. ,. LAKHS TO MOKSHA JEWELLERY ETC, MOBILE MONEY TRANSFERS RS. . LAKHS IN  INSTANCES. FY - FROM  TO  : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH RS.  BY CASH, REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RS.  BY CASH, REMITTANCE RS. ,. LAKHS. CUSTOMER IS INTO GOLD ORNAMENTS MANUFACTURING. FOR MANUFACTURING PURPOSE, CUSTOMER IS USING JOB WORK VENDORS AND GOLD SMITHS. EXCEPT OFFICE SET UP, COULD NOT ASCERTAIN ANY BUSINESS ACTIVITIES. HIGH VALUE NON CASHC CREDITS AND DEBITS WERE SEEN IN THE ACCOUNT. THE ACTIVITY IS NOT LOOKING USUAL AS THE TURNOVER IN JUST  YEAR IS MUCH HIGHER IN COMPARE TO THE DECLARED TURNOVER WHICH LESS THAN RS.  LACS. FOR EXAMPLE, ON  RS . LACS WERE CREDITED THROUGH RTGS FROM J B GOLD , KKBK AND ON THE SAME DAY RS . CRORE WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PVT LTD , INDB. ALSO ON  RS . LACS WERE CREDITED THROUGH RTGS FROM A V JEWELLERS , YESB, LEO JEWELS , HDFC ETC AND ON THE SAME DAY RS . LACS WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PRIVATE LIMITED. SIMILAR PATTERN WERE SEEN IN THE ACCOUNT. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT WHICH SEEMS LIKE ROUTING OF FUNDS WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILED.",{"entities":[(1345, 1370,"FOREIGN ORG"),(1372, 1394,"FOREIGN ORG"),(1581, 1606,"FOREIGN ORG")]}),
            (" TO TOTAL CREDIT SIDE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS  TRANSACTION PATTERN SHOWS LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT ROUTING OF FUNDS WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS OF HIGH VALUE FUNDS FOR EXAMPLE ONRS  CRORES WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM DN ENERGY PRIVATE LIMITEDAND ON THE SAME DAY ACCOUNT WAS DEBITED WITH RS CRORES THROUGH CMSTO RILLICICI ALSO ONRSCRORES WERE CREDITED THROUGH CMS FROM REFEX INDUSTRIES LIMITED AND ON THE SAME DAY ACCOUNT WAS DEBITED WITH SAME AMOUNT THROUGH RTGS IN PARTS TO SIL ROOFTOP SOLAR POWER PVT LTD ACCOUNT NOIFSC HDFC SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED UNUSUAL ALSO AS PER MEDIA REPORT SEBI OBSERVED THAT DURING THE PERIOD OF JULYTO JULY  COMPANY HAD ALLEGEDLY VIOLATED THE PROVISIONS OF SEBI SUBSTANTIAL ACQUISITION OF SHARES AND TAKEOVER SAST REGULATION HTTPSWWWSEBIGOVINENFORCEMENTORDERSJANADJUDICATIONORDERINRESPECTOFJAGDISHTJAINSHERISHATECHNOLOGIESPVTLTDTARACHANDJAINANILJAINUGAMDEVIJAINDIMPLEJAINANDSEEMAJAININTHEMATTEROFREFEXINDUSTRIESLIMITED_HTML IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS STR IS BEING FILEDSHERISHA TECHNOLOGIES PVT LTD PRIVATE LTD HAVING ADDRESSOLD NO L NEW NO TH FLR VENKATNARAYANA ROADT NAGAR CHENNAI TAMIL NADU INDIACONTACT NUMBER + AND EMAIL ID LALITHAU@REFEXCOIN IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD CHENNAI  T NAGAR NEW NO BAZULLAH ROAD T NAGAR CHENNAI TAMIL NADU SINCE  DATE OF INCORPORATION IS  PAN NUMBER OF THE ACCOUNT IS AAHCSP IECCODE OF THE CUSTOMER AS PER BANK RECORDS IS  OTHER RELATIONSHIP OF CUSTOMER ARE   TRANSACTION LINKED ACCOUNTS ARE   AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE LALITHA AUTHORISED SIGNATORY ACQPLFTAPPI KUMAR DAS AUTHORISED SIGNATORY BDKPDC DINESH KUMAR AGARWAL AUTHORISED SIGNATORY AKHPAA  BENEFICIALOWNER OF THE REPORTED ACCOUNT IS A TARACHAND JAIN WITH PAN NO AAFPJLT ANIL JAIN WITH PAN NO AADPAA THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAHCSP CIN UTNPTC BOARD RESOLUTION MOA ETC DATE OF LAST KYC UPDATION KYC UPDATION IS NOT DUE ENTITY IS INTO MANAGEMENT CONSULTANCY SERVICES  DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO LESS THAN CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS BALANCE IN THE ACCOUNT ATWAS RS LAKHSCREDIT FURTHER THE QUANTUM OFTRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WEREFY  FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN TIRUPPUR BRANCHTRANSFERS RS LAKHS MAJORLY FROM ARADHYAA INFRAHOLDINGS PRIVATE LIMITEDPAVAGADA PROCESSING INDUSTRIES PVT LTDCLEARING RS LAKHS MAJORLY FROM SHERISHA SOLAR PVT LTDREFEX INDUSTRIES LIMITEDDECCAN CHARTERS PVT LTDNEFT RS LAKHS MAJORLY FROM MEDHA SERVO DRIVES PVT LTDL T LTDILABS CAPITAL I LLPRTGS RS LAKHS MAJORLY FROM DUGAR FINANCE AND INVEREFEX ENERGY LIMITEDILABS CAPITAL I LLPREMITTANCE RS LAKHS MAJORLY FROM VISTA INDIA DIGITAL MEDIA INCINTERNET FUND TRANSFERS RS LAKHS FROM DN ENERGY PRIVATE LIMITEDMOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCES FIXED DEPOSIT RS LAKHS AND OTHERS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCHTRANSFERS RS LAKHS MAJORLY TO EMPLOYEES PROVIDENT FUND ORGANISATION SHERISHA TECHNOLOGIES PVT LTDKHAJANE II DEPT OF TRESURIESCLEARING RS LAKHS RTGS RS LAKHS MAJORLY TO ILABS CAPITAL I LLPDUGAR FINANCE AND INVESTMENT  REMITTANCE RS LAKHS MAJORLY TO GINLONG TECHNOLOGIES CO LTDTUAH AND SUPARTO INTERNET FUND TRANSFER RS LAKHS CARD TRANSACTIONS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES  AND OTHERS RS LAKHS FY  FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS  TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS FY  FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS  TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS FY  FROM",{"entities":[(3612, 3643,"FOREIGN ORG")]}),
            ("IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT ROUTING OF FUND AND HIGH TURNOVER IN THE ACCOUNT STR IS BEING FILEDA N DIAMONDS A PROPRIETORSHIP FIRM HAVING ADDRESSH N KUMBHAR SHERIND FLOOR MAHIDHARPURAROOM  SURAT GUJARAT INDIA  CONTACT NUMBER + AND EMAIL ID ANDIAMONDS@YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD GROUND FLOOR TOWER A SWASTIK UNIVERSAL UDHNA MAGDALLA ROAD SURAT GUJARAT  SINCE  DATE OF INCORPORATION IS  PAN NUMBER OF THE ACCOUNT IS AAZPDP GSTIN AS PER BANK RECORDS IS AAZPDPZX TRANSACTION LINKED ACCOUNTS ARE   AUTHORIZED SIGNATORY PROPRIETOR OF THE REPORTED ACCOUNT IS ASHESH NANALAL DOSHI PAN AAZPDP  THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAZPDP AADHAAR GST REGISTRATION CERTIFICATE HENCE KYC UPDATION IS NOT DUE CUSTOMER IS INTO IMPORT AND EXPORT OF ROUGH DIAMONDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RSLAKHS TO RSCRORES ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT ATWAS RS LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY  FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN SURATLH ROAD VARACHHA BRANCH TRANSFERS RS LAKHS MAJORLY FROM JIGAR SHAH JT ASHESH NANALAL DOSHI SUMO JEWEL CLEARING RS LAKHS MAJORLY FROM KANCHANBEN VADILAL VADILAL CHHOTALAL NEFT RS LAKHS MAJORLY FROM A N DIAMONDS NISHCHAL C DESAI HUF RTGS RS LAKHS MAJORLY FROM AASHESH NANALAL DOSHI VAISHALI GEMS REMITTANCE RS LAKHS MAJORLY FROM STAR IMPEX HONG KONG USD CURRENCY INTERNET FUND TRANSFERS RS LAKHS FROM A N DIAMONDS  MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES  UPI TRANSFERS RS LAKHS ININSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH TRANSFERS RS LAKHS MAJORLY TO SAHAJANAND TECHNOLOGIES PVT LTD A N DIAMONDSANU IMPEX CLEARING RS LAKHS MAJORLY TO TORRENT POWER LIMITEDMARUTI HARDWARE NEFT RS LAKHS MAJORLY TO RITESH B MEHTAPADSALA VITTHALBHAI RTGS RS LAKHS MAJORLY TO GIA INDIA LABORATORY PVT LTD ASHESH NANALAL DOSHI REMITTANCE RS LAKHS MAJORLY TO GIRDHARLAL DMCC IN USD TO UAE FOR PAYMT TOWARDS IMPORT SETTLEMENT OTHER THAN NEPAL BHUTAN DIAMONDS INTERNET FUND TRANSFER RS LAKHS TO A N DIAMONDS  MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RS LAKHS DTAXFURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY  FROMTO  TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE ON  RSLAKHS WERE CREDITED THROUGH RTGS FROM AASHESH NANALAL DOSHI ACCOUNT NO IFSC CODECBIN BANK CENTRAL BANK OF INDIA AND ON SAME DAY RSLAKHS THROUGH RTGS TO GIA INDIA LABORATORY PVT LTD ACCOUNT NO  IFSC CODE HDFC BANK HDFC ON  RSLAKHS WERE CREDITED THROUGH INTERNET FUND TRANSFERS FROM A N DIAMONDSAND ON NEXT DAY RSLAKHS WERE DEBITED THROUGH A N DIAMONDS SIMILAR TRANSACTIONS ARE BEING REPEATED IN THE ACCOUNT WHICH SEEMS TO BE ROUTING OF FUNDS IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY  WAS RS LAKHS WHICH SEEMS TO BE VERY HIGH",{"entities":[(1607, 1617,"FOREIGN ORG")  ]}),
            ("BELOW ARE RED FLAGS OBSERVED OUT OF TOTALOUTWARD REMITTANCES VALUE RS CRORES PERTAINING TO ADVANCE AGAINST IMPORTS CLIENT HAS NOT SUBMITTED BILL OF ENTRY PERTAINING TOTRANSACTIONS OF VALUE RS CRORES PENDING SINCE NOVEMBER IN THIS REGARD YES BANK OFFICIALS HAD FOLLOWED UP WITH THE CUSTOMER ON MULTIPLE OCCASIONS AND HAD REQUESTED TO SHARE TRANSPORT DOCUMENTS FOR WHICH IMPORT REMITTANCES ARE BEING UNDERTAKEN HOWEVER CUSTOMER HAS TILL DATE NEITHER SUBMITTED BILL OF ENTRIES PERTAINING TOTRANSACTIONS NOR HAS HE FURNISHED ANY TRANSPORT DOCUMENTS PERTAINING TOOUTWARD REMITTANCES THUS UNDERTAKENAS PER RECENT COMMUNICATION CLIENT HAS REPLIED STATING THAT SINCE HE NO LONGER WANTS TO TRANSACT VIA YES BANK HE HAS SUBMITTED BILL OF ENTRY AND LODGED RELATED TRANSPORT DOCUMENTS WITH ANOTHER BANKAS PER PUBLIC DOMAIN INFORMATION ON OVERSEAS ENTITIES NAFDA TRADING FZE UAE AND CAMELOT OVERSEAS PTE LTD SINGAPORE BELOW RED FLAGS WERE OBSERVEDPUBLIC DOMAIN INFORMATION OF OVERSEAS ENTITY NAFDA TRADING FZE UAE IS INSUFFICIENT MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS KITCHENER ROAD  JALAN BESAE PLAZA SINGAPORE OF OVERSEAS ENTITY CAMELOT OVERSEAS PTE LTDCAMELOT OVERSEAS PTE LTD HAD PREVIOUSLY CHANGED NAMES MULTIPLE TIMES REFER HTTPSWWWCOMPANIESSGZHBUSINESSHCAMELOTOVERSEASPTELTDWEBSITE OF CAMELOT OVERSEAS PTE LTD SINGAPORE IS NEWLY REGISTERED IN THE YEAR OCTOBERAND REGISTRATION LOCATION IS NEW DELHI INDIA HTTPSWWWWHOISCOMWHOISCAMELOTOVERSEASCOM INCIDENTALLY IT WAS OBSERVED THAT CONTENTS OF WEBSITE OF OVERSEAS ENTITY CAMELOT OVERSEAS PTE LTD HTTPSCAMELOTOVERSEASCOM IS COPY PASTED FROM CUSTOMERS OWN WEBSITE OF KALEEN BHAWAN REFER HTTPSKALEENBHAWANCOM?PAGE_ID=FURTHER STATED ANNUAL INCOME OF IMEXTRA TRADERS PRIVATE LIMITED IS LESS THAN RS CRORES HOWEVER IN LESS THANMONTHS FROM ACCOUNT OPENING OCTOBERCLIENT HAS RECEIVED ACCOUNT CREDITS OF MORE THAN RS CRORESMAJOR CREDITS IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED IS VIA FUNDS TRANSFER FROM OTHER YES BANK CUSTOMERS VIZ M CARPETS RSCRS FROM AC NO  MISHRA AND SONS RSCRS FROM AC NO  SHIVESH GARMENTS PRIVATE LIMITED RSCRS FROM AC NOAND RTGS THE TUNE OF RS FROM SHIVESH GARMENTS PRIVATE LIMITED AC NOWITH ICICI BANK ICIC AND RS CRORES FROM MISHRA AND SONS AC NOWITH INDUSIND BANK INDBNO RED FLAGS WERE OBSERVED IN THE LINKED ACCOUNTS OF VINDHYA DAIRY UDAMYA IMPEX PRIVATE LIMITED AND ORIENTAL CARPET EXPORTERS HOWEVER TOTAL CREDITS RECORDED IN YES BANK ACCOUNTS OF OTHER LINKED ACCOUNTS OF M CARPETS MISHRA AND SONS AND SHIVESH GARMENTS PRIVATE LIMITED IS BEYOND STATED ANNUAL INCOME OF LESS THAN RSCRORES REGISTERED COMMUNICATION ADDRESS OF SHIVESH GARMENTS PRIVATE LIMITED IS THAT OF NEW DELHI DELHI AS MENTIONED IN POINT HOWEVER ITS REGISTERED PERMANENT ADDRESS IS IN FLAT NOPOONAM ORBIT MIRA ROAD THANE MAHARASHTRA AFOREMENTIONED REGISTERED PERMEANT ADDRESS OF SHIVESH GARMENTS PRIVATE LIMITED DOES NOT MATCH WITH THE ONE ON PUBLIC DOMAIN IE C FIRST FLOOR AJANTA SQUARE MALL BHAJI MARKET LANE  BORIVALI W MUMBAI CITY MAHARASHTRA IN LIGHT OF CROSS BORDER REMITTANCES BEING UNDERTAKEN IN PRETEXT OF IMPORT OF GOODS FROM OVERSEAS ENTITIES WITH OPAQUE CREDENTIALS WHEREIN VERACITY OF UNDERLYING TRANSACTIONS COULDNT BE DETERMINED DUE TO ABSENCE OF SUPPORTING DOCUMENTS AND CUSTOMER BEING NON COOPERATIVE IT HAS BEEN DECIDED ON MAY TO FILE STR ON SAID CUSTOMER THE CUSTOMER IMEXTRA TRADERS PRIVATE LIMITED PAN AAECIK DOI  COMMUNICATION ADDRESS THIRD FLOOR  EAST PATEL NAGAR NEW DELHI DELHI  PERMANENT ADDRESS C FIRST FLOOR POLE NO D PALAM  NEW DELHI   GSTIN AAECIKZD REGISTRATION NUMBER  CIN UDLPTC  IEC CODE AAECIK MOBILE NUMBER  EMAIL ID IMEXTRATRADERSPVTLTD@GMAILCOM IS MAINTAINING A CURRENT AC NOSINCEWITH DELHI ASAF ALI ROAD BRANCH ADDRESS GROUND FLOOR  ASAF ALI ROAD NEW DELHI   THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN OCTOBER  SAT PAL SHARMA AND ASHISH MISHRA ARE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS GARMENT STORE WITH AN ANNUAL INCOME LESS THAN RS CRORES CUSTOMER HAS AVAILED TERM DEPOSITS AMOUNTING TO RS ADDITIONALLYMORE ACCOUNTS ARE FOUND TO BE LINKED TO THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NO AS UNDERCURRENT ACCOUNT OF KALEEN BHAWAN EXPORTS PRIVATE LIMITED AC NOIS LINKED WITH AFOREMENTIONED ACCOUNT BY COMMON MOBILE NUMBERAND COMMUNICATION ADDRESS IN POINT NOCURRENT ACCOUNT OF SHIVESH GARMENTS PRIVATE LIMITED AC NO  VINDHYA DAIRY AC NO  UDAMYA IMPEX PRIVATE LIMITED AC NOAND ORIENTAL CARPET EXPORTERS AC NOS  LINKED WITH AFOREMENTIONED ACCOUNT BY COMMON COMMUNICATION ADDRESS IN POINT NO CURRENT ACCOUNT OF M CARPETS AC NOAND MISHRA SONS AC NOARE LINKED WITH ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED BY FUND TRANSFERS IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NOANDLINKED ACCOUNTS TRANSACTIONS ANALYSIS FROMTILL  SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS  ANDDEBIT TRANSACTIONS AMOUNTING TO RS OUT OF THE TOTAL CREDITS OF RS  IN THE ACCOUNTS FUND TRANSFERS AMOUNTS TO RS  CONSTITUTING AROUNDPERCENT OF THE TOTAL CREDITS NEFTRTGSIMPS AMOUNTS TO RS  CONSTITUTING AROUNDPERCENT OF THE TOTAL CREDITS OUT OF THE TOTAL DEBITS OF RS  IN THE ACCOUNTS FUND TRANSFERS AMOUNTS TO RS  CONSTITUTING AROUNDPERCENT OF THE TOTAL DEBITS NEFTRTGSIMPS AMOUNTS TO RS  CONSTITUTING AROUNDPERCENT OF THE TOTAL DEBITSIN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NOANDLINKED ACCOUNTS TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS  COMPRISING OF CASH DEPOSIT AMOUNTING TO RS  ANDDEBIT TRANSACTIONS AMOUNTING TO RS  COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RSFROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS  COMPRISING OF CASH DEPOSIT AMOUNTING TO RS  ANDDEBIT TRANSACTIONS AMOUNTING TO RS  COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RSFROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS  COMPRISING OF NIL CASH DEPOSIT ANDDEBIT TRANSACTIONS AMOUNTING TO RS  COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RSBALANCE IN THE ACCOUNT AS ONIS RSON REVIEWING THE TRANSACTION PATTERN OF IMEXTRA TRADERS PRIVATE LIMITED ACAND KALEEN BHAWAN EXPORTS PRIVATE LIMITED AC NOIT WAS OBSERVED THAT SINCE OCTOBERSAID ENTITIES HAD REMITTED TOTAL OF RS CRORES UNDERTRANSACTIONS TO OVERSEAS ENTITIES CAMELOT OVERSEAS PTE LTD SINGAPORE RS CRORES INTRACHES AND NADFA TRADING FZE UAE RS CRORES INTRANCHES TOWARDS ADVANCE AGAINST IMPORT OF CARPET  CARPET",{"entities":[(870, 894,"FOREIGN ORG"),(979, 996,"FOREIGN ORG"),(1128, 1152,"FOREIGN ORG"),(1227, 1313,"FOREIGN ORG"),(6324, 6341,"FOREIGN ORG"),(6265, 6289,"FOREIGN ORG")]}),
            ("FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH, REMITTANCE RS. . LAKHS. FY  FROMTO : TOTAL CREDIT SIDE RS. . LAKHS TOTAL DEBIT SIDE RS. . LAKHS. FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS. THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBERIN THE NAME OF BORA MOBILITY, REPORT SERIAL NUMBER . TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS WHICH SEEMS LIKE ROUTING OF FUNDS OBSERVED. FOR EXAMPLE, ONRS. .LACS WAS CREDITED BY RTGS IN PARTS FROM JAY JALARAM ENTERPRISE INDB, BORA EXIM PRIVATE LIMITED , NEFT FROM SAMA DISTRIBUTOR HDFC, AND RS.LACS WAS DEBITED BY RTGS TO BORA MOBILITY , HDFC. IN OTHER EXAMPLE, ONR. .LACS CREDITED BY INTERNET FUND TRANSFERS FROM BORA EXIM PRIVATE LIMITEDAND SAME WAS DEBITED BY INTERNET FUND TRANSFERS TO BORA MOBILITY , HDFC. SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD. TOTAL TURNOVER IN THE FINANCIAL YEAR , WAS RS. ,. LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL INCOME OF RS. LACS TO RS. CRS. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF FUNDS OBSERVED AND HIGH TURNOVER, STR IS BEING FILED.BORA MOBILITY, PROPRIETORSHIP FIRM, HAVING ADDRESS: OFFICE NO. AND , NEARMITRAMANDALCHOW, PARVATI PUNE, CENTRE POINT PLOT NO , PUNE, MAHARASHTRA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: PRASHANT.BORA@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, PUNE  SATARA ROAD, ICICI BANK LTD, SOMSHANK CHAMBERS, SURVEY NO. , PLOT , PUNE SATARA ROAD, PUNE, MAHARASHTRA, , SINCE , DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AISPBL. TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE PRASHANT PRAKASH BORA PROPRIETOR  AISPBL THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN AISPBL, AADHAR. DATE OF LAST KYC UPDATION . BORA MOBILITY IS AN INDIAN COMPANY ENGAGED IN TRADING AND DISTRIBUTION OF MOBILE HANDSETS, TABLETS, LAPTOPS AND MOBILE ACCESSORIES WITH DIRECT PRESENCE IN OVER TWENTY COUNTRIES. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. LACS TO RS. CRS. A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NONCASH TRANSACTIONS. BALANCE IN THE ACCOUNT ATWAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROMTILLSHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH CLEARING RS. . LAKHS MAJORLY FROM NEFT, SSK INCORPORATION, NEFT RS. . LAKHS MAJORLY FROM MUKUL MADHAV FOUNDATION, MAHARASHTRA VALUE ADDED TAX, MUKUL MADHAV FOUNDATION FIL CSR, RTGS RS. . LAKHS MAJORLY FROM KAMALETERNITY PVT LTD, JAY JALARAM ENTERPRISE, BORA MOBILITY, INTERNET FUND TRANSFERS RS. . LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES, UPI TRANSFERS RS. . LAKHS ININSTANCES, FIXED DEPOSIT RS. . LAKHS,. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH CASH WAS NIL, CLEARING RS. . LAKHS, RTGS RS. . LAKHS, REMITTANCE RS. . LAKHS MAJORLY TO SHENZHEN DEDA HEALTH CO LTD, MEDMSL LIMITED, ZHENGZHOU OLIVE ELECTRONIC, INTERNET FUND TRANSFER RS. . LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES, AND OTHERS RS. . LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS MAJORLY IN DELHI KAROL BAGH GURDWARA ROAD, BULDHANA, KAROL BAGH N.DBRANCH, TRANSFERS RS. . LAKHS MAJORLY FROM BORA MOBILITY, PRASHANT PRAKASH BORA, SEANA, CLEARING RS. . LAKHS MAJORLY FROM SSK INCORPORATION, BORA MOBILITY LLP, A Z LUNAWAT AND, NEFT RS. . LAKHS MAJORLY FROM OMKAR COMPUTERS PVT LTD, BITFLIGHT ELECTRONIC SYSTEMS, BORA MOBILITY, RTGS RS. ,. LAKHS MAJORLY FROM BORA MOBILITY, BATHLA TELETECH PRIVATE, E PAO GST REFUNDS, REMITTANCE RS. . LAKHS MAJORLY FROM CAPTAINS FREIGHT SERVICES HK, INTERNET FUND TRANSFERS RS. ,. LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES, UPI TRANSFERS RS. . LAKHS ININSTANCES, FIXED DEPOSIT RS. . LAKHS,. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKH BY BRANCH, TRANSFERS RS. . LAKHS MAJORLY TO BORA MOBILITY, CLEARING RS. . LAKHS, RTGS RS. . LAKHS MAJORLY TO BORA AND BORA TECHNOLOGIES, BORA MOBILITY, REMITTANCE RS. ,. LAKHS MAJORLY TO ESTAR TECHNOLOGY GROUP CO LTD, WONDAFONE TECHNOLOGY HONGKONG CO, AYAAN ELECTRONICS LLC, INTERNET FUND TRANSFER RS. . LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES.",{"entities":[(3136, 3163,"FOREIGN ORG"),(3165, 3179,"FOREIGN ORG"),(3181, 3207,"FOREIGN ORG") ,(4477, 4506,"FOREIGN ORG"),(4508, 4540,"FOREIGN ORG"),(4542, 4563,"FOREIGN ORG"),(4009, 4036,"FOREIGN ORG")]}),
            ("BEETREAT LOGISTICS PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS: OFFICE NO , ST FLOOR, SECTOR , CBD BELAPUR, PLOT NO , NAVI MUMBAI, MAHARASHTRA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: BEETREATLOGISTICS@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, BALAJI BHAVAN, NEAR BELAPUR RAILWAY STN., CBD BELAPUR, NAVI MUMBAI, MAHARASHTRA, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AAICBG. GSTIN AS PER BANK RECORDS IS AAICBGZZ. TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS/ DIRECTORS OF THE REPORTED ACCOUNT ARE DINESH KUMAR WITH PAN NO ANXPKBVIKAS NARAWAT PAN: AIJPVD. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON . KYC DOCUMENTS OBTAINED WERE PAN  AAICBG, COI: UMHPTC, MOA AOA AND PAN OF AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS. CUSTOMER IS INTO THE BUSINESS OF LOGISTICS SERVICES, TRADING, WHOLES RETAILS, IMPORT AND EXPORT OF GOODS AND SERVICES FOR MAKING AND EARING PROFITS, WORKS ON COMMISSION BASIS FOR GOODS TRADING. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS GREATER THAN EQUALS TO RS.CRORES. ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY FROM DINESH KUMAR, NEFT RS. . LAKHS MAJORLY FROM BONIRY LOGISTICS, RTGS RS. ,. LAKHS MAJORLY FROM BONIRY LOGISTICS, AEGIS LOGISTICS, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY ATM AT THANE, NEW BOMBAY BELAPUR CIBD BRANCH, CLEARING RS. . LAKHS MAJORLY TO BEETREAT LOGISTICS PRIVATE LIMITED, RTGS RS. . LAKHS MAJORLY TO BONIRY LOGISTICS, REMITTANCE RS. ,. LAKHS MAJORLY TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG, IN USD CURRENCY FOR WHICH PURPOSE WAS MENTIONED AS TOWARDS OTHER PAYMENTS BY SHIPPING COMPANIES, KM LOGISTICS AND SHIPPING PTE LTD IN USD TO SINGAPORE FOR OTHER PAYMENTS BY SHIPPING COMPANIES, HMMA LOGISTICS CO LIMITED IN USD TO HONG KONG FOR OTHER PAYMENTS BY SHIPPING COMPANIES, LUCKY GAINS LIMITED IN USD TO HONG KONG FOR WHICH PURPOSE WAS MENTIONED AS PAYMENT TOWARDS SHIPPING CHARGES, INTERNET FUND TRANSFER RS. . LAKHS TO KEMPS LOGISTICS PRIVATE LIMITED , MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES.ON ANALYZING TRANSACTION PATTERN IN THE ACCOUNT, IT HAS BEEN OBSERVED THAT THERE WERE HIGH VALUE NON CASH CREDITS FOLLOWED BY DEBITS THROUGH REMITTANCES. FOR EXAMPLE, ON , RS. . LAKHS WERE CREDITED THROUGH RTGS FROM BONIRY LOGISTICS ACCOUNT NO: , IFSC CODEUTIB AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH REMITTANCE TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG, IN USD CURRENCY FOR WHICH PURPOSE WAS MENTIONED AS TOWARDS OTHER PAYMENTS BY SHIPPING COMPANIES. ON , RS.LAKHS WERE CREDITED THROUGH RTGS FROM AEGIS LOGISTICS ACCOUNT NO: , IFSC CODE: ASBL AND ONRS. . LAKHS WERE DEBITED THROUGH REMITTANCE TO LUCKY GAINS LIMITED IN USD TO HONG KONG FOR WHICH PURPOSE WAS MENTIONED AS PAYMENT TOWARDS SHIPPING CHARGES. SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT WITHOUT ANY BUSINESS EXPENDITURE. TRANSACTIONS SEEM UNUSUAL AND NO SATISFACTORY EXPLANATION ABOUT IT CAN BE OBTAINED FROM THE CUSTOMER. ACCOUNT CLOSED BY CUSTOMER WITHIN SPAN OFMONTHS AND HIGH TURNOVER IN THE ACCOUNT WITHIN SPAN OFMONTHS RS. ,. LAKHS, WHICH RAISED FURTHER SUSPICION. IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT, STR IS BEING FILED.",{"entities":[(1905, 1932,"FOREIGN ORG"),(2044, 2077,"FOREIGN ORG"),(2140, 2165,"FOREIGN ORG"),(2226,2246,"FOREIGN ORG")    ,(2779,2806,"FOREIGN ORG")]}),
            ("SUBSEQUENT TO SUCH CREDITS FUNDS WERE REMITTED ON THE SAME OR NEXT DAY OVERSEAS FAVOURING ENTEGRATED LOGISTIC PTE LTDSINGAPORE RS.. CR TOWARDS FREIGHT ON EXPORTSSHIPPING COMPANIES.DURING WEB CHECK NO SPECIFIC INFORMATION/ WEBSITE COULD BE SEEN IN THE NAME OF ENTEGRATED LOGISTIC PTE LTD.ON SCRUTINY OF SAMPLE DOCUMENTS FURNISHED BY CUSTOMER FOR EFFECTING REMITTANCES IN WAS NOTED THAT CUSTOMER HAD FURNISHED COPIES OF B/L, FREIGHT INVOICE, AGREEMENT ANDCA/ CB. AS PER THE B/L GOODS WERE SHIPPED FROM INDIA TO CHINA HOWEVER FREIGHT PAYMENT WAS BEING MADE TO SINGAPORE BASED ENTITY.A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE RTGS/NEFT CREDITS WERE FOLLOWED BY HIGH VALUE FOREIGN OUTWARD REMITTANCES ON THE SAME DAY. NO OTHER BUSINESS RELATED TRANSACTIONS LIKE PAYMENTS TOWARDS TAXES, SALARIES, RENT ETC., COULD BE NOTED IN THE ACCOUNT DURING REVIEW PERIOD.IT IS NOTED THAT WITHINMONTHS OF ACCOUNT OPENING TRANSACTIONS IN THE ACCOUNT FAR EXCEEDED THE ANNUAL TURNOVER INDICATED BY CUSTOMER DURING ACCOUNT OPENING.DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT THE OFFICE WAS ASQ. FT. RENTED PREMISE WHERE SIGN BOARD, EMPLOYEES AND OFFICE SET UP WERE SEEN.BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT THE DIRECTORS WHO ARE AGED YEARS MOHIT CHAUDHARY ANDYEARS ASHISH CHOWREKAR MAY NOT BE THE ACTUAL BENEFICIAL OWNER OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNT IN A SHORT SPAN. ALSO CUSTOMER REMITTING FUNDS TO SINGAPORE AS FREIGHT CHARGES WHEN THE GOODS ARE SHIPPED FROM INDIA TO CHINA APPEARS UNUSUAL. IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH THE ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING OR HAWALA SETTLEMENT. ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED IS ACTIVE WITH BALANCE OF RS.. LAKHS AS ON MAY , . QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR.A CURRENT ACCOUNT BEARING NUMBERWAS OPENED IN THE NAME OF OCEANS PRIDE MARINE SERVICES P LTD ON FEB AT OUR BRANCH LOCATED IN LAKE PRIMROSE PHASEIV, WING C LAKE HOMES, POWAI, MUMBAI MAHARASHTRA  . THE ENTITY WAS FORMED ON DEC AND HAS SUBMITTED COPIES OF PAN AACCOA, CERT. OF INCORP UMHPTC, SHOPS AND ESTABLISHMENT REG. CERTIFICATEAND IEC CERT AACCOA AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO.SAMRUDDHI C COMPLEX CHINCHOLI BUNDER ROAD MALAD WEST MUMBAI . MOHIT CHAUDHARY AND ASHISH DINESH CHOWKEKAR ARE THE DIRECTORS WHILE MOHIT CHAUDHARY IS THE BENEFICIAL OWNER IN THE SAID COMPANY. MOHIT CHAUDHARY AND ASHISH DINESH CHOWKEKAR HAVE REGISTERED THEIR MOBILE NUMBERSANDWITH THE BANK. THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS.. CR.BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT ACCOUNTS IN THE NAMES OF TRIUMPH TECHNOMATE SOLUTIONS , A/C OPENED ON JUL, RAJKOTIA CLOUD CONSULTANCY P LTD , A/C OPENED ON AUG AND SAVINGS ACCOUNT IN THE NAME OF ASHISH DINESH CHOWKEKAR , A/C OPENED ON JAN. THE CURRENT ACCOUNT OF TRIUMPH TECHNOMATE SOLUTIONS AND SAVINGS ACCOUNT OF ASHISH DINESH CHOWKEKAR ARE DORMANT. IT WAS INDICATED THAT RAJKOTIA CLOUD CONSULTANCY P LTD IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS.. CR. NO MAJOR TRANSACTIONS WERE NOTED IN THE ACCOUNT OF RAJKOTIA CLOUD CONSULTANCY P LTD DURING REVIEW PERIOD. WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WERE ROUTED THROUGH THE ABOVE MENTIONED NEWLY OPENED ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED. ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH FREIGHT PAYMENTS WERE DONE IMMEDIATELY POST RECEIPT OF HIGH VALUE RTGS/NEFT CREDITS AND HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION:ON REVIEWING ACCOUNT TRANSACTIONS SINCE APRIL ,TILL MAY ,IT WAS NOTED THAT RTGS/NEFT CREDITS AMOUNTED TO RS.. CR MAINLY FROM QUICK SOLUTION LOGISTICS RS.. CR, RBL BANK  , BLUE SEA LOGISTIC RS.. CR, RBL AND PREMIUM LOGISTICS SOLUTIONS RS.. CR, RBL ",{"entities":[(80, 120,"FOREIGN ORG"),(259, 289,"FOREIGN ORG")]}),
            ("TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BRANCH, TRANSFERS RS. ,. LAKHS MAJORLY TO BINDRA TOURS N TRAVELS PRIVATE LIMITED, EMPLOYEES PROVIDENT FUND ORGANISATION, TRAVEL BOUTIQUE ONLINE A U/O TEK TRA.PL, CLEARING RS. . LAKHS MAJORLY TO WELFARE COMMISSIONER HARY, POOJA PRINTERS, PHD CHAMBER OF COMMERCE, RTGS RS. ,. LAKHS MAJORLY TO TRAVEL BOUTIQUE ONLINE, TRAVEL BOUTIQUE ONLINE, REMITTANCE RS. ., INTERNET FUND TRANSFER RS. ,. LAKHS MAJORLY TO R K TRAVELTLC SOLUTION, KANDHARI TRAVEL ETC. FY  FROMTO : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. ,. LAKHS, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS.FY  FROMTO : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. ,. LAKHS, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. . LAKHS.FY  FROMTO : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. ,. LAKHS, REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. . LAKHS, REMITTANCE RS. .. PAST STR REPORTED: THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBERIN THE NAME OF TRAVEL BOUTIQUE ONLINE A U/O TEK TRA.PL ONVIDE FIU BATCH ID , REPORT SERIAL NUMBERCUSTOMER IS INTO TRAVEL RELATED SUPPORT THROUGH ONLINE BASED PORTAL. AS PER TRANSACTION PATTERN VOLUME OF TRANSACTION IS VERY HIGH MULTIPLE NUMBER OF NONCASH CREDITED IN THE ACCOUNT FOLLOWED BY NONCASH WITHDRAWAL OBSERVED. FOR EXAMPLE, ONRS.CRORES APPROX. CREDITED IN MULTIPLE PARTS FROM DIFFERENT ENTITIES SUCH AS NEFT FROM NEXUS TRAVELS, MICHELIN STAR HOLIDAYS, AIONZ SOJOURN PRIVATE LIMITED, ACE TRAVELS ETC, TRANSFERS FROM DIVVYA TOURS AND TRAVELS, OM TRAVELS, SACH TRAVEL, SEVEN OCEAN, SURAJ TRAVELS, KRISHNA TRAVELS, TRIP STORE LTD ETC, CASH DEPOSITS FROM VIJAYNAGARBANGALORE BRANCH, MALAD BRANCH ETC AND ON THE SAME DAY RS.CRORES GOT DEBITED BY RTGS INPARTS TO TRAVEL BOUTIQUE ONLINE ACC, IFSCSCBL. SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT. AS OBSERVED THE ACCOUNTS ACTIVITY SEEN COLLECTIONS FROM VARIOUS PARTIES, RELATED TO BOOKING OF HOTEL, TRAVEL TICKETS, AND ITS RELATED SERVICES. TOTAL TURNOVER IN THE FINANCIAL YEAR , WAS RS. ,,. LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS. CRS TO LESS THAN RS. CRS. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT AND HIGH TURNOVER, STR IS BEING FILED.TRAVEL BOUTIQUE ONLINE A U/O TEK TRA.PL, PRIVATE LIMITED COMPANY, HAVING ADDRESS: P NO, PHV, PHV U/VIHAR, NR HORIZON BPO, GURGAON, HARYANA, INDIA, , EMAIL ID: RAJESHJ@TRAVELBOUTIQUEONLINE.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, PLOT NO. B B VANIJAYA NIKUNJ, ENKAY TOWER, PHASEIV, UDYOG VIHAR, GURGAON, HARYANA, , SINCE , DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AACCTK. GSTIN AS PER BANK RECORDS IS AACCTKZZ. TRANSACTION LINKED ACCOUNTS ARE , . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN  AACCTK. DATE OF LAST KYC UPDATION . CUSTOMER IS INTO TRAVEL RELATED SUPPORT THROUGH ONLINE BASED PORTAL WITH OFFICIAL ADDRESSHTTPS://WWW.TRAVELBOUTIQUEONLINE.COM/. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. CRS TO LESS THAN RS. CRS. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT. BALANCE IN THE ACCOUNT ATWAS RS.. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROMTILLSHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS MAJORLY IN BANGALORE  RESIDENCY ROAD, CHENNAI  TRIPLICANE, DELHI  TAIMUR NAGAR BRANCH, TRANSFERS RS. . LAKHS MAJORLY FROM ARYAN INTERNATIONAL, TRAVEL BOUTIQUE ONLINE A U/O TEK TRA.PL, GALAXY TRAVEL SOLUTIONS, CLEARING RS. . LAKHS MAJORLY FROM YATRA ONLINE PRIVATE LIMITED, FLAMINGO TRANSWORLD PVT LTD, SHRESTH UDAN TRAVELS, NEFT RS. ,. LAKHS MAJORLY FROM CSC E GOVERNANCE SERVICES IN, SUGAL AND DAMANI U S P LTD, WILLOWTREE, RTGS RS. ,. LAKHS MAJORLY FROM NEARBY TRAVEL SOLUTION, SPICE MONEY LIMITED, TRAVEL BOUTIQUE, REMITTANCE RS. . LAKHS MAJORLY FROM MTG GLOBAL SERVICES FZ, INTERNET FUND TRANSFERS RS. ,. LAKHS MAJORLY FROM R K TRAVELTLC SOLUTION, KANDHARI TRAVEL ETC, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES, UPI TRANSFERS RS. . LAKHS ININSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKH BY BRANCH, CLEARING RS. . LAKHS, RTGS RS. ,. LAKHS MAJORLY TO TRAVEL BOUTIQUE ONLINE, TRAVEL BOUTIQUE ONLINE. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,,. LAKHS OF WHICH CASH WAS RS. ,. LAKHS MAJORLY IN CHENNAI  G P ROAD, CHENNAI  TRIPLICANE, BANGALORE N.R RDBRANCH, TRANSFERS RS. ,. LAKHS MAJORLY FROM GLOBE HOLIDAY, TRAVEL BOUTIQUE ONLINE A U/O TEK TRA.PL, RAMBLE INDIA TOUR TRAVEL PVT LTD, CLEARING RS. ,. LAKHS MAJORLY FROM EBIX TRAVELS PVT LTD, FLAMINGO TRANSWORLD PVT LTD, TRAVEL BOUTIQUE ONLINE, NEFT RS. ,. LAKHS MAJORLY FROM TRAVCLAN TECHNOLOGY INDIA PVT LTD, LOVELY HOLIDAYS, CSC E GOVERNANCE SERVICES IN, RTGS RS. ,. LAKHS MAJORLY FROM TRAVELBOUTIQUEONLINEAU, TRAVEL BOUTIQUE ONLINE, HAPPYEASYGO INDIA P, REMITTANCE RS. . LAKHS MAJORLY FROM MTG GLOBAL SERVICES FZC SM OFFICE, MTG GLOBAL SERVICES FZC SM OFFICE C, MTG GLOBAL SERVICES FZC, INTERNET FUND TRANSFERS RS. ,. LAKHS MAJORLY FROM R K TRAVELTLC SOLUTION, KANDHARI TRAVEL ETC, MOBILE MONEY TRANSFERS RS. ,. LAKHS ININSTANCES, UPI TRANSFERS RS. . LAKHS ININSTANCES, FIXED DEPOSIT RS. ,. LAKHS.",{"entities":[(1584, 1606,"FOREIGN ORG"),(4123, 4145,"FOREIGN ORG"),(5268, 5287,"FOREIGN ORG"),(5303, 5322,"FOREIGN ORG"),(5340, 5359,"FOREIGN ORG")]}),
            ("FUNDS WERE CREDITED FROM THE BUSINESS CLIENT AND THE SAME HAS BEEN USE FOR BUSINESS PAYMENTS THROUGH REMITTANCES. FOR EXAMPLE, ONRSCRORES WERE CREDITED INPARTS THROUGH RTGS FROM SHIVALAY VYAPAAR PRIVATE LIMITED ACCOUNT NO: , IFSC CODEHDFC, SR BULLION JEWELLERS ACCOUNT NO: , IFSC CODEUTIB, ARIHANT BULLION AND JEWELS LLP ACCOUNT NO: , IFSC CODEKKBK ETC. AND ON THE SAME DAY THE ACCOUNT WAS DEBITED WITH SAME AMOUNT INPARTS THROUGH OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, DIJLLAH JEWELLERY FZCO OF U.A.E. IN USD FOR PAYMENT TOWARDS IMPORT OF GOLD. ONRS . CRORES WERE CREDITED INPARTS THROUGH RTGS FROM ARIHANT BULLION AND JEWELS LLP ACCOUNT NO: , IFSC CODEKKBK, ELEGANCE TRADELINKS PVT LTD ACCOUNT NO: , IFSC CODEUTIB AND ON THE SAME DAY THE ACCOUNT WAS DEBITED WITH RS . CRORES INPARTS THROUGH OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, DIJLLAH JEWELLERY FZCO OF U.A.E. IN USD FOR PAYMENT TOWARDS IMPORT OF GOLD. SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT. THERE WERE TOTALBOE WERE PENDING FOR SUBMISSION SINCE ACCOUNT OPENING. TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY  WAS RS. ,. LAKHS, WHICH SEEMS TO BE VERY HIGH. IN VIEW OF NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS, HIGH TURNOVER IN THE ACCOUNT AND BOE PENDING FOR SUBMISSION, STR IS BEING REPORTED.AGNIS BULLION HOUSE LLP, A LIMITED LIABILITY PARTNERSHIP FIRM, HAVING ADDRESS: P NO  SEC ,IMT MNS,G MANESAR, GURGAON, HARYANA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: MOHAMMED@DIJLLAHGOLD.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, PLOT NO. B B VANIJAYA NIKUNJ, ENKAY TOWER, PHASEIV, UDYOG VIHAR, GURGAON, HARYANA, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS ABNFAP. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS ABNFAP. TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS/ PARTNERS OF THE REPORTED ACCOUNT ARE MOHAMMED HAMMOODI HASHIM ALABID RESIDENT ID OF UAE:  SHARMIYA SHAROFUDDIN WITH PAN NO KVUPSD. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN: ABNFAP, COI: AAN, LLP AGREEMENT. CUSTOMER IS INTO BULLION BUSINESS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS.LAKHS TO LESS THAN RS.CRORES. ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS. BALANCE IN THE ACCOUNT ATWAS RS. ,. LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROMTILLSHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH NEFT RS. . LAKHS MAJORLY FROM SHIVALAY VYAPAAR PRIVATE LIMITED, RAKSHA BULLION, JALAN COMPANY PVT LTD, RTGS RS. ,. LAKHS MAJORLY FROM RAKSHA BULLION, AGNIS BULLION HOUSE LLP, SR BULLION JEWELLERS, INTERNET FUND TRANSFERS RS. ,. LAKHS FROM VISTARAN COMMODITIES LLP , MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH TRANSFERS RS. . LAKHS MAJORLY TO AGNIS BULLION HOUSE LLP, RTGS RS. ,. LAKHS MAJORLY TO REMITTANCE RS. ,. LAKHS MAJORLY TO BRENLEY QUARTZ COMPANY LIMITED, DIJLLAH JEWELLERY FZCO, INTERNET FUND TRANSFER RS. . LAKHS TO JALAN SILVER GARDEN PRIVATE LIMITED , MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES.FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF CLEARING RS. . LAKHS MAJORLY FROM AGNIS BULLION HOUSE LLP, NEFT RS. ,. LAKHS MAJORLY FROM AGNIS BULLION HOUSE LLP, SALASAR SYNTHETICS, SR BULLION JEWELLERS, RTGS RS. ,. LAKHS MAJORLY FROM SR BULLION JEWELLERS, ARIHANT BULLION AND JEWELS LLP, BILASA IMPEX, REMITTANCE RS. . LAKHS MAJORLY FROM MOHAMMED HAMMOODI HASHIM ALABID IN USD FROM U.A.E. FOR FOREIGN DIRECT INVESTMENT IN EQUITY, INTERNET FUND TRANSFERS RS. ,. LAKHS FROM ARIHANT BULLION AND JEWELS LLP , MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES, FIXED DEPOSIT RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY TO ICICI LOMBARD GIC LTDPERSONAL LINE COLL, AGNIS BULLION HOUSE LLP, CUSTOM DUTY, NEFT RS. . LAKHS MAJORLY TO MMTC PAMP INDIA PVT LTD, RTGS RS. . LAKHS MAJORLY TO MMTC PAMP, REMITTANCE RS. ,. LAKHS MAJORLY TO DIJLLAH JEWELLERY FZCO OF U.A.E. FOR GOLD IN USD, MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, INTERNET FUND TRANSFER RS. . LAKHS TO BHOLASONS JEWELLERS GOLD SOUK , MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCE.",{"entities":[(936, 954,"FOREIGN ORG"),(561, 583,"FOREIGN ORG"),(1014, 1036,"FOREIGN ORG"),(3294, 3315,"FOREIGN ORG"),(3362, 3391,"FOREIGN ORG"),(4412, 4434,"FOREIGN ORG"),(4487, 4505,"FOREIGN ORG")]}),
            ("SIMILAR TRANSACTION PATTERN WERE SEEN IN THE ACCOUNT. NEGATIVE MEDIA REPORT FOUND AGAINST CUSTOMER, SEBI IMPOSES RS . CRORE FINE ONENTITIES FOR FOR INDULGING IN FRAUDULENT TRADING THAT LED TO ARTIFICIAL VOLUME IN THE SHARES OF SHAKTI PUMPS INDIA LTD. WEBLINK IS HTTPS://WWW.BUSINESSSTANDARD.COM/ARTICLE/PTISTORIES/SEBIIMPOSESRSCRFINEONENTITIESFORFRAUDTRADE_.HTML::TEXT=CAPITALMARKETSREGULATORSEBIMONDAY,SHAKTIPUMPSINDIALTD. IN VIEW OF HIGH VALUE TRANSACTIONS IN THE ACCOUNT AND NEGATIVE MEDIA REPORT FOUND AGAINST THE CUSTOMER, STR IS BEING FILED.SHAKTI PUMPS INDIA LTD, HAVING ADDRESS : PLOT NO , ,INDL, AREA SECTOR  , PITHAMPUR, DHAR, MADHYAPRADESH, INDIA, , CONTACT NUMBER: + AND EMAIL ID: ACCOUNTS@SHAKTIPUMPSINDIA.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, , CHHOTI KHAJRANI, MALAV PARISAR, INDORE, MADHYA PRADESH, , SINCE . DATE OF INCORPORATION IS . PAN OF THE ACCOUNT IS AAECSL. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . OTHER RELATIONSHIP OF CUSTOMER ARECURRENT ACCOUNT,CARD,FASTAG,FD. TRANSACTION LINKED ACCOUNT IS . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. DIRECTOR OF THE REPORTED ACCOUNT ARE SUNIL MANOHARLAL PATIDAR ADHPPP, DINESH PATIDAR ADHPPA, RAMESH PATIDAR AHPPPN, NAVIN SUNDERLAL PATWA AGXPPK. KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION. KYC UPDATION IS NOT DUE. CUSTOMER IS BUSINESS OF SOLAR PUMPS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS GREATER THAN EQUAL TOCR. MANUAL ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS AND NEGATIVE MEDIA REPORT. BALANCE IN THE ACCOUNT ATWAS RS. ,. LAKHS DEBIT. FY  FROMTO: TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH REMITTANCE RS. ,. LAKHS MAJORLY FROM SHAKTI PUMPS FZE,SHAKTI PUMPS USA LLCFLORIDA, SHAKTI PUMPS USA LLC. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH OTHERS RS. ,. LAKHS SHAKTI PUMPS INDIA LTD. FY  FROMTO: TOTAL CREDIT SIDE RS. . LAKHS OF WHICH REMITTANCE RS. . LAKHS. TOTAL DEBIT SIDE RS. . LAKHS. CREDITS FOLLOWED BY DEBITS WERE SEEN. FOR EXAMPLE, FROMTO , RS. . CRORES WERE CREDITED IN PARTS THROUGH TRANSFER FROM FC SETTLEMENT A/C USD, INWARD REMITTANCE FROM MUTLUSU POMPA SAN. TIC. A. IN THE CURRENCY USD FROM THE COUNTRY TURKEY FOR WHICH THE PURPOSE MENTIONED WAS ADVANCE RECEIVED AGAINST EXPORT, KONG PISETH IN THE CURRENCY OF USD FROM THE COUNTRY COMBODIA FOR WHICH THE PURPOSE MENTIONED WAS  EXPORT BILL COLLECTION, SHAKTI PUMPS USA LLC IN THE CURRENCY USD FROM THE COUNTRY US FOR WHICH THE PURPOSE MENTIONED WAS ADVANCE RECEIVED AGAINST EXPORT AND SIMULTANEOUSLY SAME AMOUNT GOT DEBITED IN PARTS THROUGH TRANSFER TOWARDS SHAKTI PUMPS INDIA LTD . ALSO FROMTO , RS. . LAKHS WERE CREDITED IN PARTS THROUGH TRANSFER FROM FC SETTLEMENT A/C USD, INWARD REMITTANCE FROM SHAKTI PUMPS FZE IN THE CURRENCY USD FROM THE COUNTRY U.A.E. FOR WHICH THE PURPOSE MENTIONED WAS ADVANCE RECEIVED AGAINST EXPORT AND ON , RS.LAKHS WERE DEBITED THROUGH TRANSFER TOWARDS SHAKTI PUMPS INDIA LTD .",{"entities":[(1829, 1845,"FOREIGN ORG"),(1846, 1873,"FOREIGN ORG"),(1875, 1895,"FOREIGN ORG") ,(2249, 2266,"FOREIGN ORG"),(2389, 2400,"FOREIGN ORG"),(2511, 2531,"FOREIGN ORG"),(2859, 2875,"FOREIGN ORG") ]}),
            ("INTERCONNECTED FUNDS TRANSFER TXNS WITHIN THE A/CS AMOUNTED TO RS.. CR. UTURN TXNS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS.. CR WAS RECEIVED AND RS.. CR WAS SENT BACK TO THE SAME GROUP OF A/CS MAINTAINED WITH OTHER BANK WHICH MAINLY COMPRISE OF A/CS OF APPLE INTERNATIONAL, MAHADEV TRADERS, NISHA GOEL, RAJNISH TRADERS, GOPAL OVERSEAS AND SHRI MAHADEV JI EXPORTS.DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY SHRI MAHADEVJI EXPORTS, IT WAS NOTED THAT THE PREMISE WAS ASQ. FT. OWNED PREMISE WHERE OFFICE SET UP WAS SEEN, BUT NO SIGN BOARD OR STOCK OF GOODS COULD BE SIGHTED.ON ENQUIRY IT WAS LEARNT THAT ALL THE ABOVE MENTIONED ARE FAMILY A/CS WITH PRANSHU GOEL, HIS MOTHER NISHA GOEL AND HIS RELATIVESMANOJ JAIN AND AJAY KUMAR AS THE PROPRIETORS. HOWEVER, VIJAY GOEL  FATHER OF PRANSHU GOEL WAS MANAGING THE BUSINESS. ON ENQUIRING REASON FOR VIJAY GOEL NOT BEING A PROPRIETOR / RELATED TO ANY OF THE ABOVE MENTIONED A/CS, IT WAS INFORMED THAT DUE TO HEALTH CONDITIONS HE NAMED HIS FAMILY MEMBERS AS THE PROPRIETORS. IT IS PERTINENT TO NOTE THAT THE SAID VIJAY GOEL PAN NO. AFUPGJ IS A DIRECTOR IN KVM APPARELS P LTD AND SOGO TEXTILES P LTD WHICH AS PER WEB CHECK ARE STRUCK OF FOR EFILING BY REGISTRAR OF COMPANIES. IT APPEARS THAT THE PROPRIETORS MAY BE BENAMIS OF VIJAY GOEL AND JUSTIFICATIONS FURNISHED MAY HAVE BEEN FABRICATED TO SATISFY BANK OFFICIALS TO CONCEAL IDENTITY OR LINKAGE OF VIJAY GOEL TO THESE A/CS.SHRI MAHADEV JI EXPORTS, GOEL EXIM AND VINAYAK STEEL ARE YET TO SUBMIT BOES PERTAINING TOCASES OF AIR TOTALLING TO USD ,, WHICH COMPRISE OFCASES  USD , PENDING FOR A PERIOD ABOVEMONTHS TOYEAR,CASES  USD , PENDING FOR A PERIOD ABOVEMONTHS TOMONTHS ANDCASES  USD ,, PENDING FOR A PERIOD LESS THANMONTHS.AS A PART OF DUE DILIGENCE, BALANCE SHEET OF SHRI MAHADEVJI EXPORTS WERE OBTAINED, WHEREIN FOR YEAR ENDING MARCH , DUTY DRAWBACK RECEIVED AMOUNTED TO RS.. CR AND ROSL WAS INDICATED AS RS.. CR. ROSL AS PER WEB CHECK IS REBATE OF STATE LEVIES TOWARDS EXPORT OF GARMENTS. IT APPEARS UNUSUAL THAT THE CUSTOMER WHO HAS INDICATED THAT HE IS INTO BUSINESS OF STEEL IMPORT HAS RECEIVED DUTY DRAWBACK CREDIT WHICH IS EXPORTERS INCENTIVE AND ROSL WHICH IS GOVERNMENT INCENTIVE TOWARDS GARMENT EXPORTS. BOTH THE NATURE OF BUSINESS AND THE PRODUCT APPEARS TO CONTRADICT THE TYPE OF INCENTIVE INDICATED IN THE BALANCE SHEET.IT IS ALSO NOTED THAT THOUGH THE CUSTOMER HAS FORMED ENTITY AND OPENED A/C IN THE NAME OF SHRI MAHADEV JI EXPORTS, THE CUSTOMER HAS ROUTED IMPORT RELATED TXNS THROUGH THE A/C.BASIS TXN PATTERN AND INFORMATION AVAILABLE, IT COULD BE POSSIBLE THAT THE CUSTOMER MAY HAVE RECEIVED EXPORT CREDITS IN A/C MAINTAINED WITH OTHER BANK INDICATING AS EXPORT OF GARMENTS, MAY HAVE CLAIMED EXPORT INCENTIVES LIKE DUTY DRAWBACK AND ROCL THEREBY DEPRIVING EXCHEQUER WITH SUCH EXPORT INCENTIVES AND OPENED ANOTHER A/C INDICATING STEEL IMPORT BUSINESS TO REMIT SUCH FUNDS RECEIVED OVERSEAS AS IMPORT OF STEEL. SUCH TXNS MAY HAVE NO ECONOMIC RATIONALE AND MAY HAVE BEEN DONE POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING AND HAWALA SETTLEMENTS. A/C OF SHRI MAHADEVJI EXPORTS IS ACTIVE WITH BALANCE OF RS./ AS ON MAY , . QUANTUM OF TXNS FOR CURRENT FINANCIAL YEAR  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. CORRESPONDING TXN TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE:  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR.PRANSHU GOELYEAR OLD AS A PROPRIETOR MAINTAINS A CURRENT IN THE NAME OF SHRI MAHADEV JI EXPORTSSINCE JUL AT OUR BRANCH LOCATED IN C/ , ASHOK VIHAR, PHASEII, NEAR DEEP MARKET NEW DELHI, . THE ENTITY WAS FORMED ON JUN AND HAS SUBMITTED COPIES OF PAN CPTPGF, GST REG. CERT. CPTPGFZM AND IEC CERT CPTPGF AS KYC DOCS DURING A/C OPENING. CUSTOMERS REG. ADDRESS WITH BANK IS ST FLOOR PLOT NO. ABLOCK A WAZIRPUR INDUSTRIAL AREA NEAR SHRI RAM CHOWK NORTH WEST DELHIAND HAS REGISTERED MOBILE NO.. THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED THAT THE ENTITY IS INTO WHOLESALE BUSINESS OF STEEL WITH ANNUAL TURNOVER OF RS.. CR. BASIS INTERCONNECTEDNESS, OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A/CS IN THE NAMES OF SOLE PROPRIETORSHIP CONCERNS  GOEL EXIM , PROP.  NISHA GOEL/ MANDATE HOLDER  PRANSHU GOEL, SHRI SIDDHI VINAYAK EXPORTS , PROPNISHA GOEL, VINAYAK STEEL /, PROP  AJAY KUMAR AND APPLE INTL.L , PROP  MANISH JAIN. IT WAS INDICATED THAT THE ENTITIES ARE INTO BUSINESS OF IRON/ STEEL WITH ANNUAL TURNOVER IN THE RANGE OF RS.. CRRS.. CR. BESIDES SAVINGS A/CS WERE OPENED IN THE NAMES OF PRANSHU GOEL , NISHA GOEL , MANISH JAIN , PRANSHU GOEL JOINTLY WITH NISHA GOELAND RIYA GOEL JOINTLY WITH NISHA GOELAND IN THE NAME OF MINOR RIYA GOEL WITH NISHA GOEL AS THE NATURAL GUARDIAN . THE MENTIONED A/CS WERE OPENED DURING THE PERIOD OCTOBER , APR ,AT ASHOK VIHAR, NETAJI SUBHASH PLACE, ROHINI, SHALIMAR BAUG AND PITAMPURA BRANCHES IN NEW DELHI. WHILE REVIEWING REPORT ON ADVANCE IMPORT REMITTANCES AIR DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE AIRS WERE DONE THROUGH THE A/C OF SHRI MAHADEV JI EXPORTS. ON VERIFYING THE A/C ACTIVITY, RELATED A/CS WERE IDENTIFIED WHEREIN SIMILAR PATTERN AND INTERCONNECTED TXNS WERE NOTED. HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED A/C TXNS SINCE APRIL ,MAY ,IT WAS NOTED THAT CHEQUE DEPOSITS AMOUNTED TO RS.. CR AND IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.. CR MAINLY FROM SHIV ENTPS RS.. CR, ICICI/ UBI  , SHREE PARWATI STEEL AND ALLOYS RS.. CR, HDFC  , SHRI MAHADEV JI EXPORTS RS.. CR, CANARA BANK AND SINGHAL STEELS RS.. CR, ICICI.INTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS.. CR MAINLY FROM SWASTIK OVERSEAS RS.. CR, A/C NO AND SOHUM TRADING CO. RS.. CR, A/C NO.. THE SAID A/CS WOULD BE REVIEWED SEPARATELY AND STR WILL BE FILED IN CASE SUSPICIOUS.SUBSEQUENT TO SUCH CREDITS, ADVANCE IMPORT REMITTANCES AMOUNTED TO RS.. CR AND DIRECT IMPORT REMITTANCES AMOUNTED TO RS.. CR MAINLY FAVOURING PT UNIQUE STEEL INDONESIAINDONESIA RS.. CR, GUANGDONG GUANGXIN GOLDTEC  CHINA RS.. CR, LEO METALS LIMITEDHK RS.. CR, SHANDONG MENGYIN  CHINA RS.. CR AND HONG KONG WINNERHK RS.. CR TOWARDS IMPORT OF COLD ROLLED STAINLESS STEEL COIL.IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.. CR MAINLY FAVOURING SVJ EXIM SOLUTIONS RS.. CR, ICICI/ IOB/ AXIS, BALJEET SINGH RS.. CR, ICICI AND CHITRANJAN PAL SINGH RS.. CR, ICICI  .INTERNAL FUNDS TRANSFER PAYMENTS AMOUNTED TO RS.. CR MAINLY FAVOURING EMPIRE TRADING RS.. CR, A/C NO. AND JAINIT ENTPS RS.. CR, A/C NO. . BANK HAS ALREADY FILED STR ON A/CS OF EMPIRE TRADING VIDE BATCH IDAND JAINIT ENTPS VIDE BATCH ID .",{"entities":[(6002, 6046,"FOREIGN ORG"),(6099, 6119,"FOREIGN ORG"),(6056, 6084,"FOREIGN ORG"),(6165, 6183,"FOREIGN ORG"),(6129, 6152,"FOREIGN ORG")]}),
            ("AS PER WEB CHECK SKIRE HK LTD HONG KONG IS A MANUFACTURER AND EXPORTER OF GENERAL MERCHANDISE LIKE BULBS, ELECTRONICS ETC. LINK  HTTPS://WWW.SKIREHK.COM/AND LAKSH JEWELLERY LLCUAE IS INTO BUSINESS OF DIAMOND, PRECIOUS STONES, GEMS TRADE ETC. NO WEBSITE NOTED. NO SPECIFIC INFORMATION WAS AVAILABLE ABOUT NATURE OF BUSINESS OR GOODS DEALT WITH BY BEST RISE ENTP LTD SUITE. IT APPEARS THAT THE CUSTOMER JOWELA EXIM P LTD MAY HAVE RECEIVED EXPORT PROCEEDS FROM SKIRE HK LTD HONG KONG WHICH IS IN UNRELATED BUSINESS OR LAKSH JEWELLERY LLC UAE WHOSE BUSINESS CREDENTIALS ARE NOT VERIFIABLE BASIS INFORMATION ON PUBLIC DOMAIN.IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS..CR MAINLY FROM LAXMI DIA JEWEL P LTD RS.. CR, SBI/ , BOI  , SHREENATHJI GEMS AND JEWELS PLTD RS.. CR, SBI AND DIAMOND INDIA LTD RS.. CR, AXIS  .INTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS.. CR MAINLY FROM RONAK GEMS P LTD RS.. CR, A/C NO, . BANK HAS ALREADY FILED STR ON THE A/C OF RONAK GEMS P LTD VIDE STR NO. ON .. ALONG WITH RIDHISIDDHI BULLION AND RELATED A/CS BASIS NEGATIVE MEDIA REPORT, VIDE BATCH IDIN REGARDS TO NEGATIVE MEDIA REPORT AND DURING DEMONETISATION VIDE BATCH ID .SUBSEQUENT TO SUCH CREDITS, IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.. CR MAINLY FAVOURING DIAMOND INDIA LTD	RS.. CR, AXIS AND JOWELA EXIM PVT LTD RS.. CR, HDFC/ BCBM  .INTERCONNECTED FUNDS TRANSFER TXNS WITHIN THE A/CS AMOUNTED TO RS.. CR. UTURN IMPS/NEFT/RTGS TXNS WERE NOTED WHEREIN RS.. CR WAS RECEIVED AND RS.. CR WAS SENT BACK TO THE A/CS OF DIAMOND INDIA LTD, KAVISH EXPORTS, SHREENATHJI GEMS AND JEWELS P LTD AND LAXMI DIA JEWEL P LTD MAINTAINED WITH OTHER BANKS. IT APPEARS THAT SUCH TXNS MAY HAVE BEEN DONE WITHOUT ANY ECONOMIC RATIONALE FOR LAYERING OF FUNDS OR POSSIBLY TO INFLATE TURNOVER.DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, NAME OF JOWELA EXIM P LTD WAS DIGITAL PRINTED ON GLASS BOARD. IT WAS ASQ. FT. RENTED COMMERCIAL PREMISE WHERE OFFICE SET UP WAS SEEN WITHEMPLOYEES. HOWEVER IT IS NOT KNOWN IF THE CUSTOMER HAS THE NET WORTH / BUSINESS EXISTENCE TO ROUTE SUCH HIGH VALUE AND VOLUME OF TXNS.TXNS IN THE A/C OF JOWELA EXIM P LTD FAR EXCEED THE ANNUAL TURNOVER INDICATED AT THE TIME OF A/C OPENING. NO PAYMENTS TOWARDS TAXES WERE NOTED THROUGH THE SAID A/C DURING REVIEW PERIOD. FURTHER TOTAL TAX PAYMENTS AMOUNTING TO RS.. LAKHS WERE NOTED ONLY FROM THE A/C OF PRITI SUNIL DOKANIA DURING REVIEW PERIOD.CUSTOMER JOWELA EXIM P LTD IS YET TO SUBMIT DOCUMENTS PERTAINING TOCASES OF BILL REALIZATIONS TOTALLING TO USD , PENDING FOR A PERIOD LESS THAN A YEAR.BASIS TXN PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN THE NAME OF JOWELA EXIM P LTD AND A/C MAY HAVE BEEN OPENED TO RECEIVE EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS AND SUCH TXNS MAY NOT HAVE ANY ECONOMIC RATIONALE. FURTHER, FUNDS RECEIVED MAY HAVE BEEN CHURNED WITHIN THE A/CS AND REMITTED AS RTGS/NEFT REMITTANCES FOR LAYERING OF FUNDS. A/C OF JOWELA EXIM P LTD IS ACTIVE WITH BALANCE OF RS.. CR AS ON JUNE , . QUANTUM OF TXNS FOR CURRENT FINANCIAL YEAR  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. CORRESPONDING TXN TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE:  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. : DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR.A CURRENT A/C BEARING NO.WAS OPENED IN THE NAME OF JOWELA EXIM P LTD ON AUG AT OUR BRANCH LOCATED IN OM SHREE GANESH CHS, SHOP NO,,,,, CHIKOOWADI, BORIVALI WEST, MUMBAI ,MAHARASHTRA . THE ENTITY WAS FORMED ON JUL AND HAS SUBMITTED COPIES OF PAN AAECJH, BOARD RESOLUTION, CERT. OF INCORP. UMHPTC AND MOA/ AOA AS KYC DOCS AT THE TIME OF A/C OPENING. CUSTOMERS REG. ADDRESS WITH THE BANK ISAURA BIPLEX SENATE SV ROAD BORIVALI WEST MUMBAI  . PRANAV SUNIL DOKANIA, BHAVIN CHANDRAKANT SHAH AND HARDIK BHARAT SHAH ARE THE AUTHORISED SIGNATORIES WHILE BHAVIN CHANDRAKANT SHAH IS THE BENEFICIAL OWNER. PRANAV SUNIL DOKANIA, BHAVIN CHANDRAKANT SHAH AND HARDIK BHARAT SHAH HAVE REGISTERED MOBILE NOS. ,ANDRESPECTIVELY WITH THE BANK. THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED AT THE TIME OF A/C OPENING THAT THE ENTITY IS INTO TRADING OF JEWELLERY BUSINESS WITH ANNUAL TURNOVER OF RS.. CR. BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED A/C, OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A/CS IN THE NAMES OF PRIT DIAJEWELS P LTD , , , , SHREENATHJI GEMS AND JEWELS P LTD , , AND LAXMI DIA JEWEL P LTD . PRANAV DOKANIA AND PRITI DOKANIA ARE THE COMMON DIRECTORS IN THE MENTIONED A/CS. IT WAS INDICATED THAT THE ENTITIES ARE INTO BUSINESS OF JEWELLERY WITH ANNUAL TURNOVER IN THE RANGE OF RS.. CR TO RS.. CR. BESIDES SAVINGS A/C WERE OPENED IN THE NAMES OF PRANAV SUNIL DOKANIA , PRITI SUNIL DOKANIA JOINTLY WITH PRANAV SUNIL DOKANIA //, SUNIL DOKANIA HUF WITH PRANAV SUNIL DOKANIA AS KARTA , BHAVIN CHANDRAKANT SHAHAND HARDIK BHARAT SHAH .THE MENTIONED A/CS WERE OPENED DURING THE PERIOD JULY ,TILL OCT ,AT BORIVALI W, J B NAGAR, BORIVALI C.CIRCLE BRANCHES IN MUMBAI. NO TXNS / UNUSUAL TXNS WERE NOTED IN THE A/CS MENTIONED ABOVE DURING REVIEW PERIOD EXPECT IN THE A/CS OF JOWELA EXIM P LTD , PRITI SUNIL DOKANIA JOINTLY WITH PRANAV SUNIL DOKANIA / , PRIT DIAJEWELS P LTDAND LAXMI DIA JEWEL PRIVATE LTD . WHILE REVIEWING REPORT ON EXPORT ADVANCE REMITTANCE CREDITS RECEIVED IN A/CS MAINTAINED WITH THE BANK IT WAS NOTED THAT HIGH VALUE EXPORT ADVANCE REMITTANCE CREDITS WERE RECEIVED IN THE ABOVE MENTIONED A/C OF JOWELA EXIM P LTD. ON VERIFYING THE A/C ACTIVITY IT WAS NOTICED THAT SUCH CREDITS WERE FOLLOWED BY HIGH VALUE IMPS/NEFT/RTGS PAYMENTS. HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLL. GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED A/C TXNS SINCE APRIL , JUNE , , IT WAS NOTED THAT FUNDS TOTALLING TO RS.. CR WERE RECEIVED IN THE A/C OF JOWELA EXIM PRIVATE LTD BY WAY OF EXPORT ADVANCE REMITTANCE RS.. CR AND DIRECT BILL REALIZATION RS.. CR MAINLY FROM SKIRE HK LTDHONG KONG RS.. CR, LAKSH JEWELLERY LLC  UAE RS.. CR AND BEST RISE ENTP LTD SUITEHONG KONG RS.. CR TOWARDS EXPORT OF INDIAN MADE JEWELLERY CONTAINING ASSORTED KT GOLD .",{"entities":[(17, 29,"FOREIGN ORG"),(458, 470,"FOREIGN ORG"),(515, 538,"FOREIGN ORG"),(6114, 6141,"FOREIGN ORG"),(6083, 6095,"FOREIGN ORG")]}),
            ("MARIYAM ENTERPRISE, MOBILE MONEY TRANSFERS RS.. LAKHS ININSTANCES AND OTHERS RS..LAKHS. FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. LAKH, REMITTANCE RS.. LAKHS. TRANSACTION PATTER SHOWS FUNDS CREDITED THROUGH INWARD REMITTANCE, INTERNET FUND TRANSFERS AND WHICH IS IMMEDIATELY DEBITED THROUGH INTERNET FUND TRANSFERS. FOR EXAMPLE, ONRS.. CRORES CREDITED THROUGH INWARD REMITTANCE FROM M/S. SIDDHART ENTERPRISE, BANGLADESH IN USD CURRENCY AND REASON WAS MENTIONED AS CUMINJEERALOOSE, FROM M/S. SHOTOTA TRADERS, BANGLADESH IN IN USD CURRENCY AND REASON WAS MENTIONED AS INDIAN WHEAT AND ONRS..CRORES DEBITED THROUGH INTERNET FUND TRANSFERS TO MANGALAM ENTERPRISES , DIAMOND FRUIT COMPANY . ALSO ONRS..LACS CREDITED FROM R.K.P. ENTERPRISE, BANGLADESH IN IN USD CURRENCY AND REASON WAS MENTIONED AS DRY RED CHILLY GENUS CAPSICUM AND ONRS..LACS DEBITED THROUGH RTGS TO POTENT PRODUCTS PVT. LTD ACCOUNT NUMBER: , IFSC CODE UTIB, SITALADAS GOVINDRAM KEWLANI ACCOUNT NUMBER , IFSC CODE UCBA. SIMILAR PATTERN WAS OBSERVED WHERE HIGH VALUE TRANSACTION, INWARD REMITTANCE AND HIGH CREDIT TURNOVER IN THE FINANCIAL YEAR FY :TILLIS RS. CRORES WHICH IS VERY HIGH INMONTHS AS COMPARED TO THE DECLARED TURNOVER. TOTAL TURNOVER IN THE FINANCIAL YEAR , WAS RS. ,. LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS. LAC TO LESS THAN RS. CRS. IN VIEW OF LARGE VALUE NON CASH TRANSACTION IN CURRENT ACCOUNT AND HIGH TURNOVER, STR IS BEING FIELD.MTE EXIM PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS: , ARMENIAN STREET, TH FLOOR, BURRABAZAR, ROOM NO , KOLKATA, WEST BENGAL, INDIA, , CONTACT NUMBER: + AND EMAIL ID: MTEEXIM_@YAHOO.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, KOLKATA  PORTUGUESE CHURCH STREET, ICICI BANK LTD, CATHEDRAL ANNEXE, GROUND FLOOR,PORTUGUESE CHURCH STREET, KOLKATA, WEST BENGAL, , SINCE . DATE OF INCORPORATION IS . RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT. PAN NUMBER OF THE ACCOUNT IS AANCMP. GSTIN AS PER BANK RECORDS IS AANCMPZM. OTHER RELATIONSHIP OF CUSTOMER ARE , , . TRANSACTION LINKED ACCOUNTS ARE , . BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS JAYEETI JAISWAL WITH PAN NO ASJPCG, SUMAN JAISWAL WITH PAN NO ASYPPN. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE ENTITY PROOF MOA, AOA, BOARD RESOLUTION, PAN AANCMP, CINUWBPTC. CUSTOMER PROFILE IS INTO FOOD GRAINS AS PER BANK RECORDS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. LAC TO LESS THAN RS. CRS. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTION IN CURRENT ACCOUNT. BALANCE IN THE ACCOUNT ATWAS RS.. LAKHS DEBIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROMTILLSHOWS TOTAL CREDIT RS. ,. LAKHS OF WHICH RS. BY CASH, CLEARING RS.LAKHS FROM MAA TARA EXIM PRIVATE LIMITED, S P ENTERPRISE, NEFT RS. , FROM EDI DUTY DRAWBACK DISBURSAL, RTGS RS. . LAKHS FROM MAHEMUD PAN HOU, JP FLOUR MILLS, ABHI TRADERS, REMITTANCE RS. ,. LAKHS FROM M/S RAJ ENTERPRISE, R.K.P. ENTERPRISE, M/S. SIDDHART ENTERPRISE, INTERNET FUND TRANSFERS RS.LAKHS FROM MTE EXIM PRIVATE LIMITED , MOBILE MONEY TRANSFERS RS. , ININSTANCES AND OTHERS RS. ,. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. ,. LAKHS OF WHICH CASH WAS RS.LAKHS BY BRANCH, CLEARING RS. ,. LAKHS TO SRIVISHNUPRIYA CHILLI, ANILTRADING INDIA PVT LTD, BALASAHEB GANGARAMSHE, RTGS RS. ,. LAKHS MAJORLY TO TO KRS TRADERS, SRI VISHNU PRIYA CHILL, MANGALAM ENTERPRISES, DIAMOND FRUIT COMPANY, REMITTANCE RS.. LAKHS, INTERNET FUND TRANSFER RS. ,.LAKHS MAJORLY TO S P ENTERPRISE , MARIYAM ENTERPRISES , MOBILE MONEY TRANSFERS RS.. LAKHS ININSTANCES AND OTHERS RS.. LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL, TRANSFERS RS. . LAKHS FROM MTE EXIM PRIVATE LIMITED, S P ENTERPRISE, CLEARING RS. . LAKHS FROM CMS_, MAA TARA EXIM PRIVATE LIMITED, S P ENTERPRISE, NEFT RS. . LAKHS FROM BONGAON, SBI BASIRHAT, GHOJADANGA CUSTOMS, RTGS RS. . LAKHS FROM SNOWBLUE ENCLAVE PVT LTD, ABHI TRADERS, DILIP KUMAR CHANDRA, REMITTANCE RS. ,. LAKHS FROM M/S RAJ ENTERPRISE, R.K.P. ENTERPRISE, M/S. SIDDHART ENTERPRISE, INTERNET FUND TRANSFERS RS. ,. LAKHS MAJORLY FROM MTE EXIM PRIVATE LIMITED, MARIYAM ENTERPRISE, MOBILE MONEY TRANSFERS RS.. LAKHS ININSTANCES AND OTHERS RS.. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS.LAKHS BY BRANCH, TRANSFERS RS. ,. LAKHS TO MAMA EXPORT, KISSAN JAI KISSAN TRADERS, MTE EXIM PRIVATE LIMITED, CLEARING RS. ,. LAKHS TO KISSANJAIKISSANTRADE, MAAVINDHWASHINITRADE, SRIVISHNUPRIYACHILLI, NEFT RS. . LAKHS TO NANDHINI CHILLIES, HK ORGANIC, MANOJ TRADING COMPAN, RTGS RS. ,. LAKHS TO KRS TRADERS, SRI VISHNU PRIYA CHILL, REMITTANCE RS. . LAKHS, INTERNET FUND TRANSFER RS. ,. LAKHS MAJORLY TO MTE EXIM PRIVATE LIMITED,",{"entities":[(492, 509,"FOREIGN ORG"),(594, 608,"FOREIGN ORG"),(818, 835,"FOREIGN ORG"),(3175, 3192,"FOREIGN ORG"),(4311, 4328,"FOREIGN ORG")]}),
            ("SOURCE OF FUNDS IS FROM SAME ENTITIES TO MULTIPLE COMPANIES UNDER THE SAME GROUP FOLLOWED BY OUTWARD REMITTANCE SEEMS TO BE UNUSUAL. TOTAL TURNOVER IN THE FINANCIAL YEAR , WAS RS. ,. LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL INCOME OF RS. LACS. IN VIEW OF HIGH TURNOVER IN NEWLY OPENED GET CLOSED IN SHORT SPAN OF TIME PERIOD AND HIGH TURNOVER, STR IS BEING FILED.WATERMARK STUDIOS, PARTNERSHIP FIRM, HAVING ADDRESS: ND, FLOOR / FLAT NO, , RELIANCE CLASSIC, ABOVE ANDHRA, HYDERABAD, TELANGANA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: HI@WATERMARKSTUDIOS.IN IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LIMITED, HYDERABAD  BEGUMPET, //A, JAYALAKSHMI TOWERS, MOTILALNEHRU NAGAR, PRAKASHNAGAR, BEGUMPET, HYDERABAD, TELANGANA, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS AADFWA. GSTIN AS PER BANK RECORDS IS AADFWAZ. TRANSACTION LINKED ACCOUNTS IS , . BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH CHAWLA WITH PAN NO AGRPCE, MOHIT BANSAL WITH PAN NO ALRPBC. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON . KYC DOCUMENTS OBTAINED WERE AADHAR, PARTNERSHIP LETTER, PAN  AADFWA, GSTIN. AS PER ENHANCE DUE DILIGENCE, CUSTOMER IS INTO CINE PRODUCTION SERVICES PROVIDER FOR MOVIE OR FILM PRODUCTION. OBSERVED THE BUSINESS BOARD OF THREE ENTITIES IE. CLOUDZ TOURS AND TRAVEL, DIGI MEDIA AND FILMS SOLUTIONS, WATER MARK STUDIO, TINGLE CINE PRODUCTIONS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. LACS. ALERT WAS TRIGGERED FOR HIGH TURNOVER IN NEWLY OPENED GET CLOSED IN SHORT SPAN OF TIME PERIOD. BALANCE IN THE ACCOUNT ATWAS NIL. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHERE CASH IS NIL, CLEARING RS. . LAKH MAJORLY FROM DIGI MEDIA, NEFT RS. . LAKHS MAJORLY FROM DIVERSE MEDIA, BRASH FILM SERVICES, RTGS RS. ,. LAKHS MAJORLY FROM MAGNETIC FILMS PRODUCTION, LIQUID STUDIOS AND SERVICE, DIVERSE MEDIA, REMITTANCE RS. . LAKHS MAJORLY FROM ICICI BANK LIMITED, HONG KONG IN USD FOR REVERSAL OF WRONG ENTRIES, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BRANCH, CLEARING RS. . LAKHS MAJORLY TO T VINAY ASSOCIATES, RTGS RS. . LAKHS MAJORLY TO BRASH FILMS SERVICES, BRASH FILM, LIQUID STUDIO, REMITTANCE RS. ,. LAKHS MAJORLY TO TRISTAR FILMS LIMITED, TRINITY WORLD PTE LIMITED, TRISTAR FILMS LIMITED, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES. FUNDS ARE GETTING CREDITED FROM LIQUID STUDIOS AND SERVICES, BRASH FILM SERVICES, MAGNETIC FILMS PRODUCTION AND ENTIRE FUNDS ARE MADE OUTWARD REMITTANCE TO TRISTAR FILMS LIMITED IN USD TO SINGAPORE FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE. NO PROPER DETAILS FOUND FOR CREDITED SIDE PARTIES ON PUBLIC DOMAIN OF LIQUID STUDIOS AND SERVICES, BRASH FILM SERVICES, MAGNETIC FILMS AND OUTWARD REMITTANCE MADE TO TRINITY WORLD PTE WHICH IS INTO CINE PRODUCTION COMPANY. WWW.TRINITYWORLDPTELTD.COM. CUSTOMER WAS RELUCTANT WHILE SHARING DETAILS AND SUDDENLY STOP TRANSACTION. FOR EXAMPLE, ONRS. . CRORES GOT CREDITED BY RTGS FROM DIVERSE MEDIA IFSC YESB, ACCAND ON THE SAME DAY RS. . CRORES GOT DEBITED BY OUTWARD REMITTANCE TO TRISTAR FILMS LTD ADD BEDOK RESEROIR ROAD, , SINGAPORE, IN USD, WITH INFORMATION AS TOWARDS AUDIO VISUAL SERVICES. IN OTHER EXAMPLE, ONRS. . CRORES GOT CREDITED BY RTGS FROM LIQUID STUDIOS AND SERVICES IFSC SBIN, ACCAND ON THE SAME DAY RS. . CRORES GOT DEBITED BY OUTWARD REMITTANCE TO TRISTAR FILMS LTD ADD BEDOK RESEROIR ROAD, , SINGAPORE, IN USD, WITH INFORMATION AS TOWARDS AUDIO VISUAL SERVICES.",{"entities":[(2476, 2497,"FOREIGN ORG"),(2499, 2524,"FOREIGN ORG"),(2526, 2546,"FOREIGN ORG"),(2753, 2773,"FOREIGN ORG"),(3011, 3027,"FOREIGN ORG"),(3324, 3341,"FOREIGN ORG"),(3610, 3627,"FOREIGN ORG")]}),
            ("ON REVIEWING THE TRANSACTION PATTERN OF STORAIL SOFTWARES PVT LTD AC.IT WAS OBSERVED THAT DURING ENHANCE DUE DILIGENCE CONDUCTED BY BANK OFFICIAL ON // IT WAS FOUND, CLIENT STORAIL SOFTWARES PVT LTD IS PROVIDING CLOUD STORAGE SERVICES TO DOMESTIC CLIENT ZICOM ELECTRONIC SECURITY SYSTEMS LTD, INDIA AND RECEIVES CLOUD STORAGE FROM OVERSEAS ENTITY SAFE DATA STORAGE LTD. CLIENT WAS ATTEMPTING TO MAKE OUTWARD REMITTANCE TO OVERSEAS ENTITY SAFE DATA STORAGE LTD TOWARDS PURCHASE OF CLOUD STORAGE SERVICES. CLIENT STORAIL SOFTWARES PVT LTD HAD SUBMITTED AGREEMENT COPY OF BOTH THE ENTITIES ZICOM ELECTRONIC SECURITY SYSTEMS LTD AND SAFE DATA STORAGE LTD.ON CONDUCTING FURTHER DUE DILIGENCE AND REVIEW OF DOCUMENT IT WAS FOUND THAT THE AGREEMENT COPY WAS FAKE DOMESTIC CLIENT ZICOM ELECTRONIC SECURITY SYSTEMS LTD HAS ALSO CONFIRMED THAT THE AGREEMENT IS NOT SIGNED BY ANY AUTHORIZED PERSON IN THE COMPANY. THEY HAVE NO BUSINESS WITH STORAIL SOFTWARE PVT.LTD. THEY ALSO DENY ANY ASSOCIATION WITH THE SAID COMPANY AS PER AGREEMENT SHARED BY US.DOMESTIC ENTITY ZICOM ELECTRONIC SECURITY SYSTEMS LTD HAS ALSO REMITTED FUND AMOUNTING TO INR .CR TO ANOTHER YES BANK CUSTOMER STOCKPILE SOFTECH SOLUTIONS PVT LTD ACFOR WHICH STR WAS FILED VIDE FIU BATCH ID SAFE DATA STORAGE LTD IS A COMMON OVERSEAS ENTITY RECEIVING CROSS BORDER REMITTANCE FROM YBL CUSTOMER SUPERNITCO INDIA OVERSEAS FOR WHICH STR WAS FILED VIDE FIU BATCH ID , YBL CUSTOMER GREEN GOLD ENT FOR WHICH STR WAS FILED VIDE FIU BATCH IDAND STR WAS FILED ON NON CUSTOMER SPAN SOFTWARE SOLUTIONS PRIVATE LIMITED VIDE FIU BATCH IDTHAT HAD ATTEMPTED OUTWARD REMITTANCE IN FAVOR OF SAFE DATA STORAGE LTD.THERE ARE NO OTHER BUSINESS TRANSACTIONS IN THE ACCOUNTS AND ARE SUSPECTED TO BE USED WITH THE SOLE INTENSION OF REMITTING THE FUNDS ABROADTHE TRANSACTION WAS DECLINED BY BANK FOR FURTHER PROCESSINGDUE TO SUSPICIOUS AGREEMENT COPY AND DOMESTIC/OVERSEAS COUNTERPARTY THAT APPEARS IN DEFAULTER LIST, IT GIVES RISE TO SUSPICION THAT THE CUSTOMER IS INDULGING IN MONEY LAUNDERING IN GUISE OF TRADE TRANSACTIONS. WE SUSPECT THAT CUSTOMER WAS INTENDING TO USE HIS CURRENT ACCOUNT FOR ROUTING THE MONEY FROM THIS ARRANGEMENT.IN LIGHT OF THE ABOVE FACTS, IT HAS BEEN DECIDED ON // TO FILE A STR ON THE SAID CUSTOMER. THE CUSTOMER STORAIL SOFTWARES PVT LTD PAN ABFCSN, DOI , COMMUNICATION PERMANENT ADDRESS INDUSTRIAL PLOT NO E , PHASE A INDUSTRIAL AREA S.A.S. NAGAR MOHALI PUNJAB , REGISTRATION NUMBER , CIN UPBPTC, MOBILE NUMBER , EMAIL ID GURSIMRAN@STORAILSOFTWARES.COM IS MAINTAINING A CURRENT AC NO.SINCEWITH CHANDIGARH BRANCH ADDRESS SCO  SECTOR C CHANDIGARH . THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN FEB . GURSIMRAN SINGH NAVDEEP SINGH ARE AUTHORIZED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT. CUSTOMER IS AN IT SERVICE PROVIDER WITH AN ANNUAL GROSS TURNOVER BETWEEN RS. CRCR.IN THE ACCOUNT OF STORAIL SOFTWARES PVT LTD AC NO. , TRANSACTIONS ANALYSIS FROMTILL , SHOWS NIL CREDIT/DEBITS TRANSACTIONS. IN THE ACCOUNT OF STORAIL SOFTWARES PVT LTD AC NO. , TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW: FROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS. , COMPRISING OF CASH DEPOSIT AMOUNTING TO RS.ANDDEBIT TRANSACTIONS AMOUNTING TO RS. , COMPRISING OF NIL CASH WITHDRAWAL.NO TRANSACTIONS WERE OBSERVED FROM // TO // AS ACCOUNT WAS OPENED ON TH FEB .BALANCE IN THE ACCOUNT AS ONIS RS.",{"entities":[(629, 650,"FOREIGN ORG")]}),
            ("SAMURAI TRADEX, A PROPRIETORSHIP FIRM, HAVING ADDRESS : , B BLOCK, MADANGANJ, KISHANGARH, VINAYAK NAGAR JAIPUR ROAD, AJMER, RAJASTHAN, INDIA, , CONTACT NUMBER: + AND EMAIL ID: SAMURAITRADEX@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, INDIA SHOPPING MALL, INDIA MOTOR CIRCLE, KUTCHERI ROAD, AJMER, RAJASTHAN,SINCE . DATE OF FORMATION IS . PAN OF THE ACCOUNT IS ANMPPC. GSTIN AS PER BANK RECORDS IS ANMPPCZ. TRANSACTION LINKED ACCOUNTS ARE , . AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS RAKESH PANCHOLI PROPRIETOR  ANMPPC, RAHUL PANCHOLY AUTHORISED SIGNATORY ANPPPQ. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN AND LETTER OF DECLARATION OFPROPRIETORSHIPFIRM. KYC UPDATION IS NOT DUE. CUSTOMER IS INTO BUSINESS OF FOOD GRAIN .DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO CR. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS. BALANCE IN THE ACCOUNT ATWAS RS. . LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROMTILLSHOWS TOTAL CREDIT RS. . LAKHS OF WHICH RTGS RS. . LAKHS MAJORLY FROM GANESH AGRO INDUSTRIES, HINAL ENTERPRISE, INTERNET FUND TRANSFERS RS. . LAKHS FROM RAHUL AGRO INDUSTRIES ETC, FIXED DEPOSIT RS. . LAKHS. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. . LAKHS OF WHICH RTGS RS. . LAKHS TO MK INDUSTRIES, INTERNET FUND TRANSFER RS. . LAKHS TO RAHUL AGRO INDUSTRIES ETC. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO: TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY FROM RAHUL AGRO INDUSTRIES, KAPISH INTERNATIONAL, RTGS RS. ,. LAKHS MAJORLY FROM HINAL ENTERPRISE, GUJARAT PEANUT PRODUCT, G S TRADING CO, REMITTANCE RS. ,. LAKHS MAJORLY FROM AGRI COMMODITIES AND FINANCE FZ LLC, KOREA AGROFISHERIES AND FOOD TRADE, INTERNET FUND TRANSFERS RS. ,. LAKHS FROM SAGAR INTERNATIONAL ETC, FIXED DEPOSIT RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH TRANSFERS RS. ,. LAKHS MAJORLY TO SHREESHIV ENTERPRISE, RAHUL AGRO INDUSTRIES, SAMURAI TRADEX, NEFT RS. . LAKHS MAJORLY TO SHREE BHANDERI INDUSTRIES, GANESH ENTERPRISES, RTGS RS. ,. LAKHS MAJORLY TO GUJARAT PEANUT PRODUCT, REMITTANCE RS. . LAKHS MAJORLY TO KOKAI INDO FOOD STUFF TRADING LLC, INTERTEK VIETNAM LIMITED, INTERNET FUND TRANSFER RS. ,. LAKHS TO RAHUL AGRO INDUSTRIES ETC. FY  FROMTO: TOTAL CREDIT SIDE RS. ,. LAKHS. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH RS.BY CASH, REMITTANCE RS. . LAKHS. AS PER TRANSACTION IT IS OBSERVED THAT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT. FOR EXAMPLE ONRS . CRORE WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM SAGAR INTERNATIONALAND ON THE SAME DAY RS . CRORE WERE DEPLOYED THROUGH INTERNET FUND TRANSFER TO RAHUL AGRO INDUSTRIES . ALSO ONRSLACS WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM RAHUL AGRO INDUSTRIES AND ON THE SAME DAY SAME AMOUNT WERE DEBITED THROUGH RTGS TO M K INDUSTRIES , UTIB. SIMILAR TRANSACTION PATTERN WERE SEEN IN THE ACCOUNT. IN VIEW OF LARGE VALUE ROUTING OF FUNDS WITHIN ACCOUNTS WITH HIGH TURNOVER, STR IS BEING REPORTED.",{"entities":[(1893, 1907,"FOREIGN ORG"),(1914,1943,"FOREIGN ORG"),(2336, 2369,"FOREIGN ORG"),(2371, 2395,"FOREIGN ORG")]}),
            ("SAGAR ENTERPRISE, PARTNERSHIP FIRM, HAVING ADDRESS:PLOT NOTOND FLR, KATARGAM, SKY HIGH BLDG,VASTA DEVDI RD, OPP MEHTA PETROL PUMP CASA NGR, SURAT, GUJARAT, INDIA, , CONTACT NUMBER: + AND EMAIL ID: AK.SAGAR@GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, SHREE SHYAM CHAMBERS, RING ROAD, SURAT, GUJARAT, , SINCE . DATE OF INCORPORATION IS . PAN NUMBER OF THE ACCOUNT IS ABXFSH. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS . AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE ARVIND K SHAH PTR ADOPSH , JIGISHABEN P SHAH PTR AQRPSG, DILIP D SHAH PTR ACQPSH , CONSIDERING THAT THERE ARE AN ADDITIONAL AUTHORIZED SIGNATORIES, THEIR DETAILS ARE PROVIDED IN RESPECTIVE FIELDS WITHIN THE XML FILE. BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ARVIND KESHVLAL SHAH WITH PAN NO ADOPSH, DILIP D SHAH WITH PAN NO ACQPSH, JIGISHABEN PRAWNESH BHAI WITH PAN NO AQRPSG. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN  ABXFSH, PARTNERSHIP DEED. DATE OF LAST KYC UPDATION . CUSTOMER IS INTO BUSINESS IMPORT AND EXPORT OF DIAMONDS. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT. BALANCE IN THE ACCOUNT ATWAS RS.. DEBIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO: TOTAL CREDIT SIDE RS. . LAKHS OF WHICH RS.BY CASH, REMITTANCE RS. . LAKHS, . TOTAL DEBIT SIDE RS. . LAKHS OF WHICH RS.BY CASH, REMITTANCE RS. . LAKHS, .FY  FROMTO: TOTAL CREDIT SIDE RS. . LAKHS OF WHICH RS.BY CASH , REMITTANCE RS. . LAKHS . TOTAL DEBIT SIDE RS. . LAKHS OF WHICH RS.BY CASH , REMITTANCE RS. . LAKHS .FY  FROMTO: TOTAL CREDIT SIDE RS. . LAKHS OF WHICH RS.BY CASH , REMITTANCE RS. . LAKHS . TOTAL DEBIT SIDE RS. . LAKHS OF WHICH RS.BY CASH , REMITTANCE RS. . LAKHS .FY  FROMTO: TOTAL CREDIT SIDE RS. . LAKHS OF WHICH RS.BY CASH , REMITTANCE RS. . LAKHS . TOTAL DEBIT SIDE RS. . LAKHS OF WHICH RS.BY CASH , REMITTANCE RS. . LAKHS . TRANSACTION PATTERN SHOWS ACCOUNT WAS CREDITED BY FOREIGN INWARD REMITTANCE FROM SAMBHAV DIAM HK LTD, HONKONG. IN USD AND SAME IS DEBITED THROUGH REMITTANCES. ROR EXAMPLE ONUSD..LACS CREDITED THROUGH GRS/GRS SAMBHAV DIAM HK LIMITED FLAT, WHICH IS DEBITED TOWARDS VINAY DIAMONDS DIAMOND BOURSE BANDRA KURLA COMPLEX MUMBAI INDIA : REMITTANCE INFORMATION PAYMENT AGAINST INVOICE ONFOR THE SAME VALUE OF FUND THROUGH REMITTANCE ON , ANOTHER EXAMPLE ONUSD .LACS CREDITED THROUGH GRS/GRS SAMBHAV DIAM HK LIMITED FLAT, OUT OF WHICH SAME VALUE OF FUND DEBITED THROUGH REMITTANCE ON TO PADMA GEMS DIAMOND BOURSE BANDRA KURLA COMPLEX MUMBAI INDIA : REMITTANCE INFORMATION PAYMENT AGAINST INVOICE ON THE SAME DATE. HIGH VALUE FUNDS WERE CREDITED AND IMMEDIATELY DEBITED THROUGH REMITTANCES LEAVING VERY LOW BALANCE AT EOD. NO OTHER BUSINESS RELATED ACTIVITY OBSERVED INTO THE ACCOUNT. ALSO THERE ARE NO TRANSACTION IN THE ACCOUNT AFTER DEC. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS, STR IS BEING FILED.",{"entities":[(2130, 2149,"FOREIGN ORG"),(2257, 2280,"FOREIGN ORG"),(2531,2554,"FOREIGN ORG")]}),
            ("IN OTHER EXAMPLE ONRS. . CRORE GOT CREDITED THROUGH RTGS FROM MATRIX SECURITY STATE BANK OF INDIA, IFSCSBIN, REF WHICH IS DEPLOYED ON SAME DAY WITH RS. . CRORES THROUGH REMITTANCE TOWARDS SAFE DATA STORAGE LTD, IN USD, WHERE PURPOSE IS MENTIONED AS PAYMENT FOR DATA BASE DATA PROCESSING CHARGES. SIMILAR TRANSACTION PATTERN SEEN IN THE ACCOUNT. THERE SEEMS TO BE NO ECONOMIC RATIONALE FOR SUCH TRANSACTIONS. HIGH VALUE TURNOVER OF . CRORE OBSERVED IN SHORT SPAN OFMONTHS. TOTAL TURNOVER IN THE FINANCIAL YEAR , WAS RS. ,. LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS. LACS TO RS. CRS. IN VIEW OF LARGE VALUE ROUTING OF FUNDS WITHIN ACCOUNTS AND HIGH TURNOVER, STR IS BEING FILED.ORCHAD TECHNO SOLUTIONS LLP, LIMITED LIABILITY PARTNERSHIP, HAVING ADDRESS: PLATINUM SQUARE, SR NO/A/B, VADGOAN SHERI, OFFICE NO, PUNE, MAHARASHTRA, INDIA, , CONTACT NUMBER: + AND EMAIL ID: ACCOUNTS@ORCHADTECHNO.COM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD, PANCHSHIL TECH PARK, NEAR GANAPATHI CHOWK, / VIMAN NAGAR, PUNE, MAHARASHTRA, , SINCE . DATE OF INCORPORATION IS . PAN OF THE ACCOUNT IS AAGFOG. GSTIN AS PER BANK RECORDS IS AAGFOGZO. TRANSACTION LINKED ACCOUNTS ARE , . BENEFICIAL OWNERAND AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT IS DEEPAK. WITH PAN BFVPDH, SACHIN AGGARWAL WITH PAN AXYPAQ. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF ACCOUNT IS CLOSED ON . KYC DOCUMENTS OBTAINED WERE AADHAR , PAN AAGFOG OF FIRM, GSTIN. KYC UPDATION IS NOT DUE. AS PER INFORMATION FOUND IN PUBLIC DOMAIN, CUSTOMER IS ENGAGED INTO COMPUTER AND RELATED ACTIVITIES. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. LACS TO RS. CRS. A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE ROUTING OF FUNDS WITHIN ACCOUNTS. BALANCE IN THE ACCOUNT ATWAS RS... FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE: FY  FROMTO : TOTAL CREDIT SIDE RS. ,. LAKHS OF WHICH CASH WAS NIL, CLEARING RS. . LAKHS MAJORLY FROM DEEPAK, NEFT RS. . LAKHS MAJORLY FROM MATRIX SECURITY, DEEPAK MOHAN JETHMALANI, RTGS RS. ,. LAKHS MAJORLY FROM MATRIX SECURITY. TOTAL DEBIT SIDE RS. ,. LAKHS OF WHICH CASH WAS RS. . LAKHS BY BRANCH, TRANSFERS RS. . LAKHS, RTGS RS. . LAKHS MAJORLY TO MATRIX SECURITY, SPARSH SECURITIES, REMITTANCE RS. ,. LAKHS MAJORLY TO SAFE DATA STORAGE LTD, PROFISEA LTD, INTERNET FUND TRANSFER RS. . LAKHS MAJORLY TO E SQUARE IT, MAKEMYTRIP INDIA PVT LTD, CARD TRANSACTIONS RS. . LAKHS, MOBILE MONEY TRANSFERS RS. . LAKHS ININSTANCES. ALERT WAS RAISED AS CUSTOMER IS ROUTING MONEY BY USING NAME OF BIG BRANDS LIKE MATRIX SECURITIES SURVEILLANCE, SPARSH SECURITIES SAFE STORAGE. IN ACTUAL THESE COMPANIES HAS NO RELATION WITH IT. AS PER INVOICE AND AGREEMENT UPLOADED BY SAFE DATA STORAGE LTD WHICH IS UK BASED COMPANY, THE PAYMENT WAS GOING TO SINGAPORE ACCOUNT. BANK HAD REPORTED THIS CLIENT IN DEC END AND CLOSED IT IN . , WHICH IS SUSPICIOUS. AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT. CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT WHICH SEEMS TO BE UNUSUAL. FOR EXAMPLE ONRS. . CRORE GOT CREDITED THROUGH RTGS FROM MATRIX SECURITY STATE BANK OF INDIA, IFSCSBIN, REF WHICH IS DEPLOYED ON SAME DAY WITH SAME AMOUNT THROUGH REMITTANCE TOWARDS SAFE DATA STORAGE LTD, UNIT A VICTORIA WORKS,FAIRWAY PETTS WOOD, UNITED KINGDOM IN USD, WHERE PURPOSE IS MENTIONED AS PAYMENT FOR DATA BASE DATA PROCESSING CHARGES.",{"entities":[(489, 509,"FOREIGN ORG"),(2399, 2420,"FOREIGN ORG"),(2422, 2434,"FOREIGN ORG"),(2835,2855,"FOREIGN ORG"),(3438, 3456,"FOREIGN ORG")]}),
            ("DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT IT WAS ASQ. FT. RENTED PREMISE WITH SIGN BOARD, OFFICE SET UP ANDEMPLOYEES SEEN. HOWEVER NO STOCK OF GOODS WERE SIGHTED AT THE ADDRESS. ON VISIT BY BRANCH OFFICIAL AT THE SITE ADDRESS INDICATED BY CUSTOMER I.E., PLOT NO., SECTOR , PHASE , KASEZ, GANDHIDHAM, KACHCHH, GUJARAT , IT WAS NOTED THAT IT WAS AN UNDER CONSTRUCTION RENTED PREMISE WHERE NO SIGN BOARD WAS SEEN.CUSTOMER IS YET TO SUBMIT BILLS OF ENTRY PERTAINING TOCASES OF ADVANCE IMPORT REMITTANCES TOTALLING TO USD ,, PENDING FOR A PERIOD LESS THANMONTHS ANDCASES TOTALLING TO USD , PENDING FOR A PERIOD ABOVEMONTHS TO LESS THANMONTHS. ALSO CUSTOMER IS YET TO SUBMIT SHIPPING DOCUMENTS PERTAINING TOCASES TOTALLING TO USD , PENDING FOR A PERIOD LESS THAN A YEAR.DURING AN ENQUIRY IT WAS LEARNT FROM CUSTOMER THAT IT IS INTO BUSINESS OF CHEMICALS, DOMESTIC FUNDS RECEIVED IN THE ACCOUNT WERE FROM OWN ACCOUNT MAINTAINED AND THAT FOREIGN IMPORT PAYMENTS WERE TOWARDS IMPORT OF AROMATIC SOLVENT. ON FURTHER ENQUIRY ABOUT REASON FOR HIGH VALUE ADVANCE IMPORT PAYMENTS AS WELL AS EXPORT ADVANCE REMITTANCES FROM A COMMON OVERSEAS PARTY M UNION CHEMICALS TRADING FZC UAE, IT WAS LEARNT THAT THE CUSTOMER PROCURES MATERIALS FROM THE SAID PARTY, PROCESSES IT IN INDIA AND EXPORTS IT BACK TO SAME PARTY. IT COULD BE POSSIBLE THAT THE CUSTOMER MAY HAVE RESORTED TO ROUND TRIPPING OF FUNDS BY RECEIVING AND REMITTING BACK FUNDS TO A COMMON OVERSEAS PARTY AND THE JUSTIFICATION FURNISHED APPEARS TO BE UNUSUAL.ON SCRUTINY OF SAMPLE DOCUMENTS FURNISHED BY THE CUSTOMER FOR EFFECTING ADVANCE IMPORT REMITTANCES IT WAS NOTED THAT THE INVOICES FURNISHED BY DIFFERENT OVERSEAS PARTIES APPEARED TO HAVE IDENTICAL FORMATS, INVOICE NUMBER FORMATS AND STAMPS.NO PAYMENTS TOWARDS TAXES, CUSTOM DUTY OR DOMESTIC TRANSACTIONS WITH ENTITIES IN RELATED BUSINESS WERE NOTED THROUGH THE ACCOUNT DURING REVIEW PERIOD DESPITE OF ROUTING HIGH VALUE AND VOLUME OF TRANSACTIONS.BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS DOUBTFUL THAT THE PROPRIETOR WHO ISYEARS OLD MAY BE ACTUAL BENEFICIAL OWNER OF SUCH HIGH VALUE AND VOLUME OF FUNDS ROUTED THROUGH THE ACCOUNT. IT COULD BE POSSIBLE THAT SHELL ENTITY MAY HAVE BEEN FORMED IN THE NAME OF GKN ENTERPRISES AND TRANSACTIONS MAY HAVE BEEN ROUTED WITHOUT ECONOMIC RATIONALE FOR LAYERING AND ROUND TRIPPING OF FUNDS AND POSSIBLY TO REMIT FUNDS OVERSEAS UNDER THE PRETEXT OF ADVANCE IMPORT REMITTANCES WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENT. ACCOUNTSANDOF GKN ENTERPRISES ARE ACTIVE WITH BALANCE OF RS.,,/ AND RS.,,/ RESPECTIVELY AS ON JULY ,WHILE ACCOUNT NO. IS CLOSED SINCE APRIL , . QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE:  IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. IS DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR. : DEBIT: RS.. CR IN CASH RS.. CR AND CREDITS: RS.. CR IN CASH RS.. CR.SEENIVASAN NYEARS AS A PROPRIETOR MAINTAINS CURRENT ACCOUNTS IN THE NAME OF GKN ENTERPRISES BEARING NUMBERS ,ANDSINCE OCT, OCT AND NOV RESPECTIVELY AT OUR OFFICE NO.,SINDHUI, GROUND FLOOR, PLOT NO., WARD /B, GANDHIDHAM, KUTCH, GUJARAT  BRANCH . THE ENTITY WAS FORMED ON AUG AND HAS SUBMITTED COPIES OF PAN BSTPSD, IEC CERTAND VAT REG. CERT.AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO SECOND FLOOR PLOT NO WARDB POLICE STATION ROAD GANDHIDHAM GJ . SUJITH S PADAYACHIYEARS IS THE MANDATE HOLDER FOR ACCOUNT OPERATIONS IN THE SAID ENTITY. SEENIVASAN N AND SUJITH S PADAYACHI HAVE UPDATED THEIR MOBILE NUMBERSANDRESPECTIVELY WITH THE BANK. THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO BUSINESS OF EXPORT/ IMPORT OF CHEMICALS WITH ANNUAL TURNOVER OF RS.. CR. NO TRANSACTIONS WERE NOTED IN THE ACCOUNTOF GKN ENTERPRISES DURING REVIEW PERIOD. WHILE REVIEWING REPORT ON ADVANCE IMPORT REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE AND VOLUME OF ADVANCE IMPORT REMITTANCES WERE ROUTED THROUGH THE ABOVE MENTIONED ACCOUNTS OF GKN ENTERPRISES. ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH ADVANCE IMPORT REMITTANCES WERE DONE SUBSEQUENT TO HIGH VALUE IMPS/NEFT/RTGS CREDITS AND HENCE THIS REPORT IS FILED ON GKN ENTERPRISES ON THE FOLLOWING GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED ACCOUNT TRANSACTIONS SINCE APRIL ,TILL JUNE , , IT WAS NOTED THAT HIGH VALUE ROUND SUM IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.. CR MAINLY FROM OWN ACCOUNT OF GKN ENTERPRISES MAINTAINED WITH OTHER BANK RS.. CR, SBI, THE LAKSHMI VILAS BANK LTD, SHIV SHAKTI ENTERPRISES RS.. CR, LAKSHMI VILAS BANK LTD AND PARASNATH ORGANICS P LTD RS.. CR, ICICI.EXPORT CREDITS AMOUNTED TO RS.. CR FROM M UNION CHEMICALS TRADING  F Z C RS.. CR AND ARAMEX INTERNATIONAL EXCHANGE PTE. REMITTER AS PER SWIFTAFFHAN INTERNATIONAL PTE LTDSINGAPORE RS.. CR TOWARDS ADVANCE AGAINST EXPORT OF GOODSWATER BASED VARNISHES.SUBSEQUENT TO SUCH CREDITS FUNDS HIGH VALUE ADVANCE IMPORT REMITTANCES AMOUNTED TO RS.. CR FAVOURING GALAXY OIL INTERNATIONAL DMCC  UAE RS.. CR, GREEN PETROCHEM INDUSTRY FZCUAE RS.. CR, SMB GOLDEN GENERAL TRADING CO  KUWAIT RS.. CR ANDUNION CHEMICALS TRADING  F Z CUAE RS.. CR TOWARDS IMPORT OF AROMATIC SOLVENT.DURING WEB CHECK, WEBSITES HTTPS://WWW.UCTGLOBAL.COM/CHEMCOATPRODUCT/ IN THE NAME OF UNION CHEMICAL TRADING, HTTP://GREENPETROCHEM.COM/ IN THE NAME OF GREEN PETROCHEM INDUSTRY AND HTTP://WWW.SMBKUWAIT.COM/ IN THE NAME OF SMB GOLDEN GENERAL TRADING CO WERE NOTED. NO WEBSITE OR SPECIFIC INFORMATION ABOUT NATURE OF BUSINESS OR GOODS DEALT WITH BY GALAXY OIL INTERNATIONAL DMCC UAE COULD BE SEEN. FURTHER THE GENUINENESS OF DETAILS IN THE GIVEN WEBSITES IS NOT KNOWN.INTERCONNECTED FUNDS TRANSFER TRANSACTIONS WITHIN THE ACCOUNTS AMOUNTED TO RS.. CR.IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.. CR MAINLY FAVOURING RBI RS.. CR, GKN ENTERPRISES RS.. CR AND PITAMBER ORGANICS RS.. CR, AU SMALL FINANCE BANK LIMITED  .",{"entities":[(1193, 1221,"FOREIGN ORG"),(5043, 5078,"FOREIGN ORG"),(5088, 5121,"FOREIGN ORG"),(5140,5169,"FOREIGN ORG"),(5354,5389,"FOREIGN ORG"),(5396,5427,"FOREIGN ORG"),(5437, 5477,"FOREIGN ORG"),(5480,5518,"FOREIGN ORG"),(5649,5670,"FOREIGN ORG"),(5715,5738,"FOREIGN ORG"),(5785,5813,"FOREIGN ORG"),(5908,5941,"FOREIGN ORG")]}),
            ("ON SCRUTINIZING THE ACCOUNT OF COGO FREIGHT PRIVATE LIMITED ACCOUNT NOIT WAS OBSERVED THAT CUSTOMER HAS SUBMITTED DOCUMENTS FOR FOREIGN REMITTANCE SEEMS TO BE IRRELEVANT FOR FURTHER PROCEDURE, SAME ON CONFIRMING WITH THE CUSTOMER THEY SAID THAT THIS IS AN ADVANCE PAYMENT, DOCUMENT SEEMS TO BE SUSPICIOUS. FURTHER IT WAS OBSERVED BANK HAS GIVEN DEFERRAL GUARANTEE FOR TRANSPORT DOCUMENT WHICH IS STILL UNSETTLED HAS RAISE CASE OF SUSPICION, HENCE STR TO BE RAISED.COGO FREIGHT PRIVATE LIMITED A/C NO , PAN NO. AAGCCP, DOI, STATUS  FREEZED, ADD  STH FLOOR PINNACLE BUSINESS PARK SHANTI,NAGAR MAHAKALI CAVES ROAD,MUMBAI  , MOB NO.  , CONSTITUTION CODE PRIVATE LIMITED COMPANY, AUTHORISED SIGNATORIES AND BENEFICIARY OWNER  MAYUR CHAMARIA, AADHAR REFERENCE NO., RUIYA TYAGI AADHAR REFERENCE NO.AND SHEKHAR PURNENDU AADHAR REFERENCE NO.IN VIEW OF THE RESTRICTION OF SHARING THE AADHAAR NO. AS PER REGULATIONOF AADHAAR SHARING OF INFORMATION, MEDIUM RISK,KYC COMPLIANCE STATUS & LAST KYC UPDATION DATE  YES &OPENED A CURRENT ACCOUNT ONAT RBL BANK LOWER PAREL MUMBAI BRANCH ADD  ONE INDIABULLS CENTRE, TOWER , RD FLOOR,SENAPATI BAPAT MARG, LOWER PAREL W, MUMBAI  . ACCOUNT HOLDER HAS DECLARED PROFILE AS LOGISTICS/ WAREHOUSING HAVING EXPECTED TURNOVER OF RSCRORE TO RSCRORE. SCRUTINY OF THE ACCOUNT STATEMENT FOR THE PERIOD FROMUP TO DATE OF FILING STR OF COGO FREIGHT PRIVATE LIMITED A/C NOREVEALED THAT THE TOTAL CREDITS OF RS . CRORE IN THE ACCOUNT CONSISTS OF RTGS TRANSFER OF RS . CRORE FROM GEE ESS WOOD STOCK HDFC BANK , GAF GLOBAL LOGISTICS PVT LTD IDBI BANK , HERO CYCLES LTD ICICI BANK ,ETC, NEFT TRANSFER OF RS . CRORE FROM MALBROS MARBLES AND GRANITESHDFC BANK , NIMBUS OVERSEAS INDUSIND BANK, BHATIA SHIPPING HDFC BANK,ETC AND OUTWARD CHEQUE OF RS . CRORE FROM K K OVERSEAS ICICI BANK , BABDANA BEARING HDFC BANK , MICRO LABS LTD CANARA BANK , ETC. OF THE TOTAL DEBITS OF RS . CRORE IN THE ACCOUNT CONSISTS OF CMS NEFT OF RS . CRORE TO SILVERFOX GLOBAL LOGISTIC, MSC AGENCY INDIA PVT LTD , OCEAN NETWORK EXPRESS PTE, ETC , CMS RTGS TRANSFER OF RS . CRORE TO CARE HEALTH INSURANCE LIM, DESIGNER WATCHES PVT LTD, MARINETRANS INDIA PVT LTD,ETC AND FOREIGN REMITTANCE OF RS . CRORE TO DYJ LOGISTICS LTD, KINGS AIR LOGISTICS COMPANY, AS LOGISTICS CO LTD,ETC. SCRUTINY OF THE ACCOUNT STATEMENT FOR THE PERIOD FROMUP TOOF COGO FREIGHT PRIVATE LIMITED A/C NOREVEALED THAT THE TOTAL CREDITS OF RS . CRORE, CASH DEPOSIT NIL. OF THE TOTAL DEBITS IS RS . CRORE, CASH WITHDRAWAL NIL. SCRUTINY OF THE ACCOUNT STATEMENT FOR THE PERIOD FROMUP TOOF COGO FREIGHT PRIVATE LIMITED A/C NOREVEALED THAT THE TOTAL CREDITS OF RS . CRORE, CASH DEPOSIT NIL. OF THE TOTAL DEBITS IS RS . CRORE, CASH WITHDRAWAL . LAKH. SCRUTINY OF THE ACCOUNT STATEMENT FOR THE PERIOD FROMUP TOOF COGO FREIGHT PRIVATE LIMITED A/C NOREVEALED THAT THE TOTAL CREDITS OF RS . CRORE, CASH DEPOSIT NIL. OF THE TOTAL DEBITS IS RS . CRORE, CASH WITHDRAWAL . LAKH. BALANCE AS ON THE DATE OF FILING STR IN ACCOUNT OF COGO FREIGHT PRIVATE LIMITED A/C NOIS RS . CREDIT.",{"entities":[(1997, 2022,"FOREIGN ORG"),(2188, 2205,"FOREIGN ORG"),(2207, 2234,"FOREIGN ORG")]}),
            ("IT WAS ALSO NOTED THAT RS CR WAS REMITTED OVERSEAS AS FOREIGN OUTWARD REMITTANCES NONIMPORT TOWARDS FILM PRODUCTION / SHOOTING FAVOURING OCEAN THEAX PTE LTD  SINGAPORE RS CR FROM THE NEWLY OPENED A/C OF MOVIZZY ENTERTAINMENT P LTD AS PER WEB CHECK A WEBSITE WAS NOTED IN THE NAME OF SAID OVERSEAS PARTY HTTPS://WWWOCEANTHEAXPTELTDCOM/HOWEVER THE WEBSITE CARRIED GENERIC INFORMATION ON MOVIE PRODUCTION/ SHOOTING AND NO INFORMATION ABOUT THE FOUNDERS COMPANY ETC COULD BE SEEN HENCE THE GENUINENESS OF THE WEBSITE OR CONTENTS UPDATED IS NOT KNOWN TO THE BANKINTERCONNECTED FUNDS TRANSFER TRANSACTIONS WITHIN THE ACCOUNTS AMOUNTED TO RS CR UTURN TRANSACTIONS WERE ALSO NOTED WHEREIN FUNDS AMOUNTING TO RS CR WAS RECEIVED AND RS CR WAS SENT BACK TO THE SAME GROUP ACCOUNTS MAINTAINED WITH OTHER BANKS WHICH COMPRISE OF ACCOUNTS OF MOVIZZY ENTERTAINMENT PVT LTD CRAZY FILMS PRODUCTION KANCANAM TRADING OPC P LTD AND DIAMOND START GLOBAL P LTD IT APPEARS THAT SUCH TRANSACTIONS MAY HAVE BEEN DONE WITHOUT ANY ECONOMIC RATIONALE FOR LAYERING OF FUNDSMOVIZZY ENTERTAINMENT P LTD ENTERTAINMENT BUSINESS DIAMOND START GLOBAL P LTD IT/ SOFTWARE BUSINESS AND MARINE TIME P LTDLOGISTICS BUSINESS HAVE UPDATED A COMMON MAILING ADDRESS  OFFICE NO BPLOT NOSECTORCBD BELAPUR NAVI MUMBAI DURING A VISIT BY BRANCH OFFICIAL AT THE SAID MAILING ADDRESS IT WAS NOTED THAT IT WAS ASQ FT RENTED PREMISE WITH DIFFERENT SIGN BOARDS OF EACH COMPANY AND OFFICE SET UP COULD BE SEEN WITH TABLES AND FEW CHAIRS SINCE ENTITIES WITH DIFFERENT LINE OF BUSINESS HAVE UPDATED A COMMON MAILING ADDRESS AND NO MAJOR BUSINESS ACTIVITY WAS SEEN IN THE FIELD VISIT DETAILS FURNISHED THE EXISTENCE OF BUSINESS AND GENUINENESS TRANSACTIONS ROUTED THROUGH ACCOUNT APPEARS DOUBTFUL IT APPEARS THAT THE CUSTOMER MAY HAVE DELIBERATELY INDICATED INFLATED TURNOVERS FOR ALL ACCOUNTS POSSIBLY TO AVOID ENQUIRIES ON ROUTING HIGH VALUE AND VOLUME OF TRANSACTIONS THROUGH ACCOUNTSON ENQUIRY IT WAS LEARNT THAT ACCOUNTS RELATED TO THE ABOVE MENTIONED ENTITIES NAMELY LIBERTON LOGISTICS P LTD AND NEXASHIP LOGISTICS PVT LTD WERE OPENED WITH THE BANK BANK HAS ALREADY FILED STR ON THE SAID ACCOUNTS VIDE BATCH IDTOWARDS HIGH VALUE FOREIGN OUTWARD REMITTANCES THROUGH NEWLY OPENED ACCOUNT TOWARDS FREIGHT PAYMENTSA PATTERN OF TRANSACTIONS WERE NOTED IN ACCOUNT OF MOVIZZY ENTERTAINMENT P LTD WHEREIN HIGH VALUE ROUND SUM RTGS/NEFT CREDITS WERE FOLLOWED BY IMMEDIATE FOREIGN OUTWARD REMITTANCES NONIMPORT ALSO HIGH VALUE AND VOLUME DOMESTIC CHURNING OF FUNDS VIDE RTGS/NEFT REMITTANCES WERE NOTED IN THE ACCOUNT OF DIAMOND START GLOBAL P LTDNO OTHER BUSINESS RELATED TRANSACTIONS LIKE PAYMENTS TOWARDS TAXES RENT UTILITY OR MAINTENANCE WERE NOTED THROUGH THE ACCOUNTS DURING REVIEW PERIODBASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS MOVIE SHOOTING EXPENSES WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING / HAWALA SETTLEMENTS ACCOUNT OF MOVIZZY ENTERTAINMENT P LTD IS ACTIVE WITH BALANCE OF RS LAKHS AS ON JULY   QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR  IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE:  IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CRA CURRENT A/C BEARING NO WAS OPENED IN THE NAME OF MOVIZZY ENTERTAINMENT P LTD ON APR AT OUR BRANCH LOCATED IN SHARDA TERRACES PLOT NO SEC   CBD BELAPURNAVI MUMBAI MAHARASHTRA  THE ENTITY WAS FORMED ON FEB AND HAS SUBMITTED COPIES OF PAN AAOCMN BOARD RESOLUTION MOA/ AOA CERT OF INCORP UMHPTC AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO BPLOT NOSECTORCBD BELAPUR NAVI MUMBAI  AJAY BALWANT SINGH AND SANDEEP GAWADE ARE THE DIRECTORS AND UBOS IN THE SAID COMPANY AND HAVE REGISTERED MOBILE NUMBERSANDRESPECTIVELY WITH THE BANK THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO ENTERTAINMENT SERVICES BUSINESS WITH ANNUAL TURNOVER OF RS CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A/CS IN THE NAMES OF DIAMOND START GLOBAL P LTD  MARINE TIME P LTD  CRAZY FILMS PRODUCTION OPC P LTDAND KANCANAM TRADING OPC P LTD  SANDEEP GAWADE AJAY BALWANT SINGH AND SARASWATI YADAV ARE THE COMMON DIRECTORS IN THE SAID ACCOUNTS IN DIFFERENT COMBINATIONS THE ACCOUNTS WERE OPENED DURING THE PERIOD APRIL TILL JUNE AT OUR BELAPUR AND KHARGHAR BRANCHES IN NAVI MUMBAI IT WAS INDICATED THAT THE ENTITIES ARE INTO BUSINESS OF CLEARING / FORWARDING ENTERTAINMENT / IT INDUSTRY WITH ANNUAL TURNOVER IN THE RANGE OF RS CR TO RS CR BESIDES SAVINGS ACCOUNTS WERE OPENED IN THE NAMES OF SANDEEP DEEPAK GAWADEAND AJAY BALWANT SINGHON NOV AND MAR RESPECTIVELY NO TRANSACTIONS WERE NOTED IN THE NEWLY OPENED ACCOUNTS OF MARINE TIME P LTD KANCANAM TRADING OPC P LTD AND CRAZY FILMS PRODUCTION OPC P LTD DURING REVIEW PERIOD EXCEPT INITIAL FUNDING PAYMENT TOWARDS ACCOUNT OPENING WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NONIMPORT WERE DONE THROUGH THE NEWLY OPENED ACCOUNT OF MOVIZZY ENTERTAINMENT P LTD ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH FOREIGN OUTWARD REMITTANCES WERE DONE SUBSEQUENT TO HIGH VALUE ROUND SUM RTGS/NEFT CREDITS AND HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED ACCOUNT TRANSACTIONS SINCE INCEPTION TILL JULY   IT WAS NOTED THAT IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS CR MAINLY FROM NEXASHIP LOGISTICS P LTD RS CR SBI   MOVIZZY ENTERTAINMENT PVT LTD RS CR ICICI/  MAA SHEETLA IMPEX INDIA RS CR SBI AND NAMO SAI INTERNATIONAL RS CR EQUITAS SMALL FINANCE BANK LTD  SUBSEQUENT TO SUCH CREDITS IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING OWN GROUP ACCOUNTS MOVIZZY ENTERTAINMENT PVT LTD RS CR CRAZY FILMS PRODUCTION RS CR ICICI   KANCANAM TRADING OPC P LTD RS CR IDFC AND DIAMOND START GLOBAL P LTD RS CR ICICI/ IDFC ",{"entities":[(138, 156,"FOREIGN ORG")]}),
            ("PART B DETAILS OF INVESTIGATIONSIT WAS OBSERVED THAT THE ACCOUNT HAD FOREIGN INWARD REMITTANCES ININSTANCES BETWEENTO  AFTER WHICH NO INWARD REMITTANCE TRANSACTIONS WERE OBSERVED IN THE ACCOUNT ALSO THE ACCOUNT HAD RECEIVED CREDITS AGGREGATING TO RS  LAKHS DURING THE PERIOD BETWEEN APRTO JUN  WHICH CONTRIBUTES TOPERCENT OF TOTAL CREDITS RECEIVED DURING THE REVIEW PERIOD AFTER WHICH THERE WAS A SUBSTANTIAL DROP IN TRANSACTIONS IN THE ACCOUNT DURING BY BRANCH THE CUSTOMER HAD REVEALED THAT THE UNITED STATES BASED COMPANY  A SQUARE MARKETING LLC FROM WHERE THE SUBJECT ACCOUNT HAD RECEIVED FREQUENT FOREIGN INWARD REMITTANCES IS FAMILY OWNED COMPANY HOWEVER THE CUSTOMER DID NOT SUBMIT THE KYC AND OTHER DOCUMENTS REQUESTED BY BRANCH ON FURTHER DUE DILIGENCE BY BRANCH IT WAS FOUND THAT THE TRACER REPORT FROM TRADE TEAM WAS NEGATIVEIT WAS OBSERVED THAT THE ACCOUNT HAS RECEIVED MULTIPLE REMITTANCE TRANSACTIONS FROM A SINGLE ENTITY FOLLOWED BY IMMEDIATE DEBITS THROUGH ELECTRONIC FUND TRANSFER TO MULTIPLE PARTIES WHICH SEEMS SUSPICIOUS ALSO THE REASON FOR THE SPURT AND THE SUBSEQUENT DROP IN TRANSACTIONS COULD NOT BE ASCERTAINED FURTHER THE FACT THAT THE CUSTOMER IS NOT PROVIDING SUPPORTING DOCUMENTS FOR THE TRANSACTIONS IN THE ACCOUNT ADDS TO THE COMPLEXITIES HENCE RECOMMENDING THE CASE FOR STR FILINGSTR TO BE FILED IN THE LIGHT OF AFOREIGN INWARD REMITTANCES FROM A SINGLE ENTITY FOLLOWED BY IMMEDIATE IMPS DEBITSBSPURT AND SUBSTANTIAL DROP IN TRANSACTIONSPART A GROUND OF SUSPICIONDURING THE REGULAR REVIEW IT WAS OBSERVED THAT THE TRANSACTION PATTERN OF CUSTOMER TRAVELOMATRIX TRIP PLANNERS PVT LTD A/C NOSEEMED SUSPICIOUS WHEREIN MULTIPLE FOREIGN INWARD REMITTANCES FOLLOWED BY IMMEDIATE DEBIT TRANSACTIONS TO MULTIPLE PARTIES WERE OBSERVEDTRAVELOMATRIX TRIP PLANNERS PVT LTD MAINTAINS CORPORATECURRENT ACCOUNT NO  WITH OUR KALKAJI  NEW DELHI BRANCH SINCE  CUSTOMER DETAILS UPDATED IN BANK RECORDS ARE DATE OF INCORPORATION   PAN  AAGCTL PROFILESERVICE PROVIDERTOURS AND TRAVEL AGENCY GROSS TURNOVER OF RSLAKHS REGISTERED ADDRESS  C/O TRAVELOMATRIX TRIP PLANNERS PVT GA KALKAJI NEW DELHI  RISK PROFILE IS LOW MOBILE NO  THE ACCOUNT IS ACTIVE MR ANUJ TANDON AND MR AMARPAL SINGH PAHWA ARE THE AUTHORISED SIGNATORIES IN THE A/C CURRENT BALANCE IN A/C AS ON DATEIS RS /SCRUTINY OF ACCOUNT FROMTO  REVEALS THAT THE ACCOUNT HAD RECEIVED TOTAL CREDITS OF RS  LAKHS OUT OF WHICH RS  LAKHS WERE RECEIVED THROUGH FOREIGN INWARD REMITTANCES FROM A SQUARE MARKETING LLC COUNTRYUNITED STATES PURPOSE  DATA BASE DATA PROCESSING CHARGES WHILE FUNDS AGGREGATING TO RS  LAKHS WERE RECEIVED THROUGH NEFT/RTGS TRANSFER FROM RED ARROWS RSLAKHS A/C NOHELD WITH IDBI BANK AT IFSC  IBKL SHREE SAI OVERSEAS RSLAKHS A/C NOHELD WITH IDBI BANK AT IFSC  IBKLNEFT THE MENTIONED CREDITS WERE FOLLOWED BY DEBITS AGGREGATING TO RS  LAKHS OUT OF WHICH RS  LAKHS WERE TRANSFERRED THROUGH IMPS MAJORLY TO AMIT KANT VERMA RS  LAKHS WHILE FUNDS AGGREGATING TO RS  LAKHS WERE DEBITED THROUGH NEFT/RTGS MAJORLY TO ELITE LANDBASE PRIVATE LIMITED RSLAKHS A/C NOHELD WITH INDUSIND BANK AT IFSC  INDBTHE MOBILE LINKED ACCOUNTS ARE AMR ANUJ TANDON PAN ADTPTN MAINTAINED SAVINGS ACCOUNT NO  SINCEAT DLF GALERIA GURGOAN BRANCH AS PER BANK RECORDS CUSTOMERS MOBILE NUMBER AND ADDRESS IS  F/ RD FLOOR DLF PHASE  OPP PHASECOMMUNITY CENTER GURGAON HARYANA RESPECTIVELY CUSTOMER HAS DECLARED HIS PROFILE AS SERVICE PROVIDER WITH ANNUAL INCOME BETWEEN RSLAKHS CUSTOMER FALLS UNDER LOW RISK CATEGORY THE ACCOUNT IS ACTIVE WITH AN AVAILABLE BALANCE OF RS / AS ON  THE SCRUTINY OF THE ACCOUNT REVEALS THAT THE ACCOUNT RECEIVED TOTAL CREDIT AGGREGATING TO RS  LAKHS OUT OF WHICH RS  LAKHS WERE RECEIVED THROUGH FUND TRANSFER MAJORLY FROM ARTI TANDON RS  LAKHS FROM A/C NO  THE MENTIONED CREDITS WERE FOLLOWED BY DEBITS AGGREGATING TO RS  LAKHS OUT OF WHICH RS  LAKHS WERE TRANSFERRED MAJORLY TO ARTI TANDON RS  LAKHS FROM A/C NO BTHE OTHER MOBILE LINKED ACCOUNTS ARE A/C NOOF ARTI TANDON A/C NOOF PROPMATRIX DEVELOPERS LLP A/C NOOF TECHMATRIX SOFTWARE SOLUTIONS PVT LTD A/C NOOF ALLTIME TECH SUPPORT AND A/C NOOF EQS PLACEMENT SERVICES WHEREIN CREDITS FROM FUND TRANSFER FOLLOWED BY IMMEDIATE DEBITS WERE OBSERVED",{"entities":[(528, 548,"FOREIGN ORG"),(2454, 2474,"FOREIGN ORG")]}),
            ("ININSTANCES CASH WITHDRAWALS TOTALLING TO RS CR WAS STRUCTURED AND SPLIT WITHIN THE A/CS BELOW RS LAKHS POSSIBLY TO AVOID FURNISHING PURPOSE OF CASH WITHDRAWALS THROUGH THE A/CSINTERCONNECTED FUNDS TRANSFER TXNS WITHIN A/CS AMOUNTED TO RS CR ALSO UTURN TXNS WERE NOTED WHEREIN RS CR WAS RECEIVED AND RS CR WAS SENT BACK TO THE SAME GROUP OF A/CS MAINLY COMPRISING OF VANDANA IMPEX SHREE MAHAVIR SALES CORP PLG IMPEX AND ISOE PRINT PACK INDUSTRIES PVT LTD MAINTAINED WITH OTHER BANKS IT COULD BE POSSIBLE THAT SUCH INTERCONNECTED AND UTURN TRANSACTIONS MAY HAVE BEEN DONE WITHOUT ANY ECONOMIC RATIONALE FOR LAYERING OF FUNDS OR POSSIBLY TO INFLATE TURNOVER TO AVAIL CREDIT FACILITIESFUNDS WERE ALSO REMITTED OVERSEAS AS ADV IMPORT REMITTANCES RS CR AND DIRECT BILL RETIREMENTS RS CR MAINLY FAVOURING DING HENG NEW MATERIALS CO LTD  THAILAND RS CR J N EXPORTSHONG KONG RS CR N P INTL S PTE LTD  SINGAPORE RS CR YUNNAN HAOXIN ALUMINIUMCHINA RS CR EASTERN VALLEY CO LTDSOUTH KOREA RS CR AND LOFTEN THAILAND CO LTDTHAILAND RS CR TOWARDS IMPORT OF ALUMINIUM FOILDURING BRANCH OFFICIAL VISIT AT CUSTOMER MAILING ADDRESSSHREE MAHAVIR SALES CORP   MADARSA ROAD KASHMERE G DELHINO BUSINESS ACTIVITY OR STOCK OF GOODS WERE SEEN ON VISITING THE OFFICE ADDRESS OF SHREE MAHAVIR SALES AT  NDM  NETAJI SUBHASH PLACE PITAMPURA DELHI SIGN BOARD IN THE NAME OF VANDANA TRADING CO OFFICE SET UP AND FEW SAMPLE STOCK OF GOODS WERE SEEN IT WAS LEARNT THAT THE STOCK OF GOODS IMPORTED WERE KEPT AT ALIPUR DELHISHREE MAHAVIR SALES CORP VANDANA IMPEX AND BALAJI EXIM ARE YET TO SUBMIT BILLS OF ENTRY PERTAINING TOCASES OF ADVANCE IMPORT REMITTANCES ANDCASES OF DIRECT BILL RETIREMENTS THEY COMPRISE OFCASES TOTALLING TO USD  PENDING FOR A PERIOD LESS THANMONTHSCASES TOTALLING TO USD  PENDING FOR A PERIOD ABOVEMONTHS TO LESS THANMONTHS ANDCASES TOTALLING TO USD  PENDING FOR A PERIOD ABOVEMONTHS TILLYEARAS PER INFORMATION/ NEWS ARTICLES AVAILABLE ON PUBLIC DOMAIN LINK  HTTPS://ECONOMICTIMESINDIATIMESCOM/NEWS/ECONOMY/FOREIGNTRADE/INDIAINITIATESANTIDUMPINGDUTYPROBEAGAINSTALUMINIUMFOILIMPORTSFROMCHINAOTHERS/ARTICLESHOW/CMSANTIDUMPING DUTY PROBE IS INITIATED IN INDIA AGAINST CERTAIN KIND OF ALUMINIUM FOIL IMPORTED FROM CHINA INDONESIA MALAYSIA AND THAILAND VARIOUS REPUTED COMPANIES HAVE FILED APPLICATION CLAIMING INJURY RESULTING FROM THE ALLEGED DUMPING IN THE FORM OF INCREASED VOLUME OF DUMPED IMPORTS IN RELATION TO PRODUCTION AND CONSUMPTION PRICE UNDER CUTTING AND PRICE SUPPRESSION EFFECTING DOMESTIC INDUSTRYBASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE SINCE NO MAJOR BUSINESS ACTIVITY OR STOCK OF GOODS IN LINE WITH THE VALUE AND VOLUME OF TRANSACTIONS ROUTED THROUGH A/CS WERE SEEN AT THE MAILING ADDRESS AND OFFICE ADDRESS OF SHREE MAHAVIR SALES CORP IT COULD BE POSSIBLE THAT TRANSACTIONS ROUTED THROUGH THE A/C MAY HAVE NO ECONOMIC RATIONALE AND MAY HAVE BEEN ROUTED FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TBML A/CS OF SHREE MAHAVIR SALES CORPIS ACTIVE WITH SANCTIONED LIMIT OF RS CR UNUTILISED BALANCERS CR AND A/C NO IS ACTIVE WITH BALANCE OF RS/ AS ON JULY   QUANTUM OF TRANSACTIONS FOR CURRENT FY  IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE:  IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR  IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CRGOPAL CHAND TAPARIA AS A PROPRIETOR MAINTAINS CURRENT A/CS BEARING NOSANDIN THE NAME OF A SOLE PROPRIETORSHIP CONCERN SHREE MAHAVIR SALES CORPORATION SINCEANDRESPECTIVELY AT OUR BRANCH LOCATED IN C/ ASHOK VIHARNEW DELHI  THE ENTITY WAS FORMED ONAND HAS SUBMITTED COPIES OF PAN AAEPTE IEC CERTAND GST REG CERT AAEPTEZP AS KYC DOCS AT THE TIME OF A/C OPENING CUSTOMERS REG ADDRESS WITH BANK IS  MADARSA ROAD KASHMERE G DELHI  VARDHAN TAPARIA AND MANISH TAPARIA ARE THE MANDATE HOLDERS FOR A/C OPERATION IN THE SAID A/C CUSTOMER HAS REGISTERED LANDLINE NOAND MOBILE NUMBERWITH THE BANK THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF A/C OPENING THAT THE ENTITY IS INTO BUSINESS OF ALUMINIUM AND ALUMINIUM PRODUCTS WITH ANNUAL TURNOVER OF RS CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED A/C OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A/CS IN THE NAMES OF SOLE PROPRIETORSHIP CONCERNS VANDANA TRADING CO A/C NO  NATURE OF BUSINESS  ALUMINIUM AND ALUMINIUM PRODUCTS ANNUAL TURNOVERRS CR VANDANA IMPEX A/C NO NATURE OF BUSINESS  ALUMINIUM AND ALUMINIUM PRODUCTS ANNUAL TURNOVERRS CR AND BALAJI EXIM A/C NO NATURE OF BUSINESSPACKAGING ANNUAL TURNOVERRS CR BESIDES CREDIT CARD FACILITY WAS AVAILED BY MANISH TAPARIA /  GOPAL CHAND TAPARIA VARDHAN TAPARIA MANISH TAPARIA AND VANDANA TAPARIA ARE THE COMMON PROPRIETORS/ MANDATE HOLDERS FOR A/C OPERATIONS IN THE MENTIONED A/CS THE MENTIONED A/CS WERE OPENED DURING THE PERIOD JANUARY TILL OCTOBER AT THE ABOVE MENTIONED BRANCH SHREE MAHAVIR SALES CORP HAS AVAILED CREDIT FACILITY OF RS CR CURRENT UNUTILISED AMOUNT IS RS CR AND VANDANA TRADING CO HAS AVAILED CREDIT FACILITY OF RS CR CURRENT UNUTILISED AMOUNT IS RS CR WHILE REVIEWING REPORT ON ADV IMPORT REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE ADVANCE IMPORT PAYMENTS WERE DONE THROUGH THE A/C OF SHREE MAHAVIR SALES CORP ON VERIFYING THE A/C ACTIVITY IT WAS NOTICED THAT SUCH ADV IMPORT REMITTANCES WERE DONE SUBSEQUENT TO HIGH VALUE ROUND SUM RTGS/NEFT / INTERNAL FUNDS TRANSFER CREDITS AND HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLL GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED A/C TRANSACTIONS SINCE APRIL TILL JULY   IT WAS NOTED THAT IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS CR MAINLY FROM ISOE PRINT PACK INDS P LTD RS CR ICICI   S S IMPEX RS CR THE NAVNIRMAN COOP BANK LTD/  VANDANA IMPEX RS CR SBI AND SHREE MAHAVIR SALES CORP RS CR HDFC  / ICICI/SBI  INTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS CR MAINLY FROM SHREE BABOSA PAPER P LTD RS CR A/C NO  VINOD TRADING CO RS CR A/C NO DOLPHIN INTL RS CR A/C NOAND KOPAL PAPERS P LTD RS CR A/C NO  TRANSACTIONS IN THE INTERNAL FUNDS TRANSFER A/CS WOULD BE REVIEWED SEPARATELY AND IN CASE SUSPICIOUS STR WOULD BE FILED LATER CHEQUE DEPOSITS AMOUNTED TO RS CRSUBSEQUENT TO SUCH CREDITS IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING VANDANA IMPEX RS CR HDFC/ SBI   SHREE MAHAVIR SALES CORP RS CR BILT GRAPHIC PAPER PRODUCTS LTD RS CR ICICI   PLG IMPEX RS CR HDFC/AND ISOE PRINT PACK INDUSTRIES PVT LTD RS CR ICICI  ",{"entities":[(845, 857,"FOREIGN ORG"),(873, 892,"FOREIGN ORG"),(944, 965,"FOREIGN ORG"),(987, 1014,"FOREIGN ORG")]}),
            ("ASG LEATHER PRIVATE LIMITED PRIVATE LTD COMPANY HAVING ADDRESS: PLOT A AND B BASANTI HIGHWAY KOLKATA LEATHER COMPLEX SOUTHPARGANAS WEST BENGAL INDIA  CONTACT NUMBER: + AND EMAIL ID: ALOKE@ASGLEATHERCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD   SIR RN MUKHERJEE ROAD KOLKATA WEST BENGAL  SINCE  DATE OF INCORPORATION IS  PAN NUMBER OF THE ACCOUNT IS AAECAL IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS  TRANSACTION LINKED ACCOUNTS IS  AUTHORISED SIGNATORIES AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT ARE ALOKE KUMAR SENGUPTA WITH PAN NO ALSPSK DEBIKA SAHA WITH PAN NO BREPSA INDRANATH SENGUPTA WITH PAN NO CLWPSP MITALI SENGUPTA WITH PAN NO ALJPSM THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON  KYC DOCUMENTS OBTAINED WERE AADHAR PAN CIN IEC BOARD RESOLUTION MOA ETC KYC UPDATION IS COMPLIED CUSTOMER IS A KOLKATA BASED COMPANY FORMERLY KNOWN AS ASG ASSOCIATES STARTED ITS OPERATIONS WAY BACK IN  MANUFACTURER OF LEATHER LEATHER PRODUCTS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO CR ALERT WAS TRIGGERED FOR HIGH VALUE FOREX TURNOVER IN ACCOUNTS QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROMTILLSHOWS TOTAL CREDIT RS  LAKHS OF WHICH RSBY CASH REMITTANCE RS  LAKHS MAJORLY FROM /HTF HAMBURG LEATHER GMBHHAMLED  HAMBURGER LEDERWAREN /HAMLED  HAMBURGER LEDERWAREN  AND OTHERS RS  LAKHS  TOTAL DEBITS DURING THE SAME PERIOD WAS RS  LAKHS OF WHICH RSBY CASH AND OTHERS RS  LAKHSFURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY  FROMTO: TOTAL CREDIT SIDE RS  LAKHS OF WHICH RSBY CASH REMITTANCE RS  LAKHS MAJORLY FROM HTF HAMBURG LEATHER GMBH  AND OTHERS RS  LAKHS TOTAL DEBIT SIDE RS  LAKHS OF WHICH RSBY CASH AND OTHERS RS  LAKHS TRANSACTION PATTERN SHOWS ACCOUNT WAS MAINLY GETTING CREDITED THROUGH TRANSFER REMITTANCE AND DEBITED THROUGH TRANSFER ONTHE ACCOUNT GOT CREDITED WITH RSCRORES APPROX THROUGH REMITTANCE FROM HTF HAMBURG LEATHER GMBH IN EUR FROM GERMANY FOR WHICH PURPOSE WAS MENTIONED AS ADV RECPT AGAINST EXPORTOTHER THAN NEPAL AND BHUTAN AND ONTHE ACCOUNT GOT DEBITED WITH RSCRORES THROUGH TRANSFER TO ASG LEATHER PRIVATE LIMITED FUNDS GOT DEBITED THROUGH CLEARING TO HB IMPEX/SBI UPS EXPRESS PRIVATE LTDC/CIT ETC SIMILARLY ONTHE ACCOUNT GOT CREDITED WITH RS CRORES APPROX THROUGH REMITTANCE FROM HTF HAMBURG LEATHER GMBH IN EUR FROM GERMANY FOR WHICH PURPOSE WAS MENTIONED AS ADV RECPT AGAINST EXPORTOTHER THAN NEPAL AND BHUTAN AND ONANDTHE ACCOUNT GOT DEBITED WITH RS CRORES THROUGH TRANSFER TO ASG LEATHER PRIVATE LIMITED LARGE VALUE CREDITS THROUGH REMITTANCE WERE OBSERVED TRANSACTION PATTERN SEEMS TO BE UNUSUAL AND NOT MATCHING WITH THE PROFILE THERE SEEMS TO BE NO ECONOMIC RATIONALE FOR SUCH TRANSACTIONS IN VIEW OF HIGH VALUE FOREX TURNOVER IN ACCOUNTS STR IS BEING REPORTED",{"entities":[(1365, 1432,"FOREIGN ORG"),(2046, 2070,"FOREIGN ORG"),(2437,2461,"FOREIGN ORG")]}),
            ("DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS OF VKR TRADERS, IT WAS OBSERVED THAT IT WAS A RESIDENTIAL SETUP IN 1300 SQ FT AREA WITH A TEMPORARY SIGN BOARD AND NO STOCK OF GOODS OR BUSINESS SETUP WAS SIGHTED STOCK OF TILES WERE SEEN AT THE REGISTERED ADDRESS WHICH APPEARS TO BE PRODUCT UNRELATED TO INDICATED BUSINESS ON ENQUIRY BY BRANCH OFFICIAL, CUSTOMER INDICATED THAT HE IS NOT MAINTAINING STOCK OF GOODS SINCE GOODS WERE DIRECTLY SENT BY DOMESTIC VENDORS TO THE AIRPORT FOR ONWARD EXPORT TO RESPECTIVE OVERSEAS CLIENTS  CUSTOMER ALSO INDICATED THAT ROUND SUM DOMESTIC PAYMENTS WERE TOWARDS HIS REGULAR SUPPLIERS BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE THE GENUINENESS OF BUSINESS SET-UP IS NOT KNOWN SINCE DESPITE OF HIGH VALUE TRANSACTIONS, NO PROPER BUSINESS ACTIVITY IN LINE WITH TRANSACTIONS ROUTED COULD BE SIGHTED IT COULD BE POSSIBLE THAT SHELL ENTITY MAY HAVE BEEN FORMED IN THE NAME OF VKR TRADERS AND ACCOUNT MAY HAVE BEEN OPENED TO ROUTE TRANSACTIONS WITHOUT GENUINE COMMERCIAL BUSINESS FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS 550 CR IN CASH RS 010 CR AND CREDIT: RS 551 CR IN CASH RS 018 CR AND THE TRANSACTIONS TURNOVER FOR PREVIOUS FINANCIAL YEARS ARE 2020-21: DEBIT: RS 1056 CR IN CASH RS 023 CR AND CREDIT: RS 1056 CR IN CASH RS 074 CR, 2019-20: DEBIT: RS 192 CR IN CASH RS 013 CR AND CREDIT: RS 192 CR IN CASH RS 048 CR THE ACCOUNT NO 4913193163 OF VKR TRADERS IS CURRENTLY ACTIVE WITH A BALANCE OF RS29,606/- AS ON 23072021R HARI BABU AS A PROPRIETOR MAINTAINS A CURRENT ACCOUNT A/C NO4913193163 IN THE NAME OF A SOLE PROP CONCERN VKR TRADERS SINCE 04062019 AT OUR BRANCH LOCATED AT UPPER GROUND FLOOR, KRISHNA TULASI PARADISE, 392A, TILAK ROAD, TIRUPATHI  517501 ANDHRA PRADESH THE ENTITY WAS ESTABLISHED ON 09072008 AND HAS SUBMITTED COPIES OF PAN AARPH5424Q, GST REG CERT 37AARPH5424Q1ZZ AND CONTACT POINT VERIFICATION REPORT AS KYC DOCS CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS  FLAT NO 310, GCR CROSS WINDS APARTMENT, LINGESWARA NAGAR, TIRUPATI  517501 ANDHRA PRADESH AND HAS REGISTERED MOBILE NUMBER 9700726406THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND THE ACCOUNT IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF FOOD MATERIALS WITH ANNUAL TURNOVER OF RS900 CR NOTICING HIGH VALUE EXPORT ADVANCE REMITTANCE CREDITS AND SUBSEQUENT ROUND SUM RTGS/NEFT/IMPS PAYMENTS THROUGH THE ACCOUNT 4913193163 OF VKR TRADERS, AN ALERT WAS RAISED ON BRANCH TO CARRY OUT ENHANCED DUE DILIGENCE AND THIS REPORT IS FILED ON THE ABOVE MENTIONED ACCOUNT ON THE FOLLOWING GROUNDS OF SUSPICION: ON REVIEWING ACCOUNT TRANSACTIONS SINCE 01042020 TILL 21062021, IT WAS NOTED THAT EXPORT ADVANCE REMITTANCE RECEIVED AMOUNTED TO RS1091 CR MAINLY FROM R H S FOODSTUFF TRADING LLC- UAE RS472 CR, ALANA FOOD STUFF TRADING LLC- USA RS153 CR AND DRIP CAPITAL, INC- USA RS151 CR TOWARDS EXPORT OF VEGETABLES/ FISH AND PRAWN AS PER WEB CHECK R H S FOODSTUFF TRADING LLC AND ALANA FOOD STUFF TRADING LLC ARE INTO IMPORT/EXPORT OF FOOD STUFF / FRUITS/VEGETABLE AND DRIP CAPITAL IS A DIGITAL TRADE FINANCE COMPANY BASED IN USA VKR TRADERS IS YET TO SUBMIT EXPORT DOCUMENTS PERTAINING TO 57 CASES OF EXPORT ADVANCE REMITTANCES TOTALING TO USD 680,077 PENDING FOR A PERIOD LESS THAN A YEAR RTGS/NEFT/IMPS CREDITS AMOUNTED TO RS 413 CR MAINLY FROM NAZEER SEA FOODS RS122 CR, KARUR VYSYA BANK - 1431135000011615, CBEC ICE GATE RS 068 CR , SBI - 33325824153 AND KMR EXPORTS RS 051 CR , IDBI - IDB919912143344 CASH DEPOSITS AMOUNTED TO RS092 CR SUBSEQUENT TO SUCH CREDIT, RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS1537 CR MAINLY FAVORING  RVV INTERNATIONAL P LTD RS 265 CR , ICICI BANK - 111505500798, VBY SEA FOOD RS 149 CR, AXIS BANK - 917020051747645 AND M KANTHA RAO RS 092 CR , ICICI BANK - 130901503947 CASH WITHDRAWALS THROUGH ATMS AND ACROSS BRANCH COUNTERS LOCATED IN TIRUPATI AMOUNTED TO RS033 CR NO BUSINESS EXPENSE TRANSACTIONS LIKE PAYMENTS TOWARDS TAXES, RENT ETC, WERE NOTED THROUGH THE ACCOUNT DURING REVIEW PERIOD WHICH IS UNUSUAL WHILE RUNNING A COMMERCIAL BUSINESS",{"entities":[(2895, 2923,"FOREIGN ORG"),(3068, 3095,"FOREIGN ORG"),(3175, 3204,"FOREIGN ORG")]}),
            ("ON 28-09-2020, RS 3522 LAKHS WERE CREDITED THROUGH RTGS FROM MONZADV PRIVATE LIMITED (ACCOUNT NO: 50200036482270, IFSC CODE-HDFC0000240, BANK: HDFC BANK), CASH DEPOSIT FROM KARTARPUR BRANCH AND ON 28-09-2020, RS 3534 LAKHS WERE DEBITED THROUGH REMITTANCES TO EXOCLICK S L IN USD TO SPAIN FOR ADVERTISING, TRADE FAIR SERVICE SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT THERE WERE NO PAYMENTS SEEN TO ANY AURVEDIC PRODUCT COMPANIES FOR PROCUREMENT OF MATERIAL AND NO ROUTINE NATURE BUSINESS EXPENSES SUCH AS UTILITY BILL PAYMENTS , SALARIES , RENT TAXES ETC SEEN IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2020-21 WAS RS 3,86391 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF SUDDEN INCREASE IN VALUE AND VOLUME OF TRANSACTIONS IN THE ACCOUNT AND HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDMONZADV PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS: E-330 RANJIT AVENUE, AMRITSAR, PUNJAB, INDIA, 143001, CONTACT NUMBER: +918559009158 AND EMAIL ID: PAVNEET@ADMONZCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 060805500350 WITH ICICI BANK LTD, SCO 112, DISTT SHOPPING COMPLEX, RANJIT AVENUE, AMRITSAR, PUNJAB, 143001, SINCE 09072019 DATE OF INCORPORATION IS 23112017 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES LOAN AND FIXED DEPOSITS PAN NUMBER OF THE ACCOUNT IS AALCM4586D PAN LINKED ACCOUNTS ARE 060813004478, 060813004489, 060813005134, 060813005140, 060813005141, 060813005142, FLAAMT00038600616 TRANSACTION LINKED ACCOUNTS ARE 060813005140 AND 060813005141 BENEFICIAL OWNERS/ AUTHORIZED SIGNATORIES/ DIRECTORS OF THE REPORTED ACCOUNT ARE GURINDER MOHAN SINGH (PAN: APMPS9360J) AND PAVNEET SINGH GULATI (PAN: ALQPG0965M) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- AALCM4586D, COI: U74990PB2017PTC047211, MOA AOACUSTOMER PROFILE AND DUE DILIGENCE: CUSTOMER IS INTO TRADING AND ONLINE MARKETING OF AYURVEDA PRODUCTS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO LESS THAN RS 5 CRORES ALERT WAS TRIGGERED FOR SUDDEN INCREASE IN VALUE AND VOLUME OF TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 30062021 WAS RS1158 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 30062021 SHOWS TOTAL CREDIT RS 1,23123 LAKHS OF WHICH CASH WAS RS 4209 LAKHS MAJORLY IN THANE KOPRI, MUZZAFARNAGAR, LUCKNOW - TELIBAGH, MUMBAI TILAK NAGAR BRANCH, NEFT RS 88842 LAKHS MAJORLY FROM BUSY BEES LOGISTICS PRIVATE LIMITED, MONZADV PRIVATE LIMITED, ECOM EXPRESS, RTGS RS 29480 LAKHS MAJORLY FROM MONZADV PRIVATE LIMITED TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,20985 LAKHS OF WHICH RTGS RS 50475 LAKHS MAJORLY TO MONZADV PRIVATE LIMITED, REMITTANCE RS 51556 LAKHS MAJORLY TO EXOCLICK S L IN USD TO SPAIN FOR ADVERTISING, TRADE FAIR SERVICE, TRAFFIC STARS LIMITED IN USD TO CYPRUS FOR ADVERTISING FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 2 YEARS WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 3,86391 LAKHS OF WHICH CASH WAS RS 29914 LAKHS MAJORLY IN RANCHI - KANTA TOLI, VIJAYNAGAR(BANGALORE), LUCKNOW - TELIBAGH BRANCH, NEFT RS 3,01792 LAKHS MAJORLY FROM ECOM EXPRESS, BUSY BEES LOGISTICS PRIVATE, MONZADV PRIVATE LIMITED, RTGS RS 40250 LAKHS MAJORLY FROM MONZADV PRIVATE LIMITED, INTERNET FUND TRANSFERS RS 2592 LAKHS FROM ANAND SINGH, NAYYER IQBAL, FIXED DEPOSIT RS 3000 LAKHS TOTAL DEBIT SIDE RS 3,84827 LAKHS OF WHICH CASH WAS RS 3750 LAKHS BY AMRITSAR, PUNJAB BRANCH, TRANSFERS RS 3170 LAKHS MAJORLY TO KUNDAN JEWELLERS, SMARTBIZ TECHNOLOGIES PRIVATE LIMITED, MONZADV PRIVATE LIMITED, RTGS RS 2,03025 LAKHS MAJORLY TO MONZADV PRIVATE LIMITED, REMITTANCE RS 1,60325 LAKHS MAJORLY TO EXOCLICK S L IN USD TO SPAIN FOR ADVERTISING, TRADE FAIR SERVICE, TRAFFIC STARS LIMITED IN USD TO CYPRUS FOR ADVERTISING, INTERNET FUND TRANSFER RS 12023 LAKHS TO PAVNEET SINGH GULATI (039901559183) FY 2019-20 FROM 09072019 TO 31032020: TOTAL CREDIT SIDE RS 3,23318 LAKHS OF WHICH CASH WAS RS 1527 LAKHS TOTAL DEBIT SIDE RS 3,21532 LAKHS OF WHICH CASH WAS RS 720 LAKHS AS PER THE TRANSACTION PATTERN IN THE ACCOUNT, THERE ARE FREQUENT FUNDS CREDITED THROUGH CASH DEPOSITS FROM DIFFERENT LOCATIONS AND FROM ENTITIES WHICH ARE LATER DEPLOYED TOWARDS THE ENTITIES, REMITTANCES SEEMS UNUSUAL FOR EXAMPLE, ON 31-08-2020, RS 31 LAKHS WERE CREDITED THROUGH RTGS FROM MONZADV PRIVATE LIMITED (ACCOUNT NO: 50200036482270, IFSC CODE-HDFC0000240, BANK: HDFC BANK) AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH REMITTANCES TO EXOCLICK S L IN USD TO SPAIN FOR ADVERTISING, TRADE FAIR SERVICE",{"entities":[(259, 271,"FOREIGN ORG"),(2805, 2817,"FOREIGN ORG"),(3749, 3761,"FOREIGN ORG"),(4565,4577,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN INDICATES- LARGE VALUE OF FUNDS WERE CREDITED THROUGH REMITTANCES AND THOSE FUNDS WERE DEPLOYED TOWARDS ENTITIES FOR EXAMPLE, ON 29-06-2021, RS 221CRORES CREDITED THROUGH INWARD REMITTANCES FROM DUTEL INTERNATIONAL FZE, SM OFFICE B 405 H SM OFFICE PO BOX 46023, AJMAN, UNITED ARAB EMIRATES IN USD CURRENCY WHERE THE REASON/PURPOSE MENTIONED WAS GOODS BOUGHT/ AWB NO 17663154490 FINAL PAYMENT (EXPORT OF MOBILES) AND ON 29-06-2021, RS 22CRORES DEBITED THROUGH RTGS TO FUTURE MOBILE LLP (AC NO- 37265890010, IFSC- SBIN0004115, BANK: SBI) SIMILARLY, ON 21-05-2021, RS 145CRORES CREDITED THROUGH INWARD REMITTANCES FROM DUTEL INTERNATIONAL FZE, SM OFFICE B 405 H SM OFFICE PO BOX 46023, AJMAN, UNITED ARAB EMIRATES IN USD CURRENCY WHERE THE REASON/PURPOSE MENTIONED WAS GOODS BOUGHT/ AWB NO 17628092890 FINAL PAYMENT (EXPORT OF MOBILES) AND ON 21-05-2021, RS 14CRORES DEBITED THROUGH RTGS TO FUTURE MOBILE LLP (AC NO- 37265890010, IFSC- SBIN0004115, BANK: SBI) SIMILAR TRANSACTION SEEN IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2020-21 WAS RS 12,84282 LAKHS, WHICH SEEMS TO BE VERY HIGH SUCH HIGH TURNOVER IS NOT SUPPORTED BY THE BUSINESS SETUP IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT WITH HIGH TURNOVER WITHOUT PROPER BUSINESS SETUP AND NEGATIVE EDD FINDINGS WHEREIN CUSTOMER WAS UNCOMFORTABLE TO GIVE THE INFORMATION REGARDING TRANSACTION PATTERN IN THE ACCOUNT , STR IS BEING FILEDKRISHNA EXPORTS, A PARTNERSHIP FIRM, HAVING ADDRESS: FLAT NO-102 AISHA MANZIL, PLOT NO: 335/337/A 1ST FLOOR, ANDHERI, MUMBAI, MAHARASHTRA, INDIA, 400058, CONTACT NUMBER: +918657462481 AND EMAIL ID: KRISHNAEXPORTAC@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 001105030724 WITH ICICI BANK LTD, SAGAR AVENUE, GROUND FLOOR, OPP SHOPPERS STOP, SV ROAD, ANDHERI W, MUMBAI, MAHARASHTRA, 400058, SINCE 20082019 DATE OF FORMATION IS 02012016 PAN OF THE ACCOUNT IS AAQFK1444G GSTIN AS PER BANK RECORDS IS 27AAQFK1444G1Z8 TRANSACTION LINKED ACCOUNTS ARE 001101618209, 347405000656 AUTHORIZED SIGNATORIES/ PARTNERS OF THE REPORTED ACCOUNT ARE SAGAR RATANDHARA (PAN- ASRPR2360K) AND NEHA AGGARWAL (PAN: AIOPM5007B) BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS NEHA AGGARWAL (PAN: AIOPM5007B) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN - AAQFK1444G, PARTNERSHIP DEED AND PAN AADHAAR OF PARTNERS KYC UPDATION IS NOT DUE AS PER EDD, THEY WERE IN THE BUSINESS TO EXPORT MOBILE PHONE TO UAE THE SAME ARE PROCURED FROM THE LOCAL MARKETS THE CUSTOMER WAS UNCOMFORTABLE TO GIVE THE INFORMATION AS PER VISIT FINDINGS, NO PROPER BOARD WAS AVAILABLE AND IT WAS SEEN ON A4 SIZE PAPER NO PROPER BUSINESS SETUP WAS VISIBLE DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 25 CRORES TO RS 100 CRORES ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 17072021 WAS RS1621 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 17072021 SHOWS TOTAL CREDIT RS 4,42366 LAKHS OF WHICH TRANSFERS RS 644 LAKHS MAJORLY FROM OM PRAKASH UPADHYAYA, RTGS RS 25000 LAKHS MAJORLY FROM S M EDIBLES PVT LTD, REMITTANCE RS 4,16706 LAKHS MAJORLY FROM ACTION LOGISTIC FZE, DUTEL INTERNATIONAL FZE TOTAL DEBITS DURING THE SAME PERIOD WAS RS 4,42544 LAKHS OF WHICH CASH WAS RS 350 LAKHS BY ANDHERI BRANCH, TRANSFERS RS 060 LAKHS MAJORLY TO OM PRAKASH UPADHYAYA, EMPLOYEES PROVIDENT FUND ORGANISATION, CLEARING RS 25089 LAKHS MAJORLY TO PALLATS AND CRATES MOVERSAND PRIVATE LIMITED, FEDERATION OF INDIAN EXP, RTGS RS 4,15208 LAKHS MAJORLY TO FUTURE MOBILES LLP, INTERNET FUND TRANSFER RS 1498 LAKHS- NEFT TO REEYA STORE AND OTHERS RS 207 LAKHS- DTAXFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 12,84282 LAKHS OF WHICH NEFT RS 2381 LAKHS MAJORLY FROM IMPREST FUTURE MOBILE LLP, RTGS RS 2950 LAKHS MAJORLY FROM KRISHNA EXPORTS, FUTURE MOBILE LLP, REMITTANCE RS 12,78925 LAKHS MAJORLY FROM DUTEL INTERNATIONAL FZE, ACTON LOGISTIC FZE TOTAL DEBIT SIDE RS 12,83507 LAKHS OF WHICH CASH WAS RS 200 LAKHS BY BRANCH, CLEARING RS 420 LAKHS MAJORLY TO ARIF DAWOOD HALAI, RAZAK DAWOOD HALAI, RAFIQUE DAWOOD HALAI, RTGS RS 12,13232 LAKHS MAJORLY TO FUTURE MOBILE LLP, INTERNET FUND TRANSFER RS 51150 LAKHS TO IDEAL FREIGHT FORWARDERS PRIVATE LIMITED (347405000656), CARD TRANSACTIONS RS 62400 AND OTHERS RS 17724 LAKHS- DTAX, GSTFY 2019-20 FROM 20082019 TO 31032020: TOTAL CREDIT SIDE RS 14,83378 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 14,80218 LAKHS TOTAL DEBIT SIDE RS 14,82353 LAKHS OF WHICH CASH WAS RS 150 LAKHS, REMITTANCE RS 1767 LAKHS",{"entities":[(215, 238,"FOREIGN ORG"),(636, 659,"FOREIGN ORG"),(3293, 3311,"FOREIGN ORG"),(3313, 3335,"FOREIGN ORG"),(4154, 4177,"FOREIGN ORG"),(4179, 4196,"FOREIGN ORG")]}),
            ("FY 2019-20 FROM 01042019 TO 31032020: TOTAL CREDIT SIDE RS 11,92903 LAKHS OF WHICH CASH WAS RS 6650 LAKHS, REMITTANCE RS 10,34635 LAKHS TOTAL DEBIT SIDE RS 11,90788 LAKHS OF WHICH CASH WAS RS 8326 LAKHS, REMITTANCE RS 1940 LAKHSFY 2018-19 FROM 01042018 TO 31032019: TOTAL CREDIT SIDE RS 8,70847 LAKHS OF WHICH CASH WAS RS 1800 LAKHS, REMITTANCE RS 6,82254 LAKHS TOTAL DEBIT SIDE RS 8,70959 LAKHS OF WHICH CASH WAS RS 5490 LAKHS, REMITTANCE RS 4497 LAKHSFY 2017-18 FROM 26122017 TO 31032018: TOTAL CREDIT SIDE RS 80425 LAKHS OF WHICH CASH WAS RS 305 LAKHS, REMITTANCE RS 36910 LAKHS TOTAL DEBIT SIDE RS 80025 LAKHS OF WHICH CASH WAS RS 475 LAKHS, REMITTANCE RS 021 LAKHSTRANSACTION PATTERN INDICATES- LARGE VALUE OF FUNDS WERE CREDITED THROUGH DIFFERENT ENTITIES AND THOSE FUNDS WERE DEPLOYED AGAIN TOWARDS ENTITIES FOR EXAMPLE, ON 15-06-2021, RS 25 LAKHS WERE CREDITED THROUGH TRANSFER FROM POTUS KIRANA PVT LTD (739305000001) AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH RTGS TO DESHBANDHU IMPEX PVT LTD (ACCOUNT NO: 50200025132894, IFSC CODE: HDFC0009141, BANK: HDFC BANK) ON 30-06-2021, RS 3032 LAKHS WERE CREDITED THROUGH REMITTANCES FROM AL YASRA FOOD CO WLL IN USD FROM KUWAIT FOR EXPORT OF RICE AND PULSES AND ON 01-07-2021, RS 2612 LAKHS WERE DEBITED THROUGH RTGS TO SRI RAJALAKSHMI AGRO FOODS (ACCOUNT NO: 1412917023, IFSC CODE: KKBK0008306, BANK: KOTAK MAHINDRA BANK), M P TRADERS (ACCOUNT NO: 6784509771, IFSC CODE: IDIB000R028, BANK: INDIAN BANK), GOURAV JINDAL (ACCOUNT NO: 50100117084842, IFSC CODE: HDFC0000525, BANK: HDFC BANK), NEW ERA CONTRACTS (ACCOUNT NO: 3951002100004711, IFSC CODE: PUNB0413700, BANK: PUNJAB NATIONAL BANK) SIMILAR TRANSACTION SEEN IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2020-21 WAS RS 12,26110 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT AND HIGH TURNOVER IN THE ACCOUNT WHICH FAR EXCEEDS THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER, STR IS BEING FILEDCROWN GLOBAL TRADING PRIVATE LIMITED, A PRIVATE LIMITED COMPANY, HAVING ADDRESS: D 1/5 FLOOR D RAJOURI, GARDEN, NEAR BINDAR PARK, NEW DELHI, INDIA, 110027, CONTACT NUMBER: +919810625655 AND EMAIL ID: NIHARMITTAL@SHRIVISHNUCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 135305001234 WITH ICICI BANK LTD, 1ST FLOOR, UNIT NO 106,107,108,109, PLOT NO C9, PEARL BEST HEIGHT II, NETAJI SUBHASH PLACE, DELHI, 110034, SINCE 26122017 DATE OF INCORPORATION IS 18022015 PAN NUMBER OF THE ACCOUNT IS AAFCC9630E TRANSACTION LINKED ACCOUNTS ARE 739305000001 ,031651000008 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE PAVANI MITTAL (DIRECTOR, PAN- BZJPM8829G) AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS NEHA MITTAL (DIRECTOR, PAN: APZPM9509C) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN: AAFCC9630E, COI: U51909DL2015PTC276866, MOA-AOA AND PAN AADHAAR OF DIRECTORS KYC UPDATION IS DUE ENTITY IS INTO TRADING OF PULSES AND CEREALS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 5 CRORES TO RS 25 CRORES ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 22072021 WAS RS175 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 22072021 SHOWS TOTAL CREDIT RS 1,99467 LAKHS OF WHICH CASH WAS RS 4800 LAKHS MAJORLY IN DELHI - NETAJI SUBHASH PLACE, HANSI BRANCH, TRANSFERS RS 2500 LAKHS MAJORLY FROM POTUS KIRANA PRIVATE LIMITED, CLEARING RS 298 LAKHS MAJORLY FROM CROWN GLOBAL TRADING PRIVATE LIMITED, CMS TRANSFERS, NEFT RS 3304 LAKHS MAJORLY FROM A V OVERSEAS, STAR RICE MUNDY, A V EXPORTS, RTGS RS 60696 LAKHS MAJORLY FROM GORI TRADING CO, A V EXPORTS, E PAO GST REFUNDS, REMITTANCE RS 1,27869 LAKHS MAJORLY FROM SUNCONS TRADING AND CONTRACTING WLL, ALTAWAF GENERAL TRADING CO NE, MR DASHTY FOOD STUFF COMPANY TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,99455 LAKHS OF WHICH CASH WAS RS 2498 LAKHS BY DELHI - NETAJI SUBHASH PLACE BRANCH, TRANSFERS RS 064 LAKHS MAJORLY TO NWC ROADWAYS, PRAHLAD KUMAR JAT, CLEARING RS 26088 LAKHS MAJORLY TO CMS TRANSFERS, RTGS RS 1,69003 LAKHS MAJORLY TO SHREE LUXMI FASTNERS, GORI TRADING CO AND OTHERS RS 1708 LAKHS- DTAX, GST FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 12,26110 LAKHS OF WHICH CASH WAS RS 5700 LAKHS MAJORLY IN DELHI - NETAJI SUBHASH PLACE BRANCH, TRANSFERS RS 2759 LAKHS MAJORLY FROM POTUS KIRANA PRIVATE LIMITED, SHREE NATH JI AGRO PRODUCTS PRIVATE LIMITED, CLEARING RS 5894 LAKHS MAJORLY FROM ICICI LOMBARD GENERAL INSURANCE, BLUE WATER EXPOTRADE INDIA PRIVATE LIMITED, NEFT RS 32632 LAKHS MAJORLY FROM A V EXPORTS, NOBLE INTERNATIONAL, E PAO GST REFUNDS, RTGS RS 81493 LAKHS MAJORLY FROM A V OVERSEAS, A V EXPORTS, REMITTANCE RS 10,95676 LAKHS MAJORLY FROM YAQUBY INTERNATIONAL, ALTAWAF GENERAL TRADING CO, 1 MEZZAN FOODS COMPANY LIMITED, INTERNET FUND TRANSFERS RS 1811 LAKHS FROM SHYAM SHRI AGROTECH (031605002187), MOBILE MONEY TRANSFERS RS 037 LAKHS IN 1 INSTANCE, UPI TRANSFERS RS 087 LAKHS IN 5 INSTANCES TOTAL DEBIT SIDE RS 12,28351 LAKHS OF WHICH CASH WAS RS 9958 LAKHS BY DELHI - NETAJI SUBHASH PLACE BRANCH, CLEARING RS 2,08851 LAKHS MAJORLY TO CMS TRANSFERS, RTGS RS 9,88989 LAKHS MAJORLY TO A V OVERSEAS, BANSAL MARBLE TRADERS, A V EXPORTS, REMITTANCE RS 1128 LAKHS MAJORLY TO SALALAH MACARONI, INTERNET FUND TRANSFER RS 475 LAKHS, MOBILE MONEY TRANSFERS RS 704 LAKHS IN 5 INSTANCES AND OTHERS RS 17793 LAKHS- DTAX, GST",{"entities":[(3820, 3855,"FOREIGN ORG"),(3857,3886,"FOREIGN ORG"),(3891,3915,"FOREIGN ORG"),(4953, 4973,"FOREIGN ORG"),(4975, 5002,"FOREIGN ORG"),(5005, 5033,"FOREIGN ORG"),(5486,5501,"FOREIGN ORG")    ]}),
            ("ON 01-03-2021, RS 5170 LAKHS WERE CREDITED THROUGH REMITTANCE FROM VIRAGO INTERNATIONAL TRADING PTE LTD IN USD FROM SINGAPORE FOR EXPORT OF RICE AND ON SAME DAY RS 55 LAKHS WERE DEBITED THROUGH RTGS TO MODINA AGRO PRODUCT (ACCOUNT NO: 916020072720756, IFSC CODE: UTIB0000148, BANK: AXIS BANK), RAYAL RICE TECH (ACCOUNT NO: 917020042177697, IFSC CODE: UTIB0000148, BANK: AXIS BANK) SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2020-21 WAS RS 2,57142 LAKHS, WHICH SEEMS TO BE VERY HIGH SHIPMENTS MADE EXPORT BILLS PENDING FOR SUBMISSION ARE 13 BILLS OF WHICH 10 ARE OVERDUE ADVANCE RECEIVED EXPORT BILLS PENDING SUBMISSION ARE 6 BILLS AND NONE ARE OVERDUE IN VIEW OF LARGE VALUE FOREX CREDIT IN CURRENT ACCOUNT, HIGH TURNOVER IN THE ACCOUNT AND EXPORT BILLS PENDING SUBMISSION ARE OVERDUE, STR IS BEING FILEDRAINBOW EXPORTS, A PROPRIETORSHIP FIRM, HAVING ADDRESS: 1SST FLOOR FLOOR, 57, ELLIOT ROAD, KOLKATA, WEST BENGAL, INDIA, 700016, CONTACT NUMBER: +919434251148 AND EMAIL ID: RICERAINBOWEXPORTS@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000605501507 WITH ICICI BANK LTD, 22, SIR RN MUKHERJEE ROAD, KOLKATA, WEST BENGAL, 700001, SINCE 31082018 DATE OF INCORPORATION IS 01052016 PAN NUMBER OF THE ACCOUNT IS BYRPS9174B IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0216902266 TRANSACTION LINKED ACCOUNTS ARE 000605035561, 154305500794 AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS SEKH MAINUDDIN (PAN- BYRPS9174B) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN-BYRPS9174B, AADHAAR AND IMPORT EXPORT CERTIFICATE (0216902266) AS PER DUE DILIGENCE THE FIRM IS INVOLVED INTO EXPORT OF RICE, HAVING OFFICE SET UP IN THE COMMERCIAL AREA OF THE CITY DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS 40 LAKHS ALERT WAS TRIGGERED FOR LARGE VALUE FOREX CREDIT IN CURRENT ACCOUNT (DAILY) BALANCE IN THE ACCOUNT AT 31052021 WAS RS018 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31052021 SHOWS TOTAL CREDIT RS 1,16378 LAKHS OF WHICH NEFT RS 169 LAKHS MAJORLY FROM RAINBOW EXPORTS, MODINA AGRO PRODUCT, RTGS RS 4347 LAKHS MAJORLY FROM MODINA AGRO PRODUCT, RAYAL RICE TECH, RAINBOW AGRI TECH, REMITTANCE RS 1,11858 LAKHS MAJORLY FROM VEGA FOODS CORP PTE LTD IN USD FROM SINGAPORE FOR EXPORT OF RICE, VIRAGO INTERNATIONAL TRADING PTE LTD IN USD FROM SINGAPORE FOR EXPORT OF RICE, MOBILE MONEY TRANSFERS RS 002 LAKHS IN 1 INSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,16408 LAKHS OF WHICH CLEARING RS 4838 LAKHS MAJORLY TO CMS TRANSFERS, RTGS RS 1,10545 LAKHS MAJORLY TO MODINA AGRO PRODUCT, MOBILE MONEY TRANSFERS RS 747 LAKHS IN 9 INSTANCESFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 2,57142 LAKHS OF WHICH NEFT RS 995 LAKHS MAJORLY FROM RAINBOW EXPORTS, RTGS RS 7364 LAKHS MAJORLY FROM RAINBOW EXPORTS, REMITTANCE RS 2,48773 LAKHS MAJORLY FROM FROM VEGA FOODS CORP PTE LTD IN USD FROM SINGAPORE FOR EXPORT OF RICE, VIRAGO INTERNATIONAL TRADING PTE LTD IN USD FROM SINGAPORE FOR EXPORT OF RICE TOTAL DEBIT SIDE RS 2,57112 LAKHS OF WHICH CASH WAS RS 532 LAKHS BY KOLKATA-RN MUKHERJEE ROAD BRANCH, CLEARING RS 7207 LAKHS MAJORLY TO DMT LOGISTICS PRIVATE LIMITED, RTGS RS 2,47198 LAKHS MAJORLY TO MODINA AGRO PRODUCT, RAYAL RICE TECH, INTERNET FUND TRANSFER RS 745 LAKHS- SALARY PAYMENTS, CARD TRANSACTIONS RS 033 LAKHS, MOBILE MONEY TRANSFERS RS 954 LAKHS IN 17 INSTANCESFY 2019-20 FROM 01042019 TO 31032020: TOTAL CREDIT SIDE RS 196 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 547 LAKHS OF WHICH CASH WAS NILFY 2018-19 FROM 01042018 TO 31032019: TOTAL CREDIT SIDE RS 9492 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 9123 LAKHS OF WHICH CASH WAS RS 014 LAKHS TRANSACTION PATTERN SHOWS, REGULAR REMITTANCE CREDIT FROM EXPORTED FOOD GRAINS, AND THOSE ARE DEBITED TOWARDS VARIOUS FOOD MANUFACTURING BUSINESS, RICE MILLS, ETC, IMMEDIATELY AFTER THE CREDIT IN THE ACCOUNT FOR EXAMPLE, ON 22-02-2021, RS 6683 LAKHS WERE CREDITED THROUGH REMITTANCE FROM VIRAGO INTERNATIONAL TRADING PTE LTD IN USD FROM SINGAPORE FOR EXPORT OF RICE AND ON SAME DAY RS 5515 LAKHS WERE DEBITED THROUGH RTGS TO MODINA AGRO PRODUCT (ACCOUNT NO: 916020072720756, IFSC CODE: UTIB0000148, BANK: AXIS BANK), RAYAL RICE TECH (ACCOUNT NO: 917020042177697, IFSC CODE: UTIB0000148, BANK: AXIS BANK)",{"entities":[(67, 103,"FOREIGN ORG"),(2397, 2420,"FOREIGN ORG"),(2463, 2499,"FOREIGN ORG"),(3152, 3175,"FOREIGN ORG"),(3218, 3254,"FOREIGN ORG"),(4255, 4291,"FOREIGN ORG")]}),
            ("THERE ARE 47 EXPORT BILLS PENDING SUBMISSION, WHERE 45 BILL ARE OVERDUE, MORE THAN 365 DAYS SHIPMENTS MADE EXPORT BILLS PENDING SUBMISSION OF 149 BILLS ALSO OVERDUE IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, HIGH TURNOVER IN THE ACCOUNT AND BOE PENDING WITH OVERDUE FOR SUBMISSION, STR IS BEING FILEDBALAJI TRADERS, A PROPRIETORSHIP FIRM, HAVING ADDRESS : 15B, KALAKAR STREET, GROUND FLOOR, KOLKATA, WEST BENGAL, INDIA,  700007,  CONTACT NUMBER: +919883318170 AND EMAIL ID: BALAJITRADERS03516@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 627705501012 WITH ICICI BANK LTD, PBNO 6702, 11, VIVEKANANDA ROAD, KOLKATA, WEST BENGAL, 700007,  SINCE 08022018 DATE OF INCORPORATION IS 01022018 PAN OF THE ACCOUNT IS ANVPA7889B PAN LINKED ACCOUNTS ARE 329301500981, 627705501141, 627706000008, 627752000001, 627766000002, 627766000003, 627766000005, 627955000074 TRANSACTION LINKED ACCOUNTS ARE 627905501601, 627752000001 AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS AJAY AGARWAL (PAN- ANVPA7889B) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN: ANVPA7889B AND LETTER OF DECLARATION OF PROPRIETORSHIP FIRM THE FIRM IS INVOLVED INTO EXPORT IMPORT OF FMCG PRODUCTS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO RS 5 CRORES MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRASACTIONS IN THE ACCOUNT AND BOE PENDING WITH OVERDUE FOR SUBMISSION BALANCE IN THE ACCOUNT AT 28022021 WAS RS 3815 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2020-21 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 12,28893 LAKHS OF WHICH TRANSFERS RS 4900 LAKHS MAJORLY FROM BALAJI TRADERS, CLEARING RS 5795 LAKHS MAJORLY FROM MARIYAM ENTERPRISE, ASIAN STAR ENTERPRISE, RTGS RS 40318 LAKHS MAJORLY FROM CHHAJER AGRO PRODUCTS, SATI INTERNATIONAL, EXIM TRADE SERVICE PRIVATE LIMITED, REMITTANCE RS 11,40493 LAKHS MAJORLY FROM SONALI TRADE INTERNATIONAL AT BANGLADESH IN USD FOR EXPORT OF FRESH FRUITS, BK TRADERS AT BANGLADESH IN USD FOR EXPORT OF FRESH FRUITS, INTERNET FUND TRANSFERS RS 23016 LAKHS FROM GYANIRAM AGARWAL  CO (627905501601), KOLKATA AAYAT NIRYAT PRIVATE LIMITED (627905016044) AND OTHERS RS 11232 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS  12,41980 LAKHS OF WHICH CASH WAS RS 10990 LAKHS BY VK ROAD CALCUTTA BRANCH, CLEARING RS 7,23099 LAKHS MAJORLY TO ASIAN STAR ENTERPRISE, MOHAMMED FAIYAZ AHMED, SHIFA FOODS COMPANY, DIAMOND FRUIT COMPANY, MKCAGRO FRESH LTD, SALEEM IBRAHIM SHEIKH, JGT VEGETABLES, RTGS RS 5,03741 LAKHS MAJORLY TO AMAR VEGETABLE COMPANY, SANMAN PUNDALIK GAIKAWAD FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2019-20 FROM 01042019 TO 31032020: TOTAL CREDIT SIDE RS 12,84587 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 12,79533 LAKHS OF WHICH CASH WAS RS 8325 LAKHS FY 2018-19 FROM 01042018 TO 31032019: TOTAL CREDIT SIDE RS 10,99903 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 11,00970 LAKHS OF WHICH CASH WAS RS 10080 LAKHS FY 2017-18 FROM 08022018 TO 31032018: TOTAL CREDIT SIDE RS 27023 LAKHS OF WHICH CASH WAS RS 015 LAKHS TOTAL DEBIT SIDE RS 14108 LAKHS OF WHICH CASH WAS RS 300 LAKHS TRANSACTIONS IN THE ACCOUNT ARE WITH RELATED PARTIES, BUT THE VOLUME OF THE TRANSACTIONS IS NOT INLINE WITH BUSINESS SETUP FOR EXAMPLE, ON 23-02-2021 RS 7250 LACS WERE CREDITED THROUGH REMITTANCE FROM SONALI TRADE INTERNATIONAL AT BANGLADESH IN USD FOR EXPORT OF FRESH FRUITS AND ON THE SAME DAY RS 8010 LACS WAS DEBITED TO MKCAGRO FRESH LTD, SALEEM IBRAHIM SHEIKH, JGT VEGETABLES ETC THROUGH CMS TRANSFERS ALSO ON 17-02-2021 RS 6295 LACS CREDITED THROUGH INWARDS REMITTANCE BK TRADERS AT BANGLADESH IN USD FOR EXPORT OF FRESH FRUITS AND ON THE SAME DAY RS 65 LACS DEPLOYED THROUGH CLEARING (CMS TRANSFERS) TO SHIFA FOODS COMPANY, DIAMOND FRUIT COMPANY SIMILAR TRANSACTION WERE SEEN IN THE ACCOUNT",{"entities":[(2535, 2556,"FOREIGN ORG")]}),
            ("ALSO MOST OF THE FUNDS DEPLOYED ARE IN ROUND FIGURES FOR EXAMPLE ON 15-04-2021 FUNDS OF RS RS2 CRORE CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM ESLI ENTERPRISES (AC NO: 36726441344, IFSC: SBIN0063788, SBI BANK) AND FURTHER FUNDS ARE DEPLOYED IN PARTS THROUGH RTGS TOWARDS SRI DURGA MALLESWARASWAMYVARLA VIJYAWADA (AC NO: 10442746280, IFSC: SBIN0001208) IN ANOTHER INSTANCE ON 19-01-2021 FUNDS OF RS 25 LACS CREDITED IN THE ACCOUNT THROUGH RTGS FROM SIDDHANT ENTERPRISES (AC NO: 911020013042791, IFSC: UTIB0000789, AXIS BANK) AND DEPLOYED IN PARTS THROUGH RTGS TOWARDS DIFFERENT ENTITIES LIKE SHIVA KARUNYA ENTERPRISES (AC NO: 39540684509, IFSC: SBIN0021978, SBI BANK) SEEMS UNUSUAL NO OTHER BUSINESS RELATED EXPENSES, BILL PAYMENTS, UTILITY PAYMENTS SEEN IN ACCOUNT TOTAL ANNUAL TURNOVER FOR THE FY 21-22 WAS RS60 CRORES WHICH FAR EXCEEDS THE DECLARED TURNOVER OF RS 5 CRORES FREQUENT HIGH VALUE CREDITS FOLLOWED BY IMMEDIATE DEBITS SEEMS TO BE UNUSUAL AND RAISES GROUND OF SUSPICION IN VIEW OF SUDDEN INCREASE IN VALUE AND VOLUME OF TRANSACTIONS  IN A MONTH FOR THE CLIENT AND HIGH TURNOVER, STR IS BEING REPORTEDNARESH WOMEN HAIR ENTERPRISES, A PARTNERSHIP FIRM, HAVING ADDRESS: BAHADURGUDA,3-4-39,RTC COLONY, L B NAGAR, HYDERABAD, TELANGANA, INDIA, 500074, CONTACT NUMBER: +919866688827 AND EMAIL ID: CHINNAMNARESH27@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 131405500654 WITH ICICI BANK LTD, 3-11-467, BESIDE KUSHAL CONVENTION, LB NAGAR, MANSOORABAD, HYDERABAD, TELANGANA, 500074, SINCE 07022020 DATE OF INCORPORATION IS 16042011 PAN NUMBER OF THE ACCOUNT IS AAIFN4820H GSTIN AS PER BANK RECORDS IS 36AAIFN4820H1ZATRANSACTION LINKED ACCOUNTS ARE 038205001855 AND 236605002049 AUTHORIZED SIGNATORY/PARTNER OF THE REPORTED ACCOUNT IS CHINNAM NARESH (PAN-AHQPC1338J) BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS CH EGAIAHA (AFWPC1509H), CH NARESH (AHQPC1338J), CHINNA KRISHNAVENI (APDPC6555J), TELLAMEKALA SRINU (AKWPT6847Q) THE CUSTOMER IS CATEGORIZED AS MEDIUM RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE AAIFN4820H, GST-36AAIFN4820H1ZA ENTITY IS IN TO TRADING OF WOMEN HAIR DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO RS 5 CRORES ALERT WAS TRIGGERED FOR SUDDEN INCREASE IN VALUE AND VOLUME OF TRANSACTIONS IN A MONTH FOR THE CLIENT BALANCE IN THE ACCOUNT AT 18072021 WAS RS42495 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 18072021 SHOWS TOTAL CREDIT RS 6,48346 LAKHS OF WHICH TRANSFERS RS 1,51220 LAKHS MAJORLY FROM EXCELLENT HAIR PRIVATE LIMITED, NEFT RS 3581 LAKHS MAJORLY FROM MD MAINUL HAQUE, MD ENATUR RAHAMAN, RTGS RS 1,52900 LAKHS MAJORLY FROM ESLI ENTERPRISES, SIDDHANT ENTERPRISES, INDIGO HAIR COMPANY, REMITTANCE RS 3,39076 LAKHS MAJORLY FROM CLASSIC LOGISTICS PTE LTD, INTERNET FUND TRANSFERS RS 600 LAKHS MAJORLY FROM NAY LA FAMILY EXPORTS PRIVATE LIMITED AND EXCELLENT HAIR PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 650 LAKHS IN 4 INSTANCES, UPI TRANSFERS RS 320 LAKHS IN 5 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 6,05969 LAKHS OF WHICH RTGS RS 5,05210 LAKHS MAJORLY TO REMITTANCE RS 078 LAKHS, INTERNET FUND TRANSFER RS 98737 LAKHS MAJORLY TO NAY LA FAMILY EXPORTS PRIVATE LIMITED AND EXCELLENT HAIR PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 1944 LAKHS IN 29 INSTANCES FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 3,01359 LAKHS OF WHICH RS 0 BY CASH, TRANSFERS RS 2,42136 LAKHS MAJORLY FROM NAY LA FAMILY EXPORTS PRIVATE LIMITED, CLEARING RS 7500 LAKHS, NEFT RS 10028 LAKHS MAJORLY FROM ELIM ENTERPRISES, ESLI ENTERPRISES, CBEC ICEGATE, RTGS RS 21899 LAKHS MAJORLY FROM SIDDHANT ENTERPRISES, HANIF SEKH,SEKH DELABAR, REMITTANCE RS 18719 LAKHS, INTERNET FUND TRANSFERS RS 500 LAKHS MAJORLY FROM TO NAY LA FAMILY EXPORTS PRIVATE LIMITED AND EXCELLENT HAIR PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 491 LAKHS IN 11 INSTANCES, UPI TRANSFERS RS 001 LAKHS IN 1 INSTANCE AND OTHERS RS 085 LAKHS TOTAL DEBIT SIDE RS 3,05740 LAKHS OF WHICH CASH WAS RS 500 LAKHS BY BRANCH, CLEARING RS 7500 LAKHS, NEFT RS 300 LAKHS MAJORLY TO GOVERNMENT OF TELANGANA, RTGS RS 2,58247 LAKHS MAJORLY TO MOTHER ENTERPRISES, SRI DURGA MALLESWARAS, YASODHA KRISHNAVENI WAST, REMITTANCE RS 004 LAKHS, INTERNET FUND TRANSFER RS 28041 LAKHS MAJORLY TO TO NAY LA FAMILY EXPORTS PRIVATE LIMITED AND EXCELLENT HAIR PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 11148 LAKHS IN 283 INSTANCES FY 2019-20 FROM 01042019 TO 31032020: TOTAL CREDIT SIDE RS 4501 LAKHS TOTAL DEBIT SIDE RS 003 LAKHS AS PER TRANSACTION IT IS OBSERVED THAT FUNDS CREDITED IN THE ACCOUNT THROUGH TRANSFER, NEFT, RTGS AND DEPLOYED THROUGH MMT, RTGS SEEMS UNUSUAL CREDITS FOLLOWED BY IMMEDIATE DEBITS ON THE SAME DAY FOR SAME AMOUNT IS SEEN WHICH SEEMS TO BE SUSPICIOUS",{"entities":[(2850, 2875,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN SHOWS, HIGH VALUE NON CASH CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE, ON 05-05-2021, RS 25 LACS AMOUNT OF FUND CREDITED THROUGH TRANSFERS FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED (193705002450) WHERE FUND DEBITED THROUGH RTGS TOWARDS (SIDDHISHREE ISPAT UDYOG, ACCOUNT: 50200047733005, IFSC: HDFC0000529) ALSO FROM 12-03-2021 TO 15-03-2021 RS 50 LACS GOT CREDITED BY INTERNET FUND TRANSFERS FROM RAMUKA GLOBAL ISPAT METALS PRIVATE LIMITED (193705002450), NEFT FROM BIG C TECHNOLOGIES PRIVATE LIMITED (650014108634, IFSC-INDB0000006) AND ON 15-03-2021 RS 4838 LACS GOT DEBITED THROUGH OUTWARD REMITTANCE TO OSCAR INTERNATIONAL GROUP CO LTD, (ADD- 6 F M BUILDING NUMBER 2 NORTH PART, NORTH PART, CHINA), IN USD FOR ADVANCE PAYMENT AGAINST IMPORTS SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED LIKE ROUTING OF FUNDS IN VIEW OF NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS AND ROUTING OF FUNDS, STR IS BEING FILEDDIA ENTERPRISES, A PROPRIETORSHIP FIRM, HAVING ADDRESS: NO 1, OLD NO 550 1ST MAIN ROAD, 1ST FLOOR, 14TH CRS, SAMPANGIRAMANAGAR, BANGALORE, KARNATAKA, INDIA, 560027, CONTACT NUMBER: +919901489460 AND EMAIL ID: DEEPAKRAMUKA@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 193705002384 WITH ICICI BANK LIMITED, NO 102, K H ROAD, BANGALORE, KARNATAKA, 560027, SINCE 27092019 DATE OF INCORPORATION IS 01072017 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT, SAVINGS ACCOUNT PAN OF THE ACCOUNT IS AKRPR8734P OTHER RELATIONSHIP OF CUSTOMER IS 000201098361, 085205000408, 193705002450, 193705002724, 627501212632 TRANSACTION LINKED ACCOUNTS ARE 193705002450, 054405008827 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS DEEPAK RAMUKA (PROPRIETOR - AKRPR8734P) THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE LETTER OF DECLARATION OF PROPRIETORSHIP FIRM, PAN - AKRPR8734P AS PER ENHANCE DUE DILIGENCE, ENTITY IS ENGAGED IN IMPORTING OF COMPUTER RAM, USB FLASH DRIVES, BLUETOOTH ADAPTERS ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40LACS TO BELOW RS 5CRS ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT 06052021 WAS RS 021 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 06052021 SHOWS TOTAL CREDIT RS 2823 LAKHS OF WHICH NEFT RS 4261 LAKHS MAJORLY FROM DEEPAK RAMUKA, BIG C TECHNOLOGIES PRIVATE LIMITED, RTGS RS 9210 LAKHS MAJORLY FROM MEGHANI TEXTILES, MOHINI SAREE CENTRE, DEEPAK RAMUKA, INTERNET FUND TRANSFERS RS 14085 LAKHS MAJORLY FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED TOTAL DEBITS DURING THE SAME PERIOD WAS RS 28307 LAKHS OF WHICH RTGS RS 17005 LAKHS MAJORLY TO SIDDHISHREE ISPAT UDYOG, REMITTANCE RS 5416 LAKHS MAJORLY TO OSCAR INTERNATIONAL GROUP CO LIMITED, INTERNET FUND TRANSFER RS 5499 LAKHS MAJORLY TO RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS (OR ACCOUNT OPENING DATE IF EARLIER), YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 1,75110 LAKHS OF WHICH CASH WAS RS 511 LAKHS MAJORLY IN BANGALORE - K H ROAD BRANCH, CLEARING RS 4227 LAKHS MAJORLY FROM ELECTRO STEEL STEELS LIMITED, NEFT RS 21811 LAKHS MAJORLY FROM MS RAJ CONSTRUCTION COMPANY, DIA ENTERPRISES, BIG C TECHNOLOGIES PRIVATE LIMITED, RTGS RS 1,08660 LAKHS MAJORLY FROM BIG C TECHNOLOGIES PRIVATE, MITTAL TRADERS, DEEPAK RAMUKA, INTERNET FUND TRANSFERS RS 32387 LAKHS MAJORLY FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED, MOBILE MONEY TRANSFERS RS 7451 LAKHS IN 56 INSTANCES TOTAL DEBIT SIDE RS 1,75535 LAKHS OF WHICH RTGS RS 62463 LAKHS MAJORLY TO JD ENTERPRISES, DEEPAK RAMUKA, REMITTANCE RS 34793 LAKHS MAJORLY TO OSCAR INTERNATIONAL GROUP, OSCAR INTERNATIONAL GROUP COLTD, OSCAR INTERNATIONAL GROUP CO LTD, INTERNET FUND TRANSFER RS 75413 LAKHS MAJORLY FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED FY 2019-20 FROM 27092019 TO 31032020: TOTAL CREDIT SIDE RS 36282 LAKHS TOTAL DEBIT SIDE RS 35760 LAKHS OF WHICH CASH WAS RS 060 LAKHS, REMITTANCE RS 15985 LAKHS",{"entities":[(662, 694,"FOREIGN ORG"),(452, 494,"INIDAN ORG"),(2586, 2620,"INIDAN ORG"),(2982, 3018,"FOREIGN ORG"),(3075, 3114,"INIDAN ORG"),(3116, 3133,"INIDAN ORG"),(3428, 3456,"INIDAN ORG"),(3520, 3535,"INIDAN ORG"),(3767, 3784,"INIDAN ORG"),(3981, 4006,"FOREIGN ORG") ,(4008,4039,"FOREIGN ORG"),(4041, 4073,"FOREIGN ORG")]}),
            ("6 CUSTOMER IS YET TO SUBMIT BILLS OF ENTRY PERTAINING TO 4 ADVANCE IMPORT REMITTANCES TOTALLING TO USD 181235 PENDING FOR A PERIOD LESS THAN 3 MONTHS 7 ON REVIEWING CONSOLIDATED ACCOUNT ACTIVITY IT WAS NOTED THAT INTER-CONNECTED FUNDS TRANSFER TRANSACTIONS WITHIN OCEAN GREEN ENERGY, VEDYA ENTERPRISE AND VISHINE FUEL STATION AMOUNTED TO RS098 CR  FURTHER U-TURN TRANSACTIONS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS452 CR WAS RECEIVED AND RS3030 CR WAS SENT BACK TO THE SAME ACCOUNTS OF KHANDA BIOFUELS  HDFC  50200048479801/ SBI  00000062411606638, JITIN BAJAJ HDFC 01821000023024 , RADHEY ENTERPRISE FEDERAL BANK  18640200007261, AGARWAL ENTERPRISES  KARNATAKA BANK  5167000600009601/ AXIS BANK  455010200002356 AND VISHINE FUEL STATION BOB- 08950200001811/ UBI - UBI917350138880 MAINTAINED WITH OTHER BANKS IT APPEARS THAT SUCH INTER-CONNECTED AND U-TURN TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE 8 IN 12 INSTANCES TOTALLING TO RS064 CR DURING PERIOD DECEMBER 2020 TILL JULY 2021, CASH WITHDRAWALS FROM THE ACCOUNT OF VISHINE FUEL STATION WERE STRUCTURED BELOW RS1000 LAKHS POSSIBLY TO AVOID FURNISHING PURPOSE OF WITHDRAWALS TO BANK BANK HAS FILED CASH TRANSACTION REPORT ON THE ACCOUNT OF VISHINE FUEL STATION IN MARCH AND MAY 21 9 DESPITE ROUTING HIGH VALUE AND VOLUME OF TRANSACTIONS NO PAYMENTS TOWARDS TAXES, CUSTOM DUTIES ETC, COULD BE NOTED THROUGH THE ACCOUNTS DURING REVIEW PERIOD 10 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN THE NAME OF OCEAN GREEN ENERGY AND ACCOUNT MAY HAVE BEEN OPENED TO ROUTE TRANSACTIONS RELATED TO VISHINE FUEL STATION AND VEDYA ENTERPRISE WITHOUT ANY ECONOMIC RATIONALE FOR LAYERING AND POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING ACCOUNT OF OCEAN GREEN ENERGY IS ACTIVE WITH BALANCE OF RS5,01,986/-  AS ON AUGUST 1, 2021 QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-2022 IS DEBIT: RS1858 CR IN CASH RS029 CR AND CREDITS: RS1855 CR IN CASH RS000 CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE: 2020-2021 IS DEBIT: RS2497 CR IN CASH RS068 CR AND CREDITS: RS2538 CR IN CASH RS004 CR2019-20 IS DEBIT: RS054 CR IN CASH RS000 CR AND CREDITS: RS062 CR IN CASH RS001 CRA CURRENT ACCOUNT BEARING NUMBER 2022220222 WAS OPENED IN THE NAME OF A PARTNERSHIP FIRM OCEAN GREEN ENERGY ON 02-JUN-21 AT OUR BRANCH LOCATED IN RAJHANS HOUSE,VARACCHA ROAD - SURAT, GUJARAT - 395006 THE ENTITY WAS FORMED ON 19-MAY-21 AND HAS SUBMITTED COPIES OF PAN AAHFO4390G, PARTNERSHIP DEED, GST REG CERT 24AAHFO4390G1Z8 AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS B-4-1201 CELEBRITY GREENS ABHVA ROAD NR HAPPY HALLMARK SHOPPERS VESU B-H AAGAM SHOPPING SURAT 395007 VORA MANUBHAI GOBARBHAI AND VEKARIYA VIPULKUMAR KANTILAL ARE THE PARTNERS AND HAVE REGISTERED MOBILE NUMBERS 9537507900 AND 7383956620 RESPECTIVELY WITH THE BANK VEKARIYA VIPULKUMAR KANTILAL 1813 PERCENT SHAREHOLDER, JITIN BAJAJ 50 PERCENT AND HIREN SAVAILAL DESAI 1812 PERCENT ARE THE UBOS IN THE SAID FIRM THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO OIL AND GAS BUSINESS WITH ANNUAL TURNOVER OF RS500 CR AND THAT THE INDIVIDUALS ARE BUSINESSMEN WITH ANNUAL INCOME IN THE RANGE OF RS005 CR TO RS010 CR ONE OF THE UBO MENTIONED ABOVE JITIN BAJAJ MAINTAINS TWO OTHER CURRENT ACCOUNTS AS A PROPRIETOR IN THE NAMES OF VEDYA ENTERPRISE 9825045309, A/C OPENED ON 17-MAY-21, NATURE OF BUSINESS  OIL AND GAS, ANNUAL TURNOVER  RS2000 CR AND VISHINE FUEL STATION 9713967785, A/C OPENED ON 09-JAN-20, NATURE OF BUSINESS  PETROLEUM PRODUCTS, ANNUAL TURNOVER  RS2400 CR THE ACCOUNTS WERE OPENED AT OUR SURAT - RING ROAD BRANCH WHILE REVIEWING REPORT ON ADVANCE IMPORT REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE ADVANCE IMPORT REMITTANCES WERE ROUTED THROUGH THE ABOVE MENTIONED ACCOUNT OF OCEAN GREEN ENERGY ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH PAYMENTS WERE DONE SUBSEQUENT TO ROUND SUM HIGH VALUE ROUND SUM RTGS/NEFT / INTERNAL FUNDS TRANSFER CREDITS HENCE DUE DILIGENCE WAS CARRIED OUT AND THIS CONSOLIDATED REPORT IS FILED ON OCEAN GREEN ENERGY AND INTERCONNECTED ACCOUNTS ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL AUGUST 1, 2021 IT WAS NOTED THAT IN THE ACCOUNT OF OCEAN GREEN ENERGY A PATTERN OF TRANSACTIONS WERE FOLLOWED WHEREIN ROUND SUM RTGS/NEFT CREDITS WERE FOLLOWED BY IMMEDIATE ADVANCE IMPORT REMITTANCES 2 IMPS/NEFT/RTGS REMITTANCE CREDITS AMOUNTED TO RS423 CR MAINLY FROM JITIN BAJAJ RS178 CR, HDFC  01821000023024, HIREN SAVAILAL DESAI RS055 CR, ICICI  138701503926, VIPULKUMAR KANTILAL VEKARIYA RS035 CR, BOB- 78220100002241, GLOBAL PETRO OIL RS030 CR, ICICI  777705385555 AND V D ENTERPRISE RS017 CR, ICICI  162305500254 CHEQUE DEPOSITS AMOUNTED TO RS015 CR 3 SUBSEQUENT TO SUCH CREDITS ADVANCE IMPORT REMITTANCES AMOUNTED TO RS308 CR FAVOURING UAE BASED ENTITIES ENERGIA BIOCHEM FZE RS247 CR AND BIOENERGIA FZC RS060 CR TOWARDS IMPORT OF CASHEW KERNELS / OTHER FRUIT WASTES USED TO EXTRACT OIL / GREASE 4 RTGS/NEFT PAYMENTS AMOUNTED TO RS121 CR FAVOURING OIKOS RS079 CR, SBI  39568156589 AND PALAK LOGISTICS PVT LTD RS041 CR, IDBI - 0183102000017693 5 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY OCEAN GREEN ENERGY, IT WAS NOTED THAT IT WAS A RESIDENCE ADDRESS WHERE NO SIGN BOARD STOCK OF GOODS OR BUSINESS ACTIVITY WAS SEEN CUSTOMER INDICATED THAT THE BUSINESS IS AT ANOTHER LOCATION  C-1052/53, CENTRAL BAZAAR, OPPO VARACHHA POLICE STATION, VARACHHA  SURAT  395006 DURING VISIT BY BRANCH OFFICIAL AT THE BUSINESS ADDRESS, IT WAS A 400 SQ FT RENTED OFFICE WHERE OFFICE SET UP LIKE CHAIRS, TABLES WERE SEEN HOWEVER NO SIGN BOARD, BUSINESS ACTIVITY OR STOCK OF GOODS COULD BE SEEN",{"entities":[(264, 281,"FOREIGN ORG"),(1763, 1781,"FOREIGN ORG")   ,(2311, 2327,"FOREIGN ORG"),(3992, 4009,"FOREIGN ORG"),(4255, 4273,"FOREIGN ORG") ,(5421, 5439,"FOREIGN ORG")]}),
            ("THE CURRENT ACCOUNT OF COMPANY IS RECEIVING HIGH VALUE ROUND AMOUNT CREDIT FROM ITS OTHER BANK ACCOUNT AND FROM AERO STRUCTURES IMMEDIATELY AFTER RECEIVING THESE HIGH VALUE CREDITS, THE COMPANY USE TO REMIT THESE FUNDS OUTSIDE INDIA THE ROUND AMOUNT HIGH VALUE TRANSACTIONS IN THE ACCOUNT OF THE COMPANY MAY NOT BE IN CONFORMITY WITH THE NORMAL BUSINESS ACTIVITY THE USE OF THE ACCOUNT OF THE COMPANY IN CHANNELIZING OF HIGH VALUE ROUND AMOUNT FUNDS COUPLED WITH OUTWARD REMITTANCES CREATES SUSPICION OF MONEY LAUNDERING ACTIVITY THROUGH LAYERING OF FUNDSTHIS STR IS BEING ESCALATED BASED ON RED FLAG INDICATOR (RFI)IBA_HIGH VALUE NON-CASH WITHDRAWALS IN A MONTHCUSTOMER KYC AND ACCOUNT DETAILS: 1) CUSTOMER NAME : AEQUS PRIVATE LIMITED AND CUSTOMER ADDRESS : 21/2,FIRST FLOOR,BHATTARAHALLI,14 TH KM,OLD MADRAS ROAD, BANGALORE,KARNATAKA, 560049 INDIA 2) DATE OF INCORPORATION: 27032000 3) PAN NO: AACCM3073E  4) PASSPORT NO:  5)OTHER IDENTIFICATION DOCUMENTS: CIN- U80302KA2000PTC026760 6)IEC CODE :  7)GSTIN NO: 29 AACCM3073E1ZL 8)LANDLINE NO: 08042592000  9)OTHER FACILITIES AVAILED: 2 FIXED DEPOSITS AMOUNTING RS708 LAC 10)RISK CATEGORY: HIGH RISK 11)NAME OF THE AUTHORISED SIGNING AUTHORITIES: SAURABH MITTAL, MANOJ PRADHAN, HARISH BANG, MURLIDHAR BODDAPATI, SANTOSH J 12)KYC COMPLIANCE STATUS: YES 13)DATE OF LAST KYC UPDATION: 24032021 CUSTOMER PROFILE: 14)NATURE OF BUSINESS: MANUFACTURING OF MACHINERY 15) OCCUPATION:  16) ANNUAL INCOME AS DECLARED IN KYC:  17)ANNUAL TURNOVER AS DECLARED IN KYC: THRESHOLD RS1000 LAC 18)BENEFICIAL OWNERS AS DETERMINED UNDER RULE 9(3) OF PMLA RULES: AJAY ARAVIND PRABHU, ARAVIND S MELLIGERI, RAJEEV KAUL, RAVI MALLIKARJUN HUGAR, SOURABH MITTAL, EBERHARD KLAUS RICHTER, SHUBHADA MANOHAR RAO ACCOUNT DETAILS : 19)BANK AND BRANCH DETAILS: STATE BANK OF INDIA , INDUSTRIAL FINANCE BRANCH BANGALORE, 560025 20)BANK ACCOUNT NO: 00000030048771736 21)BANK ACCOUNT TYPE: CA 22)BANK A/C OPEN ON: 11-05-2006 23)BANK ACCOUNT STATUS : ACTIVETRANSACTION-03062021 RS425000 LAC CREDIT TRF FROM AXIS BANK A/C OF AEQUS PVT LTD BY RTGS IN 5 TRANSACTIONS07062021 RS416600 LAC DEBIT FOR OUTWARD REMITTANCE30062021 RS10000 LAC CREDIT TRF FROM AXIS BANK A/C OF AERO STRUCTURES BY RTGS30062021 RS3410 LAC CREDIT TRF FROM AXIS BANK A/C OF AERO STRUCTURES BY RTGS30062021 RS10000 LAC CREDIT TRF FROM AXIS BANK A/C OF AERO STRUCTURES BY RTGS01072021 RS3000 LAC CREDIT TRF FROM AXIS BANK A/C OF AERO STRUCTURES BY RTG01072021 RS10000 LAC CREDIT TRF FROM AXIS BANK A/C OF AERO STRUCTURES BY RTG01072021 RS4000 LAC CREDIT TRF FROM HDFC BANK A/C OF AEQUS PVT LTD BY RTG02072021 RS10000 LAC CREDIT TRF FROM HDFC BANK A/C OF AERO STRUCTURES BY RTG02072021 RS2000 LAC CREDIT TRF FROM HDFC BANK A/C OF AERO STRUCTURES BY RTG02072021 RS10000 LAC CREDIT TRF FROM HDFC BANK A/C OF AERO STRUCTURES BY RTG02072021 RS10000 LAC CREDIT TRF FROM HDFC BANK A/C OF AERO STRUCTURES BY RTG08072021 RS74822 LAC DEBIT FOR OUTWARD REMITTANCE TRANSACTION SUMMARY : 24)QUANTUM OF TRANSACTIONS FOR THE CURRENT YEAR: FROM 01-04-2021 TO 31-07-2021  TOTAL CREDIT RS 508992 LAC AND TOTAL DEBIT RS 509628 LAC AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 25) QUANTUM OF TRANSACTIONS FOR PREVIOUS YEARS: FROM 01-04-2020 TO 31-03-2021 TOTAL CREDIT RS 1154136 LAC AND TOTAL DEBIT RS 1158179 LAC AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 FROM 01-04-2019 TO 31-03-2020  TOTAL CREDIT RS 702966 LAC AND TOTAL DEBIT RS 700914 LAC AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 FROM 01-04-2018 TO 31-03-2019  TOTAL CREDIT RS 182364 LAC AND TOTAL DEBIT RS 185555 LAC AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 26)BALANCE IN ACCOUNT ON DATE OF FILING STR : RS127 LAC HISTORY : 27)DETAILS OF EARLIER STRS FILED ON THE ACCOUNT OR ACCOUNT HOLDERS/RELATED PERSONS:",{"entities":[(2180, 2195,"FOREIGN ORG"),(2256, 2271,"FOREIGN ORG"),(2333, 2348,"FOREIGN ORG"),(2392, 2424,"FOREIGN ORG"),(2635, 2649,"FOREIGN ORG"),(2861, 2876,"FOREIGN ORG") ]}),
            ("ON REVIEWING THE TRANSACTION PATTERN OF DOSSIER DEN TECHNOLOGIES PRIVATE LIMITED AC 063963700000485 IT WAS OBSERVED THAT 1SINCE 21-JAN-2021 CUSTOMER HAS RECEIVED CREDITS MAINLY VIA RTGS FROM ZICOM ELECTRONIC SECURITY SYSTEMS TO THE TUNE OF RS 103 CRORES (IN 3 TRANCHES FROM ITS ACCOUNTS HELD WITH ICICI BANK VIZ AC NO - 425405000575) AND FROM HONEYWELL SECURITY SYSTEMS TO THE TUNE OF RS 6827 LAKHS (IN 2 TRANCHES FROM ITS ACCOUNT- 4403120040000211 HELD WITH UJJIVAN SMALL FINANCE BANK LIMITED) ACCOUNT CREDITS WERE FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TO THE TUNE OF RS128 CRORES IN FAVOR OF OVERSEAS ENTITY SAFE DATA STORAGE LTD, UK TOWARDS DATA BASE DATA PROCESSING CHARGES2OVERSEAS ENTITY SAFE DATA STORAGE LTD IS BASED IN UK HOWEVER FUNDS ARE BEING REMITTED TO ITS BANK ACCOUNT HELD IN UNRELATED JURISDICTION OF SINGAPORE 3SAFE DATA STORAGE LTD IS A COMMON OVERSEAS ENTITY RECEIVING CROSS BORDER REMITTANCE FROM ANOTHER YBL CUSTOMER SUPERNITCO INDIA OVERSEAS FOR WHICH STR WAS FILED VIDE FIU BATCH ID 2105271293, YBL CUSTOMER GREEN GOLD ENT FOR WHICH STR WAS FILED VIDE FIU BATCH ID 2105271293 AND STR WAS ALSO FILED ON NON- CUSTOMER SPAN SOFTWARE SOLUTIONS PRIVATE LIMITED VIDE FIU BATCH ID- 2009052265 THAT HAD ATTEMPTED OUTWARD REMITTANCE IN FAVOR OF SAFE DATA STORAGE LTD4THERE IS NO CREDIBLE INFORMATION ON THE OVERSEAS ENTITIES SAFE DATA STORAGE IN PUBLIC DOMAIN APART FROM THEIR WEBSITES WHICH TOO ARE BELOW PAR THE INDUSTRY STANDARDS 5CUSTOMER HAS RECEIVED HIGH VALUE CREDITS TO THE TUNE OF RS 103 CRORES WITHIN A SHORT SPAN OF ONE MONTH (25TH FEB 2021 TO MARCH 2021) FROM DOMESTIC COUNTERPARTY ZICOM ELECTRONIC SECURITY SYSTEMS AND 6827 LAKHS IN SHORT SPAN OF 15 DAYS FROM DOMESTIC COUNTERPARTY HONEYWELL SECURITY SYSTEM INDIA LTD 6DOMESTIC ENTITY ZICOM ELECTRONIC SECURITY SYSTEMS LTD HAS ALSO REMITTED FUND AMOUNTING TO INR 18CR TO ANOTHER YES BANK CUSTOMER STOCKPILE SOFTECH SOLUTIONS PVT LTD AC 004663300003244 FOR WHICH STR WAS FILED VIDE FIU BATCH ID 21060230617NO PUBLIC DOMAIN INFORMATION AVAILABLE ON DOMESTIC ENTITY HONEYWELL SECURITY SYSTEMS INDIA LTD8DURING THE ENHANCED DUE DILIGENCE CARRIED BY THE BRANCH OFFICIALS, THE CUSTOMER CLAIMS THAT IT IS GIVING CCTV FOOTAGE CLOUD STORAGE SERVICES TO BIG SECURITY SERVICES COMPANY LIKE HONEYWELL SECURITY SYSTEMS INDIA LTD ZICOM ELECTRONIC SECURITY SYSTEMS LTD WHICH ARE LOCATED AT MUMBAI 9BRANCH OFFICIALS REQUESTED FOR INVOICES COPIES WHICH WERE RAISED ON THESE BIG COMPANIES CUSTOMER COULD NOT PROVIDE INVOICES AT THAT TIME 10IMMEDIATE CREDIT AND DEBIT TRANSACTIONS PATTERN HAS BEEN OBSERVED IN THE ACCOUNT AND AFTER THE TRANSACTIONS CARRIED OUT, CUSTOMER HAS CLOSED THE ACCOUNT IN SHORT PERIOD WHICH RAISE SUSPICIONIN LIGHT OF THE ABOVE FACTS, IT HAS BEEN DECIDED ON 3RD-AUG-2021 TO FILE A STR ON THE SAID CUSTOMER1) THE CUSTOMER DOSSIER DEN TECHNOLOGIES PRIVATE LIMITED (PAN- AAICD0369A, DOI 10-05-2020, COMMUNICATION AND PERMANENT ADDRESS- UNIT NO 106 FIRST FLOOR PRIZM GRESTONE NO 70/1 MILLERS ROAD WARD NO 78 VASANTHNAGAR BANGALORE KARNATAKA 560052, GSTIN- 29AAICD0369A1ZV, REGISTRATION NUMBER- 139348, CIN- U72900KA2020PTC139348, IEC CODE - AAICD0369A, MOBILE/LANDLINE NUMBER- 9663005808, EMAIL ID- SINGHGURJOT0801@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT AC NO 063963700000485 (SINCE 19-01-2021 , CLOSED ON-27-07-2021) WITH CUNNINGHAM ROAD, BENGALURU BRANCH (ADDRESS- YES BANK LTD, GROUND FLOOR, BATRA CHAMBERS, 1/3, CUNNINGHAM ROAD, BANGALORE, KARNATAKA PIN 560052)2) THE CUSTOMER FALLS UNDER MEDIUM RISK CATEGORY WITH KYC LAST UPDATED IN JAN 2021 GURJOT SINGH AND JIWANJOT SINGH IS AUTHORISED SIGNATORY AND THE BENEFICIAL OWNER IN THE ACCOUNT CUSTOMER IS IT SERVICE PROVIDER WITH AN ANNUAL TURNOVER UP BETWEEN RS 50 CRORES TO RS 75 CRORES 3) IN THE ACCOUNT OF DOSSIER DEN TECHNOLOGIES PRIVATE LIMITED AC NO 063963700000485 TRANSACTIONS ANALYSIS FROM 01-04-2021 TILL 28-07-2021, SHOWS 2 CREDIT TRANSACTIONS AMOUNTING TO RS 950 AND 6 DEBIT TRANSACTIONS AMOUNTING TO RS 819,330OUT OF THE TOTAL CREDITS OF RS 950 IN THE ACCOUNTS, IMPS AMOUNTS TO RS 950 CONSTITUTING 100 PERCENT OF THE TOTAL CREDITSOUT OF THE TOTAL DEBITS OF RS 819,330 IN THE ACCOUNTS, RTGS/NEFT AMOUNTS TO RS 803,370 CONSTITUTING AROUND 98 PERCENT OF THE TOTAL DEBITS4) IN THE ACCOUNT OF DOSSIER DEN TECHNOLOGIES PRIVATE LIMITED AC NO 063963700000485 TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW: FROM 01-04-2020 TILL 31-03-2021, TRANSACTIONS ANALYSIS SHOWS CREDIT TRANSACTIONS AMOUNTING TO RS 18,154,555 COMPRISING NIL CASH DEPOSIT AND DEBIT TRANSACTIONS AMOUNTING TO RS 17,336,175 COMPRISING NIL CASH WITHDRAWALNO TRANSACTIONS WERE OBSERVED FROM 01/04/2018 TO 31/03/2020 AS ACCOUNT WAS OPENED ON 19TH JAN 2021BALANCE IN THE ACCOUNT AS ON 27-07-2021 IS RS 0",{"entities":[(614, 634,"FOREIGN ORG"),(697,717,"FOREIGN ORG"),(832,852,"FOREIGN ORG"),(1264, 1284,"FOREIGN ORG"),(1344,1364,"FOREIGN ORG")   ]}),
            ("5 IT WAS OBSERVED THAT, IN THE A/C NO 50100269447991 OF PRASHANT P PEDNEKAR THERE WAS NO MAJOR CREDITS SINCE INCEPTION TO 13-05-2020 AFTERWHICH, THERE WAS SUDDEN SURGE IN TRANSACTIONS WERE OBSERVED BETWEEN THE PERIOD 14-05-2020 TO 23-06-2020, WHEREIN THE ACCOUNT RECEIVED CREDITS AGGREGATING TO RS 13220 LAKHS, IN 24 INSTANCES WHICH CONTRIBUTES TO 90 PERCENT OF THE OVERALL TRANSACTIONS DURING THE REVIEW PERIOD AFTER WICH THERE WAS A SUBSTANTIAL DROP IN TRANSACTIONS IN THE ACCOUNT ALSO, IT WAS NOTICED THAT THE IN THE ABOVE MENTIONED 24 INSTANCES, THERE WAS IDENTICAL CREDITS OF RS 6 LAKHS, IN 22 INSTANCES FURTHER FUNDS WERE TRANSFERRED TOWARDS HIS OWN BUSINESS CURRENT ACCOUNT AND INVESTED IN THE FORM OF FIXED DEPOSIT6 DURING BRANCH INVESTIGATION, IT WAS OBSERVED THAT THE CUSTOMERS ACCOUNT IS INOPERATIVE FOR LONG PERIOD THERE ARE SUDDEN CREDITS OF RS 24 LAKHS IN THE ACCOUNT WHEREIN RS 6 LACS HAVE BEEN RECEIVED IN THE ACCOUNT FROM 4 DIFFERENT INDIVIDUALS WHO DO NOT SEEM TO BE RELATED FURTHER, MR PRASHANT P PEDNEKAR WAS NOT ABLE TO GIVE CONVINCING CLARIFICATION ON THE CREDITS IN THE ACCOUNT IT IS UNDERSTOOD THAT SIMILAR CREDITS ARE FURTHER EXPECTED IN THE ACCOUNT SINCE THE SOURCE AND PURPOSE OF THE MONEY BEING RECEIVED IN THE ACCOUNT WAS NOT CLEAR, THE TRANSACTIONS SEEM TO BE SUSPICIOUS IN NATURE7 THE SAVINGS ACCOUNTS OF INDIVIDUAL, RECEIVING SUDDEN HIGH VALUE IDENTICAL CREDITS FROM SEEMINGLY UNRELATED PARTIES FOLLOWED BY FUND TRANSFER SEEMS SUSPICIOUS ALSO, THE REASON FOR THE SUDDEN SURGE IN TRANSACTIONS IN INOPERATIVE ACCOUNTS FOLLOWED BY SUBSEQUENT DROP IN TRANSACTIONS COULD NOT BE ASCERTAINED FURTHER, THE FACT THAT THE CUSTOMER WAS UNABLE TO EXPLAIN THE SOURCE OF FUNDS ADDS TO COMPLEXITIES HENCE, RECOMMENDING THE CASE FOR STR FILING8 STR TO BE FILED IN THE LIGHT OF A SUDDEN HIGH VALUE CREDITS IN INACTIVE ACCOUNT B SUDDEN SPURT FOLLOWED BY SUBSTANTIAL DROP IN TRANSACTIONS C CUSTOMER UNABLE TO EXPLAIN THE DETAILS OF THE TRANSACTIONS IN THE ACCOUNT1 OUR KALYANINAGAR - MAHARASHTRA BRANCH HAS RAISED THE SUSPICION BASED ON THE TRANSACTION PATTERN OF CUSTOMER MR PRASHANT P PEDNEKAR A/C NO 50100269447991, WHEREIN SUDDEN HIGH VALUE TRANSACTIONS WERE OBSERVED2 MR PRASHANT P PEDNEKAR MAINTAINS SAVINGS ACCOUNT NO 50100269447991 WITH OUR KALYANINAGAR - MAHARASHTRA BRANCH, SINCE 05-03-2019 CUSTOMER DETAILS UPDATED IN BANK RECORDS ARE DATE OF BIRTH 21-09-1975, PAN AMJPP4914F, SERVICE PROVIDER, GROSS INCOME OF RS25 50 LAKHS REGISTERED ADDRESS B-20 TULIP SOCIETY, OPPOSITE SKY LOUNGE, KALYANI NAGAR, DUKIRK LINE, PUNE, MAHARASHTRA - 411014 RISK PROFILE LOW, MOBILE NO 9820403330 STATUS OF THE ACCOUNT IS ACTIVE THE CURRENT BALANCE IN THE ACCOUNT IS RS 235527/- AS ON 05-07-20213 SCRUTINY OF THE ACCOUNT FROM 01-04-2019 TO 05-08-2021 REVEALS THAT CUSTOMER HAS RECEIVED TOTAL CREDITS AGGREGATING TO RS 14645 LAKHS OUT OF WHICH FUNDS AGGREGATING TO RS 10189 LAKHS WERE IN THE FORM OF NEFT/RTGS MAINLY FROM ASHISH KUSHWAHA RS 17 LAKHS, A/C NO 918010097337899, IFSC- UTIB0000073, POOJA RAJENDRA HIWALE RS 12 LAKHS, A/C NO 918010098552040, IFSC- UTIB0000073, ANKUR PARASHAR RS 12 LAKHS, A/C NO 918010097338630, IFSC- UTIB0000073, SNEHA LAXMIKANT BUNDE RS 12 LAKHS, A/C NO 919010049830242, IFSC- UTIB0001576, VIKAS SANJAY LONDHE RS 12 LAKHS, A/C NO 918010103680483, IFSC- UTIB0000073, MEHUL PUROHIT RS 12 LAKHS, A/C NO 22911241074, IFSC- SCBL0036001 ETC THE MENTIONED CREDITS WERE FOLLOWED BY DEBITS AGGREGATING TO RS 9402 LAKHS, OUT OF WHICH RS 92 LAKHS WERE DEBITED THROUGH FUND TRANSFERS TO PANGEA GLOBAL SERVICES A/C NO 502000173468374 THE PAN/MOBILE LINKED ACCOUNTS ARE M/S PANGEA GLOBAL SERVICES MAINTAINS PROPRIETORSHIP - CURRENT A/C NO - 50200017346837 WITH OUR KALYANINAGA - MAHARASHTRA BRANCH SINCE 06-02-2016 CUSTOMER DETAILS UPDATED IN BANK RECORDS ARE PAN AMJPP4914F, PROFILE BPO KPO MARKET RESEARCH, GROSS INCOME/TURNOVER OF RS4000 LAKHS REGISTERED ADDRESS - MARISOFT BUILDING NO 1, 602 MARIGOLD, KALYANI NAGAR, HAVELI, PUNE, MAHARASHTRA - 411014 RISK PROFILE IS HIGH, MOBILE NO 9820403330 STATUS OF THE A/C IS ACTIVE WITH BALANCE OF 7537467/- AS ON 05-08-2021SCRUTINY OF THE ACCOUNT FROM 01-04-2019 TO 05-08-2021 REVEALS THAT CUSTOMER HAS RECEIVED TOTAL CREDITS AGGREGATING TO RS 220806 LAKHS OUT OF WHICH FUNDS AGGREGATING TO RS 141318 LAKHS WERE IN THE FORM OF NEFT/RTGS MAINLY FROM PANGEA GLOBAL SERVICES RS 91099 LAKHS, IFSC- UTIB0000073, PRASHANT PANDURANG PEDNEKAR RS 500 LAKHS, IFSC- KKBK0000958 ETC THE MENTIONED CREDITS WERE FOLLOWED BY DEBITS AGGREGATING TO RS 213689 LAKHS, OUT OF WHICH RS 121786 LAKHS WERE DEBITED THROUGH FUND TRANSFERS, MAJORLY TOWARDS FIXED DEPOSIT BOOKING RS 1200 LAKHS OTHER ACCOUNTS ARE, A/C NO 50200031987376 OF MARKET CODE TECHSYS PVT LTD, AND A/C NO 50200029635503 OF MARKET CODE, WHEREIN CREDITS ARE MAJORLY THROUGH FOREIGN INWARD REMITTANCE FROM INTEGRATECOM INC COUNTRY UNITED STATES, PURPOSE - BUSINESS AND MANAGEMENT CONSULTANCY AND PUBLIC RELATIONS SERVICES FOLLOWED BY DEBITS THROUGH CHEQUE PAYMENTS WERE OBSERVED A/C NO 50200015573572 OF PANGEA GLOBAL SERVICES AND A/C NO 50100033383601 OF PRASHANT P PEDNEKAR, WHEREIN NO MAJOR TRANSACTIONS WERE OBSERVED DURING THE REVIEW PERIOD",{"entities":[(4818, 4833,"FOREIGN ORG") ]}),
            ("2 FOREIGN CURRENCY USD ACCOUNT 166-410530-511 CREDITS WERE BY WAY OF INWARD REMITTANCES AGGREGATING TO RS4022 CRORES TOWARDS BUSINESS RECEIPTS, ADVANCE IMPORT PAYMENTS, ETC, OF WHICH:O RS972 CRORES 13 TRANCES WERE FROM AL AHLIA CONVERTING INDUSTRIES CO, MUSCAT, OMAN WHICH INTO MANUFACTURING AND DISTRIBUTION OF PAPER PRODUCTS HSBC BANK USA: 6101062556001 AND OMAN ARAB BANK SAOC, OMAN: 3101-059696-500 O RS886 CRORES 12 TRANCHES WERE FROM AL BAYADER MODERN IND CO LLC, SHARJAH UAE, INTO MANUFACTURING AND SUPPLYING COMPREHENSIVE FOOD PACKAGING SOLUTIONS A/C NO: /AE800200000041269119001 OF HSBC BANK MIDDLE EAST LTD, UNITED ARAB EMIRATESO RS495 CRORES WERE FROM PINDO DELI PULP AND PAPER MILLS, INDONESIA, MANUFACTURES AND SELLS PAPER PRODUCTS A/C NO: 0000157527 OF BANK SINARMAS, INDONESIAO RS480 CRORES 19 TRANCHES WERE FROM NICE TISSUE FACTORY WLL, MANUFACTURERS AND SUPPLIERS OF PAPER AND CATERING PRODUCTS A/C NO: QA60CBQA000000004680158105051 OF THE COMMERCIAL BANK PSQC, QATARO RS366 CRORES 4 TRANCHES WERE FROM GOLDEN TISSUE FOR TRADING A/C NO: QA67MAFR000000000004104239001 OF MASRAF AL RAYAN, QATARO RS234 CRORES 7 TRANCHES WERE FROM NICE TECH SERVICES TRADING CO, QATAR A/C NO: QA59CBQA000000004530577820051 OF THE COMMERCIAL BANK PSQC, QATARO RS214 CRORES 2 TRANCHES WERE FROM GULF MANUFACTURING CO LLC, INTO THE BUSINESS OF QUALITY PAPER PRODUCTS A/C NO: AE220380000012000294697 OF NATIONAL BANK OF FUJAIRAH, UNITED ARAB EMIRATES, ETC DEBITS WERE BY WAY OF:O IMPORT BILL PAYMENTS AGGREGATING TO RS3521 CRORES 71 TRANCHESO TRANSFERS OF RS615 CRORES 7 TRANCHES TO THE CURRENT ACCOUNT HELD WITH THE BANKO OUTWARD REMITTANCES AGGREGATING TO RS6120 LAKHS 5 TRANCHES WERE TOWARDS FREIGHT CHARGES PAID TO WORLDWIDE LOGISTICS CO LTD, CHINA, ETC3 TRADE FACILITIES EXPORT BILLS: 124 EXPORT BILLS AGGREGATING TO USD 885 MILLION RS6489 CRORES APPROX WERE BOOKED AGAINST THE COUNTERPARTIES, VIZ; NICE TISSUE FACTORY WLL, AL BAYADER MODERN IND COLLC, GOLDEN TISSUE FOR TRADING, AL AHLIA CONVERTING INDUSTRIES, FOUR STARS TRADING, GULF MANUFACTURING CO LLC, DURING THE REVIEW PERIOD IMPORT BILL: 123 IMPORT BILLS UNDER OPEN ACCOUNT PAYMENT, ADVANCE OPEN ACCOUNT PAYMENT, IMPORT COLLECTION BILL, IMPORT UNDER LC BILL AGGREGATING TO USD 860 MILLION RS6305 CRORES WERE PROCESSED THE UNDERLYING GOODS WERE PAPER CUPS BOTTOM, PAPER CUP STOCK, PAPER IN REELS, KITCHEN TOWEL TISSUE, ETC WHICH WERE SHIPPED FROM CHINA, HONG KONG, INDONESIA, BANGLADESH AND SHIPPED TO COUNTRIES LIKE OMAN, QATAR, UAE, UGANDA, KENYA, MEXICO, ETC AS PER THE IDPMS RECORDS AS ON 6 JULY 2021 AND EDPMS AS ON 13 JULY 2021, NO BILLS ARE SHOWN UNDER HSBC AD CODE AS PENDING BY THE CUSTOMER FOR SUBMISSION AND PAYMENT FOLLOWING ARE THE OBSERVATIONS NOTED AS PER THE AVAILABLE INFORMATION: CLIENT IS INTO MERCHANT TRADE BUSINESS OF PAPER PRODUCTS WHERE THE GOODS ARE IMPORTED FROM CHINA, HONG KONG, INDONESIA, BANGLADESH AND SHIPPED TO COUNTRIES LIKE OMAN, QATAR, UAE, UGANDA, KENYA, MEXICO, ETC NO SHIPPING BILLS ARE AVAILABLE FOR MERCHANT TRADE TRANSACTIONS AS PER RBI REGULATIONS, MERCHANT TRADE TRANSACTIONS SHOULD BE ROUTED FROM THE SAME AD BANK HOWEVER, FOR SOME TRANSACTIONS, FUNDS WERE RECEIVED FROM THE ACCOUNT HELD WITH OTHER BANKS IT IS ALSO NOTED THAT THE CUSTOMER HAS DEFAULTED IN FOLLOWING THE RBI GUIDELINE, WHICH STATES EXPORT FUNDS RECEIVED BEFORE MAKING IMPORT PAYMENT IN MERCHANT TRADE TRANSACTIONS, SHOULD BE PUT UNDER HOLD, TO ENABLE IMPORT PAYMENT AGAINST THESE PROCEEDS THERE ARE TIMES WHEN ADVANCE PAYMENT RECEIVED AGAINST SOME EXPORT ORDER, BUT ACTUAL SHIPMENT WAS CANCELLED OR REDUCED DIFFERENTIAL WAS ADJUSTED AGAINST OTHER ORDERS CHANGE IN ORDERS VALUE WERE NOTED OFTEN, FOR MAKING THE BALANCE PAYMENT OF AN ORDER IN VIEW OF THE AFORESAID OBSERVATIONS, AN STR IS BEING FILEDNARSINGH DASS AND COMPANY PRIVATE LIMITED PAN: AAACN0601K; GST: 07AAACN0601K1ZV CIN: U74899DL1990PTC039721; HOLDS THE FOLLOWING ACCOUNTS WITH THE BARAKHAMBA ROAD BRANCH, NEW DELHI:1 CURRENT ACCOUNT 166-410530-001, OPENED ON 260920182 FOREIGN CURRENCY USD ACCOUNT 166-410530-511, OPENED ON 260920183 TERM DEPOSIT FOR RS2034 LAKHS4 IT HAS ALSO AVAILED TRADE FACILITIES BOTH EXPORT & IMPORTTHE COMPANY WAS INCORPORATED ON 02041990 IT IS INTO WHOLESALE TRADING OF PAPER PRODUCTS AND BOARD GRADES THE RECORDED TURNOVER IS IN RANGE OF RS100 -150 CRORES THE REGISTERED ADDRESS IS E - 3/16, BALRAM HOUSE, DARYAGANJ, NEW DELHI 110002 AND PHONE IS 011 23266743 OMR SHIVANAND GUPTA, MR KAPIL GUPTA, MR PRADEEP GUPTA, MR ADITYA GUPTA, MR ASHOK KUMAR GUPTA, MR RAJENDRA PRASAD GUPTA, MR SATYA PAL GUPTA, MR RAM AVTAR GUPTA, AND MR ABHINAV GUPTA ARE THE DIRECTORS AND KEY CONTROLLERS OF THE COMPANY MR RAM AVTAR GUPTA, MR NIRMAL GUPTA, MS GITA GUPTA, AND MS SUMITRA GUPTA ARE THE BENEFICIAL OWNERS OF THE COMPANY ALERTS WERE RAISED BY THE STAFF MEMBERS, AS IT WAS OBSERVED THAT THE CUSTOMER DOES NOT FOLLOW THE BASIC RBI GUIDELINES WHILE ROUTING MERCHANT TRADE TRANSACTIONS ACCORDINGLY, TRANSACTIONS IN THE ACCOUNTS WERE REVIEWED FOR THE PERIOD 01082020 TO 31072021:1 CURRENT ACCOUNT 166-410530-001 CREDITS WERE BY WAY OF:O FUND TRANSFERS AGGREGATING TO RS615 CRORES 7 TRANCHES FROM THE FOREIGN CURRENCY ACCOUNT HELD WITH THE BANKO NEFT/RTGS AGGREGATING RS351 CRORES 13 TRANCHES WERE FROM THE CUSTOMERS OWN ACCOUNT HELD WITH HDFC BANK A/C NO: 03142790000333 O INWARD REMITTANCES OF RS6758 LAKHS 5 TRANCHES FROM FOUR STARS TRADING, DOHA, QATAR, WHO IS INTO THE BUSINESS OF PRODUCING WIDE RANGE OF HIGH QUALITY TISSUE PAPER PRODUCTS THROUGH ICICI BANK A/C NO: 0035SLCBPEFT, DEBITS WERE BY WAY OF:O IMPORT BILL PAYMENTS AGGREGATING TO RS888 CRORES 21 TRANCHESO NEFT/RTGS AGGREGATING TO RS526 CRORES 18 TRANCHES, OF WHICH:? RS310 CRORES 9 TRANCHES WERE TOWARDS CUSTOMERS OWN ACCOUNT HELD WITH HDFC BANK A/C NO: 03142790000333? RS213 CRORES 6 TRANCHES WERE TOWARDS VENDOR PAYMENTS TO CENTURY PULP AND PAPER A/C NO: /31962422247 OF STATE BANK OF INDIA, ETC O OUTWARD REMITTANCES OF RS3275 LAKHS 4 TRANCHES TOWARDS FREIGHT CHARGES, OF WHICH:? RS2979 LAKHS 3 TRANCHES WERE TOWARDS WORLDWIDE LOGISTICS CO LTD, CHINA A/C NO: 09421014040013387 OF AGRICULTURAL BANK OF CHINA LTD, CHINA ? RS296 LAKHS WERE TOWARDS RELIANCE FREIGHT SYSTEMS LLC, DUBAI, UAE A/C NO: AE390260001022309351101 OF EMIRATES NBD BANK PJSC, UNITED ARAB EMIRATES",{"entities":[(219, 252,"FOREIGN ORG"),(440, 468,"FOREIGN ORG"),(663, 693,"FOREIGN ORG"),(1145, 1174,"FOREIGN ORG"), (1290, 1315,"FOREIGN ORG"),(1396, 1421,"FOREIGN ORG"),(1722, 1738,"FOREIGN ORG"),(1897, 1920,"FOREIGN ORG"),(1922, 1948,"FOREIGN ORG"),(1978, 2008,"FOREIGN ORG"),(2030, 2055,"FOREIGN ORG") ,(6026 ,6051,"FOREIGN ORG") ,(6088, 6118,"FOREIGN ORG"), (6153, 6176,"FOREIGN ORG")]}),  
            ("TRANSACTION PATTERN SHOWS CREDITS MAINLY BY INWARD REMITTANCE AND DEBITS BY RTGS, TRANSFER FOR EXAMPLE, ON 15-07-2021, RS 148CRORES APPROX GOT CREDITED THROUGH INWARD REMITTANCE FROM SAPHIRE FOODSTUFF TRADING LLC IN THE CURRENCY OF USD FROM THE COUNTRY UAE FOR WHICH THE PURPOSE MENTIONED WAS AGAINST PROFORMA NF 0010 PRASHANT V IHAR NEW DELHI AND ON THE SAME DAY, RS 157CRORES APPROX GOT DEBITED IN PARTS THROUGH RTGS IN FAVOR OF PIYUSH SUGAR AND POWER PVT LTD ACCNO: 072011100000063, IFSC CODE: DNSB0000072, THE MALEGAON SAHAKARI SAKHAR ACCNO: 135010200004718, IFSC CODE: UTIB0000135, SARWESHWAR FOODS ACCNO: 0230020100000154, IFSC CODE: JAKA0ESTATE, ETC ALSO ON 08-07-2021, RS 167CRORES APPROX GOT CREDITED THROUGH INWARD REMITTANCE FROM ZEMEN GENERAL TRADING IN THE CURRENCY OF USD FROM THE COUNTRY DJ FOR WHICH THE PURPOSE MENTIONED WAS PAYMENT AGAINST PROFORMA INV N F/28/2021-22 DTD JULY/01/2021 FOR PURCHASE OF SUGAR AND RICE AND ON THE SAME DAY, RS 206CRORES APPROX GOT DEBITED IN PARTS THROUGH RTGS IN FAVOR OF VICHI AGRO PRODUCTS PVT LTD ACCNO: 16852320000070, IFSC CODE: HDFC0001685, DISHA FOODS ACCNO: 8747002100000580, IFSC CODE: PUNB0874700, BISHNOI AGRO INDIA PVT LTD ACCNO: 0971008700100526, IFSC CODE: PUNB0097100, ETC, TRANSFER TOWARDS JAI HANUMAN TRANSPORT 653305050782, SHRIRAM TRANSPORT 352305500114, GEOCHEM LABORATORIES PVT LTD 039305008622 SIMILAR TRANSACTION PATTERN WAS OBSERVED HUGE TURNOVER OF MORE THAN RS1000 CRORES WAS OBSERVED IN THE ACCOUNT CUSTOMERS IS DOING SUCH HUGE TURNOVER HOWEVER NO INFORMATION REGARDING ITS PROFILE AVAILABLE ON ANY PUBLIC DOMAIN NO OWN WEBSITES BASIS WHICH MARKING THIS ACCOUNT AS SUSPICIOUS IN VIEW OF HIGH VALUE TRANSACTIONS AND HUGE TURNOVER, STR IS BEING FILEDNAVYAA FOODS, PARTNERSHIP FIRM, HAVING ADDRESS:D 2 VINOBA KUNJ,ROHINI,SECTOR 9, NEW DELHI, DELHI, INDIA, 110085, CONTACT NUMBER: +919910339884 AND EMAIL ID: NAVYAAFOODS@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 082705001245 WITH ICICI BANK LTD, D16, NEAR PVR CINEMA, PRASHANT VIHAR, NEW DELHI, DELHI, 110085, SINCE 11112019 DATE OF INCORPORATION IS 08042019 PAN NUMBER OF THE ACCOUNT IS AAQFN3993H TRANSACTION LINKED ACCOUNT IS 073105000100 AUTHORIZED SIGNATORIES AND BENEFICIAL OWNERS OF THE REPORTED ACCOUNT ARE SUBHASH CHAND AUTHORISED SIGNATORY- AHMPC7934L, GEETIKA JINDAL WITH PAN NO AHSPM6483A, RICHA JINDAL WITH PAN NO AWPPD3610C THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN OF THE FIRM, PARTNERSHIP DEED KYC UPDATION IS NOT DUE AS PER BANK RECORDS, CUSTOMER PROFILE IS BUSINESS TRADING DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 5CR TO 25CR ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS AND HUGE TURNOVER BALANCE IN THE ACCOUNT AT 22072021 WAS RS464 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 22072021 SHOWS TOTAL CREDIT RS 22,75802 LAKHS OF WHICH TRANSFERS RS 5000 LAKHS MAJORLY FROM BHARDWAJ OVERSEAS PRIVATE LIMITED, CLEARING RS 1435 LAKHS MAJORLY FROM BHARDWAJ OVERSEAS PVT L,SWARUPA TRADERS, NEFT RS 2354 LAKHS MAJORLY FROM CBEC CUSTOM,CBEC ICEGATE,BULLRING HOTELS PRIVATE LIMI, RTGS RS 2,41435 LAKHS MAJORLY FROM ANCHIT CONSULTING GROU,NAVYAA FOODS,DISHA FOODS, REMITTANCE RS 20,24472 LAKHS MAJORLY FROM LUWAM GENERAL TRADING L L C P,HAJI NOUR INTERNATIONAL FZC,SAPHIRE FOODSTUFF TRADING LLC, AND OTHERS RS 1079 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 22,83757 LAKHS OF WHICH RTGS RS 21,43353 LAKHS, REMITTANCE RS 623 LAKHS MAJORLY TO AKTIF YATIRIM BANKASI AS, INTERNET FUND TRANSFER RS 1,27127 LAKHS, MOBILE MONEY TRANSFERS RS 1605 LAKHS IN 30 INSTANCES , , AND OTHERS RS 11049 LAKHS FY 2020-21 FROM 01042020 TO 31032021 : TOTAL CREDIT SIDE RS 47,83750 LAKHS OF WHICH TRANSFERS RS 10207 LAKHS MAJORLY FROM UTTARANCHAL COOPERATIVE SUGAR FACTORIES FEDERATION LTD,NAVYAA FOODS, CLEARING RS 007 LAKHS MAJORLY FROM B MANGATRAM AND CO, NEFT RS 52479 LAKHS MAJORLY FROM JINDAL FOODS,NAVYAA FOODS,RUSHABH SEALINK PVT, RTGS RS 12,70174 LAKHS MAJORLY FROM NAVYAA FOODS,ANCHIT CONSULTING GROU,JINDAL FOODS, REMITTANCE RS 33,86297 LAKHS MAJORLY FROM LUWAM GENERAL TRADING L L C P,SAPHIRE FOODSTUFF TRADING LLC,OMINCO GROUP LTD OFFICE NO 3, INTERNET FUND TRANSFERS RS 62999 LAKHS, MOBILE MONEY TRANSFERS RS 1515 LAKHS IN 12 INSTANCES , , AND OTHERS RS 073 LAKHS TOTAL DEBIT SIDE RS 47,75456 LAKHS OF WHICH CASH WAS RS 200 LAKHS BY BRANCH, TRANSFERS RS 100 LAKH MAJORLY TO NAVYAA FOODS, CLEARING RS 11617 LAKHS, NEFT RS 9699 LAKHS MAJORLY TO JINDAL ARYA IMPEX PV, RTGS RS 40,62392 LAKHS MAJORLY TO BISHNOI AGRO INDIA PVT ETC , REMITTANCE RS 1215 LAKHS, INTERNET FUND TRANSFER RS 6,61854 LAKHS, MOBILE MONEY TRANSFERS RS 11102 LAKHS IN 185 INSTANCES , , AND OTHERS RS 17276 LAKHS FY 2019-20 FROM 11112019 TO 31032020 : TOTAL CREDIT SIDE RS 1,34907 LAKHS OF WHICH REMITTANCE RS 89347 LAKHS TOTAL DEBIT SIDE RS 1,34782 LAKHS OF WHICH REMITTANCE RS 021 LAKHS",{"entities":[(183, 212,"FOREIGN ORG") ,(741, 762,"FOREIGN ORG")     , (3355, 3384,"FOREIGN ORG"),(3413, 3442,"FOREIGN ORG")    ,(4229, 4258,"FOREIGN ORG"), (4199, 4228,"FOREIGN ORG")   , (4259, 4275,"FOREIGN ORG") ]}),
            ("ALSO ON 13-05-2021, RS 450 CRORES WERE CREDITED IN 2 PARTS THROUGH RTGS FROM E PAO GST REFUNDS ACCNO: 31860389541, IFSC CODE: SBIN0000TBU AND ON 14-05-2021, RS 452CRORES WERE DEBITED THROUGH RTGS TO ESSEL MINING AND INDUSTRIES LIMITED ACCNO: 00140110000244, IFSC CODE: HDFC0000014 PATTERN IS SIMILAR IN THE ACCOUNT LARGE OUTWARD REMITTANCE WERE ALSO SEEN IN THE ACCOUNT IN VIEW OF LARGE VALUE E PAO GST REFUND TRANSACTIONS IN THE ACCOUNT WITH VERY HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDESSEL MINING INDUSTRIES LTD, COMPANY, HAVING ADDRESS : INDUSTRY HOUSE, 10 CAMAC STREET, 18TH FLR, KOLKATA, WEST BENGAL, INDIA, 700017, CONTACT NUMBER: +03327828330 AND EMAIL ID: NAVNEETSHARDA@ADITYABIRLACOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000651000108 WITH ICICI BANK LTD, 22, SIR RN MUKHERJEE ROAD, KOLKATA, WEST BENGAL, 700001, SINCE 10-06-2003 DATE OF BIRTH IS 01-04-1950 PAN OF THE ACCOUNT IS AAACE6607L IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0288027183 GSTIN AS PER BANK RECORDS IS 19AAACE6607L1ZF OTHER RELATIONSHIP OF CUSTOMER ARE 42 CURRENT ACCOUNT, 5 FASTAG, 20 FIXED DEPOSIT, 1 LOANRAPG TRANSACTION LINKED ACCOUNTS ARE 603090012277, 000610071883 BENEFICIAL OWNERS / DIRECTORS/ AUTHORISED SIGNATORIES OF THE REPORTED ACCOUNT ARE MANGALAM BIRLA KUMAR WITH PAN AEFPB5926H, RAJASHREE BIRLA WITH PAN AAFPB1583L THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN OF COMPANY, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION DATE OF LAST KYC UPDATION 31102019KYC UPDATION IS NOT DUE CUSTOME IS MANUFACTURERS OF NOBLE FERRO-ALLOYS, PRODUCING FERRO-MOLYBDENUM AND FERRO-VANADIUM AN ALERT WAS TRIGGERED FOR LARGE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 31-05-2021 WAS RS 560 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01-04-2021 TILL 31-05-2021 SHOWS TOTAL CREDIT RS 93,54954 LAKHS OF WHICH NEFT RS 15149 LAKHS MAJORLY FROM ESSEL MINING AND INDUSTRIES, E PAO GST REFUNDS, RTGS RS 86,10702 LAKHS MAJORLY FROM E PAO GST REFUNDS, BIRLA SUN LIFE PAYLI, REMITTANCE RS 7,29100 LAKHS MAJORLY FROM BRIGHT RUBY RESOURCES PTE LTD, ELECTROSTEEL CASTINGS LTD TOTAL DEBITS DURING THE SAME PERIOD WAS RS 93,55929 LAKHS OF WHICH RTGS RS 90,44450 LAKHS MAJORLY TO IGH HOLDINGS, REMITTANCE RS 2,38529 LAKHS MAJORLY TO GUARDIAN NAVIGATION WORLDWIDE, GS GLOBAL CORPORATION AND OTHERS RS 72359 LAKHS FY 2020-21 FROM 01-04-2020 TO 31-03-2021 : TOTAL CREDIT SIDE RS 1,10,25496 LAKHS OF WHICH TRANSFERS RS 51,81000 LAKHS MAJORLY FROM ESSEL MINING INDUSTRIES LTD, NEFT RS 5,11803 LAKHS MAJORLY FROM ESSEL MINING AND INDUSTRIES, E PAO GST REFUNDS, RTGS RS 22,73082 LAKHS MAJORLY FROM ESSEL MINING INDUSTRIES LTD, IGH HOLDINGS PRIVATE LTD, REMITTANCE RS 26,21651 LAKHS MAJORLY FROM BRIGHT RUBY RESOURCES PTE LTD, CARAVEL METALLURGICAL LIMITED, ELECTROSTEEL STEELS LIMITED AND OTHERS RS 4,35004 LAKHS TOTAL DEBIT SIDE RS 1,10,28307 LAKHS OF WHICH TRANSFERS RS 7,76833 LAKHS MAJORLY TO ESSEL MINING INDUSTRIES LTD, ADITYA BIRLA SUN LIFE LIQUID FUND COLLECTION A/C, RTGS RS 95,04369 LAKHS MAJORLY TO ESSEL MINING AND INDUSTRIES LTD, REMITTANCE RS 6,56639 LAKHS MAJORLY TO GUARDIAN NAVIGATION WORLDWIDE, ULTRABULK SHIPHOLDING SINGAPORE PTE LTD AND OTHERS RS 79633 LAKHS FY 2019-20 FROM 01-04-2019 TO 31-03-2020 : TOTAL CREDIT SIDE RS 2,56,57927 LAKHS OF WHICH REMITTANCE RS 50,35555 LAKHS TOTAL DEBIT SIDE RS 2,56,53605 LAKHS OF WHICH REMITTANCE RS 7,30111 LAKHS FY 2018-19 FROM 01-04-2018 TO 31-03-2019 : TOTAL CREDIT SIDE RS 40,99505 LAKHS OF WHICH REMITTANCE RS 19,02430 LAKHS TOTAL DEBIT SIDE RS 39,70562 LAKHS OF WHICH REMITTANCE RS 79515 LAKHS FY 2017-18 FROM 01-04-2017 TO 31-03-2018 : TOTAL CREDIT SIDE RS 39,54129 LAKHS OF WHICH REMITTANCE RS 10,00108 LAKHS TOTAL DEBIT SIDE RS 41,36833 LAKHS OF WHICH REMITTANCE RS 73904 LAKHS TRANSACTION PATTERN SHOWS HIGH VALUE TRANSACTIONS ON THE SAME DAY WITH SAME AMOUNT WERE SEEN IN THE ACCOUNT FOR EXAMPLE, ON 25-05-2021, RS 221 CRORES WERE CREDITED THROUGH RTGS FROM E PAO GST REFUNDS ACCNO: 31860389541, IFSC CODE: SBIN0000TBU AND ON 26-05-2021, RS 222 CRORES WERE DEBITED THROUGH RTGS TO ESSEL MINING AND INDUSTRIES LIMITED ACCNO: 00140110000244, IFSC CODE: HDFC0000014",{"entities":[(2215, 2244,"FOREIGN ORG"),  (2426,2455,"FOREIGN ORG"),(2457, 2478, "FOREIGN ORG")   , (2881, 2910, "FOREIGN ORG"),(2912, 2941, "FOREIGN ORG")  , (3268, 3297, "FOREIGN ORG"),(3299, 3338, "FOREIGN ORG")]}),
            ("ON 09-07-2021, RS 107CRORES GOT CREDITED IN PARTS THROUGH RTGS FROM CLASSIC PETRO AND ENERGY PRIVATE ACCNO: 001563700003920, IFSC CODE: YESB0000001, BANK: YES BANK AND ON 12-07-2021, RS 105CRORES GOT DEBITED IN PARTS THROUGH OUTWARD REMITTANCE IN FAVOR OF MOTOTIVE GENERAL TRADING LLC IN THE CURRENCY OF AED TO THE COUNTRY UAE FOR WHICH THE PURPOSE MENTIONED WAS PAYMENT TOWARDS IMPORT OTHER THAN NEPAL BHUTAN, RTGS IN FAVOR OF CLASSIC BITUMEN SISTER CONCERN OF THE CUSTOMER, ACCNO: 50200027529522, IFSC CODE: HDFC0002658, BANK: HDFC BANK SIMILAR TRANSACTIONS WERE OBSERVED IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2020-21 WAS RS 7,72354 LAKHS, WHICH SEEMS TO BE VERY HIGH NO BUSINESS RELATED EXPENDITURE SEEN IN THE ACCOUNT IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDCLASSIC PETRO ENERGY PRIVATE LIMITED, A PRIVATE HAVING ADDRESS: SCO55, FIRST, SECTOR 30C, CHANDIGARH, INDIA, 160030, CONTACT NUMBER: +919888664778 AND EMAIL ID: CPEPLCHD@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 632205010412 WITH ICICI BANK LTD, SCO 485486, SECTOR 35C, CHANDIGARH, 160022, SINCE 09112020 DATE OF INCORPORATION IS 14032014 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT, LOAN PAN NUMBER OF THE ACCOUNT IS AAFCC7079K IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 2215004851 GSTIN AS PER BANK RECORDS IS 04AAFCC7079K1ZD PAN LINKED ACCOUNTS ARE 632205009408, FLACHD00043284232 TRANSACTION LINKED ACCOUNTS ARE 246905500252, 643805053140 AUTHORIZED SIGNATORIES/ DIRECTORS/ BENEFICIAL OWNERS OF THE REPORTED ACCOUNT ARE BRICHI ARORA PAN: BYOPK8861Q AND ROHIT RAJ ARORA PAN: AHTPA8475D THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN: AAFCC7079K, COI: U23200CH2014PTC035021, MOA AOA AND PAN OF AUTHORIZED SIGNATORIES/ DIRECTORS/ BENEFICIAL OWNERS KYC UPDATION IS NOT DUE CUSTOMER IS INVOLVED INTO THE BUSINESS OF MANUFACTURING REFINED PETROLEUM PRODUCTS THE COMPANY IS ALSO INVOLVED IN THE BUSINESS OF SUPPLYING AND TRADING OF RUBBER PROCESSING OIL, BITUMEN, WHITE SPIRIT AND LIGHT DIESEL OIL DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO LESS THAN RS 5 CRORES ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31052021 WAS RS174 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31052021 SHOWS TOTAL CREDIT RS 3,19120 LAKHS OF WHICH RTGS RS 3,15100 LAKHS MAJORLY FROM CLASSIC BITUMEN, CLASSIC PETRO ENERGY PRIVATE LIMITED TOTAL DEBITS DURING THE SAME PERIOD WAS RS 3,36586 LAKHS OF WHICH RTGS RS 18414 LAKHS TO MARUTI TRADING CO, ARMITA INDIA SHIPPING PVT LTD, REMITTANCE RS 3,11298 LAKHS MAJORLY TO RENEWABLE ENERGY FZE, MOTOTIVE GENERAL TRADING LLC, MAN TRADING LLC, INTERNET FUND TRANSFER RS 5010 LAKHS TO RADIANT MARITIME INDIA PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 1224 LAKHS IN 16 INSTANCES FY 2020-21 FROM 09112020 TO 31032021: TOTAL CREDIT SIDE RS 7,72354 LAKHS OF WHICH CLEARING RS 500 LAKHS MAJORLY FROM CLASSIC PETRO ENERGY PRIVATE LIMITED, NEFT RS 54880 LAKHS MAJORLY FROM CLASSIC PETRO ENERGY PRIVATE LIMITED, CLASSIC BITUMEN, RTGS RS 7,03154 LAKHS MAJORLY FROM CLASSIC BITUMEN, CLASSIC PETRO ENERGY PRIVATE LIMITED, INTERNET FUND TRANSFERS RS 10000 LAKHS FROM GOYAL AND COMPANY 246905500252, MOBILE MONEY TRANSFERS RS 3820 LAKHS IN 25 INSTANCES TOTAL DEBIT SIDE RS 7,54714 LAKHS OF RTGS RS 70855 LAKHS, REMITTANCE RS 6,69987 LAKHS MAJORLY TO MOTOTIVE GENERAL TRADING LLC, TASHBULAK TRADING LLC, RENEWABLE ENERGY FZE, INTERNET FUND TRANSFER RS 10155 LAKHS TO GIL SHIPPING SERVICES PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 3517 LAKHS IN 31 INSTANCES TRANSACTION PATTERN SHOWS HIGH VALUE TRANSACTIONS IN NEW ACCOUNT FOR EXAMPLE, ON 13-07-2021, RS 102CRORES APPROX GOT CREDITED IN PARTS THROUGH RTGS FROM CLASSIC PETRO AND ENERGY PRIVATE LIMITED ACCNO: 001563700003920, IFSC CODE: YESB0000001, BANK: YES BANK, CLASSIC BITUMEN ACCNO: 50200027529522, IFSC CODE: HDFC0000240, BANK: HDFC BANK AND ON THE SAME DAY, RS 101CRORES APPROX GOT DEBITED THROUGH OUTWARD REMITTANCE IN FAVOR OF MOTOTIVE GENERAL TRADING LLC IN THE CURRENCY OF AED TO THE COUNTRY UAE FOR WHICH THE PURPOSE MENTIONED WAS PAYMENT TOWARDS IMPORT OTHER THAN NEPAL BHUTAN",{"entities":[(256, 284,"FOREIGN ORG")       ,(2862,2890,"FOREIGN ORG"),(2840,2860,"FOREIGN ORG")  ,(3631,3652,"FOREIGN ORG"),(3654, 3674,"FOREIGN ORG") ]}),         
            ("CLOUDBOTIX TECHNOLOGIES PRIVATE LIMITED, COMPANY, HAVING ADDRESS: OFF-,ADITYA BANARASI HERITAGE,306,MALAD WEST,, CHINCHOLI BUNDER ROAD, MUMBAI, MAHARASHTRA, INDIA, 400064, CONTACT NUMBER: +917873866570 AND EMAIL ID: CLOUDBOTIXTECH@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 122505001216 WITH ICICI BANK LTD, SHOP NO7, BHARAT APT, VEERA DESAI ROAD, ANDHERI W, MUMBAI MAHARASHTRA, 400701, SINCE 19062021 DATE OF INCORPORATION IS 25032021 PAN NUMBER OF THE ACCOUNT IS AAJCC3855FTRANSACTION LINKED ACCOUNT IS 699110024470 AUTHORISED SIGNATORY AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS GANESH WITH PAN ALAPG9304L THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN - AAJCC3855F OF COMPANY, BOARD RESOLUTION , MOACUSTOMER IS ENGAGED IN SOFTWARE PUBLISHING, CONSULTANCY AND SUPPLY SOFTWARE, OPERATING SYSTEMS SOFTWARE ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 25 CR TO LESS THAN 100CR ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31072021 WAS RS8293 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR:FY 2021-22 FROM 19062021 TILL 31072021 SHOWS TOTAL CREDIT RS 51472 LAKHS OF WHICH CLEARING RS 500 LAKHS MAJORLY FROM GANESH APPU, RTGS RS 49972 LAKHS MAJORLY FROM CLOUDBOTIX TECHNOLOGIES PRIVATE LIMITED, SKYLINE CLOUD TECHNOLOGIES, FIXED DEPOSIT RS 1000 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 43179 LAKHS OF WHICH TRANSFERS RS 1000 LAKHS MAJORLY TO CLOUDBOTIX TECHNOLOGIES PRIVATE LIMITED, REMITTANCE RS 42157 LAKHS MAJORLY TO INFINITY DIGITAL SOLUTIONS, INTERNET FUND TRANSFER RS 020 LAKHS AND OTHERS RS 002 LAKHS AS PER TRANSACTIONS PATTERN, IT IS OBSERVED THAT THERE ARE HIGH VALUE NON-CASH TRANSACTIONS BEING CARRIED OUT IN THE NEWLY OPENED ACCOUNT FOR EXAMPLE- ON 28-07-2021 RS 8140 LAKHS WERE CREDITED IN MULTIPLE TRANCHES THROUGH RTGS FROM SKYLINE CLOUD TECHNOLOGIES ACCOUNT NUMBER : 020163400003375, IFSC CODE : YESB0000001,BANK : YES BANK AND ON THE SAME DAY RS 9249 LAKHS WERE DEPLOYED THROUGH REMITTANCE TO INFINITY DIGITAL SOLUTIONS ACCOUNT NUMBER : 059802000001145, CURRENCY : USD, AMOUNT : 124,22496, COUNTRY : HONG KONG, PURPOSE : TOWARDS SOFTWARE CONSULTANCY AND IMPLEMENTATION ALSO ON 22-07-2021 RS 8401 LAKHS WERE CREDITED INTO THE ACCOUNT IN 2 TRANCHES THROUGH RTGS FROM SKYLINE CLOUD TECHNOLOGIES ACCOUNT NUMBER : 020163400003375, IFSC CODE : YESB0000001,BANK : YES BANK AND ON THE SAME DAY RS 8577 LAKHS WERE DEPLOYED THROUGH REMITTANCE TO INFINITY DIGITAL SOLUTIONS ACCOUNT NUMBER : 059802000001145, CURRENCY : USD, AMOUNT : 114,98544, COUNTRY : HONG KONG, PURPOSE : TOWARDS SOFTWARE IMPLEMENTATION LARGE VALUE OF FUNDS ARE BEING ROUTED INTO THE ACCOUNT AND THE FUNDS ARE DEPLOYED ON THE SAME DAY THROUGH REMITTANCE, IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT, STR IS BEING FILED",{"entities":[(2143, 2169,"FOREIGN ORG"), (2586, 2612,"FOREIGN ORG")  ]}),
            ("FY 2019-20 FROM 01042019 TO 31032020: TOTAL CREDIT SIDE RS 14,30072 LAKHS OF WHICH CASH WAS RS 020 LAKHS, REMITTANCE RS 9852 LAKHS TOTAL DEBIT SIDE RS 14,28672 LAKHS OF WHICH CASH WAS RS 454 LAKHS, REMITTANCE RS 1,84336 LAKHS FY 2018-19 FROM 19062018 TO 31032019: TOTAL CREDIT SIDE RS 2,61964 LAKHS OF WHICH CASH WAS NIL, TOTAL DEBIT SIDE RS 2,61700 LAKHS OF WHICH CASH WAS RS 060 LAKHS, REMITTANCE RS 1,14313 LAKHS THERE ARE 38 BILL OF ENTRY PENDING FOR SUBMISSION AMOUNTING USD 4,526,877 WHICH ARE PENDING FOR SUBMISSION AND OVERDUE FOR LESS THAN EQUAL TO 944 DAYS THERE ARE 32 IMPORT MADE-PAYMENT PENDING AMOUNTING USD 1,027,525 WHICH ARE PENDING PAYMENT FOR LESS THAN EQUAL TO 1068 DAYS AS PER PATTERN HIGH VALUE TRANSACTIONS SEEN IN THE ACCOUNT FOR EXAMPLE, ON 29-07-2021, RS 127CRORES GOT CREDITED IN PARTS THROUGH RTGS FROM LEELA TUBES PRIVATE LIMITED ACCNO: 008284600000441, IFSC CODE: YESB0000001, K M EXPORTS ACCNO: 03900200001626, IFSC CODE: BARB0CPTANK, JINRATNA METAL ACCNO: 139007102002936, IFSC CODE: NVNM0000007, ETC, NEFT FROM METAL SOURCE ACCNO: 910020043649967, IFSC CODE: UTIB0000563, STANCOR ALLOYS INC ACCNO: 018984600000200, IFSC CODE: YESB0000001, BHARAT STEELS ACCNO: 917020011115538, IFSC CODE: UTIB0002622, ETC, AND ON THE SAME DAY, RS 150CRORES GOT DEBITED IN PARTS THROUGH RTGS IN FAVOR OF REFLEX TUBES AND INDUSTRIES ACCNO: 59209930502130, IFSC CODE: HDFC0000626, STEEL SPACE PVT LTD ACCNO: 35935114031, IFSC CODE: SBIN0031282, MONTEX TUBES LLP ACCNO: 018961900001478, IFSC CODE: YESB0000189, OUTWARD REMITTANCE IN FAVOR OF FOSHAN XUANZHENG TRADING CO LTD IN THE CURRENCY OF USD TO THE COUNTRY CHINA FOR WHICH THE PURPOSE MENTIONED WAS PAYMT TOWARDS IMP SETTLMT OTR THAN NEPL BHUTAN AS PER EDD RECEIVED COMPLAINT STATING THERE ARE 28 BOE PENDING FOR SUBMISSION, AND THERE ARE NO FILING AVAILABLE ON PUBLIC DOMAIN, AND ONE OF THE COMPANY DESIGNATED PARTNER RAHUL RAJMAL GANDHI IS DISQUALIFIED BY ROC 164 2, WHICH SEEMS SUSPICIOUS AND RAISED THE GROUND OF SUSPICION TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 26,62987 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL INCOME OF RS 100CRS IN VIEW OF HIGH VALUE TRANSACTIONS IN NEW ACCOUNT, HIGH TURNOVER AND NEGATIVE EDD FEEDBACK, STR IS BEING FILEDNICKEL IMPEX LLP, LIMITED LIABILITY PARTNERSHIP, HAVING ADDRESS: D-4, SHREEPATI CASTLE, 1ST FLR, KHETWADI 11TH LANE, KHETWADI BACK, ROAD GIRGAON, MUMBAI, MAHARASHTRA, INDIA, 400004, CONTACT NUMBER: +919833064164 AND EMAIL ID: NICKELIMPEXLLP@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 034805005356 WITH ICICI BANK LTD, SHOP NO 02, RAWAL BUILDING 01, DR BHADKAMKAR MARG, OPERA HOUSE, MUMBAI, MAHARASHTRA, 400004, SINCE 19062018 DATE OF INCORPORATION IS 12072017 PAN NUMBER OF THE ACCOUNT IS AAOFN4344E TRANSACTION LINKED ACCOUNTS ARE 034851000024, 034805500776 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS HARSH JITENDRA SANGHVI WITH PAN NO HLTPS4810PRAHUL RAJMAL GANDHI WITH PAN NO AIXPG0918Q THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN - AAOFN4344E, LLPIN: AAJ-9799 DATE OF LAST KYC UPDATION 21082020 HENCE, KYC UPDATION IS NOT DUE CUSTOMER PROFILE IS OTHER RETAIL TRADE AS PER BANK RECORD DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 100CRS ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN NEW ACCOUNT AND NEGATIVE EDD FEEDBACK BALANCE IN THE ACCOUNT AT 30072021 WAS RS1375 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 30072021 SHOWS TOTAL CREDIT RS 12,81767 LAKHS OF WHICH TRANSFERS RS 36854 LAKHS MAJORLY FROM RATNAMANI METALS AND TUBES LTD, PREMDEEP METAL, SUPREME AND COMPANY PVT LTD, CLEARING RS 5675 LAKHS MAJORLY FROM ALFA FORGE INDUSTRIES, MEGHDOOT REFRIGERATION, STEEL SPACE PVT LTD, NEFT RS 1,09261 LAKHS MAJORLY FROM SKM INOX A DIVISION, AMAR WOOD INDUSTRIES PRIVATE LTD, NISHAN IMPEX, RTGS RS 9,49091 LAKHS MAJORLY FROM JNB STEEL INDUSTRIES PRIVATE LTD, NIRLON TUBE INDIA, METAL TRADING VENTURES, REMITTANCE RS 13191 LAKHS, INTERNET FUND TRANSFERS RS 1,43608 LAKHS MAJORLY FROM AMANAT STEELS PVT LTD, DHANERA IMPEX, AND OTHERS RS 20893 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 12,82666 LAKHS OF WHICH RTGS RS 9,02557 LAKHS, REMITTANCE RS 2,51274 LAKHS MAJORLY TO EMETAL COMPANY LIMITED, HYOSUNG TNC CORPORATION, LEO METALS LIMITED, INTERNET FUND TRANSFER RS 1,24164 LAKHS MAJORLY TO AMANAT STEELS PVT LTD, DHANERA IMPEX FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 26,62987 LAKHS OF WHICH TRANSFERS RS 62370 LAKHS MAJORLY FROM MUDIT IMPEX PVT LTD, CLASSIC STEEL CENTRE, RATNAMANI METALS AND TUBES LTD, CLEARING RS 24280 LAKHS MAJORLY FROM AKASH STEEL CRAFTS PVT LTD, ROLEX INDUSTRIAL CORPORATION, AKASH, NEFT RS 3,72251 LAKHS MAJORLY FROM NG INDUSTRIES, AARON INDUSTRIES LIMITED, SUN MARK STAINLESS P, RTGS RS 19,23396 LAKHS MAJORLY FROM K P ENTERPRISE, MARUDHAR STAINLESS, DHANERA IMPEX, REMITTANCE RS 5030 LAKHS, INTERNET FUND TRANSFERS RS 2,69275 LAKHS MAJORLY FROM AMANAT STEELS PVT LTD, DHANERA IMPEX, MOBILE MONEY TRANSFERS RS 6275 LAKHS IN 70 INSTANCES, UPI TRANSFERS RS 100 LAKH IN 9 INSTANCES, AND OTHERS RS 011 LAKHS TOTAL DEBIT SIDE RS 26,62379 LAKHS OF WHICH CASH WAS RS 929 LAKHS BY ATM, BRANCH, TRANSFERS RS 5500 LAKHS MAJORLY TO MUDIT IMPEX PVT LTD, AMANAT STEELS PVT LTD, HOME ZONE STAINLESS PRIVATE LIMITED, CLEARING RS 6712 LAKHS MAJORLY TO LIFE INSURANCE CORPORATIO, NAVDEEP STEEL PRIVATE LIMITED, AARAV STEEL INDUSTRIES, RTGS RS 19,25161 LAKHS MAJORLY TO SUNRISE STAINLESS PVT LTD, JOY REAP METAL INDIA PVT LTD, JOY REAP METAL, REMITTANCE RS 3,66099 LAKHS MAJORLY TO LEO METALS LIMITED, HYOSUNG TNC CORPORATION, SONHA SSP VIETNAM SOLE MEMBER, INTERNET FUND TRANSFER RS 3,47859 LAKHS MAJORLY TO AMANAT STEELS PVT LTD, DHANERA IMPEX, CARD TRANSACTIONS RS 854 LAKHS, MOBILE MONEY TRANSFERS RS 5295 LAKHS IN 161 INSTANCES, AND OTHERS RS 3969 LAKHS",{"entities":[( 1554, 1585, "FOREIGN ORG")  , (4389, 4406, "FOREIGN ORG") ,(4340, 4361,"FOREIGN ORG"), (4364, 4386 ,"FOREIGN ORG") ,(5698, 5720,"FOREIGN ORG"),(5678, 5695,"FOREIGN ORG")]}),
            ("CLEARING RS 2526 LAKHS MAJORLY TO P P SOFTTECH PVT LTD, RTGS RS 11,70854 LAKHS MAJORLY TO PJS OVERSEAS LTD, CARGO LINKS, REMITTANCE RS 9,60050 LAKHS MAJORLY TO KIARA PETROLEUM FZE IN AED CURRENCY TO UAE WHERE THE REASON/PURPOSE MENTIONED WAS PAYMT TOWARDS IMPORT SETTLEMENT OTHER THAN NEPAL AND BHUTAN, INTERNET FUND TRANSFER RS 4,07266 LAKHS MAJORLY TO NOVEL LINES LOGISTICS PVT LTD 122705000197, GULF PETROCHEM INDIA PVT LTD 054405005391, CARD TRANSACTIONS RS 2451 LAKHS, MOBILE MONEY TRANSFERS RS 6158 LAKHS IN 162 INSTANCES AND OTHERS RS 8500 LAKHS FY 2019-20 FROM 03022020 TO 31032020: TOTAL CREDIT SIDE RS 1,60441 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 1,60354 LAKHS OF WHICH CASH WAS RS 159 LAKHS THE CUSTOMER ENTITY HAS IMPORT/EXPORT BILLS INVOLVING IRAN PENDING FOR REGULARIZATION IN IDPMS/EDPMS SYSTEM WHICH ARE REGISTERED IN ICICI BANK AD CODE THERE IS TOTAL 12 BILLS PENDING UNDER BOTH EXPORT AND IMPORT CATEGORY FOR SAME COUNTERPARTY - PAYAM TEJARAT JASHNVREH CO FROM THE BOE DETAILS, IT IS OBSERVED THAT THE COMMODITY INVOLVED IN THE UNDERLYING TRANSACTIONS IS IMPORT OF BITUMEN GRADE VG40 AND EXPORT OF BASMATI RICE FROM/TO PAYAM TEJARAT JASHNVREH CO AND THE COUNTRY OF IMPORT/EXPORT MENTIONED IS IRAN PORT - BANDAR ABBASTHE CUSTOMER HAD APPROACHED OUR BANK FOR REGULARIZATION OF THE MENTIONED 12 BILLS TO WHICH BANK HAD RESPONDED WITH OUR INABILITY TO HANDLE IRAN TRANSACTIONS SINCE IRAN IS A COMPREHENSIVE SANCTIONED COUNTRY UNDER THE US OFAC SANCTIONS PROGRAM AND LISTED AS A COUNTRY UNDER FATFS CALL FOR ACTION ACCORDINGLY AS PER BANKS INTERNAL POLICY , BANK DOES NOT DEAL WITH IRAN LINKED TRANSACTIONS DIRECTLY OR INDIRECTLY WE UNDERSTAND THAT THE CUSTOMER IS ROUTING IRAN RELATED TRANSACTIONS THROUGH IDBI BANK, HOWEVER, THE CUSTOMER IS USING AD CODE OF ICICI BANK THE CUSTOMER HAS A HISTORY OF DEALING WITH UAE PARTIES FOR BITUMEN WHEREIN IRAN WAS NOT EVIDENT IN ANY PREVIOUS BOES SETTLED THROUGH OUR BANK AS PER SAMPLE CHECKS IN OUR READING, IRAN IS LARGEST EXPORTER OF BITUMEN WHICH IS SMUGGLED INTO UAE, THEN PACKAGED/LABELED THEREIN AND SOLD TO OTHER COUNTRIES TO EVADE TRADE SANCTIONS IMPOSED BY GLOBAL SANCTIONING AUTHORITIESCUSTOMER HAS CONFIRMED THAT THEY HAVE KNOWINGLY INVOICED THESE TRANSACTIONS IN INR AND DEALING WITH SAME IRANIAN PARTY FOR BOTH IMPORT AND EXPORT LEGS SO THAT CUSTOMERS INTERNAL BOOKS GETS SETTLED AND THESE TRANSACTIONS SEEMED SUSPICIOUS DUE TO THE WAY THEY ARE DENOMINATED IN INR TO AVOID TOUCHING ANY BANKING CHANNELS TO EVADE SANCTIONS CHECKINGFURTHER, IT WAS OBSERVED A TOTAL 10 BILLS OF IMPORT MADE BUT PAYMENT PENDING AGAINST SAME FOREIGN PARTY - PAYAM TEJARAT JASHNVREH CO, THE OUTSTANDING AMOUNT OF THESE PENDING BILLS IS INR 109 CRORES AND AGEING OF THESE BILLS ARE FROM 15 DAYS TO 237 DAYS TOTAL ANNUAL TURNOVER FOR THE FY 21-22 WAS RS 27,833 LAKHS WHICH IS ABNORMALLY HIGHER THAN THE DECLARED ANNUAL TURNOVER OF RS500 LAKHS ALSO IT IS OBSERVED THAT THE COMPANY HAS RECENTLY SHIFTED TO THE BUSINESS OF MANUFACTURING AND SUPPLYING OF BITUMEN FROM SOFTWARE INDUSTRY WHICH ARISES SUSPICIOUSTHE BANK HAS ALREADY FILED STR ON THE SAID ACCOUNT AS MAIN VIDE STR REPORT SERIAL NUMBER :311017 AND DATE OF STR FILING:31-07-2021 WITH FIU BATCH ID:2107308350IN VIEW OF SUSPICIOUS TRANSACTIONS WITH THE COUNTRYIRAN LISTED BY FATF, SUDDEN CHANGE IN THE BUSINESS TYPE AND ABNORMAL HIGH ANNUAL TURNOVER MAKES IT SUSPICIOUS, STR IS BEING REPORTEDPP SOFTTECH PRIVATE LIMITED, A PRIVATE LIMITED COMPANY, HAVING ADDRESS: T-1, PURI ANAND VILAS, GFD, SECTOR-81, FARIDABAD, HARYANA, INDIA, 121004, CONTACT NUMBER: +919899895811 AND EMAIL ID: PPBITUMEN@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 732305000042 WITH ICICI BANK LTD, SCO-45, OMAXE WORLD STREET, SECTOR-79, NEHAR PAR, FARIDABAD - 121002, SINCE 03022020 DATE OF INCORPORATION IS 29022016 PAN OF THE ENTITY IS AAICP5254C GSTIN AS PER BANK RECORDS IS 29AAICP5254C1ZD TRANSACTION LINKED ACCOUNTS ARE 277905500492, 732310000272 AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS/ DIRECTORS OF THE REPORTED ACCOUNT ARE PRASHANT GARG PAN-ACYPG4021D AND PULKIT JAIN PAN-AMGPJ6429E THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN - AAICP5254C, COI-U11100HR2016PTC058403, GST-29AAICP5254C1ZD, MOA AND AOA ON VERIFICATION OF GSTIN NUMBER IN GST WEBSITE, CUSTOMER DEALS IN GOODS AND SERVICES OF HSN 27132000PETROLEUM COKE, PETROLEUM BITUMEN AND OTHER RESIDUES OF PETROLEUM OILS OR OF OILS OBTAINED FROM BITUMINOUS MINERALS - PETROLEUM BITUMEN AS PER THE EDD, THE COMPANY WAS INCORPORATED TO CARRY ON THE BUSINESS OF TRADING AND DEVELOPMENT OF SOFTWARE, SOFTWARE DESIGN AND WEB DESIGN BUT DUE TO CORONA PANDEMIC COMPANY HAS SHIFTED TO DIFFERENT BUSINESS LINES SUCH AS PETROCHEMICAL, TRADING OF BITUMEN AND BASMATI RICE TRADING DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO RS 5 CRORES A MANUAL ALERT WAS RAISED FOR SUSPICIOUS TRANSACTIONS WITH THE COUNTRYIRAN LISTED BY FATF BALANCE IN THE ACCOUNT AT 13092021 WAS RS100946 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 13092021 SHOWS TOTAL CREDIT RS 27,83382 LAKHS OF WHICH CLEARING RS 200 LAKHS FROM AA ASPHALTS, NEFT RS 2,56913 LAKHS MAJORLY FROM JUPITER PETROCHEM, SHREE EZHUMALAI ASPHALTS, SHREE BALAJI ASSOCIATES, RTGS RS 20,81555 LAKHS MAJORLY FROM V R PETROCHEM, SHREE BALAJI ASSOCIATE, ABPETROCHEM, INTERNET FUND TRANSFERS RS 3,92256 LAKHS MAJORLY FROM SIDDHIVINAYAK CONSTRUCTION 277905500492, VRAJ MANGAL LIQUID MOVERS AND TRADERS PVT LTD AC NO- 001851000130, MOBILE MONEY TRANSFERS RS 1475 LAKHS IN 19 INSTANCES, FIXED DEPOSIT RS 32377 LAKHS AND OTHERS RS 7633 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 27,23253 LAKHS, TRANSFERS RS 43610 LAKHS MAJORLY TO PP SOFTTECH PRIVATE LIMITED, NATIONAL COLLATERAL MANAGEMENT SERLTD, PARMOD MITTAL, CLEARING RS 47 LAKHS MAJORLY TO PP SOFTTECH PRIVATE LIMITED, PURI CONSTRUCTION PVT LTD, RTGS RS 4,53219 LAKHS MAJORLY TO ARMITA INDIA SHIPPING PVT LTD, SAMUDRA MARINE SERVICES PRIVATE LIMITED, REMITTANCE RS 15,8979 LAKHS MAJORLY TO KIARA PETROLEUM FZE IN AED CURRENCY TO UAE WHERE THE REASON/PURPOSE MENTIONED WAS PAYMT TOWARDS IMPORT SETTLEMENT OTHER THAN NEPAL AND BHUTAN, INTERNET FUND TRANSFER RS 6,24215 LAKHS MAJORLY TO GULF PETROCHEM INDIA PVT LTD 054405005391, CARD TRANSACTIONS RS 559 LAKHS, MOBILE MONEY TRANSFERS RS 2835 LAKHS IN 77 INSTANCES, UPI TRANSFERS RS 144 LAKHS IN 20 INSTANCES FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 27,24478 LAKHS, CLEARING RS 17629 LAKHS MAJORLY FROM AZUS ASPHALTS INDIA PRIVATE LIMITED, PRASHANT GARG, NEFT RS 3,59588 LAKHS MAJORLY FROM SHREE EZHUMALAI ASPHALTS, SHREE BALAJI ASSOCIATES, UNIVERSAL ASPHALT PVT LTD, RTGS RS 20,07902 LAKHS MAJORLY FROM V R PETROCHEM, SHREE BALAJI ASSOCIATE, ABPETROCHEM, INTERNET FUND TRANSFERS RS 2,10195 LAKHS MAJORLY FROM SIDDHIVINAYAK CONSTRUCTION 277905500492, MOBILE MONEY TRANSFERS RS 7664 LAKHS IN 70 INSTANCES, FIXED DEPOSIT RS 1,13092 LAKHS AND OTHERS RS 6957 LAKHS TOTAL DEBIT SIDE RS 26,83747 LAKHS OF WHICH CASH WAS RS 2100 LAKHS BY BRANCH, TRANSFERS RS 1,23844 LAKHS MAJORLY TO RUHI AHUJA, NOVEL LINES LOGISTICS PVT LTD, P P SOFTTECH PRIVATE LIMITED,",{"entities":[(160,179,"FOREIGN ORG"),(6140, 6159,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN SHOWS HIGH VALUE INWARD REMITTANCE FOLLOWED BY DEBITS BY REMITTANCE, CMS, RTGS ETC FOR EXAMPLE, ON 12-07-2021, RS 616 CRORES APPROX GOT CREDITED THROUGH INWARD REMITTANCE FROM IMAEX TRADING COMPANY INC IN THE CURRENCY OF USD FROM THE COUNTRY US FOR WHICH THE PURPOSE MENTIONED WAS INVVVMP 071,77,78,79,81,83,84 PO47621,99716,99725,99664 476622,47623,99726 AND ON THE SAME DAY, RS 3445LAKHS APPROX GOT DEBITED THROUGH OUTWARD REMITTANCE IN FAVOR OF BUDENHEIM IN THE CURRENCY OF EUR TO THE COUNTRY GERMANY FOR WHICH THE PURPOSE MENTIONED WAS ADV PAYMENT AGAINST IMP OTR THAN NEPL BHUTAN ALSO ON 01-07-2021, RS 336CRORES APPROX GOT CREDITED IN PARTS THROUGH INWARD REMITTANCE FROM IMAEX TRADING COMPANY INC IN THE CURRENCY OF USD FROM THE COUNTRY US FOR WHICH THE PURPOSE MENTIONED WAS 067,068,069202122 VVMP067202122, PO49254 VVMP068202122, PO99735 VVMP069202122, PO47072 AND ON 03-07-2021, RS 332CRORES APPROX GOT DEBITED IN PARTS THROUGH NEFT IN FAVOR OF S SESHAN ACCNO: 6028000100006276, IFSC CODE: PUNB0602800, VIN SEA FOODS ACCNO: 40128028945, IFSC CODE: SBIN0070246, KK SEAFOODS ACCNO: 920020036915207, IFSC CODE: UTIB0004292, ETC, RTGS IN FAVOR OF POORVIKA MOBILES PRIVATE LIMITED ACCNO: 18648020000120, IFSC CODE: HDFC0001864, TRANSFER TOWARDS SRI SOWBHAGYA TRADERS 070405002498, AMMAN NSD 609405008653, ETC AS PER BRANCH FEEDBACK, THERE WAS INCOME TAX RAID HAPPENED TO THE ENTITY AND ITS GROUP COMPANIES AND CHARGE SHEET WAS FILED AGAINST THE CUSTOMER AND THE GROUP ENTITIES FOR NEARLY RS 800 CR TAX EVASION ALSO HIGH TURNOVER SEEN IN THE ACCOUNT IN VIEW OF HIGH VALUE TRANSACTIONS IN THE ACCOUNT, COMPLAINT RECEIVED AGAINST CUSTOMER THAT CUSTOMER IS INTO TAX EVASION, STR IS BEEN FILEDV V MARINE PRODUCTS, PARTNERSHIP FIRM, HAVING ADDRESS:17 C,KEERAIKARANTHATTU,MAHADEVANKULAM PO, TISAIYANVILLAI, TIRUNELVELI, TAMIL NADU, INDIA, 627657, CONTACT NUMBER: +919443205149 AND EMAIL ID: ACCOUNTS@VVMARINEPRODUCTSCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 660905600804 WITH ICICI BANK LTD, NEW NO 4, OLD NO 41, NEW TANK STREET, VALLUVARKOTTAM HIGH ROAD, CHENNAI, TAMIL NADU, 600034, SINCE 28052020 DATE OF INCORPORATION IS 06012014 PAN NUMBER OF THE ACCOUNT IS AALFV2122R IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 3513012586 GSTIN AS PER BANK RECORDS IS 33AALFV2122R1ZTTRANSACTION LINKED ACCOUNTS ARE 070405002498, 609405008653 AUTHORISED SIGNATORIES AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT ARE CHENTHIL RAJAN WITH PAN NO AHZPC0253FJ, MUTHU RAJAN WITH PAN NO AKZPM1405G, JAGATHEESAN SUBBIAH NADAR WITH PAN NO ABXPJ7672F, SUBBU RAJAN WITH PAN NO CWJPS9725F THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- AALFV2122R OF THE FIRM, GSTIN, PARTNERSHIP DEED KYC UPDATION IS NOT DUE CUSTOMER PROFILE IS BUSINESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40LAC ALERT WAS TRIGGERED FOR COMPLAINT RECEIVED AGAINST CUSTOMER THAT CUSTOMER IS INTO TAX AVIATION BALANCE IN THE ACCOUNT AT 27072021 WAS RS6723 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 27072021 SHOWS TOTAL CREDIT RS 9,13151 LAKHS OF WHICH NEFT RS 27681 LAKHS MAJORLY FROM DBK,DRAW BACK,CHS DUTY DRAW BACK, RTGS RS 5211 LAKHS MAJORLY FROM VELAVAN STORES,ABDUL RAHMAN, REMITTANCE RS 8,80260 LAKHS MAJORLY FROM PALMETTO FOOD SERVICE LLC 436,BML FOOD GROUP HOLLAND BV,IMAEX TRADING COMPANY INC DBA TOTAL DEBITS DURING THE SAME PERIOD WAS RS 9,08808 LAKHS OF WHICH CASH WAS RS 5400 LAKHS BY BRANCH, TRANSFERS RS 38605 LAKHS MAJORLY TO KURINCHI SEA FOODS,AMMAN NSD,SRI SOWBHAGYA TRADERS, CLEARING RS 13071 LAKHS MAJORLY TO SE TANGEDCO,SE TANGEDCO TUTICORIN,SE TEDC TANGEDCO, NEFT RS 3354 LAKHS MAJORLY TO DHL EXPRESS INDIA PV,SRI NITHYAKALYANI SE, RTGS RS 5,85660 LAKHS MAJORLY TO GODAVARI A, REMITTANCE RS 3718 LAKHS MAJORLY TO BUDENHEIM, INTERNET FUND TRANSFER RS 2,34030 LAKHS, AND OTHERS RS 24714 LAKHS FY 2020-21 FROM 01042020 TO 31032021 : TOTAL CREDIT SIDE RS 16,59471 LAKHS OF WHICH TRANSFERS RS 2481 LAKHS, NEFT RS 44730 LAKHS MAJORLY FROM VV MARINE PRODUCTS, CHS DUTY DRAW BACK, SANDHYA AQUA EXPORTS, RTGS RS 8590 LAKHS MAJORLY FROM NMN TRADING AND, REMITTANCE RS 15,94911 LAKHS MAJORLY FROM SEALAND FOODS INC 7418 RANCO, RED CHAMBER CO, IMAEX TRADING COMPANY INC DBA, INTERNET FUND TRANSFERS RS 4500 LAKHS, FIXED DEPOSIT RS 4219 LAKHS, , AND OTHERS RS 042 LAKHS TOTAL DEBIT SIDE RS 16,57091 LAKHS OF WHICH CASH WAS RS 14335 LAKHS BY BRANCH, TRANSFERS RS 2,38702 LAKHS MAJORLY TO SRI SOWBHAGYA TRADERS,CYR SEA FOODS,AMMAN NSD, CLEARING RS 29840 LAKHS MAJORLY TO TANGEDCO SE TEDC,SE TEDC TANGEDCO,SETEDCTNEBTTN, NEFT RS 46773 LAKHS MAJORLY TO SRI SIVA SAI SEA FOO,SRINIVASA SEA FOODS,SRI VENKATA LAKSHMI, RTGS RS 11,41073 LAKHS MAJORLY TO SSS SEA FO,SRI VENKAT,SRI SIVA S, REMITTANCE RS 1462 LAKHS, INTERNET FUND TRANSFER RS 17549 LAKHS, , AND OTHERS RS 1,67358 LAKHS",{"entities":[(698,723,"FOREIGN ORG"),(196, 221,"FOREIGN ORG") , (3396, 3421,"FOREIGN ORG"),(3452,3477,"FOREIGN ORG"),(3911,3919, "FOREIGN ORG"), (4284,4301,"FOREIGN ORG"),(4330,4355,"FOREIGN ORG")]}),
            ("MEDIAMINE PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS: 529 5TH FLOOR BLOCK D-E, OFFICE NO, CHANDIGARH CITI CENTER CCC, VIP, ROAD, ZIRAKPUR, PUNJAB, INDIA, 140603, CONTACT NUMBER: +917877248015 AND EMAIL ID: MEDIAMINECOIN@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 050705003477 WITH ICICI BANK LTD, NEAR REEBOK SHOWROOM, AMBALA, CHANDIGARH HIGHWAY, ZIRAKPUR, PUNJAB, 140603, SINCE 09062021 DATE OF INCORPORATION IS 22042021 PAN NUMBER OF THE ACCOUNT IS AAOCM5411C THERE ARE NO PAN LINKED ACCOUNT AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE KUMAD KUMAR WADHWA DIRECTOR, PAN- AALPW5130G, MONALI SHIVAJI SHELKI AUTHORISED SIGNATORY, PAN: FCWPS5814P BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS YASH AHUJA PAN: BXWPA1494A THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN: AAOCM5411C, U92490PB2021PTC053281, MOA AOA AND PAN OF AUTHORIZED SIGNATORIES KYC UPDATION IS NOT DUE CUSTOMER IS INTO FILM PRODUCTION, HAVING PRODUCTION HOUSE AND SERVICE PROVIDER DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS 40 LACS ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 23082021 WAS RS026 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 09062021 TILL 23082021 SHOWS TOTAL CREDIT RS 39113 LAKHS OF CLEARING RS 025 LAKHS MAJORLY FROM YASH AHUJA, RTGS RS 38923 LAKHS MAJORLY FROM SHREEJI INFOTECH, MOBILE MONEY TRANSFERS RS 100 LAKH IN 1 INSTANCE, UPI TRANSFERS RS 065 LAKHS IN 2 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 39087 LAKHS OF WHICH REMITTANCE RS 38881 LAKHS MAJORLY TO MAGNETIC MOTION PICTURE, DREAMBOX STUDIOS LIMITED, INTERNET FUND TRANSFER RS 063 LAKHS, CARD TRANSACTIONS RS 002 LAKHS, MOBILE MONEY TRANSFERS RS 097 LAKHS IN 5 INSTANCESAS PER EDD, DURING THE VISIT WE OBSERVED THE OFFICE SIZE IS TOO SHORT BUT TRANSACTION VOLUME IS HIGH IN NUMBERS AND ONE THING WHICH IS MOST DOUBTFUL AS FUNDS RECEIVED FROM COMMON PARTY SHREE JI INFOTECHTRANSACTIONS PATTERN SHOWS CREDITS THROUGH RTGS, MMT TRANSFERS FROM ENTITY FOLLOWED BY CROSS BORDER REMITTANCES ON SAME OR NEXT DAY TO ENTITIES BASED OUT OF HONG KONG AND UAE THE CROSS BORDER TRANSACTIONS ARE UNDERTAKEN WITH PURPOSE OF REMITTANCE AS FOR AUDIO VISUAL AND RELATED SERVICES FOR EXAMPLE: ON 27-07-2021 ACCOUNT WAS CREDITED WITH RS 5475 LAKHS THROUGH RTGS FROM SHREEJI INFOTECH ACCOUNT NO: 200001394029, IFSC: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK LIMITED AND ON 28-07-2021 THE ACCOUNT WAS DEBITED WITH RS 5453 LAKHS THROUGH OUTWARD REMITTANCE TO DREAMBOX STUDIOS LIMITED, HONG KONG IN USD CURRENCY WITH PURPOSE MENTIONED AS PAYMENT FOR AUDIO VISUAL AND RELATED SERVICES- 3D ANIMATION AND VFX RELATED SERVICES SIMILARLY, ON 17-06-2021 ACCOUNT WAS CREDITED WITH RS 4871 LAKHS IN PARTS THROUGH RTGS FROM SHREEJI INFOTECH ACCOUNT NO: 200001394029, IFSC: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK LIMITED, UPI TRANSFER FROM YASH AHUJA ACCOUNT NO: 00000051108833851, VPA: AHUJAYASH148@OKSBI, IFSC: SBIN0031541, BANK: STATE BANK OF INDIA AND ON 19-06-2021 ACCOUNT WAS DEBITED WITH RS 4846 LAKHS THROUGH OUTWARD REMITTANCE TO MAGNETIC MOTION PICTURE, UAE IN USD CURRENCY WITH PURPOSE MENTIONED AS PAYMENT FOR AUDIO VISUAL AND RELATED SERVICES SIMILAR TYPE OF TRANSACTIONS ARE OBSERVED IN THE ACCOUNT AS OBSERVED, THIS IS A NEWLY INCORPORATED / OPENED COMPANY ACCOUNT ON 09062021 TRANSACTION STARTED FROM 14062021 AND TURNOVER IS RS 39113 LAKHS IN SHORT SPAN OF LESS THAN 3 MONTHS AND WHICH IS FAR EXCEEDING THE DECLARED TURNOVER AT THE TIME OF OPENING ACCOUNT IN VIEW OF RECENTLY FORMED ENTITY, HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT WITH HIGH TURNOVER, CUSTOMER IS UNDERTAKING HIGH VALUE OUTWARD REMITTANCE TO ENTITIES BASED IN HONG KONG AND UAE, STR IS BEING REPORTED",{"entities":[(3239,3261,"FOREIGN ORG"),(1724,1746,"FOREIGN ORG"),(1749,1772,"FOREIGN ORG"),(2663,2686,"FOREIGN ORG")  ]}),
            ("TOTAL DEBIT SIDE RS 4,64366 LAKHS OF WHICH CASH WAS RS 1190 LAKHS BY BRANCH, TRANSFERS RS 60697 LAKHS MAJORLY TO KAMLESH AGGARWAL JT1, GAURAV IMPEX, CLEARING RS 10922 LAKHS MAJORLY TO TANISHA RAJKUMAR MITTAL, POPULAR CHEMISTS AND DRUG, NEFT RS 176 LAKHS, RTGS RS 95785 LAKHS MAJORLY TO GAURAV IMPEX, REMITTANCE RS 2,43346 LAKHS MAJORLY TO DONGJUE SILICONE NANJING CO LTD, TECH LINK SILICONES VIETNAM COMPANY, INTERNET FUND TRANSFER RS 18256 LAKHS TO MANAN AGGARWAL 109801502710 AND OTHERS RS 33994 LAKHS FY 2019-20 FROM 01042019 TO 31032020: TOTAL CREDIT SIDE RS 3,22827 LAKHS OF WHICH CASH WAS RS 174 LAKHS, REMITTANCE RS 264 LAKHS TOTAL DEBIT SIDE RS 3,28548 LAKHS OF WHICH CASH WAS RS 1574 LAKHS, REMITTANCE RS 2,18851 LAKHS FY 2018-19 FROM 07082018 TO 31032019: TOTAL CREDIT SIDE RS 1,46833 LAKHS OF WHICH CASH WAS RS 010 LAKHS TOTAL DEBIT SIDE RS 1,52207 LAKHS OF WHICH CASH WAS RS 1616 LAKHS, REMITTANCE RS 1,07272 LAKHS TRANSACTIONS IN THE ACCOUNT ARE WITH MULTIPLE ENTITIES CREDITED THROUGH NEFT, RTGS, CLEARING, TRANSFERS AND ARE FOLLOWED BY DEBIT THROUGH VICE VERSA AND REMITTANCES, WHICH SEEMS UNUSUAL FOR EXAMPLE: ON 25-08-2021 ACCOUNT WAS CREDITED WITH RS 25 LAKHS APPROX IN PARTS THROUGH CLEARING FROM RC TRADERS, HORIAKI INDIA PVT LTD, NEFT FROM GAURI POLYMERS ACCOUNT NO: 1111118464, IFSC: KKBK0000958, BANK: KOTAK MAHINDRA BANK, PADAM CHEMICALS ACCOUNT NO: 71810200000111, IFSC: BARB0DBCOIM, BANK: BAKN OF BARODA, RTGS FROM ANTRA ENTERPRISES ACCOUNT NO: 59208076799783, IFSC: HDFC0009038, BANK: HDFC BANK AND ON THE SAME DATE ACCOUNT WAS DEBITED WITH RS 1537 LAKHS APPROX THROUGH REMITTANCE TO TECH LINK SILICONES VIETNAM COMPANY, VIETNAM, IN USD CURRENCY WITH DESCRIPTION HS-39100090-SILICONES IN PRIMA WITH PURPOSE ADVANCE PAYMENT AGAINST IMPORT OTHER THAN NEPAL BHUTAN SIMILARLY, ON 20-08-02021 ACCOUNT WAS CREDITED WITH RS 12 LAKHS APPROX IN PARTS THROUGH RTGS FROM ELASYO SEAL ACCOUNT NO: 008130100000122, IFSC: BACB0000021, BANK: BASSEIN CATHOLIC CO-OP BANK LTD, AMIT RUBBER CHEM TRADING CO PVT LTD ACCOUNT NO: 010563700001476, IFSC: YESB0000001, BANK: YES BANK LTD, NEFT FROM R R POLYMERS ACCOUNT NO: 02522900000578, IFSC: DCBL0000025, BANK: DEVELOPMENT CREDIT BANK LIMITED, MONTY RUBBER PRODUCT ACCOUNT NO: 50211010004130, IFSC: CNRB0015021, BANK: CANARA BANK AND ON THE SAME DATE ACCOUNT WAS DEBITED WITH RS 1550 LAKHS IN PARTS THROUGH CASH PAID SELF, THROUGH OUTWARD REMITTANCE TO TECH LINK SILICONES VIETNAM COMPANY, VIETNAM, IN USD CURRENCY WITH DESCRIPTION HS-39100090-SILICONES IN PRIMA WITH PURPOSE ADVANCE PAYMENT AGAINST IMPORT OTHER THAN NEPAL BHUTAN AS PER THE REPORT AVAILABLE AS ON DATE, IT WAS OBSERVED THAT THERE ARE TOTAL 8 BILLS OVERDUE PENDING AGAINST THE CUSTOMER WITH REGARDS TO IMPORTS OF WHICH 2 BILLS OF IMPORT COLLECTION BILLS RECEIVED-NOT PAID/ACCEPTED, 3 BILLS OF BILL OF ENTRY PENDING FOR SUBMISSION WITH TRANSACTION DATE 18-06-2019 AND 08-03-2021, THE MAIN COUNTERPARTIES ARE DONGGUAN XINRUN SILICONE NEW MATERIALS CO LTD, ECO INFINIC CO LTD AND 3 BILLS OF IMPORT MADE-PAYMENT PENDING OF AGEING 832 DAYS WITH FIRST BILL OF ENTRY DATE IS 28-05-2019 OF PARTY COLOR SOURCE HK LTD, 665 DAYS OF 11-11-2019 OF PARTY DONGJUE SILICONE NANJING COLTD SIMILAR TYPE OF TRANSACTIONS ARE OBSERVED IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2020-21 WAS RS 4,45188 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF UNUSUAL FOREX REMITTANCE PATTERNS WITH HIGH TURNOVER IN THE ACCOUNT AND HIGH VALUE OF IMPORT BILL PENDING AGAINST THE CUSTOMER DESPITE MULTIPLE REMINDERS AND FOLLOW-UPS BY THE BANK, STR IS BEING FILEDGAURAV IMPEX, A PARTNERSHIP FIRM, HAVING ADDRESS: 8, SAI COMMERCIAL BUILDING, GROUND FLOOR, GOVANDI STATION RD, OPP BEST, MUMBAI, MAHARASHTRA, INDIA, 400088, CONTACT NUMBER: +919892313450 AND EMAIL ID: GAURAVSGCHEM@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 109805500976 WITH ICICI BANK LTD, GROUND FLOOR, UNIT NO 1, SAI COMMERCIAL ANNEX, BKS DEVASHI MARG, GOVANDI EAST, MUMBAI, MAHARASHTRA, 400088, SINCE 07082018 DATE OF INCORPORATION IS 01102008 PAN NUMBER OF THE ACCOUNT IS AAIFG0858Q IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0309044278 GSTIN AS PER BANK RECORDS IS 27AAIFG0858Q2ZQ TRANSACTION LINKED ACCOUNTS ARE 109801508579, 623913033154 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE KAMLESH BIDER KUMAR AGGARWAL AUTHORISED SIGNATORY, PAN- AADPA6553B, GAURAV AGGARWAL AUTHORISED SIGNATORY, PAN- AADPA6554G, RISHIKA AGGARWAL PAN- DSMPR3316H BENEFICIAL OWNER/ PARTNERS OF THE REPORTED ACCOUNT IS RINKU GAURAV AGGARWAL PAN: ACSPA2488C AND RISHIKA AGGARWAL PAN: DSMPR3316H THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN - AAIFG0858Q, PARTNERSHIP DEED AND GST REGISTRATION CERTIFICATE KYC UPDATION IS DUE CUSTOMER IS CHEMICAL TRADER DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LACS ALERT WAS TRIGGERED FOR UNUSUAL FOREX REMITTANCE PATTERNS BALANCE IN THE ACCOUNT AT 27082021 WAS RS12152 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 27082021 SHOWS TOTAL CREDIT RS 2,62544 LAKHS OF WHICH TRANSFERS RS 2406 LAKHS MAJORLY FROM GAURAV IMPEX, VIJAY RUBBER PRODUCTS, SUNDHA POLYMERS, CLEARING RS 49800 LAKHS MAJORLY FROM GAURAV IMPEX, R C TRADERS, SUPREME RUBPLAST, NEFT RS 91473 LAKHS MAJORLY FROM ANTRA INDUSTRIES, SHORE AUTO RUBBER EXPORTS PVT LTD, JAYSUN RUBBER INDUSTRIES, RTGS RS 87978 LAKHS MAJORLY FROM MIDAS RUBBER PLASTIC, RK POLYMER, M/S ENKAY POLYCHEM LLP, INTERNET FUND TRANSFERS RS 9468 LAKHS FROM AKT INDUSTRIES 106905500902, GUPTA CHEMICALS 664005010084, MOBILE MONEY TRANSFERS RS 3367 LAKHS IN 63 INSTANCES, UPI TRANSFERS RS 740 LAKHS IN 23 INSTANCES, FIXED DEPOSIT RS 15786 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 2,44422 LAKHS OF WHICH CASH WAS RS 506 LAKHS BY DEONAR BRANCH, TRANSFERS RS 024 LAKHS MAJORLY TO SAI COMMERCIAL CENTER CO-OP SOCIETY LTD, CLEARING RS 2992 LAKHS MAJORLY TO BD-INVESCOMF, LNT MUTUAL FUND OTM, AXIS MUTUAL FUND, RTGS RS 43306 LAKHS, REMITTANCE RS 1,65336 LAKHS MAJORLY TO DONG JUE SILICONE NANJING CO LTD, TECH LINK SILICONES VIETNAM COMPANY, DONGJUE SILICONE NANJING COMPANY LIMITED, INTERNET FUND TRANSFER RS 15764 LAKHS TO KAMLESH AGGARWAL 109801508579, MOBILE MONEY TRANSFERS RS 3329 LAKHS IN 29 INSTANCES AND OTHERS RS 13165 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY 2020-21 FROM 01042020 TO 31032021: TOTAL CREDIT SIDE RS 4,45188 LAKHS OF WHICH CASH WAS RS 491 LAKHS MAJORLY IN MUMBAI-GHATKOPAR LBS ROAD, DHAMPUR, PUNE - NARHE BRANCH, TRANSFERS RS 6060 LAKHS MAJORLY FROM MANAN AGGARWAL, MS POLYMERS, VIJAY RUBBER PRODUCTS, CLEARING RS 1,35590 LAKHS MAJORLY FROM R C TRADERS, GAURAV IMPEX, GARG CABLES, NEFT RS 1,49540 LAKHS MAJORLY FROM POLYBOND INDIA PRIVATE LTD, GAURAV IMPEX, JAYSUN RUBBER INDUSTRIES, RTGS RS 1,18744 LAKHS MAJORLY FROM M/S ENKAY POLYCHEM LLP, RUBBER CENTRE, V R RUBBER CHEMICALS, INTERNET FUND TRANSFERS RS 25710 LAKHS FROM LOTUS INC 437305500033, MOBILE MONEY TRANSFERS RS 7278 LAKHS IN 185 INSTANCES, UPI TRANSFERS RS 1761 LAKHS IN 76 INSTANCES",{"entities":[(339, 369,"FOREIGN ORG")  , (372, 406,"FOREIGN ORG") ,(1611,1645,"FOREIGN ORG"),(1986,2021,"FOREIGN ORG"),(2408,2442,"FOREIGN ORG"),(2929,2973,"FOREIGN ORG") ,(2976,2994,"FOREIGN ORG"),(3104,3128,"FOREIGN ORG"), (3163, 3193,"FOREIGN ORG")  , (7075, 7084,"FOREIGN ORG")]}),
            ("U V GRAPHIC TECHNOLOGIES PRIVATE LIMITED, A PRIVATE LTD COMPNAY, HAVING ADDRESS: PLOT-15 SECTOR-140A, GAUTAM BUDDHA NAGAR, NOIDA, UTTAR PRADESH, INDIA, 201305, CONTACT NUMBER: +919810035696 AND EMAIL ID: INFO@GTIGTICOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 071605003711 WITH ICICI BANK LTD, A194, OKHLA PHASE1, NEW DELHI, 110020, SINCE 15012021 DATE OF INCORPORATION IS 30012004 PAN OF THE ACCOUNT IS AAACU6652M GSTIN AS PER BANK RECORDS IS 09AAACU6652M1ZT OTHER RELATIONSHIP OF CUSTOMER ARE 071605001332, 071605001337, 071605001521, 071606000125, 071606000126, 071606000127, 071606000216, 071606000217, 071606000218, 071613010616, 071613011248, 757713000381, 757713000382 TRANSACTION LINKED ACCOUNTS ARE 071613011248, 000305015935 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS SONU KUMAR TAYAL AJAPT0655P BENEFICIAL OWNER/ AUTHORISED SIGNATORY/ DIRECTOR OF THE REPORTED ACCOUNT IS ABHAY DATTA WITH PAN NO ACLPD4737A THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, CIN - U29199DL2004PTC124380, COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION KYC UPDATION IS NOT DUE CUSTOMER PROFILE IS BUSINESS - OTHERS AS PER BANK RECORDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40 LACS TO LESS THAN 5CR ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31082021 WAS RS 100 LAKH CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31082021 SHOWS TOTAL CREDIT RS 49765 LAKHS OF WHICH CLEARING RS 10200 LAKHS FROM HOLOSTIC INDIA, THOMSON PRESS INDIA, NEFT RS 4284 LAKHS FROM KOHINOOR PRINTERS P LTD, UV GRAPHIC TECHNOLOGIES PVT LTD, RTGS RS 35000 LAKHS FROM UV GRAPHIC TECHNOLOGIES PVT LTD TOTAL DEBITS DURING THE SAME PERIOD WAS RS 55237 LAKHS OF WHICH TRANSFERS RS 10150 LAKHS TO U V GRAPHIC TECHNOLOGIES PRIVATE LIMITED, EMPLOYEES PROVIDENT FUND ORGANISATION, RTGS RS 7427 LAKHS TO INDER ENTERPRISE, REMITTANCE RS9606 LAKHS TO CHUAN FAN ELECTRIC CO LTD, HIGH POWER EQUIPMENT SRL, JIAXING VICTORY ELECTRONICS CO LTD, INTERNET FUND TRANSFER RS15132 LAKHS TO COSMOS IMPEX INDIA PVT LTD ETC, MOBILE MONEY TRANSFERS RS 429 LAKHS IN 17 INSTANCES AND OTHERS RS 12493 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS YEAR-WISE WERE: FY 2020-21 FROM 15012021 TO 31032021: TOTAL CREDIT SIDE RS 23561 LAKHS OF WHICH CLEARING RS 836 LAKH FROM NEBULA H PRODUCT, VIDYA POLYMER PVT LTD, RAJA ISPAT PLTD, RTGS RS22700 LAKHS FROM UV GRAPHIC TECHNOLOGIES PVT LTD TOTAL DEBIT SIDE RS 17989 LAKHS OF WHICH TRANSFERS RS 1658 LAKHS TO U V GRAPHIC TECHNOLOGIES PRIVATE LIMITED, ESI FUND A/C NO 1, INTERNET FUND TRANSFER RS 748 LAKHS TO COSMOS IMPEX INDIA PVT LTD AND OTHERS RS 15051 LAKHS TRANSACTION PATTERN SHOWS HIGH VALUE FUNDS CREDITED THROUGH RTGS FROM UV GRAPHIC TECHNOLOGIES PVT LTD AND WHICH IS BEING DEBITED THROUGH NON CASH ON THE SAME DAY WITH SAME AMOUNT FOR EXAMPLE, ON 01062021 RS 50 LACS WERE CREDITED THROUGH RTGS FROM UV GRAPHIC TECHNOLOGIES PRIVATE LIMITED: ACCOUNT NUMBER: 02052180000183: IFSC CODE: 02052180000183 AND ON 010620201 RS 50 LACS WERE DEBITED THROUGH INTERNET FUND TRANSFER TO COSMOS IMPEX INDIA PRIVATE LIMITED 00030501 5935 ALSO FROM 03062021 TO 04062021 RS 35 LACS WERE CREDITED THROUGH RTGS FROM UV GRAPHIC TECHNOLOGIE PVT LTD: ACCOUNT NUMBER: 02052180000183: IFSC CODE: KKBK0000958 AND ON 04062021 RS 3092 LACS WERE DEBITED THROUGH OUTWARD REMITTANCE TO U V GRAPHIC TECHNOLOGIES PRIVATE LIMITED: IN USD CURRENCY, FOR WHICH PURPOSE WAS MENTIONED AS ADVANCE PAYMENT AGANIST IMPORT INVOICE NO I1100601 PATTERN IS SIMILAR IN THE ACCOUNT IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT WHICH SEEMS LIKE ROUTING OF FUNDS WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILED",{"entities":[(2046,2070,"FOREIGN ORG"),(2073,2096,"FOREIGN ORG"),(2099,2132,"FOREIGN ORG")]}),
            ("HENCE, STR IS BEING REPORTEDWORLD WIDE LOGISTICS, HAVING ADDRESS: LEVEL 13 PLATINUM, TECHNO, PARK SECTOR, 30 VASHI, NAVI MUMBAI, THANE, MAHARASHTRA, INDIA, 400703, CONTACT NUMBER: +919004049717 AND EMAIL ID: WORLDWIDELOGISTICS28@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 039305013247 WITH ICICI BANK LTD, 163, HT PAREKH MARG, BACKBAY RECLAMATION, CHURCHGATE, MUMBAI, MAHARASHTRA, 400020, SINCE 20032021 DATE OF INCORPORATION IS 02112020 PAN NUMBER OF THE ACCOUNT IS BBJPC9260K TRANSACTION LINKED ACCOUNTS ARE 015105018315, 418405000394 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MAHESH CHOUDHARY PAN-BBJPC9260K THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- BBJPC9260K CUSTOMER IS INTO SERVICE BUSINESS AS PER BANK RECORDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS20 LAKHS TO RS25 LAKHS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNTS BALANCE IN THE ACCOUNT AT 02092021 WAS RS16132 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 02092021 SHOWS TOTAL CREDIT RS 13, 10085 LAKHS OF WHICH CLEARING RS 009 LAKHS MAJORLY FROM WORLD WIDE LOGISTICS, NEFT RS 3900 LAKHS MAJORLY FROM WORLD WIDE LOGISTICS, RTGS RS 13, 00718 LAKHS MAJORLY FROM WORLDWIDE LOGISTICS PVT LTD, WORLD WIDE LOGISTICS,INTERNET FUND TRANSFERS RS 5326 LAKHS MAJORLY FROM DAZZLING INDIA FREIGHT FORWARDING OPC PRIVATE LIMITED 015105018315 AND PAYU PAYMENTS PRIVATE LIMITED 418405000394, MOBILE MONEY TRANSFERS RS 113 LAKHS IN 4 INSTANCES, AND OTHERS RS 019 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 12, 94053 LAKHS OF WHICH RTGS RS 2600 LAKHS , REMITTANCE RS 12, 91285 LAKHS MAJORLY TO GALAXY HOLDINGS PTE LIMITED,MOBILE MONEY TRANSFERS RS 130 LAKHS IN 15 INSTANCES, AND OTHERS RS 003 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS, TOTAL CREDIT SIDE RS 100 LAKH OF WHICH CLEARING RS 100 LAKH MAJORLY FROM MAHESH CHOUDHARY TOTAL DEBIT SIDE WAS NIL TRANSACTION PATTERN IN THE ACCOUNT INDICATES FUNDS CREDITED FROM RTGS, NEFT, MMT TRANSFERS AND DEPLOYED TOWARDS REMITTANCE FOR EXAMPLE: ON 05-08-2021, RS 205 LAKHS CREDITED IN THE ACCOUNT THROUGH RTGS TRANSFER FROM WORLD WIDE LOGISTICS ACCOUNT NO: 10064366207, IFSC CODE: IDFB0010201, BANK IDFC BANK AND ON 06-08-2021, RS105 LAKHS GOT DEBITED THROUGH REMITTANCE TRANSFER TO GALAXY HOLDINGS PTE LIMITED ACCOUNT NUMBER- 3929014322, SINGAPORE WITH REMITTANCE INFORMATION AS TOWARDS AIR FREIGHT SIMILARLY ON 12-05-2021, RS 93 LAKHS HAS BEEN CREDITED IN THE CUSTOMER ACCOUNT THROUGH RTGS FROM WORLD WIDE LOGISTICS ACCOUNT NUMBER-10064366207, IFSC CODE: IDFB0020194, IDFC BANK AND ON SAME DAY RS92 LAKHS 80 K HAS BEEN DEPLOYED TOWARDS REMITTANCE TO GALAXY HOLDINGS PTE LIMITED ACCOUNT NUMBER-3929014322, SINGAPORE WITH REMITTANCE INFORMATION AS TOWARDS AIR FREIGHT SIMILAR TRANSACTION PATTERN IS SEEN REPEATEDLY IN THE ACCOUNT BEHIND WHICH THE RATIONALE OF THE CUSTOMER IS NOT KNOWN TOTAL ANNUAL TURNOVER FOR THE FY 21-22 WAS RS 10,064 LAKHS WHICH IS ABNORMALLY HIGHER THAN THE DECLARED ANNUAL TURNOVER OF RS 25 LAKHS ALSO BANK HAS RECEIVED LETTER FROM FIU-IND ON SEPTEMBER 16, 2020REF NUMBER 1110010275 FOR SUNWODA ELECTRONIC INDIA PRIVATE LIMITED ALSO BANK HAS RECEIVED LETTER FROM FIU-IND ON JULY 24, 2020REF NUMBER 1110009922 FOR OPPO MOBILES INDIA PVT LTD AS PER BOTH LETTER, REGULATOR ASKED FOR ACCOUNT DETAILS, SOA, CARD DETAILS, SOURCE AND DESTINATION, AOF KYC CUSTOMER, WORLD WIDE LOGISTICS IS WORKING FOR BOTH OPPO MONILES INDIA PVT LTD AND SUNWODA ELECTRONIC INDIA PVT LTD AS CUSTOMER HAVING NON-FINANCIAL TRANSACTIONS IE CUSTOMER WORKING AS FREIGHT AGENT FOR ENTITIES WHICH ARE UNDER REGULATOR SCANNER , RAISE SUSPICIONIN VIEW OF ACCOUNT RECEIVING FUNDS FROM RTGS AND DEPLOYED TOWARDS REMITTANCE AND ABNORMAL HIGH ANNUAL TURNOVER MAKES IT SUSPICIOUS,",{"entities":[(1796,1822, "FOREIGN ORG"),(2451,2478, "FOREIGN ORG"),(2820,2847, "FOREIGN ORG") ]}),
            ("HIGH VALUE TRANSACTIONS IN NEW ACCOUNT CREDITS THROUGH NON-CASH FOLLOWED BY DEBITS THROUGH REMITTANCE FOR EXAMPLE, ON 04-08-2021, RS 275 LAKHS APPROX GOT CREDITED IN PARTS THROUGH RTGS FROM SILVERLINE DISTRIBUTORS ACCNO: 200001401745, IFSC CODE: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK LIMITED, FLAIR EFFECT ENTERPRISES ACCNO: 200001405568, IFSC CODE: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK LIMITED, SIDEWALK ENTERPRISES ACCNO: 10070637678, IFSC CODE: IDFB0040137, BANK: IDFC BANK LTD, ETC, AND FROM 04-08-2021 TO 06-08-2021, RS 344 LAKHS GOT DEBITED IN PARTS THROUGH OUTWARD REMITTANCE IN FAVOR OF CLIPPER LOGISTICS PTE LTD IN THE CURRENCY OF USD TO THE COUNTRY SINGAPORE FOR WHICH THE PURPOSE MENTIONED WAS - FREIGHT ON EXPORTS - SHIPPING COMPANIES SIMILARLY, ON 02-08-2021, RS 191 LAKHS APPROX GOT CREDITED IN PARTS THROUGH RTGS FROM ZIOFY TRADERS ACCNO: 200001395123, IFSC CODE: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK LIMITED AND ON 03-08-2021, RS 172 LAKHS APPROX GOT DEBITED IN PARTS THROUGH TRANSFER TOWARDS CHRISTIANSON DEVADANAM JANUMALA 041801514479, OUTWARD REMITTANCE IN FAVOR OF CLIPPER LOGISTICS PTE LTD IN THE CURRENCY OF USD TO THE COUNTRY SINGAPORE FOR WHICH THE PURPOSE MENTIONED WAS - FREIGHT ON EXPORTS - SHIPPING COMPANIES SIMILAR TRANSACTIONS ARE SEEN IN THE ACCOUNT AS PER EDD, CUSTOMER IS SUSPICIOUS AND AS PER IMB VERIFICATION REPORT, PROVIDED BILL OF LADING FOR THE SAID TRANSACTION IS NOT VALID IN VIEW OF HIGH VALUE TRANSACTIONS IN NEW ACCOUNT AND SUBMISSION OF INVALID BILL OF LADING, STR IS BEING FILEDHIS GRACE MANAGEMENT PRIVATE LIMITED, A PRIVATE LIMITED, HAVING ADDRESS: GALA NO 208, SAMRUDDHI COMMERCIAL COMPLEX, PLOT NO D, MALAD WEST, CHINCHOLI BUNDER ROAD, MUMBAI, MAHARASHTRA, INDIA, 400064, CONTACT NUMBER: +919920081473 AND EMAIL ID: HISGRACE4512@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 123905000823 WITH ICICI BANK LTD, SHREE ADINATH TOWER, SHOP NO19 19A, VEER SAVARKAR ROAD, BORIVALI EAST, OPPOSITE NANCY BUS DEPOT MUMBAI, MAHARASHTRA, 400066, SINCE 28102020 DATE OF INCORPORATION IS 04062014 PAN NUMBER OF THE ACCOUNT IS AADCH4146C TRANSACTION LINKED ACCOUNTS ARE 041801514486 AUTHORIZED SIGNATORY/ BENEFICIAL OWNER/ DIRECTOR OF THE REPORTED ACCOUNT IS MAJID SHAIKH PAN: ERPPS3676Q THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE COI - U74999MH2014PTC255376, PAN: AADCH4146C, MOA AOA KYC UPDATION IS NOT DUE CUSTOMER PROFILE IS OTHER SERVICES- MANAGEMENT CONSULTING AND MANAGEMENT SERVICES INCLUDING FINANCIAL, STRATEGIC, HUMAN RESOURCES, MARKETING, OPERATIONS AND SUPPLY CHAIN MANAGEMENT DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 25 CRORES TO RS 100 CRORES ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN NEW ACCOUNT BALANCE IN THE ACCOUNT AT 26082021 WAS RS156 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 26082021 SHOWS TOTAL CREDIT RS 8,14117 LAKHS OF WHICH NEFT RS 45672 LAKHS MAJORLY FROM CRYSTAL INTERNATIONAL, MOUNTAINTOP ENTERPRISES, SENTOR ENTERPRISES, RTGS RS 6,46713 LAKHS MAJORLY FROM MOUNTAINTOP ENTERPRISE, CRYSTAL INTERNATIONAL, KN ENTERPRISES, REMITTANCE RS 35092 LAKHS MAJORLY FROM ICICI BANK LTD, HONGKONG REVERSAL/ REFUND OF REMITTANCES , INTERNET FUND TRANSFERS RS 35300 LAKHS FROM MOHIT CHAUDHARY 041801514486, FIXED DEPOSIT RS 50265 LAKHS, AND OTHERS RS 1046 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 8,17595 LAKHS OF WHICH TRANSFERS RS 40000 LAKHS MAJORLY TO HIS GRACE MANAGEMENT PRIVATE LIMITED, RTGS RS 44948 LAKHS, REMITTANCE RS 6,18649 LAKHS MAJORLY TO FLAVIA LOGISTICS PTE LTD, CLIPPER LOGISTICS PTE LTD, FLAVIA LOGISTIC PTE LTD, INTERNET FUND TRANSFER RS 1,02761 LAKHS TO CHRISTIANSON DEVADANAM JANUMALA 041801514479 AND OTHERS RS 10145 LAKHS FY 2020-21 FROM 28102020 TO 31032021: TOTAL CREDIT SIDE RS 4,61267 LAKHS OF WHICH CASH WAS RS 020 LAKHS, TRANSFERS RS 200 LAKHS MAJORLY FROM HIS GRACE MANAGEMENT PRIVATE LIMITED, NEFT RS 33899 LAKHS MAJORLY FROM ARROW ENTERPRISES, ONE CLICK ENTERPRISES, BLUE SEA LOGISTICS, RTGS RS 3,78545 LAKHS MAJORLY FROM ARROW ENTERPRISES, KN ENTERPRISES, ABRUDHI ENTERPRISES, INTERNET FUND TRANSFERS RS 2700 LAKHS, FIXED DEPOSIT RS 43399 LAKHS AND OTHERS RS 2504 LAKHS TOTAL DEBIT SIDE RS 4,57632 LAKHS OF WHICH CASH WAS RS 973 LAKHS BY ATM AT ICICI BANK ATM KWALITY EGG CENTER, SHOP NO21, RAJIV GANDHI SHOPPING CENTER,EKTA NAGAR, KANDIVALI WEST, MUMBAI- 400067, MUMBAI - VEER SAVARKAR ROAD BRANCH, TRANSFERS RS 40700 LAKHS MAJORLY TO HIS GRACE MANAGEMENT PRIVATE LIMITED, REMITTANCE RS 4,02366 LAKHS MAJORLY TO FLAVIA LOGISTIC PTE LTD, WORLD WIDE OCEAN SHIPPING LLC, INTERNET FUND TRANSFER RS 6484 LAKHS, MOBILE MONEY TRANSFERS RS 338 LAKHS IN 5 INSTANCES AND OTHERS RS 6753 LAKHS",{"entities":[(612,637,"FORIEGN ORG") ,(1112,1137,"FOREIGN ORG")   ,(3069, 3089,"FOREIGN ORG") , (3092, 3114,"FOREIGN ORG"),(3172, 3193,"FOREIGN ORG"),(3196, 3216 ,"FOREIGNORG")  , (3662, 3687,"FORIEGN ORG"), (3688, 3712,"FORIEGN ORG"),(3715, 3737,"FOREIGN ORG")    , (4108, 4125,"FOREIGN ORG")     ,(4655, 4677,"FOREIGN ORG"),(4680, 4708,"FOREIGN ORG")]}),
            ("AS PER TRANSACTIONS IT IS OBSERVED THAT TRANSACTIONS IN THE ACCOUNT ARE WITH MULTIPLE ENTITIES, INDIVIDUALS THROUGH MMT, UPI, NEFT, RTGS, REMITTANCE, CASH, INF WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WHICH SEEMS TO BE UNUSUAL FOR EXAMPLE: ON 16-06-2021 TO 17-06-2021 ACCOUNT WAS CREDITED WITH RS 12 LAKHS IN PARTS THROUGH NEFT FROM N R CHEMICALS ACCOUNT NO: 08872020000574, IFSC: HDFC0000887, BANK; HDFC BANK, G BHAGYALAKSHMI NAND ACCOUNT NO: 6168201000015, IFSC: CNRB0006168, BANK: CANARA BANK, UPI TRANSFER FROM PABBU SANJEEVA ACCOUNT NO: 126510100066933, VPA: 6309674175@YBL, IFSC: ANDB0001265, BANK: ANDHRA BANK AND THE SAME ARE DEBITED IN PARTS THROUGH CASH PAID SELF, CLEARING TO PANKAJ KUMAR, MMT TRANSFERS TO MR AJAY KUMAR BAJPA ACCOUNT NO: 3793663655, IFSC: CBIN0281449, BANK: CENTRAL BANK OF INDIA, SETHI WHEELS ACCOUNT NO: 10932020000176, IFSC: HDFC0001093, BANK: HDFC BANK SIMILARLY, ON 08-6-2021 ACCOUNT WAS CREDITED WITH RS 36 39 LAKHS THROUGH INWARD REMITTANCE FROM LAURA GIDA INSAAT TURIZM, TURKEY BUYER: GREEN AL BAIDAR FOR FEED ADDITIVES, IRAQ WITH PURPOSE ADVANCE RECEIPTS AGAINST EXPORT CONTRACTS AND FROM 08-06-2021 TO 11-06-2021 ACCOUNT WAS DEBITED WITH RS 23 LAKHS IN PARTS THROUGH MMT TRANSFERS TO FREIGHT CARE LOGISTICS ACCOUNT NO: 59209990099657, IFSC: HDFC0000127, BANK: HDFC BANK, ROYAL PLASTIC ACCOUNT NO: 50200032284861, IFSC: HDFC0000127, BANK: HDFC BANK, INTERNET FUND TRANSFERS TO GLOBUS LOGISYS PVT LTD 054451000123, S K SALES AND MARKETING 628805025227 SIMILAR TYPE OF TRANSACTIONS ARE OBSERVED IN THE ACCOUNT THERE SEEMS TO BE NO ECONOMIC RATIONALE FOUND FOR SUCH TRANSACTIONS ALSO, ACCOUNT IS MARKED DEBIT FREEZE ON 05-07-2021 AS RECEIVED NOTICE U/S 91 CR P C, COMPLIANT NO 50/2021 OF IPC SECTION 406, 420, 464, 465, 467, 468, 471, 472, 120B, AS DURING THE INVESTIGATION THE CUSTOMER IS OPENED MULTIPLE BANK ACCOUNTS IN MULTIPLE BANKS TO EARN FRAUDULENT MONEY AND USING THESE ACCOUNTS FOR HIS OWN BENEFITS, RECEIVED FROM DAMAN POLICE ALSO THE BANK WAS ASKED TO SUBMIT KYC DOCUMENTS AND STATEMENT OF ACCOUNTS FROM THE DATE OF ACCOUNT OPENING TURNOVER IN THE ACCOUNT IS RS 1184 LAKHS IS VERY HIGH WHICH IS FAR EXCEEDING THE DECLARED TURNOVER AT THE TIME OF ACCOUNT OPENING OF RS500 LAKHS IN VIEW OF HIGH ACTIVITY IN A NEW ACCOUNT, ACCOUNT FREEZE DUE TO COMPLAINT RECEIVED AND HIGH TURNOVER MAKES IT SUSPICIOUS HENCE, STR IS BEING FILEDPROMOIS INTERNATIONAL LTD, A PROPRIETORSHIP FIRM, HAVING ADDRESS: 4TH, FLOOR ROOM NO 402, KRISHNA TOWER 15/, 63 CIVIL LINES, KANPUR NAGAR, UTTAR PRADESH, INDIA, 208001, CONTACT NUMBER: +918081650077 AND EMAIL ID: PROMOISINTERNATIONAL088@GMAIL COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 628805025242 WITH ICICI BANK LTD, 16/106, J S TOWERS, THE MALL, KANPUR, UTTAR PRADESH, 208001, SINCE 30012020 DATE OF FORMATION IS 07012019 PAN NUMBER OF THE ACCOUNT IS FNGPM6566P GSTIN AS PER BANK RECORDS IS 09FNGPM6566P1ZX TRANSACTION LINKED ACCOUNTS ARE 054451000123, 695705500123 AUTHORIZED SIGNATORIES/PROPRIETOR/DIRECTOR OF THE REPORTED ACCOUNT ARE ANJUM MASIH PAN- FNGPM6566P THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS DEBIT FREEZED ON 05-07-2021 AS RECEIVED NOTICE U/S 91 CR P C, COMPLIANT NO 50/2021 OF IPC SECTION 406, 420, 464, 465, 467, 468, 471, 472, 120B, AS DURING THE INVESTIGATION THE CUSTOMER IS OPENED MULTIPLE BANK ACCOUNTS IN MULTIPLE BANKS TO EARN FRAUDULENT MONEY AND USING THESE ACCOUNTS FOR HIS OWN BENEFITS, RECEIVED FROM DAMAN POLICE KYC DOCUMENTS OBTAINED WERE GSTIN-09FNGPM6566P1ZX, PAN-FNGPM6566P, COI-U24100UP2013PLC060504 ENTITY IS MANUFACTURER, WHOLESALER AND EXPORTER OF AMINO ACID POWDER, ANIMAL FEED SUPPLEMENT DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS40 LAKHS TO LESS THAN RS5CR ALERT WAS TRIGGERED FOR HIGH ACTIVITY IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31072021 WAS RS 1 91 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31072021 SHOWS TOTAL CREDIT RS 237 75 LAKHS, NEFT RS 1 63 LAKHS MAJORLY FROM G BHAGYALAKSHMI NAND, RTGS RS 10 80 LAKHS MAJORLY FROM N R CHEMICALS, REMITTANCE RS 219 58 LAKHS MAJORLY FROM 1 LAURA GIDA INSAAT TURIZM 1, 1 ANIMAL NUTRITION DEVELOPMENT, AGROVET MMC 5501007541 IMISLI, INTERNET FUND TRANSFERS RS 4 00 LAKHS MAJORLY FROM GLOBUS LOGISYS PVT LTD 054451000123 AND SHRI ANANDESHWAR TRADERS 695705500123, MOBILE MONEY TRANSFERS RS 0 94 LAKHS IN 5 INSTANCES, UPI TRANSFERS RS 0 70 LAKHS IN 2 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 239 48 LAKHS, CLEARING RS 2 54 LAKHS MAJORLY TO FIEO FEDERATION OF INDIA, PANKAJ KUMAR, NEFT RS 1 30 LAKHS MAJORLY TO SWATI KESHARI BABU M, NIKHIL TRADE AND EXPORT, REMITTANCE RS 0 16 LAKHS, INTERNET FUND TRANSFER RS 35 25 LAKHS MAJORLY TO GLOBUS LOGISYS PVT LTD 054451000123 AND SHRI ANANDESHWAR TRADERS 695705500123, MOBILE MONEY TRANSFERS RS 173 28 LAKHS IN 371 INSTANCES AND OTHERS RS 0 25 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS, TOTAL CREDIT SIDE RS 800 51 LAKHS, TRANSFERS RS 1 50 LAKHS, NEFT RS 0 32 LAKHS MAJORLY FROM SHREE D M ELECTRICALS, REMITTANCE RS 788 93 LAKHS MAJORLY FROM M S THE NATIONAL AGRICULTURAL, EQUALIVET INCORPORATED 18 BET, 1 ANIMAL NUTRITION DEVELOPMENT, MOBILE MONEY TRANSFERS RS 9 29 LAKHS IN 32 INSTANCES AND OTHERS RS 0 22 LAKHS TOTAL DEBIT SIDE RS 817 28 LAKHS, TRANSFERS RS 20 11 LAKHS, CLEARING RS 25 31 LAKHS MAJORLY TO PILLAI EXIMS CONSORTIUM, RADHA KRISHNA PILLAI, NEFT RS 75 34 LAKHS MAJORLY TO RENU VISHWAKARMA, UFLEX LIMITED, PMS STOCK VENTURE PVT LTD, RTGS RS 29 36 LAKHS, REMITTANCE RS 1 29 LAKHS MAJORLY TO SGS GREECE S A, INTERNET FUND TRANSFER RS 54 82 LAKHS MAJORLY TO GLOBUS LOGISYS PVT LTD 054451000123 AND SHRI ANANDESHWAR TRADERS 695705500123, CARD TRANSACTIONS RS 1 75 LAKHS, MOBILE MONEY TRANSFERS RS 342 17 LAKHS IN 764 INSTANCES AND OTHERS RS 9 97 LAKHS FY 2019-20 FROM 30012020 TO 31032020: TOTAL CREDIT SIDE RS 146 69 LAKHS OF WHICH REMITTANCE RS 142 08 LAKHS TOTAL DEBIT SIDE RS 126 28 LAKHS OF WHICH CASH WAS RS 44 92 LAKHS, REMITTANCE RS 0 11 LAKHS",{"entities":[(1008,1031,"FOREIGN ORG") , (4205,4229,"FOREIGN ORG")]}),   
            ("ON COMPARISON OF SAMPLE DOCUMENTS INVOICES FURNISHED BY VIEW CINECRAFT TECHNOLOGY P LTD ON WHOM BANK HAS ALREADY FILED STR AND SAMPLE DOCUMENTS FURNISHED BY ARROW SERVICES, IT WAS NOTED THAT WHILE THE FUNDS WERE REMITTED TO A COMMON OVERSEAS PARTY NETCLICKS MEDIA S PTE LTD SINGAPORE, THE INVOICES HAD DIFFERENT FORMATS ALSO THE INVOICE NUMBERS DID NOT APPEAR TO BE IN SERIAL ORDER WHICH APPEARS TO BE UNUSUAL DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS OF ARROW SERVICES, IT WAS NOTED THAT THE OFFICE SETUP WAS A 600 SQUARE FEET RENTED PREMISE WITH TEMPORARILY SIGN BOARD NAME OF ENTITY PRINTED ON BANNER, CHAIR, TABLE, LAPTOP AND TWO EMPLOYEES HOWEVER, NO EQUIPMENT OR ANY BANNER/PICTURES RELATED TO ADVERTISEMENT SERVICES GIVEN BY THE CUSTOMER WAS OBSERVED IN THE OFFICE IT IS PERTINENT TO NOTE THAT THE ENTITY IS FORMED ON 2072021, ACCOUNT IS OPENED WITHIN 9 DAYS IE, ON 2972021 AND WITHIN 2 MONTHS OF ACCOUNT OPENING TRANSACTIONS ROUTED AMOUNTED TO RS433 CR HENCE IT APPEARS DOUBTFUL THAT THE CUSTOMER HAS THE NET WORTH / GENUINE BUSINESS ACTIVITY TO ROUTE SUCH HIGH VALUE TRANSACTIONS AND CUSTOMER MAY HAVE DELIBERATELY INDICATED INFLATED TURNOVER TO AVOID BEING TRACKED ON ROUTING HIGH VALUE TRANSACTIONS ALSO, NO MAJOR BUSINESS RELATED TRANSACTIONS APPEARS TO BE ROUTED THROUGH THE ACCOUNT LIKE PAYMENTS TOWARDS SALARY, RENT AND ETC, WHICH IS UNUSUAL WHILE RUNNING A COMMERCIAL BUSINESS BASIS THE TRANSACTION PATTERN AND INFORMATION AVAILABLE, IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN NAME OF ARROW SERVICES AND ACCOUNT MAY HAVE BEEN OPENED TO ROUTE TRANSACTIONS FOR LAYERING OF FUNDS AND TO REMIT FUNDS OVERSEAS UNDER PRETEXT OF ADVERTISING SERVICES WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENT QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS 517 CR IN CASH RS 000 CR AND CREDIT: RS 520 CR IN CASH RS 000 CR THE ACCOUNT OF ARROW SERVICES IS ACTIVE WITH BALANCE OF RS282 LAKHS AS ON 17092021DAYA KISHAN AS A PROPRIETOR MAINTAINS A CURRENT ACCOUNT BEARING NO 0945488284 IN THE NAME OF A SOLE PROPRIETORSHIP CONCERN ARROW SERVICES SINCE 29072021 AT OUR BRANCH LOCATED AT C-2/11, ASHOK VIHAR-2, 110052 - NEW DELHI THE ENTITY WAS ESTABLISHED ON 20072021 AND HAS SUBMITTED COPIES OF PAN GYLPK7247C, GST REG CERT 07GYLPK7247C1Z6, IEC CERT GYLPK7247C AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 5TH FLOOR, UNIT NO 514 P NO 2, AMBA TOWER COMMUNITY CENTER, SEC-9, ROHINI, NORTH WEST DELHI, 110085 - DELHI AND HAS REGISTERED MOBILE NUMBER 7840015796 THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND CUSTOMER IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF ADVERTISING WITH ANNUAL TURNOVER OF RS1000 CR WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES NON-IMPORT DONE THROUGH THE BANK, IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NON-IMPORT WERE MADE THROUGH THE NEWLY OPENED ACCOUNT OF ARROW SERVICES FAVORING SINGAPORE BASED ENTITY NAMED NETCLICKS BANK HAS ALREADY FILED STRS ON ANOTHER CUSTOMER VIEW CINECRAFT TECHNOLOGY PRIVATE LIMITED A/C NO 5545467301 VIDE BATCH ID 2107035260 WHEREIN FUNDS WERE REMITTED TO THE SAID OVERSEAS PARTY AND THE ACCOUNT ACTIVITY APPEARED TO BE SUSPICION HENCE DUE DILIGENCE WAS CARRIED OUT ON THE ACCOUNT OF ARROW SERVICES AND THIS REPORT IS FILED ON THE FOLLOWING GROUNDS OF SUSPICION: ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL 17092021, IT WAS NOTED THAT RTGS/NEFT/IMPS CREDITS AMOUNTED TO RS 519 CR MAINLY FROM OWN ACCOUNT OF ARROW SERVICES MAINTAINED WITH OTHER BANK RS 311 CR , YES BANK - 107563300002139, BALAJI ENTERPRISES RS157 CR, PUNJAB NATIONAL BANK - 50591132000948 AND SHRI RAM ENTERPRISES RS051 CR, PUNJAB NATIONAL BANK - 50591132000893 SUBSEQUENT TO SUCH CREDIT, FOREIGN OUTWARD REMITTANCE NON-IMPORT AMOUNTED TO RS398 CR MAINLY FAVORING ADVANCE MEDIA FILM PRODUCTION LLC- UAE RS 173 CR, QUICKODE PTE LTD- SINGAPORE RS 104 CR, DREAMBOX STUDIOS LIMITED- HONG KONG RS 088 CR AND NETCLICKS - SINGAPORE RS031 CR TOWARDS ADVERTISEMENT SERVICES RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS076 CR MAINLY FAVORING ARROW SERVICES RS 047 CR , YES BANK - 107563300002139, SPARX TRADING RS022 CR, PUNJAB NATIONAL BANK - 0120002100765727 AND VIVEK ENTERPRISES RS020 CR, THE RATNAKAR BANK LTD BANK - 409001312097",{"entities":[(248, 273,"FOREIGN ORG"), (3085, 3099,"FOREIGN ORG") ,(3945, 3979,"FOREIGN ORG"), (3995, 4012,"FOREIGN ORG"),(4034, 4059,"FOREIGN ORG"),(4084,4095,"FOREIGN ORG") ]}),
            ("1 THE COMBINED ACCOUNT SCRUTINY REVEALED ROUTINE BUSINESS TRANSACTIONS WHEREIN CREDITS VIA FUND TRANSFERS AND RTGS AND DEBIT TOWARDS CHEQUE ISSUANCE, RTGS/NEFT TRANSFERS OBSERVED2 AS PER MEDIA NEWS, ENFORCEMENT DIRECTORATE ED IS INVESTIGATING CHARGES OF MONEY LAUNDERING IN THE EMBRAER SA DEFENSE DEAL THE FEDERAL AGENCY HAD INITIATED A PROBE AFTER THE CENTRAL BUREAU OF INVESTIGATION CBI HAD REGISTERED AN FIR ON THE BASIS OF A REFERENCE RECEIVED FROM CHIEF CONTROLLER, R AND D R AND M AND DS, DEFENSE RESEARCH AND DEVELOPMENT ORGANIZATION DRDO, DEPARTMENT OF DEFENSE RESEARCH AND DEVELOPMENT, MINISTRY OF DEFENSE THE ALLEGATIONS ARE THAT M/S EMBRAER WAS BEING INVESTIGATED BY AUTHORITIES OF US AND BRAZIL FOR ALLEGED EMPLOYMENT OF AN AGENT IN FACILITATING THE CONTRACT FOR AEW AND C PROJECT WITH CABS/DRDO FOR PROCUREMENT OF THREE FULLY MODIFIED EMB-145 AIRCRAFT AND HAVE LAUNDERED MONEY ED HAS CLAIMED THAT ITS PROBE HAS REVEALED THAT, M/S EMBRAER SA BRAZIL OBTAINED THE CONTRACT FOR SUPPLY OF AIRCRAFTS TO INDIAN AIR FORCE FOR USD 210 MILLION AND PAID COMMISSION OF USD 576 MILLION TO A MIDDLEMAN NAMED VIPIN KHANNA FOR INFLUENCING THE SAID CONTRACT IN ITS FAVOR THE SAID KICKBACKS OF USD 576 MILLION WERE ALLEGEDLY ROUTED BY M/S EMBRAER SA, BRAZIL THROUGH ITS SUBSIDIARIES TO M/S INTERDEV AVIATION SERVICES PTE LTD, SINGAPORE IN LIEU OF SHAM AGREEMENT 3 IT IS FURTHER ALLEGED THAT KICKBACKS TO THE TUNE OF AROUND USD 3275 MILLION WERE FURTHER ROUTED FROM M/S INTERDEV AVIATION SERVICES PTE LTD TO M/S KRBL DMCC, DUBAI A 100% OWNED SUBSIDIARY OF KRBL LTD IN LIEU OF SHAM AGREEMENT TO FINALLY REACH INDIA THROUGH M/S KRBL LTD PROJECTING THE SAME AS UNTAINTED MONEY 4 ED HAS FURTHER CLAIMED THAT ANOOP KUMAR GUPTA, DIRECTOR OF KRBL LIMITED SIGNED THE SHAM AGREEMENT HELD BETWEEN INTERDEV AVIATION SERVICES PTE LTD AND KRBL DMCC TO RECEIVE PROCEEDS OF CRIME POC WHICH WERE FINALLY RECEIVED IN THE BANK A/C OF KRBL LTD IN WHICH HE IS ONE OF THE DIRECTOR THE AGENCY ED HAS SO FAR ATTACHED IMMOVABLE PROPERTIES WORTH RS 1629 CRORE BELONGING TO KRBL LTD LINK - ECONOMICTIMESINDIATIMESCOM/NEWS/DEFENCE/ED-FILES-CHARGE-SHEET-IN-EMBRAER-DEFENCE-DEAL/ARTICLESHOW/79642508CMS?FROM=MDR , WWWLIVEMINTCOM/NEWS/INDIA/ED-FILES-CHARGESHEET-IN-EMBRAER-DEFENCE-DEAL-MONEY-LAUNDERING-CASE-11607508980682HTML 5 CASE IS THEREFORE RECOMMENDED FOR FILING BASIS ADVERSE MEDIA NEWS AND INVESTIGATION BY ENFORCEMENT DIRECTORATE COUPLED WITH SIGNIFICANT HIGH VALUE TRANSACTIONS IN THE GROUP ACCOUNTS MUCH HIGHER THAN THE DECLARED TURNOVER AND PROFILE OF THE CUSTOMER ALSO CUSTOMER CORE BUSINESS ACTIVITY IS RELATED TO RICE MANUFACTURING HOWEVER HIS INVOLVEMENT IN DEFENSE DEALS SEEMS TO BE UNUSUAL AND DOES NOT SEEM TO HAVE ANY RATIONALE STR FILED BASIS -A ADVERSE MEDIA NEWS AGAINST CUSTOMERB CUSTOMER UNDER INVESTIGATION OF ENFORCEMENT DIRECTORATE FOR MONEY LAUNDERING OFFENCEC CUSTOMER MAINTAINING MULTIPLE ACCOUNTS WHERE BUSINESS ACTIVITY IS PERTAINING TO RICE MANUFACTURING HOWEVER ADVERSE NEWS ABOUT CUSTOMER IS PERTAINING TO HIS INVOLVEMENT IN DEFENSE DEAL1 MEDIA NEWS PUBLISHED ON 09TH DECEMBER, 2020 REVEALED THE ENFORCEMENT DIRECTORATE ED ON WEDNESDAY FILED A CHARGE SHEET AGAINST M/S EMBRAER SA, BRAZIL, M/S INTERDEV AVIATION SERVICES PTE LTD, ANOOP KUMAR GUPTA DIRECTOR OF KRBL LIMITED, KRBL LIMITED, ANURAG POTDAR NEPHEW OF ANOOP KUMAR GUPTA AND OTHERS IN A CASE OF EMBRAER SA DEFENSE DEAL SCREENING WAS CARRIED OUT WITH BANK S DATABASE WHERE POSITIVE MATCH CUSTOMER DETAILS MATCHING WITH DETAILS OF ENTITY APPEARING IN ADVERSE MEDIA NEWS WAS FOUND FOR M/S KRBL LIMITED WHO IS MAINTAINING A/C 02620330000013 AT OUR SANGRUR PUNJAB BRANCH 2 M/S KRBL LIMITED IS HAVING PAN NO AAACK4644H AND IS MAINTAINING CURRENT ACCOUNT NO 02620330000013 WITH OUR SANGRUR PUNJAB BRANCH SINCE 17-01-2006 CUSTOMERS REGISTERED OFFICE ADDRESS IS VILLAG BHASAUR, DHURI - 148024 WITH MOBILE NUMBER UPDATED 911123915813 AND EMAIL ID UPDATED AS RAJEEVGARG@KRBLINDIACOM CUSTOMER DECLARED PROFILE AS MANUFACTURER OF RICE WITH DECLARED TURNOVER OF RS4498 CRORES ACCOUNT STATUS IS ACTIVE WITH ACCOUNT BALANCE OF RS 4,87,14,32100/- AS ON 23-08-2021 RISK CATEGORY OF THE ACCOUNT IS LOW LATEST KYC UPDATED ON 13-01-2020 MR ANIL K MITTAL, ARUN K GUPTA , MAHESH C JAIN AND ANOOP K GUPTA ARE THE AUTHORISED SIGNATORIES IN THE ACCOUNT BUSINESS INCORPORATION DATE IS 30-03-1993 3 SCRUTINY OF THE ACCOUNT STATEMENT OF KRBL LIMITED A/C NO 02620330000013 FOR THE PERIOD APR-2019 TO DEC-2020 REVEALED THAT THE ACCOUNT RECEIVED CREDITS OF AGGREGATING TO RS 3804657 LAKHS OF WHICH RS 10441 LAKHS WERE RECEIVED THROUGH CHEQUE CLEARING, RS 926850 LAKHS WERE RECEIVED FROM FUND TRANSFER MAINLY FROM A/C NO 01952320000164 - CHAMAN LAL SETIA EXPORTS LIMITED RS 50693 LAKHS, A/C NO 50200021771627 - SANDEEP RICE AND GEN MILLS RS 88141 LAKHS, A/C NO 50200010683284 - RICELA HEALTH FOODS LIMITED RS 199458 LAKHS ,RS 487301 LAKHS RECEIVED FROM SWEEP TRANSFERS AND WORKING CAPITAL DISBURSED AND RS 2379808 LAKHS WERE RECEIVED THROUGH RTGS/NEFT/IMPS/UPI TRANSFERS MAINLY FROM AMIR CHAND JAGDISH KUMAR BANK OF INDIA, UNION BANK OF INDIA AND BANK OF INDIA, RS 225547 LAKHS, SHREE BALAJI FOOD PRODUCTS SBI, RS 20529 LAKHS, BALAJI OVERSEAS PUNJAB NATIONAL BANK, RS 107306 LAKHS AND OTHERS DEBITS AGGREGATING TO RS 3765259 LAKHS OF WHICH RS 669154 LAKHS WERE TOWARDS CLEARING RS 626166 LAKHS WERE UTILIZED FOR FUND TRANSFER MAINLY TO SALARY PAYMENT RS 120032 LAKHS, A/C NO 08738710000011 - KRISHNA AUTO SALES RS 10604 LAKHS, A/C NO 50200013474602-S R S ENGG CO RS 40377 LAKHS AND OTHERS, RS 1679204 LAKHS WERE UTILIZED TOWARDS SWEEP TRANSFERS AND RS 790346 LAKHS WERE UTILIZED FOR RTGS/NEFT/IMPS/UPI TRANSFERS MAINLY TO OWN A/C SBI, RS 447463 LAKHS OBSERVED 4 SCRUTINY OF THE ACCOUNT 02620330000013 - KRBL LIMITED FOR THE PERIOD APRIL 2020 - AUG 2021 REVEALED THAT CUSTOMER HAS RECEIVED TOTAL CREDITS AGGREGATING TO RS 2978703 LAKHS MAINLY VIA RTGS/NEFT AGGREGATING TO RS 2109321 LAKHS MAINLY FROM NATURE BIOFOODS LIMITED AGGREGATING TO RS 78124 LAKHS AND FROM AROMA AGROTECH P LTD AGGREGATING TO RS 19116 LAKHS THE PROCEEDS ARE FURTHER WITHDRAWN MAINLY VIA FUNDS TRANSFER AGGREGATING TO RS 2176114 MAINLY TO SELF ACCOUNT - 00030120000180 AGGREGATING TO RS 1551154 LAKHS 5 CUSTOMER PAN AND MOBILE LINKED ACCOUNTS IDENTIFIED AS BELOW - 08610310000014, 21520310000016, 00432000029540, 00032230001959, 01670700000024, 03800310000053, 50200004290422, 50200002729739, 50200002729742, 50200004290296, 28390310000013, 26730310000019, 22090310000014, 02682320002302, 14822020000465, 50200011340961, 50200010607610, 00562320001807, 50200002474458, 01530330000203, 00620310000927, 05730310000021, 09270310000015, 00030310014350, 00030340043146, 26480310000013, 00030310014343, 00030120000180, 07700310000014, 00032220002569, 05710310000012, 00030310014058, 00431000202264, 00432000029550, 00432000029516, 00432000029577 AND 00432000029523, 50200058968801",{"entities":[(278,288,"FOREIGN ORG")  ,(943,953,"FOREIGN ORG"),(1234,1243,"FOREIGN ORG"),(1285,1318,"FOREIGN ORG"),(1464,1498,"FOREIGN ORG"),(1506,1514,"FOREIGN ORG"),(1820, 1829,"FOREIGN ORG"),(1781,1815,"FOREIGN ORG")  ,(3170,3179,"FOREIGN ORG"),(3194,3227,"FOREIGN ORG")        ]}),
            ("4 ON CONDUCTING ENHANCED DUE DILIGENCE AND WEB SEARCH IT WAS FOUND THAT THE ENTITY WITH THE NAME OF M/S GOMONEY SERVICOS DIGITAIS LTD IS INVOLVED IN THE BUSINESS OF ONLINE MONEY TRANSFER, CURRENCY CONVERSION ETC HTTPS://GOMONEYME/ WHICH SEEMS TO BE IN CONTRADICTION OF THE CLARIFICATION PROVIDED BY OUR CUSTOMER ON THE PURPOSE OF THE TRANSACTIONS CARRIED OUT WITH THE SAID ENTITY AND PROBABLY MISGUIDING THE BRANCH OFFICIALS5 IT IS ALSO OBSERVED THAT FUNDS RECEIVED VIA INWARD REMITTANCES WERE IMMEDIATELY REMITTED OUT FROM THE ACCOUNT ON THE SAME OR NEXT DAY MAJORLY TO THE CUSTOMER S OWN ACCOUNT MAINTAINED WITH OTHER BANK WITHOUT ANY ECONOMIC RATIONALE6 THE BANK HAD REPORTED THE CROSS BORDER INWARD/OUTWARD REMITTANCE TRANSACTIONS IN THE ACCOUNT IN THEIR RESPECTIVE MONTHLY CROSS-BORDER WIRE TRANSFER REPORTS TO FIU-IND FOR THE MONTH DECEMBER 2020 TO JULY 20217 BASED ON THE TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT THE TRANSACTIONS ROUTED THROUGH THIS ACCOUNT MAY NOT HAVE ANY ECONOMIC RATIONALE AND THE ENTITY MAY HAVE RECEIVED FUNDS FROM HIGH RISK COUNTRIES WITHOUT ANY UNDERLYING COMMERCIAL ACTIVITY WITH AN INTENTION OF TRADE-BASED MONEY LAUNDERINGTHE TRANSACTION TURNOVER IN THE CURRENT FINANCIAL YEAR 2021-2022 IS DEBIT: RS56717 LAKHS IN CASH- NIL AND CREDIT: RS56099 LAKHS IN CASH- NIL AND THE TRANSACTION TURNOVER IN PREVIOUS THREE FINANCIAL YEARS 2020-2021 IS DEBIT: RS851 LAKHS IN CASH- NIL AND CREDIT: RS850 LAKHS IN CASH- NIL , 2019-20 IS DEBIT: RS29708 LAKHS IN CASH: RS065 LAKHS AND CREDIT: RS29604 LAKHS IN CASH: NIL , 2018-19 : DEBITS: RS 25792 LAKHS, IN CASH RS796 LAKHS , CREDITS: RS25605 IN CASH: NIL THE AFORESAID CURRENT ACCOUNT IS IN ACTIVE STATUS AND THE BALANCE IN THE ACCOUNT AS ON DATE OF FILING THIS STR IS RS15,92737/-FURTHER THE SIGNATORY MR AMBARISH ROY ALSO HOLDS A CURRENT ACCOUNT IN THE NAME M/S DKVR VENTURES PRIVATE LIMITED A/C NO- 10057773016, CIN-U45201KA2020PTC135403, PAN-AAHCD8045N TRANSACTIONS IN THIS ACCOUNT ARE UNUSUAL AND NOT IN LINE WITH THE PROFILE OF THE CUSTOMERA CURRENT ACCOUNT BEARING NUMBER 10018004514 WAS OPENED IN THE NAME OF A PRIVATE LIMITED COMPANY NAMELY, M/S MERRIANT FASHIONTECH PVT LTD DATE OF ESTABLISHMENT 28/02/2018 ON 21/03/2018 AT OUR BANGALORE RESIDENCY ROAD BRANCH SITUATED AT PLOT NO 79 RESIDENCY ROAD, RICHMOND TOWN, RESIDENCY BUILDING, BANGALORE- 560025, KARNATAKA MR SUBHRO SEN GUPTA DOB-03/04/1973, PAN- ACDPG0544Q AND AADHAR-XXXXXXXX9469 AND MR AMBARISH ROY DOB 20/12/1982, PAN-AKDPR2930D AND AADHAR NO- XXXXXXXX2347 ARE THE DIRECTORS / SIGNATORIES OF THE SAID ENTITY COPY OF PAN-AALCM6493P, CERTIFICATE OF INCORPORATION- U74999KA2018PTC110612, MEMORANDUM AND ARTICLES OF ASSOCIATION, BOARD RESOLUTION WERE FURNISHED AS KYC S AS PER THE BANK S RECORDS, CUSTOMER S MAILING ADDRESS IS FLAT NO 307, JYOTHI CLIQUE APPTS, 4TH B CROSS, ABBAYA REDDY LAYOUT, KHAGGADASAPURA MAIN RD, BANGALORE- 560093, KARNATAKA, EMAIL ID-SUBHROSENGUPTAITC@GMAILCOM AND MOBILE NUMBER IS 918013333100 THE ENTITY IS INTO THE BUSINESS OF TEXTILES/GARMENTS/ HANDLOOMS HAVING ANNUAL TURNOVER BETWEEN RS 5 CRORES TO RS10 CRORES THE ENTITY HAS BEEN CATEGORIZED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC UPDATION OF KYCCONSIDERING HIGH VALUE INWARD REMITTANCES FROM M/S GOMONEY SERVICOS DIGITAIS LTD BRAZIL FOLLOWED BY SUBSEQUENT NEFT TRANSFERS IN THE AFORESAID CURRENT ACCOUNT, AN ALERT WAS RAISED TO THE BRANCH TO CARRY OUT ENHANCED DUE DILIGENCE ON THE CUSTOMER AND THIS REPORT IS BEING FILED BASED ON THE FOLLOWING GROUNDS OF SUSPICION1 ON REVIEWING THE TRANSACTIONS IN THE AFORESAID CURRENT ACCOUNT FOR THE PERIOD 01/04/2021 TO 31/07/2021, IT WAS NOTED THAT INWARD REMITTANCE CREDITS AMOUNTING TO RS 114984 LAKHS WERE RECEIVED MAINLY FROM M/S GOMONEY SERVICOS DIGITAIS LTD RS104624 LAKHS, COUNTRY- BRAZIL AND M/S ANHUI FORTUNE FASHION CULTURE CO RS10363 LAKHS, COUNTRY-CHINA MAINLY FOR THE PURPOSE OF ADVANCE RECEIPTS AGAINST EXPORT CONTRACTS IN ADDITION, NEFT/RTGS/IMPS CREDITS RECEIVED MAINLY FROM ENTITY OWN ACCOUNT MAINTAINED WITH OTHER BANK RS2004 LAKHS, ORIENTAL BANK OF COMMERCE/ PUNJAB NATIONAL BANK: A/C NO- 09384011000480, KOTAK MAHINDRA BANK: A/C NO- 2214128969 AND MR AMBARISH ROY RS831 LAKHS, ICICI BANK: A/C NO- 106401502902, CITI BANK: A/C NO-5862507802 2 SUBSEQUENT TO THESE CREDITS, NEFT/RTGS/IMPS PAYMENTS AMOUNTING TO RS117355 LAKHS WERE MADE MAINLY TO MR FARID KHAN RS52736 LAKHS, STATE BANK OF INDIA, A/C NO- 30940570749 , ENTITY S OWN ACCOUNTS MAINTAINED WITH OTHER BANKS RS18769 LAKHS, ORIENTAL BANK OF COMMERCE/ PUNJAB NATIONAL BANK, A/C NO- 09384011000480, KOTAK MAHINDRA BANK, A/C NO- 2214128969 AND TO M/S HAARON HAIR EXPORTS RS8278 LAKHS, ICICI BANK: A/C NO-130005001182 IN ADDITION, CASH WITHDRAWALS AMOUNTING TO RS 132 LAKHS WERE WITHDRAWN THROUGH VARIOUS ATM AT HARYANA AND WEST BENGAL LOCATIONS AND POINT OF SALE PAYMENTS AMOUNTING TO RS087 LAKHS, WERE ALSO NOTED IN THE ACCOUNT3 DURING DISCREET ENQUIRY BY BRANCH OFFICIALS, IT WAS REVEALED THAT THE CUSTOMER IS MAINLY INTO EXPORT OF PROCESSED HUMAN HAIRS, WHEREAS THE CUSTOMER WAS RECEIVING INWARD REMITTANCE MAINLY FROM M/S GOMONEY SERVICOS DIGITAIS LTD BRAZIL WHICH IS CONTRADICTING WITH THE NATURE OF BUSINESS THAT THE CUSTOMER HAS DECLARED AT THE TIME OF OPENING OF THE ACCOUNT",{"entities":[(104,133,"FOREIGN ORG") ,(3244,3273,"FOREIGN ORG"),(3722,3751,"FOREIGN ORG")]}),
            ("BELOW ARE RED FLAGS OBSERVED 1STATED ANNUAL INCOME OF CUSTOMER IS LESS THAN RS1CRORE HOWEVER IN LESS THAN 6 MONTHS CUSTOMER HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS23 CRORES 2IT WAS OBSERVED THAT ALTHOUGH OVERSEAS ENTITIES ELECTRONORD TEKNIKA PTE LTD AND SPACEX GLOBAL LIMITED ARE TWO DIFFERENT ENTITIES LOCATED IN DIFFERENT JURISDICTIONS OF SINGAPORE AND HONG KONG RESPECTIVELY THEIR INVOICE FORMAT AND PARTICULARS LOOK SIMILAR AND PER UNIT PRICE IN THE INVOICE IS THE SAME3AS PER PUBLIC DOMAIN MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS: 121 MCNAIR ROAD, 07-69, SINGAPORE 320121 OF ELECTRONORD TEKNIKA PTE LTD4FURTHER, SPACEX GLOBAL LIMITED, HONG KONG IS NEWLY INCORPORATED ENTITY IN THE YEAR APRIL 2021 REFER: HTTPS:WWWHKCORPORATIONSEARCHCOM/COMPANIES/3037936/ 5PUBLIC DOMAIN INFORMATION OF OVERSEAS ENTITY SOFT SERVE LLC, UAE IS INSUFFICIENT6OPINION REPORT OF ELECTRONORD TEKNIKA PTE LTD IS INCONCLUSIVE WITH LIMITED INFORMATION ON SCOPE AND SCALE OF BUSINESS 7ALSO, ELECTRONORD TEKNIKA PTE LTD IS COMMON OVERSEAS ENTITY RECEIVING CROSS BORDER REMITTANCES FROM OTHER YBL CUSTOMERS VIZ RANKPIE INDIA PRIVATE LIMITED AND FLYDER AMENITIES ON WHICH STR WAS FILED VIDE FIU BATCH ID 2001282005 AND 2011177781IN LIGHT OF CROSS BORDER REMITTANCES BEING UNDERTAKEN IN FAVOR OF OVERSEAS ENTITIES WITH OPAQUE CREDENTIALS IT HAS BEEN DECIDED ON 22- SEPTEMBER- 2021 TO FILE STR ON SAID CUSTOMER1 THE CUSTOMER BLUE PEAK SERVICE, A PARTNERSHIP FIRM PAN- AAXFB6215F, DOI 11-03-2021, COMMUNICATION AND PERMANENT ADDRESS- FIRST FLOOR SHOP NO 16 F BLOCK ASHOK VIHAR, PH 1 NORTH WEST DELHI, NEW DELHI, 110052, DELHI, GSTIN- 07AAXFB6215F1ZB, MOBILE NUMBER- 919818417955, EMAIL ID- ASHARMA43021@GMAILCOM IS MAINTAINING A CURRENT AC NO 065763400000604 SINCE 18-05-2021 WITH DWARKA MORE,DELHI BRANCH ADDRESS- GROUND FLOOR , PLOT NO-B-6, RAMA PARK, UTTAM NAGAR, DWARKA MORE, NEW DELHI -1100592 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN MAY 2021 AAKASH KUMAR SHARMA AND AMIT ARE AUTHORIZED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS IT SERVICE PROVIDER WITH AN ANNUAL TURNOVER LESS THAN RS1CRORE 3 IN THE ACCOUNT OF BLUE PEAK SERVICE AC NO 065763400000604, TRANSACTIONS ANALYSIS FROM 01-04-2021 TILL 23-09-2021, SHOWS 14 CREDIT TRANSACTIONS AMOUNTING TO RS 23,375,675 AND 21 DEBIT TRANSACTIONS AMOUNTING TO RS 23,374,950OUT OF THE TOTAL CREDITS OF RS 23,375,675 IN THE ACCOUNTS, RTGS AMOUNTS TO RS 23,175,675 CONSTITUTING AROUND 99 PERCENT OF THE TOTAL CREDITS, OUT OF THE TOTAL DEBITS OF RS 23,374,950 IN THE ACCOUNTS, RTGS AMOUNTS TO RS 10,492,520 CONSTITUTING AROUND 45 PERCENT OF THE TOTAL DEBITS, TRADE TRANSACTION AMOUNTS TO RS 12,287,430 CONSTITUTING AROUND 53 PERCENT OF THE TOTAL DEBITS, 4 IN THE ACCOUNT OF BLUE PEAK SERVICE AC NO 065763400000604, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW: FROM 01-04-2028 TILL 31-03-2021 NO TRANSACTIONS WERE RECORDED IN AFOREMENTIONED ACCOUNT SINCE ACCOUNT WAS OPENED IN THE YEAR MAY-2021BALANCE IN THE ACCOUNT AS ON 23-09-2021 IS RS 72533ON REVIEWING TRANSACTION PATTERN OF BLUE PEAK SERVICES AC NO 065763400000604, IT WAS OBSERVED THAT SINCE MAY 2021, CUSTOMER HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS23 CRORES WITH MAJOR CREDITS VIA RTGS FROM DAT SOLUTION RS67 LACS, AC NO 10050733002 WITH IDFC BANK, OCN INTERNATIONAL RS11 CRORE, AC NO 10049745342 WITH IDFC BANK AND APS INTERNATIONAL RS28 LACS, AC NO 10049539859 WITH IDFC BANK ACCOUNT CREDITS WERE FOLLOWED BY DEBITS TO THE TUNE OF RS23 CRORES WITH MAJOR DEBITS PERTAINING TO RTGS IN FAVOR OF BLUE PEAK SERVICE RS85LACS AC NO 135205000765 AND 135205000743 WITH ICICI BANK LTD, APS INTERNATIONAL RS20LACS AC NO 10049539859 WITH IDFC BANK AND CROSS BORDER REMITTANCES TOWARDS DATABASE AND DATA PROCESSING CHARGES, AMOUNTING TO RS121 CRORES IN FAVOR OF OVERSEAS ENTITIES ELECTRONORD TEKNIKA PTE LTD, SINGAPORE RS27 LACS, SPACEX GLOBAL LIMITED, HONG KONG RS60LACS AND SOFT SERVE LLC, UAE RS34 LACS",{"entities":[(229,256,"FOREIGN ORG"),(261,282,"FOREIGN ORG"),(595,621,"FOREIGN ORG"),(632,652,"FOREIGN ORG"),(821,834,"FOREIGN ORG"),(875,902,"FOREIGN ORG"),(1096,1129,"FOREIGN ORG"),(982,1009,"FOREIGN ORG"),(3259,3271,"FOREIGN ORG"),(3317,3334,"FOREIGN ORG"),(3933,3946,"FOREIGN ORG"),(3887,3907,"FOREIGN ORG"),(3837,3863,"FOREIGN ORG"),(3384,3401,"FOREIGN ORG"),(3646,3663,"FOREIGN ORG")]}),
            ("ZIOFI EXPORTER, A PROPRIETORSHIP FIRM, HAVING ADDRESS: VILL, MURADPUR PO, CHAKMAHIMA PO, BELDA MURADPUR, PASCHIM BARDHAMAN, MEDINIPUR, WEST BENGAL, INDIA, 721424, CONTACT NUMBER: +919874889010 AND EMAIL ID: ZIOFIEXPORTER@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 037105007658 WITH ICICI BANK LTD, SWASTIK, GROUND FLOOR, NAZRUL ISLAM AVENUE, TEGHARIA, VIP ROAD, KOLKATA, WEST BENGAL, 700059, SINCE 04062021 DATE OF FORMATION IS 24032021 PAN NUMBER OF THE ACCOUNT IS APBPK7473B PAN LINKED ACCOUNTS ARE 037106000022, 634205002966 TRANSACTION LINKED ACCOUNTS ARE 037105007381, 333105500239 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE MOHAMMAD HAMIDULLA KHAN PAN- APBPK7473B THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- APBPK7473B KYC UPDATION IS NOT DUE AS PER ENHANCED DUE DILIGENCE, CUSTOMER IS INTO BUSINESS OF IMPORT EXPORT OF HUMAN HAIR PROPRIETOR OF FIRM IS MOHAMMAD HAMIDULLA KHAN VISIT HAS BEEN CONDUCTED BY BRANCH, ADDRESS IS TRACEABLE AND BUSINESS BOARD IN NAME OF FIRM WAS ALSO SEENDECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO LESS THAN RS 5 CRORES ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31082021 WAS RS199 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 04062021 TILL 31082021 SHOWS TOTAL CREDIT RS 90380 LAKHS OF WHICH CLEARING RS 150 LAKHS MAJORLY FROM CHQ, NEFT RS 5150 LAKHS MAJORLY FROM PRASADAM VENKATASURESH KUMAR, HS ENTERPRISE, MUSTAFIJUR RAHAMAN, RTGS RS 2387 LAKHS MAJORLY FROM NADIA HUMAN HAIR DOT C, SEIKH NABAB ALI, MD ALOMGIR, REMITTANCE RS 77850 LAKHS MAJORLY FROM DHAKA GLOBAL PTE LTD, MU MU INTERNATIONAL TRADING P, CTY TNHH MTV TAN NAM A 059 PH, INTERNET FUND TRANSFERS RS 2266 LAKHS FROM ENLIAN SANG NGAIHTE 131401511807, MOBILE MONEY TRANSFERS RS 2497 LAKHS IN 20 INSTANCES, UPI TRANSFERS RS 081 LAKHS IN 4 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 90180 LAKHS OF WHICH RTGS RS 15501 LAKHS MAJORLY TO ANSHU ENTERPRISE, INTERNET FUND TRANSFER RS 71231 LAKHS TO KSIX CARGO INTERNATIONAL 037105007381, R H HAIR ENTERPRISE 333105500239, RAJUDDIN SK 090205025024, MOBILE MONEY TRANSFERS RS 3413 LAKHS IN 24 INSTANCES TRANSACTION PATTERN SHOWS THERE WERE HIGH VALUE CREDITS FOLLOWED BY DEBITS IN THE ACCOUNT FOR EXAMPLE, ON 13-08-2021, RS 2589 LAKHS WERE CREDITED THROUGH REMITTANCES FROM DHAKA GLOBAL PTE LTD AT SINGAPORE IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS PAYMENT FOR WASTE HUMAN HAIR AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH INTERNET FUND TRANSFERS TO RAJUDDIN SK 090205025024SIMILARLY, ON 02-08-2021, RS 3335 LAKHS WERE CREDITED THROUGH REMITTANCES FROM DHAKA GLOBAL PTE LTD AT SINGAPORE IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS PAYMENT FOR WASTE HUMAN HAIR AND ON 03-08-2021, RS 30 LAKHS WERE DEBITED THROUGH INTENET FUND TRANSFERS TO R H HAIR ENTERPRISE 333105500239, KSIX CARGO INTERNATIONAL 037105007381 SIMILAR TRANSACTIONS ARE BEING REPEATED IN THE ACCOUNT HIGH TURNOVER RS 90380 LAKHS IN THE ACCOUNTS WITH IN A VERY SHORT SPAN OF TIME 3 MONTHS AND HENCE SEEM TO BE SUSPICIOUS IN NATURE IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT AND HIGH TURNOVER IN THE ACCOUNT WITHIN SPAN OF 3 MONTHS, STR IS BEING FILED",{"entities":[(1754,1773,"FOREIGN ORG"),(1776,1804,"FOREIGN ORG"),(2488,2508,"FOREIGN ORG"),(2776,2796,"FOREIGN ORG")  ]}),
            ("ON 20-08-2021 ACCOUNT WAS CREDITED WITH RS 148 LAKHS THROUGH REMITTANCE FROM XUCHANG SUPERLINE HAIR PRODUCTS IN USD AND ON 20-08-2021 AND 21-08-2021 RS 57 5 LAKHS WAS DEBITED THROUGH TRANSFER TO SEKH JAMIUL 090305500918, KSIX CARGO INTERNATIONAL 037105007381, ANAN ENTERPRISE 090205024961 SIMILARLY, ON 17-08-2021 ACCOUNT WAS CREDITED WITH RS 200 LAKHS THROUGH REMITTANCE FROM XUCHANG RUIJIATE HAIR INDUSTRIES AND THE SAME DAY RS 175 LAKHS WAS DEBITED THROUGH TRANSFER TO S S ENTERPRISE 214905000085, HMS EXPORTS 090205022116, H S EXPORTS 090205024285 SIMILAR TYPE OF TRANSACTIONS ARE OBSERVED IN THE ACCOUNT ENTITY IS ENGAGED IN EXPORT IMPORT BUSINESS, SHIPMENTS MADE AND 67 EXPORT BILLS PENDING FOR SUBMISSION WHICH IS OVERDUE AS PER TRANSACTION PATTERN IT IS OBSERVED THAT TRANSACTIONS IN THE ACCOUNT ARE WITH MULTIPLE ENTITIES, INDIVIDUALS THROUGH TRANSFER, RTGS, WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT WHICH SEEMS TO BE UNUSUAL ALSO IT IS OBSERVED THAT UNUSUAL INWARD REMITTANCE WERE SEEN IN THE ACCOUNT IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT, ABNORMAL INWARD REMITTANCE AND HIGH ANNUAL TURNOVER MAKES IT SUSPICIOUS HENCE, STR IS BEING FILEDKSIX INTERNATIONAL, A PROPRIETORSHIP FIRM, HAVING ADDRESS: KATRA 160 JAMUNALAL, PUNJABI, BAJAJ STREET, KOLKATA, WEST BENGAL 700007, CONTACT NUMBER: +917872244569 AND EMAIL ID: KSIXINTERNATIONAL@GMAIL COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 694905501585 WITH ICICI BANK LTD, CATHEDRAL ANNEXE, GROUND FLOOR, 15 PORTUGUESE CHURCH STREET, KOLKATA, WEST BENGAL, 700001, SINCE 20012021 DATE OF INCORPORATION IS 05012021 PAN NUMBER OF THE ACCOUNT IS BGDPK2879K IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS BGDPK2879K GSTIN AS PER BANK RECORDS IS 19BGDPK2879K1ZU OTHER RELATIONSHIP OF THE CUSTOMER IS 694906000034 TRANSACTION LINKED ACCOUNTS ARE 694905602501, 090205022116 AUTHORIZED SIGNATORIES/PROPRIETOR OF THE REPORTED ACCOUNT ARE ASFAK KHAN PAN- BGDPK2879K BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ASFAK KHAN THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- BGDPK2879K, GST-19BGDPK2879K1ZU AS PER EDD, ENTITY IS INVOLVED INTO HUMAN HAIR, DRESSED, THINNED, BLEACHED OR OTHERWISE WORKED, WOOL OR OTHER ANIMAL HAIR OR OTHER TEXTILE MATERIALS, PREPARED FOR USE IN MAKING WIGS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 5CR TO LESS THAN RS 25CR ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31082021 WAS RS 0 22 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31082021 SHOWS TOTAL CREDIT RS 8, 155 62 LAKHS OF NEFT RS 35 23 LAKHS MAJORLY FROM JULIE THENKHOCHIN TOUTHANG, CBEC ICEGATE, NEMNEIHCHING, RTGS RS 40 00 LAKHS MAJORLY FROM A VENKANNA, HUMAN HAIR ENTERPRISES, REMITTANCE RS 8, 038 36 LAKHS MAJORLY FROM 1 XUCHANG HUACAI HAIR PRODUCT, DHAKA GLOBAL PTE LTD 89 SHORT, QINGDAO YUANZHEN HAIR PRODUCT, INTERNET FUND TRANSFERS RS 30 00 LAKHS MAJORLY FROM ABD OASIS IMPEX 694905602501 AND HMS EXPORTS 090205022116, MOBILE MONEY TRANSFERS RS 0 03 LAKHS IN 2 INSTANCES, FIXED DEPOSIT RS 12 01 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 8, 259 65 LAKHS OF TRANSFERS RS 494 00 LAKHS MAJORLY TO MAINUL HAIR COMPANY, HMS EXPORTS, S K HABIB HAIR ENTERPRISE, RTGS RS 2, 435 80 LAKHS, REMITTANCE RS 2 35 LAKHS, INTERNET FUND TRANSFER RS 5, 211 87 LAKHS MAJORLY TO ABD OASIS IMPEX 694905602501 AND HMS EXPORTS 090205022116, CARD TRANSACTIONS RS 0 43 LAKHS, MOBILE MONEY TRANSFERS RS 58 65 LAKHS IN 39 INSTANCES, AND OTHERS RS 4 54 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS, FY 20-21 FROM 20012021 TO 31032021 SHOWS TOTAL CREDIT SIDE RS 341 21 LAKHS OF CLEARING RS 0 10 LAKHS MAJORLY FROM ASFAQ KHAN, NEFT RS 0 02 LAKHS MAJORLY FROM CBEC ICEGATE, REMITTANCE RS 341 08 LAKHS MAJORLY FROM IPAYLINKS LIMITED UNIT A 19 F, HONGKONG GRACE HOUSEWARES PRO, HENAN SHENLONG HAIR PRODUCTS TOTAL DEBIT SIDE RS 236 97 LAKHS, TRANSFERS RS 20 00 LAKHS MAJORLY TO ABD OASIS IMPEX, RTGS RS 103 44 LAKHS MAJORLY TO SK MEHABUB, SK SABIR A, MUJIBAR IN, REMITTANCE RS 0 10 LAKHS, INTERNET FUND TRANSFER RS 113 41 LAKHS MAJORLY TO ABD OASIS IMPEX 694905602501 AND HMS EXPORTS 090205022116, AND OTHERS RS 885 00 AS PER TRANSACTION PATTERN IT IS OBSERVED THAT TRANSACTIONS IN THE ACCOUNT ARE WITH MULTIPLE ENTITIES, INDIVIDUALS THROUGH TRANSFER, RTGS, WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT WHICH SEEMS TO BE UNUSUAL",{"entities":[(377,409,"FOREIGN ORG")   ,(2992,3012,"FOREIGN ORG") ,(3023,3039,"FOREIGN ORG"),(2963,2977,"FOREIGN ORG"),(3106,3121,"FOREIGN ORG")   ,(3511,3526,"FOREIGN ORG"),(3544,3555,"FOREIGN ORG"),(3956,3973,"FOREIGN ORG"),(3987,4015,"FOREIGN ORG"),(4018,4032,"FOREIGN ORG"),(4117,4131,"FOREIGN ORG"),(4278,4293,"FOREIGN ORG")]}),
            ("ROUTING OF FUNDS SEEN IN THE ACCOUNT FOR EXAMPLE, ON 31-07-2021, RS 18 LAKHS WERE CREDITED THROUGH RTGS FROM MAA SHEETLA IMPEX INDIA ACCOUNT NO: 39076770144, IFSC CODE-SBIN0000731, BANK: SBI AND ON SAME DAY RS 1750 LAKHS WERE DEBITED THROUGH INTENET FUND TRANSFER TO WINGMANN ENTERPRISE 102805500432- MAHENDRA MOHAN ARYA PAN- AIVPA5618F IS A PROPRIETOR DUE TO HIGH VALUE FUNDS BEING ROUTED THROUGH THIS ACCOUNT, TO LOCAL TRADERS AND TO ABROAD TO HONG KONG AND SINGAPORE, LOOKS LIKE THIS ACCOUNT IS BEING MISUSED FOR ROUTING PURPOSE ONLY AND ALSO THE CUSTOMERS NATURE OF BUSINESS AND ACTIVITY IS NOT LOOKING INLINE, ALSO SETUP AND TURNOVER IS NOT MATCHING TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2021-22 WAS RS 1,67132 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDBLLUEFIN ENTERPRISE, A PROPRIETORSHIP FIRM, HAVING ADDRESS: 814, HUBTOWN VIVA, SHANKARWADI, JOGESHWARI EAST, MUMBAI, MAHARASHTRA, INDIA, 400060, CONTACT NUMBER: +919152093213 AND EMAIL ID: BLLUEFINEENTERPRISE@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 777705961983 WITH ICICI BANK LTD, UNIT NO 02A2, HALLMARK BUSINESS PLAZA, SANT DNYANESHWAR ROAD, BANDRA EAST, MUMBAI, MAHARASHTRA, 400051, SINCE 09042021 DATE OF FORMATION IS 22032021 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AIVPA5618F TRANSACTION LINKED ACCOUNTS ARE 102805500432, 123105003042 AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS MAHENDRA MOHAN ARYA PAN- AIVPA5618F THE CUSTOMER IS CATEGORIZED AS MEDIUM RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN - AIVPA5618F, AADHAAR AND IMPORT EXPORT CERTIFICATE AIVPA5618F CUSTOMER IS ENGAGED OF TRADING BUSINESS OF MATHEMATICAL CALCULATING INSTRUMENTS SUCH AS DRAFTING MACHINES, PANTOGRAPHS, PROTRACTORS, DRAWING SETS, SLIDE RULES, DISC CALCULATORS AND INSTRUMENTS FOR MEASURING LENGTH DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS 40 LAKHS ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 31082021 WAS RS3380 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 09042021 TILL 31082021 SHOWS TOTAL CREDIT RS 1,67132 LAKHS OF WHICH TRANSFERS RS 050 LAKHS MAJORLY FROM WINGMANN ENTERPRISE, NEFT RS 4600 LAKHS MAJORLY FROM MOHAN TRADERS, RTGS RS 1,39977 LAKHS MAJORLY FROM MAA SHEETLA IMPEX INDIA, MOHAN TRADERS, SUMAN ENTERPRISE, INTERNET FUND TRANSFERS RS 22084 LAKHS FROM ROBIN MITTAL 019505011371, SHIV ENTERPRISES 085105502548, PINNACLLE ENTERPRISES 123105003042, MOBILE MONEY TRANSFERS RS 320 LAKHS IN 2 INSTANCES, UPI TRANSFERS RS 100 IN 1 INSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,63752 LAKHS OF WHICH TRANSFERS RS 100 LAKH MAJORLY TO PINNACLLE ENTERPRISES, CLEARING RS 100 LAKH MAJORLY TO RBL BANK LTD BLLUEFIN ENTERPRISES, REMITTANCE RS 1,15591 LAKHS MAJORLY TO INFINITY WORLDIWDE SG PTE LTD, INFINITY WORLDWIDE SG PTE LTD, JIANGUO TRADING, INTERNET FUND TRANSFER RS 47937 LAKHS TO WINGMANN ENTERPRISE 102805500432- MAHENDRA MOHAN ARYA PAN- AIVPA5618F IS A PROPRIETOR, CARD TRANSACTIONS RS 018 LAKHS AS PER VISIT FEEDBACK, THERE WERE MULTIPLE COMPANY NAME PASTED SEEN WHILE ENTERING THE PREMISES, OFFICE IS NEARLY 300 SFT, AND CUBICLES SEEN, AND ONLY ONE PERSON AVAILABLE, WHO WAS LOOKING LIKE OFFICE BOY, NO OTHER EMPLOYEE PRESENT AT THE TIME OF VISIT, THE GUY PRESENT THERE COMMUNICATED TO COME LATER WITH PROPER APPOINTMENT TRANSACTION OVERVIEW, REGULAR HIGH VALUE FUNDS ARE BEING CREDITED THROUGH RTGS OUT OF THOSE ALMOST ALL THE FUNDS ARE BEING DEBITED TO ABROAD THROUGH REMITTANCE TO INFINITY WORLDIWDE-SINGAPORE IN USD CURRENCY FOR PAYMENT TOWARDS TRADE FAIR SERVICE, JIANGUO TRADING- HONG KONG IN USD CURRENCY FOR PAYMENT TOWARDS ADVERTISING, TRADE FAIR SERVICE, REGULARLY IN HIGH VALUE FUNDS, FOR EXAMPLE ON 26-08-2021, RS366LACS CREDITED THROUGH INTENET FUND TRANSFER FROM PINNACLLE ENTERPRISE, [ACCOUNT-123105003042 IN THIS ACCOUNT FUNDS WERE CREDITED THROUGH RTGS FROM SHIV ENTERPRISES, ACCOUNT NO: 4729005500000370, IFSC CODE-PUNB0472900, BANK: PNB IN PINNACLLE ENTERPRISE, PROPRIETOR IS REETU MAHENDRA ARYA PAN: AIVPA5620M ], THROUGH RTGS FROM SUMAN ENTERPRISE, ACCOUNT- 510909010041973 IFSC- CIUB0000499, BANK: CITY UNION BANK LTD, OUT OF WHICH RS3661LACS DEBITED THROUGH REMITTANCE TO JIANGUO TRADING- HONG KONG IN USD CURRENCY FOR PAYMENT TOWARDS ADVERTISING, TRADE FAIR SERVICE",{"entities":[(869,887,"FOREIGN ORG")      ,(2961,2980,"FOREIGN ORG"),(3022,3050,"FOREIGN ORG"),(3053,3081,"FOREIGN ORG"),(3084,3098,"FOREIGN ORG") ,(3750,3767,"FOREIGN ORG"),(3815,3832,"FOREIGN ORG"),(3835,3850,"FOREIGN ORG")    ,(4461, 4476,"FOREIGN ORG")]}),
            ("DEBITS AGGREGATED TO RS25861 CRORES IN 435 TRANCHES COMPRISING OF: RS11578 CRORES IN 19 TRANCHES TOWARD IMPORT BILL PAYMENTS AND IMPORT BILL CHARGES NEFT OF RS8010 CRORES IN 3 TRANCHES TOWARDS CUSTOMERS OWN ACCOUNT 10324773582 HELD WITH SBI AND ACCOUNT 00030330017866 HELD WITH HDFC OUTWARD REMITTANCE OF RS1435 CRORES IN 3 TRANCHES TO HSBC SINGAPORE ACCOUNT 000038865 TOWARDS REPAYMENT OF INTEREST AND PRINCIPAL OF EXTERNAL COMMERCIAL BORROWINGS RS3720 LAKHS IN 10 TRANCHES TOWARDS REPAYMENT OF LOAN FACILITIES AVAILED FROM HSBC OUTWARD REMITTANCE OF RS186 CRORES IN 2 TRANCHES TO HSBC BANK MAURITIUS LTD TOWARDS INTEREST ON EXTERNAL COMMERCIAL BORROWINGS OUTWARD REMITTANCE OF RS803 LAKHS IN 2 TRANCHES TO PRECURSOR GROUP PTE LTD 629160177001, OVERSEA-CHINESE BANKING CORP LTD WITH THE NARRATION AS ACCOUNTING, AUDITING AND BOOK KEEPING2 REVIEW OF TRADE TRANSACTIONS NO IMPORTS OR EXPORTS OBSERVED DURING THE REVIEW PERIOD TOTAL IDPMS OUTSTANDING IS RS10220 CRORES IN 56 TRANCHES OF WHICH RS3841 CRORES IN 30 TRANCHES PERTAINS TO THE YEAR 2016, RS 189 CRORES IN 2 TRANCHES PERTAINS TO THE YEAR 2017 AND RS409 CRORES IN 3 TRANCHES PERTAINS TO THE YEAR 2020, RS5824 CRORES IN 18 TRANCHES PERTAINS TO THE YEAR 20213 NO TRANSACTIONS OBSERVED IN THE ESCROW ACCOUNT 166-176743-002 DURING THE REVIEW PERIOD TRANSACTIONS IN THE ACCOUNT DO NOT INDICATE ANY UNUSUAL PATTERN HOWEVER, THE NOTICE RECEIVED FROM CBI, APPEARS TO BE IN THE SAME CASE PERTAINING TO THE TERMINAL EXCISE DUTY REFUNDS CASE FOR WHICH NOTICES WERE RECEIVED EARLIER FROM OTHER AUTHORITIES ACCORDINGLY, A SUPPLEMENTARY STR IS BEING FILEDCRYSTAL CROP PROTECTION LTD CCPL PAN: AABCJ3574E, DATE OF INCORPORATION: 13071994 HOLDS BELOW ACCOUNTS WITH THE NEW DELHI BRANCH - CURRENT ACCOUNT 166-176743-001, SINCE 20012011 ESCROW ACCOUNT 166-176743-002 SINCE 06102016 MULTIPLE TERM DEPOSITS AGGREGATING TO RS483 CRORES LOAN ACCOUNT 041-024134-700 FOR USD 50 LAKH WORKING CAPITAL LOAN OF RS123 CRORESITS DIRECTORS ARE MR MOHIT KUMAR GOEL, MR ANIL JAIN, MR SARTAJ SEWA SINGH, MR CHETAN RAMESHCHANDRA DESAI, MRS SANGEETA KAPIL JIT SINGH ITS KEY CONTROLLERS ARE MR NAND KISHORE AGGARWAL, MR ANKUR AGGARWAL, MR SUMEET SOOD, MR DINESH KUMAR GUPTA AND MR PIYUSH JAIN AND THE BENEFICIAL OWNER IS MR KANAK AGGARWAL THE REGISTERED ADDRESS IS B-95, WAZIPUR INDUSTRIAL AREA, DELHI, INDIA 110052 AND CONTACT NUMBERS ARE M 9958793104 AND O 011-27006800/11 COMPANYS TURNOVER FOR FINANCIAL YEAR 2019-2020 WAS RS1,450 CRORESSTRS WERE PREVIOUSLY FILED ON CCPL, AS UNDER: 25012018 VIDE FIU REFERENCE NUMBER 1801255099, IN VIEW OF MULTIPLE DISCREPANCIES IN REGULARIZATION OF EXPORT BILLS OF 2014 AND SIGNIFICANT VALUE OF OLD EXPORT BILLS PENDING FOR REGULARIZATION 03062020 VIDE FIU REFERENCE NUMBER 2006035602, IN VIEW OF ADVERSE MEDIA REPORTS REGARDING INVESTIGATION BY CBI AGAINST MR A K SINGH, THE THEN JOINT DIRECTOR GENERAL OF FOREIGN TRADE, AHMEDABAD AND CCPL AND ITS DIRECTORS IN THE MATTER OF FRAUDULENT CLAIM OF TERMINAL EXCISE DUTY BENEFITS THAT CAUSED WRONGFUL LOSS TO THE GOVERNMENT OF INDIA AND IN VIEW OF HIGH VALUE OF OLD OUTSTANDING IMPORT BILLS 02022021 VIDE FIU REFERENCE NUMBER 2102029851, IN VIEW OF FRESH NEGATIVE NEWS REGARDING CASE FILED BY ED UNDER PMLA IN RELATION TO ALLEGED FAKE REFUND CLAIMS OF TERMINAL EXCISE DUTYALERT WAS GENERATED ON CCPL SINCE BANK HAS RECEIVED A NOTICE ISSUED BY SHRI S S BHADORIYA, DY SUPDT OF POLICE, CBI-ACB, GANDHINAGAR, DATED 29052021, IN CONNECTION TO A CASE, REGISTERED AGAINST SHRI A K SINGH FORMER JOINT DIRECTOR, DGFT, AHMEDABAD, CCPL, ITS DIRECTORS AND UNKNOWN OTHERS, U/S 120B R/W 420 OF IPC AND SEC 132 R/W SEC 131D OF THE PREVENTION OF CORRUPTION ACT, 1988 ACCORDINGLY, ACCOUNTS WERE REVIEWED FROM 20012021 TO 02092021, WHICH HIGHLIGHTED THE FOLLOWING - 1 IN CURRENT ACCOUNT NO 166-176743-001? CREDITS AGGREGATED TO RS26808 CRORES IN 63 TRANCHES COMPRISING OF: NEFT/RTGS OF RS18162 CRORES IN 49 TRANCHES FROM OWN ACCOUNT 00030330017866 HELD WITH HDFC INWARD REMITTANCE OF RS3712 CRORES FROM CUSTOMERS NOSTRO ACCOUNT AS SURPLUS FUNDS TRANSFERRED LOAN DISBURSAL OF RS3049 CRORES IN 10 TRANCHES AND RS4945 LAKHS IN 9 TRANCHES AS DRAWDOWN FROM 166-176743-451, 166-176743-452, 166-176743-453 AND 166-176743-454 INWARD REMITTANCE OF RS1784 CRORES FROM LOTUS GLOBAL PTE LTDS ACCOUNT 054361452 HELD WITH HSBC, SINGAPORE INWARD REMITTANCE OF RS9225 LAKHS ON 22042021 FROM HEBEI BESTAR COMMERCE AND TRADE ACCOUNT: 50354014040000583, AGRICULTURAL BANK OF CHINA WITH THE NARRATION AS PAYMENT FOR CONSULTING FEE INWARD REMITTANCE OF RS739 LAKHS ON 04082021 FROM BEIJING LUCKYSTAR CO LTD ACCOUNT: 77012025000010095, BANK OF NINGBO CO LTD, CHINA",{"entities":[(708,731,"FOREIGN ORG"),(746,778,"FOREIGN ORG") ,(4245,4266,"FOREIGN ORG"),(4362,4393,"FOREIGN ORG"),(4422,4448,"FOREIGN ORG"),(4548,4572,"FOREIGN ORG"),(4601,4621,"FOREIGN ORG")]}),
            ("CUSTOMER KYC AND ACCOUNT DETAILS: 1 CUSTOMER NAME : STEER ENGINEERING PVT LTD AND CUSTOMER ADDRESS : 290, 4TH MAIN, ,4TH PHASE, ,PEENYA IND ESTATE , ,KARNATAKA,560058 INDIA 2 DATE OF BIRTH / DATE OF INCORPORATION : 29-03-1993 3 PAN NO : AABCS8840E 4 PASSPORT NO : 5 OTHER IDENTIFICATION DOCUMENTS : CIN -U05190KA1993PTC015303 6 IEC CODE : 7 GSTIN NO:29AABCS8840E1Z7 8 MOBILE NUMBER/LANDLINE NO : 9900077042 9 OTHER FACILITIES AVAILED: 10 RISK CATEGORY : MEDIUM RISK 11 NAME OF THE AUTHORISED SIGNING AUTHORITIES: 1 SATISH PADMANABHAN 2 UTTAM KUMAR 12 KYC COMPLIANCE STATUS: YES 13 DATE OF LAST KYC UPDATION: 27062018 CUSTOMER PROFILE : 14 NATURE OF BUSINESS : FISHING, OPERATION OF FISH HATCHERIES AND FISH FAR 15 OCCUPATION : 16 ANNUAL INCOME AS DECLARED IN KYC : 17 ANNUAL TURNOVER AS DECLARED IN KYC :RS1000 LACS 18 BENEFICIAL OWNERS AS DETERMINED UNDER RULE 93 OF PMLA RULES: 1 SATISH PADMANABHAN 2 UTTAM KUMAR CIF OF BENEFIACIARY OWNER IS NOT AVAILABLE IN THE CBS ACCOUNT DETAILS : 19 BANK AND BRANCH DETAILS : STATE BANK OF INDIA , SME BRANCH PEENYA IND ESTATE ,NEAR PEENYA POLICE STATION, 560058 20 BANK ACCOUNT NO : 00000035333802430 21 BANK ACCOUNT TYPE : STDR 22 BANK A/C OPEN ON: 09-11-2015 23 BANK ACCOUNT STATUS : CLOSEDTRANSACTION DETAIL :SR NO BILL REF NO COLLECTION AMOUNT IN FC DRAWEE NAME INVOICE NO SHIPMENT DATE1 0505316GC1001768 USD 11000000 STEER JAPAN CORPORATION 100882 2/19/20162 0505316GC1001054 USD 27611600 A SCHULMAN PLASTICS DONGGUAN LTD 100837 1/30/2016 TRANSACTION SUMMARY : 24 QUANTUM OF TRANSACTIONS FOR THE CURRENT YEAR : FROM01-04-2021 TO30-09-2021 TOTAL CREDIT RS 0 AND TOTAL DEBIT RS 0 AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 25 QUANTUM OF TRANSACTIONS FOR PREVIOUS YEARS : FROM 01-04-2020 TO 31-03-2021 TOTAL CREDIT RS 0 AND TOTAL DEBIT RS 0 AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 FROM 01-04-2019 TO 31-03-2020 TOTAL CREDIT RS 0 AND TOTAL DEBIT RS 0 AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 FROM 01-04-2018 TO 31-03-2019 TOTAL CREDIT RS 0 AND TOTAL DEBIT RS 0 AND TOTAL CASH CREDIT RS 0 AND TOTAL CASH DEBIT RS 0 26 BALANCE IN ACCOUNT ON DATE OF FILING STR : 0 HISTORY : 27 DETAILS OF EARLIER STRS FILED ON THE ACCOUNT OR ACCOUNT HOLDERS/RELATED PERSONS: NILGROUND OF SUSPICION:STEER ENGINEERING PRIVATE LIMITED INCORPORATED ON 29 MARCH 1993 AND IS REGISTERED AT REGISTRAR OF COMPANIES, BANGALORE ITS CIN IS U05190KA1993PTC015303, AND ITS REGISTRATION NUMBER IS 15303 IT IS INVOLVED IN FISHING, OPERATION OF FISH HATCHERIES AND FISH FARMS; SERVICE ACTIVITIES INCIDENTAL TO FISHING DIRECTORS OF STEER ENGINEERING PRIVATE LIMITED ARE SATISH PADMANABHAN, PADMANABHAN RAMASWAMY, SATISH PREETHI, CHITHRA BABU, BABU PADMANABHANCOMPANY HAS EXPORTED SOME GOODS AND SUBMITTED THE DOCUMENTS THROUGH BANK HOWEVER PAYMENT OF THE BILL IS STILL PENDING DETAILS ARE GIVEN IN TRANSACTION DETAILS FIELDS IT IS OBLIGATORY ON THE PART OF THE EXPORTER TO REALIZE AND REPATRIATE THE FULL VALUE OF GOODS / SOFTWARE / SERVICES TO INDIA WITHIN NINE MONTHS FROM THE DATE OF EXPORT TOTAL VALUE OF THE OUTSTANDING BILLS IN INR IS APROX RS 8256 LAKH AS THE GOODS HAS BEEN EXPORTED BUT PAYMENT NOT RECEIVED THEREFORE, IT SEEMS TO BE SUSPICIOUS AS PER THE TBML ANGLE HENCE STR HAS BEEN ESCALATED",{"entities":[(1363,1386,"FOREIGN ORG"),(1437,1467,"FOREIGN ORG")  ]}),
            ("FOR EXAMPLE CUSTOMER HAS UNDERTAKEN OUTWARD REMITTANCE ON 03092021 FOR AN AMOUNT OF AED 1,21,110 TOWARDS IMPORT OF BITUMINUS TO KING STONE BUILDING HARDWARE AND TOOLS TRADING LLC BUT THE PAYMENT WAS REJECTED BY EMIRATES BANK AS STATED ABOVE AND SIMILAR 4 OTHER TRANSACTIONS WERE ALSO OBSERVED CUSTOMER IS IMPORTING GOODS FROM A FOREIGN COUNTERPARTY WHICH SEEMS TO BE NOT IN SAME LINE OF BUSINESS ALSO CUSTOMER HAS MADE AN DIRECT IMPORT PAYMENT TOWARDS IMPORTING OF BITUMINUS FROM WESTON DMCC, ON PUBLIC DOMAIN DATA SEARCH, THE COMPANY IS FOUND TO BE ENGAGED IN BUSINESS OF DIAMOND, PRECIOUS STONES AND GEMS TRADE PUBLIC DOMAIN LINK: HTTPS://LOCALINFOBELAE/AE100428963/WESTONDMCC-DUBAIHTML ON PUBLIC DOMAIN DATA SEARCH, IT WAS OBSERVED THAT IRAN IS LARGEST EXPORTER OF BITUMEN WHICH IS SMUGGLED INTO UAE, THEN PACKAGED/LABELED THEREIN AND SOLD TO OTHER COUNTRIES THIS APPROACH IS FOLLOWED TO MAINLY AVOID TRADE SANCTIONS IMPOSED BY UNITED STATES REFERENCE PUBLIC DOMAIN DATA SEARCH LINKS: HTTPS://WWWDAILY-SUNCOM/POST/554406/MYSTERY-OVER-THE-ORIGIN-OF-IMPORTED-BITUMEN-, HTTPS://WWWDAILY-SUNCOM/POST/560125/IRANIAN-BITUMEN-ARRIVES-ON-IRANIAN-SHIPS- , HTTPS://WWWARGUSMEDIACOM/EN/NEWS/2236606-IRANS-JUNE-BITUMEN-EXPORTS-RISE-ON-FIRMER-DEMAND THERE ARE 35 BOES PENDING FOR THE CUSTOMER HAVING AGEING OF 5 DAYS TO 233 DAYS 6 OUT OF 35 BILLS, ARE CURRENTLY UNDER OVERDUE STATUS AMOUNTING TO AED: 14,06,149 THE COMMODITY INVOLVED IN THE UNDERLYING TRANSACTION IS BITUMEN GRADE VG40 CUSTOMER WAS IMPORTING BITUMEN FROM THE SAID UAE FROM COUNTER PARTIES LIKE DOA INTERNATIONAL DMCC, M/S WESTON DMCC, HARWICH GLOBAL DMCC AND M/S MOTOTIVE GENERAL TRADING LLC FURTHER, IT WAS OBSERVED A TOTAL 63 BILL OF IMPORT MADE BUT PAYMENT PENDING AGAINST FOREIGN PARTIES M/S CITY PULSE GENERAL TRADING LL, M/S WESTON DMCC AND OTHERS THE OUTSTANDING OF THESE PENDING BILLS IS AED 11,94,21300 AND USD 49779798 AND AGEING OF THESE BILLS ARE FROM 55 DAYS TO 580 DAYS 20 BILLS OUT OF THESE 63 BILLS HAVE AGEING BEYOND ONE YEAR IT WAS NOTICED THAT TOTAL TURNOVER IN THE ACCOUNT IS FAR EXCEEDING THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING IE RS 40 LACS TO RS 5 CR IN VIEW OF CUSTOMER DOING HIGH VALUE OUTWARD REMITTANCES TOWARDS IMPORTING OF BITUMINUS FROM UAE, IMPORT TRANSACTIONS TREATED AS NULL AND VOID BY EMIRATES BANK DUE TO THEIR INTERNAL POLICY, PENDING BILLS AND TURNOVER FAR EXCEEDING THE DECLARED TURNOVER, STR IS BEING REPORTEDCATALYST PETROCHEM LLP, A LIMITED LIABILITY PARTNERSHIP FIRM, HAVING ADDRESS: HOUSE NO 197 KD BLOCK, PITAMPURA, NORTH WEST DELHI, NEW DELHI, INDIA, 110034, CONTACT NUMBER: +919871294951 AND EMAIL ID: ACCOUNTS@CATALYSTPETROCHEMLLPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 015405006903 WITH ICICI BANK LTD, RP I PITAMPURA, NEW DELHI, 110034, SINCE 06032020 DATE OF INCORPORATION IS 17012017 PAN OF THE ACCOUNT IS AALFC9514G IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0517501902 GSTIN AS PER BANK RECORDS IS 07AALFC9514G1ZC TRANSACTION LINKED ACCOUNTS ARE 104105500420 - VARDHMAN TRADING CO, 246905500252 - GOYAL AND COMPANY BENEFICIAL OWNERS/ AUTHORISED SIGNATORIES/ PARTNERS OF THE REPORTED ACCOUNT ARE NIPUN GOYAL WITH PAN NO ANVPG5288N, SANJAY GOYAL WITH PAN NO AAOPG1061D THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AND GSTIN KYC UPDATION IS NOT DUE COMPANY IS ENGAGED IN THE BUSINESS OF MANUFACTURING, SUPPLIER OF BITUMEN CUSTOMER IS ENGAGED IN MANUFACTURING AND SUPPLYING A WIDE ASSORTMENT OF INDUSTRIAL OILS AND METALS AS PER CROSS BORDER TRANSACTIONS, CUSTOMER DEALS IN GOODS AND SERVICES OF HSN 27132000PETROLEUM COKE, PETROLEUM BITUMEN AND OTHER RESIDUES OF PETROLEUM OILS OR OF OILS OBTAINED FROM BITUMINOUS MINERALS - PETROLEUM BITUMEN DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAC TO RS 5 CR ALERT WAS TRIGGERED FOR HIGH OUTWARDS REMITTANCE IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 27092021 WAS RS527 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 27092021 SHOWS TOTAL CREDIT RS 6,35000 LAKHS OF WHICH TRANSFERS RS 7456 LAKHS MAJORLY FROM ANEJA TRADERS, GHANSHAYMDAS IMPEX, RADHA DEVI, CLEARING RS 887 LAKHS MAJORLY FROM NEEL KAMAL RUBBER P LTD, NEFT RS 35477 LAKHS MAJORLY FROM RABHYA PETRO CHEM LLP, SHUBHAM ENTERPRISES, AGGARWAL TRADERS PARTNERSHIP FIRM, RTGS RS 5,04919 LAKHS MAJORLY FROM CATALYST PETROCHEM LLP, SHREE JI PETROCHEM, REMITTANCE RS 8251 LAKHS MAJORLY FROM ICICI BANK LTD, INTERNET FUND TRANSFERS RS 77772 LAKHS FROM TEJAS ENERGY RESOURCES ETC TOTAL DEBITS DURING THE SAME PERIOD WAS RS 6,42284 LAKHS OF WHICH RTGS RS 3,18849 LAKHS, REMITTANCE RS 2,10709 LAKHS MAJORLY TO OULA ALNUHA INTERNATIONAL DMCC, HARWICH GLOBAL DMCC, MOTOTIVE GENERAL TRADING LLC, INTERNET FUND TRANSFER RS 1,03014 LAKHS TO GOYAL AND COMPANY ETC, MOBILE MONEY TRANSFERS RS 1324 LAKHS IN 11 INSTANCES AND OTHERS RS 7624 LAKHS FY 2020-21 FROM 01042020 TO 31032021 : TOTAL CREDIT SIDE RS 9,42629 LAKHS OF WHICH TRANSFERS RS 3520 LAKHS, CLEARING RS 11884 LAKHS MAJORLY FROM BHAGWATI BITUMEN, IMMORTAL TYRES, NEFT RS 71678 LAKHS MAJORLY FROM STAR BITUMEN, GWC ASPHALT, CATALYST PETROCHEM LLP, RTGS RS 7,44333 LAKHS MAJORLY FROM J P AND SONS, CATALYST PETROCHEM LLP, REMITTANCE RS 4111 LAKHS, INTERNET FUND TRANSFERS RS 94809 LAKHS FROM TEJAS ENERGY RESOURCES 023505007619 ETC, MOBILE MONEY TRANSFERS RS 2166 LAKHS IN 21 INSTANCES AND OTHERS RS 10128 LAKHS TOTAL DEBIT SIDE RS 9,43824 LAKHS OF WHICH CASH WAS RS 10900 LAKHS BY BRANCH, RTGS RS 2,40011 LAKHS MAJORLY TO SANJAY GOYAL, REMITTANCE RS 6,01407 LAKHS, INTERNET FUND TRANSFER RS 70198 LAKHS TO GOYAL AND COMPANY ETC, MOBILE MONEY TRANSFERS RS 9537 LAKHS IN 72 INSTANCES AS PER TRANSACTION PATTERN, HIGH VALUE REMITTANCES ARE SEEN IN A RECENTLY OPENED ACCOUNT CUSTOMER IS UNDERTAKING OUTWARD REMITTANCE TRANSACTIONS FOR IMPORT OF BITUMINUS FROM UAE IT WAS OBSERVED THAT CUSTOMER HAS MADE 5 ADVANCE PAYMENT AGAINST IMPORT OF BITUMINUS TO KING STONE BUILDING HARDWARE TOOLS TRADING LLC THROUGH EMIRATES BANK IN AED CURRENCY AND ALL 5 TRANSACTIONS WERE TREATED AS NULL AND VOID DUE TO INTERNAL POLICY OF EMIRATES BANK INTERMEDIARY BANK",{"entities":[(1551,1572,"FOREIGN ORG"),(1579,1589,"FOREIGN ORG"),(1592,1611,"FOREIGN ORG"),(1620,1648,"FOREIGN ORG"),(1758,1781,"FOREIGN ORG"),(1788,1799,"FOREIGN ORG")    ,(4751,4780,"FOREIGN ORG"),(4783,4801,"FOREIGN ORG"),(4804,4831,"FOREIGN ORG")      ,(6042,6087,"FOREIGN ORG")]}),
            ("FY 2020-21 FROM 10082020 TO 31032021 : TOTAL CREDIT SIDE RS 8,23080 LAKHS OF WHICH CASH WAS RS 100 LAKH, CLEARING RS 59400 LAKHS, NEFT RS 36431 LAKHS MAJORLY FROM BLACK DIAMOD EXPLOSIVES P,VINAYAK OIL AND FATS PVT LTD,VINAYAK OIL FATS PVT LTD, RTGS RS 7,24172 LAKHS MAJORLY FROM M/S VINAYAK OIL AND FA,VINAYAK OIL FATS PVT L,VINAYAK OIL FATS PVT, INTERNET FUND TRANSFERS RS 378 LAKHS, MOBILE MONEY TRANSFERS RS 2600 LAKHS IN 13 INSTANCES , TOTAL DEBIT SIDE RS 8,21830 LAKHS OF WHICH CASH WAS RS 1545 LAKHS BY BRANCH, TRANSFERS RS 025 LAKHS MAJORLY TO TANUSREEDEB PAUL, CLEARING RS 59934 LAKHS MAJORLY TO SPARROWIGST311220,SPARROWSHREETRANSPO, SPARROWSHREE104122, RTGS RS 1,03253 LAKHS MAJORLY TO BLACK DIAM, MS AK INTERNATIONAL, SOWALLOW ENTERPRISE, REMITTANCE RS 6,37229 LAKHS MAJORLY TO A S ENTERPRISE, INTERNET FUND TRANSFER RS 8603 LAKHS, MOBILE MONEY TRANSFERS RS 10794 LAKHS IN 98 INSTANCES , , AND OTHERS RS 447 LAKHS TRANSACTION PATTERN SHOWS HIGH VALUE NON CASH TRANSACTIONS IN THE ACCOUNT FOR EXAMPLE: ON 09-08-2021 ACCOUNT WAS CREDITED WITH RS244 CR THROUGH RTGS FROM VINAYAK OIL AND FATS PVT LTDACCOUNT NO:00080340052425, IFSC:HDFC0000240 AND THE ON SAME DAY RS121 CR WAS DEBITED THROUGH REMITTANCE TO MIM INTERNATIONAL, BENAPOLE BAZAR JESSORE BANGLADESH, PURPOSE AS ADVANCE IMPORT PAYMENT INV NO -MISI2692021, P K INTERNATIONAL 50 PURANA POLTAN LANE DHAKA BANGLADESH PURPOSE AS ADVANCE PAYMENT AGAINST IMPORT PI NO PKI/SI/EX/06/2021 SIMILARLY ON 13-08-2021 ACCOUNT WAS CREDITED WITH RS274 CR THROUGH RTGS FROM VINAYAK OIL AND FATS PVT LTD ACCOUNT NO:00080340052425, IFSC:HDFC0000240 AND THE ON SAME DAY RS230 CR WAS DEBITED THROUGH REMITTANCE TO MAHIM ENTERPRISE BENAPOLE BAZAR SHARSHA BANGLADESH PURPOSE WAS ADVANCE PAYMENT AGAINST IMPORTS INV - ME/SI65/2021, AS ENTERPRISE BENAPOLE BAZAR BENAPOLE BANGLADESH ADVANCE PAYMENT AGAINST IMPORT INV NO: ASE/SIS/01/21-22 SIMILAR TYPE OF TRANSACTIONS WERE OBSERVED IN THE ACCOUNT WHEREIN FREQUENT CREDITS BY RTGS WERE FOLLOWED BY IMMEDIATE DEBITS BY REMITTANCE, RTGS ALSO ACCOUNT TURNOVER IS MORE THAN RS168 CR IN SHORTER DURATION IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS THE ACCOUNTS WITH HIGH ACCOUNT TURNOVER, STR IS BEING FILEDSPARROW INTERNATIONAL, PROPRIETORSHIP FIRM, HAVING ADDRESS:432/109 BABU PARA,PO GOBARDANGA,NORTH 24 PGS, GOBARDANGA, WEST BENGAL, INDIA, 743252, CONTACT NUMBER: +919143129043 AND EMAIL ID: SPARROWINTERNATIONAL2020@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 161205003385 WITH ICICI BANK LTD, COURT ROAD, BICHALIHATA, BONGAON NORTH 24 PARGANAS DIST, WEST BENGAL, 743235, SINCE 10082020 DATE OF INCORPORATION IS 30072020 PAN NUMBER OF THE ACCOUNT IS ANRPP6146J GSTIN AS PER BANK RECORDS IS 19ANRPP6146J1Z7 TRANSACTION LINKED ACCOUNTS ARE 039205003735 ,161213001927 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT IS TANUSREEDEB PAUL PROP- ANRPP6146J THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- ANRPP6146J, GSTIN KYC UPDATION IS NOT DUE AS PER WEB SEARCH, SPARROW INTERNATIONAL PROVIDE FIRST CLASS BRANDS- IN BOTH EQUIPMENT AND INGREDIENTS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40LAC TO LESS THAN 5CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS BALANCE IN THE ACCOUNT AT 31082021 WAS RS25611 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31082021 SHOWS TOTAL CREDIT RS 6,18365 LAKHS OF WHICH NIL BY CASH, NEFT RS 3048 LAKHS MAJORLY FROM VINAYAK OIL AND FATS PVT LTD,ASHIAN OILS PVT LTD, RTGS RS 6,04657 LAKHS MAJORLY FROM ASHIANOILSPVTLTD,VINAYAK OIL AND FATS P,VINAYAK OIL FATS PVT, REMITTANCE RS 10636 LAKHS MAJORLY FROM MAHIM ENTERPRISE, ASSOCIATION ROAD,, MOBILE MONEY TRANSFERS RS 024 LAKHS IN 1 INSTANCE , TOTAL DEBITS DURING THE SAME PERIOD WAS RS 5,94005 LAKHS OF WHICH CASH WAS RS 800 LAKHS BY BRANCH, CLEARING RS 007 LAKHS MAJORLY TO RUMPA DEBNATH, RTGS RS 32439 LAKHS, REMITTANCE RS 5,51053 LAKHS MAJORLY TO A S ENTERPRISE,MIM INTERNATIONAL,P K INTERNATIONAL, INTERNET FUND TRANSFER RS 3626 LAKHS, MOBILE MONEY TRANSFERS RS 4226 LAKHS IN 49 INSTANCES , , AND OTHERS RS 1854 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE:",{"entities":[(1214,1230,"FOREIGN ORG") ,(1774,1787,"FOREIGN ORG")   ,(3756,3771,"FOREIGN ORG"),(4052,4065,"FOREIGN ORG"),(4067,4083,"FOREIGN ORG"),(4085,4101,"FOREIGN ORG")]}),
            ("SIMILARLY, ON 23-08-2021, RS 6710 LAKHS WERE CREDITED INTO THE ACCOUNT THROUGH RTGS FROM SHREEJI INFOTECH ACCOUNT NUMBER:200001394029, IFSC: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK AND ON 24-08-2021, RS 6643 LAKHS WERE DEPLOYED THROUGH OUTWARD REMITTANCE TO DREAMBOX STUDIOS LTD ACCOUNT NUMBER: 57412001958, CURRENCY: USD, AMOUNT: 8785000, BANK: STANDARD CHARTERED BANK, COUNTRY: HONG KONG, PURPOSE: TWDS DATABASE AND DATA PROCESSING THE BENEFICIARY IS A NEWLY INCORPORATED ENTITY AND REMITTER HAS STARTED REMITTING TO THIS PARTICULAR BENEFICIARY ONLY ACCOUNT HAS BEEN MARKED CREDIT FREEZE ON 30-09-2021 DUE TO UNSATISFACTORY EDD FEEDBACK FROM THE BRANCH AND SHALL BE BANK INDUCED CLOSURE FOR THE SAID ACCOUNT IN VIEW OF NEGATIVE EDD FEEDBACK RECEIVED AGAINST THE CUSTOMER, HIGH TURNOVER DOESNT MATCH WITH THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER, NEGATIVE FINDINGS DURING THE VISIT AND SHALL BE FOLLOWED BY BANK INDUCED CLOSURE, STR IS BEING FILEDSERVINITY TECHNOLOGIES OPC PRIVATE LIMITED IS A ONE DIRECTOR COMPANY, HAVING ADDRESS : PLT-IID/31/1 QUBE BLK-1,FLT-205,NEWTOWN, PS-BIDHAN NAGAR EAST, KOLKATA, WEST BENGAL, INDIA, 700135, CONTACT NUMBER: +919874737578 AND EMAIL ID: SERVINITYTECHNOLOGY2019@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 330305000374 WITH ICICI BANK LTD, RAJARHAT GOPALPUR - SIDDHA PINES, OPP DEROZIO COLLEGE, RAJARHAT ROAD, KALI PARK, BABLATALA, KOLKATA, WEST BENGAL, 700136, SINCE 31122020 DATE OF INCORPORATION IS 16082019 PAN NUMBER OF THE ACCOUNT IS ABCCS5072N TRANSACTION LINKED ACCOUNTS ARE 330305000375 AND 127201504114 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS RAJNEESH KUMAR BINE AUTHORIZED SIGNATORY- ANUPB0407D BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS AJIT KUMAR RANA DIRECTOR - AHOPR2008Q THE CUSTOMER IS CATEGORIZED AS LOW RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CREDIT FREEZE ON 30-09-2021 KYC DOCUMENTS OBTAINED WERE IDENTIFICATION PROOF AS PAN - ABCCS5072N, CERTIFICATE OF INCORPORATION WITH CIN: U72900WB2019OPC233460, A COPY OF ARTICLES, MEMORANDUM OF ASSOCIATION KYC UPDATION IS NOT DUE CUSTOMER PROFILE IS INTO BUSINESS OF DEALING IN COMPUTER AND RELATED ACTIVITIES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40 LAKHS TO 5 CRALERT WAS TRIGGERED FOR NEGATIVE EDD FEEDBACK RECEIVED AGAINST THE CUSTOMER BALANCE IN THE ACCOUNT AT 31082021 WAS RS 2675 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31082021 SHOWS TOTAL CREDIT RS 88926 LAKHS OF WHICH TRANSFERS RS 200 LAKHS MAJORLY FROM ABUL KASHEM MOLLA, SERVINITY TECHNOLOGIES OPC PRIVATE LIMITED, NEFT RS 879 LAKHS MAJORLY FROM ZEROPLUS DESIGNING SOLUTIONS LLP, RAJNEESH KUMAR BINE, RTGS RS 87122 LAKHS MAJORLY FROM SHREEJI INFOTECH, MOBILE MONEY TRANSFERS RS 537 LAKHS IN 15 INSTANCES, UPI TRANSFERS RS 188 LAKHS IN 13 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 86251 LAKHS OF WHICH CASH WAS RS 855 LAKHS BY ATM, BRANCH, RTGS RS 850 LAKHS MAJORLY TO INDB0000850, REMITTANCE RS 83025 LAKHS MAJORLY TO INFINITY DIGITAL SOLUTIONS, DREAMBOX STUDIOS LIMITED, INTERNET FUND TRANSFER RS 857 LAKHS, MOBILE MONEY TRANSFERS RS 604 LAKHS IN 39 INSTANCESFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY 2020-21 FROM 31122020 TO 31032021: TOTAL CREDIT SIDE RS 051 LAKHS OF WHICH UPI TRANSFERS RS 051 LAKHS IN 3 INSTANCES TOTAL DEBIT SIDE RS 051 LAKHS OF WHICH MOBILE MONEY TRANSFERS RS 051 LAKHS IN 4 INSTANCESON ANALYZING THE TRANSACTION PATTERN OF THE ACCOUNT HIGH VALUE CREDITS ARE RECEIVED INTO THE ACCOUNT THROUGH RTGS AND ON THE SAME DAY OR THE NEXT DAY THE HIGH VALUE FUNDS ARE DEPLOYED THROUGH REMITTANCE FOR EXAMPLE, ON 27-08-2021, RS 6545 LAKHS WERE CREDITED INTO THE ACCOUNT THROUGH RTGS FROM SHREEJI INFOTECH ACCOUNT NUMBER:200001394029, IFSC: ESFB0000002, BANK: EQUITAS SMALL FINANCE BANK AND ON 30-08-2021, RS 6471 LAKHS WERE DEPLOYED THROUGH OUTWARD REMITTANCE TO DREAMBOX STUDIOS LTD ACCOUNT NUMBER: 57412001958, CURRENCY: USD, AMOUNT: 8785000, BANK: STANDARD CHARTERED BANK, COUNTRY: HONG KONG, PURPOSE: TWDS DATABASE AND DATA PROCESSING",{"entities":[(264,284,"FOREIGN ORG")    ,(3061,3086,"FOREIGN ORG"),(3089,3112,"FOREIGN ORG"),(3995,4015,"FOREIGN ORG")  ]}),
            ("AND ON THE SAME DAY AN AMOUNT OF RS 241 LAKH GOT REMITTED TO MINGKUN INTERNATIONAL LOGISTICS LIMITED, CHINA COUNTRY IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS TOWARDS FREIGHT AS PER DUE DILIGENCE FEEDBACK AT THE TIME OF BANK OFFICIALS VISIT, NO BUSINESS WAS OBSERVED DURING CUSTOMER VISIT TO BRANCH FOR REMITTANCE PURPOSE , THE BANK OFFICIALS HAVE ASKED FOR THE REQUIRED DOCUMENTS, CUSTOMER WAS ABUSING THE BANK OFFICIAL AND LATER THE DOCUMENTS PROVIDED SEEMS TO BE NOT GENUINE BASED ON THE ABOVE FINDINGS, STR IS BEING FILEDSURVE IMPEX LLP, LIMITED LIABILITY PARTNERSHIP FIRM, HAVING ADDRESS: 1004, 3 RAJVAIBHAV COMPLEX, RAJU NAGAR, DOMBIVALI W, BEHIND CHURCH GANESH NAGAR, THANE, MAHARASHTRA, INDIA, 421202, CONTACT NUMBER: +918828809888 AND EMAIL ID: MANGESH@SURVE-GROUPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 148905001468 WITH ICICI BANK LTD, THANE - DOMBIVALI WEST, I GOPI CINE PLAZA, NSSROAD, THANE MAHARASHTRA, 421202, SINCE 14092020 DATE OF INCORPORATION IS 04072020 PAN NUMBER OF THE ACCOUNT IS AEFFS0779F TRANSACTION LINKED ACCOUNTS ARE 003501567743, 087305000725 AUTHORIZED SIGNATORIES/ BENEFICIAL OWNERS/ DESIGNATED PARTNERS OF THE REPORTED ACCOUNT ARE MANGESH N SURVE PAN: AVBPS9984M AND MANJIRI M SURVE PAN: DOZPS1305N THE CUSTOMER IS CATEGORIZED AS LOW RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN- AEFFS0779F, COI: AAS-7758, LLP AGRREMENT AND PAN OF PARTNERS KYC UPDATION IS NOT DUE AS PER BANK RECORDS, CUSTOMER IS INTO BUSINESS OF PHARMA AND MEDICAL GOODS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS 40 LAKHS ALERT WAS TRIGGERED FOR NEGATIVE EDD FEEDBACK RECEIVED AGAINST THE CUSTOMER BALANCE IN THE ACCOUNT AT 31082021 WAS RS8967 CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31082021 SHOWS TOTAL CREDIT RS 5217 LAKHS OF WHICH CLEARING RS 350 LAKHS MAJORLY FROM CHEQUE- SURVE IMPEX LLP, NEFT RS 2477 LAKHS MAJORLY FROM MAMTA, KANKAI TOTAL LOGISTICS LLP, RITU ENTERPRISE, RTGS RS 1908 LAKHS MAJORLY FROM PREM ENTERPRISES, SURYAA BATTERIES PVT LTD, INTERNET FUND TRANSFERS RS 424 LAKHS FROM MANGESH NARESH SURVE 003501567743, MOBILE MONEY TRANSFERS RS 056 LAKHS IN 1 INSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS 5221 LAKHS OF WHICH REMITTANCE RS 1878 LAKHS MAJORLY TO UNION OCEAN SHIP MANAGEMENT CO LTD AT MAJURO REPUBLIC MARSHALL ISLANDS IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS TOWARDS FREIGHT, INTERNET FUND TRANSFER RS 2782 LAKHS TO MANGESH NARESH SURVE 003501567743, LANCER CONTAINER LINES LIMITED 087305000725, MOBILE MONEY TRANSFERS RS 559 LAKHS IN 20 INSTANCESFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEAR-WISE WERE: FY 2020-21 FROM 14092020 TO 31032021: TOTAL CREDIT SIDE RS 3251 LAKHS OF WHICH CASH WAS RS 250 LAKHS MAJORLY IN DOMBIVALI WEST BRANCH, TRANSFERS RS 040 LAKHS MAJORLY FROM SRI VENKATESHWARA GEAR WHEELS, MANGESH NARESH SURVE, NEFT RS 703 LAKHS MAJORLY FROM KANKAI TOTAL LOGISTICS LLP, R A ENGINEERES, SURVE IMPEX LLP, RTGS RS 2246 LAKHS MAJORLY FROM KANKAI TOTAL LOGISTICS, PREM ENTERPRISES, R A ENGINEERS, MOBILE MONEY TRANSFERS RS 100, UPI TRANSFERS RS 008 LAKHS IN 2 INSTANCES TOTAL DEBIT SIDE RS 3246 LAKHS OF WHICH CASH WAS RS 002 LAKHS BY ATM, REMITTANCE RS 194 LAKHS TO MINGKUN INTERNATIONAL LOGISTICS LIMITED, CHINA COUNTRY IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS TOWARDS FREIGHT, INTERNET FUND TRANSFER RS 2828 LAKHS TO MANGESH NARESH SURVE 003501567743, CARD TRANSACTIONS RS 016 LAKHS, MOBILE MONEY TRANSFERS RS 202 LAKHS IN 10 INSTANCES TRANSACTION PATTERN SHOWS HIGH VALUE CREDITS ARE FOLLOWED BY DEBITS FOR WHICH APPARENT RATIONALE IS NOT CLEAR FOR EXAMPLE, ON 18-05-2021, RS 18 LAKHS WERE CREDITED THROUGH NEFT FROM MAMTA ACCOUNT NO: 0897102000004435, IFSC CODE-IBKL0NEFT01, BANK: IDBI BANK LTD AND ON 27-05-2021, RS 1623 LAKHS WERE CREDITED THROUGH REMITTANCE TO UNION OCEAN SHIP MANAGEMENT CO LTD AT MAJURO REPUBLIC MARSHALL ISLANDS IN USD CURRENCY FOR WHICH PURPOSE MENTIONED AS TOWARDS FREIGHT ON 25-08-2021, AN AMOUNT OF RS 250 LAKHS GOT CREDITED THROUGH TRANSFERS FROM MANGESH NARESH SURVE ACCOUNT NO: 003501567743 ON THE SAME DAY AN AMOUNT OF RS 149 LAKH GOT CREDITED THROUGH MMT TRANSFERS AND 1 LAKH GOT CREDITED THROUGH UPI TRANSFERS",{"entities":[(61,99,"FOREIGN ORG")     ,(2371,2405,"FOREIGN ORG"),(2581,2611,"FOREIGN ORG")   ,(3366,3404,"FOREIGN ORG"),(3980,4008,"FOREIGN ORG")]}),
            ("3 A PATTERN OF TRANSACTIONS WERE SEEN IN THE ACCOUNTS WHEREIN HIGH VALUE ROUND SUM RTGS/ NEFT CREDITS WERE FOLLOWED BY HIGH VALUE FOREIGN OUTWARD REMITTANCES NON IMPORT NO OTHER BUSINESS EXPENSE TRANSACTIONS LIKE PAYMENTS TOWARDS TAXES, RENT, UTILITY OR MAINTENANCE WERE NOTED THROUGH THE ACCOUNTS DURING REVIEW PERIOD 4 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS A 1200 SQ FT RENTED PREMISE WITH OFFICE SET UP AND EMPLOYEES ON ENQUIRY CUSTOMER INFORMED BRANCH OFFICIAL THAT THEY ARE INTO BUSINESS OF PROVIDING FACILITY FOR FILM SHOOTING AT DIFFERENT LOCATIONS THE CUSTOMER HOWEVER DID NOT ALLOW THE BRANCH OFFICIALS TO TAKE PICTURES OF THE OFFICE 5 IT IS PERTINENT TO NOTE THAT ACCOUNT OF SHOOLIN MEDIA AND ENTERTAINMENT WAS OPENED IN AT ANDHERI WEST BRANCH IN MUMBAI WITH MANOJ KUMAR VERMA ANLPV6738L AND LALITKUMAR YADAV AS THE PARTNERS HOWEVER THE ACCOUNTS OF GOLDEN FLUTE ENTERTAINMENT AND KFILMS MOTION PICTURES WERE OPENED AT LUCKNOW BRANCH IN UP WHEREIN CUSTOMER HAD DONE A NAME CHANGE FROM MANOJ KUMAR TO KRISHNA VERMA ANLPV6738L THE RATIONALE BEHIND OPENING ACCOUNT AT DIFFERENT CITIES POST CHANGING NAME IS NOT KNOWN TO BANK 6 AS PER CREDIT REPORT ON THE OVERSEAS PARTIES SUNRISE AUDIO VISUALS PTE LTD AND AUT INTERNATIONAL PTE LTD FURNISHED BY DUN AND BRADSTREET THE ENTITIES ARE FORMED IN 2019 WITH NATURE OF BUSINESS AS MOTION PICTURE AND VIDEO PRODUCTION HAVING A COMMON MAILING ADDRESS 200 JALAN SULTAN 10-10 TEXTILE CENTRE 199018, SINGAPORE AND COMMON DIRECTOR- BHANDARI RAJENDER KUMAR, SONI INDERJEET SINGH AND ASHISH KUMAR SINGH WHO APPEAR TO BE INDIANS ALSO, AS PER THE REPORTS, THE OVERALL ASSESSMENT IS INDICATED AS HEIGHTENED POTENTIAL PAYMENT CONCERNS AND LOW ABILITY TO PAY OBLIGATIONS FURTHER THE OVERSEAS COMPANIES ARE FORMED WITH A MERE CAPITAL OF SGD 1000 EACH APPROX INR 55000 BASIS THE INFORMATION IN CREDIT REPORT IT APPEARS THAT A COMMON INDIVIDUAL MAY HAVE SET UP BOTH ENTITIES OVERSEAS WITH A NOMINAL CAPITAL AND THE EXISTENCE OF BUSINESS AND GENUINENESS OF TRANSACTIONS IS A SUSPECT 7 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS OVERSEAS SHOOTING PAYMENT MOVIE WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENT AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE ACCOUNT OF GOLDEN FLUTE ENTERTAINMENT IS ACTIVE WITH BALANCE OF RS74,52,926/- AS ON 7112021 QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS674 CR IN CASH RS002 CR AND CREDIT: RS751 CR IN CASH RS0 CR AND THE TRANSACTIONS TURNOVER FOR PREVIOUS THREE FINANCIAL YEARS ARE 2020-21: DEBIT: RS001 CR IN CASH RS0 CR AND CREDIT: RS0 CR IN CASH RS0 CR, 2019-20: DEBIT: RS004 CR IN CASH RS0 CR AND CREDIT: RS005 CR IN CASH RS0 CR, 2018-19: DEBIT: RS0 CR IN CASH RS0 CR AND CREDIT: RS0 CR IN CASH RS0 CRA CURRENT ACCOUNT BEARING NUMBER 1045600170 WAS OPENED IN THE NAME OF A PARTNERSHIP FIRM GOLDEN FLUTE ENTERTAINMENT ON 05082021 AT OUR BRANCH LOCATED AT GROUND FLOOR, A-3/L, SECTOR-L KURSI ROAD, ALIGANJ OPPOSITE VIKAS NAGAR, LUCKNOW, UTTAR PRADESH-226006 THE ENTITY WAS ESTABLISHED ON 06042020 AND HAS SUBMITTED COPIES OF PAN AAWFG3158R, SHOPS AND ESTABLISHMENT CERT REG NO UPSA28736616, PARTNERSHIP DEED, GST REG CERT , AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING KUNAL PATEL AND NAZNEEN ABDUL RAZZAK CHAUDHARY ARE THE PARTNERS AND UBOS IN THE ENTITY CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS B-3-10 VJAYANT KHAND GOMTINAGAR LUCKNOW UP - 226010 AND HAS REGISTERED MOBILE NUMBER 7400142696 THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND THE ACCOUNT IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF ENTERTAINMENT SERVICES WITH ANNUAL TURNOVER OF RS1000 CR BASIS INTER-CONNECTEDNESS WITH ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT ACCOUNTS IN THE NAMES OF KFILMS MOTION PICTURES A/C NO2845285048, CONSTITUTION-PARTNERSHIP FIRM, NATURE OF BUSINESS - ENTERTAINMENT - SERVICES, ANNUAL TURNOVER-RS030 CR, PAN-AAWFK9895R, DATE OF INCORPORATION - 04-APR-20, PARTNERS -KUNAL PATEL AND KRISHNA PATEL AND SHOOLIN MEDIA AND ENTERTAINMENT A/C NO8713473500, CONSTITUTION PARTNERSHIP FIRM, NATURE OF BUSINESS- MEDIA SERVICES, ANNUAL TURNOVER RS070 CR, PAN-ADZFS1194H, DATE OF INCORPORATION- 18-JUL-19, PARTNER - MANOJ KUMAR VERMA AND LALITKUMAR YADAV AND SAVINGS ACCOUNTS IN THE NAMES OF KUNAL PATEL A/C NO0613183282 AND KRISHNA VERMA A/C NO9839404012 ALL THE MENTIONED ACCOUNTS WERE OPENED DURING THE PERIOD 03-NOV-18 TILL 21-OCT-21 AT OUR GOMTI NAGAR AND VIKAS NAGAR BRANCHES IN LUCKNOW AND LOKHANDWALA BRANCH MUMBAI WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NON-IMPORT WERE DONE FROM THE ABOVE MENTIONED NEWLY OPENED ACCOUNTS OF GOLDEN FLUTE ENTERTAINMENT HENCE AN ALERT WAS RAISED ON BRANCH TO CARRY OUT DUE DILIGENCE AND THIS CONSOLIDATED REPORT IS BEING FILED BASIS INTER-CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE APRIL 1, 2020 TILL NOVEMBER 7, 2021, IT WAS NOTED THAT IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS740 MAINLY FROM CAPITAL ENTERTAINMENT RS501 CR, IDFC 10067204005, EQUITAS SMALL FINANCE BANK - 200001375028 AND FRIENDS FILMS AND ENTERTAINMENT RS239 CR, IDFC 10067353761 2 SUBSEQUENT TO SUCH CREDITS FOREIGN OUTWARD REMITTANCES AMOUNTED TO RS606 CR FAVOURING SINGAPORE BASED ENTITIES SUNRISE AUDIO VISUALS PTE LTD RS398 CR AND AUT INTERNATIONAL PTE LTD RS207 CR FROM THE ACCOUNTS OF GOLDEN FLUTE ENTERTAINMENT AND KFILMS MOTION PICTURES TOWARDS AUDIO VISUAL AND RELATED SERVICES",{"entities":[(1231,1260,"FOREIGN ORG"),(1265,1290,"FOREIGN ORG") ,(5693,5722,"FOREIGN ORG"),(5736,5761,"FOREIGN ORG")  ]}),
            ("DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER KYTEUS P LTD, IT WAS NOTED THAT THE OFFICE WAS SHIFTED TO AN ALTERNATE LOCATION IE, 503, DDA 2 BUILDING, DISTRICT CENTRE, JANAKPURI, WEST DELHI AT THE MENTIONED ADDRESS SMALL SIGN BOARDS IN THE NAMES OF KYTEUS P LTD AND GLOBUS HOLIDAYS WERE SEEN BY BRANCH OFFICIAL WITH OFFICE SET UP AND FEW FREELANCERS NO PERMANENT EMPLOYEES IN 700 SQ FT RENTED PREMISE 4 ALSO MIDDLE ASIA OVERSEAS AND PROCON INTERNATIONAL SHARE A COMMON ADDRESS - A-49 FIRST FLOOR MAHIPALPUR EXTN NEW DELHI SOUTH WEST DELHI 110037 DURING VISIT BY BRANCH OFFICIAL AT THE SAID ADDRESS TWO DIFFERENT TEMPORARY SIGN BOARDS WERE SEEN IN THE NAMES OF MIDDLE ASIA OVERSEAS AND IN THE NAME OF PROCON INTERNATIONAL IMPORT AND EXPORT OFFICE SET UP WAS SEEN WITH TABLES, CHAIRS AND FEW DESKTOPS HOWEVER NO MAJOR BUSINESS ACTIVITY IN LINE WITH THE TURNOVER INDICATED OR TRANSACTIONS ROUTED COULD BE SIGHTED 5 IT IS PERTINENT TO NOTE THAT THE CUSTOMER PROCON INTERNATIONAL HAS INDICATED NATURE OF BUSINESS AS AUTOMOBILE WHOLESALER AND THE SIGN BOARD ALSO INDICATES IMPORT AND EXPORT BUSINESS HOWEVER CUSTOMER HAS ROUTED TRANSACTIONS RELATED TO FREIGHT PAYMENTS FURTHER, KYTEUS P LTD WHICH IS INTO BUSINESS OF IT SERVICES HAS RECEIVED RTGS CREDITS FROM MIDDLE ASIA OVERSEAS WHICH HAS DECLARED BUSINESS AS COURIER AND FREIGHT SERVICES HENCE, IT APPEARS THAT THE ENTITIES KYTEUS P LTD, MIDDLE ASIA OVERSEAS AND PROCON INTERNATIONAL HAVE ROUTED TRANSACTIONS UNRELATED TO BUSINESS ACTIVITY WHICH MAY HAVE NO ECONOMIC RATIONALE AND SHELL COMPANIES MAY HAVE BEEN FORMED WITH DIFFERENT NATURE OF BUSINESSES BY A COMMON GROUP OF INDIVIDUALS WITHOUT GENUINE COMMERCIAL BUSINESS 6 A PATTERN OF TRANSACTIONS WERE NOTED IN THE ACCOUNTS WHEREIN HIGH VALUE ROUND SUM RTGS/NEFT CREDITS RECEIVED MAINLY FROM OWN ACCOUNTS MAINTAINED WITH OTHER BANKS WERE REMITTED OVERSEAS AS FOREIGN OUTWARD REMITTANCES NON-IMPORT TOWARDS FREIGHT PAYMENTS OR TOWARDS SOFTWARE CONSULTANCY NO PAYMENTS TOWARDS BUSINESS EXPENSES LIKE TAXES, RENT, UTILITY PAYMENTS COULD BE SEEN THROUGH THE ACCOUNTS DURING REVIEW PERIOD 7 DURING SCRUTINY OF SAMPLE DOCUMENTS FURNISHED BY THE CUSTOMER IT WAS NOTED THAT ALL INVOICES WERE COMPUTER GENERATED WITHOUT THE SIGN AND STAMP OF OVERSEAS PARTIES IT COULD BE POSSIBLE THAT THE DOCUMENTS MAY HAVE BEEN FABRICATED MERELY TO SATISFY BANK OFFICIALS TO REMIT FUNDS OVERSEAS 8 POST QUERIES, NO MAJOR TRANSACTIONS WERE ROUTED THROUGH THE CURRENT ACCOUNTS 9 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS IT SERVICES / FREIGHT PAYMENTS WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF LAYERING AND TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENTS ACCOUNT OF KYTEUS P LTDIS ACTIVE WITH BALANCE OF RS23,562/- AS ON NOVEMBER 9, 2021 QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS665 CR IN CASH RS0 00 CR AND CREDIT: RS669 CR IN CASH RS000 CRA CURRENT ACCOUNT BEARING NUMBER 0345219266 WAS OPENED IN THE NAME OF KYTEUS PRIVATE LIMITED ON 06072021 AT OUR BRANCH LOCATED AT B-1 BLOCK, JANAKPURI, NEW DELHI PIN CODE 110027 THE ENTITY WAS ESTABLISHED ON 01082020 AND HAS SUBMITTED COPIES OF PAN AAICK2043E, BOARD RESOLUTION, MOA/ AOA, CERT OF INCORP U52520DL2020PTC367254, GST REG CERT 07AAICK2043E1ZV, IEC CERT AAICK2043E AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING SARANSH SHARMA AND AMIT KANOJIA ARE THE DIRECTORS AND UBOS IN THE COMPANY CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 322 JAINA TOWER 1 DISTRICT CENTRE JANAKPURI NEW DELHI WEST DELHI, 110058 AND HAS REGISTERED MOBILE NUMBER 9971331110 THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND THE ACCOUNT IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF IT INDUSTRY - SERVICES WITH ANNUAL TURNOVER OF RS300 CR BASIS INTER-CONNECTEDNESS WITH ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT ACCOUNTS IN THE NAMES OF MIDDLE ASIA OVERSEAS A/C NO0945390907, CONSTITUTION-SOLE PROPRIETORSHIP, NATURE OF BUSINESS- LOGISTICS/COURIER - SERVICES, ANNUAL TURNOVER-RS1500 CR, PAN-NFWPS0876E, DATE OF INCORPORATION-21-OCT-20, PROCON INTERNATIONAL A/C NO1345552377, CONSTITUTION-PARTNERSHIP FIRM, NATURE OF BUSINESS - AUTOMOBILES - WHOLESALERS, ANNUAL TURNOVER- RS6000 CR, PAN-AAZFP2473P, DATE OF INCORPORATION - 30-SEP-20 AND THE SWISS TOURS A/C NO3411625497, CONSTITUTION-PARTNERSHIP, NATURE OF BUSINESS - TRAVEL AGENCIES, ANNUAL TURNOVER-RS100 CR, PAN-AAKFT1288M, DATE OF INCORPORATION-08-MAY-15 THE SAID ACCOUNTS WERE OPENED DURING THE PERIOD 28-JUL-21 TILL 19-AUG-21 AT OUR ABOVE MENTIONED BRANCH EXCEPT THE SWISS TOURS WHICH WAS OPENED ON 22-JUL-15 AND CLOSED SINCE 30-JUL-2016 GAURAV SEHGAL, JASPAL SINGH BINDRA, SARANSH SHARMA AND AMIT KANOJIA ARE THE COMMON PROPRIETORS/ PARTNERS IN THE MENTIONED ACCOUNTS WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES NON-IMPORT DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES WERE DONE FROM THE ABOVE MENTIONED NEWLY OPENED ACCOUNT OF KYTEUS P LTD THOUGH KYTEUS P LTD IS INTO BUSINESS OF IT SERVICES HAD RECEIVED RTGS/NEFT CREDITS FROM MIDDLE ASIA OVERSEAS WHICH ALSO MAINTAINS AN ACCOUNT BEARING NUMBER 0945390907 HENCE DUE DILIGENCE WAS CARRIED OUT AND THIS CONSOLIDATED REPORT IS FILED ON THE ABOVE MENTIONED ACCOUNTS BASIS INTER-CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL NOVEMBER 5, 2021 IT WAS NOTED THAT IMPS/ RTGS/NEFT CREDITS TOTALLING TO RS665 CR WERE RECEIVED IN THE ACCOUNTS BY WAY MAINLY FROM KYTEUS PRIVATE LIMITED RS266 CR, FINCARE SMALL FINANCE BANK 21200000014691, HDFC 50200053989840, ICICI 663005601270, AXIS BANK - 920020058932574, MIDDLE ASIA OVERSEAS RS180 CR, BANDHAN BANK 11210001677347, LAKSHMI VILAS BANK - 0499360000005708 AND PROCON INTERNATIONAL RS060 CR, BANDHAN BANK 11210001951731/ ICICI - ICI8882032973 2 SUBSEQUENT TO SUCH CREDITS FOREIGN OUTWARD REMITTANCES NON-IMPORT AMOUNTED TO RS617 CR MAINLY FAVOURING TROJEN SG PTE LTD - SINGAPORE RS197 CR FROM MIDDLE ASIA OVERSEAS TOWARDS FREIGHT PAYMENTS, SUCCESS BLOSSOM SG PTE LTD SINGAPORE RS139 CR FROM A/C OF KYTEUS P LTD TOWARDS SOFTWARE CONSULTANCY AND BLISS QUAY SG PTE LTD - SINGAPORE RS098 CR FROM A/C OF PROCON INTERNATIONAL TOWARDS FREIGHT ON IMPORTS",{"entities":[(6232,6249,"FOREIGN ORG"),(6331,6349,"FOREIGN ORG"),(6427,6448,"FOREIGN ORG")    ]}),
            ("ON SCRUTINY OF THE ACCOUNT OF SSG GLOBAL ACCOUNT NUMBER - 409825039180, WE HAVE OBSERVED THAT HUGE AMOUNT OF FUNDS BY NEFT/RTGS/FOREIGN REMITTANCE HAVE BEEN ROUTED THROUGH THIS ACCOUNT WHEREIN WE ARE NOT ABLE TO ASCERTAIN THE RATIONALE FOR THESE TRANSACTIONS ALSO, CUSTOMER DECLARED HIS ANNUAL TURNOVER WHILE ACCOUNT OPENING WAS RS15 CRORES ON THE OTHER HAND CUSTOMER HAS ALREADY MADE MORE THAN RS15 CRORES OUTWARD IMPORT PAYMENT TILL THE DATE ALL THE ABOVE POINTS HAVE GIVEN RISE TO SUSPICION, HENCE STR RAISEDSSG GLOBAL A/C NO 409825039180, PAN NO ADRPS3350F, DOI 20-08-2020, STATUS - A, ADD - 263 GVMMSAV LTD ODHAV ROAD ODHAV,,AHMEDABAD - 382415, MOB NO - 919825039180, AUTHORISED SIGNATORY - SURESHCHANDRA GOVERDHANLAL SHAH, MEDIUM RISK,KYC COMPLIANCE STATUS AND LAST KYC UPDATION DATE - YES AND NA OPENED A CURRENT ACCOUNT ON 31-08-2020 AT RBL BANK AHMEDABAD BRANCH ADD - SHOP NO 8,9,10,13 & 14, MALLINATH COMPLEX, OPP SUJATA FLATS, SHAHIBAUG, AHMEDABAD - 380004 ACCOUNT HOLDER HAS DECLARED PROFILE AS CUSTOMER DEALING IN SCRAPE METALS BUSINESS CONSTITUTION - PROPRIETORSHIP HAVING EXPECTED INCOME BETWEEN OF RS1 CRORE - 5 CRORE IN VIEW OF THE RESTRICTION ON SHARING THE AADHAAR NO, AS PER REGULATION 6 OF AADHAAR SHARING OF INFORMATION REGULATIONS 2016, WE ARE PROVIDING REFERENCE KEY ASSIGNED TO THE AADHAAR NOSCRUTINY OF ACCOUNT STATEMENT FOR THE PERIOD 2021-22 UP TO THE DATE OF FILING OF STR OF SSG GLOBAL ACCOUNT NO: 409825039180 REVEALED THAT TOTAL CREDITS OF RS3804 CRORES IN THE ACCOUNT MAINLY CONSIST OF INTERNAL FUND TRANSFER OF RS201 CRORES FROM AC NO - 409898880152, 400040228158, 309009309436 RTGS/NEFT OF RS3416 LAKHS FROM RADHA RAMAN METAL PRIVATE LI AC NO - 917030028869205/AXIS BANK, JINDAL ALUMINIUM LIMITED R AC NO- 45605134686/STANDARD CHARTERED BANK CHEQUE DEPOSIT OF RS4357 LAKHS FROM ELEANOR INDUSTRI AC NO - 0260658400000019/IDBI BANK AND JAGDISHCHANDRA D AC NO - 002101001005373/APNI SAH BANK OF THE TOTAL DEBITS BEING RS3814 CRORES UTILIZED BY FOREIGN REMITTANCE OF RS228 CRORES TOWARDS VARIOUS PARTIES LIKE INDICAA GROUP LIMITED AC NO - AE390030000974710021001/ABU DHABI COMMERCIAL BANK, FPT INTERNATIONAL FERROUS AC NO - 4720459080/WELLS FARGO BANK, TARSAN INTERNATIONAL PTE L AC NO - 1379014473/STANDARD CHARTERED BANK SCRUTINY OF ACCOUNT STATEMENT FOR THE PERIOD 2020-21 UP TO THE DATE OF FILING OF STR OF SSG GLOBAL ACCOUNT NO: 409825039180 REVEALED THAT TOTAL CREDITS OF RS3543 CRORES CASH TRANSACTIONS BEING NIL OF THE TOTAL DEBITS BEING RS3486 CRORES CASH TRANSACTIONS BEING NILBALANCE AS ON DATE OF FILING BEING RS300170639/-",{"entities":[(30,40,"FOREIGN ORG"),(1405,1415,"FOREIGN ORG"),(2342,2352,"FOREIGN ORG"),(2040,2061,"FOREIGN ORG"),(2121,2138,"FOREIGN ORG"),(2184,2210,"FOREIGN ORG")]}),
            ("AN AMOUNT OF RS 30 LAKHS WERE CREDITED THROUGH RTGS FROM RIDDHYMIC TECHNOLOGIE SPRIVATE LIMITED A/C 50200036609140, IFSC HDFC0000240 AND ON 29-09-2021 RS 2836 LAKHS WERE DEBITED THROUGH CMS TRANSFER IN 3 INSTANCES ALSO ON 27-10-2021, RS 19 LAKHS WERE CREDITED THROUGH RTGS FROM RIDDHYMIC TECHNOLOGIES PRIVATE LIMITED A/C 50200036609140, IFSC HDFC0000240 AND ON 30-10-2021 RS 1487 LAKHS WERE DEBITED THROUGH CMS TRANSFER IN 4 INSTANCES OVERALL ACCOUNT ACTIVITY SEEMS TO BE SUSPICIOUS IN VIEW OF HIGH VALUE FUNDS BEING ROUTED, STR IS BEING FILEDRIDDHYMIC TECHNOLOGIES PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS : 171/B MITTAL TOWER, 210, NARIMAN POINT, MUMBAI, MAHARASHTRA, INDIA, 400021, CONTACT NUMBER: +919821038771 AND EMAIL ID: AMITTAL@RIDDHYMICCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000405111318 WITH ICICI BANK LTD, 215, FREE PRESS HOUSE, FREE PRESS MARG, NARIMAN POINT, MUMBAI, MAHARASHTRA, 400021, SINCE 15022016 DATE OF INCORPORATION IS 23032000 PAN OF THE ACCOUNT IS AABCI0121N IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0316977900 GSTIN AS PER BANK RECORDS IS 27AABCI0121N1ZS OTHER RELATIONSHIP OF CUSTOMER ARE 000405071677, 000406000878, 000410421880, 000414863995, 000414943708 TRANSACTION LINKED ACCOUNTS ARE 000205001402, 000401737660 BENEFICIAL OWNER/ AUTHORISED SIGNATORY/ DIRECTOR OF THE REPORTED ACCOUNT IS ASHWIN RAMESH MITTAL WITH PAN NO AAEPM3835F THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION KYC UPDATION IS DUE MANUAL ALERT WAS RAISED FOR HIGH VALUE FUNDS BEING ROUTED BALANCE IN THE ACCOUNT AT 31102021 WAS RS554 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY 2021-22 FROM 01042021 TILL 31102021 SHOWS TOTAL CREDIT RS 1,89600 LAKHS OF WHICH TRANSFERS RS 1450 LAKHS MAJORLY FROM RIDDHYMIC TECHNOSERVE LLP, RAMESH BRAHMADUTT MITTAL JT1, NEFT RS 512 LAKHS MAJORLY FROM RIDDHYMIC TECHNOLOGIES PRIVATE LTD, MR KUMAR MEHTA, E PAO GST REFUNDS, RTGS RS 32501 LAKHS MAJORLY FROM RIDDHYMICTECHNOLOGIESPVTLTD, E PAO GST REFUNDS, REMITTANCE RS 16913 LAKHS MAJORLY FROM TRADE TECH CHINA CO LTD A39 3, INTERNET FUND TRANSFERS RS 1,38215 LAKHS FROM COURSES INTELLIGENCE PVT LTD ETC TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,90485 LAKHS OF WHICH CLEARING RS 46315 LAKHS, RTGS RS 1,38200 LAKHS MAJORLY TO RIDDHYMIC, INTERNET FUND TRANSFER RS 4962 LAKH TO DEEPAL MARUTI KAYANDEKA ETC FY 2020-21 FROM 01042020 TO 31032021 : TOTAL CREDIT SIDE RS 38376 LAKHS OF WHICH TRANSFERS RS 920 LAKHS MAJORLY FROM MAYURI SAMEER ADHIKARI, RAMESH BRAHMADUTT MITTAL JT1, NEFT RS 1907 LAKHS MAJORLY FROM WELLNESS FORE UNPAID, RTGS RS 13510 LAKHS MAJORLY FROM RIDDHYMICTECHNOLOGIESP, REMITTANCE RS 21607 LAKHS TOTAL DEBIT SIDE RS 38144 LAKHS OF WHICH CASH WAS RS 1720 LAKHS BY BRANCH, TRANSFERS RS 1721 LAKHS MAJORLY TO RAMESH BRAHMADUTT MITTAL JT1, ANUPAM MITTAL, EMPLOYEES PROVIDENT FUND ORGANISATION, CLEARING RS 9377 LAKHS, INTERNET FUND TRANSFER RS 24554 LAKHS TO DEEPAK MARUTI KAYANDEKA ETC FY 2019-20 FROM 01042019 TO 31032020 : TOTAL CREDIT SIDE RS 40819 LAKHS OF WHICH RS 0 BY CASH, REMITTANCE RS 19496 LAKHS TOTAL DEBIT SIDE RS 39620 LAKHS OF WHICH CASH WAS RS 372 LAKHS, REMITTANCE RS 013 LAKHS FY 2018-19 FROM 01042018 TO 31032019 : TOTAL CREDIT SIDE RS 2,69960 LAKHS OF WHICH RS 0 BY CASH, REMITTANCE RS 17567 LAKHS TOTAL DEBIT SIDE RS 2,70460 LAKHS OF WHICH CASH WAS RS 1069 LAKHS, REMITTANCE RS 014 LAKHS FY 2017-18 FROM 01042017 TO 31032018 : TOTAL CREDIT SIDE RS 1,21339 LAKHS OF WHICH RS 0 BY CASH , REMITTANCE RS 11332 LAKHS TOTAL DEBIT SIDE RS 1,22945 LAKHS OF WHICH CASH WAS RS 1334 LAKHS, REMITTANCE RS 010 LAKHS AS PER EDD, THE COMPANY IS ENGAGED IN THE BUSINESS OF PROVIDING BPO SERVICES IT IS ALSO ENGAGED IN IT HOSTING OR INFRASTRUCTURE PROVISIONING SERVICES FUNDS WERE CREDITED MAJORLY FROM OWN OTHER ACCOUNT AND DEBITED THROUGH CMS",{"entities":[(2231,2254,"FOREIGN ORG")]}),
            ("AS PER CREDIT OPINION REPORT FURNISHED BY DUN AND BRADSTREET DATED 2552021 ON OVERSEAS PARTY JOY REAP METAL PTE LTD SINGAPORE, THE ENTITY IS FORMED IN 2017 AND IS INTO BUSINESS OF DURABLE GOODS THE COMPANY COULD NOT BE LOCATED ON THE ADDRESS FURNISHED 80 ANSON ROAD, 25-01, FUJI XEROX TOWERS, SINGAPORE AND DID NOT HAVE LISTING IN THE LOCAL TELEPHONE COMPANY IN SINGAPORE FURTHER THE COMPANY IS FORMED WITH A MERE PAID UP CAPITAL OF USD 3 AND THE RISK RATING IS UNDETERMINED DUE TO INSUFFICIENT INFORMATION BASIS THE INFORMATION, EXISTENCE OF THE SAID OVERSEAS PARTY APPEARS TO BE DOUBTFUL AND HENCE THE GENUINENESS OF THE IMPORT PAYMENTS DONE BY CUSTOMER FAVOURING JOY REAP METAL PTE LTD SINGAPORE IS SUSPECT 5 RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS427 CR MAINLY FAVOURING MUKESH STEEL INDIA RS113 CR , SCB - 22506161842, DHANERA IMPEX RS095 CR, ICICI BANK - 034805500776 AND SHREE ASHAPURA STEEL CENTRE RS062 CR , YES BANK - 000484100001388 6 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS OF YAVI OVERSEAS, IT WAS NOTED THAT THE PREMISE WAS UNDER RENOVATION AND CLIENT HAD PROVIDED A NEW ADDRESS AS OFFICE ADDRESS HOWEVER BUSINESS ACTIVITY WAS YET TO COMMENCE AT THE NEW ADDRESS HENCE EXISTENCE OF BUSINESS ACTIVITY APPEARS TO BE A SUSPECT 6 CUSTOMER IS YET TO SUBMIT BOES PERTAINING TO 15 CASES OF ADVANCE IMPORT REMITTANCES TOTALLING TO USD 838,439 WHICH COMPRISE OF 1 CASE USD 18000 PENDING FOR A PERIOD ABOVE 3 MONTHS UPTO 6 MONTHS AND 14 CASES USD 820439 PENDING BELOW 3 MONTHS 7 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT MULTIPLE SHELL ENTITIES MAY HAVE BEEN FORMED BY A COMMON INDIVIDUAL - DIXIT DEVCHAND CHHAJED ALONG WITH RELATED PARTIES AND ACCOUNT MAY HAVE BEEN OPENED TO ROUTE TRANSACTIONS FOR LAYERING OF FUNDS AND TO REMIT FUNDS OVERSEAS AS ADVANCE IMPORT PAYMENTS POSSIBLY WITH THE INTENTION OF LAYERING AND TBML AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALEA CURRENT ACCOUNT BEARING NUMBER 8845290891 WAS OPENED IN THE NAME OF A PARTNERSHIP FIRM YAVI OVERSEAS ON 28042021 AT OUR BRANCH LOCATED AT 480, SANGHRAJKA HOUSE, S V P ROAD, OPERA HOUSE, MUMBAI - 400004 THE ENTITY WAS ESTABLISHED ON 27022021 AND HAS SUBMITTED COPIES OF PAN AACFY7409H, GST REG CERT 27AACFY7409H1ZX, IEC CERT AACFY7409H, PARTNERSHIP DEED AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING DINESH KUMAR JAIN AND DIXIT DEVCHAND CHHAJED ARE THE PARTNERS AND UBOS IN THE FIRM CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 1ST FLOOR OFFICE NO 102 PRIME VIEW 8TH KHETWADI LANE GIRGAON MUMBAI 400004 AND HAS REGISTERED MOBILE NUMBER 9820644999 THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND THE ACCOUNT IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF METALS - WHOLESALERS WITH ANNUAL TURNOVER OF RS2500 CR ONE OF THE PARTNERS DINESH KUMAR JAIN MAINTAINS A SAVINGS ACCOUNT BEARING NO1514336807 NO UNUSUAL ACTIVITY WAS NOTED IN THE SAID ACCOUNT DURING REVIEW PERIOD WHILE REVIEWING REPORT ON ADVANCE IMPORT REMITTANCES ROUTED THROUGH THE BANK, IT WAS NOTED THAT HIGH VALUE ADVANCE IMPORT REMITTANCES WERE DONE FROM THE ABOVE MENTIONED ACCOUNT OF YAVI OVERSEAS HENCE AN ALERT WAS RAISED ON BRANCH TO CARRY OUT DUE DILIGENCE AND THIS REPORT IS FILED ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS FROM INCEPTION TILL 29112021, IT WAS NOTED THAT RTGS/NEFT/IMPS CREDITS AMOUNTED TO RS759 CR MAINLY FROM AMANAT STEELS PVT LTD RS267 CR , ICICI BANK - 034851000024, JAY JAGDAMBA METAL INDUSTRIES RS082 CR , HDFC BANK - 59209920307730 AND CLASSIC STEEL CENTRE RS052 CR, ICICI BANK - 034805005453 CHEQUE DEPOSITS AMOUNTED TO RS130 CR 2 INTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS1051 CR FROM THE ACCOUNT OF DWEEP INTERNATIONAL A/C NO 5411965887 BANK HAS ALREADY FILED STR ON THE ACCOUNT OF DWEEP INTERNATIONAL ALONG WITH HINDAVI LOGISTICS SERVICES VIDE BATCH ID 2103131151 TOWARDS HIGH VALUE RTGS/NEFT CREDITS AND SUBSEQUENT ADVANCE IMPORT REMITTANCE 3 SUBSEQUENT TO SUCH CREDITS ADVANCE IMPORT REMITTANCES AMOUNTED TO RS926 CR MAINLY FAVOURING JOY REAP METAL PTE LTD- SINGAPORE RS433 CR, WENZHOU TONGSHANG IMPORT- CHINA RS206 CR, UNISILK LIMITED HONG KONG RS15 CR AND HONG KONG WINNER STEEL CO LIMITED- HONG KONG RS106 CR TOWARDS IMPORT OF METAL",{"entities":[(93,115,"FOREIGN ORG"),(666,688,"FOREIGN ORG"),(4076,4099,"FOREIGN ORG") ,(4120,4137,"FOREIGN ORG"),(4162,4177,"FOREIGN ORG"),(4210,4234,"FOREIGN ORG")  ]}),
            ("TRANSACTIONS WERE NOTED WHEREIN RS 194 CR WERE RECEIVED AND RS045 CR WAS SENT BACK BY RTGS/NEFT/IMPS TO OWN ACCOUNT OF KUMAR INTERTRADERS MAINTAINED WITH OTHER BANKS 6 NO BUSINESS RELATED TRANSACTIONS WERE NOTED THROUGH THE ACCOUNTS LIKE PAYMENTS TOWARDS TAXES, RENT AND SALARIES ETC, DURING REVIEW PERIOD WHICH IS UNUSUAL WHILE RUNNING A COMMERCIAL BUSINESS 7 AS A PART OF DUE DILIGENCE WHEN BRANCH OFFICIAL CONTACTED THE CUSTOMER, HE WAS RELUCTANT IN MEETING, SUBSEQUENTLY CUSTOMER STOPPED ATTENDING CALLS FROM THE OFFICIAL DURING A BRANCH OFFICIAL VISIT AT THE MAILING ADDRESS OF KUMAR INTERTRADERS, THE CUSTOMER WAS NOT AVAILABLE AND NO BUSINESS RELATED ACTIVITIES WERE OBSERVED ON ENQUIRY WITH NEIGHBOURS IT WAS LEARNT THAT CUSTOMER HAS SHIFTED TO OTHER LOCATION 8 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN THE NAME OF KUMAR INTERTRADERS AND ACCOUNT MAY HAVE BEEN OPENED TO ROUTE HIGH VALUE TRANSACTIONS WITHOUT GENUINE COMMERCIAL ACTIVITY FOR LAYERING OF FUNDS IT ALSO APPEARS THAT THE CUSTOMER MAY HAVE REMITTED FUNDS OVERSEAS TOWARDS IMPORT OF GOODS LIKE FABRIC YARN , DRY FRUITS , BICYCLE SPOKE NIPPLE, ETC WHICH MAY BE UNRELATED WITH INDICATED BUSINESS OF DECORATIVE / PLYWOOD / FURNITURE POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING AND HAWALA SETTLEMENT QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS 390 CR IN CASH RS 002 CR AND CREDIT: RS 390 CR IN CASH RS 025 CR AND THE TRANSACTIONS TURNOVER FOR PREVIOUS FINANCIAL YEAR ARE 2020-21: DEBIT: RS 000 CR IN CASH RS 000 CR AND CREDIT: RS 001 CR IN CASH RS 000 CR THE ACCOUNT STATUS OF KUMAR INTERTRADERS IS TOTAL FREEZE WITH NIL BALANCE AS ON 04122021ASHISH KUMAR AS A PROPRIETOR MAINTAINS A CURRENT ACCOUNT BEARING NO 3014532079 IN THE NAME OF A SOLE PROPRIETORSHIP CONCERN KUMAR INTERTRADERS SINCE 22032021 AT OUR BRANCH LOCATED AT SCO-31, SECTOR -32, CHANDIGARH ROAD, LUDHIANA 141001- PUNJAB THE ENTITY WAS ESTABLISHED ON 26022021 AND HAS SUBMITTED COPIES OF PAN KLBPK7182R, IEC CERT KLBPK7182R AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 513 5TH FLOOR, K -10 TOWER, FEROZE GANDHI MARKET, LUDHIANA 141001- PUNJAB AND HAS REGISTERED MOBILE NUMBER 9056208360 THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND CUSTOMER IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF DECORATIVE / PLYWOOD / FURNITURE WITH ANNUAL TURNOVER OF RS 050 CR CUSTOMER HAS ALSO MAINTAINED SAVINGS ACCOUNT NO BEARING 9045390572 SINCE 29052021 AT OUR FEROZE GANDHI MARKET LUDHIANA BRANCH NO UNUSUAL ACTIVITY WAS OBSERVED IN THE AFORESAID SAVINGS ACCOUNT 9045390572 AND THE SAID ACCOUNT WAS CLOSED ON 06092021 NOTICING HIGH VALUE RTGS/NEFT/IMPS CREDITS AND SUBSEQUENTLY HIGH VALUE ADVANCE IMPORT REMITTANCE WERE MADE FROM THE ACCOUNT 3014532079 OF KUMAR INTERTRADERS, AN ALERT WAS RAISED ON BRANCH TO CARRY OUT DUE DILIGENCE AND THIS REPORT IS FILED ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS FROM 01042020 TILL 02122021, IT WAS NOTED THAT RTGS/NEFT/IMPS CREDITS AMOUNTED TO RS 342 CR MAINLY FROM OWN ACCOUNT OF KUMAR INTERTRADERS MAINTAINED WITH OTHER BANKS RS194 CR , ICICI BANK - 001705014065, IDFC BANK - 10068494995, INDUSIND BANK LTD- 259056208360 , MANTAJ TRADERS RS059 CR , ICICI BANK - 151505500386 AND ACCURATE TRADER RS020 CR, CAPITAL LOCAL AREA BANK LTD - 030105000250 2 CASH DEPOSIT AMOUNTED TO RS 025 CR 3 ADVANCE IMPORT REMITTANCE AMOUNTED TO RS316 CR MAINLY FAVOURING CHANGSHU HUASEN- CHINA RS045 CR, WEIFANG ALICE FOOD CO LTD- CHINA RS039 CR, HEILONGJIANG- CHINA RS038 CR , HEBO BICYCLE CO LTD- CHINA RS022 CR AND PROGRESSER HOLDING- CHINA RS022 CR TOWARDS IMPORT OF FABRIC YARN, DRY FRUITS, BICYCLE SPOKE NIPPLE, ETC 4 KUMAR INTERTRADERS HAS NOT SUBMITTED BOES PERTAINING TO 17 CASES OF ADVANCE IMPORT REMITTANCE TOTALLING TO USD 423,396 PENDING FOR THE PERIOD BEYOND 3 MONTHS TILL 6 MONTHS",{"entities":[(3552,3560,"FOREIGN ORG"),(3585,3611,"FOREIGN ORG"),(3659,3679,"FOREIGN ORG")]}),
            ("FOLLOWING ARE THE OBSERVATIONS NOTED:1 ACCOUNT ACTIVITY AND TURNOVER IN A NEWLY OPENED RELATIONSHIP ARE NOT IN LINE WITH THE DECLARED TURNOVER AND KNOWN PROFILE OF THE CUSTOMER2 INTERNET SEARCH DID NOT REVEAL ANY PROPER WEBSITE FOR THE INDIAN COUNTERPARTIES FROM WHERE MAJORITY OF CREDITS RECEIVED IN THE ACCOUNT3 MAJORITY OF THE FUNDS CREDITED IN THE ACCOUNT FOLLOWED BY IMMEDIATE OUTWARD REMITTANCES IN A SHORT SPAN OF TIME APPEARED UNUSUALIN VIEW OF THE AFORESAID REASONS, AN STR IS BEING FILED3 WINGS PRODUCTION PAN: AKZPR7517G; GSTN: 27AKZPR7517G1ZC HOLDS A CURRENT ACCOUNT 006-366728-001 OPENED ON 30082021 WITH THE M G ROAD BRANCH, MUMBAIIT IS A SOLE PROPRIETORSHIP FIRM INCORPORATED ON 26042018 IT IS INTO THE BUSINESS OF MEDIA/VIDEO PRODUCTION, EXECUTING ASSIGNMENTS FOR MAJOR BRANDS, AD AGENCIES, CORPORATE COMPANIES AND SMALL BUSINESSES NATIONWIDE HTTPS://3WINGSPRODUCTIONCOM/ABOUTPHP THE ANNUAL TURNOVER IS DECLARED AS BELOW RS1 CRORE ITS ADDRESS IS: 406 ABHISHEK CHS LTD, OFF LINK ROAD, PLOT NO C 5 DALIA INDUSTRIAL, ANDHERI WEST, MUMBAI - 400053, AND PHONE NUMBER IS 9619690070 M MR TSANTOSH TKAMRAJU RAO DOB: 02011981 IS THE PROPRIETOR OF THE FIRM WITH A DECLARED ANNUAL INCOME OF RS20 LAKHS HE ALSO HOLDS A SAVINGS ACCOUNT 006-358741-006 SINCE 17082021 WITH THE M G ROAD BRANCH, MUMBAI HIS ADDRESS IS: FLAT NO A103 MANODAY BUILDING, DATTAPADA ROAD, NEAR SBI BANK, BORIVALI EAST, MUMBAI - 400066, AND PHONE NUMBER IS 9152990026 M AN ALERT WAS RAISED BY A STAFF MEMBER AS TRANSACTIONS IN THE ACCOUNT WERE NOTED TO BE SIGNIFICANTLY HIGH COMPARED TO THE DECLARED EXPECTED TURNOVER OF THE FIRM SIGNIFICANT AMOUNT OF CREDITS NOTED IN THE ACCOUNT WERE SUBSEQUENTLY REMITTED TO OVERSEAS COUNTERPARTIES IN VIEW OF THE SAME, THE TRANSACTIONS IN THE ABOVE MENTIONED ACCOUNTS WERE REVIEWED, AS BELOW:1 CURRENT ACCOUNT 006-366728-001 REVIEW PERIOD: 27082021 TO 05122021 CREDITS WERE BY WAY OF:O RTGS/NEFT/IMPS AGGREGATING TO RS2248 CRORES 75 TRANCHES, OF WHICH:? RS1234 CRORES 32 TRANCHES WERE FROM RAPID ENTERTAINMENT AND MEDIAWORKS A/C: 2121234033182742 OF AU SMALL FINANCE BANK LIMITED? RS898 CRORES 34 TRANCHES WERE FROM CAPITAL ENTERTAINMENT A/C:10067204005 OF IDFC BANK & A/C: 200001375028 OF EQUITAS SMALL FINANCE BANK LIMITED? RS58 LAKHS 3 TRANCHES WERE FROM TRIFOLD FILMS A/C: 2121234033430278 OF AU SMALL FINANCE BANK LIMITED? RS3288 LAKHS 4 TRANCHES WERE FROM Z S ENTERPRISES A/C: 0100102000094540 OF IDBI BANK, AND? RS25 LAKHS 2 TRANCHE WERE FROM HENAL ENTERPRISES A/C: 0179360000001649 OF DBS BANK LTDO A CHEQUE DEPOSIT OF RS50,000 FROM ITS OWN ACCOUNT HELD WITH ICICI BANK TOWARDS OPENING THE ACCOUNT DEBITS WERE BY WAY OF:O OUTWARD REMITTANCES AGGREGATING TO RS2036 CRORES 35 TRANCHES BEING ADVISED AS AUDIO VISUAL RELATED SERVICES LIKE MOTION PICTURES, VIDEO TAPE PRODUCTION, DISTRIBUTION AND PRODUCTION SERVICES, OF WHICH:? RS1036 CRORES 17 TRANCHES WERE TOWARDS FILMNATION ENTERTAINMENT LLC, UNITED ARAB EMIRATES? RS768 CRORES 14 TRANCHES WERE TOWARDS SUNRISE AUDIO VISUALS PTE LTD, SINGAPORE? RS176 CRORES 3 TRANCHES WERE TOWARDS MARK1 GLOBAL ENTERPRISES PTE LTD, SINGAPORE, AND ? RS5641 LAKHS WERE TOWARDS EFFORT POINT INTERNATIONAL GROUP COMPANY LIMITED, CHINA O RTGS/NEFT AGGREGATING TO RS130 CRORES 9 TRANCHES, OF WHICH:? RS60 LAKHS WERE TOWARDS CAPITAL ENTERTAINMENT VENDOR PAYMENT? RS5865 LAKHS WERE TOWARDS FRIENDS FILMS AND ENTERTAINMENT VENDOR PAYMENT? RS630 LAKHS 6 TRANCHES WERE TOWARDS THE PROPRIETORS PERSONAL ACCOUNT HELD WITH ICICI BANK A/C: 041801004577 AND INDIAN BANK A/C: 6024524004? RS5 LAKHS WERE TOWARDS THE STANDARD FILM COMPANY VENDOR PAYMENTO CASH WITHDRAWALS AGGREGATING TO RS201 LAKHS 3 TRANCHES O CHEQUES ISSUED AGGREGATING TO RS115 LAKHS 3 TRANCHES2 SAVINGS ACCOUNT 006-358741-006 REVIEW PERIOD: 17082021 TO 05122021 CREDITS WERE BY WAY OF IMPS FOR RS150 LAKHS FROM 3 WINGS PRODUCTION DEBITS WERE BY WAY OF IMPS AGGREGATING TO RS127 LAKHS 9 TRANCHES TO HIS OWN ACCOUNT HELD WITH ICICI BANK",{"entities":[(2883,2910,"FOREIGN ORG"),(2973,3001,"FOREIGN ORG"),(3052,3083,"FOREIGN ORG"),(3129,3176,"FOREIGN ORG")  ]}),
            ("OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO HIGH VOLUME OF SHIPPING BILLS PENDING AS PER EDPMS AMOUNTING TO RS 49541 LAKHS CUSTOMER IS NOT SUBMITTING DOCUMENTS AFTER MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASE 2 SINCE THE SHIPPING BILLS ARE GENERATED, THE EXPORTS HAS ALREADY TAKEN PLACE NEITHER CUSTOMER HAS SUBMITTED EXPORT DOCUMENT WITH US NOR ANY OTHER BANK HAS REQUESTED FOR AD TRANSFER OF THE SHIPPING BILLS ACCORDINGLY, POSSIBLY EXPORT IS DONE ONLY FOR VALUE TRANSFER FROM INDIA TO OVERSEAS 3 SHIPPING BILLS PENDING FOR FIVE YEARS 2016-2021 FOR INR 38173 LAKHS WHILE TOTAL EXPORT DONE BY CUSTOMER AMOUNTING TO INR 330717 LAKHS 4 THE EXPORT PENDENCY REFLECTING IN EDPMS ALSO FOR FOUR YEARS 2017-2021 FOR INR 49541 LAKHS 5CUSTOMER HAS RECEIVED REMITTANCE MAINLY FROM TOPMOST SHIPPING LLC, CN INTERNATIONAL, YH TRADING, MOH TRADING CO LTD MAINLY FROM UNITED STATES , DUBAI, SOUTH KOREA ONLY TRANSACTIONS SEEMS IN LINE WITH CONTRA PARTIES 6 CUSTOMER HAS DONE TRANSACTIONS WITHIN GROUP ACCOUNT ONLY 3 STR IS FILED IN LIGHT OF A SHIPPING BILLS PENDING AS PER EDPMS AGGREGATING TO RS49541 LAKHS B CUSTOMER IS NOT READY TO SUBMIT SUPPORTIVE DOCUMENTS C CUSTOMER HAS DONE TRANSACTIONS WITHIN GROUP ACCOUNT ONLY C THE CASE IS BEING REPORTED UNDER TRADE BASED MONEY LAUNDERING TBML SCENARIO1 IN OUR REGULAR TRANSACTION MONITORING, CUSTOMER U S CLOTHING INDIA PVT LTD MAINTAINING A/C 50200010273600 GOT IDENTIFIED UNDER HIGH VOLUME OF SHIPPING BILLS PENDING AS PER EDPMS AMOUNTING TO RS 49541 LAKHS BASIS WHICH CASE IS CONSIDERED FOR REPORTING 2CUSTOMER U S CLOTHING INDIA PVT LTD DOI-13-DEC-01 HAS DECLARED UNDER KYC DOCUMENT AS A MANUFACTURER OF GARMENT WITH ANNUAL TURNOVER OF RS 500 LAKHS CUSTOMER IS MAINTAINING A A/C 50200010273600 WITH KANDLA KANDLA GUJARAT BRANCH SINCE 09 FEBRUARY 2015 CUSTOMER PAN, MOBILE AND REGISTERED ADDRESS ARE AAACU4836H,+ 919825251414 AND PLOT NO 274 SECTOR - 4 KASEZ GANDHIDHAM 370201 RESPECTIVELY CUSTOMER ACCOUNT STATUS IS REGULAR CUSTOMER IS CORPORATE CONCERN RISK CATEGORIZATION IS LOW CUSTOMER ACCOUNT KYC HAS BEEN DONE CURRENT AVAILABLE BALANCE IN THE ACCOUNT IS RS -490618200 GURVINDER SINGH TOOR AND HANIF SALE MOHAMMAD TALU ARE THE AUTHORIZED SIGNATORY OF ACCOUNT 2DETAILS OF LINKED ACCOUNT 1CUSTOMER IS HAVING ACCOUNT NUMBER 06098730000018 NAME OF US CLOTHING INDIA PVT LTD SINCE 03-NOV-2012 2 CUSTOMER IS HAVING ACCOUNT NUMBER 06092320000074 NAME OF US CLOTHING INDIA PVT LTD SINCE 22-SEP-2007 3CUSTOMER IS HAVING ACCOUNT NUMBER 23128730000121 NAME OF US CLOTHING INDIA PVT LTD SINCE 07-MAY-2013 4CUSTOMER IS HAVING ACCOUNT NUMBER 50200016042550 NAME OF US CLOTHING INDIA PVT LTD SINCE 05-DEC-2015 3 SCRUTINY OF CUSTOMER U S CLOTHING INDIA PVT LTD ACCOUNT 50200010273600 STATEMENTS FOR THE PERIOD 01-07-2020 TO TILL DATE, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 36673 LAKHS WHICH LARGELY COMPRISES OF, FUND TRANSFER AGGREGATING TO RS 2612 LAKHS MAINLY FROM ACCOUNT NUMBER 50200034627071 - B V H MANUFACTURING INDIA PVT LTD, RTGS/NEFT AGGREGATING TO RS 12250 LAKHS MAINLY FROM OWN GROUP ACCOUNT U S CLOTHING INDIA PVT LTD -MAINTAINED ACCOUNT WITH AXIS BANK IFSC CODE--UTIB0000178, INWARD REMITTANCE AGGREGATING TO RS 20352 LAKHS MAINLY FROM TOPMOST SHIPPING LLC FROM UNITED STATES, ALOYS NIYONAMBAZA FROM UNITED STATE OUT OF TOTAL DEPOSITS, WHICH ARE UTILIZED TOWARDS RTGS/NEFT AGGREGATING TO RS 33251 LAKHS MAINLY TO OWN GROUP ACCOUNT U S CLOTHING INDIA PVT LTD -MAINTAINED ACCOUNT WITH AXIS BANK IFSC CODE--UTIB0000178, CLEARING DEBIT AGGREGATING TO RS 227 LAKHS 4 NO TRANSACTIONS HAVE BEEN OBSERVED IN ALL THE LINKED OF CUSTOMER SINCE JUL20",{"entities":[(792,811,"FOREIGN ORG"),(814,829,"FOREIGN ORG"),(832,841,"FOREIGN ORG"),(844,862,"FOREIGN ORG")         ]}),
            ("DETAILS OF INVESTIGATION 1 OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO CUSTOMER SHIPPING BILLS PENDING AMOUNTING TO RS 2138 LAKHS CUSTOMER IS NOT SUBMITTING DOCUMENTS AFTER MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASE 2 CUSTOMER HAS NEITHER SUBMITTED EXPORT DOCUMENT WITH US NOR REQUESTED US TO ENABLE E-FIRC IN EDPMS FOR SUBMISSION OF DOCUMENTS TO ANOTHER BANK HENCE POSSIBLY EXPORT SHIPMENT IS NOT PROCESSED AND ADVANCE IS RECEIVED ONLY FOR FUND TRANSFER FROM OVERSEAS TO INDIA 3 SHIPPING BILLS PENDING FOR TWO YEARS 2017-2021 FOR INR 2138 LAKHS WHILE TOTAL EXPORT DONE BY CUSTOMER AMOUNTING TO INR 2138 LAKHS 4 CUSTOMER HAS RECEIVED REMITTANCE MAINLY FROM MAINLY FROM TRITON POLAND SP Z OO TRITON BIG SAMS LIMITED UNITED STATES, GLOBUS RESOURCES LIMITED WE HAVE NOT FOUND ANY INFORMATION RELATED TO CONTRA PARTIES ON PUBLIC DOMAIN WHICH SEEMS UNUSUAL 5 ONE LEG OF MERCHANTING TRADE PENDING AMOUNTING TO RS125709 LAKHS WHICH ARE IN OVERDUE STATUS,4 STR IS FILED IN LIGHT OF A SHIPPING BILLS PENDING AGGREGATING TO RS2138 LAKHS B CUSTOMER IS NOT READY TO SUBMIT SUPPORTIVE DOCUMENTS C THE CASE IS BEING REPORTED UNDER TRADE BASED MONEY LAUNDERING TBML SCENARIO1 IN OUR REGULAR TRANSACTION MONITORING, CUSTOMER INDEPESCA OVERSEAS PVT LTD ANDHERI UNIT MAINTAINING A/C 00162560000796 GOT IDENTIFIED UNDER HIGH VOLUME OF SHIPPING BILLS PENDING AMOUNTING TO RS 2138 LAKHS BASIS WHICH CASE IS CONSIDERED FOR REPORTING 2CUSTOMER INDEPESCA OVERSEAS PVT LTDANDHERI UNIT DOI-4-MAR-04 HAS DECLARED UNDER KYC DOCUMENT AS A SEA FOOD EXPORTS WITH ANNUAL TURNOVER OF RS4000 LAKHS CUSTOMER IS MAINTAINING A A/C 00162560000796 WITH KURLA - KAMANI MUMBAI MAHARASHTRA BRANCH SINCE 14 JULY 2007 CUSTOMER PAN, MOBILE AND REGISTERED ADDRESS ARE AAICS1304G,+ 919820300388 AND UNIT 11A TOWER-I COMMERCIAL-II KOHINOOR CITY KIROL ROAD OFF LBS ROAD KURLA W OLD PREMIER COMPOUND MUMBAI 400070 RESPECTIVELY CUSTOMER ACCOUNT STATUS IS REGULAR CUSTOMER IS CORPORATE CONCERN AND ACCOUNT RISK CATEGORIZATION IS HIGH CUSTOMER ACCOUNT KYC HAS BEEN DONE CURRENT AVAILABLE BALANCE IN THE ACCOUNT IS RS 329562069 ANIMESH VIJAY KUMAR, PRAVENN KUMAR RAO,RAJU SAMTANI, SHRI HARISHNKAR MAIRAL AND LAVIN A SAMTHANI ARE THE AUTHORIZED SIGNATORY OF ACCOUNT 2DETAILS OF LINKED ACCOUNT 1 CUSTOMER IS HAVING ACCOUNT NUMBER 00162320001634 NAME OF INDEPESCA OVERSEAS PVT LTD-AQUACULTURE SINCE 27-MAY-2008 2 CUSTOMER IS HAVING ACCOUNT NUMBER 00162430000375 NAME OF INDEPESCA OVERSEAS PRIVATE LIMITED SINCE 16-DEC-2011 3 CUSTOMER IS HAVING ACCOUNT NUMBER 00162440000214 NAME OF INDEPESCA OVERSEAS PRIVATE LIMITED SINCE 06-NOV-2012 4CUSTOMER IS HAVING ACCOUNT NUMBER 00162320000537 NAME OF INDEPESCA OVERSEAS PRIVATE LIMITED SINCE 04-OCT-2004 5 CUSTOMER IS HAVING ACCOUNT NUMBER 02402430000425 NAME OF INDEPESCA OVERSEAS PRIVATE LIMITED SINCE 06-SEP-2005 6 CUSTOMER IS HAVING ACCOUNT NUMBER 00162320001600 NAME OF INDEPESCA OVERSEAS PVT LTD CORPORATE SINCE 23-MAY-2008 3 SCRUTINY OF CUSTOMER INDEPESCA OVERSEAS PVT LTD ANDHERI UNIT ACCOUNT 00162560000796 STATEMENTS FOR THE PERIOD 01-07-2020 TO TILL DATE, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 100746 LAKHS WHICH LARGELY COMPRISES OF, FUND TRANSFER AGGREGATING TO RTGS/NEFT AGGREGATING TO RS 9132 LAKHS MAINLY FROM AMPA ORTHODONTICS PRIVATE LIMITED MAINTAINED ACCOUNT WITH ICICI BANK IFSC-ICIC0000104, INWARD REMITTANCE AGGREGATING TO RS 90337 LAKHS MAINLY FROM TRITON POLAND SP Z OO TRITON BIG SAMS LIMITED FROM UNITED STATES OUT OF TOTAL DEPOSITS, WHICH ARE UTILIZED, RTGS/NEFT AGGREGATING TO RS 5864 LAKHS MAINLY TO INDEPESCA OVERSEAS PRIVATE LTD MAINTAINED ACCOUNT WITH INDIAN BANK IFSC-IDIB000V021, FUND TRANSFER AGGREGATING TO RS 88922 LAKHS MAINLY TOWARDS A/C- 00162320000537 - INDEPESCA OVERSEAS PRIVATE LIMITED, REGULAR EXPENSES SALARY PAYMENT , CLEARING DEBIT AGGREGATING TO RS 1810 LAKHS 4 SIMILAR TRANSACTIONS HAVE BEEN OBSERVED IN ALL THE LINKED OF CUSTOMER SINCE JUL20",{"entities":[(690,708,"FOREIGN ORG"),(709,735,"FOREIGN ORG"),(751,775,"FOREIGN ORG")       ,(3412,3438,"FOREIGN ORG")]}),
            ("BRANCH HAS BEEN FOLLOWING UP WITH THE CUSTOMER FOR SUBMISSION OF EXPORT REGULARIZATION DOCUMENT HOWEVER CUSTOMER NOT SUBMITTING THE SAME 6 CUSTOMER HAS PROCESSED TOTAL EXPORTS OF INR 4552 LAKH AND IMPORT OF 3640 LAKHS WITH HDFC BANK HENCE THE EXPORT PENDENCY IS APPROX 13% 7 CUSTOMER HAS RECEIVED MAJOR ADVANCES FROM TAWAZON CHEMICAL COMPANY LLC DUBAI, D T ENGINEERING SYSTEMS LIMITED GHANA, SPRINT CHEMIE LTD SOUTH AFRICA, HAVING SAME LINE OF BUSINESS 8 NO NEGATIVE MEDIA NEW/ INFORMATION FOUND FOR THE CUSTOMER IN THE WEB SEARCH9 RECOMMENDED FOR STR TO BE FILED BASIS A CUSTOMER HAS HIGH VOLUME PENDENCY OF ADVANCE AGAINST EXPORT OF RS 70698 LAKHS, EDPMS OF RS 7975 LAKHS, EXPORT O/S OF RS 1163 LAKHS FOR WHICH SHIPPING BILLS NOT SUBMITTED, IDPMS OF RS 726 LAKHS, AND IDPMS IMPORT DONE PAYMENT NOT SENT OF RS 16362 LAKHSB CUSTOMER IS NOT SUBMIT ANY RELEVANT DOCUMENTS AFTER MULTIPLE FOLLOW-UPS AND REMINDERS, HE IS SAYING HE WILL CHECK AND CONFIRM FROM LAST ONE YEAR AS ADVANCE AGAINST EXPORT PENDENCY IS FROM 20141 IN OUR REGULAR TRANSACTION MONITORING, CUSTOMER SETCO CHEMICALS I PVT LIMITED MAINTAINING A/C NO 01702560002238 GOT IDENTIFIED FOR TRADE PENDENCY- ADVANCE AGAINST EXPORT PENDENCY OF RS 70698 LAKHS FOR THE PERIOD 2014-2021, EXPORT DATA PROCESSING AND MONITORING SYSTEM EDPMS PENDENCY FOR PERIOD 2018-2020 OF RS 7975 LAKHS FOR WHICH SHIPPING BILL SUBMISSION AND REGULARIZATION IS PENDING CUSTOMER HAS EXPORT O/S PENDENCY FOR PERIOD 2018 AND 2019 OF RS 1163 LAKHS, FOR WHICH EXPORT REALIZATION IS PENDING CUSTOMER HAS PENDENCY OF IDPMS IMPORT DONE PAYMENT NOT SENT OF RS 16362 LAKHS FOR PERIOD 2016, 2018, 2019, 2020 AND 2021 FOR WHICH BILL OF ENTRY SUBMISSION AND SUBSEQUENT IMPORT PAYMENT IS PENDING 2 CUSTOMER SETCO CHEMICALS I PVT LIMITED MAINTAINING A/C NO 01702560002238 DATE OF INCORPORATION 03-06-1985, SINCE 25-03-2004 WITH OUR VASAI EAST - EVERSHINE CITY BRANCH, SHOP NO 30 TO 32 AND 34 TO 36, VRUNDAVAN SHOPPING CENTRE, NEAR EVERSHINE GATE, VASAI EAST, MUMBAI, MAHARASHTRA THE PAN NO IS AADCS1688L AND PROFILE OF THE CUSTOMER IS MANUFACTURING CHEMICAL MANUFACTURING CATEGORY CORPORATE PRIVATE LIMITED ANNUAL TURNOVER OF THE CUSTOMER IS RS 7929 LAKHS THE MOBILE NUMBER IS 9920031535 AND THE ADDRESS UPDATED AS SETCO CHEMICALS I PVT LTD PLOT NO290/1 PHASE 2 NEAR SARDAR CHOK G ID C VAPI, VAPI, GUJARAT-396195 RISK PROFILE OF THE CUSTOMER IS LOW AND STATUS OF THE ACCOUNT IS NO DEBIT ALLOWED BALANCE MAINTAIN IN THE ACCOUNT IS RS 000 YOGESH H SOLANKI, SHAMIT V DESAI, REKHA SOLANKI ARE AUTHORIZE SIGNATORIES OF THE ACCOUNT3 SCRUTINY OF CUSTOMER SETCO CHEMICALS I PVT LIMITED, ACCOUNT NO 01702560002238 AND 00382430000104 STATEMENTS FOR THE PERIOD JAN 2021 TO NOV-21, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 179160 LAKHS WHICH LARGELY COMPRISES OF NEFT/RTGS AGGREGATING TO RS 145809 LAKHS MAJORLY RECEIVED FROM KRIS FLEXIPACKS PVT LTD - ICIC0000104 OF INR 12076 LAKHS AND PARLE BISCUITS PVT LTD - KKBK0000958 OF INR 60850 LAKHS, INTERNAL FUND TRANSFER AGGREGATING TO RS 21444 LAKHS MAJORLY RECEIVED FROM 00051000008189 - YOGESH HARIPRASAD SOLANKI OF INR 20000 LAKHS, OTHERS CREDITS AGGREGATING TO RS 11066 LAKHS MAJOR RECEIVED FROM ACH CREDIT OUT TOTAL DEPOSITS, NEFT/RTGS AGGREGATING TO RS 145929 LAKHS TRANSFERRED TO BARB0INDVAP-SETCO CHEMICALS I PVT LTD AND KKBK0001347-SETCO CHEMICALS I PVT LIMITED OF INR 137438 LAKHS, INTERNAL FUND TRANSFER AGGREGATING TO RS 26461 LAKHS TRANSFERRED TO 00051000008076 - REKHA SOLANKI OF INR 20000 LAKHS, OTHERS DEBITS AGGREGATING TO RS 7291 LAKHS 4 NO MAJOR TRANSACTION FOUND IN ACCOUNT NO 07372790000091 AND 00388640000016 IN LAST ONE YEAR AND ACCOUNT NO 50200018286015 CLOSED ON 2017, 50200018286015 CLOSED ON 2017, 00388640000016 CLOSED ON 2021 AND 00388640000016 CLOSED ON 2021",{"entities":[(317,345,"FOREIGN ORG"),(353,384,"FOREIGN ORG"),(392,409,"FOREIGN ORG")        ]}),
            ("OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO CUSTOMER S SHIPPING BILLS PENDENCY AMOUNTING TO RS 270152 LAKHS AS PER ADVANCE AGAINST EXPORT DATA AND RS 43530 LAKHS AS PER EDPMS WHICH THE CUSTOMER IS NOT SUBMITTING DESPITE MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASE CUSTOMER IS REFLECTING IN THE CAUTION LIST OF RBI AS WELLCUSTOMER HAS NOT PROCESSED ANY EXPORT WITH HDFC BANK HENCE THE ADVANCE AGAINST EXPORT PENDENCY AND EDPMS PENDENCY IS 100% OF THE EXPORT PROCESSED THROUGH US 8 SINCE THE SHIPPING BILLS ARE GENERATED, THE EXPORTS HAS ALREADY TAKEN PLACE NEITHER CUSTOMER HAS SUBMITTED EXPORT DOCUMENT WITH US NOR ANY OTHER BANK HAS REQUESTED FOR AD TRANSFER OF THE SHIPPING BILLS ACCORDINGLY POSSIBLY EXPORT IS DONE ONLY FOR VALUE TRANSFER FROM INDIA TO OVERSEAS9 CUSTOMER HAS NEITHER SUBMITTED EXPORT DOCUMENT WITH US NOR REQUESTED US TO ENABLE E-FIRC IN EDPMS FOR SUBMISSION OF DOCUMENTS TO ANOTHER BANK HENCE POSSIBLY EXPORT SHIPMENT IS NOT PROCESSED AND ADVANCE IS RECEIVED ONLY FOR FUND TRANSFER FROM OVERSEAS TO INDIA THE MAJOR ADVANCE AGAINST EXPORT WERE RECEIVED FROM M/S SILK CRAFTS INC USA, KRAVET INC USA, ROMO LIMITED GREAT BRITAIN AS PER WEB SEARCH CONTRA PARTIES ARE SAME LINE OF BUSINESS 10 STR FILED IN THE LIGHT OF I CUSTOMER IS NOT READY SUBMIT THE SHIPPING DESPITE REGULAR REMINDERS BY THE BANK II CUSTOMER IS ENLISTED IN THE CAUTION LIST OF RBI III THE CASE IS BEING REPORTED UNDER TRADE BASED MONEY LAUNDERING TBML SCENARIO1 IN OUR REGULAR TRANSACTION MONITORING, CUSTOMER EASTERN SILK INDUSTRIES LIMITED MAINTAINING A/C 00082050006906 GOT IDENTIFIED UNDER HIGH VOLUME OF SHIPPING BILLS PENDENCY AMOUNTING TO RS 270152 LAKHS AS PER ADVANCE AGAINST EXPORT DATA 2 CUSTOMER EASTERN SILK INDUSTRIES LIMITED MAINTAINING A/C NO 00082050006906 DATE OF INCORPORATION 09-04-1946, SINCE 28-03-2006 WITH OUR STEPHEN HOUSE BRANCH, KOLKATA, WEST BENGAL THE PAN NO IS AAACE5807C AND PROFILE OF THE CUSTOMER IS MANUFACTURING TEXTILE MANUFACTURING ANNUAL TURNOVER OF THE CUSTOMER IS RS 500 LAKHS THE MOBILE NUMBER IS 9038017486 AND THE ADDRESS UPDATED AS 19 R N MUKHERJEE ROAD, KOLKATA, WEST BENGAL-700001 RISK PROFILE OF THE CUSTOMER IS LOW AND STATUS OF THE ACCOUNT IS CLOSED BALANCE MAINTAIN IN THE ACCOUNT IS RS 000 SHYAM SUNDER SHAH, SUNDEEP SHAH ARE AUTHORIZE SIGNATORIES OF THE ACCOUNT 3 CUSTOMER ALSO MAINTANS 20 MORE ACCOUNT WITH OUR BANK DETAILS OF THE ACCOUNT MENTIONED BELOW A/C NO 00142220001114 SINCE 30-08-2007, A/C NO 00532050000392 SINCE 09-08-2006, A/C NO 05098730000128 SINCE 17-09-2012, A/C NO 00142220000960 SINCE 30-10-2006, A/C NO 00142220001539 SINCE 21-08-2009, A/C NO 50200005079263 SINCE 10-04-2014, A/C NO 50200002898486 SINCE 11-11-2013, A/C NO 50200007167940 SINCE 18-08-2014, A/C NO 00262320000050 SINCE 23-10-2000, A/C NO 10152320001531 SINCE 23-11-2009, A/C NO 00532320000724 SINCE 30-03-2002, A/C NO 50200012136233 SINCE 22-05-2015, A/C NO 50200015958208 SINCE 01-12-2015, A/C NO 00142220001772 SINCE 20-08-2010, A/C NO 00652320001412 SINCE 28-03-2007, A/C NO 00142220000943 SINCE 11-09-2006, A/C NO 13552320000033 SINCE 10-09-2005, A/C NO 13552020001681 SINCE 07-03-2003, A/C NO 00142220001313 SINCE 16-08-2008, A/C NO 10152320001521 SINCE 23-11-2009 4 SCRUTINY OF CUSTOMER EASTERN SILK INDUSTRIES LIMITED NO 50200007167940STATEMENTS FOR THE PERIOD JAN 2014 TO DEC 2016, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 1491626 LAKHS WHICH LARGELY COMPRISES OF INWARD REMITTANCE AGGREGATING TO RS 1082473 LAKHS, INTERNAL FUND TRANSFER AGGREGATING TO RS 360713 LAKHS MAINLY FROM A/C NO 50200004525649 OF GEMINI OVERSEAS LIMITED, A/C NO 50200004748540 OF ETHICS COMMERCIALS LIMITEDOUT TOTAL DEPOSITS, INTERNAL FUND TRANSFER AGGREGATING TO RS 11913646 LAKHS MAINLY TO A/C NO 00532050000392 OF EASTERN SILK INDUSTRIES LTD-KORAMANGALA, A/C NO 50200004525649 OF GEMINI OVERSEAS LIMITED, NEFT/RTGS AGGREGATING TO RS 221856 LAKHS MAINLY TO EASTERN SILK INDUSTRIES LTD SBIN0010508 5 SCRUTINY OF CUSTOMER EASTERN SILK INDUSTRIES LTD-KORAMANGALA A/C NO 00532050000392 STATEMENTS FOR THE PERIOD JAN 2014 TO DEC 2016, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 1250169 LAKHS WHICH LARGELY COMPRISES OF INTERNAL FUND TRANSFER AGGREGATING TO RS 659269 LAKHS MAINLY FROM A/C NO 50200007167940 OF EASTERN SILK INDUSTRIES LIMITED, NEFT/RTGS AGGREGATING TO RS 488215 MAINLY FROM EASTERN SILK INDUSTRIES LTD SBMY0040259 AND SBIN0010508OUT TOTAL DEPOSITS, INTERNAL FUND TRANSFER AGGREGATING TO RS 796463 LAKHS MAINLY TO A/C NO 00532320000724 OF EASTERN SILK INDUSTRIES LIMITED, A/C NO 10152320001418 OF LUCKY GOLDSTAR CO LTD, CHEQUE WITHDRAWAL AGGREGATING TO RS 298696 LAKHS 6 NO MAJOR TRANSACTION OBSERVED IN THE ABOVE MENTIONED ACCOUNT FROM JAN 2019",{"entities":[(1102,1117,"FOREIGN ORG"),(1123,1133,"FOREIGN ORG"),(1139,1151,"FOREIGN ORG")          ]}),
            ("OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO CUSTOMERS ADVANCE AGAINST EXPORT, EDPMS, IDPMS AND IDPMS IMPORT DONE PAYMENT NOT SENT PENDENCY, WHICH THE CUSTOMER IS NOT SUBMITTING DESPITE MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASETHE MAJOR PENDENCY ARE WITH FOLLOWING PARTIES1/CHORI INDONESIA, PT UNITED STATES OF AMERICA, ACETO CORP UNITED STATES OF AMERICA, ACETO FRANCE SAS FRANCE, AJAY EUROPE SARL FRANCE, ALHOBAIL MEDICAL OFFICE CO UNITED STATES OF AMERICA, CHATTEM, INC UNITED STATES OF AMERICA, FAMAR NEDERLAND BV NEDERLAND, MACARTHYS LABORATORIES LIMITED UNITED STATES OF AMERICA, HAVING SAME LINE OF BUSINESS6 TOTAL EXPORTS PROCESSED BY THE CUSTOMER WITH HDFC BANK IS INR 1464117 LAKHS, HENCE THE EDPMS PENDENCY IS 283% AND ADVANCE PENDENCY IS781 % OF THE EXPORT PROCESSED THROUGH US SINCE THE SHIPPING BILLS ARE GENERATED, THE EXPORTS HAS ALREADY TAKEN PLACE NEITHER CUSTOMER HAS SUBMITTED EXPORT DOCUMENT WITH US NOR ANY OTHER BANK HAS REQUESTED FOR AD TRANSFER OF THE SHIPPING BILLS ACCORDINGLY POSSIBLY EXPORT IS DONE ONLY FOR VALUE TRANSFER FROM INDIA TO OVERSEAS7 CUSTOMER HAS NEITHER SUBMITTED EXPORT DOCUMENT WITH US NOR REQUESTED US TO ENABLE E-FIRC IN EDPMS FOR SUBMISSION OF DOCUMENTS TO ANOTHER BANK HENCE POSSIBLY EXPORT SHIPMENT IS NOT PROCESSED AND ADVANCE IS RECEIVED ONLY FOR FUND TRANSFER FROM OVERSEAS TO INDIA8 NO NEGATIVE MEDIA NEW/ INFORMATION FOUND FOR THE CUSTOMER IN THE WEB SEARCH 9 RECOMMENDED FOR STR TO BE FILED BASIS A CUSTOMER HAS HIGH VOLUME PENDENCY OF ADVANCE AGAINST EXPORT OF RS 114373 LAKHS, EDPMS PENDENCY OF RS 48370 LAKHS, IDPMS PENDENCY OF RS 19883 LAKHS AND IDPMS IMPORT DONE PAYMENT NOT SENT PENDENCY FOR RS 15815 LAKHSB CUSTOMER IS NOT SUBMITTED ANY RELEVANT DOCUMENTS AFTER MULTIPLE FOLLOW-UPS AND REMINDERSC CUSTOMER IS ENLISTED IN THE CAUTION LIST OF RBI1 DURING THE REVIEW OF ADVANCE AGAINST EXPORT, OUR BANK HAS IDENTIFIED ONE OF OUR CUSTOMER ESKAY SPECIALITY CHEMICALS A/C NO 02402000014147, WHEREIN IT WAS OBSERVED THAT CUSTOMER HAS NOT SUBMITTED EXPORT DOCUMENT WORTH RS 114373 LAKHS FOR THE YEAR 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, OF EDPMS PENDENCY OF RS 48370 LAKHS FOR PERIOD 2018-2021 FOR WHICH SHIPPING BILL ARE PENDING FOR REGULARIZATION IDPMS PENDENCY OF RS 19883 LAKHS FOR PERIOD 2017, 2019 AND 2021 AND IDPMS IMPORT DONE PAYMENT NOT SENT PENDENCY FOR RS 15815 LAKHS FOR PERIOD 2016-20212 CUSTOMER ESKAY SPECIALITY CHEMICALS MAINTAINING A/C NO 02402000014147 DATE OF INCORPORATION 06-12-2006, SINCE 12-12-2006 WITH OUR WORLI - SANDOZ HOUSE BRANCH, MUMBAI, MAHARASHTRA THE PAN NO IS AACFE2324F AND PROFILE OF THE CUSTOMER IS MANUFACTURING PHARMA MANUFACTURING OF CHEMICAL ANNUAL TURNOVER OF THE CUSTOMER IS RS 3096 LAKHS THE MOBILE NUMBER IS AND THE ADDRESS UPDATED AS A-3 SHIVSAGAR ESTATE DR ANNIE BESANT ROAD WORLI, MUMBAI, MAHARASHTRA-400018 RISK PROFILE OF THE CUSTOMER IS HIGH AND STATUS OF THE ACCOUNT IS REGULAR CUSTOMER IS A PARTNERSHIP AND BALANCE MAINTAIN IN THE ACCOUNT IS RS 3571741 BIPIN NEMCHAND SHAH POA, KETAN L SHAH, LALIT P SHAH, NEETA K SHAH, SAMIR J SHAH ARE AUTHORIZE SIGNATORIES OF THE ACCOUNT3 FOLLOWING ARE LINKED ACCOUNT OF THE CUSTOMER A/C NO 50200058281002, A/C NO 02400330002626, A/C NO 02402670000321 OF ESKAY SPECIALITY CHEMICALS4 SCRUTINY OF CUSTOMER ESKAY SPECIALITY CHEMICALS, ACCOUNT NO 02402000014147, 50200058281002, 02400330002626 AND 02402670000321 STATEMENTS FOR THE PERIOD JAN 2021 TO NOV-21, TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 728150 LAKHS WHICH LARGELY COMPRISES OF NEFT/RTGS AGGREGATING TO RS 5305 LAKHS, INTERNAL FUND TRANSFER AGGREGATING TO RS 388610 LAKHS MAJORLY RECEIVED FROM HDFC MUTUAL FUND RE OF INR 119479 LAKHS, KETAN LALITKUM-LOAN TAKEN OF INR 94925 LAKHS, EXPORT BILL LIQUIDATED OF INR 280408 LAKHS, OTHER CREDITS AGGREGATING TO RS 53041 LAKHS RECEIVED FROM INWARD REMITTANCE OF INR 51074 LAKHS OUT TOTAL DEPOSITS NEFT/RTGS AGGREGATING TO RS 53878 LAKHS MAJORLY TRANSFERRED TO ANITA GARG - BKID0000049 OF INR 1425 LAKHS, ANANT SHANKAR MATE - BKID0000076 OF INR 1671 LAKHS, HEMAL G GALAIYA - BKID0000049 OF INR 1000 LAKHS, INTERNAL FUND TRANSFER AGGREGATING TO RS 529315 LAKHS MAJORLY TRANSFERRED TO 02401000107628-LOAN REPAY OF INR 125670 LAKHS, 00600350006482-INVESTMENT OF INR 41500 LAKHS",{"entities":[(341,351,"FOREIGN ORG"),(378,394,"FOREIGN ORG"),(403,419,"FOREIGN ORG"),(428,454,"FOREIGN ORG"),(481,493,"FOREIGN ORG"),(550,580,"FOREIGN ORG")]}),
            ("OUR BRANCH OFFICIALS HAD RAISED THE CASE FOR REPORTING DUE TO CUSTOMERS EDPMS AND ADVANCE AGAINST EXPORT PENDENCY, WHICH THE CUSTOMER IS NOT SUBMITTING DESPITE MULTIPLE FOLLOW UPS AND HENCE RAISED THE CASETHE MAJOR PENDENCY ARE WITH FOLLOWING PARTIESNUANCE COMMUNICATIONS INC UNITED STATES OF AMERICA, HAVING SAME LINE OF BUSINESS6 CUSTOMER HAS NOT PROCESSED ANY EXPORT WITH HDFC BANK HENCE THE ADVANCE AGAINST EXPORT PENDENCY IS 100% SINCE THE SHIPPING BILLS ARE GENERATED, THE EXPORTS HAS ALREADY TAKEN PLACE NEITHER CUSTOMER HAS SUBMITTED EXPORT DOCUMENT WITH US NOR ANY OTHER BANK HAS REQUESTED FOR AD TRANSFER OF THE SHIPPING BILLS ACCORDINGLY POSSIBLY EXPORT IS DONE ONLY FOR VALUE TRANSFER FROM INDIA TO OVERSEAS7 CUSTOMER HAS NEITHER SUBMITTED EXPORT DOCUMENT WITH US NOR REQUESTED US TO ENABLE E-FIRC IN EDPMS FOR SUBMISSION OF DOCUMENTS TO ANOTHER BANK HENCE POSSIBLY EXPORT SHIPMENT IS NOT PROCESSED AND ADVANCE IS RECEIVED ONLY FOR FUND TRANSFER FROM OVERSEAS TO INDIA8 NO NEGATIVE MEDIA NEW/ INFORMATION FOUND FOR THE CUSTOMER IN THE WEB SEARCH 9 RECOMMENDED FOR STR TO BE FILED BASIS A CUSTOMER HAS HIGH VOLUME PENDENCY OF EDPMS OF RS 59367 LAKHS AND ADVANCE AGAINST EXPORT OF RS 43858 LAKHSB CUSTOMER IS NOT SUBMITTED ANY RELEVANT DOCUMENTS AFTER MULTIPLE FOLLOW-UPS AND REMINDERSC CUSTOMER IS ENLISTED IN THE CAUTION LIST OF RBI1 DURING THE REVIEW OUR BANK HAS IDENTIFIED ONE OF OUR CUSTOMER SERVTECH SYSTEMS INDIA PRIVATE LIMITED A/C NO 05442320000153, WHEREIN IT WAS OBSERVED THAT CUSTOMER HAS NOT SUBMITTED EXPORT DOCUMENT FOR EDPMS WORTH RS 59367 LAKHS FOR THE 2014-2016 AND PENDENCY OF ADVANCE AGAINST EXPORT WORTH RS 43858 LAKHS FOR THE YEAR 2015 FOR WHICH SHIPPING BILL ARE PENDING FOR REGULARIZATION2 CUSTOMER SERVTECH SYSTEMS INDIA PRIVATE LIMITED MAINTAINING A/C NO 05442320000153 DATE OF INCORPORATION 00-01-1900, SINCE 02-03-2005 WITH OUR TARDEO WADIA STREET BRANCH, MUMBAI, MAHARASHTRA THE PAN NO IS AAFCS6389E AND PROFILE OF THE CUSTOMER IS HARDWARE AND SOFTWARE CONSULTAN ANNUAL TURNOVER OF THE CUSTOMER IS RS 150 LAKHS THE MOBILE NUMBER IS 9901904358 AND THE ADDRESS UPDATED AS TODI ESTATEA-WING 2ND FLOOR SUN MILL COMPOUND LOWER PAREL, MUMBAI, MAHARASHTRA-400013 RISK PROFILE OF THE CUSTOMER IS HIGH AND STATUS OF THE ACCOUNT IS BLOCKED AND DORMANT CUSTOMER IS A CORPORATE AND BALANCE MAINTAIN IN THE ACCOUNT IS RS 10188878 MUSTAK B ODIYA, RAVINDRA DHARESHIVKAR ARE AUTHORIZE SIGNATORIES OF THE ACCOUNT3 FOLLOWING ARE LINKED ACCOUNT OF THE CUSTOMER A/C NO 05442430000016 OF SERVTECH SYSTEMS INDIA PRIVATE LIMITED4 SCRUTINY OF CUSTOMER SERVTECH SYSTEMS INDIA PRIVATE LIMITED, ACCOUNT NO 05442320000153 AND 05442430000016 STATEMENTS FOR THE PERIOD JAN 2015 TO DEC-17, AS ACCOUNT ARE BLOCKED AND DORMANT OF CLOSED TOTAL DEPOSITS IN THE ACCOUNT ARE AGGREGATING TO RS 128565 LAKHS WHICH LARGELY COMPRISES OF OTHERS CREDITS AGGREGATING TO RS 127136 LAKHS MAJORLY RECEIVED FROM INWARD REMITTANCE OF INR 127104 LAKHS OUT TOTAL DEPOSITS, NEFT/RTGS AGGREGATING TO RS 58898 LAKHS MAJORLY TRANSFERRED TO CITI0000004-SERVTECH SYSTEMS INDIA PVT LTD OF INR 58350 LAKHS, INTERNAL FUND TRANSFER AGGREGATING TO RS 12059 LAKHS MAJORLY TRANSFERRED TO 50200004366861 - VORA OPTICS OF INR 2682 LAKHS, JUBILEEHILLS-54-505299 DR 57500000326520 OF INR 1200 LAKHS, CHEQUE ISSUANCE AGGREGATING TO RS 14308 LAKHS, OTHERS DEBITS AGGREGATING TO RS 56019 LAKHS MAJORLY TRANSFERRED TO 051030812031502036- CBDT TAX OF INR 11500 LAKHS, SALARY OF INR 36212 LAKHS",{"entities":[(243,275,"FOREIGN ORG")     ]}),
            ("FUNDS WERE ALSO REMITTED OVERSEAS AS DIRECT IMPORT REMITTANCE RS498 CR MAINLY FAVOURING AUSUM INTERNATIONAL PTE LTD- SINGAPORE RS260 CR AND STARGRADE MEDIA AND EQUIPMENT PTE LTD SINGAPORE RS238 CR TOWARDS IMPORT OF SOFTWARE 4 TRANSACTIONS IN THE ACCOUNT WITHIN A SHORT SPAN OF 3 MONTHS FAR EXCEED THE ANNUAL TURNOVER INDICATED BY CUSTOMER DURING ACCOUNT OPENING A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE RTGS/NEFT CREDITS WERE FOLLOWED BY IMMEDIATE DIRECT IMPORT PAYMENTS 5 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS OF PERFECT SALES AND MARKETING, IT WAS A 1000 SQ FT RENTED PREMISE WITH SIGN BOARD, EMPLOYEES AND OFFICE SET UP SEEN HOWEVER THE GENUINENESS OF BUSINESS EXISTENCE IS A DOUBTFUL 6 ON SCRUTINY OF SAMPLE DOCUMENTS OUR TRADE DESK NOTED THAT AS PER INVOICE FUNDS WERE REMITTED AS DIRECT IMPORT PAYMENTS TOWARDS SOFTWARE CONSULTANCY ADVISORY MAINTENANCE/ GPRS TRACKER HOWEVER CERTIFICATE FURNISHED BY CHARTERED ACCOUNTANT INDICATED THAT THE SOFTWARE WAS PURCHASED AND SUCCESSFULLY DOWNLOADED SINCE THE INFORMATION IN THE INVOICE APPEARS TO CONTRADICT THE DETAILS CONFIRMED IN CA CERTIFICATE, IT COULD BE POSSIBLE THAT THE DOCUMENTS MAY HAVE BEEN FABRICATED MERELY TO SATISFY BANK OFFICIAL TO REMIT FUNDS OVERSEAS IT WAS ALSO NOTED THAT THOUGH THE CUSTOMER WAS BASED OUT OF MUMBAI, THE CHARTERED ACCOUNTANT CERTIFYING THE TRANSACTION WAS BASED OUT OF DELHI 7 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS DOUBTFUL THAT THE 33 YEAR OLD PROPRIETOR MAY BE THE ACTUAL BENEFICIAL OWNER OF HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNT IT ALSO APPEARS THAT THE TRANSACTIONS ROUTED THROUGH ACCOUNT OF PERFECT SALES AND MARKETING MAY HAVE NO ECONOMIC RATIONALE AND MAY HAVE BEEN ROUTED FOR LAYERING OF FUNDS AND TO REMIT FUNDS OVERSEAS UNDER PRETEXT OF NON-PHYSICAL IMPORT SOFTWARE POSSIBLY WITH THE INTENTION OF TBML QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021-22 IS DEBIT: RS1180 CR IN CASH RS004 CR AND CREDIT: RS1180 CR IN CASH RS0 CRAMIT KUMAR AS A PROPRIETOR MAINTAINS A CURRENT ACCOUNT BEARING NUMBER 2845432961 IN THE NAME OF A SOLE PROPRIETORSHIP CONCERN PERFECT SALES AND MARKETING SINCE 27092021 AT OUR BRANCH LOCATED AT SHOP NO3,4 14, KHANDELWAL HOUSE, PODDAR ROAD, MALAD EAST, MUMBAI 400097 THE ENTITY WAS ESTABLISHED ON 05072021 AND HAS SUBMITTED COPIES OF PAN BSHPK3995E, GST REG CERT 27BSHPK3995E1ZE, IEC CERT BSHPK3995E AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO 301 WING A MASTERMIND 1 ROYAL PALMS, GOREGAON EAST, AAREY COLONY, S NO 169 NEAR UNIT MUMBAI 400065 AND HAS REGISTERED MOBILE NUMBER 8591798452 THE ENTITY IS CATEGORIZED AS MEDIUM RISK AND THE ACCOUNT IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF SOFTWARE SERVICES WITH ANNUAL TURNOVER OF RS200 CR THE PROPRIETOR ALSO MAINTAINS A SAVINGS ACCOUNT BEARING NO6513245150 SINCE 18-SEP-19 NO UNUSUAL TRANSACTIONS WERE NOTED IN THE SAID ACCOUNT DURING REVIEW PERIOD OUR TRADE DESK NOTICED HIGH VALUE REMITTANCES IN THE ABOVE MENTIONED NEWLY OPENED ACCOUNT OF PERFECT SALES AND MARKETING AND HENCE HIGHLIGHTED THE SAME TO AML CELL AN ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD ON THE CUSTOMER AND THIS REPORT IS FILED ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL 17122021, IT WAS NOTED THAT RTGS/NEFT/IMPS CREDITS AMOUNTED TO RS940 CR MAINLY FROM ORIENTAL TRIMEX LTD RS143 CR, ICICI BANK - 000705024167, FORTUNE OVERSEAS RS080 CR, YES BANK - 034063400000800 AND PERFECT SALES AND MARKETING RS071 CR, ICICI BANK 028105500990/ YES BANK - 100263400002018 2 SUBSEQUENT TO SUCH CREDITS, RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS432 CR MAINLY FAVOURING PURPLE MARTINI PRODUCTIONS RS192 CR, EQUITAS SMALL FINANCE BANK - 200001423553, SNOWFLY BRANDING AND ADVERTISING PVT LTD RS117 CR, ICICI - 000705052180 AND QUICK BYTES TECHNOLOGIES RS060 CR , RBL - 409001489546 CASH WITHDRAWALS THROUGH ATMS AMOUNTED TO RS004 CR",{"entities":[(78,116,"FOREIGN ORG"),(140,177,"FOREIGN ORG")     ]}),     
            ("GROUNDS OF SUSPICION THIS IS AN LEA CASE AND A SUPPLEMENTARY CASE BANK WAS IN RECEIPT OF A SUMMON DATED 2/1/221 VIDE REFERENCE NUMBER ITBA/INV/S/11/2221/129792 1 U/S 11 1A FROM NARENDRA MOLAI PRASAD, DYDIRECTOR OF INCOME TAX INV UNIT , INCOME TAX DEPARTMENT, OFFICE OF THE ASSISTANT DIRECTOR OF INCOME TAX, INVESTIGATION THE NOTICE DIRECTED THE BANK TO PROVIDE ACCOUNT RELATED INFORMATION OF R N EXPORTS HOLDING ACCOUNT NUMBERS 225686, 22562 AND 2256129 FOR INVESTIGATION PURPOSE THE INFORMATION SOUGHT HAS BEEN PROVIDED TO THE INCOME TAX DEPARTMENT REVIEW OF THE TRANSACTIONS FOR ACCOUNT 225686 FOR THE PERIOD FROM 1//22 TO 1//221 REVEALED TOTAL CREDITS OF RS7 CRORES AND TOTAL DEBITS OF RS5 CRORES MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF INTERNAL FUND TRANSFERS TOTALLING TO RS276 CRORES ON 7 INSTANCES WHICH WERE RECEIVED FROM OWN USD DIAMOND DOLLAR ACCOUNT ACCOUNT NUMBER 2256129 , RTGS/NEFT CREDITS TOTALLING TO RS5 LAKHS ON 8 INSTANCES OF WHICH RS2851 LAKHS FROM OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 21226 ON 5 INSTANCES, RS959 LAKHS FROM CARAT LANE TRADING PRIVATE LIMITED ICICI BANK LTD, ACCOUNT NUMBER 1SLNEFTPL ON 9 INSTANCES, RS82 LAKHS FROM KANTILAL CHHOTALAL BANK OF INDIA, ACCOUNT NUMBER 1672112 ON 12 INSTANCES, RS88 LAKHS FROM NAICE JEWELS PRIVATE LIMITED KOTAK MAHINDRA BANK LIMITED, ACCOUNT NUMBER 51156 ON 8 INSTANCES, RS16 LAKHS FROM CREATIVE GEMS AND JEWELLERY LTD STATE BANK OF INDIA, ACCOUNT NUMBER 2886667 ON INSTANCES, RTGS RETURNS TOTALLING TO RS661 LAKHS MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGS/NEFT TOTALLING TO RS269 CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 21226 ON 8 INSTANCES, FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS657 LAKHS, CHEQUES ISSUANCE TOTALLING TO RS829 LAKHS, IMPORT PAYMENT TOTALLING TO RS61 LAKHS TO TANU STAR HK LTD, HONG KONG ETC A REVIEW OF THE TRANSACTIONS IN USD DIAMOND DOLLAR ACCOUNT 2256129 FOR ABOVE REVIEW PERIOD REVEALED BUSINESS RELATED TRANSACTIONS SEARCHES CONDUCTED IN PUBLIC DOMAIN DID NOT REVEAL ANY ADVERSE MEDIA ON THE CUSTOMER CASE IS BEING REPORTED TO FIUIND DUE TO THE NOTICE RECEIVED FROM THE INCOME TAX DEPARTMENTCUSTOMER R N EXPORTS DATE OF INCORPORATION 1//1991; PAN AAAFR22B WITH IEC CODE 91821 ISSUED BY DGFT, GSTIN 27AAAFR22B1Z ISSUED BY CBEC HAS ADDRESS AS 1, PETIT TOWER, RD FLOOR, CUMBALLA HILL, A K MARG, MUMBAI26 MOBILE NUMBER 9826978 AND LANDLINE NUMBER 22288882 THE CUSTOMER HAS BEEN CATEGORISED AS MEDIUM RISK AND THE LAST KYC WAS UPDATED ON 8/9/22 WITH THE STATUS AS COMPLETED THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER 225686 WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT SHREE PANT BHUVAN, SANDHURUST BRIDGE, MUMBAI7 SINCE 27/8/21 WITH THE STATUS AS ACTIVE THE CUSTOMER ALSO HOLDS AN USD EEFC ACCOUNT NUMBER 22562 SINCE 27/8/21 WHICH IS IN DORMANT STATUS SINCE 1/6/219 AND USD DIAMOND DOLLAR ACCOUNT NUMBER 2256129 SINCE 6/7/216 WHICH IS IN ACTIVE STATUS AS PER THE KYC RECORDS HELD WITH THE BANK, CUSTOMER R N EXPORTS IS A PARTNERSHIP FIRM ENGAGED IN THE BUSINESS OF MANUFACTURING OF JEWELLERY AND WORKING OF DIAMONDS RAHUL ROHIT JHAVERI AND SONALI RAHUL JHAVERI ARE THE BENEFICIAL OWNERS AND AUTHORISED SIGNATORIES OF THE FIRM THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS6295 CRORES REVIEW OF THE TRANSACTIONS FOR ACCOUNT 225686 FOR THE CURRENT FINANCIAL YEAR FROM 1//221 TO 1//221 REVEALED TOTAL CREDITS OF IN CASH AND TOTAL DEBITS OF IN CASH REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 1//22 TO 1//221 REVEALED TOTAL CREDITS OF RS7 CRORES IN CASH AND TOTAL DEBITS OF RS5 CRORES IN CASH RS7, ; REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 1//219 TO 1//22 REVEALED TOTAL CREDITS OF RS2112 CRORES IN CASH AND TOTAL DEBITS OF RS217 CRORES IN CASH RS2 LAKHS BALANCE IN THE ACCOUNT AS ON 1//221 WAS A REVIEW OF THE INTERNAL RECORDS REVEALED THAT AN STR WAS REPORTED PREVIOUSLY VIDE BATCH ID 17267988, SNO 1 WITH THE REPORTED ACCOUNTS 225686, 22562 AND 2256129 IN THE NAME OF R N EXPORTS DUE TO CIRCULATION OF FUNDS BETWEEN THE CLIENT AND A HONG KONG BASED ENTITY ON ACCOUNT OF PURCHASE AND SALE OF ROUGH AND POLISHED DIAMONDS",{"entities":[(1837,1852,"FOREIGN ORG")   ]}),
            ("HIGH VALUE ADV IMP REM AMOUNTED TO RS11761 CR FROM THE A/C OF PREMIUM PETRO PRODUCTS MAINLY FAVOURING UAE BASED ENTITIES MOTOTIVE GENERAL TRADING LLC RS97 CR, PETRO STAR FZE RS71 CR, KIARA PETROLEUM FZE RS178 CR, OULA ALNUHA INTERNATIONAL RS11 CR, TIME TECHNO PACKAGING LLC RS9 CR, TASHBULAK TRADING LLC RS61 CR AND MAN TRADING LLC RS8 CR TOWARDS IMPORT OF BITUMEN PREMIUM PETRO PRODUCTS IS YET TO SUBMIT BOES PERTAINING TO 72 CASES TOTALLING TO USD 8,25,89 WHICH COMPRISES OF 1 CASES TOTALLING TO USD 985,122 PENDING FOR A PERIOD ABOVE MONTHS UPTO 6 MONTHS AND 58 CASES TOTALLING TO USD 7,265,266 PENDING FOR A PERIOD LESS THAN MONTHS 5 IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS7177 CR MAINLY FAVOURING PREMIUM PETRO PRODUCTS RS16 CR, PP SOFTTECH P LTD RS715 CR, ICICI 7252, VISHAL BOHRA RS2 CR, HDFC 8598662562/ 591982999999, IDBI 11688, YES BANK 6521982999999, VINAYAK EXIMS RS115 CR, AXIS BANK 91217166 AND AUM KRIPA ENTPS RS1 CR, HDFC 592721 6 INTERCONNECTED FUNDS TRANSFER TXNS WITHIN GROUP A/CS AMOUNTED TO RS771 CR UTURN TRANSACTIONS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS1525 CR WERE RECEIVED AND RS59 CR WERE SENT OUT TO A/CS OF PREMIUM PETRO PRODUCTS, VISHAL BOHRA, ABHA PRECISION FARMING LLP, BHEEM RAJ BOHRA, MANISH BOHRA IDBI 112995 AND VCI EXPORTS P LTD MAINTAINED WITH OTHER BANKS 7 PREMIUM PETRO PRODUCTS, B L INVESTMENT, RISHI ENTPS, HERO PETRO PRODUCTS, REEVA BUILDCON P LTD, REALTECH BUILDHOME P LTD, ABHA PRECISION FARMING P LTD, AONE BUILDHOME LLP AND BODHI FARM DEVELOPERS P LTD HAVE UPDATED A COMMON MAILING ADDRESS 1 HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN 21 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT IT WAS A 2 SQ FT OWNED PREMISE WITH SIGN BOARD, EMPLOYEES AND OFFICE SET UP SEEN HOWEVER NO STOCK OF GOODS WERE SIGHTED ON ENQUIRY IT WAS LEARNT FROM CUSTOMER THAT GOODS WERE STOCKED AT MOKHAMPURA, TEHSILMOZAMABAD, DISTRICT JAIPUR 8 PAN OF INTERCONNECTED A/CS SHREEBALAJI INFRAPROJECTS P LTD AAPCS589K AND DENIS LAND DEVELOPERS P LTD AACCD6725C ARE REFLECTING IN THE LIST OF POSSIBLE SHELL ENTITIES CIRCULATED BY FIU INDIA 9 TOTAL TAX PAYMENTS OF RS7 CR WAS NOTED THROUGH THE A/CS OF UJJWAL BHANDARI HUF 9512972 AND VCI EXPORTS P LTD 19959 DURING REVIEW PERIOD, WHICH APPEARS TO BE DISPROPORTIONATE TO HIGH VALUE AND VOLUME OF TRANSACTIONS ROUTED THROUGH A/CS1 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH MULTIPLE A/CS OPENED BY THE SAME GROUP OF INDIVIDUALS FOR LAYERING OF FUNDS AND POSSIBLY TO REMIT FUNDS OVERSEAS WITHOUT ACTUAL IMPORTS TAKING PLACE POSSIBLY WITH THE INTENTION OF TBML AND HAWALA SETTLEMENT IT ALSO APPEARS THAT SUCH HIGH VALUE AND VOLUME OF TRANSACTIONS MAY HAVE BEEN CARRIED OUT WITHIN GROUP A/CS WITHOUT ANY ECONOMIC RATIONALE POSSIBLY WITH INTENTION OF TAX EVASION A/C OF PREMIUM PETRO PRODUCTS BEARING NUMBER 681952 IS ACTIVE WITH BALANCE OF RS28955/ AND 6819559 HAS OUTSTANDING BALANCE OF RS7,5,25,6/ ON CREDIT LIMIT OF RS12,/ AS ON APRIL 15, 221QUANTUM OF TRANSACTIONS FOR CURRENT FY 221222 IS DEBIT RS277 CR IN CASH RS CR AND CREDITS RS2781 CR IN CASH RS CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FY 2221 IS DEBIT RS2286 CR IN CASH RS11 CR AND CREDITS RS226 CR IN CASH RS CR 2192 IS DEBIT RS761 CR IN CASH RS1 CR AND CREDITS RS7589 CR IN CASH RS7 CR 21819 DEBIT RS2976 CR IN CASH RS CR AND CREDITS RS2987 CR IN CASH RS CRCURRENT A/CS WERE OPENED IN THE NAME OF A PARTNERSHIP FIRM M/S PREMIUM PETRO PRODUCTS NOS 6819559 AND 681952 ON SEP19 AT OUR BRANCH LOCATED IN 57 KRISHNA TOWER, BASEMENT, 1ST FLOOR, SARDAR PATEL MARGH JAIPUR RAJASTHAN 21 THE ENTITY WAS FORMED ON 9MAY17 AND HAS SUBMITTED COPIES OF PAN AAUFP267N, PARTNERSHIP DEED, CERT OF REG NO1/97/217, IEC AAUFP267N AND GST REG CERT 8AAUFP267N1ZF AS KYC DOCS AT THE TIME OF A/C OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 1 HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN 21 RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA AND LALIT KUMAR NAREDI ARE THE PARTNERS WHILE MANISH KUMAR BOHRA AND UJJWAL BHANDARI ARE THE UBOS IN THE SAID FIRM CUSTOMER HAS REGISTERED MOBILE NO 982999999 WITH THE BANK THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF PETROLEUM PRODUCTS WITH ANNUAL TURNOVER OF RS1 CR BASIS INTERCONNECTEDNESS, OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF 25 CURRENT A/CS IN THE NAMES OF 9 REGISTERED COS, 1 NBFC, 8 PARTNERSHIP FIRMS, 2 LLP FIRMS AND 1 INDIVIDUAL CURRENT A/C BESIDES 15 SAVINGS A/CS WERE OPENED IN THE NAMES OF INDIVIDUALS AND 1 HUF DETAILS OF A/CS ARE UPDATED IN XML FILE THE MENTIONED A/CS WERE OPENED DURING THE PERIOD 6SEP9 TILL NOV2 AT ABOVE MENTIONED AJMER ROAD, RAJA PARK, NORTH AVENUE, VAISHALI NAGAR BRANCHES IN JAIPUR RAJASTHAN AND SECTOR 17 VASHI BRANCH IN NAVI MUMBAI IT WAS INDICATED THAT THE ENTITIES ARE INTO VARIED BUSINESSES LIKE REAL ESTATE SERVICES, WHOLESALER OF CHARCOAL/ SODA / ASH, INFRASTRUCTURE, GENERAL TRADING, TEXTILE PRODUCTS, CONSTRUCTION, FINANCE AND INVESTMENT, DEALING IN STONES/MARBLE/GRANITE, TRADE/SALE OF HANDICRAFTS WITH ANNUAL TURNOVER IN THE RANGE OF RS1 CR TO RS CR RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA, LALIT KUMAR NAREDI AND BHEEM RAJ BOHRA ARE THE COMMON DIRECTORS/ PARTNERS/ KARTA IN THE ABOVE MENTIONED A/CS WITH OTHER INDIVIDUALS IN DIFFERENT COMBINATIONS NOTICING HIGH VALUE ROUND SUM NEFT/RTGS CREDITS FROM OWN A/C AND SUBSEQUENT HIGH VALUE ADVANCE IMP REM THROUGH THE A/C OF PREMIUM PETRO PRODUCTS 6819559, AN ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD AND THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION 1 ON REVIEWING A/C TRANSACTIONS SINCE APRIL 1, 22 APRIL 15, 221 IT WAS NOTED THAT IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS1782 CR MAINLY FROM PREMIUM PETRO PRODUCTS RS128 CR, IDBI 1128662, VCI EXPORTS P LTD RS9 CR, IDFC 198826, SURYA NAGARI TRADING RS297 CR, YES BANK 1672899, ABHA PRECISION FARMING LLP RS22 CR, IDBI 11277552 AND BHEEM RAJ BOHRA RS22 CR, IDBI 11658 2 EXPORT ADVANCE REMITTANCES AMOUNTED TO RS12 CR IN THE A/CS OF INDIA STONE HOUSE LLP AND VCI EXPORTS P LTD MAINLY FROM WONDERSTONE LLC USA RS29 CR, TRANQUILLO GMBH RS21 CR, GRAMACO SOURCE AND MARBLE RS18 CR, STONE SOURCE INTERNATIONAL INC USA RS1 CR AND STONELAND INC USA RS1 CR AS PER WEB CHECK WONDERSTONE LLC USA IS INTO BUSINESS OF JEWELLERY STORES HTTPS//WWWDNBCOM/BUSINESSDIRECTORY/COMPANYPROFILESWONDERSTONE LLCD21F25EAAE1E6AA861651D299FHTML, TRANQUILLO GMBH IS INTO CLOTHING STORE INDUSTRY, STONE SOURCE INTERNATIONAL INC IS INTO BUSINESS OF STONES/ GRANITES AND STONELAND INC IS INTO GRANITE/ MARBLE BUSINESS IT APPEARS THAT INDIA STONE HOUSE LLP WHICH HAS INDICATED THAT IT IS INTO BUSINESS OF STONES/ GRANITES MAY HAVE RECEIVED EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS LIKE WONDERSTONE LLC USA JEWELLERY STONES AND TRANQUILLO GMBH CLOTHING STORE INDUSTRY AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE",{"entities":[(121,149,"FOREIGN ORG"),(159,173,"FOREIGN ORG"),(183,202,"FOREIGN ORG"),(218,238,"FOREIGN ORG"),(253,273,"FOREIGN ORG"),(282,303,"FOREIGN ORG"),(316,331,"FOREIGN ORG")  ,(6211 , 6224,"FOREIGN ORG"),(6240 , 6255,"FOREIGN ORG"),(6300 , 6330,"FOREIGN ORG"),(6346 , 6359,"FOREIGN ORG"),(6388 , 6403,"FOREIGN ORG"),(6542 ,6557,"FOREIGN ORG"),(6591, 6621,"FOREIGN ORG"),(6663, 6676,"FOREIGN ORG") ,(6929, 6944,"FOREIGN ORG"),(6888, 6903,"FOREIGN ORG") ]}),
            ("HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED ON 8522 TO TUNE OF USD 29297 FROM MUNDRA GENERAL TRADING FZE IN UAEDEALS IN METAL SCRAPHTTPS//PANJIVACOM/MUNDRAGENERALTRADINGFZE/2896816 WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY OF MORE THAN 12 DAYS, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON 9122 USD 2891712 TO TSAI SHEN TRADING LIMITED IN HONGKONGCLIENT IS INTO CERAMIC AND AGRO PRODUCTS RELATED HTTPS//TSAISHENTRADINGCOM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 87171 STORAGE DEVICES THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED USD 29297 RECEIVED ON 1722 FROM MUNDRA GENERAL TRADING FZE IN UAEDEALS IN METAL SCRAPHTTPS//PANJIVACOM/MUNDRAGENERALTRADINGFZE/2896816 IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 115CRORES WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDKOTHARI PRODUCTS LIMITED,PRIVATE LTD, HAVING ADDRESS2/19, THE MALL, KANPUR,UTTAR PRADESH,INDIA,281, CONTACT NUMBER 12171 AND EMAIL ID KOTHARILW1@HOTMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 628851951 WITH ICICI BANK LTD, KANPUR,16/16, JSTOWERS, THE MALL, KANPUR, UTTAR PRADESH, 281, SINCE 18122 DATE OF INCORPORATION IS 179198 PAN NUMBER OF THE ACCOUNT IS AAACK5571F IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 68827 THERE ARE NO LINKED ACCOUNT AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE DEEPAK KOTHARI DIRECTOR AAYPK611C ,MITESH KOTHARI DIRECTOR AFUPK95M ,VIJAY KUMAR MODI DIRECTOR ACVPM668F ,SAURABH BHATNAGAR AUTHORISED SIGNATORY AHQPB297K ,MITESH KOTHARI AUTHORISED SIGNATORY AGUPK95M BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK KOTHARIAAYPK611C THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI, IEC DATE OF LAST KYC UPDATION 122 ENTITY IS PRIMARILY ENGAGED IN THE INTERNATIONAL TRADING OF COMMODITIES IT IS A PART OF KOTHARI GROUP KPL ALSO EARNS REVENUE FROM REAL ESTATE THROUGH VARIOUS COMMERCIAL PROPERTIES OWNED BY ITS SUBSIDIARIES THE COMPANY IS ALSO ENGAGED IN IMPORT EXPORT TRADE OF VARIOUS PRODUCTS/COMMODITIES INCLUDING AGRO BASED COMMODITIES, MINERALS, METALS, PETROLEUM PRODUCTS AND COAL, TILES, NOTEBOOK, COMPUTER STORAGE DEVICES, COMPUTER HARDWARE, ELECTRONIC COMPONENTS, TRANSFORMERS, COPIER PAPER, STEEL, SCRAP AND PVC THE COMPANY WAS INCORPORATED IN 198 AND HAS ITS REGISTERED OFFICE LOCATED IN KANPUR, UTTAR PRADESHALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 221 WAS RS1,172 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 25,1597 LAKHS OF WHICH RTGS RS 18,925 LAKHS MAJORLY FROM KOTHARI PRODUCTS LIMIT,KOTHARI PRODUCTS LIM,KPL EXPORTS LIMITED, REMITTANCE RS 6,22997 LAKHS MAJORLY FROM MUNDRA GENERAL TRADING F Z E,HIMALAYAN GENERAL TRADING FZE,MUNDRA GENERAL TRADING FZE,OFFICE NUMBER 1, TOTAL DEBIT SIDE RS 2,1221 LAKHS OF WHICH RTGS RS ,55 LAKHS MAJORLY TO KOTHARI PR,KPL EXPORT, REMITTANCE RS 19,5722 LAKHS MAJORLY TO KINGSTON PTE LTD,TSAI SHEN TRADING LIMITED,KINGSTON PTE LTD AND OTHERS RS 7 LAKHS FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 6,752 LAKHS OF WHICH CASH WAS RS 1 LAKHS, REMITTANCE RS 1,7697 LAKHS TOTAL DEBIT SIDE RS 6,6755 LAKHS OF WHICH, REMITTANCE RS 5,8752 LAKHS FY 21718 FROM 1217 TO 1218 TOTAL CREDIT SIDE RS 1722 LAKHS TOTAL DEBIT SIDE RS 7 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE FUNDS OUTLAY OF MORE THAN 12 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE 22 OF THE VALUE OF USD 115CRORES FOR EXAMPLE, VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN 12 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON 2122 USD 289128 LACS REMITTED TOTSAI SHEN TRADING LIMITED IN HONGKONGCLIENT IS INTO CERAMIC AND AGRO PRODUCTS RELATED HTTPS//TSAISHENTRADINGCOM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 87171 STORAGE DEVICE",{"entities":[(97,123,"FOREIGN ORG"),(168,190,"FOREIGN ORG"),(468,493,"FOREIGN ORG"),(835,858,"FOREIGN ORG"),(906,928,"FOREIGN ORG"),(3294,3321,"FOREIGN ORG"),(3323,3351,"FOREIGN ORG"),(3353,3378,"FOREIGN ORG"),(3530,3545,"FOREIGN ORG"),(3547,3571,"FOREIGN ORG"),(3573,3589,"FOREIGN ORG"),(4251,4269,"FOREIGN ORG")]}),
            ("FLYON FILM PRODUCTION, A PROPRIETORSHIP FIRM, HAVING ADDRESS77/89, HEMKUNT CHAMBER, NEHRU PLACE, SOUTH DELHI, NEW DELHI, INDIA, 1119, CONTACT NUMBER 917972 AND EMAIL ID AKANKSHASINGH91@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 755196 WITH ICICI BANK LTD, NEW DELHI CONNAUGHT PLACE, ICICI BANK LTD, 9A, PHELPS BUILDING, CONNAUGHT PLACE, DELHI, 111, SINCE 21221 DATE OF INCORPORATION IS 17822 PAN NUMBER OF THE ACCOUNT IS CGHPS898F GSTIN AS PER BANK RECORDS IS 7CGHPS898F2ZP OTHER RELATIONSHIP OF CUSTOMER IS 1695167 TRANSACTION LINKED ACCOUNTS ARE 685527, 11951855 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKANKSHA SINGH PROPRIETOR CGHPS898F THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON 6221 KYC DOCUMENTS OBTAINED WERE PAN CGHPS898F, GSTIN, AADHAR CLIENT DEALS MEDIA AND FILMES SERVICES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAC TO RS 5 CR A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 6221 WAS NIL FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 21221 TO 6221 TOTAL CREDIT SIDE RS 1,96158 LAKHS OF WHICH CLEARING RS 1 LAKH MAJORLY FROM FLYON FILM PRODUCTION, NEFT RS 976 LAKHS MAJORLY FROM DAT SOLUTIONS, APS INTERNATIONAL, IMAGINARY FILMS PRIVATE LIMITED, RTGS RS 1,962 LAKHS MAJORLY FROM GWAY TO WORLD, OCN INTERNATIONAL, CARIO SERVICES, MOBILE MONEY TRANSFERS RS 5 LAKHS IN INSTANCES, TOTAL DEBIT SIDE RS 1,96158 LAKHS OF WHICH CLEARING RS 151 LAKHS MAJORLY TO FLYCON FILM PRODUCTION, RTGS RS 81 LAKHS MAJORLY TO IMAGINARY FILMS PRIVATE LIMITED, REMITTANCE RS 15 LAKHS MAJORLY TO THE PRODUCTION SERVICES FZCO, GATE STUDIOS PTE LTD, INTERNET FUND TRANSFER RS 9821 LAKHS TO ONE FRAME PRODUCTIONS, MOBILE MONEY TRANSFERS RS 1 LAKH IN 1 INSTANCE, AND OTHERS RS 2 LAKHS TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS ROUTING OF FUNDS OBSERVED SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD FOR EXAMPLE, ON 22221 RS 611 LACS CREDITED BY RTGS FROM IMAGINARY FILMS PRIVATE LIMITED 9227297, IFSCUTIB1625, AXIS BANK LTD AND SAME WAS DEBITED THROUGH REMITTANCE TO GATE STUDIOS PTE LTD IN SINGAPORE FOR PERSONAL, CULTURAL AND RECREATIONAL SERVICES IN OTHER EXAMPLE, BETWEEN 1221 TO 2221 RS 62 LACS CREDITED BY RTGS FROM IMAGINARY FILMS PRIVATE LIMITED 9227297, IFSCUTIB1625, AXIS BANK LTD AND SAME WAS DEBITED THROUGH NEFT TO IMAGINARY FILMS PRIVATE LIMITED 9227297, IFSCUTIB1625, AXIS BANK LTD SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT TOTAL TURNOVER IN THE FINANCIAL YEAR 221, WAS RS 1,96158 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS LAC TO 5CR IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS, HIGH TURNOVER AND ACCOUNT BEING CLOSED IN SHORT DURATION, STR IS BEING FILED",{"entities":[(1473,1489,"FOREIGN ORG"),(1492,1505,"FOREIGN ORG"),(1781,1800,"FOREIGN ORG"),(2244, 2264,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN SHOWS, FUNDS ARE RECEIVED FROM DIFFERENT ENTITIES AND ARE IMMEDIATELY DEBITED TOWARDS OUTWARD REMITTANCE FOR EG, ON 2922 ACCOUNT WAS CREDITED WITH RS 1 CRORES IN PARTS THROUGH RTGS FROM SHREEJI JEWELLERS ACCOUNT NO 919922, IFSC CODE UTIB175, BANK AXIS BANK AND THE SAME IS DEBITED IN PARTS THROUGH RTGS TO DIAMAX ACCOUNT NO 25982521725, IFSC CODE INDB57, BANK INDUSIND BANK LTD SIMILARLY, ON 922 ACCOUNT WAS CREDITED WITH RS 158 CRORES IN PARTS THROUGH RTGS FROM DIAMAX ACCOUNT NO 521986889, IFSC CODE HDFC896, PURVI EXPORTS ACCOUNT NO 71616, IFSC CODE KKBK958, VANSH GEMS, ACCOUNT NO 112515, IFSC CODE KKBK958 AND ON THE SAME DATE ACCOUNT WAS DEBITED WITH RS 182 CRORES APPROX IN PARTS THROUGH REMITTANCE TO MARS VENTURE LIMITED, HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT FUNDS WERE CREDITED IN THE ACCOUNT FROM MAHAPUJA PRODUCTS LTD, NYSNA MULTIVENTURES PVT LTD, SHREEJI JEWELLERS ETC AND MAJORITY REMITTANCES MADE TO MARS VENTURE LIMITED SINCE ACCOUNT OPENING OVERSEAS ENTITIES EXISTENCE CHECKED HOWEVER NO FURTHER INFORMATION AVAILABLE ON THEM REGARDING WHAT PRODUCT THEY SELL NO BUSINESS RELATED PAYMENTS ARE MADE FROM THIS ACCOUNT IN VIEW OF LARGE VALUE FOREX DEBIT DAILY AND NEGATIVE FINDINGS, STR IS BEING FILEDDIAMAX, A PROPRIETORSHIP FIRM, HAVING ADDRESS/5, DAIMOND APPARTMENT, SAIYEDPURA, NAGORIVAD, SURAT, GUJARAT, INDIA, 95, CONTACT NUMBER 91982521725 AND EMAIL ID DIAMAX95@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 29755111 WITH ICICI BANK LTD, SHUBHAM POINT, RUWALA TEKRA, HARIPURA BAGAL, LAL GATE, SURAT GUJARAT, 95, SINCE 21219 DATE OF INCORPORATION IS 112216 PAN NUMBER OF THE ACCOUNT IS FOJPS7176R IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 5216926682 GSTIN AS PER BANK RECORDS IS 2FOJPS7176R1ZQ THERE ARE NO PAN LINKED ACCOUNTS TRANSACTION LINKED ACCOUNT IS 52125697 AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS DHRUMIL DHARNENDRA SHAH PAN FOJPS7176R THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN FOJPS7176R, AADHAAR OF PROPRIETOR, IMPORT EXPORT CERTIFICATE AND GST REGISTRATION CERTIFICATE CUSTOMER IS AN IMPORTER OF LAB GROWN THAT IS SYNTHETIC DIAMONDS AND FURTHER HE IS SELLING IN LOCAL MARKET AS PER ENHANCED DUE DILIGENCE EDD, NAME BOARD AFFIXED OUTSIDE OFFICE WHICH IS PRINTED ON A SIZE PAPER ALSO THERE ARE CREDITS FROM NYSNA MULTIVENTRES PVT LTD SEEN AND IS ONE SUCH COMPANY WHICH HAS TRANSACTED WITH MANY SUSPICIOUS DIAMOND COMPANIES OF SURAT AND WHOSE ACCOUNTS ARE CLOSED UNDER BANK INDUCED PROCESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 25 CRORES TO LESS THAN RS 1 CRORES ALERT WAS TRIGGERED FOR LARGE VALUE FOREX DEBIT DAILY BALANCE IN THE ACCOUNT AT 282221 WAS RS1 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2221 FROM 122 TILL 282221 SHOWS TOTAL CREDIT RS 1,9817 LAKHS OF WHICH CLEARING RS 771 LAKHS MAJORLY FROM RAHIL GEM, NEFT RS 9 LAKHS MAJORLY FROM SAHAJ DIAM, INTEGRITY JEWELS, RTGS RS 1,89 LAKHS MAJORLY FROM DIAMAX, VANSH GEMS, SHREEJI JEWELLERS, MOBILE MONEY TRANSFERS RS 2 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,92985 LAKHS OF WHICH RTGS RS 2851 LAKHS MAJORLY TO PARI IMPEX, RADHE KRISHNA IMPEX, DIAMAX, REMITTANCE RS 1,881 LAKHS MAJORLY TO MARS VENTURE LIMITED, HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT , AARAS DIAMONDS LLC, ALICAT SCIENTIFIC INC IN USD TO UNITED STATES FOR ADV PAYMENT AGAINST IMPORT OTHER THAN NEPAL BHUTAN , NEFT RS 1121 LAKHS TO MAYAN EXPORTS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2192 FROM 21219 TO 122 TOTAL CREDIT SIDE RS 2,51 LAKHS OF WHICH CLEARING RS 5 LAKHS FROM DIAMAX, NEFT RS 6727 LAKHS FROM SHREE RAM DIAMOND, RTGS RS 1,9679 LAKHS FROM NYSNA MULTIVENTURES PVT LTD, SHREEJI JEWELLERS, INTERNET FUND TRANSFERS RS 568 LAKHS FROM MAHAPUJA PRODUCTS LTD, MOBILE MONEY TRANSFERS RS 85 LAKHS TOTAL DEBIT SIDE RS 2,521 LAKHS OF WHICH CASH WAS RS 5 LAKHS TO SURAT BHAGAL BRANCH, RTGS RS 152 LAKHS, REMITTANCE RS 2,6986 LAKHS TO MARS VENTURE LIMITED, HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT",{"entities":[(729,748,"FOREIGN ORG"),(970,990,"FOREIGN ORG"),(3354,3337,"FOREIGN ORG"),(4170,4189,"FOREIGN ORG")      ]}),
            ("IMAGINARY FILMS PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS 2, EMBASSY CHAMBERS, RD ROAD, KHAR WEST, MUMBAI, MAHARASHTRA, INDIA, 52, CONTACT NUMBER 9179581 AND EMAIL ID IMAGINARYFILMS@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 11951855 WITH ICICI BANK LTD, PLOT NO 26 221, 27TH RD, CRYSTAL BUILDING, WATERFIELD ROAD, BANDRA W, MUMBAI, MAHARASHTRA, 5, SINCE 29122 DATE OF INCORPORATION IS 22122 PAN NUMBER OF THE ACCOUNT IS AAFCI555Q THERE ARE NO PAN LINKED ACCOUNTS TRANSACTION LINKED ACCOUNTS ARE 152559, 151158 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH KUMAR SINGH WITH PAN NO GSRPS7969M, SAPNA KUMARI WITH PAN NO DGSPK718Q THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS TOTAL FREEZED ON 172221 DUE TO SUSPICIOUS NATURE OF TRANSACTIONS IN THE ACCOUNT 8NSMT21, AS PER BANK RECORDS KYC DOCUMENTS OBTAINED WERE PAN AAFCI555Q, MOA, AOA, COI U929MH22PTC62 AS PER ENHANCE DUE DILIGENCE COMPANY IS IN THE BUSINESS OF THE CINEMATOGRAPH TRADE AND INDUSTRY IN ALL THEIR BRANCHES AND ACTIVITIES AND PARTICULARLY THE BUSINESS OF MANUFACTURE, PRODUCTION, DISTRIBUTION, EXPLOITATION, EXHIBITION, IMPORT AND EXPORT OF ALL KINDS OF CINE FILMS, TALKIE FILMS, VIDEO FILMS, TELEFILMS, DOCUMENTARY FILMS, ADVERTISING FILMS, ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 5 CRS TO 25 CRS A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 16221 WAS RS87 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2221 FROM 122 TILL 16221 SHOWS TOTAL CREDIT RS 6,67792 LAKHS OF WHICH TRANSFERS RS 117 LAKHS MAJORLY FROM IMAGINARY FILMS PRIVATE LIMITED, KESARI TOURS PVT LTDFOREX, CLEARING RS 26 LAKHS MAJORLY FROM IMAGINARY FILMS PVT LTD, RTGS RS 26 LAKHS MAJORLY FROM IMAGINARY FILMS PVT LTD, GILPIN TOURS AND TRAVEL, TRANSCORP INTERNATIONAL, INTERNET FUND TRANSFERS RS 6,2198 LAKHS FROM SAWARTHIYA FILM PRODUCTIONS, FLYON FILM PRODUCTION, ETC, MOBILE MONEY TRANSFERS RS 1, AND OTHERS RS 252 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 6,6818 LAKHS OF WHICH CASH WAS RS 6 LAKHS BY BRANCH, CLEARING RS 181 LAKHS MAJORLY TO IMAGINARY FILMS PVT LTD, HDFC BANK LTD, YES BANK LTD, RTGS RS ,1626 LAKHS MAJORLY TO GILPIN TOURS N TRAVEL, IMAGINARY FILMS PVTLTD, WORLD ONE INDIA FOREX, REMITTANCE RS 519 LAKHS MAJORLY TO ROSEBERRY INTERNATIONAL, ROSEBERRY INTERNATIONAL SERVICES, LIVING FILMS PTE LTD, INTERNET FUND TRANSFER RS 2,1165 LAKHS TO SAWARTHIYA FILM PRODUCTIONS, MULTIMONEY FOREX LTD, ETC, MOBILE MONEY TRANSFERS RS 2926 LAKHS IN 5 INSTANCES AND OTHERS RS 286 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2192 FROM 29122 TO 122 TOTAL CREDIT SIDE RS LAKHS OF WHICH CLEARING RS 2 LAKHS, RTGS RS 1 LAKHS TOTAL DEBIT SIDE RS 252 LAKHS OF WHICH CASH WAS RS 5 LAKHS, CLEARING RS 2 LAKHS TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS WHICH SEEMS LIKE ROUTING OF FUNDS FOR EXAMPLE, BETWEEN 122221 TO 152221 RS 62LACS CREDITED BY TRANSFERS FROM SAWARTHIYA FILM PRODUCTIONS 152559 AND DURING SAME DURATION RS6156LACS WAS DEBITED BY TRANSFERS TO SAWARTHIYA FILM PRODUCTIONS 152559 IN OTHER EXAMPLE, ON 82221 RS 626LACS WAS CREDITED BY TRANSFERS FROM FLYON FILM PRODUCTION 755196 AND SAME WAS DEBITED BY TRANSFERS TO MULTIMONEY FOREX LTD 151158 SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS AND ROUTING OF FUNDS, STR IS BEING FILED",{"entities":[(2413,2444,"FOREIGN ORG"),(2447, 2466,"FOREIGN ORG")     ]}),
            ("DIAMAX, A PROPRIETORSHIP FIRM, HAVING ADDRESS /5, DAIMOND APPARTMENT, SAIYEDPURA, NAGORIVAD, SURAT, GUJARAT, INDIA, 95, CONTACT NUMBER 91982521725 AND EMAIL ID DIAMAX95@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 29755111 WITH ICICI BANK LTD, SHUBHAM POINT, RUWALA TEKRA, HARIPURA BAGAL, LAL GATE, SURAT, GUJARAT, 95, SINCE 21219 DATE OF INCORPORATION IS 112216 PAN OF THE ACCOUNT IS FOJPS7176R IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 5216926682 GSTIN AS PER BANK RECORDS IS 2FOJPS7176R1ZQ TRANSACTION LINKED ACCOUNT IS 52125697 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS DHRUMIL DHARNENDRA SHAH PROPRIETOR FOJPS7176R THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, GSTIN KYC UPDATION IS NOT DUE CUSTOMER IS AN IMPORTER OF LAB GROWN IE SYNTHETIC DIAMONDS WHICH HE SELL IN LOCAL MARKET DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 25CR TO LESS THAN 1CR ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 282221 WAS RS1 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2221 FROM 122 TILL 282221 SHOWS TOTAL CREDIT RS 1,9817 LAKHS OF WHICH RTGS RS 1,89 LAKHS MAJORLY FROM DIAMAX, VANSH GEMS, SHREEJI JEWELLERS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,92985 LAKHS OF WHICH RTGS RS 2851 LAKHS MAJORLY TO PARI IMPEX, DIAMAX, REMITTANCE RS 1,881 LAKHS MAJORLY TO MARS VENTURE LIMITED, AARAS DIAMONDS LLC, ALICAT SCIENTIFIC INC FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2192 FROM 21219 TO 122 TOTAL CREDIT SIDE RS 2,51 LAKHS OF WHICH RS BY CASH TOTAL DEBIT SIDE RS 2,521 LAKHS OF WHICH CASH WAS RS 5 LAKHS FUNDS ARE FROM DIFFERENT ENTITIES AND ARE IMMEDIATELY DEBITED TOWARDS OUTWARD REMITTANCE FOR EXAMPLE, ON 21922 ACCOUNT WAS CREDITED WITH RS 1 CR THROUGH RTGS FROM SHREEJI JEWELLERS 91922115268, BANK AXIS IFSC UTIB175 AND ON SAME DURATION ACCOUNT WAS DEBITED WITH RS 8 CRORE THROUGH REMITTANCE TO MARS VENTURE LIMITED AT HONG KONG FOR PAYMENT AGAINST INVOICE ALSO ON 922 ACCOUNT WAS CREDITED WITH RS 158 CRORE THROUGH RTGS FROM DIAMAX521986889, BANK HDFC IFSC HDFC896, PURVI EXPORT, VANSH GEMS AND ON SAME DURATION ACCOUNT WAS DEBITED WITH RS 182 CRORE THROUGH REMITTANCE TO MARS VENTURE LIMITED AT HONG KONG FOR PAYMENT AGAINST INVOICE MAJORITY REMITTANCES MADE TO MARS VENTURE LIMITED OVERSEAS ENTITIES EXISTENCE CHECKED HOWEVER NO FURTHER INFORMATION AVAILABLE ON THEM REGARDING WHAT PRODUCT THEY SELL NO BUSINESS RELATED PAYMENTS ARE MADE FROM THIS ACCOUNT NAME BOARD AFFIXED OUTSIDE OFFICE WHICH IS PRINTED ON A SIZE PAPER ALSO THERE ARE CREDITS FROM NYSNA MULTIVENTRES PVT LTD IT IS ONE SUCH COMPANY WHICH HAS TRANSACTED WITH MANY SUSPICIOUS DIAMOND COMPANIES OF SURAT WHOSE ACCOUNTS WE HAVE CLOSED UNDER BANK INDUCED PROCESS IN VIEW OF LARGE VALUE FOREX OUTWARD REMITTANCE IN THE ACCOUNT TO A SUSPICIOUS OVERSEAS COUNTER PARTY, HIGH TURNOVER AND NEGATIVE FINDINGS IN TERMS OF TRANSACTIONS WITH A DOMESTIC SUSPICIOUS ENTITY, STR IS BEING FILED",{"entities":[(1484,1503,"FOREIGN ORG"),(1526,1547 ,"FOREIGN ORG"),(2093 ,2112 ,"FOREIGN ORG"),(2371,2391 ,"FOREIGN ORG"),(2462, 2482 ,"FOREIGN ORG"),(1506,1523,"FOREIGN ORG")   ]}),          
            ("ON 21221 FUNDS OF RS LACS APPROX CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM SUYOG SYNTHETICSACC NO918167262, IFSC UTIB1 AND DEPLOYED THROUGH RTGS TOWARDS SHREE BALAJI CREATIONACC NO591157868, IFSC KKBK876 SEEMS UNUSUAL, RATIONALE FOR SUCH ROUTING OF FUNDS ARE NOT CLEAR ACCOUNT TURN OVER IS TOO HIGH MORE THAN CRORES IS SEEN IN ACCOUNT IN VIEW OF SUSPICIOUS LARGE VALUE NON CASH DEPOSIT TRANSACTIONS AND SUSPECTED ROUTING OF FUNDS, STR IS BEING REPORTEDSUYOG SYNTHETICS, PROPRIETOR, HAVING ADDRESS21 RAGHUVEER BUSINESS,EMPIRE,AAI MATA CHOWK, PARVAT PATIYA, SURAT, GUJARAT, INDIA, 951, CONTACT NUMBER 91992511 AND EMAIL ID JAGDISHGARG@TRIBENIGROUPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 6265511 WITH ICICI BANK LTD, SURAT RING ROAD, ICICI BANK LTD SINCE 17217 DATE OF INCORPORATION IS 2972 PAN NUMBER OF THE ACCOUNT IS AAPPG629J IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 522882 PAN LINKED ACCOUNTS ARE 62615889 AND 62651155 TRANSACTION LINKED ACCOUNTS ARE 626558,6988511787 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE JAGDISHPRASAD RAMKISHAN GARG PROPREITOR AAPPG629J THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN CARDAAPPG629J, GSTIN2AAPPG629J1ZE, IEC522882 AND AADHAR CARD DATE OF LAST KYC UPDATION 111219 HENCE, KYC UPDATION IS NOT DUE CUSTOMER IS INTO IMPORTING AND TRADING OF YARN ANNUAL TURN OVER DECLARED BY CUSTOMER IS RS LAC TO 5 CRORE ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT 282221 WAS RS5 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2221 FROM 122 TILL 282221 SHOWS TOTAL CREDIT RS 5,266 LAKHS OF WHICH CASH WAS RS 96 LAKHS MAJORLY IN JUHU , SURAT TEXTILE MARKET BRANCH, TRANSFERS RS 12528 LAKHS MAJORLY FROM RAJ TEXTILE, SHREE GANESH TEXTILE,TRIBENI WORLDWIDE PVT LTD, CLEARING RS 98 LAKHS MAJORLY FROM SUMILON, BHATHIJI SILK MILLS,GEETA RAYON, NEFT RS 667 LAKHS MAJORLY FROM PADMAVATI INTERNATIONAL ,GURUKRUPA RAYON, GURU FIBERS, RTGS RS ,125 LAKHS MAJORLY FROM SUYOG SYNTHETICS, R R TRADING CO, SAKSHI YARNS PVT LTD, REMITTANCE RS 21 LAKHS MAJORLY FROM INDORAMA SYNTHETICS TBK PT PP,UNITED RAW MATERIAL PTE LTD, INTERNET FUND TRANSFERS RS 1,92 LAKHS, MOBILE MONEY TRANSFERS RS 1675 LAKHS IN 127 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 5,288 LAKHS OF WHICH CASH WAS RS 5176 LAKHS BY BR ANCH, TRANSFERS RS 176 LAKHS MAJORLY TO SINGHAL BROTHERS, S K KABRA AND COMPANY, CLEARING RS 775 LAKHS MAJORLY TO MONIT TRADING PRIVATE LIM,S R GROUP, KAYAVLON IMPEX PRIVATE LIMITED, RTGS RS 1,99 LAKHS MAJORLY TO DEEP SHIPPING AGENCY,SUYOG AXIS CC, NIRANJAN SHIPPING AGENCY, REMITTANCE RS 1,7597 LAKHS MAJORLY TO FTEX ASIA INTERNATIONAL LIMITED, PT INDORAMA POLYCHEM INDONESIA, INTERNET FUND TRANSFER RS 1 ,62529 LAKHS, CARD TRANSACTIONS RS 15 LAKHS, MOBILE MONEY TRANSFERS RS 229 LAKHS IN 12 INSTANCES AND OTHERS RS 2 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 9,19 LAKHS OF WHICH CASH WAS RS 56 LAKHS TOTAL DEBIT SIDE RS 9,5278 LAKHS OF WHICH CASH WAS RS 791 LAKHS FY 21819 FROM 1218 TO 1219 TOTAL CREDIT SIDE RS 9,92 LAKHS OF WHICH CASH WAS RS 5 LAKHS TOTAL DEBIT SIDE RS 9,77 LAKHS OF WHICH CASH WAS RS 62 LAKHS FY 21718 FROM 1217 TO 1218 TOTAL CREDIT SIDE RS ,7876 LAKHS OF WHICH CASH WAS RS 229 LAKHS TOTAL DEBIT SIDE RS ,7128 LAKHS OF WHICH CASH WAS RS 2 LAKHS ON ANALYZING THE TRANSACTION PATTERN, IT HAS BEEN NOTICED THAT THERE ARE HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS FOR EXAMPLE ON 252221 FUNDS OF RS 8 LACS APPROX CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM M/S ULTIMATEACC NO6551571, IFSC SCBL61, NEFT FROM SUYOG SYNTHETICSACC NO918167262, IFSC UTIB1 AND DEPLOYED THROUGH TRANSFER TOWARDS TRIBENI WORLDWIDE PVT LTD 626558 AND REMITTANCE TOWARDS F TEX ASIA INTERNATIONAL LIMITED , CHINA FOR ADV PAYMENT AGAINST IMP OTR THAN NEPL BHUT AN AND",{"entities":[(2233,2261,"FOREIGN ORG"),(2263,2289,"FOREIGN ORG")  ,(2791,2821,"FOREIGN ORG"),(2824,2853,"FOREIGN ORG"),(4019,4051,"FOREIGN ORG") ]}),          
            ("SRI SNJ JEWELS, A PROPRIETORSHIP FIRM, HAVING ADDRESS 2, OLD NO 116 SHOP NO 1/, RD FLR, NSC BOSE ROAD, SOWCARPET, TARUS TOWER, CHENNAI, TAMIL NADU, INDIA, 679, CONTACT NUMBER 917718666 AND EMAIL ID SRISNJJEWELS@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 615265 WITH ICICI BANK LTD, 8, NSC BOSE ROAD, SOWCARPET, CHENNAI, TAMIL NADU, 679, SINCE 2511219 DATE OF BIRTH IS 27219 PAN OF THE ACCOUNT IS BADPJ7R PAN LINKED ACCOUNTS ARE 61526562, 6155627 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS NIKET SOHANLAL JAIN PROPRIETOR BADPJ7R THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN BADPJ7R AND LETTER OF DECLARATION OF PROPRIETORSHIP FIRM KYC UPDATION IS NOT DUE DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN LACS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 1221 WAS RS 5 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 15,26228 LAKHS OF WHICH TRANSFERS RS 295 LAKHS MAJORLY FROM KANTILAL, JAI GANESH JEWELLERY, NEFT RS 6297 LAKHS MAJORLY FROM DAIMOND, SANGITA MADAN JAIN, RTGS RS 2,87156 LAKHS MAJORLY FROM LEO JEWELS, DIAMOND, REMITTANCE RS 9,7595 LAKHS MAJORLY FROM CHAIPHET ANYAMANEE CO LTD, SUAY THAI THONG CO LTD, INTERNET FUND TRANSFERS RS 1,78812 LAKHS FROM MOKSHA JEWELLERY TOTAL DEBIT SIDE RS 15,288 LAKHS OF WHICH RTGS RS 1,5756 LAKHS MAJORLY TO DIAMOND INDIA LIMITED, REMITTANCE RS 289 LAKHS MAJORLY TO CHAIPHET ANYAMANEE CO LTD, INTERNET FUND TRANSFER RS 1,172 LAKHS TO MOKSHA JEWELLERY ETC, MOBILE MONEY TRANSFERS RS 598 LAKHS IN 9 INSTANCES FY 2192 FROM 2511219 TO 122 TOTAL CREDIT SIDE RS ,8951 LAKHS OF WHICH RS BY CASH, REMITTANCE RS ,9812 LAKHS TOTAL DEBIT SIDE RS ,8688 LAKHS OF WHICH RS BY CASH, REMITTANCE RS 1,61 LAKHS CUSTOMER IS INTO GOLD ORNAMENTS MANUFACTURING FOR MANUFACTURING PURPOSE, CUSTOMER IS USING JOB WORK VENDORS AND GOLD SMITHS EXCEPT OFFICE SET UP, COULD NOT ASCERTAIN ANY BUSINESS ACTIVITIES HIGH VALUE NON CASHC CREDITS AND DEBITS WERE SEEN IN THE ACCOUNT THE ACTIVITY IS NOT LOOKING USUAL AS THE TURNOVER IN JUST 2 YEAR IS MUCH HIGHER IN COMPARE TO THE DECLARED TURNOVER WHICH LESS THAN RS LACS FOR EXAMPLE, ON 221 RS 212 LACS WERE CREDITED THROUGH RTGS FROM J B GOLD 9852977, KKBK958 AND ON THE SAME DAY RS 18 CRORE WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PVT LTD 65181212, INDB7 ALSO ON 22221 RS 698 LACS WERE CREDITED THROUGH RTGS FROM A V JEWELLERS 92767, YESB1, LEO JEWELS 522191, HDFC2 ETC AND ON THE SAME DAY RS 691 LACS WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PRIVATE LIMITED SIMILAR PATTERN WERE SEEN IN THE ACCOUNT IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT WHICH SEEMS LIKE ROUTING OF FUNDS WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILED",{"entities":[(1403,1427,"FOREIGN ORG"),(1430,1451,"FOREIGN ORG"),(1649,1673,"FOREIGN ORG")   ]}),
            ("1217 TO 1218 TOTAL CREDIT SIDE RS 1,55768 LAKHS TOTAL DEBIT SIDE RS 1,691 LAKHS OF WHICH CASH WAS RS 968 LAKHS, REMITTANCE RS 58 LAKHS TRANSACTION PATTERN SHOWS LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF FUNDS WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS OF HIGH VALUE FUNDS FOR EXAMPLE, ON 25221 RS 75 CRORES WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM DN ENERGY PRIVATE LIMITED 626551156 AND ON THE SAME DAY ACCOUNT WAS DEBITED WITH RS 6CRORES THROUGH CMS TO RILLICICI ALSO ON 26221 RS CRORES WERE CREDITED THROUGH CMS FROM REFEX INDUSTRIES LIMITED AND ON THE SAME DAY ACCOUNT WAS DEBITED WITH SAME AMOUNT THROUGH RTGS IN PARTS TO SIL ROOFTOP SOLAR POWER PVT LTD ACCOUNT NO 52685 IFSC HDFC11 SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED UNUSUAL ALSO AS PER MEDIA REPORT, SEBI OBSERVED THAT DURING THE PERIOD OF JULY1, 215 TO JULY 2, 215 COMPANY HAD ALLEGEDLY VIOLATED THE PROVISIONS OF SEBI, SUBSTANTIAL ACQUISITION OF SHARES AND TAKEOVER SAST REGULATION, 211HTTPS//WWWSEBIGOVIN/ENFORCEMENT/ORDERS/JAN218/ADJUDICATIONORDERINRESPECTOFJAGDISHT JAIN SHERISHA TECHNOLOGIES PVT LTD TARA CHAND JAINANILJAINUGAMDEVIJAINDIMPLEJAINANDSEEMAJAININTHE MATTERO FREFEX INDUSTRIES LIMITED _766HTML IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS STR IS BEING FILED SHERISHA TECHNOLOGIES PVT LTD, PRIVATE LTD, HAVING ADDRESSOLD NO 56L NEW NO 1/2,11TH FLR VENKATNARAYANA ROAD,T NAGAR CHENNAI, TAMIL NADU, INDIA, 617, CONTACT NUMBER 91996299719 AND EMAIL ID LALITHAU@REFEXCOIN IS MAINTAINING A CURRENT ACCOUNT NUMBER 62655662 WITH ICICI BANK LTD, CHENNAI T NAGAR, NEW NO, BAZULLAH ROAD, T NAGAR, CHENNAI, TAMIL NADU, 617, SINCE 11221 DATE OF INCORPORATION IS 1122 PAN NUMBER OF THE ACCOUNT IS AAHCS671P IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 522 OTHER RELATIONSHIP OF CUSTOMER ARE 626552818, 6265598 TRANSACTION LINKED ACCOUNTS ARE 6265587, 626551156 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE LALITHA AUTHORISED SIGNATORY ACQPL1682F,TAPPI KUMAR DAS AUTHORISED SIGNATORY BDKPD69C ,DINESH KUMAR AGARWAL AUTHORISED SIGNATORY AKHPA67A BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS A TARACHAND JAIN WITH PAN NO AAFPJ8LT, ANIL JAIN WITH PAN NO AADPA8582A THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAHCS671P, CIN U711TN22PTC9676, BOARD RESOLUTION, MOA ETC DATE OF LAST KYC UPDATION 122 KYC UPDATION IS NOT DUE ENTITY IS INTO MANAGEMENT CONSULTANCY SERVICES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO LESS THAN 5CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS BALANCE IN THE ACCOUNT AT 1221 WAS RS1556 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 92,98597 LAKHS OF WHICH CASH WAS RS 5 LAKHS MAJORLY IN TIRUPPUR 615 BRANCH, TRANSFERS RS ,22 LAKHS MAJORLY FROM ARADHYAA INFRA HOLDINGS PRIVATE LIMITED,PAVAGADA PROCESSING INDUSTRIES PVT LTD, CLEARING RS 18,821 LAKHS MAJORLY FROM SHERISHA SOLAR PVT LTD,REFEX INDUSTRIES LIMITED,DECCAN CHARTERS PVT LTD, NEFT RS 9,8919 LAKHS MAJORLY FROM MEDHA SERVO DRIVES PVT LTD,L T LTD,ILABS CAPITAL I LLP, RTGS RS 8,68 LAKHS MAJORLY FROM DUGAR FINANCE AND INVE,REFEX ENERGY LIMITED,ILABS CAPITAL I LLP, REMITTANCE RS 962 LAKHS MAJORLY FROM VISTA INDIA DIGITAL MEDIA INC, INTERNET FUND TRANSFERS RS 21,8199 LAKHS FROM DN ENERGY PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 156 LAKHS IN 19 INSTANCES , UPI TRANSFERS RS 5 LAKHS IN 2 INSTANCES , FIXED DEPOSIT RS 29987 LAKHS, AND OTHERS RS 271 LAKHS TOTAL DEBIT SIDE RS 9,1598 LAKHS OF WHICH CASH WAS RS 6 LAKHS BY BRANCH, TRANSFERS RS 7 LAKHS MAJORLY TO EMPLOYEES PROVIDENT FUND ORGANISATION, SHERISHA TECHNOLOGIES PVT LTD,KHAJANE II DEPT OF TRESURIES, CLEARING RS 21,88661 LAKHS, RTGS RS 58,89 LAKHS MAJORLY TO ILABS CAPITAL I LLP,DUGAR FINANCE AND INVESTMENT, 1, REMITTANCE RS 17 LAKHS MAJORLY TO GINLONG TECHNOLOGIES CO LTD,TUAH AND SUPARTO, INTERNET FUND TRANSFER RS 11,966 LAKHS, CARD TRANSACTIONS RS 159 LAKHS, MOBILE MONEY TRANSFERS RS 27162 LAKHS IN 26 INSTANCES , AND OTHERS RS 1,8 LAKHS FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 6,788 LAKHS OF WHICH CASH WAS RS 5 LAKHS, REMITTANCE RS 2117 LAKHS TOTAL DEBIT SIDE RS 6,7285 LAKHS OF WHICH CASH WAS RS 111 LAKHS, REMITTANCE RS 82 LAKHS FY 21819 FROM 1218 TO 1219 TOTAL CREDIT SIDE RS ,88298 LAKHS OF WHICH CASH WAS RS 2 LAKHS, REMITTANCE RS 5658 LAKHS TOTAL DEBIT SIDE RS ,758 LAKHS OF WHICH CASH WAS RS 86 LAKHS, REMITTANCE RS 6661 LAKHS FY 21718 FROM",{"entities":[(3430,3458,"FOREIGN ORG")     , (4034,4060,"FOREIGN ORG"),(4062,4077,"FOREIGN ORG")]}),        
            ("IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF FUND AND HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDA N DIAMONDS, A PROPRIETORSHIP FIRM, HAVING ADDRESSH N6, KUMBHAR SHERI,2ND FLOOR,1582 MAHIDHARPURA,ROOM 1, SURAT, GUJARAT, INDIA, 95, CONTACT NUMBER 91982511966 AND EMAIL ID ANDIAMONDS9@YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 75656 WITH ICICI BANK LTD, GROUND FLOOR, TOWER A, SWASTIK UNIVERSAL, UDHNA MAGDALLA ROAD, SURAT, GUJARAT, 957, SINCE 212219 DATE OF INCORPORATION IS 1217 PAN NUMBER OF THE ACCOUNT IS AAZPD2576P GSTIN AS PER BANK RECORDS IS 2AAZPD2576P1ZX TRANSACTION LINKED ACCOUNTS ARE 756552, 75657 AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS ASHESH NANALAL DOSHI PAN AAZPD2576P THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAZPD2576P, AADHAAR, GST REGISTRATION CERTIFICATE HENCE, KYC UPDATION IS NOT DUE CUSTOMER IS INTO IMPORT AND EXPORT OF ROUGH DIAMONDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAKHS TO RS 5 CRORES ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 282221 WAS RS2689 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2221 FROM 122 TILL 282221 SHOWS TOTAL CREDIT RS 5,8996 LAKHS OF WHICH CASH WAS RS 1 LAKHS MAJORLY IN SURATLH ROAD VARACHHA BRANCH, TRANSFERS RS 21925 LAKHS MAJORLY FROM JIGAR SHAH JT1, ASHESH NANALAL DOSHI, SUMO JEWEL, CLEARING RS 122 LAKHS MAJORLY FROM KANCHANBEN VADILAL, VADILAL CHHOTALAL, NEFT RS 1789 LAKHS MAJORLY FROM A N DIAMONDS, NISHCHAL C DESAI HUF, RTGS RS 2,1757 LAKHS MAJORLY FROM AASHESH NANALAL DOSHI, VAISHALI GEMS, REMITTANCE RS 791 LAKHS MAJORLY FROM STAR IMPEX HONG KONG, USD CURRENCY, INTERNET FUND TRANSFERS RS ,2551 LAKHS FROM A N DIAMONDS 756552, MOBILE MONEY TRANSFERS RS 1775 LAKHS IN 27 INSTANCES , UPI TRANSFERS RS 25 LAKHS IN 1 INSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS 6,16897 LAKHS OF WHICH CASH WAS RS 125 LAKHS BY BRANCH, TRANSFERS RS 217 LAKHS MAJORLY TO SAHAJANAND TECHNOLOGIES PVT LTD, A N DIAMONDS,ANU IMPEX, CLEARING RS 1977 LAKHS MAJORLY TO TORRENT POWER LIMITED,MARUTI HARDWARE, NEFT RS 79 LAKHS MAJORLY TO RITESH B MEHTA,PADSALA VITTHALBHAI, RTGS RS 1,955 LAKHS MAJORLY TO GIA INDIA LABORATORY PVT LTD, ASHESH NANALAL DOSHI, REMITTANCE RS 2171 LAKHS MAJORLY TO GIRDHARLAL DMCC IN USD TO UAE FOR PAYMT TOWARDS IMPORT SETTLEMENT OTHER THAN NEPAL BHUTAN DIAMONDS, INTERNET FUND TRANSFER RS ,12272 LAKHS TO A N DIAMONDS 756552, MOBILE MONEY TRANSFERS RS 119 LAKHS IN 227 INSTANCES AND OTHERS RS 965 LAKHS DTAXFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2192 FROM 212219 TO 122 TOTAL CREDIT SIDE RS 26 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 287 LAKHS OF WHICH CASH WAS RS 9 LAKHS AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE, ON 281222, RS 27 LAKHS WERE CREDITED THROUGH RTGS FROM AASHESH NANALAL DOSHI ACCOUNT NO155565, IFSC CODECBIN28512, BANK CENTRAL BANK OF INDIA AND ON SAME DAY RS 25 LAKHS THROUGH RTGS TO GIA INDIA LABORATORY PVT LTD ACCOUNT NO 15721, IFSC CODE HDFC157, BANK HDFC ON 15622, RS 87 LAKHS WERE CREDITED THROUGH INTERNET FUND TRANSFERS FROM A N DIAMONDS 756552 AND ON NEXT DAY RS 86 LAKHS WERE DEBITED THROUGH A N DIAMONDS 756552SIMILAR TRANSACTIONS ARE BEING REPEATED IN THE ACCOUNT, WHICH SEEMS TO BE ROUTING OF FUNDS IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 2221 WAS RS 5,8996 LAKHS, WHICH SEEMS TO BE VERY HIGH",{"entities":[(1780,1785,"FOREIGN ORG")      ]}),           
            ("WHILE REVIEWING TXNS IN GROUP A/CS OF A R CORP MENTIONED ABOVE DURING THE SAID REVIEW PERIOD, IT WAS NOTED THAT RTGS/NEFT/IMPS CREDITS AMOUNTED TO RS8778 CR MAINLY FROM A R CORP RS771 CR, THE SURAT PEOPLES COOP BANK LTD 612, SKWEAVING P LTD RS156 CR , LAXMI VILAS BANK, TAMILNAD MERCANTILE BANK LTD 515525, 115288517 AND SHIVEN YARN P LTD RS729 CR, THE SURAT PEOPLES COOP BANK LTD 116877 SUBSEQUENT TO SUCH CREDITS, RS7911 CR WERE REMITTED OVERSEAS AS AIRS RS2 CR, DIRECT IMPORT REMS RS2789 CR AND DIRECT BILL RETIREMENT RS7218 CR MAINLY FAVORING CHINA BASED JILIN CHEMICAL FIBER STOCK CO LTD RS5 CR, JILIN ENKA VISCOSE CO LTD RS965 CR AND GRACE HONG KONG INTL TRADING RS22 CR TOWARDS IMPORT OF RAYON YARN, VISCOSE FILAMENT YARN, ETC AS PER WEB CHECK, JILIN CHEMICAL FIBRE STOCK CO, LTD CHINA IS INTO PRODUCTION AND DISTRIBUTION OF CHEMICAL FIBER PRODUCTS, HOWEVER NO SPECIFIC INFO RELATED TO BUSINESS OF JILIN ENKA VISCOSE CO LTD AND GRACE HONG KONG INTL TRADING WAS FOUND RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS277 CR MAINLY FAVORING WELLKNOWN POLYESTERS LTD RS79 CR , SBI 166691519, GRASIM INDS LTD RS788 CR, HDFC 1112 AND RELIANCE INDS LTD RS82 CR, HDFC RILDFY856829 CHEQUE ISSUANCE AMOUNTED TO RS 195 CR MAINLY FAVORING GRASIM INDU LTD RS125 CR, A R CORP RS965 CR AND AGLON INDS PVT RS6 CR INTERCONNECTED FUNDS TRANSFER TXNS WITHIN THESE A/CS AMOUNTED TO RS128 CR KHUSHI ENTP AND A R CORP ARE YET TO SUBMITTED BOES FOR 2 CASES OF AIRS AND DIRECT BILL RETIREMENTS TOTALING TO USD 1,71,698, OF WHICH 17 CASES TOTALING TO USD 1,8,11 FOR THE PERIOD LESS THAN MONTHS, 15 CASES USD 1,117,19 FOR THE PERIOD BEYOND MONTHS 6 MONTHS, 91 CASES USD 8,19,72 FOR THE PERIOD BEYOND 6 MONTH 1 YEAR, 2 CASES USD 2 FOR THE PERIOD BEYOND 1 YEAR TILL 2 YEARS, 5 CASES USD 1,66,18 FOR A PERIOD BEYOND 2 YEARS TXNS IN THE NEWLY OPENED A/CS OF KHUSHI ENTP FAR EXCEED THE ANNUAL TURNOVER INDICATED NO BUSINESS EXPENSE TXNS LIKE PAYMENTS TOWARDS TAXES, RENT ETC, COULD BE SEEN ROUTED THROUGH THE A/CS DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS OF KHUSHI ENTP, IT WAS NOTED THAT THE PREMISE WAS 15 SQ FT RESIDENTIAL AREA WITH NO SIGN BOARD, STOCK OF GOODS OR BUSINESS ACTIVITY SEEN ON ENQUIRING ABOUT THE REASON FOR ROUND SUM TXNS WITH A R CORP, IT WAS LEARNT FROM CUSTOMER THAT A R CORP IS CUSTOMERS GROUP COMPANY HAVING SAME LINE OF BUSINESS HE FURTHER ADDED THAT SINCE HE IS INTO TRADING, STOCKS WERE NOT MAINTAINED THE CUSTOMER HAD FURNISHED ALTERNATE BUSINESS ADDRESS 22 SUPER YARN MARKET, ZAPABAZAAR, SURAT 951 HOWEVER SAME IS NOT UPDATED IN BANK RECORD BASIS TXN PATTERN AND INFORMATION AVAILABLE, IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN NAME OF KHUSHI ENTP AND A/C MAY HAVE BEEN OPENED TO ROUTE TXNS RELATED TO A R CORP FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TBML IT APPEARS THAT THE PROPRIETOR OF KHUSHI ENTP MAY NOT BE THE ACTUAL UBO OF THE FUNDS ROUTED THROUGH THE A/C QUANTUM OF TXNS FOR CURRENT FY 22122 IS DEBIT RS 1967 CR IN CASH RS 8 CR AND CREDIT RS 19152 CR IN CASH RS CR AND THE TXNS TURNOVER FOR PREVIOUS THREE FY ARE 2221 DEBIT RS 8211 CR IN CASH RS 28 CR AND CREDIT RS 85 CR IN CASH RS 2 CR, 2192 DEBIT RS 967 CR IN CASH RS CR AND CREDIT RS 951 CR IN CASH RS CR, 21819 DEBIT RS 7299 CR IN CASH RS 21 CR AND CREDIT RS 716 CR IN CASH RS 2 CR THE A/C 5957721 OF KHUSHI ENTP IS CURRENTLY ACTIVE WITH A BAL OF RS 28 LAKHS AS ON 186221PARIMAL HARSHADRAI HALVAWALA AS A PROPRIETOR MAINTAINS A CURRENT A/C 5957721 IN THE NAME OF A SOLE PROP CONCERN KHUSHI ENTP SINCE 51221 AT OUR KG POINT, GHOD DOD ROAD, NEAR GANGA PALACE, SURAT 957 GUJRAT BRANCH THE ENTITY WAS EST ON 17217 AND HAS SUBMITTED COPIES OF PAN AAFPH25P, GST REG CERT 2AAFPH25P1ZL, IEC CERT AAFPH25P AND CONTACT POINT VERIFICATION REPORT AS KYC DOCS CUSTOMERS REG ADDRESS WITH THE BANK IS BANGLOW NO 1, RAJ LAXMI BANGLOW, OPP RAJHANS CINEMA, PIPLOD ROAD, SURAT 957, GUJRAT AND HAS REG MOBILE NO 982511168 THE ENTITY IS CATEGORIZED AS HIGH RISK AND THE A/C IS NOT YET DUE FOR PERIODIC UPDATION OF KYC IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF TEXTILES WITH ANNUAL TURNOVER OF RS9 CR FUNDS WERE RECEIVED AS RTGS/NEFT/INTERNAL FUNDS TRANSFER CREDITS IN THE A/C OF KHUSHI ENTP ONLY FROM A R CORPORATION WHICH ALSO MAINTAINS AN A/C 21168262 WITH THE BANK ONE OF THE PARTNERS OF A R CORP NATVERLAL DAHYABHAI RUWALA IS A COMMON PARTNER/ DIRECTOR IN VARIOUS A/CS OPENED ALONG WITH RELATED INDIVIDUALS DETAILS OF SUCH INDIVIDUALS ARE UPDATED IN XML FILE THE A/CS OPENED BY NATVERLAL D RUWALA COMPRISE OF CURRENT A/CS IN THE NAMES OF PARTNERSHIP FIRMS, N D R SILK INDS 12792697, N D R YARNS 1112672 AND S R SILK 99125656, 99125657, A REG CO NAMED SHIVEN YARN P LTD 6122552, IN THE NAME OF RATNA MILAN APT CO OPERATIVE HOUSING SOC 151285921, INDIVIDUAL CURRENT A/CS IN THE NAME OF NATVERLAL D RUWALA 71299822, 7126977, 7128966 BESIDES SAVINGS A/CS WERE OPENED IN THE NAMES OF JASH JITESHKUMAR RUWALA 511151221, 6811996686, RUWALA RIDDESH 9212261889, 921252 AND VINAY AGARWAL 21182995, 578159 THE MENTIONED A/CS WERE OPENED DURING THE PERIOD 22112 17221 AT OUR SURAT BRANCH AND GHOD DOD ROAD SURAT BRANCH LOCATED IN GUJARAT CREDIT FACILITIES WERE ALSO AVAILED BY NATVERLAL DAHYABHAI RUWALA A/C NO7568 SANCTIONED LIMIT RS5 CR / UTILIZED AMOUNT IS RS6 CR, N D R YARNS 1156821 SANCTIONED LIMIT RS 5 CR / UTILIZED AMOUNT IS RS CR, 11126676 SANCTIONED LIMIT RS1 CR / UTILIZED AMOUNT IS RS927 CR N D R SILK INDS 11182 SANCTIONED LIMIT RS6 CR / UTILIZED AMOUNT IS RS58 CR AS PER PROFILE SHEET, THE ENTITIES MENTIONED ABOVE ARE INTO BUSINESS OF TEXTILE WITH ANNUAL TURNOVER IN THE RANGE OF RS6 CR RS6 CR NO UNUSUAL ACTIVITY WAS OBSERVED IN THE AFORESAID A/CS OF RATNA MILAN APT CHS, S R SILK AND SHIVEN YARN P LTD AND IN THE SAVINGS A/CS511151221, 578159, 6811996686, 921252, 9212261889 21182995 DURING THE REVIEW PERIOD NOTICING HIGH VALUE ROUND SUM IMPS/NEFT/RTGS CREDITS AND SUBSEQUENT ADVANCE IMPORT REM ROUTED THROUGH ABOVE MENTIONED A/C OF KHUSHI ENTP, ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD AND THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLL GROUNDS OF SUSPICION ON REVIEWING TXNS IN THE A/C OF KHUSHI ENTP 5957721 SINCE 122 166221, A PATTERN OF TXNS WERE NOTED WHEREIN HIGH VALUE CREDITS RECEIVED FROM A R CORP BY WAY OF RTGS/NEFT/IMPS REM RS16 CR, THE SURAT PEOPLES COOP BANK LTD 612 AND INTERNAL FUNDS TRANSFER CREDITS RS CR, A/C NO21168262 WERE IMMEDIATELY FOLLOWED BY HIGH VALUE ADVANCE IMPORT REM AIR AND DIRECT BILL RETIREMENTS TOTALING TO RS1 CR MAINLY FAVORING XINXIANG CHEMICAL FIBER CO LTD CHINA TOWARDS IMPORT OF YARNS AS PER WEB CHECK XINXIANG CHEMICAL FIBER CO LTD IS A CHINA BASED COMPANY ENGAGED IN THE MANUFACTURE AND DISTRIBUTION OF CHEMICAL FIBER TEXTILE RAW MATERIALS RTGS/NEFT/IMPS PAYMENTS AMOUNTED TO RS199 CR FROM THE A/C OF KHUSHI ENTP MAINLY FAVORING NIRANJAN SHIPPING AGENCY P LTD RS19 CR, IDBI 2812128",{"entities":[(559,592,"FOREIGN ORG"),(601,626,"FOREIGN ORG"),(640,626,"FOREIGN ORG"),(752,786,"FOREIGN ORG"),(905,930,"FOREIGN ORG"),(935,963,"FOREIGN ORG")      ,(6501, 6531,"FOREIGN ORG"),(6579,6609,"FOREIGN ORG")   ]}),
            ("CUSTOMER KYC AND ACCOUNT DETAILS 1 CUSTOMER NAME BHARAT INDL ENTERPRISES LTD AND CUSTOMER ADDRESS VPO TARAORI,DISTTKARNAL ,KARNAL ,KARNAL HR,1211 INDIA 2 DATE OF BIRTH / DATE OF INCORPORATION 2881997 PAN NO AABCB2611C PASSPORT NO 5OTHER IDENTIFICATION DOCUMENTS CINU1111HR1997PLC69 6IEC CODE 7GSTIN NO 8MOBILE NUMBER/LANDLINE NO 955818 9OTHER FACILITIES AVAILED 1RISK CATEGORY HIGH RISK 11NAME OF THE AUTHORISED SIGNING AUTHORITIES NATHI RAM GUPTA, RAM PAL, MOHIT GUPTA 12KYC COMPLIANCE STATUS 1DATE OF LAST KYC UPDATION 29722 CUSTOMER PROFILE 1NATURE OF BUSINESS MFG OF GRAIN MILL PRODUCTRICE/DAL ETC 15 OCCUPATION 16 ANNUAL INCOME AS DECLARED IN KYC 17ANNUAL TURNOVER AS DECLARED IN KYC RS28 LACS 18BENEFICIAL OWNERS AS DETERMINED UNDER RULE 9 OF PMLA RULES MONIKA GUPTA, SUSHIL KUMAR GUPTA, NATHI RAM GUPTA, RAM PAL, MOHIT GUPTA ACCOUNT DETAILS 19BANK AND BRANCH DETAILS STATE BANK OF INDIA , SME BRANCH KARNAL ,DISTKARNAL HARYANA, 121 2BANK ACCOUNT NO 127971719 21BANK ACCOUNT TYPE CA 22BANK A/C OPEN ON 1998 2BANK ACCOUNT STATUS ACTIVE TRANSACTION SUMMARY 2 QUANTUM OF TRANSACTIONS FOR THE CURRENT YEAR FROM1221 TO15221 TOTAL CREDIT RS 172 LAC AND TOTAL DEBIT RS 1595 LAC AND TOTAL CASH CREDIT RS AND TOTAL CASH DEBIT RS 25 QUANTUM OF TRANSACTIONS FOR PREVIOUS YEARS FROM 122 TO 1221 TOTAL CREDIT RS 1556 LAC AND TOTAL DEBIT RS 1579 LAC AND TOTAL CASH CREDIT RS AND TOTAL CASH DEBIT RS FROM 1219 TO 122 TOTAL CREDIT RS 11976 LAC AND TOTAL DEBIT RS 1199 LAC AND TOTAL CASH CREDIT RS AND TOTAL CASH DEBIT RS 12 LAC FROM 1218 TO 1219 TOTAL CREDIT RS 298 LAC AND TOTAL DEBIT RS 29917 LAC AND TOTAL CASH CREDIT RS AND TOTAL CASH DEBIT RS 26 BALANCE IN ACCOUNT ON DATE OF FILING STR RS 81 LAC HISTORY 27DETAILS OF EARLIER STRS FILED ON THE ACCOUNT OR ACCOUNT HOLDERS/RELATED PERSONS GROUND OF SUSPICIONBHARAT INDUSTRIAL ENTERPRISES LIMITED IS A PUBLIC INCORPORATED ON 28 AUGUST 1997 IT IS CLASSIFIED AS NONGOVT COMPANY AND IS REGISTERED AT REGISTRAR OF COMPANIES, DELHI IT IS INVOLVED IN GROWING OF CROPS; MARKET GARDENING; HORTICULTURE CORPORATE IDENTIFICATION NUMBER IS CINU1111HR1997PLC69GSTIN 6AAECR1217R1Z5 6AAECR1217R2ZDIRECTORS OF BHARAT INDUSTRIAL ENTERPRISES LIMITED ARE MONIKA GUPTA, SUSHIL KUMAR GUPTA, NATHI RAM GUPTA, RAM PAL, MOHIT GUPTA COMPANY HAS EXPORTED SOME GOODS AND SUBMITTED THE DOCUMENTS THROUGH BANK HOWEVER PAYMENT OF THE BILL IS STILL PENDING DETAILS OF PENDING EXPORT BILLS ARE AS UNDERSR NO BILLREF NO CURR OUTSTANDING AMT DRAWEE NAME INVOICE_NO SHIPMENT DATE1 719GC57 AED1891769 ARAD INTERNATIONAL GENERAL TRADING BIEL/67/17 /27/2182 719GC59 AED16821 ARAD INTERNATIONAL GENERAL TRADING BIEL/12/18 /22/218 719GC58 AED17592 ARAD INTERNATIONAL GENERAL TRADING BIEL/116/18 5//218 719GC517 USD5687 HASSAN AND PARTNERS LLC BIEL/5/18 /15/2195 719GC82 USD1 ALGRO INTERNATIONAL LLC BIEL/168/18 7/18/2186 719GC75 USD272667 ALGRO INTERNATIONAL LLC BIEL/262/17 12/17/217IT IS OBLIGATORY ON THE PART OF THE EXPORTER TO REALIZE AND REPATRIATE THE FULL VALUE OF GOODS / SOFTWARE / SERVICES TO INDIA WITHIN NINE MONTHS FROM THE DATE OF EXPORT TOTAL VALUE OF THE OUTSTANDING BILLS IN INR AT PRESENT RATE IS APROX RS 182 LAC AT PRESENT VALUE OF CONVERSION RATE HENCE STR HAS BEEN ESCALATED",{"entities":[(2508,2542,"FOREIGN ORG"),(2580,2614,"FOREIGN ORG"),(2651,2685,"FOREIGN ORG"),(2722,2745,"FOREIGN ORG"),(2842,2865,"FOREIGN ORG"),( 2778,2801,"FOREIGN ORG")]}),
            ("INWARD REMITTANCE IN FOREIGN CURRENCY ACCOUNT 122299511O GBP 52 LAKHS 2 TRANCHES OF WHICH GBP 25 LAKHS 22 TRANCHES WERE FROM INFOPLUS TECHNOLOGIES UK LTD, GBP 97,65 9 TRANCHES FROM ZAX TECHNOLOGIES LTD, GBP 7,15 7 TRANCHES FROM FUTURECOM SOLUTIONS LTD ETC5 INWARD REMITTANCE IN FOREIGN CURRENCY ACCOUNT 122299512O EUR 58 LAKHS 29 TRANCHES OF WHICH EUR 78 LAKHS 25 TRANCHES WERE FROM INFOPLUS TECHNOLOGIES UK LTD, EUR 8, TRANCHES BRAINSTREAM UK LTDO USD 5,525 2 TRANCHES OF WHICH USD 289 WAS FROM SHELL MDS MALAYSIA SDN BHD AND USD 2,61 FROM GMR MALE' INTERNATIONAL AIRPORTIN VIEW OF THE INVESTIGATIONS BY THE ENFORCEMENT DIRECTORATE, AN STR IS BEING FILEDINFO PLUS TECHNOLOGIES PRIVATE LIMITED PAN AABCI759E; GST AABCI759E1ZQ WAS HOLDING A CURRENT ACCOUNT 1222991 OPENED ON 11227 AND CLOSED ON 152221 WITH THE CHENNAI MAIN BRANCH OF THE BANK IT WAS ALSO HOLDING 2 FOREIGN CURRENCY ACCOUNTS, 122299511 GBP OPENED ON 7228 AND 122299512 EUR OPENED ON 71129THE COMPANY IS INTO IT SERVICES COMPANY FOCUSING ON ENTERPRISE SOLUTIONS AND INTEGRATION SERVICES CONSULTING, SOFTWARE DEVELOPMENT, PRODUCT SOLUTIONS FOR EDUCATION SEGMENT AND IT CONSULTING SERVICES OFFERING OUTSOURCING SOLUTIONS TO ENTERPRISES WORLDWIDE THE DECLARED ANNUAL SALE TURNOVER IS RS 2515 CRORES THE BUSINESS ADDRESS IS NO61, DEVELOPED PLOT ESTATE PERUNGUDI CHENNAI 696, AND THE CONTACT DETAIL IS 9888777 M MS SAKUNTHALA DEVI SETTIPALLI, SRIKANTH S AND MOHAMED NABI ARE THE DIRECTORS OF THE COMPANYAN ALERT WAS RAISED AS THE BANK RECEIVED AN EMAIL DATED 5 MARCH 221 VIDE REFERENCE NUMBER F NO T2/HYZO/FEMA/2/221, FROM THE OFFICE OF MR BENJAMIN CHETTIAR, DEPUTY DIRECTOR, DIRECTORATE OF ENFORCEMENT, HYDERABAD, FOR THE INVESTIGATION AGAINST THE CUSTOMER UNDER THE PROVISION OF FEMA 1999 AND PMLA 22 IN VIEW OF THE SAME, THE TRANSACTIONS IN THE AFORESAID ACCOUNTS WERE REVIEWED, AS INDICATED BELOW1 CURRENT ACCOUNT 1222991 122 TO 152221 DATE OF CLOSURE CREDITS WERE BY WAY OFO CREDITS AGGREGATED TO RS 68 LAKHS 9 TRANCHES OF WHICH RS 99 LAKHS 1 TRANCHE WAS INCOME TAX REFUND, RS 51 LAKHS 2 TRANCHES FROM GMR AIRPORTS LTD, RS LAKHS 1 TRANCHE FROM MAARUTHI ART FILMS, RS 2 LAKHS FROM VEDANTA LTDO IMPS CREDITS AGGREGATING TO RS 82, TRANCHES DEBITS WERE BY WAY OFO DEBITS AGGREGATED TO RS 6667 LAKHS 9 TRANCHES OF WHICH RS 75 LAKHS TRANCES WERE TRANSFERRED TO OWN ACCOUNT HELD WITH ICICI BANK, RS 15 LAKHS 5 TRANCHES WERE PAID TO INFOPLUS MDM PVT LTD, RS RS 7 LAKHS 2 TRANCHES WERE PAID AS SALARY, RS 5 LAKHS 1 TRANCHES WERE FOR OFFICE EXPENSES COURIER, MOBILE, INTERNET2 NO TRANSACTION IN THE FOREIGN EXCHANGE ACCOUNTS 122299511 GBP AND 122299512 EUR DURING THE REVIEW PERIOD 122 TO 152221 DATE OF CLOSURE\ED HAS ASKED FOR THE DETAILS OF THE REMITTANCES DONE BY THE CUSTOMER, SUMMARY OF WHICH IS PROVIDED BELOW REVIEW OF INWARD / OUTWARD REMITTANCE IN CURRENT ACCOUNT 1222991 FROM 11227 TO 152221 ARE HIGHLIGHTED AS UNDER OUTWARD REMITTANCES WERE CARRIED OUT FROM CURRENT ACCOUNT 1222991 WHICH INCLUDED RS CRORES 6 TRANCHES PAID TO ONLEY SOLUTIONS LTD UAE TOWARDS SOFTWARE DEVELOPMENT AND IMPLEMENTATION AND ONSITE IT CONSULTING AND RS 2659 LAKHS TRANCHES TO MATAHARI ICT M SDN BHD MAURITIUS TOWARDS CONSULTING CHARGES INWARD REMITTANCE INCLUDESO USD 595 LAKHS 151 TRANCHES OF WHICH USD 22 LAKHS 8 TRANCHES WERE FROM INFOPLUS TECHNOLOGIES UK LTD, USD 8 LAKHS 21 TRANCES FROM BRAINSTREAM UK LTD, USD 159 LAKHS 18 TRANCHES FROM SPARESFINDER LTD, USD 199 LAKHS 9 TRANCHES FROM BASRAH GAS COMPANY, USD 127 LAKHS 7 TRANCHES FROM STAR LINK MIDDLE EASTO RECEIPT IN CAD AGGREGATED TO CAD 17,55 2 TRANCHES FROM PHOENX AMC INCO RECEIPT IN INR AGGREGATED TO RS 5185 LAKHS 6 TRANCHES OF WHICH RS 11 LAKHS TRANCHES WERE FROM SPARESFINDER LTD, RS 7 LAKHS 1 TRANCHE FROM DR S CHANDRU AND RS 9 LAKHS 1 TRANCHE FROM SAP ECHO LTDO EUR 1697 LAKHS 77 TRANCHES OF WHICH EUR 1525 LAKHS 66 TRANCHES WERE FROM INFOPLUS TECHNOLOGIES UK LTD, EUR 19 LAKHS 2 TRANCHES BRAINSTREAM UK LTD ETCO GBP 266 LAKHS 16 TRANCHES OF WHICH GBP 15 LAKHS 77 TRANCHES FROM ZAX TECHNOLOGIES LTD, GBP 72 LAKHS TRANCHES WERE FROM SAP ECHO LTD, GBP 2 LAKHS 19 TRANCHES FROM INFOPLUS TECHNOLOGIES UK LTD, GBP 269 LAKHS 2 TRANCHES FROM BRAINSTREAM UK LTD ETC",{"entities":[(125,154,"FOREIGN ORG"),(181,200,"FOREIGN ORG"),(228,251,"FOREIGN ORG"),(383,410,"FOREIGN ORG"),(429,446,"FOREIGN ORG"),(496,522,"FOREIGN ORG")     ,(3277,3304,"FOREIGN ORG"),(3846,3873,"FOREIGN ORG"),(4086,4113,"FOREIGN ORG"),(2992,3011,"FOREIGN ORG"),(3119,3141,"FOREIGN ORG"),(3335,3352,"FOREIGN ORG"),(3386,3401,"FOREIGN ORG"),(3434,3451,"FOREIGN ORG"),(3484,3506,"FOREIGN ORG"),(3562,3577,"FOREIGN ORG"),(3672,3687,"FOREIGN ORG"),(3759,3770,"FOREIGN ORG"),(3900,3918,"FOREIGN ORG"),(4043,4054,"FOREIGN ORG"),(4146,4164,"FOREIGN ORG")]}),
            ("FY 21819 FROM 26219 TO 1219 TOTAL CREDIT SIDE RS 7 LAKHS TOTAL DEBIT SIDE RS TRANSACTION PATTERN SHOWS HIGH VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, CREDITS FOLLOWED BY IMMEDIATE DEBITS SEEN IN THE ACCOUNT FOR EXAMPLE, ON 28221, RS15 CR HAS BEEN CREDITED IN THE CUSTOMER ACCOUNT THROUGH INTERNET FUND TRANSFER FROM RINTAC TRADING PRIVATE LIMITED ACCOUNT NUMBER5752166 AND ON SAME DAY RS16 CR HAS BEEN DEPLOYED TOWARDS INTERNET FUND TRANSFER TO KAY KAY TRADING COMPANY ACCOUNT NUMBER2615175 FURTHER THE AMOUNT HAS BEEN TRANSFERRED TO DIFFERENT INDIVIDUALS AND ENTITIESSIMILARLY ON 76221, RS95 LAKHS HAS BEEN CREDITED IN THE CUSTOMER ACCOUNT THROUGH RTGS FROM HIS ANOTHER ACCOUNT ALLIANCE IMPORT AND EXPORT ACCOUNT NUMBER 51611, IFSC CODE DCBL9 AND ON SAME DAY WITH SAME AMOUNT HAS BEEN DEPLOYED TOWARDS RTGS TO ASAD N MOULVI ACCOUNT NUMBER 511212712, IFSC CODE HDFC2 ALSO BRANCH HAS RAISED A COMPLIANT STATING THAT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT WHICH SEEMS TO BE UNUSUAL IN VIEW OF HIGH VALUE TRANSACTIONS, CREDITS FOLLOWED BY IMMEDIATE DEBITS FOR THE ACCOUNT, STR IS BEING REPORTEDALLIANCE IMPORT AND EXPORT, PARTNERSHIP FIRM, HAVING ADDRESS27 FAIZ E AKHTAR GR,FLR,SHOP NO 1 MEMONWADA RD NR SHALIMAR,HOTEL, MUMBAI, MAHARASHTRA, INDIA CONTACT NUMBER 91955 AND EMAIL ID RABMETAL@LIVEIN IS MAINTAINING A CURRENT ACCOUNT NUMBER 5752221 WITH ICICI BANK LTD, MUMBAI PRABHADEVI, ICICI BANK LTD RAVINDRA NATYA MANDIR,KALA ACADEMY,PRABHADEVI, MUMBAI, MAHARASHTRA, 25, SINCE 26219 DATE OF INCORPORATION IS 9215 PAN NUMBER OF THE ACCOUNT IS ABEFA1656K TRANSACTION LINKED ACCOUNTS ARE 5752166,12255186 AUTHORISED SIGNATORIES AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT ARE MOHAMMED ASAD NAEEMUDDIN MOULVI WITH PAN NO AFCPM8595F, RASHID ABBAS BUSHERI WITH PAN NO ABPPB9217L THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE REGISTRATION CERTIFICATE, PAN CARD ABEFA1656K, PARTNERSHIP DEED ETC KYC UPDATION IS NOT DUE CUSTOMER IS SELFEMPLOYED AS PER BANK RECORDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAKHS TO RS 5 CRORES ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS BALANCE IN THE ACCOUNT AT 77221 WAS RS89 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 77221 SHOWS TOTAL CREDIT RS 2,165 LAKHS OF WHICH CASH WAS RS 175 LAKHS MAJORLY IN PRABHADEVI BRANCH, TRANSFERS RS 75 LAKHS MAJORLY FROM KAY KAY TRADING COMPANY,ALLIANCE IMPORT AND EXPORT, CLEARING RS 1591 LAKHS, NEFT RS 12 LAKHS MAJORLY FROM NIRMA LIMITED,VELJI P AND SONS AGENCIES PR, RTGS RS 1,6152 LAKHS MAJORLY FROM NIRMA LIMITED,ALLIANCE IMPORT AND EX,SHIV SHIPPING SERVICES, REMITTANCE RS 2969 LAKHS MAJORLY FROM ALLIANCE IMPORT AND EXPORT, INTERNET FUND TRANSFERS RS 296 LAKHS, MOBILE MONEY TRANSFERS RS 996 LAKHS IN 15 INSTANCES, FIXED DEPOSIT RS 22 LAKHS, AND OTHERS RS 5 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 2,2672 LAKHS OF WHICH CASH WAS RS 292 LAKHS BY BRANCH, TRANSFERS RS 12 LAKHS MAJORLY TO ALLIANCE IMPORT AND EXPORT, CLEARING RS 2 LAKHS MAJORLY TO CHRIST ACADEMY, RTGS RS 8287 LAKHS, REMITTANCE RS 591 LAKHS MAJORLY TO WONDER STAR TRADING LLC,ATLAS SHIPPING LINES LTD,BLUE POWER BUILDING AND, INTERNET FUND TRANSFER RS 8 LAKHS, MOBILE MONEY TRANSFERS RS 1681 LAKHS IN INSTANCES, AND OTHERS RS 529 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 16,56 LAKHS OF WHICH CASH WAS RS 5896 LAKHS MAJORLY IN WORLI,PRABHADEVI BRANCH, TRANSFERS RS 792 LAKHS MAJORLY FROM KAY KAY TRADING COMPANY,ALLIANCE IMPORT AND EXPORT,OCEANIC TRADE MINERALS PRIVATE LIMITED, CLEARING RS 98 LAKHS MAJORLY FROM ARNAV SHIPPING PRIVATE LIMITED, NEFT RS 7921 LAKHS MAJORLY FROM OCEANIC TRADE MINERALS PRIVATE L,REFINE ALLOY PRIVATE LIMITED,KAY KAY TRADING CO, RTGS RS 1,6978 LAKHS MAJORLY FROM REFINE ALLOY PRIVATE L,KAY KAY TRADING CO,OCEANIC TRADE MINERALS, REMITTANCE RS 2,728 LAKHS, INTERNET FUND TRANSFERS RS 788 LAKHS, MOBILE MONEY TRANSFERS RS 1996 LAKHS IN 2 INSTANCES, FIXED DEPOSIT RS 25 LAKHS AND OTHERS RS 77 LAKHS TOTAL DEBIT SIDE RS 16,5591 LAKHS OF WHICH CASH WAS RS 61 LAKHS BY BRANCH, TRANSFERS RS 97192 LAKHS MAJORLY TO BABU KANAGRAJ,RINTAC TRADING PRIVATE LIMITED,ALLIANCE IMPORT AND EXPORT, CLEARING RS 9279 LAKHS MAJORLY TO MITRA S K PVT LTD, RTGS RS ,697 LAKHS MAJORLY TO REFINE ALL, REMITTANCE RS 6,5929 LAKHS MAJORLY TO INFINITE ENERGY FZE,ALASKA MARINE SERVICES LLC,BLUE POWER BUILDING AND, INTERNET FUND TRANSFER RS ,551 LAKHS, MOBILE MONEY TRANSFERS RS 1798 LAKHS IN 88 INSTANCES, AND OTHERS RS 9 LAKHS FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 77 LAKHS TOTAL DEBIT SIDE RS 1176 LAKHS",{"entities":[(681,707,"FOREIGN ORG"),(1143,1168,"FOREIGN ORG"),(2557,2582,"FOREIGN ORG"),(2731,2752,"FOREIGN ORG"),(2816,2841,"FOREIGN ORG"),(3115,3140,"FOREIGN ORG"), (3725,3750,"FOREIGN ORG") ,(3245,3267,"FOREIGN ORG"),(3269,3292,"FOREIGN ORG"),(3294,3304,"FOREIGN ORG"),(4395,4420,"FOREIGN ORG") ,(4556,4574,"FOREIGN ORG"),(4576,4601,"FOREIGN ORG"),(4603,4613,"FOREIGN ORG")]}),
            ("THE FEEDBACK ABOUT THE CUSTOMERS BILL PENDING ARE NEGATIVE AS THERE ARE 2 BOES PENDING FOR SUBMISSION WITH OVERDUE ABOVE 56 DAYS, 2 IMPORT MADE PAYMENT PENDING OUT WHICH 21 ARE OVERDUE FOR SUBMISSION AGEING ABOVE 27 AND MORE THAN 1 DAYS IN VIEW OF HIGH NUMBERS OF VARIOUS PENDING DOCUMENTS RELATED TO BOE, IMPORT BILLS, STR IS BEING FILEDVEEKAY PRINT SERVICES, A PARTNERSHIP FIRM, HAVING ADDRESS NO 69, OLD NO 5, ROYAPETTAH HIGH RD, 1ST FLR, JUSTICE PRATAPSINGH COMPLEX, CHENNAI, TAMIL NADU, INDIA, 61, CONTACT NUMBER 91921625 AND EMAIL ID VEEKAYPRINT@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 622599 WITH ICICI BANK LTD, NO 1A, P S SIVASAMY SALAI, MYLAPORE, CHENNAI, TAMIL NADU, 6, SINCE 2525 DATE OF INCORPORATION IS 1125 PAN OF THE ACCOUNT IS AAFFV16M IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 2756 TRANSACTION LINKED ACCOUNTS ARE 622159912, 626551582 AUTHORIZED SIGNATORY/ PARTNER OF THE REPORTED ACCOUNT IS KARTHIKEYAN VADIVELU THE CUSTOMER IS CATEGORIZED AS MEDIUM RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAFFV16M, PARTNERSHIP DEED CUSTOMER IS INTO IMPORT AND TRADING OF PRINTER MACHINES AND SPARES, PRESS RIM AND RELATED SERVICES ALERT WAS TRIGGERED FOR UNUSUAL FOREX REMITTANCE PATTERN IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 15221 WAS RS 88 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 15221 SHOWS TOTAL CREDIT RS 12 LAKHS OF WHICH NEFT RS 66 LAKHS MAJORLY FROM THE MALAYALA MANORAMA CO, MANOHARAN V, DINAMALAR, INTERNET FUND TRANSFERS RS 126 LAKHS FROM V KARTIKHEYAN ETC, MOBILE MONEY TRANSFERS MMT RS 221 LAKHS IN 5 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 92 LAKHS OF WHICH REMITTANCE RS 2 LAKHS MAJORLY TO KOENIG AND BAUER DIGITAL, INTERNET FUND TRANSFER RS 62 LAKHS TO V KARTHIKEYAN ETC AND OTHERS RS 18 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 1129 LAKHS OF WHICH CLEARING RS 55 LAKHS MAJORLY FROM PRINTERS PVT LTD, THOMSON PRESS, RATHANKUMARI D, NEFT RS 8765 LAKHS MAJORLY FROM BENNETT COLEMAN AND CO LTD, THE MALAYALA MANORAMA CO, RTGS RS 77 LAKHS MAJORLY FROM DHAYANAND, INTERNET FUND TRANSFERS RS 71 LAKHS FROM V KARTIKHEYAN ETC, MOBILE MONEY TRANSFERS RS 28 LAKHS IN 18 INSTANCES, UPI TRANSFERS RS 6 LAKHS IN 7 INSTANCES TOTAL DEBIT SIDE RS 1152 LAKHS OF WHICH CLEARING RS 815 LAKHS MAJORLY TO DHL EXPRESS INDIA PVT LTD, REMITTANCE RS 81 LAKHS MAJORLY TO ERNST REHFUSS WERKZEUGBAU GMBH, KOENIG AND BAUER DIGITAL, INTERNET FUND TRANSFER RS 658 LAKHS TO V KARTIKHEYAN ETC AND OTHERS RS 19 LAKHS FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 1718 LAKHS OF WHICH CASH WAS RS 12 LAKHS TOTAL DEBIT SIDE RS 16981 LAKHS OF WHICH CASH WAS RS 2 LAKHS, REMITTANCE RS 598 LAKHS FY 21819 FROM 1218 TO 1219 TOTAL CREDIT SIDE RS 221 LAKHS OF WHICH CASH WAS RS 58 LAKHS TOTAL DEBIT SIDE RS 259 LAKHS OF WHICH CASH WAS RS 2 LAKHS, REMITTANCE RS 652 LAKHS FY 21718 FROM 1217 TO 1218 TOTAL CREDIT SIDE RS 18999 LAKHS OF WHICH CASH WAS RS 55 LAKHS TOTAL DEBIT SIDE RS 18817 LAKHS OF WHICH CASH WAS RS 15 LAKHS, REMITTANCE RS 67 LAKHS TRANSACTION PATTERN SHOWS NON CASH CREDITS AND DEBITS IN THE ACCOUNT FOR EXAMPLE, ON 21221 RS LACS WERE CREDITED THROUGH MMT AND NEFT FROM MANOHARAN V 1891879, SBIN16 AND ON 26221 RS 1 LACS WERE DEBITED THROUGH REMITTANCE TO KOENIG AND BAUER DIGITAL AND WEBFED AG, GERMANY FOR PAYMENT AGAINST INVOICE ALSO ON 221 RS 27 LACS WERE CREDITED THROUGH RTGS FROM DHAYANAND 16778, CBIN28126 AND ON THE SAME DAY RS 19 LACS WERE DEBITED THROUGH NEFT TO CRAFT MACHINE CORPORATION 958992, IDIBA19, REMITTANCE TO BAIER GMBH TRAGOSSERSTRABE, AUSTRIA FOR ADVANCE PAYMENT AGAINST IMPORT",{"entities":[(2560,2589,"FOREIGN ORG"),(2592,2615,"FOREIGN ORG"),(3444,3468,"FOREIGN ORG"),(3719,3744,"FOREIGN ORG"),(3473,3481,"FOREIGN ORG")]}),
            ("AS PER TRANSACTION PATTERN, IT IS OBSERVED THAT FREQUENT CREDITS THROUGH REMITTANCE FROM COMPANIES WHICH ARE INTO ONLINE BETTING ARE SEEN FOR EXAMPLE, ON 2722, RS 6,771 GOT CREDITED THROUGH INWARD REMITTANCE FROM GENESIS GLOBAL LIMITED IN THE CURRENCY OF EUR FROM THE COUNTRY GB FOR WHICH THE PURPOSE MENTIONED WAS COMMISSION AND ON THE SAME DAY, RS 8LAKHS GOT DEBITED IN PARTS THROUGH TRANSFER TOWARDS ST MARKETING 29568, ASHU CHAUHAN 62915756, NEFT IN FAVOR OF TENGO TECH MEDIA PRIVATE LIMITED ACCNO 2119261, IFSC CODE INDB21, INDUSIND BANK, GOPAISA NETVENTURES PVT LTD ACCNO 698972, IFSC CODE YESB69, YES BANK, NXTIFY TECHNOLOGIES PVT LTD ACCNO 95892, IFSC CODE RATN91, THE RATNAKAR BANK, JUBILANT SERVICES ACCNO 592911129, IFSC CODE HDFC168, HDFC BANK SIMILARLY ON 1982 RS 75 LAKHS GOT CREDITED FROM MOBILE MOMENTUM THROUGH INWARD REMITTANCE IN THE CURRENCY OF EUR FROM THE COUNTRY GB FOR WHICH PURPOSE MENTIONED WAS UNCLEAR AND THE SAME WAS DEBITED IN PARTS MAJORLY THROUGH INTERNET FUND TRANSFERS TO MOHAMMED SHAZ QAMAR 5151565, SHEENU SANJEEV 2251277 ON THE NEXT DAY SIMILAR TRANSACTION PATTERN WERE OBSERVED IN THE ACCOUNT AS PER PUBLIC DOMAIN SEARCH, GENESIS GLOBAL LIMITED IS LICENSED AND REGULATED BY THE MALTA GAMING AUTHORITY REFER HTTPS//GENESISCOMMT/ AND ITS BETTING WEBSITE NAME IS FUNBET IN VIEW OF CUSTOMER BEING INVOLVED IN ONLINE BETTING AND CREDIT IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON THE SAME OR NEXT DAY MAKES IT SUSPICIOUS HENCE, STR IS BEING FILEDRAPIDMILE BUSINESS SOLUTIONS PVT LTD, A PRIVATE LIMITED COMPANY, HAVING ADDRESS E8 SUNCITY INFRONT OF SUNCITY HIGHTS, SECTOR E 5, GURGAON, HARYANA, INDIA, 1222, CONTACT NUMBER 9871228 AND EMAIL ID SHIRAZISCOOL@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 99558 WITH ICICI BANK LTD, SHOP NO19, 2 21, VIPUL ORCHID PLAZA, SUNCITY, SECTOR5, GURGAON, HARYANA, 1222, SINCE 17215 DATE OF INCORPORATION IS 11215 PAN OF THE REPORTED ACCOUNT IS AAHCR251D TRANSACTION LINKED ACCOUNTS ARE 5151565, 59279 AUTHORIZED SIGNATORIES/DIRECTOR OF THE REPORTED ACCOUNT ARE MOHAMMED SHIRAZ QAMAR PAN AACPQ95F, MOHD SHAZ QAMAR PAN AADPQ7595F BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MOHAMMED SHIRAZ QAMAR AACPQ95F MOHD SHAZ QAMAR AADPQ7595F THE CUSTOMER IS CATEGORIZED AS MEDIUM RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PANAAHCR251D OF COMPANY, MOA AND AOA HENCE, KYC UPDATION IS NOT DUE CUSTOMER PROFILE IS OTHER DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAKHS TO RS 5CR ALERT WAS TRIGGERED FOR CUSTOMER BEING INVOLVED IN ONLINE BETTING BALANCE IN THE ACCOUNT AT 87221 WAS RS2116 CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 87221 SHOWS TOTAL CREDIT RS 285 LAKHS OF WHICH CLEARING RS 117 LAKHS MAJORLY FROM INSTAMOJO TECHNOLOGIES PVT LTD, NEFT RS 81 LAKHS MAJORLY FROM PLABS VENTURES PVT LTD, SLICK ORGANICS PVT LTD, VCOMMISSION MEDIA PVT LTD, REMITTANCE RS 95 LAKHS MAJORLY FROM GAMETECH MARKETING LIMITED LE, INTERNET FUND TRANSFERS RS 128 LAKHS MAJORLY FROM MOHAMMED SHAZ QAMAR AND NETCORE CLOUD PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 5 LAKHS IN 8 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 27 LAKHS OF WHICH REMITTANCE RS 2887, INTERNET FUND TRANSFER RS 269 LAKHS MAJORLY TO MOHAMMED SHAZ QAMAR AND NETCORE CLOUD PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 718 LAKHS IN 28 INSTANCES FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 181 LAKHS OF WHICH CLEARING RS 268 LAKHS MAJORLY FROM INSTAMOJO TECHNOLOGIES PVT LTD, NEFT RS 727 LAKHS MAJORLY FROM VCOMMISSION MEDIA PVT LTD, TIMES INTERNET LTD 2875, POLYVALENT DIGITAL SERVICES, RTGS RS 126 LAKHS MAJORLY FROM VCOMMISSION MEDIA PVT, REMITTANCE RS 67 LAKHS MAJORLY FROM VALYUZ UAB OUR CHARGES G, SUPRMEDIA LTD SUPRHOLDINGS, ASPIRE GLOBAL PLC LEVEL G OFF, INTERNET FUND TRANSFERS RS 718 LAKHS MAJORLY FROM MOHAMMED SHAZ QAMAR AND NETCORE CLOUD PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 95 LAKHS IN 7 INSTANCES , UPI TRANSFERS RS 1 LAKHS IN 1 INSTANCE, AND OTHERS RS 81 LAKHS TOTAL DEBIT SIDE RS 122 LAKHS, REMITTANCE RS 5 LAKHS, INTERNET FUND TRANSFER RS 12 LAKHS MAJORLY TO MOHAMMED SHAZ QAMAR AND NETCORE CLOUD PRIVATE LIMITED, CARD TRANSACTIONS RS 27 LAKHS, MOBILE MONEY TRANSFERS RS 27 LAKHS IN 1 INSTANCES , AND OTHERS RS 597 LAKHS FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 96 LAKHS OF WHICH CASH WAS RS 5 LAKHS TOTAL DEBIT SIDE RS 9617 LAKHS OF WHICH CASH WAS RS 15 LAKHS FY 21819 FROM 1218 TO 1219 TOTAL CREDIT SIDE RS 26 LAKHS OF WHICH CASH WAS RS LAKHS TOTAL DEBIT SIDE RS 261 LAKHS OF WHICH CASH WAS NIL FY 21718 FROM 1217 TO 1218 TOTAL CREDIT SIDE RS 978 LAKHS OF WHICH CASH WAS RS 5 LAKHS TOTAL DEBIT SIDE RS 989 LAKHS OF WHICH CASH WAS NIL",{"entities":[(213,235,"FOREIGN ORG")   ,(1160,1182,"FOREIGN ORG")    ,( 3004,3032,"FOREIGN ORG") ,(3341,3369,"FOREIGN ORG"),(3524,3553,"FOREIGN ORG")  ,(3758,3768,"FOREIGN ORG"),(3781,3797,"FOREIGN ORG"),(3812,3829,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN SHOWS, HIGH VALUE NON CASH CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE, ON 55221, RS 25 LACS AMOUNT OF FUND CREDITED THROUGH TRANSFERS FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED 197525 WHERE FUND DEBITED THROUGH RTGS TOWARDS SIDDHISHREE ISPAT UDYOG, ACCOUNT 52775, IFSC HDFC529 ALSO FROM 12221 TO 15221 RS 5 LACS GOT CREDITED BY INTERNET FUND TRANSFERS FROM RAMUKA GLOBAL ISPAT METALS PRIVATE LIMITED 197525, NEFT FROM BIG C TECHNOLOGIES PRIVATE LIMITED 651186, IFSCINDB6 AND ON 15221 RS 88 LACS GOT DEBITED THROUGH OUTWARD REMITTANCE TO OSCAR INTERNATIONAL GROUP CO LTD, ADD 6 F M BUILDING NUMBER 2 NORTH PART, NORTH PART, CHINA, IN USD FOR ADVANCE PAYMENT AGAINST IMPORTS SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED LIKE ROUTING OF FUNDS IN VIEW OF NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS AND ROUTING OF FUNDS, STR IS BEING FILEDDIA ENTERPRISES, A PROPRIETORSHIP FIRM, HAVING ADDRESS NO 1, OLD NO 55 1ST MAIN ROAD, 1ST FLOOR, 1TH CRS, SAMPANGIRAMANAGAR, BANGALORE, KARNATAKA, INDIA, 5627, CONTACT NUMBER 91991896 AND EMAIL ID DEEPAKRAMUKA@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 197528 WITH ICICI BANK LIMITED, NO 12, K H ROAD, BANGALORE, KARNATAKA, 5627, SINCE 279219 DATE OF INCORPORATION IS 17217 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT, SAVINGS ACCOUNT PAN OF THE ACCOUNT IS AKRPR87P OTHER RELATIONSHIP OF CUSTOMER IS 219861, 85258, 197525, 1975272, 6275121262 TRANSACTION LINKED ACCOUNTS ARE 197525, 558827 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS DEEPAK RAMUKA PROPRIETOR AKRPR87P THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE LETTER OF DECLARATION OF PROPRIETORSHIP FIRM, PAN AKRPR87P AS PER ENHANCE DUE DILIGENCE, ENTITY IS ENGAGED IN IMPORTING OF COMPUTER RAM, USB FLASH DRIVES, BLUETOOTH ADAPTERS ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LACS TO BELOW RS 5CRS ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT 65221 WAS RS 21 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 65221 SHOWS TOTAL CREDIT RS 282 LAKHS OF WHICH NEFT RS 261 LAKHS MAJORLY FROM DEEPAK RAMUKA, BIG C TECHNOLOGIES PRIVATE LIMITED, RTGS RS 921 LAKHS MAJORLY FROM MEGHANI TEXTILES, MOHINI SAREE CENTRE, DEEPAK RAMUKA, INTERNET FUND TRANSFERS RS 185 LAKHS MAJORLY FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED TOTAL DEBITS DURING THE SAME PERIOD WAS RS 287 LAKHS OF WHICH RTGS RS 175 LAKHS MAJORLY TO SIDDHISHREE ISPAT UDYOG, REMITTANCE RS 516 LAKHS MAJORLY TO OSCAR INTERNATIONAL GROUP CO LIMITED, INTERNET FUND TRANSFER RS 599 LAKHS MAJORLY TO RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 1,7511 LAKHS OF WHICH CASH WAS RS 511 LAKHS MAJORLY IN BANGALORE K H ROAD BRANCH, CLEARING RS 227 LAKHS MAJORLY FROM ELECTRO STEEL STEELS LIMITED, NEFT RS 21811 LAKHS MAJORLY FROM MS RAJ CONSTRUCTION COMPANY, DIA ENTERPRISES, BIG C TECHNOLOGIES PRIVATE LIMITED, RTGS RS 1,866 LAKHS MAJORLY FROM BIG C TECHNOLOGIES PRIVATE, MITTAL TRADERS, DEEPAK RAMUKA, INTERNET FUND TRANSFERS RS 287 LAKHS MAJORLY FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED, MOBILE MONEY TRANSFERS RS 751 LAKHS IN 56 INSTANCES TOTAL DEBIT SIDE RS 1,7555 LAKHS OF WHICH RTGS RS 626 LAKHS MAJORLY TO JD ENTERPRISES, DEEPAK RAMUKA, REMITTANCE RS 79 LAKHS MAJORLY TO OSCAR INTERNATIONAL GROUP, OSCAR INTERNATIONAL GROUP COLTD, OSCAR INTERNATIONAL GROUP CO LTD, INTERNET FUND TRANSFER RS 751 LAKHS MAJORLY FROM RAMUKA GLOBAL ISPAT AND METALS PRIVATE LIMITED, ESL STEEL LIMITED FY 2192 FROM 279219 TO 122 TOTAL CREDIT SIDE RS 6282 LAKHS TOTAL DEBIT SIDE RS 576 LAKHS OF WHICH CASH WAS RS 6 LAKHS, REMITTANCE RS 15985 LAKHS",{"entities":[(591,623,"FOREIGN ORG"),(2806,2842,"FOREIGN ORG"),(3772,3796,"FOREIGN ORG"),(3799,3829,"FOREIGN ORG"),(3832,3863,"FOREIGN ORG")]}),
            ("5 YINUCO OVERSEAS SERVICES AND YINUCO OVERSEAS LOGISTICS SERVICES P LTD HAVE UPDATED A COMMON MAILING ADDRESS 9TH FLOOR, OFFICE NO 9, A WING, PRANIK CHAMBERS, SAKI VIHAR ROAD, SAKI NAKA, ANDHERI, MUMBAI 72 DURING A VISIT BY BRANCH OFFICIAL AT THE GIVEN MAILING ADDRESS IT WAS NOTED THAT THE PREMISE WAS A 25 SQ FT RENTED PREMISE WITH 2 EMPLOYEES WHERE THE CUSTOMER WAS OPERATING FOR LESS THAN MONTHS FURTHER YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED IS FORMED WITH A MERE PAID UP CAPITAL OF RS1 LAKH IT APPEARS THAT THE CUSTOMERS MAY HAVE DELIBERATELY INDICATED INFLATED ANNUAL TURNOVERS OF RS5 CR AND RS8 CR WITHOUT HAVING GENUINE BUSINESS SET UP OR COMMERCIAL BUSINESS ACTIVITY POSSIBLY TO AVOID BEING TRACKED ON ROUTING HIGH VALUE TRANSACTIONS 6 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED AND ACCOUNTS MAY HAVE BEEN OPENED IN THE NAMES OF YINUCO OVERSEAS SERVICES AND YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED TO ROUTE TRANSACTIONS FOR LAYERING OF FUNDS AND TO REMIT FUNDS OVERSEAS UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS SPACE BOOKING FOR CONTAINERISED CARGO WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING/ HAWALA SETTLEMENT ACCOUNT OF YINUCO OVERSEAS SERVICES IS ACTIVE WITH BALANCE OF RS1,25,896/ AS ON AUGUST 1, 221 QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 221222 IS DEBIT RS61 CR IN CASH RS2 CR AND CREDITS RS62 CR IN CASH RS CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE 22221 IS DEBIT RS11 CR IN CASH RS5 CR AND CREDITS RS11 CR IN CASH RS1 CRPANKAJ RAJENDRA PRASAD KESHARI AS A PROPRIETOR MAINTAINS A CURRENT ACCOUNT BEARING NUMBER 65121 IN THE NAME OF YINUCO OVERSEAS SERVICES SINCE 2JAN21 AT OUR BRANCH LOCATED AT SHOP NO1, AUM SAHEEL TOWER, KASHINATH DHURU COMPOUND, OPP KAMALA MILLS, SB MARG, LOWER PAREL, MUMBAI 1 THE ENTITY WAS FORMED ON 12DEC2 AND HAS SUBMITTED COPIES OF PAN HFKPK52C, IEC CERT HFKPK52C, GST REG CERT 27HFKPK52C1Z1 AND CONTACT POINT VERIFICATION REPORT AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 9TH FLOOR, OFFICE NO 9, A WING PRANIK CHAMBERS, SAKI VIHAR ROAD, SAKI NAKA, ANDHERI, MUMBAI 72 CUSTOMER HAS REGISTERED HIS MOBILE NUMBER 75767 WITH THE BANK THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS5 CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF A CURRENT ACCOUNT IN THE NAME OF YINUCO OVERSEAS LOGISTICS SERVICES P LTD A/C NO71572716, DATE OF A/C OPENING 25FEB21, NATURE OF BUSINESS LOGISTIC AND COURIER, ANNUAL TURNOVER RS8 CR, DATE OF INCORP DEC2, DIRECTORS PANKAJ RAJENDRA PRASAD KESHARI AND AJIT YADAV AND SAVINGS ACCOUNTS IN THE NAMES OF PANKAJ RAJENDRA PRASAD KESHARI 51522267, A/C OPENED ON 15DEC2 AND AJIT YADAV 112887, A/C OPENED ON 27AUG2 THE SAID ACCOUNTS WERE OPENED AT OUR ABOVE MENTIONED LOWER PAREL AND MALAD BRANCHES IN MUMBAI WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES NONIMPORT DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WERE DONE THROUGH THE ABOVE MENTIONED NEWLY OPENED ACCOUNTS OF YINUCO OVERSEAS SERVICES AND YINUCO OVERSEAS LOGISTICS SERVICES PRIVATE LIMITED FAVOURING HONG KONG BASED ENTITIES ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH PAYMENTS WERE DONE SUBSEQUENT TO HIGH VALUE RTGS/NEFT CREDITS AND HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE INCEPTION TILL AUGUST 9, 221 IT WAS NOTED THAT RTGS/NEFT CREDITS AMOUNTED TO RS1 CR MAINLY FROM PRIME VISION INTERNATIONAL RS256 CR, EQUITAS SMALL FINANCE BANK 21979, EVERGREEN INTERNATIONAL RS CR, ICICI 12185572/ IDFC FIRST 16627966 AND CLASSIC MULTITRADE RS27 CR, YES BANK 61852 BASIS THE NAMES OF THE REMITTERS THE NATURE OF BUSINESS IS NOT KNOWN 2 INTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS5 CR MAINLY FROM CLASSIC MULTITRADE RS CR, A/C NO 51556975 BANK HAS ALREADY FILED STR ON THE ACCOUNT OF CLASSIC MULTITRADE AND RELATED ACCOUNTS VIDE BATCH ID 2151787 TOWARDS HIGH VALUE RTGS CREDITS AND FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS SUBSEQUENT TO SUCH CREDITS, FOREIGN OUTWARD REMITTANCE NONIMPORT AMOUNTED TO RS2 CR FAVOURING HONG KONG BASED BENEFICIARIES NAMELY AGILE LOGISTICS LIMITED RS17 CR, SUNSHINE LOGISTIC HK RS17 CR, MIDNIGHT SUN LOGISTICS RS8 CR AND HONG KONG RICH TREND LOGISTICS LIMITED RS CR TOWARDS SPACE BOOKING FOR CONTAINERISED CARGO IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS5 CR MAINLY FAVOURING P LAXMAN AND COMPANY RS51 CR, ICICI 751",{"entities":[(4560,4583,"FOREIGN ORG"),(4593,4613,"FOREIGN ORG"),(4623,4645,"FOREIGN ORG"),(4657,4695,"FOREIGN ORG")]}),
            ("AS PER TRANSACTIONS IT IS OBSERVED THAT TRANSACTIONS IN THE ACCOUNT ARE WITH MULTIPLE ENTITIES, INDIVIDUALS THROUGH MMT, UPI, NEFT, RTGS, REMITTANCE, CASH, INF WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WHICH SEEMS TO BE UNUSUAL FOR EXAMPLE ON 166221 TO 176221 ACCOUNT WAS CREDITED WITH RS 12 LAKHS IN PARTS THROUGH NEFT FROM N R CHEMICALS ACCOUNT NO 8872257, IFSC HDFC887, BANK; HDFC BANK, G BHAGYALAKSHMI NAND ACCOUNT NO 61682115, IFSC CNRB6168, BANK CANARA BANK, UPI TRANSFER FROM PABBU SANJEEVA ACCOUNT NO 126511669, VPA 6967175@YBL, IFSC ANDB1265, BANK ANDHRA BANK AND THE SAME ARE DEBITED IN PARTS THROUGH CASH PAID SELF, CLEARING TO PANKAJ KUMAR, MMT TRANSFERS TO MR AJAY KUMAR BAJPA ACCOUNT NO 7966655, IFSC CBIN2819, BANK CENTRAL BANK OF INDIA, SETHI WHEELS ACCOUNT NO 1922176, IFSC HDFC19, BANK HDFC BANK SIMILARLY, ON 86221 ACCOUNT WAS CREDITED WITH RS 6 9 LAKHS THROUGH INWARD REMITTANCE FROM LAURA GIDA INSAAT TURIZM, TURKEY BUYER GREEN AL BAIDAR FOR FEED ADDITIVES, IRAQ WITH PURPOSE ADVANCE RECEIPTS AGAINST EXPORT CONTRACTS AND FROM 86221 TO 116221 ACCOUNT WAS DEBITED WITH RS 2 LAKHS IN PARTS THROUGH MMT TRANSFERS TO FREIGHT CARE LOGISTICS ACCOUNT NO 59299999657, IFSC HDFC127, BANK HDFC BANK, ROYAL PLASTIC ACCOUNT NO 52228861, IFSC HDFC127, BANK HDFC BANK, INTERNET FUND TRANSFERS TO GLOBUS LOGISYS PVT LTD 55112, S K SALES AND MARKETING 6288525227 SIMILAR TYPE OF TRANSACTIONS ARE OBSERVED IN THE ACCOUNT THERE SEEMS TO BE NO ECONOMIC RATIONALE FOUND FOR SUCH TRANSACTIONS ALSO, ACCOUNT IS MARKED DEBIT FREEZE ON 57221 AS RECEIVED NOTICE U/S 91 CR P C, COMPLIANT NO 5/221 OF IPC SECTION 6, 2, 6, 65, 67, 68, 71, 72, 12B, AS DURING THE INVESTIGATION THE CUSTOMER IS OPENED MULTIPLE BANK ACCOUNTS IN MULTIPLE BANKS TO EARN FRAUDULENT MONEY AND USING THESE ACCOUNTS FOR HIS OWN BENEFITS, RECEIVED FROM DAMAN POLICE ALSO THE BANK WAS ASKED TO SUBMIT KYC DOCUMENTS AND STATEMENT OF ACCOUNTS FROM THE DATE OF ACCOUNT OPENING TURNOVER IN THE ACCOUNT IS RS 118 LAKHS IS VERY HIGH WHICH IS FAR EXCEEDING THE DECLARED TURNOVER AT THE TIME OF ACCOUNT OPENING OF RS5 LAKHS IN VIEW OF HIGH ACTIVITY IN A NEW ACCOUNT, ACCOUNT FREEZE DUE TO COMPLAINT RECEIVED AND HIGH TURNOVER MAKES IT SUSPICIOUS HENCE, STR IS BEING FILEDPROMOIS INTERNATIONAL LTD, A PROPRIETORSHIP FIRM, HAVING ADDRESS TH, FLOOR ROOM NO 2, KRISHNA TOWER 15/, 6 CIVIL LINES, KANPUR NAGAR, UTTAR PRADESH, INDIA, 281, CONTACT NUMBER 918816577 AND EMAIL ID PROMOISINTERNATIONAL88@GMAIL COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 628852522 WITH ICICI BANK LTD, 16/16, J S TOWERS, THE MALL, KANPUR, UTTAR PRADESH, 281, SINCE 122 DATE OF FORMATION IS 71219 PAN NUMBER OF THE ACCOUNT IS FNGPM6566P GSTIN AS PER BANK RECORDS IS 9FNGPM6566P1ZX TRANSACTION LINKED ACCOUNTS ARE 55112, 69575512 AUTHORIZED SIGNATORIES/PROPRIETOR/DIRECTOR OF THE REPORTED ACCOUNT ARE ANJUM MASIH PAN FNGPM6566P THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS DEBIT FREEZED ON 57221 AS RECEIVED NOTICE U/S 91 CR P C, COMPLIANT NO 5/221 OF IPC SECTION 6, 2, 6, 65, 67, 68, 71, 72, 12B, AS DURING THE INVESTIGATION THE CUSTOMER IS OPENED MULTIPLE BANK ACCOUNTS IN MULTIPLE BANKS TO EARN FRAUDULENT MONEY AND USING THESE ACCOUNTS FOR HIS OWN BENEFITS, RECEIVED FROM DAMAN POLICE KYC DOCUMENTS OBTAINED WERE GSTIN9FNGPM6566P1ZX, PANFNGPM6566P, COIU21UP21PLC65 ENTITY IS MANUFACTURER, WHOLESALER AND EXPORTER OF AMINO ACID POWDER, ANIMAL FEED SUPPLEMENT DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAKHS TO LESS THAN RS5CR ALERT WAS TRIGGERED FOR HIGH ACTIVITY IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 17221 WAS RS 1 91 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 17221 SHOWS TOTAL CREDIT RS 27 75 LAKHS, NEFT RS 1 6 LAKHS MAJORLY FROM G BHAGYALAKSHMI NAND, RTGS RS 1 8 LAKHS MAJORLY FROM N R CHEMICALS, REMITTANCE RS 219 58 LAKHS MAJORLY FROM 1 LAURA GIDA INSAAT TURIZM 1, 1 ANIMAL NUTRITION DEVELOPMENT, AGROVET MMC 551751 IMISLI, INTERNET FUND TRANSFERS RS LAKHS MAJORLY FROM GLOBUS LOGISYS PVT LTD 55112 AND SHRI ANANDESHWAR TRADERS 69575512, MOBILE MONEY TRANSFERS RS 9 LAKHS IN 5 INSTANCES, UPI TRANSFERS RS 7 LAKHS IN 2 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 29 8 LAKHS, CLEARING RS 2 5 LAKHS MAJORLY TO FIEO FEDERATION OF INDIA, PANKAJ KUMAR, NEFT RS 1 LAKHS MAJORLY TO SWATI KESHARI BABU M, NIKHIL TRADE AND EXPORT, REMITTANCE RS 16 LAKHS, INTERNET FUND TRANSFER RS 5 25 LAKHS MAJORLY TO GLOBUS LOGISYS PVT LTD 55112 AND SHRI ANANDESHWAR TRADERS 69575512, MOBILE MONEY TRANSFERS RS 17 28 LAKHS IN 71 INSTANCES AND OTHERS RS 25 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS, TOTAL CREDIT SIDE RS 8 51 LAKHS, TRANSFERS RS 1 5 LAKHS, NEFT RS 2 LAKHS MAJORLY FROM SHREE D M ELECTRICALS, REMITTANCE RS 788 9 LAKHS MAJORLY FROM M S THE NATIONAL AGRICULTURAL, EQUALIVET INCORPORATED 18 BET, 1 ANIMAL NUTRITION DEVELOPMENT, MOBILE MONEY TRANSFERS RS 9 29 LAKHS IN 2 INSTANCES AND OTHERS RS 22 LAKHS TOTAL DEBIT SIDE RS 817 28 LAKHS, TRANSFERS RS 2 11 LAKHS, CLEARING RS 25 1 LAKHS MAJORLY TO PILLAI EXIMS CONSORTIUM, RADHA KRISHNA PILLAI, NEFT RS 75 LAKHS MAJORLY TO RENU VISHWAKARMA, UFLEX LIMITED, PMS STOCK VENTURE PVT LTD, RTGS RS 29 6 LAKHS, REMITTANCE RS 1 29 LAKHS MAJORLY TO SGS GREECE S A, INTERNET FUND TRANSFER RS 5 82 LAKHS MAJORLY TO GLOBUS LOGISYS PVT LTD 55112 AND SHRI ANANDESHWAR TRADERS 69575512, CARD TRANSACTIONS RS 1 75 LAKHS, MOBILE MONEY TRANSFERS RS 2 17 LAKHS IN 76 INSTANCES AND OTHERS RS 9 97 LAKHS FY 2192 FROM 122 TO 122 TOTAL CREDIT SIDE RS 16 69 LAKHS OF WHICH REMITTANCE RS 12 8 LAKHS TOTAL DEBIT SIDE RS 126 28 LAKHS OF WHICH CASH WAS RS 92 LAKHS, REMITTANCE RS 11 LAKHS",{"entities":[(929,952,"FOREIGN ORG"),(3967,3991,"FOREIGN ORG"),(4890,4914,"FOREIGN ORG"),(5339,5352,"FOREIGN ORG")]}),
            ("1 THE COMBINED ACCOUNT SCRUTINY REVEALED ROUTINE BUSINESS TRANSACTIONS WHEREIN CREDITS VIA FUND TRANSFERS AND RTGS AND DEBIT TOWARDS CHEQUE ISSUANCE, RTGS/NEFT TRANSFERS OBSERVED2 AS PER MEDIA NEWS, ENFORCEMENT DIRECTORATE ED IS INVESTIGATING CHARGES OF MONEY LAUNDERING IN THE EMBRAER SA DEFENSE DEAL THE FEDERAL AGENCY HAD INITIATED A PROBE AFTER THE CENTRAL BUREAU OF INVESTIGATION CBI HAD REGISTERED AN FIR ON THE BASIS OF A REFERENCE RECEIVED FROM CHIEF CONTROLLER, R AND D R AND M AND DS, DEFENSE RESEARCH AND DEVELOPMENT ORGANIZATION DRDO, DEPARTMENT OF DEFENSE RESEARCH AND DEVELOPMENT, MINISTRY OF DEFENSE THE ALLEGATIONS ARE THAT M/S EMBRAER WAS BEING INVESTIGATED BY AUTHORITIES OF US AND BRAZIL FOR ALLEGED EMPLOYMENT OF AN AGENT IN FACILITATING THE CONTRACT FOR AEW AND C PROJECT WITH CABS/DRDO FOR PROCUREMENT OF THREE FULLY MODIFIED EMB15 AIRCRAFT AND HAVE LAUNDERED MONEY ED HAS CLAIMED THAT ITS PROBE HAS REVEALED THAT, M/S EMBRAER SA BRAZIL OBTAINED THE CONTRACT FOR SUPPLY OF AIRCRAFTS TO INDIAN AIR FORCE FOR USD 21 MILLION AND PAID COMMISSION OF USD 576 MILLION TO A MIDDLEMAN NAMED VIPIN KHANNA FOR INFLUENCING THE SAID CONTRACT IN ITS FAVOR THE SAID KICKBACKS OF USD 576 MILLION WERE ALLEGEDLY ROUTED BY M/S EMBRAER SA, BRAZIL THROUGH ITS SUBSIDIARIES TO M/S INTERDEV AVIATION SERVICES PTE LTD, SINGAPORE IN LIEU OF SHAM AGREEMENT IT IS FURTHER ALLEGED THAT KICKBACKS TO THE TUNE OF AROUND USD 275 MILLION WERE FURTHER ROUTED FROM M/S INTERDEV AVIATION SERVICES PTE LTD TO M/S KRBL DMCC, DUBAI A 1% OWNED SUBSIDIARY OF KRBL LTD IN LIEU OF SHAM AGREEMENT TO FINALLY REACH INDIA THROUGH M/S KRBL LTD PROJECTING THE SAME AS UNTAINTED MONEY ED HAS FURTHER CLAIMED THAT ANOOP KUMAR GUPTA, DIRECTOR OF KRBL LIMITED SIGNED THE SHAM AGREEMENT HELD BETWEEN INTERDEV AVIATION SERVICES PTE LTD AND KRBL DMCC TO RECEIVE PROCEEDS OF CRIME POC WHICH WERE FINALLY RECEIVED IN THE BANK A/C OF KRBL LTD IN WHICH HE IS ONE OF THE DIRECTOR THE AGENCY ED HAS SO FAR ATTACHED IMMOVABLE PROPERTIES WORTH RS 1629 CRORE BELONGING TO KRBL LTD LINK ECONOMICTIMESINDIATIMESCOM/NEWS/DEFENCE/EDFILESCHARGESHEETINEMBRAERDEFENCEDEAL/ARTICLESHOW/796258CMS?FROM=MDR , WWWLIVEMINTCOM/NEWS/INDIA/EDFILESCHARGESHEETINEMBRAERDEFENCEDEALMONEYLAUNDERINGCASE11675898682HTML 5 CASE IS THEREFORE RECOMMENDED FOR FILING BASIS ADVERSE MEDIA NEWS AND INVESTIGATION BY ENFORCEMENT DIRECTORATE COUPLED WITH SIGNIFICANT HIGH VALUE TRANSACTIONS IN THE GROUP ACCOUNTS MUCH HIGHER THAN THE DECLARED TURNOVER AND PROFILE OF THE CUSTOMER ALSO CUSTOMER CORE BUSINESS ACTIVITY IS RELATED TO RICE MANUFACTURING HOWEVER HIS INVOLVEMENT IN DEFENSE DEALS SEEMS TO BE UNUSUAL AND DOES NOT SEEM TO HAVE ANY RATIONALE STR FILED BASIS A ADVERSE MEDIA NEWS AGAINST CUSTOMERB CUSTOMER UNDER INVESTIGATION OF ENFORCEMENT DIRECTORATE FOR MONEY LAUNDERING OFFENCEC CUSTOMER MAINTAINING MULTIPLE ACCOUNTS WHERE BUSINESS ACTIVITY IS PERTAINING TO RICE MANUFACTURING HOWEVER ADVERSE NEWS ABOUT CUSTOMER IS PERTAINING TO HIS INVOLVEMENT IN DEFENSE DEAL1 MEDIA NEWS PUBLISHED ON 9TH DECEMBER, 22 REVEALED THE ENFORCEMENT DIRECTORATE ED ON WEDNESDAY FILED A CHARGE SHEET AGAINST M/S EMBRAER SA, BRAZIL, M/S INTERDEV AVIATION SERVICES PTE LTD, ANOOP KUMAR GUPTA DIRECTOR OF KRBL LIMITED, KRBL LIMITED, ANURAG POTDAR NEPHEW OF ANOOP KUMAR GUPTA AND OTHERS IN A CASE OF EMBRAER SA DEFENSE DEAL SCREENING WAS CARRIED OUT WITH BANK S DATABASE WHERE POSITIVE MATCH CUSTOMER DETAILS MATCHING WITH DETAILS OF ENTITY APPEARING IN ADVERSE MEDIA NEWS WAS FOUND FOR M/S KRBL LIMITED WHO IS MAINTAINING A/C 2621 AT OUR SANGRUR PUNJAB BRANCH 2 M/S KRBL LIMITED IS HAVING PAN NO AAACK6H AND IS MAINTAINING CURRENT ACCOUNT NO 2621 WITH OUR SANGRUR PUNJAB BRANCH SINCE 17126 CUSTOMERS REGISTERED OFFICE ADDRESS IS VILLAG BHASAUR, DHURI 182 WITH MOBILE NUMBER UPDATED 9111291581 AND EMAIL ID UPDATED AS RAJEEVGARG@KRBLINDIACOM CUSTOMER DECLARED PROFILE AS MANUFACTURER OF RICE WITH DECLARED TURNOVER OF RS98 CRORES ACCOUNT STATUS IS ACTIVE WITH ACCOUNT BALANCE OF RS ,87,1,21/ AS ON 28221 RISK CATEGORY OF THE ACCOUNT IS LOW LATEST KYC UPDATED ON 1122 MR ANIL K MITTAL, ARUN K GUPTA , MAHESH C JAIN AND ANOOP K GUPTA ARE THE AUTHORISED SIGNATORIES IN THE ACCOUNT BUSINESS INCORPORATION DATE IS 199 SCRUTINY OF THE ACCOUNT STATEMENT OF KRBL LIMITED A/C NO 2621 FOR THE PERIOD APR219 TO DEC22 REVEALED THAT THE ACCOUNT RECEIVED CREDITS OF AGGREGATING TO RS 8657 LAKHS OF WHICH RS 11 LAKHS WERE RECEIVED THROUGH CHEQUE CLEARING, RS 92685 LAKHS WERE RECEIVED FROM FUND TRANSFER MAINLY FROM A/C NO 1952216 CHAMAN LAL SETIA EXPORTS LIMITED RS 569 LAKHS, A/C NO 5221771627 SANDEEP RICE AND GEN MILLS RS 8811 LAKHS, A/C NO 5216828 RICELA HEALTH FOODS LIMITED RS 19958 LAKHS ,RS 871 LAKHS RECEIVED FROM SWEEP TRANSFERS AND WORKING CAPITAL DISBURSED AND RS 27988 LAKHS WERE RECEIVED THROUGH RTGS/NEFT/IMPS/UPI TRANSFERS MAINLY FROM AMIR CHAND JAGDISH KUMAR BANK OF INDIA, UNION BANK OF INDIA AND BANK OF INDIA, RS 22557 LAKHS, SHREE BALAJI FOOD PRODUCTS SBI, RS 2529 LAKHS, BALAJI OVERSEAS PUNJAB NATIONAL BANK, RS 176 LAKHS AND OTHERS DEBITS AGGREGATING TO RS 765259 LAKHS OF WHICH RS 66915 LAKHS WERE TOWARDS CLEARING RS 626166 LAKHS WERE UTILIZED FOR FUND TRANSFER MAINLY TO SALARY PAYMENT RS 122 LAKHS, A/C NO 8787111 KRISHNA AUTO SALES RS 16 LAKHS, A/C NO 521762S R S ENGG CO RS 77 LAKHS AND OTHERS, RS 16792 LAKHS WERE UTILIZED TOWARDS SWEEP TRANSFERS AND RS 796 LAKHS WERE UTILIZED FOR RTGS/NEFT/IMPS/UPI TRANSFERS MAINLY TO OWN A/C SBI, RS 76 LAKHS OBSERVED SCRUTINY OF THE ACCOUNT 2621 KRBL LIMITED FOR THE PERIOD APRIL 22 AUG 221 REVEALED THAT CUSTOMER HAS RECEIVED TOTAL CREDITS AGGREGATING TO RS 29787 LAKHS MAINLY VIA RTGS/NEFT AGGREGATING TO RS 21921 LAKHS MAINLY FROM NATURE BIOFOODS LIMITED AGGREGATING TO RS 7812 LAKHS AND FROM AROMA AGROTECH P LTD AGGREGATING TO RS 19116 LAKHS THE PROCEEDS ARE FURTHER WITHDRAWN MAINLY VIA FUNDS TRANSFER AGGREGATING TO RS 217611 MAINLY TO SELF ACCOUNT 1218 AGGREGATING TO RS 155115 LAKHS 5 CUSTOMER PAN AND MOBILE LINKED ACCOUNTS IDENTIFIED AS BELOW 86111, 2152116, 2295, 221959, 16772, 815, 522922, 52272979, 52272972, 5229296, 28911, 267119, 22911, 2682222, 1822265, 5211961, 5216761, 5622187, 522758, 152, 621927, 57121, 927115, 115, 16, 26811, 11, 1218, 7711, 2222569, 571112, 1158, 12226, 22955, 229516, 229577 AND 22952, 525896881",{"entities":[(274,296,"FOREIGN ORG"),(941,951,"FOREIGN ORG"),(1231,1240,"FOREIGN ORG"),(3132,3141,"FOREIGN ORG"),(3316,3326,"FOREIGN ORG"),(1282,1315,"FOREIGN ORG"),(1458,1492,"FOREIGN ORG"),(1771,1805,"FOREIGN ORG"),(1810,1819,"FOREIGN ORG"),(3156,3189,"FOREIGN ORG")]}),
            ("IN OTHER EXAMPLE ON RS LACS AMOUNT OF FUND CREDITED IN PARTS THROUGH RTGS FROM MAGNETIC FILMS PRODUCTION DIVERSE MEDIA WHERE FUND DEBITED THROUGH OUTWARD REMITTANCE TOWARDS WEB VIDEOS LIMITED UNITED KINGDOM HTTP://WWWWEBVIDEOSCOUK/ SINGAPORE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE WHITE RABBIT MOTION PICTURE PRODUCTION LLC UAE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE HTTP://WWWWHITERABBITDUBAICOM/ THIS ACCOUNT GET CLOSED IN SHORT SPAN OF PERIOD OFMONTHS WITH RS CRORES CLIENT WAS UNABLE TO PROVIDE THE INVESTMENT AGREEMENT WITH THE INVESTORS IN SPITE SEVERAL ASSURANCES WHICH SEEMS SUSPICIOUS TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF BELOW RS LACS IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS IN NEWLY OPENED ACCOUNT AND HIGH TURNOVER STR IS BEING FILEDTINGLE CINE PRODUCTIONS PARTNERSHIP HAVING ADDRESS:ND FLOOR / FLAT NORELIANCE CLASSIC ABOVE ANDHRA HYDERABAD TELANGANA INDIA CONTACT NUMBER: + AND EMAIL ID: INFO@TINGLECINEPRODUCTIONSCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LIMITED HYDERABAD BEGUMPET //A JAYALAKSHMI TOWERS MOTILALNEHRU NAGAR PRAKASHNAGAR BEGUMPET HYDERABAD TELANGANA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAQFTK GSTIN AS PER BANK RECORDS IS AAQFTKZL TRANSACTION LINKED ACCOUNTS ARE BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH CHAWLA WITH PAN NO AGRPCE MOHIT BANSAL WITH PAN NO ALRPBC THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON KYC DOCUMENTS OBTAINED WERE GSTIN AADHAR PANAAQFTK AS PER ENHANCE DUE DILIGENCE CUSTOMER IS IN BUSINESS OF THE INVESTORS AND THE BENEFICIARY TO BE IN THE LINE MEDIA AND FILMS THEY ARE THE SERVICE PROVIDER FOR MOVIE OR FILM PRODUCTION DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS BELOW RS LACS A MANUAL ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT ATWAS NIL FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY FROMTO : TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL CLEARING RS LAKH MAJORLY FROM CHQ NEFT RS LAKHS MAJORLY FROM DIVERSE MEDIA LIQUID STUDIOS AND SERVICES RTGS RS LAKHS MAJORLY FROM DIVERSE MEDIA LIQUID STUDIOS AND SERVICE MAGNETIC FILMS PRODUCTION MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKH BY BRANCH CLEARING RS LAKHS MAJORLY TO T VINAY ASSOCIATES RTGS RS LAKHS MAJORLY TO LIQUID STUDIO SERVICE LIQUID STUDIO SERVICE REMITTANCE RS LAKHS MAJORLY TO SKY DENVAX PTE LTD WEB VIDEOS LIMITED WHITE RABBIT MOTION PICTURE INTERNET FUND TRANSFER RS LAKHS MAJORLY TO MANISH CHAWLA MOHIT BANSAL MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TRANSACTION PATTERN SHOWS HIGH VALUE NON CASH CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE ON RS CRORES AMOUNT OF FUND CREDITED IN PARTS THROUGH RTGS FROM LIQUID STUDIOS AND SERVICE MAGNETIC FILMS PRODUCTION DIVERSE MEDIA AND ON THE SAME DAY SAME AMOUNT OF FUND DEBITED THROUGH OUTWARD REMITTANCE TOWARDS WEB VIDEOS LIMITED UNITED KINGDOM HTTP://WWWWEBVIDEOSCOUK/ SINGAPORE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE WHITE RABBIT MOTION PICTURE PRODUCTION LLC UAE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE",{"entities":[(173,206,"FOREIGN ORG"),(298,340,"FOREIGN ORG"),(2656,2674,"FOREIGN ORG"),(2675,2693,"FOREIGN ORG"),(2694,2721,"FOREIGN ORG"),(3164,3182,"FOREIGN ORG"),(3289,3331,"FOREIGN ORG")]}),
            ("QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2021 22 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS. 0.09 LAKHS OF WHICH REMITTANCE RS. 0.09 LAKHS MAJORLY FROM 1 FIXIER SA DE CV 2 CARR DR M,1 INNOVATION ORTHO LINE LTD. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 0.00. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 86.31 LAKHS OF WHICH REMITTANCE RS. 86.31 LAKHS MAJORLY FROM 1 ZOLDAN CORPOREISHON CIA L,1 MEDICAL ORTOVIT SRL 2 BUCUR,1 ATRON HEALTH S A 2 8 LEOFO. TOTAL DEBIT SIDE RS. 86.42 LAKHS OF WHICH REMITTANCE RS. 86.29 LAKHS MAJORLY TO CHANGZHOU NUBE INTERNATIONAL,CHANGZHOU XIETONG INDUSTRIES CO,CHANGZHOU XIETONG INDUSTRIES CO LTD",{"entities":[(686,713,"FOREIGN ORG"), (715,745,"FOREIGN ORG"),(747,781,"FOREIGN ORG"),(520,543,"FOREIGN ORG"),(548,568,"FOREIGN ORG"),(578,594,"FOREIGN ORG")]}),
            ("THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 11,501.52 LAKHS OF WHICH TRANSFERS RS. 3,055.50 LAKHS MAJORLY FROM RAHUL AGRO INDUSTRIES, KAPISH INTERNATIONAL, RTGS RS. 1,781.19 LAKHS MAJORLY FROM HINAL ENTERPRISE, GUJARAT PEANUT PRODUCT, G S TRADING CO, REMITTANCE RS. 3,187.04 LAKHS MAJORLY FROM AGRI COMMODITIES AND FINANCE FZ LLC, KOREA AGRO FISHERIES AND FOOD TRADE, INTERNET FUND TRANSFERS RS. 2,257.89 LAKHS FROM SAGAR INTERNATIONAL ETC, FIXED DEPOSIT RS. 1,219.77 LAKHS. TOTAL DEBIT SIDE RS. 11,497.48 LAKHS OF WHICH TRANSFERS RS. 5,032.47 LAKHS MAJORLY TO SHREESHIV ENTERPRISE, RAHUL AGRO INDUSTRIES, SAMURAI TRADEX, NEFT RS. 468.40 LAKHS MAJORLY TO SHREE BHANDERI INDUSTRIES, GANESH ENTERPRISES, RTGS RS. 3,082.58 LAKHS MAJORLY TO GUJARAT PEANUT PRODUCT, REMITTANCE RS. 95.29 LAKHS MAJORLY TO KOKAI INDO FOOD STUFF TRADING LLC, INTERTEK VIETNAM LIMITED",{"entities":[(414,448,"FOREIGN ORG"), ( 451,485,"FOREIGN ORG"),(919,951,"FOREIGN ORG"),( 954,977,"FOREIGN ORG")]}),
            ("MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGS/NEFT TOTALLING TO RS.2.69 CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 2413200206 ON 8 INSTANCES, FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS.65.47 LAKHS, CHEQUES ISSUANCE TOTALLING TO RS.8.29 LAKHS, IMPORT PAYMENT TOTALLING TO RS.6.41 LAKHS TO TANU STAR HK LTD, HONG KONG ETC",{"entities":[(351,366,"FOREIGN ORG")]}),
            ("MALISTA FREIGHT FORWARDING OPC PRIVATE LIMITED IS A PRIVATE LIMITED COMPANY, HAVING ADDRESS 61, RUPA SOLITAIRE, MILLENIUM ,BUSINESS PARK, THANE, MAHARASHTRA, INDIA, 71, CONTACT NUMBER 9191571 AND EMAIL ID ACCOUNTS MALISTAFREIGHTCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 95115 WITH ICICI BANK LTD, 16, HT PAREKH MARG,BACKBAY RECLAMATION, CHURCHGATE, MUMBAI, MAHARASHTRA, 2, SINCE 291221 DATE OF INCORPORATION IS 21122 PAN NUMBER OF THE ACCOUNT IS AANCM962P IEC CODE OF THE ACCOUNT IS AANCM962P TRANSACTION LINKED ACCOUNTS ARE 11152559 AND 69956591 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS RAHUL KUMAR PAN CMZPK265G THE CUSTOMER IS CATEGORIZED AS MEDIUM RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE IDENTIFICATION PROOF AS PAN AANCM962P, CERTIFICATE OF INCORPORATION WITH CIN NUMBER U5259DL22OPC71956, A COPY OF ARTICLES AND MEMORANDUM OF ASSOCIATION KYC UPDATION IS NOT DUE CUSTOMER IS ENGAGED INTO LOGISTICS BUSINESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAKHS ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS AND HIGH TURNOVER SEEN IN NEW ACCOUNT BALANCE IN THE ACCOUNT AT 151221 WAS RS16 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 151221 SHOWS TOTAL CREDIT RS 9,6296 LAKHS OF WHICH NEFT RS 1887 LAKHS MAJORLY FROM MALISTA FREIGHT FORWARDING, NEXASHIP LOGISTICS PRIVATE LIMITED, NAMO SAI INTERNATIONAL, RTGS RS 9,85 LAKHS MAJORLY FROM MALISTA FREIGHT FORWARDING, NAMO SAI INTERNATIONAL, NEXASHIP LOGISTICS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 9,6929 LAKHS OF WHICH RTGS RS 25 LAKHS TO MALISTA FREIGHT FORWARDING OPC PRIVATE LIMITED, REMITTANCE RS 9, LAKHS MAJORLY TO STATICE INTERNATIONAL PTE LTD,ASIAN ETHICS LIMITED, EXPRESS LUCK PTE LIMITEDFY 2221 FROM 291221 TO 1221 TOTAL CREDIT SIDE RS 15957 LAKHS OF WHICH CLEARING RS 2 LAKHS MAJORLY FROM MUM CLG, RTGS RS 15757 LAKHS MAJORLY FROM NAMO SAI INTERNATIONAL TOTAL DEBIT SIDE RS 1828 LAKHS OF WHICH REMITTANCE RS 1727 LAKHS, MOBILE MONEY TRANSFERS RS 11 LAKHS IN 2 INSTANCES , HIGH VALUE TRANSACTIONS IN NEW ACCOUNT AND HIGH TURNOVER SEEN IN THE ACCOUNT FOR EXAMPLE, ON 28221, RS 11112 LAKHS WERE CREDITED INTO THE ACCOUNT IN TWO TRANCHES THROUGH RTGS FROM MALISTA FREIGHT FORWARDING OPC PRIVATE LIMITED ACCOUNT NUMBER 212859, IFSC ESFB25, BANK EQUITAS SMALL FINANCE BANK LIMITED AND ON THE SAME DAY, RS 1116 LAKHS WERE DEPLOYED THROUGH OUTWARD REMITTANCE TO EXPRESS LUCK PTE LIMITED ACCOUNT NUMBER 77171, CURRENCY USD, AMOUNT 18176, COUNTRY HONG KONG, PURPOSE TWDS FREIGHT CHARGESSIMILARLY, ON 121221, RS 197 LAKHS WERE CREDITED INTO THE ACCOUNT THROUGH RTGS FROM MALISTA FREIGHT FORWARDING OPC PRIVATE LIMITED ACCOUNT NUMBER 212859, IFSC ESFB25, BANK EQUITAS SMALL FINANCE BANK LIMITED AND ON THE SAME DAY, RS 166 LAKHS WERE DEPLOYED THROUGH OUTWARD REMITTANCE MAJORLY OF RS 192 LAKHS TO EXPRESS LUCK PTE LIMITED ACCOUNT NUMBER 77171, CURRENCY USD, AMOUNT 18795, COUNTRY HONG KONG, PURPOSE TWDS AIR FREIGHT AND HIGH TURNOVER COMPARED TO DECLARED TURNOVER SEEN IN THE ACCOUNT SIMILAR TRANSACTION PATTERN IS OBSERVED IN MULTIPLE INSTANCES IN THE ACCOUNT HIGH TURNOVER OBSERVED IN THE ACCOUNT WHICH DOESNT MATCH WITH THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER HIGH TURNOVER FOR AN NEWLY INCORPORATED ENTITIY IN VIEW OF HIGH VALUE TRANSACTIONS AND HIGH TURNOVER IN A NEW ACCOUNT RS 96296 LAKHS WHICH DOESNT MATCH WITH THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER AT THE TIME OF ACCOUNT OPENING, STR HAS BEEN FILED",{"entities":[(1743,1771,"FOREIGN ORG"),(1773,1792,"FOREIGN ORG"),(1795,1818,"FOREIGN ORG"),(2918 , 2942,"FOREIGN ORG")]}),
            ("REMITTANCE RS 6,297 LAKHS, INTERNET FUND TRANSFERS RS 2,8197 LAKHS FROM WILWORTH TECHSOL PRIVATE LIMITED 695658, MOBILE MONEY TRANSFERS RS 969 LAKHS IN 1 INSTANCES, UPI TRANSFERS RS 21 LAKHS IN 9 INSTANCES AND OTHERS RS 2,1989 LAKHS TOTAL DEBIT SIDE RS 276 LAKHS OF WHICH CASH WAS RS 678 LAKHS BY SHIVAJI NAGAR BRANCH, TRANSFERS RS 12,577 LAKHS MAJORLY TO SANY HEAVY INDUSTRY INDIA PVT LTD, A S ENTERPRISES, CLEARING RS 1,916 LAKHS, NEFT RS 89 LAKHS MAJORLY TO MAHA GOVT STATUATORY, E PROCUREMENT GOVERN, GEM PORTAL, RTGS RS 1,17,95672 LAKHS MAJORLY TO SANY HEAVY INDUSTRY INDIA PVT LTD, REMITTANCE RS 5,6271 LAKHS, INTERNET FUND TRANSFER RS 56,7827 LAKHS TO AADITYA BIRLA SUN LIFE OVERNIGHT FUND, MOBILE MONEY TRANSFERS RS 12 LAKHS IN 12 INSTANCES AND OTHERS RS 6,11281 LAKHS FY 2192 FROM 1219 TO 122 TOTAL CREDIT SIDE RS 2,82,6728 LAKHS OF WHICH CASH WAS RS 291 LAKHS, REMITTANCE RS 9,169 LAKHS TOTAL DEBIT SIDE RS 2,78,5 LAKHS OF WHICH CASH WAS RS 91 LAKHS, REMITTANCE RS 11,869 LAKHSFY 21819 FROM 1218 TO 1219 TOTAL CREDIT SIDE RS 2,21,5579 LAKHS OF WHICH CASH WAS RS 25 LAKHS, REMITTANCE RS 22,1 LAKHS TOTAL DEBIT SIDE RS 2,22,9218 LAKHS OF WHICH CASH WAS RS 685 LAKHS, REMITTANCE RS 16,597 LAKHSFY 21718 FROM 1217 TO 1218 TOTAL CREDIT SIDE RS 1,52,7818 LAKHS OF WHICH CASH WAS RS 165 LAKHS, REMITTANCE RS 9,96676 LAKHS TOTAL DEBIT SIDE RS 1,57,67 LAKHS OF WHICH CASH WAS RS 1 LAKHS, REMITTANCE RS 12,627 LAKHSTHE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER 95195 IN THE NAME OF SANY HEAVY INDUSTRY INDIA PVT LTD ON 28217 VIDE FIU BATCH ID 1728727, REPORT SERIAL NUMBER 1651THERE ARE 66 SHIPMENTS MADE EXPORT BILLS PENDING SUBMISSION WHICH ARE OVERDUE UNDER CAUTION LIST FOR LESS THAN 276 DAYS THERE ARE 125 ADVANCE RECEIVED EXPORT BILLS PENDING SUBMISSION WHICH ARE OVERDUE FOR LESS THAN 62 DAYS THERE ARE 886 IMPORT MADEPAYMENT PENDING WHICH ARE PENDING PAYMENT FOR LESS THAN 25 DAYS THERE ARE 17 BILL OF ENTRY PENDING FOR SUBMISSION THERE ARE 7 IMPORT BILLS UNDER LCS BUYERS CREDIT OUTSTANDING HUGE TURNOVER SEEN IN THE ACCOUNT FOR EXAMPLE, ON 121221, RS 81 LAKHS GOT CREDITED IN PARTS THROUGH RTGS FROM SANY HEAVY INDUSTRY INDIA PVT LTD ACCOUNT NO 52116927, IFSC CODE HDFC19, BANK HDFC BANK, MADHURA ENGINEERING SERVICES PVT LTD ACCOUNT NO 52669761, IFSC CODE HDFC2, BANK HDFC BANK, NILACHAL GREENTECH ENGINEERS PRIVATE LIMITED ACCOUNT NO 2176959, IFSC CODE INDB6, BANK INDUSIND BANK LTD, NEFT FROM RETAIL ASSET BULK LO ACCOUNT NO 289955, IFSC CODE YESB1, BANK YES BANK, MN LIFTERS ACCOUNT NO 115521126, IFSC CODE CNRB1155, BANK CANARA BANK, INTEC INFRA PROJECTS ACCOUNT NO 916219512815, IFSC CODE UTIB7, BANK AXIS BANK AND ON THE SAME DAY, RS 7 LAKH GOT DEBITED IN PARTS THROUGH NEFT TO ACCOUNT NO 2295128725, IFSC CODE SCBL61, BANK STANDARD CHARTERED BANK SIMILAR TRANSACTION PATTERN IS SEEN REPEATEDLY IN THE ACCOUNT BEHIND WHICH APPARENT RATIONALE OF THE CUSTOMER IS NOT KNOWN TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 22122 WAS RS 1,52,692 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF PENDING BILLS AND HUGE TURNOVER IN THE ACCOUNT, STR IS BEING FILEDSANY HEAVY INDUSTRY INDIA PVT LTD, A PRIVATE LIMITED COMPANY, HAVING ADDRESS PLOT NO CHAKAN MIDC PHASEIII TALUKA KHED, PUNE, MAHARASHTRA, INDIA, 151, CONTACT NUMBER 91997512 AND EMAIL ID ANANDPATHAK SANYIN IS MAINTAINING A CURRENT ACCOUNT NUMBER 95195 WITH ICICI BANK LTD, 119 8, RAMCHANDRA SABHAMANDAP, GHOLE ROAD, SHIVAJI NAGAR, PUNE, MAHARASHTRA, 115, SINCE 17827 DATE OF INCORPORATION IS 21222 PAN NUMBER OF THE ACCOUNT IS AAGCS775L IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 526885 GSTIN AS PER BANK RECORDS IS 27AAGCS775L1ZO AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE ANAND RAMKRISHNA PATHAK AUTHORISED SIGNATORY, PAN AUNPP571H, MANISH KUMAR PATADIA AUTHORISED SIGNATORY, PAN ALXPP2951M, ANAND DEEPARAM CHOUDHARY AUTHORISED SIGNATORY, PAN APNPC6816J ,ANAND RAMKRISHNA PATHAK AUTHORISED SIGNATORY, PAN AUNPP571H CONSIDERING THAT THERE ARE ADDITIONAL AUTHORIZED SIGNATORIES, THEIR DETAILS ARE PROVIDED IN RESPECTIVE FIELDS WITHIN THE XML FILE THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAGCS775L, COI UA521DL22PTC117856, MOA AOA, GST REGISTRATION CERTIFICATE DATE OF LAST KYC UPDATION 2711219 KYC UPDATION IS NOT DUE AS PER BANK RECORDS, CUSTOMER IS INTO MANUFACTURING OF HEAVY MACHINESAS PER PROBE, SANY HEAVY INDUSTRY INDIA PRIVATE LIMITED SHIIPL, AS PER ITS CREDIT RATING REPORT, MANUFACTURES OR ASSEMBLES EXCAVATORS AND CONCRETE EQUIPMENT LIKE CONCRETE MIXERS AND BATCHING PLANTS, TRUCK CRANES AND MOTOR GRADERS IN INDIA DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS 1 CRORES ALERT WAS TRIGGERED FOR PENDING BILLS AND HUGE TURNOVER IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 11221 WAS RS298 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 22122 FROM 1221 TILL 11221 SHOWS TOTAL CREDIT RS 1,52,692 LAKHS OF WHICH TRANSFERS RS 1,872 LAKHS MAJORLY FROM ROYAL INFRA ENGINEERING PVT LTD, SHRI DINESH ROADLINES, SHREE BALAJEE INFRA EQUIPMENT PVT LTD, CLEARING RS 1,59 LAKHS MAJORLY FROM BIRLA SUNLIFE MUTUAL FUND, SUNDARAM FINANCE, NEFT RS 2,672 LAKHS MAJORLY FROM G R INFRAPROJECTS LTD, SANY HEAVY INDUSTRY INDIA PRIVATE LIMITED, KOTAK MAHINDRA BANK LTD, RTGS RS 8,51552 LAKHS MAJORLY BIRLA SUN LIFE, SANY HEAVY INDUSTRY INDIA PVT LTD, REMITTANCE RS 1,56728 LAKHS MAJORLY FROM MEGHA ENGINEERING AND INFRASTRUCTU, MEGHA ENGINEERING AND INFRASTRUCTURE, BHAJURATNA AGENCY PVT LTD, INTERNET FUND TRANSFERS RS 2,568 LAKHS FROM SHREE AMAR CRANE HIRING CO 2152, MOBILE MONEY TRANSFERS RS 125 LAKHS IN 29 INSTANCES, UPI TRANSFERS RS 67 LAKHS IN 58 INSTANCES AND OTHERS RS 15,9615 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,5,791 LAKHS OF WHICH TRANSFERS RS 1,57166 LAKHS MAJORLY TO SANY HEAVY INDUSTRY INDIA PVT LTD, CLEARING RS 1,1 LAKHS REJECTION OF CLEARING, RTGS RS 91,2561 LAKHS TO SANY HEAVY INDUSTRY INDIA PVT LTD, REMITTANCE RS 1,877 LAKHS MAJORLY TO KAWASAKI HEAVY INDUSTRIES, MOTTROL CO LTD, KAWASAKI HEAVY INDUSTRIES, INTERNET FUND TRANSFER RS ,7271 LAKHS, OTHERS RS ,65759 LAKHSFY 2221 FROM 122 TO 1221 TOTAL CREDIT SIDE RS 1,96,5865 LAKHS OF WHICH CASH WAS RS 792 LAKHS MAJORLY AT DURGAPUR, BARHI, BARUIPUR, ALWAR BRANCHES, TRANSFERS RS 22,89562 LAKHS MAJORLY FROM GLOBAL TRADING CORPORATION ESCROW ACCOUNT, SANY HEAVY INDUSTRY INDIA PVT LTD, BVS EQUIPMENTS ESCROW ACCOUNT, CLEARING RS 28,7515 LAKHS MAJORLY FROM BIRLA SUNLIFE MUTUAL FUND, SUNDARAM FINANCE,HINDUJA LEYLAND FINANCE LIMITED, NEFT RS 5,155 LAKHS MAJORLY FROM G R INFRAPROJECTS LTD, SANY HEAVY INDUSTRY, RTGS RS 8,825 LAKHS MAJORLY FROM SANY HEAVY INDUSTRY INDIA PRIVATE LIMITED",{"entities":[(6013,6037,"FOREIGN ORG"),(6040,6053,"FOREIGN ORG"),(6056,6080,"FOREIGN ORG")]}),
            ("SUBSEQUENT TO SUCH CREDITS FOREIGN OUTWARD REMITTANCES NONIMPORT AMOUNTED TO RS CR FAVOURING SPEED STREAM FREIGHT BROK UAE RS CR STATICE INTERNATIONAL PTE LTD SINGAPORE RS CR AND EXCEL SEA SHIPPING LLC UAE RS CR TOWARDS FREIGHT ON IMPORTS BANK HAS ALREADY STR ON OTHER ACCOUNTS NAMELY BARA AFRICA ON SHORE SERVICES PRIVATE LIMITED BATCH ID BOXTRON SHIPPING AND LOGISTICS PVT LTD BATCH ID AND MILE LOGISTICS P LIMITED BATCH ID WHEREIN SIMILAR PATTERN OF TRANSACTIONS WERE NOTED AND FUNDS WERE REMITTED TO THE ABOVE MENTIONED COMMON OVERSEAS PARTY EXCEL SEA SHIPPING LLC UAE IMPS NEFT RTGS PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING NEXASHIP LOGISTICS PRIVATE LIMITED RS CR IDFC LIBERTON LOGISTICS PRIVATE LIMITED RS CR IDFC AND T S GLOBAL IMPEX RS CR IDFC LIBERTON LOGISTICS PRIVATE LIMITED AND NEXASHIP LOGISTICS PRIVATE LIMITED HAVE UPDATED A COMMON MAILING ADDRESS DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS SIGN BOARDS IN THE NAMES OF THE COMPANIES WERE SEEN AT THE PREMISE AND OFFICE SET UP WITH CHAIRS AND TABLES COULD BE SEEN HOWEVER NO MAJOR BUSINESS ACTIVITY OR EMPLOYEES COULD BE SEEN TO SUBSTANTIATE THE HIGH VALUE TRANSACTIONS ROUTED THROUGH ACCOUNT IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED TO ROUTE TRANSACTIONS WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE ROUND SUM RTGS NEFT CREDITS WERE FOLLOWED BY IMMEDIATE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS NO OTHER BUSINESS RELATED TRANSACTIONS COULD BE NOTED THROUGH THE ACCOUNT WITHIN MONTHS OF ACCOUNT OPENING HIGH VALUE AND VOLUME OF TRANSACTIONS WERE ROUTED THROUGH ALL ACCOUNTS BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING ALSO THE PROPRIETORS AND DIRECTORS ARE IN THE AGE GROUP OF YEARS AND MAY NOT BE THE ACTUAL BENEFICIAL OWNERS OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNTS ACCOUNT OF LIBERTON LOGISTICS P LIMITED IS ACTIVE WITH BALANCE OF RS AS ON APRIL QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR IS DEBIT RS IN CASH RS CR AND CREDITS RS CR IN CASH RS CR IS DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CRA CURRENT ACCOUNT BEARING NUMBER WAS OPENED IN THE NAME OF LIBERTON LOGISTICS P LTD ON FEB AT OUR BRANCH LOCATED IN SHARDA TERRACES PLOT NO SEC CBD BELAPUR NAVI MUMBAI THE ENTITY WAS FORMED ON JAN AND HAS SUBMITTED COPIES OF PAN AAGCAR ALONG WITH BOARD RESOLUTION MOA AOA AND CERT OF INCORP UMHPTC AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO B SHELTON SAPPHIRE PL SECTOR CBD BELAPUR NAVI MUMBAI DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS AND ULTIMATE BENEFICIAL OWNERS DIRECTORS HAVE REGISTERED THEIR MOBILE NUMBERS AND WITH THE BANK THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS CR AND THAT THE INDIVIDUALS HAVE ANNUAL INCOME IN THE RANGE OF RS CR TO RS CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT OTHER ACCOUNTS WERE IDENTIFIED IN THE NAMES OF NEXASHIP LOGISTICS PRIVATE LIMITED AND A SOLE PROPRIETORSHIP CONCERN T S GLOBAL IMPEX OPENED ON FEB AND FEB DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS IN NEXASHIP LOGISTICS PRIVATE LIMITED WHILE TSHERINGLA SHERPA IS THE PROPRIETOR IN T S GLOBAL IMPEX IT WAS INDICATED THAT THE ENTITIES ARE INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS CR AND CR RESPECTIVELY WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NONIMPORT WERE ROUTED THROUGH THE NEWLY OPENED ACCOUNT OF LIBERTON LOGISTICS PRIVATE LIMITED SIMILAR PATTERN OF TRANSACTIONS AND COMMON OVERSEAS BENEFICIARIES WERE NOTED IN THE ACCOUNTS OF NEXASHIP LOGISTICS PRIVATE LIMITED AND T S GLOBAL IMPEX WHICH AS PER CUSTOMER ARE RELATED ACCOUNTS HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION ON REVIEWING CONSOLIDATED ACCOUNT TRANSACTIONS SINCE INCEPTION TILL APRIL IT WAS NOTED THAT ROUND SUM HIGH VALUE IMPS NEFT RTGS CREDITS AMOUNTED TO RS CR MAINLY FROM NAMO SAI INTERNATIONAL RS CR EQUITAS SMALL FINANCE BANK LIMITED A C NO AND NEXASHIP LOGISTICS PRIVATE LIMITED RS CR IDBI IDFC SBI YES BANK",{"entities":[(93,118,"FOREIGN ORG"),(129,158,"FOREIGN ORG"),(179,201,"FOREIGN ORG"),(546,568,"FOREIGN ORG")]}),
            ("TO THE TUNE OF INR CR BY HDFC BANK LTD SANDOZ HOUSE BRANCH HDFC INR CR CREDITS ALSO SEEN BY WAY OF NET TRANSACTIONS CREDIT FROM RAKSHA BULLIONTHESE INWARD REMITTANCES ARE IMMEDIATELY FOLLOWED BY CROSS BORDER OPEN ACCOUNT PAYMENT FOR IMPORT OF GOLD DORE BARS SINCE JANUARY CUSTOMER HAS UNDERTAKEN TRANSACTIONS OF GOLD DORE IMPORTS FROM PERU AMOUNTING TO INR CR TO THE PARTIES SMRL AMERICAN ZAFIRO EIRL INR CR CONSORCIO PACAYPAMPA SAC INR CR CONSORCIO VILLA DEL MAR S A C INR CR MINERA HOLDER E I R L INR CR EMPRESA MINERA JCHH E I R L INR INR CR WHOSE PUBLIC DOMAIN INFORMATION ARE NOT AVAILABLE EXPLICITLY AND ARE ESTABLISHED RECENTLY FOR EXAMPLE CONSORCIO PACAYPAMPA SAC HTTPS COMPUEMPRESACOM INFO CONSORCIOPACAYPAMPASOCIEDADANONIMACERRADA WAS ESTABLISHED ON TH JULY SMRL AMERICAN ZAFIRO EIRL HTTPS WWWUNIVERSIDADPERUCOM EMPRESAS SRMLAMERICANZAFIROPHP WAS ESTABLISHED ON TH DECEMBER UPON REVIEWING THE TRANSACTION REFERENCE BM IT WAS FOUND THAT THE CUSTOMER WAS IMPORTING GOLD DORE BARS FROM SMRL AMERICAN ZAFIRO EIRL ADDRESSED AT CALLE EL GRECIO SAN BORIALILNA PERU THE GOLD DORES WERE MINED ASSAYED AND SHIPPED BY COMPANIA MINERA PAUCARTAMBO SAC HTTPS WWWUNIVERSIDADPERUCOM EMPRESAS COMPANIAMINERAPAUCARTAMBOPH WHICH WAS RECENTLY ESTABLISHED ON TH JAN SINCE THE OVERSEAS PARTY DETAILS ARE NOT SUFFICIENT CREDIT INFORMATION REPORT CIR OF OVERSEAS PARTY WAS SOUGHT AND IT WAS ALSO FOUND UNSATISFACTORY FOLLOWING ARE THE NOTEWORTHY POINTS IN CIR DETAILS LIKE CONTACT NUMBER TRADE REFERENCES ETC WERE NOT PROVIDED AND HENCE THE DETAILS CANNOT BE VERIFIED THE COMPANY IS NOT REGISTERED IN THE NATIONAL REGISTRY OF SUPPLIERS RNP BANKING REFERENCES WERE NOT OBTAINED CUSTOMERS LINE OF BUSINESS IS PROVIDED AS CUSTOM HOUSE BROKERSOVERALL RATING OF CLIENT AS PER DB REPORT IS NB NEW BUSINESS USUALLY LESS THAN YEARS AND MONTHS THE ADDRESS OF OVERSEAS PARTY AS PROVIDED ON INVOICE AT CALLE EL GRECIO SAN BORIALILNA PERU WHICH IS A VILLA AND ALSO HOUSES VARIOUS COMPANIES NOTEWORTHY THAT THERE ARE TRANSACTIONS OUTSTANDING FOR BOE SUBMISSION AGEING BEYOND DAYS AND AMOUNTING TO USD MILLION SINCE THE COMMODITY OF IMPORT IS GOLD DORE THE LIKELIHOOD OF NONSUBMISSION OF BOE CREATES SUSPICION IN LIGHT OF THE ABOVE REDFLAGS IT IS SUSPECTED THAT CUSTOMER MAY BE INVOLVED IN MONEY LAUNDERING INSTANCES ARISING OUT OF THE GENUINESS OF THE COMMODITY OF IMPORT AND THE AREA OF ITS ORIGIN BY UNDERTAKING HIGH VALUE REMITTANCES TO HIGH RISK COUNTRY IN DISGUISE OF DIRECT IMPORT REMITTANCES IN LIGHT OF THE ABOVE FACTS IT HAS BEEN DECIDED ON APRIL TO FILE A STR ON THE SAID CUSTOMERTHE CUSTOMER GGC GUJRAT GOLD CENTRE PRIVATE LIMITED PAN AAFCGM DOB DOI COMMUNICATION AND PERMANENT ADDRESS GUJARAT HOSIERY MILL COMPRAKHIAL ROAD RAKHIALB H VIJAY PETROL PUMP AHMEDABAD GUJARAT GSTIN AAFCGMZV REGISTRATION NUMBER CIN UGJPTC IEC CODE MOBILE NUMBER EMAIL ID GUJARATGOLD GMAILCOM IS MAINTAINING A CURRENT AC NO SINCE WITH AHMEDABAD BRANCH ADDRESS UNIT NOG CGCENTRE CGROAD AHMEDABAD THE CUSTOMER FALLS UNDER HIGH RISK CATEGORY WITH KYC LAST UPDATED IN SEPTEMBER ALKESH DASHARATHLAL PATEL RAMESHBHAI PATEL ARE AUTHORIZED SIGNATORIES AND SURESH C KANSARA RAMESH KANSARA ANIL KANSARA SHARAD KANSARA ARE THE BENEFICIAL OWNER IN THE ACCOUNT CUSTOMER IS HAVING A GEMS AND JEWELLERY MANUFACTURING UNIT WITH AN ANNUAL GROSS TURNOVER GREATER THAN OR EQUAL TO INR CR CUSTOMER HAS AVAILED TERM DEPOSITS AMOUNTING TO RS ADDITIONALLY ACCOUNTS ARE FOUND TO BE LINKED TO THE ACCOUNT OF GGC GUJRAT GOLD CENTRE PRIVATE LIMITED AC NO BY COMMON PAN AAFCGM UNDER SAME NAME ACCOUNT NO IS ACTIVE BUT NONTRANSACTING SINCE NOV AND ACCOUNT NO WAS OPENED ON CLOSED ON IN THE ACCOUNT OF GGC GUJRAT GOLD CENTRE PRIVATE LIMITED AC NO AND LINKED ACCOUNTS TRANSACTIONS ANALYSIS FROM TILL SHOWS CREDIT TRANSACTIONS AMOUNTING TO RS AND DEBIT TRANSACTIONS AMOUNTING TO RS OUT OF THE TOTAL CREDITS OF RS IN THE ACCOUNTS RTGS NEFT TRANSACTIONS AMOUNTS TO RS CONSTITUTING AROUND PERCENT OF THE TOTAL CREDITS FUND TRANSFER AMOUNTS TO RS CONSTITUTING AROUND PERCENT OF THE TOTAL CREDITS OUT OF THE TOTAL DEBITS OF RS IN THE ACCOUNTS TRADE TRANSACTIONS AMOUNTS TO RS CONSTITUTING AROUND PERCENT OF THE TOTAL DEBITS IN THE ACCOUNT OF GGC GUJRAT GOLD CENTRE PRIVATE LIMITED AC NO AND LINKED ACCOUNTS TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM TILL TRANSACTIONS ANALYSIS SHOWS CREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH DEPOSIT AND DEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS FROM TILL TRANSACTIONS ANALYSIS SHOWS CREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH DEPOSIT AND DEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH WITHDRAWALFROM TILL TRANSACTIONS ANALYSIS SHOWS CREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH DEPOSIT AND DEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH WITHDRAWAL BALANCE IN THE ACCOUNT AS ON IS RS ON REVIEWING THE TRANSACTION PATTERN OF GGC GUJRAT GOLD CENTRE PRIVATE LIMITED AC AND LINK ACCOUNT IT WAS OBSERVED THATCUSTOMER IS DEALING IN ASSAYING AND REFINING OF SCRAP GOLD SILVER AS WELL AS MANUFACTURE OF BARS AND COINS OF HIGH FINENESS CUSTOMER IS SEEN IMPORTING GOLD DORE BARS FROM UAE PREDOMINANTLY FOLLOWED BY SINGAPORE GHANA BRAZIL PERUCUSTOMER IS UNDERTAKING THE IMPORT OF GOLD DORES POST OBTAINING LICENSE FROM DGFT THROUGH OPEN ACCOUNT DIRECT CROSS BORDER REMITTANCES FOLLOWING ARE THE RED FLAGS OBSERVEDIN LAST MONTHS JANUARY MARCH CUSTOMER ACCOUNT WAS SEEN FUNDED BY AMRAPALI INDUSTRIES LIMITED HTTPS WWWAMRAPALICOM AMRAPALI IS STOCK AND COMMODITY BROKER BY WAY OF RTGS NEFT TO THE TUNE OF INR CR BY HDFC BANK LTD SANDOZ HOUSE BRANCH HDFC AND CREDITS BY WAY OF RTGS NEFT ALSO SEEN FROM AMRAPALIFINANCE TO THE TUNE OF INR CR BY HDFC BANK LTD SA",{"entities":[( 375,400,"FOREIGN ORG"),(768,794,"FOREIGN ORG"),(993,1018,"FOREIGN ORG")]}),
            ("GROUNDS OF SUSPICION THIS IS AN LEA CASE AND A SUPPLEMENTARY CASE BANK WAS IN RECEIPT OF A SUMMON DATED VIDE REFERENCE NUMBER ITBA INV S U S A FROM NARENDRA MOLAI PRASAD DYDIRECTOR OF INCOME TAX INV UNIT INCOME TAX DEPARTMENT OFFICE OF THE ASSISTANT DIRECTOR OF INCOME TAX INVESTIGATION THE NOTICE DIRECTED THE BANK TO PROVIDE ACCOUNT RELATED INFORMATION OF R N EXPORTS HOLDING ACCOUNT NUMBERS AND FOR INVESTIGATION PURPOSE THE INFORMATION SOUGHT HAS BEEN PROVIDED TO THE INCOME TAX DEPARTMENT REVIEW OF THE TRANSACTIONS FOR ACCOUNT FOR THE PERIOD FROM TO REVEALED TOTAL CREDITS OF RS CRORES AND TOTAL DEBITS OF RS CRORES MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF INTERNAL FUND TRANSFERS TOTALLING TO RS CRORES ON INSTANCES WHICH WERE RECEIVED FROM OWN USD DIAMOND DOLLAR ACCOUNT ACCOUNT NUMBER RTGS NEFT CREDITS TOTALLING TO RS LAKHS ON INSTANCES OF WHICH RS LAKHS FROM OWN ACCOUNT KOTAK MAHINDRA BANK LTD ACCOUNT NUMBER ON INSTANCES RS LAKHS FROM CARAT LANE TRADING PRIVATE LIMITED ICICI BANK LTD ACCOUNT NUMBER SLNEFTPL ON INSTANCES RS LAKHS FROM KANTILAL CHHOTALAL BANK OF INDIA ACCOUNT NUMBER ON INSTANCES RS LAKHS FROM NAICE JEWELS PRIVATE LIMITED KOTAK MAHINDRA BANK LIMITED ACCOUNT NUMBER ON INSTANCES RS LAKHS FROM CREATIVE GEMS AND JEWELLERY LTD STATE BANK OF INDIA ACCOUNT NUMBER ON INSTANCES RTGS RETURNS TOTALLING TO RS LAKHS MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGS NEFT TOTALLING TO RS CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT KOTAK MAHINDRA BANK LTD ACCOUNT NUMBER ON INSTANCES FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS LAKHS CHEQUES ISSUANCE TOTALLING TO RS LAKHS IMPORT PAYMENT TOTALLING TO RS LAKHS TO TANU STAR HK LTD HONG KONG ETC A REVIEW OF THE TRANSACTIONS IN USD DIAMOND DOLLAR ACCOUNT FOR ABOVE REVIEW PERIOD REVEALED BUSINESS RELATED TRANSACTIONS SEARCHES CONDUCTED IN PUBLIC DOMAIN DID NOT REVEAL ANY ADVERSE MEDIA ON THE CUSTOMER CASE IS BEING REPORTED TO FIUIND DUE TO THE NOTICE RECEIVED FROM THE INCOME TAX DEPARTMENTCUSTOMER R N EXPORTS DATE OF INCORPORATION PAN AAAFRB WITH IEC CODE ISSUED BY DGFT GSTIN AAAFRBZ ISSUED BY CBEC HAS ADDRESS AS PETIT TOWER RD FLOOR CUMBALLA HILL A K MARG MUMBAI MOBILE NUMBER AND LANDLINE NUMBER THE CUSTOMER HAS BEEN CATEGORISED AS MEDIUM RISK AND THE LAST KYC WAS UPDATED ON WITH THE STATUS AS COMPLETED THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT SHREE PANT BHUVAN SANDHURUST BRIDGE MUMBAI SINCE WITH THE STATUS AS ACTIVE THE CUSTOMER ALSO HOLDS AN USD EEFC ACCOUNT NUMBER SINCE WHICH IS IN DORMANT STATUS SINCE AND USD DIAMOND DOLLAR ACCOUNT NUMBER SINCE WHICH IS IN ACTIVE STATUS AS PER THE KYC RECORDS HELD WITH THE BANK CUSTOMER R N EXPORTS IS A PARTNERSHIP FIRM ENGAGED IN THE BUSINESS OF MANUFACTURING OF JEWELLERY AND WORKING OF DIAMONDS RAHUL ROHIT JHAVERI AND SONALI RAHUL JHAVERI ARE THE BENEFICIAL OWNERS AND AUTHORISED SIGNATORIES OF THE FIRM THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS CRORES REVIEW OF THE TRANSACTIONS FOR ACCOUNT FOR THE CURRENT FINANCIAL YEAR FROM TO REVEALED TOTAL CREDITS OF IN CASH AND TOTAL DEBITS OF IN CASH REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM TO REVEALED TOTAL CREDITS OF RS CRORES IN CASH AND TOTAL DEBITS OF RS CRORES IN CASH RS REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM TO REVEALED TOTAL CREDITS OF RS CRORES IN CASH AND TOTAL DEBITS OF RS CRORES IN CASH RS LAKHS BALANCE IN THE ACCOUNT AS ON WAS A REVIEW OF THE INTERNAL RECORDS REVEALED THAT AN STR WAS REPORTED PREVIOUSLY VIDE BATCH ID SNO WITH THE REPORTED ACCOUNTS AND IN THE NAME OF R N EXPORTS DUE TO CIRCULATION OF FUNDS BETWEEN THE CLIENT AND A HONG KONG BASED ENTITY ON ACCOUNT OF PURCHASE AND SALE OF ROUGH AND POLISHED DIAMONDS",{"entities":[(1654,1670,"FOREIGN ORG")]}),
            ("HIGH VALUE ADV IMP REM AMOUNTED TO RS CR FROM THE A C OF PREMIUM PETRO PRODUCTS MAINLY FAVOURING UAE BASED ENTITIES MOTOTIVE GENERAL TRADING LLC RS CR PETRO STAR FZE RS CR KIARA PETROLEUM FZE RS CR OULA ALNUHA INTERNATIONAL RS CR TIME TECHNO PACKAGING LLC RS CR TASHBULAK TRADING LLC RS CR AND MAN TRADING LLC RS CR TOWARDS IMPORT OF BITUMEN PREMIUM PETRO PRODUCTS IS YET TO SUBMIT BOES PERTAINING TO CASES TOTALLING TO USD WHICH COMPRISES OF CASES TOTALLING TO USD PENDING FOR A PERIOD ABOVE MONTHS UPTO MONTHS AND CASES TOTALLING TO USD PENDING FOR A PERIOD LESS THAN MONTHS IMPS NEFT RTGS PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING PREMIUM PETRO PRODUCTS RS CR PP SOFTTECH P LTD RS CR ICICI VISHAL BOHRA RS CR HDFC IDBI YES BANK VINAYAK EXIMS RS CR AXIS BANK AND AUM KRIPA ENTPS RS CR HDFC INTERCONNECTED FUNDS TRANSFER TXNS WITHIN GROUP A CS AMOUNTED TO RS CR UTURN TRANSACTIONS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS CR WERE RECEIVED AND RS CR WERE SENT OUT TO A CS OF PREMIUM PETRO PRODUCTS VISHAL BOHRA ABHA PRECISION FARMING LLP BHEEM RAJ BOHRA MANISH BOHRA IDBI AND VCI EXPORTS P LTD MAINTAINED WITH OTHER BANKS PREMIUM PETRO PRODUCTS B L INVESTMENT RISHI ENTPS HERO PETRO PRODUCTS REEVA BUILDCON P LTD REALTECH BUILDHOME P LTD ABHA PRECISION FARMING P LTD AONE BUILDHOME LLP AND BODHI FARM DEVELOPERS P LTD HAVE UPDATED A COMMON MAILING ADDRESS HATHROI MARKET AJMER ROAD JAIPUR RAJASTHAN DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER IT WAS NOTED THAT IT WAS A SQ FT OWNED PREMISE WITH SIGN BOARD EMPLOYEES AND OFFICE SET UP SEEN HOWEVER NO STOCK OF GOODS WERE SIGHTED ON ENQUIRY IT WAS LEARNT FROM CUSTOMER THAT GOODS WERE STOCKED AT MOKHAMPURA TEHSILMOZAMABAD DISTRICT JAIPUR PAN OF INTERCONNECTED A CS SHREEBALAJI INFRAPROJECTS P LTD AAPCSK AND DENIS LAND DEVELOPERS P LTD AACCDC ARE REFLECTING IN THE LIST OF POSSIBLE SHELL ENTITIES CIRCULATED BY FIU INDIA TOTAL TAX PAYMENTS OF RS CR WAS NOTED THROUGH THE A CS OF UJJWAL BHANDARI HUF AND VCI EXPORTS P LTD DURING REVIEW PERIOD WHICH APPEARS TO BE DISPROPORTIONATE TO HIGH VALUE AND VOLUME OF TRANSACTIONS ROUTED THROUGH A CS BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH MULTIPLE A CS OPENED BY THE SAME GROUP OF INDIVIDUALS FOR LAYERING OF FUNDS AND POSSIBLY TO REMIT FUNDS OVERSEAS WITHOUT ACTUAL IMPORTS TAKING PLACE POSSIBLY WITH THE INTENTION OF TBML AND HAWALA SETTLEMENT IT ALSO APPEARS THAT SUCH HIGH VALUE AND VOLUME OF TRANSACTIONS MAY HAVE BEEN CARRIED OUT WITHIN GROUP A CS WITHOUT ANY ECONOMIC RATIONALE POSSIBLY WITH INTENTION OF TAX EVASION A C OF PREMIUM PETRO PRODUCTS BEARING NUMBER IS ACTIVE WITH BALANCE OF RS AND HAS OUTSTANDING BALANCE OF RS ON CREDIT LIMIT OF RS AS ON APRIL QUANTUM OF TRANSACTIONS FOR CURRENT FY IS DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FY IS DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CR IS DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CR DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CRCURRENT A CS WERE OPENED IN THE NAME OF A PARTNERSHIP FIRM M S PREMIUM PETRO PRODUCTS NOS AND ON SEP AT OUR BRANCH LOCATED IN KRISHNA TOWER BASEMENT ST FLOOR SARDAR PATEL MARGH JAIPUR RAJASTHAN THE ENTITY WAS FORMED ON MAY AND HAS SUBMITTED COPIES OF PAN AAUFPN PARTNERSHIP DEED CERT OF REG NO IEC AAUFPN AND GST REG CERT AAUFPNZF AS KYC DOCS AT THE TIME OF A C OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS HATHROI MARKET AJMER ROAD JAIPUR RAJASTHAN RITIKA BOHRA MANISH KUMAR BOHRA UJJWAL BHANDARI VISHAL BOHRA AND LALIT KUMAR NAREDI ARE THE PARTNERS WHILE MANISH KUMAR BOHRA AND UJJWAL BHANDARI ARE THE UBOS IN THE SAID FIRM CUSTOMER HAS REGISTERED MOBILE NO WITH THE BANK THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF PETROLEUM PRODUCTS WITH ANNUAL TURNOVER OF RS CR BASIS INTERCONNECTEDNESS OTHER A CS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A CS IN THE NAMES OF REGISTERED COS NBFC PARTNERSHIP FIRMS LLP FIRMS AND INDIVIDUAL CURRENT A C BESIDES SAVINGS A CS WERE OPENED IN THE NAMES OF INDIVIDUALS AND HUF DETAILS OF A CS ARE UPDATED IN XML FILE THE MENTIONED A CS WERE OPENED DURING THE PERIOD SEP TILL NOV AT ABOVE MENTIONED AJMER ROAD RAJA PARK NORTH AVENUE VAISHALI NAGAR BRANCHES IN JAIPUR RAJASTHAN AND SECTOR VASHI BRANCH IN NAVI MUMBAI IT WAS INDICATED THAT THE ENTITIES ARE INTO VARIED BUSINESSES LIKE REAL ESTATE SERVICES WHOLESALER OF CHARCOAL SODA ASH INFRASTRUCTURE GENERAL TRADING TEXTILE PRODUCTS CONSTRUCTION FINANCE AND INVESTMENT DEALING IN STONES MARBLE GRANITE TRADE SALE OF HANDICRAFTS WITH ANNUAL TURNOVER IN THE RANGE OF RS CR TO RS CR RITIKA BOHRA MANISH KUMAR BOHRA UJJWAL BHANDARI VISHAL BOHRA LALIT KUMAR NAREDI AND BHEEM RAJ BOHRA ARE THE COMMON DIRECTORS PARTNERS KARTA IN THE ABOVE MENTIONED A CS WITH OTHER INDIVIDUALS IN DIFFERENT COMBINATIONS NOTICING HIGH VALUE ROUND SUM NEFT RTGS CREDITS FROM OWN A C AND SUBSEQUENT HIGH VALUE ADVANCE IMP REM THROUGH THE A C OF PREMIUM PETRO PRODUCTS AN ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD AND THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION ON REVIEWING A C TRANSACTIONS SINCE APRIL APRIL IT WAS NOTED THAT IMPS NEFT RTGS CREDITS AMOUNTED TO RS CR MAINLY FROM PREMIUM PETRO PRODUCTS RS CR IDBI VCI EXPORTS P LTD RS CR IDFC SURYA NAGARI TRADING RS CR YES BANK ABHA PRECISION FARMING LLP RS CR IDBI AND BHEEM RAJ BOHRA RS CR IDBI EXPORT ADVANCE REMITTANCES AMOUNTED TO RS CR IN THE A CS OF INDIA STONE HOUSE LLP AND VCI EXPORTS P LTD MAINLY FROM WONDERSTONE LLC USA RS CR TRANQUILLO GMBH RS CR GRAMACO SOURCE AND MARBLE RS CR STONE SOURCE INTERNATIONAL INC USA RS CR AND STONELAND INC USA RS CR AS PER WEB CHECK WONDERSTONE LLC USA IS INTO BUSINESS OF JEWELLERY STORES HTTPS WWWDNBCOM BUSINESSDIRECTORY COMPANYPROFILESWONDERSTONE LLCDFEAAEEAADFHTML TRANQUILLO GMBH IS INTO CLOTHING STORE INDUSTRY STONE SOURCE INTERNATIONAL INC IS INTO BUSINESS OF STONES GRANITES AND STONELAND INC IS INTO GRANITE MARBLE BUSINESS IT APPEARS THAT INDIA STONE HOUSE LLP WHICH HAS INDICATED THAT IT IS INTO BUSINESS OF STONES GRANITES MAY HAVE RECEIVED EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS LIKE WONDERSTONE LLC USA JEWELLERY STONES AND TRANQUILLO GMBH CLOTHING STORE INDUSTRY AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE",{"entities":[(116,144,"FOREIGN ORG"),(151,165,"FOREIGN ORG"),(172,191,"FOREIGN ORG"),(203,223,"FOREIGN ORG"),(230,255,"FOREIGN ORG"),(262,283,"FOREIGN ORG"),(294,309,"FOREIGN ORG"),(5682,5697,"FOREIGN ORG"),(5708,5718,"FOREIGN ORG"),(5730,5755,"FOREIGN ORG"),(5762,5793,"FOREIGN ORG"),(5807,5820,"FOREIGN ORG"),(5848,5863,"FOREIGN ORG"),(5954,5969,"FOREIGN ORG"),(5985,6000,"FOREIGN ORG"),(6033,6063,"FOREIGN ORG"),(6084,6117,"FOREIGN ORG"),(6327,6342,"FOREIGN ORG"),(6368,6383,"FOREIGN ORG")]}),
            ("THE TRANSACTION PATTERN OBSERVED IN CUSTOMER S ACCOUNT IS RECEIPT OF HIGH VOLUME OF DOMESTIC INWARD RTGS FROM CUSTOMER S OWN ACCOUNT WITH AXIS BANK LTD KHAR MUMBAI AND ICICI BANK LTD ANDHERI MUMBAI AND FROM ENTITY NAMED SAWARTHIYA FILM PRODUCTION WHOSE TRANSACTIONS ARE FROM SAME IFSC AS CUSTOMER S ACCOUNT ICIC IE ANDHERI MUMBAI IMMEDIATELY FOLLOWED BY FOREIGN OUTWARD REMITTANCES THEY SEEM TO BE RELATED PARTIES FOLLOWING EXACT SAME TRANSACTION ROUTE OF SENDING DOMESTIC RTGS THERE ARE NO DETAILS FOUND ON SAWARTHIYA FILM PRODUCTION ON PUBLIC DOMAIN EXCEPT A SMALL SNIPPET FOUND ON SOCIAL MEDIA THAT THEY ARE COPRODUCERS TO OUR CUSTOMER ON ONE OF THE PROJECTS REFER LINK HTTPS WWWFACEBOOKCOM WATCH V SAWARTHIYA FILM PRODUCTION IFSC ICIC HAS SENT RTGS TO OUR CUSTOMER AMOUNTING TO INR BETWEEN TH MARCH AND TH APRIL CUSTOMER HAS SENT BACK INR LAKHS TO SAWARTHIYA FILM PRODUCTION AT IFSC ICIC NEW DELHI IN MARCH THESE TRANSACTIONS SEEMS TO HAVE NO ECONOMIC RATIONALE NOR THERE SEEMS TO BE RATIONALE BEHIND OPENING OF ACCOUNTS GETTING BANKING SERVICES AT VARIOUS BANKS AND LOCATIONS CUSTOMER HAS A FACEBOOK PAGE YOUTUBE CHANNEL WITH FEW VIDEOS AND THERE ARE NEWS ARTICLES ON THE PRODUCTION OF THE VIDEOS WITH SOCIAL MEDIA INFLUENCERS OR RELATIVELY KNOWN ARTISTS HOWEVER THE NUMBER OF FOLLOWERS OR LIKES OR VIEWS PROJECT THE EARNINGS A COMPANY COULD MAKE AND THE SAME DOESN T SEEM TO BE IN TANDEM WITH THE VOLUME OF OUTWARD REMITTANCES MADE REFER LINK HTTPS WWWFORBESCOM SITES JROSE HOWMUCHDOYOUTUBERSREALLYMAKE SH ABDB HTTPS MINTINTUITCOM BLOG RELATIONSHIPS HOWMUCHDOYOUTUBERSMAKE TEXT WITH THE AVERAGE YOUTUBE PAYFOR EVERY C VIDEO VIEWSHTTPS WWWSANTABANTACOM BOLLYWOOD MANISHKUMARSINGHANDSAPNAKUMARIGRACETHEMAASHUKASONGLAUNCHEVENTPRESENTEDBYIMAGINARYFILMSINCOLLABORATIONWITHTAGDOGMEDIA HTTPS INDUSDAILYHUNTIN NEWS INDIA URDU GOSSIP GANJEPAPERGOSGANEN MANISH SINGH AND SAPNA S IMAGINARY FILMS RELEASES A GROOVY CHARTBUSTER MAASHUKA FEATURING BHAVIN BHANUSHALINEWSIDN PGS N AND PGN AND SNIPPET FROM ONE OF THE ARTICLES FORBES ALSO ESTIMATES THAT FOR TOP TALENT A YOUTUBER CAN MAKE ABOUT FOR EVERY VIDEO VIEWS AND CUSTOMER HAS ONLY MN VIEWS OVERALL WITH TOTAL VIDEOS UPLOADED IN LAST MONTHS CUSTOMER HAS MINIMAL LEGITIMATE WORK AND IT IS NOT IN LINE WITH THE INFERRED EARNINGS ONE COULD MAKE IN THAT LINE OF ACTIVITY THROUGH YOUTUBE AND THE TRANSACTIONS THAT CUSTOMER IS MAKING THIS WAY WE ARE UNABLE TO GAUGE THE SOURCE OF WEALTH ALSO UNABLE TO UNDERSTAND THE SOURCE OF FUNDS SINCE CUSTOMER IS ROUTING RTGS FROM VARIOUS DOMESTIC BANK ACCOUNTS TO OUR BANK THE ANNUAL TURNOVER CLAIMED BY THE CUSTOMER WITH OUR BANK IS NOT IN TANDEM WITH THE ACTUAL TURNOVER SEEN IN THE ACCOUNT IN A SINGLE QUARTER CUSTOMER BEING NEWLY INCORPORATED HAS OPENED ACCOUNTS IN MULTIPLE BANKS ICICI BANK AXIS BANK AND OUR BANK WHICH RAISES SUSPICION ON THE INTENT AND THE PURPOSE REQUESTING TO FILE STR BASIS BELOW POINTS A THE INVOICE OF ONE OF THE OVERSEAS PARTIES IS NOT GENUINEB ALL RELATED PARTIES AND ACCOUNTS ARE RELATIVELY NEWLY INCORPORATEDC UNABLE TO GAUGE THE SOURCE OF WEALTH AND SOURCE OF FUNDSD THE OPENING OF ACCOUNTS IN VARIOUS BANKS AND THEN PROCEEDING WITH THE TRANSACTIONS IN A PATTERN THAT MAKES NO ECONOMICAL SENSEE THE ACTUAL TURNOVER IS HIGHER THAN THE CLAIMED ANNUAL TURNOVER TURNOVER IN A SINGLE QUARTER HAS CROSSED THE CLAIMED ANNUAL TURNOVERPART A GROUNDS OF SUSPICION CUSTOMER IMAGINARY FILMS PRIVATE LIMITED A C IS A NEWLY OPENED ACCOUNT WHO HAS REMITTED HUGE VOLUME OF FOREIGN OUTWARD REMITTANCES IN A SHORT PERIOD OF TIME CUSTOMER IMAGINARY FILMS PRIVATE LIMITED MAINTAINS CURRENT ACCOUNT NUMBER WITH OUR KHAR WEST MUMBAI BRANCH SINCE PAN NUMBER IS UPDATED AS AAFCIQMOBILE NUMBER IS UPDATED AS PROFILE DECLARED IS FILM PRODUCTION WITH ANNUAL TURNOVER OF INR LAKHS EMAIL ID UPDATED IS INFO IMAGINARYFILMSIN DATE OF INCORPORATION IS GSTIN IS AAFCIQZWRISK PROFILE IS MEDIUMBALANCE IN THE ACCOUNT AS ON TH APRIL IS INR THE CUSTOMER IS A PRIVATE LIMITEDMAILING AND PERMANENT ADDRESS OF THE CUSTOMER IS C O IMAGINARY FILMS PRIVATE LIMITED EMBASSY CHAMBERS RD ROAD KHAR WEST MUMBAI MAHARASHTRA AUTHORISED SIGNATORIES ARE SAPNA KUMARI AND MANISH KUMAR SINGH BENEFICIAL OWNERS ARE SAPNA KUMARI AND MANISH KUMAR SINGH SCRUTINY OF ACCOUNT STATEMENT OF IMAGINARY FILMS PRIVATE LIMITED A C NO FOR PERIOD SINCE JAN ACCOUNT OPENING DATE TILL DATE REVEALED THAT CUSTOMER HAS RECEIVED A TOTAL CREDIT OF INR LAKHS OUT OF THE TOTAL CREDITS OF INR LAKHS RTGS NEFT UPI IMPS CREDITS ARE INR LAKHS FROM CUSTOMER S OWN ACCOUNT IN ICICI BANK AND AXIS BANK AND FROM ANOTHER ENTITY NAMED SAWARTHIYA FILM PRODUCTION WHICH TOO IS IN ICICI BANKMAJOR DEBITS HAVE BEEN OBSERVED VIA FOREIGN OUTWARD REMITTANCES OF INR LAKHS TO JUST OVERSEAS PARTIES BASED OUT OF SINGAPORETHESE AFOREMENTIONED DEBITS ARE OUT OF TOTAL DEBITS OF INR LAKHS NO LINKED ACCOUNTS FOUND EMAIL ID MOBILE NO AND PAN WERE CHECKEDPART B DETAILS OF INVESTIGATION CUSTOMER HAS MADE FOREIGN OUTWARD REMITTANCES AMOUNTING TO INR LAKHS AGAINST PURPOSE CODE SAUDIOVISUAL AND RELATED SERVICES LIKE MOTION PICTURE AND VIDEO TAPE PRODUCTION DISTRIBUTION AND PROJECTION SERVICES TO JUST TWO OVERSEAS PARTIES NAMED LIVING FILMS PTE LTD SG AND GATE STUDIOS PTE LTD SG WITHIN MONTHS OF OPENING THE ACCOUNT WITH OUR BANK THE OVERSEAS PARTIES WHEN CHECKED ON PUBLIC DOMAIN IT WAS FOUND THAT THEY TOO ARE NEWLY INCORPORATED LIVING FILMS PTE LTD HAS BEEN INCORPORATED ON TH SEPTEMBER REFER LINK HTTPS WWWSGPBUSINESSCOM COMPANY LIVINGFILMSPTELTD AND GATE STUDIOS PTE LTD HAS BEEN INCORPORATED ON ST OCTOBER REFER LINK HTTPS SINGAPORECORPCOM CO GATESTUDIOSPTELTD OUR CUSTOMER TOO HAS BEEN INCORPORATED IN IE ND JANUARY THERE IS AN ADDITIONAL INFORMATION FOUND WRT LIVING FILMS PTE LTD ON PUBLIC DOMAIN SEARCH IT WAS FOUND THAT THERE IS A RELATIVELY KNOWN COMPANY BY THE SAME NAME IN THAILAND REFER LINK HTTPS ENWIKIPEDIAORG WIKI LIVING FILMS HTTPS WWWLIVINGFILMSCOM THE CUSTOMER HAS SUBMITTED INVOICE WHEREIN LIVING FILMS PTE LTD SG CLAIMS THAT LIVING FILMS THAILAND IS THEIR HEADOFFICE AND HAS MENTIONED ITS GEOGRAPHICAL ADDRESS AND EMAIL ID FROM LIVING FILMS THAILAND S WEBSITE HEREON WOULD BE MENTIONED AS WEBSITE HOWEVER ON OBSERVING THE INVOICE CLOSELY IT IS FOUND THAT THE PINCODE GIVEN ON WEBSITE IS BUT IN INVOICE IT IS AND EMAIL ADDRESS ON WEBSITE IS LIVING LIVINGFILMSCOM BUT IN INVOICE IT IS LIVING LIVINGFILMCOM THE CONTACT NUMBERS IN THE INVOICE ALSO DO NOT MATCH WITH THE ONES GIVEN ON THE WEBSITE ALSO ONE OF THE NUMBERS BELONGS TO HONGKONG WHERE AS FOR SINGAPORE IT IS AND FOR THAILAND IT IS REFER LINK HTTPS ENWIKIPEDIAORG WIKI TELEPHONE NUMBERS IN HONG KONG HTTPS ENWIKIPEDIAORG WIKI TELEPHONE NUMBERS IN SINGAPORE HTTPS ENWIKIPEDIAORG WIKI TELEPHONE NUMBERS IN THAILAND AND THERE IS NO MENTION OF SINGAPORE BRANCH ON THE LIVING FILMS THAILAND S WEBSITE THE INVOICE DOES NOT SEEM GENUINE AND SEEMS TO BE FRAUDULENT THE AGREEMENTS WRT BOTH THE OVERSEAS PARTIES HAVE NO MENTION OF ANY STAKEHOLDER AND IT IS EXACTLY THE SAME",{"entities":[(5171,5191,"FOREIGN ORG"),(5471,5491,"FOREIGN ORG"),(5565,5581,"FOREIGN ORG")]}),
            ("IN CHINA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS INDUCTION CHAFER HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON FROM PREMIER HOTEL MANAGEMENT CO LTD FROM BANGLADESH DETAILS OF WHOOM IS NOT AVAILBLE IN PUBLIC DOMAIN IN AN ANOTHER MTT TRANSACTION VIOLATION OF MTT CYCLE NOT COMPLETED IN DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT TO TO TUNE OF EUR TO ERNST NEUMARKER GMBH AND COENTITY DEALS IN FOOD SUBSTITUES SNACKS MAKING MACHINES WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS KITCHENWARE HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON TO THE TUNE OF USD IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF EUR LACS GBP USDLACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF KITCHINWARE IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDA H INTERNATIONAL PARTNERSHIP FIRM HAVING ADDRESS ST FLOOR GANDHIPATHCHITRAKOOT SCHEME JAIPUR RAJASTHAN INDIA CONTACT NUMBER AND EMAIL ID VINAY AHHOSPITALITYCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD JAIPUR VAISHALI NAGARGROUND FLOOR SAURAV TOWERS PLOT NO C C BLOCK VAISHALI NAGAR JAIPUR RAJASTHAN SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AATFAB IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS GSTIN AS PER BANK RECORDS IS AATFABZR OTHER RELATIONSHIP OF CUSTOMER ARE TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VINAY LODHA AUTHORISED SIGNATORY ALTPLN BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS AMRAW DEVI LODHAHIMANSHU LODHAACWPLM THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN GSTIN CUSTOMER INVOLVE IN BUSINESS OF HOSPITALITY SERVICE AND PROVIDES PRODUCTS RELATED TO HOSPITALITY CUSTOMER IMPORT MATERIAL LIKE SCHOTT ZWIESEL PURE LOOP IE WINE GLASS ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO CRALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN JAIPURVAISHALI NAGAR BRANCH TRANSFERS RS LAKHS MAJORLY FROM HIMANSHU LODHAAMRAW DEVI LODHA JTCLEARING RS LAKHS MAJORLY FROM APA HOTELS PVT LTDTHE INDIAHOTRELCROWN PROBUILD NEFT RS LAKHS MAJORLY FROM RELIANCE INDUSTRIESA H INTERNATIONALVINAY LODHA RTGS RS LAKHS MAJORLY FROM A H INTERNATIONALBAID LEASING AND FINANCERSB WEALTH PRIVATE LIMITED REMITTANCE RS LAKHS MAJORLY FROM MEYER INTERNATIONAL MARKETING PTE ALETA AZUL CAPITAL SL CK D U ADVENTURES PRIVATE LIMITED INTERNET FUND TRANSFERS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO KAMAL NANDLAL MANSINGKAPINKCITY MOTORS PVTLTDMKTRADERS CLEARING RS LAKHS MAJORLY TO YASH BHANDARICHAIN ROOP BAIDNIRMALA ASHOKRAJ REMITTANCE RS LAKHS MAJORLY TO ABERT SPASAMBONET PADERNO INDUSTRIE SPAABERT SPA INTERNET FUND TRANSFER RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN DAYS LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF EUR LACS GBP USDLACS FOR EXAMPLE VIOLATION OF LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN TO USD REMITTED TO JLIP GROUP CO LTD CLIENT DEALS TO HOSPITALITY INDUSTRY ITS I TRADER OF FURNITURE ADN RELATED ITEMSHTTP WWWJLIPCOMCN ENGLISH",{"entities":[(2894,2927,"FOREIGN ORG"),(4190,4208,"FOREIGN ORG")]}),
            ("FAISHING INDUSTRIAL HK CO LTD IN HONGKONG WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS TAGS HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED ON FROM FUNSTATION LIMITEDFUNSTATION ARE FAMILY ENTERTAINMENT CENTRES WITH CARNIVAL GAMES LASER TAG ADVENTURE GOLF BOWLING VR AND MUCH MORE FUN GAMES HTTPS WWWFUNSTATIONUKCOM IN AN ANOTHER MTT TRANSACTION VIOLATION OF FUNDS OUTLAY MORE THAN DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN TO TO TUNE OF USD TO CARDY INTELLIGENT TECHNOLOGY CO LTD CUSTOMER IS INTO MANUFACTURE OF RFID TAGS WRISTBAND ETCHTTPS WWWSMARTCARDYCOM WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS RFID CARDS HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN AND TO THE TUNE OF USD LACS FROM BAHJAT BAHLA BAHLA AL MAMOORAH DETAILS OF THE WHOM IS NOT AVAILABLE IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDSEMNOX SOLUTIONS PVT LTD PRIVATE LTD HAVING ADDRESS RD FLOOR FLRMG ROAD LALBAGHMANGALOREPUNJA BUILDING ANNEXE MANGALORE KARNATAKA INDIA CONTACT NUMBER AND EMAIL ID IQBAL SEMNOXCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD MANGALORE P M RAO ROADBHARATH BUILDINGGROUND FLOOR PMRAO ROADMANGALORE KARNATAKA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AALCSB IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE KIRAN KARANKI AUTHORISED SIGNATORY SAMPATH KUMAR DIRECTOR IQBAL MOHAMMAD DIRECTOR AAHPIA RAJEEV GOPALAKRISHNAN DIRECTOR BALAJI VENUGOPAL DIRECTORBENEFICIAL OWNER OF THE REPORTED ACCOUNT IS KIRAN KARANKADZPKQMOHAMMAD IQBALAAHPIA THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN MOA AOA COI DATE OF LAST KYC UPDATION SEMNOX SOLUTIONS PRIVATE LIMITED SSPL IS ENGAGED IN THE BUSINESS OF PROVIDING IT SOLUTIONS FOR THE ENTERTAINMENT AND LEISURE INDUSTRY SSPL OFFERS SOLUTIONS SUCH AS FAMILY ENTERTAINMENT WATER AND THEME PARKS AND FOOD AND BEVERAGES THE COMPANY CATERS TO WATER PARKS MUSEUMS KIDS PLAY AREAS RESTAURANTS FOOD COURTS AND AQUARIUMS AMONG OTHERS SSPLS FEC AND PARK CLIENTS INCLUDE SUCH AS COWPLAY CPWMOO SINGAPORE ARENA REKREASI KELUARGA AEON FANTASY MALAYSIA AND SPARKS INDONESIA AND CAMAYA COAST PHILIPPINES AMONG OTHERS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO CRALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS REMITTANCE RS LAKHS MAJORLY FROM SEMNOX SOLUTIONS LLC LUAEON FANTASY MALAYSIA SDN BHDSEMNOX SOLUTIONS DMCC P O BOX TOTAL DEBIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO SEMNOX SOLUTIONS PRIVATE LIMITED REMITTANCE RS LAKHS MAJORLY TO FAISHING INDUSTRIAL HK CO LIMITEDTITOMA MANUFACTURING LTDAMAZON WEB SERVICES INC FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NIL REMITTANCE RS LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN DAYS LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF USLACS FOR EXAMPLE VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON USD REMITTED TO FAISHING INDUSTRIAL HK CO LTDTHIS COMPANY DEALS IN EXPORT UNDER DISCS TAPES SOLIDSTATE NONVOLATILE STORAGE DEVICES SMART CARDS AND OTHER MEDIA FOR THE RECORDING OF SOUND OR OF OTHER PHENOMENA WHETHER OR NOT RECORDED INCLUDING MATRICES AND MASTERS FOR THE PRODUCTION OF DISCS BUT EXCLUDING PRODUCTS OF CHAPTER CATEGORY",{"entities":[(0,29,"FOREIGN ORG"),(576,611,"FOREIGN ORG"),(3199,3219,"FOREIGN ORG"),(3220,3234,"FOREIGN ORG"),(3251,3272,"FOREIGN ORG"),(3410,3435,"FOREIGN ORG"),(3443,3466,"FOREIGN ORG"),(3467,3490,"FOREIGN ORG"),(4279,4307,"FOREIGN ORG")]}),
            ("IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINE TOOL AND ACCESSORIES IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDTHERMAX BABCOCK WILCOX ENERGY SOLUPLTDPRIVATE LTD HAVING ADDRESS  ADISAICONGAT NO FLR BANGALORE HIGHWAYBAVDHAN BUDRUKOPPOSITE HEMRLMUMBAI PUNE MAHARASHTRA INDIA CONTACT NUMBER   AND EMAIL ID   RAMNATHNAYAK@TBWESCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD PUNE BUND GARDENA WING SHANGRILA GARDENS BUND GARDEN ROAD PUNE MAHARASHTRA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AADCTA IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS GSTIN AS PER BANK RECORDS IS AADCTAZJ TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NIRANJAN PAWGI AUTHORISED SIGNATORY ABKPPN NIRANJAN PAWGI AUTHORISED SIGNATORY ABKPPN LATA KUMAR AUTHORISED SIGNATORY ASHISH MOHAN LAL BHANDARI AUTHORISED SIGNATORY AEXPBF MOHAN LAL BHANDARI ASHISH AUTHORISED SIGNATORY AEXPBF THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN MOAAOA COI THERMAX BABCOCK WILCOX ENERGY SOLUTIONS PRIVATE LIMITED TBWES IS A JOINT VENTURE BETWEEN THERMAX LIMITED PUNE INDIA AND BABCOCK WILCOX POWER GENERATION GROUP INC USA THE COMPANY IS ENGAGED IN MANUFACTURING SUBCRITICAL AND SUPERCRITICAL BOILERS FOR THERMAL POWER GENERATION THE COMPANYS OTHER PRODUCTS INCLUDE LOW NOX COMBUSTION SYSTEMS AND ROLL WHEEL PULVERIZERSALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE   FY FROM TO    TOTAL CREDIT SIDE RS LAKHS OF WHICH CLEARING RS LAKHS MAJORLY FROM ICICI PRUDENTIAL M NEFT RS LAKHS MAJORLY FROM MAHARASHTRA VALUE ADDED TAXUTI MUTUAL FUNDTHERMAX LIMITED RTGS RS LAKHS MAJORLY FROM UTI MF COMMON FUNDSBI MUTUAL FUNDDSP MUTUAL FUND REDEMP INTERNET FUND TRANSFERS RS LAKHS AND OTHERS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO THERMAX BABCOCK WILCOX ENERGY SOLUPLTD CLEARING RS LAKHS RTGS RS REMITTANCE RS LAKHS MAJORLY TO MANTRA SWITCHGEAR CO LTDLSIS CO LTD INTERNET FUND TRANSFER RS LAKHS FY FROM TO    TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RS BY CASH REMITTANCE RS LAKHS FY FROM TO    TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS FY FROM TO    TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NO COMPLETED IN DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF USD LACS FOR EXAMPLE MTT CYCLE NOT COMPLETED IN DAYS WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN ON USD CREDITED FROM LSIS CO LTD AUTOMATION PLC ELECTRONIC DEVICES MANUFACTURING FROM SOUTH KOREA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSOTHERS HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED FROM THE BABCOCK WILCOX COMPANY FROM USA THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE",{"entities":[(1422,1463,"FOREIGN ORG"),(2382,2405,"FOREIGN ORG"),(2406,2417,"FOREIGN ORG"),(3175,3186,"FOREIGN ORG"),(3466,3492,"FOREIGN ORG"),(3187,3201,"FOREIGN ORG")]}),
            ("AND GOODS MENTIONED AS HS ORTHOPAEDIC GOODSHS DRILLING MACHINEHS CONNECTING RODS HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED TO FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS HTTPS WWWDNBCOM BUSINESSDIRECTORY COMPANYPROFILESINNOVATION ORTHO LINE LIMITEDCABBABBECCEAHTML IN AN ANOTHER MTT TRANSACTION VIOLATION OF MTT CYCLE NOT COMPLETED IN DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN AND TO TUNE OF USD LACS TO JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS DRILLING HS SELF DRILLING HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN AND TO THE TUNE OF USDLACS FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS HTTPS WWWDNBCOM BUSINESSDIRECTORY COMPANYPROFILESINNOVATION ORTHO LINE LIMITEDCABBABBECCEAHTML IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USDLACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNTSTR IS BEING FILEDAUXEIN MEDICAL PRIVATE LIMITEDPRIVATE LTD HAVING ADDRESS PLOT NO HSIIDC FLRPHASE IVKUNDLI SONIPAT SONIPAT HARYANA INDIA CONTACT NUMBER AND EMAIL ID ACCOUNTS AUXEINMEDICALCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD NEW DELHI MODEL TOWNH MODEL TOWN III DELHI SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAKCAK IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS THERE ARE NO LINKED ACCOUNT AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE RAHUL LUTHRA AUTHORISED SIGNATORY DEEPAK LUTHRA AUTHORISED SIGNATORY GAURAV LUTHRA ASHOK LUTHRABENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK LUTHRAACBPLARAHUL LUTHRAADYPLJ THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN MOA AOA COI AS PER ENHANCED DUE DILIGENCE AUXEIN MEDICAL PRIVATE LIMITED AMPL IS ENGAGED IN MANUFACTURING OF ORTHOPEDIC IMPLANTS THE COMPANYS PRODUCTS INCLUDE MINI FRAGMENT HIP PROSTHESIS INTERLOCKING NAILS INSTRUMENT AND IMPLANT BOX SPINE IMPLANTS MAXILLOFACIAL IMPLANTS WISE LOCKING PLATES AND BONE PLATES THE COMPANYS UNIT IS EQUIPPED WITH CNC MACHINES DRILL MACHINES COMPARATORS CUTTING MACHINES AND SPECIAL PURPOSE MACHINES AMPL HAS A RANGE OF IMPLANTS AVAILABLE IN COUNTRIES IN ASSOCIATION WITH PARTNERSALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH REMITTANCE RS LAKHS MAJORLY FROM FIXIER SA DE CV CARR DR M INNOVATION ORTHO LINE LTD TOTAL DEBITS DURING THE SAME PERIOD WAS RS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS MAJORLY FROM ZOLDAN CORPOREISHON CIA L MEDICAL ORTOVIT SRL BUCUR ATRON HEALTH S A LEOFO TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS MAJORLY TO CHANGZHOU NUBE INTERNATIONALCHANGZHOU XIETONG INDUSTRIES COCHANGZHOU XIETONG INDUSTRIES CO LTD FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHSTOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF USDLACS FOR EXAMPLE VIOLATION OF FUNDS OUTLAY MORE THAN DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN TO USD LACS REMITTED IN PARTS FROM JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE",{"entities":[(2905,2914,"FOREIGN ORG"),( 2931,2956,"FOREIGN ORG"),(3188,3213,"FOREIGN ORG"),(3222,3233,"FOREIGN ORG"),(3240,3262,"FOREIGN ORG"),(3329,3356,"FOREIGN ORG"),(3357,3385,"FOREIGN ORG"),(3386,3423,"FOREIGN ORG"),(4222,4261,"FOREIGN ORG")]}),
            ("OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS REFINED SUNFLOWER OIL HOWEVER THE RESPECTIVE EXPORT WAS RECEIVED ON BETWEEN AND TO THE TUNE OF RSLACS FROM DJT IMPORT EXPORT IN MADAGASCAR WHOSSE DETAILS ARE NOT FOUND IN PUBLIC DOMAIN IN AN ANOTHER MTT TRANSACTION VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN AND TO TUNE OF RS LACS WAS REMITTED TO HEBEI LUEDING IMPORT AND EXPORT CO LTDCOMPANY DEALS IN PPGI GALVANISED STEEL COIL ROLL FORMING MACHINE ROOFING SHEETS WELDING ELECTRODEHTTPS //WWWMACHINERYOFFERSCOM/COMPANY/LEADINGROOFING/CONTACTHTML AND SHOUGUANG DASEN WOOD CO LTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED ASHS PLYWOOD GALVANIZED CORRUGATED SHEET HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN AND TO THE TUNE OF EUR IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF CRORES WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF DIVERSIFIDE PRODUCTS OILS GA SHEETS WOOD ETC IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDTHAKKAR EXIMPROPRIETORSHIP FIRM HAVING ADDRESS THAKKAR HOUSERAJDEV STREET SANGANVA CHOWK RAJKOT GUJARAT INDIA CONTACT NUMBER AND EMAIL ID INFO THAKKAREXIMCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD RAJKOT YAGNIK ROAD SHOP NO CONVENTION CENTER NR HARIBHAI HALLOFF YAGNIK ROAD RAJKOT GUJARAT SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAXPTQ PAN LINKED ACCOUNTS ARE IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NITESH HARIDAS THAKKAR PROP AAXPTQ NITESH HARIDAS THAKKAR AUTHORISED SIGNATORY AAXPTQ THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PANMOAAOAIE CODE DATE OF LAST KYC UPDATION CUSTOMER IS INTO IMPORT EXPORT AND MERCHANTING TRADER HAVING BEGUN WITH ONLY ONE TEXTILE PRODUCT THE COMPANY HAS NOW WELL DEVELOPED INTO AN STAR EXPORT HOUSE DEALING IN VARIOUS PRODUCTS LIKE AGRICULTURAL COMMODITIES SUCH AS RICE OF DIFFERENT VARIETIES WHEAT FLOUR BAKERY FLOUR BISCUITS PLASTIC HOUSEHOLD PRODUCTS KITCHEN APPLIANCES AND HOME APPLIANCES FIRM IS HAVING WEBSITE WWWTHAKKAREXIMCOM ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY FROM NITESH HARIDAS THAKKARTHAKKER ELECTRICALSTHAKKAR EXIMCLEARING RS LAKHS NEFT RS LAKHS MAJORLY FROM THAKKAR EXIMGOPALSWAMY BASKARANE PAO GST REFUNDS RTGS RS LAKHS MAJORLY FROM THAKKAR EXIMPMS ENTERPRISESARTI INTERNATIONAL REMITTANCE RS LAKHS MAJORLY FROM THAKKAR EXIMTHAKKAR EXIM RAJDEV STREETSOCIETE RAG INTERNET FUND TRANSFERS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH TRANSFERS RS LAKHS MAJORLY TO THAKKAR EXIMBHAVNA MAHESH PARMARPARMAR RAJDEEP MAHESH CLEARING RS LAKHS RTGS RS LAKHS MAJORLY TO VIRAJ IMPEX REMITTANCE RS LAKHS MAJORLY TO HEBEI TOMATO INDUSTRY CO LTD FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHSTHE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER IN THE NAME OF THAKKAR EXIM REPORT SERIAL NUMBER VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF CRORES FOR EXAMPLE VIOLATION OF MTT CYCLE NOT COMPLETED IN DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON TO THE TUNE OF USD TO OIL TEC FOR OILS AND DETERGENTS MANUFACTURER OF SOAPS ADN OILS HTTPS //WWWGULFOODCOM/EXHIBITORS/OILTECFOROILSANDDETERGENTS IN EGYPT WITH PURPOSE",{"entities":[(3447,3475,"FOREIGN ORG"),(233,250,"FOREIGN ORG"),(486,523,"FOREIGN ORG"),(690,716,"FOREIGN ORG")]}),
            ("OF THE VALUE OF USD LACS FOR EXAMPLE VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN  DAYS IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT  BETWEEN  TO  USD  LACS REMITTED IN PARTS TK FUJIKIN CORPORATION ENTITY DEALS IN PIPES AND FITTINGS HTTP://TKFUJIKINCOM/TKSCT//ENG/SUB/HISTORYPHP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS  CONTROLLER HOWEVER RESPECTIVE INWARDS REMITTANCE TOWARDS EXPORTS RECEIVED ON  TO TUNE OF USD LACS CARBON CRAFT ANG DETAILS OF WHOM ARE NOT AVAILABLE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE IN AN ANOTHER MTT TRANSACTION VIOLATION OF FUNDS OUTLAY OF MORE THAN  DAYS OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON  USD  TO  TK FUJIKIN CORPORATION ENTITY DEALS IN PIPES AND FITTINGS HTTP://TKFUJIKINCOM/TKSCT//ENG/SUB/HISTORYPHP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS  FITTINGS THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS  CLIENT IS DEALING IN MERCHANT TRADE OF PIPES FITTINGS IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDPRAMA INSTRUMENTS PVT LTD PRIVATE LTD HAVING ADDRESS:PLOT RRABALE NAVI MUMBAI MAHARASHTRA INDIA  CONTACT NUMBER: + AND EMAIL ID: MADAN@PRAMAGROUPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD MUMBAI  MAHAPESHOP NO   PLOT NO X TECHNOCITY  NEXT TO SAROVAR PORTICO MAHAPE THANE DIST NAVI MUMBAI MAHARASHTRA  SINCE  DATE OF INCORPORATION IS  PAN NUMBER OF THE ACCOUNT IS AAACPF IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS  OTHER RELATIONSHIP OF CUSTOMER ARE   TRANSACTION LINKED ACCOUNTS ARE   AUTHORISED SIGNATORY AND DIRETOR OF THE REPORTED ACCOUNT IS  MADANMOHAN AMBIKAPRASAD MISHRAAABPMBVIMLADEVI MADANMOHAN MISHRAAHWPMH BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MADAN MOHAN A MISHRAAABPMBMADANMOHAN A MISHRAAABPMB THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN IEC COI ENTITY IS ENGAGED IN MANUFACTURING AND TRADING OF LABORATORY INSTRUMENTS AND FILTRATION SYSTEMS THE COMPANYS PRODUCT PORTFOLIO INCLUDES CHROMATOGRAPHY AND INSTRUMENTS FILTER HOLDER STERILITY TEST MANIFOLD SYSTEM WATER JET CUTTING MACHINE TURN OVER SILICON SEPTA SYRINGE FILTER HOLDER STERILITY MANIFOLD TURN OVER SILICONE SEPTA AND STABILITY CHAMBER PIPL CATERS TO PHARMACEUTICAL CHEMICAL FOOD FRAGRANCES FORENSIC AND ENVIRONMENT MONITORING SECTORS PIPL WAS INCORPORATED IN  AND HAS ITS REGISTERED OFFICE LOCATED IN MUMBAI MAHARASHTRA DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAC TO RS CR ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT  WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROM  TILL  SHOWS TOTAL CREDIT RS  TOTAL DEBITS DURING THE SAME PERIOD WAS RS  LAKHS OF WHICH INTERNET FUND TRANSFER RS  LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH TRANSFERS RS  LAKHS MAJORLY FROM BIORADIUS THERAPEUTIC RESEARCH PVT LTDPRESSHOT ENGINEERSHOTEL ROYAL ELITE CLEARING RS  LAKHS MAJORLY FROM GUJARAT FLUOROCHEMICALS LIMITEDPRAMA INSTRUMENTS PVT LTDABB INDIA LTD NEFT RS  LAKHS MAJORLY FROM PRAMA INSTRUMENTS PVT LTDPRAMA INSTRUMENTS PRIVATE LTDABB INDIA LIMITED RTGS RS  LAKHS MAJORLY FROM TRUSHNA EXIMPRAMA INSTRUMENTS PVT LTD INTERNET FUND TRANSFERS RS  LAKHS MOBILE MONEY TRANSFERS RS  LAKHS IN  INSTANCES TOTAL DEBIT SIDE RS  LAKHS OF WHICH  TRANSFERS RS  LAKHS MAJORLY TO ICICI PRUD LIFE INS CO LTD EMPLOYEES PROVIDENT FUND ORGANISATIONESI FUND A/C NO  CLEARING RS  LAKHS MAJORLY TO DEUTSCHE BANKMR ASHOK BHUTA RTGS RS  LAKHS MAJORLY TO PRAMA INSTRUMENTS PVT REMITTANCE RS  LAKHS MAJORLY TO TK FUJIKIN CORPORATIONGARDNER DENVER THOMAS PNEUMATIC INTERNET FUND TRANSFER RS  LAKHS MOBILE MONEY TRANSFERS RS  LAKHS IN  INSTANCES AND OTHERS RS  LAKHS FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS NILREMITTANCE RS  LAKHS  TOTAL DEBIT SIDE RS  LAKHS OF WHICH CASH IS NIL REMITTANCE RS  LAKHS FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS NILTOTAL DEBIT SIDE RS  LAKHS OF WHICH CASH WAS NILREMITTANCE RS  LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN  DAYS FUNDS OUTLAY MORE THAN  DAYS LOSS ON MTT CYCLE FUNDS OUTLAY OF MORE THAN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE ",{"entities":[(201,229,"FOREIGN ORG")]}),
            ("PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSNIGER SEED FREE HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON  TO TUNE OF USD  WAS RECEIVED FROM SARL RENAUDAT J C LES CHAUMES WHOSES DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN IN AN ANOTHER MTT TRANSACTION VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN  DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN  AND TO TUNE OF USD TO BLACK SEA GROUP LOGISTICS IN RUSSIA DETAILS OF WHICH IS NOT AVAILABE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSCHICKPEAS HSRED MILLET HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON   TO THE TUNE OF USD IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF SEEDS MILLETS MAIZE ETC IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDAGRICOM IMPEXPROPRIETORSHIP FIRM HAVING ADDRESS:F NO AMALTAS APPTRAJ NAGAR NAGPUR MAHARASHTRA INDIA    CONTACT NUMBER: + AND EMAIL ID: AGRICOMIMPEX@HOTMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD NAGPUR  RAMDASPETHAKARSHAN BUSIPLEX  CENTRAL BAZAAR ROAD RAMDASPETH NAGPUR MAHARASHTRA   SINCE  DATE OF INCORPORATION IS   PAN NUMBER OF THE ACCOUNT IS AJRPGD IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS  GSTIN AS PER BANK RECORDS IS AJRPGDZYTRANSACTION LINKED ACCOUNTS ARE   AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKASH GHADSE PROP AJRPGDTHE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN GSTIN DATE OF LAST KYC UPDATION  AGRICOM IMPEX IS A LEADING WHOLESALE MANUFACTURER AND EXPORTER OF AGRICULTURAL OIL SEEDS BIRD SEEDS LIKE YELLOW MILLET SAFFLOWER SEED NIGER SEED SORGHUM SESAME SEEDS CHICK PEAS AND PEANUTS IN INTERNATIONAL MARKET DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAC TO RS CRALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT  WAS RS LAKHS  CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL  YEAR: FY  FROM  TILL  SHOWS TOTAL CREDIT RS  LAKHS OF WHICH  TRANSFERS RS  LAKHS MAJORLY FROM AGRICOM IMPEXSHRI BALAJI INDUSTRIES  CLEARING RS  LAKHS MAJORLY FROM FWD CANC GAIN NEFT RS  LAKHS MAJORLY FROM NARAYANA SAI TRADERSSURGO RESOURCES  RTGS RS  LAKHS MAJORLY FROM TALOJA AGRO FOODS PVT LTDHITASHI AGRI TRADEAGRICOM IMPEX  REMITTANCE RS  LAKHS MAJORLY FROM EEFCEEFC CONVERSIONAGRICOM IMPEX  MOBILE MONEY TRANSFERS RS  LAKHS IN  INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS   LAKHS OF WHICH CASH WAS RS  LAKHS BY BRANCH  TRANSFERS RS  LAKHS MAJORLY TO BHAIYAJI RAMBHAUJI ROKDE JEWELLERSICICI PRUD LIFE INS CO LTDCASH COLLNAGRICOM IMPEX  CLEARING RS  LAKHS MAJORLY TO PANKAJ GULATI AND ASSOCIAM S E D C LV P SALES AND SERVICES  NEFT RS  LAKHS MAJORLY TO B NARAYANATHAPARSONS K V SHIPPGIDDDIAH TRADERSRTGS RS  LAKHS MAJORLY TO SHREE BHAGAWATI AGRO PNARAYAN SAI TRADERMILAP TRADING  REMITTANCE RS  LAKHS MAJORLY TO GENERAL COMMODITIES LTDSPRINGHAUS LLCKAMAL LLC  INTERNET FUND TRANSFER RS  LAKHS  MOBILE MONEY TRANSFERS RS  LAKHS IN  INSTANCES FURTHER THE QUANTUM OF    TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE:    FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS NIL  REMITTANCE RS  LAKHS TOTAL DEBIT SIDE RS  LAKHS OF WHICH CASH WAS RS  LAKHS  REMITTANCE RS  LAKHS FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS NILREMITTANCE RS  LAKHS TOTAL DEBIT SIDE RS  LAKHS OF WHICH  CASH WAS RS  LAKHS REMITTANCE RS  LAKHS FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH  CASH WAS RS  LAKHS REMITTANCE RS  LAKHS TOTAL DEBIT SIDE RS  LAKHS OF WHICH  CASH WAS RS  LAKHS REMITTANCE RS  LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN  DAYS LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD LACS FOR EXAMPLE VIOLATION OF LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON  USD  WAS DEBITED TO KAMAL GLOBAL TRADING PTE LTD IN SINGAPORE WHOSE DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN WITH",{"entities":[(259,273,"FOREIGN ORG"),(497,512,"FOREIGN ORG"),(4329,4357,"FOREIGN ORG")]}),
            ("STANDARD CCENTRAL BOGE POWER I REMITTANCE RS  LAKHS MAJORLY TO JINDAL STEEL AND POWER LIMITEDATI FLAT ROLLED PRODUCTSKANEMATSU CORPORATION INTERNET FUND TRANSFER RS  LAKHS  AND OTHERS RS  LAKHSFY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS RS  LAKHS REMITTANCE RS  LAKHS  TOTAL DEBIT SIDE RS  LAKHS OF WHICH CASH WAS NILREMITTANCE RS  LAKHS FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS RS  LAKHS REMITTANCE RS  LAKHS  TOTAL DEBIT SIDE RS  LAKHS OF WHICH CASH WAS NILREMITTANCE RS  LAKHS FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS RS  LAKHS REMITTANCE RS  LAKHS  TOTAL DEBIT SIDE RS  LAKHS OF WHICH CASH WAS NIL REMITTANCE RS  LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN  DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR  TRANSACTIONS FOR THE YEAR ENDING JUNE  OF THE VALUE OF USD LACS EUR LACS CNY LACS FOR EXAMPLE VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN  DAYS IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON ON  USD  WAS REMITTED TO VICENTE BERRIZBEITIA S L CLIENT IS INTO SUPPLY OF BOLTING MATERIALS HTTP://BERRIZBEITIACOM/EN/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSTUBES AND PIPES HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE IN AN ANOTHER MTT TRANSACTION VIOLATION OF FUNDS OUTLAY MORE THAN  DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT ON  TO TUNE OF USDLACS TO GE HYDRO CHINA CO LTDENTITY IS JV WITH  YALONG RIVER HYDROPOWER DEVELOPMENT CO LTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSHYDRO SET HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE IN TOTAL  DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD LACS EUR LACS CNY LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONSCLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNTSTR IS BEING FILEDGE POWER INDIA LIMITEDPUBLIC LTD HAVING ADDRESS:T  T PLOT  TO JAYPEE WISHTOWN SECTOR AXIS HOUSE NOIDA UTTAR PRADESH INDIA  CONTACT NUMBER: + AND EMAIL ID: CHANDNISISODIA@POWERALSTOMCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER  WITH ICICI BANK LTD NEW DELHI  CONNAUGHT PLACEA PHELPS BUILDING CONNAUGHT PLACENEW DELHI DELHI  SINCE  DATE OF INCORPORATION IS  PAN NUMBER OF THE ACCOUNT IS AABCAF IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS  TRANSACTION LINKED ACCOUNTS ARE   AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE P JAYALAKSHMI AUTHORISED SIGNATORY AAEPPB AMARESH SINGH AUTHORISED SIGNATORY AICPSL RAJESH AGGARWAL AUTHORISED SIGNATORY AGIPAC HIREN RAJNIKANT VYAS AUTHORISED SIGNATORY ABTPVP NAVEEN SHANKARAN AUTHORISED SIGNATORY AIOPNM THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AABCAFMOAAOA DATE OF LAST KYC UPDATION  GE POWER INDIA LIMITED GPIL FORMERLY KNOWN AS ALSTOM INDIA LIMITED IS ENGAGED IN ENGINEERING PROCUREMENT MANUFACTURING CONSTRUCTION AND SERVICING OF POWER PLANTS AND POWER EQUIPMENT IN BOTH THERMAL AND HYDRO SEGMENTS THE COMPANY DESIGNS MANUFACTURE AND DELIVERS TURNKEY POWER PLANTS POWER GENERATION EQUIPMENT AND AIR QUALITY CONTROL SYSTEMS AND SERVICES AND ALSO PROVIDES RENEWABLE POWER GENERATION SOLUTIONS FOR INTEGRATED POWER PLANTS COVERING HYDROELECTRICITY WIND GEOTHERMAL BIOMASS SOLAR AND WAVE AND TIDAL STREAM ENERGIES IT ALSO OFFERS INDIVIDUAL COMPONENTS FOR A RANGE OF TURBINES AND GENERATORS AS WELL AS VARIOUS SERVICES INCLUDING PLANT MODERNIZATION MAINTENANCE AND OPERATIONAL SUPPORT AND PROVIDES PRODUCTS AND SYSTEMS FOR ELECTRICAL UTILITIES AND INDUSTRIESALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT  WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY  FROM  TILL  SHOWS TOTAL CREDIT RS  LAKHS OF WHICH RS  BY CASH NEFT RS  LAKHS MAJORLY FROM LALITPUR POWER GENERATION COTORRENT POWER LIMITED RTGS RS  LAKHS MAJORLY FROM BHARAT HEAVY ELECTRICAL TOTAL DEBITS DURING THE SAME PERIOD WAS RS  LAKHS OF WHICH RS  BY CASH RTGS RS  LAKHS MAJORLY TO GE POWER I REMITTANCE RS  LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS  YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY  FROM  TO  : TOTAL CREDIT SIDE RS  LAKHS OF WHICH CASH WAS RS  LAKHS MAJORLY IN GULBARGA BRANCH TRANSFERS RS  LAKHS MAJORLY FROM GE POWER INDIA LTD TELENGANA FGD PROJECT  ESCROW ACCOUNTGE POWER INDIA LTDIGSTPP JHAJJAR FGD PROJECT  ESCROW ACCOUNTGE POWER INDIA LTDSIPAT FGD PROJECT  ESCROW ACCOUNT CLEARING RS  LAKHS MAJORLY FROM LT MHPS BOILERS PRIVATE LIMITNTPC LIMITEDVEDANTA LIMITED NEFT RS  LAKHS MAJORLY FROM GENERAL ELECTRIC COMPANYNHPC LIMITEDMEJA URJA NIGAM PVT LTD RTGS RS  LAKHS MAJORLY FROM M/S DOOSAN POWER SYSTEBHARAT HEAVY ELECTRICAGE POWER INDIA LIMITED REMITTANCE RS  LAKHS MAJORLY FROM ARAVALI POWER COMPANY PRIVATE LTDGENERAL ELECTRIC SWITZERLANNTPC LIMITED INTERNET FUND TRANSFERS RS  LAKHS MOBILE MONEY TRANSFERS RS  LAKHS IN  INSTANCES  UPI TRANSFERS RS  LAKHS IN  INSTANCE AND OTHERS RS  LAKHS TOTAL DEBIT SIDE RS  LAKHS OF WHICH TRANSFERS RS  LAKHS MAJORLY TO GE POWER INDIA LTD UNPAID DIVIDEND ACCOUNT  GE POWER INDIA LIMITEDGE POWER SYSTEMS INDIA PVT LTD CLEARING RS  LAKHS NEFT RS  LAKHS MAJORLY TO GE POWER INDIA LIMITCENTRAL EXCISE SERVIWEST BENGAL STATE EL RTGS RS  LAKHS MAJORLY",{"entities":[(0,0,"FOREIGN ORG"),(0,0,"FOREIGN ORG")]}),
            ("ALSO ON RS LACS GOT CREDITED BY INWARD REMITTANCE FROM WORLD FIRST ASIA LTD HONG KONG WITH INFORMATION AS IMPORT OF GARMENTS AND ON RS LACS GOT DEBITED BY CMS CLEARING WITH REMARK AS SDI SD INTERNATIONAL IN OTHER EXAMPLE ON RS LACS WAS CREDITED THROUGH RTGS FROM SD INTERNATIONAL ACC IFSCINDB AND ON THE SAME DAY RS LACS ARE IMMEDIATELY DEPLOYED THROUGH CMS CLEARING WITH REMARK AS SDI SD INTERNATIONAL SIMILAR SUCH TRANSACTIONS ARE OBSERVED THROUGHOUT THE ACCOUNT FOR WHICH NO RATIONAL FOUND TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF LESS THAN RS LACS IN VIEW OF INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT ROUTING OF HIGH VALUE FUNDS AND HIGH TURNOVER STR IS BEING FILEDSD INTERNATIONAL PROPRIETORSHIP FIRM HAVING ADDRESS ND FLOOR FLR /B NIMTOLA GHAT STREET NEAR JORABAGAN MORE KOLKATA WEST BENGAL INDIA CONTACT NUMBER AND EMAIL ID BANKS SDWORLDIN IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD KOLKATA BURRA BAZAR ICICI BANK LTD P KALAKAR STREET BURRABAZAR KOLKATA WEST BENGAL SINCE DATE OF INCORPORATION IS RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AFYPDH OTHER RELATIONSHIP OF CUSTOMER ARE TRANSACTION LINKED ACCOUNTS ARE AND AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE SANJOY DEY PROPRIETOR AFYPDH THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AFYPDH AADHAR DATE OF LAST KYC UPDATION KYC UPDATION IS NOT DUE ENTITY IS INTO BUSINESS OF KIDS GARMENTS AND WOMEN SUIT PCS SET TRADERS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS LACS ALERT WAS TRIGGERED FOR INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY FROM S D INTERNATIONAL CLEARING RS LAKHS MAJORLY FROM S D S D TRADE IMPEX PRIVATE LIMITE CMS RTGS RS LAKHS MAJORLY FROM S D INTERNATIONAL SDINTERNATIONAL REMITTANCE RS LAKHS MAJORLY FROM ADVANTAGE GENERAL TRADING LIMITED COSMIC EDGE GENERAL TRADING LIMITED LIGHTWAYS GENERAL TRADING LLC TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH TRANSFERS RS LAKHS MAJORLY TO HETAL SYNTEX AMIT ROADLINES S D INTERNATIONAL CLEARING RS LAKHS MAJORL TO SDI RTGS SDI RTGS SDI TRF RTGS RS LAKHS MAJORLY TO NITFUL COMMERCIAL PVT LAXMIWAN SUPPLIERS PVT SURAKSHIT HOUSING ADVI FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TRANSFERS RS LAKHS RTGS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS CLEARING RS LAKHS RTGS RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH RS BY CASH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS AS PER THE TRANSACTION PATTERN THERE ARE FREQUENT HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS FOR EXAMPLE ON RS LACS WAS CREDITED THROUGH RTGS FROM S D INTERNATIONAL ACCOUNT NO BANK NAMEFEDERAL BANK IFSC CODE ICIC AND ON RS LACS WAS DEPLOYED THROUGH RTGS TOWARDS S D INTERNATIONAL ACC IFSCINDB AND THE REST OF THE FUNDS WERE DEPLOYED THROUGH CMS WITH REMARK AS SDI SD INTERNATIONAL",{"entities":[(55 ,75,"FOREIGN ORG"),(2283,2315,"FOREIGN ORG"),(2317,2351,"FOREIGN ORG"),(2353,2382,"FOREIGN ORG")]}),
            ("MY ENG NEWS PRESS BOSTON MEGA TEXTILE LLC IN KARAKALPAKSTANHTML WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSCOTTON YARN HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN AND TO THE TUNE OF USD LACS FROM CHENG CHIH TRANSCEND ENTERPRISE CORPORATION CLIENT DEAAL SIN TRADING OF CONTTON YARNSHTTPS //PANJIVACOM/CHENGCHIHTRANSCENDENTERPRISE/ IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDSQUARE CORPORATION PROPRIETORSHIP FIRM HAVING ADDRESS C BRS NAGARLUDHIANAPUNJAB LUDHIANA PUNJAB INDIA CONTACT NUMBER AND EMAIL ID ACCOUNTS SQUARECORPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD LUDHIANA FEROZE GANDHI MARKETNEHRU SIDHANT KENDER TRUST BUILDING FEROZ GANDHI MARKET LUDHIANA PUNJAB SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAOPVG IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS GSTIN AS PER BANK RECORDS IS AAOPVGZF TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VIVEK VERMA PROP RAHUL TALWAR AUTHORISED SIGNATORY ADBPTB VANDANA VERMA AUTHORISED SIGNATORY AADPVM THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE DATE OF LAST KYC UPDATION CUSTOMER IS INTO YARN BUSINESS HTTP //WWWSQUARECORPCOM/ DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS EQUAL TO CR ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS DEBIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY FROM PROMINENT EXPORTS INTERNATIONAL REMITTANCE RS LAKHS MAJORLY FROM THERMAX YARN DYED FABRICS LIMITEDEVINCE TEXTILES LTDMEGHNA KNIT COMPOSITE LIMITED TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS MAJORLY TO WEIQIAO TEXTILE COMPANY LIMITEDWEIHAI WEIQIAO INDUSTRIAL PARK COMIF INTERNATIONAL LIMITEDFY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHSTOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NILREMITTANCE RS LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN DAYS LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF USLACS FOR EXAMPLE VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN TO USD LACS REMITTED TO IRS UNIVERSAL PTE LTDIRS UNIVERSAL PTE LTD ISIN INTERNATIONAL PTE LTD WAS FOUNDED IN THE COMPANYS LINE OF BUSINESS INCLUDES THE WHOLESALE DISTRIBUTION OF NONDURABLE GOODSHTTPS //WWWBLOOMBERGCOM/PROFILE/COMPANY/Z SP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HSYARN HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT IS DUE IN AN ANOTHER MTT TRANSACTION VIOLATION OF MTT CYCLE NOT COMPLETED IN DAYS WHERE OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN TO TO TUNE OF USDLACS TO BOSTON MEGA TEKSTIL LLC ENTITY IS INTO MANUFACTURE OF TEXTILE",{"entities":[(25,41,"FOREIGN ORG"),(2110,2142,"FOREIGN ORG"),(2143,2161,"FOREIGN ORG"),(2258,2288,"FOREIGN ORG"),(2296,2346,"FOREIGN ORG"),(3154,3174,"FOREIGN ORG"),(3175,3196,"FOREIGN ORG"),(3736,3759,"FOREIGN ORG")]}),
            ("HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED ON TO TUNE OF USD FROM MUNDRA GENERAL TRADING FZE IN UAEDEALS IN METAL SCRAPHTTPS //PANJIVACOM/MUNDRAGENERALTRADINGFZE/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE IN AN ANOTHER MTT TRANSACTION VIOLATION OF FUNDS OUTLAY OF MORE THAN DAYS OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON USD TO TSAI SHEN TRADING LIMITED IN HONGKONGCLIENT IS INTO CERAMIC AND AGRO PRODUCTS RELATED HTTPS //TSAISHENTRADINGCOM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS STORAGE DEVICES THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED USD RECEIVED ON FROM MUNDRA GENERAL TRADING FZE IN UAEDEALS IN METAL SCRAPHTTPS //PANJIVACOM/MUNDRAGENERALTRADINGFZE/ IN TOTAL DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD CRORES WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT STR IS BEING FILEDKOTHARI PRODUCTS LIMITEDPRIVATE LTD HAVING ADDRESS / THE MALL KANPURUTTAR PRADESHINDIA CONTACT NUMBER AND EMAIL ID KOTHARILW HOTMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD KANPUR/ JSTOWERS THE MALL KANPUR UTTAR PRADESH SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAACKF IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS THERE ARE NO LINKED ACCOUNT AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE DEEPAK KOTHARI DIRECTOR AAYPKC MITESH KOTHARI DIRECTOR AFUPKM VIJAY KUMAR MODI DIRECTOR ACVPMF SAURABH BHATNAGAR AUTHORISED SIGNATORY AHQPBK MITESH KOTHARI AUTHORISED SIGNATORY AGUPKM BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK KOTHARIAAYPKC THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN MOA AOA COI IEC DATE OF LAST KYC UPDATION ENTITY IS PRIMARILY ENGAGED IN THE INTERNATIONAL TRADING OF COMMODITIES IT IS A PART OF KOTHARI GROUP KPL ALSO EARNS REVENUE FROM REAL ESTATE THROUGH VARIOUS COMMERCIAL PROPERTIES OWNED BY ITS SUBSIDIARIES THE COMPANY IS ALSO ENGAGED IN IMPORT EXPORT TRADE OF VARIOUS PRODUCTS/COMMODITIES INCLUDING AGRO BASED COMMODITIES MINERALS METALS PETROLEUM PRODUCTS AND COAL TILES NOTEBOOK COMPUTER STORAGE DEVICES COMPUTER HARDWARE ELECTRONIC COMPONENTS TRANSFORMERS COPIER PAPER STEEL SCRAP AND PVC THE COMPANY WAS INCORPORATED IN AND HAS ITS REGISTERED OFFICE LOCATED IN KANPUR UTTAR PRADESHALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH RTGS RS LAKHS MAJORLY FROM KOTHARI PRODUCTS LIMITKOTHARI PRODUCTS LIMKPL EXPORTS LIMITED REMITTANCE RS LAKHS MAJORLY FROM MUNDRA GENERAL TRADING F Z EHIMALAYAN GENERAL TRADING FZEMUNDRA GENERAL TRADING FZEOFFICE NUMBER TOTAL DEBIT SIDE RS LAKHS OF WHICH RTGS RS LAKHS MAJORLY TO KOTHARI PRKPL EXPORT REMITTANCE RS LAKHS MAJORLY TO KINGSTON PTE LTDTSAI SHEN TRADING LIMITEDKINGSTON PTE LTD AND OTHERS RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS TOTAL DEBIT SIDE RS LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE FUNDS OUTLAY OF MORE THAN DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR TRANSACTIONS FOR THE YEAR ENDING JUNE OF THE VALUE OF USD CRORES FOR EXAMPLE VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN DAYS IN THE TRANSACTION WHERE THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON USD LACS REMITTED TOTSAI SHEN TRADING LIMITED IN HONGKONGCLIENT IS INTO CERAMIC AND AGRO PRODUCTS RELATED HTTPS //TSAISHENTRADINGCOM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS STORAGE DEVICE",{"entities":[(86,112,"FOREIGN ORG"),(158,181,"FOREIGN ORG"),(433,461,"FOREIGN ORG"),(525,545,"FOREIGN ORG"),(783,809,"FOREIGN ORG"),(855,878,"FOREIGN ORG"),(3067,3093,"FOREIGN ORG"),(3094,3123,"FOREIGN ORG"),(3124,3149,"FOREIGN ORG"),(3276,3291,"FOREIGN ORG"),(3292,3316,"FOREIGN ORG"),(3317,3333,"FOREIGN ORG"),(3899,3924,"FOREIGN ORG"),(3991,4012,"FOREIGN ORG")]}),
            ("BITEX TECHNOLOGIES INDIA PRIVATE LIMITED A PRIVATE LTD COMPANY HAVING ADDRESS OFF KANAKIA WALL B WING STREET CHAKALA SIGNAL ANDHERI EAST OPP CENTRAL PARK MUMBAI MAHARASHTRA INDIA CONTACT NUMBER AND EMAIL ID BITEXINDIA GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD SAGAR AVENUE GROUND FLOOR OPP SHOPPERS STOP SV ROAD ANDHERI W MUMBAI MAHARASHTRA SINCE DATE OF INCORPORATION IS PAN OF THE ACCOUNT IS AAJCBN GSTIN AS PER BANK RECORDS IS AAJCBNZA OTHER RELATIONSHIP OF CUSTOMER ARE TRANSACTION LINKED ACCOUNT IS BENEFICIAL OWNER OF THE REPORTED ACCOUNT ARE CHIRAG ASHVINBHAI RUPARELIYA WITH PAN NO BLTPRH MONARK PARESHBHAI MODI AUTHORISED SIGNATORY/ DIRECTOR WITH PAN NO CLAPMM THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION COI AAJCBNZA AS PER ENHANCED DUE DILIGENCE CUSTOMER IS INTO PROVIDING SOFTWARE SERVICES RELATED TO TRADING INTO CRYPTO CURRENCIES HENCE RECOMMENDING ACCOUNT FOR CLOSURE REFER WEBSITE WWWBITEXCOM BITEX TECH IN INDIA COLLECTS PAYMENTS FROM DIFFERENT USERS INTERESTED TO DEAL IN CRYPTOCURRENCY TRADING AND COLLECTS PAYMENTS FOR ACCOUNT MAINTAINED WITH KOTAK BANK AND DOES OWN REMITTANCE TRANSACTION WITH ICICI BANK INITIALLY THEY HAVE CONFIRMED DEALING INTO SOFTWARE SERVICES BUT ON DEEPER INVESTIGATION CAME TO KNOW OF TRADING IN CRYPTO CURRENCY DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT WAS NIL QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH CLEARING RS LAKHS MAJORLY FROM BITEX TECHNOLOGIES NEFT RS LAKHS MAJORLY FROM YBL SIGNET PAYMENT S BITEX TECHNOLOGIES INDIA PVT LTD RTGS RS LAKHS MAJORLY FROM BITEX TECHNOLOGIES INDUSTRIES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH RTGS RS LAKHS TO BITEX TECHNOLOGIES INDIA PVT LTD REMITTANCE RS LAKHS MAJORLY TO NICHOLAS LEVENSTEIN AND COMPANY TRADE BIT COMMERCIAL BROKERS LLC TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE ON RS LACS WERE CREDITED THROUGH NEFT FROM YBL SIGNET PAYMENTS AND SAME WAS DEBITED THROUGH RTGS TO BITEX TECHNOLOGIES INDIA PVT LTD YESB ON ALSO ON RS WERE CREDITED THROUGH NEFT FROM YBL SIGNET PAYMENTS YESB AND ON THE NEXT DAY SAME AMOUNT WAS DEBITED THROUGH RTGS TO SIGNET PAYMENT SOLUTIONS PRIVATE LIMITED AUBL ROUTING OF FUNDS OBSERVED SIMILAR PATTERN WERE OBSERVED THROUGHOUT THE PERIOD IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS WHERE ROUTING OF FUNDS OBSERVED STR IS BEING FILED",{"entities":[(2057 ,2088,"FOREIGN ORG"),(2089,2122,"FOREIGN ORG")]}),
            ("TRANSACTION PATTERN SHOWS FUNDS ARE RECEIVED FROM DIFFERENT ENTITIES AND ARE IMMEDIATELY DEBITED TOWARDS OUTWARD REMITTANCE FOR EG ON ACCOUNT WAS CREDITED WITH RS CRORES IN PARTS THROUGH RTGS FROM SHREEJI JEWELLERS ACCOUNT NO IFSC CODE UTIB BANK AXIS BANK AND THE SAME IS DEBITED IN PARTS THROUGH RTGS TO DIAMAX ACCOUNT NO IFSC CODE INDB BANK INDUSIND BANK LTD SIMILARLY ON ACCOUNT WAS CREDITED WITH RS CRORES IN PARTS THROUGH RTGS FROM DIAMAX ACCOUNT NO IFSC CODE HDFC PURVI EXPORTS ACCOUNT NO IFSC CODE KKBK VANSH GEMS ACCOUNT NO IFSC CODE KKBK AND ON THE SAME DATE ACCOUNT WAS DEBITED WITH RS CRORES APPROX IN PARTS THROUGH REMITTANCE TO MARS VENTURE LIMITED HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT FUNDS WERE CREDITED IN THE ACCOUNT FROM MAHAPUJA PRODUCTS LTD NYSNA MULTIVENTURES PVT LTD SHREEJI JEWELLERS ETC AND MAJORITY REMITTANCES MADE TO MARS VENTURE LIMITED SINCE ACCOUNT OPENING OVERSEAS ENTITIES EXISTENCE CHECKED HOWEVER NO FURTHER INFORMATION AVAILABLE ON THEM REGARDING WHAT PRODUCT THEY SELL NO BUSINESS RELATED PAYMENTS ARE MADE FROM THIS ACCOUNT IN VIEW OF LARGE VALUE FOREX DEBIT DAILY AND NEGATIVE FINDINGS STR IS BEING FILEDDIAMAX A PROPRIETORSHIP FIRM HAVING ADDRESS / DAIMOND APPARTMENT SAIYEDPURA NAGORIVAD SURAT GUJARAT INDIA CONTACT NUMBER AND EMAIL ID DIAMAX GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD SHUBHAM POINT RUWALA TEKRA HARIPURA BAGAL LAL GATE SURAT GUJARAT SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS FOJPSR IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS GSTIN AS PER BANK RECORDS IS FOJPSRZQ THERE ARE NO PAN LINKED ACCOUNTS TRANSACTION LINKED ACCOUNT IS AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS DHRUMIL DHARNENDRA SHAH PAN FOJPSR THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN FOJPSR AADHAAR OF PROPRIETOR IMPORT EXPORT CERTIFICATE AND GST REGISTRATION CERTIFICATE CUSTOMER IS AN IMPORTER OF LAB GROWN THAT IS SYNTHETIC DIAMONDS AND FURTHER HE IS SELLING IN LOCAL MARKET AS PER ENHANCED DUE DILIGENCE EDD NAME BOARD AFFIXED OUTSIDE OFFICE WHICH IS PRINTED ON A SIZE PAPER ALSO THERE ARE CREDITS FROM NYSNA MULTIVENTRES PVT LTD SEEN AND IS ONE SUCH COMPANY WHICH HAS TRANSACTED WITH MANY SUSPICIOUS DIAMOND COMPANIES OF SURAT AND WHOSE ACCOUNTS ARE CLOSED UNDER BANK INDUCED PROCESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS CRORES TO LESS THAN RS CRORES ALERT WAS TRIGGERED FOR LARGE VALUE FOREX DEBIT DAILY BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH CLEARING RS LAKHS MAJORLY FROM RAHIL GEM NEFT RS LAKHS MAJORLY FROM SAHAJ DIAM INTEGRITY JEWELS RTGS RS LAKHS MAJORLY FROM DIAMAX VANSH GEMS SHREEJI JEWELLERS MOBILE MONEY TRANSFERS RS LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH RTGS RS LAKHS MAJORLY TO PARI IMPEX RADHE KRISHNA IMPEX DIAMAX REMITTANCE RS LAKHS MAJORLY TO MARS VENTURE LIMITED HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT AARAS DIAMONDS LLC ALICAT SCIENTIFIC INC IN USD TO UNITED STATES FOR ADV PAYMENT AGAINST IMPORT OTHER THAN NEPAL BHUTAN NEFT RS LAKHS TO MAYAN EXPORTS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CLEARING RS LAKHS FROM DIAMAX NEFT RS LAKHS FROM SHREE RAM DIAMOND RTGS RS LAKHS FROM NYSNA MULTIVENTURES PVT LTD SHREEJI JEWELLERS INTERNET FUND TRANSFERS RS LAKHS FROM MAHAPUJA PRODUCTS LTD MOBILE MONEY TRANSFERS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TO SURAT BHAGAL BRANCH RTGS RS LAKHS REMITTANCE RS LAKHS TO MARS VENTURE LIMITED HONGKONG IN USD CURRENCY WITH PURPOSE PAYMENT TOWARDS IMPORT SETTLEMENT",{"entities":[(641,661,"FOREIGN ORG"),(879,900,"FOREIGN ORG"),(3085,3105,"FOREIGN ORG"),(3178,3196,"FOREIGN ORG"),(3197,3218,"FOREIGN ORG"),(3821,3841,"FOREIGN ORG")]}),
            ("ROUTING OF LARGE VOLUME OF FUNDS WERE OBSERVED IN ACCOUNT LEADING TO HIGH TURNOVER WHICH SEEMS UNUSUAL IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS STR IS BEING FILEDWANDERLAND REAL ESTATE PRIVATE LIMITED SMEAGPRIVATE LTD COMPANY HAVING ADDRESS:TH FLR TREASURE ISLAND TUKOGANJ MAINRD INDORE MADHYAPRADESH INDIA CONTACT NUMBER: + IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD INDORE MALAV PARISAR ICICI BANK LTD CHHOTI KHAJRANI MALAV PARISAR INDORE INDORE MADHYA PRADESH SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAACWF OTHER RELATIONSHIP OF CUSTOMER ARE TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VINAYAK KALANI AUTHORISED SIGNATORY ATGPKP QURESH YUSUF MATKAWALA AUTHORISED SIGNATORY AGSPMA YOGESH GOSWAMI AUTHORISED SIGNATORY ACFPGK NIRMAL KUMAR SHRIVASTAVA DIRECTOR KAPIL GOPLANI DIRECTOR BSEPGN THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAACWF MOAAOA BOARD RESOLUTION ETC KYC UPDATION IS NOT DUE AS PER ENHANCED DUE DILIGENCE: CUSTOMER DEALS IN EXPORT OF PP BAGS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS CR TO LESS THAN CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH TRANSFERS RS LAKH MAJORLY FROM WANDERLAND REAL ESTATES PVT LTD CLEARING RS LAKHS MAJORLY FROM NEFTWANDERLAND REAL ESTATE PRIVATECMS_ NEFT RS LAKHS MAJORLY FROM TATWA PLASTCO EXTRUSIONS PVTMAERSK LINE INDIA PRIVATE LI REMITTANCE RS LAKHS MAJORLY FROM FLEXI GLOBAL PTE LTD A BEPANKTI INTERNATIONAL PTE LTDDHAMPUR INTERNATIONAL PTE LT INTERNET FUND TRANSFERS RS LAKHS AND OTHERS RS LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO WANDERLAND REAL ESTATE PRIVATE LIMITED CLEARING RS LAKHS RTGS RS LAKHS MAJORLY TO FLEXITUFF VENTURES INDFLEXITUFF VENTURES INTSBIN REMITTANCE RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES AND OTHERS RS LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY FROM TO : TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NIL TRANSACTION PATTERN SHOWS FUNDS WERE CREDITED THROUGH INWARD REMITTANCES AND THOSE FUNDS WERE DEPLOYED TOWARDS OTHER ENTITIES FOR EXAMPLE ON RS LAKHS CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE LTD SHENTON WAY SHENTON HOUSE SINGAPORE WHERE THE PURPOSE MENTIONED AS PAYMENT OF IMPORT BILL AND ON RS LAKHS DEBITED IN PARTS THROUGH RTGS TO FLEXITUFF VENTURES INTERNATIONAL LTD ACCOUNT NO IFSC SBIN ALSO ON RS LAKHS CREDITED THROUGH TRANSFER FROM NANOFIL TECHNOLOGIES PRIVATE LIMITED WHERE THOSE FUNDS WERE CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE LTD SHENTON WAY SHENTON HOUSE SINGAPORE WHERE THE PURPOSE MENTIONED AS EXPORT BILLS REALN COLLN AND ON RS CRORES DEBITED IN PARTS THROUGH CMS TRANSFER AS PAYMENTS ALSO ON RS LACS WERE CREDITED BY REMITTANCE FROM FLEXI GLOBAL PTE LTD A BE SINGAPORE AND ON SAME DAY ENTIRE AMOUNT WAS DEBITED IN PARTS BY RTGS TOWARDS FLEXITUFF VENTURES INTERNATIONAL LTD ACCOUNT NO IFSC SBIN",{"entities":[(1737, 1757,"FOREIGN ORG"),(1762,1789,"FOREIGN ORG"),(1790,1818,"FOREIGN ORG"),(3195,3215,"FOREIGN ORG"),(2583,2611,"FOREIGN ORG"),(2958,2986,"FOREIGN ORG")]}),
            ("SHIPMENT MADE EXPORT BILLS ARE PENDING FOR THE SUBMISSION OUT OF WHICH BILLS WERE OVERDUE AND IS IN CAUTIONED LISTED FOR MORE THAN TWO YEARS SEEMS SUSPICIOUS IN VIEW OF BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE STR IS BEING REPORTEDKGR RIGS AND MINING EQUIPMENT PARTNERSHIP HAVING ADDRESS PLOTNO / / IDA PHASEII CHERLAPALLY HCL POST RANGAREDDI HYDERABAD TELANGANA INDIA CONTACT NUMBER AND EMAIL ID GOVINDREDDY K YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD PLOT NO HNO / SUBHASH NAGAR OPP JUNA NAGAR KUSHAIGUDA KAPRA RANGAREDDY DIST HYDERABAD TELANGANA SINCE DATE OF INCORPORATION IS NA PAN NUMBER OF THE ACCOUNT IS AAHFKN OTHER RELATIONSHIP OF CUSTOMER ARE AND TRANSACTION LINKED ACCOUNTS ARE AND AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE GOVINDAREDDY KATIREDDY PARTNER ADFPKK AND KATIREDDY NITHIN REDDY PARTNER DECPKF THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PARTNERSHIP DEED AND PAN DATE OF LAST KYC UPDATION KYC UPDATION IS NOT DUEENTITY IS A MANUFACTURER AND TRADER OF DRILLING EQUIPMENT SUCH AS DRILLING RIGS DTH HAMMERS BUTTON BITS ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS CRORES TO LESS THAN CRORESALERT WAS TRIGGERED FOR MANUAL STR FOR BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE BALANCE IN THE ACCOUNT AT WAS RS LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH CASH WAS RS LAKHS TRANSFERS RS LAKHS MAJORLY FROM KGR RIGS MINING EQUIPMENT KGR INDUSTRIES KATIREDDY GOVINDA REDDY CLEARING RS LAKHS MAJORLY FROM BIMIDI MAHENDER LAXMI NARASIMHA RAO NEFT RS LAKHS MAJORLY FROM COMINT SYSTEMS AND SOLUTIONS EPAO GST REFUNDS MASTER DRILLERS RTGS RS LAKHS MAJORLY FROM E PAO GST REFUNDS BHAVIKA PLAS TEK PRIVA NANO POLYMERS INDIA PVT LTD REMITTANCE RS LAKHS MAJORLY FROM BAKORI MEGA SERVICES ROYAL TECHNO INDUSTRIES LIMITED PROACTIVE ENERGY TRADING INTERNET FUND TRANSFERS RS LAKHS FROM KGR INDUCTION DRILL PIPES COMPANY MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES UPI TRANSFERS RS LAKHS IN INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKHS BY KAPRA BRAN CH TRANSFERS RS LAKHS MAJORLY TO SEKO BEC PRIVATE LIMITED KGR RIGS MINING EQUIPMENT GROC MINING AND CONSTRUCTION PRIVATE LIMITED CLEARING RS LAKHS MAJORLY TO ELGI EQUIPMENTS LIMITED TSSPDCL NEFT RS LAKHS MAJORLY TO OSMAN PIPES AND TUBE RTGS RS LAKHS MAJORLY TO ELGI EQUIPMENTS LTD ASHOK LEYLAND LIMITED REMITTANCE RS LAKHS INTERNET FUND TRANSFER RS LAKHS TO SOUTHERN STEELS FORGINGS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES AND OTHERS RS LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NIL FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS NIL FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHSTOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHSAS PER THE TRANSACTION PATTERN FUNDS ARE GETTING CREDITED FROM DOMESTIC AND INTERNATIONAL ENTITIES AND THOSE FUNDS ARE GETTING DEBITED TO CUSTOMER OTHER BANK ACCOUNT OD ACCOUNT IS WITH US AND VARIOUS SUPPLIERS VENDORS AND OPERATIONAL EXPENSES ETC SEEMS INLINE HOWEVER CUSTOMER IS HAVING ADVANCES RECEIVED THERE ARE EXPORT BILLS ARE PENDING FOR SUBMISSION OUT OF WHICH BILLS ARE OVERDUE",{"entities":[( 1945,1964,"FOREIGN ORG"),(1966,1997,"FOREIGN ORG")]}),
            ("ON FUNDS OF RS LACS APPROX CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM SUYOG SYNTHETICSACC NO IFSC UTIB AND DEPLOYED THROUGH RTGS TOWARDS SHREE BALAJI CREATIONACC NO IFSC KKBK SEEMS UNUSUAL RATIONALE FOR SUCH ROUTING OF FUNDS ARE NOT CLEAR ACCOUNT TURN OVER IS TOO HIGH MORE THAN CRORES IS SEEN IN ACCOUNT IN VIEW OF SUSPICIOUS LARGE VALUE NON CASH DEPOSIT TRANSACTIONS AND SUSPECTED ROUTING OF FUNDS STR IS BEING REPORTEDSUYOG SYNTHETICS PROPRIETOR HAVING ADDRESS RAGHUVEER BUSINESSEMPIREAAI MATA CHOWK PARVAT PATIYA SURAT GUJARAT INDIA CONTACT NUMBER AND EMAIL ID JAGDISHGARG TRIBENIGROUPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD SURAT RING ROAD ICICI BANK LTD SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAPPGJ IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS PAN LINKED ACCOUNTS ARE AND TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE JAGDISHPRASAD RAMKISHAN GARG PROPREITOR AAPPGJ THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN CARDAAPPGJ GSTINAAPPGJZE IEC AND AADHAR CARD DATE OF LAST KYC UPDATION HENCE KYC UPDATION IS NOT DUE CUSTOMER IS INTO IMPORTING AND TRADING OF YARN ANNUAL TURN OVER DECLARED BY CUSTOMER IS RS LAC TO CRORE ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROM TILL SHOWS TOTAL CREDIT RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN JUHU SURAT TEXTILE MARKET BRANCH TRANSFERS RS LAKHS MAJORLY FROM RAJ TEXTILE SHREE GANESH TEXTILETRIBENI WORLDWIDE PVT LTD CLEARING RS LAKHS MAJORLY FROM SUMILON BHATHIJI SILK MILLSGEETA RAYON NEFT RS LAKHS MAJORLY FROM PADMAVATI INTERNATIONAL GURUKRUPA RAYON GURU FIBERS RTGS RS LAKHS MAJORLY FROM SUYOG SYNTHETICS R R TRADING CO SAKSHI YARNS PVT LTD REMITTANCE RS LAKHS MAJORLY FROM INDORAMA SYNTHETICS TBK PT PPUNITED RAW MATERIAL PTE LTD INTERNET FUND TRANSFERS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKHS BY BR ANCH TRANSFERS RS LAKHS MAJORLY TO SINGHAL BROTHERS S K KABRA AND COMPANY CLEARING RS LAKHS MAJORLY TO MONIT TRADING PRIVATE LIMS R GROUP KAYAVLON IMPEX PRIVATE LIMITED RTGS RS LAKHS MAJORLY TO DEEP SHIPPING AGENCYSUYOG AXIS CC NIRANJAN SHIPPING AGENCY REMITTANCE RS LAKHS MAJORLY TO FTEX ASIA INTERNATIONAL LIMITED PT INDORAMA POLYCHEM INDONESIA INTERNET FUND TRANSFER RS LAKHS CARD TRANSACTIONS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES AND OTHERS RS LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS ON ANALYZING THE TRANSACTION PATTERN IT HAS BEEN NOTICED THAT THERE ARE HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS FOR EXAMPLE ON FUNDS OF RS LACS APPROX CREDITED IN THE ACCOUNT IN PARTS THROUGH RTGS FROM M/S ULTIMATEACC NO IFSC SCBL NEFT FROM SUYOG SYNTHETICSACC NO IFSC UTIB AND DEPLOYED THROUGH TRANSFER TOWARDS TRIBENI WORLDWIDE PVT LTD AND REMITTANCE TOWARDS F TEX ASIA INTERNATIONAL LIMITED CHINA FOR ADV PAYMENT AGAINST IMP OTR THAN NEPL BHUT AN AND",{"entities":[(2011,2037,"FOREIGN ORG"),(2040,2067,"FOREIGN ORG"),(2512,2543,"FOREIGN ORG"),(2547,2574,"FOREIGN ORG")]}),
            ("ACCOUNT OPENED ON CURRENT STATUS OF THE ACCOUNT IS TRANSACTIONS NOTICED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES CHEQUE DEPOSITS IMPS CREDITS CASH DEPOSITS ETC CREDITS IN THE ACCOUNT VIA VARIOUS MODES ARE UTILISED VIA RTGS/NEFT TO LOCAL PARTIES IMMEDIATELY BANK FILED STR ON FIU BATCH ID RELATED ACCOUNTS ARE VALIDATED VIA UCIC DIRECTOR MR SUSHIL JAIN IS ALSO PROPRIETOR IN THE FIRMS SHANTI OVERSEAS SUNSHINE ENTERPRISES DIRECTOR MRRISHABH JAIN IS ONE OF THE PROPRIETOR/DIRECTOR IN THE BULK CIRCLE LLP SHALINI TRADEWELL INDIA PVT LTDINR GBPEUR USD SHALINI ENTERPRISES RELATED ACCOUNT DETAILS SHANTI OVERSEAS OPENED ACCOUNT WITH US ON CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS TOTAL CREDIT TURNOVER AS ON DATE IN THE ACCOUNT IS RSCRORESAVAILABLE BALANCE IN THE ACCOUNT IS RSTRANSACTIONS NOTICED ARE HIGH VALUE CREDITS FROM E PAO GST REFUNDSDUTY DBK DISBURSAL ACCOUNT RTGS/NEFT FROM LOCAL PARTIES INWARD REMITTANCES ARE UTILISED IMMEDIATELY BANK FILED STR ON VIA FIU BATCH ID SUNSHINE ENTERPRISES ACCOUNT OPENED ON CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS AVAILABLE BALANCE IN THE ACCOUNT IS RS CREDIT BANK FILED STR ON VIA FIU BATCH ID BULK CIRCLE LLP ACCOUNT OPENED ON CURRENT STATUS OF THE ACCOUNT CLOSED ON SHALINI TRADEWELL INDIA PVT LTDINR GBP EUR USDACCOUNT OPENED ON CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS AS ON TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RSCRORES AVAILABLE BALANCE IN THE ACCOUNT IS RS CREDITTRANSACTIONS OBSERVED ARE HIGH VALUE CREDITS VIA E PAO GST REFUNDS INWARD REMITTANCES UTILISED TO MAKE IMMEDIATELY RTGS TO LOCAL PARTIES DESPITE OF VOLUMES CLIENT IS HOLDING VERY MINIMAL BALANCE IN THE ACCOUNT ACCOUNT WAS UTILISED TO ROUTING FUNDS IMMEDIATELY TO OTHER LOCAL PARTIES KEEPING MINIMAL BALANCE IN THE ACCOUNT BRINGS SUSPICION IN THE ACCOUNT PATTERN SHALINI ENTERPRISES ACCOUNT OPENED WITH US ON CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS AS ON TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RSCRORES AVAILABLE BALANCE IN THE ACCOUNT IS RS CREDIT BANK FILED STR ON VIA FIU BATCH ID HIGH VALUE CASH DEPOSITS AMOUNTING TO INR CRORES WERE EVIDENCED SINCE TH NOV IN THE ACCOUNT NO OTHER CREDITS OTHER THAN CASH DEPOSITS WERE EVIDENCED IN THE ACCOUNT TURNOVER IN THE ACCOUNT HAS SURPASSED THE DECLARED TURNOVER MULTIPLE CURRENT ACCOUNTS ARE OPENED TOWARDS ROUTING FUNDS CREDITS RECEIVED IN THE ACCOUNT VIA INWARD REMITTANCES ARE UTILISED TO MAKE LOCAL PARTIES PAYMENTS VIA RTGS/NEFT SAME ACCOUNT PATTERN EVIDENCED IN THE ALL THE RELATED ACCOUNTS HIGH TURNOVER IN THE ACCOUNT HOWEVER VERY MINIMAL BALANCE MAINTAINED IN THE ACCOUNT SUSPICION RAISED DUE TO BELOW MENTIONED RED FLAGS UNABLE TO ASCERTAIN OVERSEAS PARTY PROFILE THROUGH WEB SEARCH MAJORITY OF INWARD REMITTANCES TOWARDS EXPORT OF MOBILE PHONES RECEIVED FROM LOGISTICS FIRMS IMMEDIATE UTILISATION OF HIGH VALUE RTGS/NEFT CREDITS INWARD REMITTANCE PROCEEDS TO MAKE RTGS/NEFT TO LOCAL PARTIES HIGH CREDIT TURNOVER IN A NEWLY INCORPORATED ACCOUNT MULTIPLE CURRENT ACCOUNTS ARE OPENED FOR ROUTING FUNDS FOR MAKING HIGH VALUE TRANSACTIONS DESPITE HIGH CREDIT TURNOVER IN RELATED ACCOUNT VERY MINIMAL BALANCE MAINTAINED FUNDS TRANSFER BETWEEN STR FILED ACCOUNTS STR FILED FOR THE RELATED ACCOUNTS TOWARDS SERIOUS RED FLAGSCLIENT HAS BEEN REVIEWED FROM AML PERSPECTIVE BASIS HIGH VALUE INWARD REMITTANCES OUR OBSERVATIONS ARE M/S AVIK TELEVENTURES PVT LTD IS A PVT LTD FIRM INCORPORATED ON AT FIRST FLOOR A NARAINA INDUSTRIAL AREA PHASE SOUTH WEST DELHI INDIA CLIENT OPENED CURRENT ACCOUNT WITH US ON AT CHANDANI CHOWK BRANCH NEW DELHI INDUSIND BANK LTD MUNICIPAL NO TO LAXMI NIWAS BUILDING CHANDNI CHOWK DELHI AS ON STATUS OF ACCOUNT IS ACTIVE CUMULATIVE DEBIT IS RS CUMULATIVE CREDIT IS RS AVAILABLE BALANCE IS RS CREDIT LAST CUSTOMER INDUCED TRANSACTION DATE AT THE TIME OF INVESTIGATION IS NATURE OF BUSINESS IS IMPORT EXPORT OF TELECOMMUNICATION ITEMS MOBILE/TELEPHONE ITEMS/ELECTRICAL/ELECTRONIC ITEMS DECLARED TURNOVER IS ABOVE RSCRORES PER ANNUM DIRECTORS MR SAHIL JAIN MOBILE MRRISHABH JAINMOBILE PAN IEC IS AATCAH RISK CATEGORY OF THE CLIENT IS HIGH LAST KYC DONE ON DOCUMENTS OBTAINED DURING ACCOUNT OPENING ARE PAN BOARD RESOLUTION CERTIFICATE OF INCORPORATION ASSOCIATION MEMO ARTICLE GST REGISTRATION FORM GSTIN IS AATCAHZC CURRENT STATUS IS ACTIVE AS PER GST SITE AS ON CLIENT HAD DONE DIRECT/ADVANCE IMPORTS FOR RSLAKHS TOWARDS IMPORT OF OXYMETER GAS SENSOR OUTDOOR PM MONITOR POLLUTION MONITOR SQUAIR POLLUTION MONITORCAIR MONITOR TO OVERSEAS PARTIES RAINBOW COMMUNICATION CO SHAANXI BOHU ZHICHUANG ELECTRONIC TECHNOLOGY CO BASED IN CHINA HONGKONG UNABLE AS FETCH OVERSEAS PARTIES PROFILE THROUGH WEB SEARCH DURING THE SAME PERIOD CLIENT HAS RECEIVED EXPORT ADVANCE FOR RSCRORES TOWARDS EXPORT OF MOBILE PHONES FROM THE OVERSEAS PARTIES BASED IN UAE UNITED STATESMAJORITY OF REMITTANCES ARE RECEIVED FROM LOGISTICS FIRMS RANDOM CHECK DONE ON ONE OF THE OVERSEAS PARTY PROFILE OF LOGIX ONE FZE PARTY IS INTO LOGISTICS SERVICES HTTPS //CONNECTAE/PROFILES/DUBAI/LOGIXONEFZE/PROFILEID ER AS ON CLIENT HAS ALL BOE OUTSTANDING AND EXPORT ADVANCE OUTSTANDINGTOTAL CREDIT TURNOVER BETWEEN TO IS RSCRORES WHICH INCLUDES TOTAL INWARD REMITTANCES RECEIVED FOR RSCRORES OTHER CREDITS DETAILS ARE ATOTAL RTGS CREDITS RECEIVED FOR RSCRORES OF WHICH RSCRORES RECEIVED FROM E PAO GST REFUNDS ACCOUNT WITH STATE BANK OF INDIA ACCOUNT NO RSLAKHS RECEIVED FROM INDIVIDUAL ACCOUNT RAJ KUMAR SANJEEV KUM ACCOUNT WITH BANDHAN BANK LIMITED ACCOUNT NO BMAJORITY OF INWARD REMITTANCES ARE RECEIVED FROM LOGISTICS FIRM TOTAL DEBIT TURNOVER BETWEEN TO IS RSCRORES A RSCRORES VIA RTGS/NEFTOF WHICH MAJORLY SENT TO CLIENT OWN ACCOUNT FOR RSCRORES HELD ACCOUNT WITH AU SMALL FINANCE BANK LIMITED RSCRORES TO BLUE BERRY ACCOUNT WITH ICICI BANK RSCRORES TO HEMANG ELECTRONICS ACCOUNT WITH PUNJAB NATIONAL BANK LAXMI VILAS BANK B TOTAL TRANSFER DEBIT FOR RSCRORES OF WHICH MAJORLY SENT TO YASHOM INTERNATIONAL TRADERS PVT LTDACCOUNT NO FOR RSLAKHS RSLAKHS TO JAY JALARAM ENTERPRISE ACCOUNT NO YASHOM INTERNATIONAL TRADERS PVT LTD ACCOUNT OPENED ON STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS TOTAL CREDIT BALANCE IN THE ACCOUNT IS RSCRORESAVAILABLE BALANCE AS ON DATE IS RS CREDITTRANSACTIONS OBSERVED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES UTILISED VIA RTGS/NEFT TO LOCAL PARTIES IMMEDIATELY DESPITE OF HIGH VOLUMES CLIENT HOLDING MINIMAL BALANCE IN THE ACCOUNT BRINGS MORE SUSPICION IN THE ACCOUNT PATTERN UNABLE TO ASCERTAIN RELATIONSHIP BETWEEN OUR CLIENT YASHOM INTERNATIONAL TRADERS PVT LTD C JAY JALARAM ENTERPRISEACCOUNT NO",{"entities":[(4582,4605,"FOREIGN ORG"),(4607,4654,"FOREIGN ORG"),(5010,5023,"FOREIGN ORG"),(5090,5100,"FOREIGN ORG"),(0,0,"FOREIGN ORG"),(0,0,"FOREIGN ORG")]}),
            ("BHARAT TRADING CO A PROPRIETORSHIP FIRM HAVING ADDRESS NEW COLONY HOUSE NO A PANCHWATI BLOCK A GHAZIABAD UTTAR PRADESH INDIA CONTACT NUMBER AND EMAIL ID CANNYCOCFA GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD R/ RAJ NAGAR GHAZIABAD UTTAR PRADESH SINCE DATE OF INCORPORATION IS PAN OF THE ACCOUNT IS BHIPAE GSTIN AS PER BANK RECORDS IS BHIPAEZO OTHER RELATIONSHIP OF CUSTOMER IS TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS AMAR AGARWAL PROPRIETOR BHIPAE THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN BHIPAE GSTIN BHIPAEZO KYC UPDATION IS NOT DUE THE CUSTOMER IS INVOLVE IN BAMBOO STICKS TRADING BUSINESS ANNUAL TURN OVER DECLARED BY CUSTOMER IS LESS THAN LACS ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH NEFT RS LAKHS MAJORLY FROM RAJA TRADERS PROP MOHAMMAD R GUPTA TRADERS SHUBHAM KUMAR RTGS RS LAKHS MAJORLY FROM SHUBHAM KUMAR BHARAT TRADING CO DHARISH ENTERPRISES MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES TOTAL DEBIT SIDE RS LAKHS OF WHICH RTGS RS LAKHS MAJORLY TO MERINOAIR CONTAINER LIMITED BHARAT TRADING CO REMITTANCE RS LAKHS MAJORLY TO CV AGUNG MAKMUR JAYA CV DUNIA GLOBAL NUSANTARA PT NUANSA DUA WARNA INTERNET FUND TRANSFER RS LAKHS TO MANOJ FY FROM TO TOTAL CREDIT SIDE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS HIGH VALUE CREDITS FOLLOWED BY IMMEDIATE DEBITS ARE SEEN IN ACCOUNT WHICH SEEMS UNUSUAL FOR EXAMPLE ON FUNDS OF RS LACS CREDITED IN THE ACCOUNT THROUGH RTGS FROM BHARAT TRADING CO ACC NO IFSC BARBUTTAMX AND ON THE SAME DAY RS LACS WERE DEPLOYED THROUGH REMITTANCE TOWARDS PT NUANSA DUA WARNA INDONESIA IN USD FOR BAMBOOS FOR ADV PAYMENT AGAINST IMP OTR THAN NEPL BHUTAN ALSO ON RS LACS WERE CREDITED THROUGH RTGS FROM BHARAT TRADING COMPANY AND ON THE SAME DAY RS LACS WERE DEBITED THROUGH OUTWARD REMITTANCE TO PT NUANSA DUA WARNA INDONESIA IN USD FOR ADVANCE PAYMENT AGAINST IMPORT AS PRE EDD VISIT FINDINGS IT WAS RESIDENTIAL ADDRESS CUSTOMER ALSO NOT HAVING ANY OTHER OFFICE NO BUSINESS SETUP BUSINESS ACTIVITY STAFFS STOCKS SIGN BOARD WAS SEEN TRIGGERED TRANSACTIONS ARE SAME LINE OF BUSINESS HOWEVER VOLUME OF TRANSACTIONS ARE NOT MATCHING WITH BUSINESS SETUP HUGE TRANSACTIONS OBSERVED IN ACCOUNT AND IMMEDIATE DEBIT SEEMS SUSPICIOUS IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT AND NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS WITH NEGATIVE EDD FINDINGS STR IS BEING REPORTED",{"entities":[(1543,1562,"FOREIGN ORG"),(1969,1988,"FOREIGN ORG"),(2209,2228,"FOREIGN ORG"),(1526,1541,"FOREIGN ORG")]}),
            ("SRI SNJ JEWELS A PROPRIETORSHIP FIRM HAVING ADDRESS OLD NO SHOP NO / RD FLR NSC BOSE ROAD SOWCARPET TARUS TOWER CHENNAI TAMIL NADU INDIA CONTACT NUMBER AND EMAIL ID SRISNJJEWELS GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER WITH ICICI BANK LTD NSC BOSE ROAD SOWCARPET CHENNAI TAMIL NADU SINCE DATE OF BIRTH IS PAN OF THE ACCOUNT IS BADPJR PAN LINKED ACCOUNTS ARE AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS NIKET SOHANLAL JAIN PROPRIETOR BADPJR THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN BADPJR AND LETTER OF DECLARATION OF PROPRIETORSHIP FIRM KYC UPDATION IS NOT DUE DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN LACS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT WAS RS LAKHS CREDIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY FROM KANTILAL JAI GANESH JEWELLERY NEFT RS LAKHS MAJORLY FROM DAIMOND SANGITA MADAN JAIN RTGS RS LAKHS MAJORLY FROM LEO JEWELS DIAMOND REMITTANCE RS LAKHS MAJORLY FROM CHAIPHET ANYAMANEE CO LTD SUAY THAI THONG CO LTD INTERNET FUND TRANSFERS RS LAKHS FROM MOKSHA JEWELLERY TOTAL DEBIT SIDE RS LAKHS OF WHICH RTGS RS LAKHS MAJORLY TO DIAMOND INDIA LIMITED REMITTANCE RS LAKHS MAJORLY TO CHAIPHET ANYAMANEE CO LTD INTERNET FUND TRANSFER RS LAKHS TO MOKSHA JEWELLERY ETC MOBILE MONEY TRANSFERS RS LAKHS IN INSTANCES FY FROM TO TOTAL CREDIT SIDE RS LAKHS OF WHICH RS BY CASH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RS BY CASH REMITTANCE RS LAKHS CUSTOMER IS INTO GOLD ORNAMENTS MANUFACTURING FOR MANUFACTURING PURPOSE CUSTOMER IS USING JOB WORK VENDORS AND GOLD SMITHS EXCEPT OFFICE SET UP COULD NOT ASCERTAIN ANY BUSINESS ACTIVITIES HIGH VALUE NON CASHC CREDITS AND DEBITS WERE SEEN IN THE ACCOUNT THE ACTIVITY IS NOT LOOKING USUAL AS THE TURNOVER IN JUST YEAR IS MUCH HIGHER IN COMPARE TO THE DECLARED TURNOVER WHICH LESS THAN RS LACS FOR EXAMPLE ON RS LACS WERE CREDITED THROUGH RTGS FROM J B GOLD KKBK AND ON THE SAME DAY RS CRORE WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PVT LTD INDB ALSO ON RS LACS WERE CREDITED THROUGH RTGS FROM A V JEWELLERS YESB LEO JEWELS HDFC ETC AND ON THE SAME DAY RS LACS WERE DEBITED THROUGH RTGS TO PRAKASH GOLD PALACE PRIVATE LIMITED SIMILAR PATTERN WERE SEEN IN THE ACCOUNT IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT WHICH SEEMS LIKE ROUTING OF FUNDS WITH HIGH TURNOVER IN THE ACCOUNT STR IS BEING FILED",{"entities":[(1253,1277,"FOREIGN ORG"),(1279,1300,"FOREIGN ORG"),(1470,1494,"FOREIGN ORG")]}),
            ("TO TOTAL CREDIT SIDE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TRANSACTION PATTERN SHOWS LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT ROUTING OF FUNDS WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS OF HIGH VALUE FUNDS FOR EXAMPLE ONRS CRORES WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM DN ENERGY PRIVATE LIMITEDAND ON THE SAME DAY ACCOUNT WAS DEBITED WITH RS CRORES THROUGH CMSTO RILLICICI ALSO ONRSCRORES WERE CREDITED THROUGH CMS FROM REFEX INDUSTRIES LIMITED AND ON THE SAME DAY ACCOUNT WAS DEBITED WITH SAME AMOUNT THROUGH RTGS IN PARTS TO SIL ROOFTOP SOLAR POWER PVT LTD ACCOUNT NOIFSC HDFC SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED UNUSUAL ALSO AS PER MEDIA REPORT SEBI OBSERVED THAT DURING THE PERIOD OF JULYTO JULY COMPANY HAD ALLEGEDLY VIOLATED THE PROVISIONS OF SEBI SUBSTANTIAL ACQUISITION OF SHARES AND TAKEOVER SAST REGULATION HTTPS //WWWSEBIGOVIN/ENFORCEMENT/ORDERS/JAN/ADJUDICATIONORDERINRESPECTOFJAGDISHTJAINSHERISHATECHNOLOGIESPVTLTDTARACHANDJAINANILJAINUGAMDEVIJAINDIMPLEJAINANDSEEMAJAININTHEMATTEROFREFEXINDUSTRIESLIMITED HTML IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS STR IS BEING FILEDSHERISHA TECHNOLOGIES PVT LTD PRIVATE LTD HAVING ADDRESS OLD NO L NEW NO /TH FLR VENKATNARAYANA ROADT NAGAR CHENNAI TAMIL NADU INDIACONTACT NUMBER AND EMAIL ID LALITHAU REFEXCOIN IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD CHENNAI T NAGAR NEW NO BAZULLAH ROAD T NAGAR CHENNAI TAMIL NADU SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAHCSP IECCODE OF THE CUSTOMER AS PER BANK RECORDS IS OTHER RELATIONSHIP OF CUSTOMER ARE TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE LALITHA AUTHORISED SIGNATORY ACQPLFTAPPI KUMAR DAS AUTHORISED SIGNATORY BDKPDC DINESH KUMAR AGARWAL AUTHORISED SIGNATORY AKHPAA BENEFICIALOWNER OF THE REPORTED ACCOUNT IS A TARACHAND JAIN WITH PAN NO AAFPJLT ANIL JAIN WITH PAN NO AADPAA THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAHCSP CIN UTNPTC BOARD RESOLUTION MOA ETC DATE OF LAST KYC UPDATION KYC UPDATION IS NOT DUE ENTITY IS INTO MANAGEMENT CONSULTANCY SERVICES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO LESS THAN CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CAA ACCOUNTS BALANCE IN THE ACCOUNT ATWAS RS LAKHSCREDIT FURTHER THE QUANTUM OFTRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN TIRUPPUR BRANCHTRANSFERS RS LAKHS MAJORLY FROM ARADHYAA INFRAHOLDINGS PRIVATE LIMITEDPAVAGADA PROCESSING INDUSTRIES PVT LTDCLEARING RS LAKHS MAJORLY FROM SHERISHA SOLAR PVT LTDREFEX INDUSTRIES LIMITEDDECCAN CHARTERS PVT LTDNEFT RS LAKHS MAJORLY FROM MEDHA SERVO DRIVES PVT LTDL T LTDILABS CAPITAL I LLPRTGS RS LAKHS MAJORLY FROM DUGAR FINANCE AND INVEREFEX ENERGY LIMITEDILABS CAPITAL I LLPREMITTANCE RS LAKHS MAJORLY FROM VISTA INDIA DIGITAL MEDIA INCINTERNET FUND TRANSFERS RS LAKHS FROM DN ENERGY PRIVATE LIMITEDMOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCES FIXED DEPOSIT RS LAKHS AND OTHERS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCHTRANSFERS RS LAKHS MAJORLY TO EMPLOYEES PROVIDENT FUND ORGANISATION SHERISHA TECHNOLOGIES PVT LTDKHAJANE II DEPT OF TRESURIESCLEARING RS LAKHS RTGS RS LAKHS MAJORLY TO ILABS CAPITAL I LLPDUGAR FINANCE AND INVESTMENT REMITTANCE RS LAKHS MAJORLY TO GINLONG TECHNOLOGIES CO LTDTUAH AND SUPARTO INTERNET FUND TRANSFER RS LAKHS CARD TRANSACTIONS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RS LAKHS FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS FY FROM",{"entities":[(3602,3628,"FOREIGN ORG")]}),
            ("IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT ROUTING OF FUND AND HIGH TURNOVER IN THE ACCOUNT STR IS BEING FILEDA N DIAMONDS A PROPRIETORSHIP FIRM HAVING ADDRESS H N KUMBHAR SHERIND FLOOR MAHIDHARPURAROOM SURAT GUJARAT INDIA CONTACT NUMBER AND EMAIL ID ANDIAMONDS YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD GROUND FLOOR TOWER A SWASTIK UNIVERSAL UDHNA MAGDALLA ROAD SURAT GUJARAT SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAZPDP GSTIN AS PER BANK RECORDS IS AAZPDPZX TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORY/ PROPRIETOR OF THE REPORTED ACCOUNT IS ASHESH NANALAL DOSHI PAN AAZPDP THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAZPDP AADHAAR GST REGISTRATION CERTIFICATE HENCE KYC UPDATION IS NOT DUE CUSTOMER IS INTO IMPORT AND EXPORT OF ROUGH DIAMONDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RSLAKHS TO RSCRORES ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT ATWAS RS LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN SURATLH ROAD VARACHHA BRANCH TRANSFERS RS LAKHS MAJORLY FROM JIGAR SHAH JT ASHESH NANALAL DOSHI SUMO JEWEL CLEARING RS LAKHS MAJORLY FROM KANCHANBEN VADILAL VADILAL CHHOTALAL NEFT RS LAKHS MAJORLY FROM A N DIAMONDS NISHCHAL C DESAI HUF RTGS RS LAKHS MAJORLY FROM AASHESH NANALAL DOSHI VAISHALI GEMS REMITTANCE RS LAKHS MAJORLY FROM STAR IMPEX HONG KONG USD CURRENCY INTERNET FUND TRANSFERS RS LAKHS FROM A N DIAMONDS MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH TRANSFERS RS LAKHS MAJORLY TO SAHAJANAND TECHNOLOGIES PVT LTD A N DIAMONDSANU IMPEX CLEARING RS LAKHS MAJORLY TO TORRENT POWER LIMITEDMARUTI HARDWARE NEFT RS LAKHS MAJORLY TO RITESH B MEHTAPADSALA VITTHALBHAI RTGS RS LAKHS MAJORLY TO GIA INDIA LABORATORY PVT LTD ASHESH NANALAL DOSHI REMITTANCE RS LAKHS MAJORLY TO GIRDHARLAL DMCC IN USD TO UAE FOR PAYMT TOWARDS IMPORT SETTLEMENT OTHER THAN NEPAL BHUTAN DIAMONDS INTERNET FUND TRANSFER RS LAKHS TO A N DIAMONDS MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RS LAKHS DTAXFURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE ON RSLAKHS WERE CREDITED THROUGH RTGS FROM AASHESH NANALAL DOSHI ACCOUNT NO IFSC CODECBIN BANK CENTRAL BANK OF INDIA AND ON SAME DAY RSLAKHS THROUGH RTGS TO GIA INDIA LABORATORY PVT LTD ACCOUNT NO IFSC CODE HDFC BANK HDFC ON RSLAKHS WERE CREDITED THROUGH INTERNET FUND TRANSFERS FROM A N DIAMONDSAND ON NEXT DAY RSLAKHS WERE DEBITED THROUGH A N DIAMONDS SIMILAR TRANSACTIONS ARE BEING REPEATED IN THE ACCOUNT WHICH SEEMS TO BE ROUTING OF FUNDS IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY WAS RS LAKHS WHICH SEEMS TO BE VERY HIGH",{"entities":[(1598,1607,"FOREIGN ORG")]}),
            ("PANAM ENGINEERS LIMITED MAINTAINS A CURRENT A/C BEARING A/C NO WITH OUR ANDHERI WEST SHOPPERS STOP BRANCH MAHARASHTRA SINCE CUSTOMER DETAILS UPDATED IN BANK RECORDS IS PAN NO AADCPC DATE OF INCORPORATION IS PROFILE IS BUSINESS INSTRUMENT FITTING AND VALUES WITH ANNUAL TURNOVER OF RSCRORE REGISTERED ADDRESS IS C/O PANAM ENGINEERS LIMITEDJAISINGH BUSINESS CENTER PARSIWADA SAHAR RD ANDHERI E MUMBAI MAHARASHTRA RISK PROFILE IS LOW MOBILE NO IS STATUS OF THE A/C IS ACTIVE CURRENT BALANCE IN THE A/C AS ONIS RS / AUTHORIZED SIGNATORY IN THE A/C IS JAGDISH B PRAJAPATI SCRUTINY OF A/C NO FROM MAR TO MAY REVEALED THAT CUSTOMER HAS RECEIVED TOTAL CREDITS TOTALLING TO RS CRORES MAINLY THROUGH NEFT/RTGS CREDITS TOTALLING TO RS CRORES MAINLY FROM OWN A/C WITH SABR KKBK BKID TOTAL TO RS CRORES E PAO GST REFUNDS TOTAL TO RS CRORES SBIN JAGDISH B PRAJAPATI TOTAL TO RS CRORES KKBK DUTY DRAWBACK TOTAL TO RS LACS FUND TRANSFER CREDITS TOTALLING TO RS CRORES MAINLY FROM JAGDISH B PRAJAPATI TOTAL TO RS CRORES A/C NO SPIRARE ENERGY PRIVATE LIMITED TOTAL TO RSLACS A/C NO CHEQUE DEPOSIT TOTALLING TO RS LACS FOREIGN INWARD REMITTANCE CREDITS TOTALLING TO RS LACS IN OCTBY PETROLCOM OIL GAS SERVICES FOR PADVANCE RECEIPTS AGAINST EXPORT CONTRACTS WHICH WILL BE COVERED LATER BY GR/PP/SOFTEX/SDF FROM UAE FASTENAL COMPANY PURCHASING FOR P FROM US A I M SYSTEMS GENERAL TRADING LLC FOR P FROM UAE THE PROCEEDS ARE UTILIZED BY NEFT/RTGS DEBITS TOTALLING TO RS CRORES MAINLY TO TITAN STAINLESS JAGDISH B PRAJAPATI FUND TRANSFER DEBITS TOTALLING TO RS CRORES MAINLY TO JAGDISH PRAJAPATI CBDT TAX PAYMENTS TOTALLING TO RS CRORES SIMILAR PATTERN OF TRANSACTIONS NOTED IN THE LINKED A/CS DURING REVIEW OF THE ACCOUNT STATEMENT IT HAS BEEN OBSERVED THAT CUSTOMER HAS DONETRANSFER TRANSACTIONS BETWEEN GROUP COMPANIES AND ITS DIRECTORSIT HAS BEEN OBSERVD THAT CREDITS TOTALLING TO RS CRORES THROUGHCREDIT TRANSACTIONS THROUGH FUND TRANSFERS FROM TRUSHNA JAGDISH PRAJAPATI JAGDISH B PRAJAPATI DARSHIL J PRAJAPATI U/G J B PRAJAPATI JAGDISH B PRAJAPATI HUF PANAM ENGINEERS LIMITED BHIKHABHAI H PRAJAPATI HUF VISHVA JAGDISHI PRAJAPATI U/G JAGDISH B FOLLOWED BY SUBSEQUENT DEBITS TOTALLING TO RS CRORES THROUGHDEBIT TRANSACTIONS THROUGH FUND TRANSFERS TO JAGDISH B PRAJAPATI VISHVA JAGDISHI PRAJAPATI U/G JAGDISH B DARSHIL J PRAJAPATI U/G J B PRAJAPATI TRUSHNA JAGDISH PRAJAPATI PANAM ENGINEERS BHIKHABHAI H PRAJAPATI HUF PANAM ENGINEERS LIMITED MUKESH B PRAJAPATI JAGDISH B PRAJAPATI HUFIT SEEMS THAT LAYERING TRANSACTIONS HAVE HAPPENED BETWEEN ACCOUNTS WHEN BRANCH OFFICIALS CONTACTED THE CUSTOMER FOR TRANSACTION INQUIRY PURPOSE THEN CUSTOMER STATED THAT THE FUND TRANSFER TRANSACTIONS ARE TOWARDS LOAN REPAYMENTS WITH FAMILY MEMBERS HOWEVER CUSTOMER COULD NOT SHARE ANY INVOICES FOR THE HIGH VALUE TRANSACTIONS IN THE A/C HENCE BY LOOKING AT OVERALL TRANSACTION PATTERN AND QUANTUM OF THE AMOUNT IN SAVINGS A/C BRANCH OFFICIALS WERE NOT SATISFIED STR IS BEING FILLED FOR THE FOLLOWING REASON A HIGH VALUE AND COMPLEX TRANSACTIONS OBSERVED WHICH RESEMBLES THE PATTERN AS NORMALLY SEEN DURING LAYERING OF TRANSACTIONSB CUSTOMER COULD NOT SHARE ANY INVOICES FOR THE HIGH VALUE TRANSACTIONS IN THE A/CC JUSTIFICATION PROVIDED BY CUSTOMER WAS NOT FOUND SATISFACTORY BY BRANCH OFFICIAL DURING REGULAR REVIEW OF TRANSACTIONS A SAVING A/C OF BHIKHABHAI H PRAJAPATI HUF BEARING A/C NOHAS COME INTO LIGHT WHEREIN IT HAS BEEN OBSERVED THAT CUSTOMER HAS RECEIVED CREDITS TOTALLING TO HIGH VALUE MAINLY VIA NEFT/RTGS IMPS FUND TRANSFER CREDITS FOLLOWED BY SUBSEQUENT UTILIZATION MAINLY VIA NEFT/RTGS FUND TRANSFER DEBITS WHICH ARE NOT INLINE WITH THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER FURTHER DETAILS ARE AS UNDER BHIKHABHAI H PRAJAPATI HUF MAINTAINS A SAVING A/C BEARING A/C NO WITH OUR ANDHERI WEST SHOPPERS STOP BRANCH MAHARASHTRA SINCE CUSTOMER DETAILS UPDATED IN BANK RECORDS IS PAN NO AAGHBJ DATE OF INCORPORATION IS CUSTOMER TYPE IS HUF PROFILE IS BUSINESS CONSULTANCY SERVICES WITH ANNUAL TURNOVER OF RSLACS REGISTERED ADDRESS IS C/O BHIKHABHAI H PRAJAPATI HUFSHASHVAT JANKSHAN SUBHASH ROAD OFF MG ROAD VILE PARLE EAST MUMBAI MAHARASHTRA RISK PROFILE IS MEDIUM MOBILE NO IS STATUS OF THE A/C IS ACTIVE CURRENT BALANCE IN THE A/C AS ONIS RS / AUTHORIZED SIGNATORY IN THE A/C IS BHIKHABHAI H PRAJAPATI SCRUTINY OF THE A/C NOFROM MAR TO MAY REVEALED THAT CUSTOMER HAS RECEIVED TOTAL CREDITS TOTALLING TO RS CRORES MAINLY THROUGH NEFT/RTGS CREDITS TOTALLING TO RSCRORE MAINLY FROM JAGDISH B PRAJAPATI HUF TOTAL TO RSLACS KKBK JAGDISH B PRAJAPATI TOTAL TO RSLACS KKBK FUND TRANSFER CREDITS TOTALLING TO RSLACS FROM JAGDISH B PRAJAPATI HUF A/C NO IMPS CREDITS TOTALLING TO RSLACS FROM JAGDISH B PRAJAPATI THE PROCEEDS ARE UTILIZED BY NEFT/RTGS DEBITS TOTALLING TO RS LACS MAINLY TO JAGDISH B PRAJAPATI TOTAL TO RSLACS KKBK SANKET MUKESH PRAJAPATI TOTAL TO RSLACS KKBK ZALAK MUKESH PRAJAPATI TOTAL TO RSLACS KKBK TRUSHNA JAGDISH PRAJAPATI TOTAL TO RSLACS KKBK CBDT TAX PAYMENTS TOTALLING TO RS LACS FUND TRANSFER DEBITS TOTALLING TO RSLACS TO MUKESH B PRAJAPATI A/C NO BASIS HIGH VALUE FUND TRANSFER TRANSACTIONS WITH BHIKHABHAI H PRAJAPATI HUFSAVINGS A/C OF JAGDISH B PRAJAPATI HUF AND MUKESH B PRAJAPATI BEARING A/C NOS ANDWITH OUR ANDHERI WEST SHOPPERS STOP BRANCH MAHARASHTRA SINCEANDWERE IDENTIFIED FURTHER DETAILS ARE AS UNDER SCRUTINY OF A/C NO FROM MAR TO MAY REVEALED THAT CUSTOMER HAS RECEIVED TOTAL CREDITS TOTALLING TO RS CRORES MAINLY THROUGH NEFT/RTGS CREDITS TOTALLING TO RSLACS FROM JAGDISH B PRAJAPATI KKBK FUND TRANSFER CREDITS TOTALLING TO RSLACS FROM JAGDISH B PRAJAPATI IMPS CREDITS TOTALLING TO RSLACS FROM JAGDISH B PRAJAPATI CHEQUE DEPOSIT TOTALLING TO RS THE PROCEEDS ARE UTILIZED BY FUND TRANSFER DEBITS TOTALLING TO RSLACS MAINLY TO BHIKHABHAI H PRAJAPATI HUF CBDT TAX PAYMENTS TOTALLING TO RS LACS SCRUTINY OF A/C NO FROM MAR TO MAY REVEALED THAT CUSTOMER HAS RECEIVED TOTAL CREDITS TOTALLING TO RS LACS MAINLY THROUGH FUND TRANSFER CREDITS TOTALLING TO RSLACS FROM BHIKHABHAI H PRAJAPATI HUF NEFT CREDITS TOTALLING TO RSLACS TO MUKESH B PRAJAPATI KKBK CHEQUE DEPOSIT TOTALLING TO RSLACS IMPS CREDITS TOTALLING TO RSLACS THE PROCEEDS ARE UTILIZED BY FUND TRANSFER DEBITS TOTALLING TO RS LACS TO CBDT COLLECTION CBDT TAX PAYMENTS TOTALLING TO RS LACS BASIS COMMON MOBILE NO ADDITIONALSAVINGS A/CS BEARING A/C NOS AS WELL ASCURRENT A/CS BEARING A/C NOS WITH OUR ANDHERI WEST SHOPPERS STOP BRANCH MAHARASHTRA WERE IDENTIFIED",{"entities":[(1295,1310,"FOREIGN ORG"),(1337,1369,"FOREIGN ORG")]}),
            ("BELOW ARE RED FLAGS OBSERVED OUT OF TOTALOUTWARD REMITTANCES VALUE RS CRORES PERTAINING TO ADVANCE AGAINST IMPORTS CLIENT HAS NOT SUBMITTED BILL OF ENTRY PERTAINING TOTRANSACTIONS OF VALUE RS CRORES PENDING SINCE NOVEMBER IN THIS REGARD YES BANK OFFICIALS HAD FOLLOWED UP WITH THE CUSTOMER ON MULTIPLE OCCASIONS AND HAD REQUESTED TO SHARE TRANSPORT DOCUMENTS FOR WHICH IMPORT REMITTANCES ARE BEING UNDERTAKEN HOWEVER CUSTOMER HAS TILL DATE NEITHER SUBMITTED BILL OF ENTRIES PERTAINING TOTRANSACTIONS NOR HAS HE FURNISHED ANY TRANSPORT DOCUMENTS PERTAINING TOOUTWARD REMITTANCES THUS UNDERTAKENAS PER RECENT COMMUNICATION CLIENT HAS REPLIED STATING THAT SINCE HE NO LONGER WANTS TO TRANSACT VIA YES BANK HE HAS SUBMITTED BILL OF ENTRY AND LODGED RELATED TRANSPORT DOCUMENTS WITH ANOTHER BANKAS PER PUBLIC DOMAIN INFORMATION ON OVERSEAS ENTITIES NAFDA TRADING FZE UAE AND CAMELOT OVERSEAS PTE LTD SINGAPORE BELOW RED FLAGS WERE OBSERVED PUBLIC DOMAIN INFORMATION OF OVERSEAS ENTITY NAFDA TRADING FZE UAE IS INSUFFICIENT MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS KITCHENER ROAD JALAN BESAE PLAZA SINGAPORE OF OVERSEAS ENTITY CAMELOT OVERSEAS PTE LTDCAMELOT OVERSEAS PTE LTD HAD PREVIOUSLY CHANGED NAMES MULTIPLE TIMES REFER HTTPS WWWCOMPANIESSG/ZH/BUSINESS/H/CAMELOTOVERSEASPTELTDWEBSITE OF CAMELOT OVERSEAS PTE LTD SINGAPORE IS NEWLY REGISTERED IN THE YEAR OCTOBERAND REGISTRATION LOCATION IS NEW DELHI INDIA HTTPS WWWWHOISCOM/WHOIS/CAMELOTOVERSEASCOM INCIDENTALLY IT WAS OBSERVED THAT CONTENTS OF WEBSITE OF OVERSEAS ENTITY CAMELOT OVERSEAS PTE LTD HTTPS CAMELOTOVERSEASCOM/ IS COPY PASTED FROM CUSTOMERS OWN WEBSITE OF KALEEN BHAWAN REFER HTTPS KALEENBHAWANCOM/ PAGE ID FURTHER STATED ANNUAL INCOME OF IMEXTRA TRADERS PRIVATE LIMITED IS LESS THAN RS CRORES HOWEVER IN LESS THANMONTHS FROM ACCOUNT OPENING OCTOBERCLIENT HAS RECEIVED ACCOUNT CREDITS OF MORE THAN RS CRORESMAJOR CREDITS IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED IS VIA FUNDS TRANSFER FROM OTHER YES BANK CUSTOMERS VIZ M CARPETS RSCRS FROM AC NO MISHRA AND SONS RSCRS FROM AC NO SHIVESH GARMENTS PRIVATE LIMITED RSCRS FROM AC NOAND RTGS THE TUNE OF RS FROM SHIVESH GARMENTS PRIVATE LIMITED AC NOWITH ICICI BANK ICIC AND RS CRORES FROM MISHRA AND SONS AC NOWITH INDUSIND BANK INDBNO RED FLAGS WERE OBSERVED IN THE LINKED ACCOUNTS OF VINDHYA DAIRY UDAMYA IMPEX PRIVATE LIMITED AND ORIENTAL CARPET EXPORTERS HOWEVER TOTAL CREDITS RECORDED IN YES BANK ACCOUNTS OF OTHER LINKED ACCOUNTS OF M CARPETS MISHRA AND SONS AND SHIVESH GARMENTS PRIVATE LIMITED IS BEYOND STATED ANNUAL INCOME OF LESS THAN RSCRORES REGISTERED COMMUNICATION ADDRESS OF SHIVESH GARMENTS PRIVATE LIMITED IS THAT OF NEW DELHI DELHI AS MENTIONED IN POINT HOWEVER ITS REGISTERED PERMANENT ADDRESS IS IN FLAT NOPOONAM ORBIT MIRA ROAD THANE MAHARASHTRA AFOREMENTIONED REGISTERED PERMEANT ADDRESS OF SHIVESH GARMENTS PRIVATE LIMITED DOES NOT MATCH WITH THE ONE ON PUBLIC DOMAIN IE /C FIRST FLOOR AJANTA SQUARE MALL BHAJI MARKET LANE BORIVALI W MUMBAI CITY MAHARASHTRA IN LIGHT OF CROSS BORDER REMITTANCES BEING UNDERTAKEN IN PRETEXT OF IMPORT OF GOODS FROM OVERSEAS ENTITIES WITH OPAQUE CREDENTIALS WHEREIN VERACITY OF UNDERLYING TRANSACTIONS COULDNT BE DETERMINED DUE TO ABSENCE OF SUPPORTING DOCUMENTS AND CUSTOMER BEING NON COOPERATIVE IT HAS BEEN DECIDED ON MAY TO FILE STR ON SAID CUSTOMER THE CUSTOMER IMEXTRA TRADERS PRIVATE LIMITED PAN AAECIK DOI COMMUNICATION ADDRESS THIRD FLOOR / EAST PATEL NAGAR NEW DELHI DELHI PERMANENT ADDRESS C FIRST FLOOR POLE NO D PALAM NEW DELHI GSTIN AAECIKZD REGISTRATION NUMBER CIN UDLPTC IEC CODE AAECIK MOBILE NUMBER EMAIL ID IMEXTRATRADERSPVTLTD GMAILCOM IS MAINTAINING A CURRENT AC NOSINCEWITH DELHI ASAF ALI ROAD BRANCH ADDRESS GROUND FLOOR / ASAF ALI ROAD NEW DELHI THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN OCTOBER SAT PAL SHARMA AND ASHISH MISHRA ARE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS GARMENT STORE WITH AN ANNUAL INCOME LESS THAN RS CRORES CUSTOMER HAS AVAILED TERM DEPOSITS AMOUNTING TO RS ADDITIONALLYMORE ACCOUNTS ARE FOUND TO BE LINKED TO THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NO AS UNDERCURRENT ACCOUNT OF KALEEN BHAWAN EXPORTS PRIVATE LIMITED AC NOIS LINKED WITH AFOREMENTIONED ACCOUNT BY COMMON MOBILE NUMBERAND COMMUNICATION ADDRESS IN POINT NOCURRENT ACCOUNT OF SHIVESH GARMENTS PRIVATE LIMITED AC NO VINDHYA DAIRY AC NO UDAMYA IMPEX PRIVATE LIMITED AC NOAND ORIENTAL CARPET EXPORTERS AC NOS LINKED WITH AFOREMENTIONED ACCOUNT BY COMMON COMMUNICATION ADDRESS IN POINT NO CURRENT ACCOUNT OF M CARPETS AC NOAND MISHRA SONS AC NOARE LINKED WITH ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED BY FUND TRANSFERS IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NOANDLINKED ACCOUNTS TRANSACTIONS ANALYSIS FROMTILL SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS ANDDEBIT TRANSACTIONS AMOUNTING TO RS OUT OF THE TOTAL CREDITS OF RS IN THE ACCOUNTS FUND TRANSFERS AMOUNTS TO RS CONSTITUTING AROUNDPERCENT OF THE TOTAL CREDITS NEFT/RTGS/IMPS AMOUNTS TO RS CONSTITUTING AROUNDPERCENT OF THE TOTAL CREDITS OUT OF THE TOTAL DEBITS OF RS IN THE ACCOUNTS FUND TRANSFERS AMOUNTS TO RS CONSTITUTING AROUNDPERCENT OF THE TOTAL DEBITS NEFT/RTGS/IMPS AMOUNTS TO RS CONSTITUTING AROUNDPERCENT OF THE TOTAL DEBITSIN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NOANDLINKED ACCOUNTS TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH DEPOSIT AMOUNTING TO RS ANDDEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS FROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH DEPOSIT AMOUNTING TO RS ANDDEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS FROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH DEPOSIT ANDDEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS BALANCE IN THE ACCOUNT AS ONIS RS ON REVIEWING THE TRANSACTION PATTERN OF IMEXTRA TRADERS PRIVATE LIMITED ACAND KALEEN BHAWAN EXPORTS PRIVATE LIMITED AC NOIT WAS OBSERVED THAT SINCE OCTOBERSAID ENTITIES HAD REMITTED TOTAL OF RS CRORES UNDERTRANSACTIONS TO OVERSEAS ENTITIES CAMELOT OVERSEAS PTE LTD SINGAPORE RS CRORES INTRACHES AND NADFA TRADING FZE UAE RS CRORES INTRANCHES TOWARDS ADVANCE AGAINST IMPORT OF CARPET / CARPET",{"entities":[(844,860,"FOREIGN ORG"),(870,893,"FOREIGN ORG"),(980,996,"FOREIGN ORG"),(1128,1151,"FOREIGN ORG"),(1152,1175,"FOREIGN ORG"),(1294,1317,"FOREIGN ORG"),(1529,1552,"FOREIGN ORG"),(6255,6278,"FOREIGN ORG"),(6314,6330,"FOREIGN ORG"),(0,0,"FOREIGN ORG")]}),
            ("FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS FY FROMTO TOTAL CREDIT SIDE RS LAKHS TOTAL DEBIT SIDE RS LAKHS FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS TOTAL DEBIT SIDE RS LAKHS THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBERIN THE NAME OF BORA MOBILITY REPORT SERIAL NUMBER TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS WHICH SEEMS LIKE ROUTING OF FUNDS OBSERVED FOR EXAMPLE ONRS LACS WAS CREDITED BY RTGS IN PARTS FROM JAY JALARAM ENTERPRISE INDB BORA EXIM PRIVATE LIMITED NEFT FROM SAMA DISTRIBUTOR HDFC AND RSLACS WAS DEBITED BY RTGS TO BORA MOBILITY HDFC IN OTHER EXAMPLE ONR LACS CREDITED BY INTERNET FUND TRANSFERS FROM BORA EXIM PRIVATE LIMITEDAND SAME WAS DEBITED BY INTERNET FUND TRANSFERS TO BORA MOBILITY HDFC SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL INCOME OF RS LACS TO RS CRS IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT ROUTING OF FUNDS OBSERVED AND HIGH TURNOVER STR IS BEING FILEDBORA MOBILITY PROPRIETORSHIP FIRM HAVING ADDRESS OFFICE NO AND NEARMITRAMANDALCHOW PARVATI PUNE CENTRE POINT PLOT NO PUNE MAHARASHTRA INDIA CONTACT NUMBER AND EMAIL ID PRASHANTBORA GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD PUNE SATARA ROAD ICICI BANK LTD SOMSHANK CHAMBERS SURVEY NO PLOT PUNE SATARA ROAD PUNE MAHARASHTRA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AISPBL TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE PRASHANT PRAKASH BORA PROPRIETOR AISPBL THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AISPBL AADHAR DATE OF LAST KYC UPDATION BORA MOBILITY IS AN INDIAN COMPANY ENGAGED IN TRADING AND DISTRIBUTION OF MOBILE HANDSETS TABLETS LAPTOPS AND MOBILE ACCESSORIES WITH DIRECT PRESENCE IN OVER TWENTY COUNTRIES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LACS TO RS CRS A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NONCASH TRANSACTIONS BALANCE IN THE ACCOUNT ATWAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH CLEARING RS LAKHS MAJORLY FROM NEFT SSK INCORPORATION NEFT RS LAKHS MAJORLY FROM MUKUL MADHAV FOUNDATION MAHARASHTRA VALUE ADDED TAX MUKUL MADHAV FOUNDATION FIL CSR RTGS RS LAKHS MAJORLY FROM KAMALETERNITY PVT LTD JAY JALARAM ENTERPRISE BORA MOBILITY INTERNET FUND TRANSFERS RS LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCES FIXED DEPOSIT RS LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS NIL CLEARING RS LAKHS RTGS RS LAKHS REMITTANCE RS LAKHS MAJORLY TO SHENZHEN DEDA HEALTH CO LTD MEDMSL LIMITED ZHENGZHOU OLIVE ELECTRONIC INTERNET FUND TRANSFER RS LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RS LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN DELHI KAROL BAGH GURDWARA ROAD BULDHANA KAROL BAGH NDBRANCH TRANSFERS RS LAKHS MAJORLY FROM BORA MOBILITY PRASHANT PRAKASH BORA SEANA CLEARING RS LAKHS MAJORLY FROM SSK INCORPORATION BORA MOBILITY LLP A Z LUNAWAT AND NEFT RS LAKHS MAJORLY FROM OMKAR COMPUTERS PVT LTD BITFLIGHT ELECTRONIC SYSTEMS BORA MOBILITY RTGS RS LAKHS MAJORLY FROM BORA MOBILITY BATHLA TELETECH PRIVATE E PAO GST REFUNDS REMITTANCE RS LAKHS MAJORLY FROM CAPTAINS FREIGHT SERVICES HK INTERNET FUND TRANSFERS RS LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCES FIXED DEPOSIT RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKH BY BRANCH TRANSFERS RS LAKHS MAJORLY TO BORA MOBILITY CLEARING RS LAKHS RTGS RS LAKHS MAJORLY TO BORA AND BORA TECHNOLOGIES BORA MOBILITY REMITTANCE RS LAKHS MAJORLY TO ESTAR TECHNOLOGY GROUP CO LTD WONDAFONE TECHNOLOGY HONGKONG CO AYAAN ELECTRONICS LLC INTERNET FUND TRANSFER RS LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES",{"entities":[(2937,2963,"FOREIGN ORG"),(2965,2978,"FOREIGN ORG"),(2980,3005,"FOREIGN ORG"),(4174,4202,"FOREIGN ORG"),(4204, 4235,"FOREIGN ORG"),(4237,4257,"FOREIGN ORG")]}),
            ("BEETREAT LOGISTICS PRIVATE LIMITED A PRIVATE LTD COMPANY HAVING ADDRESS OFFICE NO ST FLOOR SECTOR CBD BELAPUR PLOT NO NAVI MUMBAI MAHARASHTRA INDIA CONTACT NUMBER AND EMAIL ID BEETREATLOGISTICS GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD BALAJI BHAVAN NEAR BELAPUR RAILWAY STN CBD BELAPUR NAVI MUMBAI MAHARASHTRA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAICBG GSTIN AS PER BANK RECORDS IS AAICBGZZ TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS/ DIRECTORS OF THE REPORTED ACCOUNT ARE DINESH KUMAR WITH PAN NO ANXPKBVIKAS NARAWAT PAN AIJPVD THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON KYC DOCUMENTS OBTAINED WERE PAN AAICBG COI UMHPTC MOA AOA AND PAN OF AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS CUSTOMER IS INTO THE BUSINESS OF LOGISTICS SERVICES TRADING WHOLES RETAILS IMPORT AND EXPORT OF GOODS AND SERVICES FOR MAKING AND EARING PROFITS WORKS ON COMMISSION BASIS FOR GOODS TRADING DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS GREATER THAN EQUALS TO RSCRORES ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY FROM DINESH KUMAR NEFT RS LAKHS MAJORLY FROM BONIRY LOGISTICS RTGS RS LAKHS MAJORLY FROM BONIRY LOGISTICS AEGIS LOGISTICS MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS BY ATM AT THANE NEW BOMBAY BELAPUR CIBD BRANCH CLEARING RS LAKHS MAJORLY TO BEETREAT LOGISTICS PRIVATE LIMITED RTGS RS LAKHS MAJORLY TO BONIRY LOGISTICS REMITTANCE RS LAKHS MAJORLY TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG IN USD CURRENCY FOR WHICH PURPOSE WAS MENTIONED AS TOWARDS OTHER PAYMENTS BY SHIPPING COMPANIES KM LOGISTICS AND SHIPPING PTE LTD IN USD TO SINGAPORE FOR OTHER PAYMENTS BY SHIPPING COMPANIES HMMA LOGISTICS CO LIMITED IN USD TO HONG KONG FOR OTHER PAYMENTS BY SHIPPING COMPANIES LUCKY GAINS LIMITED IN USD TO HONG KONG FOR WHICH PURPOSE WAS MENTIONED AS PAYMENT TOWARDS SHIPPING CHARGES INTERNET FUND TRANSFER RS LAKHS TO KEMPS LOGISTICS PRIVATE LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCESON ANALYZING TRANSACTION PATTERN IN THE ACCOUNT IT HAS BEEN OBSERVED THAT THERE WERE HIGH VALUE NON CASH CREDITS FOLLOWED BY DEBITS THROUGH REMITTANCES FOR EXAMPLE ON RS LAKHS WERE CREDITED THROUGH RTGS FROM BONIRY LOGISTICS ACCOUNT NO IFSC CODEUTIB AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH REMITTANCE TO BLUESEA SHIPPERS HK LIMITED AT HONG KONG IN USD CURRENCY FOR WHICH PURPOSE WAS MENTIONED AS TOWARDS OTHER PAYMENTS BY SHIPPING COMPANIES ON RSLAKHS WERE CREDITED THROUGH RTGS FROM AEGIS LOGISTICS ACCOUNT NO IFSC CODE ASBL AND ONRS LAKHS WERE DEBITED THROUGH REMITTANCE TO LUCKY GAINS LIMITED IN USD TO HONG KONG FOR WHICH PURPOSE WAS MENTIONED AS PAYMENT TOWARDS SHIPPING CHARGES SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT WITHOUT ANY BUSINESS EXPENDITURE TRANSACTIONS SEEM UNUSUAL AND NO SATISFACTORY EXPLANATION ABOUT IT CAN BE OBTAINED FROM THE CUSTOMER ACCOUNT CLOSED BY CUSTOMER WITHIN SPAN OFMONTHS AND HIGH TURNOVER IN THE ACCOUNT WITHIN SPAN OFMONTHS RS LAKHS WHICH RAISED FURTHER SUSPICION IN VIEW OF HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT STR IS BEING FILED",{"entities":[(1798,1824,"FOREIGN ORG"),(2647,2673,"FOREIGN ORG"),(1935,1967,"FOREIGN ORG"),(2030,2054,"FOREIGN ORG")]}),
            ("SUBSEQUENT TO SUCH CREDITS FUNDS WERE REMITTED ON THE SAME OR NEXT DAY OVERSEAS FAVOURING ENTEGRATED LOGISTIC PTE LTDSINGAPORE RS CR TOWARDS FREIGHT ON EXPORTSSHIPPING COMPANIESDURING WEB CHECK NO SPECIFIC INFORMATION/ WEBSITE COULD BE SEEN IN THE NAME OF ENTEGRATED LOGISTIC PTE LTDON SCRUTINY OF SAMPLE DOCUMENTS FURNISHED BY CUSTOMER FOR EFFECTING REMITTANCES IN WAS NOTED THAT CUSTOMER HAD FURNISHED COPIES OF B/L FREIGHT INVOICE AGREEMENT ANDCA/ CB AS PER THE B/L GOODS WERE SHIPPED FROM INDIA TO CHINA HOWEVER FREIGHT PAYMENT WAS BEING MADE TO SINGAPORE BASED ENTITYA PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE RTGS/NEFT CREDITS WERE FOLLOWED BY HIGH VALUE FOREIGN OUTWARD REMITTANCES ON THE SAME DAY NO OTHER BUSINESS RELATED TRANSACTIONS LIKE PAYMENTS TOWARDS TAXES SALARIES RENT ETC COULD BE NOTED IN THE ACCOUNT DURING REVIEW PERIODIT IS NOTED THAT WITHINMONTHS OF ACCOUNT OPENING TRANSACTIONS IN THE ACCOUNT FAR EXCEEDED THE ANNUAL TURNOVER INDICATED BY CUSTOMER DURING ACCOUNT OPENINGDURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER IT WAS NOTED THAT THE OFFICE WAS ASQ FT RENTED PREMISE WHERE SIGN BOARD EMPLOYEES AND OFFICE SET UP WERE SEENBASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT THE DIRECTORS WHO ARE AGED YEARS MOHIT CHAUDHARY ANDYEARS ASHISH CHOWREKAR MAY NOT BE THE ACTUAL BENEFICIAL OWNER OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNT IN A SHORT SPAN ALSO CUSTOMER REMITTING FUNDS TO SINGAPORE AS FREIGHT CHARGES WHEN THE GOODS ARE SHIPPED FROM INDIA TO CHINA APPEARS UNUSUAL IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH THE ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING OR HAWALA SETTLEMENT ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED IS ACTIVE WITH BALANCE OF RS LAKHS AS ON MAY QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR IS DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE IS DEBIT RS CR IN CASH RS CR AND CREDITS RS CR IN CASH RS CRA CURRENT ACCOUNT BEARING NUMBERWAS OPENED IN THE NAME OF OCEANS PRIDE MARINE SERVICES P LTD ON FEB AT OUR BRANCH LOCATED IN LAKE PRIMROSE PHASEIV WING C LAKE HOMES POWAI MUMBAI MAHARASHTRA THE ENTITY WAS FORMED ON DEC AND HAS SUBMITTED COPIES OF PAN AACCOA CERT OF INCORP UMHPTC SHOPS AND ESTABLISHMENT REG CERTIFICATEAND IEC CERT AACCOA AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NOSAMRUDDHI C COMPLEX CHINCHOLI BUNDER ROAD MALAD WEST MUMBAI MOHIT CHAUDHARY AND ASHISH DINESH CHOWKEKAR ARE THE DIRECTORS WHILE MOHIT CHAUDHARY IS THE BENEFICIAL OWNER IN THE SAID COMPANY MOHIT CHAUDHARY AND ASHISH DINESH CHOWKEKAR HAVE REGISTERED THEIR MOBILE NUMBERSANDWITH THE BANK THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS CRBASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT ACCOUNTS IN THE NAMES OF TRIUMPH TECHNOMATE SOLUTIONS A/C OPENED ON JUL RAJKOTIA CLOUD CONSULTANCY P LTD A/C OPENED ON AUG AND SAVINGS ACCOUNT IN THE NAME OF ASHISH DINESH CHOWKEKAR A/C OPENED ON JAN THE CURRENT ACCOUNT OF TRIUMPH TECHNOMATE SOLUTIONS AND SAVINGS ACCOUNT OF ASHISH DINESH CHOWKEKAR ARE DORMANT IT WAS INDICATED THAT RAJKOTIA CLOUD CONSULTANCY P LTD IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS CR NO MAJOR TRANSACTIONS WERE NOTED IN THE ACCOUNT OF RAJKOTIA CLOUD CONSULTANCY P LTD DURING REVIEW PERIOD WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WERE ROUTED THROUGH THE ABOVE MENTIONED NEWLY OPENED ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH FREIGHT PAYMENTS WERE DONE IMMEDIATELY POST RECEIPT OF HIGH VALUE RTGS/NEFT CREDITS AND HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION ON REVIEWING ACCOUNT TRANSACTIONS SINCE APRIL TILL MAY IT WAS NOTED THAT RTGS/NEFT CREDITS AMOUNTED TO RS CR MAINLY FROM QUICK SOLUTION LOGISTICS RS CR RBL BANK BLUE SEA LOGISTIC RS CR RBL AND PREMIUM LOGISTICS SOLUTIONS RS CR RBL",{"entities":[(90,116,"FOREIGN ORG"),(256,282,"FOREIGN ORG")]}),
            ("IN OTHER EXAMPLE ON RS LACS AMOUNT OF FUND CREDITED IN PARTS THROUGH RTGS FROM MAGNETIC FILMS PRODUCTION DIVERSE MEDIA WHERE FUND DEBITED THROUGH OUTWARD REMITTANCE TOWARDS WEB VIDEOS LIMITED UNITED KINGDOM HTTP //WWWWEBVIDEOSCOUK/ SINGAPORE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE WHITE RABBIT MOTION PICTURE PRODUCTION LLC UAE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE HTTP //WWWWHITERABBITDUBAICOM/ THIS ACCOUNT GET CLOSED IN SHORT SPAN OF PERIOD OFMONTHS WITH RS CRORES CLIENT WAS UNABLE TO PROVIDE THE INVESTMENT AGREEMENT WITH THE INVESTORS IN SPITE SEVERAL ASSURANCES WHICH SEEMS SUSPICIOUS TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF BELOW RS LACS IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS IN NEWLY OPENED ACCOUNT AND HIGH TURNOVER STR IS BEING FILEDTINGLE CINE PRODUCTIONS PARTNERSHIP HAVING ADDRESS ND FLOOR / FLAT NORELIANCE CLASSIC ABOVE ANDHRA HYDERABAD TELANGANA INDIA CONTACT NUMBER AND EMAIL ID INFO TINGLECINEPRODUCTIONSCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LIMITED HYDERABAD BEGUMPET //A JAYALAKSHMI TOWERS MOTILALNEHRU NAGAR PRAKASHNAGAR BEGUMPET HYDERABAD TELANGANA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AAQFTK GSTIN AS PER BANK RECORDS IS AAQFTKZL TRANSACTION LINKED ACCOUNTS ARE BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH CHAWLA WITH PAN NO AGRPCE MOHIT BANSAL WITH PAN NO ALRPBC THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON KYC DOCUMENTS OBTAINED WERE GSTIN AADHAR PANAAQFTK AS PER ENHANCE DUE DILIGENCE CUSTOMER IS IN BUSINESS OF THE INVESTORS AND THE BENEFICIARY TO BE IN THE LINE MEDIA AND FILMS THEY ARE THE SERVICE PROVIDER FOR MOVIE OR FILM PRODUCTION DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS BELOW RS LACS A MANUAL ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT ATWAS NIL FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL CLEARING RS LAKH MAJORLY FROM CHQ NEFT RS LAKHS MAJORLY FROM DIVERSE MEDIA LIQUID STUDIOS AND SERVICES RTGS RS LAKHS MAJORLY FROM DIVERSE MEDIA LIQUID STUDIOS AND SERVICE MAGNETIC FILMS PRODUCTION MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKH BY BRANCH CLEARING RS LAKHS MAJORLY TO T VINAY ASSOCIATES RTGS RS LAKHS MAJORLY TO LIQUID STUDIO SERVICE LIQUID STUDIO SERVICE REMITTANCE RS LAKHS MAJORLY TO SKY DENVAX PTE LTD WEB VIDEOS LIMITED WHITE RABBIT MOTION PICTURE INTERNET FUND TRANSFER RS LAKHS MAJORLY TO MANISH CHAWLA MOHIT BANSAL MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TRANSACTION PATTERN SHOWS HIGH VALUE NON CASH CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE ON RS CRORES AMOUNT OF FUND CREDITED IN PARTS THROUGH RTGS FROM LIQUID STUDIOS AND SERVICE MAGNETIC FILMS PRODUCTION DIVERSE MEDIA AND ON THE SAME DAY SAME AMOUNT OF FUND DEBITED THROUGH OUTWARD REMITTANCE TOWARDS WEB VIDEOS LIMITED UNITED KINGDOM HTTP //WWWWEBVIDEOSCOUK/ SINGAPORE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE WHITE RABBIT MOTION PICTURE PRODUCTION LLC UAE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE",{"entities":[(173,205,"FOREIGN ORG"),(298,339,"FOREIGN ORG"),(2649,2666,"FOREIGN ORG"),(2668,2685,"FOREIGN ORG"),(2693,2713,"FOREIGN ORG"),(3157,3174,"FOREIGN ORG")]}),
            ("TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH TRANSFERS RS LAKHS MAJORLY TO BINDRA TOURS N TRAVELS PRIVATE LIMITED EMPLOYEES PROVIDENT FUND ORGANISATION TRAVEL BOUTIQUE ONLINE A U/O TEK TRAPL CLEARING RS LAKHS MAJORLY TO WELFARE COMMISSIONER HARY POOJA PRINTERS PHD CHAMBER OF COMMERCE RTGS RS LAKHS MAJORLY TO TRAVEL BOUTIQUE ONLINE TRAVEL BOUTIQUE ONLINE REMITTANCE RS INTERNET FUND TRANSFER RS LAKHS MAJORLY TO R K TRAVELTLC SOLUTION KANDHARI TRAVEL ETC FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHSFY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHSFY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS REMITTANCE RS PAST STR REPORTED THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBERIN THE NAME OF TRAVEL BOUTIQUE ONLINE A U/O TEK TRAPL ONVIDE FIU BATCH ID REPORT SERIAL NUMBERCUSTOMER IS INTO TRAVEL RELATED SUPPORT THROUGH ONLINE BASED PORTAL AS PER TRANSACTION PATTERN VOLUME OF TRANSACTION IS VERY HIGH MULTIPLE NUMBER OF NONCASH CREDITED IN THE ACCOUNT FOLLOWED BY NONCASH WITHDRAWAL OBSERVED FOR EXAMPLE ONRSCRORES APPROX CREDITED IN MULTIPLE PARTS FROM DIFFERENT ENTITIES SUCH AS NEFT FROM NEXUS TRAVELS MICHELIN STAR HOLIDAYS AIONZ SOJOURN PRIVATE LIMITED ACE TRAVELS ETC TRANSFERS FROM DIVVYA TOURS AND TRAVELS OM TRAVELS SACH TRAVEL SEVEN OCEAN SURAJ TRAVELS KRISHNA TRAVELS TRIP STORE LTD ETC CASH DEPOSITS FROM VIJAYNAGARBANGALORE BRANCH MALAD BRANCH ETC AND ON THE SAME DAY RSCRORES GOT DEBITED BY RTGS INPARTS TO TRAVEL BOUTIQUE ONLINE ACC IFSCSCBL SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT AS OBSERVED THE ACCOUNTS ACTIVITY SEEN COLLECTIONS FROM VARIOUS PARTIES RELATED TO BOOKING OF HOTEL TRAVEL TICKETS AND ITS RELATED SERVICES TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS CRS TO LESS THAN RS CRS IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT AND HIGH TURNOVER STR IS BEING FILEDTRAVEL BOUTIQUE ONLINE A U/O TEK TRAPL PRIVATE LIMITED COMPANY HAVING ADDRESS P NO PHV PHV U/VIHAR NR HORIZON BPO GURGAON HARYANA INDIA EMAIL ID RAJESHJ TRAVELBOUTIQUEONLINECOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD PLOT NO B B VANIJAYA NIKUNJ ENKAY TOWER PHASEIV UDYOG VIHAR GURGAON HARYANA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AACCTK GSTIN AS PER BANK RECORDS IS AACCTKZZ TRANSACTION LINKED ACCOUNTS ARE THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AACCTK DATE OF LAST KYC UPDATION CUSTOMER IS INTO TRAVEL RELATED SUPPORT THROUGH ONLINE BASED PORTAL WITH OFFICIAL ADDRESSHTTPS //WWWTRAVELBOUTIQUEONLINECOM/ DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS CRS TO LESS THAN RS CRS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT BALANCE IN THE ACCOUNT ATWAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN BANGALORE RESIDENCY ROAD CHENNAI TRIPLICANE DELHI TAIMUR NAGAR BRANCH TRANSFERS RS LAKHS MAJORLY FROM ARYAN INTERNATIONAL TRAVEL BOUTIQUE ONLINE A U/O TEK TRAPL GALAXY TRAVEL SOLUTIONS CLEARING RS LAKHS MAJORLY FROM YATRA ONLINE PRIVATE LIMITED FLAMINGO TRANSWORLD PVT LTD SHRESTH UDAN TRAVELS NEFT RS LAKHS MAJORLY FROM CSC E GOVERNANCE SERVICES IN SUGAL AND DAMANI U S P LTD WILLOWTREE RTGS RS LAKHS MAJORLY FROM NEARBY TRAVEL SOLUTION SPICE MONEY LIMITED TRAVEL BOUTIQUE REMITTANCE RS LAKHS MAJORLY FROM MTG GLOBAL SERVICES FZ INTERNET FUND TRANSFERS RS LAKHS MAJORLY FROM R K TRAVELTLC SOLUTION KANDHARI TRAVEL ETC MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RS LAKH BY BRANCH CLEARING RS LAKHS RTGS RS LAKHS MAJORLY TO TRAVEL BOUTIQUE ONLINE TRAVEL BOUTIQUE ONLINE FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS MAJORLY IN CHENNAI G P ROAD CHENNAI TRIPLICANE BANGALORE NR RDBRANCH TRANSFERS RS LAKHS MAJORLY FROM GLOBE HOLIDAY TRAVEL BOUTIQUE ONLINE A U/O TEK TRAPL RAMBLE INDIA TOUR TRAVEL PVT LTD CLEARING RS LAKHS MAJORLY FROM EBIX TRAVELS PVT LTD FLAMINGO TRANSWORLD PVT LTD TRAVEL BOUTIQUE ONLINE NEFT RS LAKHS MAJORLY FROM TRAVCLAN TECHNOLOGY INDIA PVT LTD LOVELY HOLIDAYS CSC E GOVERNANCE SERVICES IN RTGS RS LAKHS MAJORLY FROM TRAVELBOUTIQUEONLINEAU TRAVEL BOUTIQUE ONLINE HAPPYEASYGO INDIA P REMITTANCE RS LAKHS MAJORLY FROM MTG GLOBAL SERVICES FZC SM OFFICE MTG GLOBAL SERVICES FZC SM OFFICE C MTG GLOBAL SERVICES FZC INTERNET FUND TRANSFERS RS LAKHS MAJORLY FROM R K TRAVELTLC SOLUTION KANDHARI TRAVEL ETC MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES UPI TRANSFERS RS LAKHS ININSTANCES FIXED DEPOSIT RS LAKHS",{"entities":[(4916,4941,"FOREIGN ORG"),(4950,4975,"FOREIGN ORG"),(4986,5008,"FOREIGN ORG")]}),
            ("FUNDS WERE CREDITED FROM THE BUSINESS CLIENT AND THE SAME HAS BEEN USE FOR BUSINESS PAYMENTS THROUGH REMITTANCES FOR EXAMPLE ONRSCRORES WERE CREDITED INPARTS THROUGH RTGS FROM SHIVALAY VYAPAAR PRIVATE LIMITED ACCOUNT NO IFSC CODEHDFC SR BULLION JEWELLERS ACCOUNT NO IFSC CODEUTIB ARIHANT BULLION AND JEWELS LLP ACCOUNT NO IFSC CODEKKBK ETC AND ON THE SAME DAY THE ACCOUNT WAS DEBITED WITH SAME AMOUNT INPARTS THROUGH OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR DIJLLAH JEWELLERY FZCO OF UAE IN USD FOR PAYMENT TOWARDS IMPORT OF GOLD ONRS CRORES WERE CREDITED INPARTS THROUGH RTGS FROM ARIHANT BULLION AND JEWELS LLP ACCOUNT NO IFSC CODEKKBK ELEGANCE TRADELINKS PVT LTD ACCOUNT NO IFSC CODEUTIB AND ON THE SAME DAY THE ACCOUNT WAS DEBITED WITH RS CRORES INPARTS THROUGH OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR DIJLLAH JEWELLERY FZCO OF UAE IN USD FOR PAYMENT TOWARDS IMPORT OF GOLD SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT THERE WERE TOTALBOE WERE PENDING FOR SUBMISSION SINCE ACCOUNT OPENING TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY WAS RS LAKHS WHICH SEEMS TO BE VERY HIGH IN VIEW OF NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS HIGH TURNOVER IN THE ACCOUNT AND BOE PENDING FOR SUBMISSION STR IS BEING REPORTEDAGNIS BULLION HOUSE LLP A LIMITED LIABILITY PARTNERSHIP FIRM HAVING ADDRESS P NO SEC IMT MNSG MANESAR GURGAON HARYANA INDIA CONTACT NUMBER AND EMAIL ID MOHAMMED DIJLLAHGOLDCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD PLOT NO B B VANIJAYA NIKUNJ ENKAY TOWER PHASEIV UDYOG VIHAR GURGAON HARYANA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS ABNFAP IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS ABNFAP TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORIES/BENEFICIAL OWNERS/ PARTNERS OF THE REPORTED ACCOUNT ARE MOHAMMED HAMMOODI HASHIM ALABID RESIDENT ID OF UAE SHARMIYA SHAROFUDDIN WITH PAN NO KVUPSD THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN ABNFAP COI AAN LLP AGREEMENT CUSTOMER IS INTO BULLION BUSINESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RSLAKHS TO LESS THAN RSCRORES ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT ATWAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH NEFT RS LAKHS MAJORLY FROM SHIVALAY VYAPAAR PRIVATE LIMITED RAKSHA BULLION JALAN COMPANY PVT LTD RTGS RS LAKHS MAJORLY FROM RAKSHA BULLION AGNIS BULLION HOUSE LLP SR BULLION JEWELLERS INTERNET FUND TRANSFERS RS LAKHS FROM VISTARAN COMMODITIES LLP MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO AGNIS BULLION HOUSE LLP RTGS RS LAKHS MAJORLY TO REMITTANCE RS LAKHS MAJORLY TO BRENLEY QUARTZ COMPANY LIMITED DIJLLAH JEWELLERY FZCO INTERNET FUND TRANSFER RS LAKHS TO JALAN SILVER GARDEN PRIVATE LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCESFURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF CLEARING RS LAKHS MAJORLY FROM AGNIS BULLION HOUSE LLP NEFT RS LAKHS MAJORLY FROM AGNIS BULLION HOUSE LLP SALASAR SYNTHETICS SR BULLION JEWELLERS RTGS RS LAKHS MAJORLY FROM SR BULLION JEWELLERS ARIHANT BULLION AND JEWELS LLP BILASA IMPEX REMITTANCE RS LAKHS MAJORLY FROM MOHAMMED HAMMOODI HASHIM ALABID IN USD FROM UAE FOR FOREIGN DIRECT INVESTMENT IN EQUITY INTERNET FUND TRANSFERS RS LAKHS FROM ARIHANT BULLION AND JEWELS LLP MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES FIXED DEPOSIT RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO ICICI LOMBARD GIC LTDPERSONAL LINE COLL AGNIS BULLION HOUSE LLP CUSTOM DUTY NEFT RS LAKHS MAJORLY TO MMTC PAMP INDIA PVT LTD RTGS RS LAKHS MAJORLY TO MMTC PAMP REMITTANCE RS LAKHS MAJORLY TO DIJLLAH JEWELLERY FZCO OF UAE FOR GOLD IN USD MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR INTERNET FUND TRANSFER RS LAKHS TO BHOLASONS JEWELLERS GOLD SOUK MOBILE MONEY TRANSFERS RS LAKHS ININSTANCE",{"entities":[(546,567,"FOREIGN ORG"),(881,920,"FOREIGN ORG"),(983,1004,"FOREIGN ORG"),(3113,3142,"FOREIGN ORG"),(3144,3165,"FOREIGN ORG"),(4179, 4200,"FOREIGN ORG"),(4225,4264,"FOREIGN ORG"),(444,483,"FOREIGN ORG")]}),
            ("SIMILAR TRANSACTION PATTERN WERE SEEN IN THE ACCOUNT NEGATIVE MEDIA REPORT FOUND AGAINST CUSTOMER SEBI IMPOSES RS CRORE FINE ONENTITIES FOR FOR INDULGING IN FRAUDULENT TRADING THAT LED TO ARTIFICIAL VOLUME IN THE SHARES OF SHAKTI PUMPS INDIA LTD WEBLINK IS HTTPS //WWWBUSINESSSTANDARDCOM/ARTICLE/PTISTORIES/SEBIIMPOSESRSCRFINEONENTITIESFORFRAUDTRADE HTML TEXT CAPITALMARKETSREGULATORSEBIMONDAYSHAKTIPUMPSINDIALTD IN VIEW OF HIGH VALUE TRANSACTIONS IN THE ACCOUNT AND NEGATIVE MEDIA REPORT FOUND AGAINST THE CUSTOMER STR IS BEING FILEDSHAKTI PUMPS INDIA LTD HAVING ADDRESS PLOT NO INDL AREA SECTOR PITHAMPUR DHAR MADHYAPRADESH INDIA CONTACT NUMBER AND EMAIL ID ACCOUNTS SHAKTIPUMPSINDIACOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD CHHOTI KHAJRANI MALAV PARISAR INDORE MADHYA PRADESH SINCE DATE OF INCORPORATION IS PAN OF THE ACCOUNT IS AAECSL IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS OTHER RELATIONSHIP OF CUSTOMER ARECURRENT ACCOUNTCARDFASTAGFD TRANSACTION LINKED ACCOUNT IS THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE DIRECTOR OF THE REPORTED ACCOUNT ARE SUNIL MANOHARLAL PATIDAR ADHPPP DINESH PATIDAR ADHPPA RAMESH PATIDAR AHPPPN NAVIN SUNDERLAL PATWA AGXPPK KYC DOCUMENTS OBTAINED WERE PAN CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION KYC UPDATION IS NOT DUE CUSTOMER IS BUSINESS OF SOLAR PUMPS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS GREATER THAN EQUAL TOCR MANUAL ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS AND NEGATIVE MEDIA REPORT BALANCE IN THE ACCOUNT ATWAS RS LAKHS DEBIT FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS MAJORLY FROM SHAKTI PUMPS FZESHAKTI PUMPS USA LLCFLORIDA SHAKTI PUMPS USA LLC TOTAL DEBIT SIDE RS LAKHS OF WHICH OTHERS RS LAKHS SHAKTI PUMPS INDIA LTD FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS CREDITS FOLLOWED BY DEBITS WERE SEEN FOR EXAMPLE FROMTO RS CRORES WERE CREDITED IN PARTS THROUGH TRANSFER FROM FC SETTLEMENT A/C USD INWARD REMITTANCE FROM MUTLUSU POMPA SAN TIC A IN THE CURRENCY USD FROM THE COUNTRY TURKEY FOR WHICH THE PURPOSE MENTIONED WAS ADVANCE RECEIVED AGAINST EXPORT KONG PISETH IN THE CURRENCY OF USD FROM THE COUNTRY COMBODIA FOR WHICH THE PURPOSE MENTIONED WAS EXPORT BILL COLLECTION SHAKTI PUMPS USA LLC IN THE CURRENCY USD FROM THE COUNTRY US FOR WHICH THE PURPOSE MENTIONED WAS ADVANCE RECEIVED AGAINST EXPORT AND SIMULTANEOUSLY SAME AMOUNT GOT DEBITED IN PARTS THROUGH TRANSFER TOWARDS SHAKTI PUMPS INDIA LTD ALSO FROMTO RS LAKHS WERE CREDITED IN PARTS THROUGH TRANSFER FROM FC SETTLEMENT A/C USD INWARD REMITTANCE FROM SHAKTI PUMPS FZE IN THE CURRENCY USD FROM THE COUNTRY UAE FOR WHICH THE PURPOSE MENTIONED WAS ADVANCE RECEIVED AGAINST EXPORT AND ON RSLAKHS WERE DEBITED THROUGH TRANSFER TOWARDS SHAKTI PUMPS INDIA LTD",{"entities":[(1746,1761,"FOREIGN ORG"),(1762,1781,"FOREIGN ORG"),(1790,1809,"FOREIGN ORG"),(2389,2408,"FOREIGN ORG"),(2729,2744,"FOREIGN ORG"),(2133,2153,"FOREIGN ORG")]}),
            ("INTERCONNECTED FUNDS TRANSFER TXNS WITHIN THE A/CS AMOUNTED TO RS CR UTURN TXNS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS CR WAS RECEIVED AND RS CR WAS SENT BACK TO THE SAME GROUP OF A/CS MAINTAINED WITH OTHER BANK WHICH MAINLY COMPRISE OF A/CS OF APPLE INTERNATIONAL MAHADEV TRADERS NISHA GOEL RAJNISH TRADERS GOPAL OVERSEAS AND SHRI MAHADEV JI EXPORTSDURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY SHRI MAHADEVJI EXPORTS IT WAS NOTED THAT THE PREMISE WAS ASQ FT OWNED PREMISE WHERE OFFICE SET UP WAS SEEN BUT NO SIGN BOARD OR STOCK OF GOODS COULD BE SIGHTEDON ENQUIRY IT WAS LEARNT THAT ALL THE ABOVE MENTIONED ARE FAMILY A/CS WITH PRANSHU GOEL HIS MOTHER NISHA GOEL AND HIS RELATIVESMANOJ JAIN AND AJAY KUMAR AS THE PROPRIETORS HOWEVER VIJAY GOEL FATHER OF PRANSHU GOEL WAS MANAGING THE BUSINESS ON ENQUIRING REASON FOR VIJAY GOEL NOT BEING A PROPRIETOR / RELATED TO ANY OF THE ABOVE MENTIONED A/CS IT WAS INFORMED THAT DUE TO HEALTH CONDITIONS HE NAMED HIS FAMILY MEMBERS AS THE PROPRIETORS IT IS PERTINENT TO NOTE THAT THE SAID VIJAY GOEL PAN NO AFUPGJ IS A DIRECTOR IN KVM APPARELS P LTD AND SOGO TEXTILES P LTD WHICH AS PER WEB CHECK ARE STRUCK OF FOR EFILING BY REGISTRAR OF COMPANIES IT APPEARS THAT THE PROPRIETORS MAY BE BENAMIS OF VIJAY GOEL AND JUSTIFICATIONS FURNISHED MAY HAVE BEEN FABRICATED TO SATISFY BANK OFFICIALS TO CONCEAL IDENTITY OR LINKAGE OF VIJAY GOEL TO THESE A/CSSHRI MAHADEV JI EXPORTS GOEL EXIM AND VINAYAK STEEL ARE YET TO SUBMIT BOES PERTAINING TOCASES OF AIR TOTALLING TO USD WHICH COMPRISE OFCASES USD PENDING FOR A PERIOD ABOVEMONTHS TOYEARCASES USD PENDING FOR A PERIOD ABOVEMONTHS TOMONTHS ANDCASES USD PENDING FOR A PERIOD LESS THANMONTHSAS A PART OF DUE DILIGENCE BALANCE SHEET OF SHRI MAHADEVJI EXPORTS WERE OBTAINED WHEREIN FOR YEAR ENDING MARCH DUTY DRAWBACK RECEIVED AMOUNTED TO RS CR AND ROSL WAS INDICATED AS RS CR ROSL AS PER WEB CHECK IS REBATE OF STATE LEVIES TOWARDS EXPORT OF GARMENTS IT APPEARS UNUSUAL THAT THE CUSTOMER WHO HAS INDICATED THAT HE IS INTO BUSINESS OF STEEL IMPORT HAS RECEIVED DUTY DRAWBACK CREDIT WHICH IS EXPORTERS INCENTIVE AND ROSL WHICH IS GOVERNMENT INCENTIVE TOWARDS GARMENT EXPORTS BOTH THE NATURE OF BUSINESS AND THE PRODUCT APPEARS TO CONTRADICT THE TYPE OF INCENTIVE INDICATED IN THE BALANCE SHEETIT IS ALSO NOTED THAT THOUGH THE CUSTOMER HAS FORMED ENTITY AND OPENED A/C IN THE NAME OF SHRI MAHADEV JI EXPORTS THE CUSTOMER HAS ROUTED IMPORT RELATED TXNS THROUGH THE A/CBASIS TXN PATTERN AND INFORMATION AVAILABLE IT COULD BE POSSIBLE THAT THE CUSTOMER MAY HAVE RECEIVED EXPORT CREDITS IN A/C MAINTAINED WITH OTHER BANK INDICATING AS EXPORT OF GARMENTS MAY HAVE CLAIMED EXPORT INCENTIVES LIKE DUTY DRAWBACK AND ROCL THEREBY DEPRIVING EXCHEQUER WITH SUCH EXPORT INCENTIVES AND OPENED ANOTHER A/C INDICATING STEEL IMPORT BUSINESS TO REMIT SUCH FUNDS RECEIVED OVERSEAS AS IMPORT OF STEEL SUCH TXNS MAY HAVE NO ECONOMIC RATIONALE AND MAY HAVE BEEN DONE POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING AND HAWALA SETTLEMENTS A/C OF SHRI MAHADEVJI EXPORTS IS ACTIVE WITH BALANCE OF RS/ AS ON MAY QUANTUM OF TXNS FOR CURRENT FINANCIAL YEAR IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR CORRESPONDING TXN TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE: IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CRPRANSHU GOELYEAR OLD AS A PROPRIETOR MAINTAINS A CURRENT IN THE NAME OF SHRI MAHADEV JI EXPORTSSINCE JUL AT OUR BRANCH LOCATED IN C/ ASHOK VIHAR PHASEII NEAR DEEP MARKET NEW DELHI THE ENTITY WAS FORMED ON JUN AND HAS SUBMITTED COPIES OF PAN CPTPGF GST REG CERT CPTPGFZM AND IEC CERT CPTPGF AS KYC DOCS DURING A/C OPENING CUSTOMERS REG ADDRESS WITH BANK IS ST FLOOR PLOT NO ABLOCK A WAZIRPUR INDUSTRIAL AREA NEAR SHRI RAM CHOWK NORTH WEST DELHIAND HAS REGISTERED MOBILE NO THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED THAT THE ENTITY IS INTO WHOLESALE BUSINESS OF STEEL WITH ANNUAL TURNOVER OF RS CR BASIS INTERCONNECTEDNESS OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A/CS IN THE NAMES OF SOLE PROPRIETORSHIP CONCERNS GOEL EXIM PROP NISHA GOEL/ MANDATE HOLDER PRANSHU GOEL SHRI SIDDHI VINAYAK EXPORTS PROPNISHA GOEL VINAYAK STEEL / PROP AJAY KUMAR AND APPLE INTLL PROP MANISH JAIN IT WAS INDICATED THAT THE ENTITIES ARE INTO BUSINESS OF IRON/ STEEL WITH ANNUAL TURNOVER IN THE RANGE OF RS CRRS CR BESIDES SAVINGS A/CS WERE OPENED IN THE NAMES OF PRANSHU GOEL NISHA GOEL MANISH JAIN PRANSHU GOEL JOINTLY WITH NISHA GOELAND RIYA GOEL JOINTLY WITH NISHA GOELAND IN THE NAME OF MINOR RIYA GOEL WITH NISHA GOEL AS THE NATURAL GUARDIAN THE MENTIONED A/CS WERE OPENED DURING THE PERIOD OCTOBER APR AT ASHOK VIHAR NETAJI SUBHASH PLACE ROHINI SHALIMAR BAUG AND PITAMPURA BRANCHES IN NEW DELHI WHILE REVIEWING REPORT ON ADVANCE IMPORT REMITTANCES AIR DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE AIRS WERE DONE THROUGH THE A/C OF SHRI MAHADEV JI EXPORTS ON VERIFYING THE A/C ACTIVITY RELATED A/CS WERE IDENTIFIED WHEREIN SIMILAR PATTERN AND INTERCONNECTED TXNS WERE NOTED HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED A/C TXNS SINCE APRIL MAY IT WAS NOTED THAT CHEQUE DEPOSITS AMOUNTED TO RS CR AND IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS CR MAINLY FROM SHIV ENTPS RS CR ICICI/ UBI SHREE PARWATI STEEL AND ALLOYS RS CR HDFC SHRI MAHADEV JI EXPORTS RS CR CANARA BANK AND SINGHAL STEELS RS CR ICICIINTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS CR MAINLY FROM SWASTIK OVERSEAS RS CR A/C NO AND SOHUM TRADING CO RS CR A/C NO THE SAID A/CS WOULD BE REVIEWED SEPARATELY AND STR WILL BE FILED IN CASE SUSPICIOUSSUBSEQUENT TO SUCH CREDITS ADVANCE IMPORT REMITTANCES AMOUNTED TO RS CR AND DIRECT IMPORT REMITTANCES AMOUNTED TO RS CR MAINLY FAVOURING PT UNIQUE STEEL INDONESIAINDONESIA RS CR GUANGDONG GUANGXIN GOLDTEC CHINA RS CR LEO METALS LIMITEDHK RS CR SHANDONG MENGYIN CHINA RS CR AND HONG KONG WINNERHK RS CR TOWARDS IMPORT OF COLD ROLLED STAINLESS STEEL COILIMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING SVJ EXIM SOLUTIONS RS CR ICICI/ IOB/ AXIS BALJEET SINGH RS CR ICICI AND CHITRANJAN PAL SINGH RS CR ICICI INTERNAL FUNDS TRANSFER PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING EMPIRE TRADING RS CR A/C NO AND JAINIT ENTPS RS CR A/C NO BANK HAS ALREADY FILED STR ON A/CS OF EMPIRE TRADING VIDE BATCH IDAND JAINIT ENTPS VIDE BATCH ID",{"entities":[(5824,5838,"FOREIGN ORG"),(5865,5890,"FOREIGN ORG"),(5904,5923,"FOREIGN ORG"),(5931,5946,"FOREIGN ORG"),(5964,5981,"FOREIGN ORG")]}),
            ("AS PER WEB CHECK SKIRE HK LTD HONG KONG IS A MANUFACTURER AND EXPORTER OF GENERAL MERCHANDISE LIKE BULBS ELECTRONICS ETC LINK HTTPS://WWWSKIREHKCOM/AND LAKSH JEWELLERY LLCUAE IS INTO BUSINESS OF DIAMOND PRECIOUS STONES GEMS TRADE ETC NO WEBSITE NOTED NO SPECIFIC INFORMATION WAS AVAILABLE ABOUT NATURE OF BUSINESS OR GOODS DEALT WITH BY BEST RISE ENTP LTD SUITE IT APPEARS THAT THE CUSTOMER JOWELA EXIM P LTD MAY HAVE RECEIVED EXPORT PROCEEDS FROM SKIRE HK LTD HONG KONG WHICH IS IN UNRELATED BUSINESS OR LAKSH JEWELLERY LLC UAE WHOSE BUSINESS CREDENTIALS ARE NOT VERIFIABLE BASIS INFORMATION ON PUBLIC DOMAINIMPS/NEFT/RTGS CREDITS AMOUNTED TO RSCR MAINLY FROM LAXMI DIA JEWEL P LTD RS CR SBI/ BOI SHREENATHJI GEMS AND JEWELS PLTD RS CR SBI AND DIAMOND INDIA LTD RS CR AXIS INTERNAL FUNDS TRANSFER CREDITS AMOUNTED TO RS CR MAINLY FROM RONAK GEMS P LTD RS CR A/C NO BANK HAS ALREADY FILED STR ON THE A/C OF RONAK GEMS P LTD VIDE STR NO ON ALONG WITH RIDHISIDDHI BULLION AND RELATED A/CS BASIS NEGATIVE MEDIA REPORT VIDE BATCH IDIN REGARDS TO NEGATIVE MEDIA REPORT AND DURING DEMONETISATION VIDE BATCH ID SUBSEQUENT TO SUCH CREDITS IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS CR MAINLY FAVOURING DIAMOND INDIA LTD	RS CR AXIS AND JOWELA EXIM PVT LTD RS CR HDFC/ BCBM INTERCONNECTED FUNDS TRANSFER TXNS WITHIN THE A/CS AMOUNTED TO RS CR UTURN IMPS/NEFT/RTGS TXNS WERE NOTED WHEREIN RS CR WAS RECEIVED AND RS CR WAS SENT BACK TO THE A/CS OF DIAMOND INDIA LTD KAVISH EXPORTS SHREENATHJI GEMS AND JEWELS P LTD AND LAXMI DIA JEWEL P LTD MAINTAINED WITH OTHER BANKS IT APPEARS THAT SUCH TXNS MAY HAVE BEEN DONE WITHOUT ANY ECONOMIC RATIONALE FOR LAYERING OF FUNDS OR POSSIBLY TO INFLATE TURNOVERDURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER NAME OF JOWELA EXIM P LTD WAS DIGITAL PRINTED ON GLASS BOARD IT WAS ASQ FT RENTED COMMERCIAL PREMISE WHERE OFFICE SET UP WAS SEEN WITHEMPLOYEES HOWEVER IT IS NOT KNOWN IF THE CUSTOMER HAS THE NET WORTH / BUSINESS EXISTENCE TO ROUTE SUCH HIGH VALUE AND VOLUME OF TXNSTXNS IN THE A/C OF JOWELA EXIM P LTD FAR EXCEED THE ANNUAL TURNOVER INDICATED AT THE TIME OF A/C OPENING NO PAYMENTS TOWARDS TAXES WERE NOTED THROUGH THE SAID A/C DURING REVIEW PERIOD FURTHER TOTAL TAX PAYMENTS AMOUNTING TO RS LAKHS WERE NOTED ONLY FROM THE A/C OF PRITI SUNIL DOKANIA DURING REVIEW PERIODCUSTOMER JOWELA EXIM P LTD IS YET TO SUBMIT DOCUMENTS PERTAINING TOCASES OF BILL REALIZATIONS TOTALLING TO USD PENDING FOR A PERIOD LESS THAN A YEARBASIS TXN PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED IN THE NAME OF JOWELA EXIM P LTD AND A/C MAY HAVE BEEN OPENED TO RECEIVE EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS AND SUCH TXNS MAY NOT HAVE ANY ECONOMIC RATIONALE FURTHER FUNDS RECEIVED MAY HAVE BEEN CHURNED WITHIN THE A/CS AND REMITTED AS RTGS/NEFT REMITTANCES FOR LAYERING OF FUNDS A/C OF JOWELA EXIM P LTD IS ACTIVE WITH BALANCE OF RS CR AS ON JUNE QUANTUM OF TXNS FOR CURRENT FINANCIAL YEAR IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR CORRESPONDING TXN TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE: IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR IS DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CR : DEBIT: RS CR IN CASH RS CR AND CREDITS: RS CR IN CASH RS CRA CURRENT A/C BEARING NOWAS OPENED IN THE NAME OF JOWELA EXIM P LTD ON AUG AT OUR BRANCH LOCATED IN OM SHREE GANESH CHS SHOP NO CHIKOOWADI BORIVALI WEST MUMBAI MAHARASHTRA THE ENTITY WAS FORMED ON JUL AND HAS SUBMITTED COPIES OF PAN AAECJH BOARD RESOLUTION CERT OF INCORP UMHPTC AND MOA/ AOA AS KYC DOCS AT THE TIME OF A/C OPENING CUSTOMERS REG ADDRESS WITH THE BANK ISAURA BIPLEX SENATE SV ROAD BORIVALI WEST MUMBAI PRANAV SUNIL DOKANIA BHAVIN CHANDRAKANT SHAH AND HARDIK BHARAT SHAH ARE THE AUTHORISED SIGNATORIES WHILE BHAVIN CHANDRAKANT SHAH IS THE BENEFICIAL OWNER PRANAV SUNIL DOKANIA BHAVIN CHANDRAKANT SHAH AND HARDIK BHARAT SHAH HAVE REGISTERED MOBILE NOS ANDRESPECTIVELY WITH THE BANK THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF A/C OPENING THAT THE ENTITY IS INTO TRADING OF JEWELLERY BUSINESS WITH ANNUAL TURNOVER OF RS CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED A/C OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF CURRENT A/CS IN THE NAMES OF PRIT DIAJEWELS P LTD SHREENATHJI GEMS AND JEWELS P LTD AND LAXMI DIA JEWEL P LTD PRANAV DOKANIA AND PRITI DOKANIA ARE THE COMMON DIRECTORS IN THE MENTIONED A/CS IT WAS INDICATED THAT THE ENTITIES ARE INTO BUSINESS OF JEWELLERY WITH ANNUAL TURNOVER IN THE RANGE OF RS CR TO RS CR BESIDES SAVINGS A/C WERE OPENED IN THE NAMES OF PRANAV SUNIL DOKANIA PRITI SUNIL DOKANIA JOINTLY WITH PRANAV SUNIL DOKANIA // SUNIL DOKANIA HUF WITH PRANAV SUNIL DOKANIA AS KARTA BHAVIN CHANDRAKANT SHAHAND HARDIK BHARAT SHAH THE MENTIONED A/CS WERE OPENED DURING THE PERIOD JULY TILL OCT AT BORIVALI W J B NAGAR BORIVALI CCIRCLE BRANCHES IN MUMBAI NO TXNS / UNUSUAL TXNS WERE NOTED IN THE A/CS MENTIONED ABOVE DURING REVIEW PERIOD EXPECT IN THE A/CS OF JOWELA EXIM P LTD PRITI SUNIL DOKANIA JOINTLY WITH PRANAV SUNIL DOKANIA / PRIT DIAJEWELS P LTDAND LAXMI DIA JEWEL PRIVATE LTD WHILE REVIEWING REPORT ON EXPORT ADVANCE REMITTANCE CREDITS RECEIVED IN A/CS MAINTAINED WITH THE BANK IT WAS NOTED THAT HIGH VALUE EXPORT ADVANCE REMITTANCE CREDITS WERE RECEIVED IN THE ABOVE MENTIONED A/C OF JOWELA EXIM P LTD ON VERIFYING THE A/C ACTIVITY IT WAS NOTICED THAT SUCH CREDITS WERE FOLLOWED BY HIGH VALUE IMPS/NEFT/RTGS PAYMENTS HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLL GROUNDS OF SUSPICION:ON REVIEWING CONSOLIDATED A/C TXNS SINCE APRIL JUNE IT WAS NOTED THAT FUNDS TOTALLING TO RS CR WERE RECEIVED IN THE A/C OF JOWELA EXIM PRIVATE LTD BY WAY OF EXPORT ADVANCE REMITTANCE RS CR AND DIRECT BILL REALIZATION RS CR MAINLY FROM SKIRE HK LTDHONG KONG RS CR LAKSH JEWELLERY LLC UAE RS CR AND BEST RISE ENTP LTD SUITEHONG KONG RS CR TOWARDS EXPORT OF INDIAN MADE JEWELLERY CONTAINING ASSORTED KT GOLD",{"entities":[(17,28,"FOREIGN ORG"),(448,459,"FOREIGN ORG"),(5872,5883,"FOREIGN ORG"),(5900,5918,"FOREIGN ORG"),(152,170,"FOREIGN ORG"),(505,523,"FOREIGN ORG")]}),
            ("MARIYAM ENTERPRISE MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RSLAKHS FY FROMTO : TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKH REMITTANCE RS LAKHS TRANSACTION PATTER SHOWS FUNDS CREDITED THROUGH INWARD REMITTANCE INTERNET FUND TRANSFERS AND WHICH IS IMMEDIATELY DEBITED THROUGH INTERNET FUND TRANSFERS FOR EXAMPLE ONRS CRORES CREDITED THROUGH INWARD REMITTANCE FROM M/S SIDDHART ENTERPRISE BANGLADESH IN USD CURRENCY AND REASON WAS MENTIONED AS CUMINJEERALOOSE FROM M/S SHOTOTA TRADERS BANGLADESH IN IN USD CURRENCY AND REASON WAS MENTIONED AS INDIAN WHEAT AND ONRSCRORES DEBITED THROUGH INTERNET FUND TRANSFERS TO MANGALAM ENTERPRISES DIAMOND FRUIT COMPANY ALSO ONRSLACS CREDITED FROM RKP ENTERPRISE BANGLADESH IN IN USD CURRENCY AND REASON WAS MENTIONED AS DRY RED CHILLY GENUS CAPSICUM AND ONRSLACS DEBITED THROUGH RTGS TO POTENT PRODUCTS PVT LTD ACCOUNT NUMBER: IFSC CODE UTIB SITALADAS GOVINDRAM KEWLANI ACCOUNT NUMBER IFSC CODE UCBA SIMILAR PATTERN WAS OBSERVED WHERE HIGH VALUE TRANSACTION INWARD REMITTANCE AND HIGH CREDIT TURNOVER IN THE FINANCIAL YEAR FY :TILLIS RS CRORES WHICH IS VERY HIGH INMONTHS AS COMPARED TO THE DECLARED TURNOVER TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS LAC TO LESS THAN RS CRS IN VIEW OF LARGE VALUE NON CASH TRANSACTION IN CURRENT ACCOUNT AND HIGH TURNOVER STR IS BEING FIELDMTE EXIM PRIVATE LIMITED A PRIVATE LTD COMPANY HAVING ADDRESS: ARMENIAN STREET TH FLOOR BURRABAZAR ROOM NO KOLKATA WEST BENGAL INDIA CONTACT NUMBER: + AND EMAIL ID: MTEEXIM_@YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD KOLKATA PORTUGUESE CHURCH STREET ICICI BANK LTD CATHEDRAL ANNEXE GROUND FLOORPORTUGUESE CHURCH STREET KOLKATA WEST BENGAL SINCE DATE OF INCORPORATION IS RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AANCMP GSTIN AS PER BANK RECORDS IS AANCMPZM OTHER RELATIONSHIP OF CUSTOMER ARE TRANSACTION LINKED ACCOUNTS ARE BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS JAYEETI JAISWAL WITH PAN NO ASJPCG SUMAN JAISWAL WITH PAN NO ASYPPN THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE ENTITY PROOF MOA AOA BOARD RESOLUTION PAN AANCMP CINUWBPTC CUSTOMER PROFILE IS INTO FOOD GRAINS AS PER BANK RECORDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LAC TO LESS THAN RS CRS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTION IN CURRENT ACCOUNT BALANCE IN THE ACCOUNT ATWAS RS LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH RS BY CASH CLEARING RSLAKHS FROM MAA TARA EXIM PRIVATE LIMITED S P ENTERPRISE NEFT RS FROM EDI DUTY DRAWBACK DISBURSAL RTGS RS LAKHS FROM MAHEMUD PAN HOU JP FLOUR MILLS ABHI TRADERS REMITTANCE RS LAKHS FROM M/S RAJ ENTERPRISE RKP ENTERPRISE M/S SIDDHART ENTERPRISE INTERNET FUND TRANSFERS RSLAKHS FROM MTE EXIM PRIVATE LIMITED MOBILE MONEY TRANSFERS RS ININSTANCES AND OTHERS RS TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH CASH WAS RSLAKHS BY BRANCH CLEARING RS LAKHS TO SRIVISHNUPRIYA CHILLI ANILTRADING INDIA PVT LTD BALASAHEB GANGARAMSHE RTGS RS LAKHS MAJORLY TO TO KRS TRADERS SRI VISHNU PRIYA CHILL MANGALAM ENTERPRISES DIAMOND FRUIT COMPANY REMITTANCE RS LAKHS INTERNET FUND TRANSFER RS LAKHS MAJORLY TO S P ENTERPRISE MARIYAM ENTERPRISES MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RS LAKHS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS YEARWISE WERE: FY FROMTO : TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL TRANSFERS RS LAKHS FROM MTE EXIM PRIVATE LIMITED S P ENTERPRISE CLEARING RS LAKHS FROM CMS_ MAA TARA EXIM PRIVATE LIMITED S P ENTERPRISE NEFT RS LAKHS FROM BONGAON SBI BASIRHAT GHOJADANGA CUSTOMS RTGS RS LAKHS FROM SNOWBLUE ENCLAVE PVT LTD ABHI TRADERS DILIP KUMAR CHANDRA REMITTANCE RS LAKHS FROM M/S RAJ ENTERPRISE RKP ENTERPRISE M/S SIDDHART ENTERPRISE INTERNET FUND TRANSFERS RS LAKHS MAJORLY FROM MTE EXIM PRIVATE LIMITED MARIYAM ENTERPRISE MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES AND OTHERS RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RSLAKHS BY BRANCH TRANSFERS RS LAKHS TO MAMA EXPORT KISSAN JAI KISSAN TRADERS MTE EXIM PRIVATE LIMITED CLEARING RS LAKHS TO KISSANJAIKISSANTRADE MAAVINDHWASHINITRADE SRIVISHNUPRIYACHILLI NEFT RS LAKHS TO NANDHINI CHILLIES HK ORGANIC MANOJ TRADING COMPAN RTGS RS LAKHS TO KRS TRADERS SRI VISHNU PRIYA CHILL REMITTANCE RS LAKHS INTERNET FUND TRANSFER RS LAKHS MAJORLY TO MTE EXIM PRIVATE LIMITED",{"entities":[(458,476,"FOREIGN ORG"),(558,572,"FOREIGN ORG"),(774,787 ,"FOREIGN ORG"),(3019,3032,"FOREIGN ORG"),(3038,3056,"FOREIGN ORG"),(4068,4081,"FOREIGN ORG"),( 4087,4105,"FOREIGN ORG")]}),
            ("SOURCE OF FUNDS IS FROM SAME ENTITIES TO MULTIPLE COMPANIES UNDER THE SAME GROUP FOLLOWED BY OUTWARD REMITTANCE SEEMS TO BE UNUSUAL TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL INCOME OF RS LACS IN VIEW OF HIGH TURNOVER IN NEWLY OPENED GET CLOSED IN SHORT SPAN OF TIME PERIOD AND HIGH TURNOVER STR IS BEING FILEDWATERMARK STUDIOS PARTNERSHIP FIRM HAVING ADDRESS: ND FLOOR / FLAT NO RELIANCE CLASSIC ABOVE ANDHRA HYDERABAD TELANGANA INDIA CONTACT NUMBER: + AND EMAIL ID: HI@WATERMARKSTUDIOSIN IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LIMITED HYDERABAD BEGUMPET //A JAYALAKSHMI TOWERS MOTILALNEHRU NAGAR PRAKASHNAGAR BEGUMPET HYDERABAD TELANGANA SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS AADFWA GSTIN AS PER BANK RECORDS IS AADFWAZ TRANSACTION LINKED ACCOUNTS IS BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH CHAWLA WITH PAN NO AGRPCE MOHIT BANSAL WITH PAN NO ALRPBC THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON KYC DOCUMENTS OBTAINED WERE AADHAR PARTNERSHIP LETTER PAN AADFWA GSTIN AS PER ENHANCE DUE DILIGENCE CUSTOMER IS INTO CINE PRODUCTION SERVICES PROVIDER FOR MOVIE OR FILM PRODUCTION OBSERVED THE BUSINESS BOARD OF THREE ENTITIES IE CLOUDZ TOURS AND TRAVEL DIGI MEDIA AND FILMS SOLUTIONS WATER MARK STUDIO TINGLE CINE PRODUCTIONS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LACS ALERT WAS TRIGGERED FOR HIGH TURNOVER IN NEWLY OPENED GET CLOSED IN SHORT SPAN OF TIME PERIOD BALANCE IN THE ACCOUNT ATWAS NIL FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY FROMTO : TOTAL CREDIT SIDE RS LAKHS OF WHERE CASH IS NIL CLEARING RS LAKH MAJORLY FROM DIGI MEDIA NEFT RS LAKHS MAJORLY FROM DIVERSE MEDIA BRASH FILM SERVICES RTGS RS LAKHS MAJORLY FROM MAGNETIC FILMS PRODUCTION LIQUID STUDIOS AND SERVICE DIVERSE MEDIA REMITTANCE RS LAKHS MAJORLY FROM ICICI BANK LIMITED HONG KONG IN USD FOR REVERSAL OF WRONG ENTRIES MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH CLEARING RS LAKHS MAJORLY TO T VINAY ASSOCIATES RTGS RS LAKHS MAJORLY TO BRASH FILMS SERVICES BRASH FILM LIQUID STUDIO REMITTANCE RS LAKHS MAJORLY TO TRISTAR FILMS LIMITED TRINITY WORLD PTE LIMITED TRISTAR FILMS LIMITED MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES FUNDS ARE GETTING CREDITED FROM LIQUID STUDIOS AND SERVICES BRASH FILM SERVICES MAGNETIC FILMS PRODUCTION AND ENTIRE FUNDS ARE MADE OUTWARD REMITTANCE TO TRISTAR FILMS LIMITED IN USD TO SINGAPORE FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE NO PROPER DETAILS FOUND FOR CREDITED SIDE PARTIES ON PUBLIC DOMAIN OF LIQUID STUDIOS AND SERVICES BRASH FILM SERVICES MAGNETIC FILMS AND OUTWARD REMITTANCE MADE TO TRINITY WORLD PTE WHICH IS INTO CINE PRODUCTION COMPANY WWWTRINITYWORLDPTELTDCOM CUSTOMER WAS RELUCTANT WHILE SHARING DETAILS AND SUDDENLY STOP TRANSACTION FOR EXAMPLE ONRS CRORES GOT CREDITED BY RTGS FROM DIVERSE MEDIA IFSC YESB ACCAND ON THE SAME DAY RS CRORES GOT DEBITED BY OUTWARD REMITTANCE TO TRISTAR FILMS LTD ADD BEDOK RESEROIR ROAD SINGAPORE IN USD WITH INFORMATION AS TOWARDS AUDIO VISUAL SERVICES IN OTHER EXAMPLE ONRS CRORES GOT CREDITED BY RTGS FROM LIQUID STUDIOS AND SERVICES IFSC SBIN ACCAND ON THE SAME DAY RS CRORES GOT DEBITED BY OUTWARD REMITTANCE TO TRISTAR FILMS LTD ADD BEDOK RESEROIR ROAD SINGAPORE IN USD WITH INFORMATION AS TOWARDS AUDIO VISUAL SERVICES",{"entities":[(2353,2373,"FOREIGN ORG"),(2375,2399,"FOREIGN ORG"),(2401,2421,"FOREIGN ORG"),(2621,2641,"FOREIGN ORG"),(2876,2892,"FOREIGN ORG"),(3176,3192,"FOREIGN ORG"),(3448,3464,"FOREIGN ORG")]}),
            ("ON REVIEWING THE TRANSACTION PATTERN OF STORAIL SOFTWARES PVT LTD ACIT WAS OBSERVED THAT DURING ENHANCE DUE DILIGENCE CONDUCTED BY BANK OFFICIAL ON // IT WAS FOUND CLIENT STORAIL SOFTWARES PVT LTD IS PROVIDING CLOUD STORAGE SERVICES TO DOMESTIC CLIENT ZICOM ELECTRONIC SECURITY SYSTEMS LTD INDIA AND RECEIVES CLOUD STORAGE FROM OVERSEAS ENTITY SAFE DATA STORAGE LTD CLIENT WAS ATTEMPTING TO MAKE OUTWARD REMITTANCE TO OVERSEAS ENTITY SAFE DATA STORAGE LTD TOWARDS PURCHASE OF CLOUD STORAGE SERVICES CLIENT STORAIL SOFTWARES PVT LTD HAD SUBMITTED AGREEMENT COPY OF BOTH THE ENTITIES ZICOM ELECTRONIC SECURITY SYSTEMS LTD AND SAFE DATA STORAGE LTDON CONDUCTING FURTHER DUE DILIGENCE AND REVIEW OF DOCUMENT IT WAS FOUND THAT THE AGREEMENT COPY WAS FAKE DOMESTIC CLIENT ZICOM ELECTRONIC SECURITY SYSTEMS LTD HAS ALSO CONFIRMED THAT THE AGREEMENT IS NOT SIGNED BY ANY AUTHORIZED PERSON IN THE COMPANY THEY HAVE NO BUSINESS WITH STORAIL SOFTWARE PVTLTD THEY ALSO DENY ANY ASSOCIATION WITH THE SAID COMPANY AS PER AGREEMENT SHARED BY USDOMESTIC ENTITY ZICOM ELECTRONIC SECURITY SYSTEMS LTD HAS ALSO REMITTED FUND AMOUNTING TO INR CR TO ANOTHER YES BANK CUSTOMER STOCKPILE SOFTECH SOLUTIONS PVT LTD ACFOR WHICH STR WAS FILED VIDE FIU BATCH ID SAFE DATA STORAGE LTD IS A COMMON OVERSEAS ENTITY RECEIVING CROSS BORDER REMITTANCE FROM YBL CUSTOMER SUPERNITCO INDIA OVERSEAS FOR WHICH STR WAS FILED VIDE FIU BATCH ID YBL CUSTOMER GREEN GOLD ENT FOR WHICH STR WAS FILED VIDE FIU BATCH IDAND STR WAS FILED ON NON CUSTOMER SPAN SOFTWARE SOLUTIONS PRIVATE LIMITED VIDE FIU BATCH IDTHAT HAD ATTEMPTED OUTWARD REMITTANCE IN FAVOR OF SAFE DATA STORAGE LTDTHERE ARE NO OTHER BUSINESS TRANSACTIONS IN THE ACCOUNTS AND ARE SUSPECTED TO BE USED WITH THE SOLE INTENSION OF REMITTING THE FUNDS ABROADTHE TRANSACTION WAS DECLINED BY BANK FOR FURTHER PROCESSINGDUE TO SUSPICIOUS AGREEMENT COPY AND DOMESTIC/OVERSEAS COUNTERPARTY THAT APPEARS IN DEFAULTER LIST IT GIVES RISE TO SUSPICION THAT THE CUSTOMER IS INDULGING IN MONEY LAUNDERING IN GUISE OF TRADE TRANSACTIONS WE SUSPECT THAT CUSTOMER WAS INTENDING TO USE HIS CURRENT ACCOUNT FOR ROUTING THE MONEY FROM THIS ARRANGEMENTIN LIGHT OF THE ABOVE FACTS IT HAS BEEN DECIDED ON // TO FILE A STR ON THE SAID CUSTOMER THE CUSTOMER STORAIL SOFTWARES PVT LTD PAN ABFCSN DOI COMMUNICATION PERMANENT ADDRESS INDUSTRIAL PLOT NO E PHASE A INDUSTRIAL AREA SAS NAGAR MOHALI PUNJAB REGISTRATION NUMBER CIN UPBPTC MOBILE NUMBER EMAIL ID GURSIMRAN@STORAILSOFTWARESCOM IS MAINTAINING A CURRENT AC NOSINCEWITH CHANDIGARH BRANCH ADDRESS SCO SECTOR C CHANDIGARH THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN FEB GURSIMRAN SINGH NAVDEEP SINGH ARE AUTHORIZED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS AN IT SERVICE PROVIDER WITH AN ANNUAL GROSS TURNOVER BETWEEN RS CRCRIN THE ACCOUNT OF STORAIL SOFTWARES PVT LTD AC NO TRANSACTIONS ANALYSIS FROMTILL SHOWS NIL CREDIT/DEBITS TRANSACTIONS IN THE ACCOUNT OF STORAIL SOFTWARES PVT LTD AC NO TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW: FROMTILLTRANSACTIONS ANALYSIS SHOWSCREDIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF CASH DEPOSIT AMOUNTING TO RSANDDEBIT TRANSACTIONS AMOUNTING TO RS COMPRISING OF NIL CASH WITHDRAWALNO TRANSACTIONS WERE OBSERVED FROM // TO // AS ACCOUNT WAS OPENED ON TH FEB BALANCE IN THE ACCOUNT AS ONIS RS",{"entities":[( 344,364,"FOREIGN ORG"),(434,454,"FOREIGN ORG"),(624,644,"FOREIGN ORG"),(1615,1635,"FOREIGN ORG")]}),
            ("SAMURAI TRADEX A PROPRIETORSHIP FIRM HAVING ADDRESS : B BLOCK MADANGANJ KISHANGARH VINAYAK NAGAR JAIPUR ROAD AJMER RAJASTHAN INDIA CONTACT NUMBER: + AND EMAIL ID: SAMURAITRADEX@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD INDIA SHOPPING MALL INDIA MOTOR CIRCLE KUTCHERI ROAD AJMER RAJASTHANSINCE DATE OF FORMATION IS PAN OF THE ACCOUNT IS ANMPPC GSTIN AS PER BANK RECORDS IS ANMPPCZ TRANSACTION LINKED ACCOUNTS ARE AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS RAKESH PANCHOLI PROPRIETOR ANMPPC RAHUL PANCHOLY AUTHORISED SIGNATORY ANPPPQ THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AND LETTER OF DECLARATION OFPROPRIETORSHIPFIRM KYC UPDATION IS NOT DUE CUSTOMER IS INTO BUSINESS OF FOOD GRAIN DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LAC TO CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT ATWAS RS LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR: FY FROMTILLSHOWS TOTAL CREDIT RS LAKHS OF WHICH RTGS RS LAKHS MAJORLY FROM GANESH AGRO INDUSTRIES HINAL ENTERPRISE INTERNET FUND TRANSFERS RS LAKHS FROM RAHUL AGRO INDUSTRIES ETC FIXED DEPOSIT RS LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS LAKHS OF WHICH RTGS RS LAKHS TO MK INDUSTRIES INTERNET FUND TRANSFER RS LAKHS TO RAHUL AGRO INDUSTRIES ETC FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY FROMTO: TOTAL CREDIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY FROM RAHUL AGRO INDUSTRIES KAPISH INTERNATIONAL RTGS RS LAKHS MAJORLY FROM HINAL ENTERPRISE GUJARAT PEANUT PRODUCT G S TRADING CO REMITTANCE RS LAKHS MAJORLY FROM AGRI COMMODITIES AND FINANCE FZ LLC KOREA AGROFISHERIES AND FOOD TRADE INTERNET FUND TRANSFERS RS LAKHS FROM SAGAR INTERNATIONAL ETC FIXED DEPOSIT RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH TRANSFERS RS LAKHS MAJORLY TO SHREESHIV ENTERPRISE RAHUL AGRO INDUSTRIES SAMURAI TRADEX NEFT RS LAKHS MAJORLY TO SHREE BHANDERI INDUSTRIES GANESH ENTERPRISES RTGS RS LAKHS MAJORLY TO GUJARAT PEANUT PRODUCT REMITTANCE RS LAKHS MAJORLY TO KOKAI INDO FOOD STUFF TRADING LLC INTERTEK VIETNAM LIMITED INTERNET FUND TRANSFER RS LAKHS TO RAHUL AGRO INDUSTRIES ETC FY FROMTO: TOTAL CREDIT SIDE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS AS PER TRANSACTION IT IS OBSERVED THAT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE ONRS CRORE WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM SAGAR INTERNATIONALAND ON THE SAME DAY RS CRORE WERE DEPLOYED THROUGH INTERNET FUND TRANSFER TO RAHUL AGRO INDUSTRIES ALSO ONRSLACS WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM RAHUL AGRO INDUSTRIES AND ON THE SAME DAY SAME AMOUNT WERE DEBITED THROUGH RTGS TO M K INDUSTRIES UTIB SIMILAR TRANSACTION PATTERN WERE SEEN IN THE ACCOUNT IN VIEW OF LARGE VALUE ROUTING OF FUNDS WITHIN ACCOUNTS WITH HIGH TURNOVER STR IS BEING REPORTED",{"entities":[(1776,1810,"FOREIGN ORG"),(2204,2236,"FOREIGN ORG"),(2238,2261,"FOREIGN ORG")]}),
            ("SAGAR ENTERPRISE PARTNERSHIP FIRM HAVING ADDRESS:PLOT NOTOND FLR KATARGAM SKY HIGH BLDGVASTA DEVDI RD OPP MEHTA PETROL PUMP CASA NGR SURAT GUJARAT INDIA CONTACT NUMBER: + AND EMAIL ID: AKSAGAR@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD SHREE SHYAM CHAMBERS RING ROAD SURAT GUJARAT SINCE DATE OF INCORPORATION IS PAN NUMBER OF THE ACCOUNT IS ABXFSH IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE ARVIND K SHAH PTR ADOPSH JIGISHABEN P SHAH PTR AQRPSG DILIP D SHAH PTR ACQPSH CONSIDERING THAT THERE ARE AN ADDITIONAL AUTHORIZED SIGNATORIES THEIR DETAILS ARE PROVIDED IN RESPECTIVE FIELDS WITHIN THE XML FILE BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ARVIND KESHVLAL SHAH WITH PAN NO ADOPSH DILIP D SHAH WITH PAN NO ACQPSH JIGISHABEN PRAWNESH BHAI WITH PAN NO AQRPSG THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN ABXFSH PARTNERSHIP DEED DATE OF LAST KYC UPDATION CUSTOMER IS INTO BUSINESS IMPORT AND EXPORT OF DIAMONDS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT BALANCE IN THE ACCOUNT ATWAS RS DEBIT FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUSYEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE: FY FROMTO: TOTAL CREDIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS FY FROMTO: TOTAL CREDIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS FY FROMTO: TOTAL CREDIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS FY FROMTO: TOTAL CREDIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS TOTAL DEBIT SIDE RS LAKHS OF WHICH RSBY CASH REMITTANCE RS LAKHS TRANSACTION PATTERN SHOWS ACCOUNT WAS CREDITED BY FOREIGN INWARD REMITTANCE FROM SAMBHAV DIAM HK LTD HONKONG IN USD AND SAME IS DEBITED THROUGH REMITTANCES ROR EXAMPLE ONUSDLACS CREDITED THROUGH GRS/GRS SAMBHAV DIAM HK LIMITED FLAT WHICH IS DEBITED TOWARDS VINAY DIAMONDS DIAMOND BOURSE BANDRA KURLA COMPLEX MUMBAI INDIA : REMITTANCE INFORMATION PAYMENT AGAINST INVOICE ONFOR THE SAME VALUE OF FUND THROUGH REMITTANCE ON ANOTHER EXAMPLE ONUSD LACS CREDITED THROUGH GRS/GRS SAMBHAV DIAM HK LIMITED FLAT OUT OF WHICH SAME VALUE OF FUND DEBITED THROUGH REMITTANCE ON TO PADMA GEMS DIAMOND BOURSE BANDRA KURLA COMPLEX MUMBAI INDIA : REMITTANCE INFORMATION PAYMENT AGAINST INVOICE ON THE SAME DATE HIGH VALUE FUNDS WERE CREDITED AND IMMEDIATELY DEBITED THROUGH REMITTANCES LEAVING VERY LOW BALANCE AT EOD NO OTHER BUSINESS RELATED ACTIVITY OBSERVED INTO THE ACCOUNT ALSO THERE ARE NO TRANSACTION IN THE ACCOUNT AFTER DEC IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS STR IS BEING FILED",{"entities":[(1989,2007,"FOREIGN ORG"),(2111,2125,"FOREIGN ORG"),(2381,2403,"FOREIGN ORG")]}),
            ("IN OTHER EXAMPLE ONRS CRORE GOT CREDITED THROUGH RTGS FROM MATRIX SECURITY STATE BANK OF INDIA IFSCSBIN REF WHICH IS DEPLOYED ON SAME DAY WITH RS CRORES THROUGH REMITTANCE TOWARDS SAFE DATA STORAGE LTD IN USD WHERE PURPOSE IS MENTIONED AS PAYMENT FOR DATA BASE DATA PROCESSING CHARGES SIMILAR TRANSACTION PATTERN SEEN IN THE ACCOUNT THERE SEEMS TO BE NO ECONOMIC RATIONALE FOR SUCH TRANSACTIONS HIGH VALUE TURNOVER OF CRORE OBSERVED IN SHORT SPAN OFMONTHS TOTAL TURNOVER IN THE FINANCIAL YEAR WAS RS LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS LACS TO RS CRS IN VIEW OF LARGE VALUE ROUTING OF FUNDS WITHIN ACCOUNTS AND HIGH TURNOVER STR IS BEING FILEDORCHAD TECHNO SOLUTIONS LLP LIMITED LIABILITY PARTNERSHIP HAVING ADDRESS PLATINUM SQUARE SR NO/A/B VADGOAN SHERI OFFICE NO PUNE MAHARASHTRA INDIA CONTACT NUMBER AND EMAIL ID ACCOUNTS ORCHADTECHNOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBERWITH ICICI BANK LTD PANCHSHIL TECH PARK NEAR GANAPATHI CHOWK / VIMAN NAGAR PUNE MAHARASHTRA SINCE DATE OF INCORPORATION IS PAN OF THE ACCOUNT IS AAGFOG GSTIN AS PER BANK RECORDS IS AAGFOGZO TRANSACTION LINKED ACCOUNTS ARE BENEFICIAL OWNERAND AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT IS DEEPAK WITH PAN BFVPDH SACHIN AGGARWAL WITH PAN AXYPAQ THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF ACCOUNT IS CLOSED ON KYC DOCUMENTS OBTAINED WERE AADHAR PAN AAGFOG OF FIRM GSTIN KYC UPDATION IS NOT DUE AS PER INFORMATION FOUND IN PUBLIC DOMAIN CUSTOMER IS ENGAGED INTO COMPUTER AND RELATED ACTIVITIES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS LACS TO RS CRS A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE ROUTING OF FUNDS WITHIN ACCOUNTS BALANCE IN THE ACCOUNT ATWAS RS FURTHER THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER YEARWISE WERE FY FROMTO TOTAL CREDIT SIDE RS LAKHS OF WHICH CASH WAS NIL CLEARING RS LAKHS MAJORLY FROM DEEPAK NEFT RS LAKHS MAJORLY FROM MATRIX SECURITY DEEPAK MOHAN JETHMALANI RTGS RS LAKHS MAJORLY FROM MATRIX SECURITY TOTAL DEBIT SIDE RS LAKHS OF WHICH CASH WAS RS LAKHS BY BRANCH TRANSFERS RS LAKHS RTGS RS LAKHS MAJORLY TO MATRIX SECURITY SPARSH SECURITIES REMITTANCE RS LAKHS MAJORLY TO SAFE DATA STORAGE LTD PROFISEA LTD INTERNET FUND TRANSFER RS LAKHS MAJORLY TO E SQUARE IT MAKEMYTRIP INDIA PVT LTD CARD TRANSACTIONS RS LAKHS MOBILE MONEY TRANSFERS RS LAKHS ININSTANCES ALERT WAS RAISED AS CUSTOMER IS ROUTING MONEY BY USING NAME OF BIG BRANDS LIKE MATRIX SECURITIES SURVEILLANCE SPARSH SECURITIES SAFE STORAGE IN ACTUAL THESE COMPANIES HAS NO RELATION WITH IT AS PER INVOICE AND AGREEMENT UPLOADED BY SAFE DATA STORAGE LTD WHICH IS UK BASED COMPANY THE PAYMENT WAS GOING TO SINGAPORE ACCOUNT BANK HAD REPORTED THIS CLIENT IN DEC END AND CLOSED IT IN WHICH IS SUSPICIOUS AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT WHICH SEEMS TO BE UNUSUAL FOR EXAMPLE ONRS CRORE GOT CREDITED THROUGH RTGS FROM MATRIX SECURITY STATE BANK OF INDIA IFSCSBIN REF WHICH IS DEPLOYED ON SAME DAY WITH SAME AMOUNT THROUGH REMITTANCE TOWARDS SAFE DATA STORAGE LTD UNIT A VICTORIA WORKSFAIRWAY PETTS WOOD UNITED KINGDOM IN USD WHERE PURPOSE IS MENTIONED AS PAYMENT FOR DATA BASE DATA PROCESSING CHARGES",{"entities":[(180,200,"FOREIGN ORG"),(2271,2291,"FOREIGN ORG"),(2293,2304,"FOREIGN ORG"),(2585,2596,"FOREIGN ORG"),(2689,2709,"FOREIGN ORG"),(3233,3253,"FOREIGN ORG"),(3262,3293,"FOREIGN ORG")]}),
            ("GROUNDS OF SUSPICION THIS IS AN EMPLOYEE INITIATED CASE. THE ACCOUNT NUMBER 22105082801 OF ISOBEL TRADEX PRIVATE LIMITED AND ACCOUNT NUMBER 22105093277 OF CAPTECK OVERSEAS PRIVATE LIMITED CAME UNDER AML REVIEW DURING THE INTERNAL REVIEW PROCESS OF THE BANK. THE CUSTOMER ISOBEL TRADEX PRIVATE LIMITED HAD SUBMITTED A REQUEST TO MAKE DIRECT IMPORT PAYMENT OF USD 350,000 TO COUNTERPARTY RONGTAI HARDWARE PRODUCTS LTD HAVING CORPORATE ADDRESS IN CHINA AND REGISTERED ADDRESS IN HONG KONG TOWARDS IMPORT OF MICRO CIRCUIT FOR HIGH ILLUMINATING CEILING LIGHTS. THE INVOICE DATE WAS 16 OCT 19 WITH TOTAL INVOICE VALUE OF USD 507,000. AS PART OF ADDITIONAL DUE DILIGENCE, A SWIFT MESSAGE WAS SENT TO THE BENEFICIARY BANK I.E. HSBC, CHINA TO PROVIDE THE KYC DETAILS OF THE COUNTERPARTY FOR WHICH THERE WAS NO REVERT RECEIVED IN SPITE OF SENDING MULTIPLE REMINDERS. IT WAS OBSERVED THAT THE ENTITIES ISOBEL TRADEX PRIVATE LIMITED AND CAPTECK OVERSEAS PRIVATE LIMITED SHARE THE SAME ADDRESS I.E OFFICE NO 1001, 10TH FLOOR, PINNACLE CORPORATE PARK, BANDRA KURLA COMPLEX BANDRA EAST, MAHARASHTRA, MUMBAI 400051. REVIEW OF THE TRANSACTIONS FOR ACCOUNT NUMBER 22105082801 OF ISOBEL TRADEX PRIVATE LIMITED FOR THE PERIOD FROM 01/04/2020 TO 02/04/2021 REVEALED TOTAL CREDITS AND TOTAL DEBITS OF RS.5.25 LAKHS RESPECTIVELY. MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF BY WAY OF RTGS/ NEFT/IMPS TOTALLING TO RS.5.25 LAKHS OF WHICH RS.4 LAKHS WERE RECEIVED FROM DRIFTECH MERCANTILE PRIVATE LIMITED ON 1 INSTANCE, RS.1 LAKH WHICH WAS RECEIVED FROM OWN ACCOUNT HELD WITH ANOTHER BANK ON 1 INSTANCE, RS.25,000 FROM CAPTECK OVERSEAS PRIVATE LIMITED ON 1 INSTANCE. MAJOR DEBITS TO THE ACCOUNT WERE BY WAY OF RTGS/NEFT TOTALLING TO RS.3 LAKHS WHICH WERE TRANSFERRED TO DRIFTECH MERCANTILE PVT LTD ON 1 INSTANCE, CHEQUE ISSUANCE TOTALLING TO RS.2.25 WHICH WAS PAID TO OWN ACCOUNT HELD WITH AXIS BANK ON 1 INSTANCE. REVIEW OF THE TRANSACTIONS FOR ACCOUNT 22105093277 OF CAPTECK OVERSEAS PRIVATE LIMITED FOR THE PERIOD FROM 01/04/2020 TO 02/04/2021 REVEALED TOTAL CREDITS AND TOTAL DEBITS OF RS.1 LAKH RESPECTIVELY. MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF BY WAY OF IMPS TOTALLING TO RS.1 LAKH WHICH WAS RECEIVED FROM OWN ACCOUNT HELD WITH ANOTHER BANK ON 1 INSTANCE. MAJOR DEBITS TO THE ACCOUNT WERE BY WAY OF RTGS/CHEQUE ISSUANCE TOTALLING TO RS.1 LAKH WHICH WERE TRANSFERRED TO OWN ACCOUNT HELD WITH IDBI BANK ON 2 INSTANCES. DURING THE ENHANCED DUE DILIGENCE PROCESS UNDERTAKEN BY THE BRANCH, IT WAS FOUND THAT THE CUSTOMER WAS UNABLE TO FURNISH THE IMPORT PAYMENT DOCUMENTS. FURTHER IT ALSO REVEALED THAT, THE DIRECTORS OF THE COMPANIES WHERE UNAWARE ABOUT THESE TRANSACTIONS AND WHEN ASKED FOR FURTHER CLARIFICATION THE ACCOUNTS WERE ABRUPTLY CLOSED IN THE MONTH OF JANUARY 2021. SEARCHES CONDUCTED IN PUBLIC DOMAIN DID NOT REVEAL ANY ADVERSE MEDIA ON THE CUSTOMERS ISOBEL TRADEX PRIVATE LIMITED AND CAPTECK OVERSEAS PRIVATE LIMITED. AS PER PUBLIC DOMAIN SEARCH, RONGTAI HARDWARE PRODUCTS LIMITED IS ENGAGED IN THE MANUFACTURING OF HARDWARE AND ELECTRICAL PRODUCTS HTTPS //WWW.EWORLDTRADE.COM/C/RONGTAIHARDWAREPRODUCTS/ AND HTTP //WEBCACHE.GOOGLEUSERCONTENT.COM/SEARCH Q CACHE CUZ6YZQVDXKJ EN.RONGTAI.HK/ABOUT.HTM CD 6 HL EN CT CLNK GL SG . NO ENQUIRIES HAVE BEEN RECEIVED ON THE CUSTOMER FROM ANY LEA TILL DATE. CASE IS BEING REPORTED TO FIU IND DUE TO AN ATTEMPTED TRANSACTION BY THE CUSTOMER POTENTIALLY FOR A FALSE IMPORT PAYMENT FOLLOWED BY ABRUPT CLOSURE OF NEWLY OPENED ACCOUNTS WHICH LACKS ECONOMIC RATIONALE.CUSTOMER ISOBEL TRADEX PRIVATE LIMITED DATE OF INCORPORATION 27/05/2019 PAN AAFCI2878G WITH COMPANY IDENTIFICATION CIN U51909MH2019PTC325904 ISSUED BY MINISTRY OF CORPORATE AFFAIRS, HAS ADDRESS AS OFFICE NO 1001, 10TH FLOOR, PINNACLE CORPORATE PARK, BANDRA KURLA COMPLEX BANDRA EAST, MAHARASHTRA, MUMBAI 400051 AND MOBILE NUMBER 8506814549. THE CUSTOMER HAS BEEN CATEGORISED AS LOW RISK AND THE LAST KYC WAS UPDATED ON 31/03/2020 WITH THE STATUS AS COMPLETED. THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER 22105082801 WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT RAHEJA TOWERS, UNIT 3, GROUND FLOOR, PLOT NO. C 30, G BLOCK, BANDRA KURLA COMPLEX, BANDRA EAST , MUMBAI 400051 SINCE 31/03/2020 WHICH WAS CLOSED ON 08/01/2021. AS PER THE KYC RECORDS HELD WITH THE BANK, THE CUSTOMER IS INTO WHOLESALE TRADE OF ELECTRONIC/TELECOMMUNICATIONS EQUIPMENT AND PARTS AND HAVING GANESH DHONDU YERLEKAR AND AVI MEHRA AS AUTHORISED SIGNATORIES. THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS.15 CRORES. CUSTOMER CAPTECK OVERSEAS PRIVATE LIMITED DATE OF INCORPORATION 16/04/2019 PAN AAICC1327M WITH COMPANY IDENTIFICATION CIN U74999MH2019PTC324169 ISSUED BY MINISTRY OF CORPORATE AFFAIRS HAS ADDRESS AS OFFICE NO 1001, 10TH FLOOR PINNACLE, CORPORATE PARK BANDRA KURLA COMPLEX, BANDRA EAST, MAHARASHTRA, MUMBAI 400051 AND MOBILE NUMBER 9873704509. THE CUSTOMER HAS BEEN CATEGORISED AS LOW RISK AND THE LAST KYC WAS UPDATED ON 31/03/2020 WITH THE STATUS AS COMPLETED. THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER 22105093277 WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT RAHEJA TOWERS, UNIT 3, GROUND FLOOR, PLOT NO. C 30, G BLOCK, BANDRA KURLA COMPLEX, BANDRA EAST , MUMBAI 400051 SINCE 31/03/2020 WHICH WAS CLOSED ON 11/01/2021. AS PER THE KYC RECORDS HELD WITH THE BANK, CAPTECK OVERSEAS PRIVATE LIMITED IS INTO WHOLESALE TRADE OF ELECTRONIC/TELECOMMUNICATIONS EQUIPMENT AND PARTS AND HAVING GANESH DHONDU YERLEKAR AND AVI MEHRA AS AUTHORISED SIGNATORIES. THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS.15 CRORES. REVIEW OF THE TRANSACTIONS FOR ACCOUNT 22105082801 FOR THE CURRENT FINANCIAL YEAR FROM 01/04/2021 TO 02/04/2021 REVEALED TOTAL CREDITS OF 0 IN CASH 0 AND TOTAL DEBITS OF 0 IN CASH 0 . REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 01/04/2020 TO 31/03/2021 REVEALED TOTAL CREDITS OF RS.5.25 LAKHS IN CASH 0 AND TOTAL DEBITS OF RS.5.25 LAKHS IN CASH 0 . BALANCE IN THE ACCOUNT AS ON 02/04/2021 WAS 0. A REVIEW OF THE INTERNAL RECORDS REVEALED THAT NO PREVIOUS STR WAS REPORTED ON THE CUSTOMER OR ANY RELATED PERSONS.",{"entities":[(386,414,"FOREIGN ORG"),(2945,2977,"FOREIGN ORG"),(3077,3099,"FOREIGN ORG"),(3172,3183,"FOREIGN ORG")]}),
            ("FOREIGN REMITTANCE RS. 18.05 LAKHS FROM WORLD METALS AND ALLOYS FZC, SAMANCOR MARKETING PTE. LTD, HWASUNG METAL CO. LTD., SEASHORE STEEL, ETC., BILLS PAYMENT RS. 8.23 CRORES, IMPS TRANSFER RS. 1.02 CRORE FROM SUN STEEL, SHREE BALAJI ALUMNIC, ALTRUISTIC METAL PRIVATE LTD., DIVYA IMPORTERS, ETC., INTEREST RS. 3.38 LAKHS, UPI TRANSFER RS. 1.87 LAKH FROM KOLAWAT ENTERPRISES, SHRI RAGHAVENDRA STEELS, ANAND RAJESH, ETC., COMMISSION RS. 3,199/ . PROCEEDS OF THE SAME ARE UTILIZED BY MAKING FOREIGN REMITTANCES RS. 1078.03 CRORES TO COMPLEJO INDUSTRIAL MOLYNO, GLENCORE INTERNATIONAL AG, SIZER METALS PTE LTD., AMBANT LIMITED, ETC., CMS DEBITS RS. 433.14 CRORES, FUND TRANSFER RS. 88.62 LAKHS TO OSWAL ALLOYS PRIVATE LTD., MOHANLAL BHARATH KUMAR, SANJAY PITLIYA AND COMPANY, ETC., RTGS TRANSFER RS. 72.90 CRORES TO OSWAL MINERALS LIMITED YES BANK LTD , D S ALLOYD PVT LTD STATE BANK OF INDIA , LALWANI FERRO ALLOYS LTD. AXIS BANK , SHYAM FERRO ALLOYS LIMITED HDFC BANK LTD. , ETC., NEFT TRANSFER RS. 19.87 CRORES TO LALWANI FERRO ALLOYS LIMITED AXIS BANK , MANGANESE PRODUCTS CORPORATION UNION BANK OF INDIA , ISPAT DAMODAR PVT. LTD. STATE BANK OF INDIA , FERRO ALLOYS CORPORATION LTD. STATE BANK OF INDIA , ETC., CASH RS. 10.29 CRORES, LOAN REPAYMENT RS. 1.26 CRORE, TAX RS. 56.03 LAKHS, CHARGES RS. 42.63 LAKHS, CHEQUE RS. 30.71 LAKHS, SALARY RS. 13.08 LAKHS, BILLS RS. 6.70 LAKHS, INTEREST RS. 3.38 LAKHS, FEE RS. 1.00 LAKH, CREDIT CARD RS. 49,442/ , ONLINE TRANSACTION RS. 33,690/ , COMMISSION RS. 6,000/ .PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO. 609000016945 FOR THE PERIOD MENTIONED BELOW FY 2019 2020 01 04 2019 TO 31 03 2020 TOTAL CREDITS RS. 1352.53 CRORES CASH 5.40 LAKHS AND TOTAL DEBITS RS. 1357.04 CRORES CASH 12.29 CRORES . FY 2018 2019 01 04 2018 TO 31 03 2019 TOTAL CREDITS RS. 1211.13 CRORES CASH 2.66 LAKHS AND TOTAL DEBITS RS. 1215.26 CRORES CASH 4.45 CRORES .FY 2017 2018 01 04 2017 TO 31 03 2018 TOTAL CREDITS RS. 999.20 CRORES CASH 91,080 AND TOTAL DEBITS RS. 979.18 CRORES CASH 6.71 CRORES .BALANCE AS ON DATE OF FILING IS RS. 36870294.59/ .WE HAVE ALREADY FILED STR ON ACCOUNT NO. 609000016945 STR NO. 6194, DATE 18 07 2017, BATCH ID 1707181814, REASON HUGE AMOUNT OF FUNDS HAVE BEEN ROUTED THROUGH THE ACCOUNT BY INWARD AND OUTWARD RTGS/NEFT AND INTERNAL FUND TRANSFERS WITHOUT ANY ECONOMIC RATIONALE WHILE CONDUCTING DISCREET CHECK, WE HAVE OBSERVED FEW DISCREPANCIES. CUSTOMER HAS RECEIVED HIGH VALUE REFUND AGAINST DIRECT IMPORT BILL PAYMENT AS QUALITY CLAIM AND CONSOLIDATED VALUE OF REFUND AMOUNTED TO USD 1.9 MILLION, HOWEVER THE SAME WAS NOT HANDLED BY CLIENT AS PER RBI DIRECTIONS TOWARDS WRITE OFF OF IMPORT DUES. FURTHER, HIGH VALUE TRANSACTIONS HANDLED UNDER DIRECT IMPORT BILL PAYMENT AS OPEN ACCOUNT PAYMENT AND PAYMENT TERMS APPEARING ON INVOICE SEEMS MODIFIED WHILE PAYMENT. THERE IS FREQUENT TRADING BETWEEN THIRD COUNTRY AND PAYMENT MADE MAINLY WITH UAE ENTITY I.E M/S WORLD METALS AND ALLOYS FZC, WHEREIN MANUFACTURER OF THE GOODS OR ORIGIN DETAILS ARE NOT SPECIFIED PROPERLY. IT APPARENTLY SEEMS THAT CUSTOMER HAS INTENTIONALLY BREACHED THE FEMA GUIDELINES.HIGH VALUE RTGS/NEFT/FUND TRANSFER/BILLS RECEIPT CREDITS FOLLOWED BY HIGH VALUE TRANSFER TO SPECIFIC THIRD PARTIES, ANOMALIES OBSERVED RAISING SUSPICION ON UNDERLYING INTENTION. HENCE, STR FILED.OSWAL MINERALS LIMITED ACCOUNT NO. 409000086197, 409001100618, 409000086203 AND 609000016945 ADDRESS 1034 2ND FLOOR DR RAJKUMAR ROAD,4TH BLOCK RAJAJINAGAR, OPPOSITE TO PLANET HONDA SERVICE CENTRE BANGALORE 560010, DATE OF INCORPORATION 04 07 1996, PAN AACCM6499G, AADHAAR NUMBER REFERENCE KEY IN VIEW OF THE RESTRICTION OF SHARING THE AADHAAR NUMBER AS PER REGULATION 6 OF AADHAAR SHARING OF INFORMATION REGULATIONS 2016, WE ARE PROVIDING REFERENCE KEY ASSIGNED TO THE AADHAAR NUMBER, WHEREVER APPLICABLE, MOBILE NUMBER 919448992115, RISK CATEGORY HIGH RISK, AUTHORISED SIGNATORY MOHANLAL BHARATHKUMAR JAIN, SUBHASH CHAND MOHANLAL, KYC COMPLIANCE STATUS YES, DATE OF LAST KYC UPDATION 04 07 2019 AND NA OPENED AN EXCHANGE EARNERS FOREIGN CURRENCY A/C EEFC ACCOUNT AND CASH CREDIT ACCOUNT RESPECTIVELY ON 04 07 2019, 30 09 2013, 30 09 2013 AND 28 09 2013 AT RBL BANK, BANGALORE BRANCH, 197, 6TH CROSS, GANDHINAGAR, BANGLORE 560009 AND FORT MUMBAI BRANCH, GROUND FLOOR, 2, 2A DHANNUR, 15, SIR FIROZ SHAH MEHTA ROAD, MUMBAI, FORT 400001. CONSTITUTION TYPE PUBLIC LIMITED COMPANY, ACCOUNT STATUS INACTIVE, ACTIVE, DORMANT AND ACTIVE RESPECTIVELY THE CUSTOMER HAS DECLARED PROFILE AS MANUFACTURER OF METALS AND MINERALS HAVING EXPECTED ANNUAL TURNOVER IS GREATER THAN 100 CRORE. SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO. 409000086197 FOR THE PERIOD FROM 01 04 2020 UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS. 116.94 LAKHS CONSISTING OF FOREIGN REMITTANCES RS. 116.94 LAKHS FROM WORLD METALS ALLOYS, KUBOTA SAUDI COMPANY FACTORY, AL GARHOUD GENERAL TRADING, PINK LOTUS IRON AND STEELS, ETC. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS. 104.21 LAKHS, FOREIGN REMITTANCES RS. 12.72 LAKHS TO ALLIANZ BULK CFR8 PTE LTD., GLENCORE INTERNATIONAL AG, PMB SILICON SDN BHD, SIZER METALS PTE LTD.PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO. 409000086197 FOR THE PERIOD MENTIONED BELOW FY 2019 2020 01 04 2019 TO 31 03 2020 TOTAL CREDITS RS. 68.25 LAKHS CASH NIL AND TOTAL DEBITS RS. 68.25 LAKHS CASH NIL .FY 2018 2019 01 04 2018 TO 31 03 2019 TOTAL CREDITS RS. 149.39 LAKHS CASH NIL AND TOTAL DEBITS RS. 149.39 LAKHS CASH NIL .FY 2017 2018 01 04 2017 TO 31 03 2018 TOTAL CREDITS RS. 223.57 LAKHS CASH NIL AND TOTAL DEBITS RS. 223.57 LAKHS CASH NIL .BALANCE AS ON DATE OF FILING IS NIL.THERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED ACCOUNT NO. 409001100618 FROM THE PERIOD OF 01 04 2020 UPTO DATE OF FILING STR.SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO. 409001100618 FOR THE PERIOD FROM 04 07 2019 UPTO 31 03 2020 REVEALED THAT THE TOTAL CREDITS OF RS. 13.28 LAKHS CONSISTING OF FOREIGN REMITTANCES RS. 13.28 LAKHS FROM STAIN LESS METALS GENERAL. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS. 13.28 LAKHS.BALANCE AS ON DATE OF FILING IS NIL.THERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED ACCOUNT NO. 409000086203 FROM THE PERIOD OF 01 04 2017 UPTO DATE OF FILING STR.BALANCE AS ON DATE OF FILING IS NIL.SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO. 609000016945 FOR THE PERIOD FROM 01 04 2020 UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS. 1715.41 CRORES CONSISTING OF RTGS TRANSFER RS. 1062.17 CRORES FROM OSWAL MINERALS YES BANK LTD. , SONIC THERMAL PVT LTD. ALLAHABAD BANK , AJAY ENTERPRISES YES BANK LTD. , MUKAND LTD YES BANK LTD. , ETC., FUND TRANSFER RS. 193.83 CRORES FROM MOHANLAL, SEEMA JAIN, SUBHASHCHAN, SRIPAL KUMAR, SARITHA DEV, ETC., NEFT TRANSFER RS. 341.61 CRORES FROM OSWAL MINERALS LIMITED HDFC BANK LTD , ARCELORMITTAL NIPPON STEEL INDIA ICICI BANK LTD , SHYAM FERRO ALLOYS LIMITED HDFC BANK LTD , MACTECH ENGINEERING CORPORATION BANK , ETC., CMS RS. 52.67 CRORES, CHEQUE RS. 37.76 CRORES RETURNED AMOUNTED RS. 45.26 LAKHS ",{"entities":[(40,65,"FOREIGN ORG"),(69,94,"FOREIGN ORG"),(98,117,"FOREIGN ORG"),(122,136,"FOREIGN ORG"),(529,553,"FOREIGN ORG"),(557,580,"FOREIGN ORG"),(584,602,"FOREIGN ORG"),(607,619,"FOREIGN ORG"),(2954,2979,"FOREIGN ORG"),(4861,4878,"FOREIGN ORG"),(4882,4908,"FOREIGN ORG"),( 4912,4936,"FOREIGN ORG"),(4940,4964,"FOREIGN ORG"),(5080,5103,"FOREIGN ORG"),(5108,5129,"FOREIGN ORG"),(5135,5152,"FOREIGN ORG"),(5156,5175,"FOREIGN ORG")]}),
            ("FOREIGN OUTWARD REMITTANCES NON IMPORT AMOUNTED TO RS.9.47 CR FAVOURING SPEED STREAM FREIGHT BROK UAE RS.6.77 CR, STATICE INTERNATIONAL PTE LTD SINGAPORE RS.1.54 CR AND EXCEL SEA SHIPPING LLC UAE RS.1.15 CR TOWARDS FREIGHT ON IMPORTS. 3 BANK HAS ALREADY STR ON OTHER ACCOUNTS NAMELY BARA AFRICA ON SHORE SERVICES PRIVATE LIMITED 5845220101, BATCH ID 2103086872, BOXTRON SHIPPING AND LOGISTICS PVT LTD 5545595905, BATCH ID 2103086851 AND MILE LOGISTICS P LIMITED 3645356679, BATCH ID 2012284410 WHEREIN SIMILAR PATTERN OF TRANSACTIONS WERE NOTED AND FUNDS WERE REMITTED TO THE ABOVE MENTIONED COMMON OVERSEAS PARTY EXCEL SEA SHIPPING LLC UAE. 4 IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.0.72 CR MAINLY FAVOURING NEXASHIP LOGISTICS PRIVATE LIMITED RS.0.64 CR, IDFC 10065573438, LIBERTON LOGISTICS PRIVATE LIMITED RS.0.05 CR, IDFC 10065171794 AND T S GLOBAL IMPEX RS.0.02 CR, IDFC 10064932707. 5 LIBERTON LOGISTICS PRIVATE LIMITED AND NEXASHIP LOGISTICS PRIVATE LIMITED HAVE UPDATED A COMMON MAILING ADDRESS. DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS SIGN BOARDS IN THE NAMES OF THE COMPANIES WERE SEEN AT THE PREMISE AND OFFICE SET UP WITH CHAIRS AND TABLES COULD BE SEEN. HOWEVER NO MAJOR BUSINESS ACTIVITY OR EMPLOYEES COULD BE SEEN TO SUBSTANTIATE THE HIGH VALUE TRANSACTIONS ROUTED THROUGH ACCOUNT. IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED TO ROUTE TRANSACTIONS WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY. 6 A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE ROUND SUM RTGS/NEFT CREDITS WERE FOLLOWED BY IMMEDIATE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS. NO OTHER BUSINESS RELATED TRANSACTIONS COULD BE NOTED THROUGH THE ACCOUNT. WITHIN 2 MONTHS OF ACCOUNT OPENING HIGH VALUE AND VOLUME OF TRANSACTIONS WERE ROUTED THROUGH ALL 3 ACCOUNTS. 7 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING. ALSO THE PROPRIETORS AND DIRECTORS ARE IN THE AGE GROUP OF 20 31 YEARS AND MAY NOT BE THE ACTUAL BENEFICIAL OWNERS OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNTS. ACCOUNT OF LIBERTON LOGISTICS P LIMITED IS ACTIVE WITH BALANCE OF RS.12,72,753/ AS ON APRIL 5, 2021. QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 2021 2022 IS DEBIT RS.0.00 IN CASH RS.0.00 CR AND CREDITS RS.0.21 CR IN CASH RS.0.00 CR. 2020 2021 IS DEBIT RS.10.42 CR IN CASH RS.0.00 CR AND CREDITS RS.10.76 CR IN CASH RS.0.00 CR.A CURRENT ACCOUNT BEARING NUMBER 6845087299 WAS OPENED IN THE NAME OF LIBERTON LOGISTICS P LTD ON 11 FEB 21 AT OUR BRANCH LOCATED IN SHARDA TERRACES, PLOT NO 65, SEC 11, CBD BELAPUR, NAVI MUMBAI 400614. THE ENTITY WAS FORMED ON 16 JAN 07 AND HAS SUBMITTED COPIES OF PAN AAGCA0882R, ALONG WITH BOARD RESOLUTION, MOA/ AOA AND CERT OF INCORP U74110MH2007PTC167056 AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO. B 608 SHELTON SAPPHIRE PL 18 19 SECTOR 15 CBD BELAPUR NAVI MUMBAI 400614. DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS AND ULTIMATE BENEFICIAL OWNERS. DIRECTORS HAVE REGISTERED THEIR MOBILE NUMBERS 9711056074 AND 8130480467 WITH THE BANK. THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS.5.00 CR AND THAT THE INDIVIDUALS HAVE ANNUAL INCOME IN THE RANGE OF RS.0.25 CR TO RS.0.50 CR. BASIS INTER CONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED IN THE NAMES OF NEXASHIP LOGISTICS PRIVATE LIMITED 6945194835 AND A SOLE PROPRIETORSHIP CONCERN T S GLOBAL IMPEX 6945110002 OPENED ON 17 FEB 21 AND 16 FEB 2021. DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS IN NEXASHIP LOGISTICS PRIVATE LIMITED WHILE TSHERINGLA SHERPA IS THE PROPRIETOR IN T S GLOBAL IMPEX. IT WAS INDICATED THAT THE ENTITIES ARE INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS.4.00 CR AND 20.00 CR RESPECTIVELY. WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NON IMPORT WERE ROUTED THROUGH THE NEWLY OPENED ACCOUNT OF LIBERTON LOGISTICS PRIVATE LIMITED. SIMILAR PATTERN OF TRANSACTIONS AND COMMON OVERSEAS BENEFICIARIES WERE NOTED IN THE ACCOUNTS OF NEXASHIP LOGISTICS PRIVATE LIMITED AND T S GLOBAL IMPEX WHICH AS PER CUSTOMER ARE RELATED ACCOUNTS. HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTER CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION 1 ON REVIEWING CONSOLIDATED ACCOUNT TRANSACTIONS SINCE INCEPTION TILL APRIL 5, 2021 IT WAS NOTED THAT ROUND SUM HIGH VALUE IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.10.65 CR MAINLY FROM NAMO SAI INTERNATIONAL RS.8.04 CR, EQUITAS SMALL FINANCE BANK LIMITED A/C NO 200001328409 AND NEXASHIP LOGISTICS PRIVATE LIMITED RS.2.33 CR.",{"entities":[(114,142,"FOREIGN ORG"),(169,190,"FOREIGN ORG"),(614,635,"FOREIGN ORG")]}),
            ("GROUNDS OF SUSPICION THIS IS AN LEA CASE AND A SUPPLEMENTARY CASE. BANK WAS IN RECEIPT OF A SUMMON DATED 02/01/2021 VIDE REFERENCE NUMBER ITBA/INV/S/131/2020 21/1029437924 1 U/S 131 1A FROM NARENDRA MOLAI PRASAD, DY.DIRECTOR OF INCOME TAX INV. UNIT 3 4 , INCOME TAX DEPARTMENT, OFFICE OF THE ASSISTANT DIRECTOR OF INCOME TAX, INVESTIGATION. THE NOTICE DIRECTED THE BANK TO PROVIDE ACCOUNT RELATED INFORMATION OF R N EXPORTS HOLDING ACCOUNT NUMBERS 22405360386, 22405360424 AND 22506142090 FOR INVESTIGATION PURPOSE. THE INFORMATION SOUGHT HAS BEEN PROVIDED TO THE INCOME TAX DEPARTMENT. REVIEW OF THE TRANSACTIONS FOR ACCOUNT 22405360386 FOR THE PERIOD FROM 01/04/2020 TO 13/04/2021 REVEALED TOTAL CREDITS OF RS.3.37 CRORES AND TOTAL DEBITS OF RS.3.50 CRORES. MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF INTERNAL FUND TRANSFERS TOTALLING TO RS.2.76 CRORES ON 7 INSTANCES WHICH WERE RECEIVED FROM OWN USD DIAMOND DOLLAR ACCOUNT ACCOUNT NUMBER 22506142090 , RTGS/NEFT CREDITS TOTALLING TO RS.54.04 LAKHS ON 38 INSTANCES OF WHICH RS.28.51 LAKHS FROM OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 2413200206 ON 5 INSTANCES, RS.9.59 LAKHS FROM CARAT LANE TRADING PRIVATE LIMITED ICICI BANK LTD, ACCOUNT NUMBER 0104SLNEFTPL ON 9 INSTANCES, RS.8.20 LAKHS FROM KANTILAL CHHOTALAL BANK OF INDIA, ACCOUNT NUMBER 016720110000024 ON 12 INSTANCES, RS.4.88 LAKHS FROM NAICE JEWELS PRIVATE LIMITED KOTAK MAHINDRA BANK LIMITED, ACCOUNT NUMBER 500044010156 ON 8 INSTANCES, RS.1.60 LAKHS FROM CREATIVE GEMS AND JEWELLERY LTD STATE BANK OF INDIA, ACCOUNT NUMBER 00000032883046667 ON 3 INSTANCES, RTGS RETURNS TOTALLING TO RS.6.61 LAKHS. MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGS/NEFT TOTALLING TO RS.2.69 CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 2413200206 ON 8 INSTANCES, FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS.65.47 LAKHS, CHEQUES ISSUANCE TOTALLING TO RS.8.29 LAKHS, IMPORT PAYMENT TOTALLING TO RS.6.41 LAKHS TO TANU STAR HK LTD, HONG KONG ETC. A REVIEW OF THE TRANSACTIONS IN USD DIAMOND DOLLAR ACCOUNT 22506142090 FOR ABOVE REVIEW PERIOD REVEALED BUSINESS RELATED TRANSACTIONS. SEARCHES CONDUCTED IN PUBLIC DOMAIN DID NOT REVEAL ANY ADVERSE MEDIA ON THE CUSTOMER. CASE IS BEING REPORTED TO FIU IND DUE TO THE NOTICE RECEIVED FROM THE INCOME TAX DEPARTMENT.CUSTOMER R N EXPORTS DATE OF INCORPORATION 01/04/1991 PAN AAAFR2200B WITH IEC CODE 0391008421 ISSUED BY DGFT, GSTIN 27AAAFR2200B1Z3 ISSUED BY CBEC HAS ADDRESS AS 301, PETIT TOWER, 3RD FLOOR, CUMBALLA HILL, A K MARG, MUMBAI 400026 MOBILE NUMBER 9820046978 AND LANDLINE NUMBER 022 23888820. THE CUSTOMER HAS BEEN CATEGORISED AS MEDIUM RISK AND THE LAST KYC WAS UPDATED ON 08/09/2020 WITH THE STATUS AS COMPLETED. THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER 22405360386 WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT SHREE PANT BHUVAN, SANDHURUST BRIDGE, MUMBAI 400007 SINCE 27/08/2010 WITH THE STATUS AS ACTIVE. THE CUSTOMER ALSO HOLDS AN USD EEFC ACCOUNT NUMBER 22405360424 SINCE 27/08/2010 WHICH IS IN DORMANT STATUS SINCE 14/06/2019 AND USD DIAMOND DOLLAR ACCOUNT NUMBER 22506142090 SINCE 06/07/2016 WHICH IS IN ACTIVE STATUS. AS PER THE KYC RECORDS HELD WITH THE BANK, CUSTOMER R N EXPORTS IS A PARTNERSHIP FIRM ENGAGED IN THE BUSINESS OF MANUFACTURING OF JEWELLERY AND WORKING OF DIAMONDS. RAHUL ROHIT JHAVERI AND SONALI RAHUL JHAVERI ARE THE BENEFICIAL OWNERS AND AUTHORISED SIGNATORIES OF THE FIRM. THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS.62.95 CRORES. REVIEW OF THE TRANSACTIONS FOR ACCOUNT 22405360386 FOR THE CURRENT FINANCIAL YEAR FROM 01/04/2021 TO 13/04/2021 REVEALED TOTAL CREDITS OF 0 IN CASH 0 AND TOTAL DEBITS OF 0 IN CASH 0 . REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 01/04/2020 TO 31/03/2021 REVEALED TOTAL CREDITS OF RS.3.37 CRORES IN CASH 0 AND TOTAL DEBITS OF RS.3.50 CRORES IN CASH RS.74,000 REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 01/04/2019 TO 31/03/2020 REVEALED TOTAL CREDITS OF RS.21.12 CRORES IN CASH 0 AND TOTAL DEBITS OF RS.21.37 CRORES IN CASH RS.2 LAKHS . BALANCE IN THE ACCOUNT AS ON 13/04/2021 WAS 0. A REVIEW OF THE INTERNAL RECORDS REVEALED THAT AN STR WAS REPORTED PREVIOUSLY VIDE BATCH ID 1702067988, S.NO. 1 WITH THE REPORTED ACCOUNTS 22405360386, 22405360424 AND 22506142090 IN THE NAME OF R N EXPORTS DUE TO CIRCULATION OF FUNDS BETWEEN THE CLIENT AND A HONG KONG BASED ENTITY ON ACCOUNT OF PURCHASE AND SALE OF ROUGH AND POLISHED DIAMONDS.",{"entities":[( 1975,1989,"FOREIGN ORG")]}),
            ("ONE OF THE ENTITIES REGISTERED AT AFOREMENTIONED ADDRESS IS WHITE RABBIT MOTION PICTURES PRODUCTION LLC. AS PER AVAILABLE OPINION REPORT WHITE RABBIT MOTION PICTURES PRODUCTION LLC AND MAGNETIC MOTION PICTURE PRODUCTION LLC ARE GROUP ENTITIES.IT IS OBSERVED THAT WHITE RABBIT MOTION PICTURES PRODUCTION LLC, UAE IS COMMON BENEFICIARY RECEIVING CROSS BORDER REMITTANCES FROM MULTIPLE YBL CUSTOMER VIZ. KEYFRAME MULTIMEDIA SERVICES, SOLITUDE ENTERTAINMENT LLP AND EYE FOCUS PRODUCTION WHEREIN INVOICE DRAWN ON EACH OF THESE CUSTOMER WHEN COMPARED DID NOT MATCH WITH RESPECT TO COMPANY LOGO AND OVERALL INVOICE FORMAT. STRS WERE THEREFORE FILED ON BELOW CUSTOMERS VIDE FIU BATCH ID AS UNDER KEYFRAME MULTIMEDIA SERVICES 2103183777SOLITUDE ENTERTAINMENT LLP 2103256408EYE FOCUS PRODUCTION 21020935392 AS PER AGREEMENT COPY, REMITTANCES ARE BEING UNDERTAKEN TOWARDS FILM SHOOTING IN FIJI ISLANDS HOWEVER FUNDS ARE BEING REMITTED IN FAVOR OF ENTITY BASED IN UNRELATED JURISDICTION OF UAE.3 WHEN YES BANK OFFICIALS REQUESTED THE CUSTOMER TO SHARE COPY OF PASSPORT AND VISA OF CREW/ STAR CAST WHO HAVE TRAVELLED TO FIJI ISLANDS TO WHICH CUSTOMER REFUSED TO SHARE ANY SUCH DETAILS AND FURTHER ADDED THAT HE SHALL SHARE THE EVIDENCES WITH THE RESPECTIVE AUTHORITY WHEN NEEDED. 4 SITE VERIFICATION CONDUCTED AT THE ADDRESS REGISTERED WITH THE BANK WAS POSITIVE HOWEVER IT WAS OBSERVED THAT BESIDES DIGI MEDIA AND FILMS SOLUTIONS CLIENT IS ALSO ASSOCIATED WITH 2 OTHER ENTITIES VIZ. TINGLE CINE PRODUCTION AND WATERMARK STUDIOS ALL 3 OF WHICH OPERATE FROM SAME COMMON ADDRESS.WHILE REVIEWING ACCOUNT STATEMENTS OF REMITTER CUSTOMERS CUSTOMER VIZ. DIVERSE MEDIA AC NO 063663400000370 MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 AND LIQUID STUDIOS AND SERVICES AC NO 063663300002507 BELOW ARE RED FLAGS OBSERVED 5 STATED ANNUAL INCOME OF DIVERSE MEDIA AND MAGNETIC FILMS PRODUCTIONS IS LESS THAN RS.50 CRORES AND THAT OF LIQUID STUDIOS AND SERVICES IS LESS THAN RS.1 CRORE HOWEVER IN LESS THAN 6 MONTHS OF ACCOUNT OPENING SAID ENTITIES HAVE RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS.30 TO 50 CRORES IN EACH OF 3 ACCOUNTS.6 AFOREMENTIONED ENTITIES STATED LINE OF BUSINESS IS MEDIA AND ENTERTAINMENT HOWEVER MAJORITY OF CREDITS RECEIVED IN THESE ACCOUNTS ARE FROM MULTIPLE ENTITIES ENGAGED IN UNRELATED LINE OF BUSINESS E.G. IN THE ACCOUNT OF MAGNETIC FILMS PRODUCTIONS CLIENT HAS RECEIVED ACCOUNT CREDITS FROM TWICLO TOUR AND PACKAGES , BHUTANI CORPORATION, SHYAM ENTERPRISES, EAGLE INTERNATIONAL LOGISTICS, R.K STEEL ALLOYS FEW OF WHICH ARE COMMON COUNTERPARTIES REMITTERS SENDING FUND TRANSFERS IN FAVOR OF DIVERSE MEDIA AND LIQUID STUDIOS AND SERVICES7 FURTHER, CUSTOMERS DIVERSE MEDIA, MAGNETIC FILMS PRODUCTIONS AND LIQUID STUDIOS AND SERVICES HAVE RECEIVED TOTAL OF RS.7.43 CRORES FROM DOMESTIC COUNTERPARTY TWICLO TOUR AND PACKAGES. TWICLO TOUR AND PACKAGES IS COMMON REMITTER/COUNTERPARTY WHICH HAS ALSO PROVIDED FUNDING TO YBL CUSTOMER DISCOVER PLANET SOLUTIONS ON WHICH STR WAS FILED VIDE FIU BATCH ID 21021582848 ACCOUNT CREDITS FROM BUSINESSES IN UNALLIED ACTIVITIES IS FOLLOWED BY FUNDS TRANSFER IN FAVOR OF VARIOUS ENTITIES IN MEDIA AND ENTERTAINMENT BUSINESS INCLUDING DIGI MEDIA AND FILMS SOLUTIONS AND ITS ASSOCIATE ENTITIES TINGLE CINE PRODUCTIONS AND WATERMARK STUDIOS 9 WHEN YES BANK OFFICIALS INQUIRED ON BUSINESS RATIONAL FOR CREDITS BEING RECEIVED FROM UNALLIED BUSINESSES TO WHICH CUSTOMER REPLIED STATING THAT THEY ARE FINANCIERS/ INVESTORS AS WELL AS PRODUCERS AND THAT FUNDS ARE BEING RECEIVED FOR ARRANGING EQUIPMENTS, LOGISTICS, MANPOWER ETC FOR MAKING OF SHORT FILMS OR WEBSERIES ETCIN LIGHT OF HIGH VALUE TRANSACTIONS BEING RECORDED BEYOND CLIENTS STATED ANNUAL INCOME, TRANSACTIONS WITH ENTITIES IN UNALLIED ACTIVITIES AND CROSS BORDER REMITTANCE IN FAVOR OF ENTITY WITH UNSATISFACTORY CREDENTIALS IT HAS BEEN DECIDED ON 16 APRIL 2021 TO FILE STR SAID CUSTOMER.1 THE CUSTOMER DIGI MEDIA AND FILMS SOLUTIONS, A PARTNERSHIP FIRM PAN AARFD0600J, DOI 17 APRIL 2019, COMMUNICATION ADDRESS 2 8 2 626/5 FLAT NO 201, BANJARAHILLS OPP SAKSHI OFFICE, HYDERABAD, TELANGANA, 500034, GSTIN 36AARFD0600J1ZK IEC CODE AARFD0600J, MOBILE NUMBER 918179551161, EMAIL ID CONTACT DIGIMEDIAFILMS.IN IS MAINTAINING A CURRENT AC NO. 025063400001118 SINCE 31 12 2020 WITH BANJARA HILLS, HYDERABAD BRANCH ADDRESS SHOP NO G 1, DEV DHANUKA PRESTIGE, ROAD NO. 12, BANJARA HILLS, HYDERABAD 500034 .2 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN DECEMBER 2020. MOHIT BANSAL AND MANISH CHAWLA ARE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT. CUSTOMER IS ENGAGED IN MEDIA AND ENTERTAINMENT BUSINESS WITH AN ANNUAL GROSS INCOME LESS THAN RS.75 CRORES . DIGI MEDIA AND FILMS SOLUTIONS AC NO.025063400001118 IS COMMON BENEFICIARY RECEIVING FUNDS TRANSFER FROM 3 OTHER YES BANK CUSTOMERS AS UNDER CURRENT ACCOUNTS OF DIVERSE MEDIA AC NO 063663400000370, MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 AND LIQUID STUDIOS AND SERVICES AC NO 063663300002507 ARE LINKED WITH ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS THROUGH FUND TRANSFERS WHEREIN LATTER IS COMMON BENEFICIARY.ALL 3 ENTITIES VIZ. DIVERSE MEDIA, MAGNETIC FILMS PRODUCTIONS AND LIQUID STUDIOS AND SERVICES HAVE COMMON BUSINESS ADDRESS FLOOR FIRST SHOP NO 55 LOCAL, SHOPPING, CENTER BLOCK A 1, NEW DELHI, DELHI 110063 AND ARE ENGAGED IN BUSINESS OF MEDIA AND ENTERTAINMENT.3 IN THE ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS AC NO. 025063400001118 AND 3 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM 01 04 2020 TILL 16 04 2021, SHOWS 1,096 CREDIT TRANSACTIONS AMOUNTING TO RS. 1,225,798,293 AND 702 DEBIT TRANSACTIONS AMOUNTING TO RS. 1,225,792,338OUT OF THE TOTAL CREDITS OF RS. 1,225,798,293 IN THE ACCOUNTS, NEFT/RTGS AMOUNTS TO RS. 1,168,123,534 CONSTITUTING AROUND 95 PERCENT OF THE TOTAL CREDITS, OUT OF THE TOTAL DEBITS OF RS. 1,225,792,338 IN THE ACCOUNTS, NEFT/RTGS AMOUNTS TO RS. 1,140,639,525 CONSTITUTING AROUND 93 PERCENT OF THE TOTAL DEBITS,4 IN THE ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS AC NO. 025063400001118 AND 3 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM 01 04 2017 TILL 31 03 2020 NO TRANSACTIONS WERE RECORDED IN THE ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS AC NO 025063400001118, DIVERSE MEDIA AC NO 063663400000370, MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 AND LIQUID STUDIOS AND SERVICES AC NO 063663300002507 SINCE ACCOUNTS WERE OPENED IN THE MONTH OF SEPTEMBER 2020 TO DECEMBER 2020 BALANCE IN THE ACCOUNT AS ON 16 04 2021 IS RS. 900.75ON REVIEWING THE TRANSACTION PATTERN OF DIGI MEDIA AND FILMS SOLUTIONS AC. 025063400001118 IT WAS OBSERVED THAT SINCE JANUARY 2021 CLIENT HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS1.8 CRORES WITH MAJOR CREDITS VIA FUNDS TRANSFERS FROM YBL CUSTOMERS DIVERSE MEDIA AC NO 063663400000370 RS.52.5 LACS MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 RS.48.1 LACS , LIQUID STUDIOS AND SERVICES AC NO 063663300002507 RS54 LACS AND RTGS FROM LIQUID STUDIOS AND SERVICES RS26LACS FROM AC NO 39631954021 WITH STATE BANK OF INDIA, SBIN0009111 ACCOUNT CREDITS WERE FOLLOWED BY DEBITS TO THE TUNE OF RS1.8 CRORES WITH MAJOR DEBITS PERTAINING TO OUTWARD REMITTANCE TO THE TUNE OF RS1.7 CRORES PERTAINING TO AUDIO VISUAL AND RELATED SERVICES IN FAVOR OF OVERSEAS ENTITY MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE 1 BELOW ARE RED FLAGS OBSERVED WITH REGARDS TO OVERSEAS ENTITY MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE PUBLIC DOMAIN INFORMATION ON MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE IS INSUFFICIENTAS PER PUBLIC DOMAIN, MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS AS PROVIDED IN INVOICE 1101, AL MASRAF TOWER, BANIYAS ROAD, DEIRA, DUBAI, UAE, P.O.BOX 242874 OF SAID OVERSEAS ENTITY",{"entities":[(60,101,"FOREIGN ORG"),(137,179,"FOREIGN ORG"),(263,306,"FOREIGN ORG"),(185,222,"FOREIGN ORG"),(7376,7412,"FOREIGN ORG"),(7449,7487 ,"FOREIGN ORG")]}),
            ("HIGH VALUE ADV. IMP. REM. AMOUNTED TO RS.117.61 CR FROM THE A/C OF PREMIUM PETRO PRODUCTS MAINLY FAVOURING UAE BASED ENTITIES - MOTOTIVE GENERAL TRADING LLC RS.34.97 CR, PETRO STAR FZE RS.33.71 CR, KIARA PETROLEUM FZE RS.13.78 CR, OULA ALNUHA INTERNATIONAL RS.11.04 CR, TIME TECHNO PACKAGING LLC RS.9.34 CR, TASHBULAK TRADING LLC RS.3.61 CR AND MAN TRADING LLC RS.3.48 CR TOWARDS IMPORT OF BITUMEN. 4 PREMIUM PETRO PRODUCTS IS YET TO SUBMIT BOES PERTAINING TO 72 CASES TOTALLING TO USD 8,250,389 WHICH COMPRISES OF 14 CASES TOTALLING TO USD 985,122 PENDING FOR A PERIOD ABOVE 3 MONTHS UPTO 6 MONTHS AND 58 CASES TOTALLING TO USD 7,265,266 PENDING FOR A PERIOD LESS THAN 3 MONTHS. 5 IMPS/NEFT/RTGS PAYMENTS AMOUNTED TO RS.71.77 CR MAINLY FAVOURING PREMIUM PETRO PRODUCTS RS.43.16 CR, PP SOFTTECH P LTD RS.7.15 CR, ICICI - 732305000042, VISHAL BOHRA RS.2.40 CR, HDFC 4854980603624562/ 59109829999990, IDBI 0131063380800, YES BANK - 65432109829999990, VINAYAK EXIMS RS.1.15 CR, AXIS BANK - 914020013741606 AND AUM KRIPA ENTPS RS.1.00 CR, HDFC - 59272000000001. 6 INTER-CONNECTED FUNDS TRANSFER TXNS WITHIN GROUP A/CS AMOUNTED TO RS.37.71 CR. U-TURN TRANSACTIONS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS.152.50 CR WERE RECEIVED AND RS.50.09 CR WERE SENT OUT TO A/CS OF PREMIUM PETRO PRODUCTS, VISHAL BOHRA, ABHA PRECISION FARMING LLP, BHEEM RAJ BOHRA, MANISH BOHRA IDBI 013104000294935 AND VCI EXPORTS P LTD MAINTAINED WITH OTHER BANKS. 7 PREMIUM PETRO PRODUCTS, B L INVESTMENT, RISHI ENTPS, HERO PETRO PRODUCTS, REEVA BUILDCON P LTD, REALTECH BUILDHOME P LTD, ABHA PRECISION FARMING P LTD, AONE BUILDHOME LLP AND BODHI FARM DEVELOPERS P LTD HAVE UPDATED A COMMON MAILING ADDRESS 1-3 HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN - 302001. DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT IT WAS A 2000 SQ. FT. OWNED PREMISE WITH SIGN BOARD, EMPLOYEES AND OFFICE SET UP SEEN. HOWEVER NO STOCK OF GOODS WERE SIGHTED. ON ENQUIRY IT WAS LEARNT FROM CUSTOMER THAT GOODS WERE STOCKED AT MOKHAMPURA, TEHSIL-MOZAMABAD, DISTRICT JAIPUR. 8 PAN OF INTER-CONNECTED A/CS - SHREEBALAJI INFRAPROJECTS P LTD AAPCS5089K AND DENIS LAND DEVELOPERS P LTD AACCD6725C ARE REFLECTING IN THE LIST OF POSSIBLE SHELL ENTITIES CIRCULATED BY FIU INDIA. 9 TOTAL TAX PAYMENTS OF RS.0.70 CR WAS NOTED THROUGH THE A/CS OF UJJWAL BHANDARI HUF 9514297302 AND VCI EXPORTS P LTD 3413099593 DURING REVIEW PERIOD, WHICH APPEARS TO BE DISPROPORTIONATE TO HIGH VALUE AND VOLUME OF TRANSACTIONS ROUTED THROUGH A/CS.10 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH MULTIPLE A/CS OPENED BY THE SAME GROUP OF INDIVIDUALS FOR LAYERING OF FUNDS AND POSSIBLY TO REMIT FUNDS OVERSEAS WITHOUT ACTUAL IMPORTS TAKING PLACE POSSIBLY WITH THE INTENTION OF TBML AND HAWALA SETTLEMENT. IT ALSO APPEARS THAT SUCH HIGH VALUE AND VOLUME OF TRANSACTIONS MAY HAVE BEEN CARRIED OUT WITHIN GROUP A/CS WITHOUT ANY ECONOMIC RATIONALE POSSIBLY WITH INTENTION OF TAX EVASION. A/C OF PREMIUM PETRO PRODUCTS BEARING NUMBER 6813095042 IS ACTIVE WITH BALANCE OF RS.208955/- AND 6813095059 HAS OUTSTANDING BALANCE OF RS.7,05,25,633/- ON CREDIT LIMIT OF RS.12,00,00,000/- AS ON APRIL 15, 2021.QUANTUM OF TRANSACTIONS FOR CURRENT FY (2021-2022) IS DEBIT: RS.27.73 CR (IN CASH RS.0.00 CR) AND CREDITS: RS.27.81 CR (IN CASH RS.0.00 CR). CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FY : 2020-21 IS DEBIT: RS.228.63 CR (IN CASH RS.0.11 CR) AND CREDITS: RS.220.63 CR (IN CASH RS.0.00 CR). 2019-20 IS DEBIT: RS.76.41 CR (IN CASH RS.0.10 CR) AND CREDITS: RS.75.89 CR (IN CASH RS.0.07 CR). (2018-19): DEBIT: RS.29.76 CR (IN CASH RS.0.40 CR) AND CREDITS: RS.29.87 CR (IN CASH RS.0.03 CR).CURRENT A/CS WERE OPENED IN THE NAME OF A PARTNERSHIP FIRM M/S PREMIUM PETRO PRODUCTS NOS. 6813095059 AND 6813095042 ON 04-SEP-19 AT OUR BRANCH LOCATED IN 57 KRISHNA TOWER, BASEMENT, 1ST FLOOR, SARDAR PATEL MARGH JAIPUR RAJASTHAN - 302001. THE ENTITY WAS FORMED ON 09-MAY-17 AND HAS SUBMITTED COPIES OF PAN AAUFP2067N, PARTNERSHIP DEED, CERT. OF REG. NO.13/974/2017, IEC AAUFP2067N AND GST REG. CERT. 08AAUFP2067N1ZF AS KYC DOCS AT THE TIME OF A/C OPENING. CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 1-3 HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN - 302001. RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA AND LALIT KUMAR NAREDI ARE THE PARTNERS WHILE MANISH KUMAR BOHRA AND UJJWAL BHANDARI ARE THE UBOS IN THE SAID FIRM. CUSTOMER HAS REGISTERED MOBILE NO. 9829999990 WITH THE BANK. THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION. IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF PETROLEUM PRODUCTS WITH ANNUAL TURNOVER OF RS.100.00 CR. BASIS INTER-CONNECTEDNESS, OTHER A/CS WERE IDENTIFIED WHICH COMPRISE OF 25 CURRENT A/CS IN THE NAMES OF 9 REGISTERED COS., 1 NBFC, 8 PARTNERSHIP FIRMS, 2 LLP FIRMS AND 1 INDIVIDUAL CURRENT A/C. BESIDES 15 SAVINGS A/CS WERE OPENED IN THE NAMES OF INDIVIDUALS AND 1 HUF. DETAILS OF A/CS ARE UPDATED IN XML FILE. THE MENTIONED A/CS WERE OPENED DURING THE PERIOD 06-SEP-09 TILL 04-NOV-20 AT ABOVE MENTIONED AJMER ROAD, RAJA PARK, NORTH AVENUE, VAISHALI NAGAR BRANCHES IN JAIPUR - RAJASTHAN AND SECTOR 17 VASHI BRANCH IN NAVI MUMBAI. IT WAS INDICATED THAT THE ENTITIES ARE INTO VARIED BUSINESSES LIKE REAL ESTATE SERVICES, WHOLESALER OF CHARCOAL/ SODA / ASH, INFRASTRUCTURE, GENERAL TRADING, TEXTILE PRODUCTS, CONSTRUCTION, FINANCE AND INVESTMENT, DEALING IN STONES/MARBLE/GRANITE, TRADE/SALE OF HANDICRAFTS WITH ANNUAL TURNOVER IN THE RANGE OF RS.0.01 CR TO RS.4.00 CR. RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA, LALIT KUMAR NAREDI AND BHEEM RAJ BOHRA ARE THE COMMON DIRECTORS/ PARTNERS/ KARTA IN THE ABOVE MENTIONED A/CS WITH OTHER INDIVIDUALS IN DIFFERENT COMBINATIONS. NOTICING HIGH VALUE ROUND SUM NEFT/RTGS CREDITS FROM OWN A/C AND SUBSEQUENT HIGH VALUE ADVANCE IMP. REM. THROUGH THE A/C OF PREMIUM PETRO PRODUCTS 6813095059, AN ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD AND THIS CONSOLIDATED REPORT IS FILED BASIS INTER-CONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION: 1 ON REVIEWING A/C TRANSACTIONS SINCE APRIL 1, 2020 - APRIL 15, 2021 IT WAS NOTED THAT IMPS/NEFT/RTGS CREDITS AMOUNTED TO RS.178.24 CR MAINLY FROM PREMIUM PETRO PRODUCTS RS.142.08 CR, IDBI - 00131020000080662, VCI EXPORTS P LTD RS.3.09 CR, IDFC 10039338826, SURYA NAGARI TRADING RS.2.97 CR, YES BANK 004163700002899, ABHA PRECISION FARMING LLP RS.2.20 CR, IDBI 0013102000077552 AND BHEEM RAJ BOHRA RS.2.02 CR, IDBI 0131063538000. 2 EXPORT ADVANCE REMITTANCES AMOUNTED TO RS.1.24 CR IN THE A/CS OF INDIA STONE HOUSE LLP AND VCI EXPORTS P LTD MAINLY FROM WONDERSTONE LLC - USA RS.0.29 CR, TRANQUILLO GMBH RS.0.21 CR, GRAMACO SOURCE AND MARBLE RS.0.18 CR, STONE SOURCE INTERNATIONAL INC USA RS.0.14 CR AND STONELAND INC - USA RS.0.10 CR. AS PER WEB CHECK WONDERSTONE LLC USA IS INTO BUSINESS OF JEWELLERY STORES HTTPS://WWW.DNB.COM/BUSINESS-DIRECTORY/COMPANY-PROFILES.WONDERSTONE LLC.4D21F25E0AAE301E36AA8061651D299F.HTML, TRANQUILLO GMBH IS INTO CLOTHING STORE INDUSTRY, STONE SOURCE INTERNATIONAL INC IS INTO BUSINESS OF STONES/ GRANITES AND STONELAND INC IS INTO GRANITE/ MARBLE BUSINESS. IT APPEARS THAT INDIA STONE HOUSE LLP WHICH HAS INDICATED THAT IT IS INTO BUSINESS OF STONES/ GRANITES MAY HAVE RECEIVED EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS LIKE WONDERSTONE LLC USA JEWELLERY STONES AND TRANQUILLO GMBH CLOTHING STORE INDUSTRY AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE.",{"entities":[(126,153,"FOREIGN ORG"),(168,181,"FOREIGN ORG"),(196,214,"FOREIGN ORG"),(229,253,"FOREIGN ORG"),(268,292,"FOREIGN ORG"),(306,326,"FOREIGN ORG"),(6623,6637,"FOREIGN ORG"),(6655,6669,"FOREIGN ORG"),(6683,6707,"FOREIGN ORG"),(6721,6750,"FOREIGN ORG"),(6771,6783,"FOREIGN ORG"),(6818,6832,"FOREIGN ORG"),(6931,6945,"FOREIGN ORG"),(6986,7000,"FOREIGN ORG"),(7035,7064,"FOREIGN ORG"),(7107,7119,"FOREIGN ORG"),(7333,7347,"FOREIGN ORG"),(7374, 7388,"FOREIGN ORG")]}),
            ("THE TRANSACTION PATTERN OBSERVED IN CUSTOMER S ACCOUNT IS RECEIPT OF HIGH VOLUME OF DOMESTIC INWARD RTGS FROM CUSTOMER S OWN ACCOUNT WITH AXIS BANK LTD, KHAR MUMBAI AND ICICI BANK LTD, ANDHERI MUMBAI AND FROM ENTITY NAMED SAWARTHIYA FILM PRODUCTION WHOSE TRANSACTIONS ARE FROM SAME IFSC AS CUSTOMER S ACCOUNT ICIC0000011 I.E, ANDHERI, MUMBAI IMMEDIATELY FOLLOWED BY FOREIGN OUTWARD REMITTANCES. THEY SEEM TO BE RELATED PARTIES FOLLOWING EXACT SAME TRANSACTION ROUTE OF SENDING DOMESTIC RTGS. THERE ARE NO DETAILS FOUND ON SAWARTHIYA FILM PRODUCTION ON PUBLIC DOMAIN EXCEPT A SMALL SNIPPET FOUND ON SOCIAL MEDIA THAT THEY ARE CO PRODUCERS TO OUR CUSTOMER ON ONE OF THE PROJECTS REFER LINK HTTPS //WWW.FACEBOOK.COM/WATCH/ V 421845292404795. SAWARTHIYA FILM PRODUCTION IFSC ICIC0000011 HAS SENT RTGS TO OUR CUSTOMER AMOUNTING TO INR 425.52 BETWEEN 15TH MARCH 21 AND 16TH APRIL 21. CUSTOMER HAS SENT BACK INR 24.48 LAKHS TO SAWARTHIYA FILM PRODUCTION AT IFSC ICIC0001352, NEW DELHI IN MARCH 21. THESE TRANSACTIONS SEEMS TO HAVE NO ECONOMIC RATIONALE, NOR THERE SEEMS TO BE RATIONALE BEHIND OPENING OF ACCOUNTS/GETTING BANKING SERVICES AT VARIOUS BANKS AND LOCATIONS.9. CUSTOMER HAS A FACEBOOK PAGE, YOUTUBE CHANNEL WITH FEW VIDEOS AND THERE ARE NEWS ARTICLES ON THE PRODUCTION OF THE VIDEOS WITH SOCIAL MEDIA INFLUENCERS OR RELATIVELY KNOWN ARTISTS. HOWEVER, THE NUMBER OF FOLLOWERS OR LIKES OR VIEWS PROJECT THE EARNINGS A COMPANY COULD MAKE AND THE SAME DOESN T SEEM TO BE IN TANDEM WITH THE VOLUME OF OUTWARD REMITTANCES MADE. REFER LINK HTTPS //WWW.FORBES.COM/SITES/JROSE/2019/03/21/HOW MUCH DO YOUTUBERS REALLY MAKE/ SH 637AB9227D2B, HTTPS //MINT.INTUIT.COM/BLOG/RELATIONSHIPS/HOW MUCH DO YOUTUBERS MAKE 5035/ TEXT WITH 20THE 20AVERAGE 20YOUTUBE 20PAY,FOR 20EVERY 201 2C000 20VIDEO 20VIEWS.,HTTPS //WWW.SANTABANTA.COM/BOLLYWOOD/MANISH KUMAR SINGH AND SAPNA KUMARI GRACE THE MAASHUKA SONG LAUNCH EVENT PRESENTED BY IMAGINARY FILMS IN COLLABORATION WITH TAGDOG MEDIA, HTTPS //INDUS.DAILYHUNT.IN/NEWS/INDIA/URDU/GOSSIP GANJ EPAPER GOSGANEN/MANISH SINGH AND SAPNA S IMAGINARY FILMS RELEASES A GROOVY CHARTBUSTER MAASHUKA FEATURING BHAVIN BHANUSHALI NEWSID N252433968 PGS N AND PGN 0 AND SNIPPET FROM ONE OF THE ARTICLES FORBES ALSO ESTIMATES THAT FOR TOP TALENT, A YOUTUBER CAN MAKE ABOUT 5 FOR EVERY 1,000 VIDEO VIEWS AND CUSTOMER HAS ONLY 1.9 MN VIEWS OVERALL WITH TOTAL 5 VIDEOS UPLOADED IN LAST 3 MONTHS . CUSTOMER HAS MINIMAL LEGITIMATE WORK AND IT IS NOT IN LINE WITH THE INFERRED EARNINGS ONE COULD MAKE IN THAT LINE OF ACTIVITY THROUGH YOUTUBE AND THE TRANSACTIONS THAT CUSTOMER IS MAKING, THIS WAY WE ARE UNABLE TO GAUGE THE SOURCE OF WEALTH. ALSO, UNABLE TO UNDERSTAND THE SOURCE OF FUNDS SINCE CUSTOMER IS ROUTING RTGS FROM VARIOUS DOMESTIC BANK ACCOUNTS TO OUR BANK.10. THE ANNUAL TURNOVER CLAIMED BY THE CUSTOMER WITH OUR BANK IS NOT IN TANDEM WITH THE ACTUAL TURNOVER SEEN IN THE ACCOUNT IN A SINGLE QUARTER.11. CUSTOMER BEING NEWLY INCORPORATED HAS OPENED ACCOUNTS IN MULTIPLE BANKS ICICI BANK, AXIS BANK AND OUR BANK WHICH RAISES SUSPICION ON THE INTENT AND THE PURPOSE.12. REQUESTING TO FILE STR BASIS BELOW POINTS A THE INVOICE OF ONE OF THE OVERSEAS PARTIES IS NOT GENUINE.B ALL RELATED PARTIES AND ACCOUNTS ARE RELATIVELY NEWLY INCORPORATED.C UNABLE TO GAUGE THE SOURCE OF WEALTH AND SOURCE OF FUNDS.D THE OPENING OF ACCOUNTS IN VARIOUS BANKS AND THEN PROCEEDING WITH THE TRANSACTIONS IN A PATTERN THAT MAKES NO ECONOMICAL SENSE.E THE ACTUAL TURNOVER IS HIGHER THAN THE CLAIMED ANNUAL TURNOVER, TURNOVER IN A SINGLE QUARTER HAS CROSSED THE CLAIMED ANNUAL TURNOVER.PART A GROUNDS OF SUSPICION 1.CUSTOMER IMAGINARY FILMS PRIVATE LIMITED A/C 59207039548313 IS A NEWLY OPENED ACCOUNT WHO HAS REMITTED HUGE VOLUME OF FOREIGN OUTWARD REMITTANCES IN A SHORT PERIOD OF TIME.2. CUSTOMER IMAGINARY FILMS PRIVATE LIMITED MAINTAINS CURRENT ACCOUNT NUMBER 59207039548313 WITH OUR KHAR WEST, MUMBAI BRANCH SINCE 15 01 2021. PAN NUMBER IS UPDATED AS AAFCI5545Q,MOBILE NUMBER IS UPDATED AS 917039548313.PROFILE DECLARED IS FILM PRODUCTION WITH ANNUAL TURNOVER OF INR 600 LAKHS. EMAIL ID UPDATED IS INFO IMAGINARYFILMS.IN. DATE OF INCORPORATION IS 22 01 2020. GSTIN IS 27AAFCI5545Q1ZW.RISK PROFILE IS MEDIUM.BALANCE IN THE ACCOUNT AS ON 17TH APRIL 2021 IS INR 164174.41.THE CUSTOMER IS A PRIVATE LIMITED.MAILING AND PERMANENT ADDRESS OF THE CUSTOMER IS C/O IMAGINARY FILMS PRIVATE LIMITED 402 EMBASSY CHAMBERS 3RD ROAD KHAR WEST, MUMBAI, MAHARASHTRA 400052. AUTHORISED SIGNATORIES ARE SAPNA KUMARI AND MANISH KUMAR SINGH. BENEFICIAL OWNERS ARE SAPNA KUMARI AND MANISH KUMAR SINGH.3. SCRUTINY OF ACCOUNT STATEMENT OF IMAGINARY FILMS PRIVATE LIMITED A/C NO 59207039548313 FOR PERIOD SINCE JAN 2021 ACCOUNT OPENING DATE TILL DATE REVEALED THAT CUSTOMER HAS RECEIVED A TOTAL CREDIT OF INR 1305.77 LAKHS. OUT OF THE TOTAL CREDITS OF INR 1305.77 LAKHS, RTGS/NEFT/UPI/IMPS CREDITS ARE INR 1303.26 LAKHS FROM CUSTOMER S OWN ACCOUNT IN ICICI BANK AND AXIS BANK AND FROM ANOTHER ENTITY NAMED SAWARTHIYA FILM PRODUCTION WHICH TOO IS IN ICICI BANK .MAJOR DEBITS HAVE BEEN OBSERVED VIA FOREIGN OUTWARD REMITTANCES OF INR 1279.54 LAKHS TO JUST 2 OVERSEAS PARTIES BASED OUT OF SINGAPORE .THESE AFOREMENTIONED DEBITS ARE OUT OF TOTAL DEBITS OF INR 1304.12 LAKHS.4. NO LINKED ACCOUNTS FOUND EMAIL ID, MOBILE NO AND PAN WERE CHECKED.PART B DETAILS OF INVESTIGATION5. CUSTOMER HAS MADE FOREIGN OUTWARD REMITTANCES AMOUNTING TO INR 1278.54 LAKHS AGAINST PURPOSE CODE S1101 AUDIO VISUAL AND RELATED SERVICES LIKE MOTION PICTURE AND VIDEO TAPE PRODUCTION, DISTRIBUTION AND PROJECTION SERVICES TO JUST TWO OVERSEAS PARTIES NAMED LIVING FILMS PTE LTD, SG AND GATE STUDIOS PTE LTD, SG, WITHIN 3 MONTHS OF OPENING THE ACCOUNT WITH OUR BANK. 6. THE OVERSEAS PARTIES WHEN CHECKED ON PUBLIC DOMAIN, IT WAS FOUND THAT THEY TOO ARE NEWLY INCORPORATED LIVING FILMS PTE LTD HAS BEEN INCORPORATED ON 18TH SEPTEMBER 2020, REFER LINK HTTPS //WWW.SGPBUSINESS.COM/COMPANY/LIVING FILMS PTE LTD AND GATE STUDIOS PTE LTD HAS BEEN INCORPORATED ON 1ST OCTOBER 2020, REFER LINK HTTPS //SINGAPORE CORP.COM/CO/GATE STUDIOS PTE LTD. OUR CUSTOMER TOO HAS BEEN INCORPORATED IN 2020, IE 22ND JANUARY 2020.7. THERE IS AN ADDITIONAL INFORMATION FOUND WRT LIVING FILMS PTE LTD. ON PUBLIC DOMAIN SEARCH, IT WAS FOUND THAT THERE IS A RELATIVELY KNOWN COMPANY BY THE SAME NAME IN THAILAND, REFER LINK HTTPS //EN.WIKIPEDIA.ORG/WIKI/LIVING FILMS, HTTPS //WWW.LIVINGFILMS.COM/. THE CUSTOMER HAS SUBMITTED INVOICE WHEREIN LIVING FILMS PTE LTD, SG CLAIMS THAT LIVING FILMS, THAILAND IS THEIR HEADOFFICE AND HAS MENTIONED ITS GEOGRAPHICAL ADDRESS AND E MAIL ID FROM LIVING FILMS, THAILAND S WEBSITE HEREON WOULD BE MENTIONED AS WEBSITE , HOWEVER ON OBSERVING THE INVOICE CLOSELY IT IS FOUND THAT THE PINCODE GIVEN ON WEBSITE IS 50100 BUT IN INVOICE IT IS 501100 AND EMAIL ADDRESS ON WEBSITE IS LIVING LIVINGFILMS.COM BUT IN INVOICE IT IS LIVING LIVINGFILM.COM, THE CONTACT NUMBERS IN THE INVOICE ALSO DO NOT MATCH WITH THE ONES GIVEN ON THE WEBSITE, ALSO ONE OF THE NUMBERS 85292124533, 852 BELONGS TO HONGKONG WHERE AS FOR SINGAPORE IT IS 65 AND FOR THAILAND IT IS 66 REFER LINK HTTPS //EN.WIKIPEDIA.ORG/WIKI/TELEPHONE NUMBERS IN HONG KONG, HTTPS //EN.WIKIPEDIA.ORG/WIKI/TELEPHONE NUMBERS IN SINGAPORE, HTTPS //EN.WIKIPEDIA.ORG/WIKI/TELEPHONE NUMBERS IN THAILAND AND THERE IS NO MENTION OF SINGAPORE BRANCH ON THE LIVING FILMS, THAILAND S WEBSITE. THE INVOICE DOES NOT SEEM GENUINE AND SEEMS TO BE FRAUDULENT. THE AGREEMENTS WRT BOTH THE OVERSEAS PARTIES HAVE NO MENTION OF ANY STAKE HOLDER AND IT IS EXACTLY THE SAME.",{"entities":[(5606,5624,"FOREIGN ORG"),(5635,5653,"FOREIGN ORG"),(5820,5839,"FOREIGN ORG"),(5934,5953,"FOREIGN ORG"),(5959,5978,"FOREIGN ORG"),(6064,6082,"FOREIGN ORG"),(6203,6221,"FOREIGN ORG"),(6375,6385,"FOREIGN ORG"),(6462,6480,"FOREIGN ORG"),(6604,6614,"FOREIGN ORG"),(7353, 7363,"FOREIGN ORG"),(0,0,"FOREIGN ORG"),(0,0,"FOREIGN ORG"),(0,0,"FOREIGN ORG")]}),
            ("FOR EXAMPLE, VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN 120 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON 17 02 2020 USD 2.005LACS WAS REMITTED TO CITY PULSE GENERAL TRADING LLC IN UAE 0007NMDC0046420 NO PROPER DETAILS ABOUT THE CLIENT IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 27132000 BITUMEN. HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS IS STILL DUE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY OF MORE THAN 120 DAYS, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON 24 01 2020 USD 1.50 LACS TO CITY PULSE GENERAL TRADING LLC IN UAE 0007NMDC0046420 NO PROPER DETAILS ABOUT THE CLIENT IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 27132000 BITUMEN GRADE. THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS DUE. IN TOTAL 5 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF 7.54LACS. WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.KAPSON EXIM PRIVATE LIMITED, PRIVATE LTD, HAVING ADDRESS 403,BAJAJ HOUSE 97,4TH FLR,NEHRU PLACE,, SOUTH DELHI, NEW DELHI, DELHI, INDIA, 110019, CONTACT NUMBER 919953111443 AND EMAIL ID KAPSONEXIM93 GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 418005000214 WITH ICICI BANK LTD, DELHI 19 NEHRU PLACE,19, CHAWLA HOUSE, NEHRU PLACE, DELHI, 110019, SINCE 21022019. DATE OF INCORPORATION IS 23081993. PAN NUMBER OF THE ACCOUNT IS AAACK0464B. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0593052391. GSTIN AS PER BANK RECORDS IS 07AAACK0464B1Z4. OTHER RELATIONSHIP OF CUSTOMER ARE 006105018350, 418005000699, 418006000002. TRANSACTION LINKED ACCOUNTS ARE 418010000315, 418010000421. AUTHORIZED SIGNATORY AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ANKITA BANSAL AFBPB9149M , MUKESH KUMAR JINDAL AOFPJ6342H . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, GSTIN, COI U74899DL1993PTC054937. KAPSON EXIM PRIVATE LIMITED IS ENGAGED IN THE BUSINESS OF SELLING ALL KINDS OF COPPER AND BRASS SCRAPS. AS PER ETRG FEEDBACK AND DEDUPE FOR PROMOTERS OF THE COMPANY HAVE BUSNIESS OF MTT IN BITUMEN AND TARCOAL. THEY HAVE FIRM IN NAME OF D TACH INNOVATION 418005500023 . DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS.5 CR TO RS.25 CR.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.1.04 LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 562.17 LAKHS OF WHICH CASH WAS RS. 0.50 LAKHS MAJORLY IN DELHI 19, NEHRU PLACE BRANCH, TRANSFERS RS. 1.00 LAKH MAJORLY FROM KAPSON EXIM PRIVATE LIMITED, NEFT RS. 2.42 LAKHS MAJORLY FROM KAPSON EXIM PVT LTD,KAPSON EXIM PRIVATE LIMITED, RTGS RS. 301.80 LAKHS MAJORLY FROM ANKITA BANSAL,KAPSON EXIM PVT LTD, REMITTANCE RS. 249.39 LAKHS, MOBILE MONEY TRANSFERS RS. 3.06 LAKHS IN 2 INSTANCES. TOTAL DEBIT SIDE RS. 567.90 LAKHS OF WHICHTRANSFERS RS. 8.17 LAKHS MAJORLY TO KAPSON EXIM PRIVATE LIMITED, CLEARING RS. 0.79 LAKHS MAJORLY TO BSES RAJDHANI POWER LTD,RAVINDER PANDITA,BIF BUILDING MAINTENAN, RTGS RS. 291.90 LAKHS MAJORLY TO KAPSON EXIM PRIVATE LI,BANYAN TREE INVESTMENT,KAPSON EXIM PRIVATE LTD, REMITTANCE RS. 259.88 LAKHS MAJORLY TO CITY PULSE GENERAL TRADING LLC,SIGNIFY BUILDING MATERIALS TRADING,SYSKEP DMCC, INTERNET FUND TRANSFER RS. 0.05 LAKHS, MOBILE MONEY TRANSFERS RS. 6.58 LAKHS IN 21 INSTANCES.FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 427.97 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 196.61 LAKHS . TOTAL DEBIT SIDE RS. 452.00 LAKHS OF WHICH CASH WAS RS. 1.00 LAKH, REMITTANCE RS. 156.93 LAKHS .FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 32.03 LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. 1.23 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 0.23 LAKHS . VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E FUNDS OUTLAY OF MORE THAN 120 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 5 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 7.54LACS.",{"entities":[(188,217,"FOREIGN ORG"),(684,713,"FOREIGN ORG"),(3760,3788,"FOREIGN ORG")]}),
            ("INFO/EXPORTERS/LISTING/FAISHING INDUSTRIAL HK CO LTD IN HONGKONG WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 85235910 TAGS. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED ON 18 02 2020 FROM FUNSTATION LIMITED FUNSTATION ARE FAMILY ENTERTAINMENT CENTRES WITH CARNIVAL GAMES, LASER TAG, ADVENTURE GOLF, BOWLING, VR AND MUCH MORE FUN GAMES HTTPS //WWW.FUNSTATIONUK.COM/ . IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 14 07 2027 TO 03 07 2018 TO TUNE OF USD2645 TO CARDY INTELLIGENT TECHNOLOGY CO LTD CUSTOMER IS INTO MANUFACTURE OF RFID TAGS, WRISTBAND ETC.HTTPS //WWW.SMARTCARDY.COM/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 85235910 RFID CARDS. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 20 09 2019 AND 20 01 2020 TO THE TUNE OF USD1.89 LACS FROM BAHJAT BAHLA BAHLA AL MAMOORAH DETAILS OF THE WHOM IS NOT AVAILABLE. IN TOTAL 23 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD2.81 LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.SEMNOX SOLUTIONS PVT. LTD, PRIVATE LTD, HAVING ADDRESS 3RD FLOOR FLR,MG ROAD LALBAGH,MANGALORE,PUNJA BUILDING ANNEXE, MANGALORE, KARNATAKA, INDIA, 575003, CONTACT NUMBER 0 08244255888 AND EMAIL ID IQBAL SEMNOX.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 001406000019 WITH ICICI BANK LTD, MANGALORE P M RAO ROAD,BHARATH BUILDING,GROUND FLOOR, P.M.RAO ROAD,MANGALORE, KARNATAKA, 575001, SINCE 03102011. DATE OF INCORPORATION IS 21022008. PAN NUMBER OF THE ACCOUNT IS AALCS7710B. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0708003036. TRANSACTION LINKED ACCOUNTS ARE 603091010708. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE KIRAN KARANKI AUTHORISED SIGNATORY , SAMPATH KUMAR DIRECTOR , IQBAL MOHAMMAD DIRECTOR AAHPI5891A ,RAJEEV GOPALAKRISHNAN DIRECTOR , BALAJI VENUGOPAL DIRECTOR .BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS KIRAN KARANK ADZPK4808Q ,MOHAMMAD IQBAL AAHPI5891A . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI. DATE OF LAST KYC UPDATION 13052019.SEMNOX SOLUTIONS PRIVATE LIMITED SSPL IS ENGAGED IN THE BUSINESS OF PROVIDING IT SOLUTIONS FOR THE ENTERTAINMENT AND LEISURE INDUSTRY. SSPL OFFERS SOLUTIONS SUCH AS FAMILY ENTERTAINMENT, WATER AND THEME PARKS, AND FOOD AND BEVERAGES. THE COMPANY CATERS TO WATER PARKS, MUSEUMS, KIDS PLAY AREAS, RESTAURANTS, FOOD COURTS, AND AQUARIUMS, AMONG OTHERS. SSPLS FEC AND PARK CLIENTS INCLUDE SUCH AS COWPLAY CPWMOO, SINGAPORE, ARENA REKREASI KELUARGA, AEON FANTASY, MALAYSIA, AND SPARKS, INDONESIA, AND CAMAYA COAST, PHILIPPINES, AMONG OTHERS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 5CR TO 25CR.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.0.61 LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 17.66 LAKHS OF WHICH TRANSFERS RS. 0.15 LAKHS, REMITTANCE RS. 17.51 LAKHS MAJORLY FROM SEMNOX SOLUTIONS LLC 11498 LU,AEON FANTASY MALAYSIA SDN BHD,SEMNOX SOLUTIONS DMCC P O BOX. TOTAL DEBIT SIDE RS. 19.75 LAKHS OF WHICH TRANSFERS RS. 4.75 LAKHS MAJORLY TO SEMNOX SOLUTIONS PRIVATE LIMITED, REMITTANCE RS. 14.49 LAKHS MAJORLY TO FAISHING INDUSTRIAL HK CO LIMITED,TITOMA MANUFACTURING LTD,AMAZON WEB SERVICES, INC. FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 55.46 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 55.46 LAKHS. TOTAL DEBIT SIDE RS. 61.27 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 61.27 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 46.27 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. 46.27 LAKHS. TOTAL DEBIT SIDE RS. 52.20 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 52.20 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 66.68 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 66.68 LAKHS. TOTAL DEBIT SIDE RS. 61.37 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. 51.81 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS, LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR 23 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF US2.81LACS. FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON 28 08 20218 USD 3575 REMITTED TO FAISHING INDUSTRIAL HK CO LTD THIS COMPANY DEALS IN EXPORT, UNDER DISCS, TAPES, SOLID STATE NON VOLATILE STORAGE DEVICES, SMART CARDS AND OTHER MEDIA FOR THE RECORDING OF SOUND OR OF OTHER PHENOMENA, WHETHER OR NOT RECORDED, INCLUDING MATRICES AND MASTERS FOR THE PRODUCTION OF DISCS, BUT EXCLUDING PRODUCTS OF CHAPTER 37 CATEGORY.",{"entities":[(23,51,"FOREIGN ORG"),(305,322,"FOREIGN ORG"),(3839,3870,"FOREIGN ORG"),(4924,4952,"FOREIGN ORG"),(665,699,"FOREIGN ORG"),(2966,2978,"FOREIGN ORG"),(2993,3014,"FOREIGN ORG"),(3018,3028,"FOREIGN ORG"),(3046,3050,"FOREIGN ORG"),(3069,3079,"FOREIGN ORG"),(3598,3617,"FOREIGN ORG"),(3628,3639,"FOREIGN ORG"),(3873,3895,"FOREIGN ORG"),(3898,3920,"FOREIGN ORG")]}),
            ("FUJIFILM SPECIALITY INK SYSTEMS LTD . WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 32151940 SCREEN PRINTING INKS. HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED BETWEEN 30 05 2017 TO 30 06 2020 TO TUNE OF GBP1.35LACS ,USD2.34LACS TO CEYLON BUSINESS APPLIANCES PVT LTD ENTITY DEALS IN EXPORT OF BANKING PRODUCTS KIOSKS, COUNTING MACHINEETC., POTOCOPIERS HTTP //WWW.CBA.LK/ ,STRETCHLINE PVT LTD LEADING THE WAY AS THE WORLDS LARGEST MANUFACTURER OF NARROW ELASTICATED FABRIC AND A MAJOR SUPPLIER OF COVERED ELASTOMERIC YARN AND TEXTILE COATING AND BONDING HTTPS //STRETCHLINE.COM/ ETC. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS PIONEER TO SPORTS WEAR,PRINTING INK. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS WHERE, HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED ON 02 03 2020 TO TUNE OF USD 12830 FROM HI FASHION HOLDING PVT LTD DEALS I N TEXTILE BUSINESS HTTPS //WWW.HIFASHION.LK/ . HOWEVER THE LINKED IMPORT PAYMENTS ARE STILL DUE. IN TOTAL 134 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 2.87LACS, GBP 2.38LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF PIONEER TO SPORTS WEAR,PRINTING INK. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.FUJIFILM SERICOL INDIA PVT LTD, PRIVATE LTD, HAVING ADDRESS 10/11,B BHANDARI INDUSTRIAL ESTATE, SANASWADI TALUKA SHIRUR, PUNE, MAHARASHTRA, INDIA, 412208, CONTACT NUMBER 917030952710 AND EMAIL ID PRASHANT.BARI FUJIFILM.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000506000141 WITH ICICI BANK LTD, PUNE BUND GARDEN,A WING SHANGRILA GARDENS BUND GARDEN ROAD, PUNE, MAHARASHTRA, 411001, SINCE 22122004. DATE OF INCORPORATION IS 19021996. PAN NUMBER OF THE ACCOUNT IS AABCS0098L. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0396015123. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE PRAVIN SHARAD KHAPARDE AUTHORISED SIGNATORY , PRASANNA RAGHAVRAO MOTUPALLI AUTHORISED SIGNATORY AATPM0665L . BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS NARENDRA AGNIHOTRI ADUPA6988J ,SREEHARI SREEDHAR ADAPS0160N . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI. DATE OF LAST KYC UPDATION 13022019. ENTITY IS PRIMARILY ENGAGED IN THE MANUFACTURING AND MARKETING OF PRINTING INKS, THINNERS AND ADDITIVES. FSIPL ALSO TRADES DIGITAL MACHINES. THE COMPANYS PRODUCT RANGE INCLUDES SCREEN INK GRAPHIC INK, INDUSTRIAL INK, PACKAGING INK, TEXTILE INK, CHEMICAL AND EMULSION , DIGITAL INK UV INKS AND SOLVENT INKS , NARROW WEB INK UV FLEXO INKS, UV FLAT SCREEN INKS AND UV ROTARY SCREEN INKS , OFFSET INK DRY OFFSET UV INK AND LETTERPRESS UV INK AND SUNGLO CHANNELS INKS SUNGLO GRAPHIC INK AND SUNGLO INDUSTRIAL INK .ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.0.19 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2021 22 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS. 0.12 LAKHS OF WHICH REMITTANCE RS. 0.12 LAKHS. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 0.02 LAKHS OF WHICH REMITTANCE RS. 0.02 LAKHS MAJORLY TO FUJIFILM SPECIALITY INK, .FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 1.47 LAKHS OF WHICH REMITTANCE RS. 1.47 LAKHS MAJORLY FROM FUJIFILM SPECIALITY INK SYSTE,THREE SQUARE TECHNOLOGY LIMIT,FOR YOU INDUSTRY CO.,LTD. TOTAL DEBIT SIDE RS. 1.47 LAKHS OF WHICH REMITTANCE RS. 1.44 LAKHS MAJORLY TO FUJIFILM SPECIALITY INK,FUJIFILM SPECIALITY INK SYSTEMS,FUJIFILM SERICOL INDIA PVT LTD. FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 2.05 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 2.05 LAKHS. TOTAL DEBIT SIDE RS. 2.28 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 2.28 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 3.05 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 3.05 LAKHS. TOTAL DEBIT SIDE RS. 3.35 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 3.35 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 2.69 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 2.69 LAKHS. TOTAL DEBIT SIDE RS. 3.46 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. 3.46 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 134 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 2.87LACS, GBP 2.38LACS. FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN 270 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN 29 06 2017 TO 24 01 2018 GBP 1.59 LACS REMITTED IN PARTS FROM FUJIFILM SPECIALITY INK SYSTEMS LTD ENTITY IS INT SPECIALTY CHEMICALS",{"entities":[(0,34,"FOREIGN ORG"),(3747,3774,"FOREIGN ORG"),(3935,3964,"FOREIGN ORG"),(5027,5061,"FOREIGN ORG")]}),
            ("IN TOTAL 3 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 25.49LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF MACHINE TOOL AND ACCESSORIES. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.THERMAX BABCOCK WILCOX ENERGY SOLU.P.LTD,PRIVATE LTD., HAVING ADDRESS ADISA,ICON,GAT NO FLR,301/2,, BANGALORE HIGHWAY,BAVDHAN BUDRUK,OPPOSITE HEMRL,MUMBAI, PUNE, MAHARASHTRA, INDIA, 411021, CONTACT NUMBER 919850891016 AND EMAIL ID RAMNATH.NAYAK TBWES.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000505019247 WITH ICICI BANK LTD, PUNE BUND GARDEN,A WING SHANGRILA GARDENS BUND GARDEN ROAD, PUNE, MAHARASHTRA, 411001, SINCE 18082010. DATE OF INCORPORATION IS 26062010. PAN NUMBER OF THE ACCOUNT IS AADCT5273A. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0310065542. GSTIN AS PER BANK RECORDS IS 27AADCT5273A1ZJ. TRANSACTION LINKED ACCOUNTS ARE 000510164069,000510164070.AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NIRANJAN PAWGI AUTHORISED SIGNATORY ABKPP6008N , NIRANJAN PAWGI AUTHORISED SIGNATORY ABKPP6008N ,LATA KUMAR AUTHORISED SIGNATORY , ASHISH MOHAN LAL BHANDARI AUTHORISED SIGNATORY AEXPB3596F ,MOHAN LAL BHANDARI ASHISH AUTHORISED SIGNATORY AEXPB3596F . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA,AOA, COI. THERMAX BABCOCK WILCOX ENERGY SOLUTIONS PRIVATE LIMITED TBWES IS A JOINT VENTURE BETWEEN THERMAX LIMITED, PUNE, INDIA AND BABCOCK WILCOX POWER GENERATION GROUP, INC., USA. THE COMPANY IS ENGAGED IN MANUFACTURING SUBCRITICAL AND SUPERCRITICAL BOILERS FOR THERMAL POWER GENERATION. THE COMPANYS OTHER PRODUCTS INCLUDE LOW NOX COMBUSTION SYSTEMS AND ROLL WHEEL PULVERIZERS.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.136.12 LAKHS CREDIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 9,083.60 LAKHS OF WHICH, CLEARING RS. 771.79 LAKHS MAJORLY FROM ICICI PRUDENTIAL M, NEFT RS. 352.72 LAKHS MAJORLY FROM MAHARASHTRA VALUE ADDED TAX,UTI MUTUAL FUND,THERMAX LIMITED, RTGS RS. 7,840.38 LAKHS MAJORLY FROM UTI MF COMMON FUND,SBI MUTUAL FUND,DSP MUTUAL FUND REDEMP, INTERNET FUND TRANSFERS RS. 50.04 LAKHS AND OTHERS RS. 68.68 LAKHS. TOTAL DEBIT SIDE RS. 8,992.20 LAKHS OF WHICH TRANSFERS RS. 6,753.00 LAKHS MAJORLY TO THERMAX BABCOCK WILCOX ENERGY SOLU.P.LTD, CLEARING RS. 65.43 LAKHS, RTGS RS. 35.40, REMITTANCE RS. 17.86 LAKHS MAJORLY TO MANTRA SWITCHGEAR CO LTD,LSIS CO LTD, INTERNET FUND TRANSFER RS. 2,000.00 LAKHS. FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 6,351.74 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 103.66 LAKHS. TOTAL DEBIT SIDE RS. 6,374.94 LAKHS OF WHICH RS. 0 BY CASH , REMITTANCE RS. 0.24 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 12,972.86 LAKHS OF WHICH CASH WAS RS. 870.00, REMITTANCE RS. 5,382.50 LAKHS . TOTAL DEBIT SIDE RS. 13,401.79 LAKHS OF WHICH REMITTANCE RS. 2,728.00 LAKHS .FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 21,006.57 LAKHS OF WHICH CASH WAS RS. 0.18 LAKHS, REMITTANCE RS. 7,650.74 LAKHS . TOTAL DEBIT SIDE RS. 21,405.45 LAKHS OF WHICH REMITTANCE RS. 2,000.98 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NO COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 3 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 25.49LACS. FOR EXAMPLE, MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN ON 16 06 2020 USD 17900 CREDITED FROM LSIS CO LTD AUTOMATION, PLC, ELECTRONIC DEVICES MANUFACTURING FROM SOUTH KOREA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 85371000 OTHERS. HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED FROM THE BABCOCK WILCOX COMPANY FROM USA. THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE.",{"entities":[( 2729,2751,"FOREIGN ORG"),(2754,2765,"FOREIGN ORG"),(3802,3812,"FOREIGN ORG"),(4109,4134,"FOREIGN ORG")]}),
            ("AND GOODS MENTIONED AS HS 90213900 ORTHOPAEDIC GOODS,HS 90213900 DRILLING MACHINE,HS 90213900 CONNECTING RODS. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED 14 01 2019 TO 24 06 2020 FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS HTTPS //WWW.DNB.COM/BUSINESS DIRECTORY/COMPANY PROFILES.INNOVATION ORTHO LINE LIMITED.C53A1BBABB68E37C869CE88A51209509.HTML . IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 04 10 2019 AND 07 03 2019 TO TUNE OF USD 3.06LACS TO JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS 90211000 DRILLING, HS 90213900 SELF DRILLING. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 11 02 2019 AND 24 06 2020 TO THE TUNE OF USD7.08LACS FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS HTTPS //WWW.DNB.COM/BUSINESS DIRECTORY/COMPANY PROFILES.INNOVATION ORTHO LINE LIMITED.C53A1BBABB68E37C869CE88A51209509.HTML . IN TOTAL 29 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD46.90LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT,STR IS BEING FILED.AUXEIN MEDICAL PRIVATE LIMITED,PRIVATE LTD, HAVING ADDRESS PLOT NO 168 169 170,HSIIDC FLR,PHASE IV,KUNDLI SONIPAT, SONIPAT, HARYANA, INDIA, 131028, CONTACT NUMBER 09136220665 AND EMAIL ID ACCOUNTS AUXEINMEDICAL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 016006000018 WITH ICICI BANK LTD, NEW DELHI MODEL TOWN,H 4, MODEL TOWN III, DELHI, 110009, SINCE 16012015. DATE OF INCORPORATION IS 03092012. PAN NUMBER OF THE ACCOUNT IS AAKCA9159K. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 3308000946. THERE ARE NO LINKED ACCOUNT. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE RAHUL LUTHRA AUTHORISED SIGNATORY , DEEPAK LUTHRA AUTHORISED SIGNATORY , GAURAV LUTHRA, ASHOK LUTHRA.BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK LUTHRA ACBPL8216A ,RAHUL LUTHRA ADYPL0242J . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI. AS PER ENHANCED DUE DILIGENCE AUXEIN MEDICAL PRIVATE LIMITED AMPL IS ENGAGED IN MANUFACTURING OF ORTHOPEDIC IMPLANTS. THE COMPANYS PRODUCTS INCLUDE MINI FRAGMENT, HIP PROSTHESIS, INTERLOCKING NAILS, INSTRUMENT AND IMPLANT BOX, SPINE IMPLANTS, MAXILLOFACIAL IMPLANTS, WISE LOCKING PLATES, AND BONE PLATES. THE COMPANYS UNIT IS EQUIPPED WITH CNC MACHINES, DRILL MACHINES, COMPARATORS, CUTTING MACHINES AND SPECIAL PURPOSE MACHINES. AMPL HAS A RANGE OF 600 IMPLANTS AVAILABLE IN 60 COUNTRIES IN ASSOCIATION WITH 100 PARTNERS.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.1.76 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2021 22 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS. 0.09 LAKHS OF WHICH REMITTANCE RS. 0.09 LAKHS MAJORLY FROM 1 FIXIER SA DE CV 2 CARR DR M,1 INNOVATION ORTHO LINE LTD. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 0.00. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 86.31 LAKHS OF WHICH REMITTANCE RS. 86.31 LAKHS MAJORLY FROM 1 ZOLDAN CORPOREISHON CIA L,1 MEDICAL ORTOVIT SRL 2 BUCUR,1 ATRON HEALTH S A 2 8 LEOFO. TOTAL DEBIT SIDE RS. 86.42 LAKHS OF WHICH REMITTANCE RS. 86.29 LAKHS MAJORLY TO CHANGZHOU NUBE INTERNATIONAL,CHANGZHOU XIETONG INDUSTRIES CO,CHANGZHOU XIETONG INDUSTRIES CO LTD. FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 82.68 LAKHS OF WHICH REMITTANCE RS. 82.68 LAKHS.TOTAL DEBIT SIDE RS. 84.89 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 83.47 LAKHS .FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 68.03 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 68.03 LAKHS . TOTAL DEBIT SIDE RS. 64.03 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 64.03 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 1.60 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 1.60 LAKHS. TOTAL DEBIT SIDE RS. 1.60 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 1.60 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 29 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD46.9LACS. FOR EXAMPLE, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN 27 04 2018 TO 20 12 2018 USD 13.22 LACS REMITTED IN PARTS FROM JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE",{"entities":[(612,650,"FOREIGN ORG"),(4951,4989,"FOREIGN ORG"),(212,240,"FOREIGN ORG"),(351,379,"FOREIGN ORG"),(1002,1030,"FOREIGN ORG"),(1141,1169,"FOREIGN ORG"),(3356,3379,"FOREIGN ORG"),(3669 ,3691,"FOREIGN ORG"),(3697,3715,"FOREIGN ORG"),(3719,3740,"FOREIGN ORG"),(3833,3859,"FOREIGN ORG"),(3862 ,3891,"FOREIGN ORG"),(3894,3927,"FOREIGN ORG")]}),
            ("OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS REFINED SUNFLOWER OIL 15121910. HOWEVER THE RESPECTIVE EXPORT WAS RECEIVED ON BETWEEN 15 06 2020 AND 10 07 2020 TO THE TUNE OF RS.2.06LACS FROM D.J.T IMPORT EXPORT IN MADAGASCAR WHOSSE DETAILS ARE NOT FOUND IN PUBLIC DOMAIN. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN 270 DAYS, WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 27 11 2018 AND 07 02 2019 TO TUNE OF RS. 48.48LACS WAS REMITTED TO HEBEI LUEDING IMPORT AND EXPORT CO LTD COMPANY DEALS IN PPGI, GALVANISED STEEL COIL, ROLL FORMING MACHINE, ROOFING SHEETS, WELDING ELECTRODE HTTPS //WWW.MACHINERYOFFERS.COM/COMPANY/LEADINGROOFING/CONTACT.HTML AND SHOUGUANG DASEN WOOD CO LTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED ASHS 44123190 PLYWOOD, GALVANIZED CORRUGATED SHEET, HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 19 08 2019 AND 09 06 2020 TO THE TUNE OF EUR 85707. IN TOTAL 508 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF 2.18CRORES WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS . CLIENT IS DEALING IN MERCHANT TRADE OF DIVERSIFIDE PRODUCTS OILS, GA SHEETS, WOOD ETC. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.THAKKAR EXIM,PROPRIETORSHIP FIRM, HAVING ADDRESS THAKKAR HOUSE,RAJDEV STREET, SANGANVA CHOWK, RAJKOT, GUJARAT, INDIA, 360001, CONTACT NUMBER 919825287634 AND EMAIL ID INFO THAKKAREXIM.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 084805000210 WITH ICICI BANK LTD, RAJKOT YAGNIK ROAD, SHOP NO. 3 4, CONVENTION CENTER, NR HARIBHAI HALL,OFF YAGNIK, ROAD, RAJKOT, GUJARAT, 360002, SINCE 09102010. DATE OF INCORPORATION IS 27092000. PAN NUMBER OF THE ACCOUNT IS AAXPT8747Q. PAN LINKED ACCOUNTS ARE 084806000003, 084806000038. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 2400002045. TRANSACTION LINKED ACCOUNTS ARE 084805000369, 331105500284. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NITESH HARIDAS THAKKAR PROP AAXPT8747Q ,NITESH HARIDAS THAKKAR AUTHORISED SIGNATORY AAXPT8747Q THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN,MOA,AOA,IE CODE. DATE OF LAST KYC UPDATION 07032019. CUSTOMER IS INTO IMPORT, EXPORT AND MERCHANTING TRADER. HAVING BEGUN WITH ONLY ONE TEXTILE PRODUCT, THE COMPANY HAS NOW WELL DEVELOPED INTO AN STAR EXPORT HOUSE DEALING IN VARIOUS PRODUCTS LIKE AGRICULTURAL COMMODITIES SUCH AS RICE OF DIFFERENT VARIETIES, WHEAT FLOUR, BAKERY FLOUR, BISCUITS, PLASTIC HOUSEHOLD PRODUCTS, KITCHEN APPLIANCES AND HOME APPLIANCES. FIRM IS HAVING WEBSITE WWW.THAKKAREXIM.COM. ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 28022021 WAS RS.11.44 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS. 2,177.84 LAKHS OF WHICH TRANSFERS RS.134.49 LAKHS MAJORLY FROM NITESH HARIDAS THAKKAR,THAKKER ELECTRICALS,THAKKAR EXIM,CLEARING RS. 40.35 LAKHS, NEFT RS. 40.71 LAKHS MAJORLY FROM THAKKAR EXIM,GOPALSWAMY BASKARAN,E PAO GST REFUNDS, RTGS RS. 314.35 LAKHS MAJORLY FROM THAKKAR EXIM,PMS ENTERPRISES,ARTI INTERNATIONAL, REMITTANCE RS. 1,533.76 LAKHS MAJORLY FROM THAKKAR EXIM,THAKKAR EXIM 1 RAJDEV STREET,SOCIETE RAG, INTERNET FUND TRANSFERS RS. 108.40 LAKHS, MOBILE MONEY TRANSFERS RS. 5.65 LAKHS IN 10 INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 2,175.74 LAKHS OF WHICH CASH WAS RS. 4.72 LAKHS BY BRANCH, TRANSFERS RS. 120.83 LAKHS MAJORLY TO THAKKAR EXIM,BHAVNA MAHESH PARMAR,PARMAR RAJDEEP MAHESH, CLEARING RS. 844.39 LAKHS, RTGS RS. 1,180.72 LAKHS MAJORLY TO VIRAJ IMPEX, REMITTANCE RS. 15.79 LAKHS MAJORLY TO HEBEI TOMATO INDUSTRY CO LTD. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 1,707.94 LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. 1,700.66 LAKHS OF WHICH CASH WAS RS. 10.66 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 2,475.81 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 2,165.02 LAKHS. TOTAL DEBIT SIDE RS. 2,480.97 LAKHS OF WHICH CASH WAS RS. 12.65 LAKHS, REMITTANCE RS. 23.04 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 2,749.60 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 2,104.41 LAKHS . TOTAL DEBIT SIDE RS. 2,747.65 LAKHS OF WHICH CASH WAS RS. 11.66 LAKHS, REMITTANCE RS. 9.54 LAKHS.THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER 084805000210 IN THE NAME OF THAKKAR EXIM, REPORT SERIAL NUMBER 300446. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS, HAS BEEN NOTICED IN THE ACCOUNT FOR 508 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF 2.18CRORES. FOR EXAMPLE, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON 14 03 2017 TO THE TUNE OF USD9729 TO OIL TEC FOR OILS AND DETERGENTS MANUFACTURER OF SOAPS ADN OILS HTTPS //WWW.GULFOOD.COM/EXHIBITORS/OIL TEC FOR OILS AND DETERGENTS IN EGYPT WITH PURPOSE",{"entities":[(271,289,"FOREIGN ORG"),(562,599,"FOREIGN ORG"),(775,801,"FOREIGN ORG"),(3964,3990,"FOREIGN ORG")]}),
            ("ENTITY DEALS IN AUTO COMPONENTS MANUFACTURE WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 87088000 UNIVERSAL JOINT. HOWEVER RESPECTIVE INWARDS REMITTANCE TOWARDS EXPORTS RECEIVED USD 12016 RECEIVED BETWEEN 31 12 2019 AND 13 05 2020 FROM MOTOXPARTS DISTRIBUCIONES SRL IN ARGENTINA DETAILS OF WHOM IS NOT AVAILABLE IN PUBLIC DOMAIN. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON 14 11 2019 USD 7850 REMITTED IN PARTS TO HANGZHOU VALE TRADING CO LTD HANGZHOU VALE TRADING CO., LTD. SPECIALIZED TRADING COMPANY OF MOTORCYCLE PARTS, CAR PARTS, HARDWARE, MECHANIC PRODUCT, ELECTRONICS HTTPS //WWW.EWORLDTRADE.COM/C/VALETRADING/ IN HONGKONG WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 87141090 MOTERCYCLE PARTS. THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED USD 12016 RECEIVED BETWEEN 31 12 2019 AND 13 05 2020 FROM MOTOXPARTS DISTRIBUCIONES SRL IN ARGENTINA. IN TOTAL 30 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 3.51LACS. WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.SEVEN IMPEX,PARTNERSHIP FIRM,HAVING ADDRESS SCO 48 FIRST FLOOR,ROYALE ESTATE, ZIRAKPUR, PUNJAB, INDIA, 140603, CONTACT NUMBER 918283841575 AND EMAIL ID FINANCE SEVENIMPEX.ORG IS MAINTAINING A CURRENT ACCOUNT NUMBER 632205500590 WITH ICICI BANK LTD, CHANDIGARH SECTOR 35C,SCO 485486, SECTOR 35C, CHANDIGARH, 160022, SINCE 31122015. DATE OF INCORPORATION IS 14052014. PAN NUMBER OF THE ACCOUNT IS ACOFS4226L. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 2214001603. TRANSACTION LINKED ACCOUNTS ARE 632201541211, 632201540621. AUTHORISED SIGNATORY, PARTNERS AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ANGAD KALANTA, SARINA BHAGTA. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, PARTNERSHIP DEED, IEC. DATE OF LAST KYC UPDATION 13022020.AS PER ENHANCED DUE DILIGENCE CUSTOMER IS INTO IMPORT AND EXPORT OF AUTO ANCILARIES. CLIENT IS INTO MERCHANTING TRADE BUSINESS. CLIENT IS INTO MERCHANTING TRADE BUSINESS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40LAC TO 5CR. ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.0.04 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2021 22 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS. 0.00. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 0.20 LAKHS OF WHICH CASH WAS RS. 0.20 LAKHS BY ATM.FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 135.15 LAKHS OF WHICH CASH WAS RS. 0.04 LAKHS, CLEARING RS. 98.00 MAJORLY FROM AMAZON SELLER, NEFT RS. 11.02 LAKHS MAJORLY FROM KNOVATIC INC,SEVEN IMPEX, REMITTANCE RS. 114.26 LAKHS MAJORLY FROM QI LIANGFEI XIAOSHAN HANGZHOU, INTERNET FUND TRANSFERS RS. 1.00 LAKH, MOBILE MONEY TRANSFERS RS. 8.82 LAKHS IN 7 INSTANCES , UPI TRANSFERS RS. 0.01 LAKHS IN 1 INSTANCE. TOTAL DEBIT SIDE RS. 134.95 LAKHS OF WHICH CASH WAS RS. 18.26 LAKHS BY ATM,BRANCH, TRANSFERS RS. 3.61 LAKHS MAJORLY TO AYUSH CHAUHAN,ADITYA CHHABRA, CLEARING RS. 3.59 LAKHS MAJORLY TO SEVEN IMPEX,ST XAVIER S INTERNATIONA,MR ADITYA CHHABRA, RTGS RS. 24.00 LAKHS MAJORLY TO SEVEN IMPEX, REMITTANCE RS. 0.33 LAKHS, INTERNET FUND TRANSFER RS. 51.94 LAKHS, CARD TRANSACTIONS RS. 0.61 LAKHS, MOBILE MONEY TRANSFERS RS. 11.71 LAKHS IN 45 INSTANCES. FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 201.18 LAKHS OF WHICH CASH WAS RS. 0.55 LAKHS, REMITTANCE RS. 138.07 LAKHS . TOTAL DEBIT SIDE RS. 202.05 LAKHS OF WHICH CASH WAS RS. 22.73 LAKHS, REMITTANCE RS. 0.84 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 261.39 LAKHS OF WHICH CASH WAS RS.0.65 LAKHS, REMITTANCE RS. 119.86 LAKHS.TOTAL DEBIT SIDE RS. 260.47 LAKHS OF WHICH CASH WAS RS. 41.25 LAKHS, REMITTANCE RS. 10.79 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 160.11 LAKHS OF WHICH CASH WAS RS. 2.00 LAKHS, REMITTANCE RS. 106.60 LAKHS. TOTAL DEBIT SIDE RS. 160.97 LAKHS OF WHICH CASH WAS RS. 40.37 LAKHS, REMITTANCE RS. 13.22 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS,ONE LEG HANDLED BY OTHER BANK HAS BEEN NOTICED IN THE ACCOUNT FOR 30 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 3.51LACS. FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN 270 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN 28 08 2019 TO 14 10 2019 USD 12399 REMITTED IN PARTS JINJIANG HANGWAN AUTOMOBILE PARTS CO LTD IN CHINA",{"entities":[(334,362,"FOREIGN ORG"),(726,753,"FOREIGN ORG"),(755,783,"FOREIGN ORG"),(1227,1255,"FOREIGN ORG"),(5168,5207,"FOREIGN ORG")]}),
            ("PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 12079930 NIGER SEED FREE . HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON 12 06 2020 TO TUNE OF USD 29792 WAS RECEIVED FROM SARL RENAUDAT J C LES CHAUMES WHOSES DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN 270 DAYS, WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 19 12 20219 AND14 02 2020 TO TUNE OF USD65283 TO BLACK SEA GROUP LOGISTICS IN RUSSIA DETAILS OF WHICH IS NOT AVAILABE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS 07132000 CHICKPEAS, HS 10089090 RED MILLET. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON 02 03 2020 TO THE TUNE OF USD25276. IN TOTAL 22 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 6.99LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS. CLIENT IS DEALING IN MERCHANT TRADE OF SEEDS, MILLETS, MAIZE ETC.. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.AGRICOM IMPEX,PROPRIETORSHIP FIRM, HAVING ADDRESS F NO 202,AMALTAS APPT,RAJ NAGAR, NAGPUR, MAHARASHTRA, INDIA, 440013, CONTACT NUMBER 09822220151 AND EMAIL ID AGRICOM.IMPEX HOTMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 624205012998 WITH ICICI BANK LTD, NAGPUR RAMDASPETH,AKARSHAN BUSIPLEX, 26, CENTRAL BAZAAR ROAD, RAMDASPETH, NAGPUR, MAHARASHTRA, 440010, SINCE 27122013. DATE OF INCORPORATION IS 04072007. PAN NUMBER OF THE ACCOUNT IS AJRPG6283D. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0307027422. GSTIN AS PER BANK RECORDS IS 27AJRPG6283D1ZY.TRANSACTION LINKED ACCOUNTS ARE 624205501426, 015705014893. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKASH GHADSE PROP AJRPG6283D .THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, GSTIN. DATE OF LAST KYC UPDATION 15102019. AGRICOM IMPEX IS A LEADING WHOLESALE MANUFACTURER AND EXPORTER OF AGRICULTURAL OIL SEEDS, BIRD SEEDS LIKE YELLOW MILLET, SAFFLOWER SEED, NIGER SEED, SORGHUM, SESAME SEEDS, CHICK PEAS AND PEANUTS IN INTERNATIONAL MARKET. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS.40 LAC TO RS.5 CR.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 28022021 WAS RS.23.56 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS. 1,669.33 LAKHS OF WHICH TRANSFERS RS. 115.54 LAKHS MAJORLY FROM AGRICOM IMPEX,SHRI BALAJI INDUSTRIES, CLEARING RS. 0.66 LAKHS MAJORLY FROM FWD CANC GAIN, NEFT RS. 40.22 LAKHS MAJORLY FROM NARAYANA SAI TRADERS,SURGO RESOURCES, RTGS RS. 139.91 LAKHS MAJORLY FROM TALOJA AGRO FOODS PVT LTD,HITASHI AGRI TRADE,AGRICOM IMPEX, REMITTANCE RS. 1,371.07 LAKHS MAJORLY FROM EEFC,EEFC CONVERSION,AGRICOM IMPEX, MOBILE MONEY TRANSFERS RS. 1.89 LAKHS IN 4 INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 1,466.76 LAKHS OF WHICH CASH WAS RS. 4.74 LAKHS BY BRANCH, TRANSFERS RS. 7.69 LAKHS MAJORLY TO BHAIYAJI RAMBHAUJI ROKDE JEWELLERS,ICICI PRUD LIFE INS CO LTD CASH COLLN,AGRICOM IMPEX, CLEARING RS. 5.19 LAKHS MAJORLY TO PANKAJ GULATI AND ASSOCIA,M S E D C L,V P SALES AND SERVICES, NEFT RS. 14.45 LAKHS MAJORLY TO B NARAYANA,THAPARSONS K V SHIPP,GIDDDIAH TRADERS,RTGS RS. 76.42 LAKHS MAJORLY TO SHREE BHAGAWATI AGRO P,NARAYAN SAI TRADER,MILAP TRADING, REMITTANCE RS. 473.44 LAKHS MAJORLY TO GENERAL COMMODITIES LTD,SPRINGHAUS LLC,KAMAL LLC, INTERNET FUND TRANSFER RS. 823.12 LAKHS, MOBILE MONEY TRANSFERS RS. 24.75 LAKHS IN 65 INSTANCES. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 1,739.91 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. 1,551.09 LAKHS. TOTAL DEBIT SIDE RS. 1,849.02 LAKHS OF WHICH CASH WAS RS. 4.40 LAKHS, REMITTANCE RS. 440.77 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 2,198.40 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 2,018.59 LAKHS. TOTAL DEBIT SIDE RS. 2,279.24 LAKHS OF WHICH CASH WAS RS. 3.25 LAKHS, REMITTANCE RS. 48.31 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 2,923.10 LAKHS OF WHICH CASH WAS RS. 0.95 LAKHS, REMITTANCE RS. 2,508.33 LAKHS. TOTAL DEBIT SIDE RS. 2,825.85 LAKHS OF WHICH CASH WAS RS. 14.14 LAKHS, REMITTANCE RS. 211.10 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS, LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR 22 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 6.99LACS. FOR EXAMPLE, VIOLATION OF LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON 15 05 2020 USD 31046 WAS DEBITED TO KAMAL GLOBAL TRADING PTE LTD IN SINGAPORE WHOSE DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN WITH",{"entities":[(562,86,"FOREIGN ORG"),(3742,3763,"FOREIGN ORG"),(3766,3778,"FOREIGN ORG"),(3781,3788,"FOREIGN ORG"),(5090,5117,"FOREIGN ORG")]}),
            ("STANDARD C,CENTRAL BO,GE POWER I, REMITTANCE RS. 73,447.51 LAKHS MAJORLY TO JINDAL STEEL AND POWER LIMITED,ATI FLAT ROLLED PRODUCTS,KANEMATSU CORPORATION, INTERNET FUND TRANSFER RS. 16.98 LAKHS, , AND OTHERS RS. 43,472.45 LAKHS.FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 4,23,158.52 LAKHS OF WHICH CASH WAS RS. 0.03 LAKHS, REMITTANCE RS. 53,998.22 LAKHS . TOTAL DEBIT SIDE RS. 4,14,401.86 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 61,397.07 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 6,28,444.52 LAKHS OF WHICH CASH WAS RS. 0.12 LAKHS, REMITTANCE RS. 69,620.82 LAKHS . TOTAL DEBIT SIDE RS. 6,26,393.15 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 54,327.14 LAKHS .FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 6,23,077.94 LAKHS OF WHICH CASH WAS RS. 0.05 LAKHS, REMITTANCE RS. 65,421.20 LAKHS . TOTAL DEBIT SIDE RS. 6,21,919.98 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. 34,050.85 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 10 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 17.15LACS, EUR 78.71LACS, CNY 36.33LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN 270 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON ON 28 04 2020 USD 68887 WAS REMITTED TO VICENTE BERRIZBEITIA S L CLIENT IS INTO SUPPLY OF BOLTING MATERIALS HTTP //BERRIZBEITIA.COM/EN/ WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 73030090 TUBES AND PIPES. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT ON 21 03 2020 TO TUNE OF USD3.63LACS TO GE HYDRO CHINA CO. LTD ENTITY IS JV WITH YALONG RIVER HYDROPOWER DEVELOPMENT CO. LTD. WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 85030010 HYDRO SET. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE. IN TOTAL 10 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 17.15LACS, EUR 78.71LACS, CNY 36.33LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS.CLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT,STR IS BEING FILED.GE POWER INDIA LIMITED,PUBLIC LTD., HAVING ADDRESS T 5 T 6 PLOT 1 TO 14,JAYPEE WISHTOWN,, SECTOR 128,AXIS HOUSE, NOIDA, UTTAR PRADESH, INDIA, 201304, CONTACT NUMBER 919810292674 AND EMAIL ID CHANDNI.SISODIA POWER.ALSTOM.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000705001143 WITH ICICI BANK LTD, NEW DELHI CONNAUGHT PLACE,9A, PHELPS BUILDING, CONNAUGHT PLACE,NEW DELHI, DELHI, 110001, SINCE 01121999. DATE OF INCORPORATION IS 02091992. PAN NUMBER OF THE ACCOUNT IS AABCA8679F. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0399054081. TRANSACTION LINKED ACCOUNTS ARE 140205002149, 179405000165. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE P JAYALAKSHMI AUTHORISED SIGNATORY AAEPP5525B ,AMARESH SINGH AUTHORISED SIGNATORY AICPS9581L ,RAJESH AGGARWAL AUTHORISED SIGNATORY AGIPA1553C ,HIREN RAJNIKANT VYAS AUTHORISED SIGNATORY ABTPV1559P ,NAVEEN SHANKARAN AUTHORISED SIGNATORY AIOPN5430M . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN AABCA8679F,MOA,AOA. DATE OF LAST KYC UPDATION 16082019. GE POWER INDIA LIMITED GPIL , FORMERLY KNOWN AS ALSTOM INDIA LIMITED, IS ENGAGED IN ENGINEERING, PROCUREMENT, MANUFACTURING, CONSTRUCTION, AND SERVICING, OF POWER PLANTS AND POWER EQUIPMENT IN BOTH THERMAL AND HYDRO SEGMENTS. THE COMPANY DESIGNS MANUFACTURE, AND DELIVERS TURNKEY POWER PLANTS, POWER GENERATION EQUIPMENT, AND AIR QUALITY CONTROL SYSTEMS AND SERVICES AND ALSO PROVIDES RENEWABLE POWER GENERATION SOLUTIONS FOR INTEGRATED POWER PLANTS COVERING HYDROELECTRICITY, WIND, GEOTHERMAL, BIOMASS, SOLAR, AND WAVE AND TIDAL STREAM ENERGIES. IT ALSO OFFERS INDIVIDUAL COMPONENTS FOR A RANGE OF TURBINES AND GENERATORS, AS WELL AS VARIOUS SERVICES, INCLUDING PLANT MODERNIZATION, MAINTENANCE, AND OPERATIONAL SUPPORT AND PROVIDES PRODUCTS AND SYSTEMS FOR ELECTRICAL UTILITIES AND INDUSTRIES.ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.270.29 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2021 22 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS. 3,559.20 LAKHS OF WHICH RS. 0 BY CASH, NEFT RS. 70.28 LAKHS MAJORLY FROM LALITPUR POWER GENERATION CO,TORRENT POWER LIMITED, RTGS RS. 3,488.92 LAKHS MAJORLY FROM BHARAT HEAVY ELECTRICAL. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 4,509.50 LAKHS OF WHICH RS. 0 BY CASH, RTGS RS. 4,500.00 LAKHS MAJORLY TO GE POWER I, REMITTANCE RS. 0.03 LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 2,64,705.85 LAKHS OF WHICH CASH WAS RS. 21.08 LAKHS MAJORLY IN GULBARGA BRANCH, TRANSFERS RS. 27,222.16 LAKHS MAJORLY FROM GE POWER INDIA LTD TELENGANA FGD PROJECT ESCROW ACCOUNT,GE POWER INDIA LTD IGSTPP JHAJJAR FGD PROJECT ESCROW ACCOUNT,GE POWER INDIA LTD SIPAT FGD PROJECT ESCROW ACCOUNT, CLEARING RS. 69,725.91 LAKHS MAJORLY FROM LT MHPS BOILERS PRIVATE LIMIT,NTPC LIMITED,VEDANTA LIMITED, NEFT RS. 27,348.10 LAKHS MAJORLY FROM GENERAL ELECTRIC COMPANY,NHPC LIMITED,MEJA URJA NIGAM PVT LTD, RTGS RS. 76,463.02 LAKHS MAJORLY FROM M/S DOOSAN POWER SYSTE,BHARAT HEAVY ELECTRICA,GE POWER INDIA LIMITED, REMITTANCE RS. 60,143.73 LAKHS MAJORLY FROM ARAVALI POWER COMPANY PRIVATE LTD,GENERAL ELECTRIC SWITZERLAN,NTPC LIMITED, INTERNET FUND TRANSFERS RS. 41.43 LAKHS, MOBILE MONEY TRANSFERS RS. 3.28 LAKHS IN 6 INSTANCES , UPI TRANSFERS RS. 0.20 LAKHS IN 1 INSTANCE AND OTHERS RS. 3,736.94 LAKHS. TOTAL DEBIT SIDE RS. 2,80,786.14 LAKHS OF WHICH TRANSFERS RS. 6,207.59 LAKHS MAJORLY TO GE POWER INDIA LTD UNPAID DIVIDEND ACCOUNT 2019 20,GE POWER INDIA LIMITED,GE POWER SYSTEMS INDIA PVT LTD, CLEARING RS. 973.55 LAKHS, NEFT RS. 5,173.24 LAKHS MAJORLY TO GE POWER INDIA LIMIT,CENTRAL EXCISE SERVI,WEST BENGAL STATE EL, RTGS RS. 1,51,494.81 LAKHS MAJORLY",{"entities":[(1355,1378,"FOREIGN ORG"),(1856,1877,"FOREIGN ORG"),(1897,1938,"FOREIGN ORG"),(5896,5921,"FOREIGN ORG")]}),
            ("ALSO ON 17 10 2020 RS. 52.12 LACS GOT CREDITED BY INWARD REMITTANCE FROM WORLD FIRST ASIA LTD, HONG KONG WITH INFORMATION AS IMPORT OF GARMENTS AND ON 19 10 2020 RS. 39 LACS GOT DEBITED BY CMS CLEARING WITH REMARK AS SDI S.D INTERNATIONAL . IN OTHER EXAMPLE, ON 04 02 2021 RS. 25 LACS WAS CREDITED THROUGH RTGS FROM SD INTERNATIONAL ACC 259830542168, IFSC INDB0000006 AND ON THE SAME DAY RS. 24.12 LACS ARE IMMEDIATELY DEPLOYED THROUGH CMS CLEARING WITH REMARK AS SDI S.D INTERNATIONAL . SIMILAR SUCH TRANSACTIONS ARE OBSERVED THROUGHOUT THE ACCOUNT FOR WHICH NO RATIONAL FOUND. TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS. 4,676.35 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF LESS THAN RS. 40LACS. IN VIEW OF INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS, LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF HIGH VALUE FUNDS AND HIGH TURNOVER, STR IS BEING FILED.S.D. INTERNATIONAL, PROPRIETORSHIP FIRM, HAVING ADDRESS 2ND FLOOR FLR, 22/1B, NIMTOLA GHAT, STREET NEAR JORABAGAN MORE, KOLKATA, WEST BENGAL, INDIA, 700006, CONTACT NUMBER 919830542167 AND EMAIL ID BANKS SDWORLD.IN IS MAINTAINING A CURRENT ACCOUNT NUMBER 627905502328 WITH ICICI BANK LTD, KOLKATA BURRA BAZAR, ICICI BANK LTD, P16, KALAKAR STREET, BURRABAZAR, KOLKATA, WEST BENGAL, 700007, SINCE 20082018. DATE OF INCORPORATION IS 01082003. RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT. PAN NUMBER OF THE ACCOUNT IS AFYPD0627H. OTHER RELATIONSHIP OF CUSTOMER ARE 627705046590, 627705051746, 627905016291, 627905501161, 627905501162, 627905502320, 627906000035, 627906000036, 627913002540, 627913002541, 627913002542, 627913002573, 627913002574, 627913002577, 627966000005, 627966000006. TRANSACTION LINKED ACCOUNTS ARE 627905501161 AND 627605256946. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE SANJOY DEY PROPRIETOR AFYPD0627H . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN AFYPD0627H, AADHAR. DATE OF LAST KYC UPDATION 20012021. KYC UPDATION IS NOT DUE. ENTITY IS INTO BUSINESS OF KIDS GARMENTS AND WOMEN SUIT PCS SET TRADERS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS. 40LACS. ALERT WAS TRIGGERED FOR INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS. BALANCE IN THE ACCOUNT AT 28022021 WAS RS.12.82 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS. 4,676.35 LAKHS OF WHICH TRANSFERS RS. 1,428.00 LAKHS MAJORLY FROM S D INTERNATIONAL, CLEARING RS. 172.45 LAKHS MAJORLY FROM S D, S D TRADE IMPEX PRIVATE LIMITE, CMS1651413254 1, RTGS RS. 1,460.00 LAKHS MAJORLY FROM S D INTERNATIONAL, SDINTERNATIONAL, REMITTANCE RS. 1,534.90 LAKHS MAJORLY FROM ADVANTAGE GENERAL TRADING LIMITED, COSMIC EDGE GENERAL TRADING LIMITED, LIGHTWAYS GENERAL TRADING LLC. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 4,823.87 LAKHS OF WHICH CASH WAS RS. 7.50 LAKHS BY BRANCH, TRANSFERS RS. 164.47 LAKHS MAJORLY TO HETAL SYNTEX, AMIT ROADLINES, S D INTERNATIONAL, CLEARING RS. 2,243.30 LAKHS MAJORL TO SDI RTGS 08 06 2020, SDI RTGS 06 08 2020, SDI TRF 01 10 2020, RTGS RS. 2,045.93 LAKHS MAJORLY TO NITFUL COMMERCIAL PVT, LAXMIWAN SUPPLIERS PVT, SURAKSHIT HOUSING ADVI. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS WERE FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 8,294.73 LAKHS OF WHICH CASH WAS RS. 2.00 LAKHS, TRANSFERS RS. 1,394.21 LAKHS, RTGS RS. 2,793.50 LAKHS, REMITTANCE RS. 3,432.20 LAKHS. TOTAL DEBIT SIDE RS. 8,354.36 LAKHS OF WHICH CASH WAS RS. 17.50 LAKHS, CLEARING RS. 4,536.09 LAKHS, RTGS RS. 3,578.73 LAKHS. FY 2018 19 FROM 20082018 TO 31032019 TOTAL CREDIT SIDE RS. 4,599.81 LAKHS OF WHICH RS. 0 BY CASH, REMITTANCE RS. 2,162.45 LAKHS. TOTAL DEBIT SIDE RS. 4,379.85 LAKHS OF WHICH CASH WAS RS. 9.50 LAKHS. AS PER THE TRANSACTION PATTERN, THERE ARE FREQUENT HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS. FOR EXAMPLE, ON 19 02 2021 RS. 30 LACS WAS CREDITED THROUGH RTGS FROM S D INTERNATIONAL ACCOUNT NO 11030200072126, BANK NAME FEDERAL BANK, IFSC CODE ICIC0006279 AND ON 20 02 2021 RS. 20 LACS WAS DEPLOYED THROUGH RTGS TOWARDS S D INTERNATIONAL ACC 259830542168, IFSC INDB0000526 AND THE REST OF THE FUNDS WERE DEPLOYED THROUGH CMS WITH REMARK AS SDI S.D INTERNATIONAL",{"entities":[(73,91,"FOREIGN ORG"),(2877,2908,"FOREIGN ORG"),(2912,2945,"FOREIGN ORG"),(2949,2976,"FOREIGN ORG")]}),
            ("BOSTON MEGA TEXTILE LLC IN KARAKALPAKSTAN.HTML WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 52054210 COTTON YARN. HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 20 09 2019 AND 20 01 2020 TO THE TUNE OF USD1.89 LACS FROM CHENG CHIH TRANSCEND ENTERPRISE CORPORATION CLIENT DEAAL SIN TRADING OF CONTTON YARNS HTTPS //PANJIVA.COM/CHENG CHIH TRANSCEND ENTERPRISE/46747920 . IN TOTAL 28 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD20.62 LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS . CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY. IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILED.SQUARE CORPORATION, PROPRIETORSHIP FIRM, HAVING ADDRESS 326 C BRS NAGAR,LUDHIANA,PUNJAB, LUDHIANA, PUNJAB, INDIA, 141002, CONTACT NUMBER 919815045100 AND EMAIL ID ACCOUNTS SQUARE CORP.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 001706000136 WITH ICICI BANK LTD, LUDHIANA FEROZE GANDHI MARKET,NEHRU SIDHANT KENDER TRUST BUILDING, FEROZ GANDHI MARKET, LUDHIANA, PUNJAB, 141001, SINCE 31082015. DATE OF INCORPORATION IS 01042001. PAN NUMBER OF THE ACCOUNT IS AAOPV1881G. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 3001010495. GSTIN AS PER BANK RECORDS IS 03AAOPV1881G1ZF. TRANSACTION LINKED ACCOUNTS ARE 001706000163. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VIVEK VERMA PROP , RAHUL TALWAR AUTHORISED SIGNATORY ADBPT6925B , VANDANA VERMA AUTHORISED SIGNATORY AADPV2010M THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE . DATE OF LAST KYC UPDATION 24122019. CUSTOMER IS INTO YARN BUSINESS. HTTP //WWW.SQUARE CORP.COM/. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS EQUAL TO 100 CR. ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONS MTT DEFAULTS BY CUSTOMER. BALANCE IN THE ACCOUNT AT 04042021 WAS RS.0.00 DEBIT. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS. 10.11 LAKHS OF WHICH TRANSFERS RS. 0.06 LAKHS MAJORLY FROM PROMINENT EXPORTS INTERNATIONAL, REMITTANCE RS. 10.06 LAKHS MAJORLY FROM THERMAX YARN DYED FABRICS LIMITED.,EVINCE TEXTILES LTD,MEGHNA KNIT COMPOSITE LIMITED. TOTAL DEBIT SIDE RS. 10.11 LAKHS OF WHICH REMITTANCE RS. 10.11 LAKHS MAJORLY TO WEIQIAO TEXTILE COMPANY LIMITED,WEIHAI WEIQIAO INDUSTRIAL PARK CO,MIF INTERNATIONAL LIMITED.FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 79.96 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 79.22 LAKHS.TOTAL DEBIT SIDE RS. 82.31 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 82.31 LAKHS. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 133.92 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 130.77 LAKHS . TOTAL DEBIT SIDE RS. 132.32 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 131.92 LAKHS. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 105.40 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS. 104.27 LAKHS. TOTAL DEBIT SIDE RS. 107.56 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS. 101.24 LAKHS. VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT I.E MTT CYCLE NOT COMPLETED IN 270 DAYS, LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR 28 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF US20.62LACS. FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN 24 09 2019 TO 17 01 2020 USD 2.19 LACS REMITTED TO IRS UNIVERSAL PTE LTD IRS UNIVERSAL PTE LTD. ISIN INTERNATIONAL PTE LTD WAS FOUNDED IN 1991. THE COMPANYS LINE OF BUSINESS INCLUDES THE WHOLESALE DISTRIBUTION OF NON DURABLE GOODS. HTTPS //WWW.BLOOMBERG.COM/PROFILE/COMPANY/6077450Z SP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING / PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 52051210 YARN. HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT IS DUE. IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 28 10 2019 TO 05 12 2019 TO TUNE OF USD1.49LACS TO BOSTON MEGA TEKSTIL LLC ENTITY IS INTO MANUFACTURE OF TEXTILE",{"entities":[(0,22,"FOREIGN ORG"),(2386, 2418,"FOREIGN ORG"),(2421,2438,"FOREIGN ORG"),(2441,2468,"FOREIGN ORG"),(2591,2615,"FOREIGN ORG"),(2552,2581,"FOREIGN ORG"),(2618,2642,"FOREIGN ORG"),(3683,3703,"FOREIGN ORG"),(3705,3724,"FOREIGN ORG"),(3728,3753,"FOREIGN ORG"),(4330,4352,"FOREIGN ORG")]}),
            ("FLYON FILM PRODUCTION, A PROPRIETORSHIP FIRM, HAVING ADDRESS 707/89, HEMKUNT CHAMBER, NEHRU PLACE, SOUTH DELHI, NEW DELHI, INDIA, 110019, CONTACT NUMBER 917303930724 AND EMAIL ID AKANKSHASINGH9014 GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000705051096 WITH ICICI BANK LTD, NEW DELHI CONNAUGHT PLACE, ICICI BANK LTD, 9A, PHELPS BUILDING, CONNAUGHT PLACE, DELHI, 110001, SINCE 20012021. DATE OF INCORPORATION IS 17082020. PAN NUMBER OF THE ACCOUNT IS CGHPS8098F. GSTIN AS PER BANK RECORDS IS 07CGHPS8098F2ZP. OTHER RELATIONSHIP OF CUSTOMER IS 163905001067. TRANSACTION LINKED ACCOUNTS ARE 643805053274, 119405001855. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKANKSHA SINGH PROPRIETOR CGHPS8098F THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON 06 03 2021. KYC DOCUMENTS OBTAINED WERE PAN CGHPS8098F, GSTIN, AADHAR. CLIENT DEALS MEDIA AND FILMES SERVICES. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. 40 LAC TO RS 5 CR. A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS. BALANCE IN THE ACCOUNT AT 06032021 WAS NIL. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2020 21 FROM 20012021 TO 06032021 TOTAL CREDIT SIDE RS. 1,961.58 LAKHS OF WHICH CLEARING RS. 1.00 LAKH MAJORLY FROM FLYON FILM PRODUCTION, NEFT RS. 49.76 LAKHS MAJORLY FROM DAT SOLUTIONS, APS INTERNATIONAL, IMAGINARY FILMS PRIVATE LIMITED, RTGS RS. 1,906.32 LAKHS MAJORLY FROM GWAY TO WORLD, OCN INTERNATIONAL, CARIO SERVICES, MOBILE MONEY TRANSFERS RS. 4.50 LAKHS IN 3 INSTANCES, TOTAL DEBIT SIDE RS. 1,961.58 LAKHS OF WHICH CLEARING RS. 1.51 LAKHS MAJORLY TO FLYCON FILM PRODUCTION, RTGS RS. 843.10 LAKHS MAJORLY TO IMAGINARY FILMS PRIVATE LIMITED, REMITTANCE RS. 133.53 LAKHS MAJORLY TO THE PRODUCTION SERVICES FZCO, GATE STUDIOS PTE LTD, INTERNET FUND TRANSFER RS. 982.41 LAKHS TO ONE FRAME PRODUCTIONS, MOBILE MONEY TRANSFERS RS. 1.00 LAKH IN 1 INSTANCE, AND OTHERS RS. 0.02 LAKHS. TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS. ROUTING OF FUNDS OBSERVED. SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD. FOR EXAMPLE, ON 24 02 2021 RS. 61.14 LACS CREDITED BY RTGS FROM IMAGINARY FILMS PRIVATE LIMITED 920020074440297, IFSC UTIB0001625, AXIS BANK LTD AND SAME WAS DEBITED THROUGH REMITTANCE TO GATE STUDIOS PTE LTD IN SINGAPORE FOR PERSONAL, CULTURAL AND RECREATIONAL SERVICES. IN OTHER EXAMPLE, BETWEEN 01 03 2021 TO 02 03 2021 RS. 64.32 LACS CREDITED BY RTGS FROM IMAGINARY FILMS PRIVATE LIMITED 920020074440297, IFSC UTIB0001625, AXIS BANK LTD AND SAME WAS DEBITED THROUGH NEFT TO IMAGINARY FILMS PRIVATE LIMITED 920020074440297, IFSC UTIB0001625, AXIS BANK LTD . SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT. TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS. 1,961.58 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS. 40LAC TO 5CR. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS, HIGH TURNOVER AND ACCOUNT BEING CLOSED IN SHORT DURATION, STR IS BEING FILED.",{"entities":[(1871,1897,"FOREIGN ORG"),(1901,1919,"FOREIGN ORG"),(2399,2418,"FOREIGN ORG")]}),
            ("REDNODE MEDIA LIMITED, A PUBLIC LTD COMPANY, HAVING ADDRESS GALA NNO 12 AND 13 SHREE, 3RD FLOOR, KAMDHENU, EST CORSS LINK RD, MALAD WEST, NR TANGENT FURNITURE SHOWROOM, MUMBAI, MAHARASHTRA, INDIA, 400064, CONTACT NUMBER 919619537557 AND EMAIL ID REDNODEMEDIALIMITED GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 015805010762 WITH ICICI BANK LTD, SADGURU DARSHAN, ROAD NO. 3, LIBERTY GARDEN, MALAD W, MUMBAI, MAHARASHTRA, 400064, SINCE 29102020. DATE OF INCORPORATION IS 12102017. PAN OF THE ACCOUNT IS AAICR5730K. GSTIN AS PER BANK RECORDS IS 27AAICR5730K1Z1. THERE ARE NO LINKED ACCOUNTS. TRANSACTION LINKED ACCOUNTS ARE 015805010832,015805010356. BENEFICIAL OWNERS/ DIRECTORS/ AUTHORISED SIGNATORIES OF THE REPORTED ACCOUNT ARE AYESHA ALI MEHBOOB SHAIKH WITH PAN NO KYSPS9802L, PATEL PINKI AWADHESH WITH PAN NO CFWPP4169L, SANDEEP RAJENDRA THAKUR WITH PAN NO ATCPT2515N, SUBHASH SITARAM MUDRAS WITH PAN NO CAKPM5598R. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION. THE FIRM IS INVOLVED IN MAKING SOFTWARES AND BUILDING PROJECTS AS PER THE CUSTOMER NEEDS AND ALSO INVOLVED IN EVENTS AND ENTERTAINMENT. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40 LACS. ALERT WAS TRIGGERED FOR HIGH FOREX TRANSACTIONS. BALANCE IN THE ACCOUNT AT 16032021 WAS RS. 0.45 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 29102020 TILL 16032021 SHOWS TOTAL CREDIT RS. 2,276.19 LAKHS OF WHICH CASH WAS RS. 7.80 LAKHS MAJORLY IN MALAD BRANCH, TRANSFERS RS. 250.00 LAKHS MAJORLY FROM DARWIN PLATFORM REFINERIES LIMITED, DARWIN PLATFORM INFRASTRUCTURE LIMITED, DARWIN PLATFORM CAR CAB TRADE LIMITED, NEFT RS. 140.69 LAKHS MAJORLY FROM GENESIS MULTI TRADE, MODERN INTERNATIONAL, INDIA TRADERS, RTGS RS. 1,661.83 LAKHS MAJORLY FROM RAZORPAY SOFTWARE PRIVATE LTD, REDNODE MEDIA LIMITED, INTERNET FUND TRANSFERS RS. 140.37 LAKHS FROM GOPAL TRADING COMPANY ETC, MOBILE MONEY TRANSFERS MMT RS. 74.49 LAKHS IN 57 INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 2,275.74 LAKHS OF WHICH CLEARING RS. 7.00 LAKHS MAJORLY TO REDNODE MEDIA LIMITED, RTGS RS. 918.62 LAKHS TO REDNODE MEDIA LIMITED, REMITTANCE RS. 1,077.33 LAKHS MAJORLY TO METRO SOLUTIONS PTE LTD, SAILLI VIROHA PTE LTD, ONE STUDIOZ PTE LTD, INTERNET FUND TRANSFER RS. 171.22 LAKHS TO SANDEEP RAJENDRA THAKUR ETC, MOBILE MONEY TRANSFERS RS. 100.48 LAKHS IN 115 INSTANCES. CREDIT ARE FROM OWN ACCOUNT, STEEL FIRMS ETC AND DEPLOYED TOWARDS REMITTANCE MAJORLY TO METRO SOLUTIONS PTE LTD TOWARDS ENTERTAINMENT WHICH SEEMS SUSPICIOUS. FOR EXAMPLE, ON 03 02 2021 RS 57.80 LACS WERE CREDITED THROUGH RTGS FROM EKTA ENTERPRISES 10200005096622, BDBL0001799 , INTERNET FUND TRANSFER FROM GOPAL TRADING COMPANY 057105002242 AND ON THE NEXT DAY RS 55.94 LACS WERE DEBITED THROUGH REMITTANCE TO METRO SOLUTIONS PTE LTD, SINGAPORE TOWARDS ENTERTAINMENT SERVICES. ALSO ON 10 02 2021 RS 49.94 LACS WERE CREDITED THROUGH RTGS FROM KUMAR STEEL IND 50200050722028, HDFC0000240 , SHIV SHAKTI TRADE 39685975810, SBIN0016237 , GURUJI INDUSTRIES 918020107115146, UTIB0000533 , OVERSEAS TRADERS 02340210003800, UCBA0000234 AND ON THE SAME DAY RS 35.66 LACS WERE DEBITED THROUGH REMITTANCE TO METRO SOLUTION PTE LTD, SINGAPORE TOWARDS ENTERTAINMENT SERVICES. SIMILAR PATTERN WERE SEEN IN THE ACCOUNT. IN VIEW OF HIGH VALUE OUTWARD REMITTANCE SEEN IN THE ACCOUNT SEEMS LIKE ROUTING OF FUNDS, STR IS BEING FILED.",{"entities":[(2376,2397,"FOREIGN ORG"),(2401,2420,"FOREIGN ORG"),(2424,2441,"FOREIGN ORG"),(2985,3006,"FOREIGN ORG"),(3371,3391,"FOREIGN ORG")]}),
            ("IMAGINARY FILMS PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS 402, EMBASSY CHAMBERS, 3RD ROAD, KHAR WEST, MUMBAI, MAHARASHTRA, INDIA, 400052, CONTACT NUMBER 917039548313 AND EMAIL ID IMAGINARYFILMS4 GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 119405001855 WITH ICICI BANK LTD, PLOT NO. 206 221, 27TH RD., CRYSTAL BUILDING, WATERFIELD ROAD, BANDRA W, MUMBAI, MAHARASHTRA, 400050, SINCE 29012020. DATE OF INCORPORATION IS 22012020. PAN NUMBER OF THE ACCOUNT IS AAFCI5545Q. THERE ARE NO PAN LINKED ACCOUNTS. TRANSACTION LINKED ACCOUNTS ARE 135205000593, 151105000348. BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH KUMAR SINGH WITH PAN NO GSRPS7969M, SAPNA KUMARI WITH PAN NO DGSPK3718Q. THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS TOTAL FREEZED ON 17 02 2021 DUE TO SUSPICIOUS NATURE OF TRANSACTIONS IN THE ACCOUNT 0038NSMT0040321, AS PER BANK RECORDS. KYC DOCUMENTS OBTAINED WERE PAN AAFCI5545Q, MOA, AOA, COI U92490MH2020PTC336424. AS PER ENHANCE DUE DILIGENCE COMPANY IS IN THE BUSINESS OF THE CINEMATOGRAPH TRADE AND INDUSTRY IN ALL THEIR BRANCHES AND ACTIVITIES AND PARTICULARLY THE BUSINESS OF MANUFACTURE, PRODUCTION, DISTRIBUTION, EXPLOITATION, EXHIBITION, IMPORT AND EXPORT OF ALL KINDS OF CINE FILMS, TALKIE FILMS, VIDEO FILMS, TELEFILMS, DOCUMENTARY FILMS, ADVERTISING FILMS, ETC. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. 5 CRS TO 25 CRS. A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS. BALANCE IN THE ACCOUNT AT 16032021 WAS RS.0.87 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01042020 TILL 16032021 SHOWS TOTAL CREDIT RS. 6,677.92 LAKHS OF WHICH TRANSFERS RS. 131.47 LAKHS MAJORLY FROM IMAGINARY FILMS PRIVATE LIMITED, KESARI TOURS PVT LTD FOREX, CLEARING RS. 2.64 LAKHS MAJORLY FROM IMAGINARY FILMS PVT LTD, RTGS RS. 326.30 LAKHS MAJORLY FROM IMAGINARY FILMS PVT LTD, GILPIN TOURS AND TRAVEL, TRANSCORP INTERNATIONAL, INTERNET FUND TRANSFERS RS. 6,214.98 LAKHS FROM SAWARTHIYA FILM PRODUCTIONS, FLYON FILM PRODUCTION, ETC, MOBILE MONEY TRANSFERS RS. 1.00, AND OTHERS RS. 2.52 LAKHS. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 6,681.80 LAKHS OF WHICH CASH WAS RS. 4.60 LAKHS BY BRANCH, CLEARING RS. 18.13 LAKHS MAJORLY TO IMAGINARY FILMS PVT LTD, HDFC BANK LTD, YES BANK LTD, RTGS RS. 4,160.26 LAKHS MAJORLY TO GILPIN TOURS N TRAVEL, IMAGINARY FILMS PVT.LTD, WORLD ONE INDIA FOREX, REMITTANCE RS. 350.19 LAKHS MAJORLY TO ROSEBERRY INTERNATIONAL, ROSEBERRY INTERNATIONAL SERVICES, LIVING FILMS PTE LTD, INTERNET FUND TRANSFER RS. 2,116.50 LAKHS TO SAWARTHIYA FILM PRODUCTIONS, MULTIMONEY FOREX LTD, ETC, MOBILE MONEY TRANSFERS RS. 29.26 LAKHS IN 45 INSTANCES AND OTHERS RS. 2.86 LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2019 20 FROM 29012020 TO 31032020 TOTAL CREDIT SIDE RS. 30.00 LAKHS OF WHICH CLEARING RS. 20.00 LAKHS, RTGS RS. 10.00 LAKHS. TOTAL DEBIT SIDE RS. 25.24 LAKHS OF WHICH CASH WAS RS. 0.50 LAKHS, CLEARING RS. 20.00 LAKHS. TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS WHICH SEEMS LIKE ROUTING OF FUNDS. FOR EXAMPLE, BETWEEN 12 02 2021 TO 15 02 2021 RS. 63.20LACS CREDITED BY TRANSFERS FROM SAWARTHIYA FILM PRODUCTIONS 1352050005930 AND DURING SAME DURATION RS.61.56LACS WAS DEBITED BY TRANSFERS TO SAWARTHIYA FILM PRODUCTIONS 1352050005930 . IN OTHER EXAMPLE, ON 08 02 2021 RS. 62.36LACS WAS CREDITED BY TRANSFERS FROM FLYON FILM PRODUCTION 000705051096 AND SAME WAS DEBITED BY TRANSFERS TO MULTIMONEY FOREX LTD 151105000348 . SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD. IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS AND ROUTING OF FUNDS, STR IS BEING FILED",{"entities":[(2555,2585,"FOREIGN ORG"),(2589,2607,"FOREIGN ORG"),(2530,2551,"FOREIGN ORG")]}),
            ("DIAMAX, A PROPRIETORSHIP FIRM, HAVING ADDRESS 404/405, DAIMOND APPARTMENT, SAIYEDPURA, NAGORIVAD, SURAT, GUJARAT, INDIA, 395003, CONTACT NUMBER 919825217325 AND EMAIL ID DIAMAX95 GMAIL.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 239705501101 WITH ICICI BANK LTD, SHUBHAM POINT, RUWALA TEKRA, HARIPURA BAGAL, LAL GATE, SURAT, GUJARAT, 95003, SINCE 23102019. DATE OF INCORPORATION IS 14122016. PAN OF THE ACCOUNT IS FOJPS7176R. IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 5216926682. GSTIN AS PER BANK RECORDS IS 24FOJPS7176R1ZQ. TRANSACTION LINKED ACCOUNT IS 005201025697. AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS DHRUMIL DHARNENDRA SHAH PROPRIETOR FOJPS7176R . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN, GSTIN. KYC UPDATION IS NOT DUE. CUSTOMER IS AN IMPORTER OF LAB GROWN I.E. SYNTHETIC DIAMONDS WHICH HE SELL IN LOCAL MARKET. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 25CR TO LESS THAN 100CR. ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT. BALANCE IN THE ACCOUNT AT 28022021 WAS RS.1.43 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS. 1,908.17 LAKHS OF WHICH RTGS RS. 1,894.40 LAKHS MAJORLY FROM DIAMAX, VANSH GEMS, SHREEJI JEWELLERS. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 1,929.85 LAKHS OF WHICH RTGS RS. 328.51 LAKHS MAJORLY TO PARI IMPEX, DIAMAX, REMITTANCE RS. 1,488.13 LAKHS MAJORLY TO MARS VENTURE LIMITED, AARAS DIAMONDS LLC, ALICAT SCIENTIFIC INC. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2019 20 FROM 23102019 TO 31032020 TOTAL CREDIT SIDE RS. 2,543.41 LAKHS OF WHICH RS. 0 BY CASH. TOTAL DEBIT SIDE RS. 2,520.31 LAKHS OF WHICH CASH WAS RS. 5.00 LAKHS. FUNDS ARE FROM DIFFERENT ENTITIES AND ARE IMMEDIATELY DEBITED TOWARDS OUTWARD REMITTANCE. FOR EXAMPLE, ON 21 09 2020 ACCOUNT WAS CREDITED WITH RS. 3.14 CR THROUGH RTGS FROM SHREEJI JEWELLERS 919020021152068, BANK AXIS IFSC UTIB0001705 AND ON SAME DURATION ACCOUNT WAS DEBITED WITH RS. 3.08 CRORE THROUGH REMITTANCE TO MARS VENTURE LIMITED AT HONG KONG FOR PAYMENT AGAINST INVOICE. ALSO ON 04 09 2020 ACCOUNT WAS CREDITED WITH RS. 1.58 CRORE THROUGH RTGS FROM DIAMAX 50200019868089, BANK HDFC IFSC HDFC0000896 , PURVI EXPORT, VANSH GEMS AND ON SAME DURATION ACCOUNT WAS DEBITED WITH RS. 1.82 CRORE THROUGH REMITTANCE TO MARS VENTURE LIMITED AT HONG KONG FOR PAYMENT AGAINST INVOICE. MAJORITY REMITTANCES MADE TO MARS VENTURE LIMITED. OVERSEAS ENTITIES EXISTENCE CHECKED HOWEVER NO FURTHER INFORMATION AVAILABLE ON THEM REGARDING WHAT PRODUCT THEY SELL. NO BUSINESS RELATED PAYMENTS ARE MADE FROM THIS ACCOUNT. NAME BOARD AFFIXED OUTSIDE OFFICE WHICH IS PRINTED ON A4 SIZE PAPER. ALSO THERE ARE CREDITS FROM NYSNA MULTIVENTRES PVT LTD. IT IS ONE SUCH COMPANY WHICH HAS TRANSACTED WITH MANY SUSPICIOUS DIAMOND COMPANIES OF SURAT WHOSE ACCOUNTS WE HAVE CLOSED UNDER BANK INDUCED PROCESS. IN VIEW OF LARGE VALUE FOREX OUTWARD REMITTANCE IN THE ACCOUNT TO A SUSPICIOUS OVERSEAS COUNTER PARTY, HIGH TURNOVER AND NEGATIVE FINDINGS IN TERMS OF TRANSACTIONS WITH A DOMESTIC SUSPICIOUS ENTITY, STR IS BEING FILED.",{"entities":[(1566,1584,"FOREIGN ORG"),(1588,1604,"FOREIGN ORG"),(1608,1627,"FOREIGN ORG"),(2532,2551,"FOREIGN ORG"),(2624,2642,"FOREIGN ORG")]}),
            ("ROUTING OF LARGE VOLUME OF FUNDS WERE OBSERVED IN ACCOUNT LEADING TO HIGH TURNOVER WHICH SEEMS UNUSUAL. IN VIEW OF, LARGE VALUE NON CASH TRANSACTIONS IN CAA ACCOUNTS, STR IS BEING FILED.WANDERLAND REAL ESTATE PRIVATE LIMITED, SMEAG PRIVATE LTD COMPANY , HAVING ADDRESS 6TH FLR TREASURE ISLAND,11 TUKOGANJ MAIN,RD, INDORE, MADHYAPRADESH, INDIA, 452001, CONTACT NUMBER 917389909730 IS MAINTAINING A CURRENT ACCOUNT NUMBER 004105017744 WITH ICICI BANK LTD, INDORE MALAV PARISAR, ICICI BANK LTD, , 4,CHHOTI KHAJRANI, MALAV PARISAR, INDORE, INDORE, MADHYA PRADESH, 452008, SINCE 30122019. DATE OF INCORPORATION IS 24022006. PAN NUMBER OF THE ACCOUNT IS AAACW6397F. OTHER RELATIONSHIP OF CUSTOMER ARE 004105015785, 004105015988, 004105015989, 004105015990, 004105015991, 004105017765. TRANSACTION LINKED ACCOUNTS ARE 004105017765 ,182005000938. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VINAYAK KALANI AUTHORISED SIGNATORY ATGPK8379P ,QURESH YUSUF MATKAWALA AUTHORISED SIGNATORY AGSPM2294A ,YOGESH GOSWAMI AUTHORISED SIGNATORY ACFPG9032K ,NIRMAL KUMAR SHRIVASTAVA DIRECTOR , KAPIL GOPLANI DIRECTOR BSEPG3954N . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PAN AAACW6397F, MOA AOA, BOARD RESOLUTION ETC. KYC UPDATION IS NOT DUE. AS PER ENHANCED DUE DILIGENCE CUSTOMER DEALS IN EXPORT OF PP BAGS. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 25CR TO LESS THAN 100CR. ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CAA ACCOUNTS. BALANCE IN THE ACCOUNT AT 31122020 WAS RS.3.91 LAKHS CREDIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01042020 TILL 31122020 SHOWS TOTAL CREDIT RS. 10,081.82 LAKHS OF WHICH TRANSFERS RS. 1.00 LAKH MAJORLY FROM WANDERLAND REAL ESTATES PVT LTD, CLEARING RS. 240.83 LAKHS MAJORLY FROM NEFT,WANDERLAND REAL ESTATE PRIVATE,CMS1732302074 1, NEFT RS. 1.67 LAKHS MAJORLY FROM TATWA PLASTCO EXTRUSIONS PVT,MAERSK LINE INDIA PRIVATE LI, REMITTANCE RS. 9,796.87 LAKHS MAJORLY FROM FLEXI GLOBAL PTE LTD 7500A BE,PANKTI INTERNATIONAL PTE LTD,DHAMPUR INTERNATIONAL PTE LT, INTERNET FUND TRANSFERS RS. 41.43 LAKHS, , AND OTHERS RS. 0.02 LAKHS . TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 10,309.16 LAKHS OF WHICH , TRANSFERS RS. 25.83 LAKHS MAJORLY TO WANDERLAND REAL ESTATE PRIVATE LIMITED, CLEARING RS. 3,656.98 LAKHS, RTGS RS. 6,306.00 LAKHS MAJORLY TO FLEXITUFF VENTURES IND,FLEXITUFF VENTURES INT,SBIN0008283, REMITTANCE RS. 15.26 LAKHS, MOBILE MONEY TRANSFERS RS. 9.31 LAKHS IN 21 INSTANCES , , AND OTHERS RS. 8.19 LAKHS .FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER , YEAR WISE WERE FY 2019 20 FROM 30122019 TO 31032020 TOTAL CREDIT SIDE RS. 4,637.06 LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. 4,405.80 LAKHS OF WHICH CASH WAS NIL. TRANSACTION PATTERN SHOWS FUNDS WERE CREDITED THROUGH INWARD REMITTANCES AND THOSE FUNDS WERE DEPLOYED TOWARDS OTHER ENTITIES. FOR EXAMPLE, ON 30 12 2020, RS. 55.31LAKHS CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE. LTD. 3 SHENTON WAY SHENTON HOUSE 10 02 SINGAPORE WHERE THE PURPOSE MENTIONED AS PAYMENT OF IMPORT BILL AND ON 30 12 2020, RS. 85LAKHS DEBITED IN PARTS THROUGH RTGS TO FLEXITUFF VENTURES INTERNATIONAL LTD ACCOUNT NO. 65064340814 IFSC SBIN0030340 . ALSO ON 24 12 2020, RS. 20LAKHS CREDITED THROUGH TRANSFER FROM NANOFIL TECHNOLOGIES PRIVATE LIMITED 182005000938 WHERE THOSE FUNDS WERE CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE. LTD. 3 SHENTON WAY SHENTON HOUSE 10 02 SINGAPORE WHERE THE PURPOSE MENTIONED AS EXPORT BILLS REALN COLLN AND ON 24 12 2020, RS. 1.04CRORES DEBITED IN PARTS THROUGH CMS TRANSFER AS PAYMENTS. ALSO ON 05 01 2021, RS. 33.85 LACS WERE CREDITED BY REMITTANCE FROM FLEXI GLOBAL PTE LTD 7500A BE, SINGAPORE AND ON SAME DAY ENTIRE AMOUNT WAS DEBITED IN 2 PARTS BY RTGS TOWARDS FLEXITUFF VENTURES INTERNATIONAL LTD ACCOUNT NO. 65064340814 IFSC SBIN0030340",{"entities":[(2087,2106,"FOREIGN ORG"),(2117,2143,"FOREIGN ORG"),(2146,2172,"FOREIGN ORG"),(3112,3139,"FOREIGN ORG"),(3846,3865,"FOREIGN ORG")]}),
            ("SHIPMENT MADE EXPORT BILLS ARE PENDING FOR THE SUBMISSION OUT OF WHICH 5 BILLS WERE OVERDUE AND 1 IS IN CAUTIONED LISTED FOR MORE THAN TWO YEARS SEEMS SUSPICIOUS. IN VIEW OF BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE, STR IS BEING REPORTED.KGR RIGS AND MINING EQUIPMENT, PARTNERSHIP, HAVING ADDRESS PLOTNO 205/8 205/9, IDA PHASE II, CHERLAPALLY, HCL POST, RANGAREDDI, HYDERABAD, TELANGANA, INDIA, 500051, CONTACT NUMBER 917901344444 AND EMAIL ID GOVINDREDDY K YAHOO.COM IS MAINTAINING A CURRENT ACCOUNT NUMBER 131805500555 WITH ICICI BANK LTD, PLOT NO. 13, H.NO. 1029/13, SUBHASH NAGAR, OPP JUNA NAGAR, KUSHAIGUDA, KAPRA, RANGAREDDY DIST., HYDERABAD, TELANGANA, 500062, SINCE 19 12 2016. DATE OF INCORPORATION IS 13 03 2006. NA. PAN NUMBER OF THE ACCOUNT IS AAHFK5493N. OTHER RELATIONSHIP OF CUSTOMER ARE 131805500990, 131813000917, 131813002050 AND 131855000006. TRANSACTION LINKED ACCOUNTS ARE 131805500146 AND 131805500275. AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE GOVINDAREDDY KATIREDDY PARTNER ADFPK0552K AND KATIREDDY NITHIN REDDY PARTNER DECPK8094F . THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL. THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE. KYC DOCUMENTS OBTAINED WERE PARTNERSHIP DEED AND PAN. DATE OF LAST KYC UPDATION 26 09 2019. KYC UPDATION IS NOT DUE.ENTITY IS A MANUFACTURER AND TRADER OF DRILLING EQUIPMENT SUCH AS DRILLING RIGS, DTH HAMMERS, BUTTON BITS ETC. DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS. 5 CRORES TO LESS THAN 25 CRORES.ALERT WAS TRIGGERED FOR MANUAL STR FOR BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE. BALANCE IN THE ACCOUNT AT 28 02 2021 WAS RS.484.54 LAKHS DEBIT. QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 2020 21 FROM 01 04 2020 TILL 28 02 2021 SHOWS TOTAL CREDIT RS. 4,972.47 LAKHS OF WHICH CASH WAS RS. 6 LAKHS, TRANSFERS RS. 773.30 LAKHS MAJORLY FROM KGR RIGS MINING EQUIPMENT, KGR INDUSTRIES, KATIREDDY GOVINDA REDDY, CLEARING RS. 253.00 LAKHS MAJORLY FROM BIMIDI MAHENDER, LAXMI NARASIMHA RAO, NEFT RS. 590.99 LAKHS MAJORLY FROM COMINT SYSTEMS AND SOLUTIONS, E PAO GST REFUNDS, MASTER DRILLERS, RTGS RS. 1,011.58 LAKHS MAJORLY FROM E PAO GST REFUNDS, BHAVIKA PLAS TEK PRIVA, NANO POLYMERS INDIA PVT LTD, REMITTANCE RS. 1,581.63 LAKHS MAJORLY FROM BAKORI MEGA SERVICES, ROYAL TECHNO INDUSTRIES LIMITED, PROACTIVE ENERGY TRADING , INTERNET FUND TRANSFERS RS. 734.65 LAKHS FROM KGR INDUCTION DRILL PIPES COMPANY 131805500239 , MOBILE MONEY TRANSFERS RS. 14.96 LAKHS IN 11 INSTANCES, UPI TRANSFERS RS. 6.35 LAKHS IN 8 INSTANCES. TOTAL DEBITS DURING THE SAME PERIOD WAS RS. 5,003.55 LAKHS OF WHICH CASH WAS RS. 9.50 LAKHS BY KAPRA BRAN CH, TRANSFERS RS. 1,038.19 LAKHS MAJORLY TO SEKO BEC PRIVATE LIMITED, KGR RIGS MINING EQUIPMENT, G ROC MINING AND CONSTRUCTION PRIVATE LIMITED, CLEARING RS. 116.41 LAKHS MAJORLY TO ELGI EQUIPMENTS LIMITED, TSSPDCL, NEFT RS. 0.47 LAKHS MAJORLY TO OSMAN PIPES AND TUBE, RTGS RS. 2,189.62 LAKHS MAJORLY TO ELGI EQUIPMENTS LTD, ASHOK LEYLAND LIMITED, REMITTANCE RS. 3.51 LAKHS, INTERNET FUND TRANSFER RS. 1,480.34 LAKHS TO SOUTHERN STEELS FORGINGS 665005021395 , MOBILE MONEY TRANSFERS RS. 2.30 LAKHS IN 8 INSTANCES, AND OTHERS RS. 163.22 LAKHS. FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS WERE FY 2019 20 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS. 3,759.72 LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. 4,172.61 LAKHS OF WHICH CASH WAS NIL. FY 2018 19 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS. 2,308.71 LAKHS OF WHICH CASH WAS NIL. TOTAL DEBIT SIDE RS. 2,311.86 LAKHS OF WHICH CASH WAS NIL. FY 2017 18 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS. 1,718.71 LAKHS OF WHICH CASH WAS RS. 22.39 LAKHS.TOTAL DEBIT SIDE RS. 1,768.97 LAKHS OF WHICH CASH WAS RS. 4.50 LAKHS.AS PER THE TRANSACTION PATTERN, FUNDS ARE GETTING CREDITED FROM DOMESTIC AND INTERNATIONAL ENTITIES AND THOSE FUNDS ARE GETTING DEBITED TO CUSTOMER OTHER BANK ACCOUNT, OD ACCOUNT IS WITH US AND VARIOUS SUPPLIERS, VENDORS AND OPERATIONAL EXPENSES ETC SEEMS INLINE. HOWEVER, CUSTOMER IS HAVING 57 ADVANCES RECEIVED, THERE ARE EXPORT BILLS ARE PENDING FOR SUBMISSION OUT OF WHICH 12 BILLS ARE OVERDUE",{"entities":[(2305,2323,"FOREIGN ORG"),(2327,2356,"FOREIGN ORG")]}),
            ("ACCOUNT OPENED ON 21 09 2012, CURRENT STATUS OF THE ACCOUNT IS 17 03 2021.TRANSACTIONS NOTICED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES, CHEQUE DEPOSITS, IMPS CREDITS, CASH DEPOSITS, ETC, CREDITS IN THE ACCOUNT VIA VARIOUS MODES ARE UTILISED VIA RTGS/NEFT TO LOCAL PARTIES IMMEDIATELY. BANK FILED STR ON 02 11 2018 FIU BATCH ID 1701309216. 9.RELATED ACCOUNTS ARE VALIDATED VIA UCIC 52580533 DIRECTOR MR. SUSHIL JAIN IS ALSO PROPRIETOR IN THE FIRMS SHANTI OVERSEAS 201001424530 SUNSHINE ENTERPRISES 259999858584 . DIRECTOR MR.RISHABH JAIN IS ONE OF THE PROPRIETOR/DIRECTOR IN THE BULK CIRCLE LLP 259899920203 , SHALINI TRADEWELL INDIA PVT LTD 259599911666INR,201002328431 GBP,201002328417EUR, 201002328400 USD SHALINI ENTERPRISES 201000675197 . RELATED ACCOUNT DETAILS 1.SHANTI OVERSEAS 201001424530 OPENED ACCOUNT WITH US ON 06 07 2017.CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS 22 05 2020.TOTAL CREDIT TURNOVER AS ON DATE IN THE ACCOUNT IS RS.64.28CRORES,AVAILABLE BALANCE IN THE ACCOUNT IS RS.5,397.39.TRANSACTIONS NOTICED ARE HIGH VALUE CREDITS FROM E PAO GST REFUNDS,DUTY DBK DISBURSAL ACCOUNT RTGS/NEFT FROM LOCAL PARTIES, INWARD REMITTANCES ARE UTILISED IMMEDIATELY. BANK FILED STR ON 17 03 2018 VIA FIU BATCH ID 1803199020. 2.SUNSHINE ENTERPRISES 259999858584 , ACCOUNT OPENED ON 22 06 2016.CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS 20 01 2020.AVAILABLE BALANCE IN THE ACCOUNT IS RS.0.00 CREDIT. BANK FILED STR ON 30 01 2017 VIA FIU BATCH ID 1701309221. 3.BULK CIRCLE LLP 259899920203 , ACCOUNT OPENED ON 16 11 2017,CURRENT STATUS OF THE ACCOUNT CLOSED ON 14 01 2021. 4.SHALINI TRADEWELL INDIA PVT LTD 259599911666INR, 201002328431 GBP, 201002328417EUR, 201002328400USD ,ACCOUNT OPENED ON 29 09 2017,CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS 24 12 2020.AS ON 17 03 2021 TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RS.148.25CRORES AVAILABLE BALANCE IN THE ACCOUNT IS RS.8,516.95 CREDIT.TRANSACTIONS OBSERVED ARE HIGH VALUE CREDITS VIA E PAO GST REFUNDS INWARD REMITTANCES UTILISED TO MAKE IMMEDIATELY RTGS TO LOCAL PARTIES, DESPITE OF VOLUMES CLIENT IS HOLDING VERY MINIMAL BALANCE IN THE ACCOUNT, ACCOUNT WAS UTILISED TO ROUTING FUNDS IMMEDIATELY TO OTHER LOCAL PARTIES KEEPING MINIMAL BALANCE IN THE ACCOUNT BRINGS SUSPICION IN THE ACCOUNT PATTERN. 5.SHALINI ENTERPRISES 201000675197 , ACCOUNT OPENED WITH US ON 24 08 2016.CURRENT STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS 20 01 2020.AS ON 17 03 2021 TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RS.121.01CRORES AVAILABLE BALANCE IN THE ACCOUNT IS RS.0.97 CREDIT. BANK FILED STR ON 29 12 2016 VIA FIU BATCH ID 1612298525. HIGH VALUE CASH DEPOSITS AMOUNTING TO INR 13.77 CRORES WERE EVIDENCED SINCE 15TH NOV 2016 IN THE ACCOUNT. NO OTHER CREDITS OTHER THAN CASH DEPOSITS WERE EVIDENCED IN THE ACCOUNT. TURNOVER IN THE ACCOUNT HAS SURPASSED THE DECLARED TURNOVER. 10.MULTIPLE CURRENT ACCOUNTS ARE OPENED TOWARDS ROUTING FUNDS. 11.CREDITS RECEIVED IN THE ACCOUNT VIA INWARD REMITTANCES ARE UTILISED TO MAKE LOCAL PARTIES PAYMENTS VIA RTGS/NEFT. SAME ACCOUNT PATTERN EVIDENCED IN THE ALL THE RELATED ACCOUNTS. HIGH TURNOVER IN THE ACCOUNT HOWEVER VERY MINIMAL BALANCE MAINTAINED IN THE ACCOUNT. SUSPICION RAISED DUE TO BELOW MENTIONED RED FLAGS. 1.UNABLE TO ASCERTAIN OVERSEAS PARTY PROFILE THROUGH WEB SEARCH. 2.MAJORITY OF INWARD REMITTANCES TOWARDS EXPORT OF MOBILE PHONES RECEIVED FROM LOGISTICS FIRMS. 3.IMMEDIATE UTILISATION OF HIGH VALUE RTGS/NEFT CREDITS, INWARD REMITTANCE PROCEEDS TO MAKE RTGS/NEFT TO LOCAL PARTIES. 4. HIGH CREDIT TURNOVER IN A NEWLY INCORPORATED ACCOUNT. 5. MULTIPLE CURRENT ACCOUNTS ARE OPENED FOR ROUTING FUNDS FOR MAKING HIGH VALUE TRANSACTIONS. 6. DESPITE HIGH CREDIT TURNOVER IN RELATED ACCOUNT VERY MINIMAL BALANCE MAINTAINED. 7. FUNDS TRANSFER BETWEEN STR FILED ACCOUNTS. 8.STR FILED FOR THE RELATED ACCOUNTS TOWARDS SERIOUS RED FLAGS.CLIENT HAS BEEN REVIEWED FROM AML PERSPECTIVE BASIS HIGH VALUE INWARD REMITTANCES OUR OBSERVATIONS ARE 1.M/S AVIK TELEVENTURES PVT LTD IS A PVT LTD FIRM INCORPORATED ON 10 02 2020 AT FIRST FLOOR A 41,NARAINA INDUSTRIAL AREA PHASE 2, SOUTH WEST DELHI INDIA 110028. CLIENT OPENED CURRENT ACCOUNT 201004073711 WITH US ON 28 05 2020 AT CHANDANI CHOWK BRANCH, NEW DELHI INDUSIND BANK LTD, MUNICIPAL NO 43 TO 46, LAXMI NIWAS BUILDING, CHANDNI CHOWK, DELHI 110006 . AS ON 04 05 2021, STATUS OF ACCOUNT IS ACTIVE, CUMULATIVE DEBIT IS RS. 89,89,77,041.61 CUMULATIVE CREDIT IS RS. 90,74,26,251.37 AVAILABLE BALANCE IS RS. 84,49,209.76 CREDIT. LAST CUSTOMER INDUCED TRANSACTION DATE, AT THE TIME OF INVESTIGATION, IS 03 05 2021. NATURE OF BUSINESS IS IMPORT EXPORT OF TELECOMMUNICATION ITEMS, MOBILE/TELEPHONE ITEMS/ELECTRICAL/ELECTRONIC ITEMS. DECLARED TURNOVER IS ABOVE RS.10CRORES PER ANNUM. DIRECTORS MR. SAHIL JAIN MOBILE 9599448444 MR.RISHABH JAIN MOBILE 9599911666 . PAN IEC IS AATCA2839H. RISK CATEGORY OF THE CLIENT IS HIGH LAST KYC DONE ON 04 12 2020. DOCUMENTS OBTAINED DURING ACCOUNT OPENING ARE PAN, BOARD RESOLUTION, CERTIFICATE OF INCORPORATION, ASSOCIATION MEMO ARTICLE, GST REGISTRATION FORM. GSTIN IS 07AATCA2839H1ZC CURRENT STATUS IS ACTIVE AS PER GST SITE. 2.AS ON 04 05 2021, CLIENT HAD DONE 2 DIRECT/ADVANCE IMPORTS FOR RS.47.51LAKHS TOWARDS IMPORT OF OXYMETER, GAS SENSOR, OUTDOOR PM MONITOR, POLLUTION MONITOR, SQUAIR POLLUTION MONITOR,CAIR MONITOR TO OVERSEAS PARTIES RAINBOW COMMUNICATION CO SHAANXI BOHU ZHICHUANG ELECTRONIC TECHNOLOGY CO BASED IN CHINA HONGKONG. UNABLE AS FETCH OVERSEAS PARTIES PROFILE THROUGH WEB SEARCH. DURING THE SAME PERIOD CLIENT HAS RECEIVED 72 EXPORT ADVANCE FOR RS.76.32CRORES TOWARDS EXPORT OF MOBILE PHONES FROM THE OVERSEAS PARTIES BASED IN UAE UNITED STATES.MAJORITY OF REMITTANCES ARE RECEIVED FROM LOGISTICS FIRMS, RANDOM CHECK DONE ON ONE OF THE OVERSEAS PARTY PROFILE OF LOGIX ONE FZE, PARTY IS INTO LOGISTICS SERVICES HTTPS //CONNECT.AE/PROFILES/DUBAI/LOGIX ONE FZE/PROFILE ID ER5044418 . 3. AS ON 30 04 2021, CLIENT HAS ALL BOE OUTSTANDING AND 31 EXPORT ADVANCE OUTSTANDING.4.TOTAL CREDIT TURNOVER BETWEEN 04 06 20 TO 03 05 2021 IS RS.90.74CRORES WHICH INCLUDES TOTAL INWARD REMITTANCES RECEIVED FOR RS.76.32CRORES, OTHER CREDITS DETAILS ARE A.TOTAL RTGS CREDITS RECEIVED FOR RS.8.87CRORES, OF WHICH RS.7.44CRORES RECEIVED FROM E PAO GST REFUNDS, ACCOUNT WITH STATE BANK OF INDIA, ACCOUNT NO 31860389541 RS.33.50LAKHS RECEIVED FROM INDIVIDUAL ACCOUNT RAJ KUMAR SANJEEV KUM ACCOUNT WITH BANDHAN BANK LIMITED, ACCOUNT NO 10180004285866. B.MAJORITY OF INWARD REMITTANCES ARE RECEIVED FROM LOGISTICS FIRM.5. TOTAL DEBIT TURNOVER BETWEEN 18 06 20 TO 29 04 2021 IS RS.89.89CRORES A. RS.82.58CRORES VIA RTGS/NEFT,OF WHICH MAJORLY SENT TO CLIENT OWN ACCOUNT FOR RS.54.05CRORES, HELD ACCOUNT WITH AU SMALL FINANCE BANK LIMITED 2021210326967220 , RS.4.70CRORES TO BLUE BERRY, ACCOUNT WITH ICICI BANK 85405500721 RS.5.67CRORES TO HEMANG ELECTRONICS ACCOUNT WITH PUNJAB NATIONAL BANK 4184002100033610 LAXMI VILAS BANK 693360000000416 . B. TOTAL TRANSFER DEBIT FOR RS.1.69CRORES OF WHICH MAJORLY SENT TO YASHOM INTERNATIONAL TRADERS PVT LTD ACCOUNT NO 201003003290 FOR RS.75.12LAKHS RS.78.55LAKHS TO JAY JALARAM ENTERPRISE ACCOUNT NO 650001403814 . YASHOM INTERNATIONAL TRADERS PVT LTD 201003003290 , ACCOUNT OPENED ON 07 01 2019. STATUS OF THE ACCOUNT IS ACTIVE. LAST CUSTOMER INDUCED TRANSACTION IS 16 03 2021. TOTAL CREDIT BALANCE IN THE ACCOUNT IS RS.18.43CRORES.AVAILABLE BALANCE AS ON DATE IS RS.83,638.49 CREDIT.TRANSACTIONS OBSERVED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES UTILISED VIA RTGS/NEFT TO LOCAL PARTIES IMMEDIATELY, DESPITE OF HIGH VOLUMES CLIENT HOLDING MINIMAL BALANCE IN THE ACCOUNT, BRINGS MORE SUSPICION IN THE ACCOUNT PATTERN. UNABLE TO ASCERTAIN RELATIONSHIP BETWEEN OUR CLIENT YASHOM INTERNATIONAL TRADERS PVT LTD. C. JAY JALARAM ENTERPRISE ACCOUNT NO 650001403814",{"entities":[(5439,5462,"FOREIGN ORG"),(5464,5510,"FOREIGN ORG"),(5880,5891,"FOREIGN ORG"),(5962,5973,"FOREIGN ORG")]}),
            ("A FEW MORE ACCOUNTS FROM WHICH INWARD REMITTANCE ARE CREDITED 10 F A A S GENERAL 4TH FLOOR OFFICE NO 65 PALLA DIUM UAE , 11 L D K TEXTILE TRADING LLC DUBAI UAE , 12 LOGUS TRADING PO BOX 46277 DUBAI UAE , 13 LOTUS EXIM GENERAL LOTUS EXIM GENERAL TRADING UAE , 14 NIMBLE SMART P O BOX 14733 M018AL KAZIM UAE , 15 REVVER SIT GOODS P O BOX 183086 301301149 DUBAI UAE , 16 SCARCE TRADING LLC P O BOX 12870 904NAWRAS TOWER UAE , 17 SULTAN BIN BANA TEXTILE DUBAI UAE , 18 TAZEEZ GENERAL AL SANAVIL BUILD,NEAR BANIYAS UAE , 19 1009 PETROLEUM GROCERY ACCOUNT 1009 N USA , 20 FLORIDA IMPORT 1009 N STATE ROAD 7 ROYAL USA , 21 MOHAMMAD R ISLAM 10200 FOX TRAIL RD S ROYAL PALM B USA , 22 SURE GENERAL TRADING INC 10200 FOX TRAIL RD S ROYAL PALM USA , 23 USBD IMPORT CO INC 2216 SHOMA DR ROYAL PALM BEACH USA .RTGS DEBITS ARE SEEN TO A/C NO 12102000036227 OF SHIVAMANI IFSC IBKL0000012 , A/C NO 93002109027456 OF TURNING FASHIONS PVT LTD IFSC PUNB0009300 , A/C NO 811837185 OF POOJA SAREE IFSC KKBK0006566 , A/C NO 627505027032 OF WORLDWIDE FREIGHT SOLUTIONS PRIVAT IFSC ICIC0006275 , A/C NO 655705502751 OF BALAJI CREATIONS IFSC ICIC0006557 , A/C NO 53660100014495 OF ASHIK ALLI UNUSH IFSC BARB0BUPGBX , A/C NO 50200027002175 OF EXCLUSIVE IFSC HDFC0004120 . HIGH VALUE RTGS CREDITS ARE SEEN FROM A/C NO 31778422304 OF CBEC ICEGATE IFSC SBIN0010418 .1. FULL NAME AND PERMANENT ADDRESS OF THE ACCOUNT HOLDER A ONE INTERNATIONAL 1ST FLOOR 16 C PARK LANE PARK STREET KOLKATA WEST BENGAL INDIA 700016 2. DATE OF BIRTH 16102019 3. PAN NO ABQFA2052H 4. PASSPORT NO 5. OTHER IDENTIFICATION DOCUMENTS 6. IEC CODE ABQFA2052H 7. GSTIN 19ABQFA2052H1ZD 8. MOBILE NUMBER/LANDLINE NO. 919883025660 9. OTHER FACILITIES AVAILED 10. RISK CATEGORY LOW RISK 11. NAME AND DETAILS OF THE AUTHORISED SIGNATORIES IN THE ACCOUNT DEBASISH SWAIN WITH DOB 10 MAY 1987 AND PAN BWJPS3468J , ATIK AHMAD WITH DOB 12 JUL 1994 AND PAN BPMPA6668E 12. KYC COMPLIANCE STATUS COMPLIED 13. DATE OF LAST KYC UPDATION 29102020 14. PROFILE/NATURE OF BUSINESS PARTNERSHIP BUSINESS EXPORTER OF WOMEN S SUIT AND SAREES 15. ANNUAL INCOME AS DECLARED IN KYC 16. ANNUAL TURNOVER OF BUSINESS AS DECLARED IN KYC 2 CRORES 17. BENEFICIAL OWNER S AS DETERMINED UNDER RULE 9 3 OF PMLA RULES DEBASISH SWAIN WITH DOB 10 MAY 1987 AND PAN BWJPS3468J , ATIK AHMAD WITH DOB 12 JUL 1994 AND PAN BPMPA6668E 18. BANK AND BRANCH DETAILS THE FEDERAL BANK LTD, BRANCH KOLKATA / NEW MARKET,8 SUDDER STREET ,GROUND FLOOR NEW MARKET,KOLKATA,WEST BENGAL,700016 19. BANK ACCOUNT NO 13130200024776 20. BANK ACCOUNT TYPE CURRENT A/C 21. BANK A/C OPENED ON/ACTIVATION DATE 29102020 22. BANK ACCOUNT STATUS ACTIVE 23. QUANTUM OF TRANSACTIONS FOR THE CURRENT YEAR FY FY 2021 22 DEBIT SIDE RS. 857.97 LAKH IN CASH RS 7 LAKHS AND CREDIT SIDE RS. 808.8 LAKH IN CASH RS 0 LAKHS 24. QUANTUM OF TRANSACTIONS FOR PREVIOUS 3 YEARS FY FY 2020 21 DEBIT SIDE RS. 3701.65 LAKH IN CASH RS 18 LAKHS AND CREDIT SIDE RS. 3780.76 LAKH IN CASH RS 1 LAKHS 25. BALANCE IN ACCOUNT ON DATE OF FILLING STR 4945701 26. DETAILS OF EARLIER STRS FILLED ON THE ACCOUNT HOLDERS/RELATED PERSONS 27. REACTIVE STRS 28. ADVERSE MEDIA REPORT 29. GROUNDS OF SUSPICION HIGH VALUE FOREIGN REMITTANCES FOLLOWED BY IMMEDIATE RTGS DEBITS ARE REGULARLY SEEN IN THIS ACCOUNT.A DETAILED ANALYSIS OF THE FOREIGN TRANSACTIONS WERE DONE.FEW OF THE REMITTANCES FROM EDGEWORKS GENERAL TRADING LIMITED , DREAMWORTH GENERAL TRADING LIMITED , COSMIC EDGE GENERAL TRADING LIMITED BASED IN HONGKONG WHERE THE REMITTANCE NARRATION IS SEEN AS M/S. RUPAHALI BD, BANGLADESH S1021, F.A.J INTERNATIONAL, BANGLADESH S1, A.R.R JOINT, BANGLADESH WITH PURPOSE CODES AS P103. IT IS INFORMED BY THE FIRM THAT RUPAHALI IS ITS IMPORTERS AND THE FORMER COMPANIES ARE BUYERS OTHER THAN CONSIGNEES. HOWEVER IT IS KNOWN THAT RUPAHALI AND COMPANIES IN HONG KONG ARE SISTER CONCERNS.BASED ON CHECKING THE INVOICES , IT IS SEEN THAT INVOICE AMOUNT IS GREATER THAN THE ACTUAL REMITTANCE.WE SUSPECT THAT THE 80 20 SCHEME WHEREIN MAJOR SHARE OF EXPORT PAYMENTS HAVE COME FROM HONG KONG BUT THE GOODS HAVE BEEN SHIPPED TO BANGLADESH TO DUTY EVASION AS IMPORT TARIFF RATE IN BANGLADESH IS 0 AND HONG KONG IS 10.72 . AS WE FIND VARIOUS DISCREPANCIES IN THE FOREIGN INWARD REMITTANCES, WE FILE STR ON TRADE BASED MONEY LAUNDERING TBML ANGLE UNDER FIU IND OFFLINE ALERT NO TY 10.24 AND TY 10.14. 30. DETAILS OF INVESTIGATION THE VARIOUS FOREIGN INWARD REMITTANCES IN THIS ACCOUNT ARE FROM 1 COSMIC EDGE GENERAL HONGKONG , 2 DREAMWORTH GENERAL 01/FL,266 TEMPLE STREET,YAU MA TEI, HONGKONG , 3 EDGEWORKS GENERAL 01/FL, NO. 266 TEMPLE STREET,YAU HONGKONG , 4 GLOBAL TIME ASIA LIMITED 13 F TUNG FAI BUILDING 27 HONGKONG , 5 KRS GLOBAL CO. RM 1001 10F TOWER AHUNG HONGKONG , 6 AL GARHOUD GENERAL P.O.BOX 46351,UNITED ARAB UAE , 7 CAPRICORN GOODS P O BOX 42836 87RASIS BUSINESS UAE , 8 ESOON TRADING LLC PO BOX NO. 385126 DUBAI U.A. UAE , 9 EVERVENTIVE TRADING LLC PO BOX NO. 384762 DUBAI U.A. UAE",{"entities":[(124,148,"FOREIGN ORG"),(226,235,"FOREIGN ORG"),(368,385,"FOREIGN ORG"),(426,448,"FOREIGN ORG"),(465,478,"FOREIGN ORG"),(676,699,"FOREIGN ORG"),(654,665,"FOREIGN ORG"),(722,731,"FOREIGN ORG"),(775,790,"FOREIGN ORG"),(3331,3363,"FOREIGN ORG"),(3367,3400,"FOREIGN ORG"),(3404,3438,"FOREIGN ORG"),(4810,4826,"FOREIGN ORG"),(4865,4887,"FOREIGN ORG")]})
            ('FOREIGN REMITTANCE RS 1805 LAKHS FROM WORLD METALS AND ALLOYS FZC, SAMANCOR MARKETING PTE LTD, HWASUNG METAL CO LTD, SEASHORE STEEL, ETC, BILLS PAYMENT RS 823 CRORES, IMPS TRANSFER RS 102 CRORE FROM SUN STEEL, SHREE BALAJI ALUMNIC, ALTRUISTIC METAL PRIVATE LTD, DIVYA IMPORTERS, ETC, INTEREST RS 338 LAKHS, UPI TRANSFER RS 187 LAKH FROM KOLAWAT ENTERPRISES, SHRI RAGHAVENDRA STEELS, ANAND RAJESH, ETC, COMMISSION RS 3,199 PROCEEDS OF THE SAME ARE UTILIZED BY MAKING FOREIGN REMITTANCES RS 107803 CRORES TO COMPLEJO INDUSTRIAL MOLYNO, GLENCORE INTERNATIONAL AG, SIZER METALS PTE LTD, AMBANT LIMITED, ETC, CMS DEBITS RS 43314 CRORES, FUND TRANSFER RS 8862 LAKHS TO OSWAL ALLOYS PRIVATE LTD, MOHANLAL BHARATH KUMAR, SANJAY PITLIYA AND COMPANY, ETC, RTGS TRANSFER RS 7290 CRORES TO OSWAL MINERALS LIMITED YES BANK LTD, D S ALLOYD PVT LTD STATE BANK OF INDIA, LALWANI FERRO ALLOYS LTD AXIS BANK, SHYAM FERRO ALLOYS LIMITED HDFC BANK LTD, ETC, NEFT TRANSFER RS 1987 CRORES TO LALWANI FERRO ALLOYS LIMITED AXIS BANK, MANGANESE PRODUCTS CORPORATION UNION BANK OF INDIA, ISPAT DAMODAR PVT LTD STATE BANK OF INDIA, FERRO ALLOYS CORPORATION LTD STATE BANK OF INDIA, ETC, CASH RS 1029 CRORES, LOAN REPAYMENT RS 126 CRORE, TAX RS 5603 LAKHS, CHARGES RS 4263 LAKHS, CHEQUE RS 3071 LAKHS, SALARY RS 1308 LAKHS, BILLS RS 670 LAKHS, INTEREST RS 338 LAKHS, FEE RS 100 LAKH, CREDIT CARD RS 49,442, ONLINE TRANSACTION RS 33,690, COMMISSION RS 6,000PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO 609000016945 FOR THE PERIOD MENTIONED BELOW FY 20192020 01042019 TO 31032020 TOTAL CREDITS RS 135253 CRORES CASH 540 LAKHS AND TOTAL DEBITS RS 135704 CRORES CASH  1229 CRORES FY 20182019 01042018 TO 31032019 TOTAL CREDITS RS 121113 CRORES CASH 266 LAKHS AND TOTAL DEBITS RS 121526 CRORES CASH 445 CRORESFY 20172018 01042017 TO 31032018 TOTAL CREDITS RS 99920 CRORES CASH  91,080 AND TOTAL DEBITS RS 97918 CRORES CASH  671 CRORESBALANCE AS ON DATE OF FILING IS RS 3687029459WE HAVE ALREADY FILED STR ON ACCOUNT NO 609000016945 STR NO6194, DATE 18072017, BATCH ID 1707181814, REASON  HUGE AMOUNT OF FUNDS HAVE BEEN ROUTED THROUGH THE ACCOUNT BY INWARD AND OUTWARD RTGSNEFT AND INTERNAL FUND TRANSFERS WITHOUT ANY ECONOMIC RATIONALEWHILE CONDUCTING DISCREET CHECK, WE HAVE OBSERVED FEW DISCREPANCIES CUSTOMER HAS RECEIVED HIGH VALUE REFUND AGAINST DIRECT IMPORT BILL PAYMENT AS QUALITY CLAIM AND CONSOLIDATED VALUE OF REFUND AMOUNTED TO USD 19 MILLION, HOWEVER THE SAME WAS NOT HANDLED BY CLIENT AS PER RBI DIRECTIONS TOWARDS WRITE OFF OF IMPORT DUES FURTHER, HIGH VALUE TRANSACTIONS HANDLED UNDER DIRECT IMPORT BILL PAYMENT AS OPEN ACCOUNT PAYMENT AND PAYMENT TERMS APPEARING ON INVOICE SEEMS MODIFIED WHILE PAYMENT THERE IS FREQUENT TRADING BETWEEN THIRD COUNTRY AND PAYMENT MADE MAINLY WITH UAE ENTITY IE MS WORLD METALS AND ALLOYS FZC, WHEREIN MANUFACTURER OF THE GOODS OR ORIGIN DETAILS ARE NOT SPECIFIED PROPERLY IT APPARENTLY SEEMS THAT CUSTOMER HAS INTENTIONALLY BREACHED THE FEMA GUIDELINESHIGH VALUE RTGSNEFTFUND TRANSFERBILLS RECEIPT CREDITS FOLLOWED BY HIGH VALUE TRANSFER TO SPECIFIC THIRD PARTIES, ANOMALIES OBSERVED RAISING SUSPICION ON UNDERLYING INTENTION HENCE, STR FILEDOSWAL MINERALS LIMITED ACCOUNT NO 409000086197, 409001100618, 409000086203 AND 609000016945 ADDRESS 1034 2ND FLOOR DR RAJKUMAR ROAD,4TH BLOCK RAJAJINAGAR, OPPOSITE TO PLANET HONDA SERVICE CENTRE BANGALORE  560010, DATE OF INCORPORATION 04071996, PAN AACCM6499G, AADHAAR NUMBER REFERENCE KEY  IN VIEW OF THE RESTRICTION OF SHARING THE AADHAAR NUMBER AS PER REGULATION 6 OF AADHAAR SHARING OF INFORMATION REGULATIONS 2016, WE ARE PROVIDING REFERENCE KEY ASSIGNED TO THE AADHAAR NUMBER, WHEREVER APPLICABLE, MOBILE NUMBER 919448992115, RISK CATEGORY HIGH RISK, AUTHORISED SIGNATORY  MOHANLAL BHARATHKUMAR JAIN, SUBHASH CHAND MOHANLAL, KYC COMPLIANCE STATUS YES, DATE OF LAST KYC UPDATION 04072019 AND NA OPENED AN EXCHANGE EARNERS FOREIGN CURRENCY AC  EEFC ACCOUNT AND CASH CREDIT ACCOUNT RESPECTIVELY ON 04072019, 30092013, 30092013 AND 28092013 AT RBL BANK, BANGALORE BRANCH, 197, 6TH CROSS, GANDHINAGAR, BANGLORE 560009 AND FORT MUMBAI BRANCH, GROUND FLOOR, 2, 2A DHANNUR, 15, SIR FIROZ SHAH MEHTA ROAD, MUMBAI, FORT  400001 CONSTITUTION TYPE PUBLIC LIMITED COMPANY, ACCOUNT STATUS INACTIVE, ACTIVE, DORMANT AND ACTIVE RESPECTIVELYTHE CUSTOMER HAS DECLARED PROFILE AS MANUFACTURER OF METALS AND MINERALS HAVING EXPECTED ANNUAL TURNOVER IS GREATER THAN 100 CRORE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO 409000086197 FOR THE PERIOD FROM 01042020 UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS 11694 LAKHS CONSISTING OF FOREIGN REMITTANCES RS 11694 LAKHS FROM WORLD METALS ALLOYS, KUBOTA SAUDI COMPANY FACTORY, AL GARHOUD GENERAL TRADING, PINK LOTUS IRON AND STEELS, ETC PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS 10421 LAKHS, FOREIGN REMITTANCES RS 1272 LAKHS TO ALLIANZ BULK CFR8 PTE LTD, GLENCORE INTERNATIONAL AG, PMB SILICON SDN BHD, SIZER METALS PTE LTDPLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO 409000086197 FOR THE PERIOD MENTIONED BELOW FY 20192020 01042019 TO 31032020 TOTAL CREDITS RS 6825 LAKHS CASH  NIL AND TOTAL DEBITS RS 6825 LAKHS CASH  NILFY 20182019 01042018 TO 31032019 TOTAL CREDITS RS 14939 LAKHS CASH  NIL AND TOTAL DEBITS RS 14939 LAKHS CASH  NILFY 20172018 01042017 TO 31032018 TOTAL CREDITS RS 22357 LAKHS CASH  NIL AND TOTAL DEBITS RS 22357 LAKHS CASH  NILBALANCE AS ON DATE OF FILING IS NILTHERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED ACCOUNT NO 409001100618 FROM THE PERIOD OF 01042020 UPTO DATE OF FILING STRSCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO 409001100618 FOR THE PERIOD FROM 04072019 UPTO 31032020 REVEALED THAT THE TOTAL CREDITS OF RS 1328 LAKHS CONSISTING OF FOREIGN REMITTANCES RS 1328 LAKHS FROM STAIN LESS METALS GENERAL PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS 1328 LAKHSBALANCE AS ON DATE OF FILING IS NILTHERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED ACCOUNT NO 409000086203 FROM THE PERIOD OF 01042017 UPTO DATE OF FILING STRBALANCE AS ON DATE OF FILING IS NILSCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED ACCOUNT NO 609000016945 FOR THE PERIOD FROM 01042020 UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS 171541 CRORES CONSISTING OF RTGS TRANSFER RS 106217 CRORES FROM OSWAL MINERALS YES BANK LTD, SONIC THERMAL PVT LTD ALLAHABAD BANK, AJAY ENTERPRISES YES BANK LTD, MUKAND LTD YES BANK LTD, ETC, FUND TRANSFER RS 19383 CRORES FROM MOHANLAL, SEEMA JAIN, SUBHASHCHAN, SRIPAL KUMAR, SARITHA DEV, ETC, NEFT TRANSFER RS 34161 CRORES FROM OSWAL MINERALS LIMITED HDFC BANK LTD, ARCELORMITTAL NIPPON STEEL INDIA ICICI BANK LTD, SHYAM FERRO ALLOYS LIMITED HDFC BANK LTD, MACTECH ENGINEERING CORPORATION BANK, ETC, CMS RS 5267 CRORES, CHEQUE RS 3776 CRORES RETURNED AMOUNTED RS 4526 LAKHS',{'entities': [(38, 65, 'FOREIGN ORG'),(2823, 2850, 'FOREIGN ORG'),(67, 93, 'FOREIGN ORG'),(95, 115, 'FOREIGN ORG'),(117, 131, 'FOREIGN ORG'),(506, 532, 'FOREIGN ORG'),(534, 559, 'FOREIGN ORG'),(4945, 4970, 'FOREIGN ORG'),(561, 581, 'FOREIGN ORG'),(4993, 5013, 'FOREIGN ORG'),(1105, 1133, 'FOREIGN ORG'),(4918, 4943, 'FOREIGN ORG'),(4972, 4991, 'FOREIGN ORG')]}),
            ('2 SUBSEQUENT TO SUCH CREDITS FOREIGN OUTWARD REMITTANCES NONIMPORT AMOUNTED TO RS947 CR FAVOURING SPEED STREAM FREIGHT BROK  UAE RS677 CR, STATICE INTERNATIONAL PTE LTD  SINGAPORE RS154 CR AND EXCEL SEA SHIPPING LLC  UAE RS115 CR TOWARDS FREIGHT ON IMPORTS 3 BANK HAS ALREADY STR ON OTHER ACCOUNTS NAMELY BARA AFRICA ON SHORE SERVICES PRIVATE LIMITED 5845220101, BATCH ID 2103086872, BOXTRON SHIPPING AND LOGISTICS PVT LTD 5545595905, BATCH ID 2103086851 AND MILE LOGISTICS P LIMITED 3645356679, BATCH ID 2012284410 WHEREIN SIMILAR PATTERN OF TRANSACTIONS WERE NOTED AND FUNDS WERE REMITTED TO THE ABOVE MENTIONED COMMON OVERSEAS PARTY EXCEL SEA SHIPPING LLC UAE 4 IMPSNEFTRTGS PAYMENTS AMOUNTED TO RS072 CR MAINLY FAVOURING NEXASHIP LOGISTICS PRIVATE LIMITED RS064 CR, IDFC  10065573438, LIBERTON LOGISTICS PRIVATE LIMITED RS005 CR, IDFC  10065171794 AND T S GLOBAL IMPEX RS002 CR, IDFC  10064932707 5 LIBERTON LOGISTICS PRIVATE LIMITED AND NEXASHIP LOGISTICS PRIVATE LIMITED HAVE UPDATED A COMMON MAILING ADDRESS DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS SIGN BOARDS IN THE NAMES OF THE COMPANIES WERE SEEN AT THE PREMISE AND OFFICE SET UP WITH CHAIRS AND TABLES COULD BE SEEN HOWEVER NO MAJOR BUSINESS ACTIVITY OR EMPLOYEES COULD BE SEEN TO SUBSTANTIATE THE HIGH VALUE TRANSACTIONS ROUTED THROUGH ACCOUNT IT APPEARS THAT SHELL ENTITY MAY HAVE BEEN FORMED TO ROUTE TRANSACTIONS WITHOUT GENUINE COMMERCIAL BUSINESS ACTIVITY 6 A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE ROUND SUM RTGSNEFT CREDITS WERE FOLLOWED BY IMMEDIATE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS NO OTHER BUSINESS RELATED TRANSACTIONS COULD BE NOTED THROUGH THE ACCOUNT WITHIN 2 MONTHS OF ACCOUNT OPENING HIGH VALUE AND VOLUME OF TRANSACTIONS WERE ROUTED THROUGH ALL 3 ACCOUNTS 7 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT SHELL ENTITIES MAY HAVE BEEN FORMED ACROSS JURISDICTIONS AND CROSS BORDER MOVEMENT OF FUNDS MAY HAVE BEEN MADE UNDER THE PRETEXT OF FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WITHOUT ANY GENUINE COMMERCIAL BUSINESS ACTIVITY POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING ALSO THE PROPRIETORS AND DIRECTORS ARE IN THE AGE GROUP OF 2031 YEARS AND MAY NOT BE THE ACTUAL BENEFICIAL OWNERS OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNTS ACCOUNT OF LIBERTON LOGISTICS P LIMITED IS ACTIVE WITH BALANCE OF RS12,72,753 AS ON APRIL 5, 2021 QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 20212022 IS DEBIT RS000 IN CASH RS000 CR AND CREDITS RS021 CR IN CASH RS000 CR 20202021 IS DEBIT RS1042 CR IN CASH RS000 CR AND CREDITS RS1076 CR IN CASH RS000 CRA CURRENT ACCOUNT BEARING NUMBER 6845087299 WAS OPENED IN THE NAME OF LIBERTON LOGISTICS P LTD ON 11FEB21 AT OUR BRANCH LOCATED IN SHARDA TERRACES, PLOT NO 65, SEC11, CBD BELAPUR, NAVI MUMBAI  400614 THE ENTITY WAS FORMED ON 16JAN07 AND HAS SUBMITTED COPIES OF PAN AAGCA0882R, ALONG WITH BOARD RESOLUTION, MOA AOA AND CERT OF INCORP U74110MH2007PTC167056 AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO B 608 SHELTON SAPPHIRE PL 18 19 SECTOR 15 CBD BELAPUR NAVI MUMBAI 400614 DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS AND ULTIMATE BENEFICIAL OWNERS DIRECTORS HAVE REGISTERED THEIR MOBILE NUMBERS 9711056074 AND 8130480467 WITH THE BANK THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS500 CR AND THAT THE INDIVIDUALS HAVE ANNUAL INCOME IN THE RANGE OF RS025 CR TO RS050 CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED IN THE NAMES OF NEXASHIP LOGISTICS PRIVATE LIMITED 6945194835 AND A SOLE PROPRIETORSHIP CONCERN T S GLOBAL IMPEX 6945110002 OPENED ON 17FEB21 AND 16FEB2021 DHIRAJ KUMAR AND SAHIL ARE THE DIRECTORS IN NEXASHIP LOGISTICS PRIVATE LIMITED WHILE TSHERINGLA SHERPA IS THE PROPRIETOR IN T S GLOBAL IMPEX IT WAS INDICATED THAT THE ENTITIES ARE INTO CLEARING AND FORWARDING BUSINESS WITH ANNUAL TURNOVER OF RS400 CR AND 2000 CR RESPECTIVELY WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES NONIMPORT WERE ROUTED THROUGH THE NEWLY OPENED ACCOUNT OF LIBERTON LOGISTICS PRIVATE LIMITED SIMILAR PATTERN OF TRANSACTIONS AND COMMON OVERSEAS BENEFICIARIES WERE NOTED IN THE ACCOUNTS OF NEXASHIP LOGISTICS PRIVATE LIMITED AND T S GLOBAL IMPEX WHICH AS PER CUSTOMER ARE RELATED ACCOUNTS HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION 1 ON REVIEWING CONSOLIDATED ACCOUNT TRANSACTIONS SINCE INCEPTION TILL APRIL 5, 2021 IT WAS NOTED THAT ROUND SUM HIGH VALUE IMPSNEFTRTGS CREDITS AMOUNTED TO RS1065 CR MAINLY FROM NAMO SAI INTERNATIONAL RS804 CR, EQUITAS SMALL FINANCE BANK LIMITED AC NO 200001328409 AND NEXASHIP LOGISTICS PRIVATE LIMITED RS233 CR, IDBI 0183102000028802, IDFC 10065573438, SBI 39961353206, YES BANK 022063400000406',{'entities': [(139, 168, 'FOREIGN ORG'),(193, 215, 'FOREIGN ORG'),(636, 658, 'FOREIGN ORG'),(856, 872, 'FOREIGN ORG'),(3811, 3827, 'FOREIGN ORG'),(3995, 4011, 'FOREIGN ORG'),(4508, 4524, 'FOREIGN ORG'),(4853, 4875, 'FOREIGN ORG')]}),
            ("THE ARTICLE CAN BE FOUND AT HTTPSWWWDNAINDIACOMBUSINESSREPORTAUROBINDOINDIACEMENTSUNDERCBISCANNERFORJAGANLINKS1578674 AND HTTPSSCROLLINLATEST983720EDCOURTSUMMONSJAGANMOHANREDDYINCASERELATEDTOALLEGEDIRREGULARITIESINLANDALLOTMENT  ON CONDUCTING A SEARCH IN THE BANK'S INTERNAL DATABASE AGAINST THE NAMES APPEARING IN THE ARTICLE, MATCHES WERE FOUND AGAINST THE CUSTOMERS AUROBINDO PHARMA LTD, NITHYANANDA REDDY AND SARATH CHANDRA REDDY BASIS THE NAME APPEARING IN THE ADVERSE MEDIA ARTICLE FURTHER, PUBLIC DOMAIN SEARCHES REVEALED THAT THE ACCUSED PERSON NITHYANANDA REDDY WAS ALSO ONE OF THE DIRECTORS OF AUROBINDO PHARMA LTD, SENOR ORGANICS PVT LTD, APL RESEARCH CENTRE LIMITED ETC AND THE ACCUSED PERSON SARATH CHANDRA REDDY WAS ALSO ONE OF THE DIRECTORS OF AUROBINDO PHARMA LTD, TRIDENT LIFE SCIENCES LIMITED, TRIDENT CHEMPHAR LIMITED ETC IT WAS FURTHER OBSERVED THAT THE NAME AND DATE OF BIRTH OF CUSTOMERS NITHYANANDA REDDY KAMBAM AND PENAKA SARATH CHANDRA REDDY WERE FOUND TO BE MATCHING WITH THE PERSON NITHYANANDA REDDY WHO IS THE DIRECTOR IN THE COMPANIES AUROBINDO PHARMA LTD, SENOR ORGANICS PVT LTD, APL RESEARCH CENTRE LIMITED ETC AND SARATH CHANDRA REDDY WAS FOUND TO BE MATCHING WITH THE PERSON WHO IS THE DIRECTOR IN THE COMPANIES AUROBINDO PHARMA LTD, TRIDENT LIFE SCIENCES LIMITED, TRIDENT CHEMPHAR LIMITED ETC HTTPWWWMYCORPORATEINFOCOMDIRECTORNITYANANDAREDDYKAMBAM1284195 , HTTPWWWMYCORPORATEINFOCOMDIRECTORPENAKASARATHCHANDRAREDDY1628013  CUSTOMER AUROBINDO PHARMA LTD HOLDS CURRENT ACCOUNT NUMBER 44700000022 SINCE 03042002 AND EEFC ACCOUNT NUMBER 44605042759 USD SINCE 26092003 THE CUSTOMER ALSO HELD CURRENT ACCOUNT NUMBERS 44605035078, 44605033881 SINCE 27092002, 31072004 WHICH WAS CLOSED ON 09122009 AND 31012020 RESPECTIVELY CUSTOMER NITHYANANDA REDDY KAMBAM HOLDS CREDIT CARD NUMBERS 4622715364209836, 4622715364210511, 5546232907299217, 5546232916071243 SINCE 26022000, 22072004, 27062007, 15042009 RESPECTIVELY WITH A COMBINED ASSIGNED CREDIT LIMIT OF RS252 LAKHS THE CREDIT CARD NUMBER 4622715364209836 IS IN ACTIVE STATUS AND 4622715364210511, 5546232907299217, 5546232916071243 ARE IN INACTIVE STATUS THE CUSTOMER HELD CREDIT CARD NUMBERS 4129058880190190 AND 4129059880022888 SINCE 22072004, 26022000 WHICH WERE CLOSED ON 17012014, 18012014 RESPECTIVELY THE CUSTOMER PENAKA SARATH CHANDRA REDDY IS AN AUTHORISED SIGNATORY IN THE OVERDRAFT CURRENT ACCOUNT NUMBER 44605162509 HELD IN THE NAME OF MS TRIDENT CHEMPHAR LTD WITH THE BANK SINCE 23102015 WITH THE STATUS AS ACTIVE AND EEFC ACCOUNT NUMBER 44605162517 USD SINCE 23102015 WITH THE STATUS AS ACTIVE THE CUSTOMER ALSO HOLDS CREDIT CARD NUMBERS 5444388756463289 AND 5444388757014719 WITH THE BANK SINCE 27112009 RESPECTIVELY WITH A COMBINED ASSIGNED CREDIT LIMIT OF RS6 LAKHS WITH THE STATUS AS ACTIVE AND HELD CREDIT CARD NUMBERS 5444387750039269, 5444388756420487, 5444388756427094 27112009 WHICH WERE CLOSED ON 22012014, 18092015, 30012019 RESPECTIVELY NO UNUSUAL TRANSACTIONS WERE OBSERVED IN ALL THE ABOVEMENTIONED ACCOUNTS AND CREDIT CARD NUMBERS FOR THE PERIOD FROM 01042020 TO 12042021 NO ENQUIRIES HAVE BEEN RECEIVED ON THE CUSTOMER FROM ANY LEA TILL DATE CASE IS BEING REPORTED TO FIUIND DUE TO ADVERSE MEDIA INFORMATION FOUND ON THE CUSTOMERSCUSTOMER AUROBINDO PHARMA LIMITED DATE OF INCORPORATION 26121986; PAN AABCA7366H HAS ADDRESS AS WATER MARK BUILDING, PLOT NO 11, SURVEY NO9, KONDAPUR, HITECH CITY, HYDERABAD500084 THE CUSTOMER HAS BEEN CATEGORISED AS HIGH RISK AND THE LAST KYC WAS UPDATED ON 25032021 WITH THE STATUS AS COMPLETED THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER 44700000022 WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT SERENITY, GROUND FLOOR, 112, KORAMANGALA INDUSTRIAL AREA, 5TH BLOCK, KORAMANGALA, BANGALORE560095 SINCE 03042002 WITH THE STATUS AS ACTIVE THE CUSTOMER ALSO HOLDS EEFC ACCOUNT NUMBER 44605042759 USD SINCE 26092003 WITH THE STATUS AS ACTIVE THE CUSTOMER ALSO HELD CURRENT ACCOUNT NUMBERS 44605035078, 44605033881 SINCE 27092002, 31072004 WHICH WAS CLOSED ON 09122009 AND 31012020 RESPECTIVELY AS PER THE KYC RECORDS HELD WITH THE BANK, CUSTOMER AUROBINDO PHARMA LIMITED IS INTO MANUFACTURING OF MEDICINAL PHARMACEUTICAL PRODUCTS AND HAVING NITYANANDA REDDY KAMBAM, MADAN MOHAN REDDY METTU, VENKATA SUBRAHAMANYA SARMA MUSUNURI AND 17 OTHERS AS AUTHORISED SIGNATORIES THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS13,26648 CRORES REVIEW OF THE TRANSACTIONS FOR ACCOUNT 44700000022 FOR THE CURRENT FINANCIAL YEAR FROM 01042021 TO 12042021 REVEALED TOTAL CREDITS 0 AND TOTAL DEBITS OF RS5,841 IN CASH 0  REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEARS FROM 01042020 TO 31032021 REVEALED TOTAL CREDITS OF RS137 CRORES IN CASH 0 AND TOTAL DEBITS OF RS123 CRORES IN CASH 0 ; FROM 01042019 TO 31032020 REVEALED TOTAL CREDITS OF RS1711 CRORES IN CASH 0 AND TOTAL DEBITS OF RS1876 CRORES IN CASH 0  BALANCE IN THE ACCOUNT AS ON 12042021 WAS RS1714 LAKHS A REVIEW OF THE INTERNAL RECORDS REVEALED THAT AN STR WAS REPORTED PREVIOUSLY VIDE BATCH ID 2011 387 SNO 1 WITH THE REPORTED ACCOUNTS 44700000022, 44605042759 USD AND 44605033881 IN THE NAME OF AUROBINDO PHARMA LIMITED DUE TO ADVERSE MEDIA INFORMATION ON THE ENTITY GROUNDS OF SUSPICION THIS IS AN ADVERSE MEDIA AND A SUPPLEMENTARY STR CASE THE ACCOUNT 44700000022 OF AUROBINDO PHARMA LIMITED WAS TRIGGERED DURING A SPECIAL NAME SCREENING REVIEW UNDERTAKEN BY THE BANK SEARCHES CONDUCTED IN PUBLIC DOMAIN REVEALED AN ARTICLE DATED 23082011 IN WHICH IT WAS STATED THAT THE PROBE INTO THE WEALTH AMASSED BY YS JAGAN MOHAN REDDY, THE KADAPA MP AND SON OF FORMER ANDHRA PRADESH CHIEF MINISTER Y S RAJASEKHARA REDDY, BY THE CENTRAL BUREAU OF INVESTIGATION CBI IS PROVING TO BE A CAUSE OF HEADACHE FOR SEVERAL LISTED COMPANIES AUROBINDO PHARMA, INDIA CEMENTS, DALMIA BHARAT, NAVABHARAT VENTURES, PVP VENTURES AND RAMKY GROUP ARE AMONG THOSE NAMED IN THE FIR BY THE CBI IN THE LIST OF ACCUSED FURTHER, AN ARTICLE DATED 11012021 IN WHICH ED COURT SUMMONS JAGAN MOHAN REDDY IN CASE RELATED TO ALLEGED IRREGULARITIES IN LAND ALLOTMENT APART FROM THE CHIEF MINISTER, SUMMONS WERE SENT TO VIJAY SAI REDDY, A RAJYA SABHA MP OF HIS PARTY YSR CONGRESS, AND TO EXECUTIVES OF THREE PHARMACEUTICAL COMPANIES VIZ SRINIVASA REDDY OF HETERO DRUGS, NITHYANANDA REDDY OF AUROBINDO AND SARATH CHANDRA REDDY OF TRIDENT LIFE SCIENCES ALONG WITH TWO FORMER BUREAUCRATS",{'entities': [(781, 810, 'FOREIGN ORG'),  (1267, 1296, 'FOREIGN ORG'),  (650, 677, 'FOREIGN ORG'),  (1110, 1137, 'FOREIGN ORG')]}),
            ('GROUNDS OF SUSPICION THIS IS AN LEA CASE AND A SUPPLEMENTARY CASE BANK WAS IN RECEIPT OF A SUMMON DATED 02012021 VIDE REFERENCE NUMBER ITBAINVS1312020211029437924 1 US 131 1A FROM NARENDRA MOLAI PRASAD, DYDIRECTOR OF INCOME TAX INV UNIT 3 4 , INCOME TAX DEPARTMENT, OFFICE OF THE ASSISTANT DIRECTOR OF INCOME TAX, INVESTIGATION THE NOTICE DIRECTED THE BANK TO PROVIDE ACCOUNT RELATED INFORMATION OF R N EXPORTS HOLDING ACCOUNT NUMBERS 22405360386, 22405360424 AND 22506142090 FOR INVESTIGATION PURPOSE THE INFORMATION SOUGHT HAS BEEN PROVIDED TO THE INCOME TAX DEPARTMENT REVIEW OF THE TRANSACTIONS FOR ACCOUNT 22405360386 FOR THE PERIOD FROM 01042020 TO 13042021 REVEALED TOTAL CREDITS OF RS337 CRORES AND TOTAL DEBITS OF RS350 CRORES MAJOR CREDITS IN THE ACCOUNT WERE BY WAY OF INTERNAL FUND TRANSFERS TOTALLING TO RS276 CRORES ON 7 INSTANCES WHICH WERE RECEIVED FROM OWN USD DIAMOND DOLLAR ACCOUNT ACCOUNT NUMBER 22506142090 , RTGSNEFT CREDITS TOTALLING TO RS5404 LAKHS ON 38 INSTANCES OF WHICH RS2851 LAKHS FROM OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 2413200206 ON 5 INSTANCES, RS959 LAKHS FROM CARAT LANE TRADING PRIVATE LIMITED ICICI BANK LTD, ACCOUNT NUMBER 0104SLNEFTPL ON 9 INSTANCES, RS820 LAKHS FROM KANTILAL CHHOTALAL BANK OF INDIA, ACCOUNT NUMBER 016720110000024 ON 12 INSTANCES, RS488 LAKHS FROM NAICE JEWELS PRIVATE LIMITED KOTAK MAHINDRA BANK LIMITED, ACCOUNT NUMBER 500044010156 ON 8 INSTANCES, RS160 LAKHS FROM CREATIVE GEMS AND JEWELLERY LTD STATE BANK OF INDIA, ACCOUNT NUMBER 00000032883046667 ON 3 INSTANCES, RTGS RETURNS TOTALLING TO RS661 LAKHS MAJOR DEBITS TO THE ACCOUNT WERE DEBITED BY WAY OF RTGSNEFT TOTALLING TO RS269 CRORES WHICH WERE TRANSFERRED TO OWN ACCOUNT KOTAK MAHINDRA BANK LTD, ACCOUNT NUMBER 2413200206 ON 8 INSTANCES, FORWARD CONTRACT CANCELLATION CHARGES TOTALLING TO RS6547 LAKHS, CHEQUES ISSUANCE TOTALLING TO RS829 LAKHS, IMPORT PAYMENT TOTALLING TO RS641 LAKHS TO TANU STAR HK LTD, HONG KONG ETC A REVIEW OF THE TRANSACTIONS IN USD DIAMOND DOLLAR ACCOUNT 22506142090 FOR ABOVE REVIEW PERIOD REVEALED BUSINESS RELATED TRANSACTIONS SEARCHES CONDUCTED IN PUBLIC DOMAIN DID NOT REVEAL ANY ADVERSE MEDIA ON THE CUSTOMER CASE IS BEING REPORTED TO FIUIND DUE TO THE NOTICE RECEIVED FROM THE INCOME TAX DEPARTMENTCUSTOMER R N EXPORTS DATE OF INCORPORATION 01041991; PAN AAAFR2200B WITH IEC CODE 0391008421 ISSUED BY DGFT, GSTIN 27AAAFR2200B1Z3 ISSUED BY CBEC HAS ADDRESS AS 301, PETIT TOWER, 3RD FLOOR, CUMBALLA HILL, A K MARG, MUMBAI400026 MOBILE NUMBER 9820046978 AND LANDLINE NUMBER 02223888820 THE CUSTOMER HAS BEEN CATEGORISED AS MEDIUM RISK AND THE LAST KYC WAS UPDATED ON 08092020 WITH THE STATUS AS COMPLETED THE CUSTOMER HOLDS CURRENT ACCOUNT NUMBER 22405360386 WITH THE STANDARD CHARTERED BANK BRANCH LOCATED AT SHREE PANT BHUVAN, SANDHURUST BRIDGE, MUMBAI400007 SINCE 27082010 WITH THE STATUS AS ACTIVE THE CUSTOMER ALSO HOLDS AN USD EEFC ACCOUNT NUMBER 22405360424 SINCE 27082010 WHICH IS IN DORMANT STATUS SINCE 14062019 AND USD DIAMOND DOLLAR ACCOUNT NUMBER 22506142090 SINCE 06072016 WHICH IS IN ACTIVE STATUS AS PER THE KYC RECORDS HELD WITH THE BANK, CUSTOMER R N EXPORTS IS A PARTNERSHIP FIRM ENGAGED IN THE BUSINESS OF MANUFACTURING OF JEWELLERY AND WORKING OF DIAMONDS RAHUL ROHIT JHAVERI AND SONALI RAHUL JHAVERI ARE THE BENEFICIAL OWNERS AND AUTHORISED SIGNATORIES OF THE FIRM THE DECLARED ANNUAL TURNOVER OF THE CUSTOMER WAS RS6295 CRORES REVIEW OF THE TRANSACTIONS FOR ACCOUNT 22405360386 FOR THE CURRENT FINANCIAL YEAR FROM 01042021 TO 13042021 REVEALED TOTAL CREDITS OF 0 IN CASH 0 AND TOTAL DEBITS OF 0 IN CASH 0  REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 01042020 TO 31032021 REVEALED TOTAL CREDITS OF RS337 CRORES IN CASH 0 AND TOTAL DEBITS OF RS350 CRORES IN CASH RS74,000 ; REVIEW OF THE TRANSACTIONS FOR THE PREVIOUS FINANCIAL YEAR FROM 01042019 TO 31032020 REVEALED TOTAL CREDITS OF RS2112 CRORES IN CASH 0 AND TOTAL DEBITS OF RS2137 CRORES IN CASH RS2 LAKHS  BALANCE IN THE ACCOUNT AS ON 13042021 WAS 0 A REVIEW OF THE INTERNAL RECORDS REVEALED THAT AN STR WAS REPORTED PREVIOUSLY VIDE BATCH ID 1702067988, SNO 1 WITH THE REPORTED ACCOUNTS 22405360386, 22405360424 AND 22506142090 IN THE NAME OF R N EXPORTS DUE TO CIRCULATION OF FUNDS BETWEEN THE CLIENT AND A HONG KONG BASED ENTITY ON ACCOUNT OF PURCHASE AND SALE OF ROUGH AND POLISHED DIAMONDS',{'entities': [(1924, 1940, 'FOREI ORG')]}),
            (' DIGNITY FOREX PRIVATE LIMITED, A            PRIVATE LIMITED COMPANY, HAVING ADDRESS BG3, PLOT16 BACHUBAI BLDG, GRD FLR, C S T RLY STATION, MANGESH SHENOY, STREET BHATIA BAUG FORT, MUMBAI, MAHARASHTRA, INDIA, 400001, CONTACT NUMBER 918433774759 AND EMAIL ID DIGNITYFOREX@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 623505388610 WITH ICICI BANK LIMITED, MUMBAI  FORT, MAHARASHTRA, 400001, SINCE 04062019 DATE OF INCORPORATION IS 05092018 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AAGCD8428G PAN LINKED ACCOUNT IS 697705600660 TRANSACTION LINKED ACCOUNT IS 623505389180, 057501509558, 032405002363, 034805003651 AUTHORIZED SIGNATORY BENEFICIAL OWNER DIRECTOR OF THE REPORTED ACCOUNT IS KAVIN ANTHONY FERNANDES PAN ABCPF4628E THE CUSTOMER IS CATEGORIZED AS MEDIUM RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE COI U67120MH2018PTC313571, PAN AAGCD8428G, MOA AOA CUSTOMER WAS ENGAGED IN THE BUSINESS OF FOREIGN EXCHANGE DEALER SERVICES THE COMPANY ACT AS FOREIGN EXCHANGE BROKERS, BUYING, SELLING, DEALERS, PRINCIPALS AND TO ENGAGE IN INTERBANK CASH, SPOT, SWAPS, FORWARDS AND OPTIONS OF VARIOUS FOREIGN CURRENCIESDECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS BELOW RS 40 LAKHSAS PER ENHANCED DUE DILIGENCE, CUSTOMER IS NOT CONTACTABLE AND NOT TRACEABLE ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNTS BALANCE IN THE ACCOUNT AT 28022021 WAS NIL QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT WAS NIL TOTAL DEBITS DURING THE SAME PERIOD WAS RS 252 LAKHS OF WHICH INTERNET FUND TRANSFER RS 244 LAKHS TO EVER SHINE FOREX TOURS AND TRAVELS PRIVATE LIMITED 623505389180 FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 04062019 TO 31032020 TOTAL CREDIT SIDE RS 50,13226 LAKHS OF WHICH CASH WAS RS 11654 LAKHS FROM FORT, MUMBAI BRANCH, NEFT RS 59229 LAKHS FROM SE7EN SEAS FOREX PVT LTD, RTGS RS 20,55710 LAKHS FROM CROWN INTERNATIONAL, FASHION ENTERPRISES, INTERNET FUND TRANSFERS RS 28,82278 LAKHS FROM EVER SHINE FOREX TOURS AND TRAVELS PRIVATE LIMITED 032405002363, EVER SHINE FOREX TOURS AND TRAVELS PRIVATE LIMITED 623505389180, MOBILE MONEY TRANSFERS RS 1097 LAKHS IN 14 INSTANCES TOTAL DEBIT SIDE RS 50,12973 LAKHS OF WHICH RTGS RS 21,00288 LAKHS TO WORLD ONE INDIA FOREX PRIVATE LIMITED, EBIXCASH WORLD MONEY LIMITED, INTERNET FUND TRANSFER RS 28,80915 LAKHS TO ARVOG FOREX PRIVATE LIMITED 034805003651 TRANSACTION PATTERN SHOWS CREDIT FOLLOWED BY IMMEDIATE DEBITS FOR EXAMPLE, ON 12032020, RS 3713 LACS AMOUNT OF FUND CREDITED THROUGH INTERNET FUND TRANSFERS FROM EVER SHINE FOREX TOURS AND TRAVELS PRIVATE LIMITED 032405002363 WHICH DEBITED THROUGH INTERNET FUND TRANSFERS TOWARDS ARVOG FOREX PRIVATE LIMITED 034805003651 ON 11032020, RS 3481 LACS AMOUNT OF FUND CREDITED THROUGH INTERNET FUND TRANSFERS FROM EVER SHINE FOREX TOURS AND TRAVELS PRIVATE LIMITED 623505389180 WHICH DEBITED THROUGH TRANSFERS TOWARDS ARVOG FOREX PRIVATE LIMITED 034805003651 SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED LIKE ROUTING OF BUSINESS FUNDS TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 201920 WAS RS 50,13226 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS IN THE ACCOUNT, HIGH TURNOVER IN THE ACCOUNT AND NEGATIVE EDD FINDINGS WHEREIN CUSTOMER IS NOT CONTACTABLE AND NOT TRACEABLE, STR IS BEING FILED',{'enties': [(2122, 2141, 'FOREIGN ORG'), (2143, 2162, 'FOREIGN ORG')]}),
            ('ONE OF THE ENTITIES REGISTERED AT AFOREMENTIONED ADDRESS IS WHITE RABBIT MOTION PICTURES PRODUCTION LLC AS PER AVAILABLE OPINION REPORT WHITE RABBIT MOTION PICTURES PRODUCTION LLC AND MAGNETIC MOTION PICTURE PRODUCTION LLC ARE GROUP ENTITIESIT IS OBSERVED THAT WHITE RABBIT MOTION PICTURES PRODUCTION LLC, UAE IS COMMON BENEFICIARY RECEIVING CROSS BORDER REMITTANCES FROM MULTIPLE YBL CUSTOMER VIZ KEYFRAME MULTIMEDIA SERVICES, SOLITUDE ENTERTAINMENT LLP AND EYE FOCUS PRODUCTION WHEREIN INVOICE DRAWN ON EACH OF THESE CUSTOMER WHEN COMPARED DID NOT MATCH WITH RESPECT TO COMPANY LOGO AND OVERALL INVOICE FORMAT STRS WERE THEREFORE FILED ON BELOW CUSTOMERS VIDE FIU BATCH ID AS UNDER KEYFRAME MULTIMEDIA SERVICES  2103183777SOLITUDE ENTERTAINMENT LLP  2103256408EYE FOCUS PRODUCTION 21020935392AS PER AGREEMENT COPY, REMITTANCES ARE BEING UNDERTAKEN TOWARDS FILM SHOOTING IN FIJI ISLANDS HOWEVER FUNDS ARE BEING REMITTED IN FAVOR OF ENTITY BASED IN UNRELATED JURISDICTION OF UAE3WHEN YES BANK OFFICIALS REQUESTED THE CUSTOMER TO SHARE COPY OF PASSPORT AND VISA OF CREW STAR CAST WHO HAVE TRAVELLED TO FIJI ISLANDS TO WHICH CUSTOMER REFUSED TO SHARE ANY SUCH DETAILS AND FURTHER ADDED THAT HE SHALL SHARE THE EVIDENCES WITH THE RESPECTIVE AUTHORITY WHEN NEEDED 4SITE VERIFICATION CONDUCTED AT THE ADDRESS REGISTERED WITH THE BANK WAS POSITIVE HOWEVER IT WAS OBSERVED THAT BESIDES DIGI MEDIA AND FILMS SOLUTIONS CLIENT IS ALSO ASSOCIATED WITH 2 OTHER ENTITIES VIZ TINGLE CINE PRODUCTION AND WATERMARK STUDIOS ALL 3 OF WHICH OPERATE FROM SAME COMMON ADDRESSWHILE REVIEWING ACCOUNT STATEMENTS OF REMITTER CUSTOMERS CUSTOMER VIZ DIVERSE MEDIA AC NO 063663400000370 MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 AND LIQUID STUDIOS AND SERVICES AC NO 063663300002507 BELOW ARE RED FLAGS OBSERVED 5STATED ANNUAL INCOME OF DIVERSE MEDIA AND MAGNETIC FILMS PRODUCTIONS IS LESS THAN RS50 CRORES AND THAT OF LIQUID STUDIOS AND SERVICES IS LESS THAN RS1 CRORE HOWEVER IN LESS THAN 6 MONTHS OF ACCOUNT OPENING SAID ENTITIES HAVE RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS30 TO 50 CRORES IN EACH OF 3 ACCOUNTS6AFOREMENTIONED ENTITIES STATED LINE OF BUSINESS IS MEDIA AND ENTERTAINMENT HOWEVER MAJORITY OF CREDITS RECEIVED IN THESE ACCOUNTS ARE FROM MULTIPLE ENTITIES ENGAGED IN UNRELATED LINE OF BUSINESS EG IN THE ACCOUNT OF MAGNETIC FILMS PRODUCTIONS CLIENT HAS RECEIVED ACCOUNT CREDITS FROM TWICLO TOUR AND PACKAGES , BHUTANI CORPORATION, SHYAM ENTERPRISES, EAGLE INTERNATIONAL LOGISTICS, RK STEEL ALLOYS FEW OF WHICH ARE COMMON COUNTERPARTIES REMITTERS SENDING FUND TRANSFERS IN FAVOR OF DIVERSE MEDIA AND LIQUID STUDIOS AND SERVICES7FURTHER, CUSTOMERS DIVERSE MEDIA, MAGNETIC FILMS PRODUCTIONS AND LIQUID STUDIOS AND SERVICES HAVE RECEIVED TOTAL OF RS743 CRORES FROM DOMESTIC COUNTERPARTY TWICLO TOUR AND PACKAGES TWICLO TOUR AND PACKAGES IS COMMON REMITTERCOUNTERPARTY WHICH HAS ALSO PROVIDED FUNDING TO YBL CUSTOMER DISCOVER PLANET SOLUTIONS ON WHICH STR WAS FILED VIDE FIU BATCH ID  21021582848ACCOUNT CREDITS FROM BUSINESSES IN UNALLIED ACTIVITIES IS FOLLOWED BY FUNDS TRANSFER IN FAVOR OF VARIOUS ENTITIES IN MEDIA AND ENTERTAINMENT BUSINESS INCLUDING DIGI MEDIA AND FILMS SOLUTIONS AND ITS ASSOCIATE ENTITIES TINGLE CINE PRODUCTIONS AND WATERMARK STUDIOS 9WHEN YES BANK OFFICIALS INQUIRED ON BUSINESS RATIONAL FOR CREDITS BEING RECEIVED FROM UNALLIED BUSINESSES TO WHICH CUSTOMER REPLIED STATING THAT THEY ARE FINANCIERS INVESTORS AS WELL AS PRODUCERS AND THAT FUNDS ARE BEING RECEIVED FOR ARRANGING EQUIPMENTS, LOGISTICS, MANPOWER ETC FOR MAKING OF SHORT FILMS OR WEBSERIES ETCIN LIGHT OF HIGH VALUE TRANSACTIONS BEING RECORDED BEYOND CLIENTS STATED ANNUAL INCOME, TRANSACTIONS WITH ENTITIES IN UNALLIED ACTIVITIES AND CROSS BORDER REMITTANCE IN FAVOR OF ENTITY WITH UNSATISFACTORY CREDENTIALS IT HAS BEEN DECIDED ON 16APRIL 2021 TO FILE STR SAID CUSTOMER1 THE CUSTOMER DIGI MEDIA AND FILMS SOLUTIONS, A PARTNERSHIP FIRM PAN AARFD0600J, DOI 17APRIL2019, COMMUNICATION ADDRESS 2 826265 FLAT NO 201, BANJARAHILLS OPP SAKSHI OFFICE, HYDERABAD, TELANGANA, 500034, GSTIN  36AARFD0600J1ZK IEC CODE  AARFD0600J, MOBILE NUMBER 918179551161, EMAIL ID CONTACT@DIGIMEDIAFILMSIN IS MAINTAINING A CURRENT AC NO 025063400001118 SINCE 31122020 WITH BANJARA HILLS, HYDERABAD BRANCH ADDRESS SHOP NO G1, DEV DHANUKA PRESTIGE, ROAD NO 12, BANJARA HILLS, HYDERABAD 5000342 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN DECEMBER2020 MOHIT BANSAL AND MANISH CHAWLA ARE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS ENGAGED IN MEDIA AND ENTERTAINMENT BUSINESS WITH AN ANNUAL GROSS INCOME LESS THAN RS75 CRORES  DIGI MEDIA AND FILMS SOLUTIONS AC NO025063400001118 IS COMMON BENEFICIARY RECEIVING FUNDS TRANSFER FROM 3 OTHER YES BANK CUSTOMERS AS UNDERCURRENT ACCOUNTS OF DIVERSE MEDIA AC NO 063663400000370, MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 AND LIQUID STUDIOS AND SERVICES AC NO 063663300002507 ARE LINKED WITH ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS THROUGH FUND TRANSFERS WHEREIN LATTER IS COMMON BENEFICIARYALL 3 ENTITIES VIZ DIVERSE MEDIA, MAGNETIC FILMS PRODUCTIONS AND LIQUID STUDIOS AND SERVICES HAVE COMMON BUSINESS ADDRESS  FLOOR FIRST SHOP NO55 LOCAL, SHOPPING, CENTER BLOCK A1, NEW DELHI, DELHI 110063 AND ARE ENGAGED IN BUSINESS OF MEDIA AND ENTERTAINMENT3 IN THE ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS AC NO 025063400001118 AND 3 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM 01042020 TILL 16042021, SHOWS 1,096 CREDIT TRANSACTIONS AMOUNTING TO RS 1,225,798,293 AND 702 DEBIT TRANSACTIONS AMOUNTING TO RS 1,225,792,338OUT OF THE TOTAL CREDITS OF RS 1,225,798,293 IN THE ACCOUNTS, NEFTRTGS AMOUNTS TO RS 1,168,123,534 CONSTITUTING AROUND 95 PERCENT OF THE TOTAL CREDITS, OUT OF THE TOTAL DEBITS OF RS 1,225,792,338 IN THE ACCOUNTS, NEFTRTGS AMOUNTS TO RS 1,140,639,525 CONSTITUTING AROUND 93 PERCENT OF THE TOTAL DEBITS,4 IN THE ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS AC NO 025063400001118 AND 3 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM 01042017 TILL 31032020 NO TRANSACTIONS WERE RECORDED IN THE ACCOUNT OF DIGI MEDIA AND FILMS SOLUTIONS AC NO 025063400001118, DIVERSE MEDIA AC NO 063663400000370, MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 AND LIQUID STUDIOS AND SERVICES AC NO 063663300002507 SINCE ACCOUNTS WERE OPENED IN THE MONTH OF SEPTEMBER 2020 TO DECEMBER 2020 BALANCE IN THE ACCOUNT AS ON 16042021 IS RS 90075ON REVIEWING THE TRANSACTION PATTERN OF DIGI MEDIA AND FILMS SOLUTIONS AC 025063400001118 IT WAS OBSERVED THAT SINCE JANUARY 2021 CLIENT HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS18 CRORES WITH MAJOR CREDITS VIA FUNDS TRANSFERS FROM YBL CUSTOMERS DIVERSE MEDIA AC NO 063663400000370 RS525 LACS MAGNETIC FILMS PRODUCTIONS AC NO 063663400000390 RS481 LACS, LIQUID STUDIOS AND SERVICES AC NO 063663300002507 RS54 LACS AND RTGS FROM LIQUID STUDIOS AND SERVICES RS26LACS FROM AC NO 39631954021 WITH STATE BANK OF INDIA, SBIN0009111 ACCOUNT CREDITS WERE FOLLOWED BY DEBITS TO THE TUNE OF RS18 CRORES WITH MAJOR DEBITS PERTAINING TO OUTWARD REMITTANCE TO THE TUNE OF RS17 CRORES PERTAINING TO AUDIOVISUAL AND RELATED SERVICES IN FAVOR OF OVERSEAS ENTITY MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE 1BELOW ARE RED FLAGS OBSERVED WITH REGARDS TO OVERSEAS ENTITY MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE PUBLIC DOMAIN INFORMATION ON MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE IS INSUFFICIENTAS PER PUBLIC DOMAIN, MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS AS PROVIDED IN INVOICE  1101, AL MASRAF TOWER, BANIYAS ROAD, DEIRA, DUBAI, UAE, POBOX 242874 OF SAID OVERSEAS ENTITY',{'entities': [(60, 103, 'FOREIGN ORG'),(136, 179, 'FOREIGN ORG'),(261, 304, 'FOREIGN ORG'),
    (184, 222, 'FOREIGN ORG'),
    (7186, 7224, 'FOREIGN ORG'),
    (7292, 7330, 'FOREIGN ORG'),
    (7365, 7403, 'FOREIGN ORG'),
    (398, 426, 'FOREIGN ORG'),
    (684, 712, 'FOREIGN ORG'),
    (459, 479, 'FOREIGN ORG'),
    (762, 782, 'FOREIGN ORG'),
    (2449, 2478, 'FOREIGN ORG'),
    (2480, 2495, 'FOREIGN ORG')]}),
 ('3 HIGH VALUE ADV IMP REM AMOUNTED TO RS11761 CR FROM THE AC OF PREMIUM PETRO PRODUCTS MAINLY FAVOURING UAE BASED ENTITIES  MOTOTIVE GENERAL TRADING LLC RS3497 CR, PETRO STAR FZE RS3371 CR, KIARA PETROLEUM FZE RS1378 CR, OULA ALNUHA INTERNATIONAL RS1104 CR, TIME TECHNO PACKAGING LLC RS934 CR, TASHBULAK TRADING LLC RS361 CR AND MAN TRADING LLC RS348 CR TOWARDS IMPORT OF BITUMEN 4 PREMIUM PETRO PRODUCTS IS YET TO SUBMIT BOES PERTAINING TO 72 CASES TOTALLING TO USD 8,250,389 WHICH COMPRISES OF 14 CASES TOTALLING TO USD 985,122 PENDING FOR A PERIOD ABOVE 3 MONTHS UPTO 6 MONTHS AND 58 CASES TOTALLING TO USD 7,265,266 PENDING FOR A PERIOD LESS THAN 3 MONTHS 5 IMPSNEFTRTGS PAYMENTS AMOUNTED TO RS7177 CR MAINLY FAVOURING PREMIUM PETRO PRODUCTS RS4316 CR, PP SOFTTECH P LTD RS715 CR, ICICI  732305000042, VISHAL BOHRA RS240 CR, HDFC 4854980603624562 59109829999990, IDBI 0131063380800, YES BANK  65432109829999990, VINAYAK EXIMS RS115 CR, AXIS BANK  914020013741606 AND AUM KRIPA ENTPS RS100 CR, HDFC  59272000000001 6 INTERCONNECTED FUNDS TRANSFER TXNS WITHIN GROUP ACS AMOUNTED TO RS3771 CR UTURN TRANSACTIONS WERE NOTED WHEREIN FUNDS AMOUNTING TO RS15250 CR WERE RECEIVED AND RS5009 CR WERE SENT OUT TO ACS OF PREMIUM PETRO PRODUCTS, VISHAL BOHRA, ABHA PRECISION FARMING LLP, BHEEM RAJ BOHRA, MANISH BOHRA IDBI 013104000294935 AND VCI EXPORTS P LTD MAINTAINED WITH OTHER BANKS 7 PREMIUM PETRO PRODUCTS, B L INVESTMENT, RISHI ENTPS, HERO PETRO PRODUCTS, REEVA BUILDCON P LTD, REALTECH BUILDHOME P LTD, ABHA PRECISION FARMING P LTD, AONE BUILDHOME LLP AND BODHI FARM DEVELOPERS P LTD HAVE UPDATED A COMMON MAILING ADDRESS 13 HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN  302001 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT IT WAS A 2000 SQ FT OWNED PREMISE WITH SIGN BOARD, EMPLOYEES AND OFFICE SET UP SEEN HOWEVER NO STOCK OF GOODS WERE SIGHTED ON ENQUIRY IT WAS LEARNT FROM CUSTOMER THAT GOODS WERE STOCKED AT MOKHAMPURA, TEHSILMOZAMABAD, DISTRICT JAIPUR 8 PAN OF INTERCONNECTED ACS  SHREEBALAJI INFRAPROJECTS P LTD AAPCS5089K AND DENIS LAND DEVELOPERS P LTD AACCD6725C ARE REFLECTING IN THE LIST OF POSSIBLE SHELL ENTITIES CIRCULATED BY FIU INDIA 9 TOTAL TAX PAYMENTS OF RS070 CR WAS NOTED THROUGH THE ACS OF UJJWAL BHANDARI HUF 9514297302 AND VCI EXPORTS P LTD 3413099593 DURING REVIEW PERIOD, WHICH APPEARS TO BE DISPROPORTIONATE TO HIGH VALUE AND VOLUME OF TRANSACTIONS ROUTED THROUGH ACS10 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH MULTIPLE ACS OPENED BY THE SAME GROUP OF INDIVIDUALS FOR LAYERING OF FUNDS AND POSSIBLY TO REMIT FUNDS OVERSEAS WITHOUT ACTUAL IMPORTS TAKING PLACE POSSIBLY WITH THE INTENTION OF TBML AND HAWALA SETTLEMENT IT ALSO APPEARS THAT SUCH HIGH VALUE AND VOLUME OF TRANSACTIONS MAY HAVE BEEN CARRIED OUT WITHIN GROUP ACS WITHOUT ANY ECONOMIC RATIONALE POSSIBLY WITH INTENTION OF TAX EVASION AC OF PREMIUM PETRO PRODUCTS BEARING NUMBER 6813095042 IS ACTIVE WITH BALANCE OF RS208955 AND 6813095059 HAS OUTSTANDING BALANCE OF RS7,05,25,633 ON CREDIT LIMIT OF RS12,00,00,000 AS ON APRIL 15, 2021QUANTUM OF TRANSACTIONS FOR CURRENT FY 20212022 IS DEBIT RS2773 CR IN CASH RS000 CR AND CREDITS RS2781 CR IN CASH RS000 CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FY  202021 IS DEBIT RS22863 CR IN CASH RS011 CR AND CREDITS RS22063 CR IN CASH RS000 CR 201920 IS DEBIT RS7641 CR IN CASH RS010 CR AND CREDITS RS7589 CR IN CASH RS007 CR 201819 DEBIT RS2976 CR IN CASH RS040 CR AND CREDITS RS2987 CR IN CASH RS003 CRCURRENT ACS WERE OPENED IN THE NAME OF A PARTNERSHIP FIRM MS PREMIUM PETRO PRODUCTS NOS 6813095059 AND 6813095042 ON 04SEP19 AT OUR BRANCH LOCATED IN 57 KRISHNA TOWER, BASEMENT, 1ST FLOOR, SARDAR PATEL MARGH JAIPUR RAJASTHAN  302001 THE ENTITY WAS FORMED ON 09MAY17 AND HAS SUBMITTED COPIES OF PAN AAUFP2067N, PARTNERSHIP DEED, CERT OF REG NO139742017, IEC AAUFP2067N AND GST REG CERT 08AAUFP2067N1ZF AS KYC DOCS AT THE TIME OF AC OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS 13 HATHROI MARKET AJMER ROAD, JAIPUR RAJASTHAN  302001 RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA AND LALIT KUMAR NAREDI ARE THE PARTNERS WHILE MANISH KUMAR BOHRA AND UJJWAL BHANDARI ARE THE UBOS IN THE SAID FIRM CUSTOMER HAS REGISTERED MOBILE NO 9829999990 WITH THE BANK THE ENTITY IS CATEGORISED AS HIGH RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED THAT THE ENTITY IS INTO BUSINESS OF PETROLEUM PRODUCTS WITH ANNUAL TURNOVER OF RS10000 CR BASIS INTERCONNECTEDNESS, OTHER ACS WERE IDENTIFIED WHICH COMPRISE OF 25 CURRENT ACS IN THE NAMES OF 9 REGISTERED COS, 1 NBFC, 8 PARTNERSHIP FIRMS, 2 LLP FIRMS AND 1 INDIVIDUAL CURRENT AC BESIDES 15 SAVINGS ACS WERE OPENED IN THE NAMES OF INDIVIDUALS AND 1 HUF DETAILS OF ACS ARE UPDATED IN XML FILE THE MENTIONED ACS WERE OPENED DURING THE PERIOD 06SEP09 TILL 04NOV20 AT ABOVE MENTIONED AJMER ROAD, RAJA PARK, NORTH AVENUE, VAISHALI NAGAR BRANCHES IN JAIPUR  RAJASTHAN AND SECTOR 17 VASHI BRANCH IN NAVI MUMBAI IT WAS INDICATED THAT THE ENTITIES ARE INTO VARIED BUSINESSES LIKE REAL ESTATE SERVICES, WHOLESALER OF CHARCOAL SODA  ASH, INFRASTRUCTURE, GENERAL TRADING, TEXTILE PRODUCTS, CONSTRUCTION, FINANCE AND INVESTMENT, DEALING IN STONESMARBLEGRANITE, TRADESALE OF HANDICRAFTS WITH ANNUAL TURNOVER IN THE RANGE OF RS001 CR TO RS400 CR RITIKA BOHRA, MANISH KUMAR BOHRA, UJJWAL BHANDARI, VISHAL BOHRA, LALIT KUMAR NAREDI AND BHEEM RAJ BOHRA ARE THE COMMON DIRECTORS PARTNERS KARTA IN THE ABOVE MENTIONED ACS WITH OTHER INDIVIDUALS IN DIFFERENT COMBINATIONS NOTICING HIGH VALUE ROUND SUM NEFTRTGS CREDITS FROM OWN AC AND SUBSEQUENT HIGH VALUE ADVANCE IMP REM THROUGH THE AC OF PREMIUM PETRO PRODUCTS 6813095059, AN ALERT WAS RAISED ON BRANCH TO CARRY OUT EDD AND THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION 1 ON REVIEWING AC TRANSACTIONS SINCE APRIL 1, 2020  APRIL 15, 2021 IT WAS NOTED THAT IMPSNEFTRTGS CREDITS AMOUNTED TO RS17824 CR MAINLY FROM PREMIUM PETRO PRODUCTS RS14208 CR, IDBI  00131020000080662, VCI EXPORTS P LTD RS309 CR, IDFC 10039338826, SURYA NAGARI TRADING RS297 CR, YES BANK 004163700002899, ABHA PRECISION FARMING LLP RS220 CR, IDBI 0013102000077552 AND BHEEM RAJ BOHRA RS202 CR, IDBI 0131063538000 2 EXPORT ADVANCE REMITTANCES AMOUNTED TO RS124 CR IN THE ACS OF INDIA STONE HOUSE LLP AND VCI EXPORTS P LTD MAINLY FROM WONDERSTONE LLC  USA RS029 CR, TRANQUILLO GMBH RS021 CR, GRAMACO SOURCE AND MARBLE RS018 CR, STONE SOURCE INTERNATIONAL INC USA RS014 CR AND STONELAND INC  USA RS010 CR AS PER WEB CHECK WONDERSTONE LLC USA IS INTO BUSINESS OF JEWELLERY STORES HTTPSWWWDNBCOMBUSINESSDIRECTORYCOMPANYPROFILESWONDERSTONE LLC4D21F25E0AAE301E36AA8061651D299FHTML, TRANQUILLO GMBH IS INTO CLOTHING STORE INDUSTRY, STONE SOURCE INTERNATIONAL INC IS INTO BUSINESS OF STONES GRANITES AND STONELAND INC IS INTO GRANITE MARBLE BUSINESS IT APPEARS THAT INDIA STONE HOUSE LLP WHICH HAS INDICATED THAT IT IS INTO BUSINESS OF STONES GRANITES MAY HAVE RECEIVED EXPORT PROCEEDS FROM ENTITIES IN UNRELATED BUSINESS LIKE WONDERSTONE LLC USA JEWELLERY STONES AND TRANQUILLO GMBH CLOTHING STORE INDUSTRY AND SUCH TRANSACTIONS MAY HAVE NO ECONOMIC RATIONALE',
  {'entities': [(123, 151, 'FOREIGN ORG'),
    (163, 177, 'FOREIGN ORG'),
    (220, 245, 'FOREIGN ORG'),
    (220, 245, 'FOREIGN ORG'),
    (257, 282, 'FOREIGN ORG'),
    (293, 314, 'FOREIGN ORG'),
    (328, 343, 'FOREIGN ORG'),
    (6435, 6450, 'FOREIGN ORG'),
    (6621, 6636, 'FOREIGN ORG'),
    (6724, 6739, 'FOREIGN ORG'),
    (7120, 7135, 'FOREIGN ORG'),
    (6492, 6517, 'FOREIGN ORG'),
    (6466, 6481, 'FOREIGN ORG'),
    (6777, 6792, 'FOREIGN ORG'),
    (7161, 7176, 'FOREIGN ORG')]}),
 ('8 THE TRANSACTION PATTERN OBSERVED IN CUSTOMER S ACCOUNT IS RECEIPT OF HIGH VOLUME OF DOMESTIC INWARD RTGS FROM CUSTOMER S OWN ACCOUNT WITH AXIS BANK LTD, KHAR MUMBAI AND ICICI BANK LTD, ANDHERI MUMBAI AND FROM ENTITY NAMED SAWARTHIYA FILM PRODUCTION WHOSE TRANSACTIONS ARE FROM SAME IFSC AS CUSTOMER S ACCOUNT ICIC0000011 IE, ANDHERI, MUMBAI IMMEDIATELY FOLLOWED BY FOREIGN OUTWARD REMITTANCES THEY SEEM TO BE RELATED PARTIES FOLLOWING EXACT SAME TRANSACTION ROUTE OF SENDING DOMESTIC RTGS THERE ARE NO DETAILS FOUND ON SAWARTHIYA FILM PRODUCTION ON PUBLIC DOMAIN EXCEPT A SMALL SNIPPET FOUND ON SOCIAL MEDIA THAT THEY ARE COPRODUCERS TO OUR CUSTOMER ON ONE OF THE PROJECTS REFER LINK HTTPS WWWFACEBOOKCOMWATCH?V=421845292404795 SAWARTHIYA FILM PRODUCTION IFSC ICIC0000011 HAS SENT RTGS TO OUR CUSTOMER AMOUNTING TO INR 42552 BETWEEN 15TH MARCH 21 AND 16TH APRIL 21 CUSTOMER HAS SENT BACK INR 2448 LAKHS TO SAWARTHIYA FILM PRODUCTION AT IFSC ICIC0001352, NEW DELHI IN MARCH 21 THESE TRANSACTIONS SEEMS TO HAVE NO ECONOMIC RATIONALE, NOR THERE SEEMS TO BE RATIONALE BEHIND OPENING OF ACCOUNTSGETTING BANKING SERVICES AT VARIOUS BANKS AND LOCATIONS9 CUSTOMER HAS A FACEBOOK PAGE, YOUTUBE CHANNEL WITH FEW VIDEOS AND THERE ARE NEWS ARTICLES ON THE PRODUCTION OF THE VIDEOS WITH SOCIAL MEDIA INFLUENCERS OR RELATIVELY KNOWN ARTISTS HOWEVER, THE NUMBER OF FOLLOWERS OR LIKES OR VIEWS PROJECT THE EARNINGS A COMPANY COULD MAKE AND THE SAME DOESN T SEEM TO BE IN TANDEM WITH THE VOLUME OF OUTWARD REMITTANCES MADE REFER LINK HTTPS WWWFORBESCOMSITESJROSE20190321HOWMUCHDOYOUTUBERSREALLYMAKE?SH=637AB9227D2B, HTTPS MINTINTUITCOMBLOGRELATIONSHIPSHOWMUCHDOYOUTUBERSMAKE5035# ~ TEXT=WITH%20THE%20AVERAGE%20YOUTUBE%20PAY,FOR%20EVERY%201%2C000%20VIDEO%20VIEWS,HTTPS WWWSANTABANTACOMBOLLYWOODMANISHKUMARSINGHANDSAPNAKUMARIGRACETHEMAASHUKASONGLAUNCHEVENTPRESENTEDBYIMAGINARYFILMSINCOLLABORATIONWITHTAGDOGMEDIA, HTTPS INDUSDAILYHUNTINNEWSINDIAURDUGOSSIPGANJEPAPERGOSGANENMANISHSINGHANDSAPNASIMAGINARYFILMSRELEASESAGROOVYCHARTBUSTERMAASHUKAFEATURINGBHAVINBHANUSHALINEWSIDN252433968?PGS=N AND PGN=0 AND SNIPPET FROM ONE OF THE ARTICLES FORBES ALSO ESTIMATES THAT FOR TOP TALENT, A YOUTUBER CAN MAKE ABOUT $5 FOR EVERY 1,000 VIDEO VIEWS AND CUSTOMER HAS ONLY ~19 MN VIEWS OVERALL WITH TOTAL 5 VIDEOS UPLOADED IN LAST 3 MONTHS CUSTOMER HAS MINIMAL LEGITIMATE WORK AND IT IS NOT IN LINE WITH THE INFERRED EARNINGS ONE COULD MAKE IN THAT LINE OF ACTIVITY THROUGH YOUTUBE AND THE TRANSACTIONS THAT CUSTOMER IS MAKING, THIS WAY WE ARE UNABLE TO GAUGE THE SOURCE OF WEALTH ALSO, UNABLE TO UNDERSTAND THE SOURCE OF FUNDS SINCE CUSTOMER IS ROUTING RTGS FROM VARIOUS DOMESTIC BANK ACCOUNTS TO OUR BANK10 THE ANNUAL TURNOVER CLAIMED BY THE CUSTOMER WITH OUR BANK IS NOT IN TANDEM WITH THE ACTUAL TURNOVER SEEN IN THE ACCOUNT IN A SINGLE QUARTER11 CUSTOMER BEING NEWLY INCORPORATED HAS OPENED ACCOUNTS IN MULTIPLE BANKS ICICI BANK, AXIS BANK AND OUR BANK WHICH RAISES SUSPICION ON THE INTENT AND THE PURPOSE12 REQUESTING TO FILE STR BASIS BELOW POINTS A THE INVOICE OF ONE OF THE OVERSEAS PARTIES IS NOT GENUINEB ALL RELATED PARTIES AND ACCOUNTS ARE RELATIVELY NEWLY INCORPORATEDC UNABLE TO GAUGE THE SOURCE OF WEALTH AND SOURCE OF FUNDSD THE OPENING OF ACCOUNTS IN VARIOUS BANKS AND THEN PROCEEDING WITH THE TRANSACTIONS IN A PATTERN THAT MAKES NO ECONOMICAL SENSEE THE ACTUAL TURNOVER IS HIGHER THAN THE CLAIMED ANNUAL TURNOVER, TURNOVER IN A SINGLE QUARTER HAS CROSSED THE CLAIMED ANNUAL TURNOVERPART A GROUNDS OF SUSPICION 1CUSTOMER IMAGINARY FILMS PRIVATE LIMITED AC 59207039548313 IS A NEWLY OPENED ACCOUNT WHO HAS REMITTED HUGE VOLUME OF FOREIGN OUTWARD REMITTANCES IN A SHORT PERIOD OF TIME2 CUSTOMER IMAGINARY FILMS PRIVATE LIMITED MAINTAINS CURRENT ACCOUNT NUMBER 59207039548313 WITH OUR KHAR WEST, MUMBAI BRANCH SINCE 15012021 PAN NUMBER IS UPDATED AS AAFCI5545Q,MOBILE NUMBER IS UPDATED AS 917039548313PROFILE DECLARED IS FILM PRODUCTION WITH ANNUAL TURNOVER OF INR 600 LAKHS EMAIL ID UPDATED IS INFO@IMAGINARYFILMSIN DATE OF INCORPORATION IS 22012020 GSTIN IS 27AAFCI5545Q1ZWRISK PROFILE IS MEDIUMBALANCE IN THE ACCOUNT AS ON 17TH APRIL 2021 IS INR 16417441THE CUSTOMER IS A PRIVATE LIMITEDMAILING AND PERMANENT ADDRESS OF THE CUSTOMER IS CO IMAGINARY FILMS PRIVATE LIMITED 402 EMBASSY CHAMBERS 3RD ROAD KHAR WEST, MUMBAI, MAHARASHTRA 400052 AUTHORISED SIGNATORIES ARE SAPNA KUMARI AND MANISH KUMAR SINGH BENEFICIAL OWNERS ARE SAPNA KUMARI AND MANISH KUMAR SINGH3 SCRUTINY OF ACCOUNT STATEMENT OF IMAGINARY FILMS PRIVATE LIMITED AC NO 59207039548313 FOR PERIOD SINCE JAN2021 ACCOUNT OPENING DATE TILL DATE REVEALED THAT CUSTOMER HAS RECEIVED A TOTAL CREDIT OF INR 130577 LAKHS OUT OF THE TOTAL CREDITS OF INR 130577 LAKHS, RTGSNEFTUPIIMPS CREDITS ARE INR 130326 LAKHS FROM CUSTOMER S OWN ACCOUNT IN ICICI BANK AND AXIS BANK AND FROM ANOTHER ENTITY NAMED SAWARTHIYA FILM PRODUCTION WHICH TOO IS IN ICICI BANKMAJOR DEBITS HAVE BEEN OBSERVED VIA FOREIGN OUTWARD REMITTANCES OF INR 127954 LAKHS TO JUST 2 OVERSEAS PARTIES BASED OUT OF SINGAPORETHESE AFOREMENTIONED DEBITS ARE OUT OF TOTAL DEBITS OF INR 130412 LAKHS4 NO LINKED ACCOUNTS FOUND EMAIL ID, MOBILE NO AND PAN WERE CHECKEDPART B DETAILS OF INVESTIGATION5 CUSTOMER HAS MADE FOREIGN OUTWARD REMITTANCES AMOUNTING TO INR 127854 LAKHS AGAINST PURPOSE CODE S1101AUDIOVISUAL AND RELATED SERVICES LIKE MOTION PICTURE AND VIDEO TAPE PRODUCTION, DISTRIBUTION AND PROJECTION SERVICES TO JUST TWO OVERSEAS PARTIES NAMED LIVING FILMS PTE LTD, SG AND GATE STUDIOS PTE LTD, SG, WITHIN 3 MONTHS OF OPENING THE ACCOUNT WITH OUR BANK 6 THE OVERSEAS PARTIES WHEN CHECKED ON PUBLIC DOMAIN, IT WAS FOUND THAT THEY TOO ARE NEWLY INCORPORATED LIVING FILMS PTE LTD HAS BEEN INCORPORATED ON 18TH SEPTEMBER 2020, REFER LINK HTTPS WWWSGPBUSINESSCOMCOMPANYLIVINGFILMSPTELTD AND GATE STUDIOS PTE LTD HAS BEEN INCORPORATED ON 1ST OCTOBER 2020, REFER LINK HTTPS SINGAPORECORPCOMCOGATESTUDIOSPTELTD OUR CUSTOMER TOO HAS BEEN INCORPORATED IN 2020, IE 22ND JANUARY 20207 THERE IS AN ADDITIONAL INFORMATION FOUND WRT LIVING FILMS PTE LTD ON PUBLIC DOMAIN SEARCH, IT WAS FOUND THAT THERE IS A RELATIVELY KNOWN COMPANY BY THE SAME NAME IN THAILAND, REFER LINK HTTPS ENWIKIPEDIAORGWIKILIVINGFILMS, HTTPS WWWLIVINGFILMSCOM THE CUSTOMER HAS SUBMITTED INVOICE WHEREIN LIVING FILMS PTE LTD, SG CLAIMS THAT LIVING FILMS, THAILAND IS THEIR HEADOFFICE AND HAS MENTIONED ITS GEOGRAPHICAL ADDRESS AND EMAIL ID FROM LIVING FILMS, THAILAND S WEBSITE HEREON WOULD BE MENTIONED AS WEBSITE , HOWEVER ON OBSERVING THE INVOICE CLOSELY IT IS FOUND THAT THE PINCODE GIVEN ON WEBSITE IS 50100 BUT IN INVOICE IT IS 501100 AND EMAIL ADDRESS ON WEBSITE IS LIVING@LIVINGFILMSCOM BUT IN INVOICE IT IS LIVING@LIVINGFILMCOM, THE CONTACT NUMBERS IN THE INVOICE ALSO DO NOT MATCH WITH THE ONES GIVEN ON THE WEBSITE, ALSO ONE OF THE NUMBERS 85292124533, 852 BELONGS TO HONGKONG WHERE AS FOR SINGAPORE IT IS 65 AND FOR THAILAND IT IS 66 REFER LINK HTTPS ENWIKIPEDIAORGWIKITELEPHONENUMBERSINHONGKONG, HTTPS ENWIKIPEDIAORGWIKITELEPHONENUMBERSINSINGAPORE, HTTPS ENWIKIPEDIAORGWIKITELEPHONENUMBERSINTHAILAND AND THERE IS NO MENTION OF SINGAPORE BRANCH ON THE LIVING FILMS, THAILAND S WEBSITE THE INVOICE DOES NOT SEEM GENUINE AND SEEMS TO BE FRAUDULENT THE AGREEMENTS WRT BOTH THE OVERSEAS PARTIES HAVE NO MENTION OF ANY STAKEHOLDER AND IT IS EXACTLY THE SAME',
  {'entities': [(5477, 5497, 'FOREIGN ORG'),
    (5790, 5810, 'FOREIGN ORG'),
    (5448, 5468, 'FOREIGN ORG'),
    (5660, 5680, 'FOREIGN ORG'),
    (6022, 6042, 'FOREIGN ORG'),
    (6267, 6287, 'FOREIGN ORG')]}),
 ('FOR EXAMPLE, VIOLATION OF LOSS IN FUNDS OUTLAY OF MORE THAN 120 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON 17022020 USD 2005LACS WAS REMITTED TO CITY PULSE GENERAL TRADING LLC IN UAE0007NMDC0046420 NO PROPER DETAILS ABOUT THE CLIENT IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS27132000BITUMEN HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS IS STILL DUE IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY OF MORE THAN 120 DAYS, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON 24012020 USD 150 LACS TO CITY PULSE GENERAL TRADING LLC IN UAE0007NMDC0046420 NO PROPER DETAILS ABOUT THE CLIENT IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS27132000BITUMEN GRADE THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS DUE IN TOTAL 5 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF 754LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDKAPSON EXIM PRIVATE LIMITED, PRIVATE LTD, HAVING ADDRESS403,BAJAJ HOUSE 97,4TH FLR,NEHRU PLACE,, SOUTH DELHI, NEW DELHI, DELHI, INDIA, 110019, CONTACT NUMBER 919953111443 AND EMAIL ID KAPSONEXIM93@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 418005000214 WITH ICICI BANK LTD, DELHI  19  NEHRU PLACE,19, CHAWLA HOUSE, NEHRU PLACE, DELHI, 110019, SINCE 21022019 DATE OF INCORPORATION IS 23081993 PAN NUMBER OF THE ACCOUNT IS AAACK0464B IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0593052391 GSTIN AS PER BANK RECORDS IS 07AAACK0464B1Z4 OTHER RELATIONSHIP OF CUSTOMER ARE 006105018350, 418005000699, 418006000002 TRANSACTION LINKED ACCOUNTS ARE 418010000315, 418010000421 AUTHORIZED SIGNATORY AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ANKITA BANSALAFBPB9149M, MUKESH KUMAR JINDALAOFPJ6342H THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, GSTIN, COI U74899DL1993PTC054937 KAPSON EXIM PRIVATE LIMITED IS ENGAGED IN THE BUSINESS OF SELLING ALL KINDS OF COPPER AND BRASS SCRAPS AS PER ETRG FEEDBACK AND DEDUPE FOR PROMOTERS OF THE COMPANY HAVE BUSNIESS OF MTT IN BITUMEN AND TARCOAL THEY HAVE FIRM IN NAME OF DTACH INNOVATION418005500023 DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS5 CR TO RS25 CRALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS104 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 56217 LAKHS OF WHICH CASH WAS RS 050 LAKHS MAJORLY IN DELHI  19, NEHRU PLACE BRANCH, TRANSFERS RS 100 LAKH MAJORLY FROM KAPSON EXIM PRIVATE LIMITED, NEFT RS 242 LAKHS MAJORLY FROM KAPSON EXIM PVT LTD,KAPSON EXIM PRIVATE LIMITED, RTGS RS 30180 LAKHS MAJORLY FROM ANKITA BANSAL,KAPSON EXIM PVT LTD, REMITTANCE RS 24939 LAKHS, MOBILE MONEY TRANSFERS RS 306 LAKHS IN 2 INSTANCES TOTAL DEBIT SIDE RS 56790 LAKHS OF WHICHTRANSFERS RS 817 LAKHS MAJORLY TO KAPSON EXIM PRIVATE LIMITED, CLEARING RS 079 LAKHS MAJORLY TO BSES RAJDHANI POWER LTD,RAVINDER PANDITA,BIF BUILDING MAINTENAN, RTGS RS 29190 LAKHS MAJORLY TO KAPSON EXIM PRIVATE LI,BANYAN TREE INVESTMENT,KAPSON EXIM PRIVATE LTD, REMITTANCE RS 25988 LAKHS MAJORLY TO CITY PULSE GENERAL TRADING LLC,SIGNIFY BUILDING MATERIALS TRADING,SYSKEP DMCC, INTERNET FUND TRANSFER RS 005 LAKHS, MOBILE MONEY TRANSFERS RS 658 LAKHS IN 21 INSTANCESFY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 42797 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 19661 LAKHS  TOTAL DEBIT SIDE RS 45200 LAKHS OF WHICH CASH WAS RS 100 LAKH, REMITTANCE RS 15693 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 3203 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 123 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 023 LAKHS  VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE FUNDS OUTLAY OF MORE THAN 120 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 5 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 754LACS',
  {'entities': [(185, 215, 'FOREIGN ORG'),
    (672, 702, 'FOREIGN ORG'),
    (3682, 3712, 'FOREIGN ORG'),
    (3713, 3747, 'FOREIGN ORG')]}),
 ('HTTPWWWCORPINESSCOMINFOEXPORTERSLISTINGFAISHINGINDUSTRIALHKCOLTD IN HONGKONG WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 85235910 TAGS HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED ON 18022020 FROM FUNSTATION LIMITEDFUNSTATION ARE FAMILY ENTERTAINMENT CENTRES WITH CARNIVAL GAMES, LASER TAG, ADVENTURE GOLF, BOWLING, VR AND MUCH MORE FUN GAMES HTTPSWWWFUNSTATIONUKCOM IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 14072027 TO 03072018 TO TUNE OF USD2645 TO CARDY INTELLIGENT TECHNOLOGY CO LTD CUSTOMER IS INTO MANUFACTURE OF RFID TAGS, WRISTBAND ETCHTTPSWWWSMARTCARDYCOM WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 85235910 RFID CARDS HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 20092019 AND 20012020 TO THE TUNE OF USD189 LACS FROM BAHJAT BAHLA BAHLA AL MAMOORAH DETAILS OF THE WHOM IS NOT AVAILABLE IN TOTAL 23 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD281 LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDSEMNOX SOLUTIONS PVT LTD, PRIVATE LTD, HAVING ADDRESS3RD FLOOR FLR,MG ROAD LALBAGH,MANGALORE,PUNJA BUILDING ANNEXE, MANGALORE, KARNATAKA, INDIA, 575003, CONTACT NUMBER 008244255888 AND EMAIL ID IQBAL@SEMNOXCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 001406000019 WITH ICICI BANK LTD, MANGALORE  P M RAO ROAD,BHARATH BUILDING,GROUND FLOOR, PMRAO ROAD,MANGALORE, KARNATAKA, 575001, SINCE 03102011 DATE OF INCORPORATION IS 21022008 PAN NUMBER OF THE ACCOUNT IS AALCS7710B IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0708003036 TRANSACTION LINKED ACCOUNTS ARE 603091010708 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE KIRAN KARANKI AUTHORISED SIGNATORY, SAMPATH KUMAR DIRECTOR, IQBAL MOHAMMAD DIRECTOR AAHPI5891A ,RAJEEV GOPALAKRISHNAN DIRECTOR, BALAJI VENUGOPAL DIRECTORBENEFICIAL OWNER OF THE REPORTED ACCOUNT IS KIRAN KARANKADZPK4808Q,MOHAMMAD IQBALAAHPI5891A THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI DATE OF LAST KYC UPDATION 13052019SEMNOX SOLUTIONS PRIVATE LIMITED SSPL IS ENGAGED IN THE BUSINESS OF PROVIDING IT SOLUTIONS FOR THE ENTERTAINMENT AND LEISURE INDUSTRY SSPL OFFERS SOLUTIONS SUCH AS FAMILY ENTERTAINMENT, WATER AND THEME PARKS, AND FOOD AND BEVERAGES THE COMPANY CATERS TO WATER PARKS, MUSEUMS, KIDS PLAY AREAS, RESTAURANTS, FOOD COURTS, AND AQUARIUMS, AMONG OTHERS SSPLS FEC AND PARK CLIENTS INCLUDE SUCH AS COWPLAY CPWMOO, SINGAPORE, ARENA REKREASI KELUARGA, AEON FANTASY, MALAYSIA, AND SPARKS, INDONESIA, AND CAMAYA COAST, PHILIPPINES, AMONG OTHERS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 5CR TO 25CRALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS061 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 1766 LAKHS OF WHICH TRANSFERS RS 015 LAKHS, REMITTANCE RS 1751 LAKHS MAJORLY FROM SEMNOX SOLUTIONS LLC 11498 LU,AEON FANTASY MALAYSIA SDN BHD,SEMNOX SOLUTIONS DMCC P O BOX TOTAL DEBIT SIDE RS 1975 LAKHS OF WHICH TRANSFERS RS 475 LAKHS MAJORLY TO SEMNOX SOLUTIONS PRIVATE LIMITED, REMITTANCE RS 1449 LAKHS MAJORLY TO FAISHING INDUSTRIAL HK CO LIMITED,TITOMA MANUFACTURING LTD,AMAZON WEB SERVICES, INC FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 5546 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 5546 LAKHS TOTAL DEBIT SIDE RS 6127 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 6127 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 4627 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 4627 LAKHS TOTAL DEBIT SIDE RS 5220 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 5220 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 6668 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 6668 LAKHS TOTAL DEBIT SIDE RS 6137 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 5181 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS, LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR 23 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF US281LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON 280820218 USD 3575 REMITTED TO FAISHING INDUSTRIAL HK CO LTDTHIS COMPANY DEALS IN EXPORT, UNDER DISCS, TAPES, SOLIDSTATE NONVOLATILE STORAGE DEVICES, SMART CARDS AND OTHER MEDIA FOR THE RECORDING OF SOUND OR OF OTHER PHENOMENA, WHETHER OR NOT RECORDED, INCLUDING MATRICES AND MASTERS FOR THE PRODUCTION OF DISCS, BUT EXCLUDING PRODUCTS OF CHAPTER 37 CATEGORY',
  {'entities': [(3533, 3553, 'FOREIGN ORG'),
    (3767, 3800, 'FOREIGN ORG'),
    (661, 696, 'FOREIGN ORG')]}),
 ('LINKS ETCHTTPSWWWFUJIFILMEUUKABOUTUSCOMPANYPROFILEFUJIFILMSPECIALITYINKSYSTEMSLTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS 32151940 SCREEN PRINTING INKS HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED BETWEEN 30052017 TO 30062020 TO TUNE OF GBP135LACS ,USD234LACS TO CEYLON BUSINESS APPLIANCES PVT LTDENTITY DEALS IN EXPORT OF BANKING PRODUCTSKIOSKS, COUNTING MACHINEETC, POTOCOPIERSHTTPWWWCBALK,STRETCHLINE PVT LTDLEADING THE WAY AS THE WORLDS LARGEST MANUFACTURER OF NARROW ELASTICATED FABRIC AND A MAJOR SUPPLIER OF COVERED ELASTOMERIC YARN AND TEXTILE COATING AND BONDINGHTTPSSTRETCHLINECOM ETC WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS PIONEER TO SPORTS WEAR,PRINTING INK IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS WHERE, HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED ON 02032020 TO TUNE OF USD 12830 FROM HI FASHION HOLDING PVT LTD DEALS I N TEXTILE BUSINESS HTTPSWWWHIFASHIONLK HOWEVER THE LINKED IMPORT PAYMENTS ARE STILL DUE IN TOTAL 134 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 287LACS, GBP 238LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF PIONEER TO SPORTS WEAR,PRINTING INK IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDFUJIFILM SERICOL INDIA PVT LTD, PRIVATE LTD, HAVING ADDRESS1011,BBHANDARI INDUSTRIAL ESTATE, SANASWADI TALUKA SHIRUR, PUNE, MAHARASHTRA, INDIA, 412208, CONTACT NUMBER 917030952710 AND EMAIL ID PRASHANTBARI@FUJIFILMCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000506000141 WITH ICICI BANK LTD, PUNE  BUND GARDEN,A WING SHANGRILA GARDENS BUND GARDEN ROAD, PUNE, MAHARASHTRA, 411001, SINCE 22122004 DATE OF INCORPORATION IS 19021996 PAN NUMBER OF THE ACCOUNT IS AABCS0098L IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0396015123 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE PRAVIN SHARAD KHAPARDE AUTHORISED SIGNATORY, PRASANNA RAGHAVRAO MOTUPALLI AUTHORISED SIGNATORY AATPM0665L BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS NARENDRA AGNIHOTRIADUPA6988J,SREEHARI SREEDHARADAPS0160N THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI DATE OF LAST KYC UPDATION 13022019 ENTITY IS PRIMARILY ENGAGED IN THE MANUFACTURING AND MARKETING OF PRINTING INKS, THINNERS AND ADDITIVES FSIPL ALSO TRADES DIGITAL MACHINES THE COMPANYS PRODUCT RANGE INCLUDES SCREEN INK GRAPHIC INK, INDUSTRIAL INK, PACKAGING INK, TEXTILE INK, CHEMICAL AND EMULSION, DIGITAL INK UV INKS AND SOLVENT INKS, NARROW WEB INK UV FLEXO INKS, UV FLAT SCREEN INKS AND UV ROTARY SCREEN INKS, OFFSET INK DRY OFFSET UV INK AND LETTERPRESS UV INK AND SUNGLO CHANNELS INKS SUNGLO GRAPHIC INK AND SUNGLO INDUSTRIAL INKALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS019 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS 012 LAKHS OF WHICH REMITTANCE RS 012 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 002 LAKHS OF WHICH REMITTANCE RS 002 LAKHS MAJORLY TO FUJIFILM SPECIALITY INK, FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 147 LAKHS OF WHICH REMITTANCE RS 147 LAKHS MAJORLY FROM FUJIFILM SPECIALITY INK SYSTE,THREE SQUARE TECHNOLOGY LIMIT,FOR YOU INDUSTRY CO,LTD TOTAL DEBIT SIDE RS 147 LAKHS OF WHICH REMITTANCE RS 144 LAKHS MAJORLY TO FUJIFILM SPECIALITY INK,FUJIFILM SPECIALITY INK SYSTEMS,FUJIFILM SERICOL INDIA PVT LTD FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 205 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 205 LAKHS TOTAL DEBIT SIDE RS 228 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 228 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 305 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 305 LAKHS TOTAL DEBIT SIDE RS 335 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 335 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 269 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 269 LAKHS TOTAL DEBIT SIDE RS 346 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 346 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 134 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 287LACS, GBP 238LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN 270 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN 29062017 TO 24012018 GBP 159 LACS REMITTED IN PARTS FROM FUJIFILM SPECIALITY INK SYSTEMS LTD ENTITY IS INT SPECIALTY CHEMICALS,',
  {'entities': [(4931, 4966, 'FOREIGN ORG'),
    (3446, 3469, 'FOREIGN ORG'),
    (3697, 3720, 'FOREIGN ORG'),
    (3855, 3878, 'FOREIGN ORG'),
    (3879, 3902, 'FOREIGN ORG'),
    (4931, 4954, 'FOREIGN ORG')]}),
 ('IN TOTAL 3 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 2549LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINE TOOL AND ACCESSORIES IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDTHERMAX BABCOCK WILCOX ENERGY SOLUPLTD,PRIVATE LTD, HAVING ADDRESSADISA,ICON,GAT NO FLR,3012,, BANGALORE HIGHWAY,BAVDHAN BUDRUK,OPPOSITE HEMRL,MUMBAI, PUNE, MAHARASHTRA, INDIA, 411021, CONTACT NUMBER 919850891016 AND EMAIL ID RAMNATHNAYAK@TBWESCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000505019247 WITH ICICI BANK LTD, PUNE  BUND GARDEN,A WING SHANGRILA GARDENS BUND GARDEN ROAD, PUNE, MAHARASHTRA, 411001, SINCE 18082010 DATE OF INCORPORATION IS 26062010 PAN NUMBER OF THE ACCOUNT IS AADCT5273A IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0310065542 GSTIN AS PER BANK RECORDS IS 27AADCT5273A1ZJ TRANSACTION LINKED ACCOUNTS ARE 000510164069,000510164070AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NIRANJAN PAWGI AUTHORISED SIGNATORY ABKPP6008N, NIRANJAN PAWGI AUTHORISED SIGNATORY ABKPP6008N ,LATA KUMAR AUTHORISED SIGNATORY, ASHISH MOHAN LAL BHANDARI AUTHORISED SIGNATORY AEXPB3596F ,MOHAN LAL BHANDARI ASHISH AUTHORISED SIGNATORY AEXPB3596F  THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, MOA,AOA, COI THERMAX BABCOCK WILCOX ENERGY SOLUTIONS PRIVATE LIMITED TBWES IS A JOINT VENTURE BETWEEN THERMAX LIMITED, PUNE, INDIA AND BABCOCK WILCOX POWER GENERATION GROUP, INC, USA THE COMPANY IS ENGAGED IN MANUFACTURING SUBCRITICAL AND SUPERCRITICAL BOILERS FOR THERMAL POWER GENERATION THE COMPANYS OTHER PRODUCTS INCLUDE LOW NOX COMBUSTION SYSTEMS AND ROLL WHEEL PULVERIZERSALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS13612 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 9,08360 LAKHS OF WHICH, CLEARING RS 77179 LAKHS MAJORLY FROM ICICI PRUDENTIAL M, NEFT RS 35272 LAKHS MAJORLY FROM MAHARASHTRA VALUE ADDED TAX,UTI MUTUAL FUND,THERMAX LIMITED, RTGS RS 7,84038 LAKHS MAJORLY FROM UTI MF COMMON FUND,SBI MUTUAL FUND,DSP MUTUAL FUND REDEMP, INTERNET FUND TRANSFERS RS 5004 LAKHS AND OTHERS RS 6868 LAKHS TOTAL DEBIT SIDE RS 8,99220 LAKHS OF WHICH TRANSFERS RS 6,75300 LAKHS MAJORLY TO THERMAX BABCOCK WILCOX ENERGY SOLUPLTD, CLEARING RS 6543 LAKHS, RTGS RS 3540, REMITTANCE RS 1786 LAKHS MAJORLY TO MANTRA SWITCHGEAR CO LTD,LSIS CO LTD, INTERNET FUND TRANSFER RS 2,00000 LAKHS FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 6,35174 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 10366 LAKHS TOTAL DEBIT SIDE RS 6,37494 LAKHS OF WHICH RS 0 BY CASH , REMITTANCE RS 024 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 12,97286 LAKHS OF WHICH CASH WAS RS 87000, REMITTANCE RS 5,38250 LAKHS  TOTAL DEBIT SIDE RS 13,40179 LAKHS OF WHICH REMITTANCE RS 2,72800 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 21,00657 LAKHS OF WHICH CASH WAS RS 018 LAKHS, REMITTANCE RS 7,65074 LAKHS  TOTAL DEBIT SIDE RS 21,40545 LAKHS OF WHICH REMITTANCE RS 2,00098 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NO COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 3 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 2549LACS FOR EXAMPLE, MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN ON 16062020 USD 17900 CREDITED FROM LSIS CO LTD  AUTOMATION, PLC, ELECTRONIC DEVICES MANUFACTURING FROM SOUTH KOREA WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS85371000OTHERS HOWEVER RESPECTIVE INWARDS REMITTANCE TOWRADS EXPORTS RECEIVED FROM THE BABCOCK  WILCOX COMPANY FROM USA THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE',
  {'entities': [(322, 360, 'FOREIGN ORG'),
    (2557, 2595, 'FOREIGN ORG'),
    (2671, 2695, 'FOREIGN ORG'),
    (3701, 3724, 'FOREIGN ORG')]}),
 ('AND GOODS MENTIONED AS HS 90213900 ORTHOPAEDIC GOODS,HS 90213900 DRILLING MACHINE,HS 90213900 CONNECTING RODS HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED 14012019 TO 24062020 FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS  HTTPSWWWDNBCOMBUSINESSDIRECTORYCOMPANYPROFILESINNOVATIONORTHOLINELIMITEDC53A1BBABB68E37C869CE88A51209509HTML IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 04102019 AND 07032019 TO TUNE OF USD 306LACS TO JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS 90211000 DRILLING, HS 90213900 SELF DRILLING HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 11022019 AND 24062020 TO THE TUNE OF USD708LACS FROM INNOVATION ORTHO LINE LIMITED CLIENT DEALS IN OTHOPEDIC IMPLANTS ADN RELATED ITEMS  HTTPSWWWDNBCOMBUSINESSDIRECTORYCOMPANYPROFILESINNOVATIONORTHOLINELIMITEDC53A1BBABB68E37C869CE88A51209509HTML IN TOTAL 29 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD4690LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT,STR IS BEING FILEDAUXEIN MEDICAL PRIVATE LIMITED,PRIVATE LTD, HAVING ADDRESSPLOT NO 168169170,HSIIDC FLR,PHASE IV,KUNDLI SONIPAT, SONIPAT, HARYANA, INDIA, 131028, CONTACT NUMBER 09136220665 AND EMAIL ID ACCOUNTS@AUXEINMEDICALCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 016006000018 WITH ICICI BANK LTD, NEW DELHI  MODEL TOWN,H4, MODEL TOWN III, DELHI, 110009, SINCE 16012015 DATE OF INCORPORATION IS 03092012 PAN NUMBER OF THE ACCOUNT IS AAKCA9159K IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 3308000946 THERE ARE NO LINKED ACCOUNT AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE RAHUL LUTHRA AUTHORISED SIGNATORY, DEEPAK LUTHRA AUTHORISED SIGNATORY, GAURAV LUTHRA, ASHOK LUTHRABENEFICIAL OWNER OF THE REPORTED ACCOUNT IS DEEPAK LUTHRAACBPL8216A,RAHUL LUTHRAADYPL0242J THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, MOA, AOA, COI AS PER ENHANCED DUE DILIGENCE AUXEIN MEDICAL PRIVATE LIMITED AMPL IS ENGAGED IN MANUFACTURING OF ORTHOPEDIC IMPLANTS THE COMPANYS PRODUCTS INCLUDE MINI FRAGMENT, HIP PROSTHESIS, INTERLOCKING NAILS, INSTRUMENT AND IMPLANT BOX, SPINE IMPLANTS, MAXILLOFACIAL IMPLANTS, WISE LOCKING PLATES, AND BONE PLATES THE COMPANYS UNIT IS EQUIPPED WITH CNC MACHINES, DRILL MACHINES, COMPARATORS, CUTTING MACHINES AND SPECIAL PURPOSE MACHINES AMPL HAS A RANGE OF 600 IMPLANTS AVAILABLE IN 60 COUNTRIES IN ASSOCIATION WITH 100 PARTNERSALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS176 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS 009 LAKHS OF WHICH REMITTANCE RS 009 LAKHS MAJORLY FROM 1 FIXIER SA DE CV 2 CARR DR M,1 INNOVATION ORTHO LINE LTD TOTAL DEBITS DURING THE SAME PERIOD WAS RS 000 FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 8631 LAKHS OF WHICH REMITTANCE RS 8631 LAKHS MAJORLY FROM 1 ZOLDAN CORPOREISHON CIA L,1 MEDICAL ORTOVIT SRL 2 BUCUR,1 ATRON HEALTH S A 2 8 LEOFO TOTAL DEBIT SIDE RS 8642 LAKHS OF WHICH REMITTANCE RS 8629 LAKHS MAJORLY TO CHANGZHOU NUBE INTERNATIONAL,CHANGZHOU XIETONG INDUSTRIES CO,CHANGZHOU XIETONG INDUSTRIES CO LTD FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 8268 LAKHS OF WHICH REMITTANCE RS 8268 LAKHSTOTAL DEBIT SIDE RS 8489 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 8347 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 6803 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 6803 LAKHS  TOTAL DEBIT SIDE RS 6403 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 6403 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 160 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 160 LAKHS TOTAL DEBIT SIDE RS 160 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 160 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 29 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD469LACS FOR EXAMPLE, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN 27042018 TO 20122018 USD 1322 LACS REMITTED IN PARTS FROM JIANGSU BAIDE MEDICAL INSTRUMENT CO LTD MEDICAL COMPONESTS TRADING WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE,',
  {'entities': [(3239, 3254, 'FOREIGN ORG'),
    (207, 236, 'FOREIGN ORG'),
    (969, 998, 'FOREIGN ORG'),
    (3733, 3761, 'FOREIGN ORG'),
    (3762, 3790, 'FOREIGN ORG'),
    (3794, 3822, 'FOREIGN ORG')]}),
 ('OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS REFINED SUNFLOWER OIL 15121910 HOWEVER THE RESPECTIVE EXPORT WAS RECEIVED ON BETWEEN 15062020 AND 10072020 TO THE TUNE OF RS206LACS FROM DJT IMPORT EXPORT IN MADAGASCAR WHOSSE DETAILS ARE NOT FOUND IN PUBLIC DOMAIN IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN 270 DAYS, WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 27112018 AND 07022019 TO TUNE OF RS 4848LACS WAS REMITTED TO HEBEI LUEDING IMPORT AND EXPORT CO LTDCOMPANY DEALS IN PPGI, GALVANISED STEEL COIL, ROLL FORMING MACHINE, ROOFING SHEETS, WELDING ELECTRODEHTTPSWWWMACHINERYOFFERSCOMCOMPANYLEADINGROOFINGCONTACTHTML AND SHOUGUANG DASEN WOOD CO LTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED ASHS 44123190 PLYWOOD, GALVANIZED CORRUGATED SHEET, HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 19082019 AND 09062020 TO THE TUNE OF EUR 85707 IN TOTAL 508 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF 218CRORES WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS  CLIENT IS DEALING IN MERCHANT TRADE OF DIVERSIFIDE PRODUCTS OILS, GA SHEETS, WOOD ETC IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDTHAKKAR EXIM,PROPRIETORSHIP FIRM, HAVING ADDRESSTHAKKAR HOUSE,RAJDEV STREET, SANGANVA CHOWK, RAJKOT, GUJARAT, INDIA, 360001, CONTACT NUMBER 919825287634 AND EMAIL ID INFO@THAKKAREXIMCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 084805000210 WITH ICICI BANK LTD, RAJKOT  YAGNIK ROAD, SHOP NO 3 4, CONVENTION CENTER, NR HARIBHAI HALL,OFF YAGNIK, ROAD, RAJKOT, GUJARAT, 360002, SINCE 09102010 DATE OF INCORPORATION IS 27092000 PAN NUMBER OF THE ACCOUNT IS AAXPT8747Q PAN LINKED ACCOUNTS ARE 084806000003, 084806000038 IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 2400002045 TRANSACTION LINKED ACCOUNTS ARE 084805000369, 331105500284 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE NITESH HARIDAS THAKKAR PROP AAXPT8747Q ,NITESH HARIDAS THAKKAR AUTHORISED SIGNATORY AAXPT8747Q THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN,MOA,AOA,IE CODE DATE OF LAST KYC UPDATION 07032019 CUSTOMER IS INTO IMPORT, EXPORT AND MERCHANTING TRADER HAVING BEGUN WITH ONLY ONE TEXTILE PRODUCT, THE COMPANY HAS NOW WELL DEVELOPED INTO AN STAR EXPORT HOUSE DEALING IN VARIOUS PRODUCTS LIKE AGRICULTURAL COMMODITIES SUCH AS RICE OF DIFFERENT VARIETIES, WHEAT FLOUR, BAKERY FLOUR, BISCUITS, PLASTIC HOUSEHOLD PRODUCTS, KITCHEN APPLIANCES AND HOME APPLIANCES FIRM IS HAVING WEBSITE WWWTHAKKAREXIMCOM ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 28022021 WAS RS1144 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 2,17784 LAKHS OF WHICH TRANSFERS RS13449 LAKHS MAJORLY FROM NITESH HARIDAS THAKKAR,THAKKER ELECTRICALS,THAKKAR EXIM,CLEARING RS 4035 LAKHS, NEFT RS 4071 LAKHS MAJORLY FROM THAKKAR EXIM,GOPALSWAMY BASKARAN,E PAO GST REFUNDS, RTGS RS 31435 LAKHS MAJORLY FROM THAKKAR EXIM,PMS ENTERPRISES,ARTI INTERNATIONAL, REMITTANCE RS 1,53376 LAKHS MAJORLY FROM THAKKAR EXIM,THAKKAR EXIM 1 RAJDEV STREET,SOCIETE RAG, INTERNET FUND TRANSFERS RS 10840 LAKHS, MOBILE MONEY TRANSFERS RS 565 LAKHS IN 10 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 2,17574 LAKHS OF WHICH CASH WAS RS 472 LAKHS BY BRANCH, TRANSFERS RS 12083 LAKHS MAJORLY TO THAKKAR EXIM,BHAVNA MAHESH PARMAR,PARMAR RAJDEEP MAHESH, CLEARING RS 84439 LAKHS, RTGS RS 1,18072 LAKHS MAJORLY TO VIRAJ IMPEX, REMITTANCE RS 1579 LAKHS MAJORLY TO HEBEI TOMATO INDUSTRY CO LTD FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 1,70794 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 1,70066 LAKHS OF WHICH CASH WAS RS 1066 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 2,47581 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 2,16502 LAKHS TOTAL DEBIT SIDE RS 2,48097 LAKHS OF WHICH CASH WAS RS 1265 LAKHS, REMITTANCE RS 2304 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 2,74960 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 2,10441 LAKHS  TOTAL DEBIT SIDE RS 2,74765 LAKHS OF WHICH CASH WAS RS 1166 LAKHS, REMITTANCE RS 954 LAKHSTHE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER 084805000210 IN THE NAME OF THAKKAR EXIM, REPORT SERIAL NUMBER 300446 VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS, HAS BEEN NOTICED IN THE ACCOUNT FOR 508 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF 218CRORES FOR EXAMPLE, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON 14032017 TO THE TUNE OF USD9729 TO OIL TEC FOR OILS AND DETERGENTS MANUFACTURER OF SOAPS ADN OILS HTTPSWWWGULFOODCOMEXHIBITORSOILTECFOROILSANDDETERGENTS IN EGYPT WITH PURPOSE',
  {'entities': [(3874, 3902, 'FOREIGN ORG'),
    (545, 583, 'FOREIGN ORG'),
    (747, 774, 'FOREIGN ORG')]}),
 (' ENTITY DEALS IN AUTO COMPONENTS MANUFACTURE AND SALE HTTPSWWWTRADESNSCOMENGLISHMB87DGDN1 WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS87088000UNIVERSAL JOINT HOWEVER RESPECTIVE INWARDS REMITTANCE TOWARDS EXPORTS RECEIVED USD 12016 RECEIVED BETWEEN 31122019 AND 13052020 FROM MOTOXPARTS DISTRIBUCIONES SRL IN ARGENTINA DETAILS OF WHOM IS NOT AVAILABLE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE FOR TOWARDS IMPORT WAS DEBITED ON 14112019 USD 7850 REMITTED IN PARTS TO HANGZHOU VALE TRADING CO LTDHANGZHOU VALE TRADING CO, LTD SPECIALIZED TRADING COMPANY OF MOTORCYCLE PARTS, CAR PARTS, HARDWARE, MECHANIC PRODUCT, ELECTRONICS  HTTPSWWWEWORLDTRADECOMCVALETRADING IN HONGKONG WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS87141090MOTERCYCLE PARTS THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT RECEIVED USD 12016 RECEIVED BETWEEN 31122019 AND 13052020 FROM MOTOXPARTS DISTRIBUCIONES SRL IN ARGENTINA IN TOTAL 30 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 351LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF SILICONE IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDSEVEN IMPEX,PARTNERSHIP FIRM,HAVING ADDRESSSCO48 FIRST FLOOR,ROYALE ESTATE, ZIRAKPUR, PUNJAB, INDIA, 140603, CONTACT NUMBER 918283841575 AND EMAIL ID FINANCE@SEVENIMPEXORG IS MAINTAINING A CURRENT ACCOUNT NUMBER 632205500590 WITH ICICI BANK LTD, CHANDIGARH  SECTOR 35C,SCO 485486, SECTOR 35C, CHANDIGARH, 160022, SINCE 31122015 DATE OF INCORPORATION IS 14052014 PAN NUMBER OF THE ACCOUNT IS ACOFS4226L IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 2214001603 TRANSACTION LINKED ACCOUNTS ARE 632201541211, 632201540621 AUTHORISED SIGNATORY, PARTNERS AND BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS ANGAD KALANTA, SARINA BHAGTA THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, PARTNERSHIP DEED, IEC DATE OF LAST KYC UPDATION 13022020AS PER ENHANCED DUE DILIGENCE CUSTOMER IS INTO IMPORT AND EXPORT OF AUTO ANCILARIES CLIENT IS INTO MERCHANTING TRADE BUSINESS CLIENT IS INTO MERCHANTING TRADE BUSINESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40LAC TO 5CR ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS004 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS 000 TOTAL DEBITS DURING THE SAME PERIOD WAS RS 020 LAKHS OF WHICH CASH WAS RS 020 LAKHS BY ATMFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 13515 LAKHS OF WHICH CASH WAS RS 004 LAKHS, CLEARING RS 9800 MAJORLY FROM AMAZON SELLER, NEFT RS 1102 LAKHS MAJORLY FROM KNOVATIC INC,SEVEN IMPEX, REMITTANCE RS 11426 LAKHS MAJORLY FROM QI LIANGFEI XIAOSHAN HANGZHOU, INTERNET FUND TRANSFERS RS 100 LAKH, MOBILE MONEY TRANSFERS RS 882 LAKHS IN 7 INSTANCES , UPI TRANSFERS RS 001 LAKHS IN 1 INSTANCE TOTAL DEBIT SIDE RS 13495 LAKHS OF WHICH CASH WAS RS 1826 LAKHS BY ATM,BRANCH, TRANSFERS RS 361 LAKHS MAJORLY TO AYUSH CHAUHAN,ADITYA CHHABRA, CLEARING RS 359 LAKHS MAJORLY TO SEVEN IMPEX,ST XAVIER S INTERNATIONA,MR ADITYA CHHABRA, RTGS RS 2400 LAKHS MAJORLY TO SEVEN IMPEX, REMITTANCE RS 033 LAKHS, INTERNET FUND TRANSFER RS 5194 LAKHS, CARD TRANSACTIONS RS 061 LAKHS, MOBILE MONEY TRANSFERS RS 1171 LAKHS IN 45 INSTANCES FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 20118 LAKHS OF WHICH CASH WAS RS 055 LAKHS, REMITTANCE RS 13807 LAKHS  TOTAL DEBIT SIDE RS 20205 LAKHS OF WHICH CASH WAS RS 2273 LAKHS, REMITTANCE RS 084 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 26139 LAKHS OF WHICH CASH WAS RS065 LAKHS, REMITTANCE RS 11986 LAKHSTOTAL DEBIT SIDE RS 26047 LAKHS OF WHICH CASH WAS RS 4125 LAKHS, REMITTANCE RS 1079 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 16011 LAKHS OF WHICH CASH WAS RS 200 LAKHS, REMITTANCE RS 10660 LAKHS TOTAL DEBIT SIDE RS 16097 LAKHS OF WHICH CASH WAS RS 4037 LAKHS, REMITTANCE RS 1322 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS,ONE LEG HANDLED BY OTHER BANK HAS BEEN NOTICED IN THE ACCOUNT FOR 30 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 351LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN 270 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN 28082019 TO 14102019 USD 12399 REMITTED IN PARTS JINJIANG HANGWAN AUTOMOBILE PARTS CO LTD IN CHINA',
  {'entities': [(5060, 5100, 'FOREIGN ORG'),
    (759, 787, 'FOREIGN ORG'),
    (3397, 3426, 'FOREIGN ORG'),
    (3332, 3344, 'FOREIGN ORG'),
    (1587, 1598, 'FOREIGN ORG'),
    (3345, 3356, 'FOREIGN ORG'),
    (3735, 3746, 'FOREIGN ORG'),
    (3821, 3832, 'FOREIGN ORG'),
    (372, 401, 'FOREIGN ORG'),
    (1242, 1271, 'FOREIGN ORG'),
    (787, 811, 'FOREIGN ORG'),
    (759, 783, 'FOREIGN ORG')]}),
 ('PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS12079930NIGER SEED FREE HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT WAS RECCEIVED ON 12062020 TO TUNE OF USD 29792 WAS RECEIVED FROM SARL RENAUDAT J C LES CHAUMES WHOSES DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE WAS NOT COMPLETED IN 270 DAYS, WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 191220219 AND14022020 TO TUNE OF USD65283 TO BLACK SEA GROUP LOGISTICS IN RUSSIA DETAILS OF WHICH IS NOT AVAILABE IN PUBLIC DOMAIN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE AND GOODS MENTIONED AS HS07132000CHICKPEAS, HS10089090RED MILLET HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED ON 02032020 TO THE TUNE OF USD25276 IN TOTAL 22 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 699LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS CLIENT IS DEALING IN MERCHANT TRADE OF SEEDS, MILLETS, MAIZE ETC IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDAGRICOM IMPEX,PROPRIETORSHIP FIRM, HAVING ADDRESSF NO 202,AMALTAS APPT,RAJ NAGAR, NAGPUR, MAHARASHTRA, INDIA, 440013, CONTACT NUMBER 09822220151 AND EMAIL ID AGRICOMIMPEX@HOTMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 624205012998 WITH ICICI BANK LTD, NAGPUR  RAMDASPETH,AKARSHAN BUSIPLEX, 26, CENTRAL BAZAAR ROAD, RAMDASPETH, NAGPUR, MAHARASHTRA, 440010, SINCE 27122013 DATE OF INCORPORATION IS 04072007 PAN NUMBER OF THE ACCOUNT IS AJRPG6283D IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0307027422 GSTIN AS PER BANK RECORDS IS 27AJRPG6283D1ZYTRANSACTION LINKED ACCOUNTS ARE 624205501426, 015705014893 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKASH GHADSE PROP AJRPG6283DTHE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, GSTIN DATE OF LAST KYC UPDATION 15102019 AGRICOM IMPEX IS A LEADING WHOLESALE MANUFACTURER AND EXPORTER OF AGRICULTURAL OIL SEEDS, BIRD SEEDS LIKE YELLOW MILLET, SAFFLOWER SEED, NIGER SEED, SORGHUM, SESAME SEEDS, CHICK PEAS AND PEANUTS IN INTERNATIONAL MARKET DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS40 LAC TO RS5 CRALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 28022021 WAS RS2356 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 1,66933 LAKHS OF WHICH TRANSFERS RS 11554 LAKHS MAJORLY FROM AGRICOM IMPEX,SHRI BALAJI INDUSTRIES, CLEARING RS 066 LAKHS MAJORLY FROM FWD CANC GAIN, NEFT RS 4022 LAKHS MAJORLY FROM NARAYANA SAI TRADERS,SURGO RESOURCES, RTGS RS 13991 LAKHS MAJORLY FROM TALOJA AGRO FOODS PVT LTD,HITASHI AGRI TRADE,AGRICOM IMPEX, REMITTANCE RS 1,37107 LAKHS MAJORLY FROM EEFC,EEFC CONVERSION,AGRICOM IMPEX, MOBILE MONEY TRANSFERS RS 189 LAKHS IN 4 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,46676 LAKHS OF WHICH CASH WAS RS 474 LAKHS BY BRANCH, TRANSFERS RS 769 LAKHS MAJORLY TO BHAIYAJI RAMBHAUJI ROKDE JEWELLERS,ICICI PRUD LIFE INS CO LTDCASH COLLN,AGRICOM IMPEX, CLEARING RS 519 LAKHS MAJORLY TO PANKAJ GULATI AND ASSOCIA,M S E D C L,V P SALES AND SERVICES, NEFT RS 1445 LAKHS MAJORLY TO B NARAYANA,THAPARSONS K V SHIPP,GIDDDIAH TRADERS,RTGS RS 7642 LAKHS MAJORLY TO SHREE BHAGAWATI AGRO P,NARAYAN SAI TRADER,MILAP TRADING, REMITTANCE RS 47344 LAKHS MAJORLY TO GENERAL COMMODITIES LTD,SPRINGHAUS LLC,KAMAL LLC, INTERNET FUND TRANSFER RS 82312 LAKHS, MOBILE MONEY TRANSFERS RS 2475 LAKHS IN 65 INSTANCES FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 1,73991 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 1,55109 LAKHS TOTAL DEBIT SIDE RS 1,84902 LAKHS OF WHICH CASH WAS RS 440 LAKHS, REMITTANCE RS 44077 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 2,19840 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 2,01859 LAKHS TOTAL DEBIT SIDE RS 2,27924 LAKHS OF WHICH CASH WAS RS 325 LAKHS, REMITTANCE RS 4831 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 2,92310 LAKHS OF WHICH CASH WAS RS 095 LAKHS, REMITTANCE RS 2,50833 LAKHS TOTAL DEBIT SIDE RS 2,82585 LAKHS OF WHICH CASH WAS RS 1414 LAKHS, REMITTANCE RS 21110 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS, LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR 22 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 699LACS FOR EXAMPLE, VIOLATION OF LOSS ON MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS PROCESSED ON 15052020 USD 31046 WAS DEBITED TO KAMAL GLOBAL TRADING PTE LTD IN SINGAPORE WHOSE DETAILS ARE NOT AVAILABLE IN PUBLIC DOMAIN WITH',
  {'entities': [(3662, 3685, 'FOREIGN ORG'),
    (3701, 3710, 'FOREIGN ORG'),
    (550, 575, 'FOREIGN ORG'),
    (3686, 3700, 'FOREIGN ORG'),
    (1240, 1253, 'FOREIGN ORG'),
    (2141, 2154, 'FOREIGN ORG'),
    (2765, 2778, 'FOREIGN ORG'),
    (3001, 3014, 'FOREIGN ORG'),
    (3078, 3091, 'FOREIGN ORG'),
    (3349, 3362, 'FOREIGN ORG')]}),
 ('STANDARD C,CENTRAL BO,GE POWER I, REMITTANCE RS 73,44751 LAKHS MAJORLY TO JINDAL STEEL AND POWER LIMITED,ATI FLAT ROLLED PRODUCTS,KANEMATSU CORPORATION, INTERNET FUND TRANSFER RS 1698 LAKHS, , AND OTHERS RS 43,47245 LAKHSFY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 4,23,15852 LAKHS OF WHICH CASH WAS RS 003 LAKHS, REMITTANCE RS 53,99822 LAKHS  TOTAL DEBIT SIDE RS 4,14,40186 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 61,39707 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 6,28,44452 LAKHS OF WHICH CASH WAS RS 012 LAKHS, REMITTANCE RS 69,62082 LAKHS  TOTAL DEBIT SIDE RS 6,26,39315 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 54,32714 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 6,23,07794 LAKHS OF WHICH CASH WAS RS 005 LAKHS, REMITTANCE RS 65,42120 LAKHS  TOTAL DEBIT SIDE RS 6,21,91998 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 34,05085 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS HAS BEEN NOTICED IN THE ACCOUNT FOR 10 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF USD 1715LACS, EUR 7871LACS, CNY 3633LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE NOT COMPLETED IN 270 DAYS IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT ON ON 28042020 USD 68887 WAS REMITTED TO VICENTE BERRIZBEITIA S L CLIENT IS INTO SUPPLY OF BOLTING MATERIALS HTTPBERRIZBEITIACOMEN WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS73030090TUBES AND PIPES HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE IN AN ANOTHER MTT TRANSACTION, VIOLATION OF FUNDS OUTLAY MORE THAN 120 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT ON 21032020 TO TUNE OF USD363LACS TO GE HYDRO CHINA CO LTDENTITY IS JV WITH YALONG RIVER HYDROPOWER DEVELOPMENT CO LTD WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS85030010HYDRO SET HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT IS STILL DUE IN TOTAL 10 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD 1715LACS, EUR 7871LACS, CNY 3633LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONSCLIENT IS DEALING IN MERCHANT TRADE OF MACHINERY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT,STR IS BEING FILEDGE POWER INDIA LIMITED,PUBLIC LTD, HAVING ADDRESST5 T6 PLOT 1 TO 14,JAYPEE WISHTOWN,, SECTOR 128,AXIS HOUSE, NOIDA, UTTAR PRADESH, INDIA, 201304, CONTACT NUMBER 919810292674 AND EMAIL ID CHANDNISISODIA@POWERALSTOMCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000705001143 WITH ICICI BANK LTD, NEW DELHI  CONNAUGHT PLACE,9A, PHELPS BUILDING, CONNAUGHT PLACE,NEW DELHI, DELHI, 110001, SINCE 01121999 DATE OF INCORPORATION IS 02091992 PAN NUMBER OF THE ACCOUNT IS AABCA8679F IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0399054081 TRANSACTION LINKED ACCOUNTS ARE 140205002149, 179405000165 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE P JAYALAKSHMI AUTHORISED SIGNATORY AAEPP5525B ,AMARESH SINGH AUTHORISED SIGNATORY AICPS9581L ,RAJESH AGGARWAL AUTHORISED SIGNATORY AGIPA1553C ,HIREN RAJNIKANT VYAS AUTHORISED SIGNATORY ABTPV1559P ,NAVEEN SHANKARAN AUTHORISED SIGNATORY AIOPN5430M THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AABCA8679F,MOA,AOA DATE OF LAST KYC UPDATION 16082019 GE POWER INDIA LIMITED GPIL, FORMERLY KNOWN AS ALSTOM INDIA LIMITED, IS ENGAGED IN ENGINEERING, PROCUREMENT, MANUFACTURING, CONSTRUCTION, AND SERVICING, OF POWER PLANTS AND POWER EQUIPMENT IN BOTH THERMAL AND HYDRO SEGMENTS THE COMPANY DESIGNS MANUFACTURE, AND DELIVERS TURNKEY POWER PLANTS, POWER GENERATION EQUIPMENT, AND AIR QUALITY CONTROL SYSTEMS AND SERVICES; AND ALSO PROVIDES RENEWABLE POWER GENERATION SOLUTIONS FOR INTEGRATED POWER PLANTS COVERING HYDROELECTRICITY, WIND, GEOTHERMAL, BIOMASS, SOLAR, AND WAVE AND TIDAL STREAM ENERGIES IT ALSO OFFERS INDIVIDUAL COMPONENTS FOR A RANGE OF TURBINES AND GENERATORS, AS WELL AS VARIOUS SERVICES, INCLUDING PLANT MODERNIZATION, MAINTENANCE, AND OPERATIONAL SUPPORT; AND PROVIDES PRODUCTS AND SYSTEMS FOR ELECTRICAL UTILITIES AND INDUSTRIESALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS27029 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 04042021 SHOWS TOTAL CREDIT RS 3,55920 LAKHS OF WHICH RS 0 BY CASH, NEFT RS 7028 LAKHS MAJORLY FROM LALITPUR POWER GENERATION CO,TORRENT POWER LIMITED, RTGS RS 3,48892 LAKHS MAJORLY FROM BHARAT HEAVY ELECTRICAL TOTAL DEBITS DURING THE SAME PERIOD WAS RS 4,50950 LAKHS OF WHICH RS 0 BY CASH, RTGS RS 4,50000 LAKHS MAJORLY TO GE POWER I, REMITTANCE RS 003 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 2,64,70585 LAKHS OF WHICH CASH WAS RS 2108 LAKHS MAJORLY IN GULBARGA BRANCH, TRANSFERS RS 27,22216 LAKHS MAJORLY FROM GE POWER INDIA LTD TELENGANA FGD PROJECT  ESCROW ACCOUNT,GE POWER INDIA LTDIGSTPP JHAJJAR FGD PROJECT  ESCROW ACCOUNT,GE POWER INDIA LTDSIPAT FGD PROJECT  ESCROW ACCOUNT, CLEARING RS 69,72591 LAKHS MAJORLY FROM LT MHPS BOILERS PRIVATE LIMIT,NTPC LIMITED,VEDANTA LIMITED, NEFT RS 27,34810 LAKHS MAJORLY FROM GENERAL ELECTRIC COMPANY,NHPC LIMITED,MEJA URJA NIGAM PVT LTD, RTGS RS 76,46302 LAKHS MAJORLY FROM MS DOOSAN POWER SYSTE,BHARAT HEAVY ELECTRICA,GE POWER INDIA LIMITED, REMITTANCE RS 60,14373 LAKHS MAJORLY FROM ARAVALI POWER COMPANY PRIVATE LTD,GENERAL ELECTRIC SWITZERLAN,NTPC LIMITED, INTERNET FUND TRANSFERS RS 4143 LAKHS, MOBILE MONEY TRANSFERS RS 328 LAKHS IN 6 INSTANCES , UPI TRANSFERS RS 020 LAKHS IN 1 INSTANCE AND OTHERS RS 3,73694 LAKHS TOTAL DEBIT SIDE RS 2,80,78614 LAKHS OF WHICH TRANSFERS RS 6,20759 LAKHS MAJORLY TO GE POWER INDIA LTD UNPAID DIVIDEND ACCOUNT 2019 20,GE POWER INDIA LIMITED,GE POWER SYSTEMS INDIA PVT LTD, CLEARING RS 97355 LAKHS, NEFT RS 5,17324 LAKHS MAJORLY TO GE POWER INDIA LIMIT,CENTRAL EXCISE SERVI,WEST BENGAL STATE EL, RTGS RS 1,51,49481 LAKHS MAJORLY',
  {'entities': [(130, 151, 'FOREIGN ORG'),
    (105, 129, 'FOREIGN ORG'),
    (1793, 1814, 'FOREIGN ORG'),
    (1832, 1874, 'FOREIGN ORG'),
    (5517, 5533, 'FOREIGN ORG'),
    (5761, 5777, 'FOREIGN ORG')]}),
 ('ALSO ON 17102020 RS 5212 LACS GOT CREDITED BY INWARD REMITTANCE FROM WORLD FIRST ASIA LTD, HONG KONG WITH INFORMATION AS IMPORT OF GARMENTS AND ON 19102020 RS 39 LACS GOT DEBITED BY CMS CLEARING WITH REMARK AS SDI SD INTERNATIONAL IN OTHER EXAMPLE, ON 04022021 RS 25 LACS WAS CREDITED THROUGH RTGS FROM SD INTERNATIONAL ACC259830542168, IFSCINDB0000006 AND ON THE SAME DAY RS 2412 LACS ARE IMMEDIATELY DEPLOYED THROUGH CMS CLEARING WITH REMARK AS SDI SD INTERNATIONAL SIMILAR SUCH TRANSACTIONS ARE OBSERVED THROUGHOUT THE ACCOUNT FOR WHICH NO RATIONAL FOUND TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 4,67635 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF LESS THAN RS 40LACS IN VIEW OF INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS, LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF HIGH VALUE FUNDS AND HIGH TURNOVER, STR IS BEING FILEDSD INTERNATIONAL, PROPRIETORSHIP FIRM, HAVING ADDRESS 2ND FLOOR FLR, 221B, NIMTOLA GHAT, STREET NEAR JORABAGAN MORE, KOLKATA, WEST BENGAL, INDIA, 700006, CONTACT NUMBER 919830542167 AND EMAIL ID BANKS@SDWORLDIN IS MAINTAINING A CURRENT ACCOUNT NUMBER 627905502328 WITH ICICI BANK LTD, KOLKATA  BURRA BAZAR, ICICI BANK LTD, P16, KALAKAR STREET, BURRABAZAR, KOLKATA, WEST BENGAL, 700007, SINCE 20082018 DATE OF INCORPORATION IS 01082003 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AFYPD0627H OTHER RELATIONSHIP OF CUSTOMER ARE 627705046590, 627705051746, 627905016291, 627905501161, 627905501162, 627905502320, 627906000035, 627906000036, 627913002540, 627913002541, 627913002542, 627913002573, 627913002574, 627913002577, 627966000005, 627966000006 TRANSACTION LINKED ACCOUNTS ARE 627905501161 AND 627605256946 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE SANJOY DEY PROPRIETOR  AFYPD0627H THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN  AFYPD0627H, AADHAR DATE OF LAST KYC UPDATION 20012021 KYC UPDATION IS NOT DUE ENTITY IS INTO BUSINESS OF KIDS GARMENTS AND WOMEN SUIT PCS SET TRADERS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS 40LACS ALERT WAS TRIGGERED FOR INWARD REMITTANCE FOLLOWED BY IMMEDIATE NON CASH WITHDRAWAL TRANSACTIONS BALANCE IN THE ACCOUNT AT 28022021 WAS RS1282 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 4,67635 LAKHS OF WHICH TRANSFERS RS 1,42800 LAKHS MAJORLY FROM S D INTERNATIONAL, CLEARING RS 17245 LAKHS MAJORLY FROM S D, S D TRADE IMPEX PRIVATE LIMITE, CMS16514132541, RTGS RS 1,46000 LAKHS MAJORLY FROM S D INTERNATIONAL, SDINTERNATIONAL, REMITTANCE RS 1,53490 LAKHS MAJORLY FROM ADVANTAGE GENERAL TRADING LIMITED, COSMIC EDGE GENERAL TRADING LIMITED, LIGHTWAYS GENERAL TRADING LLC TOTAL DEBITS DURING THE SAME PERIOD WAS RS 4,82387 LAKHS OF WHICH CASH WAS RS 750 LAKHS BY BRANCH, TRANSFERS RS 16447 LAKHS MAJORLY TO HETAL SYNTEX, AMIT ROADLINES, S D INTERNATIONAL, CLEARING RS 2,24330 LAKHS MAJORL TO SDIRTGS08062020, SDIRTGS06082020, SDITRF01102020, RTGS RS 2,04593 LAKHS MAJORLY TO NITFUL COMMERCIAL PVT, LAXMIWAN SUPPLIERS PVT, SURAKSHIT HOUSING ADVI FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS WERE FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 8,29473 LAKHS OF WHICH CASH WAS RS 200 LAKHS, TRANSFERS RS 1,39421 LAKHS, RTGS RS 2,79350 LAKHS, REMITTANCE RS 3,43220 LAKHS TOTAL DEBIT SIDE RS 8,35436 LAKHS OF WHICH CASH WAS RS 1750 LAKHS, CLEARING RS 4,53609 LAKHS, RTGS RS 3,57873 LAKHS FY 201819 FROM 20082018 TO 31032019 TOTAL CREDIT SIDE RS 4,59981 LAKHS OF WHICH RS 0 BY CASH, REMITTANCE RS 2,16245 LAKHS TOTAL DEBIT SIDE RS 4,37985 LAKHS OF WHICH CASH WAS RS 950 LAKHS AS PER THE TRANSACTION PATTERN, THERE ARE FREQUENT HIGH VALUE FUNDS CREDITED AND IMMEDIATELY DEBITED ON THE SAME DAY SEEMS LIKE ROUTING OF FUNDS FOR EXAMPLE, ON 19022021 RS 30 LACS WAS CREDITED THROUGH RTGS FROM S D INTERNATIONAL ACCOUNT NO 11030200072126, BANK NAMEFEDERAL BANK, IFSC CODE ICIC0006279 AND ON 20022021 RS 20 LACS WAS DEPLOYED THROUGH RTGS TOWARDS S D INTERNATIONAL ACC259830542168, IFSCINDB0000526 AND THE REST OF THE FUNDS WERE DEPLOYED THROUGH CMS WITH REMARK AS SDI SD INTERNATIONAL',
  {'entities': [(69, 89, 'FOREIGN ORG'),
    (2597, 2614, 'FOREIGN ORG'),
    (2741, 2758, 'FOREIGN ORG'),
    (3085, 3102, 'FOREIGN ORG'),
    (4055, 4072, 'FOREIGN ORG'),
    (4206, 4223, 'FOREIGN ORG')]}),
 ('HTTPSUZBEKEMBASSYCOMMYENGNEWSPRESSBOSTONMEGATEXTILELLCINKARAKALPAKSTANHTML WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS52054210COTTON YARN HOWEVER THE RESPECTIVE INWARD REMITTANCE TOWARDS EXPORT WAS RECEIVED BETWEEN 20092019 AND 20012020 TO THE TUNE OF USD189 LACS FROM CHENG CHIH TRANSCEND ENTERPRISE CORPORATION CLIENT DEAAL SIN TRADING OF CONTTON YARNSHTTPSPANJIVACOMCHENGCHIHTRANSCENDENTERPRISE46747920 IN TOTAL 28 DEFAULT MTT TRANSACTIONS TO A THE TUNE OF USD2062 LACS WERE OBSERVED IN CLIENTS ACCOUNT WHICH RAISES FURTHER SUSPICION VIOLATIONS  CLIENT IS DEALING IN MERCHANT TRADE OF MACHINARY IN THE VIEW OF ABOVE OBSERVATIONS AND VIOLATION OF MERCHANT TRADE TRANSACTIONS IN THE ACCOUNT, STR IS BEING FILEDSQUARE CORPORATION, PROPRIETORSHIP FIRM, HAVING ADDRESS326 C BRS NAGAR,LUDHIANA,PUNJAB, LUDHIANA, PUNJAB, INDIA, 141002, CONTACT NUMBER 919815045100 AND EMAIL ID ACCOUNTS@SQUARECORPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 001706000136 WITH ICICI BANK LTD, LUDHIANA  FEROZE GANDHI MARKET,NEHRU SIDHANT KENDER TRUST BUILDING, FEROZ GANDHI MARKET, LUDHIANA, PUNJAB, 141001, SINCE 31082015 DATE OF INCORPORATION IS 01042001 PAN NUMBER OF THE ACCOUNT IS AAOPV1881G IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 3001010495 GSTIN AS PER BANK RECORDS IS 03AAOPV1881G1ZF TRANSACTION LINKED ACCOUNTS ARE 001706000163 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VIVEK VERMA PROP, RAHUL TALWAR AUTHORISED SIGNATORY ADBPT6925B, VANDANA VERMA AUTHORISED SIGNATORY AADPV2010M THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE  DATE OF LAST KYC UPDATION 24122019 CUSTOMER IS INTO YARN BUSINESS HTTPWWWSQUARECORPCOM DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS EQUAL TO 100 CR ALERT WAS TRIGGERED FOR FREQUENT MERCHANT TRADE TRANSACTIONSMTT DEFAULTS BY CUSTOMER BALANCE IN THE ACCOUNT AT 04042021 WAS RS000 DEBIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 1011 LAKHS OF WHICH TRANSFERS RS 006 LAKHS MAJORLY FROM PROMINENT EXPORTS INTERNATIONAL, REMITTANCE RS 1006 LAKHS MAJORLY FROM THERMAX YARN DYED FABRICS LIMITED,EVINCE TEXTILES LTD,MEGHNA KNIT COMPOSITE LIMITED TOTAL DEBIT SIDE RS 1011 LAKHS OF WHICH REMITTANCE RS 1011 LAKHS MAJORLY TO WEIQIAO TEXTILE COMPANY LIMITED,WEIHAI WEIQIAO INDUSTRIAL PARK CO,MIF INTERNATIONAL LIMITEDFY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 7996 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 7922 LAKHSTOTAL DEBIT SIDE RS 8231 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 8231 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 13392 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 13077 LAKHS  TOTAL DEBIT SIDE RS 13232 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 13192 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 10540 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 10427 LAKHS TOTAL DEBIT SIDE RS 10756 LAKHS OF WHICH CASH WAS NIL,REMITTANCE RS 10124 LAKHS VIOLATION OF MERCHANT TRADE TRANSACTIONS MTT IE MTT CYCLE NOT COMPLETED IN 270 DAYS, LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE ACCOUNT FOR 28 TRANSACTIONS FOR THE YEAR ENDING JUNE 2020 OF THE VALUE OF US2062LACS FOR EXAMPLE, VIOLATION OF LOSS IN MTT CYCLE HAS BEEN NOTICED IN THE TRANSACTION WHERE, THE FIRST OUTWARD REMITTANCE FOR IMPORT WAS SENT BETWEEN BETWEEN 24092019 TO 17012020 USD 219 LACS REMITTED TO IRS UNIVERSAL PTE LTDIRS UNIVERSAL PTE LTD ISIN INTERNATIONAL PTE LTD WAS FOUNDED IN 1991 THE COMPANYS LINE OF BUSINESS INCLUDES THE WHOLESALE DISTRIBUTION OF NONDURABLE GOODSHTTPSWWWBLOOMBERGCOMPROFILECOMPANY6077450ZSP WITH PURPOSE OF REMITTANCE BEING GOODS ACQUIRED UNDER MERCHANTING  PAYMENT AGAINST IMPORT LEG OF MERCHANTING TRADE, AND GOODS MENTIONED AS HS52051210YARN HOWEVER THE RESPECTIVE INWARED REMITTANCE TOWARDS EXPORT IS DUE IN AN ANOTHER MTT TRANSACTION, VIOLATION OF MTT CYCLE NOT COMPLETED IN 270 DAYS WHERE, OUTWARD REMITTANCE TOWARDS IMPORT WAS SENT BETWEEN 28102019 TO 05122019 TO TUNE OF USD149LACS TO BOSTON MEGA TEKSTIL LLC ENTITY IS INTO MANUFACTURE OF TEXTILE',
  {'entities': [(3604, 3625, 'FOREIGN ORG'),
    (3625, 3646, 'FOREIGN ORG'),
    (3647, 3673, 'FOREIGN ORG'),
    (4226, 4249, 'FOREIGN ORG'),
    (2360, 2385, 'FOREIGN ORG'),
    (2386, 2405, 'FOREIGN ORG'),
    (2512, 2543, 'FOREIGN ORG'),
    (2578, 2603, 'FOREIGN ORG')]}),
 (' FLYON FILM PRODUCTION, A PROPRIETORSHIP FIRM, HAVING ADDRESS70789, HEMKUNT CHAMBER, NEHRU PLACE, SOUTH DELHI, NEW DELHI, INDIA, 110019, CONTACT NUMBER 917303930724 AND EMAIL ID AKANKSHASINGH9014@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000705051096 WITH ICICI BANK LTD, NEW DELHI  CONNAUGHT PLACE, ICICI BANK LTD, 9A, PHELPS BUILDING, CONNAUGHT PLACE, DELHI, 110001, SINCE 20012021 DATE OF INCORPORATION IS 17082020 PAN NUMBER OF THE ACCOUNT IS CGHPS8098F GSTIN AS PER BANK RECORDS IS 07CGHPS8098F2ZP OTHER RELATIONSHIP OF CUSTOMER IS 163905001067 TRANSACTION LINKED ACCOUNTS ARE 643805053274, 119405001855 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE AKANKSHA SINGH PROPRIETOR  CGHPS8098F THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON 06032021 KYC DOCUMENTS OBTAINED WERE PAN CGHPS8098F, GSTIN, AADHAR CLIENT DEALS MEDIA AND FILMES SERVICES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAC TO RS 5 CR A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 06032021 WAS NIL FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 20012021 TO 06032021 TOTAL CREDIT SIDE RS 1,96158 LAKHS OF WHICH CLEARING RS 100 LAKH MAJORLY FROM FLYON FILM PRODUCTION, NEFT RS 4976 LAKHS MAJORLY FROM DAT SOLUTIONS, APS INTERNATIONAL, IMAGINARY FILMS PRIVATE LIMITED, RTGS RS 1,90632 LAKHS MAJORLY FROM GWAY TO WORLD, OCN INTERNATIONAL, CARIO SERVICES, MOBILE MONEY TRANSFERS RS 450 LAKHS IN 3 INSTANCES, TOTAL DEBIT SIDE RS 1,96158 LAKHS OF WHICH CLEARING RS 151 LAKHS MAJORLY TO FLYCON FILM PRODUCTION, RTGS RS 84310 LAKHS MAJORLY TO IMAGINARY FILMS PRIVATE LIMITED, REMITTANCE RS 13353 LAKHS MAJORLY TO THE PRODUCTION SERVICES FZCO, GATE STUDIOS PTE LTD, INTERNET FUND TRANSFER RS 98241 LAKHS TO ONE FRAME PRODUCTIONS, MOBILE MONEY TRANSFERS RS 100 LAKH IN 1 INSTANCE, AND OTHERS RS 002 LAKHS TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS ROUTING OF FUNDS OBSERVED SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD FOR EXAMPLE, ON 24022021 RS 6114 LACS CREDITED BY RTGS FROM IMAGINARY FILMS PRIVATE LIMITED 920020074440297, IFSCUTIB0001625, AXIS BANK LTD AND SAME WAS DEBITED THROUGH REMITTANCE TO GATE STUDIOS PTE LTD IN SINGAPORE FOR PERSONAL, CULTURAL AND RECREATIONAL SERVICES IN OTHER EXAMPLE, BETWEEN 01032021 TO 02032021 RS 6432 LACS CREDITED BY RTGS FROM IMAGINARY FILMS PRIVATE LIMITED 920020074440297, IFSCUTIB0001625, AXIS BANK LTD AND SAME WAS DEBITED THROUGH NEFT TO IMAGINARY FILMS PRIVATE LIMITED 920020074440297, IFSCUTIB0001625, AXIS BANK LTD SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 1,96158 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS 40LAC TO 5CR IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS, HIGH TURNOVER AND ACCOUNT BEING CLOSED IN SHORT DURATION, STR IS BEING FILED',
  {'entities': [(1864, 1884, 'FOREIGN ORG'),
    (2347, 2367, 'FOREIGN ORG'),
    (1709, 1731, 'FOREIGN ORG'),
    (1838, 1862, 'FOREIGN ORG'),
    (1429, 1442, 'FOREIGN ORG'),
    (1444, 1461, 'FOREIGN ORG'),
    (1546, 1563, 'FOREIGN ORG'),
    (1565, 1579, 'FOREIGN ORG')]}),
 (' REDNODE MEDIA LIMITED, A PUBLIC LTD COMPANY, HAVING ADDRESS  GALA NNO 12 AND 13 SHREE, 3RD FLOOR, KAMDHENU, EST CORSS LINK RD, MALAD WEST, NR TANGENT FURNITURE SHOWROOM, MUMBAI, MAHARASHTRA, INDIA, 400064, CONTACT NUMBER 919619537557 AND EMAIL ID REDNODEMEDIALIMITED@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 015805010762 WITH ICICI BANK LTD, SADGURU DARSHAN, ROAD NO 3, LIBERTY GARDEN, MALAD W, MUMBAI, MAHARASHTRA, 400064, SINCE 29102020 DATE OF INCORPORATION IS 12102017 PAN OF THE ACCOUNT IS AAICR5730K GSTIN AS PER BANK RECORDS IS 27AAICR5730K1Z1 THERE ARE NO LINKED ACCOUNTS TRANSACTION LINKED ACCOUNTS ARE 015805010832,015805010356 BENEFICIAL OWNERS DIRECTORS AUTHORISED SIGNATORIES OF THE REPORTED ACCOUNT ARE AYESHA ALI MEHBOOB SHAIKH WITH PAN NO KYSPS9802L, PATEL PINKI AWADHESH WITH PAN NO CFWPP4169L, SANDEEP RAJENDRA THAKUR WITH PAN NO ATCPT2515N, SUBHASH SITARAM MUDRAS WITH PAN NO CAKPM5598R THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION THE FIRM IS INVOLVED IN MAKING SOFTWARES AND BUILDING PROJECTS AS PER THE CUSTOMER NEEDS AND ALSO INVOLVED IN EVENTS AND ENTERTAINMENT DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40 LACS ALERT WAS TRIGGERED FOR HIGH FOREX TRANSACTIONS BALANCE IN THE ACCOUNT AT 16032021 WAS RS 045 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 29102020 TILL 16032021 SHOWS TOTAL CREDIT RS 2,27619 LAKHS OF WHICH CASH WAS RS 780 LAKHS MAJORLY IN MALAD BRANCH, TRANSFERS RS 25000 LAKHS MAJORLY FROM DARWIN PLATFORM REFINERIES LIMITED, DARWIN PLATFORM INFRASTRUCTURE LIMITED, DARWIN PLATFORM CAR CAB TRADE LIMITED, NEFT RS 14069 LAKHS MAJORLY FROM GENESIS MULTI TRADE, MODERN INTERNATIONAL, INDIA TRADERS, RTGS RS 1,66183 LAKHS MAJORLY FROM RAZORPAY SOFTWARE PRIVATE LTD, REDNODE MEDIA LIMITED, INTERNET FUND TRANSFERS RS 14037 LAKHS FROM GOPAL TRADING COMPANY ETC, MOBILE MONEY TRANSFERS MMT RS 7449 LAKHS IN 57 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 2,27574 LAKHS OF WHICH CLEARING RS 700 LAKHS MAJORLY TO REDNODE MEDIA LIMITED, RTGS RS 91862 LAKHS TO REDNODE MEDIA LIMITED, REMITTANCE RS 1,07733 LAKHS MAJORLY TO METRO SOLUTIONS PTE LTD, SAILLI VIROHA PTE LTD, ONE STUDIOZ PTE LTD, INTERNET FUND TRANSFER RS 17122 LAKHS TO SANDEEP RAJENDRA THAKUR ETC, MOBILE MONEY TRANSFERS RS 10048 LAKHS IN 115 INSTANCES CREDIT ARE FROM OWN ACCOUNT, STEEL FIRMS ETC AND DEPLOYED TOWARDS REMITTANCE MAJORLY TO METRO SOLUTIONS PTE LTD TOWARDS ENTERTAINMENT WHICH SEEMS SUSPICIOUS FOR EXAMPLE, ON 03022021 RS 5780 LACS WERE CREDITED THROUGH RTGS FROM EKTA ENTERPRISES 10200005096622, BDBL0001799, INTERNET FUND TRANSFER FROM GOPAL TRADING COMPANY 057105002242 AND ON THE NEXT DAY RS 5594 LACS WERE DEBITED THROUGH REMITTANCE TO METRO SOLUTIONS PTE LTD, SINGAPORE TOWARDS ENTERTAINMENT SERVICES ALSO ON 10022021 RS 4994 LACS WERE CREDITED THROUGH RTGS FROM KUMAR STEEL IND 50200050722028, HDFC0000240, SHIV SHAKTI TRADE 39685975810, SBIN0016237, GURUJI INDUSTRIES 918020107115146, UTIB0000533 , OVERSEAS TRADERS 02340210003800, UCBA0000234 AND ON THE SAME DAY RS 3566 LACS WERE DEBITED THROUGH REMITTANCE TO METRO SOLUTION PTE LTD, SINGAPORE TOWARDS ENTERTAINMENT SERVICES SIMILAR PATTERN WERE SEEN IN THE ACCOUNT IN VIEW OF HIGH VALUE OUTWARD REMITTANCE SEEN IN THE ACCOUNT SEEMS LIKE ROUTING OF FUNDS, STR IS BEING FILED',
  {'entities': [(1565, 1579, 'FOREIGN ORG'),
    (2334, 2357, 'FOREIGN ORG'),
    (2616, 2639, 'FOREIGN ORG'),
    (2932, 2955, 'FOREIGN ORG'),
    (2382, 2401, 'FOREIGN ORG')]}),
 (' IMAGINARY FILMS PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS 402, EMBASSY CHAMBERS, 3RD ROAD, KHAR WEST, MUMBAI, MAHARASHTRA, INDIA, 400052, CONTACT NUMBER 917039548313 AND EMAIL ID IMAGINARYFILMS4@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 119405001855 WITH ICICI BANK LTD, PLOT NO 206 221, 27TH RD, CRYSTAL BUILDING, WATERFIELD ROAD, BANDRA W, MUMBAI, MAHARASHTRA, 400050, SINCE 29012020 DATE OF INCORPORATION IS 22012020 PAN NUMBER OF THE ACCOUNT IS AAFCI5545Q THERE ARE NO PAN LINKED ACCOUNTS TRANSACTION LINKED ACCOUNTS ARE 135205000593, 151105000348 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH KUMAR SINGH WITH PAN NO GSRPS7969M, SAPNA KUMARI WITH PAN NO DGSPK3718Q THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS TOTAL FREEZED ON 17022021 DUE TO SUSPICIOUS NATURE OF TRANSACTIONS IN THE ACCOUNT 0038NSMT0040321, AS PER BANK RECORDS KYC DOCUMENTS OBTAINED WERE PAN AAFCI5545Q, MOA, AOA, COI U92490MH2020PTC336424 AS PER ENHANCE DUE DILIGENCE COMPANY IS IN THE BUSINESS OF THE CINEMATOGRAPH TRADE AND INDUSTRY IN ALL THEIR BRANCHES AND ACTIVITIES AND PARTICULARLY THE BUSINESS OF MANUFACTURE, PRODUCTION, DISTRIBUTION, EXPLOITATION, EXHIBITION, IMPORT AND EXPORT OF ALL KINDS OF CINE FILMS, TALKIE FILMS, VIDEO FILMS, TELEFILMS, DOCUMENTARY FILMS, ADVERTISING FILMS, ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 5 CRS TO 25 CRS A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 16032021 WAS RS087 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 16032021 SHOWS TOTAL CREDIT RS 6,67792 LAKHS OF WHICH TRANSFERS RS 13147 LAKHS MAJORLY FROM IMAGINARY FILMS PRIVATE LIMITED, KESARI TOURS PVT LTDFOREX, CLEARING RS 264 LAKHS MAJORLY FROM IMAGINARY FILMS PVT LTD, RTGS RS 32630 LAKHS MAJORLY FROM IMAGINARY FILMS PVT LTD, GILPIN TOURS AND TRAVEL, TRANSCORP INTERNATIONAL, INTERNET FUND TRANSFERS RS 6,21498 LAKHS FROM SAWARTHIYA FILM PRODUCTIONS, FLYON FILM PRODUCTION, ETC, MOBILE MONEY TRANSFERS RS 100, AND OTHERS RS 252 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 6,68180 LAKHS OF WHICH CASH WAS RS 460 LAKHS BY BRANCH, CLEARING RS 1813 LAKHS MAJORLY TO IMAGINARY FILMS PVT LTD, HDFC BANK LTD, YES BANK LTD, RTGS RS 4,16026 LAKHS MAJORLY TO GILPIN TOURS N TRAVEL, IMAGINARY FILMS PVTLTD, WORLD ONE INDIA FOREX, REMITTANCE RS 35019 LAKHS MAJORLY TO ROSEBERRY INTERNATIONAL, ROSEBERRY INTERNATIONAL SERVICES, LIVING FILMS PTE LTD, INTERNET FUND TRANSFER RS 2,11650 LAKHS TO SAWARTHIYA FILM PRODUCTIONS, MULTIMONEY FOREX LTD, ETC, MOBILE MONEY TRANSFERS RS 2926 LAKHS IN 45 INSTANCES AND OTHERS RS 286 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 29012020 TO 31032020 TOTAL CREDIT SIDE RS 3000 LAKHS OF WHICH CLEARING RS 2000 LAKHS, RTGS RS 1000 LAKHS TOTAL DEBIT SIDE RS 2524 LAKHS OF WHICH CASH WAS RS 050 LAKHS, CLEARING RS 2000 LAKHS TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS WHICH SEEMS LIKE ROUTING OF FUNDS FOR EXAMPLE, BETWEEN 12022021 TO 15022021 RS 6320LACS CREDITED BY TRANSFERS FROM SAWARTHIYA FILM PRODUCTIONS 1352050005930 AND DURING SAME DURATION RS6156LACS WAS DEBITED BY TRANSFERS TO SAWARTHIYA FILM PRODUCTIONS 1352050005930 IN OTHER EXAMPLE, ON 08022021 RS 6236LACS WAS CREDITED BY TRANSFERS FROM FLYON FILM PRODUCTION 000705051096 AND SAME WAS DEBITED BY TRANSFERS TO MULTIMONEY FOREX LTD 151105000348 SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS AND ROUTING OF FUNDS, STR IS BEING FILED',
  {'entities': [(2482, 2505, 'FOREIGN ORG'),
    (2507, 2539, 'FOREIGN ORG'),
    (2541, 2561, 'FOREIGN ORG'),
    (2635, 2655, 'FOREIGN ORG'),
    (3530, 3550, 'FOREIGN ORG'),
    (2072, 2093, 'FOREIGN ORG'),
    (3458, 3479, 'FOREIGN ORG'),
    (1972, 1995, 'FOREIGN ORG')]}),
 (' DIAMAX, A PROPRIETORSHIP FIRM, HAVING ADDRESS  404405, DAIMOND APPARTMENT, SAIYEDPURA, NAGORIVAD, SURAT, GUJARAT, INDIA, 395003, CONTACT NUMBER 919825217325 AND EMAIL ID DIAMAX95@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 239705501101 WITH ICICI BANK LTD, SHUBHAM POINT, RUWALA TEKRA, HARIPURA BAGAL, LAL GATE, SURAT, GUJARAT, 95003, SINCE 23102019 DATE OF INCORPORATION IS 14122016 PAN OF THE ACCOUNT IS FOJPS7176R IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 5216926682 GSTIN AS PER BANK RECORDS IS 24FOJPS7176R1ZQ TRANSACTION LINKED ACCOUNT IS 005201025697 AUTHORIZED SIGNATORY OF THE REPORTED ACCOUNT IS DHRUMIL DHARNENDRA SHAH PROPRIETOR  FOJPS7176R THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, GSTIN KYC UPDATION IS NOT DUE CUSTOMER IS AN IMPORTER OF LAB GROWN IE SYNTHETIC DIAMONDS WHICH HE SELL IN LOCAL MARKET DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 25CR TO LESS THAN 100CR ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 28022021 WAS RS143 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 1,90817 LAKHS OF WHICH RTGS RS 1,89440 LAKHS MAJORLY FROM DIAMAX, VANSH GEMS, SHREEJI JEWELLERS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,92985 LAKHS OF WHICH RTGS RS 32851 LAKHS MAJORLY TO PARI IMPEX, DIAMAX, REMITTANCE RS 1,48813 LAKHS MAJORLY TO MARS VENTURE LIMITED, AARAS DIAMONDS LLC, ALICAT SCIENTIFIC INC FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 23102019 TO 31032020  TOTAL CREDIT SIDE RS 2,54341 LAKHS OF WHICH RS 0 BY CASH TOTAL DEBIT SIDE RS 2,52031 LAKHS OF WHICH CASH WAS RS 500 LAKHS FUNDS ARE FROM DIFFERENT ENTITIES AND ARE IMMEDIATELY DEBITED TOWARDS OUTWARD REMITTANCE FOR EXAMPLE, ON 21092020 ACCOUNT WAS CREDITED WITH RS 314 CR THROUGH RTGS FROM SHREEJI JEWELLERS 919020021152068, BANK AXIS IFSC UTIB0001705 AND ON SAME DURATION ACCOUNT WAS DEBITED WITH RS 308 CRORE THROUGH REMITTANCE TO MARS VENTURE LIMITED AT HONG KONG FOR PAYMENT AGAINST INVOICE ALSO ON 04092020 ACCOUNT WAS CREDITED WITH RS 158 CRORE THROUGH RTGS FROM DIAMAX50200019868089, BANK HDFC IFSC HDFC0000896, PURVI EXPORT, VANSH GEMS AND ON SAME DURATION ACCOUNT WAS DEBITED WITH RS 182 CRORE THROUGH REMITTANCE TO MARS VENTURE LIMITED AT HONG KONG FOR PAYMENT AGAINST INVOICE MAJORITY REMITTANCES MADE TO MARS VENTURE LIMITED OVERSEAS ENTITIES EXISTENCE CHECKED HOWEVER NO FURTHER INFORMATION AVAILABLE ON THEM REGARDING WHAT PRODUCT THEY SELL NO BUSINESS RELATED PAYMENTS ARE MADE FROM THIS ACCOUNT NAME BOARD AFFIXED OUTSIDE OFFICE WHICH IS PRINTED ON A4 SIZE PAPER ALSO THERE ARE CREDITS FROM NYSNA MULTIVENTRES PVT LTD IT IS ONE SUCH COMPANY WHICH HAS TRANSACTED WITH MANY SUSPICIOUS DIAMOND COMPANIES OF SURAT WHOSE ACCOUNTS WE HAVE CLOSED UNDER BANK INDUCED PROCESS IN VIEW OF LARGE VALUE FOREX OUTWARD REMITTANCE IN THE ACCOUNT TO A SUSPICIOUS OVERSEAS COUNTER PARTY, HIGH TURNOVER AND NEGATIVE FINDINGS IN TERMS OF TRANSACTIONS WITH A DOMESTIC SUSPICIOUS ENTITY, STR IS BEING FILED',
  {'entities': [(1577, 1598, 'FOREIGN ORG'),
    (1557, 1575, 'FOREIGN ORG'),
    (1535, 1555, 'FOREIGN ORG'),
    (2181, 2201, 'FOREIGN ORG'),
    (2473, 2493, 'FOREIGN ORG'),
    (2564, 2584, 'FOREIGN ORG')]}),
 ('ROUTING OF LARGE VOLUME OF FUNDS WERE OBSERVED IN ACCOUNT LEADING TO HIGH TURNOVER WHICH SEEMS UNUSUAL IN VIEW OF, LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS, STR IS BEING FILEDWANDERLAND REAL ESTATE PRIVATE LIMITED, SMEAGPRIVATE LTD COMPANY , HAVING ADDRESS6TH FLR TREASURE ISLAND,11 TUKOGANJ MAIN,RD, INDORE, MADHYAPRADESH, INDIA, 452001, CONTACT NUMBER 917389909730 IS MAINTAINING A CURRENT ACCOUNT NUMBER 004105017744 WITH ICICI BANK LTD, INDORE  MALAV PARISAR, ICICI BANK LTD, , 4,CHHOTI KHAJRANI, MALAV PARISAR, INDORE, INDORE, MADHYA PRADESH, 452008, SINCE 30122019 DATE OF INCORPORATION IS 24022006 PAN NUMBER OF THE ACCOUNT IS AAACW6397F OTHER RELATIONSHIP OF CUSTOMER ARE 004105015785, 004105015988, 004105015989, 004105015990, 004105015991, 004105017765 TRANSACTION LINKED ACCOUNTS ARE 004105017765 ,182005000938 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE VINAYAK KALANI AUTHORISED SIGNATORY ATGPK8379P ,QURESH YUSUF MATKAWALA AUTHORISED SIGNATORY AGSPM2294A ,YOGESH GOSWAMI AUTHORISED SIGNATORY ACFPG9032K ,NIRMAL KUMAR SHRIVASTAVA DIRECTOR, KAPIL GOPLANI DIRECTOR BSEPG3954N THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN  AAACW6397F, MOAAOA, BOARD RESOLUTION ETC KYC UPDATION IS NOT DUE AS PER ENHANCED DUE DILIGENCE CUSTOMER DEALS IN EXPORT OF PP BAGS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 25CR TO LESS THAN 100CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS BALANCE IN THE ACCOUNT AT 31122020 WAS RS391 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 31122020 SHOWS TOTAL CREDIT RS 10,08182 LAKHS OF WHICH TRANSFERS RS 100 LAKH MAJORLY FROM WANDERLAND REAL ESTATES PVT LTD, CLEARING RS 24083 LAKHS MAJORLY FROM NEFT,WANDERLAND REAL ESTATE PRIVATE,CMS17323020741, NEFT RS 167 LAKHS MAJORLY FROM TATWA PLASTCO EXTRUSIONS PVT,MAERSK LINE INDIA PRIVATE LI, REMITTANCE RS 9,79687 LAKHS MAJORLY FROM FLEXI GLOBAL PTE LTD 7500A BE,PANKTI INTERNATIONAL PTE LTD,DHAMPUR INTERNATIONAL PTE LT, INTERNET FUND TRANSFERS RS 4143 LAKHS, , AND OTHERS RS 002 LAKHS  TOTAL DEBITS DURING THE SAME PERIOD WAS RS 10,30916 LAKHS OF WHICH , TRANSFERS RS 2583 LAKHS MAJORLY TO WANDERLAND REAL ESTATE PRIVATE LIMITED, CLEARING RS 3,65698 LAKHS, RTGS RS 6,30600 LAKHS MAJORLY TO FLEXITUFF VENTURES IND,FLEXITUFF VENTURES INT,SBIN0008283, REMITTANCE RS 1526 LAKHS, MOBILE MONEY TRANSFERS RS 931 LAKHS IN 21 INSTANCES , , AND OTHERS RS 819 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 30122019 TO 31032020  TOTAL CREDIT SIDE RS 4,63706 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 4,40580 LAKHS OF WHICH CASH WAS NIL TRANSACTION PATTERN SHOWS FUNDS WERE CREDITED THROUGH INWARD REMITTANCES AND THOSE FUNDS WERE DEPLOYED TOWARDS OTHER ENTITIES FOR EXAMPLE, ON 30122020, RS 5531LAKHS CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE LTD 3 SHENTON WAY SHENTON HOUSE 1002 SINGAPORE WHERE THE PURPOSE MENTIONED AS PAYMENT OF IMPORT BILL AND ON 30122020, RS 85LAKHS DEBITED IN PARTS THROUGH RTGS TO FLEXITUFF VENTURES INTERNATIONAL LTD  ACCOUNT NO 65064340814 IFSC SBIN0030340 ALSO ON 24122020, RS 20LAKHS CREDITED THROUGH TRANSFER FROM NANOFIL TECHNOLOGIES PRIVATE LIMITED182005000938 WHERE THOSE FUNDS WERE CREDITED THROUGH INWARD REMITTANCES FROM PANKTI INTERNATIONAL PTE LTD 3 SHENTON WAY SHENTON HOUSE 1002 SINGAPORE WHERE THE PURPOSE MENTIONED AS EXPORT BILLS REALN COLLN AND ON 24122020, RS 104CRORES DEBITED IN PARTS THROUGH CMS TRANSFER AS PAYMENTS ALSO ON 05012021, RS 3385 LACS WERE CREDITED BY REMITTANCE FROM FLEXI GLOBAL PTE LTD 7500A BE, SINGAPORE AND ON SAME DAY ENTIRE AMOUNT WAS DEBITED IN 2 PARTS BY RTGS TOWARDS FLEXITUFF VENTURES INTERNATIONAL LTD ACCOUNT NO 65064340814 IFSC SBIN0030340',
  {'entities': [(2058, 2078, 'FOREIGN ORG'),
    (3760, 3780, 'FOREIGN ORG'),
    (2088, 2116, 'FOREIGN ORG'),
    (3050, 3078, 'FOREIGN ORG'),
    (3488, 3516, 'FOREIGN ORG'),
    (2117, 2142, 'FOREIGN ORG'),
    (3237, 3273, 'FOREIGN ORG'),
    (3870, 3906, 'FOREIGN ORG')]}),
 (' BHANU VYAPAAR PVT LTD, A PRIVATE LTD COMPANY, HAVING ADDRESS 687 ANANDAPUR, EM BYPASS 6TH FLR, KOLKATA, WEST BENGAL, INDIA, 700107, CONTACT NUMBER 003366136264 AND EMAIL ID RAJESHAGARWAL@EMANIGROUPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 000605022409 WITH ICICI BANK LTD, 22, SIR RN MUKHERJEE ROAD, KOLKATA, WEST BENGAL, 700001, SINCE 25032011 DATE OF INCORPORATION IS 29081988 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AABCB1334Q OTHER RELATIONSHIP OF CUSTOMER ARE 000605000214, 000605015965, 000605023476, 000605027592, 000605027975, 000605029093, 000605029495, 000605031270 TRANSACTION LINKED ACCOUNTS ARE 000605013763, 000605012395 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE MOHAN GOENKA PTR ADZPG7446G, MOHAN GOENKA AUTHORISED SIGNATORY ADZPG7446G, MANISH GOENKA DIRECTOR ADXPG3598B, MANISH GOENKA AUTHORISED SIGNATORY ADXPG3598B, RADHE SHYAM GOENKA DIRECTOR AEGPG8285B THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN  AABCB1334Q, MOHAN GOENKA ADZPG7446G, MANISH GOENKA ADXPG3598B, RADHE SHYAM GOENKA AEGPG8285B, COI U15520WB1988PTC045075, MOA, AOA, BOARD RESOLUTION ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT BALANCE IN THE ACCOUNT AT 31032021 WAS RS024 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS 1,28,18737 LAKHS OF WHICH TRANSFERS RS 32,21766 LAKHS MAJORLY FROM PREMIER FERRO ALLOYS AND SECURITIES LTD, EMAMI REALTY LIMITED, EMAMI AGROTECH LIMITED, CLEARING RS 82477 LAKHS MAJORLY FROM CITIBANK DD, NEFT RS 18,92062 LAKHS MAJORLY FROM BHANU VYAPAAR PRIVATE LIMITE, PYRITE POWER PRIVATE LIMITED, EMAMI FRANK ROSS LTD, RTGS RS 43,39892 LAKHS MAJORLY FROM AMRI HOSPITALS LTD, IIFL WEALTH FINANCE LIMIT, DEUTSCHE INVESTMENTS, INTERNET FUND TRANSFERS RS 31,60798 LAKHS FROM RAVIRAJ VINIYOG PRIVATE LIMITED TOTAL DEBIT SIDE RS 1,28,26841 LAKHS OF WHICH TRANSFERS RS 79762 LAKHS MAJORLY TO MIDKOT INVESTMENTS PRIVATE LIMITED, BENGAL NRI COMPLEX LTD, EMAMI CAPITAL MARKETS LIMITED, RTGS RS 50,42555 LAKHS MAJORLY TO EMAMI REALTY LIMITED, AMRI HOSPITALS LTD, DEUTSCHE I, INTERNET FUND TRANSFER RS 76,92001 LAKHS TO EMAMI REALTY LIMITED FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 1,04,09421 LAKHS TOTAL DEBIT SIDE RS 1,04,02881 LAKHS OF WHICH REMITTANCE RS 356 LAKHS FY 201819 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS 1,22,50645 LAKHS TOTAL DEBIT SIDE RS 1,22,57264 LAKHS FY 201718 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS 37,34371 LAKHS TOTAL DEBIT SIDE RS 37,27385 LAKHS OF WHICH CASH WAS RS 050 LAKHS THIS IS THE FIRST REPORTING OF THE CUSTOMER CUSTOMER MANUFACTURES BEVERAGES THE COMPANY OFFERS FOOD PRODUCTS CONSUMER STAPLES FREQUENT HIGH VALUE CREDITS FOLLOWED BY DEBITS SEEMS LIKE ROUTING OF FUNDS AMONG MULTIPLE INDIVIDUALS AND ENTITIES FOR EXAMPLE, ON 04122020, RS 640 LACS CREDITED THROUGH RTGS NAME EMAMI REALTY LTD ACCOUNT NO 5412125198 IFSC KKBK0000958 AND ON SAME DAY, RS 637 CRS DEBITED THROUGH RTGS NAME RADHE SHYAM GOENKA ACCOUNT NO 03230020001119 IFSC KKBK0000323 IN OTHER EXAMPLE, ON 08022021, RS 408 CRS CREDITED BY TRANSFERS FROM 000605017595RAVIRAJ VINIYOG PRIVATE LIMITED AND ON SAME DAY, RS 400 CRS DEBITED BY TRANSFERS TO 000605013763EMAMI REALTY LIMITED SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED LIKE ROUTING OF FUNDS IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT, ROUTING OF FUNDS, STR IS BEING FILED',
  {'entities': [(1627, 1666, 'FOREIGN ORG'),
    (1668, 1688, 'FOREIGN ORG'),
    (2273, 2293, 'FOREIGN ORG'),
    (2371, 2391, 'FOREIGN ORG'),
    (3440, 3460, 'FOREIGN ORG'),
    (1690, 1712, 'FOREIGN ORG')]}),
 ('16 SHIPMENT MADE EXPORT BILLS ARE PENDING FOR THE SUBMISSION OUT OF WHICH 5 BILLS WERE OVERDUE AND 1 IS IN CAUTIONED LISTED FOR MORE THAN TWO YEARS SEEMS SUSPICIOUS IN VIEW OF BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE, STR IS BEING REPORTEDKGR RIGS AND MINING EQUIPMENT, PARTNERSHIP, HAVING ADDRESS PLOTNO2058 2059, IDA PHASEII, CHERLAPALLY, HCL POST, RANGAREDDI, HYDERABAD, TELANGANA, INDIA, 500051, CONTACT NUMBER 917901344444 AND EMAIL ID GOVINDREDDYK@YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 131805500555 WITH ICICI BANK LTD, PLOT NO 13, HNO 102913, SUBHASH NAGAR, OPP JUNA NAGAR, KUSHAIGUDA, KAPRA, RANGAREDDY DIST, HYDERABAD, TELANGANA, 500062, SINCE 19122016 DATE OF INCORPORATION IS 13032006 NA PAN NUMBER OF THE ACCOUNT IS AAHFK5493N OTHER RELATIONSHIP OF CUSTOMER ARE 131805500990, 131813000917, 131813002050 AND 131855000006 TRANSACTION LINKED ACCOUNTS ARE 131805500146 AND 131805500275 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE GOVINDAREDDY KATIREDDY PARTNER ADFPK0552K AND KATIREDDY NITHIN REDDY PARTNER DECPK8094F THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PARTNERSHIP DEED AND PAN DATE OF LAST KYC UPDATION 26092019 KYC UPDATION IS NOT DUEENTITY IS A MANUFACTURER AND TRADER OF DRILLING EQUIPMENT SUCH AS DRILLING RIGS, DTH HAMMERS, BUTTON BITS ETC DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 5 CRORES TO LESS THAN 25 CRORESALERT WAS TRIGGERED FOR MANUAL STR FOR BILLS PENDING FOR THE SUBMISSION WHICH ARE OVERDUE BALANCE IN THE ACCOUNT AT 28022021 WAS RS48454 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 4,97247 LAKHS OF WHICH CASH WAS RS 6 LAKHS, TRANSFERS RS 77330 LAKHS MAJORLY FROM KGR RIGS MINING EQUIPMENT, KGR INDUSTRIES, KATIREDDY GOVINDA REDDY, CLEARING RS 25300 LAKHS MAJORLY FROM BIMIDI MAHENDER, LAXMI NARASIMHA RAO, NEFT RS 59099 LAKHS MAJORLY FROM COMINT SYSTEMS AND SOLUTIONS, EPAO GST REFUNDS, MASTER DRILLERS, RTGS RS 1,01158 LAKHS MAJORLY FROM E PAO GST REFUNDS, BHAVIKA PLAS TEK PRIVA, NANO POLYMERS INDIA PVT LTD, REMITTANCE RS 1,58163 LAKHS MAJORLY FROM BAKORI MEGA SERVICES, ROYAL TECHNO INDUSTRIES LIMITED, PROACTIVE ENERGY TRADING , INTERNET FUND TRANSFERS RS 73465 LAKHS FROM KGR INDUCTION DRILL PIPES COMPANY 131805500239, MOBILE MONEY TRANSFERS RS 1496 LAKHS IN 11 INSTANCES, UPI TRANSFERS RS 635 LAKHS IN 8 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 5,00355 LAKHS OF WHICH CASH WAS RS 950 LAKHS BY KAPRA BRAN CH, TRANSFERS RS 1,03819 LAKHS MAJORLY TO SEKO BEC PRIVATE LIMITED, KGR RIGS MINING EQUIPMENT, GROC MINING AND CONSTRUCTION PRIVATE LIMITED, CLEARING RS 11641 LAKHS MAJORLY TO ELGI EQUIPMENTS LIMITED, TSSPDCL, NEFT RS 047 LAKHS MAJORLY TO OSMAN PIPES AND TUBE, RTGS RS 2,18962 LAKHS MAJORLY TO ELGI EQUIPMENTS LTD, ASHOK LEYLAND LIMITED, REMITTANCE RS 351 LAKHS, INTERNET FUND TRANSFER RS 1,48034 LAKHS TO SOUTHERN STEELS FORGINGS 665005021395, MOBILE MONEY TRANSFERS RS 230 LAKHS IN 8 INSTANCES, AND OTHERS RS 16322 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS WERE FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 3,75972 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 4,17261 LAKHS OF WHICH CASH WAS NIL FY 201819 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS 2,30871 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 2,31186 LAKHS OF WHICH CASH WAS NIL FY 201718 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS 1,71871 LAKHS OF WHICH CASH WAS RS 2239 LAKHSTOTAL DEBIT SIDE RS 1,76897 LAKHS OF WHICH CASH WAS RS 450 LAKHSAS PER THE TRANSACTION PATTERN, FUNDS ARE GETTING CREDITED FROM DOMESTIC AND INTERNATIONAL ENTITIES AND THOSE FUNDS ARE GETTING DEBITED TO CUSTOMER OTHER BANK ACCOUNT, OD ACCOUNT IS WITH US AND VARIOUS SUPPLIERS, VENDORS AND OPERATIONAL EXPENSES ETC SEEMS INLINE HOWEVER, CUSTOMER IS HAVING 57 ADVANCES RECEIVED, THERE ARE EXPORT BILLS ARE PENDING FOR SUBMISSION OUT OF WHICH 12 BILLS ARE OVERDUE ',
  {'entities': [(2796, 2819, 'FOREIGN ORG'), (2859, 2879, 'FOREIGN ORG')]}),
 ('INTERNET FUND TRANSFER TO WORLD ONE INDIA FOREX PVT LTD 629405043076 SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED LIKE ROUTING OF FUNDS IN VIEW OF LARGE VALUE NON CASH ROUTING OF TRANSACTIONS IN THE ACCOUNT WITH HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDWORLD ONE INDIA FOREX PRIVATE LIMITED, A PRIVATE LIMITED COMPANY, HAVING ADDRESS 351, JAWHARLALNEHRU ROAD, 1ST FLOOR, NEAR KAILASH BUILDING, ROOM NO9, KOLKATA, WEST BENGAL, INDIA, 700071, CONTACT NUMBER 919811450009 AND EMAIL ID MUKHERJEE@WORLDONEFOREXCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 105605003658 WITH ICICI BANK LIMITED, SECTOR 5, OMEGA BUILDING GROUND FLOOR, PLOT A2, M2 AND N2, BLOCK EP AND GP, SECTOR V, SALT LAKE, KOLKATA, WEST BENGAL, 700091, SINCE 30012017 DATE OF INCORPORATION IS 19092013 PAN OF THE ACCOUNT IS AABCW4315J OTHER RELATIONSHIP OF CUSTOMER IS 000205025388, 629405043076, 629413033198, 661405600516 TRANSACTION LINKED ACCOUNTS ARE 627505500677, 629405043076 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE SUPRIYO MUKHERJEE PRE AJXPM8144M, ESHWARY KRISHNAN PRE ALXPK0990P, DEBABRATA SAHA PRE BFEPS2920D, GOURANGA MUKHERJEE PRE BGSPM1795F, DIPEN BASAK PRE AKZPB7179C BENEFICIAL OWNER OF THE REPORTED ACCOUNT ARE ESHWARY KRISHNAN WITH PAN ALXPK0990P, SUPRIYO MUKHERJEE WITH PAN AJXPM8144M THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION AS PER ENHANCED DUE DILIGENCE, CUSTOMER PROVIDES A HOST OF SERVICES AROUND THE ENTIRE FOREX ECOSYSTEM APART FROM FOREIGN CURRENCIES, THE COMPANY DELIVERS TRAVEL MONEY CARDS, TRAVELLERS CHEQUES, INTERNATIONAL CALLING CARDS, AND TRAVELLERS INSURANCE TO CLIENTS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LACS TO BELOW 5 CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 31032021 WAS RS 100 LAKH CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS 11,18438 LAKHS OF WHICH TRANSFERS RS 6573 LAKHS MAJORLY FROM SHREA BURAL, TAJSHAYAMOLI FOREX AND TRAVELS PRIVATE LIMITED, CLARITY FINANCIAL SERVICES LTD, NEFT RS 6305 LAKHS MAJORLY FROM ESTELLE FOREX PRIVATE LTD, ORIENT EXCHANGE AND FINANCIAL, SHRESTHA FOREX AND TRAVELS PVT LIMITED, RTGS RS 1,69660 LAKHS MAJORLY FROM AHMED FOREX AND TRAVEL, BALARAM FOREX PVT LTD, GATIMAAN TOURS AND TRADING, INTERNET FUND TRANSFERS RS 9,34425 LAKHS FROM FLYGATE HINA FOREX AND TRAVELS PVT LTD ETC TOTAL DEBIT SIDE RS 11,18475 LAKHS OF WHICH RTGS RS 9,86122 LAKHS MAJORLY TO WORLD ONE INDIA FOREX, INTERNET FUND TRANSFER RS 93951 LAKHS TO WORLD ONE INDIA FOREX PVT LTD ETC, MOBILE MONEY TRANSFERS RS 34764 LAKHS IN 183 INSTANCES FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 31,66882 LAKHS OF WHICH CASH WAS RS 323 LAKHS, REMITTANCE RS 11042 LAKHS TOTAL DEBIT SIDE RS 31,66855 LAKHS OF WHICH RS 0 BY CASH, REMITTANCE RS 6698 LAKHS FY 201819 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS 9,93199 LAKHS OF WHICH CASH WAS RS 2814 LAKHS, REMITTANCE RS 26989 LAKHS TOTAL DEBIT SIDE RS 9,93455 LAKHS OF WHICH RS 0 BY CASH, REMITTANCE RS 31181 LAKHS FY 201718 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS 2,21130 LAKHS OF WHICH CASH WAS RS 28966 LAKHS, REMITTANCE RS 15707 LAKHS TOTAL DEBIT SIDE RS 2,20946 LAKHS OF WHICH CASH WAS RS 700 LAKHS, REMITTANCE RS 19814 LAKHS TRANSACTION PATTERN SHOWS, HIGH VALUE NON CASH CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE, ON 29032021, RS 1462 LACS AMOUNT OF FUND CREDITED THROUGH INTERNET FUND TRANSFERS FROM PLUTO FOREX PRIVATE LIMITED 083705500902 AND ON THE SAME DAY RS 1450 LACS WERE DEBITED THROUGH RTGS TOWARDS WORLD ONE INDIA FOREX PRIVATE LIMITED, ACCOUNT 00030330020071, IFSC HDFC0000003 ALSO ON 16032021 RS 1099 LACS WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM JMD FOREOGN EXCHANGE PVT LTD 008905010485 AND ON THE SAME DAY RS 11 LACS WERE DEBITED THROUGH',
  {'entities': [(2325, 2355, 'FOREIGN ORG'), (2416, 2445, 'FOREIGN ORG')]}),
 (',ACCOUNT OPENED ON 21092012, CURRENT STATUS OF THE ACCOUNT IS 17032021TRANSACTIONS NOTICED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES, CHEQUE DEPOSITS, IMPS CREDITS, CASH DEPOSITS, ETC, CREDITS IN THE ACCOUNT VIA VARIOUS MODES ARE UTILISED VIA RTGSNEFT TO LOCAL PARTIES IMMEDIATELY BANK FILED STR ON 02112018 FIU BATCH ID 1701309216 9RELATED ACCOUNTS ARE VALIDATED VIA UCIC 52580533 & DIRECTOR MR SUSHIL JAIN IS ALSO PROPRIETOR IN THE FIRMS SHANTI OVERSEAS 201001424530 & SUNSHINE ENTERPRISES 259999858584 DIRECTOR MRRISHABH JAIN IS ONE OF THE PROPRIETORDIRECTOR IN THE BULK CIRCLE LLP 259899920203, SHALINI TRADEWELL INDIA PVT LTD259599911666INR,201002328431 GBP,201002328417EUR, 201002328400 USD & SHALINI ENTERPRISES 201000675197 RELATED ACCOUNT DETAILS 1SHANTI OVERSEAS 201001424530 OPENED ACCOUNT WITH US ON 06072017CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS 22052020TOTAL CREDIT TURNOVER AS ON DATE IN THE ACCOUNT IS RS6428CRORES,AVAILABLE BALANCE IN THE ACCOUNT IS RS5,39739TRANSACTIONS NOTICED ARE HIGH VALUE CREDITS FROM E PAO GST REFUNDS,DUTY DBK DISBURSAL ACCOUNT & RTGSNEFT FROM LOCAL PARTIES, INWARD REMITTANCES ARE UTILISED IMMEDIATELY BANK FILED STR ON 17032018 VIA FIU BATCH ID 1803199020 2SUNSHINE ENTERPRISES 259999858584, ACCOUNT OPENED ON 22062016CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS 20012020AVAILABLE BALANCE IN THE ACCOUNT IS RS000 CREDIT BANK FILED STR ON 30012017 VIA FIU BATCH ID 1701309221 3BULK CIRCLE LLP 259899920203, ACCOUNT OPENED ON 16112017,CURRENT STATUS OF THE ACCOUNT CLOSED ON 14012021 4SHALINI TRADEWELL INDIA PVT LTD259599911666INR, 201002328431 GBP, 201002328417EUR, 201002328400USD,ACCOUNT OPENED ON 29092017,CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS 24122020AS ON 17032021 TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RS14825CRORES & AVAILABLE BALANCE IN THE ACCOUNT IS RS8,51695 CREDITTRANSACTIONS OBSERVED ARE HIGH VALUE CREDITS VIA E PAO GST REFUNDS & INWARD REMITTANCES UTILISED TO MAKE IMMEDIATELY RTGS TO LOCAL PARTIES, DESPITE OF VOLUMES CLIENT IS HOLDING VERY MINIMAL BALANCE IN THE ACCOUNT, ACCOUNT WAS UTILISED TO ROUTING FUNDS IMMEDIATELY TO OTHER LOCAL PARTIES & KEEPING MINIMAL BALANCE IN THE ACCOUNT BRINGS SUSPICION IN THE ACCOUNT PATTERN 5SHALINI ENTERPRISES 201000675197, ACCOUNT OPENED WITH US ON 24082016CURRENT STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS 20012020AS ON 17032021 TOTAL CREDIT TURNOVER IN THE ACCOUNT IS RS12101CRORES & AVAILABLE BALANCE IN THE ACCOUNT IS RS097 CREDIT BANK FILED STR ON 29122016 VIA FIU BATCH ID 1612298525 HIGH VALUE CASH DEPOSITS AMOUNTING TO INR 1377 CRORES WERE EVIDENCED SINCE 15TH NOV 2016 IN THE ACCOUNT NO OTHER CREDITS OTHER THAN CASH DEPOSITS WERE EVIDENCED IN THE ACCOUNT TURNOVER IN THE ACCOUNT HAS SURPASSED THE DECLARED TURNOVER 10MULTIPLE CURRENT ACCOUNTS ARE OPENED TOWARDS ROUTING FUNDS 11CREDITS RECEIVED IN THE ACCOUNT VIA INWARD REMITTANCES ARE UTILISED TO MAKE LOCAL PARTIES PAYMENTS VIA RTGSNEFT SAME ACCOUNT PATTERN EVIDENCED IN THE ALL THE RELATED ACCOUNTS HIGH TURNOVER IN THE ACCOUNT HOWEVER VERY MINIMAL BALANCE MAINTAINED IN THE ACCOUNT SUSPICION RAISED DUE TO BELOW MENTIONED RED FLAGS 1UNABLE TO ASCERTAIN OVERSEAS PARTY PROFILE THROUGH WEB SEARCH 2MAJORITY OF INWARD REMITTANCES TOWARDS EXPORT OF MOBILE PHONES RECEIVED FROM LOGISTICS FIRMS 3IMMEDIATE UTILISATION OF HIGH VALUE RTGSNEFT CREDITS, INWARD REMITTANCE PROCEEDS TO MAKE RTGSNEFT TO LOCAL PARTIES 4 HIGH CREDIT TURNOVER IN A NEWLY INCORPORATED ACCOUNT 5 MULTIPLE CURRENT ACCOUNTS ARE OPENED FOR ROUTING FUNDS FOR MAKING HIGH VALUE TRANSACTIONS 6 DESPITE HIGH CREDIT TURNOVER IN RELATED ACCOUNT VERY MINIMAL BALANCE MAINTAINED 7 FUNDS TRANSFER BETWEEN STR FILED ACCOUNTS 8STR FILED FOR THE RELATED ACCOUNTS TOWARDS SERIOUS RED FLAGSCLIENT HAS BEEN REVIEWED FROM AML PERSPECTIVE BASIS HIGH VALUE INWARD REMITTANCES & OUR OBSERVATIONS ARE 1MS AVIK TELEVENTURES PVT LTD IS A PVT LTD FIRM INCORPORATED ON 10022020 AT FIRST FLOOR A 41,NARAINA INDUSTRIAL AREA PHASE 2, SOUTH WEST DELHI INDIA 110028 CLIENT OPENED CURRENT ACCOUNT 201004073711 WITH US ON 28052020 AT CHANDANI CHOWK BRANCH, NEW DELHI INDUSIND BANK LTD, MUNICIPAL NO 43 TO 46, LAXMI NIWAS BUILDING, CHANDNI CHOWK, DELHI 110006 AS ON 04052021, STATUS OF ACCOUNT IS ACTIVE, CUMULATIVE DEBIT IS RS 89,89,77,04161 & CUMULATIVE CREDIT IS RS 90,74,26,25137 & AVAILABLE BALANCE IS RS 84,49,20976 CREDIT LAST CUSTOMER INDUCED TRANSACTION DATE, AT THE TIME OF INVESTIGATION, IS 03052021 NATURE OF BUSINESS IS IMPORT EXPORT OF TELECOMMUNICATION ITEMS, MOBILETELEPHONE ITEMSELECTRICALELECTRONIC ITEMS DECLARED TURNOVER IS ABOVE RS10CRORES PER ANNUM DIRECTORS  MR SAHIL JAIN MOBILE9599448444 & MRRISHABH JAINMOBILE9599911666 PAN & IEC IS AATCA2839H RISK CATEGORY OF THE CLIENT IS HIGH & LAST KYC DONE ON 04122020 DOCUMENTS OBTAINED DURING ACCOUNT OPENING ARE PAN, BOARD RESOLUTION, CERTIFICATE OF INCORPORATION, ASSOCIATION MEMO ARTICLE, GST REGISTRATION FORM GSTIN IS 07AATCA2839H1ZC & CURRENT STATUS IS ACTIVE AS PER GST SITE 2AS ON 04052021, CLIENT HAD DONE 2 DIRECTADVANCE IMPORTS FOR RS4751LAKHS TOWARDS IMPORT OF OXYMETER, GAS SENSOR, OUTDOOR PM MONITOR, POLLUTION MONITOR, SQUAIR POLLUTION MONITOR,CAIR MONITOR TO OVERSEAS PARTIES RAINBOW COMMUNICATION CO & SHAANXI BOHU ZHICHUANG ELECTRONIC TECHNOLOGY CO BASED IN CHINA & HONGKONG UNABLE AS FETCH OVERSEAS PARTIES PROFILE THROUGH WEB SEARCH DURING THE SAME PERIOD CLIENT HAS RECEIVED 72 EXPORT ADVANCE FOR RS7632CRORES TOWARDS EXPORT OF MOBILE PHONES FROM THE OVERSEAS PARTIES BASED IN UAE & UNITED STATESMAJORITY OF REMITTANCES ARE RECEIVED FROM LOGISTICS FIRMS, RANDOM CHECK DONE ON ONE OF THE OVERSEAS PARTY PROFILE OF LOGIX ONE FZE, & PARTY IS INTO LOGISTICS SERVICES HTTPSCONNECTAEPROFILESDUBAILOGIXONEFZEPROFILEID=ER5044418 3 AS ON 30042021, CLIENT HAS ALL BOE OUTSTANDING AND 31 EXPORT ADVANCE OUTSTANDING4TOTAL CREDIT TURNOVER BETWEEN 040620 TO 03052021 IS RS9074CRORES WHICH INCLUDES TOTAL INWARD REMITTANCES RECEIVED FOR RS7632CRORES, OTHER CREDITS DETAILS ARE ATOTAL RTGS CREDITS RECEIVED FOR RS887CRORES, OF WHICH RS744CRORES RECEIVED FROM E PAO GST REFUNDS, ACCOUNT WITH STATE BANK OF INDIA, ACCOUNT NO 31860389541 & RS3350LAKHS RECEIVED FROM INDIVIDUAL ACCOUNT RAJ KUMAR SANJEEV KUM ACCOUNT WITH BANDHAN BANK LIMITED, ACCOUNT NO 10180004285866 BMAJORITY OF INWARD REMITTANCES ARE RECEIVED FROM LOGISTICS FIRM5 TOTAL DEBIT TURNOVER BETWEEN 180620 TO 29042021 IS RS8989CRORES A RS8258CRORES VIA RTGSNEFT,OF WHICH MAJORLY SENT TO CLIENT OWN ACCOUNT FOR RS5405CRORES, HELD ACCOUNT WITH AU SMALL FINANCE BANK LIMITED2021210326967220, RS470CRORES TO BLUE BERRY, ACCOUNT WITH ICICI BANK85405500721 & RS567CRORES TO HEMANG ELECTRONICS ACCOUNT WITH PUNJAB NATIONAL BANK4184002100033610 & LAXMI VILAS BANK 693360000000416 B TOTAL TRANSFER DEBIT FOR RS169CRORES OF WHICH MAJORLY SENT TO YASHOM INTERNATIONAL TRADERS PVT LTDACCOUNT NO 201003003290 FOR RS7512LAKHS & RS7855LAKHS TO JAY JALARAM ENTERPRISE ACCOUNT NO 650001403814 YASHOM INTERNATIONAL TRADERS PVT LTD201003003290, ACCOUNT OPENED ON 07012019 STATUS OF THE ACCOUNT IS ACTIVE LAST CUSTOMER INDUCED TRANSACTION IS 16032021 TOTAL CREDIT BALANCE IN THE ACCOUNT IS RS1843CRORESAVAILABLE BALANCE AS ON DATE IS RS83,63849 CREDITTRANSACTIONS OBSERVED ARE HIGH VALUE RTGS CREDITS FROM LOCAL PARTIES UTILISED VIA RTGSNEFT TO LOCAL PARTIES IMMEDIATELY, DESPITE OF HIGH VOLUMES CLIENT HOLDING MINIMAL BALANCE IN THE ACCOUNT, BRINGS MORE SUSPICION IN THE ACCOUNT PATTERN UNABLE TO ASCERTAIN RELATIONSHIP BETWEEN OUR CLIENT & YASHOM INTERNATIONAL TRADERS PVT LTD C JAY JALARAM ENTERPRISEACCOUNT NO 650001403814',
  {'entities': [(5741, 5754, 'FOREIGN ORG'),
    (476, 496, 'FOREIGN ORG'),
    (1244, 1264, 'FOREIGN ORG')]}),
 (' A FEW MORE ACCOUNTS FROM WHICH INWARD REMITTANCE ARE CREDITED 10 F A A S GENERAL 4TH FLOOR OFFICE NO 65 PALLA DIUM UAE , 11 L D K TEXTILE TRADING LLC DUBAI UAE , 12 LOGUS TRADING PO BOX 46277 DUBAI UAE , 13 LOTUS EXIM GENERAL LOTUS EXIM GENERAL TRADING UAE , 14 NIMBLE SMART P O BOX 14733 M018AL KAZIM UAE , 15 REVVER SIT GOODS P O BOX 183086 301301149 DUBAI UAE , 16 SCARCE TRADING LLC P O BOX 12870 904NAWRAS TOWER UAE , 17 SULTAN BIN BANA TEXTILE DUBAI UAE , 18 TAZEEZ GENERAL AL SANAVIL BUILD,NEAR BANIYAS UAE , 19 1009 PETROLEUM GROCERY ACCOUNT 1009 N USA , 20 FLORIDA IMPORT 1009 N STATE ROAD 7 ROYAL USA , 21 MOHAMMAD R ISLAM 10200 FOX TRAIL RD S ROYAL PALM B USA , 22 SURE GENERAL TRADING INC 10200 FOX TRAIL RD S ROYAL PALM USA , 23 USBD IMPORT CO INC 2216 SHOMA DR ROYAL PALM BEACH USA RTGS DEBITS ARE SEEN TO  AC NO 12102000036227 OF SHIVAMANI IFSC IBKL0000012 , AC NO 93002109027456 OF TURNING FASHIONS PVT LTD IFSC PUNB0009300 , AC NO 811837185 OF POOJA SAREE IFSC KKBK0006566 , AC NO 627505027032 OF WORLDWIDE FREIGHT SOLUTIONS PRIVAT IFSC ICIC0006275 , AC NO 655705502751 OF BALAJI CREATIONS IFSC ICIC0006557 , AC NO 53660100014495 OF ASHIK ALLI UNUSH IFSC BARB0BUPGBX , AC NO 50200027002175 OF EXCLUSIVE IFSC HDFC0004120  HIGH VALUE RTGS CREDITS ARE SEEN FROM AC NO 31778422304 OF CBEC ICEGATE IFSCSBIN00104181 FULL NAME AND PERMANENT ADDRESS OF THE ACCOUNT HOLDERA ONE INTERNATIONAL 1ST FLOOR 16 C PARK LANE PARK STREET KOLKATA WEST BENGAL INDIA 700016; 2 DATE OF BIRTH16102019; 3 PAN NOABQFA2052H; 4 PASSPORT NO; 5 OTHER IDENTIFICATION DOCUMENTS; 6 IEC CODEABQFA2052H; 7 GSTIN19ABQFA2052H1ZD; 8 MOBILE NUMBERLANDLINE NO919883025660; 9 OTHER FACILITIES AVAILED; 10 RISK CATEGORYLOW RISK; 11 NAME AND DETAILS OF THE AUTHORISED SIGNATORIES IN THE ACCOUNTDEBASISH SWAIN WITH DOB 10MAY1987 AND PAN BWJPS3468J , ATIK AHMAD WITH DOB 12JUL1994 AND PAN BPMPA6668E ; 12 KYC COMPLIANCE STATUSCOMPLIED; 13 DATE OF LAST KYC UPDATION29102020; 14 PROFILENATURE OF BUSINESSPARTNERSHIP BUSINESS  EXPORTER OF WOMEN S SUIT AND SAREES; 15 ANNUAL INCOME AS DECLARED IN KYC; 16 ANNUAL TURNOVER OF BUSINESS AS DECLARED IN KYC2 CRORES; 17 BENEFICIAL OWNERS AS DETERMINED UNDER RULE 93 OF PMLA RULESDEBASISH SWAIN WITH DOB 10MAY1987 AND PAN BWJPS3468J , ATIK AHMAD WITH DOB 12JUL1994 AND PAN BPMPA6668E ; 18 BANK AND BRANCH DETAILSTHE FEDERAL BANK LTD, BRANCHKOLKATA  NEW MARKET,8 SUDDER STREET ,GROUND FLOOR NEW MARKET,KOLKATA,WEST BENGAL,700016; 19 BANK ACCOUNT NO13130200024776; 20 BANK ACCOUNT TYPECURRENT AC; 21 BANK AC OPENED ONACTIVATION DATE29102020; 22 BANK ACCOUNT STATUSACTIVE; 23 QUANTUM OF TRANSACTIONS FOR THE CURRENT YEAR FY FY 202122 DEBIT SIDE RS 85797 LAKH IN CASH RS 7 LAKHS AND CREDIT SIDE RS 8088 LAKH IN CASH RS 0 LAKHS; 24 QUANTUM OF TRANSACTIONS FOR PREVIOUS 3 YEARS FY FY 202021 DEBIT SIDE RS 370165 LAKH IN CASH RS 18 LAKHS AND CREDIT SIDE RS 378076 LAKH IN CASH RS 1 LAKHS; 25 BALANCE IN ACCOUNT ON DATE OF FILLING STR4945701; 26 DETAILS OF EARLIER STRS FILLED ON THE ACCOUNT HOLDERSRELATED PERSONS; 27 REACTIVE STRS; 28 ADVERSE MEDIA REPORT; 29 GROUNDS OF SUSPICIONHIGH VALUE FOREIGN REMITTANCES FOLLOWED BY IMMEDIATE RTGS DEBITS ARE REGULARLY SEEN IN THIS ACCOUNTA DETAILED ANALYSIS OF THE FOREIGN TRANSACTIONS WERE DONEFEW OF THE REMITTANCES FROM EDGEWORKS GENERAL TRADING LIMITED , DREAMWORTH GENERAL TRADING LIMITED , COSMIC EDGE GENERAL TRADING LIMITED BASED IN HONGKONG WHERE THE REMITTANCE NARRATION IS SEEN AS MS RUPAHALI BD, BANGLADESHS1021, FAJ INTERNATIONAL, BANGLADESHS1, ARR JOINT, BANGLADESH WITH PURPOSE CODES AS P103 IT IS INFORMED BY THE FIRM THAT RUPAHALI IS ITS IMPORTERS AND THE FORMER COMPANIES ARE BUYERS OTHER THAN CONSIGNEES HOWEVER IT IS KNOWN THAT RUPAHALI AND COMPANIES IN HONG KONG ARE SISTER CONCERNSBASED ON CHECKING THE INVOICES , IT IS SEEN THAT INVOICE AMOUNT IS GREATER THAN THE ACTUAL REMITTANCEWE SUSPECT THAT THE 8020 SCHEME WHEREIN MAJOR SHARE OF EXPORT PAYMENTS HAVE COME FROM HONG KONG BUT THE GOODS HAVE BEEN SHIPPED TO BANGLADESH TO DUTY EVASION AS IMPORT TARIFF RATE IN BANGLADESH IS 0% AND HONG KONG IS 1072% AS WE FIND VARIOUS DISCREPANCIES IN THE FOREIGN INWARD REMITTANCES, WE FILE STR ON TRADE BASED MONEY LAUNDERING TBML ANGLE UNDER FIUIND OFFLINE ALERT NO  TY 1024 AND TY 1014; 30 DETAILS OF INVESTIGATIONTHE VARIOUS FOREIGN INWARD REMITTANCES IN THIS ACCOUNT ARE FROM  1 COSMIC EDGE GENERAL HONGKONG , 2 DREAMWORTH GENERAL 01FL,266 TEMPLE STREET,YAU MA TEI, HONGKONG , 3 EDGEWORKS GENERAL 01FL, NO 266 TEMPLE STREET,YAU HONGKONG , 4 GLOBAL TIME ASIA LIMITED 13 F TUNG FAI BUILDING 27 HONGKONG , 5 KRS GLOBAL CO RM 1001 10F TOWER AHUNG HONGKONG , 6 AL GARHOUD GENERAL POBOX 46351,UNITED ARAB UAE , 7 CAPRICORN GOODS P O BOX 42836 87RASIS BUSINESS UAE , 8 ESOON TRADING LLC PO BOX NO 385126 DUBAI UA UAE , 9 EVERVENTIVE TRADING LLC PO BOX NO 384762 DUBAI UA UAE; ',
  {'entities': [(369, 387, 'FOREIGN ORG'),
    (125, 150, 'FOREIGN ORG'),
    (743, 761, 'FOREIGN ORG'),
    (4727, 4744, 'FOREIGN ORG'),
    (4344, 4372, 'FOREIGN ORG'),
    (4779, 4802, 'FOREIGN ORG'),
    (3271, 3288, 'FOREIGN ORG'),
    (4444, 4461, 'FOREIGN ORG'),
    (3307, 3341, 'FOREIGN ORG'),
    (3344, 3379, 'FOREIGN ORG'),
    (3473, 3490, 'FOREIGN ORG')]}),
 ("PARTNERSHIP DEED IS NOT REGISTERED OR NOT NOTARIZED IT IS A JUST DRAFT ON RS100 STAMP PAPER CREDIT IN ACCOUNT IS IMMEDIATELY FOLLOWED BY FOREIGN OUTWARD REMITTANCES AS PER WEB SITE OF VOEUX INTERNATIONAL PTE LTD LINE OF BUSINESS MENTIONED IS LOGISTIC SOLUTIONS PROVIDER AND OUR CLIENT LINE OF BUSINESS MEDIA AND ENTERTAINMENT PRODUCTION TAMS AND SERVICES BOTH ENTITIES BUSINESS ACTIVITY DOESN'T HAVE ANY RELATIONTHE ACCOUNT OF MSHADOW FILM PRODUCTS WAS OPENED ON FEBRUARY 25, 2021HIGH VALUE TRANSACTIONS ARE OBSERVED IN NEWLY OPENED ACCOUNT THE PARTNERSHIP FIRM HAS PAN AEIFS4409P, GSTIN NO 27AEIFS4409P1ZECOMPANY IS INTO BUSINESS OF AUDIO VISUAL & RELATED SERVICES LIKE MOTION PICTURE & VIDEO TAPE PRODUCTION, DISTRIBUTION & PROJECTION SERVICES, GRAPHIC DESIGNING UNREGISTERED PARTNERSHIP PARTNERSHIP AGREEMENT IS ON JUST RS100 STAMP PAPER AND NOT REGISTERED OR EVEN NOTARIZED TOTAL CREDITS AS UNDER RTGS OF RS 7,230,00000 FROM MADDOCK FILMS,YES BANK,RTGS OF RS 60, 75,00000 FROM MADDOCK FILMS, AXIS BANK, RTGS OF RS 14, 62,00000 FROM MADDOCK FILMS, IDFC BANKOUTWARD REMITTANCES DEBITS AS UNDERINR 1,46,01,02380USD 196,860 TO MS VOEUX INTERNATIONAL PTE LTD, OCBC BANK, SINGAPORE TOTAL CREDIT IN ACCOUNT FORM ACCOUNT OPENING RS 14,767,00000 IMMEDIATE TOTAL DEBIT IN ACCOUNT RS 14,605,74398AS PER WEB SITE OF MSVOEUX INTERNATIONAL PTE LTD VOEUXINTL PTELTDCOM LINE OF BUSINESS MENTIONED IS LOGISTIC SOLUTIONS PROVIDER, AND OUR CLIENT LINE OF BUSINESS MEDIA AND ENTERTAINMENT PRODUCTION ITEMS AND SERVICES BOTH ENTITIES BUSINESS ACTIVITY DOESN'T HAVE ANY RELATION",
  {'entities': [(184, 211, 'FOREIGN ORG'),
    (1130, 1157, 'FOREIGN ORG'),
    (1310, 1337, 'FOREIGN ORG')]}),
 ('3CUSTOMER HAS UNDERTAKEN 14 CROSS BORDER REMITTANCES AMOUNTING TO USD 104 MN  INR 763 CR TO PARTIES BASED OUT OF SINGAPORE AND HONG KONG HALCYON SEA AND OCEAN FREIGHTS  USD 297 LACS , HORIZON SHIPPING S PTE LTD  USD 158 LACS , NAVISTAR LOGISTICS LIMITED  USD 288 LACS , TRIWOOD LOGISTICS LTD  USD 303 LACS 4CUSTOMER ACCOUNT WAS SEEN RECEIVING INWARD CREDITS VIA RTGS FROM ASIA WORLD INTERNATIONAL TO A TUNE OF INR 637CR  INR 159CR FROM IDFB0040107 AND INR 47 CR FROM ICIC0000011 AND INR 15 CR FROM ICIC0000011 FOLLOWED BY CROSS BORDER REMITTANCES AND RTGS TO ASIA WORLD INTERNATIONAL INR 25 LACS ACCOUNT NO 020163400000821 WITH YES BANK 5STATED ANNUAL INCOME OF CUSTOMER ASIA WORLD INTERNATIONAL IS LESS THAN 50 CRORES HOWEVER IN LESS THAN 6 MONTHS FROM ACCOUNT OPENING DATE 16DEC2020 SAID CUSTOMER HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS 3347 CRORES TILL DATE6UPON REVIEWING THE TRANSACTION REFERENCE 594MISC210680070 FOR USD 7479046 FAVORING NAVISTAR LOGISTICS LIMITED HONGKONG FOR SPACE BOOKING FOR CONTAINERIZED SEA CARGO , IT WAS FOUND AS PER THE TRANSPORT DOCUMENT CNNSZIN VSP61135 THE GOODS ARE SHIPPED FROM NANSHA CY CHINA TO VISHAKHAPATNAM INDIA HOWEVER PAYMENT IS GOING TO HONG KONG7THERE IS NO INFORMATION ON ULTIMATE SELLER AND BUYER OF GOODS SINCE AS PER BILL OF LADING SHIPPER IS NAVISTAR LOGISTICS LIMITED, HONG KONG AND NOTIFY PARTY IS SEA LINERS SHIPPING PVT LTD8AS PER PUBLIC DOMAIN NAVISTAR LOGISTICS LIMITED, HONG KONG HAS 2 DIFFERENT WEBSITES VIZ HTTPSNAVISLOGISTICCOM AND HTTPSWWWNAVISTARLOGISTICSCOM HAVING SAME ADDRESS 1001, 10 FLOOR , BLOCK A ,HUNG HOM COMMERCIAL CENTRE , 39 MA TAI WAI ROAD, KOWLOON , HONG KONG9FURTHER SCRUTINY ON AGREEMENT COPY REVEAL THAT WHILE BUSINESS ADDRESS OF OVERSEAS ENTITY IS THE SAME AS ABOVE, WEBSITE WWWNAVSTARLOGISTICSCOM AND COMPANY LOGO PROVIDED ON THE LETTER HEAD OF AGREEMENT COPY DOES NOT MATCH INDICATING FABRICATED DOCUMENTS ARE BEING SUBMITTED TO THE BANK10WEBSITE ADDRESS WWWNAVSTARLOGISTICSCOM IS NONFUNCTIONAL AND AS PER PUBLIC DOMAIN INFORMATION SAID WEBSITE IS RECENTLY REGISTERED IN JANUARY 2021 AND PLACE OF REGISTRATION IS MAHARASHTRA REFERHTTPSWWWWHOISCOMWHOISNAVSTARLOGISTICSCOM11ANOTHER YES BANK CUSTOMER SEA LINERS SHIPPING PRIVATE LIMITED WAS SEEN SENDING CROSS BORDER REMITTANCE TO SAME OVERSEAS PARTY NAVISTAR LOGISTICS LIMITED, HONG KONG AND WITH SIMILAR RED FLAG INDICATORS STR WAS FILED IN APRIL 2021 VIDE FIU BATCH ID  2104260057  ALSO ANOTHER CUSTOMER CYCLONE GLOBAL LOGISTICS PRIVATE LIMITED WAS SEEN REMITTING TO HORIZON SHIPPING S PTE LTD AND STR WAS FILED IN SEPTEMBER 2020 VIDE FIU BATCH ID 2009159463 FURTHER WITH ABOVE REDFLAGS IT IS SUSPECTED THAT A NEWLY OPENED ACCOUNT IS UNDERTAKING OUTWARD REMITTANCE UNDER OTHER SHIPPING RELATED PAYMENTS TO SUSPECT SHELL COMPANIES IN HIGH RISK JURISDICTION, BY SUBMITTING FORGED TRANSACTION DOCUMENTS IN LIGHT OF THE ABOVE FACTS, IT HAS BEEN DECIDED ON 06MAY2021 TO FILE A STR ON THE SAID CUSTOMER1 THE CUSTOMER CITYZONE FREIGHT AND FORWARDERS PRIVATE LIMITED PAN AAICC3285C, DOI 14082019, COMMUNICATION AND PERMANENT ADDRESS A 101 NAVKAR CHAMBERS ANDHERI KURLA ROAD NR STAR PLUS OFFICE ANDHERI EAST MUMBAI MAHARASHTRA 400059, GSTIN 27AAICC3285C1ZR, REGISTRATION NUMBER 329283, CIN U63030MH2019PTC329283, MOBILE NUMBER 919820081814, EMAIL ID LOGISTICSTROJAN@GMAILCOM IS MAINTAINING A CURRENT AC NO 041981300001420 SINCE 05022021 WITH IFC BRANCH ADDRESS PART GROUND FLOOR, TOWER 2,,INDIABULLS FINANCE CENTRE,,SENAPATI BAPAT MARG LOWER PAREL,2 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN FEBRUARY2021 MAKRAND ARVIND DANDEKAR, ROHIT RAMESH SHINDE ARE AUTHORIZED SIGNATORY AND THE BENEFICIAL OWNER IN THE ACCOUNT CUSTOMER IS A SHIPPING SERVICES PROVIDER WITH AN ANNUAL GROSS TURNOVER GREATER THAN OR EQUAL TO 5 CR TO LESS THAN 15 CROREADDITIONALLY TWO ACCOUNTS ARE FOUND TO BE LINKED TO THE ACCOUNT OF CITYZONE FREIGHT AND FORWARDERS PRIVATE LIMITED AC NO041981300001420 BY COMMON EMAIL ADDRESS  LOGISTICSTROJAN@GMAILCOM ACCOUNT NUMBER 016263400001206 BY THE NAME OF TROJAN LOGISTICS PRIVATE LIMITED, HOWEVER THE ACCOUNT IS INACTIVE SINCE 18TH NOV 2019 ANOTHER ACCOUNT 020163400000821 OF ASIA WORLD INTERNATIONAL LINKED VIDE COMMON FUND TRANSFERS3 IN THE ACCOUNT OF CITYZONE FREIGHT AND FORWARDERS PRIVATE LIMITED AC NO 041981300001420 AND TWO LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM 01042021 TILL 06052021, SHOWS 2 CREDIT TRANSACTIONS AMOUNTING TO RS 1,025,000 AND 5 DEBIT TRANSACTIONS AMOUNTING TO RS 1,070,000OUT OF THE TOTAL CREDITS OF RS 1,025,000 IN THE ACCOUNTS, RTGSNEFT TRANSACTION AMOUNTS TO RS 1,000,000 CONSTITUTING 98 PERCENT OF THE TOTAL CREDITS, OUT OF THE TOTAL DEBITS OF RS 1,070,000 IN THE ACCOUNTS, CASH TRANSACTION AMOUNTS TO RS 155,000 CONSTITUTING AROUND 14 PERCENT OF THE TOTAL DEBITS, RTGSNEFT TRANSACTION AMOUNTS TO RS 900,000 CONSTITUTING AROUND 84 PERCENT OF THE TOTAL DEBITS4 IN THE ACCOUNT OF CITYZONE FREIGHT AND FORWARDERS PRIVATE LIMITED AC NO 041981300001420 AND TWO LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOWFROM 01042020 TILL 31032021 TRANSACTIONS ANALYSIS SHOWS 225 CREDIT TRANSACTIONS AMOUNTING TO RS 413,227,402 COMPRISING OF NIL CASH DEPOSIT AND 129 DEBIT TRANSACTIONS AMOUNTING TO RS 413,081,790 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 798,000FROM 01042019 TILL 31032020 TRANSACTIONS ANALYSIS SHOWS 1 CREDIT TRANSACTIONS AMOUNTING TO RS 100,000 COMPRISING OF NIL CASH DEPOSIT AND 1 DEBIT TRANSACTIONS AMOUNTING TO RS 100,000 COMPRISING OF NIL CASH WITHDRAWALNO TRANSACTION SEEN DURING 01042018 TO 31032019BALANCE IN THE ACCOUNT AS ON 05052021 IS RS 97,98569ON REVIEWING THE TRANSACTION PATTERN OF CITYZONE FREIGHT AND FORWARDERS PRIVATE LIMITED AC 041981300001420 AND LINK ACCOUNT TROJAN LOGISTICS PRIVATE LIMITED AC NO 016263400001206 IT WAS OBSERVED THAT WHILE THE ACCOUNT OF TROJAN LOGISTICS PRIVATE LIMITED IS INACTIVE SINCE NOVEMBER 2019 AND DO NOT HAVE ANY RED FLAGS, FOLLOWING ARE THE RED FLAGS AND OBSERVATION IN CITYZONE FREIGHT AND FORWARDERS PRIVATE LIMITED AC 0419813000014201CUSTOMER HAS OPENED AN ACCOUNT WITH BANK IN FEBRUARY 2021 AS SHIPPING SERVICE PROVIDER AND HAS UNDERTAKEN CROSS BORDER OUTWARD REMITTANCE ON ACCOUNT OF PAYMENT TO SHIPPING COMPANIES ABROAD, AND FREIGHT ON EXPORT OF SHIPPING COMPANIES 2IT IS IMPERATIVE TO NOTE THAT THE GSTIN NUMBER 27AAICC3285C1ZR LINKED TO CUSTOMERS PAN NO IS SHOWING AS CANCELLED SUOMOTO ON WEBSITE HTTPSSERVICESGSTGOVINSERVICESSEARCHTP?GSTINNO=27AAICC3285C1ZRPANNUM=AAICC3285C ',
  {'entities': [(184, 210, 'FOREIGN ORG'),
    (2508, 2534, 'FOREIGN ORG'),
    (227, 253, 'FOREIGN ORG'),
    (951, 977, 'FOREIGN ORG'),
    (1301, 1327, 'FOREIGN ORG'),
    (1408, 1434, 'FOREIGN ORG'),
    (2289, 2315, 'FOREIGN ORG'),
    (270, 291, 'FOREIGN ORG'),
    (372, 396, 'FOREIGN ORG'),
    (559, 583, 'FOREIGN ORG'),
    (671, 695, 'FOREIGN ORG'),
    (4148, 4172, 'FOREIGN ORG')]}),
 ('AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE ON 25022021 ACCOUNT GOT CREDITED FROM SAILAJA CHAPPIDI THROUGH TRANSFERS WITH 300 CRORES WHICH IS DEPLOYED ON SAME DAY WITH 200 CRORES TOWARDS VIRUPAKSHA ORGANICS LIMITED THROUGH TRANSFERS 000851000291 SIMILARLY, ON 16022021 ACCOUNT GOT CREDITED FROM GANGAVARAM RAMACHANDRA REDDY AND VEDAVATHI DEVI REDDY THROUGH TRANSFERS 111101500187 WITH RS1 CRORE WHICH IS DEPLOYED ON SAME DAY WITH SAME AMOUNT TOWARDS VIRUPAKSHA ORGANICS LTD THROUGH TRANSFERS 000851000291 SIMILAR TRANSACTION PATTERN SEEN IN THE ACCOUNT IT SEEMS THAT SAVINGS ACCOUNT WAS BEING MISUSED FOR ROUTING OF BUSINESS FUNDS TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 202021 WAS RS 1,87283 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, MISUSE OF SAVINGS ACCOUNT FOR ROUTING OF BUSINESS FUNDS AND HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDGANGAVARAM CHANDRA MOULISWAR REDDY, AN INDIVIDUAL, HAVING ADDRESS 83169143A 1ST FLOOR, SR NAGAR, SIDDARTHA NAGAR, NEAR KRISHNAKANTH PARK, HYDERABAD, ANDHRA PPRADESH, INDIA, 500038, CONTACT NUMBER 919848019749 AND EMAIL ID ACCTS@VIRUPAKSHACOM IS MAINTAINING A SAVING ACCOUNT NUMBER 111101500190 WITH ICICI BANK LTD, PLOT NO 238, KALYANNAGAR, HYDERABAD TELANGANA, 500038, SINCE 08042010 DATE OF BIRTH IS 03021963 PAN NUMBER OF THE ACCOUNT IS AEFPG2141F TRANSACTION LINKED ACCOUNTS ARE 000851000291, 007505501306 THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE AADHAR, PAN AEFPG2141F DATE OF LAST KYC UPDATION 25082020 CUSTOMER IS DIRECTOR IN VIRUPAKSHA ORGANICS LIMITED, VIRUPAKSHA SECURITIES PRIVATE LIMITED, VIRUPAKSHA LIFE SCIENCES LLP, VIRUPAKSHA LABORATORIES PRIVATE LIMITED, VECTOR BIO SCIENCES PRIVATE LIMITED DECLARED ANNUAL INCOME OF THE CUSTOMER DURING ACCOUNT OPENING WAS LESS THAN RS 25 LAKHS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 30042021 WAS RS246 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 30042021 SHOWS TOTAL CREDIT RS 3453 LAKHS OF WHICH TRANSFERS RS 1000 LAKHS MAJORLY FROM VEDAVATHI DEVI REDDY, NEFT RS 739 LAKHS MAJORLY FROM VIRUPAKSHA ORGANICS PVT LTD, RTGS RS 1700 LAKHS MAJORLY FROM VIRUPAKSHA ORGANICS PVT LTD TOTAL DEBITS DURING THE SAME PERIOD WAS RS 13492 LAKHS OF WHICH TRANSFERS RS 11500 LAKHS MAJORLY TO VEDAVATHI DEVI REDDY, VIRUPAKSHA ORGANICS LIMITED, CLEARING RS 1988 LAKHS MAJORLY TO MS MAK PROJECTS PVT LTDFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS 1,87283 LAKHS OF WHICH TRANSFERS RS 94450 LAKHS MAJORLY FROM VEDAVATHI DEVI REDDY, VIRUPAKSHA ORGANICS LIMITED, SAILAJA CHAPPIDI, CLEARING RS 22288 LAKHS MAJORLY FROM RAJAMOHANAREDDY P, NEERADA SYAM REDDY, SANDADI KALYANA KUMAR, NEFT RS 9176 LAKHS MAJORLY FROM VIRUPAKSHA ORGANICS PVT LTD, WESTERN UNION FINANCIAL SERVICES, RTGS RS 59300 LAKHS MAJORLY FROM VIRUPAKSHA LABORATORIES PRIVATE LIMITED, V INDAF OVERSEAS PRIVATE LIMITED, VIRUPAKSHA ORGANICS PVT LTD, INTERNET FUND TRANSFERS RS 2001 LAKHS FROM GANGAVARAM RAMACHANDRA REDDY TOTAL DEBIT SIDE RS 1,88741 LAKHS OF WHICH CASH WAS RS 800 LAKHS BY KALYANNAGARHYDERABAD BRANCH, TRANSFERS RS 1,14470 LAKHS MAJORLY TO SAI SURYA DEVELOPERS, VIRUPAKSHA ORGANICS LIMITED, VIRUPAKSHA INFRA DEVELOPERS PVT LTD, CLEARING RS 58320 LAKHS MAJORLY TO VIRUPAKSHA ORGANICS LTD, NAGASURI CHITFUNDS P LTD, M PADMAJA, NEFT RS 060 LAKHS MAJORLY TO ULTRATECH CEMENT LTD, RTGS RS 14261 LAKHS MAJORLY TO VIRUPAKSHA, VINDAFOVER, SRINIVAS R, REMITTANCE RS 496 LAKHS MAJORLY TO SRI LAKSHMI GANGAVARAM, CARD TRANSACTIONS RS 043 LAKHS FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 91409 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 85870 LAKHS OF WHICH CASH WAS RS 1000 LAKHS FY 201819 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS 58899 LAKHS OF WHICH CASH WAS RS 111 LAKHS TOTAL DEBIT SIDE RS 68129 LAKHS OF WHICH CASH WAS NIL FY 201718 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS 40917 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 40208 LAKHS OF WHICH CASH WAS RS 1100 LAKHS THE BANK HAD FILED EARLIER STR ON THE CUSTOMER S ACCOUNT NUMBER 111101500190 IN THE NAME OF GANGAVARAM CHANDRA MOULISWAR REDDY ON 18062019 VIDE FIU BATCH ID 1906180635, REPORT SERIAL NUMBER 252752',
  {'entities': [(327, 354, 'FOREIGN ORG'),
    (1805, 1832, 'FOREIGN ORG'),
    (2634, 2661, 'FOREIGN ORG'),
    (2972, 2999, 'FOREIGN ORG'),
    (3579, 3606, 'FOREIGN ORG')]}),
 ('AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE ON 29092020, RS 37 LAKHS WERE CREDITED THROUGH TRANSFERS FROM SHAPES APPARELS PRIVATE LIMITED 081605008265 AND ON SAME DAY SAME AMOUNT GOT DEBITED THROUGH TRANSFERS TO ART BEYOND PRIVATE LIMITED 081605009341SIMILARLY, ON 19032021 ACCOUNT GOT CREDITED FROM DAG PRIVATE LIMITED THROUGH RTGS ACCOUNT NUUMBER 650014025340, IFSC INDB0000168 WITH RS 35 LAKHS WHICH IS DEPLOYED ON SAME DAY WITH SAME AMOUNT TOWARDS SAKSHI ENTERPRISES PRIVATE LIMITED THROUGH RTGS ACCOUNT NUMBER 510101005786941, IFSC CORP0000501, SAKSHI ENTERPRISES PRIVATE LIMITED THROUGH NEFT ACCOUNT NUMBER 510101005786941, IFSC CORP0000501 SIMILAR TRANSACTION PATTERN SEEN IN THE ACCOUNT IT SEEMS THAT SAVINGS ACCOUNT WAS BEING MISUSED FOR ROUTING OF BUSINESS FUNDS IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 202021 WAS RS 57353 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, MISUSE OF SAVINGS ACCOUNT FOR ROUTING OF BUSINESS FUNDS AND HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDABHINAV ANAND, AN INDIVIDUAL, HAVING ADDRESS W126, GREATER KAILASH PART2,2ND FLOOR FLR, GREATER KAILASH SOUTH DELHI, INDIA, 110048, CONTACT NUMBER 919810044119 AND EMAIL ID ABHINAV@SHAPESINDIACOM IS MAINTAINING A SAVING ACCOUNT NUMBER 025301610250 WITH ICICI BANK LTD, SUPERTECH SHOPPRIX, C134 B ,SECTOR 61 NOIDA, UTTAR PRADESH, 201307, SINCE 07072018 DATE OF BIRTH IS 13111967 PAN NUMBER OF THE ACCOUNT IS AACPA8955A TRANSACTION LINKED ACCOUNTS ARE 000751000550, 081605008265,081605009341 THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE AADHAAR, PAN AACPA8955A CUSTOMER IS DIRECTOR IN DESIGNERS POINT INDIA PRIVATE LIMITED, SAKSHI ENTERPRISES PRIVATE LIMITED, TDM SOFTWARE DEVELOPMENT PRIVATE LIMITED, SAHEJ INFRASTRUCTURE PRIVATE LIMITED, SHAPES APPARELS PRIVATE LIMITED, SHAPES CLOTHING COMPANY PRIVATE LIMITED AND ART BEYOND PRIVATE LIMITED DECLARED ANNUAL INCOME OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 25 LAKHS TO RS 50 LAKHS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT BALANCE IN THE ACCOUNT AT 30042021 WAS RS030 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 30042021 SHOWS TOTAL CREDIT RS 1189 LAKHS OF WHICH TRANSFERS RS 143 LAKHS MAJORLY FROM HARPREET SINGH SACHDEVA, CLEARING RS 671 LAKHS MAJORLY FROM SAKSHI ENTERPRISES PRIVATE LIMITED, DESIGNERS POINT INDIA PVT LTD, RTGS RS 375 LAKHS MAJORLY FROM SAKSHI ENTERPRISES PRIVATE LIMITED TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1264 LAKHS OF WHICH CLEARING RS 019 LAKHS MAJORLY TO BSES RAJDHANI POWER LTD, INTERNET FUND TRANSFER RS 1245 LAKHS TO DESIGNERS POINT INDIA PVT LTD 000751000550FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 57353 LAKHS OF WHICH CASH WAS RS 500 LAKHS MAJORLY IN NOIDA SEC  61 BRANCH, TRANSFERS RS 26820 LAKHS MAJORLY FROM DESIGNERS POINT INDIA PVT LTD, SHAPES APPARELS PRIVATE LIMITED, SHAPES CLOTHING COMPANY PRIVATE LIMITED, CLEARING RS 11705 LAKHS MAJORLY FROM DESIGNERS POINT INDIA PVT LTD, NEFT RS 3823 LAKHS MAJORLY FROM AMIT CHOPRA, SHAPES CLOTHING CO PVT LTD, DAG PRIVATE LIMITED, RTGS RS 14500 LAKHS MAJORLY FROM SAKSHI ENTERPRISES P LTD, DAG PRIVATE LIMITED, ASHOK KUMAR ANAND TOTAL DEBIT SIDE RS 57253 LAKHS OF WHICH CASH WAS RS 095 LAKHS BY NOIDA SEC 61, GREATER KAILASH II BRANCH, TRANSFERS RS 9995 LAKHS MAJORLY TO ART BEYOND PRIVATE LIMITED, SHAPES APPARELS PRIVATE LIMITED, DESIGNERS POINT INDIA PVT LTD, CLEARING RS 1148 LAKHS MAJORLY TO POST MASTER, BSES RAJDHANI POWER LTD, AMIT CHOPRA, NEFT RS 558 LAKHS MAJORLY TO HARBANS MAHAJAN REA, SAKSHI ENTERPRISES PRIVATE LIMITED, SHAPES CLOTHING COMPANY PRIVATE LIMITED, RTGS RS 19560 LAKHS MAJORLY TO ASHIM ESTATES PRIVATE LIMITED, SHAPES CLOTHING COMPANY, DD GLOBAL CAPITAL PRIVATE LIMITED, REMITTANCE RS 2466 LAKHS MAJORLY TO NIDO SPITALFIELDS LTD RENT COLLECTION IN GBP TO UK FOR EDUCATION ABROAD, CITY UNIVERSITY LONDON IN GBP TO UK FOR EDUCATION ABROAD, INTERNET FUND TRANSFER RS 22666 LAKHS TO DESIGNERS POINT INDIA PVT LTD 000751000550, CARD TRANSACTIONS RS 310 LAKHS, MOBILE MONEY TRANSFERS RS 453 LAKHS IN 7 INSTANCESFY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 65316 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 65585 LAKHS OF WHICH CASH WAS RS 645 LAKHSFY 201819 FROM 07072018 TO 31032019  TOTAL CREDIT SIDE RS 23226 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 22953 LAKHS OF WHICH CASH WAS RS 475 LAKHSTHE BANK HAD FILED EARLIER STR ON THE CUSTOMER S ACCOUNT NUMBER 025301610250 IN THE NAME OF ABHINAV ANAND REPORT SERIAL NUMBER 246752 ON 14012019, BATCH ID S0000001193',
  {'entities': [(4208, 4229, 'FOREIGN ORG')]}),
 ('01042017 TO 31032018  TOTAL CREDIT SIDE RS 14,55768 LAKHS TOTAL DEBIT SIDE RS 14,60491 LAKHS OF WHICH CASH WAS RS 968 LAKHS, REMITTANCE RS 5084 LAKHS  TRANSACTION PATTERN SHOWS LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF FUNDS WHERE CREDITS ARE FOLLOWED BY IMMEDIATE DEBITS OF HIGH VALUE FUNDS FOR EXAMPLE, ON 25032021 RS 75 CRORES WERE CREDITED THROUGH INTERNET FUND TRANSFER FROM DN ENERGY PRIVATE LIMITED 602605501156 AND ON THE SAME DAY ACCOUNT WAS DEBITED WITH RS 6CRORES THROUGH CMS TO RILLICICI ALSO ON 26032021 RS 3 CRORES WERE CREDITED THROUGH CMS FROM REFEX INDUSTRIES LIMITED AND ON THE SAME DAY ACCOUNT WAS DEBITED WITH SAME AMOUNT THROUGH RTGS IN PARTS TO SIL ROOFTOP SOLAR POWER PVT LTD ACCOUNT NO 50200046348353 IFSC HDFC0000141 SIMILAR TRANSACTIONS WERE SEEN THROUGH THE ACCOUNT WHICH SEEMED UNUSUAL ALSO AS PER MEDIA REPORT, SEBI OBSERVED THAT DURING THE PERIOD OF JULY1, 2015 TO JULY 23, 2015 COMPANY HAD ALLEGEDLY VIOLATED THE PROVISIONS OF SEBI, SUBSTANTIAL ACQUISITION OF SHARES AND TAKEOVER SAST REGULATION, 2011HTTPSWWWSEBIGOVINENFORCEMENTORDERSJAN2018ADJUDICATIONORDERINRESPECTOFJAGDISHTJAINSHERISHATECHNOLOGIESPVTLTDTARACHANDJAINANILJAINUGAMDEVIJAINDIMPLEJAINANDSEEMAJAININTHEMATTEROFREFEXINDUSTRIESLIMITED37646HTML IN VIEW OF LARGE VALUE NON CASH ROUTING TRANSACTIONS STR IS BEING FILEDSHERISHA TECHNOLOGIES PVT LTD, PRIVATE LTD, HAVING ADDRESSOLD NO 56L NEW NO 102,11TH FLR,, VENKATNARAYANA ROAD,T NAGAR,, CHENNAI, TAMIL NADU, INDIA, 600017, CONTACT NUMBER 919962997139 AND EMAIL ID LALITHAU@REFEXCOIN IS MAINTAINING A CURRENT ACCOUNT NUMBER 602605054662 WITH ICICI BANK LTD, CHENNAI  T NAGAR, NEW NO40, BAZULLAH ROAD, T NAGAR, CHENNAI, TAMIL NADU, 600017, SINCE 31122014 DATE OF INCORPORATION IS 01102002 PAN NUMBER OF THE ACCOUNT IS AAHCS6471P IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0405032200 OTHER RELATIONSHIP OF CUSTOMER ARE 602605052818, 602605054938 TRANSACTION LINKED ACCOUNTS ARE 602605053837, 602605501156 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE LALITHA AUTHORISED SIGNATORY ACQPL1682F,TAPPI KUMAR DAS AUTHORISED SIGNATORY BDKPD6009C ,DINESH KUMAR AGARWAL AUTHORISED SIGNATORY AKHPA0067A  BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS A TARACHAND JAIN WITH PAN NO AAFPJ8044LT, ANIL JAIN WITH PAN NO AADPA8582A THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAHCS6471P, CIN U74110TN2002PTC049676, BOARD RESOLUTION, MOA ETC DATE OF LAST KYC UPDATION 13032020 KYC UPDATION IS NOT DUE ENTITY IS INTO MANAGEMENT CONSULTANCY SERVICES  DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS 40LAC TO LESS THAN 5CR ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN [CAA] ACCOUNTS BALANCE IN THE ACCOUNT AT 31032021 WAS RS1556 LAKHS CREDIT FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 92,98597 LAKHS OF WHICH CASH WAS RS 005 LAKHS MAJORLY IN TIRUPPUR 6154 BRANCH, TRANSFERS RS 3,22000 LAKHS MAJORLY FROM ARADHYAA INFRA HOLDINGS PRIVATE LIMITED,PAVAGADA PROCESSING INDUSTRIES PVT LTD, CLEARING RS 18,80201 LAKHS MAJORLY FROM SHERISHA SOLAR PVT LTD,REFEX INDUSTRIES LIMITED,DECCAN CHARTERS PVT LTD, NEFT RS 9,89019 LAKHS MAJORLY FROM MEDHA SERVO DRIVES PVT LTD,L T LTD,ILABS CAPITAL I LLP, RTGS RS 38,60383 LAKHS MAJORLY FROM DUGAR FINANCE AND INVE,REFEX ENERGY LIMITED,ILABS CAPITAL I LLP, REMITTANCE RS 962 LAKHS MAJORLY FROM VISTA INDIA DIGITAL MEDIA INC, INTERNET FUND TRANSFERS RS 21,81949 LAKHS FROM DN ENERGY PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 1356 LAKHS IN 19 INSTANCES , UPI TRANSFERS RS 005 LAKHS IN 2 INSTANCES , FIXED DEPOSIT RS 29987 LAKHS, AND OTHERS RS 32731 LAKHS TOTAL DEBIT SIDE RS 93,15984 LAKHS OF WHICH CASH WAS RS 600 LAKHS BY BRANCH, TRANSFERS RS 30700 LAKHS MAJORLY TO EMPLOYEES PROVIDENT FUND ORGANISATION, SHERISHA TECHNOLOGIES PVT LTD,KHAJANE II DEPT OF TRESURIES, CLEARING RS 21,88661 LAKHS, RTGS RS 58,33809 LAKHS MAJORLY TO ILABS CAPITAL I LLP,DUGAR FINANCE AND INVESTMENT, 1, REMITTANCE RS 1734 LAKHS MAJORLY TO GINLONG TECHNOLOGIES CO LTD,TUAH AND SUPARTO, INTERNET FUND TRANSFER RS 11,30966 LAKHS, CARD TRANSACTIONS RS 1509 LAKHS, MOBILE MONEY TRANSFERS RS 27162 LAKHS IN 246 INSTANCES , AND OTHERS RS 1,00843 LAKHS FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 60,78480 LAKHS OF WHICH CASH WAS RS 005 LAKHS, REMITTANCE RS 2117 LAKHS  TOTAL DEBIT SIDE RS 60,72485 LAKHS OF WHICH CASH WAS RS 1113 LAKHS, REMITTANCE RS 4802 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 34,88298 LAKHS OF WHICH CASH WAS RS 023 LAKHS, REMITTANCE RS 56583 LAKHS  TOTAL DEBIT SIDE RS 34,75483 LAKHS OF WHICH CASH WAS RS 863 LAKHS, REMITTANCE RS 66601 LAKHS FY 201718 FROM',
  {'entities': [(4184, 4211, 'FOREIGN ORG'),
    (579, 603, 'FOREIGN ORG'),
    (3282, 3306, 'FOREIGN ORG')]}),
 ('IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF FUND AND HIGH TURNOVER IN THE ACCOUNT, STR IS BEING FILEDA N DIAMONDS, A PROPRIETORSHIP FIRM, HAVING ADDRESSH N6, KUMBHAR SHERI,2ND FLOOR,1582,, MAHIDHARPURA,ROOM 1, SURAT, GUJARAT, INDIA, 395003, CONTACT NUMBER 919825119366 AND EMAIL ID ANDIAMONDS9@YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 750605000006 WITH ICICI BANK LTD, GROUND FLOOR, TOWER A, SWASTIK UNIVERSAL, UDHNA MAGDALLA ROAD, SURAT, GUJARAT, 395007, SINCE 20122019 DATE OF INCORPORATION IS 01042017 PAN NUMBER OF THE ACCOUNT IS AAZPD2576P GSTIN AS PER BANK RECORDS IS 24AAZPD2576P1ZX TRANSACTION LINKED ACCOUNTS ARE 750605500042, 750605000007 AUTHORIZED SIGNATORY PROPRIETOR OF THE REPORTED ACCOUNT IS ASHESH NANALAL DOSHI PAN AAZPD2576P  THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AAZPD2576P, AADHAAR, GST REGISTRATION CERTIFICATE HENCE, KYC UPDATION IS NOT DUE CUSTOMER IS INTO IMPORT AND EXPORT OF ROUGH DIAMONDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO RS 5 CRORES ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 28022021 WAS RS26894 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202021 FROM 01042020 TILL 28022021 SHOWS TOTAL CREDIT RS 5,89964 LAKHS OF WHICH CASH WAS RS 010 LAKHS MAJORLY IN SURATLH ROAD VARACHHA BRANCH, TRANSFERS RS 21925 LAKHS MAJORLY FROM JIGAR SHAH JT1, ASHESH NANALAL DOSHI, SUMO JEWEL, CLEARING RS 1220 LAKHS MAJORLY FROM KANCHANBEN VADILAL, VADILAL CHHOTALAL, NEFT RS 13789 LAKHS MAJORLY FROM A N DIAMONDS, NISHCHAL C DESAI HUF, RTGS RS 2,17457 LAKHS MAJORLY FROM AASHESH NANALAL DOSHI, VAISHALI GEMS, REMITTANCE RS 7913 LAKHS MAJORLY FROM STAR IMPEX HONG KONG, USD CURRENCY, INTERNET FUND TRANSFERS RS 3,25541 LAKHS FROM A N DIAMONDS 750605500042, MOBILE MONEY TRANSFERS RS 1775 LAKHS IN 27 INSTANCES , UPI TRANSFERS RS 025 LAKHS IN 1 INSTANCE TOTAL DEBITS DURING THE SAME PERIOD WAS RS 6,16897 LAKHS OF WHICH CASH WAS RS 12350 LAKHS BY BRANCH, TRANSFERS RS 32147 LAKHS MAJORLY TO SAHAJANAND TECHNOLOGIES PVT LTD, A N DIAMONDS,ANU IMPEX, CLEARING RS 19774 LAKHS MAJORLY TO TORRENT POWER LIMITED,MARUTI HARDWARE, NEFT RS 079 LAKHS MAJORLY TO RITESH B MEHTA,PADSALA VITTHALBHAI, RTGS RS 1,95345 LAKHS MAJORLY TO GIA INDIA LABORATORY PVT LTD, ASHESH NANALAL DOSHI, REMITTANCE RS 24171 LAKHS MAJORLY TO GIRDHARLAL DMCC IN USD TO UAE FOR PAYMT TOWARDS IMPORT SETTLEMENT OTHER THAN NEPAL BHUTAN DIAMONDS, INTERNET FUND TRANSFER RS 3,12272 LAKHS TO A N DIAMONDS 750605500042, MOBILE MONEY TRANSFERS RS 11394 LAKHS IN 227 INSTANCES AND OTHERS RS 9365 LAKHS DTAXFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 201920 FROM 20122019 TO 31032020 TOTAL CREDIT SIDE RS 44326 LAKHS OF WHICH CASH WAS NIL TOTAL DEBIT SIDE RS 44287 LAKHS OF WHICH CASH WAS RS 900 LAKHS AS PER TRANSACTION IT IS OBSERVED THAT FUNDS ARE BEING ROUTED THROUGH THIS ACCOUNT CREDITS IN THE ACCOUNT FOLLOWED BY IMMEDIATE DEBITS ON SAME OR NEXT DAY WITH SAME AMOUNT FOR EXAMPLE, ON 28122020, RS 27 LAKHS WERE CREDITED THROUGH RTGS FROM AASHESH NANALAL DOSHI ACCOUNT NO00000001355500365, IFSC CODECBIN0280512, BANK CENTRAL BANK OF INDIA AND ON SAME DAY RS 25 LAKHS THROUGH RTGS TO GIA INDIA LABORATORY PVT LTD ACCOUNT NO 15730330000021, IFSC CODE HDFC0001573, BANK HDFC ON 15062020, RS 87 LAKHS WERE CREDITED THROUGH INTERNET FUND TRANSFERS FROM A N DIAMONDS 750605500042 AND ON NEXT DAY RS 86 LAKHS WERE DEBITED THROUGH A N DIAMONDS 750605500042SIMILAR TRANSACTIONS ARE BEING REPEATED IN THE ACCOUNT, WHICH SEEMS TO BE ROUTING OF FUNDS IN THE ACCOUNT TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 202021 WAS RS 5,89964 LAKHS, WHICH SEEMS TO BE VERY HIGH',
  {'entities': [(2499, 2514, 'FOREIGN ORG')]}),
 ('BELOW ARE RED FLAGS OBSERVED 1OUT OF TOTAL 17 OUTWARD REMITTANCES VALUE RS222 CRORES PERTAINING TO ADVANCE AGAINST IMPORTS, CLIENT HAS NOT SUBMITTED BILL OF ENTRY PERTAINING TO 12 TRANSACTIONS OF VALUE RS163 CRORES PENDING SINCE NOVEMBER 20202IN THIS REGARD, YES BANK OFFICIALS HAD FOLLOWED UP WITH THE CUSTOMER ON MULTIPLE OCCASIONS AND HAD REQUESTED TO SHARE TRANSPORT DOCUMENTS FOR WHICH IMPORT REMITTANCES ARE BEING UNDERTAKEN, HOWEVER, CUSTOMER HAS TILL DATE NEITHER SUBMITTED BILL OF ENTRIES PERTAINING TO 12 TRANSACTIONS NOR HAS HE FURNISHED ANY TRANSPORT DOCUMENTS PERTAINING TO 17 OUTWARD REMITTANCES THUS UNDERTAKEN3AS PER RECENT COMMUNICATION, CLIENT HAS REPLIED STATING THAT SINCE HE NO LONGER WANTS TO TRANSACT VIA YES BANK, HE HAS SUBMITTED BILL OF ENTRY AND LODGED RELATED TRANSPORT DOCUMENTS WITH ANOTHER BANK4AS PER PUBLIC DOMAIN INFORMATION ON OVERSEAS ENTITIES NAFDA TRADING FZE, UAE AND CAMELOT OVERSEAS PTE LTD, SINGAPORE, BELOW RED FLAGS WERE OBSERVEDPUBLIC DOMAIN INFORMATION OF OVERSEAS ENTITY NAFDA TRADING FZE, UAE IS INSUFFICIENT MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS  101 KITCHENER ROAD, 0343, JALAN BESAE PLAZA, SINGAPORE208511 OF OVERSEAS ENTITY CAMELOT OVERSEAS PTE LTDCAMELOT OVERSEAS PTE LTD HAD PREVIOUSLY CHANGED NAMES MULTIPLE TIMES REFER HTTPSWWWCOMPANIESSGZHBUSINESS201936480HCAMELOTOVERSEASPTELTDWEBSITE OF CAMELOT OVERSEAS PTE LTD, SINGAPORE IS NEWLY REGISTERED IN THE YEAR OCTOBER 2020 AND REGISTRATION LOCATION IS NEW DELHI, INDIA HTTPSWWWWHOISCOMWHOISCAMELOTOVERSEASCOM INCIDENTALLY IT WAS OBSERVED THAT CONTENTS OF WEBSITE OF OVERSEAS ENTITY, CAMELOT OVERSEAS PTE LTD, HTTPSCAMELOTOVERSEASCOM IS COPY PASTED FROM CUSTOMERS OWN WEBSITE OF KALEEN BHAWAN REFER HTTPSKALEENBHAWANCOM?PAGEID=4475FURTHER, STATED ANNUAL INCOME OF IMEXTRA TRADERS PRIVATE LIMITED IS LESS THAN RS15 CRORES HOWEVER IN LESS THAN 6 MONTHS FROM ACCOUNT OPENING OCTOBER 2020 CLIENT HAS RECEIVED ACCOUNT CREDITS OF MORE THAN RS20 CRORES6MAJOR CREDITS IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED IS VIA FUNDS TRANSFER FROM OTHER YES BANK CUSTOMERS VIZ M CARPETS RS55CRS FROM AC NO 045061900000590, MISHRA AND SONS RS11CRS FROM AC NO 012763300000435, SHIVESH GARMENTS PRIVATE LIMITED RS15CRS FROM AC NO 080063400002280 AND RTGS THE TUNE OF RS68 FROM SHIVESH GARMENTS PRIVATE LIMITED AC NO 103105004994 WITH ICICI BANK ICIC0000104 AND RS3 CRORES FROM MISHRA AND SONS AC NO 201004063378 WITH INDUSIND BANK INDB00000067NO RED FLAGS WERE OBSERVED IN THE LINKED ACCOUNTS OF VINDHYA DAIRY, UDAMYA IMPEX PRIVATE LIMITED AND ORIENTAL CARPET EXPORTERS HOWEVER TOTAL CREDITS RECORDED IN YES BANK ACCOUNTS OF OTHER LINKED ACCOUNTS OF M CARPETS, MISHRA AND SONS AND SHIVESH GARMENTS PRIVATE LIMITED IS BEYOND STATED ANNUAL INCOME OF LESS THAN RS5CRORES 8REGISTERED COMMUNICATION ADDRESS OF SHIVESH GARMENTS PRIVATE LIMITED IS THAT OF NEW DELHI, DELHI AS MENTIONED IN POINT 1  HOWEVER ITS REGISTERED PERMANENT ADDRESS IS IN FLAT NO 001 POONAM ORBIT MIRA ROAD, THANE, MAHARASHTRA, 4011079AFOREMENTIONED REGISTERED PERMEANT ADDRESS OF SHIVESH GARMENTS PRIVATE LIMITED DOES NOT MATCH WITH THE ONE ON PUBLIC DOMAIN IE 118C, FIRST FLOOR, AJANTA, SQUARE MALL, BHAJI MARKET LANE 604, BORIVALI, W, MUMBAI CITY, MAHARASHTRA, 400092IN LIGHT OF CROSS BORDER REMITTANCES BEING UNDERTAKEN IN PRETEXT OF IMPORT OF GOODS FROM OVERSEAS ENTITIES WITH OPAQUE CREDENTIALS WHEREIN VERACITY OF UNDERLYING TRANSACTIONS COULDNT BE DETERMINED DUE TO ABSENCE OF SUPPORTING DOCUMENTS AND CUSTOMER BEING NON COOPERATIVE IT HAS BEEN DECIDED ON 13MAY2021 TO FILE STR ON SAID CUSTOMER1 THE CUSTOMER IMEXTRA TRADERS PRIVATE LIMITED PAN AAECI3076K, DOI 25102016, COMMUNICATION ADDRESS THIRD FLOOR 418 EAST PATEL NAGAR, NEW DELHI, DELHI 110008, PERMANENT ADDRESS  49 C FIRST FLOOR POLE NO 36D PALAM , NEW DELHI , 110045, GSTIN 07AAECI3076K1ZD, REGISTRATION NUMBER 307523, CIN U74999DL2016PTC307523 , IEC CODE AAECI3076K, MOBILE NUMBER 919599399002, EMAIL ID IMEXTRATRADERSPVTLTD@GMAILCOM IS MAINTAINING A CURRENT AC NO 080063700003298 SINCE 03102020 WITH DELHI ASAF ALI ROAD BRANCH ADDRESS GROUND FLOOR, 411, ASAF ALI ROAD, NEW DELHI  1100022 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN OCTOBER 2020 SAT PAL SHARMA AND ASHISH MISHRA ARE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS GARMENT STORE WITH AN ANNUAL INCOME LESS THAN RS15 CRORES CUSTOMER HAS AVAILED TERM DEPOSITS AMOUNTING TO RS 500000ADDITIONALLY 9 MORE ACCOUNTS ARE FOUND TO BE LINKED TO THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NO080063700003298 AS UNDERCURRENT ACCOUNT OF KALEEN BHAWAN EXPORTS PRIVATE LIMITED AC NO 080063700003341 IS LINKED WITH AFOREMENTIONED ACCOUNT BY COMMON MOBILE NUMBER 919599399002 AND COMMUNICATION ADDRESS IN POINT NO 1 CURRENT ACCOUNT OF SHIVESH GARMENTS PRIVATE LIMITED AC NO 080063400002280, VINDHYA DAIRY AC NO 080063700003301, UDAMYA IMPEX PRIVATE LIMITED AC NO 080063700003321 AND ORIENTAL CARPET EXPORTERS AC NOS 080063700003524, 080080600001048 080082300000031 LINKED WITH AFOREMENTIONED ACCOUNT BY COMMON COMMUNICATION ADDRESS IN POINT NO 1CURRENT ACCOUNT OF M CARPETS AC NO 045061900000590 AND MISHRA SONS AC NO 045061900000474 ARE LINKED WITH ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED BY FUND TRANSFERS3 IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NO 080063700003298 AND 9 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM 01042021 TILL 15052021, SHOWS 45 CREDIT TRANSACTIONS AMOUNTING TO RS 59,861,294 AND 39 DEBIT TRANSACTIONS AMOUNTING TO RS 59,772,544OUT OF THE TOTAL CREDITS OF RS 59,861,294 IN THE ACCOUNTS, FUND TRANSFERS AMOUNTS TO RS 9,798,500 CONSTITUTING AROUND 16 PERCENT OF THE TOTAL CREDITS, NEFTRTGSIMPS AMOUNTS TO RS 46,753,187 CONSTITUTING AROUND 78 PERCENT OF THE TOTAL CREDITS, OUT OF THE TOTAL DEBITS OF RS 59,772,544 IN THE ACCOUNTS, FUND TRANSFERS AMOUNTS TO RS 48,539,561 CONSTITUTING AROUND 81 PERCENT OF THE TOTAL DEBITS, NEFTRTGSIMPS AMOUNTS TO RS 6,162,400 CONSTITUTING AROUND 10 PERCENT OF THE TOTAL DEBITS,, 4 IN THE ACCOUNT OF IMEXTRA TRADERS PRIVATE LIMITED AC NO 080063700003298 AND 9 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM 01042020 TILL 31032021 TRANSACTIONS ANALYSIS SHOWS 583 CREDIT TRANSACTIONS AMOUNTING TO RS 974,989,432 COMPRISING OF CASH DEPOSIT AMOUNTING TO RS 5,700,000 AND 688 DEBIT TRANSACTIONS AMOUNTING TO RS 973,188,143 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 656,500FROM 01042019 TILL 31032020 TRANSACTIONS ANALYSIS SHOWS 163 CREDIT TRANSACTIONS AMOUNTING TO RS 111,038,678 COMPRISING OF CASH DEPOSIT AMOUNTING TO RS 111,000 AND 318 DEBIT TRANSACTIONS AMOUNTING TO RS 110,261,235 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 697,200FROM 01042018 TILL 31032019 TRANSACTIONS ANALYSIS SHOWS 53 CREDIT TRANSACTIONS AMOUNTING TO RS 40,950,000 COMPRISING OF NIL CASH DEPOSIT AND 43 DEBIT TRANSACTIONS AMOUNTING TO RS 40,949,768 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 15,000BALANCE IN THE ACCOUNT AS ON 15052021 IS RS 3,078,80320ON REVIEWING THE TRANSACTION PATTERN OF IMEXTRA TRADERS PRIVATE LIMITED AC 080063700003298 AND KALEEN BHAWAN EXPORTS PRIVATE LIMITED AC NO 080063700003341 IT WAS OBSERVED THAT SINCE OCTOBER 2020 SAID ENTITIES HAD REMITTED TOTAL OF RS222 CRORES UNDER 17 TRANSACTIONS TO OVERSEAS ENTITIES CAMELOT OVERSEAS PTE LTD, SINGAPORE RS94 CRORES IN 7 TRACHES AND NADFA TRADING FZE, UAE RS128 CRORES IN 10 TRANCHES TOWARDS ADVANCE AGAINST IMPORT OF CARPET  CARPET',
  {'entities': [(880, 897, 'FOREIGN ORG'),
    (1018, 1035, 'FOREIGN ORG'),
    (907, 931, 'FOREIGN ORG'),
    (1186, 1210, 'FOREIGN ORG'),
    (1210, 1234, 'FOREIGN ORG'),
    (1356, 1380, 'FOREIGN ORG'),
    (1597, 1621, 'FOREIGN ORG'),
    (7272, 7296, 'FOREIGN ORG'),
    (7337, 7354, 'FOREIGN ORG'),
    (2543, 2568, 'FOREIGN ORG'),
    (4920, 4945, 'FOREIGN ORG')]}),
 ('8 BRANCH HAS RAISED THE CASE ON THE GROUND OF NOTICE RECEIVED FROM OFFICE OF THE COMMISSIONER OF GST, CENTRAL EXCISE, ANNA NAGAR WEST EXTENSION, CHENNAI 600101, WHERE NOTICE IS UNDER SECTION 65 OF CENTRAL GOODS AND SERVICE TAX ACT, 20179 DURING THE REVIEW OF THE STATEMENT OF THE CUSTOMER PHOOLCHAND BHAGATSINGH AC 01432320005055, ON 21042020, CUSTOMER RECEIVED CREDITS FROM HIS OWN LINKED ACCOUNT PHOOLCHAND BHAGATSINGH AC NO  03422790000877 THROUGH FUND TRANSFER OF RS 3 CRORE WHICH WAS IMMEDIATELY TRANSFER TO HIS OWN ACCOUNT PHOOLCHAND BHAGATSINGH AC NO  01432790000715, SIMILAR INSTANCES CAN BE OBSERVE ON 0204202003042020, ETC, THE CUSTOMER IS ROUTING HIGH VALUE COMPLEX FUNDS TRANSFER BETWEEN LINKED ACCOUNT, WHERE SOURCE AND UTILIZATION OF FUNDS COULD BE ASCERTAIN, ALSO TRANSACTION ARE NOT IN LINE WITH THE PROFILE AND TURNOVERSTR NEEDS TO BE FILED IN LIGHT OFI HIGH VALUE CREDITS FOLLOWED BY IMMEDIATE DEBITSII TRANSACTION ARE NOT IN LINE WITH THE PROFILE DECLARED TURNOVERIII NOTICE RECEIVED FROM INSPECTOR OF DR S VADIVUKARASI, DEPUTY COMMISSIONER, OFFICE OF THE COMMISSIONER OF GST, CENTRAL EXCISE, AUDIT I COMMISSIONERTATE, NO 1775, JAWAHARLAL NEHRU INNER RING ROAD, ANNA NAGAR WEST EXTENSION, CHENNAI1 BRANCH HAS RAISED THE CASE ON THE GROUND OF NOTICE RECEIVED FROM INSPECTOR OF DR S VADIVUKARASI, DEPUTY COMMISSIONER, OFFICE OF THE COMMISSIONER OF GST, CENTRAL EXCISE, AUDIT I COMMISSIONERTATE, NO 1775, JAWAHARLAL NEHRU INNER RING ROAD, ANNA NAGAR WEST EXTENSION, CHENNAI  600101, WHERE NOTICE IS UNDER SECTION 65 OF CENTRAL GOODS AND SERVICE TAX ACT, 2017 WHERE THEY HAVE REQUESTED TO FURNISH DETAILS OF THE ACCOUNT, STATEMENT OF THE ACCOUNT, AOF AND KYC FOR THE CUSTOMER P SUREKHA AC NO 501001767963122 PHOOLCHAND BHAGATSINGH DATE OF INCORPORATION 09091966 IS MAINTAINING A CURRENT ACCOUNT BEARING NO 01432320005055 WITH OUR CRAWFORD MARKET BRANCH, ADDRESS  2225 GROUND FLOOR, ASHOKA SHOPPING CENTRE, LOKMANYA TILAK ROAD, CRAWFORD MKT, MUMBAI, MAHARASHTRA, PIN 400001, SINCE 05062004 CUSTOMER HAS DECLARED PROFILE AS TRADING IMPORT FERROUS METALS WITH ANNUAL GROSS TURNOVER OF RS 8890 LAKHS, CUSTOMER ADDRESS  HILIFE MALL SHOP NO 15 1ST FLOOR, PM ROAD SANTACRUZ WEST, MUMBAI, MAHARASHTRA, PIN  400054, MOBILE NUMBER  917678060250 CUSTOMER HAS FURNISHED PAN AAAFP1146J, ACCOUNT IS IN ACTIVE STATUS, RISK PROFILE IS HIGH, BALANCE IN ACCOUNT AS ON 25032021 IS RS 579568123 ADDITIONAL LINKED ACCOUNT BASIS PAN IS BELOWPHOOLCHAND BHAGATSINGH AC NO 01432790000715, 03422790000877, 06262320000058, 04572790000121, 06262790000159, 50200046107082, 04578640000038, 03428640000256, 01432430000322, 15792320000194, 15792790000045 AND 157923200001874 SCRUTINY OF THE ACCOUNT STATEMENT OF PHOOLCHAND BHAGATSINGH AC 01432320005055 FOR THE PERIOD OF 01042012 TO 31032020 REVEALED THAT CUSTOMER HAS RECEIVED CREDITS AGGREGATING TO RS 118854 CRORE OUT OF WHICH RS 76258 CRORE VIA RTGS NEFT  IMPS  UPI CREDIT, RS 35805 CRORE VIA FUND TRANSFER AND RS 6627 CRORE VIA CLEARING CREDIT AND RS 075 CRORE VIA SALARY  STP CREDIT, AND THE TOTAL DEBIT IS RS 118763 CRORE OUT OF WHICH RS 109624 CRORE VIA FUND TRANSFER, RS 4568 CRORE VIA RTGS NEFT  IMPS  UPI DEBIT AND RS 378 CRORE VIA CLEARING DEBIT5 SCRUTINY OF THE ACCOUNT STATEMENT OF PHOOLCHAND BHAGATSINGH AC 01432320005055 FOR THE PERIOD OF 01042020 TO 08042021 REVEALED THAT CUSTOMER HAS RECEIVED CREDITS AGGREGATING TO RS 20989 CRORE OUT OF WHICH RS 12092 CRORE VIA RTGS NEFT  IMPS  UPI CREDIT FROM MRJS LEAD PVT LTD, MITHILA RASAYAN PVT, PREMIER INDUSTRIAL CORPORATION LTD, ETC, RS 7903 CRORE VIA FUND TRANSFER AND RS 863 CRORE VIA CLEARING CREDIT AND RS 125 CRORE VIA SALARY  STP CREDIT AND THE TOTAL DEBIT IS RS 21076 CRORE OUT OF WHICH RS 17199 CRORE VIA FUND TRANSFER, RS 3254 CRORE VIA RTGS NEFT  IMPS  UPI DEBIT TO SHREE JAGDISH IMPEX PVT LTD, TODINI METALS AND CHEMICALS INDIA, SHREE JAGDISH IMPEX PVT LTD, ETC AND RS 332 CRORE VIA UTILITY PAYMENT DEBIT, RS 0042 CRORE VIA OUTWARD REMITTANCE AND RS 00027 CRORE VIA CLEARING DEBIT6 SIMILAR PATTERN OF TRANSACTION OBSERVED IN THE LINKED ACCOUNT PHOOLCHAND BHAGATSINGH AC NO 01432790000715, 03422790000877, 06262320000058, 04572790000121, 06262790000159, 50200046107082, 04578640000038, 03428640000256, 01432430000322, 15792320000194, 15792790000045 AND 157923200001877 THERE ARE NO TRANSACTION IN THE ACCOUNT OF PHOOLCHAND BHAGATSINGH AC NO 01432430000322',
  {'entities': [(3488, 3522, 'FOREIGN ORG')]}),
 ('FUNDS CREDITED IN THE ACCOUNT THROUGH REMITTANCE, RTGS AND DEPLOYED THROUGH REMITTANCE, RTS SEEMS UNUSUAL FOR EXAMPLE ON 24032021 FUNDS OF RS 223 CRORE WERE CREDITED IN THE ACCOUNT THROUGH REMITTANCE FROM MEGHA ENGINEERING AND INFRASTRUCTURE LTD, INDIA AND ON THE NEXT DAY RS 7 CRORE WERE DEPLOYED THROUGH RTGS TOWARDS RASHMI METALIKS LIMITED ACC NO39826430992, IFSC SBIN0007502  ALSO ON 09032021 FUNDS OF RS 28 CRORE WERE CREDITED IN THE ACCOUNT THROUGH CLOSURE PROCEEDS OF FD AND ON THE SAME DAY RS 246 CRORE WERE DEPLOYED THROUGH REMITTANCE TOWARDS FERROPEM, 517 AVENUE, FRANCE IN EURO FOR PIPES AND SPARES AS PAYMT TOWARDS IMP SETTLMT OTR THAN NEPL BHUTAN SAME PATTERN IS SEEN IN ACCOUNT AS PER GOOGLE SEARCH, THE RASHMI GROUP HAS BEEN SLAPPED WITH RS 660 CRORE NOTICE BY THE INDIAN RAILWAYS FOR EXPORTING LAKHS OF TONNES OF IRON ORE THAT IT SAID WAS DOMESTIC CONSUMPTION BY DECEITFULLY SUBMITTING MISLEADING DOCUMENTS TO THE RAILWAYS WEB LINK HTTPSWWWONEINDIACOM20110930RS660CROREINDIANRAILWAYSCAMUNEARTHEDHTML ALSO 26 BILLS FOR IMPORT PAYMENT ARE PENDING FOR REGULARISATION OUT OF WHICH 10 ARE OVERDUE IN VIEW OF LARGE VALUE FOREX TRANSACTIONS IN THE ACCOUNT, NEGATIVE MEDIA NEWS WHEREIN CUSTOMER GROUP IS SUSPECTED TO BE INVOLVED IN ILLEGAL ACTIVITY AND LARGE NUMBER OF IMPORT BILL PENDING FOR REGULARISATION, STR IS BEING REPORTEDRASHMI METALIKS LIMITED, A PUBLIC LTD COMPANY, HAVING ADDRESS  39 SHAKESPEARE SARANI 6TH FLOOR, PREMLATA BUILDING, KOLKATA, WEST BENGAL, INDIA, 700017, CONTACT NUMBER 9830302479 AND EMAIL ID TREASURY@RASHMIGROUPCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 129305000471 WITH ICICI BANK LTD, 8 AJC BOSE ROAD, CIRCULAR COURT, KOLKATA, WEST BENGAL, 700017, SINCE 08052014 DATE OF INCORPORATION IS 30012004 PAN OF THE ACCOUNT IS AACCR7183E IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS 0205014305 GSTIN AS PER BANK RECORDS IS 19AACCR7183E1Z6 TRANSACTION LINKED ACCOUNTS ARE 129310002451, 129310002133 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE CHANDI GHARAN NEMUYA ALNPN1730E, CHANDAN KUMAR MODI AADHARXXXXXXXX9157, DHARMENDRA KUMAR SINGH AITPS4400P DIRECTORS OF THE REPORTED ACCOUNT ARE SURENDRA JHA AKNPJ8666N, SALIL RAY BENPR3036M AND CHANDICHARAN NEMUYA ALNPN1730E THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN, CERTIFICATE OF INCORPORATION AND COPY OF ARTICLE AND MEMORANDUM OF ASSOCIATION KYC UPDATION IS DUE THE FIRM IS INVOLVED INTO MANUFACTURING OF SPONGE IRON, MILD STEEL BILLET, INGOT, DUCTILE IRON PIPE TMT BAR AND WIRE RODS ANNUAL TURN OVER DECLARED BY CUSTOMER IS NIL ALERT WAS TRIGGERED FOR LARGE VALUE REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT 02052021 WAS RS 30455 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 02052021 SHOWS TOTAL CREDIT RS 1,53345 LAKHS OF WHICH REMITTANCE RS 1,21570 LAKHS MAJORLY FROM MEGHA ENGINEERING AND INFRASTURES LTD, SUBHASH INFRAENGINEERS PVT LTD, FIXED DEPOSIT RS 31538 LAKHS TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,36812 LAKHS OF WHICH RTGS RS 1,00000 LAKHS MAJORLY TO RASHMI METALS, REMITTANCE RS 35656 LAKHS MAJORLY TO GODREJ AND BOYCE MFG CO LTD, DALMIA SEVEN REFRACTORIES LTD, KHOSLA ENGINEERING PVT LTD FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021  TOTAL CREDIT SIDE RS 10,85749 LAKHS OF WHICH NEFT RS 4,74471 LAKHS MAJORLY FROM RASHMI METALIKS LTD, REMITTANCE RS 3,56745 LAKHS MAJORLY FROM SUBHASH INFRAENGINEERS PVT LTD, SUBHASH INFRAENGINEERS, SUBHASH METALIKS LIMITED, FIXED DEPOSIT RS 2,54451 LAKHS TOTAL DEBIT SIDE RS 10,71837 LAKHS OF WHICH TRANSFERS RS 3,08837 LAKHS MAJORLY TO RASHMI METALIKS LIMITED, RTGS RS 3,20593 LAKHS MAJORLY TO RASHMI METALS, REMITTANCE RS 2,54098 LAKHS MAJORLY TO POLYSUN TONGLING EXTRUSION, ETERNAL SEA GROUP LIMITED AND OTHERS RS 1,86829 LAKHS FY 201920 FROM 01042019 TO 31032020  TOTAL CREDIT SIDE RS 4,25049 LAKHS OF WHICH CASH WAS RS 030 LAKHS, REMITTANCE RS 4,22059 LAKHS  TOTAL DEBIT SIDE RS 4,25088 LAKHS OF WHICH CASH WAS RS 3658 LAKHS, REMITTANCE RS 4730 LAKHS FY 201819 FROM 01042018 TO 31032019  TOTAL CREDIT SIDE RS 4240 LAKHS OF WHICH CASH WAS RS 040 LAKHS TOTAL DEBIT SIDE RS 4212 LAKHS OF WHICH CASH WAS RS 4212 LAKHS FY 201718 FROM 01042017 TO 31032018  TOTAL CREDIT SIDE RS 2260 LAKHS OF WHICH CASH WAS RS 040 LAKHS TOTAL DEBIT SIDE RS 2287 LAKHS OF WHICH CASH WAS RS 2287 LAKHS',
  {'entities': [(2946, 2983, 'FOREIGN ORG')]}),
 ('FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 2,62522 LAKHS OF WHICH, REMITTANCE RS 83413 LAKHS TOTAL DEBIT SIDE RS 2,60623 LAKHS OF WHICH, REMITTANCE RS 128 LAKHS FY 201819 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS 95886 LAKHS TOTAL DEBIT SIDE RS 95747 LAKHS FY 201718 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS 8,09754 LAKHS OF WHICH CASH WAS RS 13140 LAKHS TOTAL DEBIT SIDE RS 8,09780 LAKHS THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER 033705500984 IN THE NAME OF BORA MOBILITY, REPORT SERIAL NUMBER 49128 TRANSACTION PATTERN INDICATES CREDITS FOLLOWED BY IMMEDIATE DEBITS WHICH SEEMS LIKE ROUTING OF FUNDS OBSERVED FOR EXAMPLE, ON 06052021 RS 7664LACS WAS CREDITED BY RTGS IN PARTS FROM JAY JALARAM ENTERPRISE 650001403814INDB0000006, BORA EXIM PRIVATE LIMITED 033705009463, NEFT FROM SAMA DISTRIBUTOR 50200028398822HDFC00002, AND RS 64 LACS WAS DEBITED BY RTGS TO BORA MOBILITY 50200028595701, HDFC0000104 IN OTHER EXAMPLE, ON 27042021 R 2500LACS CREDITED BY INTERNET FUND TRANSFERS FROM BORA EXIM PRIVATE LIMITED 033705009463 AND SAME WAS DEBITED BY INTERNET FUND TRANSFERS TO BORA MOBILITY 50200028595701, HDFC0000104 SIMILAR PATTERN OBSERVED THROUGHOUT THE PERIOD TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 3,43177 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL INCOME OF RS 40LACS TO RS 5CRS IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN THE ACCOUNT, ROUTING OF FUNDS OBSERVED AND HIGH TURNOVER, STR IS BEING FILEDBORA MOBILITY, PROPRIETORSHIP FIRM, HAVING ADDRESS OFFICE NO305 AND 306, NEARMITRAMANDALCHOW, PARVATI PUNE, CENTRE POINT PLOT NO 491, PUNE, MAHARASHTRA, INDIA, 411009, CONTACT NUMBER 919373337477 AND EMAIL ID PRASHANTBORA@GMAILCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 033705500984 WITH ICICI BANK LTD, PUNE  SATARA ROAD, ICICI BANK LTD, SOMSHANK CHAMBERS, SURVEY NO 46, PLOT 1, PUNE SATARA ROAD, PUNE, MAHARASHTRA, 411009, SINCE 26082015, DATE OF INCORPORATION IS 01072015 PAN NUMBER OF THE ACCOUNT IS AISPB2807L TRANSACTION LINKED ACCOUNTS ARE 033705009400, 033705009463 AUTHORIZED SIGNATORIES OF THE REPORTED ACCOUNT ARE PRASHANT PRAKASH BORA PROPRIETOR  AISPB2807L THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN AISPB2807L, AADHAR DATE OF LAST KYC UPDATION 09052019 BORA MOBILITY IS AN INDIAN COMPANY ENGAGED IN TRADING AND DISTRIBUTION OF MOBILE HANDSETS, TABLETS, LAPTOPS AND MOBILE ACCESSORIES WITH DIRECT PRESENCE IN OVER TWENTY COUNTRIES DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40LACS TO RS 5CRS A MANUAL ALERT WAS TRIGGERED FOR LARGE VALUE NONCASH TRANSACTIONS BALANCE IN THE ACCOUNT AT 07052021 WAS RS826 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 07052021 SHOWS TOTAL CREDIT RS 1,16267 LAKHS OF WHICH CLEARING RS 13400 LAKHS MAJORLY FROM NEFT, SSK INCORPORATION, NEFT RS 6955 LAKHS MAJORLY FROM MUKUL MADHAV FOUNDATION, MAHARASHTRA VALUE ADDED TAX, MUKUL MADHAV FOUNDATION FIL CSR, RTGS RS 33007 LAKHS MAJORLY FROM KAMALETERNITY PVT LTD, JAY JALARAM ENTERPRISE, BORA MOBILITY, INTERNET FUND TRANSFERS RS 60829 LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 1645 LAKHS IN 11 INSTANCES, UPI TRANSFERS RS 328 LAKHS IN 7 INSTANCES, FIXED DEPOSIT RS 103 LAKHS, TOTAL DEBITS DURING THE SAME PERIOD WAS RS 1,20134 LAKHS OF WHICH CASH WAS NIL, CLEARING RS 47163 LAKHS, RTGS RS 25330 LAKHS, REMITTANCE RS 34339 LAKHS MAJORLY TO SHENZHEN DEDA HEALTH CO LTD, MEDMSL LIMITED, ZHENGZHOU OLIVE ELECTRONIC, INTERNET FUND TRANSFER RS 12618 LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 569 LAKHS IN 5 INSTANCES, AND OTHERS RS 116 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS 3,43177 LAKHS OF WHICH CASH WAS RS 377 LAKHS MAJORLY IN DELHI KAROL BAGH GURDWARA ROAD, BULDHANA, KAROL BAGH ND 6291 BRANCH, TRANSFERS RS 6240 LAKHS MAJORLY FROM BORA MOBILITY, PRASHANT PRAKASH BORA, SEANA, CLEARING RS 32883 LAKHS MAJORLY FROM SSK INCORPORATION, BORA MOBILITY LLP, A Z LUNAWAT AND, NEFT RS 8701 LAKHS MAJORLY FROM OMKAR COMPUTERS PVT LTD, BITFLIGHT ELECTRONIC SYSTEMS, BORA MOBILITY, RTGS RS 1,46011 LAKHS MAJORLY FROM BORA MOBILITY, BATHLA TELETECH PRIVATE, E PAO GST REFUNDS, REMITTANCE RS 2775 LAKHS MAJORLY FROM CAPTAINS FREIGHT SERVICES HK, INTERNET FUND TRANSFERS RS 1,44505 LAKHS MAJORLY FROM BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 1492 LAKHS IN 34 INSTANCES, UPI TRANSFERS RS 043 LAKHS IN 21 INSTANCES, FIXED DEPOSIT RS 150 LAKHS, TOTAL DEBIT SIDE RS 3,40575 LAKHS OF WHICH CASH WAS RS 100 LAKH BY BRANCH, TRANSFERS RS 250 LAKHS MAJORLY TO BORA MOBILITY, CLEARING RS 82139 LAKHS, RTGS RS 81800 LAKHS MAJORLY TO BORA AND BORA TECHNOLOGIES, BORA MOBILITY, REMITTANCE RS 1,09027 LAKHS MAJORLY TO ESTAR TECHNOLOGY GROUP CO LTD, WONDAFONE TECHNOLOGY HONGKONG CO, AYAAN ELECTRONICS LLC, INTERNET FUND TRANSFER RS 43261 LAKHS MAJORLY TO BORA EXIM PRIVATE LIMITED, MOBILE MONEY TRANSFERS RS 10124 LAKHS IN 74 INSTANCES',
  {'entities': [(4891, 4917, 'FOREIGN ORG'),
    (5004, 5024, 'FOREIGN ORG'),
    (5038, 5059, 'FOREIGN ORG'),
    (4973, 5002, 'FOREIGN ORG'),
    (4392, 4415, 'FOREIGN ORG'),
    (777, 802, 'FOREIGN ORG'),
    (1031, 1056, 'FOREIGN ORG'),
    (3234, 3259, 'FOREIGN ORG'),
    (3671, 3696, 'FOREIGN ORG'),
    (4558, 4583, 'FOREIGN ORG'),
    (5110, 5135, 'FOREIGN ORG'),
    (3549, 3576, 'FOREIGN ORG'),
    (3578, 3592, 'FOREIGN ORG'),
    (4474, 4499, 'FOREIGN ORG'),
    (4297, 4325, 'FOREIGN ORG'),
    (4973, 5002, 'FOREIGN ORG'),
    (3594, 3620, 'FOREIGN ORG')]}),
 ('2 SUBSEQUENT TO SUCH CREDITS FUNDS WERE REMITTED ON THE SAME OR NEXT DAY OVERSEAS FAVOURING ENTEGRATED LOGISTIC PTE LTD SINGAPORE RS231 CR TOWARDS FREIGHT ON EXPORTS SHIPPING COMPANIES 3 DURING WEB CHECK NO SPECIFIC INFORMATION WEBSITE COULD BE SEEN IN THE NAME OF ENTEGRATED LOGISTIC PTE LTD 4 ON SCRUTINY OF SAMPLE DOCUMENTS FURNISHED BY CUSTOMER FOR EFFECTING REMITTANCES IN WAS NOTED THAT CUSTOMER HAD FURNISHED COPIES OF BL, FREIGHT INVOICE, AGREEMENT AND 15 CA CB AS PER THE BL GOODS WERE SHIPPED FROM INDIA TO CHINA HOWEVER FREIGHT PAYMENT WAS BEING MADE TO SINGAPORE BASED ENTITY 5 A PATTERN OF TRANSACTIONS WERE NOTED WHEREIN HIGH VALUE RTGSNEFT CREDITS WERE FOLLOWED BY HIGH VALUE FOREIGN OUTWARD REMITTANCES ON THE SAME DAY NO OTHER BUSINESS RELATED TRANSACTIONS LIKE PAYMENTS TOWARDS TAXES, SALARIES, RENT ETC, COULD BE NOTED IN THE ACCOUNT DURING REVIEW PERIOD 6 IT IS NOTED THAT WITHIN 3 MONTHS OF ACCOUNT OPENING TRANSACTIONS IN THE ACCOUNT FAR EXCEEDED THE ANNUAL TURNOVER INDICATED BY CUSTOMER DURING ACCOUNT OPENING 7 DURING A VISIT BY BRANCH OFFICIAL AT THE MAILING ADDRESS FURNISHED BY THE CUSTOMER, IT WAS NOTED THAT THE OFFICE WAS A 1200 SQ FT RENTED PREMISE WHERE SIGN BOARD, EMPLOYEES AND OFFICE SET UP WERE SEEN 8 BASIS TRANSACTION PATTERN AND INFORMATION AVAILABLE IT APPEARS THAT THE DIRECTORS WHO ARE AGED 34 YEARS MOHIT CHAUDHARY AND 26 YEARS ASHISH CHOWREKAR MAY NOT BE THE ACTUAL BENEFICIAL OWNER OF SUCH HIGH VALUE FUNDS ROUTED THROUGH THE ACCOUNT IN A SHORT SPAN ALSO CUSTOMER REMITTING FUNDS TO SINGAPORE AS FREIGHT CHARGES WHEN THE GOODS ARE SHIPPED FROM INDIA TO CHINA APPEARS UNUSUAL IT APPEARS THAT TRANSACTIONS MAY HAVE BEEN ROUTED THROUGH THE ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED FOR LAYERING OF FUNDS AND POSSIBLY WITH THE INTENTION OF TRADE BASED MONEY LAUNDERING OR HAWALA SETTLEMENT ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED IS ACTIVE WITH BALANCE OF RS296 LAKHS AS ON MAY 23, 2021 QUANTUM OF TRANSACTIONS FOR CURRENT FINANCIAL YEAR 20212022 IS DEBIT RS205 CR IN CASH RS002 CR AND CREDITS RS180 CR IN CASH RS000 CR CORRESPONDING TRANSACTION TURNOVER IN THE PREVIOUS FINANCIAL YEARS WERE 202021 IS DEBIT RS046 CR IN CASH RS001 CR AND CREDITS RS074 CR IN CASH RS000 CRA CURRENT ACCOUNT BEARING NUMBER 6645082401 WAS OPENED IN THE NAME OF OCEANS PRIDE MARINE SERVICES P LTD ON 01FEB21 AT OUR BRANCH LOCATED IN LAKE PRIMROSE" PHASEIV, WING "C" LAKE HOMES, POWAI, MUMBAI MAHARASHTRA  400076 THE ENTITY WAS FORMED ON 14DEC15 AND HAS SUBMITTED COPIES OF PAN AACCO2518A, CERT OF INCORP U74120MH2015PTC270983, SHOPS AND ESTABLISHMENT REG CERTIFICATE 820105941 AND IEC CERT AACCO2518A AS KYC DOCUMENTS AT THE TIME OF ACCOUNT OPENING CUSTOMERS REGISTERED ADDRESS WITH THE BANK IS OFFICE NO 401 SAMRUDDHI C COMPLEX CHINCHOLI BUNDER ROAD MALAD WEST MUMBAI 400064 MOHIT CHAUDHARY AND ASHISH DINESH CHOWKEKAR ARE THE DIRECTORS WHILE MOHIT CHAUDHARY IS THE BENEFICIAL OWNER IN THE SAID COMPANY MOHIT CHAUDHARY AND ASHISH DINESH CHOWKEKAR HAVE REGISTERED THEIR MOBILE NUMBERS 7678012087 AND 8657154653 WITH THE BANK THE ENTITY IS CATEGORISED AS MEDIUM RISK AND IS NOT YET DUE FOR PERIODIC KYC UPDATION IT WAS INDICATED AT THE TIME OF ACCOUNT OPENING THAT THE ENTITY IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS100 CR BASIS INTERCONNECTEDNESS WITH THE ABOVE MENTIONED ACCOUNT, OTHER ACCOUNTS WERE IDENTIFIED WHICH COMPRISE OF CURRENT ACCOUNTS IN THE NAMES OF TRIUMPH TECHNOMATE SOLUTIONS 04302090000280, AC OPENED ON 26JUL11, RAJKOTIA CLOUD CONSULTANCY P LTD 3445577472, AC OPENED ON 27AUG20 AND SAVINGS ACCOUNT IN THE NAME OF ASHISH DINESH CHOWKEKAR 5212728513, AC OPENED ON 24JAN18 THE CURRENT ACCOUNT OF TRIUMPH TECHNOMATE SOLUTIONS AND SAVINGS ACCOUNT OF ASHISH DINESH CHOWKEKAR ARE DORMANT IT WAS INDICATED THAT RAJKOTIA CLOUD CONSULTANCY P LTD IS INTO LOGISTIC AND COURIER BUSINESS WITH ANNUAL TURNOVER OF RS250 CR NO MAJOR TRANSACTIONS WERE NOTED IN THE ACCOUNT OF RAJKOTIA CLOUD CONSULTANCY P LTD DURING REVIEW PERIOD WHILE REVIEWING REPORT ON FOREIGN OUTWARD REMITTANCES DONE THROUGH THE BANK IT WAS NOTED THAT HIGH VALUE FOREIGN OUTWARD REMITTANCES TOWARDS FREIGHT PAYMENTS WERE ROUTED THROUGH THE ABOVE MENTIONED NEWLY OPENED ACCOUNT OF OCEANS PRIDE MARINE SERVICES PRIVATE LIMITED ON VERIFYING THE ACCOUNT ACTIVITY IT WAS NOTICED THAT SUCH FREIGHT PAYMENTS WERE DONE IMMEDIATELY POST RECEIPT OF HIGH VALUE RTGSNEFT CREDITS AND HENCE THIS CONSOLIDATED REPORT IS FILED BASIS INTERCONNECTEDNESS AND ON THE FOLLOWING GROUNDS OF SUSPICION 1 ON REVIEWING ACCOUNT TRANSACTIONS SINCE APRIL 1, 2020 TILL MAY 19, 2021 IT WAS NOTED THAT RTGSNEFT CREDITS AMOUNTED TO RS252 CR MAINLY FROM QUICK SOLUTION LOGISTICS RS196 CR, RBL BANK  409001403849, BLUE SEA LOGISTIC RS024 CR, RBL  409001403863 AND PREMIUM LOGISTICS SOLUTIONS RS018 CR, RBL 409001383226',
  {'entities': [(92, 119, 'FOREIGN ORG'),
    (265, 292, 'FOREIGN ORG'),
    (82, 119, 'FOREIGN ORG'),
    (4726, 4743, 'FOREIGN ORG'),
    (4776, 4803, 'FOREIGN ORG'),
    (4667, 4691, 'FOREIGN ORG'),
    (3438, 3466, 'FOREIGN ORG'),
    (3686, 3714, 'FOREIGN ORG'),
    (4667, 4691, 'FOREIGN ORG'),
    (4776, 4803, 'FOREIGN ORG')]}),
 ('IN OTHER EXAMPLE, ON 03022021, RS 5982 LACS AMOUNT OF FUND CREDITED IN PARTS THROUGH RTGS FROM MAGNETIC FILMS PRODUCTION, DIVERSE MEDIA WHERE FUND DEBITED THROUGH OUTWARD REMITTANCE TOWARDS WEB VIDEOS LIMITED UNITED KINGDOM HTTPWWWWEBVIDEOSCOUK, SINGAPORE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE, WHITE RABBIT MOTION PICTURE PRODUCTION LLC, UAE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE HTTPWWWWHITERABBITDUBAICOM THIS ACCOUNT GET CLOSED IN SHORT SPAN OF PERIOD OF 6 MONTHS WITH RS 5507 CRORES CLIENT WAS UNABLE TO PROVIDE THE INVESTMENT AGREEMENT WITH THE INVESTORS IN SPITE SEVERAL ASSURANCES, WHICH SEEMS SUSPICIOUS TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 5,50785 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF BELOW RS 40LACS IN VIEW OF, LARGE VALUE NON CASH ROUTING TRANSACTIONS IN NEWLY OPENED ACCOUNT AND HIGH TURNOVER, STR IS BEING FILEDTINGLE CINE PRODUCTIONS, PARTNERSHIP, HAVING ADDRESS 2 ND, FLOOR 826265 FLAT NO, 201 RELIANCE CLASSIC, ABOVE ANDHRA HYDERABAD, TELANGANA, INDIA, 500067, CONTACT NUMBER 918374697882 AND EMAIL ID INFO@TINGLECINEPRODUCTIONSCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 018305011130 WITH ICICI BANK LIMITED, HYDERABAD  BEGUMPET, 11125119A, JAYALAKSHMI TOWERS, MOTILALNEHRU NAGAR, PRAKASHNAGAR, BEGUMPET, HYDERABAD, TELANGANA, 500016, SINCE 02092020 DATE OF INCORPORATION IS 10072019 PAN NUMBER OF THE ACCOUNT IS AAQFT2670K GSTIN AS PER BANK RECORDS IS 36AAQFT2670K1ZL TRANSACTION LINKED ACCOUNTS ARE 000801654223, 000801654221 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS MANISH CHAWLA WITH PAN NO AGRPC4964E, MOHIT BANSAL WITH PAN NO ALRPB9873C THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS CLOSED ON 19032021 KYC DOCUMENTS OBTAINED WERE GSTIN, AADHAR, PANAAQFT2670K AS PER ENHANCE DUE DILIGENCE, CUSTOMER IS IN BUSINESS OF THE INVESTORS AND THE BENEFICIARY TO BE IN THE LINE MEDIA AND FILMS THEY ARE THE SERVICE PROVIDER FOR MOVIE OR FILM PRODUCTION DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS BELOW RS 40LACS A MANUAL ALERT WAS TRIGGERED FOR HIGH VALUE TRANSACTIONS IN A NEW ACCOUNT BALANCE IN THE ACCOUNT AT 19032021 WAS NIL FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 02092020 TO 19032021 TOTAL CREDIT SIDE RS 5,50785 LAKHS OF WHICH CASH WAS NIL, CLEARING RS 100 LAKH MAJORLY FROM CHQ, NEFT RS 6133 LAKHS MAJORLY FROM DIVERSE MEDIA, LIQUID STUDIOS AND SERVICES, RTGS RS 5,44123 LAKHS MAJORLY FROM DIVERSE MEDIA, LIQUID STUDIOS AND SERVICE, MAGNETIC FILMS PRODUCTION, MOBILE MONEY TRANSFERS RS 430 LAKHS IN 3 INSTANCES TOTAL DEBIT SIDE RS 5,50785 LAKHS OF WHICH CASH WAS RS 100 LAKH BY BRANCH, CLEARING RS 005 LAKHS MAJORLY TO T VINAY ASSOCIATES, RTGS RS 14375 LAKHS MAJORLY TO LIQUID STUDIO SERVICE, LIQUID STUDIO SERVICE, REMITTANCE RS 5,34783 LAKHS MAJORLY TO SKY DENVAX PTE LTD, WEB VIDEOS LIMITED, WHITE RABBIT MOTION PICTURE, INTERNET FUND TRANSFER RS 1025 LAKHS MAJORLY TO MANISH CHAWLA, MOHIT BANSAL, MOBILE MONEY TRANSFERS RS 496 LAKHS IN 7 INSTANCES TRANSACTION PATTERN SHOWS, HIGH VALUE NON CASH CREDITS FOLLOWED BY IMMEDIATE DEBITS OF THE SAME AMOUNT FOR EXAMPLE, ON 28012021, RS 133 CRORES AMOUNT OF FUND CREDITED IN PARTS THROUGH RTGS FROM LIQUID STUDIOS AND SERVICE, MAGNETIC FILMS PRODUCTION, DIVERSE MEDIA AND ON THE SAME DAY SAME AMOUNT OF FUND DEBITED THROUGH OUTWARD REMITTANCE TOWARDS WEB VIDEOS LIMITED UNITED KINGDOM HTTPWWWWEBVIDEOSCOUK, SINGAPORE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE, WHITE RABBIT MOTION PICTURE PRODUCTION LLC, UAE IN USD FOR PAYMENT FOR AUDIO VISUAL AND RELATED SERVICE HTTPWWWWHITERABBITDUBAICOM',
  {'entities': [(2961, 2979, 'FOREIGN ORG'),
    (190, 208, 'FOREIGN ORG'),
    (2981, 2999, 'FOREIGN ORG'),
    (3504, 3522, 'FOREIGN ORG'),
    (313, 340, 'FOREIGN ORG'),
    (3001, 3028, 'FOREIGN ORG'),
    (3627, 3654, 'FOREIGN ORG'),
    (2532, 2558, 'FOREIGN ORG'),
    (2611, 2637, 'FOREIGN ORG'),
    (3352, 3378, 'FOREIGN ORG'),
    (95, 120, 'FOREIGN ORG'),
    (2639, 2664, 'FOREIGN ORG'),
    (3380, 3405, 'FOREIGN ORG'),
    (190, 208, 'FOREIGN ORG'),
    (2981, 2999, 'FOREIGN ORG'),
    (3504, 3522, 'FOREIGN ORG')]}),
 ('TOTAL DEBIT SIDE RS 1,62,73798 LAKHS OF WHICH CASH WAS RS 625 LAKHS BY BRANCH, TRANSFERS RS 65,46515 LAKHS MAJORLY TO BINDRA TOURS N TRAVELS PRIVATE LIMITED, EMPLOYEES PROVIDENT FUND ORGANISATION, TRAVEL BOUTIQUE ONLINE A UO TEK TRAPL, CLEARING RS 4636 LAKHS MAJORLY TO WELFARE COMMISSIONER HARY, POOJA PRINTERS, PHD CHAMBER OF COMMERCE, RTGS RS 85,54986 LAKHS MAJORLY TO TRAVEL BOUTIQUE ONLINE, TRAVEL BOUTIQUE ONLINE, REMITTANCE RS 2500, INTERNET FUND TRANSFER RS 6,09673 LAKHS MAJORLY TO R K TRAVELTLC SOLUTION, KANDHARI TRAVEL ETC FY 201920 FROM 01042019 TO 31032020 TOTAL CREDIT SIDE RS 5,94,56245 LAKHS OF WHICH CASH WAS RS 18,31969 LAKHS, REMITTANCE RS 2388 LAKHS TOTAL DEBIT SIDE RS 5,93,54828 LAKHS OF WHICH CASH WAS RS 313 LAKHS, REMITTANCE RS 002 LAKHSFY 201819 FROM 01042018 TO 31032019 TOTAL CREDIT SIDE RS 2,55,70375 LAKHS OF WHICH CASH WAS RS 19,49270 LAKHS, REMITTANCE RS 4649 LAKHS TOTAL DEBIT SIDE RS 2,54,91744 LAKHS OF WHICH CASH WAS RS 126 LAKHS, REMITTANCE RS 002 LAKHSFY 201718 FROM 01042017 TO 31032018 TOTAL CREDIT SIDE RS 1,96,37963 LAKHS OF WHICH CASH WAS RS 14,68604 LAKHS, REMITTANCE RS 451 LAKHS TOTAL DEBIT SIDE RS 1,96,99676 LAKHS OF WHICH CASH WAS RS 379 LAKHS, REMITTANCE RS 89494 PAST STR REPORTED THE BANK HAD FILED EARLIER STR ON THE CUSTOMERS ACCOUNT NUMBER 114505000274 IN THE NAME OF TRAVEL BOUTIQUE ONLINE A UO TEK TRAPL ON 26042017 VIDE FIU BATCH ID 1704266635, REPORT SERIAL NUMBER 103426 CUSTOMER IS INTO TRAVEL RELATED SUPPORT THROUGH ONLINE BASED PORTAL AS PER TRANSACTION PATTERN VOLUME OF TRANSACTION IS VERY HIGH MULTIPLE NUMBER OF NONCASH CREDITED IN THE ACCOUNT FOLLOWED BY NONCASH WITHDRAWAL OBSERVED FOR EXAMPLE, ON 31012020 RS 10 CRORES APPROX CREDITED IN MULTIPLE PARTS FROM DIFFERENT ENTITIES SUCH AS NEFT FROM NEXUS TRAVELS, MICHELIN STAR HOLIDAYS, AIONZ SOJOURN PRIVATE LIMITED, ACE TRAVELS ETC, TRANSFERS FROM DIVVYA TOURS AND TRAVELS, OM TRAVELS, SACH TRAVEL, SEVEN OCEAN, SURAJ TRAVELS, KRISHNA TRAVELS, TRIP STORE LTD ETC, CASH DEPOSITS FROM VIJAYNAGARBANGALORE BRANCH, MALAD BRANCH ETC AND ON THE SAME DAY RS 10 CRORES GOT DEBITED BY RTGS IN 2 PARTS TO TRAVEL BOUTIQUE ONLINE ACC53105107187, IFSCSCBL0036025 SIMILAR PATTERN WAS OBSERVED IN THE ACCOUNT AS OBSERVED THE ACCOUNTS ACTIVITY SEEN COLLECTIONS FROM VARIOUS PARTIES, RELATED TO BOOKING OF HOTEL, TRAVEL TICKETS, AND ITS RELATED SERVICES TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 1,60,50994 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS 5CRS TO LESS THAN RS 25CRS IN VIEW OF LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT AND HIGH TURNOVER, STR IS BEING FILEDTRAVEL BOUTIQUE ONLINE A UO TEK TRAPL, PRIVATE LIMITED COMPANY, HAVING ADDRESS P NO728, PHV, PHV UVIHAR, NR HORIZON BPO, GURGAON, HARYANA, INDIA, 122016, EMAIL ID RAJESHJ@TRAVELBOUTIQUEONLINECOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 114505000274 WITH ICICI BANK LTD, PLOT NO B B1 VANIJAYA NIKUNJ, ENKAY TOWER, PHASEIV, UDYOG VIHAR, GURGAON, HARYANA, 122001, SINCE 01012013, DATE OF INCORPORATION IS 06112006 PAN NUMBER OF THE ACCOUNT IS AACCT6259K GSTIN AS PER BANK RECORDS IS 06AACCT6259K1ZZ TRANSACTION LINKED ACCOUNTS ARE 000705010826, 114510005079 THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN  AACCT6259K DATE OF LAST KYC UPDATION 16092020 CUSTOMER IS INTO TRAVEL RELATED SUPPORT THROUGH ONLINE BASED PORTAL WITH OFFICIAL ADDRESSHTTPSWWWTRAVELBOUTIQUEONLINECOM DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 5CRS TO LESS THAN RS 25CRS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTIONS IN CURRENT ACCOUNT BALANCE IN THE ACCOUNT AT 26042021 WAS RS76602 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 26042021 SHOWS TOTAL CREDIT RS 10,30190 LAKHS OF WHICH CASH WAS RS 26158 LAKHS MAJORLY IN BANGALORE  RESIDENCY ROAD, CHENNAI  TRIPLICANE, DELHI  TAIMUR NAGAR BRANCH, TRANSFERS RS 11569 LAKHS MAJORLY FROM ARYAN INTERNATIONAL, TRAVEL BOUTIQUE ONLINE A UO TEK TRAPL, GALAXY TRAVEL SOLUTIONS, CLEARING RS 15135 LAKHS MAJORLY FROM YATRA ONLINE PRIVATE LIMITED, FLAMINGO TRANSWORLD PVT LTD, SHRESTH UDAN TRAVELS, NEFT RS 1,84287 LAKHS MAJORLY FROM CSC E GOVERNANCE SERVICES IN, SUGAL AND DAMANI U S P LTD, WILLOWTREE, RTGS RS 2,43986 LAKHS MAJORLY FROM NEARBY TRAVEL SOLUTION, SPICE MONEY LIMITED, TRAVEL BOUTIQUE, REMITTANCE RS 1400 LAKHS MAJORLY FROM MTG GLOBAL SERVICES FZ, INTERNET FUND TRANSFERS RS 3,12146 LAKHS MAJORLY FROM R K TRAVELTLC SOLUTION, KANDHARI TRAVEL ETC, MOBILE MONEY TRANSFERS RS 33488 LAKHS IN 883 INSTANCES, UPI TRANSFERS RS 4394 LAKHS IN 330 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 10,58379 LAKHS OF WHICH CASH WAS RS 100 LAKH BY BRANCH, CLEARING RS 298 LAKHS, RTGS RS 10,44197 LAKHS MAJORLY TO TRAVEL BOUTIQUE ONLINE, TRAVEL BOUTIQUE ONLINE FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS 1,60,50994 LAKHS OF WHICH CASH WAS RS 3,76356 LAKHS MAJORLY IN CHENNAI  G P ROAD, CHENNAI  TRIPLICANE, BANGALORE NR RD 6251 BRANCH, TRANSFERS RS 1,72196 LAKHS MAJORLY FROM GLOBE HOLIDAY, TRAVEL BOUTIQUE ONLINE A UO TEK TRAPL, RAMBLE INDIA TOUR TRAVEL PVT LTD, CLEARING RS 1,53725 LAKHS MAJORLY FROM EBIX TRAVELS PVT LTD, FLAMINGO TRANSWORLD PVT LTD, TRAVEL BOUTIQUE ONLINE, NEFT RS 15,40992 LAKHS MAJORLY FROM TRAVCLAN TECHNOLOGY INDIA PVT LTD, LOVELY HOLIDAYS, CSC E GOVERNANCE SERVICES IN, RTGS RS 46,56563 LAKHS MAJORLY FROM TRAVELBOUTIQUEONLINEAU, TRAVEL BOUTIQUE ONLINE, HAPPYEASYGO INDIA P, REMITTANCE RS 46630 LAKHS MAJORLY FROM MTG GLOBAL SERVICES FZC SM OFFICE, MTG GLOBAL SERVICES FZC SM OFFICE C, MTG GLOBAL SERVICES FZC, INTERNET FUND TRANSFERS RS 25,60891 LAKHS MAJORLY FROM R K TRAVELTLC SOLUTION, KANDHARI TRAVEL ETC, MOBILE MONEY TRANSFERS RS 3,33828 LAKHS IN 9381 INSTANCES, UPI TRANSFERS RS 33322 LAKHS IN 2585 INSTANCES, FIXED DEPOSIT RS 4,99687 LAKHS',
  {'entities': [(4480, 4502, 'FOREIGN ORG'),
    (5710, 5732, 'FOREIGN ORG'),
    (5745, 5767, 'FOREIGN ORG'),
    (5782, 5804, 'FOREIGN ORG'),
    (491, 513, 'FOREIGN ORG'),
    (4558, 4580, 'FOREIGN ORG'),
    (5862, 5884, 'FOREIGN ORG'),
    (4189, 4216, 'FOREIGN ORG'),
    (5395, 5422, 'FOREIGN ORG')]}),
 ('FUNDS WERE CREDITED FROM THE BUSINESS CLIENT AND THE SAME HAS BEEN USE FOR BUSINESS PAYMENTS THROUGH REMITTANCES FOR EXAMPLE, ON 12042021 RS 104 CRORES WERE CREDITED IN 28 PARTS THROUGH RTGS FROM SHIVALAY VYAPAAR PRIVATE LIMITED ACCOUNT NO 50200016072655, IFSC CODEHDFC0000240, SR BULLION JEWELLERS ACCOUNT NO 918020033060778, IFSC CODEUTIB0000254, ARIHANT BULLION AND JEWELS LLP ACCOUNT NO 2512605193, IFSC CODEKKBK0000958 ETC AND ON THE SAME DAY THE ACCOUNT WAS DEBITED WITH SAME AMOUNT IN 26 PARTS THROUGH OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, DIJLLAH JEWELLERY FZCO OF UAE IN USD FOR PAYMENT TOWARDS IMPORT OF GOLD ON 30032021 RS 4336 CRORES WERE CREDITED IN 9 PARTS THROUGH RTGS FROM ARIHANT BULLION AND JEWELS LLP ACCOUNT NO 2512605193, IFSC CODEKKBK0000958, ELEGANCE TRADELINKS PVT LTD ACCOUNT NO 914020018093355, IFSC CODEUTIB0000268 AND ON THE SAME DAY THE ACCOUNT WAS DEBITED WITH RS 1757 CRORES IN 5 PARTS THROUGH OUTWARD REMITTANCE TOWARDS MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, DIJLLAH JEWELLERY FZCO OF UAE IN USD FOR PAYMENT TOWARDS IMPORT OF GOLD SIMILAR TRANSACTIONS SEEN IN THE ACCOUNT THERE WERE TOTAL 35 BOE WERE PENDING FOR SUBMISSION SINCE ACCOUNT OPENING TOTAL CREDIT TURNOVER IN THE ACCOUNT DURING THE FY 202021 WAS RS 63,84547 LAKHS, WHICH SEEMS TO BE VERY HIGH IN VIEW OF NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS, HIGH TURNOVER IN THE ACCOUNT AND BOE PENDING FOR SUBMISSION, STR IS BEING REPORTEDAGNIS BULLION HOUSE LLP, A LIMITED LIABILITY PARTNERSHIP FIRM, HAVING ADDRESS P NO 144 SEC 7,IMT MNS,G31 MANESAR, GURGAON, HARYANA, INDIA, 122052, CONTACT NUMBER 918448330316 AND EMAIL ID MOHAMMED@DIJLLAHGOLDCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 114505001886 WITH ICICI BANK LTD, PLOT NO B B1 VANIJAYA NIKUNJ, ENKAY TOWER, PHASEIV, UDYOG VIHAR, GURGAON, HARYANA, 122001, SINCE 06102020 DATE OF INCORPORATION IS 18122018 PAN NUMBER OF THE ACCOUNT IS ABNFA7936P IEC CODE OF THE CUSTOMER AS PER BANK RECORDS IS ABNFA7936P TRANSACTION LINKED ACCOUNTS ARE 623505389635, 114510005438 AUTHORIZED SIGNATORIESBENEFICIAL OWNERS PARTNERS OF THE REPORTED ACCOUNT ARE MOHAMMED HAMMOODI HASHIM ALABID RESIDENT ID OF UAE 784196527195497 SHARMIYA SHAROFUDDIN WITH PAN NO KVUPS6997D THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE PAN ABNFA7936P, COI AAN7913, LLP AGREEMENT CUSTOMER IS INTO BULLION BUSINESS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40 LAKHS TO LESS THAN RS 5 CRORES ALERT WAS TRIGGERED FOR NON CASH DEPOSIT TRANSACTIONS FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TRANSACTIONS BALANCE IN THE ACCOUNT AT 02052021 WAS RS 1,45411 LAKHS CREDIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 02052021 SHOWS TOTAL CREDIT RS 60,31439 LAKHS OF WHICH NEFT RS 60744 LAKHS MAJORLY FROM SHIVALAY VYAPAAR PRIVATE LIMITED, RAKSHA BULLION, JALAN COMPANY PVT LTD, RTGS RS 51,21351 LAKHS MAJORLY FROM RAKSHA BULLION, AGNIS BULLION HOUSE LLP, SR BULLION JEWELLERS, INTERNET FUND TRANSFERS RS 8,49242 LAKHS FROM VISTARAN COMMODITIES LLP 028105005781, MOBILE MONEY TRANSFERS RS 101 LAKHS IN 2 INSTANCES TOTAL DEBITS DURING THE SAME PERIOD WAS RS 60,15681 LAKHS OF WHICH TRANSFERS RS 9200 LAKHS MAJORLY TO AGNIS BULLION HOUSE LLP, RTGS RS 6,57869 LAKHS MAJORLY TO REMITTANCE RS 53,23392 LAKHS MAJORLY TO BRENLEY QUARTZ COMPANY LIMITED, DIJLLAH JEWELLERY FZCO, INTERNET FUND TRANSFER RS 24998 LAKHS TO JALAN SILVER GARDEN PRIVATE LIMITED 627905015422, MOBILE MONEY TRANSFERS RS 145 LAKHS IN 2 INSTANCESFURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS 4 YEARS OR ACCOUNT OPENING DATE IF EARLIER, YEARWISE WERE FY 202021 FROM 06102020 TO 31032021 TOTAL CREDIT SIDE RS 63,84547 LAKHS OF CLEARING RS 20150 LAKHS MAJORLY FROM AGNIS BULLION HOUSE LLP, NEFT RS 1,24460 LAKHS MAJORLY FROM AGNIS BULLION HOUSE LLP, SALASAR SYNTHETICS, SR BULLION JEWELLERS, RTGS RS 55,16215 LAKHS MAJORLY FROM SR BULLION JEWELLERS, ARIHANT BULLION AND JEWELS LLP, BILASA IMPEX, REMITTANCE RS 14619 LAKHS MAJORLY FROM MOHAMMED HAMMOODI HASHIM ALABID IN USD FROM UAE FOR FOREIGN DIRECT INVESTMENT IN EQUITY, INTERNET FUND TRANSFERS RS 2,69388 LAKHS FROM ARIHANT BULLION AND JEWELS LLP 623505389635, MOBILE MONEY TRANSFERS RS 084 LAKHS IN 7 INSTANCES, FIXED DEPOSIT RS 4,39533 LAKHS TOTAL DEBIT SIDE RS 62,54895 LAKHS OF WHICH TRANSFERS RS 13,27784 LAKHS MAJORLY TO ICICI LOMBARD GIC LTDPERSONAL LINE COLL, AGNIS BULLION HOUSE LLP, CUSTOM DUTY, NEFT RS 070 LAKHS MAJORLY TO MMTC PAMP INDIA PVT LTD, RTGS RS 603 LAKHS MAJORLY TO MMTC PAMP, REMITTANCE RS 48,27768 LAKHS MAJORLY TO DIJLLAH JEWELLERY FZCO OF UAE FOR GOLD IN USD, MERDIN KUYUMCULUK SANAYI TICARET LIMITED IN USD TO TURKEY FOR PAYMENT TOWARDS IMPORT OF GOLD DORE BAR, INTERNET FUND TRANSFER RS 95183 LAKHS TO BHOLASONS JEWELLERS GOLD SOUK 031405002444, MOBILE MONEY TRANSFERS RS 126 LAKHS IN 1 INSTANCE',
  {'entities': [(639, 661, 'FOREIGN ORG'),
    (1147, 1169, 'FOREIGN ORG'),
    (3622, 3644, 'FOREIGN ORG'),
    (4840, 4862, 'FOREIGN ORG'),
    (536, 576, 'FOREIGN ORG'),
    (1044, 1084, 'FOREIGN ORG'),
    (4887, 4927, 'FOREIGN ORG'),
    (3590, 3620, 'FOREIGN ORG')]}),
 ('ON CONDUCTING REVIEW SINCE INCEPTION TILL 05052021 FOLLOWING OBSERVATIONS WERE NOTED CREDIT NEFT TRANSFER OF RS 1 LAKH WAS RECEIVED FROM PARTNER NARESH KUMAR MIDHA IDFC FIRST BANK OTHER THAN THIS, CREDIT TOWARDS INITIAL FUNDING OF THE ACCOUNT WAS NOTED DEBIT RS 090 LAKH WERE TRANSFERRED TO NARESH KUMAR IDFC FIRST BANK SHORTLY POST INITIAL FUNDING MULTIPLE ATM WITHDRAWALS AGGREGATED TO RS 1 LAKH WERE NOTED ON 17 APRIL 2021; POST WHICH SINGLE LOW VALUE TRANSACTION WERE OBSERVED S SQUARE CONSULTANTS PVT LTD 000040418620019 THE CLIENT, SHADOW FILM PRODUCTIONS HOLDS CURRENT ACCOUNT 000040418620019 ADOPTED ON 15032021 WITH DEUTSCHE BANK, PUNE BRANCH AS PER THE KYC INFORMATION SUBMITTED DURING ADOPTION, THE CLIENT IS SERVICE PROVIDER INTO CULTURAL AND ENTERTAINMENT INDUSTRY, DECLARED ANNUAL INCOME IS 750 CRORES, ANNUAL TURNOVER IS RS 75 CRORES ENTITY IS BASED IN PUNE AND MANAGED BY FOLLOWING PARTNERS MR SHAM KESHAV CHAVAN DOB 22061973, PAN AHPPC7247M, ADDRESS 347 ADARSH NAGAR NEW LINK ROAD, JOGESHWARI WEST, MUMBAI 400102 MR NARESH KUMAR MIDHA DOB 04011977, PAN ALEPM1168N, ADDRESS FLAT NO 204 JUHU CREADO APARTMENTS JUHU CHURCH ROAD, ABOVE ANANT CLINIC, JUHU, MUMBAI 400049 DETAILS OF ENTITY DOI 07102014, PAN AAVCS0083R, GSTIN 27AAVCS0083R12D, ADDRESS OFFICE NO 401, AMANORA CHAMBER HADAPSAR KHARADI ROAD SADESATARA NALI HADAPSAR PUNE 411028 ACCOUNT WAS OPENED WITH IP CHEQUE 151680 OF RS 1 LAKH RECEIVED FROM CANARA BANK, PUNE BRANCH CPV IS POSITIVE KYC WAS UPDATED AT THE TIME OF ADOPTION & IS ACCOUNT COMPLIANT AS ON DATE; CUSTOMER HAS BEEN CATEGORIZED UNDER MEDIUM RISK ACCOUNT IS ACTIVE AS ON DATE OF REVIEW & BALANCE IS NIL LAST CUSTOMER INDUCED TRANSACTION WAS OBSERVED ON 17042021 ANNUAL TURNOVER 202021 TOTAL DEBIT NIL, TOTAL CREDIT RS 1 LAKHS CASH DEPOSIT & WITHDRAWAL NIL 202122 TOTAL DEBIT RS 1 LAKH CASH WITHDRAWAL RS 1 LAKH, TOTAL CREDIT NIL ON CONDUCTING REVIEW SINCE INCEPTION TILL 17042021 IT WAS NOTED THAT CREDIT ON ACCOUNT OF INITIAL FUNDING WAS RECEIVED, FOLLOWED BY MULTIPLE ATM WITHDRAWALS TO THE TUNE OF RS 1 LAKH ON SAME DAY, IE 17042021, POST WHICH NO TRANSACTIONS HAVE BEEN EXECUTED IN THE ACCOUNT BANK STAFF HAS RAISED ALERTS STATING THAT THE CUSTOMER SUBMITTED TO US TWO OUTWARD REMITTANCE DOCUMENTS OF USD 99230 AND USD 98030 IN LAST WEEK OF MARCH 2021 THE BENEFICIARY IN BOTH THE INVOICES WAS MAGNETIC MOTION PICTURE PRODUCTION LLC, DUBAI, UAE THE CUSTOMER SUBMITTED THE INVOICES AND THE UNDERLYING AGREEMENTS, OTHER RELATED DOCUMENTS HOWEVER THE INVOICES DID NOT MENTION THE ACTUAL PURPOSE OF PAYMENT THE CLIENT STATED THAT THEY HAD TAKEN FILM PRODUCTION RELATED INSTRUMENTS FROM THE BENEFICIARY AND THE PAYMENT IS TOWARDS THE RENT FOR THE SAME WHEN BANK OFFICIAL RAISED QUERY WITH THE CUSTOMER THAT THE INVOICE DOES NOT THE GIVEN REASON, CUSTOMER PROVIDED AN ALTERNATIVE INVOICE POST THIS, CUSTOMER PROVIDED TWO MORE INVOICES WHEREIN THE BENEFICIARIES WERE DIFFERENT HOWEVER THE PATTERN OF THE INVOICE AND THE UNDERLYING AGREEMENT ARE SIMILAR IN ALL CASES ALSO, DISCREPANCY RELATED TO PROFILE OF THE ENTITY WAS OBSERVED, SINCE CUSTOMER HAD STATED THAT THEY ARE INTO FILM PRODUCTION, HOWEVER LATER IT WAS CONFIRMED THAT THEY ARE PROVIDER OF TECHNICAL SERVICES TO PRODUCTION HOUSES, WHICH APPEARS TO BE UNUSUAL ALSO, THE TRANSACTIONS IN ALL THREE ACCOUNTS INDICATE SIMILAR PATTERN AND WITHDRAWALS ON SAME DAY WERE NOTED, POST WHICH NO ACTIVITY OBSERVED ADDITIONALLY, ALL THE THREE ENTITIES OPERATE FROM SINGLE ADDRESS & ALSO AS PER DETAILS SHARED, BOTH THE DIRECTORS ARE RESIDING IN MUMBAI, WHEREAS THESE ENTITIES ARE LOCATED IN PUNE THEREFORE, IT RAISES A CONCERN IN VIEW OF DISCREPANCIES NOTED IN THE PROFILE, PURPOSE OF TRADE TRANSACTIONS, ALONG WITH SIMILAR TREND OF TRANSACTIONS NOTED IN NEWLY ESTABLISHED ACCOUNTS, APPEARS TO BE UNUSUAL; HENCE AN STR IS BEING RAISEDTHE CLIENT, SLOW MOTION PICTURES PRIVATE LIMITED HOLDS CURRENT ACCOUNT 000040423860019 ADOPTED ON 19032021 WITH DEUTSCHE BANK, PUNE BRANCH AS PER THE KYC INFORMATION SUBMITTED DURING ADOPTION, THE CLIENT IS SERVICE PROVIDER INTO CULTURAL AND ENTERTAINMENT INDUSTRY, PROVIDING TECHNICAL SERVICES TO FILM PRODUCTION HOUSES AND CONSULTANCY DECLARED ANNUAL INCOME IS 7 CRORES, ANNUAL TURNOVER IS RS 70 CRORES ENTITY IS BASED IN PUNE AND MANAGED BY FOLLOWING DIRECTORS & UBOS MR SHAM KESHAV CHAVAN DOB 22061973, PAN AHPPC7247M, ADDRESS 347 ADARSH NAGAR NEW LINK ROAD, JOGESHWARI WEST, MUMBAI 400102 MR NARESH KUMAR MIDHA DOB 04011977, PAN ALEPM1168N, ADDRESS FLAT NO 204 JUHU CREADO APARTMENTS JUHU CHURCH ROAD, ABOVE ANANT CLINIC, JUHU, MUMBAI 400049 DETAILS OF ENTITY DOI 04082011, PAN AAQCS1627J, GSTIN 27AAQCS1627J2ZX, ADDRESS OFFICE NO 401, AMANORA CHAMBER HADAPSAR KHARADI ROAD SADESATARA NALI HADAPSAR PUNE 411028 ACCOUNT WAS OPENED WITH IP CHEQUE 151655 OF RS 1 LAKH RECEIVED FROM CANARA BANK, PUNE BRANCH CPV IS POSITIVE KYC WAS UPDATED AT THE TIME OF ADOPTION & IS COMPLIANT AS ON DATE & CUSTOMER HAS BEEN CATEGORIZED UNDER MEDIUM RISK ACCOUNT IS ACTIVE AS ON DATE OF REVIEW & BALANCE IS NIL LAST CUSTOMER INDUCED TRANSACTION WAS OBSERVED ON 17042021 AN ALERT WAS RAISED BY BANK STAFF AS DISCREPANCIES WERE NOTED IN THE INVOICES SHARED FOR TRADE TRANSACTIONS IN LIGHT OF THIS, REVIEW HAS BEEN CONDUCTED ON THE ACCOUNT SINCE INCEPTION TILL 17042021 AND FOLLOWING ARE THE MAJOR OBSERVATION NOTED ANNUAL TURNOVER 202021 TOTAL DEBIT RS 7276 LAKHS, TOTAL CREDIT RS 7376 LAKHS CASH DEPOSIT & WITHDRAWAL NIL 202122 TOTAL DEBIT RS 1 LAKH CASH WITHDRAWAL RS 1 LAKH, TOTAL CREDIT NIL CREDITS NEFT TRANSFERS AGGREGATED TO RS 7276 LAKHS WERE RECEIVED FROM CINE 1 STUDIOS COSMOS COOP BANK OTHER THAN THIS, CREDIT TOWARDS INITIAL FUNDING OF THE ACCOUNT WAS NOTED DEBITS RTGS OF RS 7186 LAKHS TO CINE 1 STUDIOS COSMOS BANK, RS 090 LAKHS TO MR SHAM CHAVAN DIRECTOR, IDFC FIRST BANK MULTIPLE ATM WITHDRAWALS AGGREGATED TO RS 1 LAKH WERE NOTED ON 17 APRIL 2021; POST WHICH NO TRANSACTIONS WERE OBSERVED FOLLOWING RELATED PARTIES ALSO HOLD ACCOUNT WITH DEUTSCHE BANK, PUNE BRANCH SHADOW FILM PRODUCTIONS 000040423890019 THE CLIENT, SHADOW FILM PRODUCTIONS HOLDS CURRENT ACCOUNT 000040423890019 ADOPTED ON 19032021 WITH DEUTSCHE BANK, PUNE BRANCH AS PER THE KYC INFORMATION SUBMITTED DURING ADOPTION, THE CLIENT IS SERVICE PROVIDER INTO CULTURAL AND ENTERTAINMENT INDUSTRY, DECLARED ANNUAL INCOME IS 3 CRORES, ANNUAL TURNOVER IS RS 30 CRORES ENTITY IS BASED IN PUNE AND MANAGED BY FOLLOWING PARTNERS MR SHAM KESHAV CHAVAN DOB 22061973, PAN AHPPC7247M, ADDRESS 347 ADARSH NAGAR NEW LINK ROAD, JOGESHWARI WEST, MUMBAI 400102 MR NARESH KUMAR MIDHA DOB 04011977, PAN ALEPM1168N, ADDRESS FLAT NO 204 JUHU CREADO APARTMENTS JUHU CHURCH ROAD, ABOVE ANANT CLINIC, JUHU, MUMBAI 400049 DETAILS OF ENTITY DOI 14062018, PAN AEIFS4409P, GSTIN27AEIFS4409P12E, ADDRESS OFFICE NO 401 AMANORA CHAMBERS TOWN CENTER AMANORA HADAPSAR KHARADI ROAD PUNE 411028 ACCOUNT WAS OPENED WITH IP CHEQUE 151706 OF RS 1 LAKH RECEIVED FROM CANARA BANK, PUNE BRANCH CPV IS POSITIVE KYC WAS UPDATED AT THE TIME OF ADOPTION & IS ACCOUNT COMPLIANT AS ON DATE; CUSTOMER HAS BEEN CATEGORIZED UNDER MEDIUM RISK ACCOUNT IS ACTIVE AS ON DATE OF REVIEW & BALANCE IS RS 010 LAKHS LAST CUSTOMER INDUCED TRANSACTION WAS OBSERVED ON 05052021 ANNUAL TURNOVER 202021 TOTAL DEBIT RS 090 LAKH, TOTAL CREDIT RS 2 LAKHS CASH DEPOSIT & WITHDRAWAL NIL 202122 TOTAL DEBIT RS 1 LAKH CASH WITHDRAWAL RS 1 LAKH, TOTAL CREDIT RS 00001 LAKH',
  {'entities': [(2334, 2372, 'FOREIGN ORG')]}),
 ('INR 2004 CRORES TOTAL RTGSNEFT CREDITSINR 874 CRORES, TOTAL FCW CREDITS  INR 1052 CRORES, TOTAL CHEQUE DEPOSITS  INR 7508 LAKHS, TOTAL IMPS CREDIT  INR 100 LAKH INITIAL PAYMENT  INR 100 LAKH MAJOR CREDITS ARE FROM RTGSNEFT CREDITS FROM CLIENTS OWN ACCOUNT MAINTAINED WITH ICICI BANK ACCOUNT NUMBER 25905005057, HDFC BANK ACCOUNT NUMBER 59209820121715 FOR VALUE INR 874 CRORES FOREIGN CURRENCY WITHDRAWALS RECEIVED IN THE ACCOUNT FOR VALUE INR 1052 CRORES CHEQUE DEPOSITS RECEIVED IN THE ACCOUNT FOR VALUE INR 7508 LAKHS IMPS CREDIT 1 NO FOR VALUE INR 100 LAKH AND INITIAL PAYMENT FOR VALUE INR 100 LAKH TOTAL DEBIT IN THE CLIENT ACCOUNT INR 1995 CRORES ,TOTAL RTGSNEFT DEBITS INR 1904 CRORES TOTAL CHEQUE WITHDRAWALSINR 7500 LAKHS TOTAL TRANSFER DEBITSINR 1500 LAKHS TOTAL CHARGES DEBITINR 9534600 MAJOR DEBITS IN THE CLIENT ACCOUNT ARE TOWARD RTGSNEFT DEBIT TO ROYAL PETRO OIL REFINERY LLP CLIENT OWN ACCOUNT MAINTAINED WITH ICICI BANK ACCOUNT NUMBER 25905005057  FOR VALUE INR 597 CRORES, UNIVERSAL IMPEX HOLDING ACCOUNT WITH ICICI BANK ACCOUNT NUMBER 25905502153  FOR VALUE INR 260 CRORES, HEMANT SHAH HOLDING ACCOUNT WITH YES BANK ACCOUNT NUMBER 5890700001662 FOR VALUE INR 7500 LAKHS , TIME TECHNOPLAST LTD HOLDING ACCOUNT WITH STATE BANK OF INDIA ACCOUNT NUMBER 38155990952 FOR VALUE INR 7273 LAKHS AND OTHER LOW VALUE CREDITS TO LOCAL PARTIES CHEQUE WITHDRAWAL TO SELF FOR VALUE INR 7500 LAKHS INTERNAL FUND TRANSFER DEBIT MADE TO ROYAL PETRO SPECIALITIES PVT LTD ACCOUNT NUMBER 201003805290 FOR VALUE INR 1500 LAKHS CHARGES DEBIT IN THE ACCOUNT FOR VALUE INR 9534600 8 HIGH VALUE INWARD CREDITS RECEIVED IN THE EEFC ACCOUNT ARE CREDITED TO CURRENT ACCOUNT AS FCW WHICH ARE IMMEDIATELY TRANSFERRED TO CLIENT OWN ACCOUNT MAINTAINED WITH ANOTHER BANK 9 MAJOR CREDITS ARE RECEIVED VIA RTGSNEFT FROM CLIENT OWN ACCOUNT MAINTAINED WITH ICICI BANK AND HDFC BANK SOURCE OF FUNDS NOT CLEAR 10UCIC OF THE CLIENT IS 48174488 AND PARTNERS OF THE COMPANY ALSO HOLDS MULTIPLE ACCOUNTS WITH US ROYAL PETRO SPECIALITIES PVT LTD ACCOUNT NUMBER 201003805290, ACCOUNT OPENED ON 14102019 CUMULATIVE DEBIT BALANCE  INR 1,27,10,81,67684 AND CUMULATIVE CREDIT BALANCE  INR 1,26,36,70,57445 ACCOUNT BALANCE  INR 1,03,60,09462 CREDIT , KYC DOCUMENTS OBTAINED AT THE TIME OF ACCOUNT OPENING ARE AADHAAR, PAN, BOARD RESOLUTION, CERTIFICATE OF INCORPORATION SIMILAR IRAN INVOLVEMENT NOTICED IN THIS ACCOUNT AS WELL DOCUMENT PENDENCY 2 BOE OVERDUE IN OUR BOOKS AND 27 BILLS OVERDUE IN IDPMS UNDER OUR AD CODE 1 BILL OVERDUE IN EDPMS UNDER OUR AD CODE CLIENT REPORTED TO ENFORCEMENT DIRECTORATE ON 30042021 UNDER EDPMS AND IDPMS DEFAULTER REPORTING ROYAL PETROCHEM ACCOUNT NUMBER 259820121715ACCOUNT OPENED ON 21072016 CUMULATIVE DEBIT BALANCE  INR 2,96,04,07,10712 AND CUMULATIVE CREDIT BALANCE  INR 2,96,07,06,44560 ACCOUNT BALANCE  INR 2,99,33848 CREDIT LINE OF BUSINESSMANUFACTURER AND TRADER OF OIL REFINERY INDUSTRIES KYC DOCUMENTS OBTAINED AT THE TIME OF ACCOUNT OPENING ARE PAN, REGISTRATION CERTIFICATE, DGFT CERTIFICATE DOCUMENT PENDENCY 5 BOE OVERDUE IN OUR BOOKS AND 243 BILLS OVERDUE IN IDPMS UNDER OUR AD CODE 9 EXPORT ADVANCE OVERDUE IN OUR BOOKS AND 292 BILL OVERDUE IN EDPMS UNDER OUR AD CODE CLIENT REPORTED TO ENFORCEMENT DIRECTORATE ON 10072018 UNDER EDPMS AND ON 07082018 UNDER IDPMS DEFAULTER REPORTING RED FLAGS SUSPICION RAISED DUE TO BELOW MENTIONED RED FLAGS 1SUSPICION OF DISGUISED TRANSACTIONS INVOLVING IRAN PORT 2OVERDUE BILL OF ENTRY IN OUR BOOKS AND OVERDUE BILLS IN IDPMS UNDER OUR AD CODE SINCE AUG 2016 3OVERDUE EXPORT ADVANCE IN OUR BOOKS AND OVERDUE BILLS IN EDPMS UNDER OUR AD CODE SINCE AUG 2016 4 HIGH VALUE INWARD REMITTANCES RECEIVED IN THE CLIENT ACCOUNT ARE IMMEDIATELY TRANSFERRED TO CLIENT OWN ACCOUNT WITH OTHER BANK 5MAJOR CREDITS IN THE ACCOUNT FROM RTGSNEFT CREDIT FROM CLIENT OWN ACCOUNT WITH OTHER BANK SOURCE OF FUNDS NOT CLEARACCOUNT WAS INVESTIGATED FOR THE ABOVE CLIENT BASED ON DISGUISED TRANSACTIONS INVOLVING IRAN 1ROYAL PETRO OIL REFINERY LLP 301 SAI COMMERCIAL,GOVANDI STATION ROAD HOVANDI EAST,SAI COMMERCIAL COMPLEX,MUMBAI,MAHARASHTRA,INDIA,400088 IS A LIMITED LIABILITY PARTNERSHIP INCORPORATED ON 05122014CURRENT ACCOUNT 201003947624 OPENED WITH US ON 15012020 AT OUR ANDHERI BRANCH, MAHARASHTRA INDUSIND BANK LTDACME PLAZA,CTS NO 32 OPP SANGAM TALKIES,ANDHERIKURLA ROAD,ANDHERIE,MUMBAI400059 CURRENT STATUS OF THE ACCOUNT IS ACTIVE CURRENT BALANCE IN THE ACCOUNT IS INR 14,92,91987 CREDIT AS ON 31052021CUMULATIVE CREDIT BALANCE IS INR 21,63,03,75508 & CUMULATIVE DEBIT BALANCE IS INR 21,60,84,61422CLIENT DECLARED ANNUAL TURNOVER OF INR 10 CRORES AND ABOVE LAST CUSTOMER INDUCED TRANSACTION IS 29052021NATURE OF BUSINESS IS TRADING OF PETROLEUM AND PETRO CHEMICAL PRODUCTS AS PER ACCOUNT OPENING FORM PARTNERS OF THE COMPANY ARE HEMANT SHAH MOBILE NUMBER 8010860621 ,VISHAL VED MOBILE NUMBER 8238081913 ,PRAMESH VED MOBILE NUMBER 8238081913KYC DOCUMENT OBTAINED AT THE TIME OF ACCOUNT OPENING ARE AADHAAR, PAN, BOARD RESOLUTION, CERTIFICATE OF INCORPORATION, DGFT CERTIFICATE OF THE FIRM IS \'\'3716900257\'\' ISSUED ON 10052016PAN OF THE COMPANY IS AARFR9211RGSTIN NO 24AARFR9211R1ZE CURRENT STATUS IS ACTIVE AS PER PUBLIC DOMAIN "HTTPSSERVICESGSTGOVINSERVICESSEARCHTPBYPAN" THE CLIENT IS MAPPED UNDER RISK CATEGORY \'\'HIGH" AND LAST KYC WAS DONE ON 27092020 2FROM THE PERIOD 13052020 TO 31052021, CLIENT HAS MADE 60 OUTWARD REMITTANCES TOWARDS IMPORT OF BASE OIL, USED MOTOR OIL, FREIGHT CHARGES, AD TRANSFERS FOR VALUE INR 2678 CRORES OVERSEAS PARTIES ARE BASED IN VARIOUS COUNTRIES RANDOM CHECK DONE ON THE OVERSEAS PARTY QATAR CHEMICAL AND PETROCHEMICAL MARKETING AND DISTRIBUTION COMPANY, QATAR, LINE OF BUSINESS MATCHES WITH THE CLIENT BUSINESS AND THE SAME HAS BEEN VERIFIED THROUGH HTTPSWWWQCHEMCOMQAAPPPAGESMARKETINGASPX 3DURING THE SAME PERIOD ABOVE, CLIENT HAS RECEIVED 108 INWARD REMITTANCES TOWARDS EXPORT OF LUBRICANTS FOR VALUE INR 4425 CRORES OVERSEAS PARTIES BASED IN VARIOUS COUNTRIES RANDOM CHECK DONE ON THE OVERSEAS PARTY NAS PETRO CHEM LTD, KENYA AND THE LINE OF BUSINESS MATCHES AND THE SAME HAS BEEN VERIFIED THROUGH WEB SEARCH HTTPSWWWBUSINESSLISTCOKECOMPANY101618NASPETROCHEMLTD 4AS ON 31052021, CLIENT HAS 20 BOE\'S OUTSTANDING AND YET TO BE OVERDUE AND OLDEST BILL DATED 16JUNE2020 5 AS ON 31052021, CLIENT HAS 53 EXPORT ADVANCE OUTSTANDING AND YET TO BE OVERDUE OLDEST BILL DATED 16JUNE2020 AS ON 31052021, CLIENT HAS 56 SHIPPING BILLS OUTSTANDING IN EDPMS OLDEST BILL DATED 20052020 6 CLIENT SUBMITTED DOCUMENTS FOR OUTWARD REMITTANCE IMPORT ADVANCE ORM0018200424908TRANSACTION VALUEAED 10239299, ORM0018200446589TRANSACTION VALUE AED 311,36300 AND ORM0018200443899 TRANSACTION VALUE AED 311,36260 AS PER BILL OF LADING NUMBER BSLJEAMUN132278 PORT OF LOADING AS JEBEL ALI,UAE AND BL ISSUE DATE AS 05112020IT WAS NOTICED THAT THE VESSEL IN THE BILL LADINGS HAVE IRAN LINKAGE IN LLOYDS SYSTEMS AND BL ISSUE DATE WAS 05112020 HOWEVER, VESSEL WAS AT BANDAR ABBAS, IRAN DURING 04112020 TO 10112020 IT FURTHER MOVED TO INDIA AND REACHED MUNDRA, INDIA ON 10112020 IN THE TRANSACTION, PORT OF LOADING WAS CAMOUFLAGED AS UAE PORT WHEREAS TRANSACTION IS IRANIAN IN NATURE AS PER BILL OF LADING NUMBER SEAMUN7155 PORT OF LOADING AS JEBEL ALI,UAE AND BL ISSUE DATE AS 27082020IT WAS NOTICED THAT THE VESSEL IN THE BILL LADINGS HAVE IRAN LINKAGE IN LLOYDS SYSTEMS AND BL ISSUE DATE WAS 27082020 HOWEVER VESSEL WAS AT BANDAR ABBAS, IRAN DURING 25082020 TO 27082020 IT FURTHER MOVED TO INDIA AND REACHED MUNDRA, INDIA ON 01092020 IN THE TRANSACTION, PORT OF LOADING WAS CAMOUFLAGED AS UAE PORT WHEREAS TRANSACTION IS IRANIAN IN NATURE 7 QUANTUM OF TRANSACTIONS FOR CURRENT YEAR JAN 2020APR 2021TOTAL CREDIT IN THE CLIENT ACCOUNT',
  {'entities': [(1191, 1211, 'FOREIGN ORG'),
    (6847, 6861, 'FOREIGN ORG'),
    (7306, 7320, 'FOREIGN ORG')]}),
 ('PREVIOUSLY, STR HAS BEEN FILED ON IMAGINARY FILMS PVT LTD AC NO 102563400000299 ON 22ND MARCH 2021 VIDE FIU BATCH ID 2103245908 FOR RED FLAGS OBSERVED IN THE ACCOUNT AND DUE TO OPAQUE CREDENTIALS OF OVERSEAS ENTITY LIVING FILMS PTE LTD, SINGAPORECUSTOMERS STATED ANNUAL INCOME IS LESS THAN RS5 CRORES HOWEVER IN LESS THAN 2 MONTHS SINCE 22ND MARCH 2021 CUSTOMER HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS206 CRORES FOLLOWED BY IMMEDIATE CROSS BORDER OUTWARD REMITTANCES IN FAVOR MULTIPLE OVERSEAS ENTITIES AS LISTED ABOVE 1BELOW ARE RED FLAGS OBSERVED ON GATE STUDIOS PTE LTD, SINGAPORE AS PER PUBLIC DOMAIN MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS 736 BEDOK RESEROIR ROAD, 0527 SINGAPORE 479264 OF AFOREMENTIONED ENTITY WEBSITE OF AFOREMENTIONED ENTITY HTTPSWWWGATESTUDIOSCOMINDEXHTML LOOKS DUBIOUS WITH CONTENTS BEING COPY PASTED FROM OTHER WEBSITES REFER HTTPSWWWB2WTVCUSTOMGIFANIMATEDGATE STUDIOS PTE LTD, SINGAPORE IS A COMMON BENEFICIARY RECEIVING REMITTANCE FROM ANOTHER YBL CUSTOMER KEYFRAME MULTIMEDIA SERVICES ON WHICH STR WAS ALREADY FILED VIDE FIU BATCH ID 2103183777ON COMPARING INVOICES DRAWN BY GATE STUDIOS PTE LTD ON YBL CUSTOMERS VIZ KEYFRAME MULTIMEDIA SERVICES AND IMAGINARY FILMS PVT LTD IT WAS OBSERVED THAT OVERSEAS ENTITY ADDRESS AND BENEFICIARY BANK ACCOUNT NUMBER IS THE SAME HOWEVER INVOICE FORMAT INCLUDING LETTER HEAD AND LOGO DOES NOT MATCH2BELOW ARE RED FLAGS OBSERVED WITH REGARDS TO OVERSEAS ENTITY MAGNETIC MOTION PICTURE PRODUCTION LLC, UAE WEBSITE ADDRESS MAGNETICPICTURECOM AS PROVIDED IN INVOICE IS INVALID MAGNETIC MOTION PICTURE PRODUCTION LLC IS A COMMON BENEFICIARY RECEIVING REMITTANCE FROM ANOTHER YBL CUSTOMER DIGI MEDIA AND FILMS SOLUTIONS ON WHICH STR WAS ALREADY FILED VIDE FIU BATCH ID 2104260055ON COMPARING INVOICES DRAWN BY MAGNETIC MOTION PICTURE PRODUCTION LLC ON YBL CUSTOMERS VIZ DIGI MEDIA AND FILMS SOLUTIONS AND IMAGINARY FILMS PVT LTD IT WAS OBSERVED THAT OVERSEAS ENTITY ADDRESS AND BENEFICIARY BANK ACCOUNT NUMBER IS THE SAME HOWEVER INVOICE FORMAT INCLUDING LETTER HEAD AND LOGO DOES NOT MATCHAS PER PUBLIC DOMAIN, MULTIPLE ENTITIES ARE REGISTERED AT THE ADDRESS AS PROVIDED IN INVOICE  1101, AL MASRAF TOWER, BANIYAS ROAD, DEIRA, DUBAI, UAE, POBOX 242874 OF SAID OVERSEAS ENTITYONE OF THE ENTITIES REGISTERED AT AFOREMENTIONED ADDRESS IS WHITE RABBIT MOTION PICTURES PRODUCTION LLC AS PER AVAILABLE OPINION REPORT WHITE RABBIT MOTION PICTURES PRODUCTION LLC AND MAGNETIC MOTION PICTURE PRODUCTION LLC ARE GROUP ENTITIESIT IS OBSERVED THAT WHITE RABBIT MOTION PICTURES PRODUCTION LLC, UAE IS COMMON BENEFICIARY RECEIVING CROSS BORDER REMITTANCES FROM MULTIPLE YBL CUSTOMER VIZ KEYFRAME MULTIMEDIA SERVICES, SOLITUDE ENTERTAINMENT LLP AND EYE FOCUS PRODUCTION WHEREIN INVOICE DRAWN ON EACH OF THESE CUSTOMER WHEN COMPARED DID NOT MATCH WITH RESPECT TO COMPANY LOGO AND OVERALL INVOICE FORMAT STRS WERE THEREFORE FILED ON BELOW CUSTOMERS VIDE FIU BATCH ID AS UNDER KEYFRAME MULTIMEDIA SERVICES  2103183777SOLITUDE ENTERTAINMENT LLP  2103256408EYE FOCUS PRODUCTION 21020935393FURTHER, THERE IS NO INFORMATION OF OVERSEAS ENTITY THE PRODUCTION SERVICE FZCO, UAE ON PUBLIC DOMAININ LIGHT OF HIGH VALUE CROSS BORDER REMITTANCES BEING UNDERTAKEN IN FAVOR OF OVERSEAS ENTITIES WITH UNSATISFACTORY CREDENTIALS IT HAS BEEN DECIDED ON 31MAY2021 TO FILE STR ON SAID CUSTOMER1 THE CUSTOMER IMAGINARY FILMS PVT LTD PAN AAFCI5545Q, DOI 22012020, COMMUNICATION AND PERMANENT ADDRESS 402 EMBASSY CHAMBERS 3RD ROAD, KHAR WEST, MUMBAI, 400052, MAHARASHTRA , GSTIN 27AAFCI5545Q1ZW, REGISTRATION NUMBER 336424, CIN U92490MH2020PTC336424, MOBILE NUMBER 917039548313, EMAIL ID INFO@IMAGINARYFILMSIN IS MAINTAINING A CURRENT AC NO 102563400000299 SINCE 28122020 WITH UDYOG VIHAR, GURGAON BRANCH ADDRESS YES BANK LTD,FIRST FLOOR,PLOT A,VANIJYA NIKUNJ,ENKAY TOWER ,UDYOG VIHAR ,PHASE 5 ,GURGAON1220162 THE CUSTOMER FALLS UNDER MEDIUM RISK CATEGORY WITH KYC LAST UPDATED IN DEC2020 MANISH KUMAR SINGH AND SAPNA KUMARI ARE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS FILM PRODUCTIONDISTRIBUTION HOUSE WITH AN ANNUAL GROSS INCOME LESS THAN RS5 CRORES ADDITIONALLY 1 MORE ACCOUNT ARE FOUND TO BE LINKED TO THE ACCOUNT OF IMAGINARY FILMS PVT LTD AC NO102563400000299 AS UNDER THE CUSTOMER RAPTAS GLOBAL SOLUTIONS AC NO 042163700000980 PAN ABAFR5558D, DOI 19112020, COMMUNICATION AND PERMANENT ADDRESS SHOP NO 45 A LEVEL II DEWAN CENTRE, 183 186 S V ROAD JOGESHWARI WEST, MUMBAI SUBURBAN, MUMBAI, 400102 , EMAIL ID RAPTASSALUTIONS@GMAILCOM IS LINKED WITH ACCOUNT OF IMAGINARY FILMS PVT LTD AC NO102563400000299 BY COMMON MOBILE NUMBER  9170395483133 IN THE ACCOUNT OF IMAGINARY FILMS PVT LTD AC NO 102563400000299 AND 1 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM 01042021 TILL 01062021, SHOWS 89 CREDIT TRANSACTIONS AMOUNTING TO RS 156,982,004 AND 22 DEBIT TRANSACTIONS AMOUNTING TO RS 144,828,224OUT OF THE TOTAL CREDITS OF RS 156,982,004 IN THE ACCOUNTS, RTGS AMOUNTS TO RS 156,832,004 CONSTITUTING AROUND 100 PERCENT OF THE TOTAL CREDITSOUT OF THE TOTAL DEBITS OF RS 144,828,224 IN THE ACCOUNTS, RTGS AMOUNTS TO RS 16,100,000 CONSTITUTING AROUND 11 PERCENT OF THE TOTAL DEBITS, TRADE TRANSACTION AMOUNTS TO RS 128,728,224 CONSTITUTING AROUND 89 PERCENT OF THE TOTAL DEBITS, ,, 4 IN THE ACCOUNT OF IMAGINARY FILMS PVT LTD AC NO 102563400000299 AND 1 LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM 01042020 TILL 31032021 TRANSACTIONS ANALYSIS SHOWS 82 CREDIT TRANSACTIONS AMOUNTING TO RS 119,234,774 COMPRISING OF NIL CASH DEPOSIT AND 35 DEBIT TRANSACTIONS AMOUNTING TO RS 111,137,183 COMPRISING OF NIL CASH WITHDRAWALFROM 01042019 TILL 31032020 TRANSACTIONS ANALYSIS SHOWS 1 CREDIT TRANSACTIONS AMOUNTING TO RS 200,000 COMPRISING OF NIL CASH DEPOSIT AND 1 DEBIT TRANSACTIONS AMOUNTING TO RS 200,000 COMPRISING OF NIL CASH WITHDRAWALFROM 01042018 TILL 31032019 NO TRANSACTIONS WERE RECORDED IN THE ACCOUNTS OF IMAGINARY FILMS PVT LTD AC NO 102563400000299 AND RAPTAS GLOBAL SOLUTIONS AC NO 042163700000980 SINCE ACCOUNTS WERE OPENED IN THE YEAR DECEMBER 2020 AND DECEMBER 2019 RESPECTIVELY BALANCE IN THE ACCOUNT AS ON 01062021 IS RS237,60591NO SUSPICION WAS OBSERVED IN THE ACCOUNT OF RAPTAS GLOBAL SOLUTIONS AC NO 042163700000980, HOWEVER, WHILE REVIEWING TRANSACTION PATTERN AND OF IMAGINARY FILMS PVT LTD AC NO 102563400000299 IT WAS OBSERVED THAT  SINCE DECEMBER 2020, CLIENT HAS RECEIVED ACCOUNT CREDITS TO THE TUNE OF RS276 CRORES WITH MAJOR CREDITS VIA RTGS FROM SAWARTHIYA FILM PRODUCTIONS RS202 CRORES IN 119 TRANCHES FROM ITS AC NO 135205000593 HELD WITH ICICI BANK, ICIC0000011 AND IMAGINARY FILMS PRIVATE LIMITED RS53 CRORES IN 13 TRANCHES FROM AC NO 119405001855 HELD WITH ICICI BANK, ICIC0000011 AND AC NO 920020074440297 WITH AXIS BANK, UTIB0000001 ACCOUNT CREDITS WERE FOLLOWED BY IMMEDIATE OUTWARD REMITTANCE TO THE TUNE OF RS224 CRORES IN FAVOR OF BELOW MENTIONED OVERSEAS ENTITIES TOWARDS AUDIOVISUAL AND RELATED SERVICES ALIVING FILMS PTE LTD, SINGAPORE  RS7 CRORESBGATE STUDIOS PTE LTD, SINGAPORE RS73 CRORESCMAGNETIC MOTION PICTURE PRODUCTION LLC, UAE RS14 CRORESDTHE PRODUCTION SERVICE FZCO, UAE RS67 CRORES',
  {'entities': [(559, 579, 'FOREIGN ORG'),
    (899, 919, 'FOREIGN ORG'),
    (1121, 1141, 'FOREIGN ORG'),
    (7004, 7024, 'FOREIGN ORG'),
    (1443, 1481, 'FOREIGN ORG'),
    (1556, 1594, 'FOREIGN ORG'),
    (1787, 1825, 'FOREIGN ORG'),
    (2437, 2475, 'FOREIGN ORG'),
    (7048, 7086, 'FOREIGN ORG'),
    (215, 235, 'FOREIGN ORG'),
    (6960, 6980, 'FOREIGN ORG'),
    (1002, 1030, 'FOREIGN ORG'),
    (1163, 1191, 'FOREIGN ORG'),
    (2651, 2679, 'FOREIGN ORG'),
    (2937, 2965, 'FOREIGN ORG'),
    (1666, 1696, 'FOREIGN ORG'),
    (1847, 1877, 'FOREIGN ORG'),
    (2313, 2356, 'FOREIGN ORG'),
    (2389, 2432, 'FOREIGN ORG'),
    (2514, 2557, 'FOREIGN ORG'),
    (2681, 2707, 'FOREIGN ORG'),
    (2977, 3003, 'FOREIGN ORG'),
    (34, 57, 'FOREIGN ORG'),
    (1196, 1219, 'FOREIGN ORG'),
    (1882, 1905, 'FOREIGN ORG'),
    (3351, 3374, 'FOREIGN ORG'),
    (4193, 4216, 'FOREIGN ORG'),
    (4536, 4559, 'FOREIGN ORG'),
    (4638, 4661, 'FOREIGN ORG'),
    (5271, 5294, 'FOREIGN ORG'),
    (5927, 5950, 'FOREIGN ORG'),
    (6302, 6325, 'FOREIGN ORG'),
    (4260, 4283, 'FOREIGN ORG'),
    (5977, 6000, 'FOREIGN ORG'),
    (6203, 6226, 'FOREIGN ORG')]}),
 (' THE CENTRAL BUREAU OF INVESTIGATION HAS BOOKED US AWASTHI, THE CEO AND MD OF INDIAN FARMERS FERTILIZER COOPERATIVE IFFCO LTD, HIS TWO SONS, AND SEVEN OTHERS FOR ALLEGED CORRUPTION AND CAUSING LOSS TO THE EXCHEQUER BY IMPORTING FERTILIZER RAW MATERIAL AT INFLATED PRICES THE AGENCY WEDNESDAY CONDUCTED SEARCHES AT 12 LOCATIONS INCLUDING AWASTHIS OFFICE AND RESIDENCE IN DELHI, GURUGRAM AND MUMBAI, AMONG OTHER PLACES ACCORDING TO CBI, IFFCO AND ANOTHER FIRM, INDIAN POTASH LTD, HAVE BEEN IMPORTING THOUSANDS OF METRIC TONNES OF FERTILIZERS AND FERTILIZER RAW MATERIALS FROM MULTIPLE FOREIGN SUPPLIERS THE FERTILIZERS, SUBSIDISED BY THE CENTRE, ARE SUPPLIED TO FARMERS IN ORDER TO CHEAT GOVT OF INDIA BY CLAIMING HIGHER SUBSIDY, THESE OFFICIALS OF IFFCO AND INDIAN POTASH LIMITED HAVE BEEN IMPORTING FERTILIZERS AND RAW MATERIALS THROUGH MS KISAN INTERNATIONAL TRADING FZE IN DUBAI A SUBSIDIARY OF IFFCO AND OTHER MIDDLEMEN AT HIGHLY INFLATED RATES COVERING THE COMMISSION MEANT FOR THE ACCUSED INCLUDING THEN MD OF IFFCO THEN MD OF INDIAN POTASH LTD, WHICH WAS PAID  TRANSFERRED BY THE OVERSEAS SUPPLIERS TO THE ACCUSED OFFICIALS THROUGH SHAM TRANSACTIONS INVOLVING THE SAID ACCUSED, THE CBI SAID IN A STATEMENT THE CBI CASE IS BASED ON REFERENCES RECEIVED FROM THE MINISTRY OF CHEMICALS AND FERTILIZERS, APART FROM INFORMATION DEVELOPED BY IT APART FROM AWASTHI, THOSE NAMED IN THE FIR INCLUDE PRAVINDER SINGH GAHLAUT, THEN MD, INDIAN POTASH LTD IPL; AMOL AWASTHI SON OF US AWASTHI, PROMOTER OF MS CATALYST BUSINESS ASSOCIATES PVT LTD; ANUPAM AWASTHI SON OF US AWASTHI, PROMOTER OF MS CATALYST BUSINESS SOLUTIONS PVT LTD; VIVEK GAHLAUT SON OF PRAVINDER SINGH GAHLAUT; PANKAJ JAIN, JYOTI GROUP OF COMPANIES AND RARE EARTH GROUP, DUBAI; SANJAY JAIN BROTHER OF PANKAJ JAIN, PRESIDENT OF MS JYOTI TRADING CORPORATION AND BENEFICIAL OWNER OF OTHER ENTITIES OF JYOTI GROUP; AMRINDER DHARI SINGH, SR VICE PRESIDENT OF MS JYOTI TRADING CORPORATION, DUBAI; RAJIV SAXENA, CHARTERED ACCOUNTANT AND PROMOTER OF MIDAS METAL INTERNATIONAL LLC AND CERTAIN OTHER COMPANIES; AND SUSHIL KUMAR PACHISIA, AN EMPLOYEE OF PANKAJ JAIN ON THE BASIS OF THIS A STR HAS BEEN FILED THE NEWS ARTICLE IS SEBI FINES BIOCON, ITS DESIGNATED PERSON FOR VIOLATING MARKET NORMS NEWS SOURCE HTTPS INDIANEXPRESSCOMARTICLEINDIAIFFCOUSAWASTHICBI7322052 FULL NAME AND ADDRESS OF THE ACCOUNT HOLDER CATALYST BUSINESS SOLUTIONS PRIVATE LIMITED, PLOT NO 221 PHASE IV UDYOG VIHAR, , GURGAON, HARYANA, 122016, INDIA, CUSTOMER TYPE PRIVATE LTD, CUSTOMER STATUS GENERAL, OCCUPATION BUSINESS, CONSTITUTION PRIVATE NONFINANCIAL CORPORATIONS, RISK CATEGORY HIGH, PAN NO AACCC5353M, NAME OF THE AUTHORISED SIGNATORIES PROSANTA KUMAR RAY, KYC COMPLIANCE STATUS , DATE OF CUST ID ACTIVATION 07062013, DATE OF LAST KYC UPDATION JUN13, ANNUAL TURNOVER OF BUSINESS AS DECLARED IN KYC RS50000LAKHRS600LAKH, BANK AND BRANCH DETAILS IDBI BANK, BRANCH 874DELHI  KRISHNA NAGAR, D221, KRISHNA NAGAR NEW DELHI, DELHI, DELHI STATE, 110051, BANK ACCOUNT NO 0874102000003193, BANK ACCOUNT TYPE CURRENT, BANK AC OPENED ON 07062013, BANK ACCOUNT STATUS CLS29122017, BALANCE IN ACCOUNT ON DATE 23052021 RS 00 LAKHS, DETAILS OF EARLIER STRS FILED ON THE ACCOUNT , CTR  0, NTR  0, STR  0, CBWT  0 FULL NAME AND ADDRESS OF THE ACCOUNT HOLDER INDIAN POTASH LIMITED, SEETHAKATHI BUSINESS CENTRE, I ST FLOOR, NO684690, ANNA SALAI, CHENNAI, TAMIL NADU, 600006, INDIA, CUSTOMER TYPE PRIVATE LTD, CUSTOMER STATUS GENERAL, OCCUPATION BUSINESS  TRADING, CONSTITUTION PRIVATE NONFINANCIAL CORPORATIONS, RISK CATEGORY HIGH, RISK CODE RL305, PAN NO AAACI0888H, IEC CODE 0493010122, MOBILE NUMBERLANDLINE NO 9999726954, DATE OF CUST ID ACTIVATION 05082008, DATE OF LAST KYC UPDATION DEC20, PROFILENATURE OF BUSINESS BUSINESS  TRADING, BANK AND BRANCH DETAILS IDBI BANK, BRANCH 129CHENNAISCB, 115, ANNA SALAI, SAIDAPET, PB NO1306, CHENNAI, TAMIL NADU, 600015, BANK ACCOUNT NO 129655100001144, BANK ACCOUNT TYPE CASH CREDIT, BANK AC OPENED ON 13112009, BANK ACCOUNT STATUS ACTIVE, QUANTUM OF TRANSACTION FOR THE CURRENT YEAR 20212022 AC 129655100001144DEBIT SIDE  RS 1339 LAKHS IN CASH RS 00 LAKHS AND CREDIT SIDE RS 150 LAKHS IN CASH RS 00 LAKHS, QUANTUM OF TRANSACTION FOR THE PREVIOUS YEARS FOR FINANCIAL YEAR 20182019DEBIT SIDE RS 7182053 LAKHS IN CASH RS 00 LAKHS AND CREDIT SIDE RS 7182447 LAKHS IN CASH RS 00 LAKHS, 20192020DEBIT SIDE RS 183147 LAKHS IN CASH RS 00 LAKHS AND CREDIT SIDE RS 1830547 LAKHS IN CASH RS 00 LAKHS, 20202021DEBIT SIDE RS 4178268 LAKHS IN CASH RS 00 LAKHS AND CREDIT SIDE RS 4179816 LAKHS IN CASH RS 00 LAKHS, BALANCE IN ACCOUNT ON DATE 23052021 RS 1291 LAKHS, DETAILS OF EARLIER STRS FILED ON THE ACCOUNT , CTR  0, NTR  0, STR  0, CBWT  49 STR IS BEING FILED ON THE BASIS OF A NEGATIVE MEDIA ALERT AND ON CATALYST BUSINESS SOLUTIONS PRIVATE LIMITED WHERE MURALIDHAR KOMANDURI AND JITENDRA VERMA ARE DIRECTORS AS PER THE MINISTRY OF CORPORATE AFFAIRS  DIRECTORS MASTER DATA THEIR DIRECTOR IDENTIFICATION NUMBER DIN IS 00058015 AND 00049808 RESPECTIVELY STR IS BEING FILED ON THE BASIS OF A NEGATIVE MEDIA ALERT AND ON INDIAN POTASH LIMITED WHERE SHRINIWAS CHANDRASHEKHAR MUDGERIKAR, MUKESH GULSHANRAI PURI, PREM CHANDRA MUNSHI, SHIVAKUMAR GOUDA SHIDDANGOUDA PATIL, UDAI SHANKER AWASTHI AND PARVINDER SINGH GAHLAUT ARE DIRECTORS AS PER THE MINISTRY OF CORPORATE AFFAIRS  DIRECTORS MASTER DATA THEIR DIRECTOR IDENTIFICATION NUMBER DIN IS 03498837, 03582870, 03597933, 08195441, 00026019 AND 00049401 RESPECTIVELY',
  {'entities': [(2000, 2029, 'FOREIGN ORG'),
    (840, 871, 'FOREIGN ORG'),
    (2000, 2029, 'FOREIGN ORG')]}),
 ('RTGSNEFT TO INKWALA S INC 10000917249, IDFC OF RS 7,145,000, FOREIGN OUTWARD REMITTANCE TO BAMBINO SOLUTIONS LIMITED {000340582, DBS BANK HONG KONG LTD} OF RS 39,008,982, HUDSON OVERSEAS HK LIMITED 848856654838, HSBC OF RS 8,522,127BANK OFFICIALS CONTACTED THE CUSTOMER WHO INFORMED THAT THEY WERE INTO EVENT MANAGEMENT AND DEALING IN SOFTWARE WHICH IS USED IN PRINTING INDUSTRY AS PER CUSTOMER, BAMBINO SOLUTIONS AND HUDSON OVERSEAS HK ARE DEALERS OF HARLEY QUEEN BIGGEST SOFTWARE MANUFACTURER AND EXPORTER IN PRINTING INDUSTRY BELOW ARE RED FLAGS OBSERVED1OUR CUSTOMER HAS REMITTED TO BAMBINO SOLUTIONS LIMITED {000340582, DBS BANK HONG KONG LTD} OF RS 39,008,982, HUDSON OVERSEAS HK LIMITED 848856654838, HSBC OF RS 8,522,1272AS PER CUSTOMER, BAMBINO SOLUTIONS AND HUDSON OVERSEAS HK ARE DEALERS OF HARLEY QUEEN BIGGEST SOFTWARE MANUFACTURER AND EXPORTER IN PRINTING INDUSTRY 3DURING PUBLIC DOMAIN CHECK, NO CREDIBLE INFORMATION COULD BE OBTAINED ABOUT HARLEY QUEEN BIGGEST SOFTWARE MANUFACTURER AND EXPORTER IN PRINTING INDUSTRY, WHICH ADDS TO THE SUSPICION4AT THE TIME OF ACCOUNT OPENING, THE CUSTOMER HAD DECLARED THAT THEY WERE INTO EVENT MANAGEMENT AS PER GST WEBSITE, THE CUSTOMER WAS DEALING IN EVENTS, EXHIBITIONS, CONVENTIONS AND TRADE SHOWS ORGANIZATION AND ASSISTANCE SERVICES HOWEVER, CUSTOMER WAS REMITTING FUNDS TO HONG KONG FOR PURCHASING SOFTWARE CUSTOMER DEALING IN SOFTWARE SERVICES NOT UPDATED IN GST WEBSITE SEEMS SUSPICIOUS 5THE WEBSITE HTTPBAMBINOSOLUTIONSHKCOM REGISTRATION DETAILS REVEALED THE SAME TO BE REGISTERED IN GUJARAT, INDIA6THE WEBSITE HTTPHUDSONOVERSEASHKCOM REGISTRATION DETAILS REVEALED THE SAME TO BE REGISTERED IN GUJARAT, INDIA FURTHER CONTACT NUMBER 919825154654 UPDATED IN THE CONTACT US SECTION WAS ALSO FOUND IN THE CONTACT US SECTION OF WWWUMAKANTDESAIFOUNDATIONCOM THE WEBSITE MENTIONED CONTACT ADDRESS IN BHAVNAGAR, GUJARAT AND MUMBAI, MAHARASHTRA7ENHANCE DUE DILIGENCE ON THE INVOICES REVEALED THE FOLLOWINGAALL THE SALES INVOICES HAD GST NO OF CLIENTS MENTIONED AS EXEMPTED WHICH SEEMS UNUSUAL HOWEVER, IN SEVERAL INVOICES, GST WAS CHARGED ON THE SALES VALUEBONE CLIENT WAS FOUND TO BE HAVING ACTIVE GSTIN KEJRIWAL MINING PVT LTD 19AADCP0161H1ZL CIN MOST OF THE INVOICES CHECKED THE SOFTWARE ITEMS QUE DIAGRAM MAPING, SMTP INSERTS, MOTION SENSOR, KIMBERLY PRECISION ESTIMATOR,ETC HAD THE SAME SALES PRICE MENTIONED IN INVOICES WHICH SEEMS UNUSUAL8FURTHER CUSTOMER WAS FOUND TO BE DEALING WITH KEJRIWAL MINING PVT LTD, AVION PHARMA PVT LTD, MIGHTY SUPPLIERS, AS GARMENTS EXIM THE SAME PARTIES WERE ALSO FOUND TO BE TRADING WITH ARYAN INTERNATIONAL WHEREIN STR WAS FILED ON 30620 VIDE FIU BATCH ID 20060356439YES BANK LTD, RELIEF ROAD HAD RECEIVED A NOTICE US 44 OF THE GUJARAT VALUE ADDED TAX 2003VIDE REFERENCE NO STO14UNIT 9RECOVERY BRANCH202021OW2615 DATED 10072020 ON 14072020 FROM DP THAKOR, STATE TAX OFFICER 1 4, UNIT 9, AHMEDABAD OFFICE OF THE ASST COMMISSIONER OF STATE TAX UNIT 9 THE NOTICE WAS REGARD TO TAX DUES AND REQUESTING FOR KYC DETAILSIN LIGHT OF THE ABOVE FACTS, IT HAS BEEN DECIDED ON 16JAN2021 TO FILE A STR ON THE SAID CUSTOMER1 THE CUSTOMER IDEAL TRADE SERVICES PAN AAHFI3683C, DOI 01112018, COMMUNICATION AND PERMANENT ADDRESS SHOP 115B BLOCKA GANESH MERIDIAN OPP HIGH COURT S G HIGHWAY AHMEDABAD GUJARAT 380061, GSTIN 24AAHFI3683C1ZK, IEC CODE  AAHFI3683C, MOBILE NO 7202977154, EMAIL ID INFO@IDEALTRADESERVICESCOIN IS MAINTAINING A PARTNERSHIP CURRENT AC 021563400001059 SINCE 15022020 WITH MANINAGAR BRANCH ADDRESS PART GROUND, 3RD AND 4TH FLOOR, SWASTIK SQUARE, OPP BRTS BUS STOP, JAWAHAR CHOWK, MANINAGAR, AHMEDABAD 3800082 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED ON JAN2020 SHILPIN NAVINCHANDRA DESAI AND KIRANKUMAR H PATEL ARE THE AUTHORISED SIGNATORIES AND BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS INTO EVENT MANAGEMENT WITH ANNUAL TURNOVER UP TO RS 15 CRORE CUSTOMER HAS AVAILED TERM DEPOSITS OF RS 21322ADDITIONALLY 3 ACS ARE FOUND TO BE LINKED TO THE ACCOUNT OF IDEAL TRADE SERVICES AC NO021563400001059 BY COMMON AUTHORIZED SIGNATORY AND FUND TRANSFER FT THE LINKED ACCOUNTS ARE INKWALA S INC AC 021563400000352, ACE CREATIVE ENTERPRISE AC 021563400001620 AND M CARPETS AC 0450619000005903 IN THE ACCOUNT OF IDEAL TRADE SERVICES AC NO 021563400001059 AND 3 LINKED ACS, TRANSACTIONS ANALYSIS FROM 01042020 TILL 15012021, SHOWS 366 CREDIT TRANSACTIONS AMOUNTING TO RS 424,438,520 AND 536 DEBIT TRANSACTIONS AMOUNTING TO RS 425,187,285OUT OF THE TOTAL CREDITS OF RS 424,438,520 IN THE ACCOUNTS, FUND TRANSFER AMOUNTS TO RS 75,170,700 CONSTITUTING AROUND 18 PERCENT OF THE TOTAL CREDITS, RTGS AMOUNTS TO RS 287,227,629 CONSTITUTING AROUND 68 PERCENT OF THE TOTAL CREDITS OUT OF THE TOTAL DEBITS OF RS 425,187,285 IN THE ACCOUNTS, FUND TRANSFER AMOUNTS TO RS 83,238,000 CONSTITUTING AROUND 20 PERCENT OF THE TOTAL DEBITS, RTGS AMOUNTS TO RS 258,096,087 CONSTITUTING AROUND 61 PERCENT OF THE TOTAL DEBITS, REMITTANCE AMOUNTS TO RS 47,535,239 CONSTITUTING AROUND 11 PERCENT OF THE TOTAL DEBITS 4 IN THE ACCOUNT OF IDEAL TRADE SERVICES AC NO 021563400001059 AND 3 LINKED ACS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM 01042019 TILL 31032020 TRANSACTIONS ANALYSIS SHOWS 168 CREDIT TRANSACTIONS AMOUNTING TO RS 111,908,678 COMPRISING OF CASH DEPOSIT AMOUNTING TO RS 111,000 AND 323 DEBIT TRANSACTIONS AMOUNTING TO RS 110,881,235 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 697,200FROM 01042018 TILL 31032019 TRANSACTIONS ANALYSIS SHOWS 53 CREDIT TRANSACTIONS AMOUNTING TO RS 40,950,000 COMPRISING OF NIL CASH DEPOSIT AND 45 DEBIT TRANSACTIONS AMOUNTING TO RS 40,950,180 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 15,000FROM 01042017 TILL 31032018 TRANSACTIONS ANALYSIS SHOWS 62 CREDIT TRANSACTIONS AMOUNTING TO RS 46,430,060 COMPRISING OF NIL CASH DEPOSIT AND 101 DEBIT TRANSACTIONS AMOUNTING TO RS 46,430,241 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 111,000BALANCE IN THE ACCOUNT AS ON 15012021 IS RS 901965 ON REVIEWING THE TRANSACTION PATTERN OF IDEAL TRADE SERVICES AC 021563400001059 SINCE 27012020 TILL 15121, IT WAS OBSERVED THAT THERE WERE CREDITS VIDE FT FROM M CARPETS AC 045061900000590 OF RS 2,187,000, ACE CREATIVE ENTERPRISE AC 021563400001620 OF RS 3,240,000, RTGSNEFT FROM IDEAL TRADE SERVICES 10034003600, IDFC OF RS 17,213,000, FRS FOUNDATION 342801000285,ICICI OF RS 2,000,000, AVION PHARMA PVT LTD 201003384382 INDUSIND OF RS 2,000,000, ISOUL SOFTWARE SOLUTION 002111101004386, GSCB OF RS 11,106,500, KEJRIWAL MINING PVT LTD 50200041425882,HDFC OF RS 2,038,000, MIGHTY SUPPLIERS 38735129687, SBI OF RS 2,020,000, AS GARMENTS EXIM 00000003726265214, CBI OF RS 1,650,000CREDITS IN THE ACCOUNT WERE FOLLOWED BY DEBITS VIDE FT TO ACE CREATIVE ENTERPRISE OF RS600,000, CASH WITHDRAWAL OF RS 1,181,000,',
  {'entities': [(171, 197, 'FOREIGN ORG'),
    (667, 693, 'FOREIGN ORG'),
    (91, 116, 'FOREIGN ORG'),
    (587, 612, 'FOREIGN ORG'),
    (2158, 2181, 'FOREIGN ORG'),
    (2445, 2468, 'FOREIGN ORG'),
    (6471, 6494, 'FOREIGN ORG'),
    (171, 186, 'FOREIGN ORG'),
    (418, 433, 'FOREIGN ORG'),
    (667, 682, 'FOREIGN ORG'),
    (768, 783, 'FOREIGN ORG'),
    (4130, 4153, 'FOREIGN ORG'),
    (6165, 6188, 'FOREIGN ORG'),
    (6696, 6719, 'FOREIGN ORG'),
    (2470, 2490, 'FOREIGN ORG'),
    (6347, 6367, 'FOREIGN ORG')]}),
 ('BANK APPEARS NOT HAVING ANY ECONOMIC RATIONALE AND USED MERELY FOR BOOK ENTRY PERSPECTIVE THE SUPPORTING AGREEMENT BETWEEN THE CAPTIONED PARTIES AS SUBMITTED BY CUSTOMER WAS VAGUE; WITH NO DETAILS ON THE PROJECTS, TITLES, CREW, DIRECTOR ETC FURNISHED 3CLIENT HAS UNDERTAKEN 02 CROSS BORDER REMITTANCES FOR AUDIO VISUAL AND RELATED SERVICES TO MAGNETIC MOTION PICTURE PRODUCTION LLC UAE, SINCE ACCOUNT OPENING IN MAY 2021 TOTALING TO RS 114 CR 4CUSTOMER HAS SUBMITTED ANOTHER CROSS BORDER REMITTANCE ON 07TH JUNE 2021 FOR USD 91,380 FAVORING SAME PARTY, HOWEVER SINCE THE AGREEMENT ATTACHED IN THIS TRANSACTION IS SAME AS THAT OF THE PREVIOUS TWO REMITTANCES AND CUSTOMER IS NOT WILLING TO PROVIDE THE DETAILS OF THE REMITTANCES MADE UNDER THE AGREEMENT BY OTHER BANKS, BANK CARRIED OUT ENHANCED DUE DILIGENCE ON THE REMITTANCES5PUBLIC DOMAIN DETAILS OF MAGNETIC MOTION PICTURE PRODUCTION LLC UAE HTTPMAGNETICMOTIONPICTURECOM ARE COPIED FROM ANOTHER RENOWN INDIAN MEDIA AGENCY HTTPTHEFILMCOMPANYNETABOUTUS  6 INFORMATION REPORT OF THE OVERSEAS PARTY IS ALSO CALLED FOR AND FOUND UNSATISFACTORY, FOLLOWING ARE SOME OF THE POINTS;AOVERSEAS PARTYS CHIEF EXECUTIVE IS AN INDIAN MR RAJESH KRISHAN BAL KRISHAN, WHO IS ALSO A MANAGER IN THE FIRM AND HOLDING A SUBSTANTIAL PERCENTAGE  49 IN THE FIRM BAS PER THE GROUP ENTITIES DETAILS, WHITE RABBIT MOTION PICTURE PRODUCTION LLC , WHICH HOLDS 49 IN OVERSEAS PARTY STR WAS ALREADY FILED IN FEB 2021 VIDE FIU BATCH ID 2102093539 FOR THE REMITTANCES SENT TO WHITE RABBIT MOTION PRODUCTION LLC BY ONE OF OUR CUSTOMER EYE FOCUS PRODUCTION7LOOKING AT THE ABOVE RED FLAGS IT WAS DECIDED BY BANK NOT TO PURSUE THE TRANSACTION OF USD 91,380 SUBMITTED BY CUSTOMERIT APPEARS THAT THE CUSTOMER WAS ATTEMPTING TO UNDERTAKE MONEY LAUNDERING TRANSACTION UNDER THE GUISE OF ENTERTAINMENT SERVICES TO OVERSEAS PARTIES WITH UNSATISFACTORY CREDENTIALSIN LIGHT OF THE ABOVE FACTS, IT HAS BEEN DECIDED ON 19 JUNE 2021 TO FILE A STR ON THE SAID CUSTOMER1 THE CUSTOMER SLOW MOTION PICTURES PRIVATE LIMITED PAN AAQCS1627J, DOI 04082011, COMMUNICATION AND PERMANENT ADDRESS OFFICE NO 401 TOWN CENTER AMANORA CHAMBER HADAPSAR KHARADI ROAD PUNE MAHARASHTRA 411028, GSTIN 27AAQCS1627J2ZX, REGISTRATION NUMBER 286075, CIN U74999MH2016PTC286075, MOBILE NUMBER 917379871723, EMAIL ID INFO@SLOWMOTIONPICTURESPVTLTDCOM IS MAINTAINING A CURRENT AC NO 058563400000321 SINCE 18052021 WITH AMANORA TOWN CENTRE, PUNE BRANCH ADDRESS GROUND FLOOR, EAST BLOCK, AMANORA TOWN CENTRE, HADAPSAR KHARADI BYPASS, HADAPSAR, PUNE 411028, MH2 THE CUSTOMER FALLS UNDER LOW RISK CATEGORY WITH KYC LAST UPDATED IN MAY2021 NARESH KUMAR MIDHA, SHAM KESHAV CHAVAN ARE AUTHORISED SIGNATORY BENEFICIAL OWNERS IN THE ACCOUNT CUSTOMER IS IN TO MEDIAENTERTAINMENT SERVICES BUSINESS WITH AN ANNUAL GROSS INCOMETURNOVER UP TO GREATER THAN OR EQUAL 15 CRORES TO LESS THAN 50 CRORES ADDITIONALLY SIX ACCOUNTS ARE FOUND TO BE LINKED TO THE ACCOUNT OF SLOW MOTION PICTURES PRIVATE LIMITED AC NO058563400000321 OUT OF THIS AC NO 058563400000341 OF S SQUARE CONSULTANTS PRIVATE LIMITED , AC NO 058563400000288 OF SHADOW FILM PRODUCTIONS ARE LINKED BY MOBILE NUMBER 917379871723 WHILE AC NO 023563400003080 LAMHO HOSPITALITY PVT LTD , 025063400001118 DIGI MEDIA AND FILMS SOLUTIONS, 102563400000299 IMAGINARY FILMS PVT LTD , 047884100000059 CINE1 STUDIOS PRIVATE LIMITED ARE LINKED BY WAYS OF FUND TRANSFERS AND SEEN MAKING REMITTANCE TO SAME OVERSEAS PARTY MAGNETIC MOTION PICTURE PRODUCTION LLC  3 IN THE ACCOUNT OF SLOW MOTION PICTURES PRIVATE LIMITED AC NO 058563400000321 AND SIX LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FROM 01042021 TILL 19062021, SHOWS 137 CREDIT TRANSACTIONS AMOUNTING TO RS 23636219232 AND 50 DEBIT TRANSACTIONS AMOUNTING TO RS 239,739,31138OUT OF THE TOTAL CREDITS OF RS 236,362,19232IN THE ACCOUNTS, RTGS NEFT TRANSACTION AMOUNTS TO RS 228,665,714 CONSTITUTING 98 PERCENT OF THE TOTAL CREDITS, OUT OF THE TOTAL DEBITS OF RS 239,739,31138IN THE ACCOUNTS, RTGS NEFT TRANSACTION AMOUNTS TO RS 36,132,000CONSTITUTING AROUND 15 PERCENT OF THE TOTAL DEBITS, TRADE TRANSACTION AMOUNTS TO RS 203,011,434 CONSTITUTING AROUND 85 PERCENT OF THE TOTAL DEBITS, 4 IN THE ACCOUNT OF SLOW MOTION PICTURES PRIVATE LIMITED AC NO 058563400000321 AND SIX LINKED ACCOUNTS, TRANSACTIONS ANALYSIS FOR PREVIOUS THREE YEARS WERE OBSERVED AS BELOW FROM 01042020 TILL 31032021 TRANSACTIONS ANALYSIS SHOWS 118 CREDIT TRANSACTIONS AMOUNTING TO RS 280,525,560 COMPRISING OF NIL CASH DEPOSIT AND 87 DEBIT TRANSACTIONS AMOUNTING TO RS 272,387,069 COMPRISING OF CASH WITHDRAWAL AMOUNTING TO RS 189,509NO TRANSACTION SEEN FROM 01042018 TO 31032020 AS THE ACCOUNTS WERE OPENED RECENTLY BALANCE IN THE ACCOUNT AS ON 18062021 IS RS 0ON REVIEWING THE TRANSACTION PATTERN OF SLOW MOTION PICTURES PRIVATE LIMITED AC 058563400000321 AND OTHER LINKED ACCOUNT IT WAS OBSERVED THAT AACCOUNT OF LAMHO HOSPITALITY PVT LTD  AC NO 023563400003080 AND DIGI MEDIA AND FILMS SOLUTIONS AC NO 025063400001118 IS SEEN MAKING CROSS BORDER REMITTANCES TOWARDS AUDIO VISUAL AND RELATED SERVICES TO MAGNETIC MOTION PICTURE PRODUCTION LLC UAE B STR IS ALREADY FILED ON IMAGINARY FILMS PVT LTD AC NO 102563400000299 IN MARCH 2021 VIDE FIU BATCH ID 2103245908 FOR MAKING CROSS BORDER REMITTANCES TO OVERSEAS PARTY LIVING FILMS PTE LTD IN SINGAPORE WITH UNVERIFIED CREDENTIALSIN ACCOUNT OF SLOW MOTION PICTURES PRIVATE LIMITED FOLLOWING RED FLAGS WERE FOUND1CUSTOMER ACCOUNT IS SEEN RECEIVING FUNDS FROM CINE 1 STUDIO  RS 183 CR RTGS IMPS FROM AXIS BANK GOREGAON LINK ROAD BRANCH RS116 CR , RTGS FROM IDFC BANK INTERNET BANKINGCPU BRANCH  RS 6773 LACS, IMMEDIATELY FOLLOWED BY CROSS BORDER REMITTANCES FOR AUDIO VISUAL AND RELATED SERVICES NO OTHER BUSINESS TRANSACTION SEEN IN ACCOUNT2CREDIT FOLLOWED BY IMMEDIATE DEBITS BETWEEN CUSTOMER AND CINE 1 STUDIOS A',
  {'entities': [(3270, 3293, 'FOREIGN ORG'),
    (5108, 5131, 'FOREIGN ORG'),
    (3312, 3341, 'FOREIGN ORG'),
    (3178, 3203, 'FOREIGN ORG'),
    (4848, 4873, 'FOREIGN ORG')]}),
 ('MARIYAM ENTERPRISE, MOBILE MONEY TRANSFERS RS6640 LAKHS IN 180 INSTANCES AND OTHERS RS6289LAKHS FY 201920 FROM 19122019 TO 31032020 TOTAL CREDIT SIDE RS 6,95295 LAKHS OF WHICH CASH WAS NIL, REMITTANCE RS 3,49970 LAKHS TOTAL DEBIT SIDE RS 6,86312 LAKHS OF WHICH CASH WAS RS 1LAKH, REMITTANCE RS190 LAKHS TRANSACTION PATTER SHOWS FUNDS CREDITED THROUGH INWARD REMITTANCE, INTERNET FUND TRANSFERS AND WHICH IS IMMEDIATELY DEBITED THROUGH INTERNET FUND TRANSFERS FOR EXAMPLE, ON 05052021 RS124 CRORES CREDITED THROUGH INWARD REMITTANCE FROM MS SIDDHART ENTERPRISE, BANGLADESH IN USD CURRENCY AND REASON WAS MENTIONED AS CUMINJEERALOOSE, FROM MS SHOTOTA TRADERS, BANGLADESH IN IN USD CURRENCY AND REASON WAS MENTIONED AS INDIAN WHEAT AND ON 05052021 RS110CRORES DEBITED THROUGH INTERNET FUND TRANSFERS TO MANGALAM ENTERPRISES 627905502039, DIAMOND FRUIT COMPANY 008605501340 ALSO ON 30042021 RS4747LACS CREDITED FROM RKP ENTERPRISE, BANGLADESH IN IN USD CURRENCY AND REASON WAS MENTIONED AS DRY RED CHILLY GENUS CAPSICUM AND ON 30042021 RS7420LACS DEBITED THROUGH RTGS TO POTENT PRODUCTS PVT LTD ACCOUNT NUMBER 919030067624577, IFSC CODE UTIB0001164, SITALADAS GOVINDRAM KEWLANI ACCOUNT NUMBER 13110510008539, IFSC CODE UCBA0001311 SIMILAR PATTERN WAS OBSERVED WHERE HIGH VALUE TRANSACTION, INWARD REMITTANCE AND HIGH CREDIT TURNOVER IN THE FINANCIAL YEAR FY 2022 01042021 TILL 31052021 IS RS 58CRORES WHICH IS VERY HIGH IN 2 MONTHS AS COMPARED TO THE DECLARED TURNOVER TOTAL TURNOVER IN THE FINANCIAL YEAR 2021, WAS RS 27,78061 LAKHS WHICH IS VERY HIGH AS COMPARED WITH THE DECLARED ANNUAL TURNOVER OF RS 40LAC TO LESS THAN RS 5CRS IN VIEW OF LARGE VALUE NON CASH TRANSACTION IN CURRENT ACCOUNT AND HIGH TURNOVER, STR IS BEING FIELDMTE EXIM PRIVATE LIMITED, A PRIVATE LTD COMPANY, HAVING ADDRESS 19, ARMENIAN STREET, 4TH FLOOR, BURRABAZAR, ROOM NO 513, KOLKATA, WEST BENGAL, INDIA, 700001, CONTACT NUMBER 919433054990 AND EMAIL ID MTEEXIM19@YAHOOCOM IS MAINTAINING A CURRENT ACCOUNT NUMBER 694905602404 WITH ICICI BANK LTD, KOLKATA  PORTUGUESE CHURCH STREET, ICICI BANK LTD, CATHEDRAL ANNEXE, GROUND FLOOR, 15 PORTUGUESE CHURCH STREET, KOLKATA, WEST BENGAL, 700001, SINCE 19122019 DATE OF INCORPORATION IS 12122019 RELATIONSHIP OF THE CUSTOMER AS PER UCIC INCLUDES CURRENT ACCOUNT PAN NUMBER OF THE ACCOUNT IS AANCM1491P GSTIN AS PER BANK RECORDS IS 19AANCM1491P1ZM OTHER RELATIONSHIP OF CUSTOMER ARE 694966000011, 694966000012, 777705054990 TRANSACTION LINKED ACCOUNTS ARE 777705054990, 127605001421 BENEFICIAL OWNER OF THE REPORTED ACCOUNT IS JAYEETI JAISWAL WITH PAN NO ASJPC5156G, SUMAN JAISWAL WITH PAN NO ASYPP7883N THE CUSTOMER IS CATEGORIZED AS HIGH RISK AS PER THE BANKS CUSTOMER RISK CATEGORIZATION MODEL THE CURRENT STATUS OF THE ACCOUNT IS ACTIVE KYC DOCUMENTS OBTAINED WERE ENTITY PROOF MOA, AOA, BOARD RESOLUTION, PAN AANCM1491P, CINU15490WB2019PTC235251 CUSTOMER PROFILE IS INTO FOOD GRAINS AS PER BANK RECORDS DECLARED ANNUAL TURNOVER OF THE CUSTOMER DURING ACCOUNT OPENING WAS RS 40LAC TO LESS THAN RS 5CRS ALERT WAS TRIGGERED FOR LARGE VALUE NON CASH TRANSACTION IN CURRENT ACCOUNT BALANCE IN THE ACCOUNT AT 31052021 WAS RS13889 LAKHS DEBIT QUANTUM OF TRANSACTIONS IN THE CURRENT FINANCIAL YEAR FY 202122 FROM 01042021 TILL 31052021 SHOWS TOTAL CREDIT RS 5,89282 LAKHS OF WHICH RS0 BY CASH, CLEARING RS263LAKHS FROM MAA TARA EXIM PRIVATE LIMITED, S P ENTERPRISE, NEFT RS 90,000 FROM EDI DUTY DRAWBACK DISBURSAL, RTGS RS 8471 LAKHS FROM MAHEMUD PAN HOU, JP FLOUR MILLS, ABHI TRADERS, REMITTANCE RS 4,55069 LAKHS FROM MS RAJ ENTERPRISE, RKP ENTERPRISE, MS SIDDHART ENTERPRISE, INTERNET FUND TRANSFERS RS 993 LAKHS FROM MTE EXIM PRIVATE LIMITED 777705054990, MOBILE MONEY TRANSFERS RS 4,000 IN 4 INSTANCES AND OTHERS RS 48,000 TOTAL DEBITS DURING THE SAME PERIOD WAS RS 5,89449 LAKHS OF WHICH CASH WAS RS 6 LAKHS BY BRANCH, CLEARING RS 2,17527 LAKHS TO SRIVISHNUPRIYA CHILLI, ANILTRADING INDIA PVT LTD, BALASAHEB GANGARAMSHE, RTGS RS 2,27943 LAKHS MAJORLY TO TO KRS TRADERS, SRI VISHNU PRIYA CHILL, MANGALAM ENTERPRISES, DIAMOND FRUIT COMPANY, REMITTANCE RS193 LAKHS, INTERNET FUND TRANSFER RS 1,40033LAKHS MAJORLY TO S P ENTERPRISE 694905501253, MARIYAM ENTERPRISES 127605001421, MOBILE MONEY TRANSFERS RS2730 LAKHS IN 39 INSTANCES AND OTHERS RS423 LAKHS FURTHER, THE QUANTUM OF TRANSACTIONS FOR THE PREVIOUS YEARS YEARWISE WERE FY 202021 FROM 01042020 TO 31032021 TOTAL CREDIT SIDE RS 27,78061 LAKHS OF WHICH CASH WAS NIL, TRANSFERS RS 7450 LAKHS FROM MTE EXIM PRIVATE LIMITED, S P ENTERPRISE, CLEARING RS 18877 LAKHS FROM CMS18215959481, MAA TARA EXIM PRIVATE LIMITED, S P ENTERPRISE, NEFT RS 2316 LAKHS FROM BONGAON, SBI BASIRHAT, GHOJADANGA CUSTOMS, RTGS RS 86459 LAKHS FROM SNOWBLUE ENCLAVE PVT LTD, ABHI TRADERS, DILIP KUMAR CHANDRA, REMITTANCE RS 24,87795 LAKHS FROM MS RAJ ENTERPRISE, RKP ENTERPRISE, MS SIDDHART ENTERPRISE, INTERNET FUND TRANSFERS RS 1,59510 LAKHS MAJORLY FROM MTE EXIM PRIVATE LIMITED, MARIYAM ENTERPRISE, MOBILE MONEY TRANSFERS RS341 LAKHS IN 12 INSTANCES AND OTHERS RS15313 LAKHS TOTAL DEBIT SIDE RS 28,00766 LAKHS OF WHICH CASH WAS RS 20 LAKHS BY BRANCH, TRANSFERS RS 1,10106 LAKHS TO MAMA EXPORT, KISSAN JAI KISSAN TRADERS, MTE EXIM PRIVATE LIMITED, CLEARING RS 8,71338 LAKHS TO KISSANJAIKISSANTRADE, MAAVINDHWASHINITRADE, SRIVISHNUPRIYACHILLI, NEFT RS 9541 LAKHS TO NANDHINI CHILLIES, HK ORGANIC, MANOJ TRADING COMPAN, RTGS RS 11,43363 LAKHS TO KRS TRADERS, SRI VISHNU PRIYA CHILL, REMITTANCE RS 963 LAKHS, INTERNET FUND TRANSFER RS 6,50526 LAKHS MAJORLY TO MTE EXIM PRIVATE LIMITED,',
  {'entities': [(4691, 4715, 'FOREIGN ORG'),
    (835, 856, 'FOREIGN ORG'),
    (4032, 4053, 'FOREIGN ORG'),
    (537, 559, 'FOREIGN ORG'),
    (3565, 3587, 'FOREIGN ORG'),
    (4821, 4843, 'FOREIGN ORG'),
    (912, 926, 'FOREIGN ORG'),
    (3549, 3563, 'FOREIGN ORG'),
    (4805, 4819, 'FOREIGN ORG'),
    (1067, 1090, 'FOREIGN ORG'),
    (3330, 3359, 'FOREIGN ORG'),
    (4552, 4581, 'FOREIGN ORG'),
    (3361, 3375, 'FOREIGN ORG'),
    (4129, 4143, 'FOREIGN ORG'),
    (4491, 4505, 'FOREIGN ORG'),
    (4583, 4597, 'FOREIGN ORG'),
    (800, 820, 'FOREIGN ORG'),
    (4010, 4030, 'FOREIGN ORG'),
    (4691, 4715, 'FOREIGN ORG')]


### Adding labels over identified foreign entities

In [ ]:
for _,annotations in train_data:
    for ent in annotations.get("entities"):
        print(ent)
        ner.add_label(ent[2])

### Training on prepared dataset

    1. Loading the optimizer
    2. Training data to create the model

In [ ]:
optimizer=nlp.create_optimizer()

In [ ]:
unaffected_pipes=[pipe for pipe in nlp.pipe_names if pipe !='ner']
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(40):
        print("Iteration  "+ str(iteration))
        random.shuffle(train_data)
        losses={}

In [ ]:
for text, annotations in train_data:
    doc =nlp.make_doc(text)
    example=Example.from_dict(doc, annotations)
    nlp.update([example],drop=0.8,
    sgd=optimizer,
    losses=losses)
    print("losses", losses)

### Saving the trained model on disk

In [ ]:
db=DocBin()

In [ ]:
for text, annot in tqdm(train_data):
    doc=nlp.make_doc(text)
    ents=[]
    for start, end, label in annot['entities']:
        
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
        if span is None:
            pass
        else:
            ents.append(span)
    doc.ents=ents
    db.add(doc)
    
    db.to_disk("./train.spacy")

### Loading the trained dataset for inference

In [4]:
nlp1=spacy.load(r"Z:\FiNnet_2.0\NTI_GOS summary\Gaurav\output\model-best")

C:\Users\SAL005\anaconda3\lib\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Passing new GOS to verify results

In [5]:
text=nlp1("FOREIGN REMITTANCE RS. 18.05 LAKHS FROM WORLD METALS AND ALLOYS FZC, SAMANCOR MARKETING PTE. LTD, HWASUNG METAL CO. LTD., SEASHORE STEEL, ETC., BILLS PAYMENT RS. 8.23 CRORES, IMPS TRANSFER RS. 1.02 CRORE FROM SUN STEEL, SHREE BALAJI ALUMNIC, ALTRUISTIC METAL PRIVATE LTD., DIVYA IMPORTERS, ETC., INTEREST RS. 3.38 LAKHS, UPI TRANSFER RS. 1.87 LAKH FROM KOLAWAT ENTERPRISES, SHRI RAGHAVENDRA STEELS, ANAND RAJESH, ETC., COMMISSION RS. 3,199/-. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING FOREIGN REMITTANCES RS. 1078.03 CRORES TO COMPLEJO INDUSTRIAL MOLYNO, GLENCORE INTERNATIONAL AG, SIZER METALS PTE LTD., AMBANT LIMITED, ETC., CMS DEBITS RS. 433.14 CRORES, FUND TRANSFER RS. 88.62 LAKHS TO OSWAL ALLOYS PRIVATE LTD., MOHANLAL BHARATH KUMAR, SANJAY PITLIYA AND COMPANY, ETC., RTGS TRANSFER RS. 72.90 CRORES TO OSWAL MINERALS LIMITED (YES BANK LTD), D S ALLOYD PVT LTD (STATE BANK OF INDIA), LALWANI FERRO ALLOYS LTD. (AXIS BANK), SHYAM FERRO ALLOYS LIMITED (HDFC BANK LTD.), ETC., NEFT TRANSFER RS. 19.87 CRORES TO LALWANI FERRO ALLOYS LIMITED (AXIS BANK), MANGANESE PRODUCTS CORPORATION (UNION BANK OF INDIA), ISPAT DAMODAR PVT. LTD. (STATE BANK OF INDIA), FERRO ALLOYS CORPORATION LTD. (STATE BANK OF INDIA), ETC., CASH RS. 10.29 CRORES, LOAN REPAYMENT RS. 1.26 CRORE, TAX RS. 56.03 LAKHS, CHARGES RS. 42.63 LAKHS, CHEQUE RS. 30.71 LAKHS, SALARY RS. 13.08 LAKHS, BILLS RS. 6.70 LAKHS, INTEREST RS. 3.38 LAKHS, FEE RS. 1.00 LAKH, CREDIT CARD RS. 49,442/-, ONLINE TRANSACTION RS. 33,690/-, COMMISSION RS. 6,000/-.PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 609000016945) FOR THE PERIOD MENTIONED BELOW: FY: 2019-2020 (01-04-2019 TO 31-03-2020): TOTAL CREDITS RS. 1352.53 CRORES (CASH  5.40 LAKHS) AND TOTAL DEBITS RS. 1357.04 CRORES (CASH - 12.29 CRORES). FY: 2018-2019 (01-04-2018 TO 31-03-2019): TOTAL CREDITS RS. 1211.13 CRORES (CASH  2.66 LAKHS) AND TOTAL DEBITS RS. 1215.26 CRORES (CASH  4.45 CRORES).FY: 2017-2018 (01-04-2017 TO 31-03-2018): TOTAL CREDITS RS. 999.20 CRORES (CASH - 91,080) AND TOTAL DEBITS RS. 979.18 CRORES (CASH - 6.71 CRORES).BALANCE AS ON DATE OF FILING IS RS. 36870294.59/-.WE HAVE ALREADY FILED STR ON ACCOUNT NO.: 609000016945 (STR NO.:6194, DATE  18-07-2017, BATCH ID  1707181814, REASON - HUGE AMOUNT OF FUNDS HAVE BEEN ROUTED THROUGH THE ACCOUNT BY INWARD AND OUTWARD RTGS/NEFT AND INTERNAL FUND TRANSFERS WITHOUT ANY ECONOMIC RATIONALE)WHILE CONDUCTING DISCREET CHECK, WE HAVE OBSERVED FEW DISCREPANCIES. CUSTOMER HAS RECEIVED HIGH VALUE REFUND AGAINST DIRECT IMPORT BILL PAYMENT AS QUALITY CLAIM AND CONSOLIDATED VALUE OF REFUND AMOUNTED TO USD 1.9 MILLION, HOWEVER THE SAME WAS NOT HANDLED BY CLIENT AS PER RBI DIRECTIONS TOWARDS WRITE OFF OF IMPORT DUES. FURTHER, HIGH VALUE TRANSACTIONS HANDLED UNDER DIRECT IMPORT BILL PAYMENT AS OPEN ACCOUNT PAYMENT AND PAYMENT TERMS APPEARING ON INVOICE SEEMS MODIFIED WHILE PAYMENT. THERE IS FREQUENT TRADING BETWEEN THIRD COUNTRY AND PAYMENT MADE MAINLY WITH UAE ENTITY I.E M/S WORLD METALS AND ALLOYS FZC, WHEREIN MANUFACTURER OF THE GOODS OR ORIGIN DETAILS ARE NOT SPECIFIED PROPERLY. IT APPARENTLY SEEMS THAT CUSTOMER HAS INTENTIONALLY BREACHED THE FEMA GUIDELINES.HIGH VALUE RTGS/NEFT/FUND TRANSFER/BILLS RECEIPT CREDITS FOLLOWED BY HIGH VALUE TRANSFER TO SPECIFIC THIRD PARTIES, ANOMALIES OBSERVED RAISING SUSPICION ON UNDERLYING INTENTION. HENCE, STR FILED.,OSWAL MINERALS LIMITED (ACCOUNT NO.: 409000086197, 409001100618, 409000086203 AND 609000016945) ADDRESS: 1034 2ND FLOOR DR RAJKUMAR ROAD,4TH BLOCK RAJAJINAGAR, OPPOSITE TO PLANET HONDA SERVICE CENTRE BANGALORE - 560010, DATE OF INCORPORATION:- 04-07-1996, PAN: AACCM6499G, AADHAAR NUMBER REFERENCE KEY: - IN VIEW OF THE RESTRICTION OF SHARING THE AADHAAR NUMBER AS PER REGULATION 6 OF AADHAAR (SHARING OF INFORMATION) REGULATIONS 2016, WE ARE PROVIDING REFERENCE KEY ASSIGNED TO THE AADHAAR NUMBER, WHEREVER APPLICABLE, MOBILE NUMBER: 919448992115, RISK CATEGORY: HIGH RISK, AUTHORISED SIGNATORY - MOHANLAL BHARATHKUMAR JAIN, SUBHASH CHAND MOHANLAL, (KYC COMPLIANCE STATUS: YES, DATE OF LAST KYC UPDATION: 04-07-2019 AND NA) OPENED AN EXCHANGE EARNERS FOREIGN CURRENCY A/C - EEFC ACCOUNT AND CASH CREDIT ACCOUNT RESPECTIVELY ON 04-07-2019, 30-09-2013, 30-09-2013 AND 28-09-2013 AT RBL BANK, BANGALORE BRANCH, 197, 6TH CROSS, GANDHINAGAR, BANGLORE  560009 AND FORT MUMBAI BRANCH, GROUND FLOOR, 2, 2A DHANNUR, 15, SIR FIROZ SHAH MEHTA ROAD, MUMBAI, FORT - 400001. (CONSTITUTION TYPE: PUBLIC LIMITED COMPANY, ACCOUNT STATUS: INACTIVE, ACTIVE, DORMANT AND ACTIVE RESPECTIVELY)THE CUSTOMER HAS DECLARED PROFILE AS MANUFACTURER OF METALS AND MINERALS HAVING EXPECTED ANNUAL TURNOVER IS GREATER THAN 100 CRORE. SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 409000086197) FOR THE PERIOD FROM 01-04-2020 UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS. 116.94 LAKHS CONSISTING OF FOREIGN REMITTANCES RS. 116.94 LAKHS FROM WORLD METALS ALLOYS, KUBOTA SAUDI COMPANY FACTORY, AL GARHOUD GENERAL TRADING, PINK LOTUS IRON AND STEELS, ETC. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS. 104.21 LAKHS, FOREIGN REMITTANCES RS. 12.72 LAKHS TO ALLIANZ BULK CFR8 PTE LTD., GLENCORE INTERNATIONAL AG, PMB SILICON SDN BHD, SIZER METALS PTE LTD.PLEASE FIND THE SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 409000086197) FOR THE PERIOD MENTIONED BELOW: FY: 2019-2020 (01-04-2019 TO 31-03-2020): TOTAL CREDITS RS. 68.25 LAKHS (CASH - NIL) AND TOTAL DEBITS RS. 68.25 LAKHS (CASH - NIL).FY: 2018-2019 (01-04-2018 TO 31-03-2019): TOTAL CREDITS RS. 149.39 LAKHS (CASH - NIL) AND TOTAL DEBITS RS. 149.39 LAKHS (CASH - NIL).FY: 2017-2018 (01-04-2017 TO 31-03-2018): TOTAL CREDITS RS. 223.57 LAKHS (CASH - NIL) AND TOTAL DEBITS RS. 223.57 LAKHS (CASH - NIL).BALANCE AS ON DATE OF FILING IS NIL.THERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 409001100618) FROM THE PERIOD OF 01-04-2020 UPTO DATE OF FILING STR.SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 409001100618) FOR THE PERIOD FROM 04-07-2019 UPTO 31-03-2020 REVEALED THAT THE TOTAL CREDITS OF RS. 13.28 LAKHS CONSISTING OF FOREIGN REMITTANCES RS. 13.28 LAKHS FROM STAIN LESS METALS GENERAL. PROCEEDS OF THE SAME ARE UTILIZED BY MAKING BILLS RS. 13.28 LAKHS.BALANCE AS ON DATE OF FILING IS NIL.THERE WERE NO CUSTOMER INDUCED TRANSACTION IN THE ACCOUNT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 409000086203) FROM THE PERIOD OF 01-04-2017 UPTO DATE OF FILING STR.BALANCE AS ON DATE OF FILING IS NIL.SCRUTINY OF THE ACCOUNT STATEMENT OF OSWAL MINERALS LIMITED (ACCOUNT NO.: 609000016945) FOR THE PERIOD FROM 01-04-2020 UPTO DATE OF FILING STR REVEALED THAT THE TOTAL CREDITS OF RS. 1715.41 CRORES CONSISTING OF RTGS TRANSFER RS. 1062.17 CRORES FROM OSWAL MINERALS (YES BANK LTD.), SONIC THERMAL PVT LTD. (ALLAHABAD BANK), AJAY ENTERPRISES (YES BANK LTD.), MUKAND LTD (YES BANK LTD.), ETC., FUND TRANSFER RS. 193.83 CRORES FROM MOHANLAL, SEEMA JAIN, SUBHASHCHAN, SRIPAL KUMAR, SARITHA DEV, ETC., NEFT TRANSFER RS. 341.61 CRORES FROM OSWAL MINERALS LIMITED (HDFC BANK LTD), ARCELORMITTAL NIPPON STEEL INDIA (ICICI BANK LTD), SHYAM FERRO ALLOYS LIMITED (HDFC BANK LTD), MACTECH ENGINEERING (CORPORATION BANK), ETC., CMS RS. 52.67 CRORES, CHEQUE RS. 37.76 CRORES (RETURNED AMOUNTED RS. 45.26 LAKHS)")

### Displaying the Final Results

In [6]:
from spacy import displacy
displacy.render(text,style='ent', jupyter=True)